In [20]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from urllib import request
import json
import random
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest



In [65]:
71663*4*50

14332600

In [21]:
df_devices = pd.read_csv("all_devices_v1.csv.gz").drop("Unnamed: 0",axis=1) #,nrows=12123
print(df_devices.shape)
print(df_devices.nunique())
# df_devices.to_csv("all_devices_v1.csv.gz",compression="gzip",index=False)
df_devices.head()

(76261, 8)
network_id                      4
device_id                   71663
type                            7
model                        1357
manufacturer                  263
operating_system               80
operating_system_version      266
id                          76261
dtype: int64


,network_id,device_id,type,model,manufacturer,operating_system,operating_system_version,id
0,0,1028623,MOBILE_PHONE,Galaxy S8,Samsung,Android,9,0_1028623
1,0,48047,MOBILE_PHONE,Galaxy Note 8,Samsung,Android,9,0_48047
2,0,123568,MOBILE_PHONE,H918,LG Electronics,Android,8.0.0,0_123568
3,0,95366,MOBILE_PHONE,iPhone 6,"Apple, Inc.",iOS,NaN,0_95366
4,0,1755023,TABLET,iPad,Apple,iOS,NaN,0_1755023


In [22]:
df_devices["manufacturer"].head()

0           Samsung
1           Samsung
2    LG Electronics
3       Apple, Inc.
4             Apple
Name: manufacturer, dtype: object

In [23]:
print("unique manuf",df_devices["manufacturer"].str.split(expand=True)[0].nunique())
le = LabelEncoder()
df_devices["manufacturer"] = df_devices["manufacturer"].str.split(expand=True)[0].str.split(",",expand=True)[0]
df_devices["manufacturer"] = le.fit_transform(df_devices["manufacturer"].astype(str))

df_devices["operating_system"] = df_devices["operating_system"].str.split(expand=True)[0].str.split(",",expand=True)[0]

le = LabelEncoder()
df_devices["type"]= le.fit_transform(df_devices["type"])

unique manuf 179


### Featurize devices
* clean manufacturer, label encode (later, OHE

In [24]:
dev_count_cols = ['manufacturer','operating_system',"model","operating_system_version"]
for col in dev_count_cols:
    df_devices[col+'_count'] = df_devices.groupby(col)["device_id"].transform("count")

# df_devices['manufacturer_count_usage'] = df_devices.groupby('manufacturer')['device_id'].transform('count')

# df_devices['operating_system_count'] = df_devices.groupby('operating_system')['device_id'].transform('count')

In [25]:
df_devices.head()

,network_id,device_id,type,model,manufacturer,operating_system,operating_system_version,id,manufacturer_count,operating_system_count,model_count,operating_system_version_count
0,0,1028623,1,Galaxy S8,131,Android,9,0_1028623,9246,18960.0,1579.0,4972.0
1,0,48047,1,Galaxy Note 8,131,Android,9,0_48047,9246,18960.0,565.0,4972.0
2,0,123568,1,H918,81,Android,8.0.0,0_123568,1711,18960.0,11.0,4428.0
3,0,95366,1,iPhone 6,12,iOS,NaN,0_95366,38702,35174.0,935.0,NaN
4,0,1755023,4,iPad,12,iOS,NaN,0_1755023,38702,35174.0,2243.0,NaN


In [26]:
df_devices.operating_system.value_counts()

iOS                   35174
Android               18960
Windows               11085
Mac                    1330
Axis                    768
watchOS                 705
Watch                   302
Cisco                   237
Linux                   204
Miui                    109
HP                       90
Debian                   35
Tizen                    34
BlackBerry               24
Link-OS                  20
Symbian                  15
Honeywell                11
Zebra                     8
Lexmark                   6
Xerox                     4
Chrome                    4
Firefox                   4
Avaya                     3
Code                      3
Spice                     2
ZebraNet                  1
Win98                     1
TOSHIBA                   1
OS/2                      1
Nintendo                  1
MTK                       1
Maemo                     1
Samsung                   1
SCCP45.9-4-2SR3-1S        1
Name: operating_system, dtype: int64

In [27]:
%%time
df = pd.read_csv("all_sessions_v1.csv.gz") #,nrows=83456

print(df.shape)
# print(df_devices.nunique())
df.tail()

(22463971, 44)
CPU times: user 1min 56s, sys: 27.6 s, total: 2min 23s
Wall time: 2min 28s


,network_id,device_id,timestamp,host,host_ip,port_dst,transport_protocol,service_device_id,packets_count,outbound_bytes_count,inbound_bytes_count,packet_loss,retransmit_count,latency,session_count,outbound_packets_count,inbound_packets_count,outbound_bytes_max,outbound_bytes_min,outbound_bytes_mean,outbound_bytes_median,outbound_bytes_stddev,inbound_bytes_max,inbound_bytes_min,inbound_bytes_mean,inbound_bytes_median,inbound_bytes_stddev,outbound_packet_size_max,outbound_packet_size_min,outbound_packet_size_mean,outbound_packet_size_median,outbound_packet_size_stddev,inbound_packet_size_max,inbound_packet_size_min,inbound_packet_size_mean,inbound_packet_size_median,inbound_packet_size_stddev,id,row_nan_count,service_device_id_count,host_count,device_id_count,id-service_device_counts,port_dst_count
22463966,3,2701197,1565287200,96162,255197,443,TCP,NaN,19,2311,0,0.00000,1,NaN,1,19,0,2311,2311,2311.00000,NaN,NaN,0,0,0.00000,NaN,NaN,121.63158,121.63158,121.63158,121.63158,NaN,NaN,NaN,NaN,NaN,NaN,3_2701197,12,NaN,109945,309,309,13380015
22463967,3,2701266,1565283600,337758,347360,443,TCP,700571.0,58,1852,3906,0.00000,0,NaN,7,29,29,451,139,264.57143,NaN,156.99879,1353,135,558.00000,NaN,565.68395,69.50000,56.37500,66.41752,69.50000,5.27282,216.33333,67.5,108.22874,67.5,60.64558,3_2701266,3,2159.0,1148,501,501,13380015
22463968,3,2702227,1565287200,381770,329778,1146,TCP,2305795.0,114,2337,2964,0.00000,0,NaN,57,57,57,41,41,41.00000,NaN,0.00000,52,52,52.00000,NaN,0.00000,41.00000,41.00000,41.00000,41.00000,0.00000,52.00000,52.0,52.00000,52.0,0.00000,3_2702227,3,5.0,5,314,314,9
22463969,3,2703918,1565298000,377492,326062,53,UDP,NaN,14,535,849,NaN,0,NaN,7,7,7,103,58,76.42857,NaN,16.17464,168,80,121.28571,NaN,35.52330,103.00000,58.00000,76.42857,72.00000,16.17464,168.00000,80.0,121.28571,131.0,35.52330,3_2703918,5,NaN,62303,675,675,999553
22463970,3,2710107,1565287200,30734,222102,443,TCP,NaN,22,4039,0,0.16458,1,NaN,1,22,0,4039,4039,4039.00000,NaN,NaN,0,0,0.00000,NaN,NaN,183.59091,183.59091,183.59091,183.59091,NaN,NaN,NaN,NaN,NaN,NaN,3_2710107,12,NaN,419135,90,90,13380015


In [28]:
list(df.columns)

['network_id',
 'device_id',
 'timestamp',
 'host',
 'host_ip',
 'port_dst',
 'transport_protocol',
 'service_device_id',
 'packets_count',
 'outbound_bytes_count',
 'inbound_bytes_count',
 'packet_loss',
 'retransmit_count',
 'latency',
 'session_count',
 'outbound_packets_count',
 'inbound_packets_count',
 'outbound_bytes_max',
 'outbound_bytes_min',
 'outbound_bytes_mean',
 'outbound_bytes_median',
 'outbound_bytes_stddev',
 'inbound_bytes_max',
 'inbound_bytes_min',
 'inbound_bytes_mean',
 'inbound_bytes_median',
 'inbound_bytes_stddev',
 'outbound_packet_size_max',
 'outbound_packet_size_min',
 'outbound_packet_size_mean',
 'outbound_packet_size_median',
 'outbound_packet_size_stddev',
 'inbound_packet_size_max',
 'inbound_packet_size_min',
 'inbound_packet_size_mean',
 'inbound_packet_size_median',
 'inbound_packet_size_stddev',
 'id',
 'row_nan_count',
 'service_device_id_count',
 'host_count',
 'device_id_count',
 'id-service_device_counts',
 'port_dst_count']

### datetime features
* cyclical datetime : https://www.kaggle.com/avanwyk/encoding-cyclical-features-for-deep-learning
* weekend, workhours

In [29]:
# df["time"] = pd.to_datetime(df.timestamp,unit="s")
# df.drop("time",axis=1,inplace=True)

In [30]:
def cyclical_encode(data, col, max_val):
    data[col + '_sin'] = np.sin(2 * np.pi * data[col]/max_val)
    data[col + '_cos'] = np.cos(2 * np.pi * data[col]/max_val)
    return data

In [31]:
# # df['month'] = df.datetime.dt.month
# # df = encode(df, 'month', 12)

# df['dayOfWeek'] = df.time.dt.dayofweek # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.dayofweek.html#pandas.Series.dt.dayofweek

# df = cyclical_encode(df, 'dayOfWeek', 7)

# df['hour'] = df.time.dt.hour
# df = cyclical_encode(df, 'hour', 24)

# # Monday=0, Sunday=6.
# df["work_day"] = (df.dayOfWeek>=6).astype(int)

# df["work_hours"] = ((df.hour>=8) & (df.hour<=18)).astype(int)

In [32]:
df_devices.head()

,network_id,device_id,type,model,manufacturer,operating_system,operating_system_version,id,manufacturer_count,operating_system_count,model_count,operating_system_version_count
0,0,1028623,1,Galaxy S8,131,Android,9,0_1028623,9246,18960.0,1579.0,4972.0
1,0,48047,1,Galaxy Note 8,131,Android,9,0_48047,9246,18960.0,565.0,4972.0
2,0,123568,1,H918,81,Android,8.0.0,0_123568,1711,18960.0,11.0,4428.0
3,0,95366,1,iPhone 6,12,iOS,NaN,0_95366,38702,35174.0,935.0,NaN
4,0,1755023,4,iPad,12,iOS,NaN,0_1755023,38702,35174.0,2243.0,NaN


In [33]:
df_devices.columns

Index(['network_id', 'device_id', 'type', 'model', 'manufacturer', 'operating_system', 'operating_system_version', 'id', 'manufacturer_count',
       'operating_system_count', 'model_count', 'operating_system_version_count'],
      dtype='object')

In [34]:
df_devices_feat_cols = ['network_id', 'device_id','type', 'manufacturer','manufacturer_count',
       'operating_system_count', 'model_count', 'operating_system_version_count']

## merge with devices

In [35]:
%%time
df = pd.merge(df, df_devices[df_devices_feat_cols], on=['network_id', 'device_id'], how='left')

CPU times: user 29.9 s, sys: 45.7 s, total: 1min 15s
Wall time: 1min 22s


In [36]:
%%time
df['service_device_id'] = df['service_device_id'].fillna(-1).astype(int)

df = pd.merge(df, df_devices[df_devices_feat_cols],
               left_on=['network_id', 'service_device_id'],right_on=['network_id', 'device_id'], how='left',suffixes=("","_service_device"))


CPU times: user 10.5 s, sys: 22.1 s, total: 32.5 s
Wall time: 37 s


In [38]:
df.shape

(22463971, 57)

In [39]:
df.head()

,network_id,device_id,timestamp,host,host_ip,port_dst,transport_protocol,service_device_id,packets_count,outbound_bytes_count,inbound_bytes_count,packet_loss,retransmit_count,latency,session_count,outbound_packets_count,inbound_packets_count,outbound_bytes_max,outbound_bytes_min,outbound_bytes_mean,outbound_bytes_median,outbound_bytes_stddev,inbound_bytes_max,inbound_bytes_min,inbound_bytes_mean,inbound_bytes_median,inbound_bytes_stddev,outbound_packet_size_max,outbound_packet_size_min,outbound_packet_size_mean,outbound_packet_size_median,outbound_packet_size_stddev,inbound_packet_size_max,inbound_packet_size_min,inbound_packet_size_mean,inbound_packet_size_median,inbound_packet_size_stddev,id,row_nan_count,service_device_id_count,host_count,device_id_count,id-service_device_counts,port_dst_count,type,manufacturer,manufacturer_count,operating_system_count,model_count,operating_system_version_count,device_id_service_device,type_service_device,manufacturer_service_device,manufacturer_count_service_device,operating_system_count_service_device,model_count_service_device,operating_system_version_count_service_device
0,0,35,1565074800,413484,357105,49152,TCP,790889,260,21855,7815,0.0675,67,NaN,39,235,25,930,465,560.38462,NaN,190.21935,1563,0,200.38462,NaN,529.37001,93.00000,93.00000,93.00000,93.00000,0.00000,312.6,312.6,312.6,312.6,0.0,0_35,3,36.0,52,7539,7539,29303,2,43,109,11085.0,69.0,3647.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,35,1565053200,252449,218408,49153,TCP,1604622,178,8326,28656,0.0000,16,NaN,18,88,90,467,427,462.55556,467.0,12.93523,1592,1592,1592.00000,1592.0,0.00000,106.75000,93.40000,94.88333,93.00000,4.31713,318.4,318.4,318.4,318.0,0.0,0_35,1,28.0,53,7539,7539,12926,2,43,109,11085.0,69.0,3647.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,57,1565082000,393482,319956,443,TCP,-1,67,4013,0,0.0000,40,NaN,7,67,0,3521,82,573.28571,NaN,1299.81982,0,0,0.00000,NaN,0.00000,64.01818,41.00000,44.28831,41.00000,8.70005,NaN,NaN,NaN,NaN,NaN,0_57,9,NaN,19752,4897,4897,13380015,2,53,559,11085.0,125.0,4571.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,57,1565082000,46068,223744,443,TCP,-1,45,9158,0,0.0000,3,NaN,4,45,0,3401,935,2289.50000,2411.0,1018.07449,0,0,0.00000,0.0,0.00000,226.73333,133.57143,194.96824,209.00000,41.77727,NaN,NaN,NaN,NaN,NaN,0_57,7,NaN,17209,4897,4897,13380015,2,53,559,11085.0,125.0,4571.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,57,1565082000,389390,113695,443,TCP,-1,9,1671,0,0.0000,0,NaN,1,9,0,1671,1671,1671.00000,NaN,NaN,0,0,0.00000,NaN,NaN,185.66667,185.66667,185.66667,185.66667,NaN,NaN,NaN,NaN,NaN,NaN,0_57,12,NaN,16114,4897,4897,13380015,2,53,559,11085.0,125.0,4571.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
# save feats 

# df.to_csv("pre_yamFeaturizing_sessions_all.csv.gz",index=False,compression="gzip")

In [37]:
# services = set(df.service_device_id.head(63456))
# print(len(services))
# df_devices.loc[df_devices.device_id.isin(services)].shape

In [ ]:
# pd.merge(df.sample(43456).drop("id",axis=1), df_devices.drop("id",axis=1),
#          left_on=['network_id', 'service_device_id'],right_on=['network_id', 'device_id'], how='inner').head()


In [ ]:
# df2 = pd.merge(df.sample(223456).drop("id",axis=1), df_devices.drop("id",axis=1), on=['network_id', 'device_id'], how='left')

# df2['service_device_id'] = df2['service_device_id'].fillna(-1).astype(int)

# df2 = pd.merge(df2.head(), df_devices.drop("id",axis=1),
#                left_on=['network_id', 'service_device_id'],right_on=['network_id', 'device_id'], how='inner',suffixes=("","_service_device"))

# df2

## Yam features

In [ ]:
list(df.columns)

In [ ]:
# def extract_features(df):
#     categ_cols = ['id'] # type
#     cols = ['packets_count', 'outbound_bytes_count', 'inbound_bytes_count',
#             'packet_loss', 'retransmit_count', 'latency', 'session_count',
#             'outbound_packets_count', 'inbound_packets_count',
#             'outbound_bytes_max', 'outbound_bytes_min', 'outbound_bytes_mean',
#             'inbound_bytes_max', 'inbound_bytes_min', 'inbound_bytes_mean', 
#            "row_nan_count", 'service_device_id_count',
#  'host_count',
#  'device_id_count',
#  'id-service_device_counts',
#  'port_dst_count']
#     operations = ['mean', 'std', 'min', 'max',"last","nunique"]

#     for categ in categ_cols:
#         for col in cols:
#             for oper in operations:
#                 df['_'.join([col, oper])] = df.groupby(categ_cols)[col].transform(oper)      
#     print("basic aggregations done")
    
#     sum_cols = ['packets_count', 'outbound_bytes_count', 'inbound_bytes_count',
#             'packet_loss', 'retransmit_count', 'latency', 'session_count',
#             'outbound_packets_count', 'inbound_packets_count',
#              'outbound_bytes_mean','inbound_bytes_mean']
#     for categ in categ_cols:
#         for col in cols:
#                 df[col+'_'+"sum"] = df.groupby(categ_cols)[col].transform("sum")
#     print("sum aggregations done")
    
#     df['device_num_of_unique_ips'] = df.groupby('device_id')['service_device_id'].transform('nunique')
    
#     df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
#     df["connection_hour"] = df['timestamp'].dt.hour

#     df['avg_time_of_day_of_connection'] = df.groupby('device_id')['connection_hour'].transform("mean")
#     df["connection_day"] = df['timestamp'].dt.day
#     df["connection_month"] = df['timestamp'].dt.month
# #     df['day_of_week_sessions'] = pd.to_datetime(df['timestamp'], unit='s').dt.dayofweek
#     df['day_of_week_sessions'] = df['timestamp'].dt.dayofweek
#     df["work_day"] = (df.day_of_week_sessions>=6).astype(int)
# #     df["connection_year"] = df['timestamp'].dt.year

# #     df = cyclical_encode(df, 'connection_day', 365)
    
#     print("datetimes done")
    
#     df['avg_times_connected_a_day'] = (
#     df.groupby('device_id')['connection_hour'].transform("count") / df.groupby('device_id')[
#         'connection_hour'].transform("nunique"))

#     # Wrong..
#     # print df['avg_times_connected_a_day']

# #     df['host_ip_is_domain'] = df['host_ip'].map(lambda x: 0 if (x[1] == '.' or x[2] == '.' or x[3] == '.') else 1)
#     df['connected_at_night'] = df['connection_hour'].map(lambda x: 1 if x > 21 or x < 6 else 0)
#     df['dest_in_network'] = df['service_device_id'].map(lambda x: 1 if x in df['device_id'] else 0)

#     # TODO: Groupby device, type
#     df['inbound_to_outbound_packet_count_proportion'] = df['inbound_packets_count'] / df['outbound_packets_count']
#     df['host_ip_device_id_different_connections'] = df.groupby('host_ip')['device_id'].transform('nunique')
#     df['host_ip_different_connections'] = df['host_ip'].nunique()
# #     df['manufacturer_count_usage'] = df.groupby('manufacturer')['device_id'].transform('count')
#     df['network_count_usage'] = df.groupby('network_id')['device_id'].transform('count')
    
#     df['day_of_week_num_of_connections'] = df.groupby('day_of_week_sessions')['device_id'].transform('count')
# #     df.groupby(["device_id"]).aggregate(
# #         {"host": "nunique", "port_dst": "nunique", "packets_count": "sum", "inbound_bytes_count": "sum",
# #          "outbound_bytes_count": "sum"})



#     #     check diff
#     for col in ['type', 'manufacturer']:
#         df[col+"_diff"] = (df[col] == df[col+"_service_device"]).astype(int)
#    # check ratio
#     for col in ['manufacturer_count','operating_system_count', 'model_count', 'operating_system_version_count']:
#         df[col+"_ratio"] = (df[col].div(df[col+"_service_device"]))
                           
#     return df

# # df = extract_features(df.sample(66))
# df = extract_features(df)

In [55]:
df["transport_protocol"] = pd.get_dummies(df["transport_protocol"],dummy_na=False, drop_first=True)

In [150]:
# https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtypes
#         print(col_type)
        if ((col_type.name != "category") & (col_type != object)): # NEW
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
#         else: df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

# df = reduce_mem_usage(df)

In [58]:
df = reduce_mem_usage(df)

Memory usage of dataframe is 9790.45 MB
Memory usage after optimization is: 4094.93 MB
Decreased by 58.2%


In [62]:
df.head(1234)["transport_protocol"].nunique()

2

In [60]:
df.head()

,network_id,device_id,timestamp,host,host_ip,port_dst,transport_protocol,service_device_id,packets_count,outbound_bytes_count,inbound_bytes_count,packet_loss,retransmit_count,latency,session_count,outbound_packets_count,inbound_packets_count,outbound_bytes_max,outbound_bytes_min,outbound_bytes_mean,outbound_bytes_median,outbound_bytes_stddev,inbound_bytes_max,inbound_bytes_min,inbound_bytes_mean,inbound_bytes_median,inbound_bytes_stddev,outbound_packet_size_max,outbound_packet_size_min,outbound_packet_size_mean,outbound_packet_size_median,outbound_packet_size_stddev,inbound_packet_size_max,inbound_packet_size_min,inbound_packet_size_mean,inbound_packet_size_median,inbound_packet_size_stddev,id,row_nan_count,service_device_id_count,host_count,device_id_count,id-service_device_counts,port_dst_count,type,manufacturer,manufacturer_count,operating_system_count,model_count,operating_system_version_count,device_id_service_device,type_service_device,manufacturer_service_device,manufacturer_count_service_device,operating_system_count_service_device,model_count_service_device,operating_system_version_count_service_device
0,0,35,1565074800,413484,357105,49152,0.0,790889,260,21855,7815,0.0675,67,NaN,39,235,25,930,465,560.38464,NaN,190.21935,1563,0,200.38461,NaN,529.37,93.000,93.000,93.00000,93.000,0.00000,312.5,312.5,312.5,312.5,0.0,0_35,3,36.0,52,7539,7539,29303,2,43,109,11088.0,69.0,3648.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,35,1565053200,252449,218408,49153,0.0,1604622,178,8326,28656,0.0000,16,NaN,18,88,90,467,427,462.55554,467.0,12.93523,1592,1592,1592.00000,1592.0,0.00,106.750,93.375,94.87500,93.000,4.31641,318.5,318.5,318.5,318.0,0.0,0_35,1,28.0,53,7539,7539,12926,2,43,109,11088.0,69.0,3648.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,57,1565082000,393482,319956,443,0.0,-1,67,4013,0,0.0000,40,NaN,7,67,0,3521,82,573.28571,NaN,1299.81982,0,0,0.00000,NaN,0.00,64.000,41.000,44.28125,41.000,8.70312,NaN,NaN,NaN,NaN,NaN,0_57,9,NaN,19752,4897,4897,13380015,2,53,559,11088.0,125.0,4572.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,57,1565082000,46068,223744,443,0.0,-1,45,9158,0,0.0000,3,NaN,4,45,0,3401,935,2289.50000,2411.0,1018.07446,0,0,0.00000,0.0,0.00,226.750,133.625,195.00000,209.000,41.78125,NaN,NaN,NaN,NaN,NaN,0_57,7,NaN,17209,4897,4897,13380015,2,53,559,11088.0,125.0,4572.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,57,1565082000,389390,113695,443,0.0,-1,9,1671,0,0.0000,0,NaN,1,9,0,1671,1671,1671.00000,NaN,NaN,0,0,0.00000,NaN,NaN,185.625,185.625,185.62500,185.625,NaN,NaN,NaN,NaN,NaN,NaN,0_57,12,NaN,16114,4897,4897,13380015,2,53,559,11088.0,125.0,4572.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
%%time

def extract_features_rowlevel(df):
#     categ_cols = ['id'] # type
#     cols = ['packets_count', 'outbound_bytes_count', 'inbound_bytes_count',
#             'packet_loss', 'retransmit_count', 'latency', 'session_count',
#             'outbound_packets_count', 'inbound_packets_count',
#             'outbound_bytes_max', 'outbound_bytes_min', 'outbound_bytes_mean',
#             'inbound_bytes_max', 'inbound_bytes_min', 'inbound_bytes_mean', 
#            "row_nan_count", 'service_device_id_count',
#  'host_count',
#  'device_id_count',
#  'id-service_device_counts',
#  'port_dst_count']
#     operations = ['mean', 'std', 'min', 'max',"last","nunique"]

#     for categ in categ_cols:
#         for col in cols:
#             for oper in operations:
#                 df['_'.join([col, oper])] = df.groupby(categ_cols)[col].transform(oper)      
#     print("basic aggregations done")
    
#     sum_cols = ['packets_count', 'outbound_bytes_count', 'inbound_bytes_count',
#             'packet_loss', 'retransmit_count', 'latency', 'session_count',
#             'outbound_packets_count', 'inbound_packets_count',
#              'outbound_bytes_mean','inbound_bytes_mean']
#     for categ in categ_cols:
#         for col in cols:
#                 df[col+'_'+"sum"] = df.groupby(categ_cols)[col].transform("sum")
#     print("sum aggregations done")
    
    df['device_num_of_unique_ips'] = df.groupby('device_id')['service_device_id'].transform('nunique')
    
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
    df["connection_hour"] = df['timestamp'].dt.hour
    df = cyclical_encode(df, 'connection_hour', 24)

    df['avg_time_of_day_of_connection'] = df.groupby('device_id')['connection_hour'].transform("mean")
    df["connection_day"] = df['timestamp'].dt.day
    df["connection_month"] = df['timestamp'].dt.month
#     df['day_of_week_sessions'] = pd.to_datetime(df['timestamp'], unit='s').dt.dayofweek
    df['day_of_week_sessions'] = df['timestamp'].dt.dayofweek
    df["work_day"] = (df.day_of_week_sessions>=6).astype(int)
    df["connection_year"] = df['timestamp'].dt.year

#     df = cyclical_encode(df, 'connection_day', 365)
    
    print("datetimes done")
    
    df['avg_times_connected_a_day'] = (
    df.groupby('device_id')['connection_hour'].transform("count") / df.groupby('device_id')[
        'connection_hour'].transform("nunique"))

    # Wrong..
    # print df['avg_times_connected_a_day']

#     df['host_ip_is_domain'] = df['host_ip'].map(lambda x: 0 if (x[1] == '.' or x[2] == '.' or x[3] == '.') else 1)
    df['connected_at_night'] = df['connection_hour'].map(lambda x: 1 if x > 21 or x < 6 else 0)
    df['dest_in_network'] = df['service_device_id'].map(lambda x: 1 if x in df['device_id'] else 0)

    # TODO: Groupby device, type
    df['inbound_to_outbound_packet_count_proportion'] = df['inbound_packets_count'] / df['outbound_packets_count']
    df['host_ip_device_id_different_connections'] = df.groupby('host_ip')['device_id'].transform('nunique')
    df['host_ip_different_connections'] = df['host_ip'].nunique()
#     df['manufacturer_count_usage'] = df.groupby('manufacturer')['device_id'].transform('count')
    df['network_count_usage'] = df.groupby('network_id')['device_id'].transform('count')
    
    df['day_of_week_num_of_connections'] = df.groupby('day_of_week_sessions')['device_id'].transform('count')
#     df.groupby(["device_id"]).aggregate(
#         {"host": "nunique", "port_dst": "nunique", "packets_count": "sum", "inbound_bytes_count": "sum",
#          "outbound_bytes_count": "sum"})


    #     check diff
    for col in ['type', 'manufacturer']:
        df[col+"_diff"] = (df[col] == df[col+"_service_device"]).astype(int)
   # check ratio
    for col in ['manufacturer_count','operating_system_count', 'model_count', 'operating_system_version_count']:
        df[col+"_ratio"] = (df[col].div(df[col+"_service_device"]))
    
    df.drop("timestamp",axis=1,inplace=True) # drop time col now
    return df

# extract_features_rowlevel(df.head(22000))
print(df.shape)
df = extract_features_rowlevel(df)
print("more feats added")
# df = reduce_mem_usage(df)

print(df.shape)

(22463971, 57)
datetimes done
more feats added
Memory usage of dataframe is 8205.51 MB


TypeError: Categorical is not ordered for operation min
you can use .as_ordered() to change the Categorical to an ordered one


In [ ]:
# df["service_device_id_count"] = df.groupby("service_device_id")["packets_count"].transform("size")
# df["host_count"] = df.groupby("host")["packets_count"].transform("size")
# df["device_id_count"] = df.groupby("device_id")["packets_count"].transform("size")
# df["id-service_device_counts"] = df.groupby(["id","device_id"])["packets_count"].transform("size")
# df["port_dst_count"] = df.groupby("port_dst")["packets_count"].transform("size")

In [71]:
# df.to_csv("all_sessions_v1.csv.gz",compression="gzip",index=False)


df.to_csv("pre_yamFeaturizing_sessions_all_expanded.csv.gz",index=False,compression="gzip")

In [91]:
df.columns

Index(['network_id', 'device_id', 'host', 'host_ip', 'port_dst', 'transport_protocol', 'service_device_id', 'packets_count', 'outbound_bytes_count',
       'inbound_bytes_count', 'packet_loss', 'retransmit_count', 'latency', 'session_count', 'outbound_packets_count', 'inbound_packets_count',
       'outbound_bytes_max', 'outbound_bytes_min', 'outbound_bytes_mean', 'outbound_bytes_median', 'outbound_bytes_stddev', 'inbound_bytes_max',
       'inbound_bytes_min', 'inbound_bytes_mean', 'inbound_bytes_median', 'inbound_bytes_stddev', 'outbound_packet_size_max', 'outbound_packet_size_min',
       'outbound_packet_size_mean', 'outbound_packet_size_median', 'outbound_packet_size_stddev', 'inbound_packet_size_max', 'inbound_packet_size_min',
       'inbound_packet_size_mean', 'inbound_packet_size_median', 'inbound_packet_size_stddev', 'id', 'row_nan_count', 'service_device_id_count', 'host_count',
       'device_id_count', 'id-service_device_counts', 'port_dst_count', 'type', 'manufacturer', '

## final aggregate features

In [82]:
df = reduce_mem_usage(df)

print(df.shape)

Memory usage of dataframe is 8205.51 MB


TypeError: data type "category" not understood

In [86]:
def extract_all_features(df):
#     return df.drop("network_id",axis=1).groupby(["device_id"]).aggregate(["min","max","std","nunique","sum","last","mean"])

    return df.groupby(["device_id","network_id"]).aggregate(["min","max","std","nunique","sum","last","mean"])

#     return df.drop("network_id",axis=1).groupby(["device_id"]).aggregate(["max","mean"])



In [87]:
# df2 = extract_all_features(df.sample(5345))
# print(df2.shape)
# print(df2.head(2))
# df2.tail()

(4228, 539)
<bound method NDFrame.head of                         host                                                         host_ip                                                        port_dst  \
                         min     max           std nunique     sum    last      mean     min     max          std nunique     sum    last      mean      min   
device_id network_id                                                                                                                                           
35        0            60264   60264           NaN       1   60264   60264   60264.0   52540   52540          NaN       1   52540   52540   52540.0    58338   
40        1            32209  195527  115483.26529       2  227736   32209  113868.0  169599  256856  61700.01641       2  426455  256856  213227.5      443   
47        1            94853   94853           NaN       1   94853   94853   94853.0   82462   82462          NaN       1   82462   82462   82462.0       80  

<bound method NDFrame.tail of                         host                                                         host_ip                                                        port_dst  \
                         min     max           std nunique     sum    last      mean     min     max          std nunique     sum    last      mean      min   
device_id network_id                                                                                                                                           
35        0            60264   60264           NaN       1   60264   60264   60264.0   52540   52540          NaN       1   52540   52540   52540.0    58338   
40        1            32209  195527  115483.26529       2  227736   32209  113868.0  169599  256856  61700.01641       2  426455  256856  213227.5      443   
47        1            94853   94853           NaN       1   94853   94853   94853.0   82462   82462          NaN       1   82462   82462   82462.0       80   
54        

In [92]:
%%time
df = extract_all_features(df)

CPU times: user 8min 41s, sys: 1min 1s, total: 9min 43s
Wall time: 9min 47s


In [93]:
print(df.shape)
df.head()

(76038, 539)


host                                                                  host_ip                                                     \
                        min     max           std nunique          sum    last          mean     min     max           std nunique          sum    last   
device_id network_id                                                                                                                                      
13        2           24319  445524  124255.94756      33  8.88392e+06  440229  227792.71795   12913  367950  116866.51919      27  7.67554e+06  255197   
20        2            3918  444247  108734.61874     139  1.42326e+08   37723  216960.28659    2010  385910  103377.16276     199  1.34592e+08  109445   
24        3           43376  377492  111159.60479       6  4.06154e+06  341142  290109.85714   11529  326062  131609.48119       6  2.62139e+06  294676   
31        3           58353  196620   76871.04850       3  4.99273e+05  196620  124818.25000   13437  170531   88150.99579       3  3.58841e+05  170531   
32        3           25122  434752  112361.59975       5  2.91119e+06   25122  291118.60000   21776  375278   97570.30508       5  2.52082e+06   21776   

                                   port_dst                                                         transport_protocol                                    \
                              mean      min    max          std nunique        sum last        mean                min  max      std nunique    sum last   
device_id network_id                                                                                                                                       
13        2           196808.69231       53   8999   1576.31665       5    27628.0  443   708.41026                0.0  1.0  0.30737       2    4.0  0.0   
20        2           205170.63110       53  65360  10149.37686      55  2296156.0  443  3500.23780                0.0  1.0  0.39673       2  128.0  0.0   
24        3           187242.00000       25   5357   2247.11157       4    16962.0  137  1211.57143                0.0  1.0  0.51904       2    7.0  1.0   
31        3            89710.25000      137   5357   3013.76841       2    10988.0  137  2747.00000                0.0  1.0  0.57715       2    2.0  1.0   
32        3           252082.20000       25   5357   1677.89519       4     5834.0  137   583.40000                0.0  1.0  0.48291       2    3.0  1.0   

                              service_device_id                                                                  packets_count                            \
                         mean               min      max           std nunique         sum     last         mean           min   max         std nunique   
device_id network_id                                                                                                                                       
13        2           0.10254                -1      359      97.18299       2      1041.0       -1  2.66923e+01             1  7039  1309.55074      33   
20        2           0.19507                -1      359      70.28892       2      8704.0       -1  1.32683e+01             1  3600   258.68101     154   
24        3           0.50000                -1  2676627  968137.46406       6  21381511.0  2580825  1.52725e+06             1  1415   460.71140      10   
31        3           0.50000           2555156  2648824   44543.41560       3  10458674.0  2555156  2.61467e+06             1    20     8.34666       4   
32        3           0.30005                -1  2673007  860536.27199       5  12917117.0  2665233  1.29171e+06             1  1405   542.84990       9   

                             ... type_diff                        manufacturer_diff                                 manufacturer_count_ratio                  \
                        sum  ...       std nunique  sum last mean               min max  std nunique  sum last mean         

In [90]:
## rename cols after groupby 
## https://stackoverflow.com/questions/43895292/how-do-you-rename-all-columns-in-multi-level-group-by-in-pandas-0-20-1
## https://stackoverflow.com/questions/44635626/rename-result-columns-from-pandas-aggregation-futurewarning-using-a-dict-with

# frame = pd.DataFrame(df2).reset_index(col_level=1)

# frame.columns = frame.columns.get_level_values(1)

# frame.head()

,device_id,network_id,min,max,std,nunique,sum,last,mean,min,max,std,nunique,sum,last,mean,min,max,std,nunique,sum,last,mean,min,max,std,nunique,sum,last,mean,min,max,std,nunique,sum,last,mean,min,max,std,...,std,nunique,sum,last,mean,min,max,std,nunique,sum,last,mean,min,max,std,nunique,sum,last,mean,min,max,std,nunique,sum,last,mean,min,max,std,nunique,sum,last,mean,min,max,std,nunique,sum,last,mean
0,35,0,60264,60264,NaN,1,60264,60264,60264.0,52540,52540,NaN,1,52540,52540,52540.0,58338,58338,NaN,1,58338,58338,58338.0,0.0,0.0,NaN,1,0.0,0.0,0.0,795441,795441,NaN,1,795441,795441,795441.0,100,100,NaN,...,NaN,1,0,0,0.0,0,0,NaN,1,0,0,0.0,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN
1,40,1,32209,195527,115483.26529,2,227736,32209,113868.0,169599,256856,61700.01641,2,426455,256856,213227.5,443,50812,35616.26146,2,51255,443,25627.5,0.0,0.0,0.0,1,0.0,0.0,0.0,-1,-1,0.0,1,-2,-1,-1.0,16,68,36.76955,...,0.0,1,0,0,0.0,0,0,0.0,1,0,0,0.0,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN
2,47,1,94853,94853,NaN,1,94853,94853,94853.0,82462,82462,NaN,1,82462,82462,82462.0,80,80,NaN,1,80,80,80.0,0.0,0.0,NaN,1,0.0,0.0,0.0,12823,12823,NaN,1,12823,12823,12823.0,2996,2996,NaN,...,NaN,1,1,1,1.0,1,1,NaN,1,1,1,1.0,1.0,1.0,NaN,1,1.0,1.0,1.0,NaN,NaN,NaN,0,0.0,NaN,NaN,1.0,1.0,NaN,1,1.0,1.0,1.0,NaN,NaN,NaN,0,0.0,NaN,NaN
3,54,1,195527,195527,0.00000,1,391054,195527,195527.0,169599,169599,0.00000,1,339198,169599,169599.0,25740,47589,15449.57606,2,73329,25740,36664.5,0.0,0.0,0.0,1,0.0,0.0,0.0,-1,-1,0.0,1,-2,-1,-1.0,16,20,2.82843,...,0.0,1,0,0,0.0,0,0,0.0,1,0,0,0.0,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN
4,56,3,412063,412063,NaN,1,412063,412063,412063.0,81410,81410,NaN,1,81410,81410,81410.0,80,80,NaN,1,80,80,80.0,0.0,0.0,NaN,1,0.0,0.0,0.0,686253,686253,NaN,1,686253,686253,686253.0,28,28,NaN,...,NaN,1,0,0,0.0,0,0,NaN,1,0,0,0.0,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN


In [94]:
df.to_csv("Featurized_sessions_all_v1.csv",index=True)

In [103]:
pd.read_csv("Featurized_sessions_all_v1.csv",nrows=4)

,Unnamed: 0,Unnamed: 1,host,host.1,host.2,host.3,host.4,host.5,host.6,host_ip,host_ip.1,host_ip.2,host_ip.3,host_ip.4,host_ip.5,host_ip.6,port_dst,port_dst.1,port_dst.2,port_dst.3,port_dst.4,port_dst.5,port_dst.6,transport_protocol,transport_protocol.1,transport_protocol.2,transport_protocol.3,transport_protocol.4,transport_protocol.5,transport_protocol.6,service_device_id,service_device_id.1,service_device_id.2,service_device_id.3,service_device_id.4,service_device_id.5,service_device_id.6,packets_count,packets_count.1,packets_count.2,...,type_diff.2,type_diff.3,type_diff.4,type_diff.5,type_diff.6,manufacturer_diff,manufacturer_diff.1,manufacturer_diff.2,manufacturer_diff.3,manufacturer_diff.4,manufacturer_diff.5,manufacturer_diff.6,manufacturer_count_ratio,manufacturer_count_ratio.1,manufacturer_count_ratio.2,manufacturer_count_ratio.3,manufacturer_count_ratio.4,manufacturer_count_ratio.5,manufacturer_count_ratio.6,operating_system_count_ratio,operating_system_count_ratio.1,operating_system_count_ratio.2,operating_system_count_ratio.3,operating_system_count_ratio.4,operating_system_count_ratio.5,operating_system_count_ratio.6,model_count_ratio,model_count_ratio.1,model_count_ratio.2,model_count_ratio.3,model_count_ratio.4,model_count_ratio.5,model_count_ratio.6,operating_system_version_count_ratio,operating_system_version_count_ratio.1,operating_system_version_count_ratio.2,operating_system_version_count_ratio.3,operating_system_version_count_ratio.4,operating_system_version_count_ratio.5,operating_system_version_count_ratio.6
0,NaN,NaN,min,max,std,nunique,sum,last,mean,min,max,std,nunique,sum,last,mean,min,max,std,nunique,sum,last,mean,min,max,std,nunique,sum,last,mean,min,max,std,nunique,sum,last,mean,min,max,std,...,std,nunique,sum,last,mean,min,max,std,nunique,sum,last,mean,min,max,std,nunique,sum,last,mean,min,max,std,nunique,sum,last,mean,min,max,std,nunique,sum,last,mean,min,max,std,nunique,sum,last,mean
1,device_id,network_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,13,2,24319,445524,124255.94756176053,33,8883916.0,440229,227792.71794871794,12913,367950,116866.51918887586,27,7675539.0,255197,196808.6923076923,53,8999,1576.3166546355817,5,27628.0,443,708.4102564102565,0.0,1.0,0.3074,2,4.0,0.0,0.10254,-1,359,97.18299446382306,2,1041.0,-1,26.692307692307693,1,7039,1309.5507409222191,...,0.0,1,0.0,0,0.0,0,0,0.0,1,0.0,0,0.0,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN
3,20,2,3918,444247,108734.61873937448,139,142325948.0,37723,216960.28658536586,2010,385910,103377.162758075,199,134591934.0,109445,205170.63109756098,53,65360,10149.376859967933,55,2296156.0,443,3500.237804878049,0.0,1.0,0.3967,2,128.0,0.0,0.1951,-1,359,70.2889215260439,2,8704.0,-1,13.268292682926829,1,3600,258.68100964863913,...,0.0,1,0.0,0,0.0,0,0,0.0,1,0.0,0,0.0,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN


In [106]:
print(df3.shape)

(76038, 541)


In [134]:
pd.read_csv("Featurized_sessions_all_v1.csv",skiprows=[0,1],nrows=4)

,device_id,network_id,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,...,Unnamed: 501,Unnamed: 502,Unnamed: 503,Unnamed: 504,Unnamed: 505,Unnamed: 506,Unnamed: 507,Unnamed: 508,Unnamed: 509,Unnamed: 510,Unnamed: 511,Unnamed: 512,Unnamed: 513,Unnamed: 514,Unnamed: 515,Unnamed: 516,Unnamed: 517,Unnamed: 518,Unnamed: 519,Unnamed: 520,Unnamed: 521,Unnamed: 522,Unnamed: 523,Unnamed: 524,Unnamed: 525,Unnamed: 526,Unnamed: 527,Unnamed: 528,Unnamed: 529,Unnamed: 530,Unnamed: 531,Unnamed: 532,Unnamed: 533,Unnamed: 534,Unnamed: 535,Unnamed: 536,Unnamed: 537,Unnamed: 538,Unnamed: 539,Unnamed: 540
0,13,2,24319,445524,124255.94756,33,8.88392e+06,440229,227792.71795,12913,367950,116866.51919,27,7.67554e+06,255197,196808.69231,53,8999,1576.31665,5,27628.0,443,708.41026,0.0,1.0,0.3074,2,4.0,0.0,0.10254,-1,359,97.18299,2,1041.0,-1,2.66923e+01,1,7039,1309.55074,...,0.0,1,0.0,0,0.0,0,0,0.0,1,0.0,0,0.0,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN
1,20,2,3918,444247,108734.61874,139,1.42326e+08,37723,216960.28659,2010,385910,103377.16276,199,1.34592e+08,109445,205170.63110,53,65360,10149.37686,55,2296156.0,443,3500.23780,0.0,1.0,0.3967,2,128.0,0.0,0.19510,-1,359,70.28892,2,8704.0,-1,1.32683e+01,1,3600,258.68101,...,0.0,1,0.0,0,0.0,0,0,0.0,1,0.0,0,0.0,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN
2,24,3,43376,377492,111159.60479,6,4.06154e+06,341142,290109.85714,11529,326062,131609.48119,6,2.62139e+06,294676,187242.00000,25,5357,2247.11157,4,16962.0,137,1211.57143,0.0,1.0,0.5190,2,7.0,1.0,0.50000,-1,2676627,968137.46406,6,21381511.0,2580825,1.52725e+06,1,1415,460.71140,...,0.0,1,0.0,0,0.0,0,0,0.0,1,0.0,0,0.0,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN
3,31,3,58353,196620,76871.04850,3,4.99273e+05,196620,124818.25000,13437,170531,88150.99579,3,3.58841e+05,170531,89710.25000,137,5357,3013.76841,2,10988.0,137,2747.00000,0.0,1.0,0.5770,2,2.0,1.0,0.50000,2555156,2648824,44543.41560,3,10458674.0,2555156,2.61467e+06,1,20,8.34666,...,0.0,1,0.0,0,0.0,0,0,0.0,1,0.0,0,0.0,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN


In [135]:
# df3 = pd.read_csv("Featurized_sessions_all_v1.csv",skiprows=[1,2,3])
df3 = pd.read_csv("Featurized_sessions_all_v1.csv",skiprows=[0,1])
# df3.rename(columns={'Unnamed: 0':"device_id",'Unnamed: 1':"network_id"},inplace=True)
print(df3.shape)
print(df3.columns)
df3.head()

(76038, 541)
Index(['device_id', 'network_id', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       ...
       'Unnamed: 531', 'Unnamed: 532', 'Unnamed: 533', 'Unnamed: 534', 'Unnamed: 535', 'Unnamed: 536', 'Unnamed: 537', 'Unnamed: 538', 'Unnamed: 539',
       'Unnamed: 540'],
      dtype='object', length=541)


,device_id,network_id,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,...,Unnamed: 501,Unnamed: 502,Unnamed: 503,Unnamed: 504,Unnamed: 505,Unnamed: 506,Unnamed: 507,Unnamed: 508,Unnamed: 509,Unnamed: 510,Unnamed: 511,Unnamed: 512,Unnamed: 513,Unnamed: 514,Unnamed: 515,Unnamed: 516,Unnamed: 517,Unnamed: 518,Unnamed: 519,Unnamed: 520,Unnamed: 521,Unnamed: 522,Unnamed: 523,Unnamed: 524,Unnamed: 525,Unnamed: 526,Unnamed: 527,Unnamed: 528,Unnamed: 529,Unnamed: 530,Unnamed: 531,Unnamed: 532,Unnamed: 533,Unnamed: 534,Unnamed: 535,Unnamed: 536,Unnamed: 537,Unnamed: 538,Unnamed: 539,Unnamed: 540
0,13,2,24319,445524,124255.94756,33,8.88392e+06,440229,227792.71795,12913,367950,116866.51919,27,7.67554e+06,255197,196808.69231,53,8999,1576.31665,5,27628.0,443,708.41026,0.0,1.0,0.3074,2,4.0,0.0,0.10254,-1,359,97.18299,2,1041.0,-1,2.66923e+01,1,7039,1309.55074,...,0.0,1,0.0,0,0.0,0,0,0.0,1,0.0,0,0.0,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN
1,20,2,3918,444247,108734.61874,139,1.42326e+08,37723,216960.28659,2010,385910,103377.16276,199,1.34592e+08,109445,205170.63110,53,65360,10149.37686,55,2296156.0,443,3500.23780,0.0,1.0,0.3967,2,128.0,0.0,0.19510,-1,359,70.28892,2,8704.0,-1,1.32683e+01,1,3600,258.68101,...,0.0,1,0.0,0,0.0,0,0,0.0,1,0.0,0,0.0,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN
2,24,3,43376,377492,111159.60479,6,4.06154e+06,341142,290109.85714,11529,326062,131609.48119,6,2.62139e+06,294676,187242.00000,25,5357,2247.11157,4,16962.0,137,1211.57143,0.0,1.0,0.5190,2,7.0,1.0,0.50000,-1,2676627,968137.46406,6,21381511.0,2580825,1.52725e+06,1,1415,460.71140,...,0.0,1,0.0,0,0.0,0,0,0.0,1,0.0,0,0.0,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN
3,31,3,58353,196620,76871.04850,3,4.99273e+05,196620,124818.25000,13437,170531,88150.99579,3,3.58841e+05,170531,89710.25000,137,5357,3013.76841,2,10988.0,137,2747.00000,0.0,1.0,0.5770,2,2.0,1.0,0.50000,2555156,2648824,44543.41560,3,10458674.0,2555156,2.61467e+06,1,20,8.34666,...,0.0,1,0.0,0,0.0,0,0,0.0,1,0.0,0,0.0,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN
4,32,3,25122,434752,112361.59975,5,2.91119e+06,25122,291118.60000,21776,375278,97570.30508,5,2.52082e+06,21776,252082.20000,25,5357,1677.89519,4,5834.0,137,583.40000,0.0,1.0,0.4830,2,3.0,1.0,0.30000,-1,2673007,860536.27199,5,12917117.0,2665233,1.29171e+06,1,1405,542.84990,...,0.0,1,0.0,0,0.0,0,0,0.0,1,0.0,0,0.0,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN


In [158]:
df3.replace([np.inf, -np.inf], np.nan,inplace=True)

In [159]:
# df3.dropna(axis=1,how="all").shape # same shape

(76038, 541)

In [160]:
df3.to_csv("Featurized_sessions_all_v1_alt_headers.csv.gz",index=True,compression="gzip")

In [107]:
df = df3.copy()

In [89]:
# df2.to_csv("sampled.csv")

# Baseline

# basic features + sub
* base on baseline

* features per network
* orig: As we mentioned before we are going to select only 5 features: unique hosts count, unique port count, total packets transferred, total inbound bytes and total outbound bytes. Lets create the feature set for each network

In [ ]:
list(df.columns)

In [ ]:
df.head()


In [ ]:
# print(df.nunique())

In [ ]:
df.shape

In [ ]:
def extract_features(df):
    return df.groupby(["device_id"]).aggregate({"host": "nunique",
                                                "port_dst": "nunique",
                                                 'row_nan_count':["min","max","std"],
                                                "service_device_id": "nunique",
 'service_device_id_count':["min","max"],
 'host_count':["min","max"],
 'device_id_count':["min","max"],
 'id-service_device_counts':["min","max"],                                
 'port_dst_count':["min","max","mean"],
                                                "row_nan_count":["min","max","mean"],
                                                
                                                "host_ip": "nunique",
                                                "packets_count":["sum","mean"], "inbound_bytes_count": ["sum","mean"], 
                                                "outbound_bytes_count": ["sum","mean"],
                                               "latency":["max","min","std","mean"],
                                               
   'dayOfWeek':["mean","std","nunique"],
 'dayOfWeek_sin':["mean","std","nunique"],
 'dayOfWeek_cos':["mean","std","nunique"],
 'hour':["mean","std","nunique"],
 'hour_sin':["mean","std","max"],
 'hour_cos':["mean","std","max"],
 'work_day':["mean","std","max"],
 'work_hours':["mean","std","max"],                                             
                                               })

In [ ]:
%%time

networks_fs = []


# for i in range(4):
#     print(i)
# #     networks_fs.append(extract_features(df[df.network_id == i])) # selected fetas
#     networks_fs.append(extract_all_features(df[df.network_id == i])) # all feats
    
#     ### fill nans
# for i in range(4):
# #     networks_fs[i].fillna(-1,inplace=True)
#     networks_fs[i].dropna(axis=1,inplace=True)


### Modeling Fun
* As we've mentioned we used the Elliptic Envelope model for our anomaly detection. We use the decision_function in order to receive the confidence and we normalize the values to values between 0 and 1.

In [168]:
# We use the simple min-max normalization in order to normalize the confidence values to 0-1 range.
# Higher score means that this device is probably more anomalous.
def calc_normalized_decision(decision_function_result):
    decision_function_result = -1 * decision_function_result
    minimum = decision_function_result.min()
    maximum = decision_function_result.max()
    return (decision_function_result - minimum) / (maximum - minimum)

def detect_anomalies(feature_set, param = 0.05, bootstrap=False, n_estimators=150,
                     behaviour='new',max_samples=800, n_jobs=3):
#     ee = EllipticEnvelope(contamination=param).fit(feature_set.values)
    ee = IsolationForest(contamination=param,n_estimators=n_estimators,bootstrap=bootstrap,
                         max_samples=max_samples, behaviour=behaviour,n_jobs=n_jobs).fit(feature_set.values)
    decision_function_result = ee.decision_function(feature_set.values)
    feature_set["confidence"] = calc_normalized_decision(decision_function_result)
#     return feature_set ## ORIG
    return feature_set["confidence"]


def detect_anomalies_ee(feature_set, param = 0.05, bootstrap=False, n_estimators=150,
                     behaviour='new',max_samples=800, n_jobs=3):
    ee = EllipticEnvelope(contamination=param).fit(feature_set.values)
#     ee = IsolationForest(contamination=param,n_estimators=n_estimators,bootstrap=bootstrap,
#                          max_samples=max_samples, behaviour=behaviour,n_jobs=n_jobs).fit(feature_set.values)
    decision_function_result = ee.decision_function(feature_set.values)
    feature_set["confidence"] = calc_normalized_decision(decision_function_result)
    return feature_set["confidence"]

In [125]:
# networks_fs = []


In [ ]:
# networks_fs = []


# param = 0.05
# bootstrap = True
# n_estimators = 80
# behaviour = "new"

# for i in range(4):
#     print(i)
# #             networks_fs.append(extract_features(df[df.network_id == i])
#     networks_fs.append(extract_all_features(df[df.network_id == i]))
# ### drop or fill nans
# for i in range(4):
# #     networks_fs[i].fillna(-1,inplace=True)
#     networks_fs[i].dropna(axis=1,inplace=True)

# for i in range(4):
#     networks_fs[i] = detect_anomalies(feature_set=networks_fs[i], param=param,  bootstrap=bootstrap,
#                                       n_estimators=n_estimators, behaviour=behaviour) # ORIG
# #         networks_fs[i] = detect_anomalies(feature_set=networks_fs[i])

# # Remove the device_id from index and Add network id to each data set
# for i in range(4):
#     networks_fs[i].reset_index(level=0, inplace=True)
#     networks_fs[i]["network_id"] = i 

# df_to_submit = pd.concat(networks_fs)[["network_id", "device_id", "confidence"]] # The column order is important!
# df_to_submit.fillna(0,inplace=True) # handle nans
# df_to_submit.head()
# arr_to_submit = df_to_submit.to_json(orient='values')


# leaderboard_name = "armis"
# host = "leaderboard.datahack.org.il"

# # Name of the user
# submitter = ".303Overfitter"

# predictions = json.loads(arr_to_submit)

# jsonStr = json.dumps({'submitter': submitter, 'predictions': predictions})
# data = jsonStr.encode('utf-8')
# req = request.Request(f"https://{host}/{leaderboard_name}/api/",
#                       headers={'Content-Type': 'application/json'},
#                       data=data)
# resp = request.urlopen(req)

# ret = json.load(resp)

# print(ret['score'])

In [ ]:
# for i in range(4):
#     networks_fs[i].drop("index",axis=1,inplace=True)
#     networks_fs[i].drop('level_0',axis=1,inplace=True)
   
# networks_fs[0].tail()

In [146]:
df3 = reduce_mem_usage(df3)

print(df3.shape)

Memory usage of dataframe is 313.85 MB
int64
int64
int64
int64
float64
int64
float64
int64
float64
int64
int64
float64
int64
float64
int64
float64
int64
int64
float64
int64
float64
int64
float64
float64
float64
float64
int64
float64
float64
float64
int64
int64
float64
int64
float64
int64
float64
int64
int64
float64
int64
int64
int64
float64
int64
int64
float64
int64
int64
int64
float64
int64
int64
float64
int64
int64
int64
float64
float64
float64
float64
int64
float64
float64
float64
int64
int64
float64
int64
int64
int64
float64
float64
float64
float64
int64
float64
float64
float64
int64
int64
float64
int64
float64
int64
float64
int64
int64
float64
int64
int64
int64
float64
int64
int64
float64
int64
int64
int64
float64
int64
int64
float64
int64
float64
int64
float64
int64
int64
float64
int64
float64
int64
float64
float64
float64
float64
int64
float64
float64
float64
float64
float64
float64
int64
float64
float64
float64
float64
float64
float64
int64
float64
float64
float64
int64
int64
f

In [163]:
df_to_submit = df3[["network_id", "device_id"]].copy()

In [165]:
df_to_submit ["confidence"] = detect_anomalies(df3.dropna(axis=1).drop(["network_id", "device_id"],axis=1))

/Users/danofer/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:223: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)


In [169]:
df_to_submit ["confidence_nan_fill"] = detect_anomalies(df3.fillna(-1).drop(["network_id", "device_id"],axis=1))

In [172]:
df_to_submit.fillna(0,inplace=True) # handle nans
df_to_submit.head()
# arr_to_submit = df_to_submit.drop("confidence",axis=1).rename(columns={"confidence_nan_fill":"confidence"}).to_json(orient='values')
arr_to_submit = df_to_submit.to_json(orient='values')


leaderboard_name = "armis"
host = "leaderboard.datahack.org.il"

# Name of the user
submitter = ".303GrimOverfitters"

predictions = json.loads(arr_to_submit)

jsonStr = json.dumps({'submitter': submitter, 'predictions': predictions})
data = jsonStr.encode('utf-8')
req = request.Request(f"https://{host}/{leaderboard_name}/api/",
                      headers={'Content-Type': 'application/json'},
                      data=data)
resp = request.urlopen(req)

ret = json.load(resp)
print(ret['score'])

,network_id,device_id,confidence,confidence_nan_fill
0,2,13,0.03198,0.04885
1,2,20,0.11254,0.07982
2,3,24,0.15004,0.11290
3,3,31,0.17045,0.09731
4,3,32,0.19617,0.20341


0.8010108642232261


## get high confidence preds + times

In [173]:
df_time = pd.read_csv("all_sessions_v1.csv.gz",usecols=["network_id","device_id","timestamp"])
df_time["timestamp"] = pd.to_datetime(df_time["timestamp"],unit="s")

In [180]:
df_time.head().groupby(["network_id","device_id"])["timestamp"].max().head()

network_id  device_id
0           35          2019-08-06 07:00:00
            57          2019-08-06 09:00:00
Name: timestamp, dtype: datetime64[ns]

In [174]:
df_devices = pd.read_csv("all_devices_v1.csv.gz").drop("Unnamed: 0",axis=1) #,nrows=12123
print(df_devices.shape)
print(df_devices.nunique())
# df_devices.to_csv("all_devices_v1.csv.gz",compression="gzip",index=False)
df_devices.head()


print("unique manuf",df_devices["manufacturer"].str.split(expand=True)[0].nunique())

df_devices["manufacturer"] = df_devices["manufacturer"].str.split(expand=True)[0].str.split(",",expand=True)[0]

df_devices["operating_system"] = df_devices["operating_system"].str.split(expand=True)[0].str.split(",",expand=True)[0]

dev_count_cols = ['manufacturer','operating_system',"model","operating_system_version"]
for col in dev_count_cols:
    df_devices[col+'_count'] = df_devices.groupby(col)["device_id"].transform("count")


(76261, 8)
network_id                      4
device_id                   71663
type                            7
model                        1357
manufacturer                  263
operating_system               80
operating_system_version      266
id                          76261
dtype: int64


,network_id,device_id,type,model,manufacturer,operating_system,operating_system_version,id
0,0,1028623,MOBILE_PHONE,Galaxy S8,Samsung,Android,9,0_1028623
1,0,48047,MOBILE_PHONE,Galaxy Note 8,Samsung,Android,9,0_48047
2,0,123568,MOBILE_PHONE,H918,LG Electronics,Android,8.0.0,0_123568
3,0,95366,MOBILE_PHONE,iPhone 6,"Apple, Inc.",iOS,NaN,0_95366
4,0,1755023,TABLET,iPad,Apple,iOS,NaN,0_1755023


unique manuf 179


In [177]:
df_to_submit["mean_pred"] = df_to_submit[["confidence","confidence_nan_fill"]].mean(axis=1)
print(df_to_submit.shape)
df_to_submit["mean_pred"].describe(percentiles=[0.5,0.75,0.95])
df_to_submit.tail()

(76038, 5)


count    76038.00000
mean         0.13833
std          0.11144
min          0.00492
50%          0.10266
75%          0.17419
95%          0.37044
max          0.99669
Name: mean_pred, dtype: float64

,network_id,device_id,confidence,confidence_nan_fill,mean_pred
76033,3,2740039,0.05947,0.05697,0.05822
76034,3,2740803,0.03114,0.05535,0.04325
76035,3,2741153,0.04829,0.09676,0.07252
76036,3,2741453,0.09437,0.09987,0.09712
76037,3,2742647,0.07600,0.06315,0.06957


In [212]:
# a = 85 submission

pd.DataFrame(a, columns=["network_id","device_id","confidence"]).head()

,network_id,device_id,confidence
0,0,33,0.01994
1,0,35,0.62804
2,0,40,0.90320
3,0,41,0.02132
4,0,53,0.02795


In [213]:
# df_confident = df_to_submit.drop(["confidence","confidence_nan_fill"],axis=1).merge(df_devices,on=["network_id","device_id"],how="left")

df_confident = pd.DataFrame(a, columns=["network_id","device_id","confidence"]).merge(df_devices,on=["network_id","device_id"],how="left")

In [214]:
df_confident  = df_confident.merge(df_time.groupby(["network_id","device_id"])["timestamp"].max(),on=["network_id","device_id"],how="left")


df_confident.head()


In [216]:
df_confident["confidence"].describe(percentiles=[.2,.5,.75,.85,.95])

count    76038.00000
mean         0.11461
std          0.14969
min          0.00000
20%          0.02119
50%          0.05553
75%          0.13895
85%          0.21802
95%          0.43782
max          1.00000
Name: confidence, dtype: float64

In [217]:
# df_confident.loc[df_confident["mean_pred"]>.3,"pred"] = 1
# df_confident.loc[df_confident["mean_pred"]<.1,"pred"] = 0

df_confident.loc[df_confident["confidence"]>.29,"pred"] = 1
df_confident.loc[df_confident["confidence"]<.06,"pred"] = 0

In [186]:
# df_confident.loc[((df_confident["mean_pred"]>.3) | (df_confident["mean_pred"]<.1))].drop("mean_pred",axis=1).to_csv("confident_preds_subset_v1.csv.gz",index=False,compression="gzip")

In [218]:
# df_confident.to_csv("get_preds_all.csv.gz",index=False,compression="gzip")

df_confident.to_csv("85_preds_subset_v1.csv.gz",index=False,compression="gzip")

In [192]:
import pickle

In [200]:
predictions[0]

TypeError: 'int' object is not subscriptable

In [198]:
# arr_to_submit = pd.read_csv("SparkBeyond_get_preds_all.csv.gz").to_json(orient='values')


leaderboard_name = "armis"
host = "leaderboard.datahack.org.il"

# Name of the user
submitter = "Overfitters"

# predictions = json.loads(arr_to_submit)

jsonStr = json.dumps({'submitter': submitter, 'predictions': predictions})
data = jsonStr.encode('utf-8')
req = request.Request(f"https://{host}/{leaderboard_name}/api/",
                      headers={'Content-Type': 'application/json'},
                      data=data)
resp = request.urlopen(req)

ret = json.load(resp)
print(ret['score'])

HTTPError: HTTP Error 500: INTERNAL SERVER ERROR

In [201]:
predictions = json.loads("85")

In [203]:
a = [0,33,0.0199444726],[0,35,0.6280394375],[0,40,0.9031979515],[0,41,0.021317463],[0,53,0.0279461945],[0,55,0.0279461945],[0,57,0.5190465283],[0,59,0.0199444726],[0,66,0.1245385134],[0,71,0.0199444726],[0,74,0.0401629957],[0,76,0.0279461945],[0,81,0.6048310182],[0,95,0.5699513109],[0,96,0.112466693],[0,97,0.1361220247],[0,99,0.4804603683],[0,100,0.0281252949],[0,110,0.3932099913],[0,118,0.0347156784],[0,121,0.0465157093],[0,122,0.0352835684],[0,131,0.0253890434],[0,138,0.0199444726],[0,140,0.0279461945],[0,142,0.0281252949],[0,147,0.0199444726],[0,154,0.4655476832],[0,157,0.2111825879],[0,159,0.087547083],[0,161,0.0281252949],[0,163,0.0381534549],[0,166,0.0465157093],[0,169,0.0480068551],[0,171,0.0700831233],[0,172,0.0281252949],[0,175,0.0279461945],[0,176,0.0221533478],[0,189,0.0364062228],[0,191,0.0279461945],[0,194,0.0199444726],[0,200,0.0858120227],[0,203,0.0279461945],[0,206,0.9853821032],[0,209,0.0332066224],[0,213,0.0345540428],[0,214,0.4600405436],[0,215,0.0231951592],[0,216,0.0374415369],[0,220,0.4153267737],[0,223,0.0281252949],[0,224,0.043086422],[0,228,0.0281252949],[0,230,0.0199444726],[0,232,0.0465157093],[0,233,0.0244511055],[0,240,0.0199444726],[0,245,0.0649285122],[0,247,0.0199444726],[0,248,0.2217334605],[0,257,0.0272960463],[0,259,0.021317463],[0,261,0.0279461945],[0,262,0.0281252949],[0,264,0.0199444726],[0,267,0.0231076133],[0,270,0.0199444726],[0,274,0.405715676],[0,275,0.0465157093],[0,276,0.0281252949],[0,278,0.0378700458],[0,280,0.0199444726],[0,292,0.0279461945],[0,294,0.0314299452],[0,302,0.0465157093],[0,307,0.0199444726],[0,309,0.0465157093],[0,312,0.0465157093],[0,314,0.0226574817],[0,315,0.0199444726],[0,316,0.1009434465],[0,327,0.0440471738],[0,329,0.0361469531],[0,334,0.0199444726],[0,344,0.0281252949],[0,346,0.909908594],[0,347,0.4471947478],[0,348,0.0465157093],[0,349,0.0465157093],[0,354,0.0200886555],[0,358,0.0561978774],[0,362,0.0231076133],[0,363,0.0279461945],[0,364,0.0457358406],[0,372,0.8886834965],[0,377,0.0279461945],[0,379,0.0200886555],[0,390,0.0199444726],[0,392,0.0411953637],[0,395,0.3014632671],[0,401,0.0281252949],[0,405,0.0743291284],[0,419,0.0448981781],[0,423,0.0409695718],[0,428,0.0465157093],[0,432,0.0281252949],[0,433,0.0281252949],[0,435,0.0279461945],[0,439,0.0199444726],[0,448,0.4497103221],[0,454,0.0331518872],[0,455,0.4548417636],[0,458,0.0322755186],[0,459,0.0199444726],[0,461,0.0199444726],[0,465,0.0465157093],[0,467,0.0369334713],[0,473,0.0381805862],[0,476,0.0201515479],[0,478,0.0445360544],[0,495,0.4732806174],[0,496,0.0743096496],[0,505,0.0293445833],[0,514,0.0454932883],[0,517,0.0590869216],[0,518,0.0279461945],[0,520,0.0465157093],[0,521,0.0201515479],[0,522,0.0199444726],[0,525,0.0378045131],[0,528,0.0272960463],[0,529,0.0188774289],[0,533,0.0403612784],[0,540,0.0201515479],[0,541,0.0281252949],[0,544,0.430971363],[0,545,0.0404335931],[0,546,0.0465157093],[0,555,0.1217113259],[0,556,0.0199444726],[0,565,0.031673969],[0,569,0.0465157093],[0,575,0.0382973316],[0,580,0.0838833369],[0,584,0.0373778101],[0,598,0.0320812623],[0,603,0.456363912],[0,604,0.0281252949],[0,606,0.0410414197],[0,616,0.0279461945],[0,618,0.2255930286],[0,634,0.0361725603],[0,639,0.0842033205],[0,641,0.0199444726],[0,645,0.0212205672],[0,648,0.0767282269],[0,651,0.0383176452],[0,659,0.1771877667],[0,669,0.1186097514],[0,674,0.0199444726],[0,679,0.0432809937],[0,681,0.0302138592],[0,682,0.1749616085],[0,690,0.4502203822],[0,692,0.4585314807],[0,693,0.1149870869],[0,696,0.0199444726],[0,703,0.4206006739],[0,707,0.0465157093],[0,710,0.0411545563],[0,711,0.0281252949],[0,717,0.0279461945],[0,719,0.0465157093],[0,721,0.0279461945],[0,726,0.0226574817],[0,728,0.1069949602],[0,731,0.0465157093],[0,739,0.0432065085],[0,741,0.0264349421],[0,743,0.0411545563],[0,744,0.0383739822],[0,748,0.0313897713],[0,754,0.0199444726],[0,756,0.0337230318],[0,772,0.4541493193],[0,776,0.0423823718],[0,777,0.0415262982],[0,779,0.1283240254],[0,780,0.0336030645],[0,785,0.0465157093],[0,786,0.0452835637],[0,788,0.023292332],[0,790,0.031943415],[0,791,0.0281252949],[0,792,0.0465157093],[0,801,0.0361469531],[0,803,0.0271264969],[0,805,0.0264988897],[0,809,0.0387646214],[0,812,0.0281252949],[0,831,0.0301196843],[0,836,0.0465157093],[0,837,0.0281252949],[0,841,0.2615859949],[0,845,0.0331373179],[0,847,0.463173866],[0,848,0.0197375767],[0,851,0.0866658381],[0,853,0.0371230288],[0,857,0.045653702],[0,861,0.0281252949],[0,863,0.0218280312],[0,865,0.0281252949],[0,866,0.0227793604],[0,868,0.0704210022],[0,869,0.0281252949],[0,874,0.0199444726],[0,878,0.0199444726],[0,879,0.4715875225],[0,880,0.048046139],[0,881,0.0281252949],[0,883,0.0299748132],[0,884,0.0281306609],[0,885,0.0279461945],[0,896,0.0353925682],[0,912,0.1157626275],[0,917,0.6070128419],[0,919,0.4618892537],[0,924,0.0279461945],[0,927,0.7228906143],[0,934,0.0300458257],[0,935,0.0215862377],[0,936,0.0545630653],[0,947,0.0340780054],[0,951,0.0745425352],[0,952,0.2670706004],[0,955,0.0199444726],[0,956,0.0279461945],[0,963,0.0522271091],[0,966,0.0331373179],[0,967,0.0361383758],[0,982,0.2106140095],[0,985,0.4653151803],[0,990,0.1005498041],[0,994,0.0199444726],[0,998,0.9499426304],[0,1000,0.1853975735],[0,1002,0.0968342914],[0,1006,0.0391375182],[0,1008,0.0650811054],[0,1013,0.0590980656],[0,1014,0.5431694294],[0,1015,0.0251633134],[0,1017,0.0457246617],[0,1021,0.0759128496],[0,1033,0.0199444726],[0,1040,0.8519716976],[0,1041,0.7072260675],[0,1043,0.2722279365],[0,1049,0.0201515479],[0,1051,0.0202906398],[0,1056,0.0212205672],[0,1058,0.6163819678],[0,1059,0.5756187519],[0,1071,0.0199444726],[0,1073,0.0200886555],[0,1074,0.0390165259],[0,1075,0.0279461945],[0,1086,0.2636444254],[0,1089,0.039104293],[0,1095,0.0399275199],[0,1098,0.0289317054],[0,1099,0.9054218832],[0,1106,0.0356434558],[0,1111,0.6254777494],[0,1116,0.0279461945],[0,1117,0.0378980068],[0,1119,0.0418199219],[0,1128,0.0281252949],[0,1138,0.3507433811],[0,1139,0.0199444726],[0,1142,0.0949404658],[0,1144,0.1191081807],[0,1149,0.510181868],[0,1157,0.0199444726],[0,1158,0.0569787714],[0,1164,0.2108341755],[0,1169,0.2107628502],[0,1171,0.1571790329],[0,1175,0.0585661788],[0,1179,0.0444025791],[0,1182,0.0281252949],[0,1184,0.0279461945],[0,1187,0.0292471811],[0,1188,0.6701214357],[0,1189,0.0279461945],[0,1194,0.0603325868],[0,1198,0.153411792],[0,1199,0.0402253543],[0,1200,0.5762618586],[0,1202,0.8726966447],[0,1210,0.0281252949],[0,1212,0.0649378109],[0,1214,0.4477788004],[0,1218,0.7322481266],[0,1229,0.0491585587],[0,1232,0.0308705068],[0,1233,0.8540315951],[0,1238,0.0943708728],[0,1239,0.0279461945],[0,1241,0.3529009844],[0,1244,0.0489853224],[0,1246,0.2228675396],[0,1254,0.0453146041],[0,1265,0.0221751444],[0,1269,0.0279461945],[0,1276,0.184603151],[0,1278,0.7681901596],[0,1279,0.0215862377],[0,1284,0.8530549771],[0,1291,0.0212205672],[0,1295,0.0279461945],[0,1296,0.0329791494],[0,1300,0.0435284285],[0,1312,0.0279461945],[0,1315,0.0231076133],[0,1316,0.021317463],[0,1317,0.0517244511],[0,1319,0.0279461945],[0,1325,0.1482163838],[0,1326,0.8037863475],[0,1327,0.4385234886],[0,1329,0.0716074173],[0,1330,0.4018953357],[0,1332,0.0286217846],[0,1333,0.5167278723],[0,1343,0.0200886555],[0,1344,0.057370677],[0,1345,0.1583987121],[0,1346,0.0410455971],[0,1349,0.0516416727],[0,1350,0.0579071178],[0,1352,0.1080485833],[0,1353,0.023292332],[0,1356,0.0398966312],[0,1358,0.0279461945],[0,1365,0.0244511055],[0,1369,0.5381907934],[0,1371,0.1405385817],[0,1373,0.1306926036],[0,1374,0.1139394877],[0,1377,0.1415170543],[0,1379,0.0199444726],[0,1380,0.1838458882],[0,1381,0.027625725],[0,1386,0.86982717],[0,1390,0.123014175],[0,1394,0.0333420617],[0,1397,0.5351363749],[0,1398,0.0244376486],[0,1402,0.9477530364],[0,1405,0.5357519364],[0,1432,0.1219356876],[0,1435,0.0279461945],[0,1440,0.0199444726],[0,1441,0.0212205672],[0,1443,0.0279461945],[0,1451,0.0231076133],[0,1454,0.9598325517],[0,1460,0.0333420617],[0,1463,0.7830046726],[0,1465,0.1802236268],[0,1467,0.0569081624],[0,1470,0.0279461945],[0,1474,0.0221751444],[0,1476,0.0212205672],[0,1479,0.4603596336],[0,1480,0.0425611219],[0,1485,0.0461236863],[0,1486,0.0517244511],[0,1490,0.0536323981],[0,1492,0.0517244511],[0,1495,0.1070829131],[0,1497,0.0349653318],[0,1500,0.0279461945],[0,1507,0.1741757097],[0,1512,0.2584811397],[0,1519,0.080456272],[0,1525,0.3820379729],[0,1527,0.0325180726],[0,1529,0.0202906398],[0,1530,0.0279461945],[0,1531,0.0279461945],[0,1536,0.0636415437],[0,1544,0.0898810703],[0,1545,0.0960955677],[0,1546,0.1502884747],[0,1548,0.279668663],[0,1552,0.0199444726],[0,1555,0.0279461945],[0,1574,0.0281252949],[0,1577,0.290736575],[0,1578,0.0212205672],[0,1581,0.4429606843],[0,1584,0.5245846183],[0,1588,0.0770944377],[0,1590,0.0752891772],[0,1601,0.0751878238],[0,1602,0.0279461945],[0,1612,0.848887073],[0,1615,0.5265608657],[0,1620,0.0279461945],[0,1621,0.0212205672],[0,1622,0.2998947295],[0,1626,0.2952620073],[0,1628,0.1194795311],[0,1629,0.0279461945],[0,1633,0.2735337707],[0,1636,0.0279461945],[0,1638,0.0532678428],[0,1646,0.0212205672],[0,1648,0.065421554],[0,1657,0.6982800793],[0,1665,0.3131225182],[0,1668,0.0509465388],[0,1670,0.0625632445],[0,1675,0.5304352631],[0,1679,0.597549782],[0,1685,0.3248513733],[0,1686,0.0317136365],[0,1690,0.0202906398],[0,1691,0.0665599329],[0,1692,0.0676744212],[0,1694,0.0748197027],[0,1695,0.0379232811],[0,1697,0.5102376319],[0,1700,0.0279461945],[0,1701,0.0279461945],[0,1703,0.0460416497],[0,1706,0.0280551086],[0,1713,0.1281366246],[0,1714,0.0279461945],[0,1720,0.6123109346],[0,1721,0.0279461945],[0,1725,0.0484812695],[0,1728,0.0325180726],[0,1729,0.0279461945],[0,1731,0.7324996307],[0,1732,0.0542703992],[0,1734,0.0279461945],[0,1741,0.0279461945],[0,1748,0.0212205672],[0,1750,0.2651962951],[0,1751,0.0278003681],[0,1753,0.0199444726],[0,1755,0.9477530364],[0,1764,0.0199444726],[0,1772,0.0293012953],[0,1773,0.0364234493],[0,1781,0.0199444726],[0,1787,0.0231076133],[0,1811,0.0199444726],[0,1813,0.0210382082],[0,1814,0.0199444726],[0,1820,0.0202523363],[0,1822,0.2977029759],[0,1824,0.0253899302],[0,1832,0.0739666189],[0,1840,0.0174175457],[0,1849,0.0253899302],[0,1864,0.0372825485],[0,1865,0.0174175457],[0,1867,0.0174175457],[0,1868,0.0253899302],[0,1874,0.0253899302],[0,1905,0.028892068],[0,1906,0.0253899302],[0,1916,0.0175612],[0,1918,0.0192941985],[0,1920,0.0334161211],[0,1944,0.0206658935],[0,1948,0.0255683741],[0,1951,0.2158177146],[0,1952,0.8895214688],[0,1965,0.0255683741],[0,1974,0.0290699608],[0,1987,0.0253899302],[0,2008,0.0174175457],[0,2015,0.301202332],[0,2017,0.0255683741],[0,2022,0.0192941985],[0,2033,0.0281197251],[0,2035,0.3884293646],[0,2039,0.4563683458],[0,2040,0.0175612],[0,2050,0.4207194134],[0,2057,0.038685115],[0,2064,0.5064104092],[0,2068,0.5332086421],[0,2082,0.3237778432],[0,2084,0.8602280384],[0,2089,0.0253899302],[0,2094,0.0216566571],[0,2098,0.3197827939],[0,2100,0.0220670759],[0,2112,0.0255683741],[0,2118,0.9598325517],[0,2126,0.979793982],[0,2127,0.079899377],[0,2133,0.5104468314],[0,2143,0.9820269622],[0,2163,0.1235263842],[0,2168,0.7693341905],[0,2171,0.5648748364],[0,2182,0.0286157233],[0,2186,0.0175612],[0,2190,0.9609351539],[0,2191,0.5723115275],[0,2195,0.0177491103],[0,2196,0.0260732046],[0,2198,0.9206316474],[0,2199,0.40597624],[0,2209,0.7471001323],[0,2217,0.891632212],[0,2220,0.2980399756],[0,2234,0.2265986476],[0,2235,0.9576295841],[0,2250,0.0253899302],[0,2255,0.5280430858],[0,2260,0.3073404406],[0,2262,0.393738217],[0,2263,0.5532764194],[0,2267,0.0231985087],[0,2275,0.1448807491],[0,2280,0.2732934215],[0,2284,0.24892632],[0,2289,0.6183812518],[0,2290,0.7771210834],[0,2294,0.6516082981],[0,2299,0.772085695],[0,2306,0.3988729464],[0,2308,0.6191666119],[0,2316,0.3846132737],[0,2318,0.3477816536],[0,2325,0.0192941985],[0,2327,0.0174175457],[0,2331,0.0174175457],[0,2337,0.0253899302],[0,2339,0.0192941985],[0,2340,0.0284368129],[0,2341,0.0174175457],[0,2343,0.1700721825],[0,2346,0.0253899302],[0,2348,0.0241741206],[0,2355,0.0174175457],[0,2356,0.0205690891],[0,2357,0.0253899302],[0,2358,0.0253899302],[0,2359,0.0174175457],[0,2362,0.6807861169],[0,2364,0.0174175457],[0,2365,0.0163544142],[0,2382,0.0175612],[0,2386,0.0415676778],[0,2394,0.1642931786],[0,2397,0.0290922977],[0,2399,0.2824010091],[0,2401,0.5559968042],[0,2410,0.0255683741],[0,2420,0.6080359394],[0,2421,0.0429448097],[0,2423,0.3182868823],[0,2426,0.0683923728],[0,2428,0.0253899302],[0,2430,0.1768434548],[0,2432,0.1527433656],[0,2437,0.0325433464],[0,2440,0.6396680542],[0,2441,0.3421984338],[0,2445,0.0678498307],[0,2446,0.138243137],[0,2447,0.7109422265],[0,2450,0.3035969254],[0,2457,0.0230356203],[0,2460,0.0600193866],[0,2461,0.0255683741],[0,2464,0.0165904324],[0,2467,0.0253899302],[0,2471,0.0255683741],[0,2473,0.056184254],[0,2480,0.0530403604],[0,2485,0.0215377878],[0,2488,0.0174175457],[0,2490,0.0174175457],[0,2491,0.0255683741],[0,2492,0.0253899302],[0,2493,0.315897981],[0,2501,0.0174175457],[0,2503,0.0174175457],[0,2506,0.0255683741],[0,2507,0.0174175457],[0,2509,0.0253899302],[0,2510,0.0174175457],[0,2516,0.0174175457],[0,2520,0.0174175457],[0,2521,0.0255683741],[0,2523,0.0253899302],[0,2524,0.0372825485],[0,2531,0.0174175457],[0,2536,0.1935286976],[0,2537,0.0174175457],[0,2542,0.0344945349],[0,2543,0.0192914472],[0,2552,0.0176238618],[0,2554,0.0440276327],[0,2559,0.4639330865],[0,2561,0.0670409706],[0,2567,0.7417886971],[0,2569,0.1797129446],[0,2570,0.0174175457],[0,2572,0.86982717],[0,2574,0.2879796619],[0,2576,0.0180667052],[0,2579,0.0253899302],[0,2580,0.2930404203],[0,2581,0.0363021258],[0,2585,0.0253899302],[0,2586,0.4195629429],[0,2587,0.0329330603],[0,2588,0.0575179716],[0,2589,0.0176238618],[0,2592,0.4935727816],[0,2595,0.0255683741],[0,2596,0.0270496682],[0,2600,0.1390880747],[0,2605,0.0325174042],[0,2608,0.5830225854],[0,2612,0.2130030235],[0,2613,0.0253899302],[0,2614,0.3630389035],[0,2616,0.0253899302],[0,2620,0.0174175457],[0,2630,0.3424551092],[0,2632,0.0255683741],[0,2635,0.1655016662],[0,2636,0.3646142876],[0,2637,0.3895168045],[0,2638,0.3895168045],[0,2639,0.0379979142],[0,2640,0.0544159692],[0,2642,0.0253899302],[0,2643,0.0255683741],[0,2644,0.0393454858],[0,2645,0.0343057328],[0,2647,0.0174175457],[0,2652,0.1067787713],[0,2656,0.0174175457],[0,2658,0.0174175457],[0,2665,0.3846132737],[0,2669,0.0253899302],[0,2672,0.0192914472],[0,2676,0.3965834444],[0,2689,0.0611960941],[0,2690,0.028785137],[0,2691,0.0431489473],[0,2693,0.0440276327],[0,2704,0.0192941985],[0,2714,0.0373955851],[0,2715,0.0409988857],[0,2716,0.5363794419],[0,2724,0.040474002],[0,2726,0.0383314312],[0,2727,0.0253899302],[0,2732,0.0824325986],[0,2734,0.0253899302],[0,2737,0.2463615667],[0,2742,0.0640021825],[0,2752,0.0253899302],[0,2755,0.0534580115],[0,2756,0.3484700104],[0,2760,0.4241077989],[0,2765,0.3804929505],[0,2766,0.0255683741],[0,2768,0.0364709058],[0,2769,0.0255683741],[0,2770,0.036045083],[0,2771,0.0253899302],[0,2775,0.0253899302],[0,2777,0.0440276327],[0,2778,0.0440276327],[0,2784,0.0269525679],[0,2791,0.0255683741],[0,2794,0.0255683741],[0,2797,0.0344945349],[0,2801,0.0174175457],[0,2806,0.0174175457],[0,2807,0.0366426607],[0,2809,0.0220300279],[0,2810,0.035858065],[0,2818,0.242372386],[0,2819,0.036474212],[0,2820,0.0174175457],[0,2821,0.0253899302],[0,2827,0.0186889617],[0,2829,0.0357794858],[0,2830,0.3301992161],[0,2832,0.1945859019],[0,2833,0.0253899302],[0,2840,0.0272438098],[0,2843,0.3723766391],[0,2844,0.3978604028],[0,2845,0.0255683741],[0,2846,0.0320021873],[0,2847,0.038867618],[0,2848,0.0186889617],[0,2852,0.0269525679],[0,2857,0.3987661696],[0,2868,0.0372825485],[0,2869,0.0440276327],[0,2871,0.0255683741],[0,2873,0.0611710196],[0,2874,0.030912461],[0,2875,0.0440276327],[0,2880,0.0205690891],[0,2882,0.0174175457],[0,2885,0.0253899302],[0,2886,0.0255683741],[0,2890,0.0255683741],[0,2892,0.0174175457],[0,2894,0.0205690891],[0,2900,0.0253899302],[0,2905,0.0235865554],[0,2906,0.2301966218],[0,2910,0.3888165161],[0,2915,0.0253899302],[0,2917,0.0305205129],[0,2919,0.0253899302],[0,2921,0.0372781736],[0,2924,0.0459205039],[0,2926,0.5979301898],[0,2928,0.0445415685],[0,2935,0.2238854195],[0,2937,0.1618330859],[0,2939,0.0255683741],[0,2940,0.0253899302],[0,2941,0.0192941985],[0,2945,0.343279867],[0,2946,0.3648710662],[0,2952,0.03235765],[0,2953,0.0253899302],[0,2959,0.0174175457],[0,2960,0.0269525679],[0,2961,0.0219551365],[0,2966,0.0260524107],[0,2972,0.2685394856],[0,2976,0.0253899302],[0,2979,0.2183594922],[0,2982,0.2362310547],[0,2983,0.0802617586],[0,2988,0.0707438962],[0,2989,0.0253899302],[0,2990,0.1073486829],[0,2991,0.1292594374],[0,2992,0.0459616125],[0,2994,0.0286394822],[0,2995,0.080584259],[0,2996,0.2761246722],[0,2997,0.0328345347],[0,2999,0.0597773977],[0,3000,0.1001849064],[0,3002,0.0704994978],[0,3003,0.2706125024],[0,3007,0.025423045],[0,3009,0.0321075449],[0,3016,0.0255683741],[0,3023,0.0306957446],[0,3027,0.0560050854],[0,3034,0.0253899302],[0,3040,0.0253899302],[0,3050,0.0253899302],[0,3053,0.0403168301],[0,3055,0.0255683741],[0,3065,0.0174175457],[0,3069,0.0253899302],[0,3071,0.0253899302],[0,3072,0.0253899302],[0,3074,0.0174175457],[0,3075,0.0192941985],[0,3077,0.0255683741],[0,3079,0.0174175457],[0,3093,0.0418394238],[0,3095,0.0651454593],[0,3100,0.1683695742],[0,3101,0.7619525894],[0,3105,0.0298369409],[0,3109,0.0300786518],[0,3112,0.5177006787],[0,3114,0.0255683741],[0,3115,0.2096401103],[0,3116,0.1265136111],[0,3118,0.0286416739],[0,3119,0.0330793501],[0,3120,0.0807868114],[0,3122,0.038685115],[0,3125,0.7359636871],[0,3128,0.5032424081],[0,3130,0.0840348755],[0,3132,0.0486374997],[0,3134,0.0300986905],[0,3135,0.0463148164],[0,3136,0.0363290786],[0,3140,0.025631201],[0,3142,0.0334371816],[0,3143,0.0266422876],[0,3145,0.0255683741],[0,3146,0.0176238618],[0,3147,0.0276886003],[0,3148,0.0253899302],[0,3150,0.0253899302],[0,3151,0.0253899302],[0,3154,0.0253899302],[0,3155,0.4572126206],[0,3160,0.0499583438],[0,3161,0.0442871689],[0,3164,0.5771209366],[0,3166,0.0311221467],[0,3169,0.0292916016],[0,3173,0.0400707168],[0,3174,0.0640021825],[0,3178,0.795034915],[0,3179,0.5549563539],[0,3182,0.2249377781],[0,3184,0.0742365066],[0,3185,0.1180469239],[0,3187,0.0676574439],[0,3193,0.2941338382],[0,3194,0.0372547903],[0,3197,0.0585148799],[0,3198,0.7570080641],[0,3200,0.6007506012],[0,3201,0.3940561875],[0,3204,0.0594250572],[0,3206,0.0603423605],[0,3207,0.0447053241],[0,3211,0.0255683741],[0,3213,0.0894814408],[0,3219,0.0832922748],[0,3220,0.5328263557],[0,3221,0.0260456957],[0,3226,0.0678498307],[0,3227,0.0798300614],[0,3229,0.0338735257],[0,3230,0.0286049562],[0,3231,0.0632867372],[0,3232,0.1935105282],[0,3238,0.0220397255],[0,3240,0.0565662527],[0,3244,0.0349375732],[0,3246,0.5670466118],[0,3250,0.0956411173],[0,3251,0.1257089726],[0,3253,0.1687965737],[0,3254,0.0255683741],[0,3255,0.0253899302],[0,3256,0.1832908181],[0,3258,0.0270856017],[0,3260,0.0175612],[0,3261,0.035096175],[0,3262,0.0440497116],[0,3265,0.1991464877],[0,3267,0.0988830015],[0,3268,0.406277443],[0,3269,0.2006515163],[0,3271,0.0427237381],[0,3273,0.6060957014],[0,3277,0.1520079609],[0,3278,0.0468350885],[0,3279,0.1733949324],[0,3280,0.0255683741],[0,3282,0.0432504748],[0,3283,0.0678498307],[0,3284,0.1324170271],[0,3285,0.0928614193],[0,3286,0.0309944669],[0,3290,0.3123961362],[0,3292,0.0375764335],[0,3293,0.0326761959],[0,3294,0.2482592355],[0,3297,0.0995180738],[0,3299,0.2914820837],[0,3301,0.030983435],[0,3309,0.0253899302],[0,3310,0.2556516843],[0,3312,0.0542146663],[0,3313,0.0470832401],[0,3315,0.038685115],[0,3317,0.0574390296],[0,3318,0.0271909567],[0,3324,0.1051165404],[0,3327,0.2602851511],[0,3330,0.0253899302],[0,3331,0.0221367362],[0,3332,0.5161836649],[0,3339,0.4769984005],[0,3342,0.0255683741],[0,3343,0.0934330348],[0,3344,0.0255683741],[0,3348,0.2740178604],[0,3349,0.2291120933],[0,3350,0.0286085586],[0,3352,0.4256353428],[0,3354,0.050983301],[0,3356,0.3460589945],[0,3359,0.0445721582],[0,3361,0.0175612],[0,3362,0.0252446385],[0,3363,0.0678498307],[0,3367,0.031222325],[0,3369,0.1296638451],[0,3371,0.3359871489],[0,3373,0.0257011434],[0,3377,0.0257064907],[0,3379,0.0337216495],[0,3381,0.0253899302],[0,3384,0.0459863606],[0,3387,0.0174175457],[0,3391,0.0253899302],[0,3400,0.3775477166],[0,3401,0.2432692775],[0,3403,0.0253899302],[0,3404,0.0340288617],[0,3408,0.0167154925],[0,3409,0.0253899302],[0,3412,0.0253899302],[0,3414,0.0174175457],[0,3415,0.0253899302],[0,3416,0.0565662527],[0,3419,0.1169139352],[0,3420,0.3570101885],[0,3421,0.0511195838],[0,3424,0.0593581474],[0,3434,0.6062825357],[0,3436,0.0455527426],[0,3437,0.4587708643],[0,3440,0.0989705054],[0,3441,0.0324811571],[0,3442,0.0323790876],[0,3443,0.1262947838],[0,3444,0.053645724],[0,3445,0.2005105701],[0,3449,0.2139302453],[0,3450,0.0372825485],[0,3453,0.0455527426],[0,3454,0.019640039],[0,3456,0.0174175457],[0,3459,0.0678498307],[0,3467,0.8295486929],[0,3469,0.1387800672],[0,3470,0.2798216718],[0,3471,0.0174175457],[0,3472,0.1044582326],[0,3477,0.0253899302],[0,3478,0.0325174042],[0,3480,0.0293131626],[0,3482,0.052047016],[0,3483,0.0399086627],[0,3485,0.0640021825],[0,3487,0.0174175457],[0,3489,0.0598420637],[0,3492,0.034267335],[0,3493,0.0441456977],[0,3496,0.291649102],[0,3501,0.3643427531],[0,3502,0.2890386189],[0,3504,0.0291058798],[0,3505,0.6056716307],[0,3507,0.0406691489],[0,3508,0.0678498307],[0,3512,0.1342493631],[0,3513,0.1120587102],[0,3520,0.0257064907],[0,3521,0.0678498307],[0,3526,0.3565514673],[0,3529,0.1467718732],[0,3530,0.0253899302],[0,3531,0.0287490827],[0,3532,0.0530032334],[0,3535,0.3762301561],[0,3536,0.4686058344],[0,3537,0.0477827468],[0,3538,0.2782699479],[0,3542,0.0678498307],[0,3545,0.4164489399],[0,3548,0.0341548873],[0,3549,0.1964983653],[0,3553,0.0333461654],[0,3554,0.0292612901],[0,3555,0.4915807034],[0,3556,0.1806421319],[0,3557,0.293957146],[0,3558,0.0579275819],[0,3559,0.0174175457],[0,3560,0.3020664842],[0,3561,0.031623192],[0,3562,0.4380047282],[0,3571,0.1940069564],[0,3573,0.0565662527],[0,3575,0.3038741304],[0,3578,0.0253899302],[0,3579,0.0174175457],[0,3580,0.038290907],[0,3581,0.0358476103],[0,3582,0.0412897552],[0,3583,0.6215970674],[0,3586,0.4447861272],[0,3587,0.0255683741],[0,3588,0.0553296201],[0,3593,0.7727021691],[0,3597,0.1171411862],[0,3601,0.0407671344],[0,3608,0.0186889617],[0,3610,0.2096860406],[0,3612,0.0956411173],[0,3613,0.0534580115],[0,3616,0.0293131626],[0,3617,0.0678809229],[0,3618,0.0678498307],[0,3622,0.0894814408],[0,3627,0.0257064907],[0,3628,0.3605364948],[0,3629,0.0414349065],[0,3635,0.0253899302],[0,3637,0.043639388],[0,3638,0.4064514068],[0,3640,0.0475016217],[0,3644,0.0325174042],[0,3646,0.2483035775],[0,3652,0.1043256215],[0,3654,0.093769598],[0,3662,0.0442421883],[0,3665,0.0660706652],[0,3667,0.0294765911],[0,3668,0.3266227478],[0,3672,0.1851482444],[0,3676,0.3393788081],[0,3678,0.1919409647],[0,3679,0.0549537672],[0,3680,0.5174049044],[0,3682,0.0534580115],[0,3683,0.1088526975],[0,3684,0.788120788],[0,3685,0.1083346215],[0,3686,0.0352984963],[0,3688,0.054886959],[0,3689,0.1386296189],[0,3690,0.0253717167],[0,3691,0.4710224387],[0,3692,0.1047049979],[0,3695,0.0257064907],[0,3696,0.0337980324],[0,3697,0.0253899302],[0,3698,0.0253899302],[0,3699,0.034870413],[0,3703,0.0253899302],[0,3704,0.0211044277],[0,3705,0.0253899302],[0,3707,0.0315812566],[0,3708,0.0255683741],[0,3709,0.3250168309],[0,3710,0.0492182689],[0,3714,0.0921257303],[0,3717,0.3035068796],[0,3718,0.0192941985],[0,3719,0.0965384508],[0,3720,0.031623192],[0,3721,0.5846839645],[0,3724,0.4213216278],[0,3725,0.0349368359],[0,3726,0.0535343506],[0,3727,0.0261959073],[0,3729,0.0253899302],[0,3736,0.0325174042],[0,3741,0.4116699763],[0,3746,0.0565662527],[0,3747,0.0282117073],[0,3750,0.5957323349],[0,3753,0.511845038],[0,3754,0.2516320457],[0,3755,0.1043256215],[0,3761,0.1585770694],[0,3762,0.03777293],[0,3763,0.0253899302],[0,3764,0.2147854162],[0,3765,0.0492182689],[0,3766,0.0413121496],[0,3767,0.0397840217],[0,3770,0.0678498307],[0,3774,0.0552063944],[0,3775,0.0619853027],[0,3777,0.0373432852],[0,3780,0.0614690656],[0,3782,0.0219551365],[0,3785,0.0253899302],[0,3786,0.2410639164],[0,3791,0.0290922977],[0,3792,0.5246484081],[0,3793,0.0525825119],[0,3800,0.0440237993],[0,3801,0.0883998549],[0,3803,0.0937711916],[0,3806,0.4265289087],[0,3811,0.0174175457],[0,3812,0.1030879059],[0,3815,0.114654128],[0,3817,0.0174175457],[0,3819,0.0551903429],[0,3820,0.0205808783],[0,3822,0.0382208159],[0,3823,0.3761320799],[0,3825,0.2432692775],[0,3827,0.09360134],[0,3831,0.0892040624],[0,3832,0.1105429683],[0,3833,0.0977900696],[0,3835,0.0257064907],[0,3837,0.1043256215],[0,3840,0.0253899302],[0,3842,0.0253899302],[0,3843,0.1363935673],[0,3844,0.052478283],[0,3845,0.0471007821],[0,3847,0.0255683741],[0,3848,0.0298790848],[0,3849,0.4598487088],[0,3851,0.2452113869],[0,3852,0.4596860887],[0,3853,0.069931061],[0,3855,0.0355757835],[0,3859,0.2442648589],[0,3865,0.1009965019],[0,3866,0.0180525869],[0,3867,0.1022585856],[0,3869,0.1073486829],[0,3870,0.0701532175],[0,3876,0.1016676746],[0,3877,0.0174175457],[0,3879,0.0250276211],[0,3882,0.0505910578],[0,3883,0.1326198065],[0,3884,0.5346871965],[0,3885,0.0623805715],[0,3886,0.0253899302],[0,3887,0.0570728693],[0,3888,0.2898882787],[0,3891,0.0628677605],[0,3894,0.0279878392],[0,3895,0.0334059019],[0,3896,0.0188220079],[0,3897,0.5863819308],[0,3898,0.1734411871],[0,3899,0.0226811071],[0,3900,0.0192941985],[0,3901,0.0604778527],[0,3902,0.0251845031],[0,3906,0.1183114701],[0,3908,0.4569929282],[0,3910,0.0253899302],[0,3913,0.0255683741],[0,3916,0.0265305465],[0,3919,0.0951237632],[0,3920,0.0205690891],[0,3923,0.0404681965],[0,3925,0.6055160917],[0,3930,0.033241924],[0,3932,0.3273419649],[0,3935,0.0305874763],[0,3936,0.4123842841],[0,3938,0.0617166056],[0,3939,0.0253899302],[0,3943,0.0435669285],[0,3945,0.2217739778],[0,3946,0.2112406624],[0,3947,0.1073486829],[0,3948,0.0174175457],[0,3949,0.1181965165],[0,3955,0.0255683741],[0,3956,0.0253899302],[0,3958,0.0680939072],[0,3962,0.0253899302],[0,3963,0.0562980088],[0,3965,0.1062306886],[0,3970,0.2470094779],[0,3971,0.038685115],[0,3972,0.6013383817],[0,3974,0.0207881448],[0,3976,0.766818426],[0,3977,0.0573735168],[0,3981,0.0418394238],[0,3983,0.0309547495],[0,3984,0.0559175665],[0,3986,0.0564226111],[0,3988,0.4397673509],[0,3990,0.082659607],[0,3992,0.0512873107],[0,3993,0.1140155831],[0,3998,0.0255683741],[0,4006,0.2676090883],[0,4007,0.8414994685],[0,4011,0.0951237632],[0,4016,0.0586867776],[0,4017,0.0219551365],[0,4018,0.1509715639],[0,4025,0.6290799409],[0,4026,0.0600193866],[0,4027,0.5254564711],[0,4031,0.0334059019],[0,4032,0.3178894324],[0,4034,0.523262763],[0,4035,0.0284368129],[0,4036,0.0484273934],[0,4037,0.1945859019],[0,4038,0.0205690891],[0,4039,0.066796808],[0,4041,0.0250276211],[0,4042,0.0257064907],[0,4044,0.0412897552],[0,4047,0.0275721867],[0,4048,0.0174175457],[0,4049,0.0394770303],[0,4055,0.0860256523],[0,4057,0.3731484536],[0,4059,0.2904968841],[0,4063,0.0950172546],[0,4068,0.156741025],[0,4069,0.1043256215],[0,4070,0.4139149105],[0,4071,0.0406020406],[0,4072,0.0362048871],[0,4077,0.0325174042],[0,4079,0.1184547617],[0,4080,0.0677311833],[0,4083,0.0255683741],[0,4084,0.0779653189],[0,4087,0.0174175457],[0,4088,0.0221146261],[0,4093,0.2481821725],[0,4094,0.0192941985],[0,4096,0.3800878123],[0,4097,0.1809614511],[0,4102,0.4405179651],[0,4104,0.0484273934],[0,4105,0.081684946],[0,4106,0.0257064907],[0,4107,0.0565662527],[0,4108,0.1073486829],[0,4110,0.0288453154],[0,4111,0.3518410385],[0,4112,0.1114335344],[0,4113,0.1065961337],[0,4114,0.059808829],[0,4116,0.02863077],[0,4117,0.2480779522],[0,4121,0.0328345347],[0,4122,0.0295059917],[0,4123,0.084932547],[0,4124,0.0255683741],[0,4125,0.4384223444],[0,4130,0.0174175457],[0,4131,0.0683049315],[0,4135,0.1135638389],[0,4136,0.0356357215],[0,4142,0.0578453062],[0,4145,0.1002720368],[0,4147,0.6338490073],[0,4150,0.253151908],[0,4151,0.0678498307],[0,4154,0.0255683741],[0,4158,0.0535600001],[0,4159,0.9076106599],[0,4160,0.0253899302],[0,4162,0.454764955],[0,4165,0.0843702048],[0,4166,0.11457883],[0,4167,0.0632181204],[0,4170,0.0253899302],[0,4172,0.0459863606],[0,4174,0.0894814408],[0,4178,0.0255683741],[0,4179,0.0174175457],[0,4182,0.0349802086],[0,4183,0.31429549],[0,4185,0.6021193518],[0,4187,0.0678498307],[0,4190,0.5333085107],[0,4191,0.0363290786],[0,4195,0.0574390296],[0,4197,0.0286049562],[0,4198,0.1196274432],[0,4199,0.061933165],[0,4200,0.0255683741],[0,4201,0.0506358745],[0,4202,0.0253899302],[0,4207,0.0477455353],[0,4208,0.0292833936],[0,4211,0.053645724],[0,4214,0.0337980324],[0,4216,0.055630592],[0,4217,0.0399086627],[0,4221,0.3269567121],[0,4224,0.4295866166],[0,4227,0.0316042481],[0,4228,0.0586739175],[0,4229,0.0565662527],[0,4238,0.0565662527],[0,4239,0.5847397835],[0,4241,0.3254138184],[0,4245,0.0452209079],[0,4246,0.0255683741],[0,4247,0.0504286726],[0,4248,0.4219064221],[0,4249,0.1073486829],[0,4251,0.3552132486],[0,4257,0.0227910451],[0,4258,0.0253899302],[0,4260,0.0946147312],[0,4262,0.0328345347],[0,4268,0.054000223],[0,4272,0.1249664352],[0,4273,0.0219551365],[0,4278,0.3559704264],[0,4282,0.3457118242],[0,4284,0.6055160917],[0,4285,0.0484273934],[0,4288,0.0248331822],[0,4293,0.3205276645],[0,4295,0.0272438098],[0,4297,0.0399508372],[0,4299,0.0451977264],[0,4302,0.0174175457],[0,4304,0.0859170172],[0,4306,0.0207881448],[0,4308,0.2145681841],[0,4312,0.1000775123],[0,4314,0.1238039008],[0,4316,0.0388521596],[0,4319,0.0230770864],[0,4320,0.0377548988],[0,4321,0.0463148164],[0,4324,0.175840163],[0,4327,0.0293041088],[0,4330,0.0600411818],[0,4331,0.4637164082],[0,4334,0.0188814635],[0,4336,0.5041692762],[0,4338,0.0676698941],[0,4339,0.0565662527],[0,4340,0.0352402623],[0,4344,0.3544129542],[0,4345,0.0336863708],[0,4346,0.2057269792],[0,4350,0.388334268],[0,4355,0.0598420637],[0,4357,0.1458770344],[0,4358,0.0173425809],[0,4360,0.0302254485],[0,4363,0.0532410472],[0,4373,0.3192799485],[0,4376,0.0817368271],[0,4378,0.1338768281],[0,4379,0.0580027959],[0,4380,0.3062646013],[0,4383,0.0253899302],[0,4385,0.0448394139],[0,4388,0.112502989],[0,4391,0.1810978492],[0,4392,0.0253899302],[0,4393,0.0475092095],[0,4395,0.0377670782],[0,4397,0.0220354526],[0,4398,0.0474239159],[0,4399,0.2353820983],[0,4402,0.0326370135],[0,4405,0.0920074402],[0,4408,0.0678498307],[0,4410,0.0275807808],[0,4418,0.0255683741],[0,4420,0.2652727466],[0,4422,0.4056578703],[0,4423,0.0300570643],[0,4424,0.1046583388],[0,4427,0.0565662527],[0,4428,0.2541147538],[0,4433,0.0523384428],[0,4435,0.5201314356],[0,4436,0.053645724],[0,4437,0.3438777023],[0,4441,0.0468350885],[0,4446,0.0365703678],[0,4448,0.9303425416],[0,4449,0.0244163666],[0,4450,0.8155043712],[0,4454,0.5911140292],[0,4455,0.0174175457],[0,4456,0.3038745447],[0,4465,0.2854779466],[0,4469,0.0792276985],[0,4470,0.0455629335],[0,4474,0.0506358745],[0,4483,0.0341453238],[0,4484,0.4153891434],[0,4487,0.0657213714],[0,4488,0.0678498307],[0,4490,0.0463148164],[0,4491,0.0174175457],[0,4492,0.0253899302],[0,4493,0.0336949182],[0,4496,0.0253899302],[0,4497,0.0253899302],[0,4504,0.0412897552],[0,4507,0.041812042],[0,4511,0.4166706399],[0,4515,0.0255683741],[0,4519,0.0253899302],[0,4520,0.0363120112],[0,4521,0.1418235095],[0,4525,0.3411398245],[0,4533,0.0174175457],[0,4536,0.0942194653],[0,4537,0.0175612],[0,4538,0.0255683741],[0,4540,0.0253899302],[0,4541,0.0253899302],[0,4542,0.0219551365],[0,4544,0.0499583438],[0,4545,0.053645724],[0,4547,0.0192941985],[0,4548,0.0565662527],[0,4550,0.5074903602],[0,4551,0.0175487576],[0,4555,0.0302080917],[0,4556,0.0920074402],[0,4557,0.1073486829],[0,4559,0.0253899302],[0,4561,0.0174175457],[0,4563,0.0798056215],[0,4564,0.4139149105],[0,4567,0.7726427441],[0,4569,0.0534580115],[0,4572,0.0640411395],[0,4574,0.0269015264],[0,4577,0.0256897913],[0,4579,0.3921718049],[0,4581,0.1119352681],[0,4583,0.872342007],[0,4584,0.418908177],[0,4587,0.0253899302],[0,4590,0.0415655017],[0,4591,0.0219551365],[0,4594,0.0255683741],[0,4596,0.0921257303],[0,4602,0.4307126235],[0,4603,0.0328090027],[0,4604,0.0328076383],[0,4607,0.0294785511],[0,4610,0.1062053924],[0,4612,0.0673192248],[0,4613,0.0253899302],[0,4614,0.3891558051],[0,4616,0.0640021825],[0,4619,0.0335955584],[0,4621,0.0253899302],[0,4622,0.4064161403],[0,4625,0.0738648943],[0,4630,0.6152545362],[0,4631,0.0253899302],[0,4635,0.0406018232],[0,4636,0.1097368929],[0,4640,0.3208069944],[0,4645,0.0345198661],[0,4647,0.0237469398],[0,4652,0.02863077],[0,4655,0.5171586956],[0,4657,0.033653848],[0,4661,0.4252542492],[0,4662,0.5085191285],[0,4665,0.2398347723],[0,4667,0.0837372353],[0,4668,0.0956411173],[0,4670,0.0640021825],[0,4674,0.1018266388],[0,4682,0.0346330637],[0,4686,0.0655144985],[0,4690,0.0280268923],[0,4693,0.0755997747],[0,4695,0.1073486829],[0,4696,0.1672485551],[0,4702,0.0255683741],[0,4703,0.0253899302],[0,4704,0.0540725093],[0,4710,0.1703764328],[0,4711,0.0451977264],[0,4712,0.0401214697],[0,4713,0.0986734443],[0,4715,0.0539875168],[0,4716,0.0257692357],[0,4718,0.2689387613],[0,4719,0.0419812266],[0,4722,0.0276149983],[0,4723,0.0364870061],[0,4724,0.0843526112],[0,4725,0.0312948345],[0,4768,0.0174175457],[0,4847,0.0289943067],[0,4850,0.0253899302],[0,4853,0.0336446779],[0,4862,0.0253899302],[0,4864,0.0174175457],[0,4865,0.0253899302],[0,4866,0.0357794858],[0,4868,0.0344285441],[0,4871,0.0334038621],[0,4873,0.0818945307],[0,4876,0.0174175457],[0,5088,0.6123682441],[0,5244,0.0782327389],[0,5414,0.0176935169],[0,5417,0.0158212268],[0,5430,0.3457248157],[0,5439,0.0237750782],[0,5451,0.0237750782],[0,5452,0.0237750782],[0,5455,0.0263318522],[0,5458,0.3116864496],[0,5479,0.0237750782],[0,5481,0.0237750782],[0,5493,0.2102531496],[0,5494,0.0170896872],[0,5506,0.0237750782],[0,5524,0.2597655932],[0,5525,0.0237750782],[0,5530,0.0158212268],[0,5533,0.2258268369],[0,5536,0.0216504932],[0,5540,0.0158212268],[0,5549,0.0189654439],[0,5551,0.0237750782],[0,5553,0.0176935169],[0,6482,0.0158212268],[0,6484,0.4337556338],[0,6485,0.0189654439],[0,6488,0.0176935169],[0,6490,0.0489623342],[0,6492,0.0158212268],[0,6495,0.0158212268],[0,6496,0.0253340833],[0,6505,0.0237750782],[0,6509,0.0237750782],[0,6512,0.0423694543],[0,6513,0.0170896872],[0,6558,0.6707150035],[0,6561,0.0170896872],[0,6564,0.0176935169],[0,6566,0.0237750782],[0,6572,0.3928073026],[0,6574,0.0239531072],[0,6581,0.0176935169],[0,6582,0.0311769039],[0,6586,0.0205294468],[0,6588,0.1999920349],[0,6589,0.0239531072],[0,6592,0.0158212268],[0,6595,0.0341404815],[0,6597,0.0160270632],[0,6599,0.0312023765],[0,6603,0.0237750782],[0,6604,0.0313500375],[0,6607,0.324324862],[0,6608,0.0176935169],[0,6610,0.0158212268],[0,6612,0.0171860033],[0,6613,0.0326701545],[0,6614,0.0159645471],[0,6616,0.0160270632],[0,6618,0.0239531072],[0,6622,0.0307125082],[0,6624,0.0307739879],[0,6626,0.0423694543],[0,6629,0.0237750782],[0,6634,0.1575991851],[0,6635,0.4108680983],[0,6638,0.0160270632],[0,6640,0.0343798749],[0,6648,0.2295813055],[0,6650,0.1665540256],[0,6666,0.0341371952],[0,6673,0.0174880167],[0,6678,0.0811143145],[0,6684,0.0237750782],[0,6687,0.1735811349],[0,6691,0.0237750782],[0,6693,0.2912113976],[0,6699,0.0170896872],[0,6700,0.1902644324],[0,6703,0.0239531072],[0,6704,0.0320218076],[0,6705,0.0237750782],[0,6706,0.1612649761],[0,6708,0.0313756833],[0,6719,0.3328657063],[0,6722,0.0237750782],[0,6723,0.0262244626],[0,6724,0.0216329797],[0,6728,0.0158212268],[0,6729,0.0171860033],[0,6732,0.0268148779],[0,6735,0.0237750782],[0,6739,0.0237750782],[0,6741,0.0239531072],[0,6744,0.0189654439],[0,6752,0.0284230474],[0,6757,0.0340203402],[0,6758,0.0661362737],[0,6773,0.0158212268],[0,6775,0.0254668078],[0,6777,0.0158212268],[0,6782,0.0355173098],[0,6784,0.0288937341],[0,6786,0.0237750782],[0,6788,0.0239531072],[0,6790,0.01547381],[0,6794,0.0228799088],[0,6797,0.0254668078],[0,6798,0.0239531072],[0,6800,0.0361226078],[0,6810,0.0158212268],[0,6817,0.0221359071],[0,6818,0.0334420489],[0,6823,0.207656422],[0,6824,0.3457248157],[0,6825,0.0158212268],[0,6826,0.2835339246],[0,6827,0.0176935169],[0,6828,0.0233646799],[0,6835,0.0836623351],[0,6837,0.0237750782],[0,6838,0.1133739233],[0,6840,0.0318179128],[0,6844,0.2881296752],[0,6845,0.0292698013],[0,6846,0.1764348582],[0,6848,0.0378359006],[0,6855,0.0365914687],[0,6856,0.0236447309],[0,6861,0.0174880167],[0,6862,0.0237750782],[0,6866,0.0204326616],[0,6869,0.0239531072],[0,6871,0.0189654439],[0,6873,0.0262244626],[0,6877,0.0320607599],[0,6881,0.0254309578],[0,6885,0.0221359071],[0,6887,0.0237750782],[0,6888,0.0158212268],[0,6892,0.0273710758],[0,6895,0.0237750782],[0,6896,0.0203482692],[0,6906,0.1911367624],[0,6911,0.0239531072],[0,6915,0.0189654439],[0,6918,0.0349137105],[0,6921,0.0391768691],[0,6922,0.3597875549],[0,6924,0.0237750782],[0,6926,0.033894209],[0,6928,0.0296268753],[0,6929,0.0158212268],[0,6930,0.363238274],[0,6933,0.18429775],[0,6934,0.0239531072],[0,6940,0.0158212268],[0,6947,0.0206959471],[0,6950,0.0517779107],[0,6951,0.0435372109],[0,6952,0.09344173],[0,6953,0.1526116354],[0,6955,0.2884769777],[0,6956,0.090237725],[0,6959,0.0423694543],[0,6964,0.0423694543],[0,6965,0.1204153338],[0,6969,0.0173171709],[0,6971,0.3264477039],[0,6974,0.0356400501],[0,6975,0.0237750782],[0,6977,0.0239531072],[0,6979,0.0318179128],[0,6980,0.056646635],[0,6985,0.0308859832],[0,6991,0.0158212268],[0,6997,0.0304770767],[0,7000,0.0237750782],[0,7009,0.0260207582],[0,7012,0.0237750782],[0,7018,0.0290685583],[0,7019,0.0158212268],[0,7021,0.0159645471],[0,7023,0.0328585178],[0,7025,0.0237750782],[0,7027,0.0333430624],[0,7031,0.0239531072],[0,7036,0.0416915631],[0,7039,0.0239531072],[0,7040,0.0158212268],[0,7043,0.0276087026],[0,7046,0.0158212268],[0,7048,0.0237750782],[0,7054,0.1882136716],[0,7056,0.0254309578],[0,7058,0.0254668078],[0,7059,0.0332386569],[0,7062,0.0173171709],[0,7063,0.317002274],[0,7066,0.0159645471],[0,7071,0.4063219749],[0,7074,0.031204676],[0,7076,0.0158212268],[0,7081,0.0423694543],[0,7084,0.0402196741],[0,7087,0.062491006],[0,7092,0.0251157812],[0,7093,0.0423694543],[0,7095,0.1911367624],[0,7100,0.0158212268],[0,7102,0.0338433683],[0,7108,0.0176935169],[0,7110,0.0276891904],[0,7113,0.3928785315],[0,7114,0.0237750782],[0,7117,0.0423694543],[0,7118,0.0158212268],[0,7120,0.3852330006],[0,7122,0.0237750782],[0,7124,0.1481203288],[0,7126,0.0300840548],[0,7128,0.1734439357],[0,7129,0.0239531072],[0,7132,0.0239531072],[0,7133,0.2053599234],[0,7139,0.0290685583],[0,7142,0.0234213221],[0,7143,0.0311755426],[0,7145,0.3470346243],[0,7146,0.0259636678],[0,7160,0.0237750782],[0,7161,0.3073451309],[0,7162,0.0158212268],[0,7164,0.0270948408],[0,7167,0.1306551174],[0,7168,0.089658803],[0,7173,0.0158212268],[0,7174,0.341277306],[0,7175,0.0158212268],[0,7181,0.0309118651],[0,7182,0.0312023765],[0,7186,0.0423694543],[0,7189,0.0239531072],[0,7190,0.0489623342],[0,7192,0.4047604198],[0,7195,0.2074537481],[0,7201,0.0260207582],[0,7206,0.0158212268],[0,7208,0.0239531072],[0,7210,0.0921435814],[0,7213,0.26461567],[0,7214,0.0443324272],[0,7220,0.2930346904],[0,7221,0.0290685583],[0,7222,0.3343761764],[0,7225,0.0423694543],[0,7226,0.0158212268],[0,7232,0.3173654806],[0,7234,0.0237750782],[0,7236,0.0690763491],[0,7237,0.0423694543],[0,7240,0.0158212268],[0,7244,0.0192333094],[0,7245,0.0341404815],[0,7246,0.0473703825],[0,7247,0.033200988],[0,7249,0.0237750782],[0,7251,0.0268148779],[0,7254,0.0237750782],[0,7255,0.0239531072],[0,7257,0.0309118651],[0,7259,0.0237750782],[0,7262,0.1787859303],[0,7264,0.0318833087],[0,7269,0.0158212268],[0,7270,0.0237750782],[0,7283,0.029350615],[0,7292,0.0327462069],[0,7296,0.0237750782],[0,7298,0.3211140138],[0,7301,0.0158212268],[0,7304,0.1859559861],[0,7305,0.0158212268],[0,7306,0.0268148779],[0,7307,0.0475903679],[0,7319,0.0203482692],[0,7320,0.0369449841],[0,7322,0.0581216892],[0,7325,0.0246606998],[0,7335,0.0239531072],[0,7336,0.0237750782],[0,7341,0.0239531072],[0,7342,0.0792349606],[0,7346,0.0230996983],[0,7348,0.0323185278],[0,7349,0.0239531072],[0,7353,0.0327824533],[0,7357,0.0347113518],[0,7365,0.2363143263],[0,7366,0.0237750782],[0,7373,0.0253340833],[0,7374,0.0423694543],[0,7376,0.0320607599],[0,7377,0.0423694543],[0,7380,0.0158212268],[0,7381,0.3901550878],[0,7384,0.0239531072],[0,7386,0.0176935169],[0,7389,0.0288830586],[0,7390,0.0221429996],[0,7397,0.0423694543],[0,7404,0.0237750782],[0,7411,0.1805898118],[0,7415,0.0159645471],[0,7417,0.0390139316],[0,7418,0.0237750782],[0,7422,0.0423694543],[0,7427,0.0225620949],[0,7430,0.3549906438],[0,7431,0.0775970157],[0,7433,0.0237750782],[0,7434,0.0174880167],[0,7441,0.0237750782],[0,7448,0.0176935169],[0,7449,0.0341404815],[0,7452,0.0449266116],[0,7454,0.0816710907],[0,7455,0.0341404815],[0,7456,0.0286673602],[0,7457,0.0189654439],[0,7464,0.2982057928],[0,7465,0.0346521599],[0,7475,0.0237750782],[0,7483,0.1341332651],[0,7486,0.0189654439],[0,7487,0.0263080308],[0,7494,0.0288937341],[0,7497,0.0229602845],[0,7498,0.0401963177],[0,7500,0.0282050403],[0,7503,0.0140744918],[0,7504,0.0216329797],[0,7505,0.1514118388],[0,7511,0.0341404815],[0,7512,0.0158212268],[0,7516,0.0158212268],[0,7523,0.3660586658],[0,7525,0.0158212268],[0,7527,0.0363225692],[0,7528,0.0237750782],[0,7536,0.0255062258],[0,7537,0.1684383048],[0,7544,0.0237750782],[0,7546,0.0237750782],[0,7547,0.2451140858],[0,7548,0.0260177705],[0,7549,0.0237750782],[0,7551,0.0266324951],[0,7553,0.0290685583],[0,7556,0.0311330734],[0,7565,0.0341362247],[0,7568,0.0699402946],[0,7571,0.0158212268],[0,7575,0.0372708862],[0,7577,0.0239531072],[0,7578,0.1154179576],[0,7582,0.0176935169],[0,7586,0.0239531072],[0,7587,0.12180036],[0,7590,0.0320607599],[0,7592,0.0158212268],[0,7596,0.0320607599],[0,7598,0.0423694543],[0,7599,0.0423694543],[0,7601,0.0286075575],[0,7605,0.3870966712],[0,7609,0.1873560863],[0,7610,0.0308859832],[0,7611,0.0569945217],[0,7612,0.2194154453],[0,7615,0.0351659867],[0,7621,0.0234136113],[0,7628,0.0592634033],[0,7635,0.0320607599],[0,7638,0.0158212268],[0,7639,0.0438372574],[0,7642,0.0176935169],[0,7645,0.0237750782],[0,7646,0.0398495818],[0,7647,0.0158212268],[0,7648,0.0632055226],[0,7650,0.0554093409],[0,7657,0.2017344582],[0,7660,0.0775944606],[0,7661,0.0237750782],[0,7665,0.0423694543],[0,7669,0.0278815712],[0,7670,0.0343798749],[0,7671,0.0275923063],[0,7672,0.1343946543],[0,7676,0.0237750782],[0,7677,0.0320607599],[0,7679,0.0732219084],[0,7681,0.0423694543],[0,7682,0.0423694543],[0,7685,0.0323113613],[0,7690,0.0260207582],[0,7694,0.1873560863],[0,7697,0.3528579048],[0,7699,0.0239531072],[0,7701,0.0311552695],[0,7706,0.0158212268],[0,7707,0.0159645471],[0,7710,0.0355004463],[0,7711,0.2237039264],[0,7715,0.026890774],[0,7716,0.1044609989],[0,7720,0.0158212268],[0,7722,0.0158212268],[0,7724,0.0216715736],[0,7728,0.0239531072],[0,7732,0.0237750782],[0,7733,0.0237750782],[0,7735,0.0158212268],[0,7736,0.3180332513],[0,7737,0.0158212268],[0,7738,0.1283989746],[0,7739,0.0299220653],[0,7741,0.2990867945],[0,7743,0.0282027497],[0,7748,0.0242720938],[0,7749,0.0158212268],[0,7750,0.0176935169],[0,7751,0.0311330734],[0,7753,0.1872287825],[0,7757,0.0237750782],[0,7761,0.0159645471],[0,7764,0.0297697974],[0,7766,0.0526590653],[0,7770,0.1671806033],[0,7771,0.0365959697],[0,7772,0.0399495736],[0,7777,0.027446554],[0,7779,0.0239531072],[0,7782,0.0423694543],[0,7786,0.323652836],[0,7787,0.0264351569],[0,7788,0.1829510152],[0,7789,0.323652836],[0,7793,0.0174880167],[0,7794,0.0158212268],[0,7799,0.0158212268],[0,7800,0.0273710758],[0,7801,0.2253253667],[0,7805,0.3991148434],[0,7808,0.0237750782],[0,7811,0.0318179128],[0,7812,0.0239531072],[0,7814,0.0221359071],[0,7815,0.0342188781],[0,7818,0.0268148779],[0,7825,0.0204283986],[0,7833,0.0176935169],[0,7834,0.206370434],[0,7837,0.0237750782],[0,7842,0.1734439357],[0,7846,0.0158212268],[0,7847,0.2762528408],[0,7856,0.0158212268],[0,7858,0.3308336731],[0,7859,0.0239531072],[0,7861,0.1193197838],[0,7864,0.0158212268],[0,7868,0.0436547713],[0,7870,0.3144128902],[0,7875,0.0368999239],[0,7880,0.1177935208],[0,7882,0.0273710758],[0,7885,0.0158212268],[0,7887,0.0176935169],[0,7890,0.0604292151],[0,7894,0.0171648095],[0,7898,0.323652836],[0,7899,0.1661797366],[0,7901,0.0273710758],[0,7904,0.340593814],[0,7906,0.0792815088],[0,7909,0.1665540256],[0,7916,0.3852330006],[0,7919,0.3604312381],[0,7924,0.0239531072],[0,7925,0.0158212268],[0,7927,0.0239531072],[0,7932,0.044651321],[0,7935,0.0158212268],[0,7939,0.0319819901],[0,7940,0.0256246481],[0,7946,0.3132304573],[0,7950,0.0244360186],[0,7959,0.0356400501],[0,7960,0.0159645471],[0,7967,0.2345623677],[0,7982,0.0237750782],[0,7993,0.3860220221],[0,7995,0.0236301243],[0,8023,0.0171860033],[0,8121,0.0140744918],[0,8173,0.4584284796],[0,8181,0.4815717551],[0,8182,0.0176935169],[0,8211,0.2678546788],[0,8249,0.1847285086],[0,8251,0.0429724461],[0,8252,0.1638512684],[0,8266,0.1346391034],[0,8289,0.5470240387],[0,8312,0.0189654439],[0,8317,0.0237750782],[0,8325,0.5609658642],[0,8331,0.031373473],[0,8332,0.1215379978],[0,8353,0.0237750782],[0,8359,0.6664861246],[0,8366,0.1055121745],[0,8367,0.0170896872],[0,8371,0.1414056307],[0,8375,0.2260096656],[0,8378,0.0605372791],[0,8380,0.2102761225],[0,8383,0.4190109872],[0,8384,0.5722522925],[0,8389,0.5064830941],[0,8390,0.7428770712],[0,8396,0.0237750782],[0,8397,0.6813882404],[0,8398,0.0602675026],[0,8400,0.5980684363],[0,8401,0.7402480826],[0,8402,0.0453370579],[0,8403,0.3540607512],[0,8408,0.7272943802],[0,8409,0.0543345426],[0,8413,0.1003927275],[0,8414,0.7810102865],[0,8416,0.2418783614],[0,8417,0.0237750782],[0,8418,0.0158212268],[0,8420,0.5686553261],[0,8428,0.355182434],[0,8429,0.155830464],[0,8431,0.2190353059],[0,8432,0.3124564556],[0,8434,0.1043432204],[0,8435,0.2372097723],[0,8436,0.1436031697],[0,8439,0.0691256262],[0,8442,0.1216930839],[0,8443,0.0237750782],[0,8446,0.1299632999],[0,8447,0.3479789843],[0,8448,0.5417571386],[0,8451,0.0158212268],[0,8452,0.0286607021],[0,8453,0.1389526124],[0,8456,0.048069187],[0,8458,0.0766426389],[0,8459,0.0239531072],[0,8460,0.040824143],[0,8463,0.0584422602],[0,8465,0.0928686456],[0,8467,0.504715075],[0,8469,0.2599216123],[0,8470,0.4406380859],[0,8473,0.1208719301],[0,8474,0.4884535301],[0,8475,0.7348864098],[0,8476,0.6990077873],[0,8479,0.0237750782],[0,8481,0.0571403486],[0,8482,0.3987745431],[0,8484,0.0675016491],[0,8485,0.6973404951],[0,8487,0.5661451822],[0,8488,0.6182393133],[0,8489,0.1932341334],[0,8490,0.6066376504],[0,8491,0.0239531072],[0,8492,0.098963841],[0,8493,0.0151041302],[0,8494,0.4891692141],[0,8495,0.0158212268],[0,8499,0.0582156866],[0,8504,0.1660100553],[0,8505,0.0158212268],[0,8506,0.042342538],[0,8507,0.0210903473],[0,8508,0.1894447244],[0,8513,0.0413957008],[0,8514,0.0156155687],[0,8517,0.1953058353],[0,8520,0.0158212268],[0,8522,0.1399162488],[0,8524,0.1182978202],[0,8526,0.0504824212],[0,8527,0.1924748404],[0,8528,0.2334627808],[0,8529,0.1038926798],[0,8530,0.0165631161],[0,8531,0.0239531072],[0,8532,0.6903255044],[0,8534,0.082983985],[0,8536,0.6779358848],[0,8537,0.0237750782],[0,8539,0.0464710432],[0,8542,0.1283250964],[0,8543,0.0239531072],[0,8545,0.0237750782],[0,8548,0.1266255137],[0,8549,0.0770893125],[0,8551,0.1108985485],[0,8558,0.1282753608],[0,8559,0.0237750782],[0,8560,0.3377075208],[0,8561,0.016165323],[0,8564,0.0636282835],[0,8566,0.0399432835],[0,8568,0.0928463579],[0,8570,0.0195741634],[0,8575,0.0251667637],[0,8577,0.477736453],[0,8579,0.1005545291],[0,8581,0.5749048372],[0,8583,0.0170896872],[0,8584,0.1131919742],[0,8585,0.0204326616],[0,8586,0.1499666745],[0,8590,0.1194302045],[0,8594,0.0158212268],[0,8597,0.0493718841],[0,8599,0.0260372571],[0,8603,0.0237750782],[0,8604,0.0237750782],[0,8605,0.2859896208],[0,8606,0.0180385536],[0,8607,0.0526721959],[0,8608,0.4681392013],[0,8610,0.0237750782],[0,8614,0.0341871539],[0,8616,0.4950347835],[0,8619,0.2341602502],[0,8621,0.3908241634],[0,8622,0.112268223],[0,8623,0.0623787137],[0,8624,0.1693433894],[0,8631,0.5071653818],[0,8633,0.0158212268],[0,8638,0.5095556894],[0,8639,0.099544366],[0,8643,0.0158212268],[0,8644,0.0237750782],[0,8646,0.0237750782],[0,8647,0.0290685583],[0,8649,0.573222391],[0,8651,0.0719657322],[0,8653,0.0256531364],[0,8654,0.1316555892],[0,8658,0.0294132813],[0,8661,0.150501931],[0,8663,0.0189654439],[0,8665,0.330123331],[0,8666,0.0360424716],[0,8667,0.0237750782],[0,8668,0.3638403069],[0,8671,0.0158212268],[0,8672,0.0370904886],[0,8673,0.0158212268],[0,8674,0.2510612289],[0,8675,0.084549959],[0,8678,0.0237750782],[0,8680,0.4210096477],[0,8681,0.0158212268],[0,8683,0.0237750782],[0,8686,0.0397608071],[0,8688,0.0661362737],[0,8689,0.1453774896],[0,8690,0.0671937906],[0,8691,0.0622124177],[0,8692,0.0239531072],[0,8694,0.0244293192],[0,8699,0.0237750782],[0,8700,0.0432923066],[0,8702,0.0711036315],[0,8704,0.1731171032],[0,8705,0.3361483916],[0,8706,0.0237750782],[0,8707,0.0276910988],[0,8709,0.0237750782],[0,8714,0.6995109034],[0,8718,0.0932674148],[0,8719,0.0827298581],[0,8720,0.0287508315],[0,8721,0.8386500925],[0,8723,0.0500005275],[0,8724,0.4234365951],[0,8725,0.086794449],[0,8731,0.0353218142],[0,8732,0.0262198207],[0,8733,0.1845731267],[0,8735,0.1709490666],[0,8739,0.0158212268],[0,8740,0.5686553261],[0,8742,0.0475290517],[0,8743,0.5050751741],[0,8744,0.0587223172],[0,8747,0.0290685583],[0,8749,0.0744616853],[0,8751,0.08136897],[0,8753,0.2009373908],[0,8755,0.2282835587],[0,8756,0.0158212268],[0,8757,0.063381343],[0,8758,0.0237750782],[0,8759,0.0237750782],[0,8760,0.0683519209],[0,8762,0.5190136192],[0,8763,0.0544863245],[0,8765,0.0237750782],[0,8766,0.0343560708],[0,8767,0.0467663336],[0,8770,0.0378450833],[0,8771,0.0158212268],[0,8773,0.0745138081],[0,8774,0.0158212268],[0,8775,0.0420009055],[0,8776,0.1850310585],[0,8777,0.0237750782],[0,8779,0.016165323],[0,8781,0.1625185427],[0,8782,0.0236301243],[0,8783,0.0357828222],[0,8784,0.0340556389],[0,8790,0.1054093612],[0,8794,0.019158023],[0,8795,0.0339372528],[0,8796,0.0912839213],[0,8798,0.0237750782],[0,8801,0.1137463856],[0,8802,0.0239531072],[0,8810,0.0977461192],[0,8813,0.0158212268],[0,8815,0.2625320532],[0,8816,0.3887942899],[0,8817,0.0158212268],[0,8820,0.0245503237],[0,8821,0.0253340833],[0,8822,0.0719100511],[0,8824,0.0247790454],[0,8827,0.0284519539],[0,8830,0.0405907405],[0,8834,0.2447286702],[0,8835,0.0237750782],[0,8836,0.4667314101],[0,8837,0.0158212268],[0,8841,0.016165323],[0,8844,0.1782392468],[0,8845,0.2628801],[0,8846,0.2456585868],[0,8847,0.2710918782],[0,8849,0.4218702702],[0,8850,0.2844182441],[0,8851,0.1437966037],[0,8852,0.0795033724],[0,8854,0.1322481041],[0,8857,0.4692598395],[0,8858,0.0733846062],[0,8861,0.0237750782],[0,8865,0.4432889432],[0,8867,0.1577693368],[0,8870,0.0159645471],[0,8871,0.0289045956],[0,8873,0.562044531],[0,8874,0.13195183],[0,8878,0.2430797626],[0,8882,0.0328585178],[0,8884,0.050590275],[0,8887,0.0498799369],[0,8888,0.0237750782],[0,8889,0.0435378885],[0,8892,0.3379448365],[0,8894,0.0158212268],[0,8896,0.1170709934],[0,8901,0.0237750782],[0,8902,0.0244679631],[0,8903,0.0237750782],[0,8905,0.0158212268],[0,8906,0.0301590646],[0,8911,0.0170896872],[0,8912,0.0954173456],[0,8913,0.0237750782],[0,8915,0.271168512],[0,8919,0.0237750782],[0,8921,0.0237750782],[0,8922,0.1055433044],[0,8923,0.0237750782],[0,8926,0.0325188195],[0,8927,0.493562221],[0,8928,0.0237750782],[0,8929,0.0158212268],[0,8931,0.047548024],[0,8933,0.0240909028],[0,8934,0.2699418868],[0,8935,0.0726031613],[0,8937,0.1254242248],[0,8939,0.0158212268],[0,8940,0.6633196865],[0,8941,0.1906719478],[0,8943,0.0158212268],[0,8944,0.0495657472],[0,8946,0.0486440917],[0,8948,0.0230008048],[0,8950,0.0158212268],[0,8959,0.0803140617],[0,8960,0.0200937098],[0,8961,0.0787407872],[0,8963,0.0915578733],[0,8966,0.0286647186],[0,8967,0.0237750782],[0,8969,0.3145761699],[0,8972,0.0961585091],[0,8976,0.0237750782],[0,8978,0.3722797003],[0,8983,0.4630594936],[0,8984,0.0237750782],[0,8986,0.0408959405],[0,8990,0.0282117511],[0,8992,0.0237750782],[0,8994,0.1065423527],[0,8998,0.048755614],[0,8999,0.1544653097],[0,9000,0.0237750782],[0,9001,0.2088807508],[0,9002,0.0237750782],[0,9003,0.0252141819],[0,9006,0.5065933857],[0,9007,0.5723820212],[0,9010,0.1016674181],[0,9014,0.0838883143],[0,9015,0.061371798],[0,9017,0.1241236056],[0,9019,0.4221674272],[0,9020,0.0314089689],[0,9021,0.1062962318],[0,9023,0.040899714],[0,9030,0.047548024],[0,9033,0.0237750782],[0,9035,0.4269072958],[0,9040,0.1509547673],[0,9042,0.0664930761],[0,9044,0.0237750782],[0,9046,0.2938307559],[0,9047,0.0895449064],[0,9049,0.1892758642],[0,9050,0.089609843],[0,9052,0.029684068],[0,9053,0.0901321062],[0,9054,0.0158212268],[0,9055,0.1702325945],[0,9057,0.0237750782],[0,9058,0.0237750782],[0,9059,0.5180304741],[0,9063,0.6292039566],[0,9065,0.0722036559],[0,9069,0.069570145],[0,9071,0.1367179824],[0,9075,0.2631236305],[0,9076,0.0237750782],[0,9086,0.3283879289],[0,9087,0.3684898044],[0,9088,0.0237750782],[0,9089,0.4719573667],[0,9091,0.0370671294],[0,9094,0.0237750782],[0,9095,0.1189163783],[0,9097,0.0367753243],[0,9100,0.0769833933],[0,9104,0.0237750782],[0,9106,0.024074855],[0,9107,0.0158212268],[0,9108,0.0836775772],[0,9109,0.2218988716],[0,9113,0.1939447],[0,9114,0.0654649255],[0,9118,0.0237750782],[0,9120,0.0523446839],[0,9121,0.0237750782],[0,9123,0.0340203402],[0,9124,0.2520484432],[0,9127,0.5405876968],[0,9128,0.5227537332],[0,9129,0.0170896872],[0,9132,0.5750348432],[0,9133,0.070586771],[0,9134,0.5661451822],[0,9137,0.1924395485],[0,9139,0.0237750782],[0,9142,0.0437138447],[0,9150,0.1316789789],[0,9153,0.2834248922],[0,9154,0.0401590141],[0,9159,0.0721884102],[0,9162,0.0170896872],[0,9164,0.024450891],[0,9166,0.2876598202],[0,9168,0.024913043],[0,9169,0.552847535],[0,9172,0.2984585839],[0,9176,0.2217910767],[0,9179,0.0947602352],[0,9180,0.0723189386],[0,9181,0.0170896872],[0,9184,0.0450598731],[0,9186,0.3440498896],[0,9187,0.0237750782],[0,9188,0.0302726894],[0,9191,0.2453666342],[0,9193,0.0237750782],[0,9195,0.1232567457],[0,9197,0.5058563835],[0,9198,0.0356540212],[0,9199,0.023512331],[0,9200,0.0237750782],[0,9203,0.3670348256],[0,9204,0.0232575354],[0,9208,0.2867582771],[0,9209,0.0237750782],[0,9210,0.0237750782],[0,9211,0.1410573958],[0,9212,0.0587223172],[0,9213,0.1721880093],[0,9215,0.1020840138],[0,9217,0.069345572],[0,9220,0.0158212268],[0,9224,0.1165227006],[0,9226,0.0173171709],[0,9228,0.0273569898],[0,9229,0.0840515689],[0,9231,0.0387428635],[0,9235,0.0237750782],[0,9239,0.0237750782],[0,9241,0.6329467196],[0,9242,0.0321777589],[0,9247,0.5002125831],[0,9248,0.5521032269],[0,9249,0.2179427919],[0,9251,0.0158212268],[0,9252,0.0638759009],[0,9253,0.0854842117],[0,9257,0.0282763036],[0,9260,0.0205031246],[0,9263,0.3112300064],[0,9270,0.0240909028],[0,9271,0.0398064159],[0,9272,0.0557760457],[0,9273,0.0237750782],[0,9277,0.0237750782],[0,9278,0.0158212268],[0,9279,0.1539774864],[0,9280,0.3407604334],[0,9281,0.0237750782],[0,9282,0.573222391],[0,9283,0.0172208364],[0,9286,0.0170896872],[0,9287,0.0674131474],[0,9288,0.0989831459],[0,9289,0.2900754471],[0,9290,0.0827010418],[0,9291,0.4638821175],[0,9294,0.0237750782],[0,9296,0.0237750782],[0,9299,0.0149960363],[0,9300,0.0408312319],[0,9302,0.685365817],[0,9304,0.0171860033],[0,9310,0.0333430624],[0,9316,0.0204326616],[0,9325,0.573803216],[0,9327,0.0237750782],[0,9328,0.0237750782],[0,9331,0.0368538338],[0,9335,0.0319773396],[0,9338,0.0171860033],[0,9341,0.1838870144],[0,9342,0.2765344434],[0,9346,0.6226888959],[0,9351,0.2616801957],[0,9352,0.0253715313],[0,9355,0.6846585929],[0,9358,0.130143721],[0,9359,0.1829805006],[0,9360,0.6728481651],[0,9362,0.1902398324],[0,9365,0.0239531072],[0,9369,0.237171066],[0,9370,0.1625016516],[0,9371,0.046758987],[0,9372,0.0237750782],[0,9373,0.0308017501],[0,9374,0.1389672565],[0,9380,0.1268966632],[0,9382,0.1567352157],[0,9384,0.147025859],[0,9387,0.6040144994],[0,9390,0.0584147451],[0,9392,0.0945926048],[0,9393,0.0170896872],[0,9395,0.0237750782],[0,9396,0.0318755412],[0,9400,0.0237750782],[0,9402,0.3446754442],[0,9403,0.0237750782],[0,9405,0.0158212268],[0,9406,0.2643067203],[0,9407,0.0642604223],[0,9412,0.4143742992],[0,9414,0.0237750782],[0,9415,0.0484382586],[0,9417,0.0840313835],[0,9418,0.1334706839],[0,9419,0.0170896872],[0,9428,0.5669813308],[0,9433,0.0237750782],[0,9434,0.1697886232],[0,9437,0.0917732114],[0,9441,0.0158212268],[0,9442,0.0256507713],[0,9446,0.0907322371],[0,9447,0.1132224881],[0,9450,0.0271522113],[0,9454,0.1716087625],[0,9455,0.0237750782],[0,9456,0.0338845337],[0,9459,0.2352330869],[0,9463,0.0237750782],[0,9464,0.0245266653],[0,9465,0.3826675144],[0,9470,0.0509727059],[0,9471,0.0273733707],[0,9473,0.0158212268],[0,9478,0.024278876],[0,9480,0.5949879451],[0,9482,0.0467253223],[0,9483,0.0543501204],[0,9484,0.0407223288],[0,9485,0.0237750782],[0,9488,0.0158212268],[0,9492,0.0189654439],[0,9495,0.2993757544],[0,9497,0.0237750782],[0,9499,0.0677816699],[0,9500,0.0189654439],[0,9502,0.5645906555],[0,9503,0.0346632652],[0,9504,0.084136986],[0,9505,0.0931817708],[0,9507,0.0237750782],[0,9509,0.0158212268],[0,9512,0.2136626479],[0,9514,0.089483797],[0,9516,0.2619143311],[0,9517,0.3512554066],[0,9518,0.0237750782],[0,9519,0.129119886],[0,9520,0.019158023],[0,9521,0.0358590523],[0,9523,0.5707029865],[0,9525,0.0237750782],[0,9526,0.0364213494],[0,9528,0.0159645471],[0,9530,0.0617573684],[0,9534,0.1747602587],[0,9540,0.0598697405],[0,9541,0.1772573288],[0,9542,0.1777949754],[0,9544,0.4093318193],[0,9547,0.025851457],[0,9549,0.0603286823],[0,9550,0.2907036432],[0,9551,0.5661451822],[0,9552,0.0355219225],[0,9554,0.5873357876],[0,9556,0.0716200784],[0,9558,0.0682097486],[0,9559,0.0486190514],[0,9562,0.5611822031],[0,9564,0.2877284621],[0,9565,0.277594138],[0,9566,0.0170896872],[0,9567,0.2689750195],[0,9569,0.0217875983],[0,9570,0.0704447959],[0,9572,0.0237750782],[0,9574,0.0237750782],[0,9576,0.3166909903],[0,9582,0.0293166924],[0,9584,0.4949179308],[0,9585,0.0273733707],[0,9587,0.0754830419],[0,9592,0.0995569262],[0,9594,0.5597660395],[0,9600,0.0158212268],[0,9603,0.0376032427],[0,9604,0.0882299392],[0,9605,0.0252424116],[0,9606,0.0405085909],[0,9617,0.115321311],[0,9618,0.1014071585],[0,9620,0.1928247934],[0,9621,0.1047929663],[0,9627,0.2355707882],[0,9630,0.1462777478],[0,9631,0.1186622739],[0,9632,0.2848600361],[0,9634,0.5494041064],[0,9635,0.4476812025],[0,9639,0.0189654439],[0,9643,0.0237750782],[0,9644,0.0268442705],[0,9645,0.3479975239],[0,9648,0.1103647647],[0,9652,0.6583587333],[0,9653,0.4716073058],[0,9654,0.0425352255],[0,9656,0.3602999307],[0,9657,0.0191839904],[0,9660,0.1773536226],[0,9662,0.0493006186],[0,9664,0.0831650443],[0,9665,0.0158212268],[0,9666,0.1456132747],[0,9667,0.4906569324],[0,9668,0.3054798983],[0,9670,0.0470120696],[0,9671,0.0799017348],[0,9672,0.0929757083],[0,9673,0.0237750782],[0,9677,0.0803981322],[0,9678,0.7358243986],[0,9680,0.0237750782],[0,9682,0.1339999321],[0,9685,0.199626283],[0,9687,0.0389803419],[0,9688,0.0234136113],[0,9689,0.0608961802],[0,9690,0.529118776],[0,9694,0.0158212268],[0,9696,0.1400036686],[0,9697,0.0926871301],[0,9700,0.0436662872],[0,9701,0.07387605],[0,9702,0.0237750782],[0,9703,0.0237750782],[0,9705,0.0579975818],[0,9707,0.532394452],[0,9708,0.0724890001],[0,9709,0.0370500355],[0,9713,0.025851457],[0,9714,0.0716607639],[0,9715,0.0450807727],[0,9716,0.0642990903],[0,9717,0.4031571431],[0,9720,0.5480168987],[0,9721,0.0389803419],[0,9722,0.2650456388],[0,9723,0.048905443],[0,9724,0.0211988824],[0,9727,0.0758389087],[0,9729,0.0619470472],[0,9731,0.6960088046],[0,9733,0.1979605271],[0,9736,0.2979516819],[0,9738,0.1991798494],[0,9739,0.0893453323],[0,9743,0.2314103202],[0,9745,0.0961970874],[0,9746,0.0865859833],[0,9747,0.0320522323],[0,9748,0.0287924024],[0,9750,0.047548024],[0,9751,0.038575267],[0,9752,0.0448902218],[0,9753,0.0529396365],[0,9754,0.0648395937],[0,9756,0.0306948196],[0,9757,0.0237750782],[0,9758,0.0643464272],[0,9760,0.0322200885],[0,9761,0.0370500355],[0,9763,0.0237750782],[0,9766,0.2508332825],[0,9768,0.2484041929],[0,9769,0.0158212268],[0,9771,0.0310055739],[0,9772,0.0753033414],[0,9774,0.0239531072],[0,9776,0.0158212268],[0,9778,0.109017917],[0,9779,0.0237750782],[0,9781,0.0327417512],[0,9782,0.0661672937],[0,9783,0.0556129289],[0,9785,0.1746361598],[0,9787,0.138294062],[0,9791,0.0306245791],[0,9793,0.0814875642],[0,9794,0.1274504611],[0,9795,0.2847122977],[0,9796,0.2453593102],[0,9801,0.0234136113],[0,9806,0.0282117511],[0,9808,0.0242686834],[0,9809,0.0430826694],[0,9810,0.0237750782],[0,9812,0.4660086354],[0,9813,0.2411672104],[0,9814,0.0237750782],[0,9815,0.1176970696],[0,9818,0.1690120309],[0,9820,0.2453983518],[0,9823,0.2583457247],[0,9825,0.0409969688],[0,9827,0.0723813712],[0,9830,0.2185791067],[0,9831,0.2629118173],[0,9832,0.1385298849],[0,9833,0.4304553921],[0,9838,0.0237750782],[0,9839,0.0416200189],[0,9840,0.2346246779],[0,9841,0.0454442859],[0,9843,0.0239531072],[0,9844,0.2766694746],[0,9845,0.2998234403],[0,9847,0.0225620949],[0,9849,0.1291118301],[0,9850,0.0237750782],[0,9852,0.0599571201],[0,9853,0.1629606523],[0,9854,0.1521756841],[0,9856,0.0866671329],[0,9857,0.0661362737],[0,9858,0.2876743225],[0,9859,0.0810578235],[0,9860,0.0237750782],[0,9862,0.0619500293],[0,9865,0.3215994381],[0,9869,0.0189654439],[0,9870,0.0872900286],[0,9871,0.0160270632],[0,9872,0.0718446939],[0,9879,0.2059246626],[0,9881,0.0398814878],[0,9883,0.089189064],[0,9884,0.7061335748],[0,9885,0.1443815412],[0,9888,0.0180385536],[0,9889,0.3790871386],[0,9890,0.0444214702],[0,9892,0.1495624172],[0,9897,0.0158212268],[0,9901,0.1745080104],[0,9904,0.3763235031],[0,9905,0.0237750782],[0,9906,0.0933468035],[0,9911,0.0778779061],[0,9913,0.1583680039],[0,9916,0.0189654439],[0,9918,0.0679688576],[0,9922,0.0237750782],[0,9923,0.351005768],[0,9924,0.0313756833],[0,9925,0.3925576239],[0,9929,0.0999866647],[0,9931,0.0237750782],[0,9932,0.0287885243],[0,9933,0.3940907974],[0,9935,0.1535180879],[0,9937,0.039236935],[0,9938,0.0228020181],[0,9939,0.0648930584],[0,9941,0.0590296003],[0,9943,0.0470120696],[0,9947,0.5981294556],[0,9950,0.019499538],[0,9952,0.0266761396],[0,9956,0.0237750782],[0,9958,0.045083005],[0,9959,0.023354717],[0,9961,0.5518593563],[0,9962,0.0189654439],[0,9964,0.0256531364],[0,9966,0.0961970874],[0,9970,0.068048847],[0,9972,0.0158212268],[0,9974,0.2463611736],[0,9977,0.4281705682],[0,9979,0.0851748738],[0,9980,0.0239531072],[0,9981,0.0239531072],[0,9984,0.0708530221],[0,9985,0.0237750782],[0,9987,0.0214676109],[0,9989,0.0386569008],[0,9990,0.0820359897],[0,9994,0.0356664176],[0,9997,0.5599596108],[0,9999,0.3349143457],[0,10001,0.0237750782],[0,10005,0.0373730303],[0,10006,0.0931743559],[0,10007,0.0290685583],[0,10014,0.0396858606],[0,10015,0.2649141376],[0,10016,0.03149919],[0,10018,0.0548789262],[0,10020,0.0795990689],[0,10021,0.0237750782],[0,10022,0.1427188579],[0,10024,0.0394017912],[0,10025,0.0561342577],[0,10026,0.1540437178],[0,10027,0.3815438872],[0,10029,0.5769461953],[0,10030,0.1026277449],[0,10031,0.0239531072],[0,10034,0.0250444591],[0,10037,0.076635433],[0,10038,0.6533105164],[0,10040,0.0853388282],[0,10043,0.0305068909],[0,10044,0.1135795788],[0,10046,0.5352116274],[0,10047,0.1217801627],[0,10048,0.0878227272],[0,10051,0.0158212268],[0,10055,0.0439447422],[0,10056,0.8595113682],[0,10057,0.0803140185],[0,10061,0.1904226606],[0,10062,0.4121385861],[0,10063,0.0189654439],[0,10065,0.0296085843],[0,10068,0.5536181443],[0,10069,0.0237750782],[0,10071,0.1157193536],[0,10072,0.0825974192],[0,10074,0.312948526],[0,10075,0.1241697431],[0,10076,0.0319048275],[0,10077,0.0405907405],[0,10078,0.1258817232],[0,10080,0.1425224677],[0,10081,0.0782155472],[0,10086,0.1356596456],[0,10090,0.8013082671],[0,10091,0.0189654439],[0,10092,0.0237750782],[0,10094,0.1392581906],[0,10096,0.5179267774],[0,10097,0.205701243],[0,10100,0.0573679264],[0,10103,0.0189654439],[0,10104,0.0510788492],[0,10106,0.161167809],[0,10107,0.2142052263],[0,10108,0.3347512063],[0,10109,0.4791126944],[0,10112,0.0158212268],[0,10114,0.0237750782],[0,10116,0.8959183607],[0,10126,0.4449623662],[0,10132,0.2298712838],[0,10134,0.0385118238],[0,10135,0.0448110046],[0,10139,0.0190969516],[0,10140,0.0299473682],[0,10141,0.4866554823],[0,10142,0.4037680913],[0,10144,0.1160260846],[0,10145,0.0237750782],[0,10146,0.048443883],[0,10147,0.0160270632],[0,10148,0.0435019909],[0,10151,0.3022729943],[0,10152,0.0944225948],[0,10154,0.0312023765],[0,10156,0.0481146731],[0,10157,0.0296508794],[0,10158,0.1025272707],[0,10159,0.0160270632],[0,10160,0.0237750782],[0,10161,0.0448815822],[0,10163,0.0237750782],[0,10164,0.1181664387],[0,10165,0.1280660905],[0,10167,0.4838062484],[0,10168,0.0237750782],[0,10174,0.0191839904],[0,10175,0.0237750782],[0,10176,0.0158212268],[0,10178,0.0158212268],[0,10180,0.0189654439],[0,10181,0.020507388],[0,10186,0.1122549629],[0,10187,0.0314777866],[0,10189,0.4561745073],[0,10190,0.5772153362],[0,10191,0.7339821877],[0,10192,0.0636447486],[0,10193,0.0574714966],[0,10194,0.0517779107],[0,10195,0.0329191012],[0,10196,0.1059107847],[0,10197,0.0528613061],[0,10200,0.1311241925],[0,10201,0.0351659867],[0,10202,0.3584729996],[0,10203,0.0672945966],[0,10204,0.0158212268],[0,10206,0.0158212268],[0,10209,0.150263176],[0,10210,0.0189654439],[0,10211,0.1475195739],[0,10213,0.2841857569],[0,10215,0.3536599118],[0,10216,0.2263022158],[0,10217,0.1899170418],[0,10218,0.5142232215],[0,10220,0.0237750782],[0,10223,0.0582853258],[0,10224,0.0423694543],[0,10225,0.1525437915],[0,10226,0.0477957104],[0,10228,0.0219186436],[0,10229,0.0317484542],[0,10231,0.0661536228],[0,10232,0.0921615141],[0,10234,0.3832817156],[0,10236,0.0237750782],[0,10237,0.1213652228],[0,10238,0.0439447422],[0,10239,0.0817671797],[0,10241,0.0158212268],[0,10242,0.0581471221],[0,10243,0.6643676894],[0,10245,0.6059024174],[0,10247,0.1423388528],[0,10248,0.2110713684],[0,10249,0.0395615248],[0,10250,0.0237750782],[0,10251,0.0544824174],[0,10252,0.5327125845],[0,10253,0.0285212137],[0,10254,0.1163061611],[0,10257,0.13782927],[0,10258,0.3702109637],[0,10259,0.3957572081],[0,10260,0.0158212268],[0,10261,0.0406114007],[0,10262,0.046758987],[0,10266,0.3632419688],[0,10268,0.0499263539],[0,10269,0.0572890957],[0,10271,0.132390466],[0,10274,0.0927041665],[0,10275,0.0998834062],[0,10277,0.0189654439],[0,10281,0.5604429407],[0,10283,0.1055433044],[0,10285,0.0921615141],[0,10286,0.0237750782],[0,10289,0.2229237143],[0,10290,0.3061760891],[0,10291,0.0237750782],[0,10294,0.0581471221],[0,10296,0.0237750782],[0,10298,0.0644097832],[0,10299,0.024450891],[0,10300,0.11066982],[0,10302,0.0237750782],[0,10304,0.0750441147],[0,10306,0.5167564192],[0,10308,0.0189654439],[0,10309,0.2271903803],[0,10310,0.063806004],[0,10311,0.3947700556],[0,10312,0.0343728957],[0,10314,0.0852280563],[0,10315,0.0176935169],[0,10317,0.0339127765],[0,10318,0.7641234999],[0,10319,0.573222391],[0,10320,0.0673271853],[0,10321,0.0924836712],[0,10322,0.4886835086],[0,10326,0.0237750782],[0,10327,0.1262778111],[0,10328,0.2790861636],[0,10329,0.1507948481],[0,10330,0.0189654439],[0,10331,0.0237750782],[0,10332,0.0590294494],[0,10333,0.0310167467],[0,10334,0.0340124986],[0,10335,0.0301300368],[0,10336,0.5669813308],[0,10338,0.0374239575],[0,10339,0.0411702985],[0,10341,0.2436877379],[0,10342,0.0158212268],[0,10343,0.0895659859],[0,10344,0.2581114409],[0,10345,0.0252831605],[0,10346,0.1619388239],[0,10347,0.1479991665],[0,10348,0.0189654439],[0,10349,0.0445027617],[0,10350,0.0189654439],[0,10351,0.1705394686],[0,10352,0.0897263389],[0,10353,0.0874408641],[0,10355,0.3536755634],[0,10356,0.025198327],[0,10357,0.0239531072],[0,10358,0.1244526484],[0,10359,0.0921615141],[0,10362,0.2288043822],[0,10366,0.0360424716],[0,10367,0.0805758753],[0,10368,0.3752467995],[0,10369,0.0583169032],[0,10371,0.2815130197],[0,10372,0.0200937098],[0,10373,0.3307899542],[0,10374,0.0661362737],[0,10375,0.0237750782],[0,10378,0.1232434887],[0,10382,0.1343495027],[0,10383,0.0995861936],[0,10384,0.158364319],[0,10385,0.1347024308],[0,10386,0.4943492444],[0,10387,0.0620135102],[0,10388,0.055068878],[0,10389,0.2749793899],[0,10390,0.3290327397],[0,10392,0.0355219225],[0,10393,0.0237750782],[0,10394,0.0991813019],[0,10395,0.0963272925],[0,10396,0.1185472376],[0,10399,0.0422444473],[0,10403,0.1070984445],[0,10405,0.0237750782],[0,10406,0.0237750782],[0,10408,0.0278162329],[0,10409,0.2088030957],[0,10411,0.0533191803],[0,10412,0.0878227272],[0,10414,0.045828767],[0,10416,0.0158212268],[0,10419,0.0785864036],[0,10420,0.6284490208],[0,10421,0.0619032615],[0,10422,0.0993455057],[0,10423,0.0237750782],[0,10425,0.0223248646],[0,10426,0.0140744918],[0,10428,0.2196692524],[0,10429,0.0160270632],[0,10430,0.0237750782],[0,10432,0.0636790499],[0,10435,0.5870576822],[0,10437,0.9238620441],[0,10439,0.1202035013],[0,10441,0.0472495941],[0,10443,0.019499538],[0,10444,0.0370500355],[0,10446,0.0176935169],[0,10448,0.5064556751],[0,10451,0.1434848021],[0,10453,0.0239531072],[0,10460,0.3047488352],[0,10461,0.0594630015],[0,10464,0.1240605281],[0,10465,0.1786913763],[0,10469,0.0389803419],[0,10470,0.3079807877],[0,10471,0.1881566919],[0,10472,0.0992800461],[0,10477,0.1211716709],[0,10479,0.1124608594],[0,10480,0.1999882526],[0,10482,0.0294206034],[0,10486,0.2226373009],[0,10488,0.3723476166],[0,10489,0.024278876],[0,10491,0.0445467575],[0,10492,0.1068902012],[0,10495,0.1463639161],[0,10497,0.0237750782],[0,10499,0.408933046],[0,10500,0.0204283986],[0,10501,0.0548085339],[0,10502,0.2425643962],[0,10503,0.0237750782],[0,10506,0.1210332736],[0,10508,0.3138430321],[0,10509,0.0389803419],[0,10511,0.7938534824],[0,10512,0.1087301643],[0,10513,0.231048444],[0,10516,0.2912628527],[0,10517,0.0244292734],[0,10518,0.1372036358],[0,10520,0.0237750782],[0,10523,0.0158212268],[0,10527,0.3847443733],[0,10528,0.0609176953],[0,10529,0.0189654439],[0,10533,0.0961970874],[0,10534,0.437189612],[0,10536,0.2210630522],[0,10537,0.0364213494],[0,10539,0.0239531072],[0,10542,0.0237750782],[0,10543,0.06050512],[0,10545,0.0239531072],[0,10546,0.0378283736],[0,10547,0.173087525],[0,10548,0.0189654439],[0,10549,0.0738043735],[0,10551,0.4523229457],[0,10552,0.5949070275],[0,10554,0.034082613],[0,10555,0.0237750782],[0,10557,0.0724471267],[0,10560,0.0364683288],[0,10562,0.875926406],[0,10563,0.2909263073],[0,10564,0.0601123776],[0,10566,0.0691951169],[0,10568,0.7342396344],[0,10582,0.6779605971],[0,10584,0.449163928],[0,10589,0.0611657852],[0,10591,0.0557496743],[0,10601,0.1265192448],[0,10603,0.1181499913],[0,10605,0.3547853798],[0,10610,0.0587021956],[0,10613,0.3097840382],[0,10614,0.2246054881],[0,10630,0.0996453401],[0,10634,0.0214399938],[0,10635,0.0237750782],[0,10639,0.2189779653],[0,10643,0.3998994063],[0,10647,0.038575267],[0,10648,0.7247757246],[0,10654,0.144688844],[0,10664,0.128778613],[0,10668,0.4815829965],[0,10672,0.0691919003],[0,10675,0.6132147656],[0,10681,0.5991407322],[0,10683,0.4632149359],[0,10688,0.1952711048],[0,10700,0.0907322371],[0,10701,0.2675421217],[0,10704,0.2672683594],[0,10705,0.0841446905],[0,10707,0.3051239473],[0,10710,0.0693049263],[0,10711,0.6594685585],[0,10713,0.8633357058],[0,10715,0.1047828314],[0,10719,0.1855759848],[0,10724,0.0442322576],[0,10725,0.2727085372],[0,10729,0.2371363345],[0,10731,0.1843957239],[0,10735,0.3806092762],[0,10742,0.0305597869],[0,10745,0.2351616435],[0,10751,0.2314448219],[0,10754,0.0807048227],[0,10755,0.3071010732],[0,10759,0.6138673231],[0,10760,0.1559713824],[0,10761,0.5162228243],[0,10765,0.0237750782],[0,10766,0.2625941103],[0,10768,0.1351232787],[0,10769,0.0589099184],[0,10771,0.0910140817],[0,10772,0.4076416206],[0,10774,0.3447475434],[0,10775,0.2803791122],[0,10778,0.6266293142],[0,10781,0.0624580966],[0,10784,0.0688601895],[0,10785,0.0611138479],[0,10788,0.0762148563],[0,10789,0.0494882309],[0,10790,0.3331797033],[0,10793,0.2199735028],[0,10796,0.2950670737],[0,10797,0.0875653783],[0,10798,0.2530401907],[0,10799,0.0664223037],[0,10801,0.0627001576],[0,10805,0.529477928],[0,10808,0.876045586],[0,10813,0.0900403736],[0,10814,0.1130504027],[0,10815,0.349001458],[0,10816,0.3450579734],[0,10821,0.0673060249],[0,10823,0.1544239485],[0,10824,0.0600551073],[0,10825,0.0660688192],[0,10826,0.4434264499],[0,10827,0.0922941262],[0,10828,0.1210355904],[0,10830,0.5504599624],[0,10832,0.3476788207],[0,10833,0.0660509174],[0,10834,0.0597522827],[0,10836,0.0865855693],[0,10840,0.2763623599],[0,10841,0.2197430386],[0,10843,0.1007400854],[0,10845,0.8334946033],[0,10846,0.0690078164],[0,10847,0.3409913398],[0,10850,0.1845685614],[0,10851,0.0409328096],[0,10852,0.212186851],[0,10853,0.0715128055],[0,10855,0.048472488],[0,10856,0.2464291561],[0,10859,0.492506162],[0,10861,0.1472565516],[0,10863,0.2038790248],[0,10864,0.0682074463],[0,10866,0.2825546135],[0,10868,0.1222713397],[0,10873,0.2851540573],[0,10874,0.0883747883],[0,10876,0.0846623428],[0,10877,0.2502744654],[0,10879,0.4323832197],[0,10881,0.1028539064],[0,10882,0.4132733928],[0,10885,0.1853404053],[0,10886,0.0734501621],[0,10889,0.0537064241],[0,10890,0.1907009069],[0,10891,0.0733068326],[0,10894,0.2715124172],[0,10896,0.0805007476],[0,10898,0.4411952148],[0,10901,0.1323698504],[0,10903,0.1963004982],[0,10904,0.0973306914],[0,10905,0.2630569445],[0,10908,0.4891753947],[0,10909,0.3584080391],[0,10912,0.187977767],[0,10915,0.5470115892],[0,10916,0.1580926823],[0,10920,0.4074833514],[0,10921,0.064455432],[0,10923,0.3637920532],[0,10924,0.4199419146],[0,10926,0.0605372791],[0,10927,0.0522172498],[0,10930,0.4368810719],[0,10931,0.0624399606],[0,10934,0.7450244641],[0,10938,0.0378743716],[0,10939,0.3095042548],[0,10941,0.1950754561],[0,10942,0.4973232017],[0,10944,0.0340140318],[0,10946,0.1944459289],[0,10950,0.6013908672],[0,10951,0.2171402996],[0,10953,0.4611698311],[0,10954,0.0138166251],[0,10955,0.0217472033],[0,10959,0.0578628908],[0,10961,0.3209690353],[0,10965,0.0219247114],[0,10967,0.2349686008],[0,10968,0.0665860131],[0,10969,0.0646595692],[0,10970,0.4250383191],[0,10971,0.2164405024],[0,10974,0.0270251945],[0,10975,0.0219247114],[0,10979,0.4393465512],[0,10983,0.1228240484],[0,10984,0.5660476258],[0,10985,0.1881599293],[0,10987,0.1724792539],[0,10992,0.0324515416],[0,10995,0.1697547241],[0,10997,0.0126918888],[0,10998,0.5157917397],[0,10999,0.0815012737],[0,11000,0.1043161138],[0,11003,0.2018495592],[0,11006,0.142763272],[0,11008,0.0195932863],[0,11012,0.2744064326],[0,11013,0.3407048439],[0,11015,0.1988995458],[0,11016,0.0335996205],[0,11017,0.0426603889],[0,11019,0.4012922807],[0,11020,0.0599953883],[0,11023,0.3370643111],[0,11024,0.3360115517],[0,11026,0.3187979228],[0,11028,0.25835247],[0,11029,0.0758007525],[0,11032,0.4660151979],[0,11033,0.2258764982],[0,11035,0.216163605],[0,11037,0.2546405915],[0,11042,0.2206535073],[0,11045,0.1966493874],[0,11048,0.5886628095],[0,11050,0.4324501306],[0,11052,0.0824771315],[0,11055,0.3027877389],[0,11056,0.2051174456],[0,11058,0.3179793677],[0,11061,0.0200246031],[0,11065,0.0611017437],[0,11067,0.4419731892],[0,11068,0.074564685],[0,11076,0.1717576351],[0,11077,0.1012339216],[0,11078,0.1191649512],[0,11079,0.0211564028],[0,11084,0.0368793296],[0,11085,0.0246500458],[0,11087,0.1712369784],[0,11090,0.0598080967],[0,11091,0.0738174879],[0,11092,0.1009840777],[0,11093,0.0620469039],[0,11095,0.2012135021],[0,11098,0.1093630682],[0,11099,0.3618926571],[0,11100,0.2952158429],[0,11104,0.056007613],[0,11105,0.0450310857],[0,11107,0.0121137946],[0,11108,0.2141016217],[0,11110,0.0403274324],[0,11111,0.0696213804],[0,11112,0.6717865422],[0,11113,0.0373647544],[0,11114,0.1422654527],[0,11115,0.0218924981],[0,11117,0.2918238216],[0,11118,0.0337862563],[0,11119,0.0407878725],[0,11121,0.1904059772],[0,11122,0.0153717926],[0,11127,0.0637370736],[0,11128,0.3116178386],[0,11130,0.0375498704],[0,11131,0.0216715218],[0,11132,0.3108561923],[0,11133,0.0200957403],[0,11137,0.191991788],[0,11138,0.4809186031],[0,11139,0.0121137946],[0,11140,0.1343690773],[0,11143,0.2219033635],[0,11144,0.1330575421],[0,11145,0.1616098285],[0,11146,0.1509242967],[0,11147,0.0322360759],[0,11150,0.4965177782],[0,11152,0.1233121299],[0,11153,0.1794861772],[0,11156,0.0971410583],[0,11157,0.0451688433],[0,11159,0.1741727666],[0,11160,0.0557716991],[0,11162,0.0978236808],[0,11163,0.0634503387],[0,11164,0.3933677582],[0,11166,0.0542162461],[0,11167,0.2624873969],[0,11168,0.1041844935],[0,11170,0.3369837825],[0,11173,0.3699749356],[0,11174,0.1467198792],[0,11175,0.0378534384],[0,11179,0.1056703472],[0,11180,0.1119532773],[0,11181,0.1997199841],[0,11187,0.1065476726],[0,11189,0.0165032659],[0,11190,0.3809572684],[0,11191,0.1432602807],[0,11192,0.2379969597],[0,11193,0.2394191549],[0,11201,0.1098230715],[0,11203,0.4390864092],[0,11205,0.1093772142],[0,11209,0.1245124071],[0,11216,0.123144858],[0,11217,0.0414950359],[0,11220,0.3028518213],[0,11221,0.0513907156],[0,11223,0.1340318533],[0,11225,0.0133753906],[0,11228,0.018213048],[0,11231,0.0133753906],[0,11232,0.1912304004],[0,11235,0.0438058815],[0,11237,0.1010355899],[0,11241,0.0554528449],[0,11242,0.0313736812],[0,11243,0.0360967529],[0,11245,0.3698170277],[0,11247,0.0189128417],[0,11249,0.0203334126],[0,11252,0.1338464312],[0,11255,0.3348960299],[0,11256,0.086992955],[0,11257,0.5749052135],[0,11259,0.0114005787],[0,11264,0.0977749038],[0,11266,0.5005644253],[0,11269,0.0404373967],[0,11270,0.0422339241],[0,11271,0.0105063787],[0,11273,0.0202016687],[0,11274,0.1361769957],[0,11276,0.0200246031],[0,11278,0.2015764205],[0,11279,0.2258574516],[0,11280,0.0751623282],[0,11282,0.1643563128],[0,11283,0.149775415],[0,11284,0.1436394355],[0,11288,0.1047782986],[0,11291,0.0366844857],[0,11292,0.1284369961],[0,11293,0.0280470991],[0,11296,0.1010327052],[0,11300,0.3307431482],[0,11301,0.2627301255],[0,11305,0.8470875064],[0,11307,0.0121137946],[0,11310,0.0872791245],[0,11313,0.2243378652],[0,11316,0.0202016687],[0,11321,0.152999355],[0,11327,0.1518463498],[0,11330,0.2402967494],[0,11332,0.1004612352],[0,11333,0.1093402379],[0,11335,0.1585916459],[0,11336,0.0305292626],[0,11340,0.1973580901],[0,11349,0.4026548968],[0,11353,0.0200246031],[0,11354,0.0390764674],[0,11356,0.1309767711],[0,11357,0.0152409966],[0,11360,0.0207227548],[0,11362,0.0209657723],[0,11364,0.5165146042],[0,11365,0.3466574094],[0,11366,0.1290388789],[0,11367,0.1522680917],[0,11368,0.0149266902],[0,11370,0.1058431385],[0,11371,0.0152409966],[0,11373,0.1877108855],[0,11374,0.2639363756],[0,11378,0.3653855691],[0,11379,0.0253829908],[0,11380,0.5054055474],[0,11381,0.0200246031],[0,11382,0.1819939573],[0,11383,0.6375003199],[0,11384,0.018628592],[0,11385,0.0299955037],[0,11386,0.023395959],[0,11396,0.0197272548],[0,11398,0.0584718585],[0,11401,0.0178342312],[0,11403,0.2428911817],[0,11404,0.717043942],[0,11405,0.0574289959],[0,11409,0.0189601117],[0,11410,0.0953192909],[0,11416,0.0963939736],[0,11419,0.0431358457],[0,11422,0.2188700099],[0,11423,0.1886715767],[0,11424,0.1558383894],[0,11429,0.2915928105],[0,11430,0.1348600719],[0,11432,0.0319936136],[0,11435,0.0839294874],[0,11439,0.1396594948],[0,11441,0.0121137946],[0,11443,0.0152409966],[0,11444,0.0628588791],[0,11446,0.3452514746],[0,11447,0.6194761465],[0,11450,0.0121137946],[0,11451,0.0200246031],[0,11452,0.0149266902],[0,11453,0.1120108398],[0,11455,0.8479552636],[0,11459,0.0291950462],[0,11461,0.2077540759],[0,11462,0.0264589334],[0,11463,0.1168923488],[0,11464,0.1392250001],[0,11465,0.0123185172],[0,11466,0.063035062],[0,11467,0.0965220716],[0,11470,0.0780594439],[0,11472,0.311916651],[0,11473,0.1183395511],[0,11475,0.0939831536],[0,11476,0.0121137946],[0,11478,0.1295436261],[0,11482,0.019509861],[0,11483,0.0431358457],[0,11485,0.3607415954],[0,11488,0.3480410997],[0,11489,0.0200246031],[0,11490,0.0121137946],[0,11491,0.1211098632],[0,11492,0.3155720565],[0,11493,0.0212672877],[0,11495,0.0158506565],[0,11497,0.3236948337],[0,11498,0.0376889322],[0,11499,0.0247434434],[0,11500,0.0599576636],[0,11503,0.4221611956],[0,11507,0.0121137946],[0,11508,0.0267165245],[0,11542,0.0200246031],[0,11547,0.1176655251],[0,11548,0.8602280384],[0,11550,0.8905764833],[0,11551,0.948847463],[0,11553,0.2783144291],[0,11554,0.0200246031],[0,11555,0.909689318],[0,11560,0.1758854449],[0,11561,0.0121137946],[0,11563,0.0832751726],[0,11564,0.0200246031],[0,11568,0.0152409966],[0,11570,0.3466108695],[0,11572,0.0133753906],[0,11577,0.0200246031],[0,11588,0.9477530364],[0,11589,0.0124560287],[0,11590,0.0124560287],[0,11599,0.0473416396],[0,11601,0.1253171343],[0,11602,0.0152409966],[0,11605,0.8031545879],[0,11611,0.0200246031],[0,11612,0.0121137946],[0,11614,0.8853085412],[0,11627,0.1134896944],[0,11629,0.7759917874],[0,11632,0.0133753906],[0,11633,0.0134711855],[0,11635,0.0121137946],[0,11636,0.0200246031],[0,11637,0.1078420781],[0,11638,0.0124560287],[0,11642,0.3557810157],[0,11647,0.0909865916],[0,11651,0.9543307169],[0,11654,0.0167662247],[0,11656,0.0354479521],[0,11663,0.0121137946],[0,11668,0.0200246031],[0,11669,0.0200246031],[0,11682,0.1707866266],[0,11687,0.0200246031],[0,11688,0.0152409966],[0,11693,0.0567251365],[0,11697,0.8937458133],[0,11704,0.0121137946],[0,11707,0.0200246031],[0,11713,0.0124560287],[0,11726,0.0200246031],[0,11728,0.0625633621],[0,11733,0.8971413646],[0,11734,0.0200246031],[0,11736,0.0121137946],[0,11738,0.0399986563],[0,11742,0.0200246031],[0,11746,0.0200246031],[0,11751,0.0200246031],[0,11756,0.0121137946],[0,11764,0.0432317873],[0,11766,0.8027900341],[0,11772,0.1737152941],[0,11773,0.0200246031],[0,11776,0.7711108226],[0,11784,0.941202004],[0,11787,0.0121137946],[0,11799,0.2142386396],[0,11801,0.0152409966],[0,11808,0.3404187998],[0,11813,0.2802178827],[0,11814,0.1132540097],[0,11817,0.0200246031],[0,11820,0.0200246031],[0,11828,0.0456249747],[0,11829,0.0943427533],[0,11834,0.0143191222],[0,11838,0.0200246031],[0,11844,0.0121137946],[0,11846,0.0121137946],[0,11848,0.095466534],[0,11859,0.0152409966],[0,11862,0.0121137946],[0,11867,0.0200246031],[0,11868,0.0200246031],[0,11875,0.0200246031],[0,11877,0.0200246031],[0,11886,0.0448464165],[0,11887,0.3609267903],[0,11889,0.944474196],[0,11897,0.0152409966],[0,11901,0.8509427936],[0,11904,0.7412146189],[0,11907,0.1916071553],[0,11908,0.1318418587],[0,11912,0.2158591037],[0,11916,0.096011412],[0,11924,0.037900561],[0,11925,0.0822337504],[0,11926,0.0133753906],[0,11930,0.0143191222],[0,11933,0.9433827273],[0,11938,0.0200246031],[0,11945,0.0200246031],[0,11948,0.0371023911],[0,11953,0.8853085412],[0,11962,0.0200246031],[0,11965,0.0152409966],[0,11968,0.2750768267],[0,11970,0.0200246031],[0,11988,0.0320796726],[0,11991,0.9664593689],[0,11995,0.0197077747],[0,12000,0.2605302944],[0,12006,0.0624820226],[0,12007,0.9977427156],[0,12010,0.7089273451],[0,12011,0.0134711855],[0,12014,0.0143191222],[0,12022,0.0497379051],[0,12023,0.0202016687],[0,12027,0.03497131],[0,12029,0.1460843634],[0,12030,0.0200246031],[0,12032,0.4482557051],[0,12033,0.9013863592],[0,12036,0.4156229484],[0,12042,0.2586057517],[0,12045,0.0124560287],[0,12046,0.0121137946],[0,12048,0.0121137946],[0,12049,0.2793022057],[0,12057,0.2098224468],[0,12059,0.0124560287],[0,12065,0.2195217127],[0,12068,0.8644140828],[0,12074,0.0200246031],[0,12076,0.5439874708],[0,12080,0.0124560287],[0,12081,0.2105987461],[0,12083,0.0200246031],[0,12086,0.0200246031],[0,12088,0.3397358884],[0,12090,0.0121137946],[0,12091,0.0124560287],[0,12098,0.0200246031],[0,12100,0.0133753906],[0,12102,0.2424571523],[0,12113,0.0200246031],[0,12114,0.0200246031],[0,12117,0.0200246031],[0,12120,0.2679855855],[0,12124,0.0121137946],[0,12145,0.0200246031],[0,12147,0.0152409966],[0,12148,0.8268885958],[0,12149,0.2834026242],[0,12151,0.0200246031],[0,12153,0.0674140946],[0,12154,0.0133753906],[0,12163,0.9054218832],[0,12164,0.0121137946],[0,12165,0.0121137946],[0,12167,0.0124560287],[0,12168,0.0133753906],[0,12174,0.0152409966],[0,12176,0.0200246031],[0,12179,0.0133753906],[0,12181,0.0200246031],[0,12182,0.3858626571],[0,12184,0.0400307634],[0,12187,0.1499957332],[0,12191,0.10995956],[0,12198,0.0114005787],[0,12203,0.8073520609],[0,12209,0.0139093693],[0,12212,0.0143191222],[0,12213,0.0273243406],[0,12214,0.0163924355],[0,12215,0.0638488903],[0,12216,0.5587252423],[0,12217,0.3888989493],[0,12218,0.0148306911],[0,12220,0.0596436078],[0,12222,0.0724429738],[0,12223,0.0318253668],[0,12224,0.1382156785],[0,12225,0.0377471989],[0,12227,0.0202016687],[0,12230,0.5221950978],[0,12231,0.7784101221],[0,12234,0.026697683],[0,12236,0.2930838054],[0,12237,0.0436885158],[0,12242,0.0200246031],[0,12243,0.310795393],[0,12247,0.0200246031],[0,12250,0.0323240506],[0,12255,0.023157689],[0,12257,0.0205825734],[0,12258,0.2224904405],[0,12259,0.0420889863],[0,12262,0.0360584191],[0,12263,0.2696821347],[0,12266,0.2171078782],[0,12267,0.0528346492],[0,12268,0.0930958133],[0,12270,0.0889516099],[0,12272,0.3176815336],[0,12273,0.0502553368],[0,12274,0.0700924876],[0,12275,0.3769613376],[0,12277,0.1325813119],[0,12278,0.1994488425],[0,12279,0.0447144204],[0,12280,0.0123185172],[0,12281,0.0385238715],[0,12283,0.0659100786],[0,12284,0.0246771106],[0,12287,0.2446922078],[0,12288,0.0444289343],[0,12291,0.2221090011],[0,12293,0.0885079892],[0,12294,0.0959395999],[0,12298,0.2308437542],[0,12300,0.0599461876],[0,12315,0.2434070151],[0,12317,0.075589107],[0,12331,0.0727440872],[0,12333,0.6854580991],[0,12337,0.2288068066],[0,12339,0.0239683517],[0,12340,0.0370609998],[0,12343,0.0567148467],[0,12345,0.0435165552],[0,12350,0.0409974164],[0,12351,0.2252640221],[0,12355,0.1450681408],[0,12357,0.3832306112],[0,12360,0.081932615],[0,12367,0.0476026019],[0,12368,0.1011717572],[0,12369,0.0630157815],[0,12370,0.0617669383],[0,12372,0.0191768229],[0,12373,0.0558757447],[0,12374,0.0349438998],[0,12377,0.2171778387],[0,12379,0.0847829447],[0,12381,0.0768999563],[0,12383,0.0514876745],[0,12386,0.3761397937],[0,12388,0.2726575517],[0,12389,0.2522930201],[0,12392,0.0998070957],[0,12413,0.0184908017],[0,12414,0.1074191914],[0,12416,0.190886379],[0,12427,0.0871980389],[0,12430,0.0304584308],[0,12431,0.0121137946],[0,12438,0.2126434402],[0,12450,0.0206753037],[0,12452,0.0563757103],[0,12456,0.0585569815],[0,12459,0.217984191],[0,12462,0.1731670153],[0,12467,0.0947356117],[0,12478,0.0654794886],[0,12489,0.0200246031],[0,12494,0.1317886476],[0,12501,0.0452786318],[0,12516,0.119866058],[0,12519,0.2758135898],[0,12523,0.0676006807],[0,12525,0.1288465952],[0,12533,0.0615927158],[0,12536,0.0877452828],[0,12537,0.142149471],[0,12545,0.0285641713],[0,12549,0.0753875745],[0,12552,0.0875764614],[0,12559,0.3405343429],[0,12560,0.3558878064],[0,12562,0.3385780317],[0,12565,0.2086926486],[0,12567,0.2764417347],[0,12568,0.0311225181],[0,12572,0.0606044069],[0,12575,0.0202016687],[0,12577,0.029498403],[0,12582,0.3864577381],[0,12586,0.0399529011],[0,12587,0.07497033],[0,12597,0.1395318779],[0,12598,0.1406823509],[0,12602,0.1380186463],[0,12604,0.0789212369],[0,12611,0.073887238],[0,12612,0.0984382712],[0,12613,0.0557075756],[0,12614,0.3898939422],[0,12617,0.0437087575],[0,12618,0.0121137946],[0,12619,0.3621108147],[0,12622,0.215714676],[0,12626,0.0678861666],[0,12629,0.0377420503],[0,12630,0.2167886354],[0,12644,0.1310391791],[0,12647,0.0607510681],[0,12668,0.0801123437],[0,12682,0.0848889228],[0,12684,0.1210873411],[0,12700,0.1354175937],[0,12702,0.2518818354],[0,12708,0.130962378],[0,12710,0.0272195166],[0,12713,0.2693579167],[0,12723,0.03233506],[0,12728,0.0450391646],[0,12734,0.0790066595],[0,12735,0.0452786318],[0,12736,0.2381532896],[0,12754,0.1208150728],[0,12765,0.2168199018],[0,12767,0.0940876211],[0,12774,0.0252929518],[0,12776,0.0707419727],[0,12777,0.1904560601],[0,12779,0.0655903084],[0,12780,0.1197863493],[0,12794,0.0375066231],[0,12796,0.1067649122],[0,12800,0.0221985323],[0,12802,0.2781808436],[0,12803,0.2499511744],[0,12807,0.3331338644],[0,12808,0.2821216443],[0,12809,0.3031860765],[0,12813,0.0452786318],[0,12819,0.0489458656],[0,12820,0.0121137946],[0,12825,0.137512751],[0,12827,0.0568630366],[0,12832,0.1809676926],[0,12838,0.0525220267],[0,12846,0.0200246031],[0,12852,0.2487197812],[0,12854,0.0200246031],[0,12857,0.1601871883],[0,12861,0.0746751995],[0,12863,0.0658906228],[0,12864,0.2018312683],[0,12871,0.210350238],[0,12872,0.1991127749],[0,12873,0.1180352943],[0,12874,0.0254147196],[0,12876,0.1189299353],[0,12880,0.09876347],[0,12881,0.3519371646],[0,12883,0.0391924815],[0,12896,0.2140943004],[0,12901,0.0285641713],[0,12902,0.2073440826],[0,12906,0.2473673015],[0,12912,0.0371023911],[0,12916,0.3057069778],[0,12918,0.1095288638],[0,12920,0.1101967511],[0,12922,0.1977080533],[0,12924,0.0404324663],[0,12927,0.3523025817],[0,12928,0.0373256878],[0,12932,0.2506123633],[0,12936,0.1968227494],[0,12937,0.215453281],[0,12939,0.2494043966],[0,12941,0.0864827927],[0,12943,0.3267997069],[0,12951,0.1632560319],[0,12953,0.4643462004],[0,12961,0.0883510422],[0,12965,0.1947916714],[0,12967,0.0581508413],[0,12975,0.0200246031],[0,12976,0.0886399687],[0,12983,0.0466832659],[0,12986,0.0955848784],[0,12989,0.0530903041],[0,12993,0.0971820292],[0,12996,0.0514564191],[0,12997,0.0668872937],[0,12999,0.3808373833],[0,13007,0.2993831351],[0,13010,0.0143191222],[0,13015,0.1185305473],[0,13018,0.0381298944],[0,13039,0.0880927825],[0,13040,0.0785393103],[0,13041,0.2319052083],[0,13043,0.1830094393],[0,13046,0.1452511911],[0,13048,0.1954670173],[0,13049,0.0447144204],[0,13051,0.624688019],[0,13052,0.0902875143],[0,13065,0.2291380926],[0,13068,0.1297243807],[0,13072,0.0534238829],[0,13073,0.9086213763],[0,13079,0.0239874225],[0,13085,0.1583492303],[0,13086,0.0397113062],[0,13095,0.0555956363],[0,13100,0.9587306953],[0,13104,0.9335926691],[0,13107,0.3269233305],[0,13110,0.2523299699],[0,13111,0.8530012979],[0,13114,0.8612632268],[0,13115,0.268645129],[0,13116,0.1563615062],[0,13120,0.3495234089],[0,13122,0.1694409469],[0,13124,0.2657035667],[0,13130,0.083874976],[0,13133,0.8214053962],[0,13135,0.2388589142],[0,13146,0.5585915024],[0,13147,0.1912380505],[0,13148,0.2770073216],[0,13151,0.0316563138],[0,13154,0.0918066008],[0,13157,0.0338954336],[0,13159,0.0545725111],[0,13160,0.0121137946],[0,13165,0.8696112207],[0,13167,0.1217253222],[0,13169,0.221658796],[0,13172,0.0774245927],[0,13176,0.1869809597],[0,13178,0.0513608766],[0,13179,0.0518758519],[0,13188,0.1096218408],[0,13191,0.2191267422],[0,13192,0.0434637801],[0,13193,0.0491616431],[0,13194,0.7631682652],[0,13195,0.0446775175],[0,13198,0.4759654441],[0,13199,0.0529722419],[0,13202,0.3804474162],[0,13203,0.0555956363],[0,13205,0.299923511],[0,13206,0.1019876678],[0,13207,0.2230243931],[0,13208,0.6700906388],[0,13215,0.0615247252],[0,13217,0.02843704],[0,13218,0.1024776537],[0,13219,0.1908829681],[0,13220,0.1322448112],[0,13221,0.0963247078],[0,13225,0.8289145285],[0,13232,0.9206316474],[0,13235,0.260944444],[0,13237,0.054325936],[0,13248,0.147848161],[0,13257,0.0200246031],[0,13258,0.0440006351],[0,13259,0.1522011776],[0,13263,0.3547764377],[0,13267,0.8519716976],[0,13269,0.57676856],[0,13277,0.1171480289],[0,13279,0.2593579641],[0,13281,0.1766598584],[0,13282,0.3703985271],[0,13285,0.1024086001],[0,13286,0.0942392623],[0,13290,0.7041791477],[0,13293,0.7889121948],[0,13294,0.0527051864],[0,13297,0.1724647287],[0,13298,0.0532093147],[0,13299,0.8664496875],[0,13301,0.0417823504],[0,13302,0.0268370374],[0,13308,0.1013796494],[0,13310,0.079846808],[0,13313,0.3673354126],[0,13314,0.167814712],[0,13317,0.1396381544],[0,13319,0.2919730162],[0,13325,0.0910297688],[0,13326,0.03497131],[0,13329,0.252417756],[0,13331,0.0519437944],[0,13335,0.8876320273],[0,13342,0.0371582649],[0,13348,0.0202016687],[0,13351,0.7032655814],[0,13353,0.0393058133],[0,13354,0.225076611],[0,13357,0.1034002238],[0,13361,0.9786786245],[0,13365,0.124362377],[0,13371,0.0548635679],[0,13376,0.1688151237],[0,13381,0.8478602553],[0,13387,0.0555956363],[0,13389,0.0557737253],[0,13397,0.9249403009],[0,13398,0.0343372054],[0,13400,0.0121137946],[0,13406,0.1195216981],[0,13413,0.311916651],[0,13414,0.0361616371],[0,13419,0.1252524969],[0,13420,0.2915178167],[0,13421,0.0152409966],[0,13423,0.0260773062],[0,13425,0.0200246031],[0,13429,0.0198804336],[0,13430,0.0401955195],[0,13431,0.0385183542],[0,13432,0.0200246031],[0,13433,0.0138455693],[0,13435,0.0692255669],[0,13439,0.0200246031],[0,13441,0.1634332495],[0,13442,0.3865576682],[0,13443,0.3967289951],[0,13444,0.3164220866],[0,13445,0.331666837],[0,13446,0.511451091],[0,13447,0.4205669489],[0,13448,0.0702967427],[0,13450,0.0404338226],[0,13451,0.0632436834],[0,13452,0.069852591],[0,13453,0.3220599174],[0,13454,0.0344311978],[0,13455,0.031463557],[0,13456,0.0790873108],[0,13457,0.3472392565],[0,13459,0.0426346871],[0,13461,0.0121137946],[0,13462,0.0257292061],[0,13464,0.2385132855],[0,13465,0.0183168474],[0,13466,0.0360624693],[0,13467,0.1147034746],[0,13468,0.089547386],[0,13470,0.125809841],[0,13472,0.2030400824],[0,13473,0.0122563394],[0,13474,0.0291540537],[0,13475,0.0813163726],[0,13477,0.0245427824],[0,13479,0.0685310773],[0,13483,0.1675759436],[0,13484,0.0121137946],[0,13485,0.1473183178],[0,13486,0.1950252706],[0,13487,0.0985363002],[0,13488,0.0976429422],[0,13489,0.0121137946],[0,13491,0.216396178],[0,13492,0.0533572569],[0,13493,0.1003436881],[0,13495,0.0697984784],[0,13496,0.1244312182],[0,13497,0.0136016433],[0,13498,0.0615247252],[0,13500,0.0200246031],[0,13501,0.0973708692],[0,13503,0.0198804336],[0,13505,0.0121137946],[0,13506,0.0202016687],[0,13508,0.0200246031],[0,13509,0.1540561432],[0,13510,0.1909571127],[0,13511,0.0881421333],[0,13512,0.0651009815],[0,13513,0.2615023764],[0,13515,0.4197306047],[0,13516,0.1914074325],[0,13517,0.0457713638],[0,13519,0.0385238715],[0,13520,0.3813127637],[0,13521,0.0385238715],[0,13523,0.0727218552],[0,13524,0.2219093997],[0,13525,0.1326992499],[0,13526,0.1168727386],[0,13527,0.0349165593],[0,13529,0.1562952954],[0,13530,0.0572062051],[0,13531,0.303329887],[0,13532,0.0786954248],[0,13533,0.0122563394],[0,13535,0.021465538],[0,13536,0.0599953883],[0,13537,0.0730159987],[0,13539,0.2343003033],[0,13541,0.0657123586],[0,13542,0.0139759527],[0,13543,0.328035357],[0,13544,0.0200246031],[0,13545,0.1889708592],[0,13546,0.0528514916],[0,13547,0.2056510725],[0,13550,0.0200246031],[0,13552,0.0121137946],[0,13554,0.3199718074],[0,13556,0.1680969511],[0,13558,0.2188916111],[0,13559,0.0813611697],[0,13561,0.2052766461],[0,13562,0.0264389077],[0,13565,0.1228489766],[0,13566,0.0340306231],[0,13567,0.0762612444],[0,13569,0.0274832566],[0,13570,0.3398021764],[0,13571,0.0790024147],[0,13572,0.018628592],[0,13574,0.0331345874],[0,13575,0.0402189361],[0,13578,0.0200246031],[0,13579,0.0314215681],[0,13580,0.0312363963],[0,13581,0.1025203189],[0,13583,0.097006597],[0,13587,0.0471411677],[0,13588,0.0200246031],[0,13589,0.0409974164],[0,13592,0.0700948993],[0,13593,0.0877330742],[0,13594,0.3054416172],[0,13595,0.4725901723],[0,13596,0.2043045705],[0,13597,0.09505151],[0,13599,0.1489490778],[0,13602,0.0953257944],[0,13603,0.1382542972],[0,13605,0.0528804988],[0,13606,0.1807271994],[0,13607,0.0391135534],[0,13608,0.0121137946],[0,13609,0.0553972027],[0,13612,0.1244900125],[0,13613,0.2098153497],[0,13614,0.0565294718],[0,13615,0.0650357277],[0,13616,0.0846540288],[0,13617,0.2382682268],[0,13620,0.3410789015],[0,13622,0.0218380347],[0,13623,0.2490803286],[0,13629,0.0530903041],[0,13630,0.1183658305],[0,13631,0.0269576046],[0,13632,0.1010174115],[0,13633,0.0812128718],[0,13634,0.0443727394],[0,13635,0.1713626899],[0,13636,0.0489810752],[0,13637,0.0419680202],[0,13638,0.0546900254],[0,13639,0.4245173247],[0,13640,0.0443452671],[0,13641,0.1173752072],[0,13642,0.0200246031],[0,13643,0.0139759527],[0,13646,0.0215751715],[0,13647,0.2328698129],[0,13649,0.1026614206],[0,13650,0.2007291428],[0,13651,0.0304268954],[0,13654,0.0123185172],[0,13656,0.1670181141],[0,13658,0.1096573823],[0,13659,0.0287376732],[0,13660,0.0464238559],[0,13663,0.0415519961],[0,13664,0.1076259281],[0,13666,0.1298529862],[0,13667,0.0600193747],[0,13668,0.3226833315],[0,13671,0.2232333873],[0,13672,0.1318418587],[0,13673,0.0699703465],[0,13674,0.1094640973],[0,13675,0.0907366892],[0,13680,0.0726017199],[0,13681,0.2491345869],[0,13682,0.0517993311],[0,13789,0.2585270061],[0,13790,0.2012261081],[0,13791,0.228053136],[0,13792,0.3135905299],[0,13793,0.0386833469],[0,13794,0.2980340575],[0,13797,0.3246639147],[0,13799,0.0121137946],[0,13800,0.0938674102],[0,13802,0.2742131913],[0,13803,0.2557245397],[0,13805,0.2420018633],[0,13806,0.0114005787],[0,13807,0.1203226527],[0,13808,0.2359992083],[0,13809,0.1771208537],[0,13810,0.0200246031],[0,13813,0.0577378478],[0,13815,0.1595198672],[0,13816,0.68180569],[0,13818,0.0359344553],[0,13820,0.1530492816],[0,13822,0.4121026826],[0,13823,0.0200246031],[0,13824,0.5479634297],[0,13826,0.0745759657],[0,13827,0.0437611006],[0,13828,0.0522891004],[0,13830,0.0550563868],[0,13831,0.1622067752],[0,13832,0.1979902323],[0,13838,0.0985131778],[0,13839,0.0704526571],[0,13841,0.0550360613],[0,13843,0.0121137946],[0,13844,0.0121137946],[0,13845,0.2106311522],[0,13853,0.0143512814],[0,13862,0.0206967587],[0,13863,0.0149266902],[0,13864,0.0431358457],[0,13865,0.0178040565],[0,13866,0.0361160669],[0,13872,0.0349931353],[0,13877,0.0200246031],[0,13878,0.2173735854],[0,13882,0.0121137946],[0,13883,0.0200246031],[0,13884,0.0646993146],[0,13892,0.0200246031],[0,13893,0.0200246031],[0,13950,0.0444023722],[0,13952,0.2084294313],[0,13955,0.0530105473],[0,13960,0.0626656515],[0,13980,0.029346267],[0,13983,0.0717385171],[0,13996,0.0200246031],[0,14017,0.152576748],[0,14020,0.0214427641],[0,14023,0.0295537372],[0,14027,0.0707496145],[0,14034,0.1220521453],[0,14037,0.0361115689],[0,14057,0.4309402608],[0,14060,0.0239175338],[0,14061,0.0244372666],[0,14076,0.0112930697],[0,14081,0.0317965768],[0,14089,0.1472666883],[0,14095,0.0123185172],[0,14096,0.0737624365],[0,14099,0.347358893],[0,14102,0.1172944621],[0,14103,0.0121137946],[0,14104,0.1764190528],[0,14106,0.0907195064],[0,14107,0.7094629166],[0,14108,0.0242709574],[0,14109,0.290763495],[0,14110,0.0464465339],[0,14111,0.146708122],[0,14115,0.026199593],[0,14116,0.0497962365],[0,14117,0.152808363],[0,14118,0.2759063611],[0,14119,0.5174899833],[0,14120,0.0371687671],[0,14121,0.1215470237],[0,14122,0.0157764346],[0,14123,0.1044915809],[0,14124,0.0202016687],[0,14125,0.2192783129],[0,14126,0.0288482755],[0,14127,0.0824022705],[0,14128,0.2675450734],[0,14130,0.0323053131],[0,14131,0.0200246031],[0,14133,0.2474360204],[0,14134,0.0291540537],[0,14135,0.1820467228],[0,14136,0.0228222303],[0,14138,0.0200246031],[0,14139,0.0156382594],[0,14140,0.1072431967],[0,14141,0.0248904101],[0,14142,0.0271689507],[0,14143,0.0863185079],[0,14144,0.0313295049],[0,14145,0.0135074092],[0,14146,0.0693574542],[0,14147,0.0363802078],[0,14148,0.0139759527],[0,14149,0.2351544184],[0,14152,0.0291540537],[0,14155,0.0266200805],[0,14156,0.7245720926],[0,14158,0.0121137946],[0,14159,0.1393046847],[0,14160,0.6574922489],[0,14162,0.1478306803],[0,14163,0.0286297522],[0,14165,0.0466954321],[0,14166,0.1142361219],[0,14168,0.0121137946],[0,14171,0.0290040081],[0,14173,0.2485804327],[0,14176,0.0110588744],[0,14177,0.0466979739],[0,14178,0.0385238715],[0,14179,0.0152409966],[0,14181,0.0877939053],[0,14182,0.0200246031],[0,14185,0.0325483291],[0,14188,0.0200246031],[0,14190,0.0460950359],[0,14191,0.0694940727],[0,14192,0.0124487547],[0,14193,0.0286918111],[0,14194,0.0103765122],[0,14197,0.0156382594],[0,14198,0.2594238432],[0,14199,0.5000992908],[0,14200,0.0139759527],[0,14203,0.1784690027],[0,14205,0.2230181365],[0,14206,0.0314505939],[0,14207,0.0724080893],[0,14210,0.0776522432],[0,14211,0.030552325],[0,14213,0.0448699547],[0,14214,0.0725778427],[0,14215,0.0202016687],[0,14216,0.0450530521],[0,14217,0.0202016687],[0,14219,0.076108654],[0,14220,0.0771085197],[0,14224,0.0163181883],[0,14228,0.1107285157],[0,14230,0.037370501],[0,14231,0.0202016687],[0,14235,0.231534777],[0,14239,0.3577713892],[0,14241,0.7897336762],[0,14242,0.1550921399],[0,14243,0.0324348614],[0,14244,0.1369648773],[0,14245,0.1416676709],[0,14246,0.1443306025],[0,14248,0.0751000281],[0,14249,0.2026782301],[0,14251,0.0649576694],[0,14254,0.0200246031],[0,14255,0.0338288684],[0,14256,0.1182956804],[0,14258,0.0472739063],[0,14259,0.0121137946],[0,14260,0.0457713638],[0,14261,0.3797225698],[0,14262,0.1320746667],[0,14264,0.0200246031],[0,14265,0.0829254247],[0,14268,0.1288048319],[0,14269,0.0152409966],[0,14271,0.0452786318],[0,14273,0.031463557],[0,14275,0.1022736804],[0,14276,0.031463557],[0,14277,0.0121137946],[0,14278,0.1452308077],[0,14279,0.194871271],[0,14280,0.3006385075],[0,14281,0.0354076629],[0,14282,0.0600510852],[0,14283,0.0291540537],[0,14284,0.0632334738],[0,14288,0.2304782777],[0,14290,0.0486133183],[0,14292,0.0166163386],[0,14293,0.0524363167],[0,14294,0.0274238834],[0,14295,0.1018638925],[0,14296,0.0485191974],[0,14297,0.0804735102],[0,14298,0.0320872035],[0,14299,0.0762922343],[0,14300,0.0200246031],[0,14669,0.2806309387],[0,14671,0.0202016687],[0,14674,0.1501383039],[0,14675,0.4390707934],[0,14676,0.0606637281],[0,14678,0.1522385015],[0,14679,0.2076931907],[0,14681,0.0866074124],[0,14682,0.0335156366],[0,14683,0.127225429],[0,14684,0.0275406907],[0,14686,0.0121137946],[0,14687,0.2829306763],[0,14694,0.0702775453],[0,14695,0.0374490343],[0,14698,0.1887075195],[0,14699,0.0382959826],[0,14700,0.0482465843],[0,14702,0.0200246031],[0,14703,0.0613039124],[0,14706,0.0121137946],[0,14707,0.3119622521],[0,14708,0.1782400368],[0,14709,0.0664614427],[0,14710,0.0121137946],[0,14711,0.066386362],[0,14712,0.1789424345],[0,14713,0.0431358457],[0,14714,0.287561092],[0,14715,0.0727440872],[0,14716,0.1612260637],[0,14717,0.3200888512],[0,14719,0.0659722829],[0,14720,0.279338598],[0,14721,0.0200246031],[0,14722,0.158176032],[0,14723,0.0200246031],[0,14724,0.3419078123],[0,14726,0.2027200016],[0,14727,0.3790119273],[0,14728,0.1156304346],[0,14729,0.2316148299],[0,14730,0.0121137946],[0,14731,0.0158464219],[0,14732,0.0395189364],[0,14733,0.2701247612],[0,14734,0.3782658367],[0,14735,0.3313856891],[0,14737,0.057121872],[0,14740,0.0878148679],[0,14741,0.1816659341],[0,14744,0.3634447282],[0,14745,0.3165997704],[0,14747,0.0369644476],[0,14748,0.1613197919],[0,14751,0.0378166533],[0,14753,0.1389278241],[0,14754,0.0700923645],[0,14757,0.3688731667],[0,14759,0.0121137946],[0,14761,0.0200246031],[0,14762,0.1151802236],[0,14763,0.5404524499],[0,14764,0.0210889278],[0,14768,0.2336269971],[0,14769,0.0777723153],[0,14770,0.1420469021],[0,14774,0.3540515927],[0,14777,0.3390701788],[0,14778,0.0410319179],[0,14779,0.0700326586],[0,14780,0.258280471],[0,14781,0.041531401],[0,14783,0.1210842568],[0,14784,0.0280890798],[0,14785,0.0354547019],[0,14786,0.0337779354],[0,14787,0.0481210261],[0,14789,0.0121137946],[0,14790,0.0962068393],[0,14793,0.2923634575],[0,14794,0.0517543041],[0,14795,0.0555956363],[0,14796,0.0200246031],[0,14799,0.0121137946],[0,14800,0.0722196585],[0,14805,0.0330923002],[0,14806,0.0206967587],[0,14807,0.0376299917],[0,14809,0.0200246031],[0,14812,0.0286842055],[0,14813,0.1169516367],[0,14814,0.3568024793],[0,14815,0.0193818418],[0,14816,0.0804012555],[0,14824,0.0347101521],[0,14828,0.0534927569],[0,14829,0.0491737518],[0,14830,0.053616394],[0,14834,0.0404284884],[0,14835,0.0262308483],[0,14840,0.1753438568],[0,14843,0.0415999349],[0,14844,0.022844339],[0,14845,0.0430423671],[0,14846,0.0385238715],[0,14851,0.0241088735],[0,14853,0.0351475823],[0,14854,0.0603094156],[0,14856,0.0491616431],[0,14861,0.0722954074],[0,14864,0.0385238715],[0,14865,0.3043652814],[0,14871,0.0442436917],[0,14875,0.1992070966],[0,14876,0.0613161399],[0,14879,0.2277726408],[0,14880,0.0404428242],[0,14881,0.0112930697],[0,14883,0.0561516984],[0,14885,0.04789497],[0,14892,0.0251687279],[0,14898,0.0112930697],[0,14899,0.0514674066],[0,14900,0.0166960344],[0,14901,0.0309528086],[0,14903,0.0938982119],[0,14904,0.0472315187],[0,14907,0.1298529862],[0,14911,0.0370681159],[0,14917,0.0961868929],[0,14920,0.0121137946],[0,14922,0.0504084071],[0,14926,0.3632277795],[0,14929,0.0905529051],[0,14933,0.0353565165],[0,14934,0.1474151953],[0,14939,0.0123185172],[0,14940,0.0317965768],[0,14943,0.0525634259],[0,14945,0.0268292261],[0,14946,0.0755644689],[0,14947,0.0270970269],[0,14948,0.0545628118],[0,14949,0.0214427641],[0,14950,0.0133753906],[0,14953,0.250512431],[0,14955,0.4686488473],[0,14956,0.3630975035],[0,14957,0.0200246031],[0,14958,0.0652292189],[0,14961,0.0200246031],[0,14962,0.1190654169],[0,14963,0.0183943026],[0,14964,0.0371523518],[0,14965,0.0496899932],[0,14967,0.1663711457],[0,14968,0.2557245397],[0,14971,0.4342548625],[0,14973,0.0395344287],[0,14977,0.2568463525],[0,14979,0.0315965552],[0,14980,0.3037860103],[0,14984,0.0598903778],[0,14985,0.0477107234],[0,14988,0.0367531982],[0,14991,0.0112930697],[0,14992,0.0200246031],[0,14993,0.0644955761],[0,14994,0.0239175338],[0,14995,0.0687175329],[0,14998,0.5965351999],[0,14999,0.0121137946],[0,15000,0.0235776815],[0,15001,0.1384409304],[0,15002,0.0407878725],[0,15006,0.0898730219],[0,15010,0.0223767856],[0,15011,0.0121137946],[0,15012,0.1309013689],[0,15014,0.2681550753],[0,15015,0.3345464234],[0,15021,0.0291540537],[0,15022,0.3891556171],[0,15023,0.025923475],[0,15027,0.0285146914],[0,15028,0.2883989675],[0,15029,0.030159191],[0,15030,0.3553121235],[0,15031,0.0449750193],[0,15034,0.010035322],[0,15037,0.4914460726],[0,15038,0.1382627157],[0,15040,0.1057074792],[0,15042,0.0792458873],[0,15044,0.0274220664],[0,15047,0.0284300582],[0,15049,0.0122563394],[0,15050,0.0393105555],[0,15051,0.0719031122],[0,15053,0.0200246031],[0,15060,0.0195896768],[0,15063,0.0271638342],[0,15065,0.4514677646],[0,15227,0.0423263532],[0,15235,0.1116778127],[0,15237,0.040198824],[0,15245,0.0200246031],[0,15251,0.0200246031],[0,15264,0.0202016687],[0,15272,0.0200246031],[0,15274,0.0152409966],[0,15275,0.0200246031],[0,15278,0.0534124407],[0,15281,0.0628659479],[0,15283,0.269410673],[0,15286,0.2174504565],[0,15287,0.0200246031],[0,15293,0.028126589],[0,15296,0.3205077351],[0,15300,0.0424991623],[0,15301,0.096959649],[0,15302,0.0200246031],[0,15304,0.0245155778],[0,15305,0.0200246031],[0,15310,0.0154583603],[0,15311,0.0551575567],[0,15312,0.0121137946],[0,15321,0.0152409966],[0,15322,0.3175636452],[0,15323,0.0200246031],[0,15414,0.0200246031],[0,15418,0.0121137946],[0,15419,0.027411708],[0,15424,0.0121137946],[0,15426,0.0152409966],[0,15433,0.027411708],[0,15435,0.1210095966],[0,15437,0.0351772363],[0,15439,0.0308838989],[0,15440,0.0595267111],[0,15441,0.1692689673],[0,15442,0.0200246031],[0,15445,0.05622562],[0,15446,0.0508985863],[0,15448,0.0202016687],[0,15450,0.0452786318],[0,15451,0.0121137946],[0,15452,0.0262196145],[0,15455,0.0310735863],[0,15456,0.0385238715],[0,15460,0.2361094217],[0,15461,0.0249962051],[0,15462,0.0352850042],[0,15464,0.0200246031],[0,15465,0.0152409966],[0,15466,0.0202016687],[0,15467,0.0252894372],[0,15470,0.1802259584],[0,15471,0.2719831065],[0,15472,0.0385238715],[0,15473,0.0948861369],[0,15475,0.0152409966],[0,15476,0.0452786318],[0,15478,0.1826940946],[0,15479,0.0465525816],[0,15480,0.0301464026],[0,15481,0.0296459424],[0,15482,0.0206753037],[0,15484,0.0385238715],[0,15485,0.028712373],[0,15486,0.0320335301],[0,15487,0.0592638822],[0,15488,0.0121137946],[0,15489,0.031463557],[0,15490,0.0235565797],[0,15491,0.0784375303],[0,15492,0.053379666],[0,15493,0.1494930114],[0,15494,0.0514246618],[0,15495,0.0137715646],[0,15496,0.0200246031],[0,15497,0.0155074125],[0,15498,0.0155074125],[0,15499,0.0313612917],[0,15501,0.0152409966],[0,15502,0.0324235822],[0,15503,0.111614322],[0,15504,0.0202016687],[0,15505,0.0694081872],[0,15507,0.0184013567],[0,15508,0.2679197809],[0,15509,0.1388722653],[0,15510,0.0121137946],[0,15511,0.0845219771],[0,15512,0.045506086],[0,15513,0.2341737459],[0,15515,0.0254895911],[0,15516,0.0540612112],[0,15518,0.0351416219],[0,15519,0.0137715646],[0,15520,0.0485326627],[0,15521,0.0367786827],[0,15522,0.0157722004],[0,15525,0.0233935865],[0,15527,0.0385238715],[0,15528,0.123601259],[0,15529,0.0112930697],[0,15531,0.1826095026],[0,15532,0.0666721438],[0,15533,0.0347101521],[0,15534,0.0313755743],[0,15535,0.0200246031],[0,15536,0.4782797616],[0,15537,0.0537281712],[0,15538,0.0291732246],[0,15539,0.0677939034],[0,15540,0.0370681159],[0,15541,0.0218924981],[0,15542,0.018980951],[0,15543,0.0114005787],[0,15544,0.0200246031],[0,15546,0.051025192],[0,15547,0.0200246031],[0,15548,0.0371794203],[0,15549,0.0495836033],[0,15550,0.027492796],[0,15551,0.0571683259],[0,15552,0.0336276977],[0,15553,0.0209054322],[0,15554,0.0192514133],[0,15555,0.0452786318],[0,15556,0.0452786318],[0,15557,0.0141255938],[0,15558,0.010035322],[0,15559,0.0121137946],[0,15560,0.1687166899],[0,15561,0.0291540537],[0,15562,0.0452786318],[0,15563,0.0274658524],[0,15566,0.0258265407],[0,15567,0.1281068608],[0,15568,0.0573486781],[0,15571,0.0200246031],[0,15572,0.1723763228],[0,15573,0.023843425],[0,15575,0.2952995279],[0,15577,0.0446370923],[0,15580,0.0311225181],[0,15581,0.0403937724],[0,15583,0.0299129644],[0,15584,0.0353565165],[0,15585,0.0480315351],[0,15586,0.1179890104],[0,15587,0.1165368024],[0,15588,0.0274238834],[0,15589,0.0295537372],[0,15592,0.0167002743],[0,15593,0.0773951209],[0,15594,0.0202016687],[0,15595,0.0592482314],[0,15596,0.0122563394],[0,15597,0.0123185172],[0,15598,0.0200246031],[0,15599,0.0200246031],[0,15601,0.0289443861],[0,15602,0.0200246031],[0,15603,0.0200246031],[0,15605,0.0272546575],[0,15606,0.0340306231],[0,15607,0.0121137946],[0,15608,0.0155544475],[0,15609,0.0406886152],[0,15610,0.0302029337],[0,15611,0.0285319437],[0,15612,0.0288420724],[0,15613,0.0200246031],[0,15614,0.029346267],[0,15616,0.0380325983],[0,15618,0.3072392848],[0,15619,0.0150698234],[0,15677,0.0853327472],[0,15731,0.0200246031],[0,15768,0.0827985775],[0,15773,0.0380785247],[0,15806,0.011712891],[0,15813,0.0645791068],[0,15856,0.0441584002],[0,15871,0.0530105473],[0,15904,0.0635369184],[0,15964,0.1971343702],[0,15969,0.0350900285],[0,16010,0.1003932291],[0,16038,0.0483013779],[0,16070,0.1613887584],[0,16092,0.059676233],[0,16116,0.073743644],[0,16122,0.0571601099],[0,16141,0.126026508],[0,16146,0.1615180613],[0,16227,0.0919289888],[0,16287,0.2335493249],[0,16288,0.2932410543],[0,16333,0.0241607738],[0,16335,0.1542710176],[0,16349,0.3128919169],[0,16372,0.0435182084],[0,16398,0.0133753906],[0,16420,0.0967946459],[0,16455,0.0200246031],[0,16458,0.0386291693],[0,16567,0.1950346965],[0,16599,0.0903292377],[0,16701,0.0628423779],[0,16710,0.022716164],[0,16711,0.1300075937],[0,16713,0.0200246031],[0,16714,0.0200246031],[0,16715,0.0121137946],[0,16716,0.0362303309],[0,16718,0.0535403951],[0,16719,0.3153175815],[0,16720,0.0487990588],[0,16721,0.0200246031],[0,16724,0.0583113156],[0,16726,0.0924801061],[0,16728,0.0452786318],[0,16729,0.0295703902],[0,16730,0.039851174],[0,16732,0.0202016687],[0,16733,0.0352540756],[0,16734,0.0788989469],[0,16737,0.0200246031],[0,16739,0.0309591246],[0,16741,0.0164308894],[0,16742,0.1280170962],[0,16747,0.0715188375],[0,16749,0.2153535556],[0,16753,0.0385238715],[0,16755,0.0274220664],[0,16756,0.0291540537],[0,16759,0.1804570982],[0,16761,0.0258217484],[0,16762,0.0412215765],[0,16763,0.0452786318],[0,16764,0.0202016687],[0,16769,0.0291540537],[0,16774,0.0274703073],[0,16775,0.0318307966],[0,16776,0.0313174382],[0,16777,0.0426240934],[0,16778,0.0200246031],[0,16780,0.0411754416],[0,16781,0.0259940179],[0,16782,0.0123185172],[0,16787,0.0416680952],[0,16788,0.0417801044],[0,16789,0.0167965357],[0,16792,0.0202016687],[0,16793,0.1806181923],[0,16796,0.0200246031],[0,16797,0.0167002743],[0,16799,0.0297300587],[0,16800,0.0200246031],[0,16802,0.146921324],[0,16806,0.0439989437],[0,16811,0.1095002492],[0,16812,0.0342882363],[0,16815,0.2824812646],[0,16818,0.0121137946],[0,16822,0.0259302059],[0,16823,0.0115186322],[0,16824,0.2152294348],[0,16826,0.0200246031],[0,16829,0.0216943309],[0,16831,0.0779385218],[0,16832,0.0385238715],[0,16836,0.0200246031],[0,16837,0.025923475],[0,16838,0.0544434763],[0,16839,0.0152409966],[0,16840,0.0233134438],[0,16842,0.0121137946],[0,16843,0.20705357],[0,16844,0.0278006902],[0,16847,0.1656023573],[0,16849,0.0524261084],[0,16853,0.0139759527],[0,16854,0.4697315652],[0,16855,0.0351475823],[0,16856,0.0720011172],[0,16864,0.0322643902],[0,16865,0.1751881274],[0,16866,0.0385238715],[0,16867,0.14838925],[0,16868,0.0228245858],[0,16870,0.0323391015],[0,16872,0.0266200805],[0,16876,0.3743075927],[0,16877,0.0343075195],[0,16880,0.0385238715],[0,16881,0.0357431713],[0,16884,0.0990021502],[0,16885,0.0590337832],[0,16886,0.1315879788],[0,16888,0.012743931],[0,16891,0.0261162576],[0,16893,0.0220897455],[0,16894,0.0209438346],[0,16895,0.0594711937],[0,16900,0.1339419689],[0,16901,0.0711827236],[0,16902,0.0202016687],[0,16904,0.0342359972],[0,16905,0.0360689107],[0,16907,0.0339120832],[0,16908,0.1364297299],[0,16909,0.0285188894],[0,16910,0.0612322363],[0,16911,0.017317106],[0,16913,0.0534124407],[0,16914,0.1298529862],[0,16915,0.0177296229],[0,16916,0.0289443861],[0,16917,0.0399716843],[0,16918,0.0200246031],[0,16919,0.0200246031],[0,16920,0.1807360594],[0,16921,0.0239607389],[0,16922,0.0820551388],[0,16923,0.0845368195],[0,16925,0.0110588744],[0,16926,0.0200246031],[0,16927,0.0134711855],[0,16928,0.0214401499],[0,16929,0.0205189291],[0,16930,0.0666611838],[0,16931,0.0489040399],[0,16932,0.03872243],[0,16933,0.0241607738],[0,16934,0.0324169868],[0,16935,0.2670779181],[0,16936,0.0152409966],[0,16937,0.0200246031],[0,16938,0.071448366],[0,16939,0.026483787],[0,16940,0.0200246031],[0,16941,0.1062109252],[0,16942,0.092707098],[0,16943,0.0344308043],[0,16945,0.0372288898],[0,16946,0.0152409966],[0,16948,0.1704519434],[0,16949,0.089278732],[0,16950,0.1930038022],[0,16951,0.0229322144],[0,16952,0.0269576046],[0,16953,0.1518392763],[0,16954,0.06883317],[0,16955,0.1557053128],[0,16956,0.081842214],[0,16957,0.0955725955],[0,16959,0.0123185172],[0,16960,0.0343075195],[0,16961,0.0200246031],[0,16962,0.2632945307],[0,16964,0.0200246031],[0,16965,0.0184013567],[0,16966,0.0320781205],[0,16968,0.1566049117],[0,16969,0.0264265298],[0,16970,0.2035737983],[0,16972,0.1273116653],[0,16973,0.1012494902],[0,16974,0.0334280706],[0,16975,0.10200408],[0,16977,0.0564718498],[0,16978,0.0121137946],[0,16979,0.0276546325],[0,16988,0.0200246031],[0,16989,0.0176326205],[0,16990,0.0121137946],[0,16991,0.2830643087],[0,16993,0.0121137946],[0,16994,0.0202016687],[0,17002,0.0200246031],[0,17008,0.0431358457],[0,17019,0.0259725408],[0,17023,0.0200246031],[0,17027,0.2735802381],[0,17030,0.0655591971],[0,17037,0.0634265887],[0,17044,0.0200246031],[0,17051,0.130629333],[0,17053,0.0152409966],[0,17055,0.0200246031],[0,17056,0.0424991623],[0,17060,0.4559507009],[0,17064,0.0324005522],[0,17068,0.0245014697],[0,17071,0.0836024061],[0,17074,0.1895167391],[0,17075,0.0246474195],[0,17079,0.0152409966],[0,17084,0.0925568777],[0,17087,0.0200246031],[0,17089,0.0200246031],[0,17090,0.2473492238],[0,17094,0.0200246031],[0,17097,0.0790178662],[0,17102,0.0205256746],[0,17105,0.0367893844],[0,17106,0.0202016687],[0,17107,0.0202016687],[0,17111,0.0380088585],[0,17115,0.0552465549],[0,17120,0.0200246031],[0,17125,0.0200246031],[0,17129,0.1886974163],[0,17137,0.0518261661],[0,17147,0.0124560287],[0,17158,0.0238879947],[0,17174,0.0121137946],[0,17175,0.0169672581],[0,17176,0.0248904101],[0,17177,0.0183316886],[0,17178,0.1091398736],[0,17179,0.0261162576],[0,17180,0.0888119915],[0,17181,0.014428189],[0,17182,0.047685331],[0,17183,0.0260194903],[0,17185,0.1580744701],[0,17186,0.0200246031],[0,17187,0.0121137946],[0,17188,0.0277260767],[0,17190,0.0200246031],[0,17191,0.0121137946],[0,17192,0.0200246031],[0,17193,0.0200246031],[0,17194,0.1195738372],[0,17197,0.0200246031],[0,17198,0.1479381459],[0,17199,0.2402561],[0,17201,0.065548689],[0,17203,0.0295408094],[0,17204,0.068224585],[0,17205,0.0200246031],[0,17207,0.0350900285],[0,17208,0.0124420813],[0,17210,0.0771143185],[0,17211,0.0389984718],[0,17212,0.0122563394],[0,17213,0.0254895911],[0,17214,0.3051484864],[0,17215,0.0624872639],[0,17216,0.0166960344],[0,17217,0.0373250633],[0,17218,0.0480222553],[0,17219,0.0318210257],[0,17221,0.0200246031],[0,17223,0.0200246031],[0,17224,0.2169749328],[0,17226,0.0200246031],[0,17227,0.2322751979],[0,17228,0.0319384023],[0,17229,0.1480683304],[0,17231,0.0121137946],[0,17232,0.0202016687],[0,17233,0.0479672624],[0,17234,0.0452786318],[0,17235,0.0409686771],[0,17236,0.1300075937],[0,17237,0.0274834755],[0,17238,0.1653653399],[0,17239,0.0200246031],[0,17240,0.035638973],[0,17241,0.1039267412],[0,17242,0.0820826687],[0,17243,0.0200246031],[0,17245,0.1209274943],[0,17246,0.0366329947],[0,17247,0.0152409966],[0,17248,0.0200246031],[0,17249,0.0363259754],[0,17250,0.0451877812],[0,17252,0.0505494109],[0,17253,0.1174048127],[0,17255,0.0323391015],[0,17256,0.0376299917],[0,17257,0.0267165245],[0,17258,0.0200246031],[0,17345,0.1629526719],[0,17346,0.0200246031],[0,17347,0.060865649],[0,17348,0.0593614033],[0,17350,0.2210092473],[0,17351,0.0133753906],[0,17352,0.0640537701],[0,17353,0.3523316738],[0,17354,0.2518263205],[0,17355,0.029974969],[0,17356,0.1131308422],[0,17357,0.0985394479],[0,17358,0.0621566608],[0,17359,0.1008766019],[0,17360,0.0456249747],[0,17361,0.1475770387],[0,17362,0.0121137946],[0,17363,0.2634809861],[0,17366,0.0447689443],[0,17367,0.1358034379],[0,17368,0.0200246031],[0,17370,0.3046228025],[0,17371,0.1538445133],[0,17372,0.0840355929],[0,17373,0.1591060347],[0,17374,0.0200246031],[0,17375,0.0121137946],[0,17377,0.0121137946],[0,17378,0.1948524202],[0,17379,0.0672967799],[0,17380,0.2451616498],[0,17381,0.0487792634],[0,17390,0.0123185172],[0,17391,0.0342859121],[0,17392,0.0478754525],[0,17398,0.1473243366],[0,17405,0.0702136303],[0,17413,0.0381517999],[0,17415,0.5635830186],[0,17424,0.0123185172],[0,17425,0.0381517999],[0,17429,0.1173645679],[0,17430,0.0121137946],[0,17431,0.5079805661],[0,17434,0.0200246031],[0,17436,0.0200246031],[0,17437,0.0245155778],[0,17449,0.0123185172],[0,17455,0.0214092302],[0,17460,0.0202016687],[0,17461,0.0399813371],[0,17462,0.0313538688],[0,17463,0.2532174439],[0,17466,0.0228245858],[0,17467,0.0516422897],[0,17468,0.0152409966],[0,17469,0.0267183149],[0,17470,0.0325107032],[0,17471,0.0531953865],[0,17472,0.0618321667],[0,17474,0.0134711855],[0,17476,0.0370533425],[0,17477,0.0648316274],[0,17478,0.0386291693],[0,17480,0.0284718165],[0,17481,0.0697465626],[0,17482,0.5101739521],[0,17483,0.0741795312],[0,17484,0.0200246031],[0,17485,0.0167002743],[0,17487,0.044267155],[0,17488,0.0669291393],[0,17489,0.0159041298],[0,17490,0.056817392],[0,17491,0.0423517931],[0,17492,0.0154583603],[0,17493,0.0200246031],[0,17494,0.0200246031],[0,17495,0.147160545],[0,17497,0.0584488774],[0,17498,0.0766821263],[0,17499,0.0149266902],[0,17500,0.0301816589],[0,17501,0.0152409966],[0,17502,0.1591527741],[0,17503,0.0123185172],[0,17504,0.0397967156],[0,17505,0.0452786318],[0,17506,0.0342537697],[0,17507,0.2605765732],[0,17508,0.0266903332],[0,17509,0.0495056171],[0,17510,0.0200246031],[0,17513,0.1042919677],[0,17514,0.0268994831],[0,17515,0.0301357693],[0,17516,0.0362304796],[0,17519,0.0200246031],[0,17521,0.2160574653],[0,17522,0.031463557],[0,17526,0.0200246031],[0,17528,0.1193750931],[0,17531,0.0121137946],[0,17540,0.0607991127],[0,17541,0.061274307],[0,17543,0.1256784303],[0,17553,0.0415405777],[0,17556,0.0121137946],[0,17560,0.0359939019],[0,17563,0.0121137946],[0,17564,0.2797390467],[0,17567,0.0450288349],[0,17587,0.0594335688],[0,17594,0.0200246031],[0,17599,0.1806675406],[0,17602,0.0176326205],[0,17605,0.0927087632],[0,17606,0.0544652844],[0,17626,0.1337373377],[0,17640,0.0121137946],[0,17645,0.0152409966],[0,17659,0.0166163386],[0,17661,0.2934138444],[0,17666,0.0753659587],[0,17668,0.0121137946],[0,17671,0.1073665333],[0,17672,0.2985694187],[0,17679,0.0400851174],[0,17683,0.0790084419],[0,17689,0.0200246031],[0,17707,0.0152409966],[0,17710,0.0200246031],[0,17711,0.3395751473],[0,17721,0.0288013746],[0,17722,0.0710805581],[0,17723,0.0200246031],[0,17726,0.0509601302],[0,17727,0.012743931],[0,17729,0.0167002743],[0,17731,0.021874643],[0,17732,0.5612564647],[0,17737,0.0121137946],[0,17740,0.0200246031],[0,17743,0.0238424379],[0,17748,0.0145241316],[0,17749,0.1784831587],[0,17756,0.0983615936],[0,17757,0.096317733],[0,17759,0.1767549607],[0,17760,0.0608930931],[0,17761,0.1625323481],[0,17762,0.0200246031],[0,17763,0.0606399393],[0,17764,0.1771148923],[0,17765,0.0200246031],[0,17766,0.0121137946],[0,17767,0.0200246031],[0,17768,0.1308021518],[0,17769,0.1795067513],[0,17770,0.0455467399],[0,17771,0.0523442027],[0,17772,0.0364996621],[0,17774,0.0401182223],[0,17775,0.2161994028],[0,17777,0.2906186921],[0,17780,0.0568396576],[0,17783,0.0363259754],[0,17785,0.1129237183],[0,17790,0.0200246031],[0,17791,0.0984662396],[0,17793,0.0894302269],[0,17795,0.0460625479],[0,17798,0.1850813274],[0,17810,0.0200246031],[0,17815,0.0407600857],[0,17830,0.2200717822],[0,17833,0.2100091064],[0,17835,0.1145100722],[0,17848,0.0733689031],[0,17849,0.0696354707],[0,17866,0.0916149027],[0,17874,0.1919472284],[0,17876,0.288292244],[0,17878,0.0200246031],[0,17881,0.0678573204],[0,17889,0.096458775],[0,17901,0.2449097174],[0,17903,0.203104302],[0,17904,0.0569706667],[0,17907,0.0333509127],[0,17908,0.054565464],[0,17909,0.0200246031],[0,17916,0.0200246031],[0,17925,0.1427815964],[0,17929,0.072824838],[0,17940,0.0446044158],[0,17941,0.0121137946],[0,17943,0.0902666948],[0,17946,0.0977279295],[0,17947,0.0539835316],[0,17948,0.0200246031],[0,17951,0.0123484086],[0,17953,0.0133753906],[0,17954,0.0424991623],[0,17955,0.1114846594],[0,17957,0.1634405477],[0,17958,0.0774281931],[0,17961,0.0250889582],[0,17962,0.0187948868],[0,17964,0.0135664056],[0,17966,0.0626705514],[0,17970,0.2108644883],[0,17971,0.0133753906],[0,17982,0.0152409966],[0,17995,0.139344192],[0,18015,0.1167926547],[0,18034,0.1840313248],[0,18039,0.0200246031],[0,18042,0.0420667698],[0,18068,0.0200246031],[0,18094,0.0327761133],[0,18097,0.1189481051],[0,18100,0.0317270785],[0,18115,0.0386832284],[0,18117,0.0202016687],[0,18124,0.0202016687],[0,18140,0.0317030158],[0,18146,0.1987336774],[0,18153,0.0324235822],[0,18201,0.0121137946],[0,18227,0.0105779531],[0,18239,0.2244607258],[0,18241,0.0105779531],[0,18249,0.0275798035],[0,18251,0.0271027604],[0,18263,0.025987916],[0,18297,0.0379237607],[0,18300,0.0184709307],[0,18301,0.0524700529],[0,18309,0.0105779531],[0,18376,0.0184709307],[0,18407,0.0184709307],[0,18409,0.2673650895],[0,18413,0.0118367055],[0,18417,0.1060929297],[0,18419,0.2302864471],[0,18421,0.1280738254],[0,18430,0.0184709307],[0,18468,0.0386268016],[0,18469,0.0406261094],[0,18478,0.2353809088],[0,18480,0.2226422258],[0,18481,0.08302825],[0,18486,0.0356777717],[0,18487,0.0184709307],[0,18504,0.0222467203],[0,18530,0.3717815901],[0,18533,0.0184709307],[0,18534,0.0200078889],[0,18535,0.0509031977],[0,18536,0.0431097166],[0,18538,0.3668788746],[0,18539,0.0493367291],[0,18540,0.3035332239],[0,18543,0.0533340187],[0,18545,0.2956234749],[0,18546,0.0583154555],[0,18547,0.1602178876],[0,18548,0.0935786494],[0,18550,0.030644437],[0,18551,0.3143185063],[0,18557,0.1065734252],[0,18562,0.0198106542],[0,18566,0.089684826],[0,18572,0.0184709307],[0,18575,0.0105779531],[0,18576,0.0623437128],[0,18580,0.2021972842],[0,18582,0.0979191869],[0,18583,0.1616125832],[0,18585,0.0983027079],[0,18587,0.3439893134],[0,18592,0.0184709307],[0,18596,0.1821767535],[0,18603,0.2177037499],[0,18605,0.0304353569],[0,18608,0.1208009482],[0,18610,0.043668407],[0,18617,0.0351701875],[0,18619,0.068454963],[0,18621,0.2224401732],[0,18622,0.0219381863],[0,18623,0.0872152982],[0,18626,0.0105779531],[0,18627,0.0184709307],[0,18634,0.3799916652],[0,18635,0.0649282395],[0,18638,0.0660156238],[0,18639,0.1339156386],[0,18640,0.1496799431],[0,18644,0.0778199662],[0,18646,0.6222566024],[0,18654,0.0639059157],[0,18659,0.3699919281],[0,18665,0.0386316991],[0,18666,0.0184709307],[0,18668,0.1416839585],[0,18679,0.0184709307],[0,18681,0.0500943906],[0,18691,0.0233191631],[0,18692,0.0647381587],[0,18699,0.1386225577],[0,18700,0.1049208199],[0,18714,0.1488445724],[0,18716,0.2261654791],[0,18717,0.0200180041],[0,18721,0.0356869798],[0,18725,0.0184709307],[0,18735,0.0639304111],[0,18736,0.2031958925],[0,18737,0.1746377468],[0,18740,0.1555004567],[0,18748,0.0105779531],[0,18749,0.0389492228],[0,18751,0.0294009552],[0,18762,0.1738319311],[0,18764,0.0488884438],[0,18776,0.0949283923],[0,18791,0.0325073782],[0,18794,0.0526321368],[0,18798,0.0522514421],[0,18805,0.0506519535],[0,18808,0.1709597076],[0,18809,0.0105779531],[0,18814,0.0437037758],[0,18822,0.0105779531],[0,18827,0.1527035571],[0,18833,0.2055991421],[0,18836,0.2791836186],[0,18841,0.0692798355],[0,18842,0.0366497391],[0,18843,0.1015895468],[0,18847,0.0783465047],[0,18848,0.2369230266],[0,18850,0.0788091145],[0,18851,0.0152792693],[0,18857,0.0184709307],[0,18858,0.1915879434],[0,18861,0.1140335037],[0,18891,0.0184709307],[0,18894,0.03450317],[0,18899,0.2065485679],[0,18900,0.0271556037],[0,18906,0.0426278925],[0,18908,0.0184709307],[0,18913,0.1880218621],[0,18915,0.0184709307],[0,18917,0.1761799162],[0,18918,0.0184709307],[0,18919,0.1236069412],[0,18920,0.0913859815],[0,18922,0.0648485995],[0,18924,0.1724413283],[0,18925,0.0105779531],[0,18927,0.1301778967],[0,18928,0.2674226649],[0,18930,0.0105779531],[0,18932,0.0105779531],[0,18933,0.0184709307],[0,18934,0.1460444933],[0,18936,0.0184709307],[0,19033,0.0136981064],[0,19038,0.042734619],[0,19041,0.0105779531],[0,19045,0.1061017089],[0,19046,0.2485546397],[0,19051,0.0660088704],[0,19052,0.0186475972],[0,19061,0.0186475972],[0,19068,0.1031145951],[0,19070,0.2636814823],[0,19071,0.1266679326],[0,19072,0.0184709307],[0,19092,0.2341644961],[0,19097,0.0415300805],[0,19098,0.0365572688],[0,19099,0.0184709307],[0,19103,0.4104813045],[0,19104,0.0139149802],[0,19105,0.0186475972],[0,19107,0.0133626782],[0,19109,0.3017187257],[0,19110,0.0118367055],[0,19111,0.0186475972],[0,19112,0.1949285664],[0,19113,0.016851343],[0,19122,0.0489941773],[0,19123,0.0136981064],[0,19125,0.1526854406],[0,19126,0.0308888859],[0,19127,0.093116194],[0,19128,0.0356390737],[0,19129,0.0985571797],[0,19130,0.2065528096],[0,19131,0.0343666166],[0,19133,0.0441013611],[0,19134,0.0822669169],[0,19135,0.0132887258],[0,19141,0.0451553928],[0,19142,0.1756285418],[0,19143,0.0095882405],[0,19145,0.0745640148],[0,19147,0.1122548272],[0,19148,0.067327825],[0,19149,0.0437599734],[0,19150,0.1097973097],[0,19154,0.0565329095],[0,19155,0.0365472179],[0,19161,0.0506519535],[0,19163,0.0247394836],[0,19165,0.0190468821],[0,19168,0.0184709307],[0,19169,0.1345412456],[0,19172,0.0105779531],[0,19173,0.2592245343],[0,19176,0.1226095409],[0,19177,0.0105779531],[0,19178,0.0184709307],[0,19179,0.035956696],[0,19180,0.0609384841],[0,19183,0.066548655],[0,19184,0.212079406],[0,19188,0.0329648592],[0,19189,0.0427926384],[0,19190,0.0661359223],[0,19192,0.0184709307],[0,19195,0.0184709307],[0,19196,0.0912143963],[0,19270,0.083247059],[0,19272,0.0351856144],[0,19281,0.1931171921],[0,19284,0.0398097181],[0,19286,0.5727884749],[0,19287,0.0452119546],[0,19296,0.0184709307],[0,19297,0.0376433643],[0,19301,0.0577860054],[0,19302,0.0588221608],[0,19305,0.0398097181],[0,19310,0.2443836382],[0,19313,0.1052282836],[0,19315,0.057258972],[0,19318,0.0609338281],[0,19319,0.257206745],[0,19320,0.0441013611],[0,19322,0.0136981064],[0,19327,0.0184709307],[0,19328,0.2304029597],[0,19335,0.0478747856],[0,19336,0.1015061007],[0,19339,0.0184709307],[0,19341,0.1589195916],[0,19346,0.0387538645],[0,19351,0.5747165843],[0,19353,0.0784161221],[0,19354,0.072763918],[0,19355,0.0766088537],[0,19356,0.0471507453],[0,19357,0.0530119125],[0,19359,0.1223378431],[0,19361,0.0485603522],[0,19363,0.3047720336],[0,19365,0.0184709307],[0,19379,0.0200180041],[0,19382,0.0151444939],[0,19386,0.0375330483],[0,19389,0.0796313752],[0,19391,0.3561453281],[0,19392,0.0324166708],[0,19393,0.0637696054],[0,19394,0.1447558475],[0,19395,0.0184709307],[0,19398,0.0184709307],[0,19399,0.2840615568],[0,19400,0.0201888292],[0,19401,0.0105779531],[0,19413,0.0150324387],[0,19418,0.0481960851],[0,19421,0.0878788033],[0,19424,0.0184709307],[0,19427,0.0352384511],[0,19428,0.0371214112],[0,19430,0.0803907739],[0,19432,0.0425391496],[0,19433,0.0874804463],[0,19437,0.0879608138],[0,19439,0.0881475293],[0,19440,0.040544376],[0,19442,0.0381520938],[0,19446,0.0498043212],[0,19447,0.0958878417],[0,19449,0.0369420676],[0,19450,0.0485447653],[0,19451,0.0590055068],[0,19452,0.0595635844],[0,19454,0.0641716239],[0,19455,0.0686216441],[0,19458,0.3076109938],[0,19461,0.0733721999],[0,19463,0.0678918824],[0,19464,0.0148088868],[0,19468,0.1824000643],[0,19469,0.1240110483],[0,19471,0.0545106796],[0,19473,0.0351979242],[0,19477,0.0329648592],[0,19481,0.1283143892],[0,19484,0.0686942397],[0,19485,0.0430944597],[0,19486,0.0738791574],[0,19487,0.0379952194],[0,19488,0.0448140981],[0,19489,0.0105779531],[0,19490,0.0105779531],[0,19491,0.0184709307],[0,19492,0.0498043212],[0,19493,0.0105779531],[0,19495,0.1401331713],[0,19496,0.0184709307],[0,19497,0.3872952244],[0,19498,0.0678047233],[0,19499,0.3094074191],[0,19500,0.0668374335],[0,19602,0.0184709307],[0,19608,0.2160342794],[0,19610,0.0973709355],[0,19614,0.3488475019],[0,19617,0.0105779531],[0,19618,0.4241465078],[0,19619,0.0109194159],[0,19622,0.0127783099],[0,19623,0.2047298463],[0,19626,0.1583412921],[0,19627,0.0519008732],[0,19628,0.0425491525],[0,19629,0.0184709307],[0,19640,0.0151540949],[0,19641,0.0493367291],[0,19642,0.2935403592],[0,19643,0.0258588802],[0,19645,0.0184709307],[0,19646,0.0599693599],[0,19647,0.0274679115],[0,19650,0.0762370818],[0,19652,0.0248131779],[0,19665,0.0268513081],[0,19667,0.0335220234],[0,19669,0.0136981064],[0,19671,0.0105779531],[0,19672,0.0189641424],[0,19675,0.0184709307],[0,19677,0.0184709307],[0,19679,0.051715284],[0,19680,0.1017847253],[0,19682,0.1399017014],[0,19683,0.0551807943],[0,19688,0.060954112],[0,19689,0.0565363641],[0,19692,0.0476304921],[0,19693,0.0476304921],[0,19694,0.0186475972],[0,19695,0.0356390737],[0,19696,0.2782596164],[0,19697,0.291457432],[0,19699,0.0714489224],[0,19701,0.0186475972],[0,19702,0.1225504882],[0,19703,0.1129724619],[0,19705,0.0539509872],[0,19706,0.0992969537],[0,19707,0.0105779531],[0,19708,0.0328464989],[0,19710,0.1361412865],[0,19711,0.038021335],[0,19712,0.2007391999],[0,19713,0.0105779531],[0,19715,0.0105779531],[0,19717,0.0887688463],[0,19718,0.0731434212],[0,19719,0.0498043212],[0,19720,0.1558114522],[0,19722,0.0523999109],[0,19723,0.3213535361],[0,19724,0.0390001915],[0,19725,0.3775090918],[0,19731,0.3921453635],[0,19732,0.050406714],[0,19739,0.0799585072],[0,19741,0.0184709307],[0,19742,0.0590875709],[0,19743,0.0441013611],[0,19745,0.060677449],[0,19906,0.0105779531],[0,19907,0.0118367055],[0,19908,0.0184709307],[0,19910,0.1093664545],[0,19911,0.0184709307],[0,19912,0.1566155701],[0,19913,0.0184709307],[0,19914,0.1574731395],[0,19915,0.2153697462],[0,19916,0.1169504808],[0,19917,0.2762478045],[0,19918,0.0367824185],[0,19919,0.0365472179],[0,19922,0.1415547881],[0,19923,0.0136981064],[0,19924,0.0441940763],[0,19925,0.0502085601],[0,19926,0.0184709307],[0,19927,0.0186475972],[0,19928,0.0675131213],[0,19929,0.0620551711],[0,19931,0.0292463588],[0,19941,0.0105779531],[0,19945,0.0329584193],[0,19946,0.0136981064],[0,19947,0.0315577603],[0,19953,0.0295278251],[0,19955,0.0186475972],[0,19956,0.0389593079],[0,19959,0.0220734408],[0,19960,0.0186475972],[0,19966,0.0151540949],[0,19967,0.0105779531],[0,19969,0.0184709307],[0,19971,0.0327867587],[0,19972,0.0105779531],[0,19973,0.0186475972],[0,19976,0.0211401906],[0,19981,0.2606243231],[0,19982,0.0105779531],[0,19984,0.0184709307],[0,19987,0.2083577337],[0,19988,0.0933043317],[0,19990,0.044178761],[0,19991,0.0184709307],[0,19993,0.0105779531],[0,19994,0.0105779531],[0,19995,0.0105779531],[0,19997,0.0437599734],[0,19998,0.0329648592],[0,20001,0.0590063951],[0,20002,0.3791149553],[0,20003,0.0184709307],[0,20005,0.0105779531],[0,20008,0.3132716033],[0,20011,0.0105779531],[0,20013,0.1301724803],[0,20014,0.0184709307],[0,20015,0.3223363656],[0,20017,0.2260968941],[0,20018,0.2355588731],[0,20019,0.177899958],[0,20020,0.0186475972],[0,20022,0.0105779531],[0,20024,0.3213966626],[0,20045,0.0184709307],[0,20095,0.0105779531],[0,20102,0.0105779531],[0,20120,0.0118367055],[0,20139,0.0184709307],[0,20162,0.2959307132],[0,20166,0.0184709307],[0,20172,0.0097327055],[0,20174,0.0088806798],[0,20198,0.0180274777],[0,20200,0.1357399817],[0,20216,0.0152198967],[0,20240,0.171606569],[0,20256,0.1288267295],[0,20305,0.0188696179],[0,20322,0.0152198967],[0,20323,0.0163272259],[0,20324,0.028776718],[0,20325,0.0344526125],[0,20352,0.0771966499],[0,20363,0.0342060692],[0,20369,0.0280891284],[0,20376,0.0127783099],[0,20379,0.0299009831],[0,20384,0.0152198967],[0,20402,0.0184709307],[0,20404,0.0140002875],[0,20408,0.0098671313],[0,20412,0.1523661302],[0,20419,0.0239408756],[0,20425,0.0098663448],[0,20441,0.0185179972],[0,20454,0.0140002875],[0,20473,0.0118367055],[0,20493,0.0325768182],[0,20505,0.0512484594],[0,20528,0.1266254698],[0,20533,0.0321544143],[0,20546,0.0414711692],[0,20555,0.0351327711],[0,20556,0.1979127682],[0,20560,0.0127783099],[0,20568,0.0235238707],[0,20583,0.3110550003],[0,20585,0.0094946754],[0,20587,0.0534392292],[0,20588,0.0461263618],[0,20600,0.0105779531],[0,20632,0.0224378836],[0,20635,0.0151185183],[0,20678,0.0136981064],[0,20693,0.0620955419],[0,20695,0.0094946754],[0,20700,0.2522026629],[0,20732,0.1643911836],[0,20742,0.0323408409],[0,20745,0.0237251216],[0,20748,0.0120272899],[0,20754,0.0179596712],[0,20781,0.0184709307],[0,20824,0.015326395],[0,20839,0.0141900841],[0,20841,0.0916854516],[0,20851,0.08633],[0,20873,0.072293339],[0,20907,0.0105779531],[0,20913,0.0099024924],[0,20918,0.0203356786],[0,20945,0.1899079891],[0,20947,0.0138294762],[0,20964,0.0088806798],[0,20974,0.0322075165],[0,20980,0.0581028801],[0,20983,0.1069689522],[0,20985,0.1041137066],[0,21000,0.0165549749],[0,21022,0.0151540949],[0,21034,0.0257724238],[0,21036,0.3330186304],[0,21037,0.0427153453],[0,21042,0.0609564376],[0,21047,0.0930148726],[0,21101,0.0105553974],[0,21102,0.0256666414],[0,21120,0.054010415],[0,21123,0.1187684584],[0,21125,0.0328464989],[0,21128,0.0184709307],[0,21130,0.3133672549],[0,21136,0.1899643608],[0,21143,0.0223550867],[0,21144,0.0315011891],[0,21145,0.1418214612],[0,21147,0.076423864],[0,21150,0.0184709307],[0,21152,0.2886736012],[0,21153,0.057181838],[0,21158,0.0341947845],[0,21159,0.0184709307],[0,21160,0.4363884048],[0,21161,0.2067447127],[0,21162,0.1057656265],[0,21163,0.0345227448],[0,21164,0.0965344049],[0,21165,0.040544376],[0,21167,0.0352818537],[0,21168,0.1540628616],[0,21169,0.0505073427],[0,21170,0.0338588051],[0,21172,0.0184709307],[0,21173,0.5569860279],[0,21175,0.0488884438],[0,21177,0.1517005149],[0,21189,0.0118367055],[0,21192,0.0332998072],[0,21204,0.0184709307],[0,21235,0.1434442413],[0,21244,0.0518489343],[0,21248,0.0105779531],[0,21249,0.1465622927],[0,21265,0.2352908861],[0,21271,0.1691796033],[0,21301,0.298672727],[0,21305,0.0184709307],[0,21310,0.0564981591],[0,21314,0.1401133288],[0,21325,0.0646019824],[0,21330,0.0887966011],[0,21331,0.0680016231],[0,21333,0.3842710018],[0,21334,0.3631073845],[0,21335,0.3623430796],[0,21338,0.3193937706],[0,21340,0.0094946754],[0,21343,0.0452494994],[0,21344,0.1030776915],[0,21345,0.0269489941],[0,21346,0.0496166915],[0,21347,0.3599170966],[0,21348,0.0105779531],[0,21349,0.0105779531],[0,21353,0.0833929633],[0,21354,0.595980836],[0,21355,0.0829412584],[0,21356,0.3926947551],[0,21368,0.0287537424],[0,21373,0.0457916003],[0,21375,0.0191214584],[0,21376,0.1027954064],[0,21399,0.0136981064],[0,21401,0.1355321445],[0,21412,0.2202590468],[0,21420,0.0105779531],[0,21429,0.0184709307],[0,21433,0.0985737625],[0,21439,0.0675482085],[0,21440,0.0105779531],[0,21532,0.032213638],[0,21584,0.0153455185],[0,21648,0.0290185743],[0,21865,0.0184709307],[0,21870,0.3017187257],[0,21873,0.0250871926],[0,21875,0.0614314288],[0,21880,0.3807546379],[0,21881,0.0136981064],[0,21885,0.4297564932],[0,21889,0.0268315474],[0,21890,0.0184709307],[0,21891,0.0184709307],[0,21892,0.3410438278],[0,21894,0.010290228],[0,21900,0.0120272899],[0,21906,0.1005658085],[0,21907,0.2812659121],[0,21910,0.1128899564],[0,21911,0.0609155654],[0,21915,0.3014848557],[0,21916,0.1794381988],[0,21918,0.0184709307],[0,21921,0.0668301547],[0,21923,0.0489663631],[0,21924,0.0184709307],[0,21927,0.0184709307],[0,21928,0.203837948],[0,21930,0.207369945],[0,21932,0.0489822069],[0,21933,0.1925956451],[0,21934,0.0388944094],[0,21935,0.0397719498],[0,21936,0.0818561546],[0,21938,0.0828913692],[0,21940,0.0184709307],[0,21942,0.1615122508],[0,21944,0.0184709307],[0,21949,0.2081261519],[0,21950,0.0455441133],[0,21957,0.0105779531],[0,21966,0.0865529575],[0,21989,0.0329648592],[0,21991,0.4308011073],[0,22003,0.0674630288],[0,22011,0.0525669899],[0,22018,0.1795791607],[0,22023,0.0184709307],[0,22027,0.0690096917],[0,22031,0.0120272899],[0,22034,0.0316442897],[0,22036,0.0191201646],[0,22040,0.020937435],[0,22043,0.2066359395],[0,22063,0.1519364625],[0,22076,0.0418172676],[0,22077,0.0136981064],[0,22087,0.0105779531],[0,22089,0.1680670912],[0,22097,0.0194099785],[0,22100,0.0105779531],[0,22111,0.0249021856],[0,22115,0.025077202],[0,22116,0.5679859557],[0,22117,0.0182463658],[0,22118,0.0103559654],[0,22121,0.2125315546],[0,22126,0.0184229746],[0,22128,0.114446857],[0,22129,0.3652503068],[0,22130,0.200258392],[0,22139,0.0184229746],[0,22170,0.0794233309],[0,22176,0.0218460213],[0,22187,0.0504361433],[0,22215,0.3195868051],[0,22223,0.2550648242],[0,22242,0.0284535213],[0,22246,0.2678906538],[0,22311,0.0182463658],[0,22321,0.1107105802],[0,22328,0.0184229746],[0,22353,0.280368435],[0,22384,0.0479117013],[0,22435,0.1030135773],[0,22456,0.0182463658],[0,22474,0.057499437],[0,22492,0.0103559654],[0,22497,0.4266427834],[0,22529,0.2785057444],[0,22543,0.0150266261],[0,22570,0.0096445894],[0,22651,0.0184229746],[0,22763,0.0134750999],[0,22765,0.3132833392],[0,22799,0.2002731841],[0,22850,0.0104981425],[0,22865,0.1928423976],[0,22874,0.0234976176],[0,22876,0.0103559654],[0,22878,0.0419627886],[0,22882,0.0182463658],[0,22884,0.1960867391],[0,22886,0.0596426466],[0,22892,0.1599076797],[0,22893,0.0439266642],[0,22895,0.0182463658],[0,22898,0.0184229746],[0,22899,0.0182463658],[0,22900,0.0592451865],[0,22901,0.0669872596],[0,22903,0.0166273069],[0,22906,0.0958871598],[0,22907,0.0184229746],[0,22908,0.1444963535],[0,22909,0.0182463658],[0,22912,0.0182463658],[0,22913,0.057688818],[0,22914,0.0182463658],[0,22990,0.0722767558],[0,22991,0.0462115904],[0,22994,0.0184229746],[0,22995,0.0243223051],[0,23004,0.475597506],[0,23011,0.0131616044],[0,23016,0.2266033329],[0,23018,0.0182463658],[0,23020,0.3559164964],[0,23023,0.0208852716],[0,23024,0.0177329516],[0,23030,0.0184229746],[0,23031,0.1602097851],[0,23048,0.0182463658],[0,23050,0.0182463658],[0,23051,0.0182463658],[0,23053,0.0615391539],[0,23054,0.0182463658],[0,23055,0.2033517694],[0,23056,0.0103559654],[0,23057,0.0103559654],[0,23058,0.0184229746],[0,23059,0.0184229746],[0,23060,0.3800598873],[0,23062,0.0184229746],[0,23064,0.015639598],[0,23067,0.0103559654],[0,23068,0.2259266819],[0,23070,0.0184229746],[0,23072,0.0182463658],[0,23081,0.0405665846],[0,23082,0.0182463658],[0,23093,0.0476962994],[0,23107,0.0134750999],[0,23113,0.2744109014],[0,23147,0.0460258082],[0,23154,0.0134750999],[0,23155,0.0184229746],[0,23236,0.0372082575],[0,23265,0.0134750999],[0,23321,0.0314048292],[0,23338,0.09962419],[0,23374,0.0091034119],[0,23422,0.0103559654],[0,23457,0.3157131316],[0,23461,0.025589144],[0,23468,0.0182463658],[0,23469,0.0182463658],[0,23473,0.2343348385],[0,23475,0.0182463658],[0,23476,0.0401770055],[0,23479,0.1848918763],[0,23481,0.0850899436],[0,23487,0.1374739371],[0,23488,0.0761244132],[0,23489,0.0562627082],[0,23490,0.0103559654],[0,23491,0.0182463658],[0,23492,0.0800236192],[0,23493,0.0322746079],[0,23499,0.0182463658],[0,23501,0.0812733057],[0,23591,0.0122133196],[0,23592,0.1051094907],[0,23595,0.023862283],[0,23601,0.0134750999],[0,23602,0.0103559654],[0,23605,0.0134750999],[0,23611,0.0182463658],[0,23614,0.1623384665],[0,23616,0.0103559654],[0,23617,0.0871455942],[0,23619,0.0456877842],[0,23621,0.2672096828],[0,23624,0.0182463658],[0,23630,0.0103559654],[0,23633,0.0191851069],[0,23636,0.0120094587],[0,23642,0.0792258263],[0,23644,0.038720209],[0,23651,0.0120094587],[0,23657,0.0182463658],[0,23660,0.0103559654],[0,23669,0.0103559654],[0,23671,0.3256902724],[0,23673,0.0103559654],[0,23675,0.0182463658],[0,23677,0.0184229746],[0,23715,0.0182463658],[0,23727,0.2775185657],[0,23748,0.0105601599],[0,23751,0.0184229746],[0,23759,0.0184229746],[0,23782,0.0103559654],[0,23786,0.0256144135],[0,23794,0.0182463658],[0,23811,0.0182463658],[0,23820,0.5957574096],[0,23825,0.0103559654],[0,23829,0.0182463658],[0,23838,0.0182463658],[0,23923,0.0103559654],[0,23931,0.0182463658],[0,23938,0.0962137094],[0,23940,0.0184229746],[0,23957,0.0182463658],[0,23961,0.3254141077],[0,23974,0.0203610515],[0,23975,0.0419088673],[0,23976,0.0273241369],[0,23977,0.0103559654],[0,23979,0.0182463658],[0,23983,0.0134750999],[0,23989,0.0182463658],[0,23993,0.0182463658],[0,24032,0.1577161337],[0,24051,0.0184229746],[0,24055,0.0182463658],[0,24057,0.0103559654],[0,24068,0.0182463658],[0,24070,0.0182463658],[0,24076,0.0103559654],[0,24091,0.0182463658],[0,24102,0.0182463658],[0,24215,0.5463355063],[0,24276,0.0389560703],[0,24283,0.0182463658],[0,24284,0.0277706089],[0,24350,0.0182463658],[0,24352,0.1047856568],[0,24383,0.0564615479],[0,24521,0.3153943967],[0,24529,0.0441391074],[0,24550,0.1401241878],[0,24558,0.0187969174],[0,24582,0.5431075921],[0,24603,0.1946709863],[0,24619,0.1305944704],[0,24643,0.2546797641],[0,24650,0.0187969174],[0,24658,0.1168100425],[0,24662,0.0391476148],[0,24700,0.0122551719],[0,24705,0.0109001986],[0,24726,0.0187969174],[0,24745,0.0187969174],[0,24752,0.0189736676],[0,24757,0.0105601599],[0,24783,0.0187969174],[0,24788,0.0187969174],[0,24808,0.0187969174],[0,24811,0.0189736676],[0,24813,0.0189736676],[0,24819,0.0187969174],[0,24831,0.0140218308],[0,24833,0.0187969174],[0,24835,0.0187969174],[0,24839,0.0187969174],[0,24861,0.0173571207],[0,24898,0.0338929615],[0,24952,0.0454752598],[0,24976,0.0135232784],[0,24982,0.0187969174],[0,24989,0.0187969174],[0,24990,0.0187969174],[0,24992,0.0186530047],[0,25001,0.0187969174],[0,25003,0.0187969174],[0,25006,0.0187969174],[0,25010,0.0187969174],[0,25011,0.0189736676],[0,25012,0.0187969174],[0,25013,0.0109001986],[0,25014,0.0187969174],[0,25015,0.0187969174],[0,25016,0.0109001986],[0,25020,0.0187969174],[0,25026,0.0187969174],[0,25037,0.0187969174],[0,25048,0.3229450619],[0,25064,0.0187969174],[0,25074,0.0187969174],[0,25088,0.0109001986],[0,25098,0.0187969174],[0,25117,0.0187969174],[0,25121,0.0187969174],[0,25126,0.0222536874],[0,25127,0.0512008042],[0,25128,0.0484646121],[0,25133,0.0216066578],[0,25134,0.0109001986],[0,25158,0.0187969174],[0,25160,0.0187969174],[0,25161,0.1459364366],[0,25162,0.0187969174],[0,25173,0.0461586228],[0,25265,0.2298025481],[0,25266,0.0188297179],[0,25267,0.2662748934],[0,25268,0.0271252185],[0,25270,0.0233257291],[0,25272,0.0187969174],[0,25273,0.0796008908],[0,25277,0.0109001986],[0,25280,0.0228916701],[0,25317,0.0100113202],[0,25382,0.3198770068],[0,25415,0.0225774565],[0,25450,0.0190358983],[0,25488,0.0153947232],[0,25490,0.1015457984],[0,25491,0.2371603275],[0,25502,0.0155526991],[0,25504,0.0925615582],[0,25510,0.0187969174],[0,25545,0.0109001986],[0,25552,0.0109001986],[0,25554,0.23277259],[0,25558,0.0253261944],[0,25559,0.0140218308],[0,25565,0.0181867743],[0,25567,0.1310985649],[0,25568,0.0187969174],[0,25579,0.0187969174],[0,25580,0.2884978844],[0,25581,0.0257464907],[0,25592,0.0187969174],[0,25613,0.0109001986],[0,25617,0.4978668184],[0,25621,0.3949237036],[0,25630,0.3161872879],[0,25636,0.0503774808],[0,25642,0.0121595476],[0,25650,0.0187969174],[0,25651,0.0109001986],[0,25652,0.0175907497],[0,25654,0.0187969174],[0,25655,0.0706892957],[0,25658,0.0291787205],[0,25703,0.0247547331],[0,25704,0.0290846031],[0,25705,0.1877068382],[0,25712,0.0121595476],[0,25713,0.0140218308],[0,25714,0.0109001986],[0,25724,0.0189736676],[0,25725,0.0299601641],[0,25753,0.0187969174],[0,25756,0.0171709758],[0,25757,0.0249710195],[0,25758,0.0194678758],[0,25759,0.0468117236],[0,25760,0.0304676351],[0,25761,0.5503308475],[0,25762,0.0760069466],[0,25763,0.1291115205],[0,25772,0.037724453],[0,25777,0.0539038412],[0,25827,0.2778199631],[0,25829,0.0187969174],[0,25830,0.0423991018],[0,25851,0.0189736676],[0,25862,0.0187969174],[0,25870,0.2157123784],[0,25871,0.0187969174],[0,25872,0.0142130266],[0,25873,0.1353465154],[0,25874,0.0739796473],[0,25875,0.0096807253],[0,25879,0.0109001986],[0,25881,0.0548566128],[0,25911,0.0143618029],[0,25959,0.3026412138],[0,25963,0.0323477566],[0,25964,0.0537260236],[0,25969,0.0338929615],[0,25970,0.1238193851],[0,25972,0.2583982192],[0,25974,0.3688416488],[0,25977,0.0523421877],[0,25978,0.0397457369],[0,25981,0.0176953431],[0,25982,0.375468019],[0,25984,0.0261708653],[0,25989,0.0187969174],[0,25993,0.0178877824],[0,26000,0.0187969174],[0,26029,0.0944538924],[0,26037,0.418550176],[0,26046,0.3428479014],[0,26062,0.0136122561],[0,26065,0.0460491089],[0,26073,0.0230899739],[0,26150,0.0109001986],[0,26180,0.0548566128],[0,26181,0.2972684856],[0,26182,0.0137771823],[0,26184,0.0140218308],[0,26192,0.0120094587],[0,26218,0.0632693694],[0,26219,0.0319282451],[0,26221,0.0109001986],[0,26228,0.028344498],[0,26229,0.3307211849],[0,26231,0.0215265521],[0,26232,0.103417485],[0,26252,0.0187969174],[0,26266,0.0190939325],[0,26267,0.0204764953],[0,26268,0.3275281051],[0,26269,0.0262334323],[0,26270,0.0791087512],[0,26276,0.0306886269],[0,26277,0.0109001986],[0,26280,0.0860618543],[0,26281,0.0517582343],[0,26282,0.0187969174],[0,26306,0.0189736676],[0,26346,0.0159134448],[0,26348,0.1482348547],[0,26349,0.0303902328],[0,26350,0.088628431],[0,26355,0.032500204],[0,26385,0.2778126374],[0,26387,0.1455322832],[0,26389,0.0187969174],[0,26390,0.0164595119],[0,26391,0.0205156302],[0,26392,0.086086849],[0,26393,0.0652070064],[0,26395,0.282740572],[0,26398,0.0109001986],[0,26399,0.1418555271],[0,26400,0.0857097738],[0,26417,0.0187969174],[0,26419,0.0109001986],[0,26439,0.0332270938],[0,26440,0.0654017631],[0,26441,0.0121595476],[0,26442,0.0220460741],[0,26443,0.0173571207],[0,26444,0.0364879479],[0,26445,0.0346678206],[0,26447,0.0372577298],[0,26448,0.0204345923],[0,26472,0.0979319625],[0,26473,0.0187969174],[0,26474,0.0489059389],[0,26476,0.3421213598],[0,26477,0.2934947139],[0,26479,0.0137080842],[0,26480,0.0140218308],[0,26482,0.0189736676],[0,26483,0.3428517351],[0,26484,0.3161872879],[0,26489,0.1985165913],[0,26520,0.2111563345],[0,26522,0.1943514138],[0,26524,0.287619938],[0,26525,0.1286309125],[0,26527,0.1162007213],[0,26528,0.0155526991],[0,26529,0.0159573061],[0,26551,0.0187969174],[0,26578,0.0381701014],[0,26580,0.0166857443],[0,26581,0.4852632051],[0,26582,0.160663315],[0,26584,0.0187969174],[0,26585,0.1948569937],[0,26586,0.0705598839],[0,26587,0.0185403088],[0,26592,0.0204409028],[0,26595,0.0465986052],[0,26596,0.0187969174],[0,26625,0.0109001986],[0,26626,0.0283681408],[0,26627,0.0182830921],[0,26628,0.4336748796],[0,26629,0.0259179537],[0,26630,0.0109001986],[0,26631,0.189045579],[0,26632,0.0196582607],[0,26672,0.0109001986],[0,26673,0.1164453207],[0,26675,0.1169737079],[0,26676,0.0395232059],[0,26677,0.0764044967],[0,26685,0.1800697061],[0,26687,0.0521929982],[0,26718,0.327374102],[0,26719,0.0187969174],[0,26720,0.0120094587],[0,26721,0.3119546726],[0,26747,0.0189736676],[0,26751,0.4647181372],[0,26752,0.0454661207],[0,26753,0.0187969174],[0,26754,0.0739624703],[0,26756,0.1671098791],[0,26759,0.0894370079],[0,26760,0.0105601599],[0,26795,0.0187969174],[0,26799,0.3321922269],[0,26800,0.0505377411],[0,26803,0.120326596],[0,26804,0.081452642],[0,26805,0.0187969174],[0,26808,0.0187969174],[0,26809,0.1786688357],[0,26811,0.0187969174],[0,26812,0.0109001986],[0,26813,0.3310453447],[0,26814,0.0109001986],[0,26840,0.0203447241],[0,26844,0.0395232059],[0,26846,0.0140218308],[0,26848,0.0187969174],[0,26850,0.0109001986],[0,26853,0.0368918283],[0,26871,0.3229450619],[0,26872,0.0194678758],[0,26873,0.340070559],[0,26874,0.0266803229],[0,26875,0.0109001986],[0,26879,0.0528033207],[0,26881,0.0987620171],[0,26883,0.0182868982],[0,26884,0.0434492122],[0,26886,0.0384167182],[0,26916,0.0182830921],[0,26917,0.052246238],[0,26923,0.1285484688],[0,26924,0.0549595062],[0,26964,0.0249218486],[0,26965,0.0451018929],[0,26970,0.1257871184],[0,26974,0.050140863],[0,26975,0.0110424895],[0,26985,0.0281115026],[0,26987,0.0109001986],[0,26988,0.1500638854],[0,26989,0.0989298636],[0,26990,0.031266601],[0,26991,0.1301223373],[0,26993,0.0140218308],[0,26994,0.0189736676],[0,26995,0.0388217024],[0,26996,0.0205156302],[0,26997,0.0688858561],[0,26998,0.0762140549],[0,26999,0.0187969174],[0,27000,0.0152851705],[0,27015,0.0187969174],[0,27020,0.0120094587],[0,27025,0.0962723102],[0,27036,0.0628345453],[0,27048,0.0859042963],[0,27049,0.0159299093],[0,27050,0.0465986052],[0,27052,0.0124240731],[0,27054,0.0187969174],[0,27074,0.3493955358],[0,27075,0.0205212749],[0,27091,0.1307089874],[0,27106,0.0186530047],[0,27109,0.0187969174],[0,27115,0.0564615479],[0,27116,0.0178877824],[0,27117,0.0172438934],[0,27118,0.0434276001],[0,27119,0.0187969174],[0,27120,0.082187044],[0,27127,0.0127590401],[0,27129,0.0376328407],[0,27135,0.0359407954],[0,27143,0.0194678758],[0,27144,0.05797541],[0,27151,0.0157827438],[0,27152,0.0232658104],[0,27153,0.0535832101],[0,27156,0.1769050084],[0,27157,0.184972053],[0,27158,0.0109001986],[0,27159,0.0187969174],[0,27160,0.0145855176],[0,27166,0.0194858445],[0,27168,0.0187969174],[0,27175,0.0914103939],[0,27181,0.0194678758],[0,27193,0.0187969174],[0,27194,0.0270230829],[0,27195,0.0388217024],[0,27196,0.0494120869],[0,27197,0.0658377082],[0,27198,0.0316679782],[0,27199,0.0451302301],[0,27200,0.0226119081],[0,27213,0.0187969174],[0,27214,0.1917440452],[0,27217,0.0187969174],[0,27224,0.0277395924],[0,27226,0.0246099109],[0,27227,0.0305517201],[0,27228,0.0206500584],[0,27230,0.0190525562],[0,27242,0.0334235689],[0,27255,0.0178877824],[0,27275,0.0109001986],[0,27276,0.0187969174],[0,27277,0.0456127317],[0,27278,0.0140218308],[0,27281,0.0187969174],[0,27282,0.0462115904],[0,27284,0.0109001986],[0,27302,0.0642898686],[0,27306,0.3645912423],[0,27308,0.1034213642],[0,27313,0.0329569491],[0,27314,0.0155653989],[0,27316,0.0187969174],[0,27323,0.0175907497],[0,27335,0.1223050043],[0,27338,0.0208431083],[0,27339,0.1353846485],[0,27341,0.0187969174],[0,27342,0.0255456632],[0,27343,0.0084897702],[0,27345,0.0613268744],[0,27346,0.019446459],[0,27347,0.0461657905],[0,27348,0.1069498194],[0,27350,0.0187969174],[0,27355,0.0187969174],[0,27356,0.0151419923],[0,27357,0.1043275541],[0,27360,0.0372632411],[0,27362,0.0187969174],[0,27363,0.0159299093],[0,27381,0.112337295],[0,27382,0.025741692],[0,27419,0.0297497005],[0,27433,0.0131015983],[0,27481,0.3652909761],[0,27486,0.3658502978],[0,27498,0.2080592155],[0,27499,0.0109001986],[0,27501,0.0187969174],[0,27502,0.1151479032],[0,27508,0.3608188458],[0,27525,0.0187969174],[0,27556,0.1210613948],[0,27557,0.0095109939],[0,27585,0.0187969174],[0,27632,0.5569342172],[0,27748,0.0223449673],[0,27819,0.0105601599],[0,27821,0.0187969174],[0,27822,0.35385551],[0,27846,0.0187969174],[0,27880,0.0109001986],[0,27882,0.0131015983],[0,27883,0.0121595476],[0,27887,0.0187969174],[0,27909,0.0187969174],[0,27910,0.0121595476],[0,27924,0.0140218308],[0,27943,0.0139475888],[0,27945,0.0183410311],[0,27946,0.0886474055],[0,27964,0.0109001986],[0,28001,0.0140218308],[0,28044,0.0187969174],[0,28046,0.0187969174],[0,28048,0.0187969174],[0,28053,0.0189736676],[0,28075,0.0245105302],[0,28095,0.0187969174],[0,28096,0.0203447241],[0,28097,0.0187969174],[0,28119,0.0189736676],[0,28145,0.1889382424],[0,28165,0.0447745977],[0,28169,0.0189736676],[0,28234,0.0194670537],[0,28240,0.0105601599],[0,28294,0.0189736676],[0,28524,0.0121595476],[0,28526,0.0424687122],[0,28527,0.0109001986],[0,28557,0.0187969174],[0,28591,0.1589985655],[0,28610,0.9054524784],[0,28617,0.0121595476],[0,28651,0.0096445894],[0,28654,0.0109001986],[0,28660,0.0440380936],[0,28674,0.0084897702],[0,28717,0.4403058432],[0,28731,0.0338670129],[0,28751,0.0471145392],[0,28753,0.638854372],[0,28812,0.0187969174],[0,28837,0.0189736676],[0,28838,0.0189736676],[0,28871,0.0268217895],[0,28888,0.0189736676],[0,28969,0.0189736676],[0,28995,0.0318420222],[0,29000,0.117325847],[0,29016,0.0187969174],[0,29018,0.3120714949],[0,29041,0.213998388],[0,29042,0.0286967311],[0,29046,0.0387774988],[0,29051,0.0187969174],[0,29058,0.0293595364],[0,29070,0.0189736676],[0,29084,0.0187969174],[0,29099,0.0137080842],[0,29125,0.0187969174],[0,29131,0.0187969174],[0,29140,0.0188297179],[0,29150,0.0137771823],[0,29165,0.0187969174],[0,29184,0.127709344],[0,29187,0.0189736676],[0,29190,0.0603921722],[0,29211,0.0521356613],[0,29221,0.0109001986],[0,29226,0.0187969174],[0,29228,0.0149963933],[0,29229,0.0391639212],[0,29251,0.0109984493],[0,29252,0.0189736676],[0,29267,0.3015439043],[0,29268,0.0187969174],[0,29277,0.1178950851],[0,29284,0.0187969174],[0,29304,0.4841215804],[0,29305,0.0109001986],[0,29312,0.0189736676],[0,29335,0.0189736676],[0,29336,0.0140218308],[0,29339,0.1551549436],[0,29353,0.0109001986],[0,29384,0.0499166175],[0,29385,0.0189736676],[0,29386,0.0187969174],[0,29405,0.0187969174],[0,29412,0.0154785094],[0,29422,0.0189736676],[0,29434,0.3748193527],[0,29458,0.0140218308],[0,29461,0.1364556482],[0,29487,0.0187969174],[0,29499,0.0189736676],[0,29503,0.0189736676],[0,29507,0.0189736676],[0,29510,0.0189736676],[0,29524,0.0189736676],[0,29529,0.0084897702],[0,29541,0.0321271666],[0,29542,0.0189736676],[0,29553,0.0221763851],[0,29568,0.2198476806],[0,29581,0.0187969174],[0,29597,0.0186530047],[0,29635,0.0189736676],[0,29654,0.0294809865],[0,29658,0.1678282231],[0,29660,0.0227096543],[0,29665,0.0187969174],[0,29688,0.0109001986],[0,29691,0.1505233998],[0,29695,0.0189736676],[0,29737,0.0187969174],[0,29741,0.0139110459],[0,29746,0.0294809865],[0,29759,0.0187969174],[0,29792,0.0106263394],[0,29797,0.0189736676],[0,29799,0.0200902815],[0,29806,0.0187969174],[0,29827,0.0189736676],[0,29846,0.0189736676],[0,29876,0.0189736676],[0,29877,0.0189736676],[0,29889,0.1609054288],[0,29890,0.0139669169],[0,29902,0.0187969174],[0,29904,0.0189736676],[0,29911,0.0187969174],[0,29924,0.0187969174],[0,29954,0.0189736676],[0,29955,0.3442222391],[0,29963,0.0230038898],[0,29981,0.0242623142],[0,30010,0.0187969174],[0,30022,0.0189736676],[0,30027,0.0189736676],[0,30038,0.0189736676],[0,30042,0.2256571101],[0,30081,0.0187969174],[0,30093,0.0151107203],[0,30108,0.3578592301],[0,30161,0.0086592463],[0,30168,0.0277395924],[0,30169,0.0109001986],[0,30177,0.0109001986],[0,30180,0.0583371313],[0,30189,0.0189736676],[0,30198,0.0109001986],[0,30212,0.0121595476],[0,30219,0.0189736676],[0,30220,0.0189736676],[0,30232,0.0109001986],[0,30246,0.1438219303],[0,30264,0.0189736676],[0,30267,0.0189736676],[0,30275,0.0528241043],[0,30277,0.0105601599],[0,30278,0.0189736676],[0,30296,0.0253003905],[0,30298,0.0189736676],[0,30312,0.0189736676],[0,30343,0.0299126789],[0,30347,0.0109001986],[0,30352,0.0189736676],[0,30362,0.0187969174],[0,30365,0.0187969174],[0,30371,0.0751889074],[0,30379,0.0187969174],[0,30426,0.0189736676],[0,30444,0.0215715918],[0,30464,0.0363312531],[0,30469,0.3654013307],[0,30485,0.0109001986],[0,30509,0.0187969174],[0,30518,0.0187969174],[0,30524,0.0187969174],[0,30531,0.0254607236],[0,30548,0.0084897702],[0,30551,0.28761615],[0,30565,0.0092730414],[0,30611,0.0187969174],[0,30613,0.033555465],[0,30617,0.0187969174],[0,30660,0.0187969174],[0,30672,0.0840546625],[0,30675,0.2471189378],[0,30686,0.3042594825],[0,30716,0.0187969174],[0,30735,0.0659253096],[0,30741,0.0109001986],[0,30763,0.1302375767],[0,30773,0.0187969174],[0,30775,0.092649889],[0,30794,0.0109001986],[0,30804,0.0289380804],[0,30807,0.1444329893],[0,30814,0.0893786629],[0,30820,0.0086231648],[0,30844,0.0112418232],[0,30845,0.0109001986],[0,30848,0.0140218308],[0,30895,0.0121595476],[0,30918,0.0201105048],[0,30928,0.0185821724],[0,30934,0.0680783072],[0,30937,0.0686909347],[0,30958,0.0136064268],[0,30975,0.2181414068],[0,30979,0.0086592463],[0,30998,0.0290182754],[0,31005,0.0781052359],[0,31021,0.0136064268],[0,31030,0.0570270634],[0,31036,0.0136064268],[0,31037,0.8004269778],[0,31038,0.1137674811],[0,31043,0.0161387294],[0,31066,0.0229711943],[0,31067,0.0201105048],[0,31096,0.0453959027],[0,31110,0.0244699315],[0,31133,0.064800662],[0,31144,0.4908365924],[0,31166,0.0264866536],[0,31178,0.0545160874],[0,31185,0.0125513895],[0,31195,0.2563315875],[0,31197,0.0587741232],[0,31202,0.0257617976],[0,31205,0.7586059671],[0,31206,0.647058592],[0,31210,0.8015799931],[0,31212,0.014616373],[0,31231,0.0253038636],[0,31259,0.0321609968],[0,31343,0.4221813714],[0,31344,0.0156700237],[0,31389,0.0096445894],[0,31509,0.3155225456],[0,31532,0.0978064978],[0,31536,0.0870673791],[0,31549,0.0567318217],[0,31576,0.0287098019],[0,31580,0.5619063897],[0,31588,0.0161387294],[0,31592,0.0273554756],[0,31614,0.0292885347],[0,31642,0.0201105048],[0,31678,0.2426153449],[0,31689,0.0269384351],[0,31728,0.0580428766],[0,31774,0.0418738035],[0,31778,0.0180965591],[0,31782,0.0131015983],[0,31784,0.0634717993],[0,31787,0.0099749878],[0,31798,0.0187969174],[0,31814,0.0201105048],[0,31815,0.023685205],[0,31833,0.1557874017],[0,31849,0.0279730415],[0,31861,0.1056367838],[0,32132,0.0559581127],[0,32134,0.0263832771],[0,32202,0.1112867326],[0,32253,0.0912660513],[0,32258,0.0166115908],[0,32259,0.0275068417],[0,32264,0.0692101771],[0,32267,0.0291625977],[0,32279,0.0195795702],[0,32302,0.1039741386],[0,32303,0.0194818031],[0,32333,0.0683286916],[0,32371,0.5420106969],[0,32373,0.0166104467],[0,32374,0.1949377292],[0,32376,0.0987862447],[0,32378,0.363751196],[0,32381,0.4238753972],[0,32386,0.0195805438],[0,32391,0.0581322411],[0,32395,0.0307058653],[0,32397,0.0109001986],[0,32398,0.4053977781],[0,32399,0.0213510709],[0,32407,0.064557072],[0,32409,0.3680468592],[0,32411,0.0145855176],[0,32415,0.0189736676],[0,32418,0.1000741397],[0,32423,0.0184806533],[0,32440,0.0156700237],[0,32441,0.0923612922],[0,32444,0.0187969174],[0,32458,0.0395173347],[0,32462,0.0278653247],[0,32465,0.0139669169],[0,32467,0.0821370347],[0,32469,0.3365670892],[0,32472,0.5780858968],[0,32474,0.7707895195],[0,32475,0.0091034119],[0,32476,0.0149963933],[0,32478,0.127103289],[0,32479,0.1723370717],[0,32481,0.0140218308],[0,32482,0.2161650579],[0,32493,0.0204730398],[0,32496,0.0315245442],[0,32500,0.204845888],[0,32502,0.034471024],[0,32503,0.0475127231],[0,32504,0.1181931028],[0,32506,0.0566491272],[0,32509,0.0736660376],[0,32514,0.0187409929],[0,32517,0.0757846676],[0,32519,0.0476181586],[0,32533,0.0612618598],[0,32536,0.0631384817],[0,32538,0.3286180867],[0,32541,0.0266637544],[0,32547,0.3598750194],[0,32550,0.0352707062],[0,32556,0.1875194066],[0,32558,0.41332533],[0,32559,0.0109001986],[0,32565,0.0109001986],[0,32566,0.0131015983],[0,32578,0.0865643235],[0,32583,0.0131015983],[0,32605,0.3530125347],[0,32606,0.0608827743],[0,32614,0.0342497546],[0,32616,0.2170611859],[0,32617,0.0086231648],[0,32622,0.0269175713],[0,32626,0.3206392777],[0,32630,0.3980092311],[0,32631,0.0506387604],[0,32635,0.0467860072],[0,32637,0.0324088306],[0,32645,0.064121077],[0,32648,0.0259009458],[0,32649,0.0415383233],[0,32656,0.3818566238],[0,32657,0.0444936301],[0,32659,0.0426768942],[0,32660,0.2334585388],[0,32665,0.0348457994],[0,32667,0.0140218308],[0,32668,0.0928531034],[0,32672,0.3803233273],[0,32673,0.0272106502],[0,32674,0.0140960902],[0,32677,0.0313252237],[0,32678,0.7764889705],[0,32687,0.0095109939],[0,32689,0.1101233762],[0,32692,0.0112418232],[0,32726,0.0691678731],[0,32727,0.0276732563],[0,32732,0.0415206624],[0,32737,0.0611237501],[0,32739,0.2542836088],[0,32742,0.1366457604],[0,32743,0.0305565733],[0,32744,0.0573054939],[0,32747,0.0140218308],[0,32749,0.186897728],[0,32753,0.0135874684],[0,32759,0.0265607282],[0,32760,0.060670227],[0,32762,0.1435560303],[0,32769,0.2572733846],[0,32778,0.1941011513],[0,32779,0.2863729367],[0,32787,0.0885903877],[0,32799,0.4726115056],[0,32802,0.2237430864],[0,32804,0.0131546484],[0,32805,0.0296876293],[0,32809,0.1272772105],[0,32812,0.0297229588],[0,32825,0.0077988697],[0,32827,0.0332837054],[0,32838,0.6415805009],[0,32839,0.0214893113],[0,32844,0.0578148134],[0,32851,0.2114047173],[0,32852,0.245381134],[0,32861,0.0086495362],[0,32876,0.0338166112],[0,33780,0.0611065666],[0,33783,0.009992627],[0,33792,0.0111520085],[0,33795,0.0076568826],[0,33801,0.0086407689],[0,33807,0.035449696],[0,33814,0.0344277125],[0,33815,0.0127672515],[0,33818,0.1351548571],[0,33819,0.1876744175],[0,33820,0.6388686829],[0,33823,0.269729696],[0,33825,0.1377566816],[0,33828,0.02970183],[0,33838,0.2016258322],[0,33840,0.0893843676],[0,33843,0.0232348388],[0,33852,0.1752861323],[0,33862,0.1807799139],[0,33865,0.0668133477],[0,33876,0.0236732223],[0,33877,0.0262269819],[0,33878,0.0931160428],[0,33884,0.0988459902],[0,33890,0.3686161569],[0,33895,0.0385288467],[0,33902,0.3602120838],[0,33903,0.549702023],[0,33904,0.0300816167],[0,33911,0.0086768513],[0,33914,0.0541534406],[0,33926,0.049951776],[0,33939,0.0127672515],[0,33940,0.2820536406],[0,33941,0.4704042747],[0,33945,0.5082311315],[0,33951,0.0125967484],[0,33952,0.0439212325],[0,33955,0.0158675795],[0,33959,0.1300537178],[0,33961,0.2530763935],[0,33965,0.0731396001],[0,33971,0.0560908008],[0,33975,0.2517591172],[0,33976,0.2211259262],[0,33978,0.0230296797],[0,33979,0.0186741569],[0,33980,0.0192284665],[0,33983,0.0076568826],[0,33989,0.9433827273],[0,34005,0.3055080108],[0,34013,0.0499219937],[0,34014,0.0085073708],[0,34018,0.0190516508],[0,34021,0.009992627],[0,34022,0.0130115382],[0,34023,0.0163570125],[0,34024,0.0381240442],[0,34046,0.0503191897],[0,34047,0.0305307605],[0,34051,0.1074071367],[0,34054,0.2121177072],[0,34081,0.0864426985],[0,34084,0.6869700268],[0,34086,0.0111520085],[0,34090,0.0074876572],[0,34094,0.384456977],[0,34095,0.523983078],[0,34096,0.0357226415],[0,34098,0.1495285164],[0,34100,0.0913492034],[0,34103,0.4441796733],[0,34104,0.2766618643],[0,34110,0.3939592681],[0,34114,0.0456422278],[0,34127,0.0111520085],[0,34142,0.0861311387],[0,34146,0.1575719509],[0,34158,0.0215686974],[0,34160,0.1246966531],[0,34163,0.1799853138],[0,34164,0.9163346389],[0,34170,0.0111520085],[0,34182,0.0337179777],[0,34183,0.5693692271],[0,34185,0.0125967484],[0,34191,0.0965619235],[0,34194,0.0111520085],[0,34195,0.0120926663],[0,34198,0.0130115382],[0,34205,0.178861601],[0,34206,0.1994505641],[0,34215,0.0540720781],[0,34226,0.0430387033],[0,34232,0.0129567055],[0,34244,0.0074876572],[0,34250,0.0324483298],[0,34251,0.0440697225],[0,34260,0.1793181824],[0,34266,0.0262615285],[0,34273,0.2438503141],[0,34274,0.0111520085],[0,34291,0.1110823338],[0,34295,0.2689102907],[0,34305,0.2137143141],[0,34334,0.0555055533],[0,34337,0.0288530749],[0,34344,0.1437958239],[0,34349,0.4543095094],[0,34371,0.1163001864],[0,34373,0.1649291738],[0,34377,0.116046182],[0,34378,0.0230060558],[0,34385,0.1026542279],[0,34386,0.3657973319],[0,34413,0.1215432356],[0,34414,0.0399521365],[0,34416,0.0366946189],[0,34427,0.0912031444],[0,34429,0.264544207],[0,34435,0.0939809416],[0,34438,0.0313613024],[0,34444,0.0280850129],[0,34450,0.0127672515],[0,34460,0.2974549079],[0,34480,0.0742468031],[0,34484,0.554365547],[0,34487,0.0349646786],[0,34503,0.021375924],[0,34505,0.0723938505],[0,34507,0.2828140268],[0,34510,0.3466026503],[0,34515,0.0328122603],[0,34516,0.3015565649],[0,34521,0.323786308],[0,34528,0.0879097429],[0,34547,0.2194769223],[0,34556,0.0515749116],[0,34557,0.1508866221],[0,34562,0.2962860811],[0,34564,0.0186741569],[0,34571,0.0360180727],[0,34572,0.0339846855],[0,34573,0.1571408622],[0,34575,0.0289364661],[0,34577,0.0783162258],[0,34585,0.0549055587],[0,34586,0.0737301882],[0,34587,0.0993034568],[0,34589,0.198393979],[0,34593,0.2407623997],[0,34594,0.2301111798],[0,34595,0.1553443621],[0,34596,0.1485213988],[0,34597,0.2529644227],[0,34599,0.2466371651],[0,34600,0.3621118618],[0,34601,0.183223601],[0,34602,0.1859321562],[0,34603,0.1566725242],[0,34604,0.1747348595],[0,34606,0.0721689651],[0,34607,0.1555520531],[0,34608,0.2514069491],[0,34609,0.0337687345],[0,34611,0.0210985999],[0,34614,0.0363883986],[0,34615,0.0465010698],[0,34616,0.223953981],[0,34617,0.1728970063],[0,34619,0.0193054972],[0,34621,0.0350079429],[0,34622,0.2035925414],[0,34625,0.1585764869],[0,34628,0.0169330392],[0,34629,0.1447220433],[0,34631,0.024915691],[0,34636,0.0096210673],[0,34638,0.2138754781],[0,34639,0.0278691688],[0,34640,0.3232610336],[0,34642,0.0111520085],[0,34644,0.0211702693],[0,34648,0.4498863702],[0,34651,0.0558774593],[0,34656,0.0386415363],[0,34657,0.4676661041],[0,34658,0.0954868731],[0,34659,0.0362773589],[0,34660,0.0357599423],[0,34662,0.1712927657],[0,34663,0.0222999742],[0,34666,0.1488104478],[0,34668,0.0685338022],[0,34670,0.1406380843],[0,34680,0.1647639742],[0,34682,0.0890012195],[0,34683,0.1406380843],[0,34684,0.050898032],[0,34686,0.0843974957],[0,34687,0.1489495118],[0,34692,0.0346985003],[0,34696,0.2650624109],[0,34697,0.1662246362],[0,34701,0.1920160585],[0,34702,0.0935161043],[0,34707,0.0111520085],[0,34708,0.4584914344],[0,34710,0.0085073708],[0,34721,0.6678041208],[0,34724,0.5923468985],[0,34727,0.1763141733],[0,34731,0.0130115382],[0,34733,0.0677060102],[0,34738,0.4347619692],[0,34743,0.377820483],[0,34744,0.0216810838],[0,34746,0.0307351583],[0,34748,0.0270851177],[0,34751,0.0190516508],[0,34758,0.1127492095],[0,34761,0.3271602229],[0,34762,0.1048229787],[0,34765,0.0633750236],[0,34768,0.0581155707],[0,34803,0.1028253477],[0,34804,0.601704807],[0,34814,0.0138683512],[0,34837,0.043384317],[0,34853,0.4242124199],[0,34858,0.0805559989],[0,34874,0.3056144259],[0,34944,0.0266215021],[0,34992,0.0076208545],[0,34999,0.0189076848],[0,35001,0.193507194],[0,35005,0.0371699369],[0,35010,0.0169266764],[0,35011,0.0885456057],[0,35017,0.0081003915],[0,35020,0.0130115382],[0,35029,0.0095549856],[0,35037,0.718897721],[0,35038,0.0111520085],[0,35060,0.0447629666],[0,35061,0.0086768513],[0,35062,0.0117106298],[0,35063,0.0130115382],[0,35069,0.0205519349],[0,35073,0.010235641],[0,35078,0.0130115382],[0,35082,0.2096026767],[0,35085,0.0277800194],[0,35086,0.0095549856],[0,35090,0.0112943521],[0,35103,0.0120926663],[0,35108,0.017463768],[0,35110,0.0095847558],[0,35117,0.0095847558],[0,35118,0.009992627],[0,35122,0.6109311893],[0,35126,0.0590500304],[0,35136,0.014466063],[0,35138,0.0120926663],[0,35145,0.0120926663],[0,35151,0.09710177],[0,35156,0.0190516508],[0,35159,0.80640537],[0,35169,0.0096846951],[0,35180,0.0375239342],[0,35198,0.0193819964],[0,35201,0.0190516508],[0,35202,0.0126764472],[0,35203,0.0501146732],[0,35220,0.0125967484],[0,35222,0.2257372516],[0,35226,0.0542329839],[0,35227,0.0666635745],[0,35228,0.0189076848],[0,35230,0.0189076848],[0,35232,0.0261935437],[0,35240,0.0983499802],[0,35244,0.0190516508],[0,35249,0.0095549856],[0,35256,0.9720023036],[0,35266,0.0111520085],[0,35280,0.0461325234],[0,35289,0.1034503479],[0,35304,0.0407569055],[0,35307,0.0111520085],[0,35316,0.7705438465],[0,35317,0.0686693284],[0,35321,0.0111520085],[0,35322,0.0161298011],[0,35326,0.0095549856],[0,35327,0.3721433378],[0,35329,0.0241268395],[0,35339,0.0390382692],[0,35352,0.0110758462],[0,35353,0.5530567737],[0,35358,0.0111520085],[0,35361,0.0632275662],[0,35386,0.022474151],[0,35397,0.0214863387],[0,35398,0.020814443],[0,35402,0.010235641],[0,35413,0.0190516508],[0,35417,0.0095549856],[0,35423,0.0124150979],[0,35427,0.0202800244],[0,35435,0.1565731047],[0,35436,0.0111520085],[0,35443,0.4865660383],[0,35444,0.0111520085],[0,35462,0.0189076848],[0,35463,0.0086407689],[0,35504,0.0190844635],[0,35507,0.0086407689],[0,35510,0.0199142841],[0,35527,0.0190516508],[0,35530,0.0129567055],[0,35541,0.0085073708],[0,35542,0.0076208545],[0,35544,0.0281478646],[0,35546,0.0161944669],[0,35549,0.0815649418],[0,35552,0.0111520085],[0,35554,0.3801376607],[0,35559,0.0140909659],[0,35561,0.1640960217],[0,35565,0.0111520085],[0,35566,0.0130115382],[0,35567,0.0192284665],[0,35569,0.0095549856],[0,35575,0.0190516508],[0,35580,0.3107980698],[0,35585,0.0120926663],[0,35594,0.0228024013],[0,35602,0.1794683799],[0,35605,0.0155974691],[0,35611,0.1619628275],[0,35618,0.0111520085],[0,36163,0.0190516508],[0,36711,0.023390485],[0,38764,0.16215881],[0,38861,0.0088416509],[0,41045,0.4590220464],[0,41046,0.0160552577],[0,41082,0.0204148848],[0,41162,0.0081900039],[0,41163,0.0041994752],[0,41164,0.0746034321],[0,41167,0.0055426544],[0,41235,0.0203462687],[0,41277,0.0431674058],[0,41282,0.0046041107],[0,41283,0.0100960199],[0,41291,0.0190275691],[0,41314,0.1773063793],[0,41315,0.2120058344],[0,41317,0.084856958],[0,41341,0.023382147],[0,41342,0.4422064159],[0,41345,0.004031114],[0,41357,0.3669649134],[0,41358,0.1653276674],[0,41362,0.0139493993],[0,41363,0.0318849408],[0,41364,0.0237837478],[0,41365,0.3863130231],[0,41366,0.0268704009],[0,41367,0.6402614238],[0,41373,0.0483015794],[0,41374,0.1064506667],[0,41381,0.0875061149],[0,41394,0.0085004593],[0,41405,0.0165306579],[0,41412,0.0660963952],[0,41414,0.0651451498],[0,41429,0.0135201424],[0,41439,0.4911270731],[0,41465,0.0100960199],[0,41475,0.0346824681],[0,41476,0.008669938],[0,41480,0.0110121992],[0,41485,0.0425595839],[0,41488,0.0134717916],[0,41490,0.0041636311],[0,41505,0.2651517141],[0,41508,0.0890010457],[0,41509,0.2951941773],[0,41513,0.3111138903],[0,41517,0.3679588398],[0,41518,0.1195873034],[0,41519,0.005911982],[0,41521,0.0072776255],[0,41525,0.0160552577],[0,41526,0.0259249735],[0,41529,0.0813425485],[0,41532,0.015471169],[0,41534,0.0905189964],[0,41538,0.0176525752],[0,41542,0.0100960199],[0,41573,0.0098800328],[0,41577,0.0041994752],[0,41580,0.1665899863],[0,41583,0.0041994752],[0,41584,0.3138020701],[0,41585,0.5403033779],[0,41588,0.2841871609],[0,41589,0.0313989709],[0,41590,0.0100960199],[0,41592,0.0204891503],[0,41593,0.0524952866],[0,41595,0.025861031],[0,41596,0.1219087914],[0,41598,0.0267210986],[0,41605,0.0344747015],[0,41606,0.0072509186],[0,41610,0.0162313036],[0,41656,0.1562494308],[0,41660,0.014878976],[0,41662,0.0297456626],[0,41664,0.0220482191],[0,41665,0.1229612699],[0,41667,0.0332386132],[0,41674,0.2011333],[0,41678,0.0081900039],[0,41681,0.0035902695],[0,41688,0.0044618153],[0,41692,0.0240340062],[0,41693,0.0135966335],[0,41694,0.1608431151],[0,41695,0.0636303679],[0,41696,0.038895442],[0,41699,0.0250538281],[0,41704,0.0124707619],[0,41707,0.0374781465],[0,41709,0.0553213443],[0,41710,0.0159119184],[0,41713,0.0087878387],[0,41716,0.0203862548],[0,41728,0.1849166719],[0,41729,0.0932018829],[0,41731,0.004031114],[0,41737,0.0095982288],[0,41741,0.0110121992],[0,41742,0.1954023542],[0,41745,0.0045682451],[0,41746,0.1329791643],[0,41749,0.0072509186],[0,41752,0.0921302254],[0,41753,0.0194492041],[0,41757,0.008858254],[0,41758,0.0865122107],[0,41761,0.0135239987],[0,41762,0.0110121992],[0,41769,0.0156734911],[0,41770,0.0098800328],[0,41774,0.0206332575],[0,41775,0.1966736314],[0,41779,0.020165873],[0,41781,0.0303482926],[0,41783,0.0539655581],[0,41784,0.1514668758],[0,41785,0.273924746],[0,41787,0.5137044006],[0,41788,0.0588855432],[0,41800,0.6143601413],[0,41810,0.0624198709],[0,41818,0.0577531258],[0,41821,0.0153569904],[0,41824,0.0160552577],[0,41825,0.0280693947],[0,41827,0.1013577844],[0,41832,0.0110121992],[0,41833,0.0185874656],[0,41834,0.0179653],[0,41835,0.0240158211],[0,41836,0.0150248232],[0,41838,0.0070073645],[0,41840,0.4379113633],[0,41842,0.0162300548],[0,41847,0.2144324197],[0,41848,0.0224958898],[0,41850,0.0961218041],[0,41851,0.0116800305],[0,41852,0.0094722292],[0,41853,0.308807162],[0,41855,0.0085004593],[0,41857,0.008669938],[0,41859,0.0418706802],[0,41861,0.0217532023],[0,41863,0.0045682451],[0,41864,0.0087878387],[0,41865,0.0055065611],[0,41866,0.6477422932],[0,41868,0.014541358],[0,41869,0.0162026226],[0,41871,0.0174075562],[0,41872,0.0445423511],[0,41873,0.0049322337],[0,41876,0.1786813731],[0,41877,0.1755628057],[0,41878,0.0035902695],[0,41882,0.0081900039],[0,41884,0.1296503582],[0,41887,0.316346881],[0,41888,0.0110121992],[0,41891,0.0100960199],[0,41892,0.0098800328],[0,41897,0.6493087909],[0,41898,0.00999788],[0,41905,0.0227939143],[0,41907,0.2161986645],[0,41909,0.0122381337],[0,41910,0.12031333],[0,41920,0.0627360274],[0,41921,0.0506380932],[0,41922,0.0035902695],[0,41925,0.0176025844],[0,41926,0.0100960199],[0,41928,0.0041994752],[0,41929,0.0662878651],[0,41930,0.0034220606],[0,41931,0.2161228056],[0,41933,0.0035902695],[0,41935,0.0243800342],[0,41937,0.0279369474],[0,41941,0.0322693021],[0,41945,0.0098800328],[0,41949,0.0173676984],[0,41950,0.0095625882],[0,41951,0.0226527158],[0,41953,0.0344759009],[0,41954,0.0049322337],[0,41955,0.3405339555],[0,41959,0.0685597217],[0,41961,0.0122666704],[0,41962,0.0294601894],[0,41966,0.0221891876],[0,41971,0.2251357781],[0,41972,0.0249258249],[0,41976,0.0098800328],[0,41977,0.358416544],[0,41978,0.0232060047],[0,41980,0.0229916677],[0,41982,0.0513912209],[0,41987,0.0034220606],[0,41988,0.0035544578],[0,41993,0.0085004593],[0,41994,0.00999788],[0,41995,0.071397838],[0,42000,0.036018348],[0,42004,0.015280302],[0,42006,0.022360237],[0,42008,0.0177193382],[0,42009,0.0153157297],[0,42010,0.1188850349],[0,42020,0.2071108737],[0,42021,0.0249258249],[0,42022,0.113018138],[0,42026,0.0297375198],[0,42028,0.070887202],[0,42031,0.0140992456],[0,42033,0.082853973],[0,42035,0.1910703938],[0,42037,0.1409453384],[0,42038,0.0222966934],[0,42042,0.0128107],[0,42043,0.0269692985],[0,42052,0.074315346],[0,42056,0.0356784182],[0,42057,0.0181525094],[0,42061,0.2931895499],[0,42062,0.0124733903],[0,42071,0.1335815874],[0,42087,0.1663853886],[0,42122,0.0081900039],[0,42123,0.013625939],[0,42141,0.1218877007],[0,42143,0.3587873628],[0,42153,0.0114831532],[0,42165,0.0418199768],[0,42168,0.0088498886],[0,42169,0.0324459141],[0,42171,0.0134583668],[0,42174,0.097863294],[0,42177,0.1225694274],[0,42183,0.008669938],[0,42184,0.0384389211],[0,42187,0.0176025844],[0,42189,0.0523978831],[0,42193,0.0408626096],[0,42194,0.0812699643],[0,42195,0.1375401375],[0,42197,0.0368545703],[0,42200,0.0086042812],[0,42201,0.0085034298],[0,42206,0.0160552577],[0,42207,0.0041994752],[0,42209,0.1177144214],[0,42216,0.008858254],[0,42217,0.1010296487],[0,42221,0.2801673436],[0,42222,0.8385113724],[0,42224,0.4077841174],[0,42225,0.052218565],[0,42227,0.0055065611],[0,42232,0.0276891705],[0,42237,0.0522923788],[0,42238,0.022427008],[0,42241,0.0969656483],[0,42242,0.0205949139],[0,42243,0.025688949],[0,42246,0.0339687602],[0,42248,0.016046544],[0,42250,0.0573891641],[0,42253,0.1096954329],[0,42255,0.0279350307],[0,42256,0.0044356484],[0,42257,0.0338680871],[0,42264,0.2454098303],[0,42265,0.0034220606],[0,42270,0.0233062514],[0,42275,0.0055065611],[0,42276,0.3079505845],[0,42283,0.0367156098],[0,42289,0.0641304954],[0,42293,0.0081900039],[0,42294,0.2588639399],[0,42363,0.0044356484],[0,42374,0.3916122124],[0,42375,0.026471502],[0,42377,0.220811289],[0,42381,0.0198513394],[0,42389,0.498604626],[0,42400,0.0176427752],[0,42402,0.0081900039],[0,42405,0.0034220606],[0,42409,0.0110121992],[0,42410,0.0422563379],[0,42411,0.2871313941],[0,42412,0.0098800328],[0,42414,0.0462731708],[0,42415,0.1180415449],[0,42418,0.0160552577],[0,42419,0.0110121992],[0,42422,0.0081900039],[0,42426,0.0085004593],[0,42433,0.0045682451],[0,42434,0.3501033691],[0,42437,0.0821784286],[0,42442,0.2493469059],[0,42444,0.0160552577],[0,42445,0.2154132881],[0,42447,0.054136918],[0,42448,0.0073626146],[0,42452,0.0177648927],[0,42454,0.0094368052],[0,42465,0.004031114],[0,42469,0.2387872361],[0,42471,0.0248817739],[0,42472,0.0110121992],[0,42473,0.4111514961],[0,42476,0.0144485648],[0,42478,0.0082658306],[0,42479,0.0035902695],[0,42482,0.1458888965],[0,42486,0.0189317659],[0,42487,0.0228112265],[0,42488,0.3190324027],[0,42489,0.0321432748],[0,42490,0.2754347262],[0,42491,0.1199722328],[0,42493,0.0297764944],[0,42497,0.0081900039],[0,42500,0.0433342624],[0,42501,0.0044356484],[0,42503,0.4440579579],[0,42504,0.2637627265],[0,42509,0.0098800328],[0,42510,0.0100960199],[0,42513,0.0185629038],[0,42520,0.0785778442],[0,42521,0.1993365737],[0,42525,0.2943610927],[0,42530,0.0395063188],[0,42533,0.1255138403],[0,42536,0.017976541],[0,42538,0.0281319129],[0,42552,0.015471169],[0,42553,0.0085004593],[0,42554,0.0192084544],[0,42555,0.2187134056],[0,42567,0.1112477935],[0,42570,0.096901736],[0,42572,0.0132580472],[0,42575,0.0175530948],[0,42577,0.0431618186],[0,42580,0.0476659838],[0,42581,0.1849079804],[0,42583,0.0139420332],[0,42591,0.0122666704],[0,42592,0.0392836809],[0,42594,0.0046041107],[0,42595,0.0162502652],[0,42597,0.0054769697],[0,42599,0.6501489142],[0,42601,0.4942969564],[0,42603,0.0181917903],[0,42608,0.035595043],[0,42617,0.0046041107],[0,42618,0.0874611849],[0,42619,0.02562529],[0,42620,0.0169402951],[0,42622,0.0386982641],[0,42623,0.0081900039],[0,42624,0.0898851461],[0,42625,0.4904133478],[0,42626,0.2067676875],[0,42627,0.3126857349],[0,42629,0.0222470303],[0,42630,0.0239961106],[0,42635,0.0272891245],[0,42636,0.0208082695],[0,42639,0.0423610003],[0,42643,0.0359127315],[0,42644,0.015471169],[0,42645,0.0160552577],[0,42649,0.0239526167],[0,42651,0.0558996149],[0,42652,0.2461765945],[0,42668,0.0245116342],[0,42671,0.0311889836],[0,42672,0.0144686227],[0,42673,0.1175354629],[0,42677,0.004031114],[0,42678,0.018372506],[0,42682,0.0045682451],[0,42684,0.0572291276],[0,42686,0.0192761853],[0,42687,0.0293747705],[0,42695,0.0385722316],[0,42696,0.1073944318],[0,42699,0.1540598454],[0,42700,0.0034220606],[0,42702,0.2215474947],[0,42705,0.3008291361],[0,42706,0.004031114],[0,42709,0.0290478552],[0,42711,0.0390249652],[0,42714,0.315625156],[0,42717,0.1561671071],[0,42720,0.0109367199],[0,42722,0.0220797608],[0,42723,0.2116686125],[0,42726,0.0235538295],[0,42727,0.1210086457],[0,42732,0.0041994752],[0,42735,0.0055065611],[0,42738,0.0100960199],[0,42739,0.4156865689],[0,42740,0.0691143593],[0,42741,0.0041994752],[0,42763,0.0081900039],[0,42860,0.073965994],[0,42862,0.0311408081],[0,42911,0.0044356484],[0,42912,0.1964106547],[0,42913,0.0122103079],[0,42917,0.0513251446],[0,42920,0.004031114],[0,42923,0.049327234],[0,42929,0.1097791078],[0,42935,0.004896173],[0,42943,0.0034220606],[0,42948,0.004031114],[0,42962,0.0044356484],[0,42964,0.0260019423],[0,42976,0.0771848604],[0,42978,0.0464306459],[0,42980,0.0193014935],[0,42981,0.1302584637],[0,42982,0.0052519222],[0,42984,0.1123372601],[0,42991,0.0041994752],[0,42992,0.0185943303],[0,42997,0.1998560134],[0,42998,0.01452516],[0,42999,0.0100739765],[0,43000,0.5662613404],[0,43003,0.0258907089],[0,43007,0.3166282837],[0,43008,0.3410459514],[0,43010,0.0244968395],[0,43011,0.3135650969],[0,43012,0.0081900039],[0,43014,0.0204258547],[0,43015,0.0669663561],[0,43016,0.1292790361],[0,43019,0.0208431433],[0,43020,0.0607143505],[0,43022,0.0081900039],[0,43024,0.8291239116],[0,43025,0.0110121992],[0,43029,0.0303163937],[0,43032,0.1449942362],[0,43036,0.007619664],[0,43040,0.7662510787],[0,43041,0.0387512959],[0,43043,0.0077441416],[0,43044,0.1241988727],[0,43045,0.0110121992],[0,43046,0.0134475157],[0,43047,0.0960313808],[0,43049,0.0126324683],[0,43050,0.1701402473],[0,43053,0.8124080808],[0,43064,0.0880272197],[0,43066,0.0081900039],[0,43069,0.1798408932],[0,43072,0.2411185199],[0,43074,0.0103441352],[0,43075,0.0501548358],[0,43078,0.1811114841],[0,43079,0.193341654],[0,43080,0.0238582027],[0,43084,0.6553950265],[0,43086,0.8074041303],[0,43092,0.0545867115],[0,43097,0.0280558824],[0,43100,0.0100960199],[0,43101,0.0055065611],[0,43102,0.0918604522],[0,43104,0.007619664],[0,43105,0.0303791571],[0,43111,0.1471171839],[0,43115,0.1693962444],[0,43119,0.0477534804],[0,43120,0.1226310902],[0,43124,0.0110138275],[0,43126,0.0236061167],[0,43127,0.0244228941],[0,43133,0.4774479614],[0,43138,0.0034220606],[0,43142,0.2492456178],[0,43146,0.0120962925],[0,43148,0.004031114],[0,43149,0.1405433713],[0,43154,0.0072091286],[0,43157,0.3929490531],[0,43158,0.008858254],[0,43162,0.0044356484],[0,43164,0.0160552577],[0,43167,0.0470376127],[0,43170,0.008669938],[0,43171,0.0308571807],[0,43172,0.008669938],[0,43173,0.0135894133],[0,43174,0.0228719986],[0,43176,0.2718160432],[0,43178,0.0418493894],[0,43179,0.0247771842],[0,43180,0.0187923719],[0,43183,0.0592881007],[0,43184,0.0055426544],[0,43191,0.613772238],[0,43195,0.0110121992],[0,43203,0.0662398677],[0,43204,0.4033850415],[0,43212,0.008669938],[0,43213,0.0422563379],[0,43218,0.0055065611],[0,43221,0.0086042812],[0,43223,0.0160552577],[0,43225,0.0584426484],[0,43230,0.0508844275],[0,43233,0.0189504696],[0,43237,0.666487084],[0,43239,0.106099007],[0,43241,0.0160879275],[0,43246,0.0162300548],[0,43248,0.0100739765],[0,43252,0.0044356484],[0,43255,0.0128408591],[0,43259,0.1854549342],[0,43263,0.0035902695],[0,43265,0.0051791173],[0,43269,0.0452238662],[0,43270,0.008669938],[0,43273,0.0627881742],[0,43274,0.0172027959],[0,43275,0.1030207074],[0,43276,0.0177802458],[0,43278,0.2381256467],[0,43281,0.022777796],[0,43284,0.0357719856],[0,43286,0.0239821288],[0,43288,0.0088498886],[0,43289,0.0157278281],[0,43291,0.0081900039],[0,43299,0.0771745627],[0,43300,0.0044356484],[0,43301,0.0353031117],[0,43303,0.0153569904],[0,43310,0.2074034924],[0,43315,0.0173789375],[0,43318,0.0438232616],[0,43322,0.0160552577],[0,43326,0.0034220606],[0,43391,0.0660205988],[0,43393,0.1660298904],[0,43395,0.0188858091],[0,43399,0.0253349929],[0,43402,0.0401209329],[0,43404,0.2639769666],[0,43411,0.2302401477],[0,43420,0.2653600648],[0,43421,0.0035544578],[0,43425,0.0044356484],[0,43426,0.0173531068],[0,43430,0.0573473968],[0,43431,0.2547708359],[0,43437,0.0182624524],[0,43440,0.2555249057],[0,43442,0.0160552577],[0,43444,0.101684493],[0,43445,0.2070136916],[0,43448,0.014541358],[0,43451,0.0081900039],[0,43454,0.0071934627],[0,43455,0.0110121992],[0,43457,0.1612472384],[0,43458,0.2419693836],[0,43460,0.0189070418],[0,43461,0.24594447],[0,43463,0.0051791173],[0,43466,0.4070190263],[0,43468,0.004031114],[0,43472,0.0623692514],[0,43473,0.0249502199],[0,43476,0.0044356484],[0,43478,0.0100739765],[0,43479,0.0406607808],[0,43482,0.0421709456],[0,43483,0.0100960199],[0,43485,0.0249820756],[0,43489,0.0035902695],[0,43492,0.0309348594],[0,43494,0.013836769],[0,43502,0.0339687602],[0,43506,0.0160552577],[0,43508,0.3726855579],[0,43511,0.004031114],[0,43512,0.0204253361],[0,43515,0.0452053935],[0,43521,0.0888072412],[0,43522,0.2786884938],[0,43526,0.1229493576],[0,43529,0.0099641479],[0,43530,0.0272054815],[0,43532,0.004896173],[0,43533,0.0134475157],[0,43537,0.0051791173],[0,43539,0.018928557],[0,43541,0.1386600771],[0,43543,0.0101987687],[0,43544,0.0045682451],[0,43546,0.0716447882],[0,43552,0.1081529548],[0,43553,0.0229224973],[0,43554,0.7573757449],[0,43557,0.0149180287],[0,43561,0.008669938],[0,43564,0.0748556337],[0,43566,0.0434648217],[0,43574,0.0230548911],[0,43576,0.008669938],[0,43582,0.10298787],[0,43585,0.0110138275],[0,43587,0.0312719698],[0,43602,0.008858254],[0,43621,0.0283391709],[0,43629,0.2174091317],[0,43637,0.0110121992],[0,43644,0.0452830914],[0,43645,0.0045682451],[0,43649,0.0915724146],[0,43662,0.2238924534],[0,43664,0.0139420332],[0,43666,0.0288616068],[0,43673,0.6742484054],[0,43680,0.0239536472],[0,43683,0.061701551],[0,43686,0.4242761023],[0,43687,0.0081900039],[0,43690,0.0377304032],[0,43693,0.024518451],[0,43695,0.0453939653],[0,43700,0.7211183212],[0,43702,0.100711916],[0,43704,0.0095625882],[0,43710,0.0221590092],[0,43711,0.0081900039],[0,43714,0.3052867011],[0,43716,0.0176025844],[0,43718,0.0180140676],[0,43721,0.0463374946],[0,43725,0.0174250785],[0,43726,0.0479310856],[0,43729,0.0488863886],[0,43731,0.0307972903],[0,43737,0.0453671445],[0,43738,0.0289924209],[0,43742,0.013172239],[0,43744,0.022360237],[0,43748,0.0154513823],[0,43751,0.1599992709],[0,43753,0.6123242465],[0,43758,0.130507693],[0,43759,0.008653345],[0,43760,0.0226157987],[0,43764,0.0472078137],[0,43766,0.0324860137],[0,43768,0.0925084792],[0,43771,0.0035544578],[0,43772,0.0046041107],[0,43776,0.0195362087],[0,43778,0.0095625882],[0,43779,0.0891517849],[0,43781,0.0252896982],[0,43786,0.1145506598],[0,43788,0.008669938],[0,43791,0.008669938],[0,43794,0.0120962925],[0,43795,0.1179027646],[0,43798,0.1101730813],[0,43799,0.0122666704],[0,43804,0.0100960199],[0,43806,0.0356033787],[0,43807,0.2297556664],[0,43808,0.120055863],[0,43810,0.0383221441],[0,43812,0.0229343744],[0,43816,0.0135894133],[0,43819,0.0081900039],[0,43823,0.0086729093],[0,43825,0.170567815],[0,43826,0.0470923327],[0,43827,0.0098800328],[0,43830,0.0081900039],[0,43833,0.0794422421],[0,43834,0.0216825343],[0,43840,0.1524453282],[0,43841,0.0574354826],[0,43843,0.008669938],[0,43845,0.0034220606],[0,43848,0.1050022409],[0,43851,0.0665913788],[0,43853,0.0079752357],[0,43864,0.0110138275],[0,43882,0.0153569904],[0,43896,0.0035902695],[0,43897,0.2496977493],[0,43898,0.1368500873],[0,43914,0.1605337176],[0,43916,0.0137789512],[0,43923,0.056949472],[0,43929,0.1196161946],[0,43934,0.017976541],[0,43936,0.0224653123],[0,43937,0.0081900039],[0,43938,0.6191292947],[0,43939,0.535155071],[0,43948,0.0035544578],[0,43950,0.040089274],[0,43951,0.0117053774],[0,43952,0.3274062089],[0,43956,0.0041994752],[0,43962,0.0100960199],[0,43965,0.0918023005],[0,43971,0.0081900039],[0,43974,0.0034220606],[0,43976,0.0041994752],[0,43977,0.0055426544],[0,43978,0.0463596307],[0,43987,0.0412093358],[0,43988,0.0065972005],[0,43989,0.0081900039],[0,43992,0.0287264167],[0,43993,0.0100960199],[0,43994,0.0672336227],[0,43995,0.0256079417],[0,43998,0.218403559],[0,43999,0.0361295594],[0,44000,0.0223690656],[0,44001,0.0110138275],[0,44003,0.0160552577],[0,44005,0.0211214324],[0,44010,0.2217651013],[0,44013,0.3855757484],[0,44015,0.0456437437],[0,44017,0.0160552577],[0,44020,0.0470784826],[0,44024,0.004031114],[0,44025,0.0182767761],[0,44029,0.0478562823],[0,44031,0.6098857142],[0,44032,0.0247807882],[0,44034,0.0278572971],[0,44040,0.0176025844],[0,44045,0.0160552577],[0,44047,0.2038236104],[0,44080,0.0135966335],[0,44081,0.0120962925],[0,44184,0.0737686603],[0,44185,0.0098800328],[0,44215,0.0340871588],[0,44216,0.0185865081],[0,44217,0.0188655672],[0,44224,0.0408364702],[0,44225,0.0098800328],[0,44230,0.0967631664],[0,44231,0.0492901429],[0,44242,0.0055843415],[0,44251,0.0034220606],[0,44253,0.0100960199],[0,44263,0.005948097],[0,44264,0.0081900039],[0,44276,0.0183653116],[0,44286,0.004896173],[0,44288,0.2073850738],[0,44290,0.4575725626],[0,44294,0.008858254],[0,44298,0.0110121992],[0,44300,0.0823341448],[0,44302,0.1456502823],[0,44303,0.0371417111],[0,44304,0.0081900039],[0,44307,0.0076529466],[0,44308,0.0144686227],[0,44312,0.4041226678],[0,44313,0.0220339234],[0,44314,0.0388625632],[0,44315,0.0332496874],[0,44317,0.0817064836],[0,44320,0.1087499166],[0,44321,0.0987281212],[0,44322,0.0154513823],[0,44326,0.004896173],[0,44328,0.0458344795],[0,44335,0.0418010671],[0,44336,0.0393274998],[0,44338,0.0135239987],[0,44339,0.0143176637],[0,44341,0.0643561321],[0,44344,0.0110121992],[0,44346,0.0338294776],[0,44348,0.0292345807],[0,44350,0.5713241637],[0,44351,0.0081900039],[0,44353,0.0214709101],[0,44354,0.0110138275],[0,44355,0.0081900039],[0,44357,0.0041994752],[0,44360,0.0296767088],[0,44367,0.0098800328],[0,44372,0.0160552577],[0,44379,0.0134475157],[0,44380,0.0090194547],[0,44381,0.0035902695],[0,44386,0.0044356484],[0,44391,0.0477929373],[0,44398,0.2579730073],[0,44400,0.0266977835],[0,44401,0.0120962925],[0,44404,0.0103441352],[0,44412,0.0083561438],[0,44419,0.0098800328],[0,44420,0.0086729093],[0,44421,0.8604417978],[0,44425,0.0120962925],[0,44428,0.0085004593],[0,44430,0.0110121992],[0,44435,0.2700569047],[0,44437,0.0160552577],[0,44440,0.2246127348],[0,44441,0.0160552577],[0,44450,0.0081900039],[0,44454,0.008858254],[0,44456,0.0430500729],[0,44460,0.0728260335],[0,44462,0.2277447803],[0,44463,0.0962668464],[0,44464,0.2461626069],[0,44468,0.0259791858],[0,44484,0.1087101261],[0,44493,0.1735202541],[0,44494,0.4162700881],[0,44496,0.2848974298],[0,44497,0.282698048],[0,44505,0.1923660358],[0,44517,0.0045682451],[0,44526,0.2507281328],[0,44531,0.1734424746],[0,44534,0.0217475754],[0,44535,0.520162797],[0,44544,0.1217975344],[0,44551,0.5870138211],[0,44552,0.138673464],[0,44554,0.1287013642],[0,44560,0.005948097],[0,44561,0.0237809443],[0,44569,0.1264264171],[0,44572,0.0381443206],[0,44582,0.0113470221],[0,44585,0.5152866805],[0,44586,0.2447715576],[0,44590,0.0083561438],[0,44594,0.0180140676],[0,44596,0.2118579559],[0,44597,0.0877879593],[0,44604,0.0160552577],[0,44605,0.0154513823],[0,44615,0.0304249057],[0,44617,0.0098800328],[0,44620,0.1966693289],[0,44621,0.0081900039],[0,44632,0.1693952468],[0,44633,0.0080263545],[0,44634,0.018729156],[0,44637,0.0034220606],[0,44638,0.0055426544],[0,44639,0.0321910381],[0,44640,0.1601493481],[0,44642,0.1147375706],[0,44643,0.0812464229],[0,44645,0.0081900039],[0,44649,0.3701145431],[0,44651,0.0445469707],[0,44652,0.0170645454],[0,44655,0.0444635272],[0,44656,0.0081900039],[0,44658,0.004031114],[0,44659,0.0045690255],[0,44660,0.0528379432],[0,44663,0.0085004593],[0,44665,0.0292743792],[0,44669,0.0134475157],[0,44671,0.0153046431],[0,44674,0.0100960199],[0,44675,0.2413109035],[0,44678,0.0068383014],[0,44681,0.0100739765],[0,44682,0.0934114761],[0,44683,0.090571782],[0,44685,0.0266496055],[0,44690,0.0347488723],[0,44693,0.004031114],[0,44699,0.0854640484],[0,44705,0.2482506931],[0,44714,0.3335850664],[0,44727,0.0068383014],[0,44732,0.025780301],[0,44739,0.1601011489],[0,44745,0.1833567569],[0,44750,0.0060993888],[0,44754,0.1390152313],[0,44760,0.0512535535],[0,44772,0.1179474409],[0,44775,0.055369998],[0,44777,0.0419120651],[0,44779,0.0046041107],[0,44782,0.3558687055],[0,44783,0.3007713005],[0,44784,0.1391755134],[0,44785,0.0081900039],[0,44786,0.0100220332],[0,44793,0.0120962925],[0,44795,0.0044356484],[0,44799,0.0190218125],[0,44802,0.094386108],[0,44807,0.013300575],[0,44808,0.0120962925],[0,44822,0.0236739219],[0,44823,0.0275392128],[0,44825,0.1891484917],[0,44827,0.0364714876],[0,44830,0.0045682451],[0,44831,0.0046041107],[0,44835,0.1230107262],[0,44836,0.11139037],[0,44842,0.046669708],[0,44848,0.0321471975],[0,44851,0.0194982648],[0,44854,0.0081900039],[0,44857,0.136181008],[0,44858,0.0100098044],[0,44859,0.0180626298],[0,44862,0.0160552577],[0,44863,0.0110121992],[0,44867,0.031705614],[0,44870,0.0170063389],[0,44875,0.022360237],[0,44889,0.0074919052],[0,44896,0.0625883772],[0,44897,0.0966185319],[0,44901,0.017976541],[0,44906,0.0464262528],[0,44909,0.2950400337],[0,44913,0.013836769],[0,44917,0.1830599436],[0,44922,0.0182767761],[0,44928,0.0867721406],[0,44931,0.0217995162],[0,44933,0.052902336],[0,44937,0.008858254],[0,44938,0.0548612966],[0,44942,0.0044356484],[0,44946,0.1664890672],[0,44948,0.004031114],[0,44954,0.00999788],[0,44957,0.0379082818],[0,44959,0.2825624983],[0,44962,0.4818866553],[0,44968,0.3876552901],[0,44970,0.0044356484],[0,44972,0.4987322719],[0,44975,0.0493912408],[0,44981,0.0081900039],[0,44991,0.116742328],[0,44995,0.1011772583],[0,44999,0.8530549771],[0,45004,0.0463364219],[0,45005,0.0446589047],[0,45008,0.0515790431],[0,45011,0.0054769697],[0,45016,0.1907483381],[0,45021,0.0608953761],[0,45031,0.1465776883],[0,45032,0.0248752903],[0,45039,0.1100743789],[0,45042,0.0100739765],[0,45048,0.034264256],[0,45056,0.0128408591],[0,45057,0.0357957232],[0,45058,0.028528382],[0,45059,0.01452516],[0,45070,0.0527249176],[0,45072,0.1843129527],[0,45073,0.0293683821],[0,45076,0.0159119184],[0,45082,0.0062682672],[0,45084,0.0034220606],[0,45088,0.107111544],[0,45089,0.0528182146],[0,45092,0.5129300486],[0,45094,0.0827377472],[0,45096,0.1376359654],[0,45098,0.1817037528],[0,45099,0.1469481494],[0,45103,0.0123795076],[0,45106,0.0204672357],[0,45109,0.0173789375],[0,45128,0.0622304882],[0,45129,0.0099857003],[0,45132,0.0045682451],[0,45133,0.0044356484],[0,45136,0.1875468943],[0,45142,0.0135894133],[0,45143,0.0081900039],[0,45144,0.0300049189],[0,45148,0.008669938],[0,45149,0.004031114],[0,45150,0.0139776924],[0,45153,0.0340566118],[0,45154,0.0103358953],[0,45158,0.020458085],[0,45188,0.0365148498],[0,45200,0.1012011577],[0,45201,0.2059776791],[0,45204,0.0072776255],[0,45206,0.0252497088],[0,45209,0.1049983345],[0,45217,0.0156196039],[0,45225,0.0215401757],[0,45226,0.0170645454],[0,45236,0.0190275691],[0,45240,0.3909356472],[0,45241,0.0098800328],[0,45248,0.029831853],[0,45255,0.2175840338],[0,45261,0.1175868653],[0,45264,0.0893096004],[0,45276,0.0085004593],[0,45278,0.2025850477],[0,45281,0.0072776255],[0,45283,0.1427245833],[0,45295,0.3307260967],[0,45300,0.0120962925],[0,45458,0.2973899482],[0,45471,0.051044597],[0,45478,0.1975198465],[0,45479,0.0433430641],[0,45480,0.0481148019],[0,45481,0.0081900039],[0,45483,0.0968849423],[0,45485,0.2158000735],[0,45487,0.0100960199],[0,45489,0.5509631158],[0,45490,0.1598911426],[0,45493,0.0708830241],[0,45496,0.2174788497],[0,45500,0.0245230275],[0,45511,0.1013122879],[0,45520,0.3119792635],[0,45532,0.1325859972],[0,45541,0.0265986298],[0,45553,0.0248356224],[0,45559,0.0164970515],[0,45562,0.009531836],[0,45566,0.0155677705],[0,45568,0.0122925673],[0,45569,0.0160552577],[0,45570,0.0045682451],[0,45578,0.0467583397],[0,45579,0.0098800328],[0,45580,0.0643197552],[0,45581,0.027897169],[0,45582,0.1876124678],[0,45586,0.0085004593],[0,45592,0.0176059001],[0,45595,0.0360158506],[0,45602,0.0302127968],[0,45605,0.008858254],[0,45609,0.2460980599],[0,45611,0.0081900039],[0,45613,0.4922990283],[0,45617,0.1321431571],[0,45620,0.0215401757],[0,45621,0.03064138],[0,45634,0.0106135703],[0,45638,0.0117727969],[0,45650,0.0486979123],[0,45651,0.1112768929],[0,45652,0.0279506714],[0,45659,0.020330929],[0,45661,0.5670382822],[0,45667,0.1379300042],[0,45676,0.0931160428],[0,45679,0.1992487468],[0,45680,0.1116357169],[0,45694,0.6531660182],[0,45695,0.0035544578],[0,45700,0.1104254424],[0,45703,0.2226185927],[0,45705,0.1420453557],[0,45710,0.134337719],[0,45712,0.004031114],[0,45718,0.0190421732],[0,45719,0.1048088194],[0,45726,0.1579922042],[0,45729,0.132546966],[0,45730,0.1972944551],[0,45732,0.0288081734],[0,45741,0.5867284363],[0,45745,0.098646799],[0,45755,0.2587697361],[0,45765,0.0628128975],[0,45767,0.0086176505],[0,45770,0.0160879275],[0,45778,0.0096171198],[0,45784,0.6739252781],[0,45788,0.0307405395],[0,45799,0.0044356484],[0,45801,0.0484132448],[0,45802,0.0034220606],[0,45805,0.0062682672],[0,45807,0.017779822],[0,45810,0.2740240093],[0,45817,0.0228642216],[0,45818,0.3112947856],[0,45819,0.0978576901],[0,45823,0.008669938],[0,45826,0.004896173],[0,45831,0.0045682451],[0,45835,0.0216267109],[0,45841,0.0035902695],[0,45842,0.0055843415],[0,45845,0.0045682451],[0,45853,0.0081900039],[0,45855,0.0086176505],[0,45858,0.0139613591],[0,45859,0.1848812008],[0,45863,0.009531836],[0,45868,0.0087994784],[0,45872,0.0292833661],[0,45873,0.0385285226],[0,45877,0.0446367224],[0,45878,0.1824273473],[0,45880,0.0159119184],[0,45882,0.0081900039],[0,45885,0.1368142674],[0,45901,0.0480530806],[0,45902,0.1015193008],[0,45903,0.0699025458],[0,45905,0.1873243048],[0,45911,0.0094722292],[0,45912,0.2085582696],[0,45917,0.2958315063],[0,45920,0.0100739765],[0,45924,0.0512705647],[0,45936,0.0234522319],[0,45946,0.2148328838],[0,45969,0.0152829973],[0,45971,0.0081900039],[0,45972,0.0187134885],[0,45981,0.0113129712],[0,45983,0.0045682451],[0,45986,0.0997374696],[0,45992,0.6705456783],[0,45993,0.0090194547],[0,46002,0.0044356484],[0,46003,0.0044356484],[0,46007,0.056890121],[0,46008,0.009531836],[0,46009,0.0086042812],[0,46012,0.0135966335],[0,46014,0.0034220606],[0,46020,0.0454145413],[0,46027,0.0070073645],[0,46028,0.0190363887],[0,46029,0.009531836],[0,46034,0.004031114],[0,46049,0.112089776],[0,46052,0.0843810411],[0,46057,0.009531836],[0,46063,0.009531836],[0,46071,0.0568705176],[0,46072,0.3208893138],[0,46079,0.0140992456],[0,46081,0.0468945331],[0,46088,0.0271614735],[0,46092,0.0232913969],[0,46101,0.0041994752],[0,46103,0.0046041107],[0,46106,0.0159119184],[0,46108,0.0177141971],[0,46112,0.017976541],[0,46117,0.1522618882],[0,46125,0.161525056],[0,46126,0.0126938129],[0,46128,0.005911982],[0,46133,0.1277336507],[0,46134,0.0260934972],[0,46136,0.0477603541],[0,46142,0.0188992789],[0,46150,0.0089333879],[0,46161,0.0161078799],[0,46166,0.031705614],[0,46170,0.1592896776],[0,46171,0.0360878579],[0,46175,0.1488429488],[0,46179,0.0339687602],[0,46187,0.0188576145],[0,46191,0.0086176505],[0,46197,0.0519859804],[0,46198,0.0257078136],[0,46200,0.026013948],[0,46210,0.0129570183],[0,46214,0.1882835902],[0,46221,0.0110138275],[0,46225,0.2685498349],[0,46226,0.776660858],[0,46239,0.1285705018],[0,46243,0.013836769],[0,46251,0.0081900039],[0,46254,0.1209551822],[0,46258,0.3357536813],[0,46265,0.0177728997],[0,46266,0.0237106565],[0,46269,0.0160552577],[0,46286,0.0110138275],[0,46297,0.0710206709],[0,46299,0.0317979023],[0,46307,0.1995721294],[0,46308,0.1238678634],[0,46315,0.196686578],[0,46316,0.2657435081],[0,46319,0.1825270245],[0,46321,0.060001818],[0,46339,0.195170248],[0,46343,0.0054769697],[0,46345,0.0160552577],[0,46350,0.0196924732],[0,46356,0.0072776255],[0,46360,0.0976436207],[0,46366,0.004896173],[0,46377,0.009531836],[0,46386,0.0162003227],[0,46404,0.0055065611],[0,46408,0.2057872752],[0,46418,0.0041636311],[0,46421,0.0233760188],[0,46422,0.0160552577],[0,46427,0.5903786436],[0,46428,0.0045682451],[0,46429,0.004031114],[0,46430,0.0541023763],[0,46435,0.0344747015],[0,46437,0.2140277239],[0,46439,0.0054769697],[0,46446,0.0153157297],[0,46447,0.221835209],[0,46450,0.031705614],[0,46454,0.0600348634],[0,46465,0.0041994752],[0,46470,0.0265559702],[0,46473,0.0707390253],[0,46491,0.1443299557],[0,46496,0.3697066057],[0,46519,0.0072776255],[0,46529,0.0326037509],[0,46531,0.015602009],[0,46533,0.0045682451],[0,46535,0.0564676822],[0,46538,0.0240069203],[0,46541,0.0642079503],[0,46544,0.0160552577],[0,46551,0.0269477937],[0,46554,0.0229980442],[0,46555,0.0526434591],[0,46564,0.0081900039],[0,46566,0.005911982],[0,46569,0.2958957223],[0,46570,0.8075584304],[0,46572,0.3321648209],[0,46575,0.0207928952],[0,46579,0.0347377152],[0,46586,0.0283517455],[0,46588,0.0055065611],[0,46590,0.1102261985],[0,46692,0.0138553133],[0,46694,0.0308677931],[0,46702,0.0107835774],[0,46715,0.0081900039],[0,46718,0.0304564993],[0,46728,0.0055065611],[0,46746,0.3343444746],[0,46748,0.0160552577],[0,46755,0.022680142],[0,46757,0.0159119184],[0,46758,0.0254714591],[0,46759,0.2822108823],[0,46760,0.0304201291],[0,46763,0.0049322337],[0,46787,0.0203462687],[0,46791,0.1085927638],[0,46803,0.0079752357],[0,46804,0.0034220606],[0,46810,0.009531836],[0,46811,0.004031114],[0,46813,0.0035544578],[0,46829,0.7285716096],[0,46832,0.009531836],[0,46833,0.0502669519],[0,46842,0.0081900039],[0,46843,0.0098800328],[0,46856,0.014541358],[0,46857,0.0639725738],[0,46864,0.0044356484],[0,46870,0.017976541],[0,46872,0.0163530258],[0,46876,0.0129737976],[0,46886,0.0160552577],[0,46892,0.0503217943],[0,46895,0.0171960454],[0,46896,0.1831079882],[0,46910,0.008669938],[0,46924,0.0141758017],[0,46927,0.0456780965],[0,46932,0.005911982],[0,46935,0.0697336677],[0,46945,0.0034220606],[0,46947,0.0086176505],[0,46954,0.0041994752],[0,46957,0.0195020723],[0,46965,0.4691556036],[0,46970,0.6396058229],[0,46971,0.3093588467],[0,46977,0.5219795336],[0,46981,0.0323848612],[0,46988,0.0447693797],[0,46991,0.0280267157],[0,46992,0.0159119184],[0,46999,0.0541762975],[0,47001,0.0035902695],[0,47007,0.0107835774],[0,47009,0.0052519222],[0,47014,0.0173789375],[0,47015,0.0406039296],[0,47018,0.005911982],[0,47038,0.0244084305],[0,47052,0.0085004593],[0,47059,0.0162313036],[0,47062,0.0968688982],[0,47065,0.0081900039],[0,47066,0.0045690255],[0,47070,0.4608381626],[0,47071,0.0339687602],[0,47083,0.0137088024],[0,47090,0.009531836],[0,47094,0.0877301797],[0,47105,0.0160552577],[0,47106,0.0045682451],[0,47111,0.0285195355],[0,47113,0.0160552577],[0,47117,0.0443020238],[0,47130,0.1472809642],[0,47131,0.0381443206],[0,47136,0.1247724177],[0,47144,0.0086176505],[0,47145,0.0098800328],[0,47152,0.008653345],[0,47153,0.0090030135],[0,47159,0.0309202547],[0,47161,0.028240495],[0,47162,0.1077262861],[0,47180,0.0076155724],[0,47187,0.0250589632],[0,47190,0.0081900039],[0,47191,0.0150539063],[0,47193,0.0159119184],[0,47196,0.3594903026],[0,47199,0.0065972005],[0,47204,0.0387402317],[0,47207,0.1043659004],[0,47208,0.0193717479],[0,47210,0.0044356484],[0,47214,0.0307990383],[0,47216,0.1408700642],[0,47226,0.0206486985],[0,47228,0.2022856288],[0,47240,0.2436713862],[0,47268,0.004896173],[0,47304,0.0489973577],[0,47323,0.3507770521],[0,47326,0.0160552577],[0,47335,0.1745214617],[0,47354,0.5206468549],[0,47356,0.0045682451],[0,47364,0.105360651],[0,47371,0.0120122873],[0,47372,0.0090194547],[0,47385,0.019245024],[0,47386,0.0301290531],[0,47402,0.009531836],[0,47409,0.3020193338],[0,47416,0.0087416973],[0,47419,0.1028107358],[0,47430,0.0268417399],[0,47434,0.0160552577],[0,47435,0.0225648771],[0,47446,0.0160552577],[0,47475,0.0098800328],[0,47485,0.5682008456],[0,47489,0.7893894561],[0,47499,0.0199798071],[0,47500,0.0250583087],[0,47503,0.1980626887],[0,47506,0.0182522881],[0,47511,0.1068638441],[0,47512,0.398709553],[0,47514,0.0208578608],[0,47515,0.022952804],[0,47532,0.2646464358],[0,47541,0.013836769],[0,47542,0.0086176505],[0,47543,0.3097701538],[0,47544,0.0098800328],[0,47546,0.009531836],[0,47552,0.0034220606],[0,47553,0.112450165],[0,47557,0.0293368734],[0,47566,0.0081900039],[0,47570,0.0144686227],[0,47574,0.0238657902],[0,47583,0.0168154574],[0,47586,0.009531836],[0,47587,0.0044356484],[0,47593,0.0320312188],[0,47595,0.2498061066],[0,47597,0.1611163783],[0,47600,0.0203730715],[0,47602,0.5450518213],[0,47619,0.1542495918],[0,47626,0.0100335912],[0,47629,0.0482293178],[0,47631,0.2735692671],[0,47633,0.0201584737],[0,47636,0.0044356484],[0,47637,0.1863825584],[0,47646,0.0271497051],[0,47647,0.0268319039],[0,47650,0.0292502783],[0,47651,0.1683754521],[0,47653,0.0055065611],[0,47654,0.037463438],[0,47657,0.005911982],[0,47663,0.0789261752],[0,47665,0.0264198992],[0,47666,0.4755269308],[0,47667,0.2104958857],[0,47670,0.005911982],[0,47684,0.0261855562],[0,47688,0.3883736405],[0,47695,0.1129520746],[0,47700,0.0437020636],[0,47708,0.3288865233],[0,47712,0.0247512438],[0,47719,0.0664222223],[0,47727,0.0673561826],[0,47728,0.0253730146],[0,47730,0.0095625882],[0,47732,0.0085004593],[0,47736,0.0867138251],[0,47740,0.0055426544],[0,47742,0.0430124497],[0,47744,0.0081900039],[0,47747,0.0621433518],[0,47748,0.0206438022],[0,47751,0.0178873345],[0,47753,0.099970173],[0,47756,0.029050828],[0,47759,0.0243626671],[0,47763,0.4197299763],[0,47769,0.1186820806],[0,47770,0.0747216648],[0,47780,0.0054769697],[0,47787,0.006232313],[0,47789,0.0169309374],[0,47792,0.0045682451],[0,47796,0.0044356484],[0,47798,0.0316144634],[0,47800,0.0708724158],[0,47802,0.008669938],[0,47809,0.0585875582],[0,47811,0.0521909611],[0,47814,0.3088851552],[0,47817,0.0325291217],[0,47827,0.0035902695],[0,47828,0.0259791858],[0,47831,0.0055426544],[0,47832,0.0230854147],[0,47834,0.1496228033],[0,47840,0.2912642157],[0,47841,0.0095625882],[0,47853,0.0660023001],[0,47857,0.053201852],[0,47864,0.0204970377],[0,47866,0.0127864678],[0,47867,0.0160552577],[0,47869,0.1609796258],[0,47870,0.0155677705],[0,47872,0.042466472],[0,47874,0.5059385671],[0,47875,0.5457121339],[0,47876,0.3076456302],[0,47877,0.0045682451],[0,47884,0.005911982],[0,47897,0.6038209836],[0,47899,0.0728486774],[0,47900,0.0035902695],[0,47902,0.0497713123],[0,47907,0.0535661585],[0,47908,0.004031114],[0,47911,0.0045682451],[0,47924,0.1431688838],[0,47926,0.0271857737],[0,47928,0.0041994752],[0,47929,0.0086176505],[0,47930,0.0176785836],[0,47935,0.0098800328],[0,47946,0.0542249933],[0,47950,0.0034220606],[0,47951,0.0045682451],[0,47952,0.009531836],[0,47959,0.0414412188],[0,47961,0.0034220606],[0,47962,0.1099337645],[0,47963,0.005911982],[0,47968,0.262652495],[0,47970,0.0212237918],[0,47985,0.0212940509],[0,47999,0.0180140676],[0,48010,0.008669938],[0,48011,0.1166772211],[0,48013,0.0140992456],[0,48014,0.2390042935],[0,48018,0.025695363],[0,48022,0.0315451582],[0,48029,0.0085004593],[0,48034,0.0408355151],[0,48035,0.015280302],[0,48038,0.0216267109],[0,48040,0.008858254],[0,48042,0.0035902695],[0,48047,0.0045682451],[0,48052,0.0946493855],[0,48054,0.0355757276],[0,48058,0.0160552577],[0,48059,0.6016276388],[0,48061,0.4627568215],[0,48066,0.004896173],[0,48067,0.013836769],[0,48076,0.022764815],[0,48079,0.2068462768],[0,48081,0.0044356484],[0,48083,0.0802595147],[0,48095,0.0230596132],[0,48096,0.2137810689],[0,48103,0.0081900039],[0,48105,0.0132315424],[0,48112,0.0244489407],[0,48113,0.0086176505],[0,48115,0.0106135703],[0,48119,0.0543406989],[0,48120,0.0090115632],[0,48121,0.1039188321],[0,48122,0.0151689056],[0,48126,0.022360237],[0,48127,0.0368873558],[0,48129,0.0054769697],[0,48139,0.0106135703],[0,48141,0.3715267222],[0,48142,0.008669938],[0,48159,0.015471169],[0,48183,0.0055426544],[0,48287,0.0046041107],[0,48300,0.161880236],[0,48301,0.0098800328],[0,48305,0.0766967736],[0,48307,0.009531836],[0,48322,0.1449655205],[0,48344,0.0161078799],[0,48345,0.0160552577],[0,48348,0.009531836],[0,48352,0.0072776255],[0,48358,0.008858254],[0,48365,0.0660511357],[0,48370,0.0845674255],[0,48382,0.0314604678],[0,48385,0.0582223216],[0,48387,0.0669794698],[0,48389,0.0963984209],[0,48391,0.0045682451],[0,48399,0.0160552577],[0,48400,0.0319146402],[0,48402,0.0072776255],[0,48405,0.0149272976],[0,48414,0.0424966686],[0,48421,0.155985683],[0,48425,0.0045682451],[0,48432,0.3393178495],[0,48434,0.103126046],[0,48443,0.0058626324],[0,48449,0.0365232646],[0,48452,0.0619103504],[0,48483,0.1258731548],[0,48500,0.0249173234],[0,48503,0.0279350307],[0,48506,0.0128107],[0,48513,0.0086176505],[0,48517,0.0046041107],[0,48519,0.005911982],[0,48524,0.0279463543],[0,48528,0.0121034419],[0,48531,0.0044356484],[0,48532,0.1398793254],[0,48533,0.0246045752],[0,48541,0.0034220606],[0,48544,0.0139366706],[0,48546,0.004031114],[0,48556,0.0266496055],[0,48559,0.0159119184],[0,48561,0.0086176505],[0,48566,0.2759243699],[0,48569,0.0338150273],[0,48573,0.036375673],[0,48574,0.0164956824],[0,48578,0.0558169467],[0,48588,0.0083561438],[0,48589,0.6215246112],[0,48594,0.0081900039],[0,48600,0.015602009],[0,48602,0.014541358],[0,48605,0.1098773209],[0,48608,0.0229467999],[0,48611,0.008633856],[0,48634,0.0343662364],[0,48647,0.0296818764],[0,48655,0.0203730715],[0,48656,0.0110138275],[0,48664,0.0121034419],[0,48666,0.0992461654],[0,48672,0.0034220606],[0,48679,0.0041994752],[0,48690,0.0332085633],[0,48691,0.0081900039],[0,48699,0.0396908077],[0,48700,0.0429784568],[0,48704,0.0239935979],[0,48712,0.0103441352],[0,48720,0.004031114],[0,48722,0.0465088659],[0,48725,0.009531836],[0,48729,0.0093497276],[0,48730,0.0174311232],[0,48732,0.0904406525],[0,48739,0.1459233244],[0,48741,0.0081900039],[0,48743,0.0120122873],[0,48756,0.0058626324],[0,48759,0.3373813972],[0,48771,0.0968151546],[0,48784,0.0302127968],[0,48789,0.0035902695],[0,48791,0.009531836],[0,48803,0.0044356484],[0,48809,0.0034220606],[0,48816,0.0692023779],[0,48820,0.0563474335],[0,48822,0.0160552577],[0,48825,0.0035544578],[0,48830,0.100828713],[0,48833,0.0229343744],[0,48834,0.009531836],[0,48848,0.0469363281],[0,48853,0.0082658306],[0,48856,0.0070073645],[0,48858,0.4875688236],[0,48865,0.005911982],[0,48867,0.004031114],[0,48876,0.0110138275],[0,48880,0.098023706],[0,48889,0.8069856251],[0,48893,0.0099857003],[0,48898,0.0044356484],[0,48903,0.0140663328],[0,48905,0.0081900039],[0,48914,0.0081900039],[0,48918,0.297451331],[0,48923,0.0156486236],[0,48931,0.051811956],[0,48933,0.0163041502],[0,48938,0.0086176505],[0,48941,0.0055065611],[0,48953,0.0045682451],[0,48954,0.004896173],[0,48959,0.2939090409],[0,48961,0.0262302472],[0,48962,0.0361531894],[0,48967,0.0139613591],[0,48979,0.0034220606],[0,48990,0.0081900039],[0,48999,0.1697797134],[0,49000,0.0191310089],[0,49001,0.0034220606],[0,49002,0.009531836],[0,49135,0.1441305717],[0,49142,0.0213389489],[0,49147,0.0212202493],[0,49150,0.0045682451],[0,49156,0.0249173234],[0,49161,0.0035902695],[0,49162,0.009531836],[0,49193,0.0548778514],[0,49196,0.0160552577],[0,49198,0.034851215],[0,49204,0.0616112011],[0,49206,0.0035902695],[0,49212,0.005911982],[0,49219,0.0212465205],[0,49232,0.004896173],[0,49233,0.0034220606],[0,49236,0.0502450117],[0,49241,0.00507181],[0,49252,0.0323490464],[0,49254,0.1648119445],[0,49256,0.0249173234],[0,49258,0.0217112946],[0,49259,0.1936242982],[0,49271,0.1128412596],[0,49280,0.3174417236],[0,49288,0.0228855716],[0,49291,0.1808131722],[0,49293,0.1740674027],[0,49295,0.0107835774],[0,49299,0.0828327898],[0,50406,0.004031114],[0,50422,0.1355302605],[0,50459,0.0253171274],[0,50461,0.0068383014],[0,50469,0.005911982],[0,50487,0.004896173],[0,50511,0.0081900039],[0,50522,0.405881669],[0,50549,0.013172239],[0,50571,0.020330929],[0,50586,0.0516103091],[0,50591,0.4515472818],[0,50611,0.0605170386],[0,50622,0.0160552577],[0,50666,0.0116800305],[0,50670,0.0542575458],[0,50686,0.0097914383],[0,50704,0.0840241923],[0,50733,0.0283847705],[0,50756,0.0054769697],[0,50803,0.0249389242],[0,50808,0.005911982],[0,50857,0.015339054],[0,50860,0.0106135703],[0,52952,0.0081900039],[0,53556,0.0173059004],[0,53559,0.0518319362],[0,53560,0.0738698804],[0,53927,0.0247180854],[0,53928,0.0081900039],[0,56579,0.0081900039],[0,56657,0.4312935884],[0,57149,0.200203798],[0,57324,0.0085004593],[0,57408,0.0151213274],[0,57945,0.9220235315],[0,57946,0.8237202083],[0,57949,0.0106135703],[0,58341,0.8614934842],[0,58345,0.8470215983],[0,58688,0.9775640214],[0,58691,0.7733903017],[0,58695,0.6137417406],[0,58703,0.7625436101],[0,58918,0.0072776255],[0,58925,0.0976100671],[0,58927,0.0241987814],[0,58930,0.109672165],[0,58931,0.1981258968],[0,58946,0.7743667169],[0,58948,0.0750201753],[0,58951,0.8563237617],[0,58952,0.9002978579],[0,58956,0.0565984035],[0,58959,0.9294582891],[0,58971,0.0406155333],[0,58972,0.0160552577],[0,58983,0.0099857003],[0,58988,0.0078296596],[0,58990,0.6766553483],[0,58997,0.0046041107],[0,59001,0.2539250543],[0,59004,0.135556839],[0,59009,0.0872343579],[0,59016,0.3033858027],[0,59018,0.0564122097],[0,59027,0.022926268],[0,59029,0.0046041107],[0,59035,0.0081900039],[0,59037,0.0335616522],[0,59055,0.3594862896],[0,59061,0.9501428459],[0,59075,0.9887440611],[0,59089,0.9523356758],[0,59090,0.852159799],[0,59099,0.99211285],[0,59111,0.0162313036],[0,59120,0.8470215983],[0,59124,0.7230627549],[0,59128,0.0162313036],[0,59132,0.0353851173],[0,59137,0.9490475432],[0,59141,0.1043342334],[0,59152,0.5084986232],[0,59155,0.9328265178],[0,59156,0.0152116297],[0,59162,0.0050808402],[0,59167,0.0086255753],[0,59172,0.7657689154],[0,59174,0.0073560579],[0,59176,0.0116341715],[0,59202,0.1388670795],[0,59205,0.2411950911],[0,59216,0.0719380612],[0,59217,0.0316511842],[0,59223,0.0402593189],[0,59225,0.0077831781],[0,59227,0.0393173027],[0,59232,0.0675560743],[0,59236,0.104191103],[0,59238,0.0037387574],[0,59240,0.1389225158],[0,59242,0.5762455412],[0,59243,0.0210225407],[0,59244,0.0150684668],[0,59245,0.0170718022],[0,59246,0.013336526],[0,59251,0.0073560579],[0,59255,0.3760225568],[0,59264,0.036354781],[0,59265,0.0192626011],[0,59267,0.0353460958],[0,59268,0.3611340897],[0,59274,0.0090440064],[0,59276,0.0153874589],[0,59281,0.6956974013],[0,59283,0.0609032916],[0,59291,0.0077831781],[0,59294,0.0281283374],[0,59295,0.0338884855],[0,59298,0.0037387574],[0,59299,0.0065296871],[0,59300,0.0641670284],[0,59303,0.0086962383],[0,59306,0.0249741156],[0,59307,0.0073560579],[0,59310,0.0740077772],[0,59314,0.3038844608],[0,59315,0.0097766409],[0,59319,0.0185323602],[0,59323,0.0180256746],[0,59329,0.0073560579],[0,59339,0.0445999415],[0,59355,0.1860072031],[0,59359,0.0148708614],[0,59361,0.0299513775],[0,59362,0.5143195486],[0,59363,0.7931591825],[0,59364,0.9222666085],[0,59371,0.0073560579],[0,59375,0.3984040237],[0,59380,0.0271200632],[0,59381,0.425226646],[0,59383,0.0343322092],[0,59384,0.1911496934],[0,59387,0.0363056367],[0,59390,0.8309849921],[0,59391,0.0330655842],[0,59393,0.0351019776],[0,59397,0.0152116297],[0,59402,0.1333934702],[0,59403,0.0067823336],[0,59405,0.0073560579],[0,59409,0.0114787407],[0,59410,0.0086367048],[0,59415,0.0025939839],[0,59416,0.3061868789],[0,59418,0.004102298],[0,59422,0.0124564447],[0,59425,0.0073560579],[0,59427,0.0332222372],[0,59431,0.0185573486],[0,59437,0.0212989439],[0,59440,0.117895692],[0,59441,0.1988602814],[0,59444,0.0131203087],[0,59446,0.0250301948],[0,59457,0.0073560579],[0,59465,0.0149146101],[0,59483,0.0667582372],[0,59489,0.1062761874],[0,59493,0.0211217119],[0,59497,0.7007413916],[0,59504,0.0371465406],[0,59511,0.1679037856],[0,59512,0.486226717],[0,59514,1.0],[0,59518,0.0093162909],[0,59520,0.0079670091],[0,59534,0.016659542],[0,59537,0.0122276885],[0,59542,0.1739788474],[0,59548,0.0348976301],[0,59550,0.0120561126],[0,59552,0.0107773074],[0,59553,0.0289667103],[0,59557,0.4108013507],[0,59558,0.0064448027],[0,59561,0.0445739158],[0,59565,0.0096561898],[0,59567,0.0073560579],[0,59575,0.0027619857],[0,59577,0.005536906],[0,59578,0.2130176692],[0,59579,0.8065595657],[0,59582,0.0248959617],[0,59583,0.1089382795],[0,59584,0.1979713378],[0,59597,0.38923959],[0,59599,0.1026962389],[0,59601,0.0190158351],[0,59602,0.5558663847],[0,59605,0.0061748743],[0,59607,0.2767127267],[0,59615,0.0783261625],[0,59616,0.0037387574],[0,59617,0.3872897382],[0,59618,0.9697828793],[0,59619,0.0404958423],[0,59622,0.0211499748],[0,59626,0.0367452124],[0,59627,0.380654827],[0,59638,0.0374933637],[0,59640,0.0167394918],[0,59645,0.1634902075],[0,59649,0.4278170213],[0,59653,0.0670346731],[0,59655,0.0037387574],[0,59669,0.0147975432],[0,59671,0.0073560579],[0,59676,0.0850164083],[0,59677,0.5211286613],[0,59679,0.0096394704],[0,59681,0.2889947191],[0,59683,0.4450498693],[0,59691,0.6694686603],[0,59694,0.0152116297],[0,59704,0.0147247426],[0,59705,0.0159683721],[0,59709,0.0239227359],[0,59713,0.0251710677],[0,59714,0.243213944],[0,59719,0.2885478257],[0,59720,0.2563373845],[0,59723,0.0086962383],[0,59727,0.4294188598],[0,59733,0.1653393741],[0,59735,0.032068339],[0,59737,0.2373559015],[0,59741,0.0073560579],[0,59744,0.0073560579],[0,59745,0.0479651123],[0,59748,0.0037387574],[0,59763,0.0150684668],[0,59768,0.0108417883],[0,59769,0.0421614274],[0,59779,0.0152442592],[0,59797,0.0097766409],[0,59812,0.6571349635],[0,59818,0.0040662817],[0,59820,0.0152116297],[0,59821,0.0080234853],[0,59823,0.0073560579],[0,59825,0.2024615373],[0,59835,0.1804837297],[0,59836,0.0562634758],[0,59840,0.0027262181],[0,59849,0.0259986231],[0,59860,0.0405502886],[0,59862,0.019848158],[0,59863,0.0906112975],[0,59865,0.0105382475],[0,59868,0.0267445774],[0,59874,0.0272716077],[0,59881,0.2519983772],[0,59882,0.3607439931],[0,59889,0.1054906812],[0,59893,0.0305122181],[0,59894,0.0762446934],[0,59899,0.3600552286],[0,59900,0.0897670042],[0,59904,0.6070222008],[0,59906,0.1423227848],[0,59907,0.1694449927],[0,59908,0.0440203486],[0,59911,0.1476489039],[0,59913,0.0518789755],[0,59915,0.0046759184],[0,59917,0.0152116297],[0,59931,0.0345563957],[0,59934,0.0096561898],[0,59939,0.0152116297],[0,59940,0.0079670091],[0,59951,0.0727269975],[0,59957,0.0948582337],[0,59962,0.004775135],[0,59964,0.012979799],[0,59967,0.0179300227],[0,59972,0.008938259],[0,59974,0.0073560579],[0,59981,0.0027262181],[0,59982,0.1541823979],[0,59986,0.876045586],[0,59991,0.4499826823],[0,60001,0.0712426922],[0,60002,0.0107468244],[0,60004,0.2802299431],[0,60005,0.1017615553],[0,60008,0.0206696664],[0,60010,0.7038350462],[0,60013,0.1719354946],[0,60023,0.0276239901],[0,60024,0.4589106411],[0,60025,0.0221507635],[0,60029,0.1462197561],[0,60031,0.0360726118],[0,60032,0.0037395369],[0,60038,0.0140545267],[0,60045,0.0077831781],[0,60046,0.0344396767],[0,60048,0.0090440064],[0,60050,0.0131598743],[0,60053,0.0267780517],[0,60064,0.0187597033],[0,60065,0.0086367048],[0,60077,0.0077698252],[0,60079,0.0064448027],[0,60088,0.9217077179],[0,60093,0.0078354012],[0,60101,0.0032022876],[0,60104,0.0132251531],[0,60106,0.0313121229],[0,60107,0.0152116297],[0,60111,0.4799541929],[0,60112,0.0152116297],[0,60119,0.0722910597],[0,60122,0.007669098],[0,60131,0.0822190745],[0,60141,0.0434482328],[0,60146,0.0033704415],[0,60148,0.050291166],[0,60150,0.0087269526],[0,60156,0.0243926825],[0,60157,0.0073560579],[0,60163,0.1535311508],[0,60164,0.0081844877],[0,60177,0.1274063169],[0,60180,0.0883678799],[0,60184,0.5652990854],[0,60191,0.0266321705],[0,60196,0.0150684668],[0,60198,0.0134396569],[0,60205,0.0833517973],[0,60209,0.0087985261],[0,60211,0.0033704415],[0,60217,0.3579375052],[0,60219,0.2067845422],[0,60220,0.3892590829],[0,60223,0.0773730399],[0,60229,0.0073560579],[0,60237,0.0027619857],[0,60240,0.0380877485],[0,60241,0.0090440064],[0,60242,0.0318364923],[0,60243,0.2314048575],[0,60245,0.011800762],[0,60255,0.1246814307],[0,60276,0.0588584025],[0,60287,0.1319237691],[0,60289,0.0159326091],[0,60291,0.0079881369],[0,60292,0.8246429399],[0,60294,0.0152442592],[0,60303,0.0040662817],[0,60309,0.4803658169],[0,60310,0.0152116297],[0,60311,0.0073560579],[0,60314,0.7268742034],[0,60316,0.0071926099],[0,60317,0.1869441089],[0,60327,0.009696086],[0,60329,0.0043488777],[0,60331,0.0281949017],[0,60335,0.0622623397],[0,60340,0.2161817089],[0,60350,0.3610845517],[0,60353,0.2285570978],[0,60357,0.0415024066],[0,60363,0.0826049212],[0,60366,0.0078354012],[0,60373,0.0145071626],[0,60374,0.8490992474],[0,60376,0.0122170585],[0,60377,0.0191636913],[0,60378,0.2491804227],[0,60382,0.0163641386],[0,60388,0.0152116297],[0,60394,0.0090440064],[0,60396,0.0447062693],[0,60407,0.0373686478],[0,60412,0.0261849629],[0,60419,0.0037387574],[0,60422,0.0033704415],[0,60424,0.2950120758],[0,60425,0.2390382451],[0,60428,0.0620764594],[0,60441,0.8992624258],[0,60442,0.0099464388],[0,60448,0.0190788708],[0,60453,0.0032022876],[0,60459,0.0131598743],[0,60461,0.0150684668],[0,60468,0.0073560579],[0,60473,0.0086255753],[0,60474,0.013658517],[0,60478,0.0321218042],[0,60483,0.0073560579],[0,60487,0.0724665191],[0,60490,0.0149711284],[0,60491,0.014590453],[0,60492,0.0235185926],[0,60495,0.0152116297],[0,60496,0.015277844],[0,60500,0.0046759184],[0,60507,0.0259064499],[0,60517,0.0316799779],[0,60546,0.3784535845],[0,60548,0.4193534769],[0,60550,0.0090440064],[0,60552,0.439000149],[0,60558,0.4133576491],[0,60562,0.1156867153],[0,60564,0.0114063312],[0,60569,0.0152116297],[0,60577,0.1537633298],[0,60578,0.0046759184],[0,60584,0.0192873573],[0,60585,0.0152116297],[0,60586,0.0033346415],[0,60595,0.0131366218],[0,60596,0.0770020107],[0,60599,0.0027262181],[0,60601,0.0399740589],[0,60607,0.0204503953],[0,60608,0.0073560579],[0,60609,0.0073560579],[0,60611,0.6567480621],[0,60616,0.0415387434],[0,60626,0.5676593161],[0,60634,0.0152116297],[0,60641,0.0076661311],[0,60643,0.0261725888],[0,60652,0.0066174122],[0,60653,0.0126142299],[0,60663,0.2139235684],[0,60665,0.0255099291],[0,60667,0.0847684222],[0,60668,0.0033346415],[0,60680,0.0152116297],[0,60687,0.1926058505],[0,60689,0.0176030471],[0,60691,0.0090830238],[0,60692,0.1341405115],[0,60693,0.0524275651],[0,60694,0.3404568386],[0,60696,0.5302767314],[0,60700,0.0306361202],[0,60704,0.0090440064],[0,60708,0.0152576595],[0,60716,0.0654840299],[0,60719,0.172676229],[0,60729,0.0086962383],[0,60734,0.0046463634],[0,60735,0.0097766409],[0,60736,0.0152116297],[0,60741,0.1017183365],[0,60745,0.0033704415],[0,60747,0.0094864645],[0,60751,0.0027619857],[0,60752,0.6078007781],[0,60756,0.120471119],[0,60763,0.0312960246],[0,60766,0.2493719474],[0,60767,0.0054366868],[0,60773,0.0232408658],[0,60789,0.2607466738],[0,60791,0.0249518185],[0,60793,0.0153874589],[0,60797,0.029179849],[0,60800,0.1254490303],[0,60803,0.015518469],[0,60805,0.01418373],[0,60806,0.0207447807],[0,60810,0.0134199077],[0,60812,0.1277542066],[0,60813,0.1920575465],[0,60814,0.044949156],[0,60819,0.0162367863],[0,60823,0.068965329],[0,60825,0.0287148607],[0,60828,0.0086962383],[0,60829,0.0302611486],[0,60834,0.4886281381],[0,60840,0.0316799779],[0,60842,0.0050808402],[0,60844,0.0097766409],[0,60857,0.0213101613],[0,60864,0.0173998944],[0,60875,0.0027619857],[0,60876,0.0037387574],[0,60878,0.0391217951],[0,60889,0.1180002251],[0,60891,0.106704072],[0,60892,0.0463839027],[0,60893,0.0088589418],[0,60894,0.0108417883],[0,60896,0.1051820172],[0,60897,0.0073560579],[0,60902,0.1634255748],[0,60903,0.003606324],[0,60908,0.0269017728],[0,60912,0.0108417883],[0,60924,0.0594780791],[0,60926,0.1144595193],[0,60931,0.2518437929],[0,60935,0.1799171206],[0,60936,0.5905023267],[0,60944,0.2423491944],[0,60954,0.0037745789],[0,60955,0.0649551381],[0,60958,0.2380195835],[0,60966,0.0425925647],[0,60967,0.0414338604],[0,60968,0.0090440064],[0,60975,0.0444209721],[0,60978,0.0236069752],[0,60994,0.0027619857],[0,60995,0.0079670091],[0,60996,0.3021584939],[0,61008,0.0073560579],[0,61009,0.0086962383],[0,61011,0.0150684668],[0,61012,0.0322847455],[0,61017,0.0105958354],[0,61018,0.0073560579],[0,61020,0.0167035439],[0,61023,0.010215175],[0,61027,0.0158248718],[0,61032,0.0073560579],[0,61040,0.0073560579],[0,61041,0.0086962383],[0,61047,0.0046463634],[0,61049,0.0131366218],[0,61050,0.0222021961],[0,61051,0.0198991936],[0,61059,0.0862762144],[0,61068,0.2350137157],[0,61070,0.1560139661],[0,61072,0.3461286531],[0,61073,0.0664831295],[0,61087,0.0091495438],[0,61088,0.0797401939],[0,61096,0.0090270104],[0,61098,0.0323852521],[0,61107,0.0199637451],[0,61110,0.0033346415],[0,61116,0.0073560579],[0,61119,0.0488585922],[0,61124,0.0152116297],[0,61135,0.0086962383],[0,61136,0.147762336],[0,61137,0.0369408975],[0,61139,0.0321842725],[0,61141,0.0756622956],[0,61145,0.0090270104],[0,61149,0.0080234853],[0,61154,0.0131203087],[0,61173,0.0027619857],[0,61178,0.0114063312],[0,61182,0.0227496365],[0,61196,0.0235327586],[0,61198,0.1109482962],[0,61199,0.0167035439],[0,61209,0.0076661311],[0,61212,0.0215762817],[0,61216,0.0153874589],[0,61218,0.0080234853],[0,61223,0.129553175],[0,61225,0.0037387574],[0,61227,0.014265133],[0,61237,0.0438054001],[0,61240,0.0073560579],[0,61241,0.0153254331],[0,61245,0.1193389094],[0,61246,0.1343992811],[0,61255,0.4704573727],[0,61257,0.0356950187],[0,61259,0.1832777111],[0,61260,0.0047119673],[0,61262,0.0566545781],[0,61263,0.0076661311],[0,61270,0.3798183333],[0,61273,0.0086962383],[0,61282,0.0505291502],[0,61285,0.009185832],[0,61289,0.0560476901],[0,61290,0.012001188],[0,61292,0.1857382889],[0,61293,0.0181756818],[0,61295,0.1025157332],[0,61300,0.079966263],[0,61301,0.3078794381],[0,61305,0.0061389251],[0,61306,0.0153638937],[0,61309,0.0152116297],[0,61311,0.4067539505],[0,61324,0.019533982],[0,61326,0.0226161038],[0,61327,0.1071440937],[0,61330,0.0037387574],[0,61339,0.0135387563],[0,61341,0.0073560579],[0,61348,0.0121172041],[0,61349,0.1916573019],[0,61353,0.0300127628],[0,61355,0.3096325883],[0,61360,0.0119468636],[0,61361,0.1258435439],[0,61375,0.0130227763],[0,61379,0.0121172041],[0,61382,0.0259019639],[0,61384,0.2574508287],[0,61386,0.0314105122],[0,61389,0.0139718479],[0,61390,0.0152116297],[0,61391,0.2116638739],[0,61392,0.0116261756],[0,61405,0.0073560579],[0,61409,0.438790531],[0,61414,0.0257998126],[0,61418,0.0201602631],[0,61424,0.0377225302],[0,61426,0.1073895213],[0,61430,0.4929772968],[0,61431,0.0805015057],[0,61436,0.0510886965],[0,61439,0.0745571538],[0,61443,0.020782615],[0,61444,0.0832855849],[0,61446,0.0050808402],[0,61447,0.0050808402],[0,61448,0.2187210656],[0,61546,0.0480835226],[0,61665,0.0078354012],[0,62927,0.0160700578],[0,63120,0.0189139809],[0,63152,0.0541378777],[0,63185,0.048503853],[0,63217,0.0322535486],[0,63225,0.1058305983],[0,63314,0.0156901513],[0,63329,0.70728719],[0,63335,0.0216087898],[0,63342,0.7578607394],[0,63366,0.0158661034],[0,63375,0.1277557712],[0,63376,0.1514118755],[0,63414,0.9447945288],[0,63419,0.9490475432],[0,63421,0.8953839855],[0,63438,0.8835644375],[0,63439,0.0158661034],[0,63441,0.8022438299],[0,63443,0.0158661034],[0,63457,0.0158661034],[0,63463,0.8836161969],[0,63528,0.9381351781],[0,63551,0.8461581653],[0,63552,0.8751821252],[0,63553,0.9208282387],[0,63582,0.7715935978],[0,63614,0.0030636846],[0,63660,0.0154938342],[0,63685,0.8503787225],[0,63716,0.8909362716],[0,63721,0.8720504368],[0,63733,0.9262165449],[0,63774,0.9392230951],[0,63802,0.9501428459],[0,63816,0.9059029785],[0,63835,0.8985630834],[0,63845,0.9601559333],[0,63854,0.0276098228],[0,63857,0.0531110599],[0,63858,0.633166649],[0,63872,0.8337675491],[0,63874,0.0084969817],[0,63881,0.0126358178],[0,63883,0.0040767324],[0,63886,0.0045370116],[0,63889,0.0092554433],[0,63891,0.1901119161],[0,63892,0.5214653128],[0,63909,0.0152029238],[0,63913,0.0078290878],[0,63914,0.065993075],[0,63922,0.0156901513],[0,63923,0.010251363],[0,63924,0.0391024315],[0,63929,0.0155468883],[0,63931,0.1223539689],[0,64012,0.8545378106],[0,64022,0.9314251845],[0,64082,0.0128451525],[0,64084,0.0040767324],[0,64088,0.013953483],[0,64089,0.0156901513],[0,64091,0.0247939965],[0,64096,0.028674136],[0,64097,0.0757834339],[0,64099,0.2148405378],[0,64107,0.011317255],[0,64108,0.0084969817],[0,64110,0.009170205],[0,64121,0.2716285196],[0,64122,0.068405776],[0,64123,0.0152757753],[0,64132,0.0092725644],[0,64139,0.0201983021],[0,64141,0.0317411363],[0,64143,0.0040767324],[0,64144,0.0092725644],[0,64146,0.0086416642],[0,64151,0.0108001296],[0,64154,0.0156901513],[0,64155,0.0848309872],[0,64163,0.0443952987],[0,64168,0.6917763004],[0,64169,0.0042092584],[0,64176,0.009170205],[0,64177,0.0121101922],[0,64179,0.0248861081],[0,64180,0.2273096376],[0,64181,0.0078290878],[0,64185,0.0072923165],[0,64190,0.0626956838],[0,64192,0.0051470745],[0,64193,0.0440957119],[0,64196,0.0150695013],[0,64205,0.0275877122],[0,64212,0.1044374259],[0,64214,0.0156901513],[0,64219,0.009170205],[0,64225,0.0083087662],[0,64227,0.2001770732],[0,64234,0.0092725644],[0,64236,0.0119546527],[0,64237,0.0315262458],[0,64238,0.6080283988],[0,64241,0.0471483704],[0,64245,0.0157427455],[0,64247,0.0055029561],[0,64269,0.0213317361],[0,64270,0.0168764727],[0,64275,0.0187087196],[0,64277,0.013806951],[0,64278,0.0119923609],[0,64282,0.0333814876],[0,64285,0.0051831486],[0,64289,0.0518308991],[0,64290,0.0316867007],[0,64294,0.0860134729],[0,64296,0.1048697019],[0,64302,0.1166588586],[0,64308,0.1560703196],[0,64314,0.0576619434],[0,64319,0.0454755267],[0,64323,0.0078290878],[0,64325,0.0160563802],[0,64326,0.0647546386],[0,64339,0.0407962491],[0,64342,0.5301384529],[0,64343,0.5296372095],[0,64344,0.0084616086],[0,64345,0.0244133072],[0,64351,0.0213351027],[0,64352,0.0226369351],[0,64356,0.0160678332],[0,64358,0.0084969817],[0,64359,0.0391753067],[0,64366,0.009170205],[0,64368,0.0322274313],[0,64370,0.0140474827],[0,64372,0.0078290878],[0,64373,0.3766514743],[0,64375,0.0078290878],[0,64377,0.0294659124],[0,64378,0.0947293418],[0,64379,0.0456572317],[0,64380,0.6775657811],[0,64385,0.0125935624],[0,64387,0.021272848],[0,64389,0.2433889715],[0,64391,0.045104227],[0,64395,0.0078290878],[0,64396,0.8285349549],[0,64405,0.009170205],[0,64408,0.0030636846],[0,64416,0.0082565065],[0,64417,0.0095182162],[0,64418,0.0889129286],[0,64425,0.1852717842],[0,64427,0.0144103091],[0,64431,0.0038406849],[0,64432,0.0154002199],[0,64433,0.0075041703],[0,64435,0.0208403354],[0,64437,0.0346072004],[0,64439,0.0195952417],[0,64440,0.1388977313],[0,64447,0.0136154668],[0,64449,0.2296618219],[0,64454,0.0042092584],[0,64459,0.0124878826],[0,64461,0.0122400034],[0,64462,0.0120163595],[0,64465,0.0151596913],[0,64466,0.009170205],[0,64470,0.1309802195],[0,64481,0.0131080616],[0,64484,0.0157116359],[0,64490,0.2956029916],[0,64492,0.1478574219],[0,64493,0.0076655255],[0,64494,0.0904175456],[0,64495,0.2119770331],[0,64502,0.0226624589],[0,64504,0.0038406849],[0,64506,0.0042092584],[0,64509,0.1677850581],[0,64511,0.0303263665],[0,64512,0.3207705035],[0,64529,0.0078290878],[0,64530,0.0030636846],[0,64531,0.0078290878],[0,64536,0.0032318038],[0,64537,0.0040767324],[0,64541,0.006848735],[0,64542,0.0157228037],[0,64544,0.0581684741],[0,64545,0.0589963352],[0,64548,0.1363797408],[0,64555,0.0795562026],[0,64557,0.0155223486],[0,64560,0.1001589604],[0,64563,0.0083087662],[0,64567,0.7859676149],[0,64568,0.0119546527],[0,64576,0.1584549379],[0,64583,0.0322253573],[0,64589,0.0527779601],[0,64592,0.0045370116],[0,64594,0.039414991],[0,64595,0.0104212795],[0,64596,0.3464278921],[0,64599,0.0576870938],[0,64602,0.0109373001],[0,64603,0.0042092584],[0,64610,0.0055883752],[0,64612,0.0436637888],[0,64613,0.0108013788],[0,64616,0.1459278151],[0,64618,0.0163564159],[0,64622,0.0470158598],[0,64623,0.0313883682],[0,64628,0.0156901513],[0,64632,0.0155468883],[0,64634,0.2902913898],[0,64635,0.009660141],[0,64640,0.0843696179],[0,64642,0.0042092584],[0,64649,0.0069171954],[0,64652,0.141934976],[0,64658,0.018635753],[0,64661,0.2980156519],[0,64664,0.0155468883],[0,64670,0.0263417691],[0,64671,0.009170205],[0,64673,0.0042092584],[0,64674,0.1929845556],[0,64679,0.4560126447],[0,64691,0.0156901513],[0,64693,0.0228302977],[0,64703,0.009170205],[0,64704,0.0783183039],[0,64706,0.011252729],[0,64707,0.0634363054],[0,64710,0.0168089529],[0,64712,0.0030636846],[0,64714,0.0177436842],[0,64718,0.0156901513],[0,64723,0.0125324282],[0,64725,0.1492946877],[0,64727,0.0104288815],[0,64729,0.0042092584],[0,64731,0.1706768749],[0,64735,0.0918757815],[0,64736,0.0156901513],[0,64737,0.262289195],[0,64740,0.3386713826],[0,64743,0.0156901513],[0,64745,0.0209284461],[0,64755,0.0192387697],[0,64757,0.2232828562],[0,64758,0.0245512375],[0,64759,0.0156901513],[0,64763,0.0111204826],[0,64765,0.1006221859],[0,64770,0.0483191567],[0,64773,0.0078290878],[0,64774,0.0128821492],[0,64776,0.0325823222],[0,64778,0.0128809465],[0,64781,0.0042092584],[0,64782,0.0048925713],[0,64784,0.4739047333],[0,64794,0.5877882465],[0,64795,0.279835225],[0,64800,0.0416311053],[0,64804,0.6060591469],[0,64808,0.0270232799],[0,64810,0.0074283616],[0,64811,0.0972157856],[0,64812,0.0032318038],[0,64818,0.0544087128],[0,64821,0.1946316396],[0,64836,0.0038406849],[0,64846,0.0076655255],[0,64847,0.0084404661],[0,64853,0.0081393777],[0,64854,0.0308339991],[0,64855,0.0165938024],[0,64858,0.3148900072],[0,64862,0.009170205],[0,64865,0.0224297066],[0,64876,0.0155468883],[0,64877,0.0051174989],[0,64880,0.0263879591],[0,64881,0.1736376508],[0,64883,0.2535604812],[0,64892,0.0082565065],[0,64896,0.0084969817],[0,64898,0.0082565065],[0,64899,0.0030636846],[0,64904,0.0182168403],[0,64909,0.0064781053],[0,64917,0.0095182162],[0,64921,0.0124231028],[0,64922,0.020376207],[0,64923,0.0231256357],[0,64925,0.042477496],[0,64929,0.0508321552],[0,64931,0.1090208289],[0,64933,0.011252729],[0,64934,0.0052756589],[0,64935,0.067440676],[0,64936,0.009987347],[0,64943,0.1061548937],[0,64946,0.2263380605],[0,64947,0.3498964419],[0,64949,0.0036724135],[0,64950,0.0078290878],[0,64958,0.0036724135],[0,64968,0.0157362134],[0,64972,0.0190358565],[0,64975,0.6366095256],[0,64977,0.0169892533],[0,64982,0.0185862426],[0,64986,0.0042092584],[0,64987,0.0081393777],[0,64988,0.011938516],[0,64997,0.4706834925],[0,65005,0.0082565065],[0,65008,0.0030636846],[0,65010,0.0294330829],[0,65022,0.5305013613],[0,65030,0.0078290878],[0,65035,0.0959699724],[0,65037,0.0309934183],[0,65039,0.0091106299],[0,65042,0.0030636846],[0,65046,0.4039418037],[0,65048,0.1408236239],[0,65053,0.0135973682],[0,65054,0.0155468883],[0,65055,0.0739933685],[0,65057,0.0059083748],[0,65059,0.009170205],[0,65061,0.1827059739],[0,65068,0.009170205],[0,65069,0.0164850282],[0,65072,0.0171390759],[0,65081,0.0321392661],[0,65090,0.2342460173],[0,65099,0.0157427455],[0,65102,0.1482432045],[0,65106,0.0427548052],[0,65109,0.0069171954],[0,65111,0.0253132418],[0,65114,0.0127260952],[0,65118,0.3575925573],[0,65121,0.0178653166],[0,65123,0.2646291396],[0,65128,0.0157228037],[0,65131,0.0074283616],[0,65142,0.0155658635],[0,65143,0.012249976],[0,65148,0.0227169279],[0,65150,0.6599388282],[0,65154,0.0214039172],[0,65155,0.0169025852],[0,65162,0.0084616086],[0,65165,0.0216326479],[0,65167,0.0168434659],[0,65168,0.0032318038],[0,65173,0.0095182162],[0,65174,0.534181495],[0,65183,0.0032318038],[0,65188,0.0032318038],[0,65190,0.2981698555],[0,65195,0.024325562],[0,65197,0.0138730655],[0,65199,0.0104212795],[0,65202,0.0336348413],[0,65208,0.0052248134],[0,65209,0.6274581503],[0,65210,0.1442314108],[0,65211,0.090085717],[0,65213,0.0435340835],[0,65215,0.0032318038],[0,65217,0.028269274],[0,65222,0.0211520038],[0,65225,0.0078290878],[0,65229,0.0204331781],[0,65231,0.1031978159],[0,65238,0.0595952624],[0,65241,0.0389937781],[0,65242,0.0357663569],[0,65244,0.038726007],[0,65246,0.0052248134],[0,65248,0.1029580544],[0,65250,0.1718392121],[0,65254,0.05082093],[0,65256,0.0078290878],[0,65258,0.0859080248],[0,65271,0.3242556697],[0,65275,0.0094290047],[0,65276,0.0571866647],[0,65277,0.0100619592],[0,65278,0.0210390768],[0,65282,0.0163994047],[0,65286,0.1704757118],[0,65289,0.2985336175],[0,65296,0.0156901513],[0,65301,0.0241195493],[0,65312,0.0161419133],[0,65313,0.0125324282],[0,65314,0.2593923528],[0,65317,0.4720289661],[0,65320,0.0051174989],[0,65327,0.009170205],[0,65330,0.7149563685],[0,65333,0.0036724135],[0,65337,0.0040767324],[0,65344,0.0531019493],[0,65348,0.0096238274],[0,65351,0.0084969817],[0,65354,0.0109373001],[0,65356,0.0078290878],[0,65357,0.0383438194],[0,65358,0.0599097607],[0,65360,0.0157228037],[0,65361,0.1041003079],[0,65365,0.4787722346],[0,65367,0.0038406849],[0,65368,0.0135973682],[0,65369,0.392299265],[0,65374,0.0166195497],[0,65375,0.009170205],[0,65376,0.009170205],[0,65378,0.421611411],[0,65380,0.1673270202],[0,65381,0.04757349],[0,65387,0.0156901513],[0,65390,0.0075041703],[0,65392,0.0156901513],[0,65393,0.0358840467],[0,65399,0.0093460817],[0,65401,0.0715878782],[0,65402,0.3157329285],[0,65404,0.0079707362],[0,65406,0.0210076726],[0,65411,0.0051831486],[0,65413,0.0278940937],[0,65415,0.034869174],[0,65427,0.0208530247],[0,65430,0.0078290878],[0,65432,0.1652818086],[0,65436,0.0388580206],[0,65438,0.0570108604],[0,65439,0.0083087662],[0,65451,0.0555789257],[0,65454,0.0129330402],[0,65470,0.0257517407],[0,65476,0.0258286835],[0,65480,0.0156901513],[0,65481,0.2767609727],[0,65482,0.0752036651],[0,65487,0.0051831486],[0,65495,0.035437482],[0,65498,0.162638794],[0,65503,0.0121075651],[0,65506,0.0326493086],[0,65509,0.0196111515],[0,65512,0.0030636846],[0,65516,0.0042092584],[0,65523,0.0603064288],[0,65526,0.026366526],[0,65527,0.0272768762],[0,65531,0.2494633876],[0,65535,0.156447229],[0,65536,0.2320837559],[0,65542,0.1414955573],[0,65543,0.0213909292],[0,65544,0.0084969817],[0,65545,0.009170205],[0,65549,0.0045370116],[0,65552,0.0280765362],[0,65553,0.0437102178],[0,65554,0.2341916238],[0,65557,0.0415491947],[0,65563,0.0550766986],[0,65564,0.0042092584],[0,65566,0.0068330774],[0,65567,0.3833685764],[0,65574,0.0217470615],[0,65585,0.0155468883],[0,65587,0.0237628472],[0,65588,0.1390755934],[0,65589,0.0222833571],[0,65597,0.0031960112],[0,65602,0.0419559075],[0,65603,0.0051831486],[0,65604,0.0185347555],[0,65605,0.009170205],[0,65620,0.0323352529],[0,65621,0.0127260952],[0,65625,0.5107161487],[0,65626,0.109255745],[0,65635,0.4973256906],[0,65639,0.011317255],[0,65643,0.6981711372],[0,65644,0.0592618166],[0,65649,0.0076655255],[0,65656,0.011317255],[0,65657,0.0040767324],[0,65662,0.0036724135],[0,65666,0.0082565065],[0,65669,0.0318817558],[0,65671,0.0091106299],[0,65672,0.0082565065],[0,65673,0.0083087662],[0,65674,0.7240116523],[0,65682,0.009170205],[0,65686,0.0155468883],[0,65689,0.0273936793],[0,65690,0.0915563873],[0,65704,0.007904874],[0,65705,0.0751329569],[0,65706,0.4591825275],[0,65708,0.009170205],[0,65717,0.0084616086],[0,65719,0.0357423256],[0,65724,0.0192134165],[0,65726,0.0414753486],[0,65727,0.0172522919],[0,65728,0.0051831486],[0,65730,0.1845133894],[0,65733,0.0869007132],[0,65734,0.009170205],[0,65736,0.0737503784],[0,65740,0.0155468883],[0,65741,0.0156901513],[0,65742,0.0086054051],[0,65749,0.0377108339],[0,65753,0.0156901513],[0,65754,0.0260516818],[0,65760,0.0078290878],[0,65764,0.0095182162],[0,65765,0.4678871433],[0,65769,0.0413133535],[0,65772,0.0156901513],[0,65774,0.009170205],[0,65775,0.0850519766],[0,65780,0.0096238274],[0,65797,0.0921387204],[0,65802,0.4588739849],[0,65822,0.015702226],[0,65824,0.0334189777],[0,65825,0.009170205],[0,65827,0.1348195064],[0,65828,0.2823217889],[0,65830,0.0180541255],[0,65832,0.0031960112],[0,65833,0.0240863985],[0,65834,0.0083087662],[0,65838,0.1112362879],[0,65839,0.0222833571],[0,65840,0.0209325794],[0,65845,0.0086083329],[0,65851,0.009170205],[0,65855,0.0095182162],[0,65856,0.0109411419],[0,65865,0.0100907341],[0,65866,0.0217990073],[0,65870,0.0563516457],[0,65878,0.0131080616],[0,65879,0.0081393777],[0,65881,0.2256099027],[0,65884,0.0596543256],[0,65887,0.0177899546],[0,65892,0.0320481892],[0,65898,0.0181753044],[0,65919,0.014571337],[0,65920,0.0030636846],[0,65921,0.0285609917],[0,65925,0.0145758194],[0,65927,0.0069171954],[0,65932,0.1959328405],[0,65934,0.0225884899],[0,65936,0.632557665],[0,65938,0.0030636846],[0,65939,0.0612828424],[0,65941,0.0055522794],[0,65942,0.0471186752],[0,65943,0.0377815465],[0,65945,0.0249761558],[0,65947,0.0114304233],[0,65948,0.8195976697],[0,65950,0.0308600101],[0,65951,0.0206267619],[0,65952,0.0085990436],[0,65955,0.0157116359],[0,65956,0.076649143],[0,65965,0.0081393777],[0,65966,0.1202841292],[0,65968,0.0872778909],[0,65969,0.0363029214],[0,65973,0.0036724135],[0,65974,0.0811489426],[0,65976,0.1256268287],[0,65988,0.009170205],[0,65989,0.009170205],[0,65992,0.0051174989],[0,66001,0.034824098],[0,66002,0.009170205],[0,66004,0.0036724135],[0,66005,0.0158040343],[0,66008,0.009170205],[0,66010,0.2529457458],[0,66018,0.0189077822],[0,66022,0.0171901357],[0,66023,0.2663583629],[0,66024,0.0219798031],[0,66027,0.0078290878],[0,66028,0.010251363],[0,66029,0.0241081488],[0,66031,0.1630321856],[0,66032,0.3479074002],[0,66036,0.0154708413],[0,66040,0.626034089],[0,66043,0.5342211297],[0,66049,0.1904224316],[0,66053,0.0031960112],[0,66056,0.0084404661],[0,66057,0.0377616902],[0,66058,0.0031960112],[0,66060,0.0874770833],[0,66062,0.0125935624],[0,66064,0.0078290878],[0,66080,0.0072590516],[0,66086,0.0040767324],[0,66087,0.0340109083],[0,66089,0.0104212795],[0,66093,0.0492436045],[0,66098,0.1281881908],[0,66100,0.0045370116],[0,66105,0.1953386772],[0,66110,0.0051174989],[0,66114,0.0040767324],[0,66116,0.2967539668],[0,66117,0.0207488792],[0,66122,0.3882566937],[0,66124,0.009170205],[0,66129,0.0128086686],[0,66134,0.0082431443],[0,66145,0.0078290878],[0,66147,0.0156901513],[0,66157,0.0066470783],[0,66159,0.1798542116],[0,66163,0.0182554274],[0,66164,0.0036724135],[0,66175,0.0084969817],[0,66181,0.0123328656],[0,66185,0.0078290878],[0,66192,0.0192746101],[0,66194,0.0032318038],[0,66195,0.0078290878],[0,66204,0.0299772419],[0,66205,0.0078290878],[0,66206,0.0368020222],[0,66210,0.0207412239],[0,66211,0.0078290878],[0,66213,0.1216293339],[0,66214,0.2685735359],[0,66221,0.009170205],[0,66225,0.0132414297],[0,66226,0.0141308477],[0,66234,0.0348753416],[0,66236,0.0084969817],[0,66237,0.3860168979],[0,66239,0.1416895109],[0,66247,0.0032318038],[0,66250,0.0234116943],[0,66254,0.1749311167],[0,66257,0.3323391144],[0,66262,0.013953483],[0,66265,0.0128821492],[0,66271,0.0078290878],[0,66272,0.0078290878],[0,66276,0.0084969817],[0,66280,0.0128809465],[0,66284,0.0038048599],[0,66286,0.0207264874],[0,66288,0.0378098392],[0,66290,0.1052473711],[0,66294,0.0078290878],[0,66297,0.0082565065],[0,66310,0.0156901513],[0,66311,0.0042092584],[0,66312,0.0120517289],[0,66313,0.0045370116],[0,66322,0.0956564088],[0,66324,0.0082565065],[0,66333,0.1051991133],[0,66344,0.464560248],[0,66348,0.0548453089],[0,66350,0.0215092764],[0,66351,0.0082565065],[0,66354,0.644057255],[0,66365,0.5324209506],[0,66369,0.0030636846],[0,66370,0.0095182162],[0,66389,0.0539320568],[0,66391,0.0124231028],[0,66393,0.0279728023],[0,66399,0.1306912626],[0,66404,0.0051470745],[0,66405,0.0095182162],[0,66411,0.0316693267],[0,66412,0.0083087662],[0,66413,0.0166646524],[0,66417,0.1207239887],[0,66420,0.0086054051],[0,66422,0.0332884292],[0,66426,0.0143917067],[0,66427,0.153349054],[0,66430,0.1097281498],[0,66432,0.392751658],[0,66434,0.1502603313],[0,66437,0.1189488717],[0,66448,0.0332001384],[0,66449,0.0078290878],[0,66452,0.0543812057],[0,66454,0.2608746567],[0,66455,0.7283072882],[0,66457,0.1757142333],[0,66462,0.0226997569],[0,66464,0.0455845059],[0,66474,0.0135973682],[0,66477,0.0078290878],[0,66486,0.0212662584],[0,66491,0.0363348927],[0,66492,0.0243226798],[0,66494,0.009170205],[0,66496,0.0093131078],[0,66500,0.1019455207],[0,66502,0.0156901513],[0,66504,0.0036724135],[0,66507,0.0078290878],[0,66508,0.022233653],[0,66512,0.3885107931],[0,66516,0.0459469493],[0,66518,0.0427103535],[0,66519,0.0031960112],[0,66520,0.4817483688],[0,66525,0.3952663963],[0,66527,0.334134353],[0,66531,0.0680142952],[0,66540,0.4975555338],[0,66541,0.5579436615],[0,66542,0.61083199],[0,66545,0.0328636616],[0,66546,0.0088354329],[0,66549,0.0154469322],[0,66555,0.6550407377],[0,66560,0.0156901513],[0,66564,0.1762552976],[0,66568,0.009170205],[0,66570,0.0386874538],[0,66576,0.0082565065],[0,66578,0.009170205],[0,66579,0.0082565065],[0,66584,0.5199237778],[0,66586,0.0203788502],[0,66590,0.1091441136],[0,66593,0.0276669518],[0,66598,0.009170205],[0,66600,0.0042100383],[0,66601,0.012295545],[0,66616,0.0217513642],[0,66619,0.0108649482],[0,66621,0.0763713798],[0,66622,0.010251363],[0,66626,0.1479893743],[0,66637,0.0212737023],[0,66640,0.0588777819],[0,66642,0.0268170496],[0,66644,0.0032318038],[0,66651,0.0217817211],[0,66658,0.0078290878],[0,66659,0.0032318038],[0,66668,0.0078290878],[0,66669,0.0038048599],[0,66678,0.0042092584],[0,66684,0.0254757406],[0,66687,0.1888029474],[0,66688,0.1849762991],[0,66691,0.0095182162],[0,66699,0.6273413201],[0,66701,0.3892487799],[0,66702,0.0322535486],[0,66703,0.1827124016],[0,66705,0.0342893278],[0,66709,0.068813788],[0,66710,0.2633437108],[0,66715,0.0078290878],[0,66716,0.0038048599],[0,66718,0.043102138],[0,66719,0.0079707362],[0,66721,0.0217817211],[0,66722,0.0091106299],[0,66726,0.0047240369],[0,66729,0.0078290878],[0,66730,0.0208460897],[0,66733,0.0078290878],[0,66736,0.0032318038],[0,66738,0.0078290878],[0,66739,0.0832849285],[0,66740,0.2045186788],[0,66746,0.0545042363],[0,66759,0.0155468883],[0,66760,0.0030636846],[0,66764,0.0096238274],[0,66772,0.0225021586],[0,66773,0.022400629],[0,66776,0.0041028852],[0,66778,0.0038406849],[0,66779,0.011487438],[0,66782,0.0040767324],[0,66783,0.0224018247],[0,66787,0.303316175],[0,66792,0.031172855],[0,66797,0.0485629284],[0,66798,0.0106462347],[0,66800,0.0227370759],[0,66804,0.0273411552],[0,66807,0.0196062864],[0,66808,0.0606905259],[0,66809,0.0070899256],[0,66812,0.0268142643],[0,66815,0.2890847864],[0,66818,0.0084404661],[0,66819,0.0868571445],[0,66825,0.0160678332],[0,66826,0.0155468883],[0,66827,0.0285175498],[0,66829,0.0156901513],[0,66833,0.0414702862],[0,66842,0.2382434379],[0,66845,0.1350782358],[0,66848,0.0092725644],[0,66855,0.009170205],[0,66858,0.033099241],[0,66861,0.0051831486],[0,66862,0.0166646524],[0,66864,0.009170205],[0,66867,0.0275242242],[0,66870,0.0117498238],[0,66874,0.0069171954],[0,66875,0.0223361598],[0,66880,0.0090752248],[0,66884,0.0078290878],[0,66892,0.0156901513],[0,66899,0.0200199734],[0,66900,0.1990348576],[0,66902,0.0083087662],[0,66903,0.0031960112],[0,66905,0.0030636846],[0,66908,0.0667156457],[0,66909,0.0156901513],[0,66916,0.1919867746],[0,66919,0.0194744815],[0,66924,0.0032318038],[0,66926,0.3349508343],[0,66927,0.0156901513],[0,66933,0.0030636846],[0,66935,0.2371499879],[0,66936,0.0155468883],[0,66938,0.1038466949],[0,66942,0.0270136374],[0,66943,0.2453443234],[0,66946,0.0344464588],[0,66947,0.0276732486],[0,66949,0.2658107252],[0,66956,0.0072923165],[0,66957,0.1028729756],[0,66965,0.0052397575],[0,66970,0.4750706056],[0,66971,0.0308720672],[0,66972,0.009170205],[0,66974,0.0156901513],[0,66980,0.3049653732],[0,66981,0.010251363],[0,66984,0.009660141],[0,66986,0.0084969817],[0,66992,0.0901729861],[0,66997,0.0158131248],[0,67000,0.010251363],[0,67004,0.0119546527],[0,67008,0.0042451049],[0,67012,0.634977461],[0,67021,0.0078290878],[0,67026,0.0042092584],[0,67027,0.0474002192],[0,67032,0.0242480196],[0,67033,0.0155468883],[0,67039,0.0207048093],[0,67044,0.0603593876],[0,67046,0.0439088216],[0,67053,0.0252408404],[0,67055,0.028736787],[0,67058,0.0095182162],[0,67059,0.0644154623],[0,67060,0.0336786566],[0,67062,0.0335714721],[0,67071,0.0823051587],[0,67072,0.0036724135],[0,67073,0.009170205],[0,67078,0.0097320051],[0,67081,0.0171670502],[0,67085,0.0644539997],[0,67088,0.0160087599],[0,67092,0.0262314985],[0,67096,0.0084404661],[0,67101,0.0196306896],[0,67106,0.0151659935],[0,67109,0.0155468883],[0,67111,0.2230259156],[0,67114,0.009170205],[0,67116,0.0774034748],[0,67118,0.4557411153],[0,67121,0.0176762792],[0,67125,0.0363855504],[0,67126,0.041956299],[0,67128,0.0320742471],[0,67130,0.0749569822],[0,67133,0.013953483],[0,67135,0.0685557794],[0,67137,0.0055522794],[0,67138,0.0098122646],[0,67143,0.1122621794],[0,67145,0.0038406849],[0,67149,0.0040767324],[0,67151,0.0178066525],[0,67152,0.4110290072],[0,67153,0.2219741529],[0,67156,0.0143917067],[0,67160,0.6369621353],[0,67166,0.022233653],[0,67167,0.0083087662],[0,67168,0.0282135305],[0,67171,0.0157427455],[0,67173,0.7104024231],[0,67538,0.0040767324],[0,67539,0.0092554433],[0,67541,0.0522276668],[0,67542,0.0156901513],[0,67544,0.0173533452],[0,67551,0.0078290878],[0,67556,0.0155468883],[0,67560,0.0309286395],[0,67575,0.0036724135],[0,67592,0.0109411419],[0,67593,0.0222833571],[0,67607,0.0051174989],[0,67609,0.2018715774],[0,67619,0.0135973682],[0,67626,0.0270232799],[0,67634,0.2643723408],[0,67641,0.1318696517],[0,67643,0.0092943857],[0,67651,0.0262431666],[0,67656,0.0176453699],[0,67664,0.0192232259],[0,67668,0.0036724135],[0,67674,0.0075041703],[0,67677,0.0078290878],[0,67680,0.0082565065],[0,67684,0.0155468883],[0,67685,0.0314906889],[0,67686,0.0156901513],[0,67689,0.0306225398],[0,67691,0.0166251113],[0,67699,0.009170205],[0,67700,0.009170205],[0,67720,0.0155468883],[0,67723,0.0078290878],[0,67725,0.0036724135],[0,67726,0.0078290878],[0,67727,0.0479930898],[0,67728,0.0075041703],[0,67740,0.1028816042],[0,67741,0.0078290878],[0,67757,0.0156901513],[0,67764,0.0576856641],[0,67779,0.0032318038],[0,67784,0.0941323282],[0,67788,0.0135973682],[0,67794,0.0088354329],[0,67796,0.0120686809],[0,67805,0.0171670502],[0,67807,0.0144682955],[0,67812,0.1588619551],[0,67813,0.0171714798],[0,67817,0.0345707689],[0,67825,0.0078290878],[0,67828,0.0117404408],[0,67832,0.0055522794],[0,67857,0.0095182162],[0,67859,0.0468952063],[0,67860,0.0111204826],[0,67875,0.0932575004],[0,67878,0.011317255],[0,67879,0.0155468883],[0,67890,0.0156901513],[0,67895,0.0078290878],[0,67904,0.0109411419],[0,67907,0.0204626684],[0,67908,0.6424552636],[0,67910,0.0128086686],[0,67916,0.0365739327],[0,67917,0.2177464847],[0,67930,0.0152029238],[0,67933,0.4861200871],[0,67945,0.1130830756],[0,67950,0.0156901513],[0,67955,0.0078290878],[0,67956,0.0128086686],[0,67971,0.0082510268],[0,67990,0.0082565065],[0,67994,0.0114285011],[0,67995,0.0030636846],[0,67998,0.0069171954],[0,68002,0.034575536],[0,68008,0.0042451049],[0,68015,0.0101140965],[0,68017,0.0230998019],[0,68019,0.0084969817],[0,68021,0.0164107375],[0,68022,0.0136136928],[0,68028,0.0156901513],[0,68029,0.0055883752],[0,68035,0.0213351027],[0,68036,0.0031960112],[0,68040,0.1757178286],[0,68041,0.2671619242],[0,68061,0.0143917067],[0,68063,0.0078290878],[0,68076,0.0078290878],[0,68077,0.0030636846],[0,68080,0.0189816922],[0,68094,0.0197750537],[0,68101,0.0434554039],[0,68118,0.0361102037],[0,68139,0.0155468883],[0,68146,0.014742993],[0,68147,0.0351580296],[0,68156,0.0078290878],[0,68160,0.7532051978],[0,68161,0.010251363],[0,68170,0.0031960112],[0,68174,0.0042092584],[0,68175,0.0246755207],[0,68177,0.011317255],[0,68181,0.0038406849],[0,68194,0.0078290878],[0,68199,0.0051174989],[0,68208,0.0078290878],[0,68209,0.0078290878],[0,68216,0.0596841349],[0,68218,0.0136136928],[0,68222,0.0541590879],[0,68224,0.0158661034],[0,68228,0.017514791],[0,68229,0.0243080558],[0,68246,0.0133142852],[0,68253,0.0933525568],[0,68255,0.0162625044],[0,68261,0.5351227814],[0,68272,0.0078290878],[0,68274,0.0095182162],[0,68284,0.0055522794],[0,68293,0.011317255],[0,68313,0.0083087662],[0,68316,0.0155468883],[0,68323,0.013953483],[0,68343,0.0076294969],[0,68344,0.0032318038],[0,68346,0.1214146294],[0,68353,0.0196252344],[0,68360,0.0693687617],[0,68368,0.0916500926],[0,68370,0.0159417377],[0,68371,0.0084404661],[0,68394,0.0051470745],[0,68402,0.010251363],[0,68405,0.0164954846],[0,68406,0.0156901513],[0,68407,0.0157427455],[0,68414,0.0159448355],[0,68418,0.0042092584],[0,68421,0.0464661712],[0,68423,0.2587022567],[0,68444,0.0038406849],[0,68456,0.0157427455],[0,68457,0.0156901513],[0,68458,0.284110413],[0,68459,0.0082565065],[0,68460,0.0486008431],[0,68464,0.0083087662],[0,68467,0.0713399714],[0,68469,0.1553186994],[0,68482,0.0042092584],[0,68483,0.0078290878],[0,68486,0.0036724135],[0,68491,0.2480687856],[0,68496,0.0069171954],[0,68498,0.0297395774],[0,68506,0.1332276102],[0,68514,0.0265826794],[0,68522,0.0045730531],[0,68531,0.1802232075],[0,68555,0.0156901513],[0,68560,0.0156901513],[0,68571,0.0156901513],[0,68574,0.0308784293],[0,68576,0.0173882695],[0,68579,0.0173533452],[0,68585,0.0084969817],[0,68587,0.0090994926],[0,68592,0.0117351198],[0,68594,0.0096238274],[0,68595,0.0078290878],[0,68596,0.0032318038],[0,68600,0.1954119918],[0,68611,0.0078290878],[0,68615,0.0155468883],[0,68635,0.0270232799],[0,68642,0.0109411419],[0,68646,0.020216965],[0,68647,0.009170205],[0,68669,0.0036724135],[0,68677,0.0083087662],[0,68680,0.0155468883],[0,68683,0.009170205],[0,68686,0.0156901513],[0,68696,0.0129333044],[0,68697,0.0083087662],[0,68703,0.0286165846],[0,68710,0.0040767324],[0,68714,0.0173533452],[0,68716,0.0172522919],[0,68734,0.0775492093],[0,68741,0.0069171954],[0,68747,0.0036724135],[0,68757,0.0485173762],[0,68770,0.1123193699],[0,68772,0.0157228037],[0,68774,0.010251363],[0,68775,0.0078290878],[0,68803,0.0230998019],[0,68804,0.0079707362],[0,68805,0.0031960112],[0,68816,0.0083087662],[0,68823,0.0038406849],[0,68843,0.0826562045],[0,68846,0.1954915649],[0,68857,0.0082565065],[0,68861,0.1855174781],[0,68864,0.0136136928],[0,68868,0.009170205],[0,68873,0.0078290878],[0,68877,0.0190830783],[0,68878,0.0135973682],[0,68887,0.0158097025],[0,68890,0.0048198052],[0,68901,0.0109373001],[0,68902,0.0078290878],[0,68914,0.0045370116],[0,68915,0.1326165159],[0,68921,0.0042092584],[0,68924,0.0042092584],[0,68927,0.0084404661],[0,68928,0.0155468883],[0,68929,0.0078290878],[0,68946,0.0078290878],[0,68953,0.0032318038],[0,68960,0.0128086686],[0,68963,0.0566481542],[0,68971,0.0156901513],[0,68975,0.0155468883],[0,68977,0.0156901513],[0,68980,0.0084404661],[0,68987,0.0070899256],[0,68993,0.0491442397],[0,68997,0.1701225289],[0,69000,0.0086083329],[0,69005,0.0051470745],[0,69006,0.010251363],[0,69008,0.1528998979],[0,69021,0.0078290878],[0,69023,0.0084969817],[0,69033,0.0111204826],[0,69037,0.0078290878],[0,69039,0.0156901513],[0,69043,0.0258068902],[0,69047,0.0118913183],[0,69050,0.0137901573],[0,69053,0.0378626594],[0,69056,0.0081393777],[0,69068,0.0030636846],[0,69069,0.2088738574],[0,69070,0.0324092586],[0,69071,0.0551961888],[0,69077,0.0577686192],[0,69079,0.0157228037],[0,69080,0.0203904685],[0,69081,0.0117404408],[0,69082,0.0078290878],[0,69094,0.0914010987],[0,69095,0.0917107851],[0,69096,0.0078290878],[0,69098,0.0036724135],[0,69104,0.009660141],[0,69106,0.0186118084],[0,69113,0.0094650073],[0,69122,0.0095182162],[0,69134,0.6907346294],[0,69139,0.0135973682],[0,69141,0.0069171954],[0,69142,0.0255597407],[0,69145,0.0078290878],[0,69146,0.4090367086],[0,69147,0.0826963536],[0,69155,0.011317255],[0,69166,0.0245582507],[0,69168,0.0109503913],[0,69181,0.0078290878],[0,69189,0.011317255],[0,69194,0.0069171954],[0,69204,0.0032318038],[0,69208,0.0083087662],[0,69211,0.0078290878],[0,69213,0.6695745158],[0,69215,0.010251363],[0,69216,0.0082727033],[0,69221,0.0078290878],[0,69231,0.0078290878],[0,69239,0.0036724135],[0,69241,0.0215078634],[0,69242,0.0296681358],[0,69243,0.0084616086],[0,69244,0.0078290878],[0,69246,0.0156901513],[0,69247,0.011317255],[0,69261,0.0082727033],[0,69262,0.0131653356],[0,69263,0.0069171954],[0,69264,0.009170205],[0,69266,0.1678080956],[0,69272,0.0040767324],[0,69282,0.0730025279],[0,69284,0.009170205],[0,69290,0.0484336335],[0,69295,0.0437589575],[0,69296,0.0151989261],[0,69298,0.0156901513],[0,69305,0.0926175178],[0,69321,0.0069171954],[0,69324,0.0095182162],[0,69326,0.0196387051],[0,69351,0.1457830775],[0,69353,0.2460794138],[0,69354,0.0735084363],[0,69357,0.0156901513],[0,69360,0.0042092584],[0,69369,0.009170205],[0,69373,0.0148505452],[0,69375,0.0045730531],[0,69379,0.5775431418],[0,69381,0.0045370116],[0,69385,0.0155468883],[0,69397,0.0078290878],[0,69406,0.0082565065],[0,69408,0.054333339],[0,69416,0.0057395864],[0,69418,0.0280742161],[0,69422,0.0206306632],[0,69430,0.0209666283],[0,69434,0.0032318038],[0,69448,0.0195634712],[0,69449,0.0036724135],[0,69450,0.011317255],[0,69451,0.0082565065],[0,69452,0.0030636846],[0,69453,0.2209064307],[0,69457,0.0078290878],[0,69459,0.0040767324],[0,69465,0.0090063225],[0,69485,0.0155468883],[0,69491,0.0356956152],[0,69500,0.0156901513],[0,69505,0.0168574168],[0,69507,0.0031960112],[0,69508,0.0116972987],[0,69520,0.0078290878],[0,69524,0.4612946362],[0,69530,0.0158536917],[0,69531,0.0168966018],[0,69532,0.029397346],[0,69533,0.0081393777],[0,69534,0.0042092584],[0,69548,0.0944048762],[0,69550,0.0131080616],[0,69553,0.0156901513],[0,69566,0.0078290878],[0,69569,0.0081393777],[0,69573,0.0091106299],[0,69581,0.0214862362],[0,69585,0.0155468883],[0,69589,0.0230493626],[0,69593,0.0042451049],[0,69596,0.0156901513],[0,69604,0.0082565065],[0,69610,0.0580990551],[0,69618,0.0082565065],[0,69630,0.1917431761],[0,69640,0.0055522794],[0,69643,0.0540060889],[0,69647,0.0117404408],[0,69651,0.0171831085],[0,69657,0.0601878855],[0,69670,0.0069171954],[0,69674,0.0152029238],[0,69681,0.0189688123],[0,69691,0.304831895],[0,69695,0.2162239454],[0,69711,0.0126358178],[0,69713,0.0069171954],[0,69714,0.0170592447],[0,69720,0.0226105441],[0,69722,0.009170205],[0,69723,0.1052736757],[0,69731,0.0254594621],[0,69733,0.032907026],[0,69736,0.009170205],[0,69740,0.0109411419],[0,69743,0.0154142258],[0,69745,0.0340407373],[0,69746,0.0078290878],[0,69764,0.0082565065],[0,69776,0.0038406849],[0,69780,0.0042451049],[0,69783,0.0129515251],[0,69789,0.012965416],[0,69795,0.0165898544],[0,69800,0.0837555314],[0,69811,0.0082565065],[0,69817,0.0118821925],[0,69818,0.0051174989],[0,69819,0.0083087662],[0,69820,0.0223502223],[0,69821,0.0345097693],[0,69829,0.0078290878],[0,69833,0.0127483837],[0,69834,0.598584926],[0,69835,0.0206810329],[0,69836,0.1469390417],[0,69854,0.0082565065],[0,69858,0.0278940937],[0,69862,0.0040767324],[0,69863,0.0036724135],[0,69868,0.0092725644],[0,69878,0.007904874],[0,69879,0.0036724135],[0,69905,0.0151209999],[0,69914,0.007904874],[0,69919,0.0463275361],[0,69921,0.011317255],[0,69928,0.0045370116],[0,69929,0.0051174989],[0,69934,0.0511487207],[0,69947,0.0156901513],[0,69950,0.0082565065],[0,69951,0.0426706925],[0,69954,0.0301479646],[0,69960,0.0030636846],[0,69962,0.0156901513],[0,69973,0.0055522794],[0,69985,0.0042451049],[0,69988,0.0149125342],[0,70000,0.0156901513],[0,70011,0.0074962978],[0,70016,0.009170205],[0,70019,0.0038406849],[0,70029,0.2478869169],[0,70032,0.0058732216],[0,70039,0.0157427455],[0,70043,0.0078290878],[0,70051,0.0437222624],[0,70053,0.0301478692],[0,70066,0.0078290878],[0,70068,0.011317255],[0,70069,0.0212452081],[0,70074,0.0092780592],[0,70077,0.0217365122],[0,70087,0.0086855252],[0,70095,0.0784228269],[0,70100,0.0190132034],[0,70103,0.0342599324],[0,70104,0.0051174989],[0,70106,0.0156901513],[0,70110,0.0625471557],[0,70111,0.0574673339],[0,70117,0.0081393777],[0,70122,0.009170205],[0,70124,0.0082764576],[0,70125,0.0275347453],[0,70139,0.0179092724],[0,70140,0.0040767324],[0,70144,0.009170205],[0,70146,0.0157427455],[0,70147,0.0128086686],[0,70148,0.1477522522],[0,70153,0.0051174989],[0,70155,0.0083087662],[0,70162,0.0157427455],[0,70174,0.078717347],[0,70179,0.253094963],[0,70183,0.083253296],[0,70184,0.0030636846],[0,70194,0.0036724135],[0,70195,0.0078290878],[0,70196,0.0036724135],[0,70202,0.0248194077],[0,70205,0.023277883],[0,70206,0.0082565065],[0,70209,0.0051174989],[0,70215,0.018375025],[0,70220,0.0156901513],[0,70223,0.011317255],[0,70224,0.0398043751],[0,70240,0.0152029238],[0,70242,0.0069171954],[0,70245,0.0078290878],[0,70246,0.0135973682],[0,70251,0.0066470783],[0,70258,0.4451828697],[0,70262,0.0156901513],[0,70263,0.1947532175],[0,70265,0.010251363],[0,70267,0.0093330223],[0,70269,0.035125632],[0,70272,0.0951339734],[0,70279,0.0165898544],[0,70282,0.0038406849],[0,70286,0.024678781],[0,70291,0.0082764576],[0,70297,0.0059083748],[0,70298,0.0156901513],[0,70318,0.0156901513],[0,70324,0.0042092584],[0,70326,0.0048198052],[0,70335,0.0081393777],[0,70336,0.0788633037],[0,70344,0.1533379417],[0,70345,0.0380805841],[0,70346,0.0078290878],[0,70353,0.011487438],[0,70362,0.0272090603],[0,70364,0.0351646275],[0,70365,0.0082565065],[0,70377,0.0126170966],[0,70389,0.0213597503],[0,70390,0.0173533452],[0,70398,0.0225884899],[0,70400,0.2549024977],[0,70404,0.0156901513],[0,70405,0.0156901513],[0,70416,0.0055883752],[0,70438,0.2870219179],[0,70457,0.0030636846],[0,70463,0.3098722297],[0,70470,0.0078290878],[0,70473,0.011487438],[0,70482,0.0078290878],[0,70500,0.0041028852],[0,70509,0.0069171954],[0,70512,0.0128086686],[0,70519,0.0075041703],[0,70526,0.0045370116],[0,70539,0.0156901513],[0,70544,0.0076302808],[0,70545,0.0038406849],[0,70547,0.1074727878],[0,70549,0.0048925713],[0,70550,0.1274182049],[0,70554,0.0392843012],[0,70555,0.0095182162],[0,70561,0.0162826698],[0,70563,0.0078290878],[0,70567,0.009170205],[0,70571,0.0032318038],[0,70576,0.0158131248],[0,70584,0.0032318038],[0,70588,0.0078290878],[0,70593,0.0117404408],[0,70597,0.011252729],[0,70598,0.0032318038],[0,70601,0.0078290878],[0,70606,0.0648164353],[0,70613,0.0515172338],[0,70617,0.0092009408],[0,70627,0.011487438],[0,70630,0.0104212795],[0,70637,0.0135973682],[0,70638,0.0752318271],[0,70650,0.0042451049],[0,70652,0.0069171954],[0,70658,0.0030636846],[0,70670,0.009170205],[0,70676,0.017514791],[0,70685,0.0157228037],[0,70690,0.3073879841],[0,70694,0.0104212795],[0,70696,0.0042092584],[0,70698,0.1234594636],[0,70701,0.0446349085],[0,70722,0.0084404661],[0,70723,0.1831389534],[0,70726,0.0151249361],[0,70745,0.0036724135],[0,70755,0.0051470745],[0,70769,0.010251363],[0,70775,0.0156901513],[0,70779,0.009170205],[0,70780,0.0143704367],[0,70781,0.0201907936],[0,70782,0.0156901513],[0,70788,0.0032318038],[0,70795,0.0321700125],[0,70798,0.0082565065],[0,70799,0.0078290878],[0,70811,0.0104281283],[0,70816,0.0118821925],[0,70818,0.0215936441],[0,70820,0.0212301851],[0,70826,0.0078290878],[0,70831,0.0179387542],[0,70832,0.0078290878],[0,70835,0.1787052204],[0,70840,0.0245557148],[0,70841,0.0078290878],[0,70843,0.0032318038],[0,70849,0.0095182162],[0,70850,0.0078290878],[0,70852,0.0226667996],[0,70857,0.0145166071],[0,70861,0.0152029238],[0,70862,0.0078290878],[0,70867,0.009170205],[0,70868,0.094690281],[0,70872,0.0338752975],[0,70884,0.0156901513],[0,70887,0.1036911523],[0,70897,0.4374447738],[0,70898,0.013702286],[0,70904,0.0411825597],[0,70910,0.0036724135],[0,70912,0.0333479123],[0,70916,0.0208269937],[0,70922,0.0270233749],[0,70930,0.0155468883],[0,70935,0.0396504137],[0,70944,0.4544409787],[0,70945,0.017514791],[0,70946,0.0152029238],[0,70955,0.0111204826],[0,70956,0.0996471025],[0,70958,0.0040767324],[0,70960,0.0074283616],[0,70963,0.2278080704],[0,70968,0.0031960112],[0,70970,0.0156901513],[0,70972,0.0927438384],[0,70973,0.0040767324],[0,70974,0.0036724135],[0,70984,0.0084969817],[0,70985,0.0042092584],[0,70988,0.011487438],[0,70996,0.0128086686],[0,70998,0.1721955683],[0,71004,0.0030636846],[0,71014,0.3876876726],[0,71019,0.038418216],[0,71021,0.024196062],[0,71024,0.4035407181],[0,71030,0.0180623008],[0,71033,0.0036724135],[0,71035,0.2029807323],[0,71037,0.0095182162],[0,71045,0.0036724135],[0,71060,0.009170205],[0,71063,0.0133142852],[0,71070,0.010251363],[0,71077,0.0047240369],[0,71078,0.0156901513],[0,71080,0.0164707881],[0,71084,0.0045730531],[0,71086,0.0128086686],[0,71088,0.006848735],[0,71090,0.0069171954],[0,71095,0.0309458103],[0,71098,0.0156901513],[0,71100,0.0051831486],[0,71102,0.0078290878],[0,71111,0.0307887363],[0,71121,0.1731727597],[0,71127,0.1043176382],[0,71128,0.0031960112],[0,71131,0.0094827898],[0,71133,0.0232523197],[0,71144,0.0078290878],[0,71149,0.0156901513],[0,71150,0.2100212701],[0,71153,0.0078290878],[0,71154,0.0156901513],[0,71158,0.0201982283],[0,71162,0.1391640774],[0,71167,0.2977773925],[0,71183,0.0042053951],[0,71189,0.1700402171],[0,71190,0.4186817994],[0,71196,0.0542003434],[0,71202,0.0082565065],[0,71205,0.0078290878],[0,71207,0.0157427455],[0,71209,0.0675488508],[0,71216,0.0078290878],[0,71220,0.0136136928],[0,71224,0.0172522919],[0,71238,0.1303855843],[0,71244,0.0128086686],[0,71265,0.0082727033],[0,71266,0.0192075379],[0,71267,0.0078290878],[0,71268,0.2652473872],[0,71270,0.0051174989],[0,71283,0.0173533452],[0,71293,0.0038406849],[0,71305,0.0155468883],[0,71309,0.0155468883],[0,71314,0.0083087662],[0,71323,0.0112984584],[0,71336,0.0551531113],[0,71340,0.0076655255],[0,71343,0.1866008727],[0,71349,0.0144706615],[0,71350,0.0162186982],[0,71353,0.0142391926],[0,71357,0.0078290878],[0,71366,0.009170205],[0,71371,0.034045128],[0,71374,0.0246942827],[0,71381,0.0125935624],[0,71385,0.1184367608],[0,71388,0.0096268029],[0,71391,0.0032318038],[0,71392,0.0597499371],[0,71393,0.1901382324],[0,71411,0.011317255],[0,71412,0.0117404408],[0,71421,0.0078290878],[0,71422,0.0156901513],[0,71439,0.0082688167],[0,71446,0.0244613452],[0,71465,0.0074283616],[0,71466,0.0156901513],[0,71477,0.0051174989],[0,71482,0.035958677],[0,71484,0.0078290878],[0,71489,0.1369134752],[0,71491,0.0078290878],[0,71492,0.0156901513],[0,71494,0.0156901513],[0,71496,0.0828188495],[0,71500,0.0141800115],[0,71506,0.0078290878],[0,71508,0.3641090592],[0,71513,0.0777544619],[0,71516,0.0156901513],[0,71520,0.0509021756],[0,71521,0.010251363],[0,71539,0.0059083748],[0,71543,0.0042451049],[0,71547,0.0040767324],[0,71548,0.0036724135],[0,71556,0.0082565065],[0,71557,0.0133618438],[0,71571,0.0078290878],[0,71574,0.0078290878],[0,71581,0.0173533452],[0,71584,0.0156901513],[0,71589,0.009741894],[0,71592,0.0156901513],[0,71597,0.0273781741],[0,71599,0.0156901513],[0,71601,0.011317255],[0,71606,0.1280804317],[0,71609,0.0135973682],[0,71611,0.0156901513],[0,71612,0.0155468883],[0,71613,0.0042451049],[0,71617,0.009170205],[0,71620,0.0078290878],[0,71622,0.0118821925],[0,71624,0.0040767324],[0,71629,0.0036724135],[0,71631,0.011317255],[0,71633,0.2179387714],[0,71637,0.0078290878],[0,71641,0.0176093764],[0,71643,0.1306009519],[0,71647,0.1010942363],[0,71648,0.0076655255],[0,71652,0.0078290878],[0,71654,0.0951797147],[0,71662,0.0069171954],[0,71665,0.0723886251],[0,71666,0.0051470745],[0,71667,0.0081393777],[0,71672,0.0156901513],[0,71673,0.042558667],[0,71678,0.0036724135],[0,71680,0.0135507048],[0,71682,0.0913850928],[0,71683,0.009170205],[0,71695,0.0101140965],[0,71696,0.0214068159],[0,71697,0.0109373001],[0,71702,0.0078290878],[0,71708,0.0051174989],[0,71719,0.052528567],[0,71724,0.0429924],[0,71725,0.0165898544],[0,71730,0.0155468883],[0,71733,0.2477957801],[0,71748,0.0090752248],[0,71749,0.3789062761],[0,71755,0.1232952279],[0,71756,0.1120840842],[0,71762,0.0072107532],[0,71779,0.0156901513],[0,71785,0.0128086686],[0,71788,0.011487438],[0,71795,0.0083087662],[0,71801,0.0127860397],[0,71802,0.0149151448],[0,71806,0.0118821925],[0,71812,0.0031960112],[0,71813,0.011317255],[0,71816,0.2024309522],[0,71818,0.0127671997],[0,71830,0.0156901513],[0,71837,0.0151120234],[0,71847,0.0040767324],[0,71848,0.018037137],[0,71853,0.0265682585],[0,71855,0.0192075379],[0,71857,0.3777182436],[0,71858,0.0078290878],[0,71859,0.2311631884],[0,71865,0.0078290878],[0,71871,0.0076294969],[0,71875,0.3463924749],[0,71878,0.2829453779],[0,71880,0.0036724135],[0,71884,0.0045370116],[0,71889,0.6038642505],[0,71897,0.0113367835],[0,71898,0.010251363],[0,71900,0.0227027044],[0,71903,0.009660141],[0,71912,0.0446254281],[0,71913,0.0099820714],[0,71916,0.0078290878],[0,71930,0.0155468883],[0,71936,0.0051831486],[0,71940,0.1091155108],[0,71961,0.02717529],[0,71966,0.009170205],[0,71970,0.0367654754],[0,71975,0.0685516615],[0,71986,0.0156901513],[0,71995,0.0359999582],[0,72006,0.0084969817],[0,72010,0.0078290878],[0,72014,0.0104212795],[0,72015,0.0222203647],[0,72024,0.0256872625],[0,72025,0.0532909497],[0,72034,0.0122511919],[0,72038,0.0078290878],[0,72044,0.0117205777],[0,72049,0.1400610599],[0,72051,0.0069171954],[0,72053,0.0078290878],[0,72057,0.5055169645],[0,72066,0.0326409176],[0,72069,0.0126934866],[0,72074,0.010251363],[0,72082,0.0415662933],[0,72087,0.0083087662],[0,72089,0.0075041703],[0,72092,0.0155468883],[0,72093,0.047479028],[0,72100,0.0249980503],[0,74265,0.0189162099],[0,74267,0.1565743789],[0,74276,0.0422298781],[0,74277,0.2012666946],[0,74281,0.0071524116],[0,74284,0.0282282072],[0,74286,0.0119435166],[0,74288,0.0234702731],[0,74298,0.0190601777],[0,74301,0.0125081531],[0,74313,0.3302079676],[0,74315,0.0065045661],[0,74316,0.2586967983],[0,74321,0.0073896204],[0,74325,0.4025610147],[0,74330,0.2339643993],[0,74338,0.2030538975],[0,74341,0.0290538407],[0,74346,0.0111604375],[0,74352,0.0115873211],[0,74353,0.0178323103],[0,74356,0.0168505873],[0,74375,0.0150910345],[0,74380,0.0069833123],[0,74406,0.0361299531],[0,74419,0.01728673],[0,74420,0.1268101038],[0,74431,0.0075588214],[0,74434,0.0162895015],[0,74585,0.0115899592],[0,74591,0.0116424759],[0,74597,0.0111604375],[0,74598,0.0292053791],[0,74600,0.0681768897],[0,74602,0.0084652287],[0,74604,0.0075588214],[0,74614,0.0271653551],[0,74619,0.0258664395],[0,74636,0.0494999552],[0,74647,0.0215992981],[0,74650,0.0115899592],[0,74660,0.06281657],[0,74665,0.0389652727],[0,74667,0.0317302771],[0,74669,0.0123528049],[0,74673,0.01593455],[0,74675,0.0118316176],[0,74677,0.0190601777],[0,74679,0.0656311414],[0,74680,0.0065045661],[0,74683,0.0065405348],[0,74707,0.0833128048],[0,74709,0.1625263875],[0,74710,0.0502616616],[0,74717,0.0117748238],[0,74732,0.2615623301],[0,74738,0.0158013937],[0,74741,0.0231471854],[0,74742,0.0190601777],[0,74746,0.0202687521],[0,74748,0.0102440586],[0,74751,0.0071524116],[0,74755,0.0116424759],[0,74763,0.0118316176],[0,74764,0.0838258505],[0,74766,0.1520080434],[0,74770,0.1430354766],[0,74773,0.0190601777],[0,74776,0.0343959473],[0,74778,0.0161039365],[0,74785,0.0160874448],[0,74789,0.0111604375],[0,74790,0.051090707],[0,74796,0.0385465735],[0,74799,0.0337773096],[0,74800,0.0190601777],[0,74801,0.018488226],[0,74802,0.0347925804],[0,74803,0.1982860479],[0,74809,0.0114722541],[0,74812,0.0255937616],[0,74819,0.0088724272],[0,75153,0.0102440586],[0,75162,0.0078883831],[0,75167,0.049470764],[0,75179,0.0063715884],[0,75181,0.1346986165],[0,75182,0.0497476101],[0,75184,0.0714136481],[0,75192,0.0071524116],[0,75530,0.0092222345],[0,75539,0.0290805019],[0,75550,0.4403270275],[0,75562,0.0092222345],[0,75567,0.0137811027],[0,75573,0.0105752521],[0,75579,0.0160325504],[0,75582,0.0145707089],[0,75601,0.0278018174],[0,75629,0.0278018174],[0,75632,0.0127042279],[0,75634,0.009088722],[0,75650,0.0142099003],[0,75655,0.0133234188],[0,75656,0.0137811027],[0,75657,0.0175863478],[0,75662,0.0415870737],[0,75663,0.0172934452],[0,75683,0.009088722],[0,75710,0.0131039147],[0,75713,0.0454806982],[0,75714,0.0127042279],[0,75715,0.0147022516],[0,75764,0.0292085849],[0,75789,0.0289309302],[0,75795,0.1671812264],[0,75800,0.0211309307],[0,75806,0.0133501988],[0,75812,0.0133501988],[0,75830,0.0115856325],[0,75835,0.0142099003],[0,75855,0.6896264872],[0,75867,0.0133501988],[0,75883,0.0147022516],[0,76070,0.068348872],[0,76088,0.0165693243],[0,76093,0.0212753619],[0,76100,0.0148916462],[0,76162,0.0231469458],[0,76167,0.0696414876],[0,77877,0.0191655142],[0,77889,0.0143640738],[0,77896,0.0172814929],[0,77898,0.0181216251],[0,77915,0.0273859106],[0,77918,0.0142099003],[0,77922,0.0129076916],[0,77936,0.0265381329],[0,77938,0.0097029059],[0,77939,0.0232478047],[0,77950,0.0619956142],[0,77976,0.3665893292],[0,77989,0.009088722],[0,77993,0.0364192134],[0,78000,0.0242134977],[0,78003,0.031138724],[0,78024,0.0168668006],[0,78031,0.0092222345],[0,78038,0.0112271793],[0,78048,0.0221903322],[0,78049,0.0859386188],[0,78057,0.0319058293],[0,78060,0.0874441731],[0,78075,0.0133501988],[0,78077,0.3499609512],[0,78105,0.0125337406],[0,79151,0.0658946396],[0,79165,0.0290805019],[0,79172,0.0909153362],[0,79186,0.0970599977],[0,79200,0.0152809764],[0,79203,0.0133501988],[0,79206,0.0212753619],[0,79216,0.0255462559],[0,79224,0.0133501988],[0,79229,0.0092222345],[0,79236,0.0282088475],[0,79267,0.0098726853],[0,79284,0.0137811027],[0,79330,0.1098362553],[0,79331,0.009088722],[0,79333,0.0300696469],[0,79342,0.0211309307],[0,79371,0.0274503031],[0,79374,0.0145707089],[0,79378,0.1448355728],[0,79408,0.0098726853],[0,79418,0.009088722],[0,79422,0.016114309],[0,79423,0.0092222345],[0,79428,0.0572410929],[0,79430,0.0133501988],[0,79431,0.0212753619],[0,79451,0.0342316399],[0,80704,0.0137811027],[0,80926,0.0223381864],[0,81055,0.0092222345],[0,81085,0.0133998879],[0,81094,0.0832033383],[0,81100,0.7972936268],[0,81148,0.0113944845],[0,81191,0.1253873694],[0,81196,0.0089681344],[0,81207,0.0103115077],[0,81209,0.0089681344],[0,81210,0.0108865174],[0,81212,0.0101808095],[0,81227,0.0068587435],[0,81240,0.0099342435],[0,81244,0.0089681344],[0,81247,0.0235255062],[0,81307,0.0142875676],[0,81309,0.0134779386],[0,82302,0.0089681344],[0,82425,0.004734014],[0,82449,0.0088035969],[0,82463,0.2670947164],[0,82519,0.0237294402],[0,82536,0.0113944845],[0,82538,0.0256003093],[0,82550,0.0264303084],[0,82556,0.0492122755],[0,82567,0.0049025509],[0,82603,0.0083263102],[0,82622,0.0090175045],[0,82633,0.0053442552],[0,83328,0.0952394956],[0,83341,0.0089681344],[0,83350,0.0251534857],[0,83569,0.7459816809],[0,83730,0.0128860673],[0,83780,0.1785393332],[0,83793,0.0093962722],[0,83795,0.0055129447],[0,83798,0.0103115077],[0,83810,0.008054708],[0,83811,0.0262845918],[0,83817,0.0289788586],[0,83819,0.1801425396],[0,83826,0.0089681344],[0,83833,0.0053442552],[0,83837,0.0575241185],[0,83846,0.7755111198],[0,83851,0.1560692254],[0,83861,0.009822317],[0,83862,0.0093962722],[0,83867,0.0168424222],[0,83871,0.0089681344],[0,83872,0.0160456218],[0,83874,0.0156755128],[0,83880,0.0049025509],[0,83911,0.0096684161],[0,83932,0.0124621695],[0,83938,0.0509065058],[0,83960,0.0053442552],[0,83973,0.0107604891],[0,83984,0.1172853771],[0,83994,0.008054708],[0,84006,0.0062110013],[0,84009,0.0431266007],[0,84021,0.0093962722],[0,84022,0.0093962722],[0,84023,0.147447557],[0,84028,0.0062110013],[0,84316,0.0986527898],[0,84633,0.1655300984],[0,84877,0.0057757964],[0,84896,0.0093962722],[0,84911,0.6171764874],[0,84922,0.0630124436],[0,84929,0.008054708],[0,84939,0.0089681344],[0,84970,0.004734014],[0,84971,0.3392805582],[0,84977,0.0298713555],[0,84983,0.0799793509],[0,84984,0.0842187078],[0,84986,0.0103115077],[0,84991,0.0238675549],[0,84994,0.495393646],[0,84997,0.0298011066],[0,85002,0.0158063821],[0,85006,0.0212464408],[0,85010,0.0103115077],[0,85012,0.0181554426],[0,85014,0.0168424222],[0,85020,0.0099921263],[0,85034,0.0089681344],[0,85039,0.4260872282],[0,85041,0.061503721],[0,85045,0.0504159267],[0,85050,0.0526227435],[0,85051,0.0083263102],[0,85062,0.0124583111],[0,85066,0.0168424222],[0,85072,0.0128860673],[0,85110,0.020299979],[0,85112,0.0101808095],[0,85118,0.0343977309],[0,85130,0.009822317],[0,85135,0.0540773226],[0,85136,0.0128108281],[0,85138,0.004734014],[0,85142,0.0249531878],[0,85161,0.0093962722],[0,85190,0.0108865174],[0,85203,0.0420054507],[0,85205,0.0104996866],[0,85211,0.0048666694],[0,86024,0.0113944845],[0,86072,0.0221715893],[0,86076,0.0586978771],[0,86090,0.080408825],[0,86101,0.0145768713],[0,86103,0.0168951048],[0,86109,0.0103115077],[0,86114,0.0093962722],[0,86116,0.0101808095],[0,86122,0.1538603899],[0,86167,0.0093962722],[0,86174,0.0174580919],[0,86175,0.3387192443],[0,86188,0.0176003166],[0,86209,0.0134779386],[0,86212,0.009822317],[0,86304,0.02935443],[0,86320,0.0685297556],[0,86368,0.0101808095],[0,86375,0.0186750354],[0,86378,0.0103115077],[0,86394,0.0140262231],[0,86399,0.0101808095],[0,86407,0.0065674444],[0,86413,0.0089681344],[0,86428,0.7000948598],[0,86435,0.0089681344],[0,86959,0.0186192197],[0,87013,0.0064944974],[0,87014,0.0876750921],[0,87302,0.009822317],[0,87304,0.0412916752],[0,87305,0.0677909629],[0,87309,0.1669130941],[0,87321,0.0767881436],[0,87323,0.016354375],[0,87328,0.0062110013],[0,87329,0.0101808095],[0,87335,0.0123857974],[0,87340,0.1435918773],[0,87351,0.0449517749],[0,87354,0.0048666694],[0,87357,0.1899226949],[0,87361,0.0334482397],[0,87381,0.004734014],[0,87389,0.0099342435],[0,87412,0.0089681344],[0,87437,0.0168424222],[0,87469,0.0062471324],[0,87472,0.02969711],[0,87478,0.0260716894],[0,87489,0.0062110013],[0,87500,0.0128860673],[0,87512,0.0166989182],[0,87522,0.0093962722],[0,87539,0.0081569174],[0,87568,0.016354375],[0,87574,0.06817297],[0,87580,0.0168424222],[0,87618,0.0103115077],[0,87619,0.1041917617],[0,87625,0.0055129447],[0,87643,0.004734014],[0,87672,0.0099558939],[0,87984,0.0053442552],[0,88482,0.0173877204],[0,88577,0.2095102914],[0,88580,0.004734014],[0,88585,0.0473503447],[0,88586,0.0168424222],[0,88591,0.0187019915],[0,88592,0.1224903595],[0,88593,0.1344521694],[0,88602,0.0227908576],[0,88604,0.0159972021],[0,88611,0.0189029035],[0,88624,0.1969428254],[0,88634,0.0062110013],[0,88639,0.0099921263],[0,88642,0.0103115077],[0,88643,0.0101808095],[0,88663,0.0260184015],[0,88688,0.0405835873],[0,88698,0.2057671616],[0,88705,0.0113944845],[0,88707,0.1164452924],[0,88709,0.0062110013],[0,88713,0.0089681344],[0,88716,0.0137089306],[0,88743,0.0113944845],[0,88759,0.0048666694],[0,88761,0.0168424222],[0,88770,0.0124621695],[0,88807,0.0048666694],[0,88826,0.0166989182],[0,88837,0.0157148696],[0,88851,0.0092164806],[0,88861,0.01892203],[0,88862,0.0102046017],[0,88863,0.072716635],[0,89048,0.0170936516],[0,89583,0.0171463535],[0,89704,0.0083027197],[0,89723,0.0132043804],[0,89725,0.0298515453],[0,89739,0.0091898627],[0,89744,0.0131358481],[0,89761,0.0055912744],[0,89767,0.0418277907],[0,89769,0.0503440548],[0,89770,0.0170936516],[0,89777,0.0308288769],[0,89780,0.0378225879],[0,89791,0.0051494084],[0,89846,0.0116437191],[0,89862,0.0092164806],[0,89879,0.0104295997],[0,89893,0.0091898627],[0,89899,0.0092164806],[0,89908,0.0049808098],[0,89917,0.0055912744],[0,89927,0.0104295997],[0,89931,0.0581265191],[0,89943,0.0351838232],[0,90324,0.0096447751],[0,90344,0.010070976],[0,90363,0.0375650969],[0,90770,0.0187076409],[0,90778,0.0169500951],[0,90784,0.0051494084],[0,90786,0.0095604274],[0,90861,0.0105603458],[0,90874,0.0049808098],[0,90881,0.0163593856],[0,90886,0.0171463535],[0,90891,0.0469494084],[0,90896,0.0154886314],[0,90900,0.1768863209],[0,90902,0.0096447751],[0,90904,0.0092164806],[0,90905,0.0116437191],[0,90913,0.0105603458],[0,90914,0.0128047021],[0,90941,0.0105603458],[0,90951,0.0049808098],[0,90955,0.0092164806],[0,90956,0.0830746414],[0,90970,0.003673165],[0,91019,0.0079006211],[0,91028,0.0079006211],[0,91031,0.1000418915],[0,91039,0.0044508697],[0,91046,0.0042824457],[0,91049,0.0082517835],[0,91054,0.0668569435],[0,91061,0.0042824457],[0,91076,0.008328085],[0,91077,0.0079006211],[0,91079,0.043541281],[0,91080,0.0294479817],[0,91087,0.0079006211],[0,91227,0.003673165],[0,91880,0.0038414367],[0,91883,0.0092418801],[0,91886,0.0856828326],[0,91891,0.5705387264],[0,91893,0.008328085],[0,91900,0.2513092672],[0,91905,0.0079006211],[0,91915,0.0044508697],[0,91916,0.246114602],[0,91918,0.0093001019],[0,91922,0.0408152185],[0,91923,0.0092658688],[0,91925,0.0268619761],[0,91928,0.0198177247],[0,91929,0.0267999563],[0,91930,0.0130241853],[0,91934,0.0421276849],[0,91935,0.0146900116],[0,91939,0.003673165],[0,91963,0.5583105289],[0,91970,0.0806059215],[0,91979,0.0038056116],[0,91980,0.1402165657],[0,91985,0.1196771226],[0,91986,0.0157625152],[0,91993,0.0104288873],[0,91994,0.0094501455],[0,91998,0.003673165],[0,92032,0.0670747338],[0,92034,0.0051478277],[0,92038,0.0092418801],[0,92048,0.0310645025],[0,92061,0.0079006211],[0,92072,0.0051839018],[0,92077,0.0529072414],[0,92085,0.0334617507],[0,92089,0.0156192371],[0,92102,0.0069886325],[0,92107,0.0092228397],[0,92111,0.3266522375],[0,92115,0.0384288304],[0,92116,0.0157625152],[0,92121,0.2378272303],[0,92124,0.0092418801],[0,92128,0.0225377042],[0,92132,0.0092418801],[0,92138,0.0201301509],[0,92440,0.0347396042],[0,92448,0.0290948696],[0,92478,0.0044508697],[0,92674,0.0110442847],[0,92678,0.0044508697],[0,92681,0.0054308776],[0,92683,0.1199571528],[0,92684,0.4975290984],[0,92685,0.186187427],[0,92686,0.0092418801],[0,92687,0.0119727801],[0,92688,0.0118825888],[0,92689,0.0252516295],[0,92690,0.0051478277],[0,92692,0.034719513],[0,92693,0.0089230013],[0,92694,0.0354598192],[0,92695,0.0772713641],[0,92696,0.0346618384],[0,92697,0.0519020555],[0,92698,0.0250858356],[0,92700,0.4285740486],[0,92707,0.0657735156],[0,92708,0.0284505104],[0,92710,0.0200759977],[0,92713,0.0137279361],[0,92714,0.222940854],[0,92720,0.0315569981],[0,92722,0.008328085],[0,92723,0.0242780062],[0,92725,0.0044508697],[0,92726,0.00860909],[0,92727,0.1567911072],[0,92732,0.0339754685],[0,92733,0.0103231523],[0,92734,0.0092418801],[0,92735,0.0051478277],[0,92737,0.0272543116],[0,92742,0.0443348459],[0,92743,0.0038056116],[0,92745,0.42054713],[0,92748,0.0092418801],[0,92750,0.0158151149],[0,92752,0.0265207711],[0,92753,0.008328085],[0,92764,0.0391462301],[0,92769,0.0091113876],[0,92770,0.0072598072],[0,92782,0.155232766],[0,92784,0.0038414367],[0,92787,0.0194635662],[0,92790,0.0156192371],[0,92797,0.0586995015],[0,92798,0.0155815446],[0,92802,0.0047133077],[0,92804,0.0196691354],[0,92805,0.0130241853],[0,92806,0.0157625152],[0,92808,0.0412061791],[0,92810,0.0051478277],[0,92818,0.0621524552],[0,92820,0.135519218],[0,92830,0.0038414367],[0,92832,0.1499473397],[0,92837,0.0092418801],[0,92842,0.008328085],[0,92845,0.0092418801],[0,92850,0.0076151899],[0,92852,0.0469580547],[0,92853,0.004820558],[0,92854,0.1471265505],[0,92855,0.0127278734],[0,92856,0.0894737757],[0,92865,0.0157625152],[0,92870,0.0092418801],[0,92874,0.0166259226],[0,92875,0.0055037097],[0,92876,0.008328085],[0,92877,0.0186299441],[0,92880,0.0051478277],[0,92883,0.5029571013],[0,92884,0.008328085],[0,92885,0.0482974355],[0,92886,0.0870887362],[0,92898,0.0485478665],[0,92900,0.0120389138],[0,92903,0.0082517835],[0,92904,0.0405146788],[0,92906,0.0079006211],[0,92907,0.0475410669],[0,92908,0.046424787],[0,92911,0.0103231523],[0,92913,0.0057945504],[0,92915,0.0537359607],[0,92916,0.0087534593],[0,92922,0.2628494801],[0,92923,0.0346136006],[0,92932,0.740976968],[0,92934,0.1394871777],[0,92935,0.0147825899],[0,92936,0.5315582435],[0,92938,0.0629474392],[0,92942,0.0487988585],[0,92944,0.0165621402],[0,92946,0.0383994746],[0,92951,0.0117372667],[0,92953,0.0117009412],[0,92954,0.0091113876],[0,92956,0.2130354789],[0,92957,0.1192630533],[0,92958,0.0246776654],[0,92960,0.2009796892],[0,92963,0.013669511],[0,92964,0.1916319119],[0,92965,0.0038414367],[0,92966,0.501222432],[0,92967,0.046261355],[0,92968,0.0102756012],[0,93007,0.015795171],[0,93012,0.0565040154],[0,93015,0.409111447],[0,93021,0.0274128314],[0,93023,0.5842072453],[0,93030,0.1407182742],[0,93032,0.003673165],[0,93034,0.0341920819],[0,93036,0.0098159847],[0,93037,0.0763601515],[0,93038,0.0051839018],[0,93055,0.069479329],[0,93062,0.0432331045],[0,93064,0.0745443393],[0,93066,0.0592232351],[0,93070,0.0235752573],[0,93081,0.0994296113],[0,93100,0.0079006211],[0,93138,0.0157625152],[0,93150,0.0188224215],[0,93153,0.3274490501],[0,93162,0.0648880401],[0,93165,0.7927751503],[0,93242,0.008328085],[0,93247,0.031640812],[0,93257,0.3238443992],[0,93261,0.0038056116],[0,93270,0.009725592],[0,93277,0.0127843354],[0,93279,0.1170516165],[0,93281,0.003673165],[0,93282,0.056332721],[0,93284,0.0390566397],[0,93285,0.0178717825],[0,93289,0.0051839018],[0,93290,0.0157625152],[0,93313,0.083471636],[0,93314,0.0241004895],[0,93325,0.1475477461],[0,93329,0.003673165],[0,93330,0.0092228397],[0,93335,0.007090681],[0,93336,0.0079006211],[0,93337,0.1421361735],[0,93344,0.7927929257],[0,93393,0.4105598164],[0,93397,0.0038056116],[0,93399,0.0688202589],[0,93407,0.0118123874],[0,93426,0.0393521973],[0,93427,0.0140391255],[0,93430,0.0102392547],[0,93442,0.0098159847],[0,93462,0.0113891569],[0,93478,0.0238879223],[0,93492,0.0091113876],[0,93498,0.0146130809],[0,93500,0.1686660647],[0,93501,0.0079006211],[0,93592,0.0038056116],[0,93597,0.1698726952],[0,93599,0.0087534593],[0,93607,0.0042824457],[0,93617,0.0102756012],[0,93629,0.0750830904],[0,93630,0.0092418801],[0,93632,0.0436974054],[0,93636,0.0214348741],[0,93637,0.0038056116],[0,93639,0.0047133077],[0,93651,0.0091113876],[0,93655,0.0495276173],[0,93658,0.0054308776],[0,93660,0.0405756255],[0,93663,0.0264032854],[0,93667,0.0235752573],[0,93687,0.1555659618],[0,93701,0.0092418801],[0,93718,0.1490875957],[0,93719,0.007090681],[0,93776,0.0038414367],[0,93779,0.0561157602],[0,93803,0.0092418801],[0,93816,0.0773219543],[0,93817,0.0089230013],[0,93823,0.0246284323],[0,93827,0.0042824457],[0,94133,0.0038056116],[0,94134,0.0411875865],[0,94147,0.0118123874],[0,94156,0.043615314],[0,94299,0.0150169235],[0,94312,0.0402265243],[0,94333,0.0079006211],[0,94737,0.0157625152],[0,95053,0.2374858693],[0,95054,0.0274191274],[0,95057,0.7101634316],[0,95065,0.3727182992],[0,95086,0.1331134828],[0,95087,0.0092418801],[0,95093,0.0072598072],[0,95139,0.0162749613],[0,95146,0.0295510502],[0,95153,0.122249497],[0,95156,0.0840389827],[0,95159,0.1361274491],[0,95165,0.0092418801],[0,95174,0.0937456178],[0,95179,0.0061146478],[0,95207,0.0414589966],[0,95231,0.0051478277],[0,95250,0.008328085],[0,95252,0.0147825899],[0,95253,0.0421024717],[0,95279,0.0146873981],[0,95281,0.0079006211],[0,95286,0.0051478277],[0,95289,0.0038414367],[0,95317,0.0836763689],[0,95324,0.0044508697],[0,95325,0.003673165],[0,95331,0.1226526725],[0,95340,0.039787628],[0,95343,0.0078740549],[0,95347,0.0771804397],[0,95350,0.0157625152],[0,95358,0.0051478277],[0,95363,0.0490664604],[0,95366,0.0092418801],[0,95368,0.0156192371],[0,95382,0.3352898574],[0,95410,0.0072598072],[0,95411,0.0079006211],[0,95418,0.4935719362],[0,95420,0.0091898627],[0,95422,0.0079006211],[0,95456,0.1792235676],[0,95464,0.3769730743],[0,95469,0.8300754913],[0,95471,0.8246429399],[0,95480,0.0419763137],[0,95485,0.0197462951],[0,95492,0.1014718293],[0,95554,0.008328085],[0,95557,0.5865587631],[0,95562,0.0157625152],[0,95580,0.0137279361],[0,95583,0.007090681],[0,95642,0.071727229],[0,95703,0.5439000454],[0,95708,0.0079006211],[0,95711,0.0458531739],[0,95714,0.0044508697],[0,95715,0.3112995101],[0,95729,0.0189815793],[0,95739,0.0038056116],[0,95787,0.0156192371],[0,95938,0.0158151149],[0,95943,0.0132116819],[0,95971,0.0180470015],[0,95990,0.0057584435],[0,96019,0.0181719719],[0,96021,0.0079006211],[0,96172,0.0175504169],[0,96205,0.013669511],[0,96220,0.003673165],[0,96258,0.1385489417],[0,96259,0.1039023197],[0,96260,0.0057584435],[0,96263,0.0092418801],[0,96266,0.0340821863],[0,96281,0.0303670837],[0,96283,0.0190821767],[0,96287,0.0204887046],[0,96289,0.0130954794],[0,96291,0.0102046017],[0,96292,0.0047133077],[0,96293,0.1536002923],[0,96294,0.1111961836],[0,96295,0.0328162482],[0,96298,0.1278594781],[0,96301,0.0079006211],[0,96304,0.0309273751],[0,96312,0.008328085],[0,96314,0.0055037097],[0,96318,0.025795621],[0,96319,0.0263447575],[0,96340,0.0103231523],[0,96342,0.0092418801],[0,96350,0.003673165],[0,96358,0.0468912083],[0,96360,0.0225472445],[0,96361,0.0137279361],[0,96363,0.0483912214],[0,96366,0.3227336325],[0,96368,0.0269117522],[0,96377,0.8552756271],[0,96380,0.0096223326],[0,96404,0.0984045806],[0,96407,0.0109415376],[0,96410,0.022196967],[0,96417,0.085922648],[0,96419,0.0394665929],[0,96421,0.1039660621],[0,96423,0.0442048731],[0,96424,0.0079006211],[0,96425,0.0195765925],[0,96427,0.0444823897],[0,96428,0.0091113876],[0,96430,0.0079006211],[0,96433,0.1630593257],[0,96439,0.008328085],[0,96474,0.0092418801],[0,96477,0.0158151149],[0,96484,0.0123785521],[0,96485,0.0979489613],[0,96488,0.0538236558],[0,96489,0.0042824457],[0,96490,0.0522932302],[0,96495,0.0336922255],[0,96496,0.0158684145],[0,96501,0.2602736787],[0,96504,0.0178717825],[0,96505,0.3619001406],[0,96511,0.017898295],[0,96512,0.0141103122],[0,96515,0.0047133077],[0,96516,0.1071815742],[0,96517,0.2671261184],[0,96518,0.10065047],[0,96519,0.0187300881],[0,96522,0.0416507928],[0,96524,0.3454623937],[0,96525,0.0455271891],[0,96526,0.0464835576],[0,96534,0.0550248188],[0,96535,0.1693274292],[0,96537,0.0092658688],[0,96543,0.0728499986],[0,96544,0.1153787688],[0,96547,0.0627302693],[0,96549,0.1867875942],[0,96553,0.0127843354],[0,96556,0.021840087],[0,96561,0.0051839018],[0,96562,0.0282134091],[0,96566,0.1866950585],[0,96569,0.0612098948],[0,96574,0.0582009823],[0,96578,0.0520734162],[0,96579,0.0963289872],[0,96581,0.0366396856],[0,96584,0.0453782278],[0,96586,0.0102046017],[0,96593,0.0158151149],[0,96596,0.138365681],[0,96597,0.249881022],[0,96599,0.0079006211],[0,96600,0.2575787152],[0,96601,0.0057945504],[0,96602,0.0316662237],[0,96606,0.3830689279],[0,96608,0.0234266958],[0,96609,0.0038414367],[0,96610,0.0089230013],[0,96611,0.0270839624],[0,96613,0.0190393654],[0,96614,0.0099095144],[0,96615,0.0492336721],[0,96619,0.0044508697],[0,96621,0.0282134091],[0,96623,0.324573764],[0,96624,0.0339543085],[0,96625,0.0051839018],[0,96626,0.0069886325],[0,96627,0.2053681771],[0,96628,0.027903437],[0,96631,0.0047133077],[0,96636,0.0057584435],[0,96641,0.0357732351],[0,96644,0.0079006211],[0,96646,0.1018100148],[0,96647,0.4591167518],[0,96651,0.0385443729],[0,96657,0.4988353541],[0,96660,0.0316500858],[0,96661,0.012725244],[0,96662,0.0292918591],[0,96669,0.1227351191],[0,96671,0.5024709019],[0,96673,0.063483306],[0,96676,0.058501052],[0,96677,0.070170563],[0,96678,0.0175873477],[0,96685,0.062209461],[0,96687,0.0274128314],[0,96688,0.0494225767],[0,96689,0.314441092],[0,96691,0.1296936596],[0,96693,0.0217516757],[0,96695,0.0944413912],[0,96696,0.0051478277],[0,96697,0.0590288283],[0,96698,0.2777115736],[0,96704,0.0350760178],[0,96706,0.0299368742],[0,96707,0.0079006211],[0,96708,0.0117372667],[0,96711,0.2128006483],[0,96716,0.0165621402],[0,96719,0.2467981393],[0,96720,0.0157625152],[0,96722,0.027263447],[0,96731,0.1977722477],[0,96732,0.0617666207],[0,96733,0.0137279361],[0,96734,0.0038414367],[0,96738,0.0038414367],[0,96742,0.0147403349],[0,96745,0.0804606105],[0,96748,0.0387557712],[0,96750,0.0147281058],[0,96753,0.003673165],[0,96758,0.0416386784],[0,96760,0.0092418801],[0,96762,0.0044508697],[0,96763,0.0162481221],[0,96764,0.0309281431],[0,96766,0.0213219009],[0,96768,0.0091113876],[0,96775,0.0038056116],[0,96776,0.0200176557],[0,96785,0.1803426495],[0,96790,0.0275849504],[0,96796,0.0089230013],[0,96797,0.0277923161],[0,96809,0.0057945504],[0,96810,0.0079006211],[0,96811,0.0387124255],[0,96812,0.0188898393],[0,96813,0.6643286357],[0,96817,0.2661398659],[0,96825,0.0238238888],[0,96826,0.0626929933],[0,96827,0.1627461382],[0,96836,0.0091113876],[0,96838,0.0102756012],[0,96871,0.0092418801],[0,96885,0.0288431882],[0,96886,0.0390214171],[0,96888,0.003673165],[0,96889,0.6129856896],[0,96893,0.1976438821],[0,96902,0.0274734637],[0,96910,0.0449756994],[0,96912,0.0553898438],[0,96915,0.0197759645],[0,96917,0.0188224215],[0,96918,0.0089230013],[0,96920,0.0103231523],[0,96921,0.0133628146],[0,96922,0.2503568423],[0,96923,0.0938795939],[0,96929,0.0295969853],[0,96940,0.0234588398],[0,96945,0.013964671],[0,96950,0.0468037499],[0,96952,0.6477369487],[0,96958,0.003673165],[0,96963,0.013669511],[0,96970,0.0764318339],[0,96973,0.0474244762],[0,96975,0.0213384243],[0,96984,0.008328085],[0,96986,0.0600533815],[0,96995,0.1508437919],[0,97001,0.003673165],[0,97006,0.0297344344],[0,97039,0.0144660011],[0,97067,0.003673165],[0,97072,0.003673165],[0,97088,0.0158943656],[0,97093,0.0290665475],[0,97094,0.0158151149],[0,97102,0.4438689652],[0,97103,0.0092418801],[0,97107,0.1474889138],[0,97117,0.0168616458],[0,97663,0.1961375708],[0,97750,0.0618538115],[0,97762,0.0198896529],[0,97793,0.0038056116],[0,97806,0.1518658737],[0,97816,0.1244560233],[0,97821,0.013669511],[0,97823,0.0038056116],[0,97833,0.0209132433],[0,97834,0.0286662317],[0,97840,0.0213219009],[0,97848,0.0038414367],[0,97860,0.0318248144],[0,97935,0.0387582769],[0,97945,0.034692959],[0,97949,0.0047133077],[0,97951,0.5648015611],[0,97963,0.0091113876],[0,97967,0.1356423788],[0,97988,0.003673165],[0,97994,0.0200451122],[0,97997,0.0157625152],[0,98000,0.0091113876],[0,98051,0.0168102218],[0,98052,0.0091113876],[0,98110,0.0087534593],[0,98134,0.0190607749],[0,98144,0.0145728352],[0,98146,0.2124423582],[0,98149,0.003673165],[0,98150,0.0038414367],[0,98154,0.4416209781],[0,98166,0.0079006211],[0,98174,0.012880728],[0,98189,0.0144108744],[0,98190,0.012880728],[0,98196,0.0147666338],[0,98246,0.0310645025],[0,98271,0.0133504234],[0,98283,0.0302371021],[0,98288,0.0193219093],[0,98297,0.003673165],[0,98365,0.193778233],[0,98370,0.0519020555],[0,98379,0.009725592],[0,98393,0.0645933424],[0,98402,0.0079006211],[0,98442,0.0877572361],[0,98447,0.0157625152],[0,98454,0.1023476182],[0,98458,0.0091113876],[0,98475,0.0296251707],[0,98487,0.0605236224],[0,98491,0.0480636777],[0,98496,0.0079006211],[0,98499,0.0175478661],[0,98575,0.115968178],[0,98577,0.0038056116],[0,98578,0.0300691848],[0,98584,0.0337235462],[0,98649,0.0047133077],[0,98666,0.0362657305],[0,98674,0.0079006211],[0,98732,0.0213219009],[0,98799,0.0910778658],[0,98806,0.0089230013],[0,98810,0.0689399278],[0,98814,0.1644634675],[0,98871,0.008328085],[0,98876,0.004820558],[0,98879,0.0242569352],[0,98894,0.4151687074],[0,98916,0.0130289127],[0,98919,0.0038056116],[0,98921,0.0556267172],[0,98949,0.0247545413],[0,98956,0.0157625152],[0,99023,0.1198963444],[0,99039,0.0157625152],[0,99054,0.0514596546],[0,99067,0.0038414367],[0,99117,0.0092418801],[0,99157,0.104913742],[0,99168,0.0199467532],[0,99175,0.0087534593],[0,99177,0.0057945504],[0,99181,0.0069886325],[0,99244,0.0095034673],[0,99250,0.2624609287],[0,99262,0.115191389],[0,99276,0.0157625152],[0,99283,0.0092418801],[0,99286,0.1051386241],[0,99288,0.0213923783],[0,99321,0.0079006211],[0,99327,0.2751096008],[0,99369,0.0914619979],[0,99400,0.003673165],[0,99484,0.0166623133],[0,99486,0.1363293871],[0,99489,0.544971323],[0,99501,0.0708981923],[0,99511,0.0080824093],[0,99513,0.2251984454],[0,99532,0.0368987691],[0,99541,0.0079006211],[0,99542,0.0079006211],[0,99546,0.0157625152],[0,99555,0.0299945515],[0,99560,0.0157625152],[0,99562,0.0092418801],[0,99572,0.3567989341],[0,99575,0.0157625152],[0,99597,0.047616761],[0,99602,0.0288763891],[0,99604,0.0079006211],[0,99608,0.0302555185],[0,99613,0.0419666956],[0,99634,0.004820558],[0,99636,0.0157625152],[0,99642,0.0051478277],[0,99646,0.0307116104],[0,99706,0.2649169499],[0,99711,0.0051478277],[0,99728,0.0064916899],[0,99740,0.1488966763],[0,99742,0.400373995],[0,99754,0.021694175],[0,99770,0.2853060448],[0,99772,0.2307942541],[0,99786,0.0900210093],[0,99806,0.0386866728],[0,99852,0.0440108838],[0,99936,0.017137793],[0,99937,0.0467125264],[0,99976,0.026497862],[0,99998,0.0079006211],[0,100015,0.0079006211],[0,100026,0.0044508697],[0,100048,0.0157625152],[0,100052,0.0157625152],[0,100064,0.0044508697],[0,100146,0.0079006211],[0,100151,0.1493447322],[0,100152,0.0051478277],[0,100180,0.0038414367],[0,100188,0.0069886325],[0,100282,0.0156192371],[0,100329,0.0211840962],[0,100334,0.1724712374],[0,100345,0.0069886325],[0,100374,0.008328085],[0,100377,0.3133434592],[0,100399,0.0092658688],[0,100415,0.0044508697],[0,100431,0.0162481221],[0,100451,0.0245484161],[0,100460,0.0171965147],[0,100472,0.0158151149],[0,100473,0.0174258848],[0,100481,0.0117009412],[0,100486,0.0091113876],[0,100526,0.6234274404],[0,100529,0.007090681],[0,100530,0.1804764997],[0,100532,0.0186858597],[0,100562,0.0091113876],[0,100565,0.0157625152],[0,100570,0.0286113219],[0,100588,0.0106168021],[0,100592,0.0299252778],[0,100640,0.0079006211],[0,100641,0.0265252612],[0,100690,0.0157625152],[0,100694,0.0424707071],[0,100699,0.0038414367],[0,100719,0.0044508697],[0,100722,0.1315626625],[0,100724,0.9598325517],[0,100750,0.3145232477],[0,100755,0.0335798421],[0,100758,0.0102046017],[0,100761,0.0091113876],[0,100781,0.1981269557],[0,100782,0.0092418801],[0,100784,0.0038414367],[0,100785,0.0079006211],[0,100786,0.0082439009],[0,100792,0.2797008539],[0,100793,0.2845759454],[0,100794,0.0042824457],[0,100795,0.0079006211],[0,100839,0.007090681],[0,100843,0.0593184969],[0,100846,0.0044508697],[0,100849,0.0038414367],[0,100853,0.0207269238],[0,100858,0.0038056116],[0,100861,0.0698091686],[0,100866,0.0038414367],[0,100867,0.0091113876],[0,100869,0.1978820348],[0,100870,0.0051478277],[0,100871,0.0888142343],[0,100881,0.0102756012],[0,100886,0.0118123874],[0,100889,0.0051478277],[0,100899,0.0092418801],[0,100901,0.0038056116],[0,100902,0.0670867912],[0,100910,0.19443664],[0,100912,0.0038056116],[0,100915,0.0186827495],[0,100916,0.0215375143],[0,100917,0.0130410677],[0,100929,0.244985932],[0,100930,0.0092418801],[0,100934,0.0103231523],[0,100937,0.0373573389],[0,100940,0.0234526911],[0,100942,0.0091113876],[0,100943,0.0079006211],[0,100945,0.0055037097],[0,100946,0.0089230013],[0,100948,0.0307472262],[0,100949,0.0123671783],[0,100950,0.0174258848],[0,100954,0.0560273594],[0,100955,0.0038414367],[0,100956,0.0594354344],[0,100972,0.2421403282],[0,100974,0.0044508697],[0,100978,0.0038056116],[0,100979,0.0061146478],[0,100981,0.0079006211],[0,100982,0.0764262944],[0,100984,0.0739529238],[0,100988,0.2612236143],[0,100990,0.0091113876],[0,100991,0.0154407673],[0,100994,0.5028411886],[0,100995,0.4580208167],[0,100998,0.9163346389],[0,100999,0.0154146241],[0,101003,0.012711787],[0,101005,0.0137266474],[0,101008,0.0051478277],[0,101009,0.0591008561],[0,101011,0.1603637061],[0,101016,0.0261241641],[0,101018,0.0157740322],[0,101019,0.1005074483],[0,101020,0.003673165],[0,101021,0.0489576687],[0,101022,0.5979558697],[0,101025,0.0148458581],[0,101028,0.017826499],[0,101030,0.0038056116],[0,101034,0.0113891569],[0,101060,0.0149934207],[0,101064,0.1850940839],[0,101120,0.0335381351],[0,101122,0.0222466482],[0,101123,0.0209436307],[0,101125,0.0057584435],[0,101127,0.2619525955],[0,101129,0.0251814035],[0,101134,0.0079006211],[0,101141,0.0038056116],[0,101149,0.00885732],[0,101153,0.0178519725],[0,101187,0.0157625152],[0,101205,0.0494155182],[0,101697,0.0264526107],[0,101704,0.0309453506],[0,101720,0.1427898749],[0,101733,0.0665786771],[0,101736,0.0031860342],[0,101766,0.0702210125],[0,101812,0.0510621394],[0,101816,0.0056238361],[0,101822,0.0140369966],[0,101827,0.8781380817],[0,101877,0.0108964406],[0,101880,0.006911365],[0,101896,0.006911365],[0,101897,0.0109907573],[0,101905,0.001361777],[0,101953,0.1331847632],[0,102035,0.0335877119],[0,102125,0.001361777],[0,102144,0.0900767849],[0,102148,0.0278079437],[0,102150,0.0065935815],[0,102157,0.0544683014],[0,102214,0.0060007086],[0,102245,0.089534642],[0,102762,0.0344442801],[0,102784,0.0067813207],[0,102798,0.0081582741],[0,102835,0.0014937687],[0,102866,0.015101203],[0,102898,0.1046815279],[0,102899,0.0134913751],[0,102910,0.0311209625],[0,102915,0.0062461687],[0,102949,0.0090308533],[0,102980,0.0127380145],[0,102983,0.0135112075],[0,102986,0.0483948304],[0,102995,0.0093015631],[0,103001,0.0759207226],[0,103005,0.0934122967],[0,103104,0.0008695689],[0,103109,0.0627373036],[0,103110,0.0049108316],[0,103115,0.020649718],[0,103116,0.0207022815],[0,103117,0.0057599045],[0,103121,0.0442061947],[0,103122,0.0116006488],[0,103124,0.0013086309],[0,103125,0.0021701921],[0,103127,0.7719834838],[0,103130,0.0412885074],[0,103143,0.0226465004],[0,103147,0.2824699059],[0,103151,0.0049108316],[0,103155,0.2415788551],[0,103158,0.0461079425],[0,103163,0.0273128799],[0,103166,0.3482502233],[0,103167,0.0239261543],[0,103168,0.0380120248],[0,103173,0.0404681637],[0,103174,0.1916579771],[0,103175,0.1419040835],[0,103176,0.0075961255],[0,103181,0.0052604435],[0,103182,0.0353618877],[0,103184,0.0342626674],[0,103185,0.0267657577],[0,103190,0.0133995303],[0,103196,0.0267234895],[0,103198,0.0062461687],[0,103200,0.0095790835],[0,103201,0.0021701921],[0,103204,0.1608585996],[0,103206,0.3182666701],[0,103207,0.2556391927],[0,103210,0.0736467593],[0,103213,0.3729966592],[0,103214,0.250986919],[0,103218,0.0127380145],[0,103220,0.0058926823],[0,103226,0.1313497422],[0,103231,0.0127380145],[0,103233,0.0127380145],[0,103234,0.0522936179],[0,103237,0.1043413674],[0,103241,0.0103577999],[0,103244,0.1432303476],[0,103249,0.0061162524],[0,103251,0.0061162524],[0,103254,0.0112526725],[0,103258,0.1368801971],[0,103265,0.0960488997],[0,103282,0.5817953259],[0,103305,0.0405237832],[0,103307,0.012790382],[0,103309,0.310810533],[0,103328,0.8266658319],[0,103329,0.000833902],[0,103332,0.0101828214],[0,103335,0.000833902],[0,103342,0.0049108316],[0,103356,0.0053364082],[0,103373,0.0061162524],[0,103374,0.0339514711],[0,103375,0.007322667],[0,103392,0.0447217047],[0,103395,0.0424473249],[0,103401,0.4018112646],[0,103417,0.0423377593],[0,103443,0.0589716299],[0,103455,0.4040312122],[0,103456,0.0061162524],[0,103463,0.0127380145],[0,103467,0.0059706918],[0,103485,0.0025245028],[0,103498,0.0049108316],[0,103516,0.1481647078],[0,103536,0.1866804781],[0,103572,0.0017375906],[0,103611,0.0640110722],[0,103613,0.0062461687],[0,103647,0.0211522205],[0,103652,0.2454666065],[0,103664,0.0013086309],[0,103676,0.1404160899],[0,103816,0.0399690954],[0,103818,0.0127380145],[0,103819,0.0110734249],[0,103830,0.1096581489],[0,103831,0.2869952341],[0,103833,0.0024519922],[0,103837,0.0958072966],[0,104020,0.0262315427],[0,104030,0.0021701921],[0,104031,0.6879017631],[0,104054,0.0155848331],[0,104085,0.0049108316],[0,104092,0.0380645338],[0,104100,0.0257501056],[0,104110,0.0042728469],[0,104121,0.0185401127],[0,104225,0.0541569636],[0,104296,0.0053364082],[0,104331,0.1150489312],[0,104333,0.0145554067],[0,104351,0.007322667],[0,104371,0.0369556526],[0,104501,0.0142589325],[0,104546,0.0014763113],[0,104552,0.0833229744],[0,104585,0.0049108316],[0,104620,0.0007020402],[0,104671,0.0183248311],[0,104691,0.0048826706],[0,104702,0.0062461687],[0,104775,0.0049108316],[0,104784,0.0342627634],[0,104893,0.0007020402],[0,104951,0.0007020402],[0,104959,0.0345446022],[0,105023,0.0126169537],[0,105027,0.0007020402],[0,105060,0.0255916278],[0,105061,0.0021701921],[0,105137,0.1568091624],[0,105141,0.0061162524],[0,105143,0.0049108316],[0,105230,0.012790382],[0,105246,0.0049108316],[0,105257,0.0127380145],[0,105264,0.0049108316],[0,105298,0.0074918512],[0,105302,0.2172733982],[0,105330,0.0058926823],[0,105331,0.0113602531],[0,105333,0.0057599045],[0,105334,0.0874521841],[0,105335,0.0134554332],[0,105340,0.0014763113],[0,105345,0.1296279688],[0,105349,0.047516081],[0,105351,0.0577852724],[0,105361,0.0017375906],[0,105384,0.0692647085],[0,105401,0.0333489248],[0,105402,0.0055800752],[0,105433,0.0007020402],[0,105439,0.0326817938],[0,105451,0.0461079425],[0,105453,0.0062461687],[0,105454,0.4969997235],[0,105456,0.0462234217],[0,105458,0.1309978628],[0,105459,0.080974818],[0,105460,0.0272594091],[0,105464,0.0008695689],[0,105473,0.0284775044],[0,105477,0.0007020402],[0,105482,0.1431608873],[0,105483,0.0413939132],[0,105487,0.0213847617],[0,105488,0.0116351415],[0,105491,0.4913380225],[0,105496,0.1379996082],[0,105497,0.0267837721],[0,105498,0.4111613416],[0,105499,0.008805327],[0,105545,0.1094501921],[0,105546,0.0133179318],[0,105548,0.0533797893],[0,105549,0.013215186],[0,105550,0.0500749388],[0,105553,0.0022061069],[0,105562,0.5968855115],[0,105563,0.2984686851],[0,105568,0.0017375906],[0,105571,0.010321548],[0,105575,0.0108860593],[0,105576,0.2712664127],[0,105582,0.0067277451],[0,105585,0.8358027966],[0,105586,0.30030261],[0,105589,0.036297686],[0,105592,0.0117624157],[0,105600,0.0127380145],[0,105603,0.0017375906],[0,105605,0.0314954071],[0,105608,0.0199240785],[0,105610,0.0204712243],[0,105613,0.0303048558],[0,105617,0.0471087666],[0,105620,0.2761019273],[0,105628,0.0061162524],[0,105631,0.0994761597],[0,105635,0.0775074315],[0,105637,0.9325085602],[0,105642,0.0096779357],[0,105650,0.0194891691],[0,105666,0.0021701921],[0,105715,0.0050918172],[0,105717,0.0074918512],[0,105727,0.0241185145],[0,105736,0.0007020402],[0,105741,0.012252887],[0,105750,0.1103863081],[0,105767,0.0049108316],[0,105793,0.0055262245],[0,105800,0.151317774],[0,105820,0.0007020402],[0,105832,0.0061162524],[0,105833,0.0127380145],[0,105848,0.0127380145],[0,105860,0.4397641395],[0,105893,0.000833902],[0,105907,0.2521909374],[0,105910,0.0093576685],[0,105927,0.1729625572],[0,105931,0.0201736944],[0,105967,0.2951420102],[0,105971,0.0014763113],[0,105976,0.5144590934],[0,105979,0.0331210633],[0,105987,0.0072391399],[0,106003,0.0112872597],[0,106016,0.0136338399],[0,106040,0.000833902],[0,106052,0.000833902],[0,106056,0.9201144397],[0,106062,0.0008695689],[0,106073,0.0028140594],[0,106098,0.0360967706],[0,106115,0.012595369],[0,106118,0.000833902],[0,106138,0.0024519922],[0,106139,0.0021701921],[0,106216,0.1314224168],[0,106380,0.0370892015],[0,106392,0.0127380145],[0,106456,0.0021701921],[0,106468,0.8170844904],[0,106489,0.0083839571],[0,106491,0.0014763113],[0,106508,0.0127380145],[0,106510,0.0048843827],[0,106511,0.3376979085],[0,106514,0.0723414957],[0,106520,0.0360171994],[0,106523,0.0053364082],[0,106528,0.1077438768],[0,106537,0.0855414483],[0,106538,0.1188959631],[0,106539,0.0391494143],[0,106543,0.0318385498],[0,106544,0.188887132],[0,106547,0.0169037787],[0,106549,0.2350521449],[0,106554,0.0510532328],[0,106562,0.0290816196],[0,106563,0.0062461687],[0,106565,0.008805327],[0,106567,0.000833902],[0,106572,0.0106542512],[0,106587,0.7668539795],[0,106592,0.180078795],[0,106593,0.0062461687],[0,106594,0.1872875193],[0,106599,0.1234742027],[0,106604,0.0240072236],[0,106618,0.0061162524],[0,106628,0.0021701921],[0,106636,0.0111080366],[0,106643,0.0382503531],[0,106644,0.0280419279],[0,106648,0.0129085728],[0,106649,0.0127380145],[0,106651,0.0235674466],[0,106666,0.0014763113],[0,106667,0.0117928981],[0,106689,0.0103577999],[0,106691,0.0151658938],[0,106716,0.0017375906],[0,106718,0.0008695689],[0,106725,0.0007020402],[0,106726,0.01140788],[0,106727,0.0008695689],[0,106731,0.0499304607],[0,106732,0.3173707367],[0,106735,0.506248374],[0,106738,0.0057599045],[0,106746,0.248191239],[0,106757,0.0857121303],[0,106758,0.0173933489],[0,106759,0.0058926823],[0,106762,0.0069108554],[0,106764,0.010127039],[0,106765,0.1208934865],[0,106766,0.0127380145],[0,106774,0.0140939604],[0,106785,0.0057599045],[0,106787,0.0049108316],[0,106789,0.0119177036],[0,106805,0.0017375906],[0,106810,0.0062461687],[0,106813,0.2099605689],[0,106816,0.0643526421],[0,106819,0.0396563514],[0,106820,0.0061162524],[0,106833,0.0137264769],[0,106838,0.0215542787],[0,106840,0.031255703],[0,106855,0.0284608777],[0,106863,0.0121608192],[0,106873,0.0377219606],[0,106877,0.0637792059],[0,106878,0.012790382],[0,106882,0.0062461687],[0,106883,0.1736916448],[0,106886,0.1595183216],[0,106888,0.1789613017],[0,106890,0.0049108316],[0,106893,0.2145135367],[0,106894,0.0223521002],[0,106904,0.0062461687],[0,106905,0.1370944564],[0,106907,0.1783626678],[0,106909,0.0118292285],[0,106917,0.2149618751],[0,106919,0.0061162524],[0,106921,0.0061162524],[0,106926,0.0092283916],[0,106931,0.3425118801],[0,106935,0.0062461687],[0,106937,0.0061162524],[0,106940,0.0127380145],[0,106941,0.0127380145],[0,106942,0.0162946752],[0,106945,0.0014763113],[0,106946,0.0180652772],[0,106947,0.0007020402],[0,106952,0.0061162524],[0,106953,0.000833902],[0,106957,0.0014763113],[0,106959,0.007322667],[0,106961,0.0049108316],[0,106962,0.0266405256],[0,106964,0.2129544055],[0,106966,0.0017375906],[0,106968,0.0439887969],[0,106971,0.012790382],[0,106978,0.0191160343],[0,106979,0.5879757558],[0,106987,0.0146713809],[0,106989,0.0326182921],[0,106990,0.0904656698],[0,106993,0.0176911826],[0,106996,0.0174226697],[0,107000,0.0082734135],[0,107005,0.0085875539],[0,107007,0.0145547902],[0,107008,0.0061162524],[0,107017,0.0062461687],[0,107026,0.012252887],[0,107027,0.0053364082],[0,107032,0.0305073219],[0,107037,0.0152457858],[0,107045,0.1945606835],[0,107049,0.032425992],[0,107053,0.1618474314],[0,107057,0.007322667],[0,107077,0.0137959869],[0,107079,0.1672050593],[0,107083,0.3160739415],[0,107090,0.0074279353],[0,107113,0.0062461687],[0,107114,0.0127380145],[0,107116,0.0194891691],[0,107124,0.0084807285],[0,107132,0.0049108316],[0,107139,0.005912172],[0,107148,0.2436962762],[0,107189,0.0127380145],[0,107211,0.0061162524],[0,107219,0.0138175555],[0,107220,0.3613470004],[0,107221,0.0057599045],[0,107222,0.0200958663],[0,107223,0.0443098184],[0,107225,0.4745732095],[0,107226,0.0470740807],[0,107227,0.0445833863],[0,107229,0.2236881281],[0,107231,0.0924736144],[0,107235,0.0107111353],[0,107236,0.633690544],[0,107239,0.0049108316],[0,107240,0.0310271181],[0,107241,0.0459898508],[0,107243,0.2142534238],[0,107245,0.01020315],[0,107246,0.1403285278],[0,107247,0.0422513942],[0,108041,0.0013086309],[0,108042,0.0062461687],[0,108043,0.0007020402],[0,108046,0.0127380145],[0,108054,0.0008695689],[0,108055,0.0053364082],[0,108056,0.0154018123],[0,108061,0.0147090564],[0,108062,0.0055262245],[0,108064,0.0062461687],[0,108065,0.0213847617],[0,108068,0.000833902],[0,108070,0.012310736],[0,108071,0.0059286979],[0,108073,0.0156064871],[0,108074,0.0129145155],[0,108077,0.1077913527],[0,108080,0.0028140594],[0,108082,0.008805327],[0,108085,0.0239373985],[0,108087,0.0061162524],[0,108089,0.0008695689],[0,108093,0.2586974182],[0,108098,0.0178519516],[0,108101,0.000833902],[0,108104,0.0103164692],[0,108109,0.0424340926],[0,108111,0.0127380145],[0,108114,0.0227756663],[0,108117,0.0127380145],[0,108118,0.0007020402],[0,108120,0.2505935007],[0,108121,0.1595021988],[0,108122,0.1961440178],[0,108123,0.0111431797],[0,108125,0.2240191627],[0,108128,0.0127380145],[0,108135,0.0377218008],[0,108137,0.0049108316],[0,108138,0.0096779357],[0,108139,0.0227131531],[0,108142,0.055261559],[0,108149,0.0127380145],[0,108151,0.06322169],[0,108156,0.0057599045],[0,108157,0.0209574281],[0,108163,0.0296443712],[0,108164,0.0154675442],[0,108167,0.0206675845],[0,108178,0.0106542512],[0,108186,0.0059286979],[0,108187,0.0468177896],[0,108190,0.0125797698],[0,108191,0.0049108316],[0,108193,0.0072391399],[0,108195,0.0216298664],[0,108196,0.0362700697],[0,108203,0.0017375906],[0,108204,0.007683909],[0,108205,0.0128402249],[0,108207,0.0677735513],[0,108210,0.0443111438],[0,108221,0.0500724337],[0,108228,0.0017375906],[0,108229,0.0288563028],[0,108232,0.0049599064],[0,108236,0.0637804323],[0,108239,0.0166595061],[0,108242,0.000833902],[0,108243,0.0049108316],[0,108245,0.0581097834],[0,108246,0.0170516878],[0,108251,0.0490825623],[0,108254,0.0062681849],[0,108257,0.0017375906],[0,108260,0.0007020402],[0,108261,0.0008695689],[0,108270,0.0049108316],[0,108274,0.1021080298],[0,108286,0.6152038842],[0,108287,0.0183461668],[0,108292,0.0150928898],[0,108297,0.0040028695],[0,108301,0.0218708337],[0,108304,0.0865073956],[0,108314,0.0017375906],[0,108316,0.0310772966],[0,108318,0.0057599045],[0,108319,0.0007020402],[0,108320,0.000833902],[0,108333,0.307619324],[0,108334,0.0127380145],[0,108336,0.0113464],[0,108354,0.012790382],[0,108359,0.3278616023],[0,108363,0.0581771187],[0,108364,0.4134861719],[0,108369,0.2090025765],[0,108371,0.0021701921],[0,108374,0.012252887],[0,108375,0.0217624253],[0,108376,0.0058926823],[0,108381,0.5081056941],[0,108390,0.0442061947],[0,108391,0.0058926823],[0,108393,0.0905376111],[0,108394,0.0062461687],[0,108397,0.0127380145],[0,108400,0.0236263761],[0,108404,0.0049599064],[0,108405,0.1206432321],[0,108408,0.0127380145],[0,108412,0.0930260434],[0,108416,0.0236693463],[0,108420,0.0196343864],[0,108422,0.0815424879],[0,108425,0.0139446385],[0,108426,0.1522825245],[0,108427,0.1330266604],[0,108431,0.0244953074],[0,108432,0.0068177387],[0,108439,0.0021701921],[0,108440,0.3074443165],[0,108442,0.0903598049],[0,108468,0.0049108316],[0,108473,0.0021701921],[0,108475,0.0150834752],[0,108477,0.0099659263],[0,108479,0.0192492673],[0,108481,0.3143712228],[0,108483,0.0127380145],[0,108484,0.0057599045],[0,108490,0.0217574782],[0,108498,0.0049108316],[0,108503,0.0127380145],[0,108510,0.0412885074],[0,108512,0.0127380145],[0,108520,0.6267511997],[0,108523,0.0950320403],[0,108525,0.018537609],[0,108526,0.0408539741],[0,108527,0.0340972037],[0,108528,0.01020315],[0,108530,0.0819784565],[0,108537,0.0372893482],[0,108538,0.0049108316],[0,108539,0.0371975993],[0,108543,0.0968721394],[0,108547,0.1835133664],[0,108556,0.0059286979],[0,108558,0.0223085795],[0,108567,0.0041044674],[0,108571,0.0007020402],[0,108572,0.0680552279],[0,108573,0.0053364082],[0,108574,0.0107587168],[0,108575,0.0049599064],[0,108576,0.0198648836],[0,108578,0.2302616922],[0,108579,0.2118447426],[0,108581,0.0062461687],[0,108583,0.8085579712],[0,108586,0.0359123037],[0,108588,0.0007020402],[0,108591,0.3276169283],[0,108592,0.0209508077],[0,108594,0.0048843827],[0,108603,0.0062461687],[0,108606,0.0374346237],[0,108610,0.010129714],[0,108616,0.0538098238],[0,108617,0.000833902],[0,108618,0.0258405454],[0,108619,0.0125013606],[0,108622,0.0254761146],[0,108625,0.0042369989],[0,108626,0.0276458989],[0,108627,0.0814903474],[0,108632,0.0049108316],[0,108633,0.2702775063],[0,108634,0.0057599045],[0,108637,0.1384344899],[0,108640,0.1926455388],[0,108642,0.0023565604],[0,108643,0.0085534147],[0,108646,0.0017375906],[0,108651,0.000833902],[0,108654,0.012595369],[0,108658,0.0069108554],[0,108660,0.2431510965],[0,108663,0.0144480495],[0,108671,0.0083839651],[0,108674,0.0394308982],[0,108681,0.1340493615],[0,108687,0.0014763113],[0,108689,0.012595369],[0,108691,0.0049108316],[0,108699,0.0138092639],[0,108701,0.4192331425],[0,108702,0.019365348],[0,108703,0.0014763113],[0,108704,0.0049108316],[0,108707,0.0014763113],[0,108708,0.0018443673],[0,108710,0.0062461687],[0,108713,0.0057599045],[0,108719,0.0133421168],[0,108732,0.3722555738],[0,108734,0.0567263745],[0,108737,0.0062461687],[0,108738,0.0025245028],[0,108740,0.0104068564],[0,108742,0.0112149904],[0,108750,0.0118292285],[0,108753,0.0072391399],[0,108754,0.0127380145],[0,108759,0.1946408462],[0,108764,0.0116589599],[0,108765,0.0049108316],[0,108769,0.0195838336],[0,108772,0.01442795],[0,108775,0.0686634239],[0,108777,0.0007020402],[0,108783,0.0083205672],[0,108784,0.0049108316],[0,108789,0.007322667],[0,108791,0.2382339755],[0,108795,0.0180056273],[0,108798,0.008805327],[0,108799,0.0915647754],[0,108808,0.0358260794],[0,108809,0.361382166],[0,108812,0.0208442223],[0,108818,0.0159417776],[0,108826,0.0127380145],[0,108830,0.0877961206],[0,108835,0.0940741474],[0,108836,0.4490666039],[0,108837,0.1186575919],[0,108839,0.0106542512],[0,108840,0.0007020402],[0,108841,0.0013086309],[0,108843,0.0211501356],[0,108855,0.0008695689],[0,108857,0.0312880885],[0,108861,0.002778112],[0,108865,0.064523168],[0,108867,0.0014763113],[0,108868,0.0048826706],[0,108871,0.0017375906],[0,108872,0.0127380145],[0,108874,0.0076150203],[0,108879,0.0648594681],[0,108880,0.1117320937],[0,108882,0.0127380145],[0,108930,0.0881796069],[0,108931,0.0127380145],[0,108935,0.0635666642],[0,108936,0.0093936577],[0,108938,0.0023565604],[0,108940,0.0375657627],[0,108942,0.026349781],[0,108943,0.0062461687],[0,108949,0.0284983394],[0,108950,0.0466289571],[0,108955,0.0189243091],[0,108958,0.1094081822],[0,108964,0.0942571654],[0,108975,0.0575942227],[0,108990,0.6052696297],[0,108993,0.2007067827],[0,109001,0.0083080498],[0,109003,0.0127380145],[0,109005,0.0127380145],[0,109006,0.0083839571],[0,109012,0.0053364082],[0,109015,0.036054847],[0,109016,0.0662287819],[0,109018,0.0117928981],[0,109021,0.0563539561],[0,109022,0.0542195956],[0,109025,0.0007020402],[0,109029,0.0062461687],[0,109045,0.0384761318],[0,109049,0.2342406319],[0,109051,0.0007020402],[0,109055,0.0216785137],[0,109062,0.0067277451],[0,109069,0.0192492477],[0,109072,0.0008695689],[0,109075,0.0049108316],[0,109089,0.0136338399],[0,109091,0.0610419407],[0,109094,0.0537540119],[0,109095,0.0013086309],[0,109096,0.0183213838],[0,109098,0.030400263],[0,109100,0.7591582923],[0,109102,0.036730696],[0,109103,0.0112149904],[0,109104,0.0096988795],[0,109107,0.0127260385],[0,109109,0.0062461687],[0,109110,0.0049108316],[0,109115,0.0059286979],[0,109117,0.0170718998],[0,109126,0.0180011922],[0,109130,0.2475519619],[0,109141,0.0049108316],[0,109150,0.0118292285],[0,109153,0.0028140594],[0,109160,0.0127380145],[0,109161,0.0103527208],[0,109162,0.0079119705],[0,109171,0.0127380145],[0,109185,0.0062461687],[0,109194,0.0061162524],[0,109195,0.0061352462],[0,109197,0.0926362024],[0,109201,0.0528083036],[0,109209,0.0049108316],[0,109224,0.0151716086],[0,109226,0.0484496039],[0,109229,0.0021701921],[0,109240,0.1625427857],[0,109249,0.012595369],[0,109252,0.0129132083],[0,109258,0.0021701921],[0,109264,0.3160919554],[0,109269,0.0061162524],[0,109286,0.0049108316],[0,109288,0.0106542512],[0,109290,0.0069108554],[0,109291,0.0062461687],[0,109297,0.0021701921],[0,109300,0.0072391399],[0,109303,0.0257217914],[0,109305,0.0013086309],[0,109309,0.0118978507],[0,109310,0.0127380145],[0,109311,0.000833902],[0,109321,0.000833902],[0,109323,0.0053364082],[0,109329,0.0074918512],[0,109331,0.000833902],[0,109340,0.0041044674],[0,109341,0.0945404694],[0,109351,0.0198767775],[0,109353,0.0053364082],[0,109369,0.0127380145],[0,109373,0.0127380145],[0,109380,0.0018443673],[0,109386,0.0136338399],[0,109392,0.0336779342],[0,109395,0.1685358153],[0,109396,0.0222961268],[0,109397,0.0197216874],[0,109403,0.0053364082],[0,109411,0.0341999335],[0,109412,0.0049108316],[0,109441,0.0095461754],[0,109452,0.0008695689],[0,109453,0.014731241],[0,109463,0.0072753259],[0,109466,0.0063041335],[0,109467,0.0170516878],[0,109469,0.0046266606],[0,109472,0.0014763113],[0,109473,0.1092169379],[0,109577,0.012595369],[0,109590,0.2660139758],[0,109607,0.1830890141],[0,109609,0.0127380145],[0,109614,0.0053364082],[0,109629,0.0083839651],[0,109632,0.0049108316],[0,109633,0.0127380145],[0,109634,0.0404108068],[0,109641,0.0007020402],[0,109643,0.0125481013],[0,109644,0.0127380145],[0,109650,0.0081840386],[0,109651,0.0754769317],[0,109652,0.0062461687],[0,109657,0.0013086309],[0,109658,0.0944305718],[0,109666,0.0028140594],[0,109667,0.0008695689],[0,109673,0.0007020402],[0,109676,0.0013086309],[0,109684,0.0062461687],[0,109687,0.0127380145],[0,109688,0.0059286979],[0,109689,0.0083839571],[0,109692,0.0049599064],[0,109695,0.4274254967],[0,109700,0.4509876217],[0,109701,0.0061162524],[0,109711,0.0007020402],[0,109722,0.056909092],[0,109723,0.0221420213],[0,109726,0.0007020402],[0,109730,0.082466983],[0,109732,0.012595369],[0,109733,0.0507291212],[0,109737,0.000833902],[0,109738,0.0022061069],[0,109743,0.0158604148],[0,109778,0.0577258286],[0,109780,0.0053364082],[0,109784,0.0127380145],[0,109792,0.008805327],[0,109802,0.0017375906],[0,109808,0.0596007205],[0,109809,0.0198726506],[0,109821,0.0127380145],[0,109831,0.0364243696],[0,109833,0.0232780435],[0,109841,0.0062461687],[0,109850,0.0117624157],[0,109855,0.0168053656],[0,109857,0.0007020402],[0,109871,0.01140429],[0,109873,0.002778112],[0,109880,0.0049108316],[0,109883,0.0077841431],[0,109887,0.0067277451],[0,109895,0.0083839651],[0,109905,0.0008695689],[0,109925,0.0014763113],[0,109928,0.0049108316],[0,109940,0.0105494072],[0,109951,0.0216230282],[0,109954,0.0066055795],[0,109956,0.0342299171],[0,109972,0.0882649115],[0,109975,0.1049889592],[0,109976,0.005243069],[0,109983,0.0025274342],[0,109998,0.3606727043],[0,110002,0.0010585012],[0,110006,0.0086685576],[0,110007,0.2367883272],[0,110008,0.0059391316],[0,110017,0.0066055795],[0,110018,0.0025274342],[0,110024,0.0261614492],[0,110033,0.0066055795],[0,110061,0.0010585012],[0,110062,0.0830064774],[0,110072,0.0129581579],[0,110090,0.4605773451],[0,110098,0.005269532],[0,110110,0.0566267852],[0,110132,0.0077879748],[0,110140,0.0070521125],[0,110143,0.0153919352],[0,110145,0.0228775498],[0,110148,0.0131008793],[0,110152,0.1433935734],[0,110162,0.0427959292],[0,110170,0.006475594],[0,110175,0.0655881507],[0,110176,0.006475594],[0,110225,0.4203151813],[0,110280,0.0271111385],[0,110282,0.0011904332],[0,110284,0.0618810431],[0,110292,0.0694748847],[0,110293,0.1018897196],[0,110296,0.0066055795],[0,110301,0.005269532],[0,110302,0.0010585012],[0,110305,0.018671289],[0,110306,0.0076826505],[0,110314,0.0010585012],[0,110315,0.005241356],[0,110320,0.0228123788],[0,110321,0.0409996433],[0,110322,0.0138745037],[0,110330,0.101325955],[0,110332,0.005695335],[0,110334,0.0011904332],[0,110340,0.060269377],[0,110343,0.0076826505],[0,110345,0.016590017],[0,110377,0.0131008793],[0,110381,0.0514571621],[0,110382,0.4330014693],[0,110389,0.0122602685],[0,110393,0.0228123788],[0,110395,0.0121916097],[0,110400,0.0190071898],[0,110407,0.0012261191],[0,110415,0.0043610867],[0,110425,0.2511376891],[0,110428,0.0069653409],[0,110430,0.0137627471],[0,110431,0.005243069],[0,110469,0.0066055795],[0,110476,0.0283076276],[0,110481,0.0062879398],[0,110485,0.0011904332],[0,110491,0.0044627387],[0,110494,0.0131008793],[0,110496,0.005269532],[0,110505,0.0131008793],[0,110510,0.0066055795],[0,110513,0.2416129688],[0,110518,0.1337730823],[0,110520,0.0066198449],[0,110523,0.005269532],[0,110531,0.0308995893],[0,110536,0.0018331842],[0,110539,0.0131008793],[0,110545,0.0138186797],[0,110547,0.0115770449],[0,110548,0.0139609321],[0,110554,0.0062879398],[0,110555,0.005269532],[0,110566,0.0699797298],[0,110579,0.0012261191],[0,110582,0.0639211687],[0,110585,0.005695335],[0,110595,0.0171948018],[0,110612,0.0010585012],[0,110614,0.0010585012],[0,110620,0.006475594],[0,110621,0.0066055795],[0,110630,0.0131008793],[0,110643,0.0131008793],[0,110645,0.005695335],[0,110647,0.005269532],[0,110654,0.0338960856],[0,110663,0.007212969],[0,110671,0.005695335],[0,110672,0.0076826505],[0,110675,0.0010585012],[0,110676,0.0131008793],[0,110682,0.0244041052],[0,110694,0.0012261191],[0,110701,0.0043610867],[0,110706,0.0149186216],[0,110707,0.006475594],[0,110709,0.006475594],[0,110715,0.0043610867],[0,110716,0.0087445133],[0,110718,0.0016654146],[0,110728,0.0129581579],[0,110732,0.0068130358],[0,110738,0.0020946026],[0,110745,0.0012261191],[0,110747,0.0213963013],[0,110748,0.1340205444],[0,110753,0.5139041333],[0,110757,0.0169373334],[0,110759,0.0087786621],[0,110775,0.0132761663],[0,110779,0.0129581579],[0,110780,0.0010585012],[0,110781,0.0131008793],[0,110787,0.005269532],[0,110788,0.0271464845],[0,110805,0.0025274342],[0,110806,0.0630860816],[0,110812,0.0131008793],[0,110822,0.0427830024],[0,110824,0.0031356776],[0,110832,0.006475594],[0,110837,0.0062879398],[0,110839,0.0569038175],[0,110842,0.0011904332],[0,110845,0.0062879398],[0,110850,0.0287320132],[0,110851,0.0010585012],[0,110854,0.3101760808],[0,110860,0.0330750459],[0,110861,0.0137277671],[0,110871,0.0066055795],[0,110872,0.7955215454],[0,110873,0.0096825169],[0,110885,0.1481196157],[0,110888,0.0917532143],[0,110891,0.2303963978],[0,110917,0.0245361764],[0,110923,0.0186470809],[0,110926,0.0120212506],[0,110928,0.1894301558],[0,110936,0.0194184824],[0,110937,0.0764210728],[0,110942,0.0767680673],[0,110943,0.0409601508],[0,110945,0.0130888969],[0,110949,0.2250745578],[0,110954,0.0199185491],[0,110958,0.0366830339],[0,110959,0.0131008793],[0,110965,0.0018331842],[0,110966,0.0010585012],[0,110972,0.2902440165],[0,110975,0.0131008793],[0,111004,0.0206679951],[0,111005,0.0227200802],[0,111006,0.0196141121],[0,111012,0.0295356077],[0,111021,0.0110160073],[0,111023,0.0755587383],[0,111026,0.0843387225],[0,111028,0.2943644463],[0,111029,0.0075776023],[0,111032,0.0200882679],[0,111042,0.0185513221],[0,111062,0.1589961135],[0,111075,0.1170979513],[0,111090,0.0199560053],[0,111092,0.0043610867],[0,111098,0.0066055795],[0,111119,0.2364813577],[0,111135,0.0159791444],[0,111140,0.0129581579],[0,111154,0.0012261191],[0,111158,0.005695335],[0,111160,0.1274263358],[0,111161,0.0131008793],[0,111164,0.006251905],[0,111169,0.006271405],[0,111173,0.0046312078],[0,111176,0.0388060918],[0,111183,0.0011904332],[0,111186,0.1048258193],[0,111192,0.2981119555],[0,111205,0.0025633682],[0,111206,0.0296437573],[0,111215,0.0243816548],[0,111217,0.0146151919],[0,111222,0.0075776023],[0,111226,0.005269532],[0,111240,0.0016654146],[0,111248,0.005269532],[0,111255,0.0333580131],[0,111256,0.0227336009],[0,111276,0.0196876237],[0,111281,0.019307016],[0,111290,0.005269532],[0,111296,0.005269532],[0,111302,0.0107612526],[0,111312,0.0268541283],[0,111317,0.0010585012],[0,111318,0.0200284131],[0,111321,0.0076352843],[0,111332,0.0294185003],[0,111338,0.005695335],[0,111339,0.3919667853],[0,111359,0.0131008793],[0,111374,0.0218389274],[0,111425,0.0057164696],[0,111434,0.0010585012],[0,111436,0.0192908383],[0,111448,0.0992518091],[0,111524,0.0149186216],[0,111528,0.6177469111],[0,111556,0.005269532],[0,111562,0.8409029805],[0,111579,0.0077517614],[0,111595,0.0532202705],[0,111597,0.0231933808],[0,111612,0.0412599513],[0,111616,0.005243069],[0,111625,0.0025633682],[0,111631,0.0011904332],[0,111635,0.0112248661],[0,111648,0.0011904332],[0,111649,0.1182440496],[0,111652,0.0061190565],[0,111653,0.0010585012],[0,111655,0.0126182779],[0,111656,0.0079751594],[0,111660,0.0288918071],[0,111662,0.006475594],[0,111663,0.0066055795],[0,111667,0.0546159331],[0,111678,0.0010585012],[0,111694,0.0131008793],[0,111696,0.0093917457],[0,111699,0.0412369856],[0,111702,0.0011904332],[0,111703,0.0750810407],[0,111704,0.7221724152],[0,111706,0.0066055795],[0,111730,0.005269532],[0,111732,0.0025633682],[0,111739,0.1226150037],[0,111786,0.0285392803],[0,111792,0.0025274342],[0,111796,0.0062879398],[0,111815,0.0275702318],[0,111818,0.0076826505],[0,111833,0.0131008793],[0,111837,0.0020946026],[0,111839,0.0062879398],[0,111847,0.0087445133],[0,111855,0.0012261191],[0,111856,0.008914053],[0,111858,0.0200268643],[0,111862,0.0010585012],[0,111866,0.0091660994],[0,111874,0.0066055795],[0,111881,0.0428259966],[0,111882,0.0131008793],[0,111886,0.0912271326],[0,111896,0.0178919801],[0,111903,0.5847713834],[0,111905,0.0104534053],[0,111913,0.0146850591],[0,111916,0.0131008793],[0,111933,0.0131783311],[0,111934,0.0347513455],[0,111946,0.0302849323],[0,111947,0.0018331842],[0,111991,0.0527680286],[0,112006,0.0138546408],[0,112013,0.0010585012],[0,112015,0.0110160073],[0,112018,0.0046312078],[0,112034,0.0043610867],[0,112036,0.014680733],[0,112037,0.0870330666],[0,112040,0.0131008793],[0,112043,0.5133126796],[0,112050,0.3664227123],[0,112052,0.0062879398],[0,112054,0.0087445133],[0,112059,0.0105646661],[0,112064,0.0156523862],[0,112068,0.0167096],[0,112075,0.1918509912],[0,112087,0.0384906202],[0,112094,0.1589211555],[0,112099,0.0066055795],[0,112121,0.0120454556],[0,112122,0.5951973159],[0,112134,0.3236239098],[0,112149,0.0061190565],[0,112153,0.0129581579],[0,112161,0.0275436505],[0,112216,0.0010585012],[0,112228,0.0381971713],[0,112229,0.0466055236],[0,112261,0.0066055795],[0,112268,0.0066055795],[0,112280,0.0135507286],[0,112302,0.0076352843],[0,112305,0.0012261191],[0,112314,0.0010585012],[0,112320,0.0120454556],[0,112321,0.0131008793],[0,112334,0.0115770449],[0,112339,0.577088991],[0,112341,0.005695335],[0,112351,0.0131008793],[0,112365,0.005269532],[0,112366,0.0066055795],[0,112398,0.0462443315],[0,112400,0.0066055795],[0,112406,0.0184433114],[0,112420,0.0557383998],[0,112421,0.0131008793],[0,112445,0.0275466056],[0,112462,0.0076826505],[0,112470,0.0498071849],[0,112473,0.006475594],[0,112485,0.0284022164],[0,112499,0.0129581579],[0,112513,0.0066055795],[0,112515,0.0062879398],[0,112524,0.0043610867],[0,112527,0.0078519248],[0,112530,0.006475594],[0,112540,0.0140494018],[0,112546,0.005269532],[0,112549,0.0219870948],[0,112570,0.1446366314],[0,112580,0.0170906375],[0,112600,0.005695335],[0,112608,0.0025274342],[0,112611,0.0248316782],[0,112615,0.0116493526],[0,112617,0.005269532],[0,112624,0.0091660994],[0,112639,0.0131008793],[0,112645,0.030772525],[0,112656,0.0924255661],[0,112679,0.0066055795],[0,112687,0.2003667646],[0,112689,0.0097547431],[0,112698,0.0097547431],[0,112710,0.0416334571],[0,112715,0.0518460242],[0,112718,0.1789923788],[0,112725,0.0110160073],[0,112739,0.0012261191],[0,112744,0.0031356776],[0,112760,0.0011904332],[0,112768,0.005269532],[0,112772,0.0053186329],[0,112773,0.0112248661],[0,112775,0.0228148036],[0,112778,0.0165168299],[0,112780,0.0016654146],[0,112785,0.0012261191],[0,112789,0.0110160073],[0,112793,0.1000348585],[0,112827,0.0131008793],[0,112831,0.0129581579],[0,112837,0.0066055795],[0,112848,0.005269532],[0,112864,0.005695335],[0,112867,0.0073882771],[0,112883,0.0234832555],[0,112884,0.3226901603],[0,112888,0.0062879398],[0,112895,0.0062879398],[0,112954,0.0553400689],[0,113016,0.005269532],[0,113017,0.0070874121],[0,113018,0.0010585012],[0,113027,0.0195823539],[0,113034,0.0361083818],[0,113050,0.0012261191],[0,113070,0.0131008793],[0,113071,0.0016654146],[0,113077,0.0055383191],[0,113080,0.005185203],[0,113121,0.0001673532],[0,113135,0.0018205529],[0,113148,0.0120233623],[0,113161,0.1094904418],[0,113184,0.0118808662],[0,113224,0.0006059551],[0,113231,0.0],[0,113235,0.0010344653],[0,113245,0.0006059551],[0,113247,0.1297231252],[0,113248,0.0099417825],[0,113257,0.0055383191],[0,113277,0.2200530344],[0,113288,0.0105019341],[0,113312,0.1809643355],[0,113338,0.0090399441],[0,113343,0.0104301986],[0,113366,0.0467421396],[0,113368,0.0152043873],[0,113377,0.012795745],[0,113384,0.0053733269],[0,113388,0.0067188472],[0,113392,0.0053733269],[0,113394,0.0086465584],[0,113412,0.0862439066],[0,113414,0.0042043812],[0,113415,0.0001317237],[0,113417,0.004908983],[0,113434,0.7515338199],[0,113436,0.268544366],[0,113449,0.0001317237],[0,113454,0.0312717038],[0,113492,0.099004453],[0,113503,0.0006059551],[0,113522,0.0042043812],[0,113553,0.5634720076],[0,113568,0.0382033836],[0,113581,0.09284608],[0,113588,0.0055383191],[0,113595,0.0340310825],[0,113623,0.0137582826],[0,113624,0.00417796],[0,113626,0.0166738185],[0,113629,0.0120233623],[0,113647,0.0076738753],[0,113674,0.0083835278],[0,113690,0.0042043812],[0,113697,0.0046295118],[0,113702,0.2535026491],[0,113717,0.0097675725],[0,113739,0.0375056341],[0,113749,0.0055383191],[0,113753,0.003567065],[0,113755,0.0018205529],[0,113760,0.0517389814],[0,113780,0.0055383191],[0,113783,0.0955053025],[0,113794,0.0850962164],[0,113798,0.0042043812],[0,113801,0.1535670525],[0,113802,0.4657597398],[0,113805,0.0167013397],[0,113808,0.029839096],[0,113813,0.0283178624],[0,113825,0.2875198453],[0,113833,0.1399017726],[0,113842,0.0042043812],[0,113843,0.0055383191],[0,113851,0.0120233623],[0,113855,0.0042043812],[0,113864,0.0062023092],[0,113878,0.0061092901],[0,113888,0.535297237],[0,113890,0.0032973705],[0,114012,0.0054085388],[0,114032,0.0042043812],[0,114043,0.0053733269],[0,114049,0.0146144002],[0,114057,0.0784726417],[0,114105,0.0001673532],[0,114106,0.0],[0,114112,0.0032973705],[0,114130,0.0859993032],[0,114136,0.0120233623],[0,114139,0.0042043812],[0,114140,0.063700407],[0,114152,0.0032973705],[0,114165,0.0007734597],[0,114171,0.026306442],[0,114180,0.0709485347],[0,114183,0.049139679],[0,114197,0.0054085388],[0,114214,0.1375628212],[0,114223,0.0055193826],[0,114237,0.0174366687],[0,114247,0.0120233623],[0,114258,0.0066136893],[0,114261,0.0513390444],[0,114292,0.0149235921],[0,114319,0.0175379395],[0,114323,0.3862286147],[0,114334,0.1700621137],[0,114349,0.0007734597],[0,114351,0.2170872905],[0,114352,0.6038383861],[0,114353,0.0065663978],[0,114356,0.0126026718],[0,114371,0.1025834039],[0,114375,0.014457681],[0,114377,0.1918149045],[0,114379,0.0054085388],[0,114442,0.233883826],[0,114484,0.2695624116],[0,114526,0.0052211809],[0,114530,0.1666505251],[0,114531,0.0086825099],[0,114543,0.4221715824],[0,114565,0.1984503584],[0,114575,0.0086465584],[0,114603,0.0091573049],[0,114610,0.1108465067],[0,114615,0.0048729235],[0,114617,0.0120756749],[0,114660,0.0014666134],[0,114778,0.0405404299],[0,114784,0.0120233623],[0,114785,0.0120233623],[0,114799,0.0062023092],[0,114804,0.0079535999],[0,114805,0.0045536268],[0,114811,0.0099417825],[0,114839,0.0164233383],[0,114847,0.0],[0,114849,0.1143923568],[0,114866,0.1021895415],[0,114903,0.0054085388],[0,114904,0.0065302497],[0,114907,0.0014666134],[0,114935,0.0042043812],[0,114952,0.0109879961],[0,114956,0.0],[0,114963,0.0060193908],[0,114968,0.0042043812],[0,114986,0.0247821148],[0,114991,0.0051890195],[0,115025,0.0046295118],[0,115038,0.0042043812],[0,115041,0.5141887044],[0,115044,0.0168304915],[0,115047,0.0],[0,115051,0.0042043812],[0,115072,0.0120233623],[0,115099,0.0172810249],[0,115125,0.1922231018],[0,115155,0.0120233623],[0,115167,0.0136776527],[0,115168,0.0042043812],[0,115174,0.0042043812],[0,115183,0.0172810249],[0,115189,0.0001673532],[0,115200,0.0098396595],[0,115210,0.0120233623],[0,115223,0.0524396223],[0,115225,0.0183509883],[0,115230,0.0068868612],[0,115231,0.0123357671],[0,115232,0.020234163],[0,115233,0.0052211809],[0,115256,0.1003935136],[0,115260,0.1913356443],[0,115261,0.0007734597],[0,115264,0.0163975543],[0,115296,0.0040078943],[0,115394,0.0001673532],[0,115407,0.1806993647],[0,115408,0.1715184126],[0,115409,0.2476111794],[0,115411,0.1371073529],[0,115412,0.2767596003],[0,115414,0.0001317237],[0,115446,0.0055383191],[0,115461,0.1489493019],[0,115486,0.0020738964],[0,115499,0.0014666134],[0,115509,0.0092716639],[0,115529,0.0007734597],[0,115532,0.0042043812],[0,115550,0.0120233623],[0,115558,0.0042043812],[0,115677,0.0256851636],[0,115696,0.0042043812],[0,115698,0.007846222],[0,115703,0.5545936708],[0,115729,0.0206876609],[0,115741,0.1757254929],[0,115750,0.0001317237],[0,115778,0.0042043812],[0,115780,0.1145715351],[0,115785,0.0076769492],[0,115797,0.0042043812],[0,115808,0.0078200909],[0,115813,0.073911202],[0,115816,0.0007734597],[0,115823,0.0106249592],[0,115828,0.0061092901],[0,115836,0.0176083359],[0,115875,0.0415649012],[0,115876,0.0001317237],[0,115881,0.1757027612],[0,115883,0.0042534046],[0,115947,0.0198196803],[0,115958,0.0102057087],[0,115987,0.3931634508],[0,115992,0.2101605275],[0,115994,0.0066136893],[0,115996,0.2093320492],[0,115997,0.00417796],[0,116001,0.04216433],[0,116015,0.0127193436],[0,116020,0.0118808662],[0,116022,0.0458242582],[0,116031,0.0032973705],[0,116035,0.0001317237],[0,116044,0.0096043544],[0,116048,0.0001317237],[0,116069,0.0042043812],[0,116082,0.0084899813],[0,116099,0.0016527864],[0,116104,0.0050525643],[0,116116,0.0055383191],[0,116190,0.0076738753],[0,116215,0.054562344],[0,116236,0.0068507944],[0,116307,0.0006059551],[0,116310,0.0399046359],[0,116323,0.0001317237],[0,116341,0.0481139851],[0,116344,0.0014666134],[0,116363,0.0054085388],[0,116399,0.0],[0,116405,0.0174461676],[0,116408,0.0200080028],[0,116499,0.0084899813],[0,116502,0.0172108764],[0,116508,0.0017481183],[0,116514,0.0120233623],[0,116523,0.0],[0,116573,0.0105137894],[0,116584,0.0046295118],[0,116586,0.0146564103],[0,116592,0.0060193908],[0,116598,0.0065302497],[0,116602,0.0556863359],[0,116620,0.0181190608],[0,116628,0.0050525643],[0,116657,0.0054085388],[0,116660,0.0051852824],[0,116756,0.2483064221],[0,116769,0.0120233623],[0,116771,0.0606975479],[0,116777,0.0032973705],[0,116783,0.0106910353],[0,116784,0.0046295118],[0,116785,0.0015024907],[0,116791,0.0015024907],[0,116798,0.0456413358],[0,116895,0.7129706848],[0,116935,0.0014666134],[0,116942,0.0054085388],[0,116949,0.0544710187],[0,116991,0.330845646],[0,116992,0.1086527991],[0,117000,0.1541177427],[0,117043,0.0080947957],[0,117044,0.0121254656],[0,117049,0.0279542063],[0,117054,0.0001673532],[0,117058,0.0042043812],[0,117062,0.0096043544],[0,117084,0.0110075683],[0,117105,0.0146144002],[0,117125,0.0496256152],[0,117127,0.0042043812],[0,117161,0.0076738753],[0,117171,0.0489596627],[0,117220,0.0042043812],[0,117254,0.0291142123],[0,117256,0.0093122129],[0,117258,0.1942705409],[0,117260,0.4343792968],[0,117271,0.0054085388],[0,117285,0.0055383191],[0,117385,0.0014666134],[0,117391,0.0120233623],[0,117392,0.0407771972],[0,117402,0.0212659753],[0,117410,0.0249784693],[0,117424,0.0119740944],[0,117471,0.0334627921],[0,117481,0.0007734597],[0,117493,0.0143458927],[0,117496,0.0120233623],[0,117525,0.0055383191],[0,117531,0.0007734597],[0,117540,0.0120233623],[0,117549,0.000737798],[0,117558,0.0042043812],[0,117561,0.0355291161],[0,117565,0.1754700204],[0,117567,0.0010344653],[0,117584,0.0120233623],[0,117588,0.0220139536],[0,117627,0.0],[0,117665,0.0037522165],[0,117682,0.0131119114],[0,117687,0.4733780186],[0,117688,0.0354505653],[0,117717,0.725998542],[0,117718,0.3197696094],[0,117731,0.2307538887],[0,117752,0.2380263307],[0,117762,0.0291279065],[0,117772,0.0032973705],[0,117796,0.0014666134],[0,117804,0.0014666134],[0,117821,0.2619004377],[0,117844,0.010994599],[0,117866,0.0055383191],[0,117884,0.0199565068],[0,117889,0.3124626581],[0,117890,0.0608331012],[0,117901,0.0046295118],[0,117907,0.0042043812],[0,117917,0.075212079],[0,118080,0.0486022973],[0,118083,0.0120233623],[0,118106,0.0080947957],[0,118120,0.1407416824],[0,118121,0.0042043812],[0,118125,0.0120233623],[0,118131,0.0042043812],[0,118139,0.0080947957],[0,118167,0.1159822928],[0,118177,0.0055383191],[0,118209,0.0227274855],[0,118250,0.2180648917],[0,118251,0.0054085388],[0,118262,0.4444826516],[0,118266,0.3341135639],[0,118269,0.0135273165],[0,118276,0.2559522207],[0,118304,0.0092716639],[0,118305,0.0125000338],[0,118350,0.0042043812],[0,118366,0.0118808662],[0,118378,0.0001673532],[0,118382,0.0052211809],[0,118386,0.0014666134],[0,118430,0.0014666134],[0,118442,0.0347355101],[0,118481,0.2202749435],[0,118494,0.0267346873],[0,118495,0.0001673532],[0,118496,0.0713509086],[0,118603,0.0266178603],[0,118605,0.0192809913],[0,118613,0.01569296],[0,118618,0.0042043812],[0,118623,0.0710414406],[0,118661,0.0138388501],[0,118685,0.00417796],[0,118689,0.0188046703],[0,118733,0.0104664563],[0,118735,0.0055383191],[0,118752,0.0120233623],[0,118777,0.0360828961],[0,118779,0.0223842485],[0,118790,0.0001673532],[0,118792,0.025064212],[0,118794,0.2666291885],[0,118799,0.3894298963],[0,118802,0.0],[0,118904,0.7484370454],[0,118937,0.0086103978],[0,118940,0.0124834042],[0,118942,0.0374071642],[0,119001,0.0571624134],[0,119107,0.0216273737],[0,119117,0.6998611348],[0,119137,0.007889315],[0,119143,0.0],[0,119181,0.0371583311],[0,119201,0.0450991611],[0,119230,0.0032973705],[0,119245,0.0120233623],[0,119246,0.0353515023],[0,119272,0.0400303599],[0,119284,0.0151380741],[0,119303,0.0055383191],[0,119316,0.0042043812],[0,119327,0.0054085388],[0,119374,0.1341550539],[0,119383,0.135175729],[0,119389,0.0007734597],[0,119509,0.0055383191],[0,119539,0.0032973705],[0,119578,0.037875276],[0,119579,0.0042043812],[0,119585,0.0041762497],[0,119586,0.0120233623],[0,119605,0.0120233623],[0,119642,0.0361198772],[0,119648,0.0300239054],[0,119658,0.0121983725],[0,119670,0.0081310275],[0,119702,0.0662381315],[0,119741,0.0045536268],[0,119770,0.0092716639],[0,119771,0.0066136893],[0,119772,0.0046295118],[0,119778,0.0240097572],[0,119798,0.0202695548],[0,119803,0.0120233623],[0,119837,0.1716947826],[0,119886,0.0001317237],[0,119890,0.0046295118],[0,119932,0.0001317237],[0,119933,0.3433277597],[0,119961,0.0171409821],[0,119988,0.0],[0,120082,0.0050525643],[0,120083,0.0066136893],[0,120087,0.0042043812],[0,120093,0.0182363501],[0,120100,0.0229501477],[0,120101,0.0094910213],[0,120105,0.0001317237],[0,120107,0.0055383191],[0,120123,0.0120233623],[0,120126,0.0],[0,120137,0.0010344653],[0,120204,0.0145530452],[0,120221,0.0055383191],[0,120231,0.0185901607],[0,120239,0.0042043812],[0,120246,0.0094417494],[0,120263,0.0094177949],[0,120272,0.0042043812],[0,120274,0.0393552296],[0,120297,0.007670044],[0,120310,0.0042043812],[0,120311,0.0001317237],[0,120315,0.2356821257],[0,120409,0.0021098061],[0,120429,0.0179320902],[0,120436,0.076729481],[0,120492,0.1293228357],[0,120497,0.787369592],[0,120502,0.0012986543],[0,120505,0.0120233623],[0,120520,0.0120233623],[0,120526,0.0001673532],[0,120527,0.0138382342],[0,120534,0.0055383191],[0,120548,0.0192892555],[0,120582,0.0120233623],[0,120603,0.0102057087],[0,120605,0.0046295118],[0,120615,0.1029920524],[0,120643,0.0726570277],[0,120644,0.0470111547],[0,120675,0.0086825099],[0,120684,0.0001673532],[0,120694,0.0042043812],[0,120767,0.0120233623],[0,120779,0.0120233623],[0,120811,0.0],[0,120813,0.0],[0,120816,0.0120233623],[0,120822,0.0506466965],[0,120830,0.003567065],[0,120833,0.0067826962],[0,120843,0.0001317237],[0,120878,0.0099417825],[0,120947,0.0032973705],[0,120961,0.0099417825],[0,120982,0.0081310275],[0,121052,0.0042043812],[0,121061,0.0046295118],[0,121070,0.0010344653],[0,121071,0.0001673532],[0,121076,0.0120233623],[0,121078,0.0018205529],[0,121092,0.0099417825],[0,121111,0.0054085388],[0,121114,0.0356736356],[0,121115,0.4594454846],[0,121118,0.0042043812],[0,121141,0.0402372233],[0,121142,0.0032973705],[0,121144,0.0046295118],[0,121148,0.0001673532],[0,121184,0.0120233623],[0,121216,0.0042043812],[0,121225,0.0105019341],[0,121226,0.0],[0,121229,0.0173535155],[0,121236,0.2091072816],[0,121237,0.0118808662],[0,121238,0.0007734597],[0,121245,0.1643341855],[0,121247,0.5292705046],[0,121263,0.0055383191],[0,121267,0.0219362715],[0,121290,0.0050525643],[0,121294,0.0158933913],[0,121296,0.0001317237],[0,121320,0.0099417825],[0,121322,0.0411459125],[0,121325,0.0689198873],[0,121328,0.2553242548],[0,121331,0.0118808662],[0,121334,0.0178168796],[0,121337,0.0103950923],[0,121339,0.0091573049],[0,121341,0.0001317237],[0,121347,0.0212049121],[0,121349,0.313521373],[0,121350,0.2257279943],[0,121351,0.6464679768],[0,121353,0.0054085388],[0,121360,0.0055383191],[0,121366,0.0112039109],[0,121367,0.0050525643],[0,121381,0.0055383191],[0,121382,0.0042043812],[0,121391,0.1513545054],[0,121396,0.0048191292],[0,121399,0.1792693936],[0,121403,0.0806476624],[0,121410,0.30243228],[0,121412,0.1676908732],[0,121417,0.1188351589],[0,121418,0.0191337296],[0,121422,0.1200843849],[0,121428,0.0014666134],[0,121429,0.0081310275],[0,121432,0.0181716749],[0,121439,0.0042043812],[0,121444,0.0206675538],[0,121445,0.0120756749],[0,121446,0.0010344653],[0,121454,0.00417796],[0,121456,0.1105101151],[0,121457,0.0350321723],[0,121461,0.0413887498],[0,121463,0.0042043812],[0,121506,0.0669848682],[0,121513,0.0462421868],[0,121520,0.0060193908],[0,121523,0.6843931418],[0,121526,0.1969252634],[0,121529,0.0055383191],[0,121547,0.0665805001],[0,121548,0.016151336],[0,121551,0.0292512421],[0,121559,0.0188882254],[0,121568,0.3522356951],[0,121570,0.0961433715],[0,121572,0.0042043812],[0,121578,0.0145528251],[0,121590,0.0092575101],[0,121616,0.0120756749],[0,121634,0.0042043812],[0,121641,0.0042043812],[0,121649,0.2053317829],[0,121663,0.017338361],[0,121678,0.0169174755],[0,121713,0.0054085388],[0,121740,0.0066136893],[0,121758,0.003567065],[0,121764,0.0],[0,121775,0.013543066],[0,121814,0.2537556475],[0,121817,0.1707349709],[0,121897,0.0254543246],[0,121913,0.0189234233],[0,121916,0.0054085388],[0,121919,0.0734304064],[0,121920,0.0112039109],[0,122005,0.2587557399],[0,122008,0.0461358458],[0,122034,0.0186210236],[0,122075,0.0035518791],[0,122103,0.006605939],[0,122158,0.4102939973],[0,122160,0.0048577081],[0,122239,0.0639481408],[0,122346,0.0146819114],[0,122364,0.0402330879],[0,122448,0.2139351455],[0,122451,0.0911091736],[0,122452,0.006605939],[0,122484,0.006605939],[0,122521,0.0266118787],[0,122529,0.0296122311],[0,122705,0.006605939],[0,122846,0.0236392478],[0,122853,0.2187366304],[0,122887,0.0027752121],[0,122894,0.3476469995],[0,122908,0.4728950149],[0,122920,0.258293572],[0,122922,0.0382891332],[0,122933,0.2658810614],[0,122944,0.4438337079],[0,123006,0.0074239208],[0,123012,0.0079446335],[0,123013,0.0078487275],[0,123016,0.006605939],[0,123019,0.006605939],[0,123049,0.5260977084],[0,123090,0.0568817968],[0,123091,0.0288024142],[0,123101,0.0677966865],[0,123125,0.024672216],[0,123130,0.0094163258],[0,123133,0.0407200728],[0,123144,0.385248434],[0,123146,0.0713400172],[0,123161,0.0114775992],[0,123169,0.011493725],[0,123170,0.0056956939],[0,123199,0.0119556964],[0,123209,0.0140098564],[0,123212,0.0050448224],[0,123214,0.0355347632],[0,123217,0.0240138608],[0,123218,0.0144528019],[0,123219,0.5501410211],[0,123221,0.0163059166],[0,123253,0.2936967819],[0,123255,0.6171550611],[0,123256,0.0144528019],[0,123259,0.006605939],[0,123263,0.0027752121],[0,123264,0.0124193921],[0,123266,0.0936175274],[0,123269,0.1308319543],[0,123273,0.0201278948],[0,123275,0.0094163258],[0,123278,0.1552151716],[0,123280,0.1660311535],[0,123282,0.1016103335],[0,123288,0.0082061781],[0,123292,0.0046033141],[0,123296,0.0372129467],[0,123297,0.0216204761],[0,123299,0.1643491359],[0,123305,0.3959897116],[0,123314,0.3573383652],[0,123324,0.004247907],[0,123328,0.0079446335],[0,123335,0.0035518791],[0,123338,0.004247907],[0,123354,0.3185937399],[0,123371,0.002907482],[0,123374,0.0070325856],[0,123375,0.0215545526],[0,123383,0.112268272],[0,123386,0.0248060614],[0,123389,0.2265251362],[0,123396,0.1026346966],[0,123405,0.0155680693],[0,123428,0.0147537656],[0,123464,0.0076143244],[0,123467,0.0027752121],[0,123472,0.0167877797],[0,123476,0.798592931],[0,123478,0.0092747137],[0,123480,0.0408017467],[0,123481,0.0556259235],[0,123488,0.015010408],[0,123497,0.0079446335],[0,123500,0.1858292068],[0,123503,0.0306199469],[0,123527,0.7771331884],[0,123533,0.0045305792],[0,123536,0.0235824207],[0,123549,0.3238471041],[0,123552,0.0106177016],[0,123556,0.0134474077],[0,123568,0.0078487275],[0,123570,0.004247907],[0,123572,0.0029432592],[0,123580,0.0060862555],[0,123581,0.0027752121],[0,123587,0.0056956939],[0,123588,0.006605939],[0,123595,0.0249686345],[0,123600,0.0034247194],[0,123606,0.0221762901],[0,123607,0.0027752121],[0,123611,0.0095398452],[0,123614,0.09027655],[0,123627,0.1803723674],[0,123664,0.0114013454],[0,123684,0.0202215117],[0,123691,0.0078487275],[0,123693,0.0094163258],[0,123696,0.4460435229],[0,123704,0.0070462539],[0,123709,0.0045305792],[0,123710,0.0069704965],[0,123712,0.0252404024],[0,123726,0.1810078947],[0,123782,0.073533942],[0,123786,0.004247907],[0,123824,0.0027752121],[0,123825,0.0027752121],[0,123836,0.0048577081],[0,123839,0.0070462539],[0,123845,0.006605939],[0,123853,0.0029432592],[0,123862,0.0183451552],[0,123878,0.6988517756],[0,123903,0.0082061781],[0,123904,0.0025187587],[0,123907,0.006605939],[0,123916,0.006605939],[0,123935,0.006605939],[0,123940,0.006605939],[0,123942,0.0191010038],[0,123943,0.0239127629],[0,123944,0.0027752121],[0,123952,0.0480067134],[0,123957,0.5871646712],[0,123966,0.0518581541],[0,123970,0.0082061781],[0,123989,0.0976239555],[0,123991,0.0104809],[0,124020,0.006605939],[0,124044,0.0143097977],[0,124049,0.006605939],[0,124070,0.0144528019],[0,124078,0.0027752121],[0,124095,0.0815541661],[0,124128,0.0035160695],[0,124157,0.0127582194],[0,124225,0.0029432592],[0,124245,0.006605939],[0,124250,0.0033836798],[0,124278,0.0029432592],[0,124285,0.0144528019],[0,124287,0.0223353045],[0,124289,0.0119704889],[0,124295,0.0138087712],[0,124301,0.3415784025],[0,124304,0.0094163258],[0,124314,0.0351853554],[0,124320,0.3544458305],[0,124358,0.0256707848],[0,124370,0.0255493549],[0,124384,0.2486806363],[0,124389,0.0121260391],[0,124399,0.1114471771],[0,124411,0.0109035738],[0,124414,0.0035518791],[0,124466,0.014005667],[0,124470,0.006605939],[0,124481,0.0080180432],[0,124484,0.0114689831],[0,124485,0.2130516693],[0,124486,0.0025187587],[0,124514,0.0252300281],[0,124519,0.0953882831],[0,124530,0.0271784661],[0,124538,0.0027752121],[0,124608,0.0034247194],[0,124615,0.0048577081],[0,124652,0.0351853554],[0,124654,0.0199737471],[0,124655,0.0274898949],[0,124657,0.0180945274],[0,124663,0.0491909079],[0,124671,0.008819563],[0,124740,0.0033836798],[0,124742,0.0610030692],[0,124746,0.1381626649],[0,124783,0.004247907],[0,124791,0.004283933],[0,124794,0.2095009517],[0,124817,0.0118007832],[0,124834,0.0079246888],[0,124852,0.015263573],[0,124860,0.0113775599],[0,124877,0.0113806506],[0,124923,0.0074974794],[0,124930,0.0074974794],[0,124933,0.0106052183],[0,124952,0.1476500142],[0,124986,0.214042859],[0,125024,0.5862302777],[0,125034,0.4557559931],[0,125037,0.0038299683],[0,125042,0.1234906252],[0,125049,0.0130248611],[0,125054,0.0069771105],[0,125057,0.1196547491],[0,125068,0.0074974794],[0,125087,0.0038299683],[0,125106,0.0136578752],[0,125110,0.0439953621],[0,125111,0.5725185603],[0,125112,0.0088379399],[0,125117,0.0406882704],[0,125134,0.0243111584],[0,125146,0.0139335969],[0,125171,0.0136578752],[0,125172,0.0038299683],[0,125221,0.0088379399],[0,125223,0.005136337],[0,125224,0.0349762873],[0,125231,0.015354693],[0,125260,0.0126279847],[0,125263,0.0074974794],[0,125276,0.015354693],[0,125290,0.0473466499],[0,125308,0.0269176783],[0,125327,0.0074974794],[0,125345,0.0089114464],[0,125353,0.0036616995],[0,125379,0.0088379399],[0,125427,0.0084802268],[0,125440,0.0089114464],[0,125444,0.0625596448],[0,125451,0.0489432479],[0,125465,0.0090998294],[0,125472,0.015354693],[0,125475,0.0051724106],[0,125477,0.0196161367],[0,125478,0.015263573],[0,125482,0.0123917129],[0,125483,0.3938805007],[0,125491,0.0079246888],[0,125501,0.0092446405],[0,125502,0.0074974794],[0,125512,0.015354693],[0,125514,0.0206266193],[0,125520,0.0074974794],[0,125540,0.015354693],[0,125551,0.041884766],[0,125572,0.0143447897],[0,125586,0.0037941438],[0,125614,0.0341684576],[0,125624,0.0090998294],[0,125628,0.0151356799],[0,125632,0.0082402399],[0,125633,0.0148373086],[0,125661,0.0089536258],[0,125697,0.0689291948],[0,125740,0.0103115734],[0,125742,0.0362009226],[0,125744,0.0202398521],[0,125746,0.4209836484],[0,125765,0.0038299683],[0,125766,0.0090644248],[0,125779,0.0326572505],[0,125912,0.0065860337],[0,125916,0.0074974794],[0,125920,0.1343277278],[0,125964,0.0038299683],[0,125992,0.008507195],[0,126016,0.0043120635],[0,126111,0.0426208498],[0,126122,0.0304625325],[0,126148,0.0238928613],[0,126150,0.0088379399],[0,126154,0.0152115002],[0,126159,0.0388602694],[0,126163,0.0175676457],[0,126165,0.0074974794],[0,126168,0.0618639037],[0,126170,0.0043120635],[0,126260,0.0090998294],[0,126293,0.0036616995],[0,126296,0.0603410761],[0,126314,0.0118007832],[0,126397,0.015354693],[0,126398,0.0171784391],[0,126403,0.0137264068],[0,126417,0.0231411993],[0,126447,0.2213947198],[0,126455,0.0074974794],[0,126461,0.0205742099],[0,126578,0.0185309329],[0,126583,0.0118007832],[0,126585,0.1140804328],[0,126599,0.0312751034],[0,126615,0.0037941438],[0,126624,0.0043120635],[0,126631,0.008684131],[0,126680,0.0088379399],[0,126683,0.0696229959],[0,126816,0.0104173067],[0,126909,0.015354693],[0,126925,0.0074974794],[0,126952,0.0042709698],[0,126955,0.004439391],[0,126967,0.0074974794],[0,126988,0.0220057765],[0,126994,0.0334507339],[0,126997,0.0083165402],[0,127019,0.015354693],[0,127029,0.0051724106],[0,127053,0.0090998294],[0,127055,0.0103115734],[0,127060,0.015354693],[0,127172,0.0813007091],[0,127178,0.0079246888],[0,127208,0.0186669412],[0,127232,0.0426208498],[0,127262,0.027874179],[0,127282,0.015354693],[0,127283,0.0090998294],[0,127290,0.1152072794],[0,127295,0.0074974794],[0,127301,0.015354693],[0,127309,0.0037941438],[0,127312,0.0192825616],[0,127319,0.0079383752],[0,127320,0.015354693],[0,127369,0.0083165402],[0,127415,0.0077248079],[0,127421,0.0142210915],[0,127433,0.0158034425],[0,127461,0.0089114464],[0,127473,0.0168484169],[0,127491,0.0090998294],[0,127517,0.0074974794],[0,127527,0.015354693],[0,127629,0.0088379399],[0,127636,0.0090998294],[0,127638,0.0037941438],[0,127642,0.1251024114],[0,127643,0.015263573],[0,127647,0.0169600454],[0,127651,0.0074974794],[0,127673,0.0036616995],[0,127684,0.0088379399],[0,127685,0.5539769716],[0,127686,0.011658815],[0,127705,0.005136337],[0,127721,0.0074974794],[0,127739,0.0088379399],[0,127758,0.0088379399],[0,127765,0.0088379399],[0,127785,0.0074974794],[0,127798,0.0038299683],[0,127818,0.0104815051],[0,127830,0.0941091939],[0,127846,0.0165075323],[0,127886,0.0144024265],[0,127907,0.005136337],[0,127917,0.1148015629],[0,127918,0.0130248611],[0,127920,0.0163458095],[0,127937,0.0072482806],[0,128053,0.0152115002],[0,128081,0.015354693],[0,128119,0.017916287],[0,128122,0.0079246888],[0,128182,0.0148303304],[0,128190,0.0036616995],[0,128200,0.0938473348],[0,128213,0.015354693],[0,128216,0.0103115734],[0,128228,0.6191238446],[0,128239,0.0038299683],[0,128240,0.3395882666],[0,128248,0.0700333656],[0,128264,0.0199350104],[0,128282,0.0074974794],[0,128284,0.2515075278],[0,128324,0.011514534],[0,128338,0.1676967231],[0,128346,0.0090998294],[0,128361,0.0285859389],[0,128401,0.1099631306],[0,128404,0.0136578752],[0,128405,0.0088379399],[0,128417,0.2979277019],[0,128440,0.0036616995],[0,128454,0.015354693],[0,128466,0.0042709698],[0,128479,0.005136337],[0,128499,0.5250453597],[0,128501,0.0321618912],[0,128503,0.0168593488],[0,128504,0.0209482837],[0,128506,0.0074974794],[0,128507,0.527351888],[0,128510,0.015354693],[0,128513,0.0458465259],[0,128514,0.0632999339],[0,128518,0.0074974794],[0,128526,0.0779346075],[0,128527,0.0372839049],[0,128535,0.0834174473],[0,128543,0.0148101188],[0,128544,0.084971894],[0,128545,0.0428175596],[0,128551,0.0220045504],[0,128557,0.1200442651],[0,128568,0.015354693],[0,128577,0.0321618912],[0,128583,0.2291375694],[0,128585,0.0530188945],[0,128588,0.5972193403],[0,128595,0.015354693],[0,128610,0.5250453597],[0,128623,0.0090464832],[0,128630,0.0074974794],[0,128634,0.0240518494],[0,128638,0.1304575128],[0,128639,0.8930646429],[0,128649,0.0566437409],[0,128650,0.0178198219],[0,128651,0.024197292],[0,128653,0.0088379399],[0,128659,0.0458465259],[0,128661,0.01870164],[0,128662,0.5980028082],[0,128665,0.015530559],[0,128667,0.025558302],[0,128668,0.057794813],[0,128677,0.015530559],[0,128690,0.0492486613],[0,128691,0.027571414],[0,128692,0.0074974794],[0,128695,0.5972727916],[0,128698,0.0541800065],[0,128702,0.0072857479],[0,128705,0.5234303363],[0,128706,0.0088379399],[0,128714,0.3202530211],[0,128721,0.6209854611],[0,128722,0.015354693],[0,128725,0.0096313319],[0,128737,0.038584992],[0,128743,0.1227705029],[0,128744,0.6394834392],[0,128760,0.0074974794],[0,128763,0.0275294581],[0,128782,0.0065860337],[0,128783,0.0447814951],[0,128784,0.015530559],[0,128787,0.5242375749],[0,128792,0.015354693],[0,128795,0.018861811],[0,128797,0.727573014],[0,128799,0.02477609],[0,128831,0.1000403314],[0,128843,0.0202715347],[0,128844,0.6082319261],[0,128848,0.0427841897],[0,128850,0.30780163],[0,128858,0.0333827568],[0,128859,0.0522541816],[0,128863,0.0704561315],[0,128865,0.0579239295],[0,128868,0.0074974794],[0,128884,0.0566437409],[0,128889,0.0321618912],[0,128894,0.0492486613],[0,128895,0.5981300078],[0,128901,0.015354693],[0,128902,0.1062004647],[0,128906,0.015354693],[0,128908,0.0334120471],[0,128911,0.0706527345],[0,128916,0.0444791563],[0,128921,0.0492965682],[0,128925,0.1916996057],[0,128927,0.0291835844],[0,128936,0.2487664352],[0,128938,0.0056190849],[0,128940,0.0088379399],[0,128947,0.0163768209],[0,128948,0.0074974794],[0,128950,0.0454530426],[0,128954,0.0074974794],[0,128957,0.0321618912],[0,128959,0.0506054999],[0,128966,0.028818019],[0,128969,0.0074974794],[0,128971,0.0072857479],[0,128977,0.0139695407],[0,128984,0.0088379399],[0,128987,0.0326907313],[0,128988,0.0530188945],[0,128992,0.015354693],[0,128995,0.0212809874],[0,128996,0.6142220083],[0,128997,0.0492486613],[0,128998,0.0634947181],[0,129003,0.015530559],[0,129007,0.0169976035],[0,129013,0.0268145877],[0,129018,0.0156391185],[0,129019,0.0898967015],[0,129023,0.1102507396],[0,129024,0.0088379399],[0,129032,0.0074974794],[0,129034,0.0133275748],[0,129038,0.0208722285],[0,129040,0.5257337456],[0,129042,0.0458465259],[0,129043,0.7930593938],[0,129044,0.015530559],[0,129046,0.0074974794],[0,129047,0.080619949],[0,129050,0.0336504792],[0,129052,0.015530559],[0,129056,0.2475301927],[0,129057,0.0489132281],[0,129060,0.034475441],[0,129065,0.0355492376],[0,129068,0.3970433449],[0,129072,0.0166051013],[0,129075,0.0189084728],[0,129076,0.015530559],[0,129079,0.015354693],[0,129080,0.0074974794],[0,129085,0.0457828721],[0,129090,0.0440580784],[0,129091,0.015354693],[0,129105,0.0802787606],[0,129112,0.2723908194],[0,129113,0.0074974794],[0,129114,0.0127101121],[0,129119,0.6242253076],[0,129124,0.015354693],[0,129125,0.0074974794],[0,129128,0.0360338412],[0,129130,0.0492486613],[0,129131,0.0160554316],[0,129141,0.1051827952],[0,129144,0.0484970993],[0,129146,0.0747184777],[0,129154,0.1420555546],[0,129155,0.0992143671],[0,129159,0.0074974794],[0,129160,0.096797811],[0,129161,0.015354693],[0,129165,0.0427841897],[0,129167,0.015354693],[0,129170,0.01415791],[0,129176,0.0074974794],[0,129178,0.0228165516],[0,129179,0.0074974794],[0,129182,0.015354693],[0,129190,0.015354693],[0,129196,0.0074974794],[0,129199,0.015354693],[0,129206,0.064664158],[0,129210,0.0074974794],[0,129211,0.015354693],[0,129215,0.0717397497],[0,129218,0.5050087244],[0,129220,0.0204820149],[0,129221,0.0162894896],[0,129225,0.0926643539],[0,129226,0.015354693],[0,129231,0.0530188945],[0,129233,0.0986288563],[0,129237,0.0074974794],[0,129238,0.0391005757],[0,129245,0.0177419787],[0,129247,0.0212691623],[0,129248,0.2183379646],[0,129249,0.015530559],[0,129254,0.4212146923],[0,129255,0.0251246595],[0,129260,0.0281408396],[0,129262,0.01448762],[0,129268,0.0321796927],[0,129274,0.014893878],[0,129277,0.123941248],[0,129279,0.0324781939],[0,129280,0.0074974794],[0,129283,0.0359956185],[0,129285,0.0442279777],[0,129289,0.0074974794],[0,129290,0.015354693],[0,129300,0.0341178385],[0,129301,0.0136587868],[0,129302,0.0492486613],[0,129304,0.0170239347],[0,129310,0.015354693],[0,129321,0.0200439463],[0,129323,0.0789589548],[0,129330,0.0224871364],[0,129342,0.3945028535],[0,129345,0.3126503365],[0,129347,0.0221316065],[0,129348,0.0090464832],[0,129351,0.0440073723],[0,129357,0.015354693],[0,129359,0.0428175596],[0,129360,0.0203483002],[0,129362,0.021784174],[0,129363,0.0458465259],[0,129364,0.0316026255],[0,129374,0.0363237684],[0,129375,0.0074974794],[0,129376,0.0564362724],[0,129379,0.5964161552],[0,129380,0.0216077222],[0,129383,0.0249656125],[0,129389,0.0074974794],[0,129397,0.0064477906],[0,129398,0.0153873294],[0,129399,0.0421860497],[0,129404,0.0289151038],[0,129407,0.015354693],[0,129409,0.0140708293],[0,129412,0.0447419302],[0,129421,0.0742152747],[0,129435,0.015354693],[0,129463,0.6043465526],[0,129464,0.3986159027],[0,129525,0.0076390585],[0,129529,0.015354693],[0,129534,0.015530559],[0,129547,0.015354693],[0,129548,0.0427841897],[0,129549,0.01448762],[0,129550,0.322702687],[0,129551,0.015354693],[0,129552,0.015530559],[0,129553,0.0458465259],[0,129554,0.0427841897],[0,129557,0.0127835916],[0,129558,0.3330828188],[0,129563,0.015354693],[0,129564,0.7195113374],[0,129566,0.015530559],[0,129573,0.015530559],[0,129575,0.0232941662],[0,129577,0.3188510909],[0,129578,0.2926088419],[0,129583,0.6706955848],[0,129589,0.515281051],[0,129591,0.0578486228],[0,129599,0.0428175596],[0,129601,0.0492486613],[0,129602,0.015354693],[0,129604,0.015530559],[0,129605,0.0074974794],[0,129612,0.0742152747],[0,129613,0.015530559],[0,129615,0.3785604043],[0,129618,0.0187676809],[0,129628,0.015530559],[0,129653,0.0074974794],[0,129654,0.0208722285],[0,129655,0.0074974794],[0,129660,0.0357464907],[0,129670,0.3307125147],[0,129672,0.015354693],[0,129681,0.0658642468],[0,129684,0.015354693],[0,129689,0.3590832947],[0,129702,0.0074974794],[0,129710,0.4169520295],[0,129713,0.611267384],[0,129722,0.0036616995],[0,129734,0.0173368326],[0,129753,0.0074974794],[0,129763,0.1311032247],[0,129764,0.1286200369],[0,129769,0.0702454941],[0,129820,0.0118007832],[0,129829,0.0036616995],[0,129844,0.0418866071],[0,129846,0.0057469425],[0,129853,0.0922207478],[0,129861,0.4153771257],[0,129870,0.0090998294],[0,129876,0.0480204824],[0,129903,0.0122992114],[0,129908,0.0074974794],[0,129915,0.0492486613],[0,129916,0.0492486613],[0,129920,0.3656170232],[0,129921,0.3288210615],[0,129936,0.0662133968],[0,129944,0.0074974794],[0,129952,0.0074974794],[0,129954,0.1066593766],[0,129955,0.3188510909],[0,129969,0.0262786868],[0,129978,0.420685451],[0,129979,0.0143847816],[0,130015,0.3314401974],[0,130016,0.1480585565],[0,130017,0.0653175028],[0,130019,0.015354693],[0,130022,0.0357464907],[0,130027,0.0492486613],[0,130029,0.015354693],[0,130031,0.3301994245],[0,130033,0.2504537902],[0,130038,0.015530559],[0,130039,0.0458465259],[0,130042,0.0193045535],[0,130046,0.0210011218],[0,130073,0.0074974794],[0,130080,0.015354693],[0,130093,0.3367003766],[0,130097,0.0129030634],[0,130105,0.0158535056],[0,130146,0.0092112796],[0,130198,0.0253024616],[0,130216,0.0051724106],[0,130245,0.004439391],[0,130258,0.0103115734],[0,130270,0.0505133318],[0,130309,0.0043120635],[0,130330,0.0088752716],[0,130337,0.0263020739],[0,130339,0.042118282],[0,130351,0.0269176783],[0,130382,0.0518821741],[0,130384,0.015354693],[0,130407,0.7118909773],[0,130424,0.0042709698],[0,130442,0.4075485152],[0,130443,0.3276070461],[0,130448,0.0042709698],[0,130455,0.4053013229],[0,130480,0.0270313716],[0,130488,0.034459343],[0,130489,0.0904518068],[0,130490,0.3153880651],[0,130509,0.2159707168],[0,130542,0.0088379399],[0,130558,0.215013964],[0,130612,0.015354693],[0,130642,0.0074974794],[0,130647,0.0209587509],[0,130656,0.0074974794],[0,130661,0.015354693],[0,130704,0.1209290074],[0,130788,0.0759885542],[0,130794,0.015354693],[0,130811,0.0104815051],[0,130815,0.015354693],[0,130817,0.0254845988],[0,130826,0.1004334444],[0,130844,0.0136578752],[0,130865,0.0104815051],[0,130879,0.005136337],[0,130881,0.015354693],[0,130882,0.1814753224],[0,130910,0.0037941438],[0,130940,0.0090998294],[0,130970,0.0036616995],[0,130997,0.3593464304],[0,131018,0.5995383234],[0,131074,0.0214538458],[0,131101,0.1214718449],[0,131117,0.1424052836],[0,131130,0.015354693],[0,131160,0.0176927509],[0,131175,0.0451016165],[0,131283,0.1212639843],[0,131288,0.2903706842],[0,131319,0.0079246888],[0,131379,0.0740660005],[0,131400,0.1010772227],[0,131415,0.0074974794],[0,131451,0.024681976],[0,131500,0.0038299683],[0,131503,0.015354693],[0,131511,0.0212033182],[0,131523,0.0292798308],[0,131541,0.437685402],[0,131547,0.0080708686],[0,131564,0.0158034425],[0,131569,0.0927027098],[0,131629,0.0088379399],[0,131637,0.0042709698],[0,131644,0.0089114464],[0,131654,0.386322693],[0,131681,0.3037256404],[0,131718,0.0089114464],[0,131828,0.0634677854],[0,131865,0.2698152324],[0,131868,0.0057830487],[0,131871,0.0074974794],[0,131880,0.0526322267],[0,131892,0.004439391],[0,131919,0.0348536246],[0,131930,0.0095551682],[0,131958,0.0379781324],[0,131969,0.1221967769],[0,131990,0.0230063056],[0,132098,0.0260563641],[0,132099,0.0088379399],[0,132221,0.015354693],[0,132229,0.018865076],[0,132244,0.0043120635],[0,132260,0.1202313914],[0,132270,0.0034049078],[0,132291,0.0130248611],[0,132318,0.0118372132],[0,132366,0.0378932273],[0,132372,0.0234322908],[0,132413,0.0079246888],[0,132423,0.0074974794],[0,132426,0.0211140229],[0,132504,0.1596555442],[0,132508,0.0088379399],[0,132513,0.0136578752],[0,132522,0.1580475792],[0,132556,0.0090998294],[0,132580,0.0074974794],[0,132583,0.0083165402],[0,132603,0.0103115734],[0,132681,0.0780543703],[0,132710,0.0037941438],[0,132714,0.0074974794],[0,132729,0.0477741653],[0,132747,0.0090998294],[0,132816,0.004439391],[0,132819,0.0149069684],[0,132829,0.4811439261],[0,132861,0.0088379399],[0,132897,0.0743557907],[0,132921,0.0244204564],[0,132940,0.0088379399],[0,133085,0.2793967768],[0,133255,0.0088379399],[0,133276,0.0481869284],[0,133289,0.0104815051],[0,133303,0.0154057236],[0,133360,0.0196186803],[0,133365,0.0037941438],[0,133386,0.0500157228],[0,133396,0.0121601759],[0,133413,0.0171838549],[0,133454,0.0038299683],[0,133474,0.0152115002],[0,133478,0.0110326936],[0,133492,0.0152115002],[0,133507,0.8085579712],[0,133518,0.075477212],[0,133678,0.2152610277],[0,133681,0.0171784391],[0,133693,0.015354693],[0,133713,0.1149592586],[0,133715,0.6681212267],[0,133802,0.0054193821],[0,133808,0.0304804403],[0,133854,0.107215988],[0,133856,0.2818226224],[0,133860,0.0346525421],[0,133863,0.0038299683],[0,133867,0.5078570027],[0,133870,0.0260602974],[0,133872,0.0823730696],[0,133873,0.0264764361],[0,133879,0.0373276726],[0,133881,0.2584385233],[0,133886,0.0287379637],[0,133893,0.0243111584],[0,133914,0.5547702419],[0,133928,0.0346143276],[0,133934,0.0036616995],[0,133935,0.0036616995],[0,133941,0.0088752716],[0,133942,0.025354599],[0,133946,0.021463062],[0,133948,0.015354693],[0,133949,0.070176132],[0,133950,0.2312484499],[0,133951,0.4732826744],[0,133952,0.0281501438],[0,134019,0.0038299683],[0,134035,0.0288715438],[0,134039,0.0279332199],[0,134043,0.015354693],[0,134055,0.0305006108],[0,134075,0.0074974794],[0,134079,0.0085612836],[0,134102,0.0034049078],[0,134110,0.2352457963],[0,134115,0.015354693],[0,134137,0.0079246888],[0,134239,0.0088379399],[0,134264,0.0080708686],[0,134273,0.1923541779],[0,134277,0.0036616995],[0,134301,0.0404374892],[0,134327,0.0250701339],[0,134355,0.0707791736],[0,134367,0.0123669383],[0,134374,0.0057469425],[0,134395,0.0074974794],[0,134412,0.2322419757],[0,134448,0.0072482806],[0,134461,0.0667140078],[0,134607,0.0090998294],[0,134622,0.0102061173],[0,134623,0.0504743246],[0,134624,0.015354693],[0,134638,0.0186092974],[0,134660,0.0036616995],[0,134775,0.0263530849],[0,134783,0.013080889],[0,134810,0.0232333645],[0,134819,0.0036616995],[0,134851,0.4092703883],[0,134855,0.0140751011],[0,134856,0.022900113],[0,134873,0.0152115002],[0,134877,0.0036616995],[0,134899,0.0723524704],[0,134908,0.0087419073],[0,134909,0.0206254012],[0,134925,0.0173368326],[0,134932,0.0057830487],[0,134933,0.0323747124],[0,134952,0.0070791572],[0,134955,0.0348530006],[0,134966,0.023448801],[0,134968,0.0037941438],[0,134978,0.1763812483],[0,134988,0.0193258419],[0,135002,0.0503366167],[0,135003,0.0089114464],[0,135004,0.0131957727],[0,135031,0.0104162821],[0,135035,0.6406739701],[0,135054,0.0604064986],[0,135058,0.0098044146],[0,135061,0.0977584354],[0,135071,0.1829793164],[0,135074,0.0541320449],[0,135084,0.015354693],[0,135087,0.0130248611],[0,135117,0.0340456557],[0,135119,0.0036616995],[0,135123,0.005136337],[0,135126,0.3698499489],[0,135137,0.0752283247],[0,135143,0.015354693],[0,135147,0.0260097678],[0,135158,0.0088752716],[0,135161,0.0104815051],[0,135185,0.0042709698],[0,135203,0.0429830263],[0,135204,0.0089114464],[0,135213,0.2597956147],[0,135218,0.0468396235],[0,135219,0.0092257598],[0,135223,0.0192531431],[0,135224,0.0055065954],[0,135315,0.0040311483],[0,135320,0.0156832657],[0,135323,0.0157381618],[0,135341,0.0214566245],[0,135343,0.0126934081],[0,135360,0.0184040126],[0,135386,0.0082964781],[0,135396,0.0292373828],[0,135443,0.0150488231],[0,135460,0.0078690341],[0,135559,0.0133808313],[0,135571,0.0157305614],[0,135578,0.0157305614],[0,135579,0.0041995094],[0,135600,0.0157305614],[0,135627,0.0225618063],[0,135644,0.0167646819],[0,135699,0.0092837773],[0,135708,0.0055065954],[0,135714,0.0106846731],[0,135735,0.0055065954],[0,135738,0.0166303176],[0,135764,0.0054363226],[0,135789,0.0091141452],[0,135846,0.0590524252],[0,135923,0.0078690341],[0,135951,0.0850098538],[0,135959,0.0058626667],[0,135966,0.0058626667],[0,135987,0.0041995094],[0,135994,0.0381384231],[0,135995,0.2528291256],[0,136009,0.0073483795],[0,136016,0.006957088],[0,136070,0.0048092667],[0,136115,0.0092102305],[0,136123,0.0389392233],[0,136125,0.0127297865],[0,136159,0.017368386],[0,136741,0.0078690341],[0,136770,0.0092837773],[0,136774,0.0213290049],[0,136803,0.0040311483],[0,136837,0.0595955821],[0,136841,0.0115943741],[0,136915,0.0608442319],[0,137196,0.0302816301],[0,137218,0.0157305614],[0,137398,0.119925048],[0,137745,0.0037742156],[0,137843,0.0082964781],[0,137845,0.140570769],[0,137894,0.0157305614],[0,137907,0.0157305614],[0,137909,0.1835733115],[0,137926,0.0170493484],[0,137947,0.2668508544],[0,137956,0.0427037783],[0,137964,0.0252872016],[0,137997,0.0109232064],[0,138006,0.0068447281],[0,138016,0.0301448601],[0,138058,0.1622524269],[0,138226,0.0245972884],[0,138338,0.0157305614],[0,138354,0.0094722638],[0,138379,0.008969699],[0,138388,0.0121747005],[0,138423,0.0094722638],[0,138438,0.0106846731],[0,138442,0.1293735476],[0,138452,0.0078690341],[0,138480,0.2635910099],[0,138547,0.0364000414],[0,138562,0.0558355758],[0,138598,0.014032812],[0,138685,0.0092102305],[0,138808,0.0145696815],[0,138818,0.0077201186],[0,138860,0.3676457469],[0,138864,0.0080852771],[0,138887,0.0173997135],[0,138929,0.0044925476],[0,138930,0.0053581995],[0,138974,0.0155799171],[0,138987,0.4948689735],[0,139058,0.2306316958],[0,139093,0.0038830768],[0,139110,0.00906102],[0,139133,0.00906102],[0,139140,0.1974193155],[0,139144,0.0050308277],[0,139150,0.0263445145],[0,139195,0.0282347857],[0,139216,0.0044925476],[0,139351,0.0290967042],[0,139530,0.0071995785],[0,139796,0.014926775],[0,140641,0.0141736458],[0,140695,0.0267461132],[0,140713,0.0340305332],[0,140731,0.0246717901],[0,140735,0.0329155847],[0,140743,0.0727368566],[0,140753,0.0048737775],[0,140758,0.0341700201],[0,140763,0.0049082574],[0,140768,0.013583302],[0,140769,0.0073196162],[0,140772,0.1245156392],[0,140775,0.0452127168],[0,140776,0.0094635821],[0,140788,0.0171827171],[0,140791,0.0054841453],[0,140800,0.0091791436],[0,140801,0.0130624182],[0,140802,0.0437113524],[0,140803,0.0094635821],[0,140835,0.0405683637],[0,140850,0.0625356995],[0,140852,0.0115704498],[0,140856,0.4136771419],[0,140920,0.0408591065],[0,140927,0.0280125537],[0,140964,0.0217732268],[0,141012,0.0122228474],[0,141025,0.0094635821],[0,141048,0.014926775],[0,141120,0.0094635821],[0,141324,0.0435796951],[0,141331,0.4698804366],[0,141464,0.0204165216],[0,141555,0.0055186564],[0,142285,0.0100940864],[0,142441,0.1534572176],[0,142479,0.1377576194],[0,142483,0.0894898825],[0,142535,0.014926775],[0,142542,0.0336642979],[0,142563,0.0063856266],[0,142589,0.0094635821],[0,142637,0.0930366106],[0,142656,0.014926775],[0,142666,0.0100940864],[0,142667,0.0091791436],[0,142669,0.0130640169],[0,142785,0.0094635821],[0,142848,0.0094635821],[0,143702,0.0049082574],[0,143818,0.014926775],[0,143827,0.0091791436],[0,143836,0.0049082574],[0,143894,0.0147836717],[0,143970,0.0211924551],[0,143997,0.0089417013],[0,144029,0.0168959319],[0,144043,0.0101136285],[0,144045,0.0117058668],[0,144060,0.0854288195],[0,144179,0.0375913118],[0,144181,0.0498072353],[0,144189,0.00716823],[0,144246,0.7874258316],[0,144464,0.0070005331],[0,144565,0.0679523502],[0,144772,0.0324940263],[0,144795,0.0077820832],[0,144804,0.0170852452],[0,144940,0.0122763272],[0,145013,0.236623832],[0,145032,0.0132285476],[0,145110,0.1656209948],[0,145123,0.0138541759],[0,145193,0.2265492386],[0,145194,0.00716823],[0,145200,0.0116048173],[0,145209,0.0122373107],[0,145238,0.0122373107],[0,145329,0.0214075916],[0,145359,0.1573204768],[0,145391,0.1050913255],[0,145395,0.1490934838],[0,145471,0.0122373107],[0,145487,0.5588789688],[0,145499,0.0262601276],[0,145571,0.0886025494],[0,145668,0.0237311176],[0,145670,0.014928826],[0,145678,0.0116048173],[0,145680,0.0633343697],[0,145720,0.0279864038],[0,145762,0.0620523076],[0,145806,0.0410701148],[0,145815,0.0202089958],[0,145837,0.0296706516],[0,145843,0.0070005331],[0,145853,0.0138439038],[0,145890,0.0203999989],[0,145980,0.0311265923],[0,146070,0.0129464431],[0,146087,0.0184376699],[0,146121,0.01513559],[0,146127,0.0135452594],[0,146129,0.0259727814],[0,146192,0.065807022],[0,146209,0.0135801828],[0,146323,0.0246263422],[0,146482,0.0237513456],[0,146518,0.0205591155],[0,146572,0.0202696286],[0,146691,0.0344782498],[0,146702,0.0309363509],[0,146713,0.0331751194],[0,146723,0.0149796392],[0,146737,0.0227773216],[0,146759,0.020872431],[0,146765,0.1213119565],[0,146769,0.0083330865],[0,146770,0.0572104411],[0,146771,0.0250539862],[0,146774,0.2736329564],[0,146775,0.830719837],[0,146804,0.0129464431],[0,146805,0.050545707],[0,146808,0.0121320218],[0,146823,0.738125473],[0,146827,0.5551526827],[0,146832,0.0184376699],[0,146843,0.0151657892],[0,146846,0.0205943976],[0,146857,0.0655584233],[0,146858,0.0083330865],[0,146862,0.136925968],[0,146863,0.0518360238],[0,146864,0.0301589545],[0,146868,0.0179890691],[0,146898,0.22497461],[0,146909,0.0134388552],[0,146923,0.2450734613],[0,146976,0.0098179607],[0,146992,0.0184376699],[0,147029,0.0670709022],[0,147078,0.0254319441],[0,147111,0.0184376699],[0,147126,0.6825216599],[0,147152,0.0138439038],[0,147203,0.0091161765],[0,147372,0.0149796392],[0,147378,0.0095817145],[0,147409,0.0126605451],[0,147412,0.0948182872],[0,147446,0.0104328049],[0,147498,0.0083677433],[0,147500,0.0895012195],[0,147743,0.0121954937],[0,147800,0.037889482],[0,147810,0.1071784956],[0,147811,0.0228257569],[0,147814,0.0605763209],[0,147815,0.5765197212],[0,147820,0.0560959147],[0,147830,0.0568378373],[0,147831,0.0236194792],[0,147845,0.0151002407],[0,147846,0.0545472651],[0,147847,0.0085011139],[0,147855,0.53295759],[0,147857,0.038484417],[0,147859,0.0084664502],[0,147865,0.0083677433],[0,147867,0.2002858898],[0,147868,0.0731144774],[0,147882,0.0083330865],[0,147884,0.0273360422],[0,147885,0.0287081239],[0,147886,0.0136192762],[0,147891,0.1432054233],[0,147892,0.0135801828],[0,147895,0.1161581264],[0,147908,0.6024696777],[0,147909,0.0184338256],[0,147924,0.0923954909],[0,147932,0.0083330865],[0,148051,0.0116715125],[0,148069,0.1974639005],[0,148079,0.0129464431],[0,148085,0.0685432293],[0,148150,0.0711324882],[0,148211,0.0138089669],[0,148246,0.0184338256],[0,148357,0.0376181076],[0,148378,0.0365130492],[0,148380,0.1171833443],[0,148424,0.0085025233],[0,148452,0.2761134409],[0,148546,0.0333897749],[0,148646,0.0135801828],[0,148703,0.0184376699],[0,148719,0.0905937562],[0,148780,0.0081091558],[0,148792,0.015064122],[0,148840,0.0202080436],[0,148947,0.0260834297],[0,149007,0.0091508734],[0,149107,0.024936515],[0,149126,0.0355469501],[0,149154,0.0180936891],[0,149164,0.0515450763],[0,149193,0.0135801828],[0,149197,0.7217462366],[0,149237,0.0085371888],[0,149261,0.2069164259],[0,149265,0.0236762892],[0,149272,0.0478894734],[0,149281,0.034463703],[0,149315,0.0184376699],[0,149327,0.0530670646],[0,149366,0.0137380376],[0,149367,0.0184376699],[0,149458,0.2418855673],[0,149554,0.0186070175],[0,149690,0.0473614051],[0,149699,0.0085025233],[0,149751,0.0501870809],[0,149758,0.0184376699],[0,149760,0.010176307],[0,149773,0.2458175364],[0,149774,0.0144274246],[0,149798,0.0098179607],[0,149855,0.0234480542],[0,149883,0.0085025233],[0,149938,0.017087303],[0,149969,0.0129464431],[0,150015,0.015451106],[0,150090,0.0292469991],[0,150092,0.0108208121],[0,150115,0.020962493],[0,150150,0.2223245041],[0,150156,0.0123459981],[0,150158,0.0108208121],[0,150219,0.0119529606],[0,150220,0.015451106],[0,150224,0.010990871],[0,150375,0.0116067772],[0,150379,0.0155601988],[0,150420,0.0149271465],[0,150429,0.015451106],[0,150443,0.0116067772],[0,150446,0.2462070959],[0,150533,0.018618759],[0,150553,0.1029673249],[0,150609,0.0228825173],[0,150650,0.0758430071],[0,150722,0.0744487036],[0,150793,0.011980392],[0,150799,0.0163518616],[0,150818,0.0213749561],[0,150881,0.0724070416],[0,150983,0.0676666314],[0,151002,0.0170292892],[0,151093,0.02045148],[0,151133,0.3695608548],[0,151210,0.0160871724],[0,151342,0.015451106],[0,151409,0.020962493],[0,151436,0.020962493],[0,151447,0.0558789243],[0,151503,0.0428606818],[0,151520,0.197262148],[0,151549,0.0161264093],[0,151619,0.0109894564],[0,151633,0.0234520006],[0,151643,0.0160871724],[0,151660,0.0925648525],[0,151717,0.1975574465],[0,151722,0.010990871],[0,151772,0.02542986],[0,151791,0.0389306751],[0,151800,0.3012193044],[0,151974,0.0353481252],[0,152040,0.0123459981],[0,152065,0.0648272426],[0,152071,0.020962493],[0,152082,0.0988948964],[0,152117,0.015451106],[0,152126,0.0174933758],[0,152155,0.0187938922],[0,152162,0.0208181273],[0,152171,0.015451106],[0,152204,0.0375575338],[0,152238,0.1121579362],[0,152315,0.0123459981],[0,152339,0.015451106],[0,152343,0.4056909365],[0,152359,0.1109259295],[0,152421,0.0163167964],[0,152447,0.1654685898],[0,152463,0.0151641584],[0,152486,0.0502882468],[0,152518,0.0688539339],[0,152556,0.0600949263],[0,152595,0.0160871724],[0,152621,0.0701972431],[0,152745,0.0108555961],[0,152817,0.0160871724],[0,152840,0.020962493],[0,152872,0.0146129524],[0,152885,0.020962493],[0,152948,0.0163518616],[0,152953,0.010990871],[0,152954,0.0828097968],[0,153304,0.0628686281],[0,153314,0.0123459981],[0,153511,0.020962493],[0,153763,0.1620177669],[0,153775,0.0129631311],[0,153870,0.4810761884],[0,153871,0.0340802089],[0,153872,0.0412452267],[0,153925,0.0163518616],[0,153932,0.020962493],[0,153948,0.0231271388],[0,153956,0.0116067772],[0,153958,0.0159550665],[0,154040,0.0206092229],[0,154175,0.0108555961],[0,154348,0.3032909456],[0,154426,0.015451106],[0,154524,0.018618759],[0,154555,0.0161264093],[0,154558,0.1725284595],[0,154570,0.0235478557],[0,154617,0.015451106],[0,154628,0.0211324624],[0,154641,0.1494292327],[0,154652,0.015451106],[0,154668,0.0426772093],[0,154690,0.0163518616],[0,154709,0.0195704232],[0,154729,0.0168136757],[0,154878,0.0109894564],[0,154953,0.0227034509],[0,154955,0.1627939032],[0,154968,0.0256750033],[0,155032,0.015451106],[0,155064,0.010990871],[0,155080,0.0347305027],[0,155099,0.0204725498],[0,155280,0.0160871724],[0,155363,0.0163518616],[0,155432,0.0177131713],[0,155556,0.0172450814],[0,155694,0.0721541183],[0,155815,0.020962493],[0,155922,0.0176292587],[0,155925,0.0163518616],[0,155949,0.0115129128],[0,155955,0.015451106],[0,155978,0.136179006],[0,156025,0.0373285732],[0,156132,0.0125003209],[0,156133,0.6108141985],[0,156160,0.020962493],[0,156327,0.020962493],[0,156341,0.0215213356],[0,156429,0.0561668144],[0,156462,0.0163518616],[0,156532,0.0494973942],[0,156598,0.0209233338],[0,156611,0.019498428],[0,156649,0.0900637442],[0,156666,0.0241770223],[0,156739,0.8874267695],[0,156764,0.0155601988],[0,156771,0.020962493],[0,156847,0.0169701687],[0,156849,0.020962493],[0,156861,0.0170213684],[0,156862,0.010990871],[0,156876,0.0155601988],[0,156892,0.0486521401],[0,156904,0.0390775572],[0,156905,0.7792760789],[0,156930,0.0114365642],[0,156932,0.0889893742],[0,156938,0.0160871724],[0,156940,0.8511968545],[0,156944,0.4358710787],[0,156971,0.0347783165],[0,156972,0.0215732753],[0,156984,0.0276245527],[0,156985,0.0184603064],[0,156989,0.301136868],[0,156990,0.020962493],[0,156991,0.0214306878],[0,156998,0.2900528147],[0,156999,0.0606762887],[0,157006,0.0637839592],[0,157009,0.015451106],[0,157011,0.0131175936],[0,157023,0.2542343173],[0,157025,0.0105960592],[0,157040,0.0561300065],[0,157048,0.0229211479],[0,157109,0.0308016087],[0,157111,0.0108208121],[0,157115,0.1759395239],[0,157117,0.1672758564],[0,157123,0.020962493],[0,157134,0.0203402093],[0,157141,0.0105960592],[0,157195,0.020962493],[0,157230,0.2932415763],[0,157238,0.0358016341],[0,157242,0.0410821381],[0,157243,0.0390661804],[0,157248,0.059350604],[0,157274,0.0108208121],[0,157284,0.0117826611],[0,157288,0.020962493],[0,158120,0.0116067772],[0,158128,0.8014216919],[0,158144,0.0231271388],[0,158232,0.1020393213],[0,158233,0.0590395845],[0,158236,0.0770355897],[0,158270,0.7140390927],[0,158278,0.0428347301],[0,158300,0.0302858389],[0,158306,0.0145289486],[0,158309,0.0163167964],[0,158317,0.020962493],[0,158318,0.2419958643],[0,158332,0.015451106],[0,158337,0.3824924081],[0,158344,0.0163518616],[0,158351,0.0433847806],[0,158361,0.1193709298],[0,158369,0.0215041637],[0,158373,0.0563431058],[0,158427,0.0507992787],[0,158501,0.055719759],[0,158529,0.0290212737],[0,158530,0.0187938922],[0,158537,0.0108555961],[0,158542,0.0461102856],[0,158545,0.0180098777],[0,158546,0.0179407148],[0,158547,0.0156167969],[0,158551,0.0231969401],[0,158552,0.0340377044],[0,158562,0.0190997887],[0,158568,0.0611907455],[0,158626,0.0365956071],[0,158627,0.0175765596],[0,158639,0.020962493],[0,158647,0.0399269472],[0,158655,0.0188099744],[0,158657,0.0201261728],[0,158666,0.0169726216],[0,158669,0.0215041637],[0,158670,0.0445925005],[0,158671,0.0652394703],[0,158681,0.0196599131],[0,158694,0.015451106],[0,158719,0.1494745611],[0,158732,0.0211778379],[0,158740,0.0108208121],[0,158839,0.0348144548],[0,158848,0.0260445499],[0,158850,0.0571081513],[0,158868,0.0229458257],[0,158958,0.1157351136],[0,159048,0.056131748],[0,159072,0.0600652346],[0,159094,0.0108208121],[0,159102,0.0313525343],[0,159103,0.3385519299],[0,159127,0.0108555961],[0,159131,0.0183019413],[0,159135,0.0201261728],[0,159148,0.040950803],[0,159150,0.0108555961],[0,159152,0.0308208135],[0,159156,0.047165831],[0,159168,0.0254843562],[0,159225,0.020962493],[0,159234,0.2630871342],[0,159237,0.0461102143],[0,159248,0.1057645274],[0,159260,0.0161264093],[0,159266,0.020962493],[0,159379,0.014274627],[0,159394,0.015451106],[0,159440,0.0341497363],[0,159447,0.3150118793],[0,159457,0.0424002074],[0,159461,0.0146973996],[0,159562,0.0163518616],[0,159565,0.0109546655],[0,159567,0.0105960592],[0,159568,0.0175050608],[0,159571,0.0879076993],[0,159573,0.020962493],[0,159584,0.0209586346],[0,159587,0.0363047415],[0,159600,0.0123111378],[0,159611,0.0481399564],[0,159618,0.0191893627],[0,159623,0.0204725498],[0,159624,0.0193767865],[0,159657,0.0163518616],[0,159661,0.0233084695],[0,159667,0.3598344237],[0,159672,0.0174917667],[0,159675,0.0579420708],[0,159681,0.020962493],[0,159703,0.0256922788],[0,159707,0.0231271388],[0,159748,0.0159550665],[0,159879,0.0972807294],[0,159892,0.015451106],[0,159906,0.0154400958],[0,159929,0.2285527828],[0,159982,0.0425539411],[0,160004,0.1572035338],[0,160009,0.0200779083],[0,160013,0.0145289486],[0,160018,0.0496629019],[0,160027,0.0175414319],[0,160029,0.0208181273],[0,160040,0.3173425436],[0,160059,0.0163167964],[0,160082,0.0628292031],[0,160090,0.0123111378],[0,160102,0.1302424845],[0,160148,0.0916104347],[0,160152,0.0116067772],[0,160158,0.0501991042],[0,160180,0.0724276453],[0,160184,0.0163167964],[0,160255,0.1999518516],[0,160280,0.0299723462],[0,160282,0.0175414319],[0,160348,0.0161614647],[0,160355,0.5630773036],[0,160357,0.0123475951],[0,160361,0.0114713798],[0,160407,0.0114365642],[0,160413,0.0300624198],[0,160440,0.0114713798],[0,160444,0.0536569257],[0,160469,0.032207347],[0,160489,0.0114365642],[0,160490,0.0187907678],[0,160494,0.059159828],[0,160500,0.026975811],[0,160566,0.0163167964],[0,160571,0.0161264093],[0,160606,0.0353402368],[0,160671,0.0205498198],[0,160722,0.0478421855],[0,160792,0.1441131813],[0,160804,0.0249918664],[0,160814,0.0163518616],[0,160832,0.0508674913],[0,160835,0.093910218],[0,160839,0.0520478879],[0,160880,0.015451106],[0,160884,0.0278566377],[0,160935,0.015451106],[0,160941,0.020962493],[0,160942,0.015451106],[0,160955,0.0192070469],[0,160979,0.0115227705],[0,160986,0.0212588175],[0,161004,0.015451106],[0,161012,0.015451106],[0,161058,0.0163518616],[0,161062,0.0169726216],[0,161073,0.015451106],[0,161092,0.0123111378],[0,161098,0.020962493],[0,161109,0.020962493],[0,161119,0.1137870487],[0,161326,0.2279263102],[0,161413,0.0212399841],[0,161419,0.0253203839],[0,161436,0.0160871724],[0,161464,0.0197801721],[0,161489,0.0346643037],[0,161491,0.0200585132],[0,161496,0.0754033991],[0,161507,0.0108208121],[0,161524,0.0676747516],[0,161527,0.0692667141],[0,161528,0.023192371],[0,161529,0.015451106],[0,161564,0.1073927367],[0,161607,0.0148198181],[0,161617,0.0253203839],[0,161622,0.0108555961],[0,161624,0.0157178572],[0,161653,0.0114780951],[0,161683,0.015451106],[0,161723,0.0341229885],[0,161734,0.0145289486],[0,161739,0.0173992137],[0,161750,0.7184701521],[0,161751,0.0108208121],[0,161775,0.0155601988],[0,161807,0.0277306395],[0,161875,0.8021476627],[0,161891,0.7027445208],[0,161892,0.0432797376],[0,161972,0.1302553339],[0,161983,0.1093993527],[0,161991,0.0238314369],[0,162019,0.0148198181],[0,162069,0.0108208121],[0,162155,0.0161096337],[0,162168,0.010990871],[0,162175,0.0114365642],[0,162233,0.0208181273],[0,162274,0.0294919849],[0,162276,0.1739782607],[0,162280,0.0190464849],[0,162285,0.0108208121],[0,162333,0.0161264093],[0,162335,0.0160871724],[0,162349,0.0231271388],[0,162363,0.0116053612],[0,162422,0.0467831533],[0,162431,0.0374138285],[0,162460,0.0215849157],[0,162463,0.0151641584],[0,162470,0.075001803],[0,162518,0.0151641584],[0,162523,0.6177930991],[0,162532,0.020962493],[0,162540,0.0934009935],[0,162544,0.2522269948],[0,162556,0.0161264093],[0,162558,0.0602809296],[0,162562,0.0165092025],[0,162582,0.212747517],[0,162583,0.0278306946],[0,162611,0.0175765596],[0,162616,0.015451106],[0,162623,0.0163518616],[0,162653,0.0552412629],[0,162680,0.015451106],[0,162702,0.020962493],[0,162725,0.0870773949],[0,162732,0.0160871724],[0,162733,0.020581444],[0,162735,0.0179506533],[0,162816,0.0168402403],[0,162818,0.0214541559],[0,163013,0.03436326],[0,163091,0.016433415],[0,163155,0.0128315233],[0,163173,0.0235190959],[0,163182,0.2035266629],[0,163188,0.0127966382],[0,163203,0.0141835589],[0,163204,0.0128315233],[0,163243,0.1141894224],[0,163257,0.0238018035],[0,163277,0.0159388431],[0,163309,0.0176123288],[0,163325,0.0182493043],[0,163344,0.0159388431],[0,163360,0.173043846],[0,163369,0.0199002578],[0,163419,0.0144162641],[0,163507,0.0381998983],[0,163565,0.0165753625],[0,163627,0.0159388431],[0,163656,0.0214502947],[0,163658,0.4886049821],[0,163664,0.0186783229],[0,163670,0.0345986964],[0,163717,0.0214541559],[0,163785,0.0113052509],[0,163790,0.0536522044],[0,163795,0.0114740153],[0,163836,0.0471609312],[0,163840,0.0374466132],[0,163893,0.0159388431],[0,163913,0.0179809573],[0,163930,0.0219672894],[0,163961,0.019536783],[0,163974,0.0114740153],[0,163975,0.0265000214],[0,164035,0.1384532233],[0,164054,0.0165753625],[0,164080,0.0165753625],[0,164126,0.0788671846],[0,164130,0.0119214417],[0,164131,0.1030286381],[0,164143,0.0587340279],[0,164150,0.1116136147],[0,164152,0.0128315233],[0,164169,0.0124382059],[0,164178,0.1176701199],[0,164182,0.0159388431],[0,164183,0.0114391997],[0,164184,0.0156516911],[0,164185,0.0228997952],[0,164235,0.0166146274],[0,164254,0.0237033193],[0,164262,0.0214541559],[0,164263,0.0679004459],[0,164300,0.0159388431],[0,164321,0.0349648169],[0,164468,0.0174614424],[0,164495,0.0349048817],[0,164551,0.0214541559],[0,164558,0.0269088201],[0,164573,0.0128315233],[0,164579,0.2171902218],[0,164598,0.0942754724],[0,164656,0.2295162376],[0,164685,0.0312225064],[0,164692,0.3696976784],[0,164705,0.0209427789],[0,164719,0.0168402403],[0,164722,0.0385455688],[0,164729,0.1534474589],[0,164777,0.4805767263],[0,164811,0.0261123218],[0,164880,0.011475431],[0,164888,0.1058981472],[0,165035,0.1969758911],[0,165046,0.0134298322],[0,165047,0.1525058038],[0,165086,0.0777740672],[0,165123,0.0128315233],[0,165151,0.0684698726],[0,165201,0.8451737633],[0,165452,0.0364274892],[0,165476,0.0127966382],[0,165480,0.0165753625],[0,165588,0.6671947322],[0,165594,0.011956282],[0,165615,0.0159388431],[0,166360,0.0192840104],[0,166383,0.011080338],[0,166465,0.0159388431],[0,166493,0.0213096874],[0,166513,0.0159388431],[0,166520,0.011475431],[0,166617,0.0114391997],[0,166631,0.0446634301],[0,166647,0.1799328875],[0,166731,0.011956282],[0,166739,0.0432045877],[0,166756,0.055274369],[0,166757,0.0214541559],[0,166769,0.0214541559],[0,166804,0.0335664744],[0,166833,0.0252139449],[0,166835,0.0168402403],[0,166846,0.0203265542],[0,166927,0.0113052509],[0,167100,0.0113400597],[0,167103,0.0214541559],[0,167144,0.0428683878],[0,167261,0.0156516911],[0,167266,0.0159388431],[0,167413,0.0165753625],[0,167735,0.0119214417],[0,167741,0.0165753625],[0,167950,0.0198673199],[0,167958,0.0159388431],[0,168077,0.0119978446],[0,168085,0.0180306579],[0,168143,0.3889014735],[0,168154,0.019536783],[0,168235,0.0159388431],[0,168349,0.0128315233],[0,168393,0.1358069531],[0,168610,0.0140479466],[0,168924,0.0110455424],[0,168989,0.0285123853],[0,169004,0.0121266249],[0,169137,0.0219962125],[0,169219,0.0227369897],[0,169247,0.02288917],[0,169324,0.0166146274],[0,169455,0.0159388431],[0,169477,0.0214541559],[0,178046,0.0887502985],[0,178201,0.1057928459],[0,178204,0.0145336634],[0,180619,0.0126214999],[0,180642,0.3159294034],[0,180704,0.0105715104],[0,180733,0.0148452018],[0,180742,0.0209672521],[0,180754,0.0154558271],[0,180848,0.0154558271],[0,180917,0.0085290031],[0,180928,0.0864253055],[0,180929,0.2545491046],[0,180931,0.243231708],[0,180932,0.90028109],[0,180933,0.0131840599],[0,180942,0.0102028522],[0,180959,0.0154558271],[0,180995,0.0320173121],[0,181000,0.0132524188],[0,181036,0.0154558271],[0,181093,0.250766246],[0,181127,0.0154558271],[0,181132,0.0209672521],[0,181140,0.0341336068],[0,181193,0.0141728547],[0,181206,0.0261366715],[0,181242,0.00856367],[0,181245,0.0127920091],[0,181274,0.0297202597],[0,181359,0.0167305035],[0,181387,0.0301253454],[0,181509,0.0246094068],[0,181512,0.0172542777],[0,181514,0.0121415259],[0,181519,0.0100506411],[0,181615,0.0270215689],[0,181695,0.0190928427],[0,181719,0.0146786914],[0,181759,0.0407919159],[0,181762,0.0465358387],[0,181765,0.0145336634],[0,181782,0.0453639734],[0,181783,0.3729913693],[0,181795,0.5225793293],[0,181974,0.0154558271],[0,181985,0.0141728547],[0,181986,0.0091426803],[0,181995,0.1471832069],[0,182014,0.0086970791],[0,182074,0.0234965208],[0,182084,0.0958596286],[0,182139,0.0137809669],[0,182193,0.0138167213],[0,182199,0.0138167213],[0,182228,0.0154558271],[0,182264,0.1331298549],[0,182273,0.0188751968],[0,182355,0.0132524188],[0,182368,0.0448488529],[0,182370,0.0318640937],[0,182412,0.0367945628],[0,182457,0.0132524188],[0,182479,0.0102028522],[0,182485,0.0968416055],[0,182518,0.0302608425],[0,182538,0.0197179928],[0,182553,0.0238862496],[0,182559,0.0109084048],[0,182563,0.0528203507],[0,182572,0.0086970791],[0,182601,0.0582188563],[0,182622,0.0392428945],[0,182626,0.0141728547],[0,182745,0.0332441435],[0,182795,0.0140414139],[0,182883,0.0198598339],[0,182889,0.011011401],[0,182890,0.0150852365],[0,182903,0.0091773786],[0,182943,0.0151188862],[0,182953,0.0209672521],[0,182968,0.0210921224],[0,182969,0.0159533114],[0,183010,0.0209672521],[0,183019,0.0140414139],[0,183084,0.018175298],[0,183097,0.0100143067],[0,183192,0.2530998819],[0,183218,0.0100490495],[0,183226,0.2659746763],[0,183233,0.0851443261],[0,183254,0.0209672521],[0,183258,0.020722554],[0,183299,0.0154558271],[0,183356,0.033162853],[0,183481,0.0154558271],[0,183529,0.6149703253],[0,183537,0.0344381601],[0,183538,0.0207938594],[0,183573,0.0902218506],[0,183587,0.0167269567],[0,183671,0.0274583555],[0,183676,0.0505307331],[0,183687,0.0207585671],[0,183690,0.0122282257],[0,183709,0.0320920556],[0,183723,0.1007818285],[0,183771,0.0223693998],[0,183776,0.0100490495],[0,183781,0.0154558271],[0,183829,0.0085290031],[0,183844,0.3769186062],[0,183934,0.6006491207],[0,183999,0.0209672521],[0,184029,0.0085290031],[0,184045,0.3110555395],[0,184060,0.0230119552],[0,184066,0.0195321081],[0,184411,0.0134850897],[0,184428,0.0112856299],[0,184464,0.3471564713],[0,184474,0.0202726498],[0,184475,0.0125655801],[0,184480,0.2699840046],[0,184483,0.014766771],[0,184484,0.0249914092],[0,184492,0.0089281971],[0,184498,0.0202726498],[0,184509,0.4302710746],[0,184516,0.0202726498],[0,184519,0.0179404097],[0,184521,0.1724376534],[0,184533,0.0368779399],[0,184576,0.1741730665],[0,184587,0.0295442657],[0,184747,0.030169284],[0,184767,0.0134850897],[0,184923,0.146867186],[0,184975,0.014766771],[0,185013,0.2077463356],[0,185025,0.0089741156],[0,185095,0.0106691156],[0,185134,0.0201284284],[0,185162,0.0129223311],[0,185264,0.0202726498],[0,185283,0.014766771],[0,185370,0.0261857974],[0,185375,0.0423104303],[0,185435,0.0300035324],[0,185503,0.0230649392],[0,185523,0.0442530114],[0,185620,0.1806935273],[0,185635,0.0135872126],[0,185648,0.0594597949],[0,185676,0.0377914961],[0,185699,0.019083597],[0,185713,0.0133589854],[0,185753,0.4639920445],[0,185767,0.081468518],[0,185853,0.0596217963],[0,185941,0.0123077414],[0,186046,0.0135872126],[0,186091,0.0072912943],[0,186117,0.0319857711],[0,186118,0.0135872126],[0,186154,0.0511338247],[0,186155,0.7040677724],[0,186307,0.008808565],[0,186317,0.0167553786],[0,186320,0.0121418076],[0,186323,0.0200780521],[0,186418,0.0113564546],[0,186441,0.0123077414],[0,186463,0.0133938993],[0,186496,0.0074604707],[0,186537,0.0128355579],[0,186564,0.1540112467],[0,186578,0.0297690608],[0,186582,0.0185627327],[0,186644,0.0881132805],[0,186669,0.019083597],[0,186676,0.0068483152],[0,186683,0.178573105],[0,186689,0.0077587067],[0,186691,0.0645074138],[0,186702,0.0226678616],[0,186723,0.0971118232],[0,186726,0.0192983556],[0,186732,0.0067138641],[0,186736,0.0305781136],[0,186856,0.0129160296],[0,186862,0.0388401966],[0,186881,0.3168672252],[0,186925,0.0074604707],[0,186931,0.0136524253],[0,186934,0.0088826258],[0,186974,0.0431243409],[0,186983,0.0113898174],[0,187035,0.0113898174],[0,187038,0.0243920639],[0,187162,0.0066792919],[0,187170,0.0068123301],[0,187179,0.0234937396],[0,187186,0.0412971599],[0,187196,0.0072912943],[0,187233,0.0068469092],[0,187242,0.0135872126],[0,187250,0.0068469092],[0,187257,0.0247775415],[0,187272,0.0135872126],[0,187274,0.019083597],[0,187282,0.1565644961],[0,187292,0.0101120743],[0,187306,0.0214090931],[0,187322,0.0658773805],[0,187451,0.1651255618],[0,187453,0.0705473584],[0,187454,0.1520175044],[0,187455,0.1435276087],[0,187456,0.1104807653],[0,187457,0.0863044783],[0,187458,0.146711603],[0,187465,0.0929252639],[0,187466,0.0909483685],[0,187467,0.099449135],[0,187468,0.1498533478],[0,187471,0.0764433056],[0,187472,0.1439307498],[0,187473,0.197766071],[0,187474,0.1111499163],[0,187487,0.0314947948],[0,187502,0.0118799887],[0,187548,0.0225194499],[0,187732,0.0148898636],[0,187733,0.019083597],[0,187806,0.0121766592],[0,187891,0.0133938993],[0,188079,0.00819519],[0,188136,0.0119525799],[0,188138,0.1129372342],[0,188142,0.0140690794],[0,188172,0.0066792919],[0,188182,0.0113898174],[0,188231,0.0121418076],[0,188333,0.0135872126],[0,188516,0.0135872126],[0,188520,0.5260465042],[0,188532,0.0123077414],[0,188545,0.0117459532],[0,188554,0.040393812],[0,188555,0.0168536913],[0,188640,0.0426171188],[0,188657,0.0086596902],[0,188734,0.0834181564],[0,188887,0.33290383],[0,189041,0.0181077595],[0,189042,0.0224788279],[0,189047,0.204039878],[0,189182,0.0149683424],[0,189194,0.0214915836],[0,189226,0.0418460227],[0,189390,0.2823818006],[0,189399,0.0242764691],[0,189402,0.0149683424],[0,189636,0.0743165668],[0,189787,0.0204758436],[0,189908,0.0127665028],[0,189958,0.0133304035],[0,189972,0.0162736281],[0,189991,0.128180805],[0,189999,0.0204758436],[0,190077,0.0140468352],[0,190145,0.0174546658],[0,190147,0.0212077745],[0,190176,0.0106776018],[0,190268,0.0174546658],[0,190272,0.0174546658],[0,190285,0.0229821796],[0,190293,0.0165298101],[0,190450,0.0221069725],[0,190462,0.0401685283],[0,190641,0.0136149883],[0,190661,0.0120321048],[0,190674,0.0106776018],[0,190678,0.0436155265],[0,190684,0.0155679514],[0,190686,0.3389332838],[0,190703,0.0174546658],[0,190713,0.0228373915],[0,190729,0.5679100949],[0,190731,0.0174546658],[0,190773,0.0174546658],[0,190779,0.0106761878],[0,190835,0.0152448254],[0,190836,0.0817409116],[0,190893,0.010641413],[0,190894,0.041113199],[0,190916,0.0229821796],[0,191194,0.043673504],[0,191196,0.0229821796],[0,191251,0.012356757],[0,191408,0.555499618],[0,191437,0.0680107281],[0,191508,0.0187646945],[0,191509,0.0174546658],[0,191648,0.4868796778],[0,191703,0.0174546658],[0,191704,0.0119972606],[0,191707,0.0221148431],[0,191738,0.081561115],[0,191758,0.055613273],[0,191769,0.0471041428],[0,191785,0.0155679514],[0,191865,0.0221434099],[0,191867,0.0105076212],[0,191874,0.0160361236],[0,191875,0.0173211598],[0,191879,0.0174546658],[0,191890,0.0192583664],[0,191907,0.1077596185],[0,191915,0.1411528408],[0,191917,0.0249304296],[0,191928,0.0324965195],[0,191929,0.0529851941],[0,191955,0.021784476],[0,192008,0.0157480179],[0,192020,0.2832481433],[0,192035,0.022692184],[0,192083,0.021784476],[0,192098,0.0868313315],[0,192109,0.1004895544],[0,192110,0.0881249394],[0,192112,0.0970372778],[0,192114,0.0906911428],[0,192115,0.0876091288],[0,192116,0.1839222158],[0,192120,0.0601156772],[0,192159,0.0298150446],[0,192168,0.0165398978],[0,192205,0.021784476],[0,192207,0.0211126238],[0,192224,0.0157480179],[0,192244,0.011793512],[0,192248,0.1774021022],[0,192259,0.3524850847],[0,192261,0.0111760219],[0,192325,0.0116580775],[0,192326,0.033766699],[0,192331,0.0177649318],[0,192362,0.021784476],[0,192365,0.0133590996],[0,192371,0.021784476],[0,192383,0.0165398978],[0,192388,0.0497658255],[0,192529,0.0179594828],[0,192577,0.0116580775],[0,192613,0.0534255019],[0,192658,0.0110073314],[0,192673,0.0163143837],[0,192680,0.2852103089],[0,192722,0.0817625429],[0,192724,0.021784476],[0,192726,0.0216399384],[0,192782,0.1543056232],[0,192836,0.021784476],[0,192891,0.0516782949],[0,193035,0.0811076688],[0,193054,0.1279190534],[0,193126,0.1975691496],[0,193241,0.0494353923],[0,193397,0.032373264],[0,193398,0.011177437],[0,193496,0.0166718193],[0,193497,0.0125329357],[0,193601,0.0209623175],[0,193616,0.0851137619],[0,193622,0.1490081478],[0,193623,0.0179594828],[0,193634,0.0111760219],[0,193672,0.4686125665],[0,193705,0.0231221101],[0,193774,0.0433539955],[0,193790,0.0177297943],[0,193800,0.0188747173],[0,193835,0.021784476],[0,193841,0.0110073314],[0,193844,0.0217107896],[0,193875,0.011042125],[0,193918,0.021784476],[0,193934,0.011177437],[0,193937,0.0211126238],[0,193954,0.011042125],[0,193958,0.0157480179],[0,193979,0.0110073314],[0,194035,0.013150238],[0,194050,0.0116232524],[0,194125,0.011793512],[0,194167,0.011793512],[0,194185,0.0162784985],[0,194197,0.0124980658],[0,194285,0.0195078096],[0,194298,0.0352958337],[0,194310,0.0389350337],[0,194318,0.0327630167],[0,194321,0.0361483337],[0,194324,0.0179594828],[0,194337,0.0124980658],[0,194340,0.0144620936],[0,194350,0.6069351946],[0,194380,0.021784476],[0,194408,0.0589414191],[0,194416,0.0894138814],[0,194445,0.0152522668],[0,194456,0.0307780503],[0,194479,0.0179594828],[0,194488,0.0165398978],[0,194494,0.0110073314],[0,194611,0.4413871652],[0,194648,0.0157480179],[0,194650,0.0179594828],[0,194687,0.011042125],[0,194692,0.0161332506],[0,194708,0.0179594828],[0,194711,0.021784476],[0,194724,0.011177437],[0,194752,0.0170339472],[0,194787,0.0111412215],[0,194829,0.0111760219],[0,194836,0.0119694443],[0,194851,0.0110073314],[0,194867,0.0207394735],[0,194869,0.0436933675],[0,194872,0.0111760219],[0,194897,0.011042125],[0,194905,0.021784476],[0,194936,0.0120936576],[0,194940,0.1670228841],[0,194964,0.021784476],[0,194974,0.0166718193],[0,194976,0.0179594828],[0,194991,0.0242345277],[0,195194,0.1827859553],[0,195220,0.0111760219],[0,195264,0.0224631997],[0,195268,0.0676764789],[0,195275,0.014633063],[0,195276,0.129667815],[0,195283,0.0245851327],[0,195345,0.0163143837],[0,195386,0.021784476],[0,195390,0.0110073314],[0,195393,0.021784476],[0,195427,0.0216399384],[0,195483,0.2138982625],[0,195542,0.0179594828],[0,195567,0.2417631085],[0,195591,0.0125345331],[0,195620,0.0111760219],[0,195666,0.0322298397],[0,195685,0.0170339472],[0,195687,0.0165398978],[0,195690,0.011177437],[0,195706,0.021784476],[0,195718,0.0157480179],[0,195729,0.0166718193],[0,195815,0.021784476],[0,195828,0.021784476],[0,195838,0.0165398978],[0,195874,0.0165398978],[0,195931,0.0119694443],[0,195951,0.021784476],[0,196043,0.0110073314],[0,196067,0.0157480179],[0,196073,0.011793512],[0,196142,0.2873179981],[0,196167,0.0661015383],[0,196225,0.0160713814],[0,196226,0.0209120126],[0,196271,0.0162100345],[0,196297,0.2460049403],[0,196321,0.0166718193],[0,196379,0.0216399384],[0,196424,0.1202710461],[0,196446,0.0110073314],[0,196456,0.0166718193],[0,196509,0.0127567368],[0,196547,0.0157480179],[0,196563,0.0124980658],[0,196569,0.0362803418],[0,196585,0.016106434],[0,196619,0.0324511377],[0,196636,0.0153762688],[0,196649,0.0152540052],[0,196664,0.0219330623],[0,196838,0.3568687452],[0,196842,0.0346690039],[0,196874,0.0110073314],[0,196915,0.0111760219],[0,196968,0.011177437],[0,196977,0.0111760219],[0,197013,0.015679409],[0,197059,0.021784476],[0,197071,0.0192352601],[0,197086,0.0515331596],[0,197154,0.0297293446],[0,197201,0.032635526],[0,197233,0.0233170332],[0,197287,0.0121397904],[0,197314,0.0166718193],[0,197352,0.013150238],[0,197396,0.021784476],[0,197399,0.0179594828],[0,197405,0.0110073314],[0,197435,0.0216399384],[0,197481,0.0179594828],[0,197482,0.0166718193],[0,197485,0.017160828],[0,197491,0.0165398978],[0,197575,0.021784476],[0,197589,0.0179594828],[0,197694,0.0244920307],[0,197708,0.180182862],[0,197801,0.0176083424],[0,197823,0.0221194936],[0,197839,0.0181032504],[0,197845,0.2148576718],[0,197868,0.0398342061],[0,197880,0.0166718193],[0,197935,0.0121672294],[0,197937,0.0111760219],[0,197940,0.0233170332],[0,198031,0.0166481295],[0,198067,0.0419981999],[0,198143,0.0355153845],[0,198159,0.0386991215],[0,198191,0.0157480179],[0,198210,0.1498897074],[0,198269,0.0111412215],[0,198307,0.0157480179],[0,198314,0.0307201869],[0,198321,0.0177649318],[0,198322,0.2872306337],[0,198325,0.0166718193],[0,198353,0.0202090097],[0,198368,0.0193781771],[0,198378,0.0125329357],[0,198380,0.021784476],[0,198391,0.0179457639],[0,198393,0.3835504404],[0,198409,0.2633826391],[0,198464,0.2562918567],[0,198469,0.0274592983],[0,198475,0.016106434],[0,198478,0.0192704745],[0,198496,0.0170339472],[0,198522,0.0606758601],[0,198587,0.2918052471],[0,198591,0.1137657841],[0,198602,0.0111760219],[0,198612,0.0165398978],[0,198660,0.0688992637],[0,198679,0.0172030153],[0,198682,0.0512941262],[0,198684,0.021784476],[0,198709,0.0111760219],[0,198733,0.0166718193],[0,198751,0.035759945],[0,198822,0.0179594828],[0,198857,0.0166718193],[0,198859,0.0217971546],[0,198869,0.0707462758],[0,199011,0.0179594828],[0,199066,0.6245600347],[0,199131,0.0166718193],[0,199142,0.0280478292],[0,199168,0.0243100798],[0,199201,0.0403429783],[0,199315,0.011177437],[0,199328,0.0166718193],[0,199333,0.022937897],[0,199369,0.0127567368],[0,199389,0.0166718193],[0,199443,0.0119694443],[0,199466,0.0179594828],[0,199617,0.0166718193],[0,199626,0.2022589912],[0,199665,0.0211359103],[0,199670,0.0125329357],[0,199679,0.0485567222],[0,199684,0.0179594828],[0,199757,0.2420653169],[0,199779,0.0124980658],[0,199786,0.016084587],[0,199798,0.0166718193],[0,199819,0.364508945],[0,199820,0.1104953245],[0,199821,0.2466298957],[0,199826,0.2164586412],[0,199828,0.2370876636],[0,199829,0.2370176077],[0,199832,0.0327021603],[0,199834,0.2551244858],[0,199837,0.1797042144],[0,199868,0.0270343164],[0,199872,0.1850179949],[0,199875,0.1489302583],[0,199876,0.1555424959],[0,199878,0.056054159],[0,199894,0.0206345051],[0,199911,0.0179594828],[0,199915,0.0165398978],[0,200005,0.0703089413],[0,200073,0.0276798556],[0,200088,0.0296931349],[0,200093,0.1595298694],[0,200108,0.0184406564],[0,200112,0.0120588102],[0,200125,0.0835973188],[0,200129,0.0337607094],[0,200144,0.2038314473],[0,200146,0.0332292862],[0,200167,0.0211479343],[0,200172,0.0432541741],[0,200194,0.050219483],[0,200252,0.0280296797],[0,200277,0.0235543022],[0,200308,0.1354332272],[0,200311,0.1387021317],[0,200314,0.021784476],[0,200317,0.0216399384],[0,200360,0.0179594828],[0,200363,0.0117572638],[0,200374,0.0165808042],[0,200387,0.1061683576],[0,200394,0.0409530137],[0,200395,0.021784476],[0,200396,0.0355717017],[0,200403,0.0365103485],[0,200405,0.0405775338],[0,200406,0.0470909449],[0,200466,0.0179594828],[0,200481,0.011042125],[0,200517,0.0110073314],[0,200547,0.0300018369],[0,200553,0.0832024308],[0,200564,0.0176083424],[0,200586,0.0110073314],[0,200634,0.0317305926],[0,200639,0.0412857206],[0,200652,0.027894638],[0,200718,0.3488526839],[0,200728,0.0166718193],[0,200743,0.0177649318],[0,200746,0.0110073314],[0,200771,0.021784476],[0,200772,0.072239883],[0,200773,0.0448749089],[0,200775,0.0259062784],[0,200783,0.1530398057],[0,200784,0.0428900806],[0,200790,0.0232665913],[0,200793,0.0207770294],[0,200794,0.021784476],[0,200800,0.0259062784],[0,200829,0.1220080319],[0,200841,0.0259369607],[0,200842,0.1317544729],[0,200847,0.0355717017],[0,200849,0.021784476],[0,200851,0.0184406564],[0,200856,0.155509486],[0,200927,0.0156060419],[0,201039,0.0246635159],[0,201048,0.0156358505],[0,201063,0.0201432927],[0,201088,0.0216399384],[0,201092,0.0353586825],[0,201093,0.0133590996],[0,201094,0.0459970641],[0,201095,0.0179594828],[0,201102,0.0111412215],[0,201103,0.4284020242],[0,201106,0.021784476],[0,201119,0.0977673536],[0,201141,0.191535408],[0,201150,0.0170339472],[0,201194,0.0186532775],[0,201207,0.0311293524],[0,201223,0.0170339472],[0,201272,0.0283547036],[0,201314,0.0125329357],[0,201316,0.0240030093],[0,201336,0.0207394735],[0,201353,0.0166718193],[0,201418,0.7301385032],[0,201426,0.0664891649],[0,201454,0.0127567368],[0,201485,0.011177437],[0,201526,0.0384668651],[0,201717,0.0872107539],[0,201734,0.060459415],[0,201740,0.0491442431],[0,201751,0.0644652049],[0,201777,0.015679409],[0,201814,0.0448371365],[0,201815,0.011177437],[0,201817,0.0128578264],[0,201824,0.0165398978],[0,201841,0.0116580775],[0,201844,0.0326817572],[0,201861,0.0564386034],[0,201867,0.0252034044],[0,201871,0.0282031292],[0,201879,0.0116232524],[0,201940,0.0988096949],[0,202006,0.0111760219],[0,202128,0.0179594828],[0,202134,0.0185491243],[0,202135,0.1590601121],[0,202149,0.1056337927],[0,202150,0.2275135077],[0,202179,0.0179594828],[0,202205,0.0407477223],[0,202234,0.014633063],[0,202294,0.018007459],[0,202324,0.011177437],[0,202326,0.0166718193],[0,202367,0.07250345],[0,202376,0.2015361686],[0,202381,0.1940467384],[0,202382,0.0691631012],[0,202387,0.2732764223],[0,202389,0.2730019302],[0,202416,0.0212468765],[0,202447,0.0987136781],[0,202499,0.0252805125],[0,202507,0.0179594828],[0,202579,0.011177437],[0,202594,0.0323115131],[0,202625,0.021784476],[0,202660,0.0284182339],[0,202664,0.0118283458],[0,202673,0.0866562657],[0,202700,0.051112103],[0,202727,0.0230868521],[0,202748,0.021784476],[0,202763,0.0216229044],[0,202851,0.0370201259],[0,202862,0.0747653945],[0,202863,0.0821426445],[0,202865,0.0179594828],[0,202872,0.0416479618],[0,202971,0.0614272483],[0,203172,0.0457089192],[0,203218,0.011042125],[0,203226,0.021784476],[0,203234,0.0648039099],[0,203248,0.0125329357],[0,203249,0.0110073314],[0,203333,0.0207394735],[0,203338,0.0318486562],[0,203347,0.021784476],[0,203389,0.0170339472],[0,203423,0.0929031205],[0,203433,0.0119694443],[0,203441,0.0948156856],[0,203450,0.0157480179],[0,203485,0.1796728916],[0,203493,0.0166718193],[0,203508,0.0133954331],[0,203520,0.0274631835],[0,203556,0.0179594828],[0,203564,0.1393382055],[0,203583,0.0355035105],[0,203663,0.1308453407],[0,203706,0.062027585],[0,203741,0.021784476],[0,203755,0.0219159651],[0,203831,0.0153762688],[0,203909,0.021784476],[0,203914,0.0179594828],[0,203984,0.0983139274],[0,204000,0.0177297943],[0,204063,0.0959244828],[0,204066,0.0207433191],[0,204137,0.0538104295],[0,204173,0.0179594828],[0,204205,0.0599192769],[0,204363,0.0170321119],[0,204434,0.0208519436],[0,204531,0.017043413],[0,204635,0.0107044319],[0,204662,0.0261152518],[0,204689,0.0569651934],[0,204692,0.2343410451],[0,204718,0.3500813094],[0,204771,0.0208519436],[0,204782,0.010259218],[0,204958,0.0208519436],[0,204995,0.3507132004],[0,205004,0.015678967],[0,205009,0.2936739615],[0,205040,0.1090216792],[0,205061,0.0169742882],[0,205062,0.010089342],[0,205064,0.0186438207],[0,205069,0.054630024],[0,205073,0.0208519436],[0,205098,0.0170321119],[0,205152,0.1046209669],[0,205169,0.1280806894],[0,205199,0.3500888043],[0,205222,0.0183413346],[0,205292,0.010089342],[0,205315,0.0108730473],[0,205330,0.0398434429],[0,205343,0.0107044319],[0,205347,0.0108744617],[0,205363,0.0203258565],[0,205380,0.0170321119],[0,205384,0.010259218],[0,205388,0.0170321119],[0,205447,0.0658764756],[0,205454,0.0223393474],[0,205462,0.0735865428],[0,205521,0.0125774545],[0,205524,0.0168378235],[0,205528,0.3789223838],[0,205541,0.0389350337],[0,205612,0.0683141109],[0,205616,0.0101240886],[0,205781,0.3490611876],[0,205806,0.0122293569],[0,205838,0.0124742211],[0,205853,0.0148236313],[0,205877,0.2669603249],[0,205882,0.0195679496],[0,205908,0.1670436983],[0,205918,0.2074558502],[0,205932,0.0208519436],[0,205957,0.0138981236],[0,206022,0.0170321119],[0,206028,0.010259218],[0,206068,0.0960003614],[0,206074,0.0198027183],[0,206090,0.0170321119],[0,206194,0.9206316474],[0,206253,0.2147899407],[0,206287,0.0185112829],[0,206291,0.105297253],[0,206353,0.0154242505],[0,206359,0.2265211808],[0,206469,0.1062898872],[0,206490,0.7855867463],[0,206590,0.010089342],[0,206623,0.0811130543],[0,206761,0.1217256151],[0,206771,0.0211208266],[0,206829,0.0162345349],[0,206840,0.0282954601],[0,206849,0.0273594864],[0,206885,0.0122101276],[0,206887,0.0656361451],[0,206904,0.0155794152],[0,206916,0.032635526],[0,206974,0.015508302],[0,207038,0.0208519436],[0,207053,0.0147886443],[0,207188,0.0234390629],[0,207204,0.0108730473],[0,207205,0.0213183719],[0,207206,0.0238363822],[0,207235,0.054093391],[0,207277,0.0168027335],[0,207279,0.0208519436],[0,207378,0.0512563244],[0,207392,0.0119373399],[0,207394,0.0112202729],[0,207413,0.0361647688],[0,207430,0.0176164515],[0,207431,0.0170321119],[0,207491,0.0343456281],[0,207574,0.0426923339],[0,207577,0.0208519436],[0,207579,0.0157461861],[0,207596,0.0170321119],[0,207631,0.018845272],[0,207633,0.0357693481],[0,207669,0.0107044319],[0,207742,0.0212138478],[0,207800,0.0193982455],[0,207802,0.024905227],[0,207839,0.0373134183],[0,207921,0.0208519436],[0,207980,0.1462519448],[0,207993,0.0162100345],[0,208037,0.0171756856],[0,208071,0.010089342],[0,208218,0.4202764415],[0,208250,0.1192852352],[0,208284,0.0208519436],[0,208339,0.0653687921],[0,208379,0.1020627182],[0,208509,0.0156144426],[0,208537,0.010089342],[0,208551,0.010089342],[0,208563,0.0234718682],[0,208567,0.5593516488],[0,208572,0.0209382689],[0,208580,0.0170321119],[0,208651,0.0246929657],[0,208675,0.0325176464],[0,208691,0.010089342],[0,208731,0.0157461861],[0,208736,0.1746072856],[0,208852,0.0170321119],[0,208879,0.0285902348],[0,208908,0.1176257218],[0,208937,0.0198121916],[0,208951,0.0172603155],[0,208953,0.0392359933],[0,208982,0.2047403677],[0,209001,0.0101240886],[0,209031,0.147062945],[0,209080,0.0170321119],[0,209100,0.0102578049],[0,209248,0.0157461861],[0,209331,0.0170321119],[0,209343,0.0170321119],[0,209403,0.0278585016],[0,209497,0.0185112829],[0,209508,0.0108744617],[0,209641,0.0202162608],[0,209655,0.029827316],[0,209745,0.2825516134],[0,209895,0.0171583745],[0,209900,0.0437466973],[0,209925,0.0156144426],[0,209942,0.106995028],[0,209964,0.0799735917],[0,209966,0.0683638583],[0,209967,0.2276592711],[0,209968,0.2096138769],[0,210042,0.8055078073],[0,210045,0.0171756856],[0,210057,0.0161078253],[0,210085,0.0441388554],[0,210124,0.0268994621],[0,210125,0.0102578049],[0,210132,0.0208519436],[0,210172,0.013055153],[0,210173,0.0218132955],[0,210178,0.0157461861],[0,210184,0.0229264628],[0,210196,0.0740282952],[0,210219,0.0101240886],[0,210250,0.0337727976],[0,210257,0.0207076011],[0,210289,0.1630107706],[0,210318,0.0513912443],[0,210470,0.0208498974],[0,210516,0.0540392112],[0,210690,0.0122293569],[0,210709,0.0153533961],[0,210974,0.0208519436],[0,211006,0.0293934665],[0,211129,0.3257070441],[0,211140,0.0332134301],[0,211167,0.0884352326],[0,211175,0.0170321119],[0,211257,0.0108744617],[0,211296,0.0208519436],[0,211451,0.0429688448],[0,211586,0.0661967812],[0,211588,0.0198438739],[0,211597,0.5559577147],[0,211669,0.043097878],[0,211745,0.0611222052],[0,211795,0.0515850133],[0,211855,0.2643419254],[0,211861,0.0329741981],[0,211862,0.063486972],[0,211904,0.0148236313],[0,211924,0.0582031161],[0,212098,0.061921923],[0,212265,0.1010105809],[0,212271,0.121821195],[0,212301,0.0202162608],[0,212361,0.1616212656],[0,212374,0.0411606334],[0,212569,0.0170321119],[0,212687,0.4454497372],[0,212714,0.1629054736],[0,212722,0.0155794152],[0,212753,0.0278862335],[0,212811,0.3423497476],[0,212869,0.010089342],[0,213002,0.0386279399],[0,213003,0.0183413346],[0,213014,0.0406719545],[0,213031,0.0869795745],[0,213051,0.0316006343],[0,213057,0.0680594195],[0,213223,0.0183413346],[0,213326,0.0208519436],[0,213348,0.0208519436],[0,213375,0.1105431389],[0,213402,0.0148236313],[0,213439,0.0107044319],[0,213441,0.0275630969],[0,213452,0.0507165035],[0,213457,0.0478237293],[0,214403,0.0895163537],[0,214459,0.0208519436],[0,214466,0.0208418683],[0,214577,0.0157461861],[0,214583,0.0151465584],[0,214599,0.0257161097],[0,214609,0.0481714684],[0,214621,0.0101240886],[0,214629,0.0157461861],[0,214697,0.010089342],[0,214721,0.0157461861],[0,214725,0.0157461861],[0,214752,0.0849622472],[0,214814,0.0413884808],[0,214815,0.0416517199],[0,214818,0.2153713991],[0,214819,0.0117670444],[0,214840,0.0997703958],[0,214850,0.0130914703],[0,214883,0.0157461861],[0,214884,0.0208519436],[0,214919,0.0156144426],[0,214967,0.5696396931],[0,214972,0.0148236313],[0,214986,0.0669098297],[0,215034,0.1112420238],[0,215038,0.0433534708],[0,215049,0.010259218],[0,215058,0.0157461861],[0,215081,0.0115780648],[0,215087,0.0116128876],[0,215255,0.2100779623],[0,215353,0.0182718677],[0,215354,0.0234257124],[0,215388,0.0219539963],[0,215392,0.0114853234],[0,215484,0.0156508436],[0,215485,0.0121030942],[0,215542,0.0182718677],[0,215678,0.0445390987],[0,215680,0.0220985996],[0,215695,0.0180772282],[0,215737,0.0833847841],[0,215798,0.1434451326],[0,215811,0.023689071],[0,215823,0.0228123129],[0,215831,0.0548307883],[0,215846,0.1649782414],[0,215881,0.0182718677],[0,215971,0.0149439359],[0,216100,0.0113165562],[0,216116,0.0257594289],[0,216117,0.0286565939],[0,216127,0.0235493307],[0,216132,0.0113165562],[0,216148,0.4040982962],[0,216153,0.0114853234],[0,216226,0.2858977904],[0,216294,0.0204528771],[0,216326,0.0134604368],[0,216375,0.1686691255],[0,216378,0.01244953],[0,216392,0.1311557871],[0,216415,0.0523485937],[0,216458,0.0965005176],[0,216562,0.0182718677],[0,216637,0.0253074745],[0,216698,0.0299143982],[0,216701,0.0220985996],[0,216759,0.0166260208],[0,216762,0.0169836189],[0,216896,0.0128428539],[0,216977,0.4268070767],[0,216978,0.0229013931],[0,216992,0.0211201344],[0,217017,0.0114853234],[0,217125,0.0166260208],[0,217185,0.0963765203],[0,217234,0.0168516375],[0,217243,0.1149600334],[0,217359,0.013332551],[0,217443,0.0114853234],[0,217455,0.0182718677],[0,217573,0.0236318534],[0,217603,0.0260274376],[0,217612,0.0113165562],[0,217639,0.03184745],[0,217653,0.0220985996],[0,217669,0.0114505071],[0,217684,0.0182718677],[0,217794,0.1156399146],[0,217805,0.0785825066],[0,217864,0.0173459114],[0,217874,0.0160593976],[0,217960,0.0182718677],[0,218080,0.0173459114],[0,218174,0.0168516375],[0,218260,0.0249341172],[0,218272,0.0166260208],[0,218317,0.0247232569],[0,218348,0.019686742],[0,218478,0.019686742],[0,218497,0.0220985996],[0,218543,0.0218304711],[0,218546,0.0220985996],[0,218657,0.2047644381],[0,218672,0.0166497998],[0,218698,0.0119285425],[0,218715,0.0187195053],[0,218732,0.0132869569],[0,218738,0.0439296068],[0,218783,0.017979333],[0,218790,0.0311235024],[0,218840,0.0342743476],[0,219239,0.0142538953],[0,219265,0.044816308],[0,219305,0.2044074894],[0,219330,0.0119299591],[0,219334,0.0187195053],[0,219353,0.073252114],[0,219381,0.012847193],[0,219391,0.0142304864],[0,219405,0.1525676403],[0,219555,0.0119285425],[0,219566,0.0161978707],[0,219576,0.0214492174],[0,219654,0.0285642109],[0,219683,0.0225487287],[0,219688,0.0230415843],[0,219710,0.012847193],[0,219721,0.0397088658],[0,219852,0.0578524061],[0,219920,0.0253215866],[0,219928,0.0187195053],[0,219932,0.072397164],[0,220156,0.0165055947],[0,220159,0.0382090144],[0,220249,0.0187195053],[0,220262,0.0474507109],[0,220267,0.0127228423],[0,220293,0.0187195053],[0,220360,0.0308653959],[0,220368,0.0132869569],[0,220495,0.014020921],[0,220505,0.0200902972],[0,220714,0.0195396959],[0,220781,0.0186120322],[0,221149,0.4348695474],[0,221159,0.0297695978],[0,221166,0.0252098483],[0,221183,0.1257931992],[0,221247,0.0138515252],[0,221285,0.0153835079],[0,221292,0.0246737283],[0,221303,0.0442183968],[0,221308,0.0195396959],[0,221312,0.0195396959],[0,221317,0.0195396959],[0,221331,0.0195396959],[0,221392,0.0153835079],[0,221407,0.0258496469],[0,221427,0.0195396959],[0,221444,0.020832743],[0,221500,0.0153484923],[0,221607,0.0462579115],[0,221633,0.0155385184],[0,221639,0.0181142084],[0,221658,0.1309292317],[0,221662,0.0195396959],[0,221663,0.014020921],[0,221781,0.0217926973],[0,221788,0.0275802902],[0,221797,0.1405490915],[0,221811,0.020832743],[0,221982,0.0417012772],[0,222010,0.0144700214],[0,222065,0.0195396959],[0,222067,0.0312151008],[0,222116,0.0305701115],[0,222257,0.0252098483],[0,222287,0.0245285864],[0,222291,0.0159683438],[0,222365,0.020832743],[0,222461,0.0263286529],[0,222473,0.020832743],[0,222499,0.0179761588],[0,222521,0.0173207315],[0,222532,0.0258496469],[0,222622,0.0176813992],[0,222733,0.1297383358],[0,222763,0.0206373786],[0,222787,0.3647578943],[0,222984,0.0138515252],[0,222999,0.020832743],[0,223049,0.0675820025],[0,223052,0.0225855759],[0,223055,0.078838203],[0,223072,0.1925775099],[0,223126,0.020832743],[0,223134,0.0245285864],[0,223136,0.014604593],[0,223139,0.0623160272],[0,223145,0.3152543327],[0,223147,0.0453530377],[0,223257,0.0195396959],[0,223314,0.0312662966],[0,223457,0.0253940229],[0,223537,0.0253940229],[0,223558,0.037678335],[0,223600,0.0795512468],[0,223603,0.0466929123],[0,223618,0.0345138591],[0,223631,0.0373337887],[0,223632,0.0416118692],[0,223635,0.0388288158],[0,223636,0.0573112357],[0,223638,0.07017447],[0,223647,0.0854518011],[0,223681,0.0894511491],[0,223697,0.0372970221],[0,223778,0.0207109346],[0,223882,0.0213534836],[0,223958,0.0270084687],[0,223960,0.0445148133],[0,223963,0.0208834775],[0,223966,0.2222482552],[0,223967,0.1501219699],[0,223968,0.1802563807],[0,223986,0.031820267],[0,223995,0.0253940229],[0,224019,0.0462897012],[0,224042,0.0814418833],[0,224050,0.0145605866],[0,224111,0.0253940229],[0,224167,0.0148610252],[0,224185,0.0145605866],[0,224203,0.5723581618],[0,224228,0.0532310109],[0,224234,0.0266480735],[0,224235,0.0292136518],[0,224243,0.1370754234],[0,224272,0.4596896797],[0,224273,0.212732409],[0,224274,0.1808341878],[0,224277,0.0202546615],[0,224310,0.0659304335],[0,224361,0.0826222994],[0,224390,0.0215490508],[0,224393,0.0472486056],[0,224404,0.0215490508],[0,224424,0.2679019746],[0,224429,0.0586477921],[0,224430,0.1816623926],[0,224432,0.1824130915],[0,224433,0.0374480699],[0,224436,0.1969539811],[0,224438,0.0488307932],[0,224439,0.0305627085],[0,224442,0.0253940229],[0,224447,0.1672186557],[0,224451,0.0389943819],[0,224456,0.0430647441],[0,224457,0.0160941595],[0,224485,0.0252098827],[0,224512,0.0577328701],[0,224522,0.0585590287],[0,224527,0.0640180708],[0,224538,0.064469385],[0,224549,0.0343634461],[0,224553,0.2551162066],[0,224559,0.014595562],[0,224561,0.0152147318],[0,224619,0.0252487303],[0,224648,0.354325422],[0,224674,0.638825298],[0,224687,0.0252487303],[0,224695,0.0193260349],[0,224704,0.0555949045],[0,224708,0.0538619517],[0,224709,0.0253940229],[0,224729,0.0215490508],[0,224730,0.0252479797],[0,224731,0.0175459172],[0,224736,0.0178500268],[0,224737,0.1998107719],[0,224738,0.0451852168],[0,224739,0.0207886321],[0,224740,0.0178500268],[0,224742,0.0469693485],[0,224747,0.0414506583],[0,224750,0.0375448201],[0,224752,0.033583753],[0,224753,0.0380698609],[0,224760,0.0253940229],[0,224761,0.0246293718],[0,224763,0.0520561273],[0,224770,0.03533401],[0,224785,0.0253940229],[0,224788,0.1778947215],[0,224807,0.0253245362],[0,224830,0.0193260349],[0,224838,0.0385680851],[0,224901,0.0312662966],[0,224929,0.0351563712],[0,224930,0.0293033026],[0,224931,0.0551925058],[0,224985,0.0465906879],[0,224998,0.0252487303],[0,225065,0.0291354113],[0,225099,0.0585304556],[0,225122,0.0247541563],[0,225132,0.0145605866],[0,225133,0.0586270581],[0,225137,0.1103248406],[0,225237,0.0207462245],[0,225248,0.0841323233],[0,225258,0.0349514786],[0,225280,0.0202546615],[0,225308,0.0215490508],[0,225430,0.0523928892],[0,225441,0.0215490508],[0,225676,0.0215490508],[0,225692,0.2655624745],[0,225694,0.0379669028],[0,225813,0.0129998696],[0,225924,0.0314653457],[0,225926,0.0220583063],[0,225953,0.0185103977],[0,226079,0.0307014089],[0,226140,0.0198015126],[0,226142,0.0236367583],[0,226218,0.0620760426],[0,226280,0.014686182],[0,226300,0.0627146373],[0,226367,0.0185103977],[0,226369,0.0128307269],[0,226406,0.0361442006],[0,226449,0.0389898836],[0,226458,0.0246269469],[0,226488,0.0246269469],[0,226640,0.0139199649],[0,226767,0.2457279782],[0,226937,0.4034660657],[0,226976,0.3906277525],[0,226994,0.0207862205],[0,226999,0.0246269469],[0,227013,0.1008887061],[0,227053,0.028536835],[0,227081,0.0246269469],[0,227085,0.0180438012],[0,227106,0.0195712086],[0,227110,0.0172999731],[0,227181,0.0183781227],[0,227213,0.2320050816],[0,227237,0.2095897931],[0,227241,0.0207862205],[0,227251,0.015458437],[0,227296,0.0266340908],[0,227300,0.0139199649],[0,227306,0.0579024068],[0,227327,0.0185103977],[0,227363,0.0442680052],[0,227500,0.0260496231],[0,227621,0.1976632036],[0,227637,0.1997142012],[0,227675,0.0693144487],[0,227684,0.0129998696],[0,227727,0.0139199649],[0,227810,0.0207862205],[0,227878,0.0130012885],[0,227926,0.0191904399],[0,227973,0.0215269492],[0,227975,0.021922268],[0,227979,0.0508475121],[0,228014,0.0260529635],[0,228154,0.0207862205],[0,228167,0.0185103977],[0,228179,0.1708183609],[0,228230,0.1374121096],[0,228366,0.0129998696],[0,228378,0.0210807095],[0,228390,0.0139199649],[0,228537,0.0129998696],[0,228578,0.0229985103],[0,228610,0.0128656138],[0,228645,0.0383785428],[0,228669,0.0183781227],[0,228717,0.0307603111],[0,228733,0.2072839054],[0,228749,0.0193154415],[0,228805,0.104478493],[0,228901,0.0304210149],[0,228924,0.0207862205],[0,228955,0.0207862205],[0,228976,0.022340916],[0,228982,0.0246269469],[0,228986,0.0236153564],[0,228990,0.0236904277],[0,229052,0.0207862205],[0,229125,0.0340760377],[0,229246,0.0234723503],[0,229402,0.1700449076],[0,229406,0.0266912363],[0,229435,0.1002141576],[0,229467,0.0270300336],[0,229489,0.0129649759],[0,229493,0.0251982285],[0,229501,0.0367089119],[0,229524,0.0129998696],[0,229646,0.051578652],[0,229685,0.0128656138],[0,229698,0.0237631139],[0,229706,0.0128307269],[0,229710,0.0292540937],[0,229720,0.0185103977],[0,229743,0.0215391454],[0,229759,0.0183781227],[0,229782,0.0211148663],[0,229849,0.0207862205],[0,229857,0.0134832173],[0,229896,0.0665965834],[0,230021,0.014095521],[0,230036,0.019225652],[0,230128,0.0246269469],[0,230150,0.1298323791],[0,230158,0.0129998696],[0,230211,0.0207862205],[0,230234,0.0211160183],[0,230262,0.0134482989],[0,230293,0.028918869],[0,230306,0.0225029702],[0,230312,0.207019553],[0,230321,0.0522149585],[0,230490,0.0183429539],[0,230499,0.0134482989],[0,230658,0.0129998696],[0,230664,0.0134482989],[0,230674,0.037895743],[0,230894,0.0720061028],[0,231176,0.0246269469],[0,231190,0.026273136],[0,231196,0.0176016152],[0,231206,0.0207862205],[0,231211,0.0226867596],[0,231214,0.42581203],[0,231246,0.0560276668],[0,231335,0.0661959276],[0,231398,0.019059836],[0,231537,0.0547837087],[0,231746,0.0191904399],[0,231763,0.0207988204],[0,231860,0.019098322],[0,231883,0.0128307269],[0,231968,0.0246269469],[0,232100,0.2598812178],[0,232159,0.0185103977],[0,232203,0.0149011077],[0,232214,0.0261035436],[0,232270,0.0473742583],[0,232321,0.0222546443],[0,232327,0.0199739786],[0,232404,0.0274690545],[0,232474,0.0479814295],[0,232610,0.0402234697],[0,232621,0.0313376829],[0,232789,0.0376109162],[0,233167,0.0316007577],[0,233397,0.2700662219],[0,233436,0.0251928307],[0,233456,0.0211749132],[0,233457,0.0334083276],[0,233497,0.0726141228],[0,233536,0.2248496584],[0,233571,0.0222546443],[0,233579,0.0261035436],[0,233594,0.0199739786],[0,233641,0.0158151702],[0,233642,0.0199739786],[0,233724,0.0142822216],[0,233743,0.26405399],[0,233759,0.0222546443],[0,233791,0.0158151702],[0,233952,0.0159464252],[0,233995,0.0199739786],[0,234017,0.0259581026],[0,234055,0.0251694675],[0,234126,0.0199739786],[0,234157,0.112015595],[0,234243,0.0199739786],[0,234275,0.0150721869],[0,234277,0.0144881159],[0,234287,0.0153737774],[0,234312,0.01904573],[0,234316,0.0645703664],[0,234320,0.0250520098],[0,234401,0.0199739786],[0,234417,0.1062091972],[0,234428,0.0222546443],[0,234514,0.0434255676],[0,234545,0.0199739786],[0,234583,0.0290103305],[0,234594,0.2786252137],[0,234609,0.0810466039],[0,234611,0.0538033519],[0,234625,0.0421302898],[0,234780,0.0144517242],[0,235030,0.2893545782],[0,235123,0.0818763518],[0,235165,0.0225497599],[0,235184,0.0149011077],[0,235270,0.0142822216],[0,235443,0.0198414221],[0,235589,0.025112447],[0,235671,0.0280365787],[0,235703,0.828691463],[0,235744,0.0558908847],[0,235745,0.0407433994],[0,235761,0.0310528044],[0,235764,0.0144517242],[0,235777,0.0595462899],[0,235781,0.0142822216],[0,235841,0.0142822216],[0,235895,0.3742589712],[0,236028,0.01904573],[0,236031,0.0283444268],[0,236138,0.0295269322],[0,236222,0.0203361395],[0,236308,0.0570449556],[0,236334,0.0199739786],[0,236464,0.0143171827],[0,236468,0.031679407],[0,236483,0.0210723534],[0,236505,0.0638888593],[0,236539,0.0222546443],[0,236561,0.0363190786],[0,236744,0.0222546443],[0,236787,0.0222546443],[0,236859,0.0261035436],[0,236866,0.105185647],[0,236878,0.0510690159],[0,236921,0.0392107392],[0,236923,0.0143171827],[0,236935,0.1513012776],[0,237048,0.0222546443],[0,237051,0.0198414221],[0,237076,0.0144517242],[0,237239,0.0295008234],[0,237290,0.0213403307],[0,237395,0.3712493943],[0,237418,0.0298194083],[0,237487,0.0261035436],[0,237592,0.0314181728],[0,237664,0.0289899284],[0,237666,0.0482281393],[0,237810,0.2180919747],[0,237815,0.0245427444],[0,237850,0.0199739786],[0,237853,0.0479452404],[0,237874,0.01904573],[0,237923,0.1054653326],[0,237928,0.0260765672],[0,238056,0.0394623592],[0,238113,0.0227074403],[0,238128,0.0303533527],[0,238268,0.0516845217],[0,238323,0.0281265477],[0,238337,0.0298028],[0,238413,0.0296565854],[0,238434,0.0259334254],[0,238572,0.023640627],[0,238602,0.0397650282],[0,238625,0.0545437724],[0,238653,0.0230033922],[0,239094,0.2696001509],[0,239097,0.6056762846],[0,239101,0.6056762846],[0,239102,0.2716901455],[0,239104,0.0527545969],[0,239105,0.0454334434],[0,239107,0.1987488615],[0,239110,0.1026021577],[0,239113,0.0799689165],[0,239125,0.023136557],[0,239126,0.2653088265],[0,239132,0.0502296993],[0,239133,0.0416741855],[0,239137,0.044765009],[0,239140,0.0246984914],[0,239144,0.0575415152],[0,239147,0.0298028],[0,239152,0.0830986188],[0,239153,0.0329360491],[0,239154,0.0236051891],[0,239155,0.0582026561],[0,239156,0.0759563409],[0,239160,0.0631917694],[0,239165,0.0298028],[0,239170,0.127893695],[0,239177,0.0612455269],[0,239180,0.0352013786],[0,239186,0.1732357655],[0,239188,0.0290613444],[0,239189,0.2480883231],[0,239191,0.0279194906],[0,239193,0.3234560749],[0,239202,0.055445084],[0,239203,0.2833057438],[0,239210,0.0377066531],[0,239212,0.122472379],[0,239216,0.4837183558],[0,239218,0.0803919374],[0,239225,0.0517682965],[0,239228,0.0507462106],[0,239229,0.0660697669],[0,239231,0.0449680473],[0,239234,0.0728484022],[0,239245,0.0458330267],[0,239249,0.0298028],[0,239252,0.0388698111],[0,239253,0.0251681806],[0,239255,0.0298468949],[0,239260,0.0497025012],[0,239262,0.0387082356],[0,239264,0.1160947754],[0,239268,0.0246984914],[0,239274,0.0516309602],[0,239279,0.0251681806],[0,239283,0.0298468949],[0,239285,0.0298028],[0,239288,0.3961044485],[0,239292,0.4765952348],[0,239295,0.0273771259],[0,239300,0.4666448028],[0,239310,0.0424896236],[0,239316,0.0650787103],[0,239321,0.0451037241],[0,239325,0.0417826423],[0,239332,0.105082947],[0,239333,0.0298028],[0,239335,0.0297488023],[0,239346,0.04629849],[0,239350,0.0560939447],[0,239354,0.0290332961],[0,239357,0.0905901376],[0,239358,0.0694059285],[0,239359,0.0308228532],[0,239363,0.0285609927],[0,239364,0.0251681806],[0,239365,0.0591634102],[0,239380,0.0891858414],[0,239386,0.0243151335],[0,239389,0.058268506],[0,239390,0.0298028],[0,239391,0.195772118],[0,239393,0.0577865873],[0,239394,0.0264478308],[0,239395,0.1227757424],[0,239400,0.0298028],[0,239406,0.0428298935],[0,239408,0.0444567515],[0,239410,0.4587867692],[0,239413,0.046490196],[0,239414,0.1305167789],[0,239417,0.2601428105],[0,239421,0.0358796273],[0,239423,0.195772118],[0,239442,0.0381308157],[0,239447,0.6133347388],[0,239449,0.3527311403],[0,239450,0.0298028],[0,239452,0.0298028],[0,239459,0.1198896316],[0,239461,0.2367739732],[0,239469,0.0298028],[0,239472,0.0512788555],[0,239473,0.0608883353],[0,239476,0.0527545969],[0,239479,0.0527545969],[0,239480,0.0577865873],[0,239488,0.0400531556],[0,239493,0.076712786],[0,239496,0.2722849124],[0,239503,0.0358796273],[0,239505,0.0310583764],[0,239510,0.0298028],[0,239513,0.1090658804],[0,239516,0.0540116681],[0,239520,0.2979822113],[0,239522,0.4935140519],[0,239526,0.0650787103],[0,239534,0.0642254371],[0,239535,0.0327325995],[0,239536,0.0298028],[0,239540,0.0304447688],[0,239542,0.0358796273],[0,239545,0.0290332961],[0,239546,0.0475268897],[0,239549,0.0298028],[0,239555,0.023136557],[0,239556,0.7608732959],[0,239563,0.0298028],[0,239565,0.04629849],[0,239575,0.1014682647],[0,239582,0.0873538195],[0,239583,0.0298028],[0,239587,0.0619962025],[0,239604,0.6422952208],[0,239607,0.0317011776],[0,239609,0.0332784973],[0,239614,0.0527545969],[0,239619,0.0236051891],[0,239629,0.0298028],[0,239631,0.0259772761],[0,239640,0.0251681806],[0,239641,0.2160040446],[0,239642,0.0247484587],[0,239647,0.0290332961],[0,239648,0.1075703986],[0,239649,0.026354755],[0,239651,0.1751766951],[0,239655,0.0330495917],[0,239658,0.0298028],[0,239659,0.1961472206],[0,239663,0.1233480966],[0,239667,0.0251681806],[0,239672,0.0259772761],[0,239673,0.0791754328],[0,239681,0.3236739076],[0,239682,0.3608452701],[0,239688,0.0392531574],[0,239694,0.0297488023],[0,239702,0.1305888862],[0,239703,0.0346630309],[0,239705,0.1199710156],[0,239710,0.0298028],[0,239711,0.0969142603],[0,239714,0.0297488023],[0,239715,0.3000704107],[0,239716,0.0313754515],[0,239717,0.0424896236],[0,239725,0.0193810362],[0,239729,0.6129710688],[0,239733,0.0281265477],[0,239737,0.0259334254],[0,239738,0.0467421868],[0,239740,0.0342416836],[0,239745,0.0498626678],[0,239749,0.0650787103],[0,239752,0.0440410962],[0,239754,0.0544613478],[0,239759,0.0503344063],[0,239760,0.0329087223],[0,239761,0.0313754515],[0,239766,0.0227275142],[0,239769,0.0612455269],[0,239771,0.0298468949],[0,239772,0.6055430772],[0,239778,0.0298028],[0,239781,0.04629849],[0,239784,0.0358796273],[0,239787,0.0450266983],[0,239789,0.180312768],[0,239790,0.4577511296],[0,239792,0.1136922802],[0,239793,0.0412523139],[0,239800,0.0298028],[0,239803,0.0302945899],[0,239809,0.0259334254],[0,239810,0.0527545969],[0,239814,0.0686426278],[0,239821,0.0337598561],[0,239828,0.0469222072],[0,239832,0.0251681806],[0,239833,0.1782589552],[0,239836,0.0464780428],[0,239846,0.0493398977],[0,239850,0.1176128167],[0,239852,0.0358202039],[0,239853,0.3507636962],[0,239857,0.0259334254],[0,239866,0.0447727274],[0,239872,0.038157916],[0,239876,0.0298028],[0,239877,0.1259248047],[0,239882,0.0437274979],[0,239885,0.1957974543],[0,239886,0.0273747296],[0,239899,0.0259334254],[0,239903,0.0402405926],[0,239904,0.0374703075],[0,239909,0.0298028],[0,239916,0.0595080889],[0,239917,0.0272285978],[0,239918,0.0650787103],[0,239934,0.0251681806],[0,239936,0.0259772761],[0,239937,0.023640627],[0,239942,0.0233540467],[0,239944,0.0399068144],[0,239946,0.0298028],[0,239959,0.0527545969],[0,239963,0.0559801477],[0,239972,0.0259772761],[0,239974,0.2724288304],[0,239975,0.1052820442],[0,239977,0.0502296993],[0,239982,0.0523851328],[0,239987,0.0489421842],[0,239988,0.0298028],[0,239997,0.0298028],[0,239998,0.0627063824],[0,240000,0.0298028],[0,240002,0.1095410325],[0,240011,0.0259334254],[0,240028,0.0298028],[0,240029,0.0374703075],[0,240031,0.0650787103],[0,240037,0.023136557],[0,240039,0.0527545969],[0,240042,0.4746731234],[0,240043,0.0298028],[0,240050,0.0281913204],[0,240057,0.0298028],[0,240058,0.0279134912],[0,240059,0.1856307671],[0,240061,0.465108752],[0,240062,0.0298028],[0,240063,0.0419097719],[0,240070,0.0527545969],[0,240071,0.04629849],[0,240072,0.0298028],[0,240075,0.0227275142],[0,240077,0.5397049701],[0,240078,0.062796392],[0,240083,0.1615167919],[0,240086,0.0341621753],[0,240092,0.0251681806],[0,240096,0.0298028],[0,240099,0.0298028],[0,240101,0.0337758931],[0,240105,0.0374703075],[0,240109,0.0298028],[0,240114,0.0226720501],[0,240115,0.0298028],[0,240122,0.0298028],[0,240126,0.0298028],[0,240133,0.3557969783],[0,240136,0.038157916],[0,240146,0.0273771259],[0,240150,0.0489949222],[0,240164,0.0527545969],[0,240167,0.6180859299],[0,240168,0.0298028],[0,240173,0.0325553599],[0,240177,0.029000501],[0,240178,0.0259772761],[0,240179,0.0259334254],[0,240180,0.0435479614],[0,240183,0.5513509136],[0,240185,0.251767172],[0,240193,0.0948696135],[0,240194,0.0284839871],[0,240195,0.029000501],[0,240196,0.0454772595],[0,240205,0.3522815383],[0,240218,0.029000501],[0,240224,0.0282318802],[0,240226,0.1526482452],[0,240233,0.2874940307],[0,240237,0.0295171759],[0,240238,0.4171635488],[0,240239,0.0300906898],[0,240245,0.029000501],[0,240252,0.0619002087],[0,240253,0.0517556425],[0,240262,0.4445616994],[0,240264,0.0374304203],[0,240265,0.0899777882],[0,240269,0.0379469055],[0,240273,0.0611520702],[0,240275,0.0289465653],[0,240282,0.1060247138],[0,240284,0.0650360273],[0,240290,0.0259334254],[0,240293,0.0611520702],[0,240300,0.029000501],[0,240302,0.1595668206],[0,240306,0.0612369086],[0,240308,0.1318823672],[0,240309,0.0870885141],[0,240312,0.0612455269],[0,240316,0.2752125284],[0,240317,0.0362854757],[0,240320,0.0325548338],[0,240323,0.0251681806],[0,240327,0.0325663559],[0,240329,0.029000501],[0,240331,0.1641525358],[0,240333,0.034436202],[0,240339,0.0358202039],[0,240342,0.0308298685],[0,240344,0.029000501],[0,240350,0.0305918899],[0,240352,0.029000501],[0,240354,0.0366606199],[0,240357,0.1060247138],[0,240368,0.0472698067],[0,240370,0.029000501],[0,240372,0.1620235658],[0,240374,0.029000501],[0,240380,0.2073089298],[0,240381,0.0233540467],[0,240382,0.378129231],[0,240385,0.2102451744],[0,240392,0.4179841436],[0,240393,0.0788183682],[0,240397,0.0305724695],[0,240416,0.029000501],[0,240425,0.6076275841],[0,240434,0.0362509667],[0,240435,0.0399195056],[0,240436,0.0242329366],[0,240442,0.029000501],[0,240449,0.0290445452],[0,240450,0.0259334254],[0,240453,0.0282221913],[0,240457,0.0279194906],[0,240458,0.0244124635],[0,240459,0.068303544],[0,240461,0.029000501],[0,240462,0.8142970728],[0,240464,0.3387697786],[0,240466,0.1011998306],[0,240470,0.0514445661],[0,240473,0.0494671171],[0,240479,0.0846280304],[0,240480,0.0429336214],[0,240485,0.0383495903],[0,240493,0.0196859583],[0,240496,0.1359156092],[0,240501,0.0290445452],[0,240505,0.6143218803],[0,240506,0.1938018919],[0,240508,0.5289385857],[0,240510,0.0311328416],[0,240527,0.2815370806],[0,240529,0.029000501],[0,240535,0.6098682574],[0,240536,0.0619420633],[0,240543,0.0255457333],[0,240550,0.0757185852],[0,240551,0.0419436021],[0,240552,0.0425779921],[0,240561,0.0358202039],[0,240563,0.0282318802],[0,240565,0.1049078303],[0,240568,0.0586428952],[0,240576,0.0570544831],[0,240579,0.0234662818],[0,240585,0.029000501],[0,240597,0.2665662654],[0,240601,0.030636644],[0,240609,0.0604657721],[0,240614,0.0653404272],[0,240615,0.1054133018],[0,240626,0.0462525793],[0,240630,0.0582589331],[0,240631,0.8291210268],[0,240638,0.0308298685],[0,240639,0.1342747781],[0,240640,0.0434755022],[0,240643,0.055445084],[0,240645,0.0489110989],[0,240647,0.029000501],[0,240648,0.029000501],[0,240653,0.03426312],[0,240655,0.0277601188],[0,240669,0.1656714059],[0,240671,0.6208543581],[0,240672,0.1042198819],[0,240678,0.029000501],[0,240679,0.029000501],[0,240681,0.043162361],[0,240684,0.0599000966],[0,240691,0.104210677],[0,240692,0.0638893974],[0,240694,0.0246984914],[0,240696,0.0251681806],[0,240697,0.0384647994],[0,240698,0.029000501],[0,240700,0.03426312],[0,240709,0.028085994],[0,240713,0.0439395685],[0,240715,0.0905901376],[0,240720,0.0463834687],[0,240722,0.029000501],[0,240723,0.029000501],[0,240724,0.0530996444],[0,240725,0.1995980788],[0,240727,0.029000501],[0,240729,0.0612455269],[0,240734,0.0875775732],[0,240739,0.029000501],[0,240743,0.029000501],[0,240744,0.029000501],[0,240749,0.148868986],[0,240752,0.0391166867],[0,240762,0.1176128167],[0,240768,0.0310119112],[0,240773,0.1188666963],[0,240775,0.3847984542],[0,240777,0.2169932169],[0,240779,0.029000501],[0,240780,0.029000501],[0,240783,0.0377370774],[0,240784,0.029000501],[0,240791,0.029000501],[0,240793,0.026354755],[0,240794,0.0404461731],[0,240799,0.040950984],[0,240802,0.029000501],[0,240803,0.0259334254],[0,240807,0.0407151668],[0,240810,0.1077014468],[0,240819,0.0736389534],[0,240820,0.039019116],[0,240823,0.026354755],[0,240832,0.0612455269],[0,240835,0.0259334254],[0,240840,0.1286418508],[0,240845,0.0442378171],[0,240850,0.0508955482],[0,240851,0.0235940542],[0,240852,0.5388872701],[0,240854,0.029000501],[0,240858,0.029000501],[0,240862,0.0791267982],[0,240864,0.6090031019],[0,240871,0.0282318802],[0,240872,0.0630641683],[0,240873,0.029000501],[0,240875,0.060528016],[0,240882,0.6098682574],[0,240883,0.0372126423],[0,240884,0.0251681806],[0,240885,0.029000501],[0,240887,0.1769631478],[0,240889,0.029000501],[0,240890,0.0636026562],[0,240891,0.0755700863],[0,240893,0.0500316853],[0,240894,0.8876256151],[0,240901,0.085098313],[0,240904,0.0299647912],[0,240906,0.0612455269],[0,240911,0.0864335828],[0,240912,0.0320555664],[0,240928,0.0360644374],[0,240939,0.0298869997],[0,240941,0.1282861764],[0,240946,0.0494349558],[0,240948,0.0247250612],[0,240949,0.027662319],[0,240950,0.2464907266],[0,240967,0.029000501],[0,240968,0.0299647912],[0,240971,0.089963061],[0,240975,0.029000501],[0,240976,0.023640627],[0,240979,0.029000501],[0,240983,0.047007077],[0,240986,0.0244004918],[0,240989,0.0612455269],[0,240991,0.0657147576],[0,240992,0.0416062446],[0,240993,0.1556998093],[0,241001,0.029000501],[0,241003,0.0630040362],[0,241010,0.1115244092],[0,241012,0.0696275694],[0,241015,0.0612455269],[0,241016,0.0381308157],[0,241018,0.0355237999],[0,241021,0.051925957],[0,241027,0.0493802082],[0,241029,0.023640627],[0,241037,0.0612455269],[0,241038,0.0520952186],[0,241043,0.0288544541],[0,241046,0.1333334057],[0,241047,0.5130623534],[0,241048,0.051925957],[0,241051,0.0258797265],[0,241052,0.0650787103],[0,241057,0.0226720501],[0,241064,0.0595080889],[0,241067,0.1564541161],[0,241070,0.0612455269],[0,241071,0.0259334254],[0,241081,0.0298072536],[0,241082,0.0562823895],[0,241086,0.0267366344],[0,241087,0.0267366344],[0,241089,0.0298072536],[0,241094,0.0399351271],[0,241098,0.1040100788],[0,241104,0.0313810383],[0,241105,0.0459560268],[0,241106,0.5899339703],[0,241119,0.0191257303],[0,241120,0.0659271461],[0,241121,0.0931593937],[0,241122,0.0439854757],[0,241123,0.1460637239],[0,241124,0.0659271461],[0,241127,0.0267366344],[0,241130,0.0845948693],[0,241131,0.0298072536],[0,241136,0.0363612499],[0,241139,0.1144430423],[0,241146,0.0370660962],[0,241147,0.1248309588],[0,241152,0.2318481483],[0,241155,0.0656058476],[0,241159,0.1401290686],[0,241161,0.0298072536],[0,241165,0.0298072536],[0,241173,0.062089534],[0,241178,0.0630641683],[0,241179,0.2689629748],[0,241188,0.1822606114],[0,241190,0.0255002737],[0,241191,0.0263135844],[0,241193,0.0354858905],[0,241197,0.0396141784],[0,241198,0.1264423842],[0,241201,0.1020899992],[0,241202,0.062089534],[0,241208,0.1450540425],[0,241216,0.0267366344],[0,241219,0.4384269474],[0,241222,0.0371227725],[0,241224,0.0656058476],[0,241226,0.1776884645],[0,241232,0.038774427],[0,241237,0.0668157965],[0,241238,0.1227737059],[0,241249,0.2480748859],[0,241251,0.0880388472],[0,241259,0.6148875173],[0,241263,0.1063083385],[0,241265,0.0269805413],[0,241271,0.0385253428],[0,241276,0.0299647912],[0,241283,0.0643437474],[0,241289,0.0474217145],[0,241290,0.0298072536],[0,241292,0.0298072536],[0,241293,0.026354755],[0,241297,0.0437373731],[0,241299,0.0285654384],[0,241303,0.0267366344],[0,241304,0.066949623],[0,241305,0.1968802464],[0,241317,0.0298072536],[0,241318,0.1469713777],[0,241320,0.0475823622],[0,241322,0.0639232799],[0,241326,0.0659271461],[0,241328,0.0327770393],[0,241331,0.0981445876],[0,241334,0.0333772284],[0,241335,0.0971977362],[0,241336,0.0311328416],[0,241337,0.618057721],[0,241338,0.0659271461],[0,241343,0.0298072536],[0,241352,0.3362673935],[0,241353,0.0581149293],[0,241358,0.3745003998],[0,241362,0.042494158],[0,241364,0.0310583764],[0,241366,0.0434444563],[0,241372,0.0610752354],[0,241376,0.0333772284],[0,241378,0.0256456189],[0,241379,0.0773526134],[0,241380,0.0281820029],[0,241381,0.2534292299],[0,241385,0.0650787417],[0,241386,0.1644142134],[0,241392,0.0484991483],[0,241397,0.6166951638],[0,241398,0.0298072536],[0,241400,0.0298072536],[0,241404,0.0298072536],[0,241412,0.1117176193],[0,241415,0.0502522055],[0,241425,0.2892973382],[0,241427,0.0928886155],[0,241428,0.254065802],[0,241429,0.0298072536],[0,241431,0.0290377448],[0,241434,0.3115469947],[0,241436,0.0659271461],[0,241439,0.0290377448],[0,241440,0.0298072536],[0,241441,0.2739842577],[0,241443,0.0285654384],[0,241449,0.0267366344],[0,241450,0.0244411871],[0,241464,0.1259405356],[0,241480,0.0298072536],[0,241489,0.0298072536],[0,241512,0.0267366344],[0,241517,0.0267366344],[0,241525,0.1823488035],[0,241530,0.025028539],[0,241532,0.0298513487],[0,241533,0.13498638],[0,241544,0.4114350676],[0,241546,0.0519769861],[0,241547,0.0540162759],[0,241550,0.045130395],[0,241556,0.1087707784],[0,241563,0.039791722],[0,241568,0.0432061754],[0,241569,0.0298072536],[0,241570,0.0402405926],[0,241571,0.0991689817],[0,241579,0.3620522057],[0,241595,0.0370216532],[0,241597,0.0267366344],[0,241604,0.0438995126],[0,241606,0.0298072536],[0,241610,0.0711559589],[0,241615,0.4362418885],[0,241624,0.0290377448],[0,241639,0.0453511733],[0,241659,0.0492671992],[0,241662,0.5783197477],[0,241674,0.3379065453],[0,241675,0.0298072536],[0,241678,0.3413579957],[0,241691,0.4093195535],[0,241693,0.0259705055],[0,241697,0.0259705055],[0,241698,0.0267366344],[0,241701,0.1191695906],[0,241702,0.0267805359],[0,241703,0.0506913772],[0,241705,0.0297532556],[0,241707,0.0298072536],[0,241709,0.0252139154],[0,241710,0.0298072536],[0,241711,0.0301895647],[0,241712,0.3727695149],[0,241715,0.39216421],[0,241716,0.4124875367],[0,241717,0.0569319999],[0,241722,0.034886259],[0,241728,0.3388750444],[0,241733,0.0473709971],[0,241734,0.1086162805],[0,241735,0.0299647912],[0,241737,0.0607425493],[0,241739,0.0227275142],[0,241742,0.3672179019],[0,241743,0.0290377448],[0,241744,0.5727391929],[0,241748,0.0298072536],[0,241749,0.2940753043],[0,241759,0.0266828736],[0,241762,0.1807182471],[0,241763,0.379513638],[0,241766,0.0298072536],[0,241768,0.106471731],[0,241771,0.0298072536],[0,241772,0.0668880489],[0,241777,0.070784909],[0,241778,0.0310338334],[0,241781,0.0440743292],[0,241785,0.041291858],[0,241788,0.3573410676],[0,241793,0.3471205266],[0,241794,0.0450966706],[0,241795,0.0619959694],[0,241799,0.0298072536],[0,241802,0.0366943277],[0,241805,0.0267366344],[0,241808,0.0269261489],[0,241810,0.740931466],[0,241811,0.0298072536],[0,241814,0.1743721985],[0,241815,0.0411289178],[0,241818,0.0900802674],[0,241821,0.3458564788],[0,241822,0.060528016],[0,241824,0.3734747839],[0,241825,0.2994049454],[0,241826,0.1063083385],[0,241827,0.0415349558],[0,241829,0.0958830658],[0,241830,0.0463030487],[0,241831,0.0914680482],[0,241836,0.3922321672],[0,241838,0.1261191583],[0,241842,0.0290377448],[0,241845,0.0298072536],[0,241846,0.344345806],[0,241847,0.026354755],[0,241848,0.0298072536],[0,241850,0.1059346936],[0,241853,0.0936517535],[0,241857,0.0290377448],[0,241858,0.1044058676],[0,241859,0.0607425493],[0,241861,0.0558644325],[0,241863,0.0280333033],[0,241864,0.0842662877],[0,241865,0.4044255729],[0,241868,0.3342207583],[0,241869,0.0410857833],[0,241874,0.0210228113],[0,241876,0.0305978762],[0,241879,0.0268312852],[0,241880,0.0299647912],[0,241882,0.0259705055],[0,241884,0.0267366344],[0,241886,0.0314004812],[0,241888,0.0298072536],[0,241890,0.0478522751],[0,241892,0.0590242634],[0,241893,0.3386408323],[0,241896,0.0317618805],[0,241897,0.1578519992],[0,241899,0.0422021504],[0,241902,0.0264638631],[0,241903,0.1302049372],[0,241904,0.1020899992],[0,241906,0.0353057989],[0,241908,0.0952155173],[0,241909,0.383898754],[0,241910,0.0463170201],[0,241915,0.0464947559],[0,241918,0.0267366344],[0,241919,0.0729663916],[0,241923,0.1586308121],[0,241929,0.0333772284],[0,241930,0.0285654384],[0,241931,0.4089160453],[0,241933,0.0298072536],[0,241937,0.0267366344],[0,241938,0.039824794],[0,241939,0.0280333033],[0,241943,0.3652919742],[0,241944,0.3816452795],[0,241945,0.3846115554],[0,241946,0.042494158],[0,241947,0.303965412],[0,241948,0.0259705055],[0,241949,0.0445053695],[0,241950,0.0234662818],[0,241951,0.0267366344],[0,241953,0.1898656309],[0,241954,0.3500152607],[0,241955,0.0587000832],[0,241956,0.0530288407],[0,241957,0.0350828142],[0,241961,0.0290377448],[0,241962,0.5093098208],[0,241964,0.0805178705],[0,241965,0.139576358],[0,241967,0.1074493274],[0,241968,0.0233540467],[0,241969,0.0549529172],[0,241971,0.0443715216],[0,241972,0.4004061339],[0,241973,0.2396455503],[0,241974,0.0193810362],[0,241978,0.1261377038],[0,241983,0.084582293],[0,241985,0.0717693181],[0,241988,0.1254117521],[0,241992,0.0305200143],[0,241993,0.0447664632],[0,241995,0.0298072536],[0,241997,0.0792491471],[0,242002,0.0350828142],[0,242003,0.120257864],[0,242004,0.0911589448],[0,242006,0.0503085121],[0,242008,0.062089534],[0,242009,0.0267366344],[0,242011,0.031006588],[0,242012,0.0196859583],[0,242014,0.0426413617],[0,242015,0.0244004918],[0,242016,0.1019621017],[0,242017,0.5417126246],[0,242019,0.041892648],[0,242020,0.046167799],[0,242023,0.0515780873],[0,242025,0.0484912069],[0,242026,0.3928775757],[0,242027,0.0880388472],[0,242029,0.0298072536],[0,242030,0.342109081],[0,242031,0.0259705055],[0,242036,0.1113137804],[0,242037,0.0485469647],[0,242038,0.0477496658],[0,242044,0.0527591967],[0,242045,0.0298072536],[0,242047,0.0442378171],[0,242048,0.0298072536],[0,242056,0.1703705822],[0,242057,0.0938872733],[0,242058,0.0584151201],[0,242061,0.0298072536],[0,242062,0.0558644325],[0,242063,0.1296642056],[0,242065,0.0633248518],[0,242068,0.0350759528],[0,242070,0.3920100092],[0,242074,0.0788204865],[0,242077,0.0298072536],[0,242082,0.0929021751],[0,242083,0.0529109962],[0,242085,0.0290377448],[0,242087,0.0303245255],[0,242092,0.0400809979],[0,242093,0.0298072536],[0,242099,0.0298072536],[0,242103,0.1114100143],[0,242105,0.0522794217],[0,242106,0.0298072536],[0,242107,0.0748503109],[0,242108,0.0751885296],[0,242115,0.0298072536],[0,242116,0.0612366337],[0,242117,0.0367120961],[0,242119,0.4079821897],[0,242124,0.0897455549],[0,242125,0.0403109565],[0,242128,0.4891500715],[0,242129,0.0450966706],[0,242130,0.1020899992],[0,242131,0.0267805359],[0,242134,0.1049382593],[0,242135,0.0427825496],[0,242136,0.0632765689],[0,242139,0.0537354015],[0,242142,0.0303222204],[0,242143,0.06258453],[0,242145,0.0664247395],[0,242154,0.0255002737],[0,242156,0.0567734552],[0,242158,0.0415809629],[0,242160,0.0259705055],[0,242161,0.0298072536],[0,242165,0.0298072536],[0,242169,0.0495096199],[0,242171,0.0290377448],[0,242172,0.0267366344],[0,242174,0.2940318306],[0,242176,0.0298072536],[0,242177,0.0484991483],[0,242179,0.0721508109],[0,242180,0.1684441529],[0,242181,0.0298072536],[0,242182,0.0341621753],[0,242183,0.0370660962],[0,242184,0.0197826388],[0,242187,0.0448778043],[0,242190,0.1152900653],[0,242191,0.0455968329],[0,242192,0.0259705055],[0,242193,0.4810255592],[0,242198,0.087494154],[0,242199,0.0298072536],[0,242203,0.0366943277],[0,242205,0.0341621753],[0,242207,0.0307254996],[0,242208,0.0310782884],[0,242210,0.06258453],[0,242211,0.0298072536],[0,242212,0.060206669],[0,242213,0.1537294199],[0,242214,0.2029567786],[0,242216,0.0724402528],[0,242218,0.3631207154],[0,242222,0.065823053],[0,242225,0.1936824179],[0,242227,0.0664247395],[0,242229,0.0664247395],[0,242230,0.0411688153],[0,242232,0.3176671862],[0,242234,0.122472379],[0,242237,0.1537313272],[0,242267,0.5308720223],[0,242269,0.3331187045],[0,242291,0.0176255226],[0,242297,0.3863481632],[0,242505,0.1421256919],[0,242542,0.0471033824],[0,242708,0.0332902963],[0,242713,0.0190159537],[0,242743,0.0330057969],[0,242817,0.0267366344],[0,242985,0.0235069222],[0,243254,0.0267366344],[0,243277,0.0174538051],[0,243304,0.0306098132],[0,243435,0.0898044534],[0,243472,0.0255111453],[0,243537,0.0332420265],[0,243559,0.02261224],[0,243618,0.0398171546],[0,243619,0.3825654449],[0,243687,0.0298072536],[0,243707,0.2098942213],[0,243738,0.1724235319],[0,243784,0.18374266],[0,243810,0.0235069222],[0,243811,0.0256848776],[0,243927,0.0427009439],[0,243975,0.0239365347],[0,243981,0.0200832683],[0,244001,0.0192205422],[0,244004,0.1076932991],[0,244011,0.0298072536],[0,244042,0.0316694993],[0,244149,0.4582157333],[0,244185,0.0878456506],[0,244188,0.0298072536],[0,244206,0.0294083703],[0,244242,0.0604673978],[0,244344,0.1309550176],[0,244379,0.0430461399],[0,244512,0.0174538051],[0,244526,0.017953738],[0,244837,0.0197729873],[0,244925,0.0419811916],[0,245029,0.08417612],[0,245116,0.0176255226],[0,245142,0.1585527581],[0,245252,0.2537965021],[0,245279,0.0180755627],[0,245321,0.0298072536],[0,245336,0.02535854],[0,245411,0.0266796792],[0,245420,0.0340289457],[0,245438,0.0235069222],[0,245455,0.0298072536],[0,245622,0.312820391],[0,245695,0.0298072536],[0,245794,0.0263566699],[0,246038,0.0286395912],[0,246110,0.0267366344],[0,246132,0.017953738],[0,246171,0.0288621826],[0,246205,0.0267366344],[0,246262,0.3921462009],[0,246270,0.0190159537],[0,246329,0.0285672568],[0,246346,0.0298072536],[0,246358,0.0190215056],[0,246391,0.0185759483],[0,246520,0.0280219699],[0,246534,0.5881648583],[0,246608,0.0642221874],[0,246676,0.0288765131],[0,246683,0.0176255226],[0,246895,0.0176255226],[0,246896,0.0298072536],[0,246929,0.0845797489],[0,246951,0.0910020701],[0,247083,0.0188429566],[0,247149,0.0531580196],[0,247187,0.1219413241],[0,247291,0.028125896],[0,247352,0.028968001],[0,247353,0.0189586661],[0,247450,0.0270336628],[0,247496,0.0185407694],[0,247564,0.0244411871],[0,247785,0.0235069222],[0,247796,0.1446296226],[0,247807,0.0250954366],[0,247825,0.0267366344],[0,247829,0.0267366344],[0,247843,0.0267366344],[0,247898,0.0255111453],[0,247900,0.0343163797],[0,247995,0.0840626583],[0,247999,0.0298072536],[0,248050,0.0198082073],[0,248186,0.1077696959],[0,248271,0.0796183136],[0,248301,0.0330665546],[0,248494,0.2133018327],[0,256325,0.0289132777],[0,259034,0.0223798415],[0,259971,0.1623721346],[0,260462,0.2561644368],[0,260518,0.0261438141],[0,260535,0.0878317315],[0,260536,0.0533169448],[0,261061,0.0344649318],[0,261078,0.0206046585],[0,261124,0.0635099089],[0,261143,0.0196755605],[0,261207,0.0326523962],[0,261395,0.1010168144],[0,261401,0.2453777171],[0,261521,0.0494445256],[0,261610,0.0331274688],[0,261670,0.1973079362],[0,261803,0.0278897723],[0,261836,0.0265750618],[0,261898,0.0332737406],[0,262035,0.0394349278],[0,262183,0.0276768861],[0,262249,0.0244889756],[0,262491,0.0602082187],[0,262577,0.0209588164],[0,262716,0.0304785168],[0,262836,0.04372481],[0,263866,0.0293858195],[0,263995,0.0266970332],[0,264057,0.0196755605],[0,264211,0.0293858195],[0,264213,0.8511062917],[0,264262,0.0311953454],[0,265549,0.022076101],[0,265629,0.0332744096],[0,265737,0.0278847083],[0,265979,0.0332744096],[0,266045,0.0274987747],[0,266332,0.0216348848],[0,266344,0.0210494236],[0,266705,0.058727592],[0,266772,0.0293858195],[0,266805,0.0331274688],[0,266886,0.8687861894],[0,266888,0.0471865288],[0,266942,0.206021864],[0,267091,0.0705633217],[0,267107,0.0511155151],[0,267118,0.0368990733],[0,267120,0.0482335056],[0,267133,0.0206046585],[0,267138,0.2136144221],[0,267180,0.072369167],[0,267181,0.0251064372],[0,268173,0.2047568773],[0,268180,0.295183623],[0,268276,0.1533710826],[0,268285,0.0332979802],[0,268295,0.0519602484],[0,268297,0.1390849806],[0,268344,0.0266106517],[0,268446,0.0332744096],[0,268511,0.0293858195],[0,268514,0.1747965543],[0,268587,0.0256734651],[0,268604,0.0251064372],[0,268628,0.0519539883],[0,268632,0.035255702],[0,268636,0.0402402423],[0,268701,0.0221114607],[0,268773,0.3141714006],[0,268776,0.0352866192],[0,268864,0.0221114607],[0,268873,0.0293858195],[0,268885,0.0332744096],[0,268909,0.1946258533],[0,268915,0.0331274688],[0,268930,0.0842099181],[0,269016,0.0214507246],[0,269109,0.1204752138],[0,269144,0.2226670288],[0,269440,0.0533639612],[0,269451,0.0343148536],[0,269487,0.0699948093],[0,269505,0.0292374607],[0,269540,0.0293204152],[0,269574,0.0246624523],[0,269591,0.0405474396],[0,269619,0.0274192771],[0,269678,0.0196755605],[0,269727,0.0196755605],[0,269735,0.0267414897],[0,269757,0.0597291911],[0,269766,0.1178271758],[0,269770,0.0216101282],[0,269777,0.0293858195],[0,269844,0.0526899104],[0,269854,0.0327024083],[0,269860,0.0338276962],[0,269918,0.2000444371],[0,270101,0.0263917866],[0,270215,0.0338276962],[0,270241,0.0229643442],[0,270248,0.034639441],[0,270306,0.0210963142],[0,270309,0.0206046585],[0,270465,0.0481769885],[0,270479,0.0332744096],[0,270541,0.0290978482],[0,270686,0.0370381019],[0,270738,0.0494729464],[0,270805,0.0293858195],[0,270806,0.0201665541],[0,270828,0.1204105723],[0,270859,0.0803951317],[0,270869,0.0894654538],[0,270890,0.0270816351],[0,270905,0.1105127875],[0,270948,0.0219120382],[0,270960,0.0288483306],[0,271176,0.0247937762],[0,271406,0.0254854641],[0,271409,0.0332744096],[0,271510,0.1662041406],[0,271552,0.1086168266],[0,271678,0.0257139356],[0,271687,0.0316256792],[0,271693,0.0583301327],[0,271734,0.0293858195],[0,271738,0.0954325616],[0,271768,0.0409774365],[0,271791,0.0419803642],[0,271827,0.2460897256],[0,271964,0.0293858195],[0,272001,0.0342009446],[0,272028,0.0331274688],[0,272029,0.0515139511],[0,272041,0.0214860523],[0,272179,0.0311516224],[0,272241,0.0792299221],[0,272251,0.1115687567],[0,272252,0.3097017954],[0,272326,0.0270816351],[0,272421,0.0507911621],[0,272427,0.0196755605],[0,272447,0.0227027902],[0,272454,0.0332744096],[0,272522,0.0214507246],[0,272523,0.0377382381],[0,272539,0.0261438141],[0,272749,0.0399151577],[0,272755,0.0332744096],[0,272803,0.0293858195],[0,272922,0.0512560929],[0,272927,0.0464167677],[0,272962,0.1460121222],[0,273067,0.0594635179],[0,273292,0.0293858195],[0,273349,0.2257580419],[0,273425,0.0382441768],[0,273582,0.0252907367],[0,273618,0.0332744096],[0,273674,0.0318257],[0,274034,0.0177138903],[0,274069,0.1724413707],[0,274214,0.0898489075],[0,274241,0.0187284044],[0,274246,0.0487642621],[0,274529,0.0973495545],[0,274590,0.0179391268],[0,275046,0.0241637266],[0,275077,0.0221968053],[0,275091,0.0227018519],[0,275429,0.0229660322],[0,275539,0.0264582559],[0,275541,0.0390165837],[0,275569,0.0277599168],[0,275598,0.0987552759],[0,275600,0.3229073907],[0,275629,0.0180665643],[0,275636,0.1529616207],[0,275648,0.0727823609],[0,275653,0.0567102353],[0,275668,0.0303765799],[0,275686,0.0356413858],[0,275781,0.0303305517],[0,275812,0.0742183387],[0,275862,0.0267907645],[0,275866,0.0221968053],[0,275950,0.0303305517],[0,275999,0.0298690325],[0,276032,0.0303305517],[0,276066,0.1505098847],[0,276071,0.0303305517],[0,276099,0.0186115205],[0,276204,0.0273915011],[0,276257,0.0186115205],[0,276284,0.0185915909],[0,276534,0.0227614572],[0,276557,0.0811690829],[0,276577,0.0542626257],[0,276584,0.0966620243],[0,276881,0.0187151468],[0,276899,0.0298573709],[0,276911,0.1445908026],[0,277025,0.0725068973],[0,277041,0.0232298354],[0,277046,0.0185564112],[0,277094,0.7239464686],[0,277097,0.2220979635],[0,277113,0.0182720611],[0,277140,0.0185564112],[0,277284,0.0528838258],[0,277339,0.017621436],[0,277343,0.0185915909],[0,277415,0.0806431646],[0,277490,0.0503866694],[0,277499,0.1337972395],[0,277523,0.0221968053],[0,277532,0.0411479756],[0,277699,0.0264582559],[0,277811,0.0308815199],[0,277821,0.1320764423],[0,277828,0.0340349695],[0,277934,0.0499069795],[0,278085,0.0578399065],[0,278105,0.0544979424],[0,278106,0.0248793954],[0,278189,0.0220544486],[0,278244,0.0188635201],[0,278255,0.0211825318],[0,278277,0.0325322426],[0,278375,0.0346689869],[0,278407,0.0237375748],[0,278434,0.0255212745],[0,278435,0.0187284044],[0,278443,0.0185564112],[0,278459,0.0325686699],[0,278486,0.0562100146],[0,278494,0.1182682751],[0,278509,0.7153663124],[0,278770,0.0267907645],[0,278835,0.0487817639],[0,278879,0.0298573709],[0,279233,0.0452908539],[0,279260,0.0200989451],[0,279318,0.018646703],[0,279415,0.2744992723],[0,279441,0.088617095],[0,279512,0.0177138903],[0,279519,0.0367090883],[0,279540,0.0226542065],[0,279544,0.0264582559],[0,279561,0.1071541229],[0,279582,0.079127548],[0,279667,0.0185564112],[0,279813,0.0395812669],[0,279879,0.0221614413],[0,279883,0.0257860842],[0,279912,0.1391042142],[0,279920,0.0408918435],[0,279927,0.128995226],[0,279930,0.0226895957],[0,279975,0.0435730966],[0,280022,0.0341141604],[0,280277,0.0389541552],[0,280373,0.0356321404],[0,280727,0.1744085788],[0,281041,0.0263514129],[0,281061,0.0531146728],[0,281292,0.0184507817],[0,281906,0.0228731246],[0,282366,0.2115526625],[0,282377,0.0225833316],[0,282395,0.0225833316],[0,282464,0.0263514129],[0,282507,0.0263514129],[0,282552,0.0787423659],[0,282553,0.0279661916],[0,282556,0.0232609788],[0,282621,0.0166833276],[0,282627,0.0586091692],[0,282693,0.0634163588],[0,282714,0.0276172309],[0,282758,0.0195347881],[0,282764,0.026030361],[0,282836,0.0727555408],[0,282846,0.0517315343],[0,282893,0.0525000602],[0,282993,0.0180979106],[0,282999,0.024057236],[0,283006,0.0191638017],[0,283011,0.0180979106],[0,283028,0.0171370815],[0,283076,0.0263514129],[0,283259,0.0970112549],[0,283288,0.0623560732],[0,283392,0.0263514129],[0,283398,0.1113219989],[0,283404,0.0466628306],[0,283405,0.0295606879],[0,283442,0.0277221122],[0,283650,0.0481434328],[0,283861,0.0231234883],[0,284006,0.0492920554],[0,284173,0.7971805204],[0,284272,0.0263514129],[0,284275,0.0306996444],[0,284424,0.0434660408],[0,284427,0.0491451745],[0,284450,0.0277221122],[0,284462,0.0180979106],[0,284480,0.0616449927],[0,284512,0.0778684649],[0,284631,0.0690215903],[0,284672,0.026648277],[0,284746,0.0302231141],[0,284895,0.0558310777],[0,284936,0.0302231141],[0,284944,0.0300768115],[0,285007,0.0166833276],[0,285046,0.1156499128],[0,285077,0.0176083903],[0,285133,0.0235882982],[0,285136,0.0507433978],[0,285138,0.0344691229],[0,285557,0.0200256813],[0,285669,0.0320758036],[0,286283,0.0556580844],[0,286286,0.0173043876],[0,286366,0.0870866341],[0,286400,0.0347464156],[0,286430,0.024057236],[0,286459,0.0356813308],[0,286501,0.1134539009],[0,286564,0.0914662462],[0,286579,0.0235921115],[0,286640,0.2180202005],[0,286742,0.0539887491],[0,286757,0.0181354686],[0,286857,0.0306965424],[0,286874,0.1251527927],[0,286886,0.0245264912],[0,286891,0.0412120988],[0,287022,0.0780256285],[0,287040,0.0406124191],[0,287081,0.0189768611],[0,287088,0.0248628944],[0,287121,0.066587908],[0,287279,0.2645767838],[0,287282,0.0456752007],[0,287283,0.1758215061],[0,287314,0.6683159453],[0,287358,0.0294061817],[0,287468,0.0176083903],[0,287482,0.0416996333],[0,287597,0.018485956],[0,287604,0.0798429148],[0,287608,0.0191086482],[0,287618,0.100565149],[0,287653,0.0166833276],[0,287791,0.0731942401],[0,287818,0.1804231062],[0,287829,0.1936005425],[0,287847,0.2665317667],[0,287888,0.0268222209],[0,287893,0.059318415],[0,288005,0.0226152244],[0,288080,0.06143777],[0,288098,0.111694961],[0,288151,0.0248628944],[0,288161,0.0176083903],[0,288252,0.0810069494],[0,288338,0.0263514129],[0,288422,0.0305812325],[0,288604,0.1032693423],[0,288615,0.0154494885],[0,288641,0.4745509705],[0,288684,0.2288347293],[0,288689,0.0250154022],[0,289188,0.0190176332],[0,289318,0.0170262622],[0,289350,0.0287938673],[0,289371,0.0289399015],[0,289475,0.0231797196],[0,289532,0.0246056766],[0,289536,0.1317605631],[0,289548,0.0340509369],[0,289570,0.4239124103],[0,289582,0.0218532999],[0,289624,0.2379005291],[0,289651,0.0288433995],[0,289695,0.0149618543],[0,289699,0.029547411],[0,289700,0.0296627604],[0,289783,0.1763802686],[0,289830,0.6579360606],[0,289932,0.0167599589],[0,289934,0.0435054936],[0,289940,0.0223172572],[0,289942,0.2857662057],[0,290026,0.0241401841],[0,290030,0.0176476328],[0,290031,0.0167248728],[0,290165,0.0173459705],[0,290171,0.0246056766],[0,290209,0.0208470516],[0,290368,0.120816581],[0,290406,0.0222305812],[0,290412,0.0236711902],[0,290431,0.0664029735],[0,290449,0.4336395273],[0,290451,0.1743643144],[0,290505,0.0208117566],[0,290542,0.0555001044],[0,290549,0.0289399015],[0,290689,0.2796212281],[0,290731,0.0220433847],[0,290811,0.0400919489],[0,290940,0.1256874955],[0,290944,0.172469305],[0,290946,0.0425422142],[0,290965,0.0223172572],[0,290976,0.059791651],[0,291002,0.0993984156],[0,291013,0.1042101925],[0,291016,0.106759724],[0,291061,0.0289399015],[0,291151,0.0289399015],[0,291152,0.0760742028],[0,291205,0.0168964081],[0,291320,0.0168149242],[0,291345,0.0825553674],[0,291448,0.023588486],[0,291899,0.1362585175],[0,291954,0.0182632994],[0,291990,0.0254707422],[0,292109,0.0289399015],[0,292155,0.0757036238],[0,292244,0.0209799583],[0,292255,0.0149618543],[0,292274,0.0225307164],[0,292284,0.2100104961],[0,292326,0.0281713474],[0,292439,0.4049346734],[0,292476,0.0391043102],[0,292604,0.0154494885],[0,292634,0.0754902736],[0,292676,0.1059396441],[0,292798,0.2074293361],[0,292799,0.0457399622],[0,292827,0.0418679331],[0,292843,0.0289399015],[0,292892,0.029415276],[0,292996,0.0686881027],[0,293053,0.0208117566],[0,293123,0.0227853347],[0,293201,0.0287938673],[0,293256,0.0343659177],[0,293341,0.2662789649],[0,293364,0.0167599589],[0,293498,0.4520424181],[0,293546,0.144026891],[0,293572,0.0320849184],[0,293618,0.0347340505],[0,293660,0.1248837133],[0,293673,0.3153069309],[0,293786,0.6178910098],[0,293872,0.0896772925],[0,293874,0.0284676613],[0,294049,0.0154144695],[0,294067,0.0166286214],[0,294724,0.0318018116],[0,294732,0.1298445243],[0,294954,0.0282012801],[0,295037,0.025075303],[0,295160,0.5447298397],[0,295223,0.017249632],[0,295326,0.1423277831],[0,295896,0.1517277951],[0,295898,0.0273013434],[0,295916,0.0397279402],[0,295941,0.0243345607],[0,295948,0.0347138036],[0,295960,0.0158845955],[0,295971,0.5025043332],[0,296038,0.0223172572],[0,296170,0.2051322573],[0,296171,0.0866712384],[0,296207,0.0220514277],[0,296208,0.0154494885],[0,296301,0.025075303],[0,296302,0.0258683178],[0,296325,0.0493119006],[0,296363,0.0167798354],[0,296420,0.0326369558],[0,296534,0.0705379579],[0,296539,0.025075303],[0,296548,0.0432290512],[0,296632,0.3879377245],[0,296705,0.035785499],[0,296708,0.0275590604],[0,296809,0.0232806827],[0,296860,0.0381758498],[0,296985,0.1256241247],[0,296990,0.1709460542],[0,297032,0.025075303],[0,297288,0.207842712],[0,297323,0.0518868639],[0,297624,0.2323397758],[0,297674,0.025351497],[0,297790,0.0227853347],[0,297815,0.0666890521],[0,297825,0.2131884433],[0,297989,0.0289399015],[0,297991,0.0154494885],[0,297993,0.1808703535],[0,298030,0.0208470516],[0,298051,0.9056428577],[0,298131,0.0826545277],[0,298170,0.0227853347],[0,298273,0.0289399015],[0,298649,0.1541307693],[0,298651,0.0175527877],[0,298668,0.2149198539],[0,298879,0.1136679817],[0,299036,0.0167248728],[0,299057,0.1308955977],[0,299067,0.0170262622],[0,299122,0.0503336783],[0,299135,0.0287938673],[0,299152,0.0248774536],[0,299161,0.2983047395],[0,299299,0.0168149242],[0,299378,0.0325732572],[0,299417,0.025924013],[0,299561,0.112403028],[0,299723,0.0149618543],[0,299884,0.0507071295],[0,299940,0.0234555867],[0,300032,0.0225226492],[0,300212,0.0169353346],[0,300293,0.0161126358],[0,300709,0.318047136],[0,300723,0.0461190231],[0,300929,0.0173892553],[0,300941,0.0298409546],[0,301010,0.0325433771],[0,301019,0.0257477728],[0,301252,0.0174243753],[0,301254,0.0591110782],[0,301538,0.0296161142],[0,301728,0.0175876374],[0,301747,0.022172083],[0,301749,0.0254715384],[0,301972,0.0264084527],[0,302023,0.2671907896],[0,302080,0.0254570847],[0,302086,0.0219451693],[0,302146,0.115461018],[0,302365,0.1132099125],[0,302413,0.1684942086],[0,302445,0.0257265161],[0,302641,0.0305143816],[0,302849,0.0183439577],[0,302894,0.0241140876],[0,303039,0.1156963453],[0,303176,0.1228710179],[0,303210,0.0180419896],[0,303275,0.0176893283],[0,303479,0.0172000989],[0,303548,0.0427373106],[0,303565,0.0472756702],[0,303887,0.1500665712],[0,303897,0.1160355279],[0,303921,0.0276251427],[0,303979,0.1135428278],[0,303993,0.0266495833],[0,304002,0.2108325651],[0,304010,0.7278988604],[0,304027,0.0621258167],[0,304035,0.0276753025],[0,304051,0.0293692265],[0,304055,0.0528082906],[0,304095,0.1379979897],[0,304163,0.0261839511],[0,304226,0.0341696914],[0,304263,0.0184786086],[0,304340,0.0179888171],[0,304382,0.0184786086],[0,304391,0.6935288464],[0,304409,0.0548661134],[0,304628,0.080844563],[0,304665,0.4809205293],[0,304670,0.0234063974],[0,304719,0.0179888171],[0,304754,0.6826389027],[0,304810,0.021402537],[0,304859,0.8437590614],[0,304873,0.0223903496],[0,304901,0.0320287016],[0,304904,0.0286902493],[0,304905,0.0266878336],[0,304923,0.0318820213],[0,304927,0.0219301118],[0,305048,0.1913877769],[0,305214,0.0261839511],[0,305215,0.0819965519],[0,305218,0.238675825],[0,305259,0.0316827594],[0,305329,0.0473510169],[0,305366,0.1945887085],[0,305418,0.0179888171],[0,305459,0.093600131],[0,305543,0.127958458],[0,305575,0.0261839511],[0,305633,0.0197948763],[0,305650,0.3293859802],[0,305761,0.0248752468],[0,305864,0.0590856593],[0,305943,0.0179888171],[0,305982,0.0682908811],[0,305993,0.3165469806],[0,305996,0.132121698],[0,306354,0.0205822719],[0,306374,0.0290930954],[0,306417,0.0863857656],[0,306689,0.1916280516],[0,306778,0.0401740993],[0,306801,0.0206864772],[0,306841,0.0944421743],[0,306975,0.0283887397],[0,306976,0.0784864816],[0,307022,0.0326469535],[0,307068,0.0617945149],[0,307077,0.0345189346],[0,307083,0.0281470065],[0,307113,0.0739670988],[0,307150,0.0790028238],[0,307170,0.0273846559],[0,307340,0.0660607084],[0,307381,0.0213048674],[0,307387,0.0493918569],[0,307434,0.0194437219],[0,307443,0.0281470065],[0,307461,0.1165837797],[0,307470,0.021568121],[0,307582,0.0320287016],[0,307672,0.0197596351],[0,307705,0.0282976759],[0,307755,0.2257254426],[0,308051,0.0299175631],[0,308113,0.4408090937],[0,308143,0.0258469077],[0,308155,0.0318820213],[0,308171,0.0179888171],[0,308304,0.0351447137],[0,308317,0.0498615926],[0,308361,0.0562520645],[0,308364,0.064900181],[0,308458,0.0283887397],[0,308507,0.0716796282],[0,308549,0.0477057809],[0,308604,0.0320287016],[0,308705,0.0716207232],[0,308745,0.0179888171],[0,308765,0.0740083157],[0,308785,0.0244816334],[0,308820,0.0724025027],[0,308864,0.0833578865],[0,308869,0.0312567474],[0,308872,0.0650621594],[0,308883,0.0996205382],[0,309005,0.0654132347],[0,309189,0.0792260641],[0,309233,0.3806629936],[0,309272,0.2567551719],[0,309402,0.0829227778],[0,309407,0.0267366821],[0,309527,0.0303828946],[0,309644,0.0258469077],[0,309651,0.0447852641],[0,309656,0.0193708711],[0,309681,0.0203834804],[0,309702,0.2063192464],[0,309703,0.1308487803],[0,309785,0.0409967584],[0,309844,0.0258469077],[0,309939,0.0331405354],[0,310034,0.0494479984],[0,310057,0.5746777843],[0,310115,0.1221183684],[0,310142,0.0258469077],[0,310416,0.0451702753],[0,310432,0.1695904096],[0,310474,0.0467650077],[0,310475,0.0790456887],[0,310491,0.0320287016],[0,310530,0.0790893939],[0,310691,0.0234063974],[0,310696,0.0556195501],[0,310699,0.0350302362],[0,310757,0.069449227],[0,310896,0.1028963601],[0,310931,0.0189156404],[0,310934,0.2032151805],[0,311073,0.0758451821],[0,311245,0.0253767594],[0,311272,0.0500697256],[0,311411,0.2702841174],[0,311455,0.0320287016],[0,311495,0.0894805295],[0,311616,0.2558479599],[0,311635,0.033052136],[0,311636,0.0340644254],[0,311721,0.0600117886],[0,311767,0.0320287016],[0,311866,0.0787828456],[0,311868,0.0197948763],[0,311870,0.1237377957],[0,311954,0.5160012253],[0,312012,0.0189156404],[0,312037,0.0514151459],[0,312172,0.0332597834],[0,312204,0.0305915575],[0,312220,0.0234063974],[0,312221,0.1702907295],[0,312281,0.069620793],[0,312451,0.067746078],[0,312511,0.1299563528],[0,312547,0.0320287016],[0,312634,0.0664817821],[0,312663,0.0189156404],[0,312696,0.0557378148],[0,312831,0.0281470065],[0,312965,0.0373546096],[0,313002,0.0568777964],[0,313023,0.0324464616],[0,313160,0.0329930095],[0,313162,0.0320287016],[0,313356,0.037543345],[0,313423,0.0808371048],[0,313457,0.0622265],[0,313544,0.0249107497],[0,313585,0.0290729651],[0,313589,0.0197948763],[0,313645,0.0485873902],[0,313746,0.0274810337],[0,313785,0.1112626542],[0,313786,0.040364802],[0,313787,0.0802044864],[0,313792,0.6781214905],[0,313820,0.0471128792],[0,313991,0.0359513189],[0,314033,0.2375367589],[0,314091,0.0656113536],[0,314095,0.0363936687],[0,314142,0.0239000499],[0,314165,0.0234320145],[0,314179,0.0376286801],[0,314212,0.073857344],[0,314448,0.0557376401],[0,314547,0.0320287016],[0,314586,0.0318820213],[0,314732,0.0213643026],[0,314833,0.0613105699],[0,315113,0.0320287016],[0,315197,0.0347591365],[0,315231,0.057287162],[0,315451,0.0509481297],[0,315480,0.0208130632],[0,315513,0.0204187535],[0,315530,0.0544134322],[0,315661,0.0189156404],[0,315668,0.0179888171],[0,315761,0.1010263316],[0,315784,0.0498145375],[0,315803,0.7194776092],[0,316076,0.2258853354],[0,316166,0.0734018611],[0,316325,0.0281470065],[0,316368,0.0189156404],[0,316372,0.2415746724],[0,316417,0.0317707468],[0,316434,0.0353447887],[0,316517,0.1083379296],[0,316586,0.2195000553],[0,316587,0.0320287016],[0,316588,0.0281470065],[0,316597,0.0189156404],[0,316775,0.0179888171],[0,316776,0.0205559305],[0,316801,0.0203834804],[0,316803,0.0253767594],[0,316820,0.0295212439],[0,316909,0.022157477],[0,316968,0.0179888171],[0,317026,0.0326011478],[0,317038,0.0255734439],[0,317245,0.0281470065],[0,317486,0.0352915195],[0,317504,0.0693599484],[0,317571,0.0315543722],[0,317574,0.0206864772],[0,319958,0.0747468378],[0,319960,0.0274744526],[0,319962,0.1185803262],[0,320012,0.0272442187],[0,320039,0.0329231773],[0,320055,0.0616192954],[0,320065,0.0768881898],[0,320204,0.0324985149],[0,320222,0.0271255962],[0,320249,0.0586754321],[0,320266,0.1640255509],[0,320282,0.0325580322],[0,320372,0.0339079],[0,320381,0.0222287701],[0,320419,0.0653399696],[0,320422,0.0345310259],[0,320510,0.0630757148],[0,320725,0.0218386129],[0,320781,0.0263484438],[0,320816,0.0581029139],[0,320894,0.038007302],[0,320960,0.0315271407],[0,321133,0.0233481103],[0,321157,0.0227215832],[0,321508,0.1907336056],[0,321527,0.0359285651],[0,321538,0.0525218709],[0,321580,0.0233481103],[0,321645,0.040960909],[0,321739,0.2644383133],[0,321838,0.0345310259],[0,321839,0.0549003744],[0,322062,0.0747707934],[0,322148,0.0209078531],[0,322171,0.1683642512],[0,322207,0.0263061788],[0,322217,0.0286077973],[0,322390,0.0209078531],[0,322431,0.0870483308],[0,322438,0.0932507555],[0,322589,0.0267731254],[0,322618,0.0349337864],[0,322911,0.0295489411],[0,323002,0.0247678028],[0,323201,0.0321693296],[0,323230,0.0226861923],[0,323376,0.0617532267],[0,323404,0.0349337864],[0,324444,0.0342316061],[0,324480,0.4563646471],[0,324506,0.0295489411],[0,324512,0.0229902008],[0,324551,0.0291377947],[0,324564,0.048588051],[0,324600,0.0699766815],[0,324610,0.0450334243],[0,324614,0.0631166299],[0,324656,0.0359178427],[0,324771,0.0491042723],[0,324772,0.0349337864],[0,324830,0.0359052451],[0,325041,0.0263484438],[0,325180,0.0295489411],[0,325206,0.0355620214],[0,325228,0.0218386129],[0,325463,0.0263484438],[0,325464,0.0278401993],[0,325526,0.5144831817],[0,325544,0.029076289],[0,325552,0.0439227595],[0,325638,0.0237796139],[0,325784,0.034689575],[0,325788,0.0412227416],[0,325842,0.0226861923],[0,325965,0.0654826409],[0,325968,0.0272442187],[0,326096,0.0209078531],[0,326105,0.0345310259],[0,326172,0.0599138933],[0,326231,0.054692506],[0,326246,0.0916953318],[0,326384,0.0213997249],[0,326408,0.0278235319],[0,326416,0.0295489411],[0,326546,0.093237584],[0,326575,0.0295489411],[0,326752,0.0268441929],[0,326839,0.0227215832],[0,326864,0.1772434205],[0,326866,0.0347864986],[0,326906,0.3011025606],[0,327031,0.1077481338],[0,327032,0.0726535966],[0,327156,0.0544135964],[0,327193,0.0329986471],[0,327366,0.0319652732],[0,327468,0.0345310259],[0,327534,0.0605864679],[0,327573,0.033920157],[0,327656,0.0295489411],[0,327830,0.049198255],[0,327834,0.1223683445],[0,327917,0.1281827648],[0,328032,0.0278591855],[0,328126,0.2889801333],[0,328168,0.0349337864],[0,328188,0.0213644016],[0,328213,0.0267731254],[0,328295,0.0485197964],[0,328299,0.1097317018],[0,328429,0.0295489411],[0,328483,0.266496155],[0,328610,0.5588821444],[0,328779,0.029076289],[0,328813,0.0295489411],[0,328891,0.0209078531],[0,328919,0.0349337864],[0,328955,0.0797477162],[0,329000,0.0314439194],[0,329030,0.4573547832],[0,329039,0.3077840219],[0,329050,0.0239662203],[0,329172,0.0218386129],[0,329613,0.0586733744],[0,329648,0.0355620214],[0,329682,0.1425989213],[0,329693,0.0248568919],[0,329697,0.0564356351],[0,329736,0.0287797166],[0,329777,0.0498639247],[0,329803,0.0387365516],[0,329897,0.0327139536],[0,329904,0.1324841038],[0,329923,0.029076289],[0,329948,0.0266339023],[0,329970,0.0269782546],[0,330167,0.0209078531],[0,330210,0.0347864986],[0,330267,0.0559591414],[0,330273,0.0251590993],[0,330403,0.0278882011],[0,330418,0.0457450005],[0,330441,0.0272442187],[0,330570,0.0687407718],[0,330618,0.0267731254],[0,330637,0.2237390771],[0,330705,0.0242025194],[0,330728,0.0977695582],[0,330752,0.034119156],[0,330800,0.0982368799],[0,330801,0.0209078531],[0,330836,0.296511992],[0,330889,0.0851350342],[0,330901,0.1385884321],[0,330910,0.0479385458],[0,330935,0.03305771],[0,330962,0.0327381038],[0,330997,0.0315080635],[0,331023,0.1605708957],[0,331035,0.18214258],[0,331052,0.0226861923],[0,331097,0.022366307],[0,331160,0.0333186379],[0,331184,0.2588572382],[0,331190,0.031857774],[0,331395,0.0209078531],[0,331456,0.070635965],[0,331534,0.0349337864],[0,331587,0.0330970051],[0,331594,0.0295489411],[0,331602,0.3494721805],[0,331617,0.3927924981],[0,331827,0.0295489411],[0,331841,0.0458258214],[0,332119,0.0313519479],[0,332196,0.0227416325],[0,332216,0.0227215832],[0,332419,0.0291377947],[0,332465,0.1464270363],[0,332473,0.0560009889],[0,332527,0.0280282417],[0,332625,0.0226861923],[0,332913,0.1663385922],[0,333031,0.1201016863],[0,333351,0.036405855],[0,333464,0.0884664376],[0,333477,0.0263061788],[0,333648,0.5017102797],[0,333691,0.0242025194],[0,333707,0.0278591855],[0,333809,0.0366660083],[0,333888,0.077145692],[0,334092,0.0345310259],[0,334248,0.0263484438],[0,334320,0.0435933321],[0,334622,0.0611249522],[0,334651,0.1373971376],[0,334816,0.043958879],[0,334895,0.6291462147],[0,334916,0.0349337864],[0,334929,0.057947574],[0,334930,0.0227215832],[0,335126,0.0222287701],[0,335707,0.0343838223],[0,335905,0.1194576932],[0,336102,0.0209078531],[0,336105,0.0263484438],[0,336288,0.0246248583],[0,336332,0.0328793465],[0,336454,0.0349337864],[0,336497,0.0387365516],[0,336857,0.0680829258],[0,336918,0.0518637238],[0,337193,0.0295489411],[0,337601,0.0209078531],[0,337607,0.0295489411],[0,337629,0.0418953554],[0,337630,0.0395010274],[0,337631,0.15824372],[0,337633,0.0432291226],[0,337649,0.2195359251],[0,337660,0.0321577119],[0,337677,0.06907475],[0,337696,0.0677336361],[0,337731,0.5942118194],[0,337747,0.0272442187],[0,337749,0.0370441446],[0,337851,0.0349337864],[0,337886,0.054086785],[0,337903,0.0544210616],[0,337918,0.0272442187],[0,337950,0.2310043384],[0,337951,0.0268085912],[0,337963,0.0349337864],[0,338020,0.031387782],[0,338023,0.0272442187],[0,338029,0.0345310259],[0,338060,0.0226861923],[0,338062,0.0209078531],[0,338065,0.0350050427],[0,338080,0.2734892083],[0,338145,0.1479801713],[0,338147,0.1365814815],[0,338196,0.0364220226],[0,338315,0.0321577119],[0,338391,0.0248658879],[0,338451,0.0295489411],[0,338455,0.3609478229],[0,338456,0.0274388186],[0,338584,0.7236550241],[0,338610,0.0468109467],[0,338710,0.0613624578],[0,338716,0.0900963296],[0,338763,0.0296255686],[0,338776,0.0349337864],[0,338968,0.0489984245],[0,338970,0.0313519479],[0,338982,0.0226861923],[0,339111,0.0208725549],[0,339243,0.0227215832],[0,339244,0.0209078531],[0,339312,0.0272442187],[0,339326,0.0281868342],[0,339427,0.7991633493],[0,339433,0.0295489411],[0,339472,0.0372958504],[0,339473,0.0242379878],[0,339599,0.1289932295],[0,339677,0.029076289],[0,339728,0.0569408823],[0,339751,0.0273876762],[0,339849,0.0315439056],[0,339870,0.0229902008],[0,339872,0.0303645014],[0,339988,0.0349337864],[0,340072,0.0349337864],[0,340099,0.0345310259],[0,340128,0.0349337864],[0,340160,0.0489781906],[0,340169,0.0719382562],[0,340177,0.071867775],[0,340179,0.0655814885],[0,340206,0.1187774569],[0,340252,0.0319423738],[0,340303,0.0273876762],[0,340351,0.0319090941],[0,340370,0.9300219475],[0,340390,0.0850694348],[0,340391,0.1366370644],[0,340395,0.0349337864],[0,340436,0.0345310259],[0,340460,0.0545942622],[0,340496,0.284000341],[0,340502,0.0856168084],[0,340505,0.0272442187],[0,340572,0.0310599101],[0,340592,0.0349337864],[0,340641,0.029076289],[0,340780,0.0548097306],[0,340812,0.0345310259],[0,340863,0.0443086399],[0,340905,0.4771753888],[0,340944,0.1024067157],[0,341059,0.4745956136],[0,341065,0.2644434426],[0,341132,0.0209078531],[0,341165,0.0789711073],[0,341186,0.0218386129],[0,341187,0.0222287701],[0,341224,0.0291377947],[0,341265,0.0221934043],[0,341292,0.0978593231],[0,341470,0.0495708199],[0,341555,0.5485809467],[0,341630,0.6904677728],[0,341715,0.0365943842],[0,342894,0.0295489411],[0,342917,0.0732213614],[0,342939,0.563470573],[0,342953,0.4450464726],[0,342967,0.0290398016],[0,343031,0.0935082088],[0,343069,0.2760794982],[0,343071,0.0461756485],[0,343080,0.0233481103],[0,343084,0.0263061788],[0,343086,0.0356201586],[0,343133,0.0349337864],[0,343137,0.9639200508],[0,343153,0.0295489411],[0,343311,0.970563495],[0,343331,0.0272442187],[0,343372,0.0712980306],[0,343379,0.0986034326],[0,343506,0.0551754586],[0,343521,0.046321144],[0,343660,0.0263484438],[0,343661,0.0233126874],[0,343662,0.0373235122],[0,343743,0.1475540289],[0,343773,0.0355259737],[0,343775,0.248269094],[0,343789,0.1992218676],[0,343794,0.2487981708],[0,343905,0.0233481103],[0,343986,0.0322886406],[0,343993,0.029076289],[0,344093,0.0309699855],[0,344126,0.0209078531],[0,344136,0.0278591855],[0,344156,0.2436724843],[0,344193,0.4433840003],[0,344312,0.0263484438],[0,344486,0.0295489411],[0,344522,0.0349337864],[0,344850,0.0278591855],[0,344863,0.0222287701],[0,344904,0.0291377947],[0,345177,0.2490680386],[0,345385,0.0438479866],[0,345389,0.0227215832],[0,345412,0.0222287701],[0,345481,0.0589799997],[0,345503,0.0349337864],[0,345596,0.0326908502],[0,346225,0.1835374533],[0,346229,0.0209078531],[0,346369,0.8347476047],[0,346372,0.0242379878],[0,346385,0.0302105889],[0,346498,0.0272442187],[0,346512,0.029076289],[0,346565,0.0363032247],[0,346573,0.0227215832],[0,346786,0.0349337864],[0,346794,0.0295489411],[0,346840,0.0237440947],[0,346862,0.0301659428],[0,346974,0.0272442187],[0,347060,0.481730915],[0,347068,0.0281357817],[0,347301,0.0349337864],[0,347322,0.2021383487],[0,347454,0.0527389036],[0,347564,0.029076289],[0,347684,0.0412627175],[0,348062,0.0555370136],[0,348088,0.0226861923],[0,348114,0.0226861923],[0,348126,0.047235291],[0,348159,0.0338655494],[0,348199,0.0469658368],[0,348207,0.0530993284],[0,348284,0.2362980862],[0,348286,0.1143693912],[0,348374,0.0268441929],[0,348382,0.0272442187],[0,348544,0.0631034022],[0,348598,0.1158526471],[0,348600,0.0222841736],[0,348730,0.0237440947],[0,348820,0.0349337864],[0,348821,0.0349337864],[0,348876,0.0333545726],[0,348939,0.034674271],[0,348994,0.0313519479],[0,349154,0.0209078531],[0,349428,0.0228458811],[0,349465,0.043958879],[0,349494,0.0397479008],[0,349731,0.0222841736],[0,349750,0.0291701478],[0,349794,0.0339079],[0,350139,0.9485231602],[0,350141,0.0408487856],[0,350268,0.2898125069],[0,350431,0.029076289],[0,350697,0.5631620206],[0,350771,0.0269637994],[0,350794,0.0913631392],[0,351094,0.0272442187],[0,351341,0.0296806513],[0,351349,0.0222287701],[0,351400,0.1515817823],[0,351522,0.0447926056],[0,351582,0.0561124347],[0,351605,0.0349337864],[0,351664,0.0233126874],[0,351675,0.0400134928],[0,351793,0.2789247678],[0,351868,0.034055004],[0,351873,0.0686975402],[0,351945,0.0710304507],[0,352150,0.2807781966],[0,352153,0.035988593],[0,352157,0.0723990808],[0,352166,0.0740942644],[0,352171,0.0349337864],[0,352203,0.0347864986],[0,352217,0.0349337864],[0,352259,0.1901019176],[0,352269,0.0349337864],[0,352458,0.2504286893],[0,352556,0.2830593431],[0,352576,0.0232155116],[0,352595,0.9224654373],[0,352696,0.0896644171],[0,352744,0.6424125127],[0,352768,0.022894618],[0,353052,0.0228592182],[0,353118,0.0716707665],[0,353176,0.7592578393],[0,353231,0.0263484438],[0,353245,0.0295511693],[0,353427,0.0274315551],[0,353478,0.0349337864],[0,353675,0.0295489411],[0,353717,0.0295489411],[0,353730,0.0464205799],[0,353811,0.0432342908],[0,353904,0.0924639194],[0,353905,0.0349337864],[0,353934,0.0267731254],[0,353945,0.1798210757],[0,353982,0.0419750123],[0,354074,0.0574296158],[0,354088,0.0555865448],[0,354122,0.1845671311],[0,354332,0.0237086515],[0,354408,0.0349337864],[0,354411,0.0248658879],[0,354648,0.0228548509],[0,354674,0.0319090941],[0,354724,0.1207062415],[0,355220,0.0226861923],[0,355275,0.0226861923],[0,355333,0.1193568191],[0,355357,0.0315080635],[0,355413,0.0263061788],[0,355705,0.0267731254],[0,355707,0.3887826069],[0,355833,0.054156689],[0,356336,0.1221139822],[0,356392,0.0297968242],[0,356447,0.0533858228],[0,356485,0.0349337864],[0,356490,0.021532738],[0,356551,0.0232155116],[0,356675,0.0349337864],[0,356734,0.0326908502],[0,356838,0.0482386257],[0,357290,0.0564067928],[0,357381,0.0246248583],[0,357425,0.0295489411],[0,357552,0.0227215832],[0,357586,0.0226861923],[0,358210,0.0954276848],[0,358218,0.0901229869],[0,358220,0.0850418757],[0,358301,0.0376401853],[0,358370,0.0319449567],[0,358379,0.0608396707],[0,358425,0.0768078748],[0,358428,0.0273876762],[0,358574,0.1157186569],[0,358576,0.1265228743],[0,358579,0.0272442187],[0,358683,0.0391614977],[0,358691,0.0263484438],[0,358930,0.0588977513],[0,358954,0.029076289],[0,359307,0.0268085912],[0,359309,0.0580785341],[0,359343,0.3925935195],[0,359362,0.0428427198],[0,359589,0.0209078531],[0,359674,0.0423506572],[0,359689,0.0272442187],[0,359727,0.0826278578],[0,359746,0.0229902008],[0,359764,0.164462361],[0,359765,0.047714119],[0,359821,0.0239971763],[0,359827,0.277207184],[0,359839,0.0361862361],[0,359855,0.029076289],[0,359863,0.0435839825],[0,359991,0.0319449567],[0,359997,0.1997349712],[0,360029,0.0779108851],[0,360033,0.0349337864],[0,360055,0.0349337864],[0,360104,0.0260467152],[0,360124,0.0295489411],[0,360153,0.1599696716],[0,360179,0.0349337864],[0,360226,0.0726363811],[0,360239,0.0295489411],[0,360255,0.0226861923],[0,360324,0.2578348842],[0,360389,0.034055004],[0,360519,0.1798493639],[0,360729,0.0360433809],[0,360739,0.0244394496],[0,360821,0.0619633245],[0,360825,0.0833508866],[0,360848,0.0478558788],[0,360850,0.2479102685],[0,360852,0.6312781197],[0,360854,0.082629188],[0,360989,0.0261193701],[0,361009,0.0363068369],[0,361104,0.0276021322],[0,361162,0.0374515649],[0,361254,0.0628321115],[0,361324,0.0360195863],[0,361333,0.0356399858],[0,361334,0.035895861],[0,361407,0.0238320162],[0,361490,0.0306500509],[0,361525,0.0486593557],[0,361539,0.0274445106],[0,361654,0.0373553391],[0,361661,0.0366726058],[0,361681,0.0306500509],[0,361692,0.0278698614],[0,361752,0.0325081867],[0,361824,0.0233738365],[0,361866,0.028341697],[0,361896,0.25865516],[0,362078,0.0337906822],[0,362099,0.0959349805],[0,362108,0.0306500509],[0,362154,0.028341697],[0,362252,0.0224526152],[0,362263,0.023420885],[0,362264,0.0306500509],[0,362370,0.0403988663],[0,362378,0.3424894952],[0,362385,0.8128260402],[0,362408,0.0272960984],[0,362410,0.0360433809],[0,362457,0.0356399858],[0,362484,0.1431027402],[0,362570,0.0304433038],[0,362622,0.0371464811],[0,362683,0.029650878],[0,362693,0.0422526069],[0,362804,0.0271794775],[0,362818,0.2498586311],[0,362899,0.0572189603],[0,362947,0.0323643039],[0,362973,0.0285366035],[0,363066,0.0520266994],[0,363071,0.0280753138],[0,363098,0.2327616349],[0,363111,0.0733253319],[0,363113,0.0239497872],[0,363123,0.0324558986],[0,363125,0.0284853806],[0,363192,0.0306500509],[0,363231,0.0306500509],[0,363284,0.0553683986],[0,363289,0.609630769],[0,363354,0.0477859063],[0,363398,0.0246113624],[0,363472,0.0360433809],[0,363491,0.0238119354],[0,363563,0.0240859283],[0,363624,0.0360433809],[0,363902,0.1088994116],[0,363957,0.0238119354],[0,363968,0.0274445106],[0,363979,0.0308321419],[0,364018,0.7320256225],[0,364271,0.0289576328],[0,364272,0.5980019679],[0,364289,0.0652910337],[0,364391,0.0325496732],[0,364992,0.2972949679],[0,365011,0.0363156922],[0,365033,0.0259965536],[0,365049,0.0290104075],[0,365064,0.028069989],[0,365070,0.0330659199],[0,365097,0.0848566752],[0,365130,0.0745881878],[0,365190,0.0308471235],[0,365388,0.0290104075],[0,365468,0.0388586102],[0,365502,0.1158030258],[0,365508,0.0624257503],[0,365528,0.0247456025],[0,365692,0.0541357959],[0,365776,0.3197524738],[0,365844,0.040202575],[0,365857,0.0386571118],[0,365874,0.0460199705],[0,366012,0.3429544245],[0,366016,0.028069989],[0,366023,0.0386370873],[0,366034,0.2991807372],[0,366122,0.0521289504],[0,366123,0.044164948],[0,366124,0.0336871119],[0,366129,0.0525204458],[0,366134,0.0332224174],[0,366212,0.0298355626],[0,366256,0.023982241],[0,366263,0.0309087852],[0,366272,0.0334570093],[0,366289,0.0759940394],[0,366298,0.0313209742],[0,366324,0.2506988485],[0,366334,0.1838803317],[0,366440,0.023982241],[0,366487,0.0367194741],[0,366492,0.0226579745],[0,366522,0.0242513479],[0,366800,0.0299520929],[0,366860,0.0808241221],[0,367065,0.0384015666],[0,367125,0.4694523429],[0,367197,0.0263991661],[0,367221,0.0621968171],[0,367236,0.059033633],[0,367273,0.0522056042],[0,367280,0.0329902042],[0,367310,0.0384015666],[0,367391,0.3948725627],[0,367534,0.0384015666],[0,367535,0.0384015666],[0,367556,0.0668397873],[0,367763,0.0665642512],[0,367794,0.0362439835],[0,367830,0.121002565],[0,367957,0.0683592023],[0,368120,0.0261493737],[0,368148,0.0302007192],[0,368225,0.082316245],[0,368232,0.1602705081],[0,368370,0.0991828075],[0,368468,0.1360740208],[0,368470,0.1054879969],[0,368477,0.1549644814],[0,368496,0.0353980187],[0,368565,0.1596486104],[0,368641,0.0902621587],[0,368645,0.0256339858],[0,368683,0.0384015666],[0,368862,0.0384015666],[0,368935,0.1814226478],[0,369104,0.0325152245],[0,369237,0.0329902042],[0,369354,0.0270290228],[0,369372,0.2066733584],[0,369525,0.0372182992],[0,369560,0.2341387828],[0,369589,0.03745699],[0,369611,0.0329902042],[0,369678,0.0828418941],[0,369741,0.0534394695],[0,369842,0.0306741324],[0,369870,0.0728407804],[0,369891,0.0297739464],[0,369903,0.0252419073],[0,369905,0.0243065641],[0,369938,0.0330892846],[0,370059,0.1828545921],[0,370067,0.0252419073],[0,370071,0.0329902042],[0,370130,0.6480623186],[0,370388,0.3127827399],[0,370409,0.0242710921],[0,370412,0.0281171461],[0,370454,0.0325152245],[0,370564,0.1127054704],[0,370604,0.0329902042],[0,370667,0.0306741324],[0,370719,0.0315700859],[0,370757,0.0540717578],[0,370921,0.0312599702],[0,370933,0.0267232407],[0,370969,0.0795227486],[0,371109,0.0348020897],[0,371142,0.2421035063],[0,371206,0.0329902042],[0,371338,0.0624131545],[0,371575,0.0306741324],[0,371581,0.204722411],[0,371810,0.5806866961],[0,371842,0.1050843748],[0,371880,0.0297739464],[0,371897,0.0558336696],[0,372155,0.0297739464],[0,372164,0.0261849418],[0,372172,0.0384015666],[0,372211,0.0302922111],[0,372280,0.3507342906],[0,372282,0.0767473683],[0,372332,0.1295378182],[0,372831,0.0393566367],[0,373034,0.0487544543],[0,373082,0.6093041947],[0,373101,0.0329902042],[0,373233,0.100461812],[0,373265,0.0255984459],[0,373292,0.0652085284],[0,373464,0.043454207],[0,374083,0.0329902042],[0,374136,0.0347033019],[0,374172,0.0364568979],[0,374341,0.0306741324],[0,374398,0.9738953442],[0,374485,0.9452427581],[0,374546,0.03562502],[0,374612,0.0400409045],[0,374630,0.0275196748],[0,375038,0.0318969992],[0,375301,0.0462081599],[0,375556,0.0298173862],[0,375560,0.0414370131],[0,375590,0.0372374902],[0,375665,0.0369898291],[0,376123,0.0632282929],[0,376260,0.0487182827],[0,376391,0.0321083255],[0,376542,0.1605060528],[0,376621,0.0273643106],[0,376631,0.0968278562],[0,376910,0.0319361309],[0,376959,0.3603505542],[0,377063,0.354721531],[0,377101,0.0410305331],[0,377128,0.0318675965],[0,377419,0.0414370131],[0,377441,0.028393076],[0,377491,0.0285251059],[0,377715,0.0334330201],[0,378147,0.0278134337],[0,378168,0.0788050663],[0,378193,0.0424803769],[0,378643,0.3271839375],[0,378646,0.0560201335],[0,378694,0.0420344874],[0,378699,0.0282543406],[0,378747,0.0752700295],[0,378750,0.1673860185],[0,378752,0.0324099122],[0,378764,0.1146619964],[0,378771,0.0524441573],[0,378792,0.0346824827],[0,378793,0.0901485139],[0,378803,0.0641209388],[0,378827,0.1499878795],[0,378828,0.0380957618],[0,378829,0.1521546291],[0,378831,0.1329326625],[0,378844,0.026426109],[0,378868,0.0353001867],[0,378873,0.0414370131],[0,378878,0.1576710707],[0,378910,0.0607891647],[0,378916,0.0360024395],[0,378921,0.0540445331],[0,378923,0.0541632715],[0,378933,0.1772297482],[0,378941,0.0538298941],[0,378945,0.0282186667],[0,378989,0.1168423933],[0,379014,0.2795006679],[0,379045,0.0371842783],[0,379054,0.0235678363],[0,379087,0.2512816477],[0,379140,0.0332033861],[0,379230,0.0361144867],[0,379231,0.0749711399],[0,379252,0.0274057227],[0,379268,0.0646363406],[0,379312,0.0270684259],[0,379338,0.0403466613],[0,379362,0.1234129685],[0,379441,0.0356294682],[0,379471,0.0792761834],[0,379541,0.0248583422],[0,379558,0.0235678363],[0,379623,0.0323352071],[0,379658,0.030072633],[0,379660,0.145914059],[0,379731,0.0317828366],[0,379816,0.0341445091],[0,379943,0.9397901962],[0,379954,0.0571074321],[0,380233,0.0583876301],[0,380261,0.0335215824],[0,380308,0.0385170681],[0,380409,0.0331048226],[0,380517,0.5198018607],[0,380567,0.0395449043],[0,380569,0.0305459595],[0,380714,0.0735602205],[0,380844,0.034108534],[0,380853,0.0332033861],[0,380882,0.0530322554],[0,380938,0.1358845016],[0,380983,0.0315920832],[0,381032,0.025163305],[0,381158,0.160497736],[0,381159,0.0424390427],[0,381230,0.1639746751],[0,381258,0.0400642598],[0,381283,0.0277118047],[0,381394,0.0412780073],[0,381414,0.0465327859],[0,381447,0.0385170681],[0,381499,0.0382422959],[0,381506,0.8537670953],[0,381510,0.0299286226],[0,381519,0.0331884955],[0,381603,0.210700741],[0,381607,0.0370493109],[0,381696,0.0262012117],[0,381764,0.034108534],[0,381798,0.0346204362],[0,381870,0.0258957931],[0,381878,0.0418345562],[0,381944,0.1192193736],[0,382126,0.0744345744],[0,382175,0.1060955924],[0,382183,0.0351220806],[0,382190,0.0565606719],[0,382284,0.0414857227],[0,382435,0.821576832],[0,382469,0.0519923126],[0,382515,0.1253012547],[0,382590,0.0246033585],[0,382663,0.0508652608],[0,382852,0.0332033861],[0,382952,0.0324301389],[0,382968,0.0255035669],[0,382971,0.2322471722],[0,382986,0.0385170681],[0,383027,0.0784731699],[0,383091,0.0331048226],[0,383216,0.2499987315],[0,383220,0.0526119124],[0,383221,0.0322573206],[0,383313,0.0581517337],[0,383529,0.0469479413],[0,383585,0.0381122582],[0,383611,0.0383690308],[0,383685,0.0458416478],[0,383772,0.0365715858],[0,383857,0.0259514904],[0,383890,0.4448457931],[0,383914,0.0632672801],[0,383920,0.0300730803],[0,384047,0.0292015219],[0,384166,0.1181460324],[0,384275,0.0385170681],[0,384511,0.0400795419],[0,384806,0.0567081945],[0,384829,0.0783038246],[0,384849,0.1874721293],[0,384863,0.030506706],[0,385095,0.0331048226],[0,385127,0.1870823954],[0,385146,0.025539102],[0,385156,0.0930736803],[0,385172,0.0332033861],[0,385259,0.0337373817],[0,385310,0.0356294682],[0,385344,0.3405617152],[0,385476,0.2230494975],[0,385808,0.0332033861],[0,385872,0.1494808175],[0,385881,0.0332033861],[0,385890,0.0300730803],[0,385902,0.0272128579],[0,385909,0.0650153916],[0,385910,0.0340881554],[0,385917,0.0279187029],[0,385942,0.0259313482],[0,385944,0.0303022414],[0,385993,0.1589684714],[0,386058,0.035214547],[0,386122,0.0467151624],[0,386124,0.0439325873],[0,386190,0.0258957931],[0,386297,0.0533390261],[0,386307,0.0322573206],[0,386342,0.1602417727],[0,386346,0.2227639696],[0,386436,0.2249806166],[0,386560,0.7015412168],[0,386569,0.1608928751],[0,386623,0.0424390427],[0,386659,0.0280416153],[0,386692,0.025539102],[0,386698,0.0289869675],[0,386781,0.0751960546],[0,386794,0.0799139207],[0,386908,0.0385170681],[0,386933,0.1668208722],[0,387191,0.0385170681],[0,387423,0.0617864041],[0,387561,0.0575442107],[0,387567,0.0427300735],[0,387589,0.0246033585],[0,387615,0.0381122582],[0,387621,0.025539102],[0,387633,0.1952882741],[0,387653,0.458297748],[0,387669,0.0385170681],[0,387691,0.0362838004],[0,387836,0.0259313482],[0,387899,0.0281244325],[0,388040,0.0299286226],[0,388076,0.110170911],[0,388090,0.0706044473],[0,388108,0.1393238812],[0,388189,0.0414072611],[0,388202,0.0922922352],[0,388229,0.0928850439],[0,388231,0.0760412078],[0,388239,0.0507280061],[0,388240,0.0267257559],[0,388250,0.0561598572],[0,388293,0.0259870483],[0,388297,0.034362722],[0,388302,0.0330001318],[0,388393,0.0347857815],[0,388478,0.071234502],[0,388536,0.025539102],[0,388542,0.0591517004],[0,388557,0.0258957931],[0,388595,0.1237624465],[0,388637,0.067148513],[0,388672,0.038063551],[0,388685,0.9042622226],[0,388698,0.0385170681],[0,388823,0.0332033861],[0,389006,0.0341445091],[0,389282,0.0258957931],[0,389350,0.1221119868],[0,389370,0.0259313482],[0,389381,0.1833510807],[0,389510,0.0424390427],[0,389565,0.1095702486],[0,389664,0.0500470466],[0,389751,0.0274547865],[0,389830,0.0569493854],[0,389873,0.0332033861],[0,389908,0.0570228949],[0,389966,0.2196921407],[0,390018,0.0507496835],[0,390102,0.0347192124],[0,390108,0.0258957931],[0,390118,0.0724532455],[0,390119,0.0701222176],[0,390123,0.034891884],[0,390177,0.0294821458],[0,390269,0.066078496],[0,390673,0.0575393621],[0,390675,0.8141723652],[0,390691,0.0320557953],[0,390762,0.1885711993],[0,390777,0.1880306248],[0,390817,0.0299286226],[0,390867,0.0396055935],[0,390917,0.0931442507],[0,391047,0.0383690308],[0,391049,0.0289869675],[0,391083,0.1066294784],[0,391084,0.046406056],[0,391125,0.0332033861],[0,391135,0.0332033861],[0,391174,0.0596937078],[0,391178,0.0484023039],[0,391183,0.0432306276],[0,391268,0.057043935],[0,391401,0.0299286226],[0,391442,0.0608689415],[0,391473,0.0605159762],[0,391517,0.0383610267],[0,391599,0.1261319517],[0,391602,0.0282564115],[0,391691,0.0262012117],[0,391827,0.0299286226],[0,391835,0.0246033585],[0,391847,0.0299286226],[0,391853,0.0317004059],[0,391856,0.0332033861],[0,391857,0.0414059202],[0,391906,0.2029627333],[0,392051,0.0381122582],[0,392058,0.2081703755],[0,392142,0.0385170681],[0,392191,0.0385170681],[0,392227,0.0500470466],[0,392229,0.0332033861],[0,392230,0.0332033861],[0,392430,0.076426942],[0,392480,0.0299286226],[0,392511,0.1435116906],[0,392512,0.0517258133],[0,392603,0.0259870483],[0,392611,0.0413276548],[0,392712,0.0382115244],[0,392729,0.2170229899],[0,392735,0.0276272687],[0,392743,0.0332033861],[0,392827,0.0586182363],[0,392864,0.0331048226],[0,392903,0.0332033861],[0,392920,0.3333416374],[0,392956,0.0356294682],[0,392983,0.0326314149],[0,392990,0.0246033585],[0,392998,0.0383690308],[0,393009,0.03380805],[0,393051,0.0352683215],[0,393095,0.0332033861],[0,393096,0.0332033861],[0,393118,0.0258957931],[0,393201,0.040481149],[0,393219,0.1692911816],[0,393301,0.2891448106],[0,393415,0.0323380824],[0,393417,0.0259870483],[0,393425,0.0276272687],[0,393488,0.045645191],[0,393574,0.2369553927],[0,393644,0.0672439326],[0,393775,0.0332033861],[0,393829,0.0385170681],[0,393862,0.0300373134],[0,393951,0.0415789368],[0,393998,0.0332033861],[0,394079,0.0987787248],[0,394180,0.0300730803],[0,394191,0.2123422014],[0,394390,0.0382115244],[0,394781,0.0331048226],[0,394801,0.038063551],[0,394818,0.3406301532],[0,394895,0.0921887446],[0,395007,0.0385170681],[0,395008,0.0259313482],[0,395053,0.036701765],[0,395097,0.0281585421],[0,395208,0.0424793334],[0,395362,0.3204065953],[0,395574,0.0740302512],[0,395613,0.0289146093],[0,395629,0.0275882715],[0,396148,0.0461934785],[0,396171,0.1906266188],[0,396181,0.0561193471],[0,396202,0.0299286226],[0,396203,0.0246033585],[0,396349,0.0258957931],[0,396351,0.0506405272],[0,396443,0.0360625261],[0,396460,0.0340521832],[0,396488,0.041658229],[0,396711,0.0246033585],[0,396767,0.5811785661],[0,396885,0.0385170681],[0,396897,0.0332033861],[0,396919,0.0260696216],[0,396955,0.4036561956],[0,396958,0.0973789203],[0,397066,0.1644286542],[0,397070,0.06761466],[0,397130,0.199794322],[0,397405,0.0464145157],[0,397473,0.3642871215],[0,397507,0.0415235806],[0,397536,0.121089449],[0,397588,0.0331048226],[0,397594,0.1006845484],[0,397648,0.0414420646],[0,397655,0.0536617475],[0,397674,0.025539102],[0,397720,0.0627900642],[0,397840,0.1468393271],[0,397844,0.0446750322],[0,397947,0.0300730803],[0,398046,0.0374327645],[0,398100,0.0262012117],[0,398169,0.0259313482],[0,398203,0.1269422954],[0,398225,0.1452548252],[0,398405,0.0556750999],[0,398436,0.051297453],[0,398510,0.0299286226],[0,398656,0.0468969506],[0,398665,0.1632089981],[0,398728,0.0319839855],[0,398827,0.3697653064],[0,398831,0.0504002627],[0,398873,0.0246033585],[0,398900,0.0886772835],[0,399026,0.0259313482],[0,399043,0.0547806367],[0,399050,0.0825528617],[0,399054,0.0790361744],[0,399060,0.0892103412],[0,399063,0.046013302],[0,399131,0.0516402649],[0,399166,0.0259870483],[0,399172,0.5875512512],[0,399202,0.0619995514],[0,399399,0.1766044762],[0,399590,0.6303314417],[0,399609,0.0299286226],[0,399617,0.0383285605],[0,399628,0.056952028],[0,399656,0.0818169845],[0,399737,0.0299286226],[0,399789,0.1474999472],[0,399793,0.0351581076],[0,399813,0.0472974086],[0,400111,0.0381122582],[0,400126,0.2960452115],[0,400195,0.0546914176],[0,400261,0.0331048226],[0,400264,0.0259870483],[0,400332,0.0383690308],[0,400764,0.0282564115],[0,400766,0.0596155401],[0,400840,0.0299286226],[0,400848,0.093199066],[0,400883,0.025539102],[0,400964,0.0246033585],[0,400988,0.2749807102],[0,401115,0.0385170681],[0,401175,0.0332033861],[0,401204,0.0323380824],[0,401283,0.0331048226],[0,401497,0.0332033861],[0,401671,0.0623989685],[0,401731,0.0385170681],[0,401762,0.1881420182],[0,401769,0.0779959871],[0,401855,0.0281585421],[0,401856,0.054546004],[0,401926,0.0260696216],[0,402027,0.7067644389],[0,402098,0.0299286226],[0,402120,0.0260696216],[0,402199,0.0489801796],[0,402289,0.025539102],[0,402346,0.1451854009],[0,402461,0.035214547],[0,402488,0.0331048226],[0,402627,0.0331048226],[0,402714,0.078584336],[0,402801,0.0338236143],[0,402882,0.0340052295],[0,402974,0.0608361254],[0,402978,0.0306292199],[0,402997,0.2599487918],[0,403012,0.0277388149],[0,403055,0.0340052295],[0,403126,0.0553421112],[0,403142,0.0299308543],[0,403297,0.0451305567],[0,403302,0.0414747697],[0,403410,0.0364970144],[0,403416,0.0341039194],[0,403450,0.0296595436],[0,403509,0.0314430862],[0,403510,0.0308249579],[0,403597,0.0400812247],[0,403626,0.027870721],[0,403736,0.0936572086],[0,403765,0.23347891],[0,403862,0.0394244133],[0,403938,0.0345194973],[0,403941,0.1378085307],[0,403946,0.1689879024],[0,404203,0.048963636],[0,404442,0.0288847957],[0,404540,0.0432235821],[0,404586,0.0394244133],[0,404629,0.0362880308],[0,404861,0.0742052981],[0,404932,0.0492400556],[0,404969,0.4029411613],[0,404984,0.1305669489],[0,405127,0.1509815879],[0,405487,0.0563896653],[0,405497,0.8703441286],[0,406039,0.0296237978],[0,406309,0.0364400105],[0,406337,0.0871978477],[0,406345,0.0311553277],[0,406480,0.0564062608],[0,406769,0.1199086068],[0,406846,0.0599396574],[0,407097,0.0388967473],[0,407173,0.2596491375],[0,407238,0.0570442076],[0,407377,0.0496982719],[0,407626,0.2232743254],[0,407663,0.0729824964],[0,407872,0.0561948966],[0,408308,0.0308249579],[0,408563,0.0317518842],[0,408591,0.4553290708],[0,408683,0.1260641227],[0,409242,0.0798092962],[0,409301,0.0491042946],[0,409327,0.0312270526],[0,409328,0.0705750295],[0,409374,0.0319472988],[0,409407,0.1088494469],[0,409408,0.0611346639],[0,409409,0.3520393531],[0,409433,0.0276002096],[0,409442,0.0521885387],[0,409463,0.3028253182],[0,409502,0.0276002096],[0,409515,0.0480276826],[0,409519,0.0277388149],[0,409520,0.0403105004],[0,409560,0.0928624667],[0,409590,0.0991127451],[0,409600,0.0275645692],[0,409684,0.0298820956],[0,409707,0.0331006559],[0,409760,0.0529051962],[0,409771,0.0525706976],[0,409843,0.027870721],[0,409851,0.0289965811],[0,409872,0.0308249579],[0,409874,0.3265712996],[0,409896,0.029127305],[0,409904,0.0443028334],[0,409910,0.2644887463],[0,409983,0.0390190844],[0,409994,0.0331566411],[0,409997,0.0290915865],[0,410025,0.0524543223],[0,410061,0.0395238636],[0,410081,0.6279990247],[0,410086,0.0481391],[0,410139,0.1176146412],[0,410140,0.0789011755],[0,410142,0.6830951252],[0,410154,0.0668460726],[0,410171,0.0317518842],[0,410231,0.0296237978],[0,410244,0.026269032],[0,410277,0.2184265109],[0,410310,0.0277388149],[0,410327,0.0397344371],[0,410349,0.0340052295],[0,410355,0.7845273698],[0,410611,0.0275645692],[0,410623,0.0440454972],[0,410653,0.0275645692],[0,410685,0.0767776612],[0,410747,0.0276560435],[0,410770,0.0804021476],[0,410851,0.0692147235],[0,410883,0.2276376209],[0,410914,0.1311638245],[0,410976,0.1036736601],[0,411045,0.069649395],[0,411148,0.0633826013],[0,411159,0.0419746732],[0,411368,0.0370521068],[0,411384,0.0374879769],[0,411500,0.2657142522],[0,411519,0.0514445282],[0,411552,0.0301639546],[0,411566,0.0487534861],[0,411585,0.0849562062],[0,411594,0.0622134093],[0,411618,0.0366350333],[0,411696,0.0366350333],[0,411842,0.076939188],[0,411918,0.028771952],[0,411923,0.0825870105],[0,412016,0.033932547],[0,412089,0.0333442724],[0,412133,0.0401062455],[0,412148,0.0697832289],[0,412242,0.2145430938],[0,412270,0.028771952],[0,412298,0.0623380388],[0,412402,0.0374879769],[0,412432,0.0366350333],[0,412589,0.0300721511],[0,412627,0.0611444323],[0,412629,0.063353538],[0,412635,0.0430979865],[0,412656,0.0297133171],[0,412679,0.2515627152],[0,412779,0.0419746732],[0,412836,0.0770103933],[0,412917,0.0844821398],[0,412927,0.0301079198],[0,413351,0.0462275451],[0,413520,0.0366350333],[0,413540,0.0336237331],[0,413572,0.0419746732],[0,413791,0.0342947235],[0,413959,0.0829879631],[0,414105,0.03213879],[0,414113,0.3793175744],[0,414176,0.0300721511],[0,414211,0.0419746732],[0,414368,0.0420744814],[0,414385,0.0568891006],[0,414605,0.2398566321],[0,415351,0.0395081406],[0,415439,0.0365359884],[0,415461,0.0709602509],[0,415485,0.0363535917],[0,415502,0.7443779773],[0,415631,0.2739688041],[0,415632,0.0297133171],[0,415766,0.0375241249],[0,415835,0.0300721511],[0,415909,0.080484667],[0,415930,0.052280003],[0,415984,0.0366350333],[0,415986,0.0793285846],[0,416068,0.0441159778],[0,416089,0.1483221338],[0,416115,0.0333690823],[0,416254,0.0374932659],[0,416290,0.0333442724],[0,416302,0.0301079198],[0,416307,0.0351511635],[0,416333,0.0487566119],[0,416342,0.0579279256],[0,416354,0.2006967357],[0,416381,0.0366350333],[0,416466,0.0559774869],[0,416473,0.0342947235],[0,416491,0.028771952],[0,416539,0.0300721511],[0,416565,0.0919415386],[0,416638,0.0491427815],[0,416901,0.2370469831],[0,416924,0.0323980172],[0,417132,0.0627791673],[0,417134,0.0475515141],[0,417166,0.0469480535],[0,417354,0.0416676368],[0,417402,0.2436149508],[0,417477,0.0531638128],[0,417635,0.0340948372],[0,417641,0.0394718911],[0,417655,0.049307637],[0,417674,0.4536325483],[0,417844,0.0597222699],[0,417906,0.0366350333],[0,417930,0.0987203237],[0,417936,0.0424511636],[0,418202,0.0394492352],[0,418391,0.0539019819],[0,418435,0.0348274712],[0,418563,0.0268742143],[0,418648,0.0342535183],[0,418670,0.1137248391],[0,418685,0.3526492535],[0,418792,0.0328690532],[0,419027,0.0414936168],[0,419138,0.0282624326],[0,419206,0.0320207794],[0,419474,0.1192239845],[0,419514,0.1454725559],[0,419691,0.0268742143],[0,419695,0.536961149],[0,419819,0.0382797445],[0,419963,0.0360586105],[0,419977,0.0573839681],[0,420008,0.0333208521],[0,420056,0.0488995956],[0,420070,0.0635970525],[0,420091,0.0658777696],[0,420107,0.1130627337],[0,420116,0.0974852864],[0,420120,0.214274332],[0,420174,0.3319140002],[0,420204,0.0328690532],[0,420218,0.0974196874],[0,420286,0.0411867881],[0,420291,0.0268742143],[0,420314,0.2476731504],[0,420442,0.0268742143],[0,420444,0.0358889809],[0,420480,0.0768456716],[0,420498,0.0317647615],[0,420573,0.0281501394],[0,420577,0.0280538442],[0,420584,0.0274861743],[0,420671,0.0371910713],[0,420679,0.0314719011],[0,420753,0.0749992385],[0,420757,0.3339485337],[0,420758,0.3100250325],[0,420760,0.2234421189],[0,420765,0.0623173234],[0,420778,0.0778773447],[0,420896,0.0325398366],[0,421004,0.2871496817],[0,421106,0.034022331],[0,421197,0.0717809446],[0,421361,0.0637396941],[0,421495,0.029903919],[0,421535,0.4105452011],[0,421593,0.0358268299],[0,421611,0.028052285],[0,421635,0.04884132],[0,421745,0.0274861743],[0,421806,0.040570364],[0,421870,0.0358268299],[0,421874,0.0367149035],[0,421894,0.2515843471],[0,421896,0.1111622213],[0,421935,0.0428988214],[0,421963,0.0414396838],[0,421964,0.042748695],[0,422005,0.0345450934],[0,422070,0.0378414785],[0,422085,0.0378414785],[0,422173,0.0285359445],[0,422189,0.2733641064],[0,422245,0.2073737749],[0,422258,0.0345450934],[0,422303,0.0551932558],[0,422305,0.0340366664],[0,422335,0.0335612738],[0,422407,0.0431902442],[0,422486,0.0413746598],[0,422500,0.0719297691],[0,422641,0.2463645112],[0,422695,0.3895309191],[0,422823,0.1703880921],[0,422833,0.0470096008],[0,423020,0.0376438061],[0,423100,0.0508694364],[0,423168,0.0368494133],[0,423281,0.0632799069],[0,423352,0.0839833798],[0,423375,0.0319016442],[0,423525,0.0413835953],[0,424224,0.0301428537],[0,424329,0.058774755],[0,424406,0.0368059219],[0,424448,0.0370960773],[0,424522,0.1024041101],[0,424578,0.0313676983],[0,424589,0.0423821335],[0,424614,0.0378414785],[0,424631,0.0660845328],[0,424661,0.198692355],[0,424685,0.033597221],[0,424688,0.0378414785],[0,424777,0.0426038788],[0,424786,0.0294769914],[0,424852,0.0378414785],[0,424854,0.0301428537],[0,425003,0.0345450934],[0,425051,0.5306761439],[0,425107,0.0407194961],[0,425233,0.0374795721],[0,425239,0.329583414],[0,425256,0.0298357041],[0,425285,0.0841133721],[0,425402,0.0342671263],[0,425456,0.1384529397],[0,425469,0.2992325919],[0,425550,0.0378414785],[0,425553,0.0636131234],[0,425557,0.0524629502],[0,425644,0.0431902442],[0,425726,0.2528273073],[0,425740,0.0625267823],[0,425759,0.0322097015],[0,425780,0.0867444045],[0,425878,0.2019086298],[0,426110,0.039006922],[0,426269,0.0294769914],[0,426508,0.0443534896],[0,426570,0.0558703994],[0,426573,0.0551179842],[0,426649,0.2005227533],[0,426653,0.0440969911],[0,426876,0.0336947947],[0,426943,0.0954360199],[0,426962,0.0386958798],[0,427054,0.0346055937],[0,427114,0.0448261751],[0,427154,0.0313676983],[0,427160,0.0521293761],[0,427175,0.0431902442],[0,427220,0.0431902442],[0,427254,0.0781460117],[0,427343,0.3005972044],[0,427413,0.0378414785],[0,427446,0.055355232],[0,427513,0.1639594471],[0,427611,0.0623477106],[0,427616,0.0378414785],[0,427791,0.0660451693],[0,427869,0.0345450934],[0,428127,0.1149479974],[0,428146,0.0302879086],[0,428175,0.0285359445],[0,428176,0.0602921246],[0,428219,0.035602436],[0,428225,0.0477573177],[0,428461,0.0971146378],[0,428556,0.3085790083],[0,428563,0.0321112774],[0,428682,0.0441579124],[0,428757,0.0674444997],[0,428763,0.0345450934],[0,428770,0.1816088531],[0,428783,0.2030596107],[0,428822,0.0510199138],[0,428975,0.0313676983],[0,429119,0.0378414785],[0,429171,0.0340366664],[0,429176,0.1088152176],[0,429300,0.0396284961],[0,429340,0.0345450934],[0,429348,0.0652454827],[0,429357,0.0345450934],[0,429437,0.0299274766],[0,429449,0.0586998229],[0,429582,0.1487743559],[0,429608,0.0285359445],[0,429618,0.0380605214],[0,429623,0.0378414785],[0,429626,0.0345090978],[0,429728,0.2054688114],[0,429754,0.0315377748],[0,429796,0.0285359445],[0,429832,0.0345450934],[0,429847,0.0430412294],[0,429999,0.1151749094],[0,430047,0.033597221],[0,430146,0.2601258203],[0,430298,0.0584330711],[0,430492,0.0377430066],[0,430515,0.0753039284],[0,430573,0.0336087447],[0,430635,0.0568201661],[0,430717,0.0322097015],[0,430761,0.0345450934],[0,430763,0.0431902442],[0,430929,0.0482082514],[0,430970,0.0403709019],[0,431063,0.083347666],[0,431138,0.0298714607],[0,431227,0.0285359445],[0,431263,0.1392473246],[0,431485,0.3577500609],[0,431536,0.0461672809],[0,432047,0.3649130915],[0,432196,0.033597221],[0,432246,0.098085576],[0,432363,0.270188479],[0,432383,0.0345450934],[0,432849,0.0340366664],[0,432967,0.0505768922],[0,433049,0.178164881],[0,433079,0.0398664059],[0,433128,0.6453841161],[0,433146,0.0712524617],[0,433218,0.2580813203],[0,433284,0.0429500833],[0,433432,0.0586780747],[0,433650,0.0398664059],[0,433887,0.0378414785],[0,434182,0.0393639285],[0,434401,0.0531117305],[0,434410,0.0514776776],[0,434522,0.4423985982],[0,434653,0.036078945],[0,434679,0.0673498203],[0,434681,0.0428826831],[0,434687,0.169954406],[0,434693,0.0473060306],[0,434731,0.0301428537],[0,434769,0.2162939925],[0,434770,0.2174680215],[0,434824,0.7515396716],[0,434843,0.0466382238],[0,434845,0.0300447196],[0,434852,0.0421539995],[0,434853,0.3152353241],[0,434934,0.0685343559],[0,434992,0.1220492275],[0,435052,0.0387888153],[0,435075,0.077979267],[0,435108,0.0438747826],[0,435158,0.0937138869],[0,435162,0.0405220889],[0,435176,0.0335101243],[0,435191,0.0724008165],[0,435200,0.0298357041],[0,435367,0.033597221],[0,435509,0.0478769482],[0,435513,0.0351168634],[0,435565,0.1006317532],[0,435573,0.4591023753],[0,435635,0.0461703464],[0,435685,0.090714556],[0,435686,0.0398664059],[0,435689,0.0736832239],[0,435716,0.0294769914],[0,435801,0.0865527954],[0,435879,0.3425535546],[0,435890,0.0427337326],[0,435891,0.0298357041],[0,436029,0.0345090978],[0,436061,0.0608390093],[0,436099,0.0319375065],[0,436105,0.0298714607],[0,436183,0.0559519442],[0,436204,0.0392096178],[0,436419,0.0354830385],[0,436432,0.0578943275],[0,436610,0.0402472898],[0,436612,0.0313676983],[0,436627,0.0641697746],[0,436639,0.0351304789],[0,436641,0.089630978],[0,436675,0.0595575882],[0,436724,0.0285359445],[0,436728,0.0431411873],[0,436919,0.1524771806],[0,436975,0.0721365727],[0,436977,0.0428826831],[0,436980,0.0345450934],[0,436998,0.0606154528],[0,437044,0.0463495278],[0,437066,0.2603588528],[0,437118,0.3430583359],[0,437312,0.0299274766],[0,437329,0.0689303941],[0,437404,0.0361503715],[0,437625,0.0379400647],[0,437745,0.0736561936],[0,437791,0.1017493777],[0,437809,0.0380174603],[0,438128,0.0595157336],[0,438129,0.0349098477],[0,438172,0.0488769238],[0,438359,0.0321415989],[0,438376,0.0321005268],[0,438464,0.0563846533],[0,438465,0.0411176839],[0,438710,0.0307607526],[0,438833,0.0364141593],[0,438836,0.0402384186],[0,439256,0.0833617496],[0,439282,0.0398752687],[0,439309,0.0665415092],[0,439429,0.0392464874],[0,439430,0.0601452231],[0,439544,0.0309636296],[0,439739,0.0362192406],[0,439861,0.0392464874],[0,439871,0.0524168854],[0,439977,0.0593780021],[0,440114,0.0360425143],[0,440324,0.0442977087],[0,440337,0.0669130449],[0,440341,0.0623877281],[0,440353,0.0788610652],[0,440661,0.1835550502],[0,440711,0.0627014077],[0,440830,0.0409069087],[0,441184,0.0393459127],[0,441198,0.0380638695],[0,441273,0.0640515709],[0,441285,0.0297718539],[0,441379,0.0392464874],[0,441504,0.0360425143],[0,441539,0.0355330056],[0,441592,0.1019428417],[0,441719,0.0392816353],[0,441807,0.0376157819],[0,441860,0.0390938692],[0,441910,0.0888020131],[0,442051,0.0664266385],[0,442249,0.0638369901],[0,442262,0.2689094553],[0,442335,0.0730763118],[0,442363,0.076963364],[0,442458,0.0447060583],[0,442628,0.0670131122],[0,442890,0.0393459127],[0,442896,0.0632648809],[0,443053,0.0297718539],[0,443062,0.0331435147],[0,443214,0.0565087358],[0,443378,0.060438399],[0,443525,0.0409599944],[0,443652,0.0447060583],[0,443770,0.0474188494],[0,443773,0.0535113603],[0,443799,0.0417495759],[0,443834,0.0383141381],[0,443836,0.1216824324],[0,443844,0.035246285],[0,443929,0.0312833013],[0,443930,0.0300205805],[0,443950,0.0585735595],[0,444140,0.0352823184],[0,444143,0.11147375],[0,444170,0.1306257334],[0,444307,0.0595290674],[0,444440,0.0300205805],[0,444459,0.1471056914],[0,444494,0.042663864],[0,444524,0.0447060583],[0,444531,0.0507921242],[0,444619,0.1053886063],[0,444638,0.0334521176],[0,444652,0.0388024472],[0,444683,0.6988701205],[0,444795,0.0310739156],[0,444906,0.0442977087],[0,445019,0.0331997928],[0,445022,0.0509178684],[0,445041,0.0542106335],[0,445204,0.0360425143],[0,445225,0.0965395106],[0,445257,0.0441440025],[0,445618,0.071835847],[0,445667,0.1887056641],[0,445727,0.1572082105],[0,445954,0.0393459127],[0,446197,0.0709738791],[0,446317,0.044382183],[0,446471,0.0390938692],[0,446522,0.0982860292],[0,446525,0.0411223831],[0,446529,0.0443978428],[0,446556,0.035246285],[0,446571,0.1793481969],[0,446573,0.0447060583],[0,446577,0.1263331401],[0,446646,0.1277220644],[0,446796,0.1108972349],[0,446846,0.1256350556],[0,446881,0.0311097355],[0,446888,0.0657281801],[0,447017,0.0393459127],[0,447028,0.0622562266],[0,447029,0.0479387524],[0,447112,0.0312490388],[0,447134,0.0445567265],[0,447200,0.0447060583],[0,447211,0.0719490244],[0,447267,0.0879079073],[0,447294,0.0431692221],[0,447327,0.0586191485],[0,447374,0.0349775347],[0,447409,0.0399232054],[0,447528,0.0316444692],[0,447545,0.040753276],[0,448138,0.0612394773],[0,448418,0.0358202379],[0,448421,0.0829829583],[0,448465,0.0551461584],[0,448566,0.037903863],[0,448569,0.0859289972],[0,448589,0.0404644317],[0,448647,0.0462459898],[0,448660,0.0405275684],[0,448690,0.0681578969],[0,448705,0.1000086784],[0,448738,0.0404644317],[0,448771,0.2541596976],[0,448852,0.0608197347],[0,448878,0.03494853],[0,448951,0.0463608969],[0,449060,0.0305902761],[0,449476,0.0305902761],[0,449481,0.0316444692],[0,449564,0.1631626376],[0,449634,0.0948465046],[0,449682,0.0384759069],[0,449853,0.0366171158],[0,449939,0.0319524135],[0,449994,0.0538479718],[0,450005,0.0390387367],[0,450261,0.0452877178],[0,450423,0.0466995737],[0,450545,0.076363851],[0,450632,0.0452877178],[0,450681,0.0399232054],[0,450798,0.0454748163],[0,450885,0.0317364791],[0,451045,0.0405275684],[0,451063,0.0428096958],[0,451123,0.0399232054],[0,451192,0.0467338692],[0,451200,0.0504551104],[0,451280,0.0638791584],[0,451296,0.0449792512],[0,451308,0.1501500078],[0,451496,0.0319524135],[0,451578,0.0366171158],[0,451638,0.0360511462],[0,451735,0.0358563005],[0,451997,0.039823699],[0,452022,0.084861074],[0,452239,0.0866857758],[0,452266,0.0446968382],[0,452267,0.0578565336],[0,452277,0.0705564952],[0,452311,0.0379400322],[0,452398,0.0354272717],[0,452567,0.3851766653],[0,452620,0.0316803184],[0,452636,0.031929724],[0,452693,0.0366171158],[0,452943,0.1139900199],[0,453026,0.3698725256],[0,453042,0.0724547459],[0,453093,0.108876191],[0,453115,0.0399232054],[0,453219,0.1022788706],[0,453224,0.036107192],[0,453256,0.0744355171],[0,453501,0.0657608869],[0,453505,0.0366171158],[0,453670,0.0408394154],[0,453693,0.0319524135],[0,453697,0.0752687526],[0,453766,0.0337157545],[0,453769,0.0377875227],[0,453819,0.2686717368],[0,453822,0.0452877178],[0,453831,0.3563007923],[0,453834,0.0316444692],[0,453913,0.0746697064],[0,453925,0.0404644317],[0,453964,0.0387161809],[0,454059,0.0405275684],[0,454171,0.278850531],[0,454417,0.0452877178],[0,454418,0.231413632],[0,454422,0.0305902761],[0,454453,0.0424346328],[0,454490,0.0405638719],[0,454570,0.1485425737],[0,454626,0.0317006272],[0,454721,0.0379646241],[0,454845,0.0343136212],[0,455041,0.0316803184],[0,455065,0.0517087064],[0,456131,0.0884627189],[0,456309,0.124178387],[0,456424,0.0754154223],[0,456494,0.0748372504],[0,456760,0.0456340587],[0,457212,0.6233946429],[0,457300,0.0629644532],[0,457449,0.0303055662],[0,457872,0.0516013956],[0,457915,0.0384759069],[0,457974,0.036107192],[0,458057,0.0691315024],[0,458239,0.060438399],[0,458297,0.2642342595],[0,458515,0.0316444692],[0,458624,0.0444350601],[0,458691,0.0326584461],[0,458848,0.0428096958],[0,458885,0.0363660429],[0,459085,0.1026350693],[0,459313,0.0323058641],[0,459391,0.4366251468],[0,459414,0.0489916445],[0,459573,0.2641984161],[0,459577,0.1041886094],[0,459609,0.0365335601],[0,459621,0.116161669],[0,459698,0.035379678],[0,459757,0.1341287974],[0,459773,0.0369276057],[0,459776,0.042520015],[0,459854,0.0452877178],[0,459870,0.1426534473],[0,459943,0.0452877178],[0,460229,0.0954676275],[0,460240,0.0399232054],[0,460241,0.0380638695],[0,460341,0.0356664529],[0,460349,0.039823699],[0,460350,0.0678454693],[0,460365,0.0333754027],[0,460373,0.0426393616],[0,460384,0.1129974115],[0,460463,0.0315340934],[0,460531,0.1228003594],[0,460576,0.0449792512],[0,460596,0.0723070676],[0,460597,0.0647251865],[0,460606,0.0806701695],[0,460629,0.1048970429],[0,460714,0.2525699655],[0,460733,0.039823699],[0,460752,0.0445758683],[0,460831,0.0399232054],[0,460843,0.0637988061],[0,460846,0.0397415388],[0,460931,0.0366171158],[0,460951,0.0661501832],[0,461007,0.0399232054],[0,461088,0.0433050724],[0,461093,0.0315340934],[0,461194,0.0316444692],[0,461495,0.0399232054],[0,461509,0.0319524135],[0,461796,0.0495701438],[0,461808,0.0592407477],[0,461904,0.5804379626],[0,461910,0.1034293502],[0,461958,0.0565567878],[0,461987,0.0316803184],[0,462017,0.3068339253],[0,462080,0.0637219189],[0,462093,0.0366171158],[0,462135,0.0317364791],[0,462141,0.0323422037],[0,462233,0.0386136642],[0,462617,0.036107192],[0,462725,0.0452877178],[0,462777,0.0396709566],[0,462902,0.0412165176],[0,462925,0.0316444692],[0,462938,0.0316803184],[0,462958,0.0945253336],[0,463025,0.0480867745],[0,463345,0.0550781264],[0,463361,0.0684893113],[0,463385,0.1674332308],[0,463594,0.0411595973],[0,463621,0.0316803184],[0,463656,0.0482896355],[0,463915,0.0494906691],[0,463978,0.0930519983],[0,464045,0.2350355003],[0,464291,0.0499904359],[0,464588,0.0340246088],[0,464602,0.0687802462],[0,464615,0.2396808035],[0,464658,0.0399232054],[0,464690,0.1244742053],[0,464804,0.0411223831],[0,464882,0.0319524135],[0,464951,0.04739954],[0,465091,0.0842863574],[0,465188,0.039823699],[0,465192,0.0423724953],[0,465238,0.0449792512],[0,465246,0.0751417935],[0,465354,0.0421033579],[0,465451,0.0505874701],[0,465483,0.0452877178],[0,465492,0.0452877178],[0,465536,0.0448774803],[0,465619,0.0452877178],[0,465642,0.0421941689],[0,465930,0.0497664583],[0,466074,0.0448790354],[0,466120,0.0432056496],[0,466351,0.0862444689],[0,466588,0.0714060008],[0,466641,0.4549134809],[0,466650,0.0354157181],[0,466677,0.047260067],[0,466837,0.0406207444],[0,466840,0.0510033047],[0,466902,0.0399232054],[0,466957,0.0917993977],[0,467254,0.0603408922],[0,467258,0.2231978875],[0,467271,0.1905786858],[0,467374,0.0377203157],[0,467399,0.0878895568],[0,467902,0.1959287357],[0,468017,0.0587985762],[0,468129,0.0319524135],[0,468245,0.0637445535],[0,468442,0.1213609933],[0,468488,0.0558980822],[0,468713,0.0319882784],[0,469412,0.5286062587],[0,469511,0.0440603129],[0,469563,0.0399232054],[0,469564,0.037963083],[0,469623,0.036107192],[0,469727,0.0407397806],[0,469828,0.0534272433],[0,469829,0.0570172574],[0,469892,0.1470854269],[0,469918,0.0639781788],[0,469931,0.05365272],[0,470002,0.032614099],[0,470055,0.0401990133],[0,470137,0.0891231972],[0,470141,0.0452877178],[0,470148,0.0711600635],[0,470261,0.0757706948],[0,470597,0.0356664529],[0,470603,0.0316444692],[0,471602,0.2572900527],[0,471692,0.0452877178],[0,471790,0.0315340934],[0,471835,0.0452877178],[0,471909,0.0315340934],[0,472034,0.0399232054],[0,472106,0.0951237214],[0,472128,0.043122537],[0,472177,0.0375939859],[0,472179,0.3126803106],[0,472181,0.08272245],[0,472219,0.0532399639],[0,472270,0.0584221878],[0,472345,0.0610922847],[0,472358,0.0785975859],[0,472385,0.0330818668],[0,472573,0.309926112],[0,472623,0.0389680899],[0,472643,0.0428096958],[0,472698,0.0337157545],[0,472702,0.039823699],[0,472708,0.0385306548],[0,472727,0.1950918365],[0,472780,0.0319524135],[0,472802,0.2343446281],[0,472841,0.0699356624],[0,472961,0.0811514109],[0,473026,0.0303413469],[0,473263,0.0399732955],[0,473340,0.0577181125],[0,473491,0.0645969448],[0,473941,0.3246478068],[0,473997,0.0553328639],[0,474308,0.1997597116],[0,474427,0.0333754027],[0,474429,0.0975315331],[0,474433,0.1033566026],[0,474680,0.0568441316],[0,474740,0.0557097492],[0,474971,0.1072965008],[0,474986,0.0765871852],[0,475055,0.039823699],[0,475064,0.0452877178],[0,475105,0.0452877178],[0,475268,0.0530863282],[0,475285,0.0372403581],[0,475309,0.0366171158],[0,475356,0.0764953627],[0,475495,0.0408394154],[0,475566,0.0316444692],[0,475575,0.0389680899],[0,475681,0.0696145192],[0,475718,0.0363299542],[0,475826,0.0399232054],[0,475842,0.6277118157],[0,475865,0.2912628951],[0,475958,0.0410566259],[0,476095,0.0430983326],[0,476214,0.0507364817],[0,476215,0.0447927786],[0,476271,0.103581605],[0,476293,0.0305902761],[0,476311,0.0316444692],[0,476388,0.1649729823],[0,476494,0.0421261224],[0,476495,0.0454199992],[0,476675,0.0369276057],[0,476683,0.065347318],[0,476684,0.039823699],[0,476740,0.0604044623],[0,476747,0.0452877178],[0,476772,0.0318197352],[0,476877,0.0375962631],[0,477188,0.0305902761],[0,477295,0.0389680899],[0,477486,0.0487689907],[0,477500,0.0399232054],[0,477537,0.0366191663],[0,477539,0.044066042],[0,477592,0.0411744589],[0,477632,0.0352157816],[0,477760,0.041074777],[0,477769,0.046548436],[0,477876,0.1564448056],[0,477877,0.036910196],[0,477953,0.0464991479],[0,478029,0.0605873373],[0,478133,0.4232410626],[0,478183,0.1658109548],[0,478437,0.1079892558],[0,478449,0.0439740146],[0,478462,0.044066042],[0,478779,0.1096262925],[0,478798,0.0407663746],[0,478833,0.4606246448],[0,478912,0.0376110204],[0,479247,0.0352991047],[0,479449,0.0636273601],[0,479547,0.0721370867],[0,479564,0.0384868782],[0,479571,0.0389838252],[0,479591,0.0318250632],[0,479616,0.0783652332],[0,479739,0.0560909495],[0,479795,0.0424078454],[0,479856,0.0370313734],[0,480004,0.037983877],[0,480187,0.0523419082],[0,480191,0.0442391345],[0,480630,0.0540641692],[0,480671,0.0417130599],[0,480691,0.0633694251],[0,480711,0.040433831],[0,480713,0.2375407298],[0,480938,0.1203698173],[0,481143,0.0466635468],[0,481145,0.0754726855],[0,481378,0.0474492693],[0,481425,0.0810553726],[0,481426,0.1241294464],[0,481729,0.0913978024],[0,481733,0.8439713829],[0,482961,0.0401221621],[0,483067,0.1209033463],[0,483259,0.0535324143],[0,483355,0.0538445748],[0,483787,0.0474492693],[0,483968,0.0538445748],[0,484245,0.0755462763],[0,484314,0.2681701031],[0,484350,0.066542906],[0,484356,0.0399262505],[0,484446,0.0784876191],[0,484667,0.0487165096],[0,484717,0.0538445748],[0,484741,0.0771901443],[0,485124,0.0907603465],[0,485262,0.0450701422],[0,485396,0.0450701422],[0,485401,0.1505214408],[0,485431,0.3375475504],[0,485446,0.0510157103],[0,485473,0.0534309985],[0,485474,0.0417130599],[0,485514,0.0538445748],[0,485530,0.0699240564],[0,485531,0.0538445748],[0,485560,0.0480300957],[0,485567,0.0643964945],[0,485617,0.0538445748],[0,485665,0.0450701422],[0,485671,0.0761620637],[0,485701,0.0466635468],[0,485750,0.2664449396],[0,485756,0.0863276953],[0,485853,0.3724279178],[0,485862,0.1001547027],[0,486007,0.0872650548],[0,486016,0.1083677138],[0,486057,0.0594145717],[0,486059,0.8645873362],[0,486101,0.1412816254],[0,486164,0.0771480366],[0,486165,0.0480300957],[0,486166,0.0496685251],[0,486200,0.0435664898],[0,486206,0.0492650197],[0,486222,0.0491494464],[0,486314,0.0400947785],[0,486418,0.3353808462],[0,486428,0.1197591996],[0,486480,0.3765792152],[0,486486,0.0766222875],[0,486498,0.0694796501],[0,486505,0.038971131],[0,486664,0.0401221621],[0,486690,0.3494295039],[0,486872,0.0483151243],[0,486877,0.4378143108],[0,486933,0.056407183],[0,487128,0.0536933317],[0,487256,0.0450701422],[0,487357,0.098185838],[0,487361,0.1856821038],[0,487362,0.0399262505],[0,487464,0.0538445748],[0,487546,0.0726029776],[0,487667,0.038971131],[0,487690,0.0484158223],[0,487705,0.038971131],[0,487714,0.0845841914],[0,487733,0.052663467],[0,487749,0.038971131],[0,487981,0.2362499494],[0,488110,0.0707030301],[0,488126,0.0605330598],[0,488172,0.0415799802],[0,488226,0.0770819279],[0,488350,0.0424805823],[0,488540,0.0704866959],[0,488731,0.2745838908],[0,488737,0.0578514817],[0,488751,0.1231442675],[0,488864,0.0917753495],[0,489012,0.1750083109],[0,489142,0.0524913754],[0,489425,0.0684326065],[0,489571,0.0630470445],[0,489647,0.0480300957],[0,489654,0.0727510324],[0,489663,0.0548387214],[0,489728,0.0807747106],[0,489745,0.0584318398],[0,489772,0.0535324143],[0,489873,0.1083824745],[0,489895,0.0590738471],[0,489898,0.0441080951],[0,490040,0.0445541121],[0,490108,0.038971131],[0,490135,0.2634240133],[0,490365,0.0538445748],[0,490452,0.4766838348],[0,490461,0.0540641692],[0,490520,0.038971131],[0,490818,0.1819786231],[0,490925,0.0484158223],[0,491028,0.0483151243],[0,491145,0.0605659198],[0,491202,0.0474043768],[0,491310,0.0398418409],[0,491313,0.091631575],[0,491369,0.0536516031],[0,491427,0.0597783725],[0,491530,0.0395666845],[0,491556,0.0484857669],[0,491666,0.0679884994],[0,491670,0.0520131095],[0,491682,0.073801192],[0,491696,0.108266848],[0,491804,0.0397915242],[0,491868,0.0576240124],[0,491944,0.0561174898],[0,491974,0.0485134745],[0,491975,0.1171091555],[0,491994,0.1855142633],[0,492053,0.039530432],[0,492054,0.038380361],[0,492089,0.0540951875],[0,492099,0.0576339068],[0,492100,0.0477165602],[0,492120,0.039530432],[0,492301,0.0429009976],[0,492522,0.0444742848],[0,493020,0.0532413983],[0,493022,0.0398418409],[0,493208,0.0559178583],[0,493210,0.5660225769],[0,494130,0.0398418409],[0,494196,0.0444742848],[0,494256,0.1057418307],[0,494293,0.0532413983],[0,494315,0.0429718867],[0,494318,0.0720750397],[0,494350,0.0395666845],[0,494434,0.0408733291],[0,494453,0.0921109232],[0,494459,0.0491081537],[0,494551,0.0878937376],[0,494553,0.3563576397],[0,494619,0.0695735974],[0,494734,0.0532413983],[0,494823,0.0532413983],[0,494941,0.0589242974],[0,494981,0.0483010749],[0,494988,0.0478171741],[0,495100,0.039530432],[0,495178,0.154268896],[0,495250,0.0477165602],[0,495343,0.1020436512],[0,495359,0.0444742848],[0,495425,0.1105238079],[0,495477,0.0762070098],[0,495555,0.1072338408],[0,495684,0.0791405808],[0,495835,0.0682196917],[0,495881,0.0419143378],[0,495974,0.0395030712],[0,496011,0.0410836708],[0,496177,0.0793723841],[0,496263,0.3824477536],[0,496590,0.107143438],[0,496710,0.0439586851],[0,496799,0.097105091],[0,496967,0.0517656201],[0,497133,0.038380361],[0,497141,0.0532413983],[0,497168,0.0812657944],[0,497333,0.0743574694],[0,497382,0.0531916494],[0,497695,0.0439586851],[0,497870,0.0927985967],[0,497964,0.0532413983],[0,498084,0.0477165602],[0,498327,0.04882883],[0,498521,0.1247810695],[0,498743,0.4823063922],[0,499109,0.4469304065],[0,499811,0.0395030712],[0,499812,0.0527784465],[0,499873,0.039530432],[0,499920,0.0398418409],[0,499938,0.0395666845],[0,499956,0.0564731215],[0,500207,0.2739936943],[0,500211,0.0485134745],[0,500251,0.038380361],[0,500269,0.039530432],[0,500321,0.0427410393],[0,500417,0.0649232726],[0,500567,0.0692111565],[0,500834,0.0496927366],[0,501173,0.0476363132],[0,501380,0.0726105944],[0,501679,0.212757869],[0,501802,0.0444742848],[0,501834,0.0532413983],[0,502371,0.0436685369],[0,502737,0.148398813],[0,502852,0.0997042865],[0,502872,0.0610466817],[0,502933,0.0444742848],[0,502971,0.0397915242],[0,503041,0.0742312568],[0,503130,0.1301040804],[0,503319,0.1927610304],[0,503750,0.0842901559],[0,503861,0.0543305324],[0,503867,0.0708073259],[0,503972,0.052828167],[0,503977,0.0521193343],[0,504026,0.1373583317],[0,504097,0.0620784805],[0,504157,0.038092482],[0,504163,0.0756183447],[0,504552,0.0532413983],[0,504596,0.0680644393],[0,504676,0.0498641244],[0,504712,0.0395030712],[0,504755,0.0776675273],[0,504847,0.0484857669],[0,504868,0.121028143],[0,504988,0.0452782944],[0,505029,0.1932317611],[0,505250,0.0397915242],[0,505290,0.0639908567],[0,505292,0.1362664195],[0,505295,0.0477165602],[0,505364,0.038380361],[0,505427,0.0398418409],[0,505463,0.0393346838],[0,505577,0.052828167],[0,505597,0.0444742848],[0,505784,0.0644445088],[0,505815,0.038380361],[0,505957,0.0444742848],[0,506000,0.039530432],[0,506003,0.086868242],[0,506173,0.0515855954],[0,506211,0.13045597],[0,506315,0.2329482703],[0,506318,0.0420965551],[0,506358,0.0608545387],[0,506508,0.0411385127],[0,506548,0.0408852844],[0,506634,0.0453280294],[0,506857,0.0423887446],[0,506932,0.3910918251],[0,506993,0.0425531474],[0,507183,0.3018632774],[0,507201,0.0520661109],[0,507257,0.2421061792],[0,507313,0.0790760384],[0,507387,0.046301175],[0,507418,0.0374672785],[0,507428,0.175276254],[0,507512,0.104009734],[0,507523,0.0740633444],[0,507544,0.0961313014],[0,507735,0.1011379799],[0,507781,0.0404558236],[0,508210,0.0566805396],[0,508264,0.0507848792],[0,508311,0.0394610151],[0,508549,0.0392838389],[0,508623,0.0564732811],[0,508633,0.1281669705],[0,508852,0.0389268728],[0,508860,0.0453280294],[0,509058,0.2138661641],[0,509118,0.0837681538],[0,509249,0.0749275707],[0,509273,0.0589866341],[0,509456,0.0468258911],[0,509524,0.1514957468],[0,509536,0.0407440648],[0,509882,0.052265807],[0,510031,0.0811749872],[0,510035,0.0922286192],[0,510132,0.3770034458],[0,510175,0.0765251513],[0,510317,0.0642312135],[0,510333,0.0983850529],[0,510427,0.0664771478],[0,510453,0.0513913374],[0,510794,0.0872551821],[0,510996,0.1156989039],[0,511051,0.2594156767],[0,511463,0.0420047811],[0,511824,0.0435463623],[0,511886,0.04350804],[0,511896,0.2380730312],[0,511897,0.0478121438],[0,511915,0.0468464039],[0,511977,0.0517441357],[0,511978,0.2441236421],[0,511982,0.0409858873],[0,512009,0.5628228185],[0,512095,0.0430758427],[0,512160,0.039572347],[0,512213,0.0398245607],[0,512244,0.0412890233],[0,512284,0.3568088337],[0,512288,0.811405936],[0,512322,0.0558297091],[0,512375,0.1372520414],[0,512477,0.0500813521],[0,512485,0.0478121438],[0,512552,0.0484452705],[0,512633,0.0478121438],[0,512715,0.0620059163],[0,512768,0.0398245607],[0,512824,0.1580685533],[0,512833,0.362310648],[0,512969,0.0454142693],[0,513059,0.3705938843],[0,513131,0.05323633],[0,513159,0.2655063366],[0,513337,0.1478891966],[0,513440,0.0477115306],[0,513449,0.09516895],[0,513520,0.0395360943],[0,514113,0.1370351863],[0,514229,0.7902758205],[0,514253,0.1264630286],[0,514264,0.5407095436],[0,514318,0.058941668],[0,514576,0.050730743],[0,514579,0.0477115306],[0,514582,0.0411545785],[0,514625,0.0481899905],[0,514672,0.06753437],[0,514767,0.0409769789],[0,514783,0.048150846],[0,514848,0.2087069829],[0,515298,0.0432180751],[0,515462,0.0444692779],[0,515537,0.8581088503],[0,515612,0.0444692779],[0,515767,0.0529244321],[0,516353,0.0571861591],[0,516367,0.05323633],[0,516375,0.0687278033],[0,516399,0.101972915],[0,516430,0.0455847497],[0,516509,0.0424365536],[0,516775,0.0409769789],[0,516925,0.0606856463],[0,516989,0.1316829051],[0,517091,0.0413253657],[0,517193,0.063401254],[0,517259,0.0439263169],[0,517300,0.5030169129],[0,517354,0.109968723],[0,517382,0.0410133053],[0,517581,0.0713425893],[0,517684,0.0409769789],[0,517707,0.0509973052],[0,517722,0.1355981736],[0,517900,0.1025482918],[0,517919,0.0433888093],[0,518254,0.0410774861],[0,518627,0.563537777],[0,518656,0.0375744966],[0,518658,0.0498028759],[0,518715,0.0661920338],[0,518728,0.1448065671],[0,518734,0.0361677456],[0,518785,0.0364548412],[0,518881,0.0417649928],[0,519014,0.0490189863],[0,519100,0.0564474003],[0,519196,0.0444044662],[0,519314,0.0567961971],[0,519364,0.0376017829],[0,519717,0.0573363977],[0,519720,0.0393227611],[0,519722,0.0416191922],[0,519872,0.0379123445],[0,519891,0.0420179867],[0,519912,0.1536479054],[0,520138,0.1144504547],[0,520143,0.0975791302],[0,520287,0.0443043311],[0,520315,0.0443043311],[0,520316,0.040002152],[0,520434,0.0398322291],[0,520604,0.0860209837],[0,521201,0.2339863629],[0,521328,0.057059796],[0,521439,0.0444044662],[0,521468,0.4734594797],[0,521743,0.1282519373],[0,521746,0.1485658708],[0,521921,0.0444563494],[0,521993,0.0444044662],[0,522076,0.0501230694],[0,522237,0.0684018076],[0,522348,0.5590883551],[0,522636,0.0375744966],[0,522731,0.0364548412],[0,522755,0.0410774861],[0,522774,0.0735936366],[0,522821,0.075518169],[0,522997,0.2414943561],[0,523059,0.0625723176],[0,523134,0.0931893907],[0,523182,0.0709166687],[0,523339,0.0607320358],[0,523400,0.0364548412],[0,523638,0.0506916081],[0,523641,0.040002152],[0,523645,0.0415334242],[0,523662,0.0454816207],[0,523835,0.0444044662],[0,524241,0.036203826],[0,524417,0.1468672679],[0,524441,0.039868497],[0,524496,0.1097127813],[0,524524,0.040002152],[0,524556,0.0414040144],[0,524601,0.04600689],[0,524606,0.0638119334],[0,524608,0.660347693],[0,524615,0.1832042709],[0,524690,0.0603040771],[0,524835,0.0553024063],[0,524902,0.0498028759],[0,525174,0.0364548412],[0,525607,0.1121897258],[0,525611,0.0660573481],[0,525967,0.0484291059],[0,526271,0.0410774861],[0,526285,0.4036427301],[0,526788,0.0809909739],[0,526899,0.0733200517],[0,526917,0.0898323896],[0,527434,0.0516082583],[0,527441,0.0444044662],[0,527445,0.1072687921],[0,527469,0.0472467254],[0,527510,0.0410774861],[0,527541,0.1312745709],[0,527568,0.0498028759],[0,527595,0.0498028759],[0,527604,0.0499037826],[0,527670,0.0374065673],[0,527832,0.0432911018],[0,527833,0.0395246202],[0,527856,0.0376017829],[0,527884,0.0929937445],[0,528057,0.110103049],[0,528079,0.0679948443],[0,528108,0.0364548412],[0,528116,0.1056710893],[0,528269,0.0410774861],[0,528349,0.0609166685],[0,528373,0.0376017829],[0,528374,0.043296633],[0,528545,0.1071329554],[0,528974,0.0524498401],[0,529041,0.0462644061],[0,529248,0.053238992],[0,529256,0.2134113492],[0,529266,0.0382687992],[0,529272,0.0434433156],[0,529590,0.0552137547],[0,529901,0.0415114313],[0,529968,0.0639608738],[0,530003,0.0465090335],[0,530057,0.0948194319],[0,530063,0.0416489134],[0,530065,0.0506526018],[0,530066,0.0506526018],[0,530108,0.0502991629],[0,530139,0.0729142066],[0,530175,0.0376017829],[0,530413,0.0506526018],[0,530414,0.0492368077],[0,530796,0.0385796538],[0,530807,0.1525981075],[0,530978,0.0397269333],[0,531406,0.0457804483],[0,531578,0.0520532224],[0,531625,0.0508061584],[0,531628,0.0567568122],[0,531902,0.0498028759],[0,531986,0.1860710695],[0,532090,0.0932143051],[0,532108,0.0439945266],[0,532229,0.0376017829],[0,532244,0.0414517841],[0,532350,0.0378621647],[0,532521,0.0701163381],[0,532543,0.3085918884],[0,532730,0.1723462567],[0,532751,0.2367760994],[0,532783,0.3725681412],[0,532870,0.0506526018],[0,532871,0.0455563954],[0,532874,0.0455563954],[0,533033,0.2503924906],[0,533043,0.0529534524],[0,533058,0.0364548412],[0,533088,0.0606430381],[0,533207,0.1187701037],[0,533286,0.1432852302],[0,533313,0.1822487458],[0,533327,0.0444044662],[0,533381,0.0467988722],[0,533410,0.4813338127],[0,533475,0.0693619922],[0,533499,0.0410774861],[0,533538,0.0467362039],[0,533838,0.120171484],[0,533843,0.0884686497],[0,533850,0.0668195377],[0,533955,0.2618904904],[0,533972,0.2908241836],[0,533988,0.2604446433],[0,534110,0.0557477362],[0,534320,0.0475807136],[0,534321,0.0424246592],[0,534323,0.0648807559],[0,534336,0.0406714333],[0,534478,0.0429500389],[0,534553,0.0361972991],[0,534925,0.0510597044],[0,534968,0.2186897431],[0,535046,0.0852491644],[0,535173,0.1135470689],[0,535248,0.0396298389],[0,535565,0.0537079146],[0,535567,0.050908758],[0,535693,0.1619136661],[0,535695,0.0800674617],[0,535754,0.041990847],[0,535757,0.0429500389],[0,535797,0.5371007878],[0,535932,0.1119769517],[0,536273,0.0439249163],[0,536349,0.0495782161],[0,536457,0.049190956],[0,536524,0.0639126354],[0,537027,0.0429500389],[0,537108,0.0798891247],[0,537157,0.0364711477],[0,537241,0.0433496283],[0,537266,0.0385566963],[0,537323,0.1182813319],[0,537361,0.0513208909],[0,537792,0.0573518847],[0,537813,0.06066877],[0,537882,0.2728110663],[0,537974,0.2407188219],[0,537988,0.0646771709],[0,538029,0.5085598874],[0,538036,0.1146581227],[0,538038,0.0673751739],[0,538040,0.469859994],[0,538110,0.0402557413],[0,538116,0.0683397526],[0,538119,0.0642406403],[0,538122,0.038592899],[0,538147,0.0350166147],[0,538199,0.0601434075],[0,538205,0.1064833623],[0,538210,0.0855463404],[0,538220,0.0448257653],[0,538224,0.055128092],[0,538227,0.1173469451],[0,538231,0.0513208909],[0,538281,0.0755534459],[0,538326,0.1013843664],[0,538338,0.0350166147],[0,538586,0.0464457913],[0,538603,0.0350166147],[0,538625,0.14421787],[0,538681,0.0443427757],[0,538686,0.0973153405],[0,538691,0.2922422061],[0,538709,0.0616343453],[0,538711,0.0440614805],[0,538736,0.0464457913],[0,538917,0.8039605838],[0,538923,0.0401624397],[0,539124,0.0419130166],[0,539145,0.0610176259],[0,539246,0.1988626704],[0,539665,0.0635712489],[0,539696,0.0430366789],[0,539713,0.0396298389],[0,539763,0.041990847],[0,539777,0.0513208909],[0,539827,0.0359664013],[0,539836,0.1031253909],[0,539869,0.4944399468],[0,539881,0.0603430571],[0,540012,0.0429500389],[0,540081,0.3914409673],[0,540115,0.0561892064],[0,540137,0.1290997874],[0,540252,0.0364711477],[0,540290,0.0866971166],[0,540308,0.0428501079],[0,540400,0.0396298389],[0,540431,0.0513208909],[0,540438,0.0969444458],[0,540700,0.0440996205],[0,540701,0.0426967135],[0,540702,0.0426967135],[0,540703,0.0427747826],[0,540704,0.0433143794],[0,540705,0.0426967135],[0,540709,0.0440996205],[0,540710,0.0440996205],[0,540711,0.0393776945],[0,540860,0.0620851725],[0,540882,0.3385036123],[0,541061,0.0449642007],[0,541202,0.0428501079],[0,541359,0.0668135831],[0,541368,0.0428501079],[0,541658,0.0755994915],[0,541688,0.1089627284],[0,542003,0.0451123429],[0,542877,0.2206598633],[0,543004,0.0518085986],[0,543063,0.0906499286],[0,543177,0.0865771221],[0,543201,0.0670880726],[0,543219,0.0518085986],[0,543223,0.0656543302],[0,543224,0.0356795044],[0,543228,0.0752095017],[0,543325,0.0360622405],[0,543526,0.0429500389],[0,543727,0.0524089444],[0,543750,0.093454428],[0,543847,0.0478688147],[0,544006,0.0468277267],[0,544007,0.1175593075],[0,544021,0.036161219],[0,544022,0.0364711477],[0,544057,0.0364711477],[0,544084,0.0662792408],[0,544119,0.0650163415],[0,544341,0.0347301042],[0,544356,0.1117701011],[0,544362,0.0396298389],[0,544442,0.0658010815],[0,544446,0.0987333294],[0,544577,0.0384573817],[0,544580,0.052057182],[0,544589,0.0415074514],[0,544598,0.0561684459],[0,544601,0.0651691354],[0,544609,0.0592524999],[0,544653,0.0453395652],[0,544786,0.3483998095],[0,544806,0.1830659171],[0,544813,0.0396298389],[0,545131,0.2744574392],[0,545294,0.1403412002],[0,546008,0.036448312],[0,546019,0.0710901153],[0,547181,0.0525551192],[0,548957,0.0449337012],[0,549114,0.061201349],[0,549239,0.0530018119],[0,549241,0.0643137261],[0,549248,0.626552308],[0,549274,0.0457056323],[0,549373,0.0678180854],[0,549383,0.0533278676],[0,549554,0.0380986667],[0,549582,0.1253222415],[0,549687,0.0438937906],[0,549697,0.037930613],[0,549718,0.0860040292],[0,550107,0.0384367652],[0,550186,0.0533278676],[0,550193,0.0449337012],[0,550215,0.0677156926],[0,550272,0.0687185693],[0,550364,0.0533278676],[0,550377,0.0399646579],[0,550378,0.0528217704],[0,550484,0.3156715104],[0,550605,0.0479625752],[0,550672,0.0426148695],[0,550776,0.0636941905],[0,550880,0.1324085308],[0,550947,0.0696879294],[0,551026,0.0931794152],[0,551031,0.0409226796],[0,551108,0.0439225131],[0,551261,0.1236070457],[0,551267,0.0614581484],[0,551360,0.504624893],[0,551370,0.0848457213],[0,551389,0.0417855198],[0,551540,0.0649378449],[0,551653,0.0619527021],[0,551663,0.0384367652],[0,551691,0.137776502],[0,551761,0.0427020221],[0,551771,0.0380986667],[0,551815,0.0401890249],[0,551895,0.0381621539],[0,552084,0.5169463147],[0,552180,0.0369781811],[0,552185,0.0406468745],[0,552441,0.0367269798],[0,552443,0.0829394662],[0,552453,0.6277635054],[0,552468,0.2233560387],[0,552480,0.0448334919],[0,552567,0.038134846],[0,552568,0.0369781811],[0,552675,0.042038521],[0,552745,0.0533278676],[0,553035,0.0384138659],[0,553109,0.0449337012],[0,553170,0.0448334919],[0,553218,0.053504785],[0,553336,0.041604254],[0,553402,0.157682085],[0,553443,0.076499956],[0,553543,0.0381621539],[0,553548,0.0786099505],[0,553928,0.0662865541],[0,554084,0.077496773],[0,554366,0.0449337012],[0,554367,0.1257770923],[0,554385,0.0389558108],[0,554411,0.2340234352],[0,554484,0.041604254],[0,554552,0.0460864847],[0,554582,0.0448334919],[0,554664,0.3374630476],[0,554674,0.0530756541],[0,554834,0.041604254],[0,554848,0.1030060492],[0,555063,0.0448334919],[0,555290,0.0565707634],[0,555693,0.0369781811],[0,555932,0.0504235708],[0,556106,0.0645269265],[0,556111,0.1095585],[0,556135,0.0662416566],[0,556351,0.2267067161],[0,556361,0.0381259733],[0,556389,0.0415603234],[0,556640,0.1432343926],[0,556641,0.0384367652],[0,556642,0.0549419328],[0,556696,0.041604254],[0,556808,0.0449237421],[0,556823,0.053504785],[0,556848,0.0417855198],[0,556860,0.1165766961],[0,556874,0.4401995764],[0,557072,0.0533278676],[0,557194,0.1254246085],[0,557365,0.0721662791],[0,557375,0.040682637],[0,557473,0.0449337012],[0,557523,0.0564825394],[0,557530,0.3109061203],[0,557538,0.0449337012],[0,557667,0.0464671882],[0,558412,0.0491045343],[0,558520,0.0384138659],[0,558537,0.1160635642],[0,558593,0.0406468745],[0,558650,0.0522195361],[0,558770,0.1044412188],[0,558810,0.0661740212],[0,558850,0.0841819375],[0,558868,0.219908654],[0,558960,0.0746384861],[0,558969,0.0428129504],[0,558979,0.0386974325],[0,559083,0.0456109643],[0,559087,0.0373603191],[0,559103,0.129240702],[0,559105,0.1250677907],[0,559162,0.0438502783],[0,559450,0.0776885204],[0,559504,0.0472977925],[0,559631,0.0424233971],[0,559780,0.0376479001],[0,559790,0.1088325249],[0,559837,0.5298745993],[0,559882,0.0413200725],[0,560010,0.3523005784],[0,560415,0.0693130696],[0,560426,0.0740082688],[0,560576,0.0907096027],[0,560577,0.1024629118],[0,560586,0.1611299931],[0,560610,0.0835254907],[0,560772,0.0741634825],[0,560921,0.0750049932],[0,560931,0.0553223819],[0,560988,0.0590247584],[0,561183,0.0748498458],[0,561256,0.1140189163],[0,561257,0.074079564],[0,561258,0.0892848908],[0,561594,0.0474709243],[0,562015,0.0470333407],[0,562025,0.0534091021],[0,562031,0.053211411],[0,562032,0.0594224934],[0,562034,0.2709038536],[0,562036,0.0474709243],[0,562123,0.3207709111],[0,562135,0.0489329149],[0,562226,0.1348889696],[0,562278,0.0460686496],[0,562305,0.0781119425],[0,562338,0.0438185871],[0,562464,0.046778563],[0,562576,0.0550712464],[0,562672,0.0494150509],[0,562712,0.0470333407],[0,562714,0.0588464898],[0,562766,0.0450904958],[0,562776,0.0528026973],[0,562807,0.0609609134],[0,562998,0.054342844],[0,563076,0.0509660105],[0,563116,0.056180023],[0,563254,0.0588464898],[0,563609,0.051123856],[0,563733,0.0797376975],[0,563747,0.0750581335],[0,563817,0.0438185871],[0,563851,0.047652191],[0,564205,0.0507677343],[0,564277,0.1514115555],[0,564293,0.1480255593],[0,564434,0.0644991623],[0,564574,0.0588464898],[0,564773,0.0722808516],[0,564786,0.0752185253],[0,564957,0.0493142128],[0,564961,0.0649347883],[0,565009,0.0794848147],[0,565044,0.0529040249],[0,565105,0.0524554456],[0,565471,0.0993280341],[0,565520,0.1938239997],[0,565526,0.07607166],[0,565581,0.0495889785],[0,565664,0.0493142128],[0,565666,0.0588464898],[0,565667,0.757628844],[0,565849,0.0829272916],[0,566000,0.1394619737],[0,566004,0.0593566604],[0,566179,0.0588464898],[0,566195,0.0509228749],[0,566202,0.0512635793],[0,566311,0.098124742],[0,566320,0.1601711028],[0,566447,0.050730909],[0,566543,0.6828379695],[0,566659,0.3136541548],[0,566675,0.0493142128],[0,566682,0.0494150509],[0,566700,0.0609842732],[0,566701,0.0496456708],[0,566703,0.2120137085],[0,567154,0.0533884185],[0,567323,0.2408580145],[0,567354,0.0463937337],[0,567392,0.0612868168],[0,567407,0.0762039985],[0,567533,0.0435009807],[0,567540,0.0473212392],[0,567544,0.0452354689],[0,567553,0.0679102559],[0,567693,0.0433316435],[0,567708,0.0425646072],[0,567755,0.1140999148],[0,567842,0.0588464898],[0,568100,0.0572001902],[0,568124,0.052210783],[0,568133,0.0435649528],[0,568494,0.0743053602],[0,568576,0.0590247584],[0,568924,0.076642682],[0,568939,0.0614557411],[0,569028,0.5035772554],[0,569203,0.5739719713],[0,569222,0.0494150509],[0,569572,0.0588464898],[0,569672,0.0667775257],[0,569683,0.0734008206],[0,570303,0.3579932965],[0,571124,0.0495775297],[0,571306,0.0481528197],[0,571353,0.1095213239],[0,571410,0.1218177921],[0,571425,0.0470333407],[0,571586,0.0595670011],[0,571821,0.0493142128],[0,571885,0.0435009807],[0,572234,0.3421808267],[0,572770,0.1219384641],[0,572834,0.0707658366],[0,572867,0.6818253397],[0,573067,0.0478319351],[0,573104,0.1733753149],[0,573268,0.051482581],[0,573873,0.2763348958],[0,573926,0.1048776658],[0,574026,0.0449054725],[0,574046,0.0474709243],[0,574086,0.0470333407],[0,574093,0.0852705392],[0,574109,0.0591868985],[0,574129,0.0973583572],[0,574143,0.0632561356],[0,574147,0.0528863049],[0,574180,0.0451270327],[0,574301,0.061385825],[0,574305,0.0882611388],[0,574315,0.0744207846],[0,574355,0.0666732838],[0,574587,0.1059121356],[0,574613,0.0579643306],[0,574676,0.0690219643],[0,574687,0.0689975118],[0,574834,0.0472015156],[0,574835,0.0798241194],[0,575063,0.0452354689],[0,575094,0.0473469171],[0,575233,0.0561899966],[0,575341,0.1226139443],[0,575344,0.0683225384],[0,575399,0.0927266776],[0,575776,0.0494150509],[0,576540,0.0493332918],[0,577021,0.0433316435],[0,577033,0.0438185871],[0,577180,0.4662834444],[0,577328,0.2363715013],[0,577381,0.0605851523],[0,578186,0.0470333407],[0,578276,0.0433316435],[0,578291,0.3282625357],[0,578300,0.7283617171],[0,578324,0.0589225719],[0,578456,0.1478684484],[0,578469,0.0599813818],[0,578604,0.0593392909],[0,578630,0.0669720196],[0,578634,0.0454797456],[0,578856,0.0452354689],[0,578900,0.0644363496],[0,579018,0.0586942004],[0,579121,0.3015067247],[0,579129,0.0470333407],[0,579136,0.0788537626],[0,579239,0.0592738615],[0,579277,0.0441604702],[0,579543,0.0977847722],[0,579547,0.0486217879],[0,579685,0.0476723683],[0,579709,0.0529679259],[0,579732,0.0436772169],[0,579872,0.0451265197],[0,579937,0.0486392221],[0,579943,0.049273079],[0,579955,0.0482274926],[0,580033,0.0451265197],[0,580063,0.0658680913],[0,580231,0.0523454143],[0,580253,0.5257409855],[0,580543,0.0533348657],[0,580614,0.0557650665],[0,580801,0.0534492467],[0,580821,0.6238110979],[0,580968,0.0464470587],[0,581017,0.1620711724],[0,581144,0.065115139],[0,581365,0.0472182785],[0,581417,0.6317045403],[0,581684,0.0628177449],[0,581738,0.0798785238],[0,582400,0.2776186772],[0,582493,0.0513801356],[0,582497,0.0997068842],[0,582501,0.0652364459],[0,582576,0.0612011329],[0,582580,0.0595345559],[0,582598,0.0535667433],[0,582689,0.0509401653],[0,582709,0.1552762417],[0,582874,0.0462533383],[0,582888,0.0626679829],[0,582904,0.1363267447],[0,582910,0.1189352128],[0,582951,0.080765432],[0,583054,0.0632738909],[0,583073,0.0628177449],[0,583093,0.0839639737],[0,583455,0.0822323349],[0,583489,0.0590219109],[0,583501,0.1953604083],[0,583756,0.0532334776],[0,583783,0.1273590987],[0,583795,0.7839170643],[0,583936,0.0474528603],[0,583939,0.2819954745],[0,584191,0.0509401653],[0,584372,0.0633132338],[0,584430,0.0462533383],[0,584535,0.0508808029],[0,584795,0.0681238138],[0,584949,0.0734577361],[0,585026,0.0958781981],[0,585131,0.0631600102],[0,585324,0.1056873608],[0,585441,0.4665720786],[0,585556,0.1839883678],[0,585760,0.2262934148],[0,586085,0.3645875317],[0,586130,0.4123219973],[0,586133,0.064091357],[0,586175,0.0611083998],[0,586182,0.0509401653],[0,586252,0.2318817881],[0,586256,0.0618322188],[0,586267,0.0786156451],[0,586279,0.1455665328],[0,586283,0.4108346316],[0,586381,0.062920478],[0,586395,0.0511238117],[0,586402,0.0511238117],[0,586408,0.4145874474],[0,586411,0.0628177449],[0,586502,0.0645542091],[0,586531,0.0533348657],[0,586538,0.0592593701],[0,586540,0.0681348399],[0,586552,0.0574248036],[0,586632,0.068643197],[0,586668,0.0585492598],[0,586741,0.0628177449],[0,587360,0.0490332953],[0,587566,0.0848227865],[0,587579,0.0520509889],[0,587602,0.0690209068],[0,587851,0.0905080438],[0,587943,0.0789333741],[0,587955,0.8646293028],[0,588015,0.3294173201],[0,588105,0.045998838],[0,588144,0.0848734505],[0,588179,0.4811598526],[0,588317,0.0795125882],[0,588320,0.0628177449],[0,588400,0.057240571],[0,588420,0.0731608582],[0,588541,0.048247524],[0,588555,0.5074773794],[0,588824,0.0631621199],[0,588862,0.0533348657],[0,588941,0.0549879878],[0,589077,0.2805097307],[0,589141,0.0550159463],[0,589159,0.0472182785],[0,589182,0.0462533383],[0,589191,0.0631600102],[0,589210,0.0629969858],[0,589297,0.0906905443],[0,589514,0.2906094972],[0,589764,0.0856784071],[0,589777,0.1050608322],[0,589790,0.107948013],[0,589849,0.0511461021],[0,590057,0.1544372435],[0,590211,0.0574287979],[0,590231,0.1751388324],[0,590259,0.2560564951],[0,590277,0.104396373],[0,590289,0.051229634],[0,590680,0.0644128711],[0,590745,0.13640064],[0,590895,0.0781259232],[0,590985,0.0462533383],[0,591007,0.10838701],[0,591046,0.0533348657],[0,591305,0.0567409887],[0,591329,0.0614543007],[0,591330,0.0717988357],[0,591378,0.0591241113],[0,591381,0.0866221331],[0,591382,0.6982361401],[0,591384,0.0858510381],[0,591453,0.062894242],[0,591454,0.0634343289],[0,591465,0.0513801356],[0,591476,0.062894242],[0,591493,0.1323098079],[0,591510,0.0618971888],[0,591681,0.0509401653],[0,591695,0.0713589119],[0,591705,0.0718666605],[0,591905,0.114681438],[0,591909,0.1357018383],[0,591911,0.1715363104],[0,591917,0.049749451],[0,591937,0.0757502635],[0,591953,0.0781722197],[0,591956,0.0462533383],[0,592072,0.120088423],[0,592085,0.0975363857],[0,592146,0.0552827428],[0,592153,0.0642665378],[0,592293,0.062894242],[0,592296,0.2227202367],[0,592328,0.0603780683],[0,592339,0.0579661743],[0,592354,0.1383299055],[0,592382,0.0583549903],[0,592402,0.0628177449],[0,592593,0.0511238117],[0,592598,0.0618039972],[0,592609,0.0539728684],[0,592687,0.0499702127],[0,592745,0.0522795629],[0,593157,0.1211411381],[0,593170,0.0633132338],[0,593276,0.0617701065],[0,593587,0.2141155157],[0,593723,0.0668933845],[0,593735,0.0628438284],[0,593937,0.0631600102],[0,593969,0.0467383377],[0,594084,0.0491067457],[0,594491,0.0951499942],[0,594631,0.1618642377],[0,594729,0.0755411273],[0,594785,0.0492790365],[0,595622,0.3180806223],[0,596237,0.2341017595],[0,596245,0.0896369624],[0,596295,0.1212046545],[0,596300,0.2234040822],[0,596572,0.0509401653],[0,596577,0.1120035169],[0,596717,0.5136203326],[0,596829,0.0514812636],[0,596894,0.076196434],[0,596935,0.0628177449],[0,597373,0.0795125882],[0,597385,0.0533348657],[0,597386,0.0474162045],[0,597409,0.0556175846],[0,597414,0.0509401653],[0,597431,0.1221828545],[0,597584,0.0628177449],[0,597644,0.7052025069],[0,597748,0.0464470587],[0,597896,0.1437756338],[0,597904,0.0533348657],[0,597933,0.0983888261],[0,598283,0.0538239364],[0,598286,0.2821817786],[0,598320,0.1615524032],[0,598448,0.0682804446],[0,598552,0.0472182785],[0,598600,0.0452714448],[0,598603,0.0487508999],[0,598644,0.2342701729],[0,598652,0.046635315],[0,598814,0.0540069595],[0,598817,0.072830747],[0,598962,0.0466963499],[0,599161,0.0487508999],[0,599285,0.0599334695],[0,599309,0.0795002092],[0,599432,0.0460668974],[0,599477,0.0540311001],[0,599487,0.0523432544],[0,599503,0.079186238],[0,599507,0.3845429806],[0,599786,0.046225862],[0,600278,0.3356822308],[0,600321,0.1227536063],[0,600327,0.0470355284],[0,600336,0.0465266663],[0,600353,0.0673554711],[0,600744,0.0552803322],[0,600887,0.0452714448],[0,600906,0.046635315],[0,600966,0.0622729112],[0,601050,0.0711212786],[0,601129,0.0471198514],[0,601301,0.0499518402],[0,601343,0.0618131205],[0,601396,0.295327235],[0,601423,0.0506698706],[0,601464,0.055211027],[0,601605,0.3420435211],[0,601732,0.0627518722],[0,601915,0.0452714448],[0,601954,0.0766915172],[0,602022,0.0618131205],[0,602080,0.1042599895],[0,602082,0.0779040683],[0,602136,0.0624284677],[0,602267,0.0621549162],[0,602306,0.4511079572],[0,602334,0.0499518402],[0,602437,0.060887443],[0,602456,0.0641274611],[0,602656,0.0791392303],[0,602800,0.0782428534],[0,602805,0.0489832186],[0,602908,0.0522420054],[0,603131,0.1746868579],[0,603163,0.315678872],[0,603466,0.0580830587],[0,603719,0.1329591094],[0,604116,0.0522316796],[0,604119,0.0914874733],[0,604122,0.1920331369],[0,604248,0.0653420602],[0,604251,0.0933409227],[0,604355,0.0522420054],[0,604370,0.0499518402],[0,604398,0.082524141],[0,604551,0.2814025955],[0,604571,0.0503308143],[0,604594,0.0844180297],[0,604720,0.0633247365],[0,604826,0.1912293912],[0,604897,0.0581245555],[0,604938,0.3879962212],[0,604971,0.3692235718],[0,604997,0.0997850486],[0,605008,0.305657813],[0,605087,0.1025277466],[0,605123,0.0741687381],[0,605314,0.0799868091],[0,605339,0.052831654],[0,605457,0.0522420054],[0,605503,0.0639530035],[0,605609,0.0769047028],[0,605694,0.0449249977],[0,605818,0.23813002],[0,605841,0.0460668974],[0,605855,0.0499518402],[0,605921,0.1186950141],[0,606253,0.130479229],[0,606258,0.1018083964],[0,606507,0.0486150432],[0,606537,0.0639723858],[0,606556,0.0449807625],[0,606649,0.0655817852],[0,606824,0.0760195158],[0,606827,0.0523432544],[0,606869,0.1435442738],[0,606951,0.2534575794],[0,606985,0.0618131205],[0,607020,0.0515223047],[0,607143,0.383110448],[0,607158,0.0617547655],[0,607199,0.11675268],[0,607272,0.0598528389],[0,607284,0.2102542555],[0,607287,0.2449737436],[0,607377,0.2750288753],[0,607465,0.1415337356],[0,607488,0.0663420327],[0,607567,0.0523432544],[0,607619,0.062988603],[0,607634,0.0618131205],[0,607744,0.0544964851],[0,607753,0.0503912067],[0,608077,0.1318819076],[0,608096,0.0502831577],[0,608128,0.1002969614],[0,608246,0.4641495582],[0,608264,0.0523432544],[0,608266,0.2066093824],[0,608270,0.0618817995],[0,608358,0.0618131205],[0,608361,0.282067941],[0,608420,0.0626555415],[0,608452,0.0869723623],[0,608551,0.0737358068],[0,608579,0.0499518402],[0,608699,0.0523432544],[0,608724,0.0525352385],[0,608726,0.0460668974],[0,608729,0.0570610313],[0,608764,0.0786532126],[0,608879,0.1685350937],[0,608904,0.0554300709],[0,609121,0.0515223047],[0,609219,0.1561837219],[0,609385,0.0452714448],[0,609423,0.0515223047],[0,609505,0.0542180258],[0,609943,0.1553311411],[0,610007,0.0918981992],[0,610025,0.076400253],[0,611120,0.0542267764],[0,611178,0.0523432544],[0,611239,0.0556015319],[0,611506,0.0618391682],[0,611662,0.0499518402],[0,611728,0.1327872829],[0,611795,0.0500984416],[0,611850,0.0628309081],[0,611860,0.2300461732],[0,611885,0.0736630015],[0,612036,0.4787760565],[0,612085,0.1863987692],[0,612294,0.3058131839],[0,612318,0.046635315],[0,612636,0.1608133895],[0,612669,0.0762847013],[0,612688,0.0487277473],[0,613024,0.0544964851],[0,613068,0.5329915379],[0,613078,0.0642404952],[0,613080,0.3121155978],[0,613114,0.0613555919],[0,613121,0.046635315],[0,613137,0.145469482],[0,613142,0.0952255022],[0,613174,0.0473270469],[0,613176,0.0501829331],[0,613177,0.2886077947],[0,613186,0.1847253612],[0,613188,0.0552052196],[0,613201,0.0619921155],[0,613202,0.1254280714],[0,613286,0.1308021756],[0,613302,0.0618131205],[0,613305,0.0652340419],[0,613312,0.0618131205],[0,613437,0.0850216647],[0,613460,0.2718298908],[0,613501,0.0500984416],[0,613586,0.1781329883],[0,613652,0.0759067181],[0,613667,0.0533641685],[0,613811,0.128561863],[0,613878,0.0500984416],[0,613895,0.1429964601],[0,613917,0.2426570434],[0,613928,0.0654775565],[0,614018,0.0675914573],[0,614107,0.1150843624],[0,614130,0.074919059],[0,614295,0.0610978845],[0,614298,0.050586856],[0,614630,0.0527693428],[0,614656,0.0791715835],[0,614672,0.0800216546],[0,614673,0.1086825982],[0,614752,0.4521763926],[0,614865,0.0615748054],[0,614907,0.0529194423],[0,614999,0.0460668974],[0,615011,0.0552052196],[0,615051,0.0915007026],[0,615057,0.0755470051],[0,615079,0.0467813545],[0,615160,0.0522420054],[0,615354,0.0518591539],[0,615368,0.046441692],[0,615407,0.0722855111],[0,615535,0.0522420054],[0,615699,0.2790034453],[0,615700,0.1273550915],[0,615774,0.0756708383],[0,615844,0.0571992479],[0,615883,0.0757028153],[0,616280,0.0917958496],[0,616325,0.0856251028],[0,616370,0.0766498391],[0,616442,0.0988202631],[0,616536,0.1152460526],[0,617035,0.1447949521],[0,617149,0.1277673164],[0,617151,0.2554408278],[0,617163,0.0753668725],[0,617168,0.3275737591],[0,617170,0.1739133335],[0,617250,0.0734473899],[0,617546,0.046635315],[0,617548,0.0691506156],[0,617562,0.1600757382],[0,617606,0.0618391682],[0,617773,0.0522611621],[0,617812,0.0717151286],[0,618011,0.0622575562],[0,618023,0.0464998586],[0,618026,0.0632599253],[0,618049,0.0624436],[0,618189,0.0526191783],[0,618195,0.1440179021],[0,618248,0.0821884161],[0,618289,0.0496627697],[0,618347,0.0514145336],[0,618424,0.0509745424],[0,618647,0.0704066607],[0,618649,0.0583416552],[0,618812,0.0525841582],[0,618969,0.0460329792],[0,619019,0.0977595117],[0,619045,0.0477995484],[0,619240,0.0674331668],[0,619451,0.0618589982],[0,619457,0.0633482015],[0,619660,0.0729056432],[0,619684,0.3493257987],[0,619818,0.0477995484],[0,619951,0.0623846941],[0,620123,0.0525472401],[0,620435,0.0812677],[0,620611,0.0698268291],[0,620614,0.0497718945],[0,620667,0.3908053438],[0,620670,0.1933893535],[0,620672,0.0509745424],[0,620745,0.1420513659],[0,620758,0.6044864824],[0,620767,0.0836744803],[0,620772,0.0459771323],[0,620774,0.051205964],[0,620841,0.0450099325],[0,620866,0.2325750957],[0,620878,0.0472524779],[0,620969,0.0511581975],[0,620989,0.3186247149],[0,620991,0.0475176522],[0,621057,0.0532679641],[0,621112,0.053369357],[0,621124,0.3213608905],[0,621140,0.0509745424],[0,621157,0.3855202714],[0,621283,0.0593775451],[0,621509,0.0532679641],[0,621637,0.0514145336],[0,621742,0.0509745424],[0,621746,0.0628787737],[0,621751,0.092323934],[0,621771,0.0509745424],[0,621777,0.072988889],[0,621789,0.0460329792],[0,621911,0.1611476601],[0,621939,0.0628526889],[0,621951,0.0550225581],[0,622016,0.178105173],[0,622168,0.0596431757],[0,622226,0.1993904028],[0,622410,0.053369357],[0,622420,0.0765026405],[0,622429,0.143360119],[0,622499,0.052724727],[0,622503,0.0614110873],[0,622519,0.0783132715],[0,622537,0.0672247312],[0,622539,0.0628526889],[0,622564,0.0568986179],[0,622682,0.1378238146],[0,622694,0.1522491722],[0,622705,0.0470840754],[0,622789,0.1104878819],[0,622934,0.0509745424],[0,622939,0.0581641278],[0,622948,0.2802676693],[0,622973,0.0459771323],[0,622974,0.0592361835],[0,623001,0.0500045434],[0,623005,0.0686667723],[0,623131,0.0743836506],[0,623468,0.4090738793],[0,623995,0.0633482015],[0,624088,0.1105450155],[0,624161,0.0794980628],[0,624165,0.059158879],[0,624166,0.0683643103],[0,624167,0.5111106039],[0,624172,0.076966327],[0,624182,0.2373829764],[0,624401,0.2866226914],[0,624576,0.0629291897],[0,624585,0.1542044447],[0,624606,0.7426370911],[0,624705,0.0768006677],[0,624711,0.0590566737],[0,624714,0.0528845682],[0,624718,0.2403487652],[0,624731,0.0598769206],[0,624859,0.8687861894],[0,624972,0.2777520511],[0,625072,0.0582322556],[0,625097,0.0558204482],[0,625196,0.053369357],[0,625241,0.6718553522],[0,625295,0.0786171328],[0,625303,0.1291138058],[0,625399,0.4360642766],[0,625641,0.05892356],[0,625672,0.0998665074],[0,625679,0.1036142938],[0,625835,0.0630319384],[0,625968,0.0631949706],[0,626057,0.0528055457],[0,626087,0.0469286067],[0,626157,0.0773650884],[0,626170,0.0646564537],[0,626184,0.079161729],[0,626441,0.0628526889],[0,626451,0.0557036993],[0,626537,0.0515156665],[0,626934,0.5022561885],[0,626956,0.0532679641],[0,626980,0.0628526889],[0,626991,0.1953921487],[0,627002,0.1028833517],[0,627033,0.0509745424],[0,627035,0.1936098722],[0,627043,0.0635463534],[0,627048,0.1019840124],[0,627119,0.0600101149],[0,627223,0.0474594029],[0,627499,0.0814003391],[0,627515,0.0509745424],[0,627553,0.080238716],[0,627635,0.2180171506],[0,627658,0.0650070008],[0,627697,0.1417721532],[0,627806,0.1316600674],[0,627905,0.0501224117],[0,627943,0.2468095581],[0,627952,0.2078413482],[0,628184,0.052838701],[0,628254,0.0704876256],[0,628447,0.0871917246],[0,628467,0.0471268427],[0,628492,0.0457620361],[0,628646,0.052838701],[0,628790,0.0598512834],[0,628960,0.1796520457],[0,628990,0.0808490991],[0,629008,0.057667129],[0,629056,0.0470279273],[0,629062,0.0508853131],[0,629067,0.0472017003],[0,629338,0.0533274359],[0,629523,0.0555563638],[0,629530,0.0494763583],[0,629946,0.0504456449],[0,629981,0.0578905409],[0,630243,0.0764220961],[0,630317,0.0755765455],[0,630338,0.0611296055],[0,630386,0.0527565524],[0,631388,0.2198850872],[0,631496,0.5530205895],[0,631649,0.0590924867],[0,631746,0.0521411055],[0,631915,0.1840700314],[0,631973,0.0868166277],[0,631976,0.1002905901],[0,631981,0.2821770104],[0,631988,0.1350854429],[0,631989,0.0454711542],[0,632085,0.070973569],[0,632118,0.548205306],[0,632257,0.0533274359],[0,632259,0.0597911187],[0,632266,0.062315069],[0,632270,0.0527373825],[0,632289,0.0860684993],[0,632297,0.0778211127],[0,632303,0.0539278949],[0,632305,0.0457620361],[0,632315,0.052838701],[0,632322,0.2650989396],[0,632329,0.0591232359],[0,632333,0.1847048748],[0,632336,0.0730744675],[0,632556,0.0549710367],[0,632569,0.0612792773],[0,632574,0.0454519046],[0,632838,0.1156542793],[0,632844,0.0790733607],[0,632848,0.1080956301],[0,632856,0.0568564921],[0,632867,0.0562424853],[0,633051,0.0490128142],[0,633068,0.052838701],[0,633069,0.0457620361],[0,633070,0.0624941868],[0,633098,0.0462480368],[0,633112,0.2286919903],[0,633153,0.062315069],[0,633168,0.416645322],[0,633179,0.1094729669],[0,633182,0.054393866],[0,633185,0.4156688995],[0,633371,0.0469330868],[0,633395,0.0457062103],[0,633399,0.0539278949],[0,633421,0.0471268427],[0,633430,0.1277687571],[0,633653,0.0690677403],[0,633677,0.044448911],[0,633758,0.2942317728],[0,633785,0.1631474359],[0,633795,0.0721301098],[0,633860,0.2139325308],[0,633861,0.0547235162],[0,633881,0.0465946467],[0,634019,0.052838701],[0,634024,0.0467263137],[0,634212,0.1022952582],[0,634222,0.1173433421],[0,634230,0.0954535144],[0,634243,0.0525629337],[0,634248,0.0457620361],[0,634442,0.074372707],[0,634444,0.1035951096],[0,634624,0.0931052609],[0,634651,0.0541857008],[0,634896,0.053030817],[0,635028,0.0471268427],[0,635048,0.0752372611],[0,635059,0.0504456449],[0,635434,0.2340122764],[0,635474,0.052838701],[0,635547,0.0777486876],[0,635682,0.0467263137],[0,635719,0.3216623491],[0,635771,0.0928766113],[0,636088,0.0759564942],[0,636133,0.8146039251],[0,636202,0.0534762657],[0,636211,0.0524917217],[0,636223,0.1303042978],[0,636344,0.0580064058],[0,636537,0.0527373825],[0,636546,0.0592820619],[0,636559,0.0626398721],[0,636565,0.0492438802],[0,636921,0.062315069],[0,637105,0.052838701],[0,637107,0.9141904916],[0,637187,0.0457620361],[0,637416,0.0478750403],[0,637544,0.062315069],[0,637583,0.0767914319],[0,637592,0.1963058133],[0,637597,0.0504456449],[0,637641,0.0647946725],[0,637645,0.0532878194],[0,637688,0.2072913085],[0,637910,0.0808074326],[0,637959,0.1187612055],[0,638032,0.1082587635],[0,638036,0.071314012],[0,638046,0.1789709114],[0,638055,0.0964519903],[0,638060,0.0511641684],[0,638183,0.0623915136],[0,638205,0.1150251384],[0,638393,0.0472017003],[0,638432,0.1661199445],[0,638488,0.5368145197],[0,638516,0.0621620542],[0,638568,0.0469912933],[0,638587,0.0683997276],[0,638614,0.052838701],[0,638626,0.0493904517],[0,638635,0.1625967305],[0,638761,0.0547814453],[0,638824,0.054436377],[0,638827,0.1479892419],[0,638855,0.0661698341],[0,638918,0.0679194366],[0,638953,0.06614851],[0,639079,0.0683836824],[0,639329,0.0649398696],[0,639436,0.1148109808],[0,639492,0.0509768323],[0,639559,0.0509015803],[0,640168,0.6439803195],[0,640169,0.0542378695],[0,640758,0.0547814453],[0,640764,0.0535186368],[0,640767,0.1181756094],[0,641044,0.0544223567],[0,641113,0.0509629789],[0,641239,0.0974756658],[0,641310,0.0507068034],[0,641314,0.0542378695],[0,641400,0.3919094783],[0,641406,0.4022572853],[0,641457,0.0513890042],[0,641483,0.0532634756],[0,641490,0.0761993949],[0,641491,0.0546798542],[0,641509,0.1154493744],[0,641578,0.3729726605],[0,641585,0.2307700904],[0,641711,0.0565416823],[0,641756,0.0661698341],[0,641763,0.0543853445],[0,641862,0.2616098191],[0,641878,0.3058048747],[0,641936,0.2827243248],[0,642264,0.9255329163],[0,642288,0.144939708],[0,642364,0.0946999706],[0,642436,0.072011466],[0,642507,0.11695129],[0,642567,0.0643924078],[0,642656,0.0618269455],[0,642850,0.0616906978],[0,642887,0.3044133692],[0,642891,0.0913581103],[0,642930,0.0565416823],[0,643053,0.0571421217],[0,643197,0.0857159292],[0,643214,0.0642395528],[0,643219,0.2383984054],[0,643240,0.1152489738],[0,643391,0.0492739168],[0,643744,0.2533968588],[0,643759,0.0509015803],[0,643812,0.2423083982],[0,643995,0.0883419399],[0,644131,0.0535186368],[0,644296,0.0697920547],[0,644768,0.0769377224],[0,644879,0.0506699813],[0,645002,0.1055189231],[0,645033,0.0566435347],[0,645124,0.0530297725],[0,645127,0.3174730594],[0,645141,0.0571348448],[0,645146,0.0565416823],[0,645348,0.1684695025],[0,645352,0.1947050076],[0,645580,0.0494734625],[0,645727,0.0678521547],[0,645733,0.1095472833],[0,645753,0.5236906539],[0,645772,0.0606618563],[0,645799,0.1150551084],[0,646013,0.0544223567],[0,646052,0.0729913264],[0,646140,0.0507653166],[0,646151,0.0627103178],[0,646175,0.0542378695],[0,646204,0.0482095384],[0,646219,0.058476608],[0,646332,0.0625859232],[0,646399,0.0661698341],[0,646482,0.2031224939],[0,646531,0.4405556985],[0,646564,0.0687514532],[0,646586,0.0742889432],[0,646603,0.0833035802],[0,646703,0.0523040404],[0,646704,0.0566435347],[0,646737,0.0542378695],[0,646928,0.0676065789],[0,647026,0.0736226886],[0,647033,0.0511946722],[0,647047,0.0542378695],[0,647288,0.1036636063],[0,647526,0.0720331764],[0,647533,0.0791698861],[0,647607,0.1035322266],[0,647622,0.0636700702],[0,647739,0.0728101433],[0,647921,0.080115232],[0,648243,0.1436603774],[0,648260,0.0735014722],[0,648389,0.078111275],[0,648535,0.0773198455],[0,648581,0.0530297725],[0,648603,0.2621710662],[0,648640,0.1315285492],[0,648703,0.0661960371],[0,648718,0.2734075567],[0,648720,0.1096917701],[0,648721,0.082805355],[0,648757,0.0879732074],[0,648767,0.2173664205],[0,648789,0.0515860447],[0,648793,0.0539828926],[0,648831,0.0555203951],[0,648905,0.0616814912],[0,648935,0.0635508417],[0,649259,0.0507533388],[0,649292,0.1310938878],[0,649295,0.0506152222],[0,649328,0.04833696],[0,649344,0.0515860447],[0,649421,0.0611628914],[0,649448,0.0468950146],[0,649510,0.1266921772],[0,649608,0.0636536777],[0,649648,0.1337191153],[0,649736,0.4688033645],[0,649788,0.3136224562],[0,649803,0.2804824888],[0,649846,0.0539828926],[0,649914,0.063474276],[0,649975,0.063474276],[0,649990,0.0615849689],[0,650027,0.0600086184],[0,650063,0.1190191097],[0,650114,0.063474276],[0,650158,0.063474276],[0,650223,0.0499948094],[0,650292,0.1199959602],[0,650327,0.0635508417],[0,650377,0.060217448],[0,650437,0.2110759198],[0,650456,0.0538814136],[0,650500,0.0544724019],[0,650521,0.0548788985],[0,650523,0.0519658799],[0,650690,0.0481163967],[0,650692,0.0639702092],[0,650787,0.0539006138],[0,650848,0.0594157637],[0,651044,0.0916491301],[0,651192,0.0789926113],[0,651197,0.155470555],[0,651261,0.0515860447],[0,651269,0.0988242237],[0,651459,0.0503823757],[0,651556,0.0481163967],[0,651603,0.0796479951],[0,651608,0.0630284249],[0,651965,0.0851607887],[0,651980,0.0482619838],[0,651998,0.2905237741],[0,652307,0.0483231574],[0,652326,0.1199555848],[0,652424,0.0988204292],[0,652528,0.1018326365],[0,652531,0.0539828926],[0,652564,0.063474276],[0,652583,0.0455798089],[0,652959,0.0544724019],[0,653058,0.0538636283],[0,653144,0.2168065392],[0,653164,0.0684933826],[0,653167,0.0819677877],[0,653171,0.0794817046],[0,653416,0.0481262196],[0,653433,0.2339884259],[0,653448,0.0855721819],[0,653449,0.063474276],[0,653462,0.0785088634],[0,653518,0.0468950146],[0,653522,0.175049448],[0,653650,0.0482619838],[0,653656,0.5731368832],[0,653657,0.302280151],[0,653668,0.0500802055],[0,653793,0.064250371],[0,653796,0.071749445],[0,653800,0.0741016072],[0,653806,0.0965847625],[0,653889,0.0887698564],[0,654025,0.0563192168],[0,654027,0.0576152491],[0,654085,0.0482619838],[0,654100,0.2264283477],[0,654105,0.047605748],[0,654128,0.0515860447],[0,654298,0.0638168483],[0,654317,0.063474276],[0,654344,0.0600040933],[0,654363,0.0924578373],[0,654373,0.0598547001],[0,654384,0.078499898],[0,654409,0.0478608201],[0,654646,0.0518710385],[0,654656,0.0481530883],[0,654804,0.1530159146],[0,654832,0.0776593344],[0,655067,0.0476922746],[0,655125,0.0717259927],[0,655324,0.0517698558],[0,655523,0.0539828926],[0,655669,0.0539828926],[0,655704,0.0513570988],[0,655765,0.0515860447],[0,656108,0.0539828926],[0,656152,0.0635508417],[0,656173,0.0838024712],[0,656189,0.2480251551],[0,656219,0.0631346808],[0,656244,0.0685643371],[0,656468,0.0463966677],[0,656473,0.0614479973],[0,656491,0.0785995986],[0,656509,0.0445976458],[0,656660,0.1192321351],[0,656682,0.0525377047],[0,656975,0.0483059195],[0,657586,0.1115942082],[0,658150,0.0526389952],[0,659616,0.1632184332],[0,659617,0.0614651965],[0,659635,0.0541993798],[0,659731,0.0489813173],[0,659749,0.0611237231],[0,659751,0.0830232093],[0,659878,0.0693401959],[0,659902,0.0525377047],[0,659996,0.0455603532],[0,660127,0.2399434671],[0,660138,0.063248694],[0,660161,0.8949803734],[0,660348,0.0659925303],[0,660376,0.04573022],[0,660388,0.1535350619],[0,660389,0.0526389952],[0,660391,0.1205413841],[0,660403,0.0457667896],[0,660577,0.0592073603],[0,660619,0.0551698473],[0,660872,0.1432754042],[0,660953,0.0526389952],[0,661110,0.0526389952],[0,661128,0.0535611772],[0,661429,0.0497125799],[0,661566,0.0525377047],[0,661576,0.0549709782],[0,661586,0.0611237231],[0,661594,0.0584483746],[0,661620,0.0492736277],[0,661657,0.0611237231],[0,661772,0.0612000432],[0,661777,0.0525377047],[0,661829,0.0442515255],[0,661953,0.0489813173],[0,662000,0.0479649529],[0,662032,0.0445976458],[0,662036,0.0518425643],[0,662166,0.4428873625],[0,662204,0.0458516838],[0,662235,0.0552901155],[0,662480,0.0749603529],[0,662583,0.0557745695],[0,662595,0.1654864758],[0,662738,0.6033119932],[0,662897,0.0526389952],[0,662937,0.5323600294],[0,663117,0.1029880329],[0,663308,0.0611497463],[0,663418,0.0469287166],[0,663462,0.3489983247],[0,663463,0.0811258203],[0,663622,0.0515878654],[0,663633,0.0479649529],[0,663702,0.1449232067],[0,663704,0.0617206302],[0,663716,0.3038962138],[0,663722,0.0901468737],[0,663727,0.0833758203],[0,663736,0.0445976458],[0,663824,0.134819333],[0,663832,0.2103668022],[0,663833,0.1321362846],[0,663855,0.0695113424],[0,663857,0.0714472596],[0,663868,0.0475602967],[0,663879,0.0462606192],[0,663907,0.0611237231],[0,663996,0.0526389952],[0,664032,0.0499080447],[0,664047,0.0490451685],[0,664235,0.0455603532],[0,664240,0.0621234585],[0,664247,0.0539045509],[0,664337,0.1803696001],[0,664351,0.0620261891],[0,664367,0.1296505118],[0,664395,0.0517386681],[0,664431,0.0617206302],[0,664448,0.1151915903],[0,664493,0.1034866913],[0,664513,0.1300331827],[0,664543,0.060238805],[0,664610,0.1541605355],[0,664645,0.0901806613],[0,664656,0.0656840645],[0,664678,0.0901806613],[0,664697,0.1130149753],[0,664710,0.0613025493],[0,664720,0.0843012902],[0,664724,0.0462047545],[0,664859,0.0613025493],[0,664889,0.0613025493],[0,664894,0.0613025493],[0,664912,0.2001827882],[0,664915,0.1918161546],[0,664923,0.0613025493],[0,664936,0.0613025493],[0,664989,0.0732282221],[0,665025,0.0490180531],[0,665033,0.0499080447],[0,665039,0.0526389952],[0,665041,0.0901806613],[0,665097,0.1646344743],[0,665104,0.0633597364],[0,665117,0.2075247546],[0,665126,0.1130149753],[0,665130,0.0901806613],[0,665186,0.0613025493],[0,665200,0.095564172],[0,665272,0.1004335618],[0,665299,0.0445976458],[0,665572,0.4991924261],[0,665967,0.0793933359],[0,665975,0.1001924966],[0,666041,0.1139003588],[0,666128,0.0469287166],[0,666492,0.0611497463],[0,666497,0.0572491818],[0,666561,0.0884958297],[0,666654,0.0497125799],[0,667170,0.2007139809],[0,667173,0.0479705134],[0,668087,0.0462047545],[0,668137,0.1718854347],[0,668187,0.3520633909],[0,668190,0.7001269668],[0,668192,0.066603445],[0,668203,0.0563853008],[0,668214,0.0458151099],[0,668313,0.046360066],[0,668338,0.0830008339],[0,668351,0.0483059195],[0,668514,0.0815541586],[0,668532,0.0626807947],[0,668648,0.05185459],[0,668771,0.0616180655],[0,668773,0.0776082091],[0,669194,0.0460212073],[0,669209,0.0786096126],[0,669220,0.0526389952],[0,669244,0.0545976837],[0,669256,0.0611237231],[0,669266,0.0512810673],[0,669621,0.0812189272],[0,669638,0.0461061317],[0,669850,0.2486791041],[0,669862,0.0616180655],[0,669888,0.0490451685],[0,669919,0.0534348355],[0,669920,0.0616180655],[0,669939,0.0675793793],[0,669945,0.0563523877],[0,669949,0.2558659845],[0,670086,0.0465862446],[0,670128,0.0490819076],[0,670129,0.1075862314],[0,670456,0.0550177334],[0,670473,0.5256016374],[0,670624,0.0918396854],[0,670774,0.6392770975],[0,670912,0.0617206302],[0,671025,0.0526389952],[0,671098,0.0492736277],[0,671185,0.0551698473],[0,671509,0.0566351496],[0,671538,0.0445976458],[0,671539,0.5618127245],[0,671546,0.0615176248],[0,671591,0.0525377047],[0,671612,0.0469287166],[0,671693,0.0594275244],[0,671854,0.0866382096],[0,672010,0.0533120837],[0,672290,0.1371972316],[0,672324,0.2067738294],[0,672352,0.2588180298],[0,672536,0.044541911],[0,672550,0.1105850161],[0,672709,0.0936076011],[0,672713,0.0785214808],[0,672739,0.0525377047],[0,672829,0.0532763836],[0,672877,0.0442515255],[0,672882,0.0742847024],[0,672889,0.0483059195],[0,672902,0.1715987181],[0,672912,0.0509474939],[0,672968,0.0445976458],[0,673194,0.0648973205],[0,673220,0.058689684],[0,673262,0.0790115646],[0,673271,0.3700953497],[0,673295,0.0460212073],[0,673314,0.0616180655],[0,673328,0.0563691952],[0,673484,0.04868146],[0,673805,0.2449590327],[0,673830,0.0549709782],[0,673989,0.1447111845],[0,674087,0.0492736277],[0,674139,0.3465916957],[0,674267,0.0758266068],[0,674539,0.1008774262],[0,674631,0.2541350338],[0,679822,0.4266232017],[0,680348,0.0649204312],[0,684768,0.1728720141],[0,684975,0.0793267424],[0,685325,0.1784329468],[0,685326,0.0601784278],[0,685344,0.0521446947],[0,686222,0.1000449267],[0,686246,0.0595325945],[0,687303,0.08328045],[0,687423,0.0652573051],[0,687442,0.0509607779],[0,687754,0.1981130667],[0,688710,0.0953483364],[0,688912,0.0596850592],[0,688974,0.0426996468],[0,689641,0.0579694677],[0,690497,0.047858305],[0,690539,0.0741540122],[0,690577,0.047858305],[0,690586,0.0559054679],[0,690587,0.062982859],[0,690956,0.0446970481],[0,691512,0.0596850592],[0,691600,0.0596850592],[0,691797,0.0591168613],[0,691886,0.0981765403],[0,691891,0.1704163276],[0,692260,0.047858305],[0,692672,0.0596850592],[0,693168,0.0539487975],[0,693284,0.1632702678],[0,693309,0.0463683955],[0,693474,0.0428460948],[0,693501,0.0771162713],[0,694874,0.0786605761],[0,695068,0.0512170437],[0,695377,0.0898284621],[0,697054,0.0438621175],[0,697073,0.2088918002],[0,697082,0.052547876],[0,698482,0.0533207282],[0,698718,0.047858305],[0,699046,0.0845244193],[0,699421,0.25622794],[0,699569,0.1670273937],[0,701403,0.047858305],[0,702322,0.0523051885],[0,703086,0.2562972353],[0,704094,0.074709365],[0,704121,0.118274455],[0,705012,0.0846513274],[0,705025,0.0607859931],[0,705053,0.0589013515],[0,705213,0.0453996079],[0,705242,0.072255168],[0,705352,0.5961753246],[0,705415,0.1152596255],[0,705449,0.0608622779],[0,705698,0.0522039448],[0,705951,0.0446676282],[0,706590,0.0489413771],[0,706687,0.5815983733],[0,708041,0.305726522],[0,709547,0.0742774374],[0,709811,0.1914118543],[0,709852,0.4657881475],[0,709853,0.0725125214],[0,709876,0.0588156403],[0,709888,0.2093087059],[0,710007,0.0841212807],[0,710128,0.4842970939],[0,710218,0.2681443292],[0,710411,0.239118161],[0,710478,0.0512545447],[0,710947,0.0607859931],[0,711207,0.0626737096],[0,711398,0.0662811436],[0,711558,0.0917691224],[0,711586,0.0609647365],[0,711621,0.0522039448],[0,711775,0.0477714794],[0,712078,0.2142000536],[0,712433,0.0617919826],[0,712566,0.1898539897],[0,712866,0.0522039448],[0,713560,0.0478081533],[0,714033,0.0613315332],[0,714034,0.2172706914],[0,714544,0.0522039448],[0,715230,0.0587201957],[0,715775,0.057381053],[0,715795,0.1364623316],[0,715839,0.0529422821],[0,715852,0.1307950382],[0,715862,0.5637423655],[0,716014,0.3280638702],[0,716019,0.115526735],[0,716405,0.0520485356],[0,716625,0.0607859931],[0,717105,0.049473721],[0,717126,0.0522039448],[0,717398,0.0773075064],[0,717866,0.7548844595],[0,717873,0.1030892081],[0,717883,0.0809478083],[0,717902,0.0523051885],[0,717917,0.0740249486],[0,717973,0.0598471741],[0,718053,0.0476959732],[0,718077,0.1262866415],[0,718102,0.0733352271],[0,718345,0.4806426977],[0,718517,0.0607859931],[0,718537,0.076592173],[0,718647,0.2787990417],[0,719229,0.2458088494],[0,719253,0.0615026791],[0,719714,0.0522039448],[0,719722,0.6564558009],[0,719759,0.0946630931],[0,719801,0.0628873527],[0,719974,0.0523051885],[0,720176,0.2439485499],[0,720473,0.047719004],[0,721834,0.1383763008],[0,721865,0.0607859931],[0,722102,0.0608622779],[0,722397,0.1608739027],[0,722471,0.8276783048],[0,722487,0.1607396192],[0,722548,0.2106333656],[0,722584,0.0851809882],[0,722649,0.0941591045],[0,722714,0.0882528246],[0,722777,0.0523051885],[0,722814,0.1331110941],[0,722867,0.0611273085],[0,723129,0.0609647365],[0,723409,0.2085394293],[0,723423,0.0607859931],[0,723632,0.0442675574],[0,723641,0.2757703197],[0,723919,0.0780379789],[0,724301,0.2553051703],[0,724571,0.0523051885],[0,724606,0.294564679],[0,725030,0.0926551093],[0,725055,0.4274603371],[0,725177,0.5316449428],[0,725209,0.0435405066],[0,725463,0.3492012693],[0,725475,0.0763523831],[0,725813,0.2588719648],[0,725861,0.0506839009],[0,725976,0.0523051885],[0,726051,0.0495262852],[0,726062,0.0890030055],[0,726075,0.0919544314],[0,726131,0.0988408369],[0,726159,0.1979980364],[0,726414,0.0553749886],[0,726639,0.0523051885],[0,726655,0.4195837444],[0,726671,0.0830546965],[0,726741,0.3774158779],[0,726987,0.1404393054],[0,727050,0.08165678],[0,727572,0.0545880528],[0,727619,0.0658941062],[0,728131,0.5085864079],[0,728453,0.2158112916],[0,728530,0.1059924097],[0,728587,0.059895838],[0,728649,0.2619343463],[0,728886,0.1891162664],[0,729415,0.1194281263],[0,729451,0.0603017351],[0,729860,0.0515117919],[0,729883,0.4692171169],[0,729916,0.4116829117],[0,730745,0.0806891691],[0,730916,0.0607859931],[0,730924,0.0486528715],[0,730975,0.0493801263],[0,731117,0.0523051885],[0,731307,0.3719939722],[0,731475,0.2612608312],[0,731527,0.0607859931],[0,731578,0.0580111985],[0,731670,0.1928924364],[0,731683,0.0607859931],[0,731716,0.0607859931],[0,731819,0.0756365977],[0,732184,0.0785761324],[0,732768,0.0538852367],[0,733917,0.363201261],[0,733940,0.0439772835],[0,734198,0.5840529898],[0,734772,0.2940767615],[0,734970,0.055669901],[0,734976,0.069071846],[0,734991,0.0607859931],[0,734992,0.0546527327],[0,735078,0.0503459314],[0,735194,0.0885550211],[0,735229,0.059895838],[0,735330,0.0609647365],[0,735384,0.1078429858],[0,735398,0.0486845849],[0,735407,0.0486845849],[0,736450,0.5542158166],[0,737661,0.3878516999],[0,737751,0.1225474045],[0,737844,0.1297630726],[0,737916,0.0504986892],[0,737950,0.2019102827],[0,738166,0.0489413771],[0,738471,0.0716332673],[0,738481,0.1917970425],[0,738600,0.0608120042],[0,738646,0.5891718473],[0,738657,0.0523051885],[0,738737,0.3919827474],[0,738848,0.0490877726],[0,738925,0.5449017758],[0,739001,0.2676842449],[0,739055,0.1026758237],[0,739122,0.2414643662],[0,739384,0.1084653971],[0,739400,0.0435405066],[0,739413,0.2455704538],[0,739568,0.0608622779],[0,739637,0.0459378994],[0,739666,0.0707291364],[0,739667,0.0550590719],[0,739739,0.0607859931],[0,739754,0.0671930384],[0,739770,0.0660768414],[0,739780,0.1125444566],[0,739845,0.1350900649],[0,739848,0.0442675574],[0,739852,0.0751949703],[0,739941,0.0502605036],[0,740038,0.0612451378],[0,740096,0.0505165203],[0,740124,0.1222699936],[0,740145,0.2625258823],[0,740281,0.1214810632],[0,740548,0.0927124775],[0,740604,0.0815465082],[0,740663,0.131387091],[0,741632,0.0499561429],[0,741698,0.1725390397],[0,741721,0.0729745719],[0,741779,0.1609383454],[0,742159,0.3999090039],[0,742168,0.0479926727],[0,742475,0.0730454723],[0,742990,0.0562683587],[0,743059,0.1346769155],[0,743503,0.1070909953],[0,743591,0.0965427217],[0,743815,0.0716165035],[0,743853,0.2909015879],[0,743875,0.0486478662],[0,744034,0.2796124272],[0,744158,0.1082644467],[0,744168,0.0523051885],[0,744261,0.0442675574],[0,744374,0.2186970247],[0,744541,0.0921080977],[0,744553,0.9013433869],[0,744797,0.1816147001],[0,744809,0.1042211428],[0,744813,0.0927619007],[0,744898,0.1194309878],[0,744964,0.1440456763],[0,745023,0.4705557858],[0,745620,0.078408105],[0,745689,0.1316683687],[0,747573,0.6383855138],[0,747843,0.0846074909],[0,748148,0.070088184],[0,748374,0.0500960387],[0,748418,0.1217824151],[0,749525,0.10139648],[0,749570,0.0845466131],[0,750110,0.053465258],[0,750111,0.0463785141],[0,750318,0.0592632917],[0,750328,0.0510412113],[0,750594,0.427713146],[0,750725,0.1051352851],[0,750901,0.0628611264],[0,751231,0.2179888354],[0,751265,0.0528903068],[0,751401,0.1194961921],[0,751428,0.053465258],[0,751499,0.0790675615],[0,751701,0.1414660972],[0,751897,0.1538229913],[0,753029,0.1571373715],[0,753143,0.2352600037],[0,753225,0.3294390395],[0,753301,0.2800262444],[0,753463,0.1272377534],[0,753724,0.1422091951],[0,753726,0.7343354093],[0,753826,0.255146134],[0,754006,0.0500960387],[0,754051,0.1243120188],[0,754169,0.0484844538],[0,754268,0.8325456595],[0,754396,0.5079237459],[0,754407,0.0594051058],[0,754461,0.876045586],[0,754546,0.1048254682],[0,754584,0.053465258],[0,754628,0.061763729],[0,755112,0.0470795815],[0,755451,0.3924513116],[0,755458,0.1314061419],[0,755651,0.0619596971],[0,755734,0.0511775272],[0,755760,0.0524129251],[0,755762,0.0787850469],[0,755856,0.246565711],[0,756095,0.2254870575],[0,756098,0.0583292962],[0,756392,0.4605270479],[0,756684,0.0498033937],[0,756795,0.0513177673],[0,756864,0.3780441166],[0,756909,0.0624546052],[0,757352,0.0802501281],[0,757463,0.1218903285],[0,757475,0.0640871001],[0,757484,0.0866962514],[0,757554,0.095449984],[0,757584,0.253138774],[0,757658,0.0532081925],[0,757671,0.1403712342],[0,757682,0.0624546052],[0,757693,0.6033451453],[0,757775,0.0958214012],[0,757788,0.0646675702],[0,757866,0.0752376273],[0,757877,0.0987042067],[0,757889,0.0621387279],[0,757905,0.0519269988],[0,757933,0.0507654763],[0,757956,0.0619596971],[0,757961,0.327851493],[0,757988,0.0507311818],[0,758013,0.0789998494],[0,758022,0.0675257613],[0,758078,0.1320909059],[0,758110,0.1077190622],[0,758120,0.1502090901],[0,758136,0.0620361045],[0,758200,0.2360233359],[0,758275,0.0558900632],[0,758330,0.0449770478],[0,758351,0.1549581664],[0,758360,0.0454147049],[0,758394,0.0797604164],[0,758408,0.5838805538],[0,758550,0.0507654763],[0,758563,0.0743463703],[0,758577,0.0463785141],[0,758600,0.0524129251],[0,758622,0.085147912],[0,758636,0.109411233],[0,758702,0.0500960387],[0,758740,0.051502851],[0,758743,0.0882631329],[0,758758,0.4772427886],[0,758764,0.0501956479],[0,758778,0.0919701021],[0,758807,0.0449770478],[0,758809,0.1093381777],[0,758834,0.053056737],[0,758860,0.0489242601],[0,758947,0.2171199846],[0,758980,0.1659094798],[0,759018,0.0801070087],[0,759103,0.0866138425],[0,759124,0.0602987845],[0,759135,0.0536193863],[0,759248,0.0557517924],[0,759333,0.0511775272],[0,759369,0.1151868298],[0,759375,0.1134949308],[0,759383,0.0853252294],[0,759424,0.0619596971],[0,759442,0.1931591745],[0,759443,0.3865511897],[0,759452,0.3902932499],[0,759544,0.0793506966],[0,759598,0.0802649691],[0,759645,0.1109794996],[0,759648,0.0517049224],[0,759653,0.0619596971],[0,759693,0.0668617337],[0,759772,0.4621763769],[0,759784,0.052701675],[0,759807,0.2169741158],[0,759810,0.216942465],[0,759890,0.1009666901],[0,759938,0.0500960387],[0,759964,0.4651531168],[0,759990,0.0494381864],[0,760014,0.0619596971],[0,760020,0.1292373456],[0,760035,0.0836953972],[0,760039,0.050975953],[0,760100,0.0679620008],[0,760134,0.1267415492],[0,760328,0.0632020706],[0,760498,0.0840238442],[0,760499,0.11283381],[0,760530,0.0624546052],[0,760563,0.0906012003],[0,760576,0.547469514],[0,760646,0.1200946084],[0,760651,0.0605524488],[0,760669,0.0500960387],[0,760732,0.3389558331],[0,760740,0.0610681109],[0,760855,0.513089658],[0,760876,0.0515618311],[0,760894,0.0577902411],[0,761097,0.0514659881],[0,761217,0.0819916117],[0,761388,0.09318386],[0,761421,0.0494381864],[0,761460,0.2077101751],[0,761487,0.0502426695],[0,761551,0.1677490809],[0,761632,0.071274792],[0,761731,0.0859062763],[0,761838,0.0611597393],[0,761844,0.0491272228],[0,761853,0.0533638516],[0,761855,0.5248597708],[0,761879,0.0517593097],[0,761909,0.0500960387],[0,761980,0.0776843568],[0,762044,0.4007205347],[0,762098,0.0645611224],[0,762107,0.0511775272],[0,762237,0.6946297661],[0,762354,0.0814348745],[0,762360,0.0454147049],[0,762511,0.1178771141],[0,762606,0.0516189837],[0,762623,0.4764218715],[0,762866,0.0669500878],[0,762892,0.1043990027],[0,762959,0.0841586678],[0,763037,0.0512925699],[0,763121,0.1054368606],[0,763144,0.0559918241],[0,763216,0.0624546052],[0,763304,0.0511775272],[0,763317,0.1173501933],[0,763500,0.0467963111],[0,763505,0.1400633634],[0,763631,0.2377540148],[0,763639,0.0533638516],[0,763855,0.0579095737],[0,763888,0.0500960387],[0,763994,0.0690435402],[0,764031,0.1435175334],[0,764055,0.0500960387],[0,764088,0.0621448362],[0,764120,0.0624871137],[0,764164,0.053465258],[0,764188,0.0500960387],[0,764297,0.0820378445],[0,764311,0.0454147049],[0,764351,0.300193818],[0,764495,0.1072098975],[0,764504,0.0511775272],[0,764517,0.0500960387],[0,764565,0.1839859668],[0,764688,0.0454147049],[0,764917,0.6753525142],[0,764950,0.0600985978],[0,765002,0.0491272228],[0,765157,0.0454147049],[0,765265,0.1534672106],[0,765277,0.3264834182],[0,765312,0.0582604684],[0,765343,0.0510412113],[0,765401,0.0643623719],[0,765438,0.0658399848],[0,765557,0.0814874828],[0,765610,0.0507311818],[0,765724,0.0747132606],[0,765989,0.0741308757],[0,765993,0.053465258],[0,766053,0.2341133065],[0,766135,0.1334438354],[0,766169,0.0484844538],[0,766261,0.2066146335],[0,766718,0.3868081265],[0,767470,0.0848607447],[0,767668,0.287795912],[0,767810,0.0807179856],[0,767891,0.048960993],[0,768099,0.053901239],[0,768179,0.1291207454],[0,768236,0.0602577509],[0,768238,0.2428483084],[0,768253,0.053465258],[0,768258,0.108106386],[0,768413,0.0533638516],[0,768445,0.3591062383],[0,768449,0.0511775272],[0,768570,0.5575369759],[0,768681,0.0642566269],[0,768713,0.0485211642],[0,768794,0.106086174],[0,768838,0.0786671107],[0,768940,0.1296818008],[0,768955,0.0608387426],[0,769010,0.06198575],[0,769016,0.1393624077],[0,769039,0.0559998371],[0,769130,0.0704872999],[0,769230,0.0975494792],[0,769240,0.0485211642],[0,769752,0.0619596971],[0,769982,0.1522482135],[0,770038,0.0557517924],[0,770506,0.0666222544],[0,770584,0.2250327105],[0,770875,0.0591459276],[0,770914,0.1120057692],[0,770915,0.0506365027],[0,771037,0.0784587992],[0,771137,0.0666565863],[0,771200,0.1104548272],[0,771269,0.0664635383],[0,771282,0.0558248442],[0,771283,0.0557231067],[0,771309,0.0807322929],[0,771349,0.0558248442],[0,771416,0.0529868536],[0,771486,0.1342354202],[0,771663,0.0835272769],[0,771728,0.0531162483],[0,771775,0.0558248442],[0,771787,0.0724004754],[0,771797,0.0512690209],[0,771809,0.0645266312],[0,772010,0.0541133263],[0,772075,0.086469733],[0,772154,0.0548053221],[0,772205,0.0681660903],[0,772256,0.0489222199],[0,772316,0.0738021016],[0,772605,0.1186457467],[0,772727,0.0489589527],[0,772743,0.0719126434],[0,772851,0.2172844767],[0,772889,0.0529654478],[0,773100,0.2108604473],[0,773113,0.0573078621],[0,773387,0.1770778192],[0,773430,0.0651776304],[0,773581,0.0555381673],[0,773584,0.0557303727],[0,773600,0.1187946097],[0,773611,0.1103793331],[0,773868,0.105216188],[0,773964,0.4021258754],[0,774015,0.0572059165],[0,774095,0.08185091],[0,774170,0.0570234222],[0,774373,0.0594381132],[0,774402,0.0566814421],[0,774463,0.0840641604],[0,774521,0.0828240399],[0,774534,0.0576317638],[0,774764,0.0628166544],[0,774778,0.0594381132],[0,774829,0.1374865837],[0,774881,0.0559529897],[0,775016,0.0572604481],[0,775022,0.0532420654],[0,775028,0.0557129993],[0,775047,0.0572604481],[0,775147,0.1443222739],[0,775338,0.3753392051],[0,775341,0.7857387667],[0,775588,0.0861008002],[0,775871,0.0759155576],[0,775943,0.0716507578],[0,776006,0.0693995839],[0,776144,0.0498960818],[0,776273,0.0681203788],[0,776290,0.0820255484],[0,776473,0.053889095],[0,776589,0.1025448505],[0,776607,0.0554083791],[0,776720,0.0558361589],[0,776875,0.1339646084],[0,776985,0.087147337],[0,777209,0.0572604481],[0,777576,0.2124097475],[0,777589,0.0683599451],[0,778193,0.0562433627],[0,778252,0.065098241],[0,778362,0.055982336],[0,778382,0.0572604481],[0,778660,0.0539790115],[0,778774,0.1968195976],[0,778792,0.0804954186],[0,778842,0.0532420654],[0,778880,0.0552199055],[0,778886,0.1453574971],[0,778915,0.0668579424],[0,778918,0.1427209664],[0,779008,0.0659794637],[0,779316,0.0566542152],[0,779387,0.0532420654],[0,779395,0.0552458952],[0,779528,0.0545684011],[0,779678,0.0659794637],[0,779691,0.0995616104],[0,779742,0.0572604481],[0,779875,0.0860208734],[0,779905,0.0562600688],[0,780009,0.0627659325],[0,780202,0.1397940835],[0,780343,0.0798684228],[0,780422,0.1041676333],[0,780424,0.1202725931],[0,780584,0.0736852384],[0,780758,0.1004715512],[0,780773,0.2911332826],[0,780796,0.0701105802],[0,780915,0.060799198],[0,780980,0.0570809329],[0,781013,0.0572604481],[0,781281,0.0497114107],[0,781305,0.0825156927],[0,781306,0.0642706459],[0,781401,0.1482021889],[0,781527,0.066400221],[0,781530,0.0734024757],[0,781541,0.1042239842],[0,781561,0.0998440147],[0,781579,0.1504826488],[0,781639,0.055539104],[0,781836,0.3197171451],[0,781848,0.0585834885],[0,781860,0.0673867909],[0,781874,0.0603420059],[0,782083,0.0682459805],[0,782162,0.0824578169],[0,782307,0.1131633566],[0,782324,0.0858210973],[0,782366,0.1038157269],[0,782524,0.1386394049],[0,782531,0.0558354323],[0,782554,0.0759976236],[0,782631,0.1935282853],[0,782633,0.056278393],[0,782641,0.1467591825],[0,782694,0.0676278848],[0,782700,0.0639884249],[0,783097,0.0568085618],[0,783162,0.4921779158],[0,783167,0.0649199738],[0,783240,0.055983233],[0,783247,0.2530489842],[0,783274,0.0623307994],[0,783306,0.593577101],[0,783326,0.0830807662],[0,783411,0.0558354323],[0,783764,0.1378042004],[0,783797,0.0996784738],[0,784049,0.0882797916],[0,784619,0.1175956317],[0,784622,0.2617563313],[0,784676,0.0536918017],[0,784746,0.0742948177],[0,784750,0.0669498211],[0,784813,0.09668011],[0,785270,0.1113535126],[0,785385,0.0880860421],[0,785386,0.0582347611],[0,785533,0.0756101108],[0,785581,0.0688471172],[0,785775,0.1444761133],[0,785798,0.1018537436],[0,785970,0.175806964],[0,786167,0.073345286],[0,786450,0.0520882478],[0,786476,0.086898856],[0,786493,0.0685758319],[0,786649,0.0558354323],[0,786671,0.0923529924],[0,787835,0.0597344616],[0,788272,0.1318056278],[0,788343,0.1023581294],[0,788527,0.0582347611],[0,788646,0.0561092021],[0,788945,0.0600888818],[0,789120,0.1685290275],[0,789423,0.1534285512],[0,789791,0.0769991898],[0,789802,0.05537596],[0,789851,0.0668885468],[0,789906,0.0628568214],[0,794920,0.0582347611],[0,800433,0.5537805484],[0,802006,0.1093396554],[0,802099,0.1124386171],[0,802105,0.2705759504],[0,802176,0.0668885468],[0,802289,0.0573988482],[0,802296,0.0796783645],[0,802766,0.0573988482],[0,802802,0.0597948356],[0,803386,0.0578579816],[0,803569,0.0801404472],[0,803692,0.0642843736],[0,803871,0.097613323],[0,803872,0.0605234145],[0,804075,0.067933295],[0,804412,0.0837184539],[0,804578,0.0565333424],[0,804740,0.0719274932],[0,805085,0.0604058474],[0,805096,0.0617416381],[0,805115,0.3039927611],[0,805581,0.0692863979],[0,805851,0.4806298105],[0,805905,0.1022428243],[0,806317,0.2844903979],[0,806325,0.0839905779],[0,806955,0.0603685291],[0,807040,0.0906371251],[0,807192,0.1175166268],[0,807226,0.1279670624],[0,807699,0.0602640029],[0,809780,0.0694415606],[0,810276,0.0642843736],[0,810354,0.2864483689],[0,810948,0.0602640029],[0,810962,0.0602640029],[0,811549,0.0604058474],[0,811644,0.0642843736],[0,812184,0.0602640029],[0,813120,0.0689419267],[0,813158,0.0983916228],[0,813301,0.1775205185],[0,814565,0.0626840391],[0,814753,0.0694415606],[0,814774,0.0602640029],[0,815210,0.3151357308],[0,816000,0.0685289988],[0,816167,0.0602640029],[0,816174,0.0780951673],[0,816436,0.0602640029],[0,816538,0.0597262],[0,817846,0.275752544],[0,817914,0.0689419267],[0,817986,0.0899809001],[0,818024,0.0692870553],[0,818274,0.1642705639],[0,818448,0.0687875259],[0,818653,0.5727516824],[0,818674,0.2755863141],[0,819805,0.057350167],[0,820173,0.6968640136],[0,820234,0.1053326124],[0,820256,0.054328401],[0,820261,0.0757853614],[0,820265,0.6589639147],[0,820822,0.3447525918],[0,822318,0.0902096903],[0,822361,0.1879708608],[0,822573,0.074044812],[0,822643,0.0568787123],[0,822855,0.0884646912],[0,823316,0.0602640029],[0,823365,0.1796378279],[0,823472,0.0671800567],[0,823619,0.0602640029],[0,823717,0.066712817],[0,823922,0.3234208225],[0,824094,0.0748718642],[0,824157,0.2357819454],[0,824160,0.0573873308],[0,824183,0.0898150734],[0,824369,0.0602640029],[0,824455,0.0921176183],[0,824637,0.1343792951],[0,824688,0.0690190638],[0,824724,0.0922313496],[0,824731,0.0716467603],[0,825106,0.9222991316],[0,825157,0.0666048814],[0,825662,0.0545400279],[0,825687,0.0602184775],[0,825854,0.7198692552],[0,825948,0.1073612353],[0,826194,0.1111008516],[0,826284,0.1024705576],[0,826285,0.4232542981],[0,826704,0.2371228003],[0,827034,0.0587704718],[0,827156,0.0695516523],[0,827443,0.0685289988],[0,827463,0.0660335129],[0,827670,0.0836208071],[0,827723,0.0861687074],[0,827760,0.2879607215],[0,828041,0.0602640029],[0,828563,0.0602640029],[0,828600,0.693389436],[0,828874,0.0573873308],[0,828998,0.0694415606],[0,829006,0.0819744691],[0,829742,0.0689419267],[0,829935,0.1053914455],[0,829944,0.1690140497],[0,830465,0.0700013321],[0,830695,0.0917610845],[0,830880,0.0699550404],[0,831337,0.3763562377],[0,831478,0.0600765052],[0,831584,0.054328401],[0,831801,0.055304231],[0,834988,0.1082696891],[0,835028,0.0565333424],[0,835155,0.0936908645],[0,835202,0.0578579816],[0,835329,0.0904432354],[0,835554,0.057350167],[0,835712,0.0536280705],[0,835748,0.0578579816],[0,838121,0.178615713],[0,838417,0.2395677386],[0,838569,0.0665396553],[0,838650,0.0618972586],[0,838699,0.325382685],[0,838841,0.0697135345],[0,839060,0.0526525344],[0,839096,0.0578899709],[0,839139,0.1264882827],[0,839174,0.0752588616],[0,839192,0.0665396553],[0,839249,0.102149265],[0,839304,0.0569106586],[0,839398,0.1036744113],[0,839401,0.0665396553],[0,839487,0.0567985043],[0,839502,0.1254587809],[0,839653,0.0576027076],[0,839794,0.0554917792],[0,839795,0.3385601315],[0,839913,0.0598092696],[0,840056,0.2145486117],[0,840073,0.0552803711],[0,840228,0.069234314],[0,840300,0.098035208],[0,840363,0.0704534655],[0,840434,0.0663857569],[0,840673,0.2000779687],[0,841246,0.8701832686],[0,841349,0.1153748737],[0,841996,0.0666165413],[0,842290,0.2258865205],[0,842318,0.084373556],[0,842341,0.12384094],[0,842505,0.0549856171],[0,842508,0.1402598012],[0,842598,0.3191223231],[0,842813,0.0875040488],[0,842857,0.0569106586],[0,843143,0.2980971228],[0,843641,0.0899586042],[0,844149,0.0943780506],[0,844603,0.1064144207],[0,845023,0.1387248614],[0,845296,0.0666165413],[0,845453,0.1559282032],[0,845457,0.2064214884],[0,845514,0.0664626269],[0,845616,0.1103532578],[0,845665,0.0618972586],[0,845769,0.0542582575],[0,845773,0.0545156966],[0,845995,0.0896545297],[0,846025,0.0943452308],[0,846638,0.1084656948],[0,846831,0.0577030834],[0,847066,0.185933076],[0,847228,0.0671250426],[0,847259,0.0922078249],[0,847345,0.1279598122],[0,847414,0.0665396553],[0,847682,0.0578899709],[0,847723,0.0647735211],[0,847810,0.0946945828],[0,847822,0.0685055657],[0,847861,0.0683984373],[0,848035,0.1693832336],[0,848073,0.0976066857],[0,848176,0.1027436487],[0,848219,0.161966798],[0,848362,0.0549856171],[0,848437,0.074044171],[0,848539,0.0578899709],[0,848623,0.0532111379],[0,848693,0.0561316864],[0,848742,0.0665396553],[0,849073,0.0554917792],[0,849284,0.0549856171],[0,849355,0.0910584878],[0,850024,0.0578899709],[0,850202,0.0665396553],[0,850368,0.1398902937],[0,850560,0.0666165413],[0,850606,0.0771956324],[0,850661,0.1445175099],[0,851032,0.0621948255],[0,851147,0.4721589468],[0,851486,0.0610694003],[0,851725,0.112393609],[0,851876,0.0940748493],[0,851968,0.0914094256],[0,851996,0.46620402],[0,852005,0.0670376632],[0,852110,0.1348956984],[0,852115,0.0568830954],[0,852159,0.080526473],[0,852483,0.0566966302],[0,852490,0.0661280711],[0,853115,0.0630295311],[0,853125,0.0702171084],[0,853164,0.1310170617],[0,853180,0.0684820072],[0,853291,0.0578899709],[0,853392,0.0937483331],[0,853426,0.0911212156],[0,853473,0.0554917792],[0,853557,0.0764771951],[0,853564,0.0840359917],[0,853690,0.3316968161],[0,853749,0.0702066643],[0,853997,0.0645224275],[0,854008,0.0600976117],[0,854021,0.0519736845],[0,854086,0.0909359687],[0,854220,0.0834111376],[0,854329,0.0578899709],[0,854378,0.0900524139],[0,854468,0.0691006441],[0,854651,0.0610694003],[0,854744,0.0608414106],[0,854790,0.094729081],[0,855023,0.1007764928],[0,855083,0.0749807442],[0,855130,0.0801188576],[0,855226,0.0545156966],[0,855275,0.0665396553],[0,855442,0.1120006364],[0,855496,0.0578899709],[0,855612,0.0946945828],[0,855646,0.0545156966],[0,855687,0.1447354231],[0,855745,0.2367997944],[0,855874,0.0959835495],[0,856157,0.0665396553],[0,856175,0.0578899709],[0,856373,0.0873664851],[0,856516,0.0549856171],[0,857797,0.1132157032],[0,857981,0.0582901354],[0,858217,0.0545156966],[0,858227,0.1162900325],[0,858396,0.3203172619],[0,858526,0.1078199627],[0,858686,0.1295232882],[0,858750,0.0588188606],[0,858780,0.0554917792],[0,858957,0.0578899709],[0,858990,0.142492125],[0,859050,0.0578899709],[0,859127,0.0567688809],[0,859910,0.0821202758],[0,860160,0.3737043525],[0,860248,0.0653913693],[0,861082,0.0651425359],[0,861125,0.0647234074],[0,861223,0.1123347815],[0,861410,0.2169068543],[0,861577,0.1103358601],[0,861697,0.0519736845],[0,862039,0.1151239346],[0,862278,0.0739317324],[0,862315,0.0519736845],[0,862354,0.0546895618],[0,862411,0.8347848286],[0,862473,0.1381903326],[0,862529,0.0662093069],[0,862707,0.1205147692],[0,862752,0.5803673492],[0,862830,0.072956122],[0,862847,0.1696390046],[0,862916,0.0517171495],[0,862948,0.0657192662],[0,863199,0.0615983781],[0,863218,0.0899537775],[0,863455,0.1643495953],[0,863765,0.0613718892],[0,863958,0.0941217179],[0,864192,0.078299588],[0,864440,0.0799137738],[0,864833,0.0917104655],[0,865638,0.5017952523],[0,865832,0.3414178794],[0,865852,0.1048918532],[0,866010,0.0578899709],[0,866501,0.0578899709],[0,866673,0.0608693793],[0,866803,0.0951377958],[0,866860,0.1764769637],[0,866905,0.0942854788],[0,867108,0.0665396553],[0,867190,0.0637656916],[0,867380,0.0581661259],[0,867495,0.0747197992],[0,867564,0.0755010423],[0,867565,0.0665396553],[0,867600,0.0665396553],[0,867738,0.1609746446],[0,867789,0.0554917792],[0,868533,0.058761876],[0,868747,0.1451749786],[0,868774,0.0578899709],[0,869102,0.0515320108],[0,869258,0.0815159113],[0,869379,0.0869281437],[0,869737,0.0644623375],[0,869816,0.1451419225],[0,869899,0.0566458914],[0,869975,0.0720520979],[0,869996,0.0828828853],[0,870308,0.0569106586],[0,871941,0.7370598956],[0,872497,0.2780164826],[0,872697,0.4134250148],[0,872749,0.0558736725],[0,873018,0.0663547981],[0,873028,0.1331487486],[0,873099,0.105915576],[0,873162,0.9609351539],[0,873291,0.0552040087],[0,873562,0.0548755116],[0,874007,0.4193158237],[0,874529,0.1089338132],[0,874530,0.0668836607],[0,875058,0.1216487681],[0,875261,0.0959812637],[0,875388,0.0613292629],[0,875521,0.0561316864],[0,875675,0.4658046461],[0,875677,0.0578899709],[0,875727,0.0553450326],[0,875773,0.064217872],[0,875833,0.0625930377],[0,876510,0.1121431266],[0,876539,0.2543688561],[0,876602,0.6160596882],[0,876610,0.0665396553],[0,876704,0.3457349633],[0,876709,0.3892584682],[0,876746,0.0865798043],[0,876857,0.0677425744],[0,877013,0.0578899709],[0,877031,0.1396105377],[0,877038,0.112207925],[0,877082,0.0914123981],[0,877282,0.3940111506],[0,877514,0.1099515527],[0,877645,0.1289513923],[0,877653,0.0851811062],[0,877800,0.220503334],[0,877885,0.1787780924],[0,878006,0.0619432077],[0,878223,0.5101415017],[0,878332,0.0554917792],[0,878401,0.1388105948],[0,878690,0.0665396553],[0,879156,0.0900217138],[0,879476,0.053861526],[0,879493,0.0909764054],[0,879708,0.094640081],[0,882085,0.1324855579],[0,882170,0.1015875118],[0,882383,0.0846898758],[0,882394,0.0554917792],[0,882436,0.0588336629],[0,883137,0.4255252964],[0,883180,0.0524958841],[0,883499,0.0578899709],[0,883662,0.0665396553],[0,884395,0.1347121399],[0,884528,0.0679903538],[0,884656,0.0543702373],[0,884872,0.0555609278],[0,884884,0.2375025735],[0,884977,0.0569206961],[0,884978,0.2022766317],[0,885167,0.0759620265],[0,885278,0.0627957203],[0,885362,0.078127799],[0,885371,0.0827258713],[0,885394,0.596821053],[0,885484,0.0569206961],[0,885545,0.0957354781],[0,885658,0.059323616],[0,885660,0.0601972401],[0,885666,0.111731483],[0,885962,0.693119318],[0,886116,0.059323616],[0,886178,0.0583386853],[0,886255,0.0935780848],[0,886512,0.1440622519],[0,886667,0.0936765125],[0,886668,0.4778820971],[0,886880,0.0625093138],[0,886891,0.0575618649],[0,886957,0.1403165767],[0,887022,0.0609973231],[0,887052,0.1128208695],[0,887199,0.059323616],[0,887237,0.0994428434],[0,887371,0.0621567099],[0,887410,0.0566624016],[0,887419,0.0850174619],[0,887481,0.1618616232],[0,887486,0.5576174342],[0,887883,0.0573033423],[0,888179,0.23687355],[0,889074,0.067836152],[0,889248,0.1358591795],[0,889303,0.0689026837],[0,889481,0.1903847622],[0,890934,0.0700290065],[0,891041,0.1004223997],[0,891217,0.058289628],[0,891346,0.1087940321],[0,891669,0.1055272406],[0,891685,0.0602691685],[0,891697,0.0864621037],[0,891840,0.1252925914],[0,891938,0.2381518616],[0,892160,0.0601972401],[0,892232,0.0991975682],[0,892725,0.2159753308],[0,892912,0.0696572277],[0,893022,0.089223746],[0,893060,0.0774592003],[0,893195,0.094380615],[0,893284,0.3368102298],[0,893295,0.070403739],[0,893340,0.0707023578],[0,893349,0.1453006642],[0,893439,0.0924964516],[0,893503,0.1390981836],[0,893751,0.0531666086],[0,893803,0.0922652032],[0,893986,0.2064128921],[0,893988,0.0592246882],[0,893995,0.1028739727],[0,894202,0.0700085665],[0,894410,0.3682957194],[0,894468,0.0700085665],[0,895451,0.0564628274],[0,895803,0.0532035584],[0,896535,0.0626420322],[0,896599,0.1734922668],[0,896695,0.3615219961],[0,896720,0.0568220946],[0,896734,0.2077569453],[0,896749,0.0704314783],[0,896862,0.070586223],[0,897536,0.0713626949],[0,897739,0.1322781248],[0,898025,0.0575327956],[0,898134,0.0532975423],[0,898308,0.1728389087],[0,898333,0.0584748119],[0,898486,0.3715603825],[0,898699,0.0580291571],[0,898702,0.1034907385],[0,898986,0.4025635746],[0,899765,0.062670293],[0,899806,0.2816500234],[0,900048,0.386561452],[0,900249,0.0577332848],[0,900297,0.070586223],[0,900527,0.3782985087],[0,900552,0.0754906655],[0,900743,0.1361945939],[0,900874,0.1111094865],[0,901016,0.0825366931],[0,901425,0.0558442203],[0,901598,0.065937019],[0,901837,0.1020144715],[0,901974,0.0848834697],[0,902056,0.1014267025],[0,902074,0.0916839026],[0,902075,0.4289484437],[0,902095,0.2540729184],[0,902102,0.1782231536],[0,902170,0.1122494847],[0,902218,0.5017834838],[0,902244,0.093624899],[0,902250,0.0856346473],[0,903166,0.0664840271],[0,903267,0.0773110103],[0,903285,0.0592246882],[0,903528,0.0988165996],[0,903593,0.0532975423],[0,903594,0.0688024552],[0,903813,0.0733296368],[0,903843,0.261725031],[0,903855,0.1682479437],[0,904265,0.0705089222],[0,904390,0.0609263235],[0,904409,0.1294730789],[0,904506,0.0935359902],[0,905131,0.0513908024],[0,905524,0.0550920593],[0,905795,0.0705089222],[0,905802,0.0916232662],[0,906158,0.0531975062],[0,906299,0.0677225241],[0,906408,0.0609263235],[0,906435,0.0700085665],[0,906501,0.0700085665],[0,906526,0.0518323895],[0,906538,0.140765718],[0,906588,0.0698539426],[0,906635,0.0629809405],[0,906737,0.1677916621],[0,906876,0.3734514314],[0,907365,0.0948260162],[0,907611,0.0592360348],[0,907658,0.0687544241],[0,907711,0.0553497951],[0,908194,0.2106751118],[0,908238,0.1006428966],[0,908278,0.4477228487],[0,908429,0.0553497951],[0,908597,0.0647892604],[0,908617,0.0580236178],[0,908635,0.2852453783],[0,908775,0.0932909996],[0,909033,0.1986330893],[0,909066,0.172960134],[0,909426,0.0529035996],[0,909543,0.0592246882],[0,909544,0.0554760988],[0,909548,0.2367383145],[0,909563,0.0521803965],[0,909953,0.0517017213],[0,910136,0.0547336482],[0,910149,0.0834358383],[0,910225,0.1311240644],[0,911207,0.5290065023],[0,911298,0.0948698375],[0,911407,0.05655441],[0,911507,0.0967989821],[0,911619,0.0523354074],[0,911627,0.1554529925],[0,911647,0.0605744823],[0,911965,0.0543739037],[0,911983,0.0772491019],[0,912038,0.0929960445],[0,912218,0.0700085665],[0,912229,0.1769791711],[0,912709,0.0592246882],[0,912711,0.0518323895],[0,913185,0.1280627095],[0,913219,0.1036891196],[0,913322,0.0637405582],[0,913588,0.0586192512],[0,914105,0.1378245533],[0,914577,0.065296939],[0,914584,0.1734510143],[0,914658,0.1205258918],[0,914706,0.0548503014],[0,914998,0.0553497951],[0,915020,0.070586223],[0,915038,0.1142540586],[0,915398,0.0884191291],[0,915449,0.2809775072],[0,915758,0.2131663476],[0,916380,0.1015160381],[0,916607,0.4205091402],[0,916844,0.0614551977],[0,917888,0.1003845024],[0,917912,0.158343608],[0,918011,0.0525111065],[0,918037,0.0610262622],[0,918315,0.2946133894],[0,918398,0.0528048535],[0,918498,0.1601745377],[0,918663,0.2156912237],[0,918797,0.0825645524],[0,918820,0.1089762906],[0,918833,0.056703708],[0,918898,0.0592246882],[0,919040,0.310056148],[0,919299,0.1044650104],[0,919316,0.1471073387],[0,919352,0.0513908024],[0,919399,0.0544361499],[0,919555,0.1633075942],[0,919577,0.0611861352],[0,919843,0.0543739037],[0,920079,0.0569991612],[0,920380,0.0592246882],[0,920521,0.0698595413],[0,920589,0.0659860405],[0,920708,0.2961072953],[0,920718,0.0634109718],[0,920811,0.0592246882],[0,921086,0.0592246882],[0,921208,0.1612158448],[0,921214,0.0559895769],[0,921233,0.2816156301],[0,921358,0.1088986762],[0,921469,0.163983205],[0,921543,0.0592246882],[0,921590,0.0812530353],[0,921665,0.2941864254],[0,921694,0.0553497951],[0,921796,0.3135782668],[0,921887,0.0952448966],[0,922040,0.0559895769],[0,922054,0.067211731],[0,922073,0.0700085665],[0,922080,0.0690563621],[0,922083,0.0777628887],[0,922295,0.0543739037],[0,922477,0.0553497951],[0,922522,0.0592246882],[0,922809,0.0856727192],[0,923051,0.0841582392],[0,923400,0.0804903489],[0,923421,0.0799823982],[0,923429,0.0700085665],[0,923457,0.1044363056],[0,923509,0.0598678912],[0,923797,0.0554595408],[0,923920,0.0585820244],[0,924242,0.0595403357],[0,924328,0.0543739037],[0,924364,0.2363384969],[0,924678,0.1404046899],[0,924742,0.2918038829],[0,924808,0.0609985133],[0,924816,0.0518323895],[0,924931,0.0976985307],[0,924960,0.2026009679],[0,925129,0.0877299685],[0,925176,0.0553497951],[0,925635,0.0619313739],[0,926418,0.1383609114],[0,926598,0.4304085172],[0,926827,0.0717277712],[0,927185,0.0568518718],[0,927208,0.0563816171],[0,927223,0.4727487278],[0,927361,0.0693186624],[0,927483,0.0829050734],[0,927499,0.2514104744],[0,927520,0.0614467936],[0,927672,0.0712995223],[0,927693,0.0699675083],[0,927712,0.187638623],[0,927785,0.053327175],[0,928106,0.053327175],[0,928172,0.0625122993],[0,928542,0.0603286437],[0,928608,0.1036365754],[0,928613,0.1007021272],[0,928739,0.0568518718],[0,928810,0.067807961],[0,928812,0.2021381518],[0,928944,0.0612128028],[0,929026,0.0630832453],[0,929144,0.1031455215],[0,929374,0.3103999569],[0,929465,0.0919883008],[0,929540,0.0625122993],[0,929621,0.0904193874],[0,929761,0.0625122993],[0,929774,0.0838011758],[0,929813,0.059384042],[0,929870,0.098922906],[0,929891,0.2002146446],[0,930274,0.0782357375],[0,930786,0.0850307134],[0,930914,0.0562140479],[0,930977,0.0764697019],[0,931119,0.2484732759],[0,931172,0.0791868994],[0,931296,0.0764697019],[0,931326,0.0562344476],[0,931352,0.1584183285],[0,931484,0.091319084],[0,931591,0.2693910069],[0,931592,0.0597533156],[0,931635,0.0603876329],[0,931640,0.2691591602],[0,931787,0.1219462978],[0,931932,0.0556178558],[0,932033,0.0572572728],[0,932105,0.077052407],[0,932261,0.0835393372],[0,932429,0.0526278135],[0,932481,0.4679672452],[0,932498,0.3027954161],[0,932564,0.1279838474],[0,932668,0.091863183],[0,932670,0.0625122993],[0,932746,0.3187908455],[0,932811,0.1961783834],[0,932872,0.1539410941],[0,933138,0.0666609843],[0,933463,0.064320858],[0,933543,0.0680170937],[0,934345,0.2539622309],[0,934395,0.2688019643],[0,934756,0.0648264707],[0,934774,0.7002446174],[0,934820,0.0771736041],[0,934832,0.6039471183],[0,935084,0.0588476698],[0,935189,0.1795749853],[0,935303,0.0803297877],[0,935313,0.0728330988],[0,935346,0.1780619652],[0,935424,0.2700135266],[0,935430,0.073966674],[0,935471,0.065335913],[0,935537,0.1133065269],[0,935565,0.1672807915],[0,935581,0.0798048736],[0,935773,0.1334297062],[0,935958,0.2891732055],[0,935975,0.3392626178],[0,936080,0.0849652744],[0,936225,0.2161423252],[0,936334,0.101617256],[0,936362,0.0746115048],[0,936366,0.0557808496],[0,936372,0.0542270259],[0,936651,0.1045089017],[0,936711,0.0588476698],[0,936750,0.0540782373],[0,936878,0.05808506],[0,937035,0.0728330988],[0,937228,0.0687632587],[0,937364,0.1235059741],[0,937368,0.0631191532],[0,937400,0.0616706432],[0,937445,0.3649587963],[0,937537,0.0770881303],[0,937598,0.0574540316],[0,937600,0.1562726512],[0,937611,0.0574540316],[0,937690,0.0592103997],[0,937706,0.321135297],[0,937920,0.0728092066],[0,937945,0.3315070796],[0,938127,0.0574405994],[0,938391,0.1207561219],[0,938468,0.0884608204],[0,938510,0.0947314283],[0,938593,0.0622799897],[0,938914,0.4186768746],[0,938986,0.0582712322],[0,939044,0.0843086303],[0,939096,0.0692105892],[0,939164,0.0631191532],[0,939166,0.0801116225],[0,939218,0.0757317429],[0,939224,0.0574540316],[0,939300,0.3513596138],[0,939418,0.0717471621],[0,939420,0.0567043812],[0,939474,0.0530954109],[0,940711,0.0631191532],[0,940947,0.0616332603],[0,941270,0.1951090364],[0,941456,0.3904640823],[0,941577,0.0770881303],[0,941615,0.1012990037],[0,941881,0.0973767632],[0,942059,0.0775740213],[0,942510,0.0953430208],[0,942580,0.099664947],[0,942600,0.0541441796],[0,942704,0.0770881303],[0,943025,0.1265019359],[0,943059,0.2765095999],[0,943287,0.0549016996],[0,943353,0.0582459115],[0,943481,0.5375361273],[0,943485,0.1620821847],[0,943792,0.0816764825],[0,944013,0.0775932776],[0,944275,0.1195860347],[0,944443,0.0775932776],[0,944450,0.0631191532],[0,944461,0.0770881303],[0,944564,0.3870769234],[0,944674,0.0587642945],[0,944695,0.0549016996],[0,944845,0.0916215271],[0,944937,0.0664438558],[0,944988,0.3123440376],[0,945094,0.0564753063],[0,945203,0.0574692872],[0,945271,0.1148536232],[0,945292,0.0754762703],[0,945303,0.1924065899],[0,945453,0.074084402],[0,945516,0.0907388877],[0,946064,0.0585673233],[0,946078,0.0584179639],[0,946772,0.0862633147],[0,947230,0.0631191532],[0,947240,0.0634101216],[0,947586,0.0739223373],[0,947732,0.1460295433],[0,947733,0.8132082492],[0,947859,0.0770881303],[0,948206,0.1254436751],[0,948254,0.0599413101],[0,948584,0.1659130899],[0,949784,0.6340786738],[0,950518,0.0771120329],[0,950783,0.0574405994],[0,950842,0.0608954602],[0,950855,0.3213018885],[0,951216,0.0617774105],[0,952340,0.0881788796],[0,952364,0.0549016996],[0,952417,0.0880710345],[0,952447,0.0573593677],[0,953188,0.3557855747],[0,953199,0.0564753063],[0,953314,0.0631191532],[0,953402,0.0587642945],[0,953634,0.0720141933],[0,953679,0.3472921241],[0,953782,0.0602033392],[0,953810,0.1211940751],[0,953857,0.0602033392],[0,953918,0.0816059837],[0,953939,0.2977906787],[0,954008,0.2136375275],[0,954190,0.0608331307],[0,954534,0.0907392203],[0,954955,0.1293190635],[0,955012,0.0711534311],[0,955058,0.0704676657],[0,955068,0.0745930328],[0,955130,0.5494103972],[0,955131,0.1007873126],[0,955261,0.1062038496],[0,955424,0.0615185591],[0,955476,0.0617685145],[0,955666,0.0780331116],[0,955702,0.070082968],[0,956022,0.0711224788],[0,956251,0.0663105989],[0,956286,0.1516018013],[0,956383,0.3244429563],[0,956506,0.1895542623],[0,956643,0.2157981275],[0,956669,0.1221170032],[0,956851,0.0803404469],[0,956906,0.0803404469],[0,956925,0.0669600582],[0,957222,0.0663105989],[0,957315,0.1391745259],[0,957365,0.1071741661],[0,957439,0.0783615487],[0,957475,0.0661256173],[0,957479,0.4119223181],[0,957593,0.0644472033],[0,957749,0.0728841496],[0,957900,0.0644472033],[0,958190,0.0644472033],[0,958249,0.0601139614],[0,958467,0.1039813275],[0,958644,0.1031835449],[0,959212,0.1189685753],[0,959317,0.0605296911],[0,959361,0.0769081098],[0,959610,0.0628385498],[0,959770,0.2063465282],[0,959870,0.1849109924],[0,960010,0.0589978502],[0,960016,0.0764118289],[0,960051,0.0583554147],[0,960554,0.0644472033],[0,960719,0.5353788749],[0,961043,0.1487580641],[0,961160,0.0583554147],[0,961439,0.2197191892],[0,961562,0.0557999171],[0,961599,0.0979383934],[0,963260,0.4685012228],[0,963633,0.0789475739],[0,963923,0.3901549391],[0,963934,0.1011303217],[0,963950,0.0646435134],[0,964096,0.0574553513],[0,964106,0.062193508],[0,964364,0.2059965248],[0,964434,0.1051935396],[0,965008,0.0825435681],[0,965529,0.1445859817],[0,965632,0.0777673269],[0,965706,0.0730837824],[0,965797,0.0755183409],[0,965845,0.5744326562],[0,965938,0.1115503544],[0,966049,0.1116514496],[0,966067,0.3260928094],[0,966081,0.0588006784],[0,966392,0.0628763565],[0,966512,0.0680506943],[0,966568,0.0613806881],[0,966751,0.1758531386],[0,966790,0.0670576979],[0,966925,0.1095543968],[0,966945,0.3632702498],[0,967106,0.0819565757],[0,967169,0.0680070016],[0,967208,0.1286483197],[0,967213,0.0869384671],[0,967293,0.0733413285],[0,967387,0.0902237781],[0,967584,0.0733088172],[0,967725,0.0729958231],[0,967732,0.0953271316],[0,967778,0.060044584],[0,968044,0.0665556266],[0,968070,0.1185300561],[0,968282,0.6448322689],[0,968392,0.0680599757],[0,968431,0.0748510681],[0,968674,0.0982913589],[0,968703,0.227365785],[0,968732,0.2674063902],[0,968866,0.0845354568],[0,968956,0.120148299],[0,968998,0.1750277169],[0,969193,0.080433311],[0,969203,0.4651981255],[0,969204,0.3014916429],[0,969222,0.0707203311],[0,969309,0.058352431],[0,969400,0.0753497429],[0,969497,0.0578772614],[0,969558,0.0749203161],[0,969564,0.0926198073],[0,969925,0.0626256862],[0,970133,0.080433311],[0,970222,0.0624047231],[0,970355,0.2055021862],[0,970767,0.0586397567],[0,971225,0.0665556266],[0,971355,0.0608098313],[0,971379,0.0665556266],[0,971560,0.7631222411],[0,971627,0.0768669946],[0,971695,0.1015311753],[0,971729,0.0612649924],[0,971735,0.0738623008],[0,971848,0.0622801327],[0,971864,0.558983879],[0,971882,0.0603338248],[0,972052,0.0628393962],[0,972059,0.0980850029],[0,972148,0.0604893784],[0,972219,0.0911222344],[0,972233,0.1219615638],[0,972250,0.080433311],[0,972423,0.0790610729],[0,972801,0.13177515],[0,972932,0.0897927819],[0,973169,0.0583152177],[0,973416,0.1001837605],[0,975118,0.1192165513],[0,975364,0.400014091],[0,975541,0.0798653685],[0,975556,0.0617244664],[0,976182,0.0807505151],[0,976188,0.2641623565],[0,976244,0.0951223693],[0,976320,0.0621427888],[0,976480,0.0860197281],[0,976714,0.0596885272],[0,976765,0.0917214004],[0,976975,0.0948956002],[0,977030,0.1264195241],[0,977286,0.0662922063],[0,977546,0.0574721448],[0,977584,0.0786479953],[0,977592,0.0743486942],[0,977750,0.0575093149],[0,977870,0.1793898666],[0,977875,0.1124219302],[0,978049,0.0830205677],[0,978528,0.117398253],[0,978812,0.1113850937],[0,980335,0.0975611532],[0,980456,0.0801649226],[0,980810,0.0604271909],[0,980986,0.1330909796],[0,980993,0.0871002207],[0,981038,0.0597834946],[0,981144,0.0662922063],[0,981603,0.0781785887],[0,981631,0.0659262539],[0,981668,0.1137287436],[0,981753,0.9163346389],[0,981779,0.1016495189],[0,982064,0.0640226521],[0,982254,0.0585004179],[0,982377,0.0884754005],[0,982458,0.221096827],[0,982464,0.0597834946],[0,982547,0.0780459235],[0,982548,0.1859287015],[0,982577,0.1543370839],[0,982696,0.1018039592],[0,982804,0.0912254],[0,982849,0.0784217754],[0,982945,0.0807505151],[0,982962,0.233053302],[0,983048,0.1070679778],[0,983180,0.1388825539],[0,983378,0.1050228043],[0,983387,0.0582331497],[0,983388,0.0662922063],[0,983672,0.0657586764],[0,983692,0.057027104],[0,983750,0.310967704],[0,984060,0.0662922063],[0,984161,0.0863775106],[0,984690,0.0725983517],[0,984890,0.0597834946],[0,985303,0.0908693967],[0,985468,0.1547224617],[0,985878,0.0580547474],[0,986034,0.3313892349],[0,986182,0.2269136618],[0,986271,0.0605440555],[0,986551,0.0656629307],[0,986588,0.058801632],[0,986630,0.107971519],[0,986745,0.0657974512],[0,986764,0.1938458057],[0,986814,0.0597834946],[0,987057,0.0597834946],[0,987348,0.179836919],[0,987376,0.1148227294],[0,987591,0.0806721523],[0,987731,0.0580547474],[0,987767,0.0750831743],[0,988134,0.2769028564],[0,988325,0.3398661042],[0,988327,0.1590768139],[0,988577,0.074598552],[0,988787,0.0597834946],[0,988901,0.0807505151],[0,988939,0.8034131402],[0,988970,0.1013358143],[0,989104,0.0732191154],[0,989105,0.0801649226],[0,989109,0.0625909642],[0,989121,0.1300820618],[0,989169,0.1726834902],[0,989191,0.1012724805],[0,989394,0.0572533751],[0,989400,0.0655690228],[0,989522,0.0818927851],[0,989874,0.0604271909],[0,990260,0.1801961646],[0,990287,0.0576169478],[0,990339,0.0863231941],[0,990998,0.1124336165],[0,991424,0.2573626383],[0,991490,0.2103931623],[0,992029,0.1051388015],[0,992030,0.0805622621],[0,992116,0.0801334049],[0,992119,0.1040694595],[0,992253,0.0571469164],[0,992830,0.0668698515],[0,992888,0.0616173526],[0,993020,0.061174514],[0,994543,0.4623902501],[0,994967,0.5195985445],[0,995014,0.2292854286],[0,995301,0.11916059],[0,995470,0.0661844232],[0,995637,0.1147462691],[0,997390,0.0800551067],[0,998948,0.222517143],[0,999267,0.0879865174],[0,999392,0.3167306161],[0,999508,0.0758968074],[0,999683,0.0595279795],[0,999868,0.1352263042],[0,1000204,0.1379941839],[0,1000262,0.1138569879],[0,1000641,0.0821878904],[0,1000727,0.1393882543],[0,1000788,0.0627920485],[0,1001018,0.0766268797],[0,1001297,0.0831812379],[0,1001303,0.0699317601],[0,1001335,0.2113398438],[0,1001597,0.2246045989],[0,1001658,0.0693228157],[0,1002168,0.0831026212],[0,1002250,0.074900427],[0,1002333,0.1017679979],[0,1003585,0.0686760716],[0,1003615,0.1001781934],[0,1003852,0.061913774],[0,1003871,0.0668978277],[0,1004091,0.3824936499],[0,1004249,0.0607900364],[0,1004911,0.5650764527],[0,1006059,0.0965379071],[0,1006086,0.0627998453],[0,1007820,0.0999138408],[0,1007844,0.1276308066],[0,1008350,0.0852234888],[0,1008588,0.0683730751],[0,1008762,0.0650918285],[0,1008833,0.061888908],[0,1009008,0.107212319],[0,1010571,0.1361385939],[0,1010580,0.2562055041],[0,1010612,0.0995063664],[0,1010636,0.0652656945],[0,1010663,0.0899502813],[0,1010998,0.0852234888],[0,1011018,0.0939401271],[0,1011122,0.1438851343],[0,1011502,0.0852234888],[0,1011586,0.2620629048],[0,1011875,0.071257134],[0,1011990,0.0621572922],[0,1012006,0.1097640604],[0,1012007,0.1166223099],[0,1012141,0.2101344972],[0,1012249,0.1052940358],[0,1012410,0.0852234888],[0,1012890,0.0742060631],[0,1012900,0.063621351],[0,1012901,0.1047287962],[0,1012937,0.0845553485],[0,1013310,0.0702598192],[0,1013311,0.071257134],[0,1013396,0.1521220543],[0,1013432,0.1038667878],[0,1013489,0.1172282736],[0,1013586,0.1088680948],[0,1013766,0.0652656945],[0,1013792,0.0890248516],[0,1013933,0.1457054297],[0,1014182,0.2250897263],[0,1014211,0.6779365133],[0,1014237,0.0647350029],[0,1014328,0.0762090336],[0,1014722,0.0639132452],[0,1014775,0.1593918137],[0,1014796,0.0722431566],[0,1014929,0.1062600281],[0,1015164,0.1218028831],[0,1015210,0.0844977856],[0,1015211,0.1896140536],[0,1015334,0.0937276193],[0,1015858,0.6347453426],[0,1016897,0.1277058834],[0,1016914,0.0858130339],[0,1017015,0.1854039094],[0,1017094,0.0621090736],[0,1017197,0.0634127523],[0,1017397,0.071257134],[0,1017710,0.2663353437],[0,1017750,0.0631436801],[0,1017821,0.1128879974],[0,1017921,0.0853204683],[0,1017955,0.0799540431],[0,1018349,0.0631436801],[0,1018541,0.2942481192],[0,1018653,0.1119953733],[0,1018733,0.1362247709],[0,1018825,0.1260580111],[0,1019036,0.2616445608],[0,1019118,0.0659750616],[0,1019203,0.2498739774],[0,1019378,0.0701662604],[0,1019498,0.1379270092],[0,1019565,0.0637159996],[0,1019774,0.0761868398],[0,1019904,0.0661012888],[0,1019957,0.0653525308],[0,1019995,0.3382570216],[0,1020054,0.1192020699],[0,1020074,0.094736731],[0,1020098,0.065256838],[0,1020142,0.3055800987],[0,1020163,0.089009342],[0,1020286,0.1930138804],[0,1020312,0.1194514826],[0,1020366,0.0675014185],[0,1020410,0.2101587155],[0,1020497,0.071257134],[0,1020777,0.322966464],[0,1021054,0.1138477105],[0,1021171,0.071257134],[0,1021194,0.0681824141],[0,1021411,0.071257134],[0,1021858,0.0703653378],[0,1021861,0.0935093764],[0,1022283,0.2700688439],[0,1022402,0.545551812],[0,1022492,0.0735050716],[0,1022767,0.1018447741],[0,1022798,0.5486462683],[0,1022799,0.0759760584],[0,1022805,0.0722616704],[0,1022896,0.152880572],[0,1022911,0.0932659859],[0,1022938,0.173931841],[0,1022952,0.1171460721],[0,1022981,0.0716739155],[0,1023063,0.0798851233],[0,1023104,0.0765170047],[0,1023340,0.3624562321],[0,1023511,0.0848964721],[0,1023515,0.2076518768],[0,1023614,0.1032418377],[0,1023669,0.3591237597],[0,1024053,0.0719979193],[0,1024892,0.1012081255],[0,1025704,0.0939820139],[0,1025728,0.1458549224],[0,1025807,0.1371484121],[0,1025827,0.1047965953],[0,1025968,0.64249834],[0,1025974,0.1075190826],[0,1026134,0.0786779718],[0,1026176,0.0862215497],[0,1026177,0.0905825606],[0,1026237,0.0694606057],[0,1026493,0.2423248708],[0,1026530,0.0765170047],[0,1026572,0.1807094669],[0,1026620,0.1221765994],[0,1026783,0.175248383],[0,1026985,0.2331245494],[0,1026997,0.0665961874],[0,1027098,0.1181192345],[0,1027124,0.0765170047],[0,1027140,0.0765170047],[0,1028535,0.1024120863],[0,1028623,0.0723014302],[0,1028721,0.2158543684],[0,1028737,0.0764198576],[0,1028977,0.0978468488],[0,1029057,0.4955809292],[0,1029171,0.0765170047],[0,1029410,0.0765170047],[0,1029455,0.0821673974],[0,1029544,0.0905825606],[0,1029545,0.2822193968],[0,1029680,0.0665961874],[0,1029790,0.067352528],[0,1029895,0.2110965584],[0,1030009,0.0863815307],[0,1030012,0.1430030764],[0,1030067,0.186395686],[0,1030533,0.0916114697],[0,1030609,0.0750305078],[0,1030823,0.0679702497],[0,1030864,0.0731183119],[0,1030884,0.0705976894],[0,1030895,0.0705704619],[0,1030975,0.2939603516],[0,1031105,0.067352528],[0,1031132,0.1042371144],[0,1031190,0.1023061054],[0,1031191,0.0891169263],[0,1031440,0.0741151003],[0,1031556,0.1377059948],[0,1031701,0.0765170047],[0,1031716,0.0802751128],[0,1031766,0.0751505621],[0,1031847,0.1140209842],[0,1031907,0.2002233944],[0,1031985,0.0941667484],[0,1031986,0.1849607964],[0,1032175,0.1018633164],[0,1032239,0.1590881748],[0,1032288,0.2626162064],[0,1032650,0.1002866602],[0,1032772,0.1621205629],[0,1032775,0.0726749663],[0,1032888,0.0671851779],[0,1032974,0.82363252],[0,1033026,0.3407242236],[0,1033079,0.6628821443],[0,1033466,0.0911762932],[0,1033551,0.1088814242],[0,1034348,0.0754713394],[0,1034422,0.0765170047],[0,1034440,0.0866721665],[0,1034457,0.0688261489],[0,1034469,0.0714707384],[0,1034548,0.0993260616],[0,1034559,0.1107060169],[0,1034702,0.0870618551],[0,1034800,0.0756873403],[0,1034996,0.1184151937],[0,1035019,0.0699178177],[0,1035098,0.1091569202],[0,1035240,0.3573289149],[0,1035432,0.1195539916],[0,1035530,0.1034909136],[0,1035631,0.0765170047],[0,1035707,0.0963269651],[0,1035798,0.1018956181],[0,1036141,0.222741271],[0,1036194,0.0699178177],[0,1036290,0.0904142345],[0,1036355,0.1499209819],[0,1036369,0.5471662937],[0,1036429,0.6377649325],[0,1036572,0.0699178177],[0,1036970,0.1171481006],[0,1037147,0.0911762932],[0,1037164,0.0709009645],[0,1037174,0.1839722367],[0,1037424,0.5983687686],[0,1037451,0.1173592187],[0,1037457,0.0765170047],[0,1037559,0.1628814915],[0,1037950,0.1656979614],[0,1038035,0.1647839094],[0,1038137,0.0699178177],[0,1038286,0.6051292452],[0,1038330,0.126642839],[0,1038379,0.0960750302],[0,1038397,0.0765170047],[0,1038740,0.0853085953],[0,1038873,0.0709009645],[0,1039215,0.0699905471],[0,1039236,0.2537044832],[0,1039801,0.0765170047],[0,1039932,0.2743408708],[0,1040102,0.0905825606],[0,1040403,0.0780686917],[0,1040608,0.1023493174],[0,1040638,0.0693487859],[0,1041334,0.0770217655],[0,1041453,0.0781660536],[0,1041563,0.0742712322],[0,1041727,0.0709228188],[0,1041833,0.0770217655],[0,1041877,0.1497550449],[0,1041927,0.1083503383],[0,1042097,0.0770217655],[0,1042408,0.1089312258],[0,1042425,0.0888192349],[0,1042478,0.1201863472],[0,1042511,0.5935793192],[0,1042614,0.360404596],[0,1043032,0.1521960465],[0,1043173,0.0759196052],[0,1043365,0.0914986969],[0,1043673,0.0959937471],[0,1044855,0.2854058461],[0,1045616,0.2012621068],[0,1046003,0.0910968411],[0,1046757,0.1015634594],[0,1046881,0.0760166871],[0,1047324,0.0818127646],[0,1047814,0.1164985189],[0,1048354,0.0882117546],[0,1048465,0.1028790939],[0,1048866,0.1338654831],[0,1049144,0.0688503292],[0,1049195,0.0950977529],[0,1049277,0.4598806553],[0,1049282,0.1521295101],[0,1049454,0.8853164588],[0,1049911,0.1009500555],[0,1049975,0.2533213592],[0,1050007,0.4871837334],[0,1050175,0.0801405751],[0,1050258,0.5497225527],[0,1050460,0.0742156943],[0,1050507,0.0719210276],[0,1050581,0.0770217655],[0,1050821,0.8896975584],[0,1050832,0.106549167],[0,1051008,0.0910968411],[0,1051142,0.0719210276],[0,1051143,0.1052686208],[0,1051204,0.1908628083],[0,1051223,0.1219742442],[0,1051407,0.0862545495],[0,1051697,0.1949274755],[0,1051816,0.0916909756],[0,1051848,0.5383332717],[0,1052093,0.0812532599],[0,1052557,0.0770217655],[0,1052592,0.1356059123],[0,1052716,0.0725754405],[0,1052819,0.6575090758],[0,1052868,0.1166443868],[0,1053054,0.084655063],[0,1053148,0.0997647162],[0,1053185,0.0770217655],[0,1053340,0.2398990221],[0,1053390,0.0916114697],[0,1053409,0.1025653266],[0,1053501,0.0770217655],[0,1053587,0.1042036783],[0,1053706,0.0994545917],[0,1053752,0.1008957754],[0,1053822,0.0717257789],[0,1053912,0.3220922223],[0,1053976,0.0719210276],[0,1054121,0.077699593],[0,1054216,0.230307857],[0,1054531,0.1372603714],[0,1054799,0.0715608837],[0,1054851,0.4891521403],[0,1054899,0.0717261371],[0,1054915,0.4895304791],[0,1054992,0.1788607492],[0,1054999,0.0817708904],[0,1055323,0.1087318313],[0,1055654,0.1100582561],[0,1055992,0.0693000932],[0,1055997,0.0753001618],[0,1056033,0.0709228188],[0,1056067,0.1867296343],[0,1056727,0.1068799487],[0,1056767,0.1304959724],[0,1056917,0.069077763],[0,1057160,0.0693000932],[0,1057207,0.0696192141],[0,1057304,0.1504134638],[0,1057314,0.1032214888],[0,1057316,0.3186811762],[0,1057322,0.0740067663],[0,1057429,0.294203001],[0,1057439,0.1066022606],[0,1057475,0.1424443168],[0,1057538,0.0910968411],[0,1057783,0.0696192141],[0,1057788,0.0918746544],[0,1057866,0.0770217655],[0,1057883,0.0838824842],[0,1058467,0.068981585],[0,1058734,0.0910968411],[0,1058805,0.0942163313],[0,1058822,0.0760166871],[0,1058924,0.0688503292],[0,1059395,0.0719210276],[0,1060487,0.0747454563],[0,1060742,0.0910968411],[0,1060993,0.2081328562],[0,1061040,0.1219921333],[0,1061091,0.4085001531],[0,1061342,0.5873925024],[0,1061346,0.1481817325],[0,1061578,0.2371541316],[0,1061673,0.3686987748],[0,1061995,0.0719939541],[0,1062101,0.1320538745],[0,1062148,0.0770217655],[0,1062284,0.1174777216],[0,1062290,0.1719400671],[0,1062303,0.0910968411],[0,1062355,0.1166336389],[0,1062693,0.0959032214],[0,1062853,0.0708264004],[0,1062986,0.1338327529],[0,1063018,0.0785686318],[0,1063025,0.0950514719],[0,1063065,0.1262721022],[0,1063067,0.0760166871],[0,1063068,0.2882851436],[0,1063173,0.0916909756],[0,1063216,0.2444477832],[0,1063328,0.0725754405],[0,1063362,0.0733316909],[0,1063585,0.2655981431],[0,1063593,0.085279713],[0,1063681,0.0811855718],[0,1063690,0.1730580537],[0,1063706,0.1597527346],[0,1063864,0.1109161661],[0,1063870,0.1605693715],[0,1063954,0.0770217655],[0,1063972,0.096991466],[0,1064004,0.0915012737],[0,1064056,0.0770217655],[0,1064252,0.1073180947],[0,1064386,0.0777646672],[0,1064417,0.0739428856],[0,1064445,0.0770217655],[0,1064497,0.0863189624],[0,1064510,0.1069242892],[0,1064511,0.2138137462],[0,1064520,0.0898605216],[0,1064597,0.0770217655],[0,1064637,0.0714625765],[0,1064717,0.0776440861],[0,1064784,0.0857053831],[0,1065338,0.0895858465],[0,1065367,0.0742795985],[0,1065456,0.0881403616],[0,1065549,0.1352997057],[0,1065552,0.5226956035],[0,1065701,0.0887688168],[0,1065723,0.0933943404],[0,1065726,0.3644954411],[0,1066036,0.0879018585],[0,1066635,0.3242194061],[0,1066701,0.081966918],[0,1066717,0.1152376045],[0,1066805,0.0910968411],[0,1066847,0.0910968411],[0,1067004,0.2975134339],[0,1067013,0.0811066596],[0,1067086,0.0814626782],[0,1068370,0.0765170047],[0,1068454,0.0765170047],[0,1068489,0.1004368371],[0,1068679,0.1372901902],[0,1068838,0.1045519645],[0,1068915,0.0704221831],[0,1068944,0.3067010086],[0,1068945,0.876045586],[0,1069076,0.266413992],[0,1069171,0.0827742605],[0,1069192,0.3083660538],[0,1069203,0.1065263036],[0,1069327,0.2565990194],[0,1069386,0.135176739],[0,1069431,0.5907984389],[0,1069505,0.1004163401],[0,1069565,0.1358900774],[0,1069626,0.2827374229],[0,1069641,0.1078718982],[0,1070181,0.1094471722],[0,1070183,0.0704221831],[0,1070270,0.0844094373],[0,1070295,0.2941106144],[0,1070368,0.093116984],[0,1070392,0.1715951727],[0,1070821,0.3040425324],[0,1071027,0.2606377115],[0,1071035,0.3169589044],[0,1071124,0.3038726057],[0,1071272,0.0765170047],[0,1071278,0.0959675775],[0,1071362,0.3274865053],[0,1071437,0.0698446273],[0,1071472,0.1078189845],[0,1071479,0.0875882237],[0,1071505,0.1398201546],[0,1071760,0.0803984348],[0,1071965,0.0905825606],[0,1072046,0.0905853081],[0,1072083,0.3122886592],[0,1072113,0.0766741839],[0,1072135,0.0873071932],[0,1072238,0.0719860562],[0,1072359,0.3722361758],[0,1072559,0.3280748037],[0,1072648,0.0899892915],[0,1072919,0.2127855113],[0,1073019,0.1042875173],[0,1073615,0.0899892915],[0,1073632,0.0789181422],[0,1073714,0.0910937251],[0,1073912,0.1115656872],[0,1073963,0.1208648062],[0,1074455,0.0925247194],[0,1074524,0.0677740711],[0,1074796,0.1066377223],[0,1074855,0.1315237862],[0,1075261,0.1764890643],[0,1075487,0.0703408288],[0,1076078,0.075430692],[0,1076689,0.0703715765],[0,1077031,0.0936049711],[0,1077104,0.1573302015],[0,1077332,0.0740375862],[0,1077468,0.0899892915],[0,1077528,0.3521534731],[0,1077601,0.075430692],[0,1077663,0.1738279888],[0,1077772,0.121478368],[0,1077778,0.2325024754],[0,1077779,0.2870094068],[0,1077780,0.1380363021],[0,1077890,0.0717484846],[0,1077916,0.0805141573],[0,1077923,0.0762007932],[0,1077959,0.1760416126],[0,1078109,0.1054124198],[0,1078250,0.1531507458],[0,1078312,0.0701463538],[0,1078397,0.3973649965],[0,1078414,0.0720203576],[0,1078431,0.0933784868],[0,1078468,0.1449619605],[0,1078541,0.0877109269],[0,1078732,0.0703408288],[0,1078801,0.1134103391],[0,1078956,0.0964644852],[0,1079335,0.1055346006],[0,1079359,0.1329399715],[0,1079415,0.0703408288],[0,1079442,0.0860143949],[0,1079677,0.1850203129],[0,1079770,0.0708810191],[0,1079825,0.0677254822],[0,1079829,0.1916043256],[0,1079912,0.2352922233],[0,1080135,0.0723969332],[0,1080243,0.0708765569],[0,1080622,0.1084576997],[0,1080624,0.0698833551],[0,1080674,0.0802010225],[0,1080764,0.0688060106],[0,1081100,0.2156184859],[0,1081106,0.6758683979],[0,1081132,0.1030969321],[0,1081195,0.0727901407],[0,1081217,0.0928643842],[0,1081315,0.123354582],[0,1081323,0.0838301766],[0,1081367,0.2092141787],[0,1081469,0.0805009293],[0,1081613,0.068025421],[0,1081619,0.7607732822],[0,1082787,0.0703408288],[0,1082833,0.0672390074],[0,1083035,0.1596514025],[0,1083099,0.0702444863],[0,1083118,0.075430692],[0,1083121,0.0703408288],[0,1083693,0.0933132322],[0,1084069,0.0951110806],[0,1084276,0.0894757601],[0,1084380,0.0852602717],[0,1084620,0.1728529824],[0,1084625,0.3564471959],[0,1084827,0.0921514807],[0,1084843,0.1047025215],[0,1085101,0.0909851858],[0,1085150,0.0887637777],[0,1085237,0.1735730161],[0,1085375,0.1080475821],[0,1085476,0.0720203576],[0,1085591,0.075430692],[0,1085592,0.0702444863],[0,1085663,0.3127635684],[0,1085759,0.0675036259],[0,1085802,0.1196706211],[0,1085852,0.1321368518],[0,1086175,0.1036467074],[0,1086182,0.134010692],[0,1086542,0.0970810649],[0,1086599,0.0900686279],[0,1086629,0.1090216024],[0,1086733,0.3067911119],[0,1086818,0.0762979117],[0,1086990,0.1075940907],[0,1087155,0.3121388423],[0,1087266,0.0940502035],[0,1087448,0.0900244851],[0,1087617,0.2173065919],[0,1087789,0.0677740711],[0,1088224,0.0769654895],[0,1088228,0.0781702344],[0,1088924,0.0928620189],[0,1088951,0.3435787534],[0,1089205,0.073061647],[0,1089437,0.0784666637],[0,1089953,0.0895079215],[0,1090135,0.132707406],[0,1090486,0.4523752965],[0,1090558,0.1040261889],[0,1090730,0.1229627089],[0,1091051,0.2426119722],[0,1091541,0.111058286],[0,1091634,0.2904522314],[0,1091644,0.0771274455],[0,1091777,0.0798801151],[0,1092159,0.0940090992],[0,1092160,0.5912576612],[0,1092266,0.2807434011],[0,1092322,0.1120768943],[0,1092444,0.094557138],[0,1092641,0.1492975237],[0,1092767,0.124290872],[0,1092930,0.3601886205],[0,1093184,0.5100195222],[0,1094033,0.4800803919],[0,1094045,0.4194632644],[0,1094259,0.1060287507],[0,1094334,0.1597664717],[0,1094542,0.59557103],[0,1094670,0.0993728981],[0,1094719,0.0747598411],[0,1094761,0.237555538],[0,1094881,0.0890512673],[0,1095109,0.0750638925],[0,1095159,0.0745642041],[0,1095227,0.0806642165],[0,1095310,0.9324647364],[0,1095449,0.0941392103],[0,1095467,0.0855630214],[0,1096416,0.0721777475],[0,1096429,0.1381571528],[0,1096571,0.1735040053],[0,1096842,0.0940090992],[0,1097335,0.0940090992],[0,1097559,0.1272148802],[0,1097569,0.1412352692],[0,1097611,0.1523070802],[0,1097693,0.1157026544],[0,1097742,0.0731776488],[0,1097796,0.1364437192],[0,1097841,0.1545509458],[0,1098281,0.084137511],[0,1098341,0.0721777475],[0,1098372,0.468868256],[0,1098391,0.0825732459],[0,1098519,0.0806258622],[0,1098561,0.1003484641],[0,1098704,0.0798801151],[0,1098873,0.0838157627],[0,1098978,0.0919555756],[0,1099035,0.1092104847],[0,1099150,0.0738299873],[0,1099292,0.0886361166],[0,1099434,0.0719056866],[0,1099459,0.1583818256],[0,1099553,0.3234683758],[0,1099556,0.0877077886],[0,1099689,0.2710601438],[0,1099778,0.0946055092],[0,1099910,0.0736610214],[0,1100000,0.2826046556],[0,1100621,0.0731776488],[0,1100798,0.0983947594],[0,1100945,0.4404250553],[0,1101027,0.0946055092],[0,1101141,0.1097067129],[0,1101163,0.113230536],[0,1101166,0.1144084778],[0,1101176,0.073757809],[0,1101336,0.0770892721],[0,1101339,0.6996513982],[0,1101399,0.1236055728],[0,1101426,0.0945256989],[0,1101599,0.1268634918],[0,1102018,0.1131578866],[0,1102146,0.102037568],[0,1102663,0.107242153],[0,1103428,0.0754441662],[0,1103561,0.0721777475],[0,1103572,0.0937604339],[0,1103963,0.0830403691],[0,1104218,0.1021602801],[0,1104290,0.0724389449],[0,1104618,0.0715076621],[0,1104649,0.074649858],[0,1104933,0.1592588014],[0,1104974,0.0731778793],[0,1105005,0.0771190471],[0,1105053,0.6109735629],[0,1105069,0.0770892721],[0,1105730,0.4479088737],[0,1106386,0.1599491524],[0,1106689,0.0856320579],[0,1106808,0.2085838835],[0,1106898,0.1132351563],[0,1107141,0.0798801151],[0,1107289,0.1724977935],[0,1107394,0.0721288683],[0,1107580,0.1001389151],[0,1107736,0.1532495271],[0,1107819,0.1456152464],[0,1107944,0.439482305],[0,1107952,0.1003399976],[0,1107977,0.0814391197],[0,1108091,0.1041338823],[0,1108200,0.2564673245],[0,1108430,0.1005646378],[0,1108649,0.0745600116],[0,1108708,0.0863994977],[0,1108770,0.3904371972],[0,1108776,0.0776594398],[0,1108861,0.1183325119],[0,1108909,0.0799060216],[0,1108942,0.0798801151],[0,1108978,0.3232151317],[0,1109043,0.2455285603],[0,1109099,0.3431967637],[0,1109329,0.1628005201],[0,1109343,0.421883707],[0,1109417,0.0940090992],[0,1109464,0.0798801151],[0,1109503,0.1066295499],[0,1109614,0.1104511686],[0,1109664,0.3469298981],[0,1109889,0.0747598411],[0,1110004,0.6811367594],[0,1110039,0.0917273473],[0,1110063,0.0798801151],[0,1110079,0.0716773817],[0,1110442,0.1104796525],[0,1110518,0.1242950543],[0,1110793,0.0798801151],[0,1111035,0.0946055092],[0,1111076,0.1129713835],[0,1111103,0.0896145426],[0,1111127,0.1473049854],[0,1111132,0.0765557691],[0,1111207,0.0990851571],[0,1111704,0.1056047849],[0,1111715,0.9510385389],[0,1111939,0.1009033915],[0,1111983,0.1655938927],[0,1112047,0.2930684645],[0,1112059,0.4360409279],[0,1112082,0.1044898774],[0,1112166,0.1058459278],[0,1112202,0.0798801151],[0,1112205,0.1261676001],[0,1112213,0.2917075127],[0,1112219,0.0798801151],[0,1112420,0.5927018206],[0,1112471,0.0798801151],[0,1112645,0.0940090992],[0,1112768,0.3626745254],[0,1112773,0.1197658444],[0,1112797,0.105591753],[0,1113128,0.085561059],[0,1113304,0.0747598411],[0,1113343,0.08301087],[0,1113673,0.4863362674],[0,1114331,0.0737816973],[0,1114410,0.0914348902],[0,1114431,0.103872961],[0,1114474,0.0940090992],[0,1114529,0.0827019379],[0,1114599,0.0736610214],[0,1114641,0.1032949837],[0,1114803,0.0798801151],[0,1115081,0.4986240898],[0,1115144,0.2672371705],[0,1115150,0.2269278436],[0,1115235,0.113942939],[0,1115293,0.0793154443],[0,1115326,0.1099780587],[0,1115340,0.1976967858],[0,1115410,0.1114461574],[0,1115539,0.2976912305],[0,1115661,0.0724871476],[0,1115679,0.0721777475],[0,1115909,0.1951215485],[0,1116051,0.0940090992],[0,1116125,0.2570992718],[0,1116173,0.0871422004],[0,1116328,0.0798801151],[0,1116346,0.1677351858],[0,1116350,0.1119286826],[0,1116477,0.1218534818],[0,1116547,0.073757809],[0,1116558,0.0753287344],[0,1116568,0.0903071162],[0,1116606,0.0793723742],[0,1116632,0.1054639961],[0,1116721,0.1581269456],[0,1116811,0.0940090992],[0,1116881,0.072166788],[0,1116967,0.1385682252],[0,1117069,0.0756308845],[0,1117262,0.0931331131],[0,1117466,0.523632903],[0,1117762,0.07922427],[0,1117898,0.0768532281],[0,1118021,0.0937288387],[0,1118080,0.0868058778],[0,1118614,0.0721288683],[0,1118652,0.0764494047],[0,1118704,0.0745642041],[0,1118719,0.0967185831],[0,1118720,0.0880698718],[0,1118885,0.1287173449],[0,1118970,0.0805221897],[0,1119038,0.0835773782],[0,1119051,0.0936491199],[0,1119222,0.0731776488],[0,1119253,0.0805821607],[0,1119416,0.3099224908],[0,1119476,0.0731776488],[0,1119659,0.0719056866],[0,1119708,0.0724492115],[0,1120028,0.1921715342],[0,1120072,0.1192721909],[0,1120154,0.1086274274],[0,1120301,0.4520045821],[0,1120303,0.0750258246],[0,1121759,0.0721288683],[0,1121923,0.0863359897],[0,1121959,0.080857766],[0,1122093,0.0859072019],[0,1122139,0.0739197455],[0,1122234,0.1248958968],[0,1122240,0.1116019947],[0,1122260,0.2393895534],[0,1122264,0.225609694],[0,1122536,0.0823276166],[0,1122562,0.0986943236],[0,1122577,0.5634890049],[0,1122599,0.3342824734],[0,1122707,0.0824943738],[0,1122752,0.0712817804],[0,1122797,0.127538994],[0,1122819,0.2735304281],[0,1123282,0.0791660567],[0,1123299,0.1419460666],[0,1123580,0.073757809],[0,1123847,0.0851297221],[0,1124022,0.0712018259],[0,1124029,0.0712326094],[0,1124319,0.1218670692],[0,1124396,0.0931496572],[0,1124401,0.0744459158],[0,1124540,0.0715925937],[0,1124564,0.1381050629],[0,1124647,0.1032078409],[0,1124661,0.3440722355],[0,1124774,0.0754419551],[0,1124782,0.0735581082],[0,1124912,0.0894872355],[0,1125029,0.155839465],[0,1125061,0.1049341146],[0,1125382,0.0817424111],[0,1125435,0.2012064108],[0,1125442,0.0788754327],[0,1125573,0.4748292501],[0,1125790,0.176433681],[0,1126112,0.0754419551],[0,1126161,0.0828825043],[0,1126188,0.0856098933],[0,1126269,0.0972102886],[0,1126279,0.0736608363],[0,1126743,0.0943662239],[0,1126806,0.2433689294],[0,1126951,0.1520874338],[0,1126978,0.072347408],[0,1127141,0.072170845],[0,1127348,0.0769963366],[0,1127525,0.0727079343],[0,1127805,0.0879794622],[0,1127840,0.0727079343],[0,1127929,0.0675134793],[0,1128274,0.1232299949],[0,1128294,0.1172865661],[0,1128306,0.0727079343],[0,1128360,0.7147818671],[0,1128410,0.0684833717],[0,1128411,0.1069567405],[0,1128484,0.0872923513],[0,1128774,0.1117222554],[0,1128811,0.0766184887],[0,1128952,0.1456894572],[0,1129090,0.1899156125],[0,1129111,0.0923901285],[0,1129256,0.1471434334],[0,1129346,0.2458720695],[0,1129454,0.121003316],[0,1129824,0.0707931521],[0,1129826,0.0767619758],[0,1129938,0.627989965],[0,1130013,0.21423562],[0,1130243,0.0656532941],[0,1130634,0.0900136101],[0,1130656,0.0684833717],[0,1130833,0.135042197],[0,1131124,0.0659230601],[0,1131134,0.0786865772],[0,1131507,0.0907478456],[0,1131630,0.1577918498],[0,1131678,0.0791451573],[0,1131758,0.0785960265],[0,1131811,0.0898161897],[0,1131829,0.0915956847],[0,1131833,0.0684833717],[0,1131845,0.0727079343],[0,1132103,0.098143972],[0,1132252,0.0915956847],[0,1132324,0.0800764755],[0,1132368,0.0727079343],[0,1132385,0.0672143638],[0,1132409,0.1067593133],[0,1132414,0.0691347494],[0,1132476,0.11552317],[0,1132620,0.0687759593],[0,1132622,0.0669145264],[0,1132753,0.0658745932],[0,1132796,0.0684833717],[0,1132936,0.0727079343],[0,1133082,0.0747598065],[0,1133098,0.0824130418],[0,1133126,0.1506270731],[0,1133127,0.1626521513],[0,1133228,0.0940626276],[0,1133262,0.0760665132],[0,1133474,0.0659230601],[0,1133630,0.0717126878],[0,1134523,0.1277689735],[0,1134600,0.1834466432],[0,1134636,0.0823000213],[0,1134936,0.0727079343],[0,1134948,0.0968920733],[0,1135064,0.6851115941],[0,1135107,0.0777049469],[0,1135253,0.1633352379],[0,1136016,0.0727079343],[0,1136041,0.1670146684],[0,1136171,0.1927719515],[0,1136613,0.0727079343],[0,1137141,0.1837868689],[0,1137237,0.0917078339],[0,1137279,0.0690222053],[0,1137479,0.0661737787],[0,1137526,0.087213305],[0,1137563,0.092251771],[0,1137960,0.072347408],[0,1138169,0.6523645184],[0,1138380,0.0662695909],[0,1138387,0.0674897925],[0,1139085,0.0983744597],[0,1139096,0.066182054],[0,1139134,0.089549985],[0,1139343,0.0872923513],[0,1139403,0.0867016512],[0,1139436,0.1078011229],[0,1139474,0.069038564],[0,1139507,0.0867016512],[0,1139618,0.0958110646],[0,1139966,0.288371539],[0,1140094,0.0872923513],[0,1140368,0.1048870122],[0,1140375,0.1828538901],[0,1140446,0.1345575274],[0,1140458,0.0659230601],[0,1140651,0.0788017874],[0,1140699,0.0685559599],[0,1140920,0.1219031969],[0,1140952,0.1074991132],[0,1141295,0.0685559599],[0,1141338,0.0684833717],[0,1141351,0.1531280769],[0,1141587,0.4680811268],[0,1141635,0.3740895587],[0,1141848,0.1105930492],[0,1141908,0.0684833717],[0,1142135,0.1267287501],[0,1142381,0.0850882536],[0,1142506,0.0684833717],[0,1142556,0.1118673588],[0,1142715,0.1193437957],[0,1142834,0.3161392559],[0,1143271,0.1151028434],[0,1143288,0.092173984],[0,1143363,0.0899931725],[0,1143374,0.0718468335],[0,1143503,0.1113686708],[0,1143602,0.0809403948],[0,1143679,0.1069034824],[0,1144313,0.4240923602],[0,1144415,0.109648802],[0,1144437,0.0698873624],[0,1144502,0.066952408],[0,1144655,0.083162231],[0,1144664,0.0682893851],[0,1144736,0.0763994797],[0,1145012,0.1038268514],[0,1145027,0.3027098594],[0,1145046,0.1440845987],[0,1145220,0.271298655],[0,1145385,0.0698873624],[0,1145577,0.0867016512],[0,1146037,0.0872923513],[0,1146153,0.065912193],[0,1146165,0.0809587192],[0,1146611,0.0684833717],[0,1146823,0.0867016512],[0,1146827,0.0691347494],[0,1146893,0.2993801659],[0,1147003,0.2833337258],[0,1147116,0.3231723428],[0,1147390,0.1090940265],[0,1147449,0.0684833717],[0,1147641,0.0650346511],[0,1147852,0.0717086657],[0,1147956,0.0788153493],[0,1148415,0.1249636304],[0,1148434,0.0867016512],[0,1148597,0.1868016946],[0,1148938,0.0661444404],[0,1149201,0.110988935],[0,1149905,0.2229211806],[0,1149969,0.3004993019],[0,1150027,0.0743379392],[0,1150065,0.0867016512],[0,1150188,0.0872923513],[0,1150307,0.0727079343],[0,1150661,0.1058479989],[0,1150724,0.1755172667],[0,1150795,0.097495392],[0,1151023,0.0669145264],[0,1151098,0.066182054],[0,1151500,0.087213305],[0,1151511,0.3362281215],[0,1151514,0.0925314508],[0,1151521,0.087213305],[0,1151542,0.310043465],[0,1151546,0.0710943394],[0,1151631,0.4568839196],[0,1152124,0.0954947694],[0,1152520,0.1437617908],[0,1152621,0.0867016512],[0,1152786,0.0771366828],[0,1153165,0.1273362871],[0,1153338,0.0761217069],[0,1153345,0.0777592113],[0,1153432,0.0761217069],[0,1153482,0.0901798074],[0,1153486,0.0901798074],[0,1153944,0.0761217069],[0,1154125,0.1221578782],[0,1154184,0.0759945371],[0,1154224,0.157706675],[0,1154305,0.0806743344],[0,1154352,0.0742277746],[0,1154546,0.0739253127],[0,1154681,0.0717165487],[0,1154700,0.1075399514],[0,1154723,0.0718777075],[0,1154803,0.2992452912],[0,1154945,0.0901798074],[0,1154949,0.0761217069],[0,1155097,0.0901798074],[0,1155192,0.1193416697],[0,1155277,0.3547807236],[0,1155345,0.070602861],[0,1155417,0.1129331778],[0,1155525,0.1026298611],[0,1155803,0.0906938154],[0,1155837,0.188262598],[0,1155876,0.5541755566],[0,1155920,0.072180581],[0,1156241,0.0901798074],[0,1156478,0.0774459538],[0,1156570,0.0856916421],[0,1156912,0.0695574883],[0,1156926,0.1012165474],[0,1157410,0.0761217069],[0,1157489,0.0901798074],[0,1157494,0.0901798074],[0,1157636,0.2602627204],[0,1157755,0.1196093505],[0,1158043,0.713160185],[0,1158108,0.0718777075],[0,1158362,0.1199091639],[0,1158567,0.1382108028],[0,1158661,0.0900728567],[0,1158731,0.0907732253],[0,1158743,0.0768144674],[0,1158756,0.1598799358],[0,1158795,0.0693605629],[0,1158895,0.26348701],[0,1158912,0.0845218212],[0,1158978,0.0725320821],[0,1158981,0.3604830389],[0,1159013,0.0761217069],[0,1159145,0.0761217069],[0,1159182,0.0924109329],[0,1159241,0.0928776613],[0,1159253,0.3408693858],[0,1159332,0.4150267082],[0,1159389,0.0715828977],[0,1159424,0.0761217069],[0,1159443,0.0761217069],[0,1159701,0.0901798074],[0,1159871,0.0718777075],[0,1159939,0.0901798074],[0,1159992,0.074500688],[0,1160157,0.3632138674],[0,1160190,0.1083789079],[0,1160209,0.0732882883],[0,1160430,0.2498063278],[0,1160453,0.1181266389],[0,1160603,0.0775593076],[0,1160893,0.0906938154],[0,1161085,0.0761217069],[0,1161120,0.2187942025],[0,1161165,0.1126377629],[0,1180779,0.0919669793],[0,1181119,0.3057199067],[0,1181141,0.080027631],[0,1181150,0.0723269548],[0,1181179,0.903470138],[0,1181203,0.0719454737],[0,1181242,0.0882206933],[0,1181425,0.0778757966],[0,1181433,0.0736218101],[0,1181445,0.0919669793],[0,1237470,0.080028335],[0,1237565,0.0937536394],[0,1238363,0.0766753608],[0,1238402,0.0776813275],[0,1238469,0.0717486943],[0,1238474,0.1162578362],[0,1238601,0.1400915941],[0,1238690,0.1327422169],[0,1238740,0.1071560033],[0,1238829,0.1888049889],[0,1239141,0.1331236794],[0,1239354,0.0937536394],[0,1239404,0.0737674305],[0,1239595,0.0745142628],[0,1239722,0.1154145118],[0,1239934,0.0718972194],[0,1240093,0.0937536394],[0,1240128,0.0857192025],[0,1240168,0.0757915418],[0,1240170,0.2308774922],[0,1240280,0.1076471701],[0,1240876,0.0745213793],[0,1241459,0.074697377],[0,1241777,0.0749720489],[0,1241836,0.0712767508],[0,1241846,0.1570555271],[0,1241892,0.5346833846],[0,1242044,0.1314591523],[0,1242211,0.0796293841],[0,1242290,0.2253985234],[0,1242878,0.073950281],[0,1242908,0.2365838028],[0,1242909,0.190573434],[0,1243090,0.128356615],[0,1243147,0.1181094302],[0,1243157,0.1299085368],[0,1243191,0.073950281],[0,1243339,0.1068313265],[0,1243584,0.3515084044],[0,1243795,0.1697790961],[0,1243816,0.1661850938],[0,1243933,0.0723698102],[0,1244256,0.1148081087],[0,1244348,0.0732769676],[0,1244480,0.144012117],[0,1244693,0.7258100107],[0,1244748,0.0757915418],[0,1244841,0.0753342408],[0,1244842,0.0777786388],[0,1244849,0.1145750399],[0,1244976,0.1275645966],[0,1244977,0.0796293841],[0,1244993,0.1439025408],[0,1245016,0.1224300565],[0,1245080,0.0851065163],[0,1245462,0.070544527],[0,1245602,0.1027370617],[0,1245672,0.0733735454],[0,1245727,0.0723245561],[0,1245850,0.2761919534],[0,1246026,0.0843040306],[0,1246074,0.1002532654],[0,1246522,0.0796293841],[0,1247404,0.1182309884],[0,1247637,0.0715402452],[0,1247741,0.073950281],[0,1247969,0.0767725412],[0,1247982,0.0783887134],[0,1248169,0.0728075296],[0,1248273,0.0830165791],[0,1248356,0.1018096123],[0,1248397,0.0796293841],[0,1248404,0.1142824187],[0,1248439,0.084335964],[0,1248796,0.1564278098],[0,1249090,0.3072884051],[0,1249728,0.0818510823],[0,1249772,0.0727852677],[0,1249819,0.076593735],[0,1249866,0.0724766739],[0,1250024,0.0924217659],[0,1250025,0.1401431283],[0,1250171,0.0947362289],[0,1250178,0.0943498498],[0,1250370,0.0891554704],[0,1250561,0.1075699508],[0,1250679,0.0757915418],[0,1250946,0.0799639368],[0,1251483,0.1027272073],[0,1251657,0.0790573418],[0,1251718,0.0812191485],[0,1252077,0.1316349666],[0,1252099,0.0776813275],[0,1252300,0.1147458816],[0,1252429,0.0871605666],[0,1252554,0.1440156667],[0,1252782,0.1217339215],[0,1252818,0.2911048351],[0,1253038,0.0747942996],[0,1253064,0.6307315908],[0,1253735,0.0937536394],[0,1254101,0.0757915418],[0,1254327,0.0747455746],[0,1254450,0.1697570821],[0,1254606,0.0937536394],[0,1254688,0.0723698102],[0,1254947,0.1028086047],[0,1255125,0.0938833392],[0,1255151,0.0796293841],[0,1255229,0.0849078453],[0,1255342,0.3431817305],[0,1255403,0.1044677895],[0,1255468,0.0942700662],[0,1255502,0.0928036908],[0,1255653,0.1440109626],[0,1256172,0.1010115667],[0,1256455,0.1963536645],[0,1256687,0.3487045383],[0,1256803,0.0726886067],[0,1256977,0.5608493847],[0,1257040,0.1312519039],[0,1257105,0.0966919085],[0,1257358,0.3389058925],[0,1257402,0.1795009819],[0,1257540,0.2075334906],[0,1257547,0.1946833422],[0,1257630,0.0943498498],[0,1257678,0.0945729602],[0,1257987,0.1212259257],[0,1258407,0.1182653296],[0,1258561,0.0777786388],[0,1258598,0.0796293841],[0,1258724,0.0865900678],[0,1258725,0.0935150839],[0,1258726,0.0907403704],[0,1258731,0.0757915418],[0,1258805,0.0796293841],[0,1258809,0.0745095873],[0,1258899,0.0969095443],[0,1258959,0.4121665041],[0,1259110,0.1816368566],[0,1259215,0.2101449454],[0,1259289,0.0757915418],[0,1259359,0.0723867709],[0,1259452,0.1314640631],[0,1259457,0.1637635156],[0,1259460,0.0775768192],[0,1259559,0.2348219285],[0,1259612,0.2842234263],[0,1259627,0.0937536394],[0,1259629,0.0796293841],[0,1259652,0.0853834144],[0,1259678,0.2535640796],[0,1259728,0.2483395711],[0,1259743,0.0825458517],[0,1259759,0.2793369993],[0,1259803,0.1207316976],[0,1259903,0.127834002],[0,1260134,0.0767628944],[0,1260149,0.0712745202],[0,1260167,0.117133003],[0,1260239,0.0796293841],[0,1260244,0.0937536394],[0,1260300,0.3505364256],[0,1260346,0.0968495066],[0,1260432,0.1813368192],[0,1260446,0.2531314302],[0,1260451,0.5645420387],[0,1260528,0.0812609312],[0,1260599,0.1050663755],[0,1260689,0.0733581664],[0,1260699,0.0723698102],[0,1260717,0.0857623964],[0,1260833,0.3309933271],[0,1261281,0.1089959201],[0,1261282,0.0828817387],[0,1261295,0.1065452477],[0,1261464,0.0943498498],[0,1261482,0.0989832788],[0,1261663,0.0937536394],[0,1261678,0.101593235],[0,1261684,0.4633781589],[0,1261694,0.0847798503],[0,1261804,0.0799639368],[0,1261906,0.0796293841],[0,1262285,0.2072771932],[0,1262358,0.1017794015],[0,1262620,0.0902060678],[0,1262645,0.294731247],[0,1262799,0.0678282803],[0,1262816,0.5357021358],[0,1262915,0.0792984831],[0,1262988,0.1056712766],[0,1263196,0.137937583],[0,1263204,0.1083252496],[0,1263390,0.0707504624],[0,1263511,0.1381114981],[0,1263558,0.1215464801],[0,1263573,0.1367065877],[0,1263580,0.0743053406],[0,1263969,0.3338753245],[0,1264423,0.1788362822],[0,1264784,0.081519436],[0,1264985,0.0834159501],[0,1264993,0.1317235146],[0,1265008,0.1651924966],[0,1265017,0.0689216595],[0,1265024,0.0744445974],[0,1265029,0.1759858077],[0,1265056,0.5031290088],[0,1265341,0.0791652141],[0,1265461,0.0679274901],[0,1265618,0.0690280264],[0,1265686,0.2616941544],[0,1265854,0.0724933922],[0,1266171,0.0902060678],[0,1266211,0.06975383],[0,1266234,0.0704947821],[0,1266237,0.0741044593],[0,1266322,0.1194943888],[0,1266749,0.2248519023],[0,1266858,0.0717320759],[0,1268484,0.0736946325],[0,1268675,0.4094259858],[0,1269622,0.1072378246],[0,1269813,0.0736946325],[0,1269872,0.2933117561],[0,1270217,0.0999093539],[0,1270358,0.1192182753],[0,1270579,0.0771945117],[0,1270770,0.3917708272],[0,1270809,0.0900769085],[0,1270829,0.0722013635],[0,1270889,0.241741507],[0,1270922,0.6446670676],[0,1270950,0.7992318835],[0,1270954,0.1544930189],[0,1270962,0.070368975],[0,1270971,0.2392850605],[0,1271015,0.0854749711],[0,1271055,0.2356538951],[0,1271154,0.3880210895],[0,1271230,0.0741788855],[0,1271590,0.1671423473],[0,1271688,0.1156913245],[0,1271695,0.2424811427],[0,1271700,0.090670246],[0,1271923,0.1522491584],[0,1271930,0.1129335729],[0,1272179,0.0757848803],[0,1272204,0.2217532159],[0,1272240,0.090670246],[0,1272339,0.0760207126],[0,1272451,0.0687133388],[0,1272478,0.3023127739],[0,1272521,0.0900769085],[0,1272679,0.1052096622],[0,1272757,0.1376047007],[0,1272835,0.0973366739],[0,1272838,0.2146222173],[0,1273173,0.1387716047],[0,1273281,0.1159803797],[0,1273380,0.0900769085],[0,1273388,0.1477871189],[0,1273516,0.0711708533],[0,1273527,0.2846817514],[0,1273540,0.076569678],[0,1273579,0.0708970771],[0,1273622,0.1348995468],[0,1273653,0.0900769085],[0,1273757,0.0748352918],[0,1273809,0.0900769085],[0,1273823,0.1245079445],[0,1273834,0.0678572782],[0,1273860,0.070368975],[0,1273865,0.1481073281],[0,1273912,0.250958207],[0,1273933,0.0677061077],[0,1273952,0.2231503617],[0,1274024,0.1135500099],[0,1274053,0.2834306012],[0,1274296,0.0735175066],[0,1274322,0.0816500319],[0,1274362,0.1046503324],[0,1274383,0.1011932973],[0,1274424,0.1064391397],[0,1274504,0.0887922156],[0,1274583,0.2947193117],[0,1274605,0.0688129989],[0,1274755,0.2907154331],[0,1274806,0.0722013635],[0,1274856,0.0693728564],[0,1275129,0.250457648],[0,1275317,0.0704085089],[0,1275428,0.0676350407],[0,1275511,0.0693728564],[0,1275543,0.0888000863],[0,1275585,0.0760207126],[0,1275598,0.2106986993],[0,1275679,0.0706716944],[0,1275800,0.07659565],[0,1275969,0.0900769085],[0,1276056,0.2805173365],[0,1276140,0.1827522939],[0,1276284,0.1582841677],[0,1276341,0.205455511],[0,1276351,0.1895752338],[0,1276365,0.4628394066],[0,1276390,0.2359714644],[0,1276430,0.1505305074],[0,1276468,0.1481268233],[0,1276495,0.0816254466],[0,1276519,0.1385309884],[0,1276524,0.173149735],[0,1276609,0.2776117424],[0,1276646,0.1258232553],[0,1276660,0.2765899232],[0,1276692,0.2703807149],[0,1276701,0.1833913002],[0,1277415,0.252579179],[0,1277441,0.2835657353],[0,1277464,0.0679533097],[0,1277711,0.0807774168],[0,1277947,0.1660867281],[0,1278021,0.0900769085],[0,1278061,0.0741788855],[0,1278553,0.0760207126],[0,1278827,0.0830334628],[0,1278964,0.0980833562],[0,1278999,0.0962776082],[0,1279025,0.0905908468],[0,1279137,0.1740376143],[0,1279390,0.1781915568],[0,1279497,0.1275637324],[0,1279528,0.1271662652],[0,1279579,0.0679533097],[0,1279612,0.0999289478],[0,1279663,0.0689619522],[0,1280057,0.0730390474],[0,1280070,0.0752808812],[0,1280269,0.0875097311],[0,1280289,0.2170876634],[0,1280471,0.0918457914],[0,1280541,0.504392167],[0,1280630,0.3521952323],[0,1280633,0.0722013635],[0,1280763,0.1546325023],[0,1280889,0.3421782232],[0,1280904,0.0760207126],[0,1281063,0.2010381202],[0,1281108,0.1075486866],[0,1281307,0.0732533089],[0,1281332,0.0742518127],[0,1281413,0.0738032018],[0,1281529,0.1647105912],[0,1281573,0.5457426644],[0,1281578,0.2279506363],[0,1281826,0.1058941989],[0,1281967,0.0750169825],[0,1282001,0.1088312236],[0,1282084,0.2856121424],[0,1282088,0.1705762635],[0,1282347,0.1312847309],[0,1282350,0.0900769085],[0,1282411,0.131315692],[0,1282466,0.0708592223],[0,1282996,0.1009666162],[0,1283090,0.080826664],[0,1283101,0.0722013635],[0,1283116,0.0720022725],[0,1283122,0.0848906137],[0,1283638,0.0731335605],[0,1283864,0.2974017349],[0,1283978,0.199249118],[0,1284057,0.0742518127],[0,1284144,0.1646678651],[0,1284186,0.2385988082],[0,1284257,0.0812390597],[0,1284283,0.0744437072],[0,1284504,0.0760207126],[0,1284674,0.1055649325],[0,1284737,0.0739219211],[0,1284872,0.3654489741],[0,1284928,0.3915686743],[0,1284935,0.2046956694],[0,1284942,0.1915396412],[0,1284964,0.2938392695],[0,1285015,0.0739780383],[0,1285122,0.2459236082],[0,1285228,0.0700353573],[0,1285379,0.107173711],[0,1285388,0.0905908468],[0,1285503,0.2072227177],[0,1285844,0.0759563712],[0,1285905,0.2483656866],[0,1285998,0.2251508788],[0,1286170,0.0696281489],[0,1286595,0.0689619522],[0,1286890,0.1077744301],[0,1287046,0.4940774304],[0,1287072,0.398961861],[0,1287302,0.0760207126],[0,1287461,0.0722013635],[0,1287524,0.0760207126],[0,1287621,0.1266910915],[0,1287675,0.0711050306],[0,1287726,0.0791072517],[0,1287963,0.0711050306],[0,1288142,0.0689024687],[0,1288220,0.2735808785],[0,1288275,0.0890596848],[0,1288369,0.2521538477],[0,1288543,0.0736946325],[0,1288770,0.079796583],[0,1288890,0.1968331818],[0,1288902,0.3610154275],[0,1288977,0.1486882199],[0,1289170,0.6667160162],[0,1289408,0.5532341407],[0,1289447,0.2602186284],[0,1289472,0.0679533097],[0,1289533,0.1086421416],[0,1289537,0.132945878],[0,1289577,0.0722013635],[0,1289638,0.0738176444],[0,1289662,0.0676706358],[0,1289687,0.1071994969],[0,1289729,0.0901562539],[0,1289732,0.090670246],[0,1289785,0.0711604365],[0,1289816,0.4230284488],[0,1289871,0.1061241738],[0,1289885,0.1531030983],[0,1290594,0.1780603905],[0,1290666,0.0702726288],[0,1290827,0.0812865562],[0,1291072,0.2070566388],[0,1291166,0.1113299366],[0,1291220,0.1018335723],[0,1291272,0.079135753],[0,1291412,0.1102547929],[0,1291598,0.1972634272],[0,1291703,0.4631827693],[0,1292134,0.0727540039],[0,1292223,0.0893599334],[0,1292237,0.107042711],[0,1292396,0.1105441422],[0,1292558,0.1536308912],[0,1292913,0.2926802359],[0,1292946,0.2552717551],[0,1292979,0.1132806105],[0,1293050,0.2987100295],[0,1293081,0.301297996],[0,1293148,0.3004296093],[0,1293384,0.3014311884],[0,1293570,0.0678594985],[0,1293655,0.1558144293],[0,1293785,0.1218784654],[0,1293786,0.0722013635],[0,1293822,0.1199374977],[0,1293861,0.1042037483],[0,1293870,0.125945196],[0,1293914,0.0901562539],[0,1294030,0.0955079363],[0,1294032,0.2086628419],[0,1294224,0.0722013635],[0,1294233,0.0678594985],[0,1294370,0.0693728564],[0,1294411,0.0748352918],[0,1294610,0.0710220175],[0,1294708,0.0683635315],[0,1294949,0.0700095852],[0,1295028,0.070368975],[0,1295063,0.7146305445],[0,1295161,0.0947533395],[0,1295362,0.1137602093],[0,1295413,0.2637696458],[0,1295492,0.0677083275],[0,1295509,0.3213197292],[0,1295704,0.1083433041],[0,1295820,0.1863907077],[0,1295829,0.1468861014],[0,1295850,0.0760207126],[0,1296021,0.0702726288],[0,1296163,0.1181188881],[0,1296275,0.0900961059],[0,1296282,0.0722013635],[0,1296431,0.0901562539],[0,1296467,0.072122887],[0,1296599,0.0876017553],[0,1296662,0.2515933614],[0,1296837,0.0736142755],[0,1296867,0.1605021892],[0,1296902,0.0762749826],[0,1296931,0.1303125027],[0,1296994,0.1940522805],[0,1297017,0.0785691036],[0,1297018,0.2046124093],[0,1297126,0.0974878512],[0,1297147,0.1206879259],[0,1297193,0.1430740835],[0,1297196,0.1821409067],[0,1297392,0.0863358492],[0,1297414,0.1717608186],[0,1297421,0.158552807],[0,1297441,0.1628130286],[0,1297476,0.0752808812],[0,1297482,0.0673729353],[0,1297594,0.1006674271],[0,1297608,0.0741788855],[0,1297740,0.0882083793],[0,1297757,0.1822474622],[0,1297839,0.0901562539],[0,1297897,0.1358710296],[0,1297922,0.0760207126],[0,1298008,0.0709255862],[0,1298110,0.5816879917],[0,1298397,0.0798480832],[0,1298724,0.441121939],[0,1298819,0.1084373727],[0,1298860,0.2609086959],[0,1299627,0.4685321477],[0,1300077,0.0692766399],[0,1300413,0.3894399637],[0,1300641,0.0911845859],[0,1301002,0.479046986],[0,1301308,0.0679533097],[0,1301430,0.0936337395],[0,1305900,0.1093734351],[0,1306821,0.0719942203],[0,1307318,0.1241457915],[0,1307402,0.1483224352],[0,1307852,0.1135084619],[0,1308056,0.0918124992],[0,1308505,0.1162958262],[0,1308578,0.3236884792],[0,1308610,0.0918124992],[0,1309029,0.0702352791],[0,1309399,0.2063366329],[0,1309518,0.1164147942],[0,1309986,0.3695523507],[0,1310214,0.1104972472],[0,1310344,0.4938566995],[0,1310527,0.1424547889],[0,1310696,0.4058326719],[0,1310698,0.0963002691],[0,1310829,0.1172382179],[0,1310993,0.7903533294],[0,1311431,0.0918124992],[0,1311461,0.0695611323],[0,1311593,0.0758002855],[0,1311604,0.0807679635],[0,1311750,0.3597220126],[0,1311772,0.0693156158],[0,1311788,0.159613571],[0,1311805,0.098523534],[0,1311859,0.1017648301],[0,1311870,0.0918124992],[0,1312121,0.0776461295],[0,1312156,0.0694096124],[0,1312176,0.126606702],[0,1312568,0.1026870767],[0,1312808,0.0928430738],[0,1312905,0.1049539282],[0,1313131,0.1110584551],[0,1313152,0.1294692121],[0,1313300,0.1222816671],[0,1313538,0.1010308472],[0,1313565,0.1077682841],[0,1313720,0.0718895315],[0,1313743,0.4662554047],[0,1313782,0.1210289287],[0,1313802,0.1136706839],[0,1313803,0.1965616964],[0,1313817,0.0998731769],[0,1314776,0.1923965383],[0,1315616,0.4004727961],[0,1315750,0.0889197232],[0,1316216,0.086716243],[0,1316242,0.197626174],[0,1316266,0.0776461295],[0,1316350,0.101248657],[0,1316894,0.1293255572],[0,1317150,0.1334191539],[0,1318017,0.1399879064],[0,1318506,0.0968092339],[0,1318513,0.1007749799],[0,1318723,0.1184613191],[0,1319096,0.1870935056],[0,1319524,0.8823315771],[0,1320168,0.0727752535],[0,1320642,0.6187937427],[0,1320688,0.1639502006],[0,1320752,0.2329401063],[0,1320869,0.1379672807],[0,1320907,0.0879204912],[0,1320936,0.1380407968],[0,1320949,0.0849660632],[0,1320952,0.0704057806],[0,1321363,0.0694096124],[0,1321372,0.0758002855],[0,1321721,0.0776461295],[0,1321729,0.1107844772],[0,1321851,0.0949708994],[0,1321967,0.0776461295],[0,1322075,0.0776461295],[0,1322077,0.0866831715],[0,1322105,0.0719820659],[0,1322128,0.144752868],[0,1322178,0.3327651983],[0,1322412,0.2101978085],[0,1322661,0.2850415157],[0,1323613,0.0751030793],[0,1323845,0.0815710093],[0,1325053,0.098857964],[0,1325809,0.160092189],[0,1325981,0.0735891921],[0,1326050,0.3762427251],[0,1326365,0.1991082897],[0,1326768,0.0798683986],[0,1326922,0.0720499476],[0,1327189,0.0751030793],[0,1327464,0.1674630598],[0,1327660,0.186041191],[0,1327894,0.0967125968],[0,1328235,0.0960519215],[0,1328864,0.0972887379],[0,1330537,0.1301386492],[0,1330719,0.0807909912],[0,1330914,0.111236075],[0,1330992,0.0979489372],[0,1331044,0.2900868932],[0,1331104,0.0751152848],[0,1331517,0.0792770156],[0,1331844,0.0981335821],[0,1332433,0.1598407326],[0,1332464,0.0897342936],[0,1332520,0.0795768511],[0,1332884,0.0996068664],[0,1333233,0.1061053963],[0,1334062,0.1250612737],[0,1334676,0.0807909912],[0,1334677,0.0759352374],[0,1334824,0.1905796539],[0,1334935,0.8833817255],[0,1334956,0.0754077362],[0,1335420,0.1053905865],[0,1335514,0.0807909912],[0,1335565,0.0741005849],[0,1335835,0.1871644535],[0,1336144,0.0807909912],[0,1336179,0.1063584447],[0,1336269,0.0735201571],[0,1336289,0.0751030793],[0,1336430,0.0724231682],[0,1336539,0.0789373753],[0,1336758,0.1592364826],[0,1336794,0.0789373753],[0,1336866,0.1960653879],[0,1336868,0.1206959452],[0,1336917,0.2389862555],[0,1337493,0.0857969461],[0,1337494,0.1550098292],[0,1337499,0.1310250254],[0,1337887,0.0804274419],[0,1338183,0.1071673932],[0,1338659,0.0892197629],[0,1338870,0.0982184792],[0,1338918,0.2775791952],[0,1338922,0.1300870034],[0,1338933,0.076727994],[0,1339038,0.13423112],[0,1339055,0.0764856399],[0,1339087,0.0779390005],[0,1339203,0.0896540666],[0,1339261,0.0965505377],[0,1339317,0.3321605379],[0,1339426,0.1091667045],[0,1340096,0.1114202592],[0,1340365,0.1619562077],[0,1340698,0.7990099263],[0,1340835,0.0994529692],[0,1341033,0.1426874268],[0,1341132,0.386220253],[0,1341259,0.0851443842],[0,1341458,0.1470248293],[0,1341874,0.160749595],[0,1341877,0.3975550248],[0,1342096,0.3002153772],[0,1342344,0.0848585446],[0,1342511,0.1128206112],[0,1342749,0.3428555485],[0,1342934,0.0826105345],[0,1343094,0.4380816151],[0,1343186,0.0771487518],[0,1343376,0.0794234594],[0,1343485,0.0851443842],[0,1343619,0.076615621],[0,1343732,0.0794357358],[0,1344927,0.0781347367],[0,1345167,0.0859802868],[0,1346881,0.0868185726],[0,1346883,0.1286882595],[0,1346888,0.3508213216],[0,1346896,0.074744787],[0,1347099,0.0761920765],[0,1347352,0.083040793],[0,1347518,0.0779814585],[0,1347578,0.3421357802],[0,1347798,0.1143536768],[0,1347857,0.1237211109],[0,1347869,0.0943410725],[0,1348000,0.146876753],[0,1348215,0.08581264],[0,1348314,0.0760683989],[0,1348390,0.6242199835],[0,1348422,0.0841165817],[0,1348485,0.0822627563],[0,1348491,0.1226009999],[0,1348577,0.0847568059],[0,1348695,0.1824653534],[0,1348784,0.0812168468],[0,1348923,0.0924551468],[0,1349033,0.0811816172],[0,1349042,0.1672779006],[0,1349212,0.2025644762],[0,1349375,0.3240187941],[0,1349578,0.0798550272],[0,1349595,0.076330319],[0,1349713,0.1420849647],[0,1349741,0.1168695251],[0,1349763,0.0978283137],[0,1349886,0.1035926652],[0,1349900,0.0988494848],[0,1349912,0.0892995727],[0,1349963,0.0973094803],[0,1350027,0.0748027064],[0,1350465,0.6375002287],[0,1350543,0.4050554419],[0,1350757,0.1061868599],[0,1350787,0.0791854685],[0,1350822,0.1790181367],[0,1350846,0.1049506848],[0,1350945,0.0847766465],[0,1350951,0.0791854685],[0,1351047,0.1004492096],[0,1351150,0.287593866],[0,1351230,0.110990074],[0,1351335,0.083040793],[0,1351339,0.1122584206],[0,1351371,0.7461002088],[0,1351417,0.0818442064],[0,1351729,0.1381846171],[0,1352052,0.107015966],[0,1352065,0.0814568684],[0,1352184,0.1104516433],[0,1352247,0.1077246246],[0,1352265,0.0783370033],[0,1352297,0.3138027912],[0,1352333,0.5694452464],[0,1352620,0.0813511439],[0,1352741,0.1314567543],[0,1352910,0.1085414621],[0,1352912,0.1354540958],[0,1353009,0.0973094803],[0,1353016,0.075996867],[0,1353182,0.077995014],[0,1353256,0.0761497381],[0,1354253,0.083040793],[0,1354409,0.0864039583],[0,1355099,0.5672214532],[0,1356543,0.1106132899],[0,1356695,0.1151066615],[0,1357181,0.1200765108],[0,1357426,0.0806528541],[0,1357457,0.1019120619],[0,1357576,0.3135066041],[0,1357688,0.0822187713],[0,1357914,0.3667117025],[0,1357918,0.083040793],[0,1358055,0.2744384627],[0,1358088,0.131812857],[0,1358113,0.0901940055],[0,1358151,0.0773358203],[0,1358162,0.3687129599],[0,1358219,0.0853575731],[0,1358490,0.0837176925],[0,1358491,0.5022105323],[0,1359471,0.0844702694],[0,1360126,0.2997228657],[0,1360170,0.0815807645],[0,1360430,0.3640935875],[0,1360437,0.0854484272],[0,1361487,0.3007540094],[0,1361596,0.1799516392],[0,1361651,0.1760839036],[0,1361694,0.0797460992],[0,1361821,0.0804037903],[0,1362075,0.118166962],[0,1362092,0.1120619138],[0,1362149,0.2821950902],[0,1362180,0.0819627769],[0,1362331,0.1024972638],[0,1362601,0.1098745235],[0,1362996,0.0873285619],[0,1363230,0.0965500044],[0,1364409,0.0873181736],[0,1364461,0.1746667319],[0,1364517,0.2276109015],[0,1364542,0.1950446158],[0,1364975,0.3571772204],[0,1367560,0.1179985853],[0,1367843,0.0912642762],[0,1368068,0.1337686967],[0,1368103,0.0873181736],[0,1368224,0.0808476376],[0,1368253,0.5581185161],[0,1369541,0.0828536816],[0,1369664,0.1310637472],[0,1370400,0.1357762701],[0,1370428,0.0800395246],[0,1370469,0.0819627769],[0,1370735,0.0819627769],[0,1371147,0.2350304006],[0,1371243,0.1414953776],[0,1371290,0.1041039894],[0,1371594,0.0846611704],[0,1371765,0.0834409291],[0,1371842,0.0805695462],[0,1371856,0.1213882092],[0,1371883,0.0956371729],[0,1372122,0.0802500467],[0,1372189,0.0798121617],[0,1372315,0.4978131113],[0,1372516,0.1684688041],[0,1372540,0.0899655093],[0,1372928,0.1051512078],[0,1373061,0.0793599633],[0,1373742,0.2853525128],[0,1373784,0.2093411173],[0,1374684,0.8397238857],[0,1374809,0.5352453726],[0,1375986,0.0885753096],[0,1375993,0.0858734234],[0,1376212,0.0805695462],[0,1376738,0.0802500467],[0,1376823,0.1450486997],[0,1377295,0.3406144389],[0,1377821,0.2461241049],[0,1377884,0.1041039894],[0,1378611,0.1359245314],[0,1379227,0.1949446478],[0,1379514,0.0873181736],[0,1380480,0.0908729982],[0,1380732,0.183611983],[0,1381229,0.0873181736],[0,1382190,0.4581644996],[0,1382726,0.0990350858],[0,1383183,0.1490423108],[0,1383441,0.0816361156],[0,1383500,0.0873181736],[0,1383563,0.2110417552],[0,1383722,0.1041039894],[0,1383782,0.5025531967],[0,1383974,0.1204000498],[0,1384107,0.1274296125],[0,1384123,0.1619834204],[0,1384355,0.1823890806],[0,1384485,0.3744115571],[0,1385129,0.3754888311],[0,1385545,0.1046274214],[0,1385831,0.2045254864],[0,1386052,0.1041039894],[0,1386431,0.083626732],[0,1387407,0.0929652324],[0,1387958,0.272947431],[0,1388258,0.0863484806],[0,1388301,0.1041039894],[0,1388336,0.2497309677],[0,1388630,0.0869812053],[0,1388973,0.2038342918],[0,1389091,0.2299657051],[0,1389202,0.0825745687],[0,1389516,0.1102187246],[0,1389651,0.1158663256],[0,1389908,0.0926508276],[0,1390102,0.0823982004],[0,1390239,0.0788797857],[0,1390299,0.12907403],[0,1390335,0.1043481119],[0,1390493,0.1592084806],[0,1391006,0.4462525114],[0,1391042,0.0819243562],[0,1391513,0.1112938925],[0,1391655,0.0885314509],[0,1392142,0.1024210494],[0,1392390,0.0799468372],[0,1392493,0.0807112709],[0,1392512,0.1158663256],[0,1392780,0.0772377896],[0,1392907,0.0856717612],[0,1392987,0.084147934],[0,1393006,0.1075431465],[0,1393119,0.1086728805],[0,1393298,0.0856717612],[0,1393518,0.0856717612],[0,1393672,0.150855077],[0,1393769,0.1566174218],[0,1393963,0.2314884513],[0,1394169,0.2113409849],[0,1394225,0.0802896813],[0,1394456,0.1750986482],[0,1394594,0.0881572306],[0,1394617,0.4505633631],[0,1394664,0.0773467427],[0,1394871,0.1051687118],[0,1394873,0.1760610516],[0,1394908,0.0856717612],[0,1394914,0.2899876841],[0,1394926,0.1880660298],[0,1395049,0.0856717612],[0,1395092,0.1334485501],[0,1395158,0.0856717612],[0,1395202,0.146245912],[0,1395330,0.0934279988],[0,1395591,0.0789378195],[0,1395965,0.1696815893],[0,1396325,0.0838414364],[0,1396376,0.0803644006],[0,1396628,0.0856717612],[0,1396809,0.0784089512],[0,1397255,0.0995056199],[0,1397381,0.1071790353],[0,1397455,0.0845679917],[0,1397458,0.081474309],[0,1397466,0.1493698909],[0,1397508,0.1475178464],[0,1397585,0.5041960411],[0,1397634,0.0864338158],[0,1397792,0.143830432],[0,1398235,0.0819501291],[0,1398264,0.1231794217],[0,1398551,0.3389886292],[0,1398563,0.1050788349],[0,1398842,0.1483308729],[0,1399121,0.0864338158],[0,1399226,0.2117199626],[0,1399398,0.0829804121],[0,1399465,0.0965470915],[0,1399728,0.0845662549],[0,1399765,0.0845662549],[0,1399822,0.3805934772],[0,1400197,0.085732093],[0,1400449,0.0818522743],[0,1400457,0.1304023627],[0,1402349,0.087289307],[0,1404220,0.1397008144],[0,1405295,0.1183966315],[0,1405515,0.1113209542],[0,1405632,0.1116223788],[0,1406018,0.1031404648],[0,1406333,0.0852984669],[0,1406528,0.0887141097],[0,1406885,0.0912830343],[0,1407120,0.0965402301],[0,1407379,0.1072016259],[0,1408284,0.0956493495],[0,1408440,0.0857582533],[0,1408448,0.1093888728],[0,1409372,0.0929424127],[0,1409432,0.0900756768],[0,1409524,0.1861619503],[0,1409556,0.2167303431],[0,1409867,0.8358027966],[0,1409973,0.6181895621],[0,1410018,0.1422804729],[0,1410137,0.0943186225],[0,1410225,0.1121508994],[0,1410423,0.4730142671],[0,1410583,0.1941631226],[0,1410653,0.0910609058],[0,1410672,0.1328896135],[0,1410988,0.2403214114],[0,1411009,0.149230838],[0,1411173,0.1116223788],[0,1411438,0.1093466657],[0,1411886,0.1228176369],[0,1412091,0.4338660118],[0,1412160,0.1404572476],[0,1412337,0.3885146948],[0,1412602,0.0994722812],[0,1412727,0.1054834248],[0,1413174,0.0856652891],[0,1413351,0.1110942156],[0,1413589,0.1724058506],[0,1413929,0.2217389405],[0,1414591,0.0967731474],[0,1414632,0.110861474],[0,1414726,0.567037349],[0,1416977,0.0941566797],[0,1417426,0.228918505],[0,1417563,0.1035687773],[0,1418795,0.1585903001],[0,1418950,0.156876667],[0,1419035,0.0941566797],[0,1419133,0.2410812116],[0,1419468,0.1984105109],[0,1419604,0.0860292254],[0,1419832,0.2297258234],[0,1419913,0.0895126542],[0,1420082,0.2113195001],[0,1420099,0.1093875441],[0,1420272,0.2414306027],[0,1420386,0.1371488718],[0,1420393,0.1583798888],[0,1420398,0.152057529],[0,1420429,0.1190932321],[0,1420451,0.0891439625],[0,1420458,0.2012864325],[0,1420535,0.0874759127],[0,1420947,0.0871843435],[0,1420996,0.0945505525],[0,1421313,0.2402908784],[0,1421462,0.1370692724],[0,1421615,0.0894138143],[0,1421684,0.170983523],[0,1422187,0.0933356591],[0,1422654,0.1047061684],[0,1422709,0.1320717705],[0,1422857,0.137218696],[0,1422926,0.098919726],[0,1423360,0.0875992258],[0,1423528,0.1228933331],[0,1423630,0.2618171073],[0,1423668,0.1152612378],[0,1423973,0.1152612378],[0,1423991,0.2412104089],[0,1424093,0.1008954895],[0,1424107,0.3418833327],[0,1424167,0.0878118719],[0,1424223,0.0913604395],[0,1424401,0.3479621942],[0,1424402,0.0923305447],[0,1424425,0.1442017793],[0,1424479,0.4390976722],[0,1424510,0.1334758047],[0,1424680,0.1218762655],[0,1424700,0.0955645149],[0,1424952,0.112835132],[0,1424957,0.1037790599],[0,1425220,0.1769251826],[0,1425577,0.114150295],[0,1425781,0.0963365375],[0,1425880,0.0903494495],[0,1425896,0.124059654],[0,1425953,0.1152612378],[0,1425983,0.0932613889],[0,1426139,0.0982332582],[0,1426202,0.089158713],[0,1426427,0.0891019776],[0,1426632,0.1247429214],[0,1426650,0.0890464095],[0,1426914,0.2419475824],[0,1427006,0.1103993741],[0,1427227,0.4170343193],[0,1427357,0.2018268159],[0,1427560,0.1957538764],[0,1427571,0.1126394428],[0,1427574,0.178408523],[0,1427584,0.1986686247],[0,1427590,0.1288810717],[0,1427626,0.0954791459],[0,1427627,0.0982332582],[0,1427865,0.1132377282],[0,1428210,0.1157922213],[0,1428729,0.1476629948],[0,1428765,0.099490256],[0,1428772,0.0961350838],[0,1428947,0.5871929934],[0,1429131,0.6131019088],[0,1429339,0.091204449],[0,1429393,0.2459493851],[0,1429997,0.7533898278],[0,1430549,0.0923305447],[0,1430768,0.0982332582],[0,1430947,0.1172489133],[0,1431089,0.0888709471],[0,1431093,0.3151004431],[0,1431109,0.2316162566],[0,1431183,0.7030503761],[0,1431194,0.1250905113],[0,1431250,0.1552144307],[0,1431344,0.0982332582],[0,1431396,0.1022699404],[0,1431561,0.0909199803],[0,1431762,0.1206387484],[0,1431806,0.3143466879],[0,1432086,0.4341811947],[0,1432189,0.0905750567],[0,1432476,0.0941164076],[0,1432766,0.0921955341],[0,1432899,0.0982332582],[0,1432971,0.0945342556],[0,1433135,0.0889851665],[0,1433497,0.1258994867],[0,1433535,0.1157922213],[0,1433651,0.092603997],[0,1433666,0.1607105856],[0,1433778,0.1179167917],[0,1434005,0.1370148205],[0,1434072,0.1324864707],[0,1434081,0.1016615473],[0,1434169,0.0951574611],[0,1434178,0.0923305447],[0,1434257,0.091261359],[0,1434272,0.0892019574],[0,1434405,0.091261359],[0,1434675,0.3727489858],[0,1434723,0.0888321731],[0,1434847,0.3456931479],[0,1434928,0.104909163],[0,1434930,0.1022903286],[0,1435115,0.0887154672],[0,1435156,0.1067872526],[0,1435162,0.0897421088],[0,1435264,0.0912781661],[0,1435281,0.1115835993],[0,1435484,0.416101833],[0,1436375,0.0923305447],[0,1436391,0.1337666927],[0,1436393,0.2873364361],[0,1436404,0.6934333333],[0,1436488,0.1078478844],[0,1436639,0.0904950179],[0,1436793,0.3459398315],[0,1436947,0.099490256],[0,1437022,0.104514191],[0,1437041,0.5502114716],[0,1437176,0.1152612378],[0,1437216,0.2501965661],[0,1437276,0.0940224739],[0,1437645,0.0911191823],[0,1437787,0.1152612378],[0,1437950,0.2533673867],[0,1437966,0.0938659374],[0,1438161,0.0878118719],[0,1438470,0.1152612378],[0,1438550,0.1106771187],[0,1438818,0.2500776234],[0,1438917,0.1082900226],[0,1438938,0.1131391761],[0,1439018,0.1658563168],[0,1439029,0.0982332582],[0,1439069,0.0912120914],[0,1439208,0.1581037544],[0,1439343,0.1003660527],[0,1439364,0.1316462666],[0,1439460,0.6129797979],[0,1440557,0.1143821371],[0,1440785,0.4483142269],[0,1440913,0.2035640261],[0,1440925,0.1393771525],[0,1441204,0.09666307],[0,1442211,0.0941554519],[0,1442305,0.1337191379],[0,1442581,0.0898529192],[0,1442710,0.1152612378],[0,1442746,0.2428928338],[0,1443062,0.0902434472],[0,1443106,0.0958643086],[0,1443261,0.2127776705],[0,1443481,0.3224814007],[0,1444061,0.1178994357],[0,1444313,0.09703497],[0,1445184,0.0963365375],[0,1446278,0.1458973528],[0,1446353,0.2092157262],[0,1446449,0.1171630707],[0,1446475,0.7354551186],[0,1446614,0.0947083936],[0,1446745,0.1366969914],[0,1446901,0.595280914],[0,1446913,0.1021090179],[0,1446915,0.1919763018],[0,1446971,0.67621912],[0,1447070,0.0922928359],[0,1447302,0.129680563],[0,1447491,0.1057316513],[0,1447834,0.0904484113],[0,1447873,0.1218230972],[0,1447954,0.3217576742],[0,1448007,0.1293145558],[0,1448332,0.1697319305],[0,1448379,0.1460580979],[0,1448522,0.144159295],[0,1448634,0.0944185047],[0,1449003,0.0982332582],[0,1449035,0.184774114],[0,1449048,0.110819636],[0,1449155,0.2368115564],[0,1449229,0.190624014],[0,1449282,0.1818753648],[0,1449381,0.1240674769],[0,1449443,0.2078334611],[0,1449515,0.1417052371],[0,1449522,0.1152612378],[0,1449767,0.1152612378],[0,1450209,0.1086198728],[0,1450429,0.0913795759],[0,1450567,0.1027493558],[0,1450700,0.1081329536],[0,1451139,0.1282801416],[0,1451163,0.1510275396],[0,1451221,0.1059789377],[0,1451229,0.0901310336],[0,1451297,0.1428009019],[0,1451438,0.121488369],[0,1453927,0.0897567977],[0,1454080,0.2013382701],[0,1455726,0.1148322611],[0,1456118,0.1005461997],[0,1456388,0.1556340951],[0,1456526,0.1100419757],[0,1456740,0.099168744],[0,1456757,0.2277924606],[0,1456799,0.1151632349],[0,1456953,0.3711785856],[0,1457029,0.126192082],[0,1457247,0.1017983628],[0,1458045,0.0986886093],[0,1458301,0.0981373824],[0,1458342,0.2284858741],[0,1458384,0.0885454139],[0,1458774,0.0981373824],[0,1458904,0.0910824837],[0,1458988,0.2759513475],[0,1459793,0.1151632349],[0,1460332,0.0981373824],[0,1460345,0.1151632349],[0,1460359,0.0882371274],[0,1460472,0.0922354062],[0,1460502,0.0883742702],[0,1460537,0.0903535079],[0,1460612,0.1112959473],[0,1460623,0.0913721474],[0,1460746,0.1852807042],[0,1460800,0.1151632349],[0,1460880,0.1419172427],[0,1461186,0.0882371274],[0,1461238,0.136546587],[0,1461772,0.1095396208],[0,1461983,0.115694152],[0,1462090,0.1151632349],[0,1462294,0.0992497974],[0,1462334,0.1051520498],[0,1462879,0.189865509],[0,1462895,0.0910824837],[0,1463006,0.0978495868],[0,1463061,0.089810211],[0,1463084,0.1556359405],[0,1463098,0.1952898282],[0,1463271,0.1275908575],[0,1463280,0.088387064],[0,1464543,0.0884296615],[0,1464796,0.095604668],[0,1464828,0.0853737362],[0,1464859,0.1621769617],[0,1465132,0.0857936515],[0,1465201,0.1724420954],[0,1465412,0.1162364553],[0,1465560,0.0885012105],[0,1465605,0.1251837308],[0,1465862,0.3247134725],[0,1465874,0.3382673543],[0,1465877,0.0905941206],[0,1469462,0.2190143594],[0,1469709,0.2042015973],[0,1470176,0.0903350537],[0,1471498,0.0963775178],[0,1471622,0.141575274],[0,1471795,0.1009579505],[0,1472681,0.1289718771],[0,1472768,0.1146111263],[0,1474139,0.1071694878],[0,1474726,0.0905473239],[0,1475677,0.1562840886],[0,1475919,0.1315262633],[0,1476489,0.1023624],[0,1476519,0.1107562779],[0,1476667,0.0952302706],[0,1476729,0.0898188612],[0,1476878,0.0894500207],[0,1476943,0.1570278764],[0,1477113,0.4361011009],[0,1477423,0.1137327541],[0,1478005,0.0974664207],[0,1478163,0.1907884755],[0,1478468,0.09352283],[0,1478472,0.0908367837],[0,1478587,0.110578488],[0,1478630,0.1090779874],[0,1478850,0.0882607113],[0,1479100,0.0962777837],[0,1479118,0.1019845665],[0,1479123,0.1572653649],[0,1479124,0.1491369147],[0,1479182,0.1018265964],[0,1479212,0.0871845066],[0,1479224,0.1151416698],[0,1479784,0.2622116248],[0,1480264,0.3455058838],[0,1480404,0.1276326662],[0,1480644,0.1277611514],[0,1480667,0.0903901093],[0,1480679,0.1481790784],[0,1480747,0.0871845066],[0,1480890,0.1191317251],[0,1481284,0.1323084798],[0,1481343,0.1189797762],[0,1481393,0.2264663078],[0,1481403,0.1122234675],[0,1481457,0.2080023176],[0,1481715,0.1728482473],[0,1481817,0.143208842],[0,1481842,0.0906809006],[0,1481846,0.5696656952],[0,1481886,0.0977079591],[0,1481966,0.1357685501],[0,1482022,0.0881579428],[0,1482044,0.1176660629],[0,1482087,0.1037681777],[0,1482156,0.0954407839],[0,1482164,0.0901908374],[0,1482250,0.4644272693],[0,1482256,0.1181665476],[0,1482487,0.0883139918],[0,1482577,0.1703131386],[0,1482599,0.1440079693],[0,1482607,0.0889059798],[0,1482612,0.0899284403],[0,1482656,0.1307845865],[0,1482695,0.0909465098],[0,1482963,0.0872937055],[0,1482998,0.0938958166],[0,1483055,0.270020446],[0,1483344,0.1045467732],[0,1483391,0.1198696936],[0,1483568,0.2323033307],[0,1483962,0.1260706886],[0,1484698,0.1198967555],[0,1484926,0.0899284403],[0,1485188,0.5748091835],[0,1485302,0.0899284403],[0,1486942,0.2248459967],[0,1487162,0.2561292351],[0,1487314,0.1016271138],[0,1487496,0.0899284403],[0,1487758,0.2016578916],[0,1488102,0.1167332],[0,1488419,0.1256248602],[0,1488744,0.1199790675],[0,1489792,0.0891014471],[0,1490103,0.0945216547],[0,1490461,0.1300654217],[0,1490676,0.5817198989],[0,1490844,0.4367665216],[0,1491097,0.1719983733],[0,1491696,0.0972488465],[0,1492871,0.1031896788],[0,1492909,0.1227543714],[0,1494841,0.1031896788],[0,1495381,0.1004164091],[0,1495517,0.0965917251],[0,1496329,0.1572575321],[0,1496462,0.109188551],[0,1498344,0.1761829716],[0,1498461,0.1037840716],[0,1498872,0.1043988079],[0,1498947,0.12176382],[0,1498976,0.1131416928],[0,1499010,0.0945887069],[0,1499910,0.1965133706],[0,1500173,0.3150223176],[0,1500890,0.0980079494],[0,1501631,0.0963877713],[0,1501854,0.4451473449],[0,1502206,0.0967119715],[0,1502238,0.3241496107],[0,1502697,0.1662718789],[0,1502774,0.104284911],[0,1502822,0.0961535713],[0,1502871,0.6562921279],[0,1503004,0.1161918242],[0,1503189,0.1479009864],[0,1503320,0.1484070532],[0,1503561,0.178037135],[0,1503564,0.0980562112],[0,1503643,0.1950548795],[0,1503987,0.1106456448],[0,1504102,0.1237616174],[0,1504223,0.6840888412],[0,1504294,0.14407381],[0,1504441,0.1197944802],[0,1504767,0.6136975778],[0,1505490,0.1718732982],[0,1505815,0.1157752632],[0,1506575,0.0940581532],[0,1506938,0.4723720395],[0,1507103,0.1349153038],[0,1507386,0.0960538663],[0,1507513,0.1162173607],[0,1508283,0.1297444276],[0,1508390,0.1092702623],[0,1508579,0.1158811417],[0,1508710,0.2143824575],[0,1508715,0.130039458],[0,1508774,0.1829389967],[0,1508883,0.1000986069],[0,1509362,0.1823413281],[0,1509378,0.1277742223],[0,1509555,0.6177453118],[0,1509888,0.0980842935],[0,1510533,0.1396807964],[0,1510805,0.1026400077],[0,1511145,0.5472342989],[0,1511319,0.105058475],[0,1511652,0.1375028897],[0,1512196,0.0996234303],[0,1512285,0.1005431259],[0,1513107,0.1040138026],[0,1513166,0.4201495755],[0,1513342,0.1020023166],[0,1513492,0.1336816085],[0,1513771,0.1151167201],[0,1513811,0.3591368037],[0,1513893,0.0982490219],[0,1513943,0.3195340925],[0,1513989,0.0962212586],[0,1513997,0.1080812489],[0,1514026,0.2891083386],[0,1514221,0.1772882947],[0,1514477,0.1021027965],[0,1514484,0.1494766264],[0,1514487,0.0996234303],[0,1514935,0.265269816],[0,1514941,0.1484372625],[0,1515444,0.1066189148],[0,1515633,0.1122317359],[0,1515641,0.1100070659],[0,1516372,0.1397552083],[0,1518584,0.5901521501],[0,1518664,0.1000366596],[0,1518971,0.1097922284],[0,1519600,0.1390497129],[0,1522746,0.2591597667],[0,1523356,0.0995671819],[0,1523408,0.1506630959],[0,1524305,0.1393369751],[0,1524337,0.1193981599],[0,1524476,0.4401513143],[0,1524758,0.6824964121],[0,1524827,0.1002118808],[0,1525014,0.4298030586],[0,1525213,0.1130358524],[0,1525379,0.1014353354],[0,1525805,0.1314399947],[0,1526090,0.1030661945],[0,1526669,0.3687766942],[0,1526954,0.0996673445],[0,1527084,0.1051008617],[0,1527094,0.5188751013],[0,1527102,0.1176634548],[0,1527327,0.1070192952],[0,1527367,0.6161755596],[0,1527379,0.1315175568],[0,1527523,0.1182777122],[0,1527710,0.1333928141],[0,1527990,0.2239636386],[0,1528161,0.2504558614],[0,1528257,0.7560926365],[0,1528284,0.1127934891],[0,1528347,0.1525703235],[0,1528353,0.108993445],[0,1528382,0.110921522],[0,1528477,0.2323773199],[0,1528629,0.0990240518],[0,1528908,0.1306820447],[0,1528954,0.1500885302],[0,1529003,0.0997560791],[0,1529010,0.2187957279],[0,1529015,0.2381905711],[0,1529192,0.131994157],[0,1529227,0.1539455146],[0,1529290,0.19300839],[0,1529326,0.127718582],[0,1530107,0.1632337065],[0,1530201,0.1240321154],[0,1530363,0.1542423252],[0,1530452,0.1030079771],[0,1530573,0.161393514],[0,1530651,0.1070524276],[0,1530863,0.1030079771],[0,1530974,0.1250089435],[0,1531049,0.1011723447],[0,1531079,0.2952578392],[0,1531173,0.1092128241],[0,1531531,0.1306820447],[0,1531670,0.1897459862],[0,1531856,0.1019679016],[0,1531939,0.1246873836],[0,1532174,0.110921522],[0,1532428,0.0987696671],[0,1532713,0.0994395846],[0,1532878,0.1633140638],[0,1533056,0.097589145],[0,1533173,0.2128904824],[0,1533510,0.7099002346],[0,1534429,0.0989763959],[0,1535396,0.1345452699],[0,1535607,0.1130091028],[0,1536209,0.1032534058],[0,1536392,0.2189345032],[0,1536529,0.1345452699],[0,1536535,0.1011140147],[0,1536895,0.228064975],[0,1537087,0.2294090027],[0,1537119,0.4327720741],[0,1537152,0.351259622],[0,1537436,0.1350893051],[0,1537527,0.1663336611],[0,1537560,0.1209729312],[0,1537613,0.1238859662],[0,1537717,0.1016615796],[0,1537788,0.1019048442],[0,1537959,0.1020945179],[0,1538099,0.1775221778],[0,1538277,0.5915445294],[0,1538301,0.1066086696],[0,1538519,0.1356337085],[0,1539011,0.1912081421],[0,1539046,0.1022725998],[0,1539078,0.3044980109],[0,1539176,0.1118526319],[0,1539292,0.1573175197],[0,1539293,0.2292971964],[0,1539373,0.3496058702],[0,1539381,0.1587348346],[0,1539401,0.1345452699],[0,1539424,0.3242887675],[0,1539512,0.1041435553],[0,1539803,0.1065683447],[0,1539880,0.1345452699],[0,1540036,0.3846644901],[0,1540156,0.114131008],[0,1540229,0.3477028489],[0,1540251,0.114131008],[0,1540266,0.1356337085],[0,1540314,0.6364276096],[0,1540563,0.1017106495],[0,1540660,0.1137160094],[0,1540858,0.114131008],[0,1541099,0.1201942846],[0,1541124,0.1233813053],[0,1541261,0.3508090606],[0,1541265,0.1020861214],[0,1541269,0.134979394],[0,1541343,0.366251472],[0,1541411,0.1208849511],[0,1541417,0.1999656341],[0,1541454,0.3415328601],[0,1541482,0.1350893051],[0,1541610,0.1210632016],[0,1541679,0.1154971037],[0,1541709,0.1098416684],[0,1542045,0.1012385181],[0,1542201,0.1144493834],[0,1542321,0.1002217468],[0,1542509,0.1345452699],[0,1542577,0.2723168987],[0,1542621,0.1002217468],[0,1542648,0.1593763488],[0,1542980,0.1079162601],[0,1543088,0.1640059787],[0,1543330,0.117469289],[0,1543399,0.1128671211],[0,1543510,0.1877526208],[0,1543595,0.1696763526],[0,1543812,0.1009289846],[0,1543879,0.1121949994],[0,1543922,0.1004573319],[0,1544039,0.1081060282],[0,1544048,0.1808802027],[0,1544180,0.1790389228],[0,1544396,0.1024241357],[0,1544467,0.397430887],[0,1544518,0.1349422341],[0,1544560,0.1674102237],[0,1544741,0.580827767],[0,1544758,0.1020053114],[0,1545416,0.4967753143],[0,1545580,0.1368605843],[0,1545739,0.1422634075],[0,1545975,0.162493096],[0,1547072,0.1108620981],[0,1548161,0.3150098521],[0,1548188,0.2773660597],[0,1548450,0.1228229088],[0,1548503,0.1299282483],[0,1548600,0.1128671211],[0,1548601,0.1126420986],[0,1550036,0.12094494],[0,1550585,0.114131008],[0,1550751,0.1112698672],[0,1551757,0.1303682655],[0,1551910,0.1053741029],[0,1552043,0.5163989397],[0,1552118,0.1066131494],[0,1552395,0.1044213467],[0,1552418,0.1143108549],[0,1552520,0.3543385452],[0,1552528,0.1436505832],[0,1552890,0.1686984825],[0,1552895,0.1062857996],[0,1553007,0.331034063],[0,1553184,0.1063740637],[0,1553360,0.1199131457],[0,1553371,0.1062461349],[0,1553564,0.1503000066],[0,1553644,0.3689256075],[0,1553735,0.1454262302],[0,1553742,0.1121860217],[0,1553800,0.1408399421],[0,1553892,0.1167176591],[0,1554072,0.1775587066],[0,1554102,0.1397444633],[0,1554118,0.5124756564],[0,1554174,0.4804519655],[0,1554179,0.1408399421],[0,1554306,0.142993107],[0,1554672,0.1305367155],[0,1554815,0.1030512156],[0,1554846,0.1408399421],[0,1554852,0.1408399421],[0,1554901,0.1378970736],[0,1554929,0.1432803874],[0,1554944,0.2109360397],[0,1555007,0.8181467748],[0,1555009,0.1442635578],[0,1555265,0.176332476],[0,1555314,0.1167814784],[0,1555520,0.1959524196],[0,1555999,0.1618181546],[0,1557253,0.1339925629],[0,1557334,0.1293474236],[0,1557582,0.1209374325],[0,1558308,0.1772320253],[0,1559095,0.157046954],[0,1559109,0.1045971476],[0,1559631,0.1143294816],[0,1559688,0.4840161458],[0,1559838,0.1344986887],[0,1560000,0.2175480851],[0,1560020,0.1040836944],[0,1560141,0.1402920174],[0,1560279,0.1911636891],[0,1560476,0.1041232485],[0,1560739,0.1664514011],[0,1561011,0.1417480498],[0,1561018,0.1099759876],[0,1561224,0.1607249384],[0,1561409,0.2627882504],[0,1561492,0.1060981794],[0,1561888,0.1146082947],[0,1562043,0.1067536586],[0,1562080,0.1053344848],[0,1562243,0.1601881718],[0,1562347,0.173958552],[0,1562642,0.1542866994],[0,1562723,0.1186648721],[0,1562857,0.5391819645],[0,1563161,0.3299187906],[0,1563162,0.1408399421],[0,1563690,0.1060585243],[0,1563768,0.1429530404],[0,1564819,0.1402920174],[0,1564933,0.2525903746],[0,1566292,0.1079908813],[0,1566785,0.1517443058],[0,1567873,0.2855703662],[0,1568319,0.1873604892],[0,1568880,0.154442759],[0,1568913,0.4418093436],[0,1569980,0.1453398358],[0,1570067,0.1099737726],[0,1570109,0.1211822573],[0,1570171,0.1474110117],[0,1570239,0.226388421],[0,1570470,0.126783449],[0,1571288,0.3754903997],[0,1571359,0.1178494723],[0,1571393,0.1417254979],[0,1571419,0.1470404824],[0,1571613,0.2571608726],[0,1571838,0.1521680885],[0,1572446,0.1723506395],[0,1572860,0.156868527],[0,1573428,0.1728050976],[0,1573541,0.1126822611],[0,1573558,0.4957734632],[0,1573580,0.2219685335],[0,1573646,0.1401221585],[0,1573651,0.1273238602],[0,1573888,0.1437162305],[0,1573902,0.358521871],[0,1573907,0.1662746804],[0,1573939,0.1083210696],[0,1573997,0.1956874585],[0,1574058,0.1666860364],[0,1574685,0.1793071274],[0,1574765,0.1212956183],[0,1574849,0.438671127],[0,1574869,0.1083406522],[0,1574884,0.11388685],[0,1575366,0.1313948493],[0,1575540,0.1616846142],[0,1575684,0.1122496647],[0,1576302,0.1619006745],[0,1576335,0.1431663605],[0,1576763,0.1202368793],[0,1576873,0.1077526443],[0,1576919,0.5360770705],[0,1576950,0.1263857717],[0,1577122,0.1936179368],[0,1577688,0.2566709278],[0,1577721,0.1083608404],[0,1577825,0.1200421596],[0,1577830,0.1526367115],[0,1577870,0.2931336585],[0,1578351,0.3720161187],[0,1578496,0.4427449142],[0,1578572,0.1077526443],[0,1578656,0.1068660492],[0,1578957,0.1064189533],[0,1579472,0.1992719307],[0,1579658,0.1288359929],[0,1581426,0.1143516419],[0,1581851,0.3502480333],[0,1582034,0.1080569103],[0,1582223,0.3204158618],[0,1582225,0.2017019419],[0,1582299,0.2062529126],[0,1582389,0.2768028728],[0,1582486,0.1110898985],[0,1582509,0.2570571834],[0,1582656,0.1140328572],[0,1582776,0.427490067],[0,1582791,0.117814045],[0,1582796,0.2678800399],[0,1582807,0.114193518],[0,1582901,0.1815757087],[0,1583123,0.1520121798],[0,1583216,0.2312072849],[0,1583258,0.1355110306],[0,1583335,0.1176442177],[0,1584000,0.1956652113],[0,1584462,0.5706477483],[0,1584934,0.2148577298],[0,1585199,0.1164959814],[0,1585367,0.1104586197],[0,1585426,0.1715686796],[0,1585762,0.3277404879],[0,1585779,0.1233368248],[0,1585823,0.4242885259],[0,1585831,0.1351520424],[0,1585847,0.4217604369],[0,1586286,0.1116402719],[0,1586298,0.2313981577],[0,1588039,0.180392209],[0,1588981,0.1171688021],[0,1589039,0.1299571848],[0,1589068,0.1161477436],[0,1589113,0.4167414382],[0,1589504,0.1213525435],[0,1589590,0.3711138188],[0,1589927,0.1593475238],[0,1590086,0.1112463832],[0,1590185,0.1087284622],[0,1590241,0.1085569541],[0,1590598,0.167951778],[0,1591004,0.374125282],[0,1591020,0.5146699594],[0,1591250,0.3517108132],[0,1592254,0.4027091121],[0,1592791,0.1827114135],[0,1592846,0.1201171884],[0,1593526,0.2980258048],[0,1593539,0.2057492059],[0,1593676,0.1142067512],[0,1594073,0.1087284622],[0,1594166,0.1189768503],[0,1594189,0.1555023884],[0,1594203,0.1223056446],[0,1594219,0.3556273253],[0,1594230,0.1259571628],[0,1594291,0.1243268617],[0,1594338,0.1472318441],[0,1594551,0.1768384791],[0,1594820,0.183918356],[0,1595466,0.1274198835],[0,1596129,0.1264660461],[0,1596707,0.1464622725],[0,1597234,0.4401540599],[0,1598053,0.2393357776],[0,1598075,0.4431527555],[0,1598120,0.1117919752],[0,1598140,0.1532413153],[0,1598441,0.1258455851],[0,1598632,0.1240999387],[0,1599519,0.1316557616],[0,1599811,0.1656203963],[0,1599917,0.1721112858],[0,1599964,0.3065774711],[0,1599966,0.1541964373],[0,1600010,0.5835645662],[0,1601543,0.1301369456],[0,1601604,0.1742699664],[0,1601616,0.1587968389],[0,1601653,0.3458236269],[0,1602394,0.1362192841],[0,1602415,0.1715883952],[0,1602488,0.1149751237],[0,1602576,0.1147898274],[0,1602588,0.1127797365],[0,1602617,0.1126807757],[0,1602631,0.1293704015],[0,1602883,0.2529008188],[0,1602979,0.119137825],[0,1603039,0.1291587145],[0,1603183,0.1653046077],[0,1603715,0.1692058726],[0,1604120,0.2714220029],[0,1604144,0.1541964373],[0,1604879,0.2363264585],[0,1605159,0.3040035964],[0,1605440,0.1691787657],[0,1605650,0.1227399006],[0,1607617,0.1152455325],[0,1607793,0.2783571129],[0,1608341,0.1127277855],[0,1609218,0.157590185],[0,1609260,0.1641798471],[0,1609485,0.1247785528],[0,1609522,0.1530828971],[0,1611115,0.1690399424],[0,1611710,0.1118474543],[0,1612034,0.6874474978],[0,1612056,0.1160252487],[0,1612194,0.1837121726],[0,1612238,0.1238005421],[0,1612667,0.1536394788],[0,1612918,0.181495884],[0,1612973,0.2536966957],[0,1613295,0.207227415],[0,1613371,0.1316557616],[0,1613532,0.1316557616],[0,1613632,0.1123905506],[0,1613938,0.1845825083],[0,1614354,0.3711425248],[0,1614460,0.7143173885],[0,1614646,0.1225983579],[0,1614759,0.1394396512],[0,1615088,0.1574316018],[0,1615196,0.1127983269],[0,1615321,0.1126020768],[0,1615371,0.1541405615],[0,1615621,0.1435936064],[0,1615661,0.5235670711],[0,1615774,0.1541964373],[0,1615862,0.1127797365],[0,1615919,0.1805792602],[0,1615976,0.1530828971],[0,1616050,0.1530828971],[0,1616237,0.3141301749],[0,1616309,0.2690318625],[0,1616609,0.3676353356],[0,1616816,0.4541723793],[0,1616891,0.210865743],[0,1617155,0.1122608543],[0,1618188,0.1175905766],[0,1618240,0.1862647595],[0,1618452,0.2996971618],[0,1618483,0.3543562282],[0,1618521,0.4159096358],[0,1618554,0.1609385459],[0,1618981,0.1311489744],[0,1618989,0.1455917987],[0,1619057,0.3606462136],[0,1619131,0.1461803689],[0,1619132,0.1113974888],[0,1619418,0.1258822392],[0,1619483,0.1246550609],[0,1620147,0.116965489],[0,1620467,0.1137901278],[0,1620570,0.1189915779],[0,1620647,0.1109194253],[0,1621460,0.1639779256],[0,1621537,0.119407143],[0,1621650,0.1125789179],[0,1621717,0.1316557616],[0,1621736,0.1513312428],[0,1621888,0.3435350379],[0,1622255,0.1316557616],[0,1622294,0.3395622572],[0,1622756,0.1912332642],[0,1622815,0.1168630731],[0,1622866,0.1364936953],[0,1623115,0.1597363704],[0,1623137,0.1611010632],[0,1623516,0.1782002779],[0,1623563,0.2862180434],[0,1623593,0.127706206],[0,1623612,0.150941883],[0,1623674,0.139268666],[0,1623861,0.1213592039],[0,1623865,0.1431355123],[0,1623924,0.4403812769],[0,1624005,0.119407143],[0,1624051,0.4949827957],[0,1624274,0.1147651126],[0,1624408,0.118913316],[0,1624621,0.1541964373],[0,1624890,0.1225435497],[0,1624985,0.2951229122],[0,1625014,0.1366440756],[0,1625306,0.1275981526],[0,1625330,0.6001343083],[0,1625340,0.2078758544],[0,1625371,0.1858611379],[0,1625372,0.118913316],[0,1625499,0.1130739064],[0,1625634,0.1239693963],[0,1625641,0.2757782076],[0,1625705,0.1174186638],[0,1625880,0.4057970654],[0,1625960,0.1133315215],[0,1625991,0.5579832075],[0,1626047,0.6199886748],[0,1626064,0.2194536096],[0,1626129,0.2144583731],[0,1626342,0.1118474543],[0,1626353,0.3137101703],[0,1626387,0.1127277855],[0,1626799,0.116965489],[0,1626868,0.1191750962],[0,1627351,0.1465216373],[0,1627392,0.2385140488],[0,1627546,0.1187516873],[0,1627580,0.1316557616],[0,1627617,0.1765513942],[0,1627680,0.1137607586],[0,1627731,0.1221616031],[0,1627786,0.6049754293],[0,1627847,0.9001600409],[0,1627849,0.4095181517],[0,1627939,0.1123008266],[0,1627964,0.1288689957],[0,1627982,0.202295743],[0,1628009,0.1530828971],[0,1628088,0.39516715],[0,1628282,0.2036692028],[0,1628502,0.1175109749],[0,1628544,0.1133315215],[0,1628594,0.2839468311],[0,1628848,0.1113974888],[0,1628963,0.2145296353],[0,1629148,0.4469517053],[0,1629425,0.1147651126],[0,1629443,0.16212056],[0,1629531,0.5610942238],[0,1629649,0.1318783827],[0,1629789,0.8729543411],[0,1629797,0.1521614157],[0,1629951,0.1226050516],[0,1630092,0.172333955],[0,1630263,0.1379671383],[0,1630293,0.1316557616],[0,1630346,0.3989738902],[0,1630498,0.1549967176],[0,1630805,0.9346775117],[0,1631055,0.241921457],[0,1631131,0.5537832899],[0,1631183,0.3148775788],[0,1631542,0.2394247448],[0,1632693,0.1117241268],[0,1632694,0.1714736416],[0,1632916,0.1536394788],[0,1633404,0.1524680797],[0,1633445,0.133443981],[0,1633616,0.2229894307],[0,1633741,0.1319653619],[0,1633784,0.1316557616],[0,1633887,0.1536394788],[0,1634055,0.1152455325],[0,1634069,0.3322925622],[0,1634124,0.216378763],[0,1634256,0.2061474253],[0,1634459,0.217681146],[0,1634587,0.3069018619],[0,1635069,0.1131555197],[0,1635127,0.1131600963],[0,1635258,0.1541964373],[0,1635596,0.1128372215],[0,1635906,0.1190723925],[0,1635907,0.1190723925],[0,1635962,0.1541964373],[0,1636019,0.1146212826],[0,1636082,0.3238851359],[0,1636099,0.1131555197],[0,1636215,0.1462812308],[0,1636278,0.150067322],[0,1636305,0.1118474543],[0,1636313,0.7550187266],[0,1636332,0.1173161889],[0,1636432,0.1116768333],[0,1636472,0.1536394788],[0,1636474,0.1136322626],[0,1636477,0.1189502649],[0,1636482,0.1316557616],[0,1636571,0.2040137474],[0,1636635,0.1305771426],[0,1636638,0.1190478202],[0,1636645,0.1301369456],[0,1636700,0.3177697163],[0,1636733,0.1239333747],[0,1636903,0.1788784323],[0,1636949,0.1755439424],[0,1636970,0.2342475481],[0,1636971,0.6524740368],[0,1636996,0.1312953261],[0,1637047,0.154932018],[0,1637074,0.1556096602],[0,1637205,0.2304953592],[0,1637256,0.5656907369],[0,1637258,0.116965489],[0,1637279,0.118913316],[0,1637344,0.1237266898],[0,1637401,0.1316557616],[0,1637597,0.1674529089],[0,1637624,0.1326185116],[0,1637703,0.1436084393],[0,1637728,0.295382547],[0,1637742,0.1316557616],[0,1637792,0.1131528307],[0,1637872,0.1296764444],[0,1637896,0.1316557616],[0,1637905,0.1401725963],[0,1638117,0.118913316],[0,1638317,0.116965489],[0,1638318,0.1131369208],[0,1638664,0.1230886531],[0,1638874,0.2593593476],[0,1638933,0.1133315215],[0,1639010,0.1530828971],[0,1639025,0.3042050263],[0,1639110,0.1109898036],[0,1639330,0.1187516873],[0,1639668,0.1202095509],[0,1639689,0.1420009166],[0,1639913,0.2729164694],[0,1640159,0.1287111554],[0,1640561,0.1896195121],[0,1640695,0.1826258714],[0,1640735,0.1156654374],[0,1640835,0.1156654374],[0,1640906,0.2733408996],[0,1640924,0.1343432285],[0,1640967,0.1131343577],[0,1641123,0.1303314194],[0,1641163,0.1518163696],[0,1641235,0.1667046182],[0,1641340,0.1303314194],[0,1641546,0.2227047145],[0,1641613,0.1113834202],[0,1641735,0.1177688571],[0,1641874,0.1523940734],[0,1641876,0.2690150818],[0,1641911,0.1176100436],[0,1641924,0.611867542],[0,1641952,0.1394434793],[0,1642000,0.111065843],[0,1642081,0.2123700847],[0,1642083,0.3524548315],[0,1642142,0.5732825415],[0,1642160,0.5946789002],[0,1642162,0.4124930644],[0,1642421,0.216678531],[0,1642480,0.1535066808],[0,1642564,0.1141392906],[0,1642727,0.1145719574],[0,1642759,0.5077512395],[0,1642877,0.159001895],[0,1642998,0.5745696925],[0,1643010,0.1110353654],[0,1643046,0.1375349207],[0,1643124,0.1298077368],[0,1643212,0.1545155274],[0,1643237,0.1136697465],[0,1643303,0.151838334],[0,1643313,0.1215100682],[0,1643426,0.1101769489],[0,1643474,0.1156654374],[0,1643560,0.7496555219],[0,1643587,0.1125754056],[0,1643666,0.1447912948],[0,1643728,0.1125754056],[0,1643810,0.1465807813],[0,1643870,0.4247844481],[0,1643956,0.1303314194],[0,1643959,0.1118431998],[0,1644058,0.1110353654],[0,1644125,0.1303314194],[0,1644136,0.1177842909],[0,1644194,0.1539656908],[0,1644209,0.1435457641],[0,1644246,0.1535066808],[0,1644301,0.1238949155],[0,1644369,0.1287111554],[0,1644427,0.1159774902],[0,1644512,0.2557091167],[0,1644568,0.2835245396],[0,1644631,0.1451150474],[0,1644685,0.2749437058],[0,1645092,0.114076021],[0,1645178,0.1918174654],[0,1645197,0.1098682329],[0,1645277,0.3432120848],[0,1645307,0.1251125845],[0,1645327,0.1529501889],[0,1645424,0.1125741926],[0,1645492,0.1673229649],[0,1645517,0.1213449012],[0,1645626,0.1123377226],[0,1645676,0.1176100436],[0,1645706,0.1361379827],[0,1645872,0.194954933],[0,1645939,0.1471732556],[0,1645979,0.5080519576],[0,1646248,0.1230829396],[0,1646285,0.1518394559],[0,1646340,0.1539809174],[0,1646504,0.1566727791],[0,1646539,0.1159774902],[0,1646560,0.2884072348],[0,1646739,0.1543062908],[0,1646905,0.1580708609],[0,1646909,0.2004701454],[0,1648019,0.1115254356],[0,1648052,0.7678417161],[0,1648271,0.1409084624],[0,1648361,0.1208208575],[0,1648496,0.151838334],[0,1648511,0.1215100682],[0,1648524,0.1535066808],[0,1648563,0.140542567],[0,1648564,0.1125741926],[0,1648567,0.2332615543],[0,1648600,0.3074963168],[0,1648606,0.1131902916],[0,1648722,0.1221890687],[0,1648727,0.1187964934],[0,1648734,0.2955916336],[0,1648913,0.7233533001],[0,1649040,0.1215100682],[0,1649044,0.2262762292],[0,1649068,0.2645839],[0,1649087,0.1633793316],[0,1649354,0.572483132],[0,1649383,0.6072796815],[0,1649421,0.1101066439],[0,1649479,0.2588226674],[0,1649659,0.9390242297],[0,1649933,0.173318015],[0,1650101,0.1366080154],[0,1650837,0.3297346156],[0,1650967,0.520987256],[0,1651093,0.112497731],[0,1651135,0.3264570074],[0,1651476,0.151566028],[0,1651750,0.1748465163],[0,1651762,0.116124286],[0,1651764,0.1289853359],[0,1651781,0.1164439219],[0,1651794,0.1513047622],[0,1651810,0.2268868687],[0,1651826,0.1127348268],[0,1651828,0.1087946076],[0,1651879,0.2878820332],[0,1651946,0.5665771389],[0,1652021,0.110722192],[0,1652041,0.1543899339],[0,1652068,0.1329906533],[0,1652154,0.1512342652],[0,1652191,0.1200646034],[0,1652773,0.1266428715],[0,1652872,0.3484463069],[0,1653003,0.1274893808],[0,1653173,0.132520729],[0,1653176,0.1635704776],[0,1653217,0.1091318827],[0,1653376,0.1229520601],[0,1653378,0.1654576214],[0,1653404,0.1320742822],[0,1653502,0.1076273166],[0,1653636,0.1762342505],[0,1653645,0.1487547537],[0,1653724,0.2928260727],[0,1653735,0.3818633396],[0,1653811,0.1314162929],[0,1653848,0.4151972696],[0,1653901,0.131673475],[0,1653904,0.1493084061],[0,1653938,0.3136795271],[0,1654045,0.2603838504],[0,1654132,0.2014639125],[0,1654199,0.5839547375],[0,1654258,0.109249576],[0,1654341,0.2448485699],[0,1654362,0.117833309],[0,1654389,0.3567941432],[0,1654411,0.1331226396],[0,1654486,0.1806764606],[0,1654591,0.1301797653],[0,1654606,0.1127177002],[0,1654620,0.2806380156],[0,1654838,0.1536802763],[0,1654888,0.1273286059],[0,1654890,0.1081168995],[0,1655083,0.34628445],[0,1655183,0.5430081835],[0,1655742,0.1493084061],[0,1655802,0.1273286059],[0,1655890,0.1237752865],[0,1656106,0.1542230757],[0,1656129,0.2772505435],[0,1656354,0.1498053691],[0,1656476,0.1273286059],[0,1656505,0.2181675194],[0,1656604,0.1504168352],[0,1656688,0.1412177063],[0,1656708,0.1104285678],[0,1656860,0.1805271793],[0,1656987,0.3184881772],[0,1657031,0.1244959861],[0,1657064,0.1081168995],[0,1657305,0.1273286059],[0,1657437,0.130686332],[0,1657478,0.1944525681],[0,1657536,0.1565425697],[0,1657701,0.409226235],[0,1657747,0.1612239308],[0,1657829,0.2514328175],[0,1657884,0.1821746275],[0,1658043,0.2622771341],[0,1658058,0.1302694658],[0,1658098,0.2001643249],[0,1658420,0.1922074666],[0,1658435,0.1487547537],[0,1658444,0.1621408937],[0,1658534,0.1755950147],[0,1658596,0.1336709672],[0,1658672,0.1476918738],[0,1658704,0.1348069899],[0,1658778,0.1273286059],[0,1658876,0.4538197208],[0,1659142,0.3802475722],[0,1659259,0.1081168995],[0,1659304,0.1762859229],[0,1659385,0.1273286059],[0,1659418,0.1620902762],[0,1659455,0.1271614919],[0,1659724,0.1329310456],[0,1659865,0.1111573688],[0,1659918,0.1438498561],[0,1659962,0.2445086307],[0,1659969,0.1075261171],[0,1660089,0.2667681189],[0,1660110,0.1971948011],[0,1660202,0.3247649404],[0,1660221,0.243654717],[0,1660253,0.145020772],[0,1660482,0.1799499771],[0,1660509,0.1097307147],[0,1660551,0.1151695488],[0,1660835,0.1629587178],[0,1660900,0.12073457],[0,1660962,0.1265748368],[0,1661102,0.2734425633],[0,1661104,0.1173049325],[0,1661190,0.1077429437],[0,1661267,0.1748097137],[0,1661331,0.1114225037],[0,1661360,0.3236867966],[0,1661424,0.1243319477],[0,1661803,0.1212039336],[0,1661938,0.1204203153],[0,1661978,0.2057706481],[0,1662119,0.1500170771],[0,1662202,0.1463152198],[0,1662208,0.159305061],[0,1662215,0.1463152198],[0,1662471,0.1615056404],[0,1662540,0.2426866875],[0,1662692,0.3953361489],[0,1662937,0.2779368856],[0,1663087,0.1399455291],[0,1663204,0.1133300597],[0,1663224,0.1200496543],[0,1663226,0.1615902602],[0,1663593,0.3542646829],[0,1664213,0.1091772201],[0,1664215,0.1244264568],[0,1664356,0.1511264659],[0,1664402,0.1114423489],[0,1664545,0.3097943466],[0,1664719,0.6442303719],[0,1665330,0.1386647074],[0,1665336,0.1115501595],[0,1665344,0.158738554],[0,1665346,0.1077147031],[0,1665459,0.5953552777],[0,1665482,0.1889976831],[0,1665630,0.6350559726],[0,1666478,0.6212310769],[0,1666509,0.1156512058],[0,1666578,0.7059137686],[0,1666603,0.1151421416],[0,1666745,0.2085472249],[0,1666795,0.4778792687],[0,1666844,0.1926676606],[0,1666876,0.2214982041],[0,1667214,0.1087390238],[0,1667272,0.119079307],[0,1667605,0.1216407185],[0,1667687,0.1077429437],[0,1667747,0.2029243358],[0,1667748,0.1781533928],[0,1668028,0.1406972778],[0,1668106,0.5290662247],[0,1668109,0.1190577272],[0,1668233,0.1166565496],[0,1668251,0.4185390524],[0,1668373,0.1857663921],[0,1668621,0.4430414363],[0,1668686,0.1153336741],[0,1668687,0.1741552486],[0,1668753,0.2846538449],[0,1668853,0.276333542],[0,1668976,0.1838985536],[0,1668995,0.1114225037],[0,1669002,0.1691171725],[0,1669050,0.1505715839],[0,1669065,0.1068897997],[0,1669079,0.1114410618],[0,1669140,0.1591732491],[0,1669437,0.1478922708],[0,1669510,0.1137438006],[0,1669526,0.1090602067],[0,1669613,0.2539595126],[0,1669819,0.2744116894],[0,1669871,0.1153336741],[0,1670011,0.1280182483],[0,1670025,0.1345409072],[0,1670156,0.3310264517],[0,1670177,0.1246226037],[0,1670513,0.1461643427],[0,1670552,0.1134252632],[0,1670578,0.3048451598],[0,1670707,0.2574167394],[0,1670855,0.1494629454],[0,1670950,0.1500170771],[0,1671008,0.1395717998],[0,1671040,0.1243616986],[0,1671044,0.2862517728],[0,1671161,0.11981046],[0,1671310,0.1261181353],[0,1671319,0.4031201522],[0,1671362,0.2123808422],[0,1671523,0.1583181925],[0,1671527,0.1245309763],[0,1671629,0.1297774717],[0,1671919,0.1458005855],[0,1671934,0.4460958294],[0,1672099,0.1568628453],[0,1672349,0.145275621],[0,1672642,0.1136589203],[0,1672689,0.1536431277],[0,1672734,0.1201320559],[0,1673137,0.1522413055],[0,1673142,0.2199110559],[0,1673259,0.1157883269],[0,1673293,0.1811271869],[0,1673388,0.1374341939],[0,1673412,0.211427467],[0,1673510,0.675479125],[0,1673544,0.225658639],[0,1673546,0.11981046],[0,1673735,0.2234216484],[0,1673756,0.1377151404],[0,1673796,0.11981046],[0,1674141,0.1178604159],[0,1674184,0.1541344656],[0,1674376,0.1540833907],[0,1674484,0.1238830613],[0,1674576,0.136341795],[0,1674608,0.1868855373],[0,1674633,0.652326443],[0,1675446,0.3288044602],[0,1675487,0.1361019871],[0,1675494,0.1325674096],[0,1675624,0.1158692368],[0,1675664,0.1139735697],[0,1675792,0.1415172676],[0,1675826,0.4338500602],[0,1675949,0.1305858395],[0,1676018,0.3434678918],[0,1676035,0.8214987478],[0,1676096,0.2850858619],[0,1676141,0.1126268329],[0,1676172,0.4077350209],[0,1676406,0.1325674096],[0,1676441,0.1217849235],[0,1676462,0.1148618942],[0,1676549,0.1238830613],[0,1676753,0.1325674096],[0,1676861,0.3023057512],[0,1676892,0.132680989],[0,1676904,0.119194188],[0,1676916,0.4904223967],[0,1677146,0.1181565154],[0,1677153,0.253832807],[0,1677157,0.1178604159],[0,1677226,0.184850144],[0,1677260,0.1134611638],[0,1677273,0.1711648603],[0,1677294,0.3101849828],[0,1677305,0.4178362837],[0,1677315,0.1690625491],[0,1677342,0.1657387715],[0,1677357,0.166297698],[0,1677554,0.1134611638],[0,1677565,0.5277355773],[0,1677603,0.1643735499],[0,1677828,0.1178604159],[0,1677954,0.3450389549],[0,1678019,0.4152736137],[0,1678024,0.1712724423],[0,1678148,0.5005871954],[0,1678174,0.1161796018],[0,1678203,0.1137797158],[0,1678427,0.2127742477],[0,1678457,0.3424747094],[0,1678487,0.1166535903],[0,1678607,0.1121764185],[0,1678788,0.1196486436],[0,1678888,0.1488210162],[0,1678960,0.1184233318],[0,1679266,0.1200725382],[0,1679270,0.1580241845],[0,1679297,0.1245942546],[0,1679337,0.1177332375],[0,1679348,0.1131381648],[0,1679368,0.1327292419],[0,1679417,0.1126268329],[0,1679516,0.12594406],[0,1679594,0.1146517363],[0,1679613,0.114112381],[0,1679646,0.1552494296],[0,1679660,0.1367011376],[0,1679693,0.148045998],[0,1679708,0.4948648595],[0,1679726,0.1209264523],[0,1679739,0.1194279826],[0,1679790,0.1797381211],[0,1679879,0.1194279826],[0,1679946,0.1979207147],[0,1679959,0.302944946],[0,1679986,0.2115993364],[0,1680110,0.154691759],[0,1680175,0.1617522796],[0,1680186,0.1121480167],[0,1680246,0.1382305434],[0,1680278,0.4155872881],[0,1680327,0.1318895539],[0,1680328,0.4774997314],[0,1680340,0.7892645144],[0,1680532,0.154691759],[0,1680659,0.3046112997],[0,1680692,0.2525178227],[0,1680757,0.3960865143],[0,1680803,0.1223863119],[0,1680930,0.1365492785],[0,1681061,0.1249261245],[0,1681200,0.1270873775],[0,1681226,0.1565299929],[0,1681227,0.1317170714],[0,1681233,0.1411661378],[0,1681318,0.1209264523],[0,1681373,0.1515462263],[0,1681441,0.1216583677],[0,1681442,0.1854745673],[0,1681546,0.1194279826],[0,1681549,0.1363953148],[0,1681567,0.3487693117],[0,1681618,0.133701444],[0,1681670,0.1397399591],[0,1681846,0.1857571817],[0,1682189,0.2348370525],[0,1682279,0.3689540408],[0,1682443,0.1189736504],[0,1682489,0.1146918309],[0,1682554,0.2716749081],[0,1682865,0.1627860353],[0,1683013,0.1121480167],[0,1683044,0.2425489447],[0,1683051,0.1354137077],[0,1683188,0.1244565424],[0,1683288,0.1131195664],[0,1683328,0.8886857631],[0,1683417,0.1712468566],[0,1683454,0.3091803711],[0,1683637,0.5157424384],[0,1683790,0.1431687544],[0,1683873,0.2609965337],[0,1683950,0.1181565154],[0,1684063,0.182412019],[0,1684125,0.133701444],[0,1684679,0.2467999391],[0,1684689,0.133701444],[0,1684714,0.308110102],[0,1684826,0.1933793303],[0,1684942,0.1162408232],[0,1684958,0.1168407245],[0,1684959,0.1516350455],[0,1684967,0.1139175861],[0,1685025,0.1620315305],[0,1685144,0.1192909412],[0,1685211,0.1917010201],[0,1685279,0.2819240667],[0,1685336,0.3198975351],[0,1685363,0.1199697176],[0,1685662,0.4066232582],[0,1685786,0.1223863119],[0,1685811,0.256090306],[0,1685823,0.1160283017],[0,1685824,0.129335471],[0,1686061,0.3757129327],[0,1686078,0.1189736504],[0,1686157,0.1255244406],[0,1686419,0.1346767269],[0,1686479,0.1427024462],[0,1686497,0.189189855],[0,1686552,0.1173149957],[0,1686561,0.1209264523],[0,1686752,0.7045514514],[0,1686756,0.2782389053],[0,1686758,0.8895773255],[0,1686781,0.1558074777],[0,1686802,0.1136175878],[0,1686808,0.5092517341],[0,1686891,0.1761176293],[0,1686895,0.7965667926],[0,1687085,0.1791965974],[0,1687168,0.1456256651],[0,1687246,0.2279039373],[0,1687291,0.1309075603],[0,1687303,0.3272941883],[0,1687341,0.1841897775],[0,1687343,0.1629108151],[0,1687426,0.1317170714],[0,1687480,0.2042903101],[0,1687541,0.1398672464],[0,1687658,0.2574783058],[0,1687713,0.3698703057],[0,1687867,0.124150514],[0,1687907,0.1328337248],[0,1687935,0.3648644889],[0,1687982,0.3099832844],[0,1688006,0.123923628],[0,1688014,0.7132868095],[0,1688016,0.1754712599],[0,1688118,0.1541367392],[0,1688276,0.8615190124],[0,1688316,0.6419475073],[0,1688355,0.118383601],[0,1688446,0.2398615712],[0,1688545,0.1144134222],[0,1688551,0.1746341466],[0,1688597,0.1534694172],[0,1688598,0.1989386696],[0,1688647,0.1552494296],[0,1688731,0.1899974506],[0,1688750,0.115657535],[0,1688752,0.1237209431],[0,1688842,0.1157489835],[0,1688900,0.1820823876],[0,1688909,0.1158769733],[0,1688935,0.113005693],[0,1689014,0.1258628076],[0,1689024,0.1838566654],[0,1689084,0.113598978],[0,1689110,0.1773157126],[0,1689174,0.4045720519],[0,1689250,0.1158769733],[0,1689327,0.1324439835],[0,1689420,0.4823839011],[0,1689425,0.1116978781],[0,1689465,0.1126268329],[0,1689514,0.1457202318],[0,1689748,0.133701444],[0,1689867,0.1188986744],[0,1689987,0.115657535],[0,1689995,0.1552494296],[0,1690010,0.2160577601],[0,1690262,0.1168407245],[0,1690277,0.1588993115],[0,1690311,0.1316127339],[0,1690346,0.1200725382],[0,1690439,0.1317170714],[0,1690501,0.1437477385],[0,1690532,0.1196486474],[0,1690568,0.1379520904],[0,1690726,0.3758092776],[0,1690792,0.1149707729],[0,1690805,0.123878422],[0,1690809,0.2901696332],[0,1690831,0.1439825361],[0,1690850,0.1959154876],[0,1690867,0.1274477124],[0,1690886,0.1637102783],[0,1690889,0.1600832229],[0,1690937,0.3681171867],[0,1690970,0.1481788508],[0,1691016,0.1160038752],[0,1691153,0.6506931518],[0,1691572,0.1232244198],[0,1692309,0.1756783343],[0,1692383,0.1924267124],[0,1692472,0.1451312129],[0,1693086,0.1609968444],[0,1693528,0.1157342397],[0,1693634,0.1288410233],[0,1693698,0.7139271354],[0,1694026,0.1154937271],[0,1694799,0.1243478974],[0,1695528,0.1351173316],[0,1695866,0.1331924634],[0,1695970,0.6555797211],[0,1696126,0.1508140316],[0,1696601,0.4184533189],[0,1696684,0.1415946022],[0,1696835,0.1351173316],[0,1696962,0.2646265782],[0,1696995,0.6283281845],[0,1697060,0.1268120213],[0,1697117,0.1483664437],[0,1697257,0.5341584191],[0,1697455,0.1590768448],[0,1697511,0.1135849268],[0,1697633,0.1516557029],[0,1698107,0.1146674141],[0,1698613,0.2411036737],[0,1698640,0.121405004],[0,1698670,0.1351173316],[0,1698685,0.196198659],[0,1698863,0.471196792],[0,1698961,0.1375568547],[0,1698968,0.2537055251],[0,1698988,0.1424044663],[0,1698990,0.2474618462],[0,1699118,0.2224588937],[0,1699179,0.1621547386],[0,1699380,0.6367837479],[0,1699608,0.5893143624],[0,1699663,0.1594826879],[0,1699728,0.1570470663],[0,1699761,0.1621956468],[0,1699880,0.1371357643],[0,1699910,0.1427880153],[0,1700037,0.1348591001],[0,1700207,0.3839578563],[0,1700220,0.1204621758],[0,1700356,0.1173073956],[0,1700366,0.1168142248],[0,1700411,0.1201100732],[0,1700442,0.1157819282],[0,1700498,0.1562474999],[0,1700499,0.1849227342],[0,1700680,0.1874265747],[0,1700829,0.1981076851],[0,1700956,0.1264131883],[0,1700965,0.1144157476],[0,1700994,0.1328718666],[0,1701183,0.1328718666],[0,1701291,0.1673392225],[0,1701299,0.1593713801],[0,1701330,0.1466405698],[0,1701404,0.1348591001],[0,1701717,0.1626804375],[0,1701830,0.1303851405],[0,1701895,0.1133030417],[0,1701927,0.1201100732],[0,1701965,0.3367691668],[0,1701966,0.1534935204],[0,1702258,0.14727834],[0,1702365,0.1424676599],[0,1702398,0.1554404629],[0,1702434,0.2786089136],[0,1702451,0.3415451985],[0,1702477,0.1629526777],[0,1702610,0.1296843064],[0,1702716,0.1946386759],[0,1702769,0.1307668536],[0,1702902,0.2328215346],[0,1703001,0.1838655679],[0,1703163,0.1209476057],[0,1703274,0.3577795122],[0,1703327,0.1317385136],[0,1703338,0.5467807749],[0,1703344,0.1587606312],[0,1703412,0.4679115879],[0,1703421,0.2272024034],[0,1703430,0.2755392004],[0,1703433,0.3179678862],[0,1703481,0.2868917032],[0,1703559,0.2771904655],[0,1703576,0.1189197734],[0,1703618,0.1337229393],[0,1703667,0.2808386055],[0,1703670,0.1199908453],[0,1703746,0.1236170708],[0,1703754,0.1487761724],[0,1703796,0.1306301251],[0,1703838,0.1122678312],[0,1703901,0.2149485682],[0,1703968,0.157081631],[0,1704143,0.1121973376],[0,1704213,0.1195361888],[0,1704232,0.1189947514],[0,1704301,0.5280746048],[0,1704308,0.1126477641],[0,1704409,0.3193331654],[0,1704451,0.1140018103],[0,1704482,0.1136199352],[0,1704867,0.1371890231],[0,1704994,0.2112207454],[0,1705031,0.1149917669],[0,1705048,0.1157808089],[0,1705078,0.1149917669],[0,1705092,0.6909250651],[0,1705138,0.2300614483],[0,1705163,0.6854807908],[0,1705180,0.2514571529],[0,1705187,0.1260183668],[0,1705217,0.6721008881],[0,1705239,0.1794277749],[0,1705388,0.1147128174],[0,1705399,0.1749644459],[0,1705459,0.4450081509],[0,1705460,0.4789299262],[0,1705554,0.1121973376],[0,1705581,0.1446038111],[0,1705678,0.1576409191],[0,1705742,0.1256730121],[0,1705813,0.1518928089],[0,1705892,0.2530998408],[0,1705977,0.2803259797],[0,1706976,0.1285267831],[0,1707214,0.157081631],[0,1707234,0.2601828108],[0,1707344,0.1576409191],[0,1707353,0.7391656384],[0,1707431,0.1246022973],[0,1707485,0.2813101828],[0,1707547,0.1189947514],[0,1707567,0.1389050881],[0,1707587,0.1153285775],[0,1707625,0.1136785883],[0,1707632,0.1675306077],[0,1707643,0.1686259894],[0,1707661,0.2062615038],[0,1707723,0.120153188],[0,1707744,0.6952210047],[0,1707761,0.1209476057],[0,1707784,0.1162618511],[0,1707791,0.1587606312],[0,1707805,0.1337229393],[0,1707842,0.2806480498],[0,1707854,0.1257354954],[0,1707883,0.1452271381],[0,1707921,0.3271240249],[0,1708038,0.1385085684],[0,1708167,0.1158777932],[0,1708181,0.3527369574],[0,1708185,0.3775232321],[0,1708196,0.1311491278],[0,1708285,0.1331654134],[0,1708376,0.7601075296],[0,1708400,0.1121973376],[0,1708409,0.3022561647],[0,1708414,0.5979157184],[0,1708497,0.1470068012],[0,1708675,0.9163346389],[0,1708682,0.1381464587],[0,1708729,0.1758241095],[0,1708778,0.1240659026],[0,1708940,0.3140641743],[0,1708953,0.6684735784],[0,1709006,0.139830988],[0,1709173,0.1869737475],[0,1709215,0.1861521076],[0,1709224,0.2484065203],[0,1709243,0.121874304],[0,1709251,0.1436950927],[0,1709260,0.113199128],[0,1709455,0.5502057738],[0,1709523,0.3427966125],[0,1709530,0.5551398954],[0,1709827,0.7014164213],[0,1709862,0.1186117345],[0,1710056,0.1372617711],[0,1710166,0.1491625599],[0,1710173,0.1589438596],[0,1710229,0.1213374471],[0,1710253,0.1149135515],[0,1710279,0.1147154781],[0,1710331,0.1153753684],[0,1710379,0.1589438596],[0,1710447,0.1285595317],[0,1710503,0.7731813959],[0,1710584,0.4203259562],[0,1710875,0.145188942],[0,1711221,0.8488364706],[0,1711338,0.1564325206],[0,1712075,0.1342843115],[0,1712164,0.6421306717],[0,1712187,0.432875242],[0,1712411,0.1370534653],[0,1712417,0.1143733333],[0,1712434,0.1202676352],[0,1712440,0.1205889573],[0,1712600,0.2847618056],[0,1712668,0.1143919615],[0,1712786,0.1221927296],[0,1712886,0.9842629661],[0,1713300,0.1729194983],[0,1713413,0.2013461195],[0,1713500,0.5314813209],[0,1713586,0.197136771],[0,1713597,0.3029964354],[0,1713667,0.171222455],[0,1713687,0.1431643251],[0,1713715,0.1485142176],[0,1713730,0.1402091023],[0,1713758,0.1978853656],[0,1713798,0.3389859455],[0,1713807,0.1800208808],[0,1713938,0.1189636929],[0,1713953,0.1648844663],[0,1713964,0.5180375526],[0,1714488,0.1402091023],[0,1714522,0.3546463789],[0,1714610,0.121963212],[0,1714623,0.1731125939],[0,1714703,0.6585763987],[0,1714777,0.2401773673],[0,1714806,0.1463608025],[0,1714832,0.8719112454],[0,1714839,0.6872953275],[0,1714853,0.4894006356],[0,1714859,0.1792891643],[0,1714861,0.1199840819],[0,1714877,0.7122010323],[0,1714897,0.7089508009],[0,1714979,0.1488761564],[0,1715046,0.1421584398],[0,1715054,0.1232117357],[0,1715065,0.2272010521],[0,1715068,0.2451624586],[0,1715107,0.1823376333],[0,1715163,0.7649751616],[0,1715224,0.1204689596],[0,1715426,0.142352746],[0,1715441,0.4838789794],[0,1715529,0.1201131376],[0,1715700,0.4070417972],[0,1715866,0.1253619478],[0,1715870,0.1420500684],[0,1715896,0.2127981372],[0,1715906,0.3821228585],[0,1715911,0.2211166877],[0,1716085,0.1764619785],[0,1716276,0.1189636929],[0,1716417,0.223593152],[0,1716469,0.1341589211],[0,1716483,0.1253619478],[0,1716511,0.3575853269],[0,1716574,0.1882562508],[0,1716577,0.1995897811],[0,1716605,0.4864473981],[0,1716637,0.1492305189],[0,1716644,0.1402091023],[0,1716673,0.2717769022],[0,1716682,0.3499820018],[0,1716740,0.2959923306],[0,1716808,0.2911475908],[0,1716927,0.1822131807],[0,1716952,0.1637564731],[0,1716983,0.1644552831],[0,1717057,0.1682000217],[0,1717067,0.2418469967],[0,1717215,0.1455549513],[0,1717259,0.1841435492],[0,1717276,0.1524121914],[0,1717287,0.1339457669],[0,1717301,0.1811976288],[0,1717302,0.3944342702],[0,1717389,0.175853085],[0,1717404,0.2389401229],[0,1717470,0.1828546147],[0,1717492,0.1219366885],[0,1717541,0.6310143664],[0,1717551,0.1305363197],[0,1717611,0.1308140285],[0,1717719,0.1414755527],[0,1717862,0.1637564731],[0,1717870,0.6223526105],[0,1717887,0.1402091023],[0,1717961,0.1198001701],[0,1717977,0.2328597143],[0,1718000,0.6508358157],[0,1718001,0.1716798892],[0,1718076,0.1643202789],[0,1718089,0.1402091023],[0,1718191,0.2943571704],[0,1718270,0.1344187638],[0,1718303,0.1578529769],[0,1718329,0.1360186057],[0,1718367,0.1654490355],[0,1718466,0.1667141933],[0,1718472,0.6051661429],[0,1718642,0.7518256961],[0,1718656,0.3033159328],[0,1718677,0.2408468736],[0,1718743,0.6165942881],[0,1718771,0.150559237],[0,1718778,0.4112589095],[0,1718803,0.1232117357],[0,1718804,0.1688638175],[0,1718805,0.1678228266],[0,1718815,0.1197814133],[0,1718817,0.1815642665],[0,1718825,0.1694266296],[0,1718900,0.1426244745],[0,1718933,0.1273305762],[0,1718957,0.5954610038],[0,1718967,0.143383219],[0,1719137,0.1782941062],[0,1719138,0.1501393142],[0,1719192,0.1357216656],[0,1719222,0.1402091023],[0,1719232,0.1943293837],[0,1719242,0.1198451437],[0,1719253,0.3335477921],[0,1719271,0.1538644328],[0,1719349,0.1942227654],[0,1719355,0.1316676675],[0,1719356,0.2133391409],[0,1719383,0.1429603412],[0,1719403,0.2147411385],[0,1719429,0.2936851075],[0,1719442,0.5799789745],[0,1719447,0.1475907183],[0,1719450,0.1402091023],[0,1719462,0.121005007],[0,1719473,0.1258199622],[0,1719487,0.1648844663],[0,1719489,0.2775748992],[0,1719498,0.1739855129],[0,1719502,0.1298754835],[0,1719530,0.1223927287],[0,1719554,0.129643017],[0,1719600,0.1483476754],[0,1719646,0.2436654211],[0,1719648,0.275433695],[0,1719654,0.1631930487],[0,1719697,0.1282831364],[0,1719757,0.2917965071],[0,1719761,0.1873019846],[0,1719805,0.1391144766],[0,1719865,0.1465022878],[0,1719899,0.1391144766],[0,1719932,0.1251366433],[0,1719957,0.2513005923],[0,1720062,0.5628409628],[0,1720066,0.1288814267],[0,1720141,0.1246413651],[0,1720178,0.1940964184],[0,1720489,0.3040122947],[0,1720667,0.1214745154],[0,1720910,0.1381357206],[0,1720929,0.1178977965],[0,1721146,0.2497208929],[0,1721300,0.2780928364],[0,1721306,0.135520311],[0,1721547,0.128652692],[0,1722500,0.119213284],[0,1722503,0.1900223218],[0,1722506,0.1631930487],[0,1722520,0.1848926266],[0,1722540,0.22250556],[0,1722574,0.1363017587],[0,1722586,0.1776144659],[0,1722615,0.1178977965],[0,1722658,0.2232765028],[0,1722675,0.1757690427],[0,1722747,0.2199172585],[0,1722759,0.1684689517],[0,1722790,0.4380215173],[0,1722813,0.1207124963],[0,1722858,0.4251300802],[0,1722917,0.1411660858],[0,1722944,0.218407578],[0,1722946,0.1184341376],[0,1722949,0.1634559044],[0,1723014,0.1304728415],[0,1723064,0.2348581475],[0,1723138,0.117062082],[0,1723158,0.1748519958],[0,1723161,0.2010233702],[0,1723163,0.133768709],[0,1723199,0.1261108635],[0,1723221,0.1475702832],[0,1723228,0.1454652108],[0,1723241,0.1177952591],[0,1723263,0.1697775511],[0,1723275,0.1944080365],[0,1723281,0.119213284],[0,1723290,0.3936153717],[0,1723308,0.1289222491],[0,1723426,0.1391144766],[0,1723428,0.152397518],[0,1723467,0.1804745665],[0,1723478,0.1285668242],[0,1723493,0.1344314932],[0,1723498,0.3658050182],[0,1723510,0.183922398],[0,1723566,0.8907953747],[0,1723568,0.8684648117],[0,1723569,0.1350576301],[0,1723581,0.1242873993],[0,1723647,0.2648750436],[0,1723691,0.1189168056],[0,1723694,0.1416816516],[0,1723695,0.5034288474],[0,1723698,0.3255623935],[0,1723747,0.1797974934],[0,1723765,0.1288507031],[0,1723791,0.8886857631],[0,1723792,0.3407599511],[0,1723800,0.1391144766],[0,1723821,0.1950878836],[0,1723833,0.1551077297],[0,1723834,0.3005427725],[0,1723837,0.1550385783],[0,1723847,0.163413965],[0,1723863,0.1192159938],[0,1723898,0.1647867479],[0,1723910,0.118876495],[0,1723984,0.1321649844],[0,1723998,0.2014373495],[0,1724002,0.1637564731],[0,1724015,0.3549098196],[0,1724031,0.1188170682],[0,1724066,0.1328185871],[0,1724081,0.1283749855],[0,1724086,0.1637564731],[0,1724087,0.1268663527],[0,1724102,0.5459563317],[0,1724115,0.1175153125],[0,1724128,0.1634683568],[0,1724172,0.2012364376],[0,1724194,0.13262958],[0,1724199,0.1383688338],[0,1724256,0.385483695],[0,1724273,0.1178977965],[0,1724294,0.1602179264],[0,1724310,0.1391144766],[0,1724344,0.1242873993],[0,1724400,0.144303843],[0,1724401,0.1185821743],[0,1724417,0.1211482097],[0,1724421,0.1369601042],[0,1724441,0.119609885],[0,1724506,0.1579406265],[0,1724532,0.3418716594],[0,1724543,0.1242873993],[0,1724556,0.1247447944],[0,1724626,0.2262086496],[0,1724645,0.1655509985],[0,1724665,0.1361103292],[0,1724671,0.152363917],[0,1724672,0.1398905332],[0,1724680,0.1481457348],[0,1724690,0.118731272],[0,1724706,0.1765733337],[0,1724720,0.1614568837],[0,1724741,0.2788069934],[0,1724770,0.4839792151],[0,1724786,0.4887523801],[0,1724790,0.1272693614],[0,1724794,0.3404319442],[0,1724829,0.1189168056],[0,1724837,0.1210005749],[0,1724840,0.118731272],[0,1724859,0.1626300053],[0,1724863,0.3005501832],[0,1724901,0.1332433529],[0,1724912,0.4263274407],[0,1724921,0.118876495],[0,1724937,0.1391144766],[0,1724946,0.1294611451],[0,1724948,0.1471893805],[0,1724968,0.1169932883],[0,1724978,0.5630576254],[0,1724981,0.1262533656],[0,1724985,0.1723876052],[0,1725025,0.219048997],[0,1725032,0.1783955677],[0,1725034,0.6051136011],[0,1725053,0.3149170447],[0,1725075,0.1174157524],[0,1725098,0.1517191704],[0,1725126,0.1386933582],[0,1725128,0.1174443457],[0,1725163,0.1391144766],[0,1725184,0.4686475153],[0,1725215,0.3053331752],[0,1725220,0.1210005749],[0,1725223,0.5885522132],[0,1725226,0.1636075987],[0,1725259,0.8108340162],[0,1725265,0.5681410685],[0,1725373,0.1391144766],[0,1725383,0.1531132201],[0,1725420,0.4306892021],[0,1725421,0.1193933589],[0,1725454,0.2069271017],[0,1725461,0.1183938516],[0,1725473,0.1516897526],[0,1725475,0.188243475],[0,1725517,0.1182487176],[0,1725528,0.2596821745],[0,1725555,0.4664182258],[0,1725614,0.5075821],[0,1725638,0.1712839043],[0,1725671,0.119213284],[0,1725808,0.1184341376],[0,1725824,0.1683726963],[0,1725845,0.1259540394],[0,1725854,0.1572922984],[0,1725887,0.1178977965],[0,1725890,0.253342066],[0,1725900,0.6332698087],[0,1725905,0.5542332422],[0,1725922,0.1895610442],[0,1725965,0.2949488039],[0,1725987,0.1391144766],[0,1726271,0.1637564731],[0,1726294,0.1391144766],[0,1726379,0.3533031388],[0,1726410,0.1371167268],[0,1726441,0.4389302708],[0,1726491,0.778752296],[0,1726495,0.2419022291],[0,1726500,0.1576503947],[0,1726514,0.1232595725],[0,1726593,0.1169625793],[0,1726595,0.162130746],[0,1726604,0.1528765787],[0,1726618,0.4772918558],[0,1726633,0.1626300053],[0,1726646,0.2064528495],[0,1726652,0.4198923682],[0,1726659,0.6015735815],[0,1726691,0.1567703342],[0,1726699,0.8425153082],[0,1726731,0.2799888056],[0,1726761,0.1199767151],[0,1726764,0.1667336918],[0,1726792,0.1302800005],[0,1726814,0.4673817735],[0,1726872,0.1153206809],[0,1726899,0.1178748234],[0,1726905,0.1254958228],[0,1726956,0.1264013601],[0,1726958,0.1461582015],[0,1727088,0.1188651731],[0,1727089,0.1746637411],[0,1727114,0.136961618],[0,1727146,0.136961618],[0,1727155,0.1346611024],[0,1727168,0.2502962247],[0,1727229,0.2466010576],[0,1727243,0.3777603547],[0,1727255,0.1270797829],[0,1727323,0.1981172452],[0,1727402,0.1238362291],[0,1727421,0.1360013073],[0,1727424,0.1189362681],[0,1727431,0.2197306856],[0,1727443,0.1571875768],[0,1727454,0.3088672001],[0,1727468,0.115699177],[0,1727496,0.1465235735],[0,1727536,0.1421646677],[0,1727537,0.2350201069],[0,1727568,0.1341595825],[0,1727569,0.2521538464],[0,1727583,0.2424124656],[0,1727593,0.423261828],[0,1727607,0.136961618],[0,1727633,0.1348644275],[0,1727639,0.1286372654],[0,1727703,0.136961618],[0,1727727,0.1529126679],[0,1727734,0.1221740276],[0,1727779,0.1631930487],[0,1727896,0.1166391596],[0,1727926,0.1230615325],[0,1727945,0.1313124292],[0,1727953,0.1513064088],[0,1727957,0.120358513],[0,1727970,0.2690147398],[0,1727971,0.1705718305],[0,1727983,0.1162961752],[0,1728024,0.1347301028],[0,1728032,0.1241347582],[0,1728035,0.180819167],[0,1728043,0.1683074351],[0,1728162,0.1873846656],[0,1728169,0.2174809224],[0,1728252,0.2157265372],[0,1728258,0.1198445478],[0,1728264,0.1158014412],[0,1728274,0.1189757712],[0,1728315,0.1350417362],[0,1728317,0.1441908311],[0,1728320,0.1545533267],[0,1728364,0.4679384644],[0,1728373,0.4546058228],[0,1728468,0.2207292686],[0,1728541,0.1322130486],[0,1728656,0.299977833],[0,1728680,0.1439037332],[0,1728769,0.1177825324],[0,1728814,0.1626300053],[0,1728934,0.4770914124],[0,1728958,0.1620673428],[0,1728972,0.1345784096],[0,1729122,0.1495889589],[0,1729158,0.1148687147],[0,1729193,0.1574808762],[0,1729239,0.136961618],[0,1729252,0.1608580152],[0,1729492,0.1215736349],[0,1729497,0.1187625098],[0,1729516,0.1638273526],[0,1729520,0.1403362074],[0,1729523,0.1241347582],[0,1729585,0.1167182648],[0,1729588,0.1289419586],[0,1729591,0.1893659118],[0,1729593,0.746888015],[0,1729604,0.1631930487],[0,1729622,0.3158793668],[0,1729683,0.1176375111],[0,1729723,0.1643652935],[0,1729881,0.3903552185],[0,1729931,0.1344045129],[0,1729955,0.1285814194],[0,1730049,0.116672893],[0,1730228,0.1266694334],[0,1730233,0.1351068648],[0,1730254,0.1171609486],[0,1730256,0.3368246354],[0,1730376,0.115349198],[0,1730419,0.1171134254],[0,1730430,0.1175952842],[0,1730433,0.1166326971],[0,1730482,0.1417689977],[0,1730540,0.3455463271],[0,1730548,0.1385454863],[0,1730572,0.1160028182],[0,1730660,0.1273063617],[0,1730665,0.2587108064],[0,1730671,0.1213370293],[0,1730690,0.1158014412],[0,1730700,0.1526577948],[0,1730715,0.1216183357],[0,1730734,0.6543215868],[0,1730767,0.11752886],[0,1731178,0.1384760993],[0,1731233,0.2106468899],[0,1731268,0.136036337],[0,1731279,0.7740068593],[0,1731310,0.1307709485],[0,1731313,0.3734789721],[0,1731321,0.1173927934],[0,1731327,0.7646280054],[0,1731331,0.1232241981],[0,1731362,0.1148687147],[0,1731373,0.1663239847],[0,1731408,0.4420822401],[0,1731428,0.1189471223],[0,1731495,0.5943295925],[0,1731555,0.171933607],[0,1731592,0.1166793558],[0,1731606,0.1261847515],[0,1731658,0.1372628429],[0,1731683,0.1171134254],[0,1731729,0.4522582808],[0,1732327,0.1166158811],[0,1732385,0.2519293899],[0,1732392,0.2390819695],[0,1732431,0.2822023909],[0,1732436,0.115349198],[0,1732450,0.1411567089],[0,1732467,0.1353994943],[0,1732488,0.4768262247],[0,1732495,0.1187300834],[0,1732520,0.1617415367],[0,1732568,0.1354573465],[0,1732570,0.3443134209],[0,1732581,0.1158014412],[0,1732585,0.1340461942],[0,1732594,0.1198445478],[0,1732603,0.136036337],[0,1732631,0.1166326971],[0,1732682,0.1616752658],[0,1732697,0.4102784188],[0,1732698,0.2638370947],[0,1732701,0.1616752658],[0,1732702,0.1169869502],[0,1732777,0.2403315349],[0,1732807,0.2296958297],[0,1732856,0.1212657177],[0,1732857,0.1882213411],[0,1732908,0.1817781365],[0,1732919,0.1612947733],[0,1732969,0.1504278014],[0,1732975,0.1171536459],[0,1733014,0.1753032592],[0,1733088,0.117116128],[0,1733119,0.1582798178],[0,1733127,0.1341186587],[0,1733153,0.1167775333],[0,1733195,0.1281770616],[0,1733220,0.1743665741],[0,1733225,0.2445570762],[0,1733256,0.1798721152],[0,1733257,0.2696195517],[0,1733280,0.3183574704],[0,1733286,0.1815901971],[0,1733305,0.2507981156],[0,1733319,0.1340461942],[0,1733356,0.176761146],[0,1733383,0.1435146212],[0,1733389,0.3388771535],[0,1733430,0.2270770155],[0,1733466,0.151894563],[0,1733476,0.1612225645],[0,1733488,0.115699177],[0,1733513,0.1616671769],[0,1733568,0.1240339916],[0,1733571,0.1628004408],[0,1733617,0.1607458521],[0,1733620,0.1607441563],[0,1733641,0.1705545517],[0,1733642,0.3016323474],[0,1733667,0.164494932],[0,1733677,0.1238362291],[0,1733712,0.1284214913],[0,1733740,0.1171207278],[0,1733743,0.162237663],[0,1733754,0.2011733991],[0,1733781,0.3748891245],[0,1733795,0.436937825],[0,1733813,0.1489749786],[0,1733837,0.1407429366],[0,1733856,0.1616752658],[0,1733930,0.3095352982],[0,1733939,0.1158014412],[0,1733976,0.117116128],[0,1733982,0.1194089706],[0,1734070,0.162237663],[0,1734098,0.1708257359],[0,1734136,0.1162961752],[0,1734295,0.1232241981],[0,1734302,0.1616752658],[0,1734313,0.1570875461],[0,1734329,0.1493412125],[0,1734361,0.1365597726],[0,1734365,0.2045028598],[0,1734553,0.1616752658],[0,1734594,0.121983145],[0,1734601,0.1392881335],[0,1734609,0.1187338686],[0,1734711,0.1867418305],[0,1734772,0.2183769409],[0,1734813,0.1616752658],[0,1734822,0.1173927934],[0,1734834,0.1684210139],[0,1734863,0.1166391596],[0,1734877,0.1258422521],[0,1734882,0.17687522],[0,1734891,0.1386950171],[0,1734893,0.1338166552],[0,1734897,0.1628004408],[0,1734899,0.1172930208],[0,1734907,0.1248387377],[0,1734912,0.1312765846],[0,1734938,0.136036337],[0,1734998,0.1160028182],[0,1735068,0.136036337],[0,1735090,0.1235032625],[0,1735098,0.4123494196],[0,1735104,0.2210935837],[0,1735124,0.1625702281],[0,1735162,0.162237663],[0,1735178,0.1444468375],[0,1735197,0.2187281382],[0,1735218,0.118493067],[0,1735246,0.1212657177],[0,1735254,0.2610212057],[0,1735278,0.1629250645],[0,1735291,0.2229952973],[0,1735304,0.181152713],[0,1735335,0.3210819238],[0,1735342,0.3769233141],[0,1735343,0.1161514278],[0,1735365,0.1806950916],[0,1735366,0.2131427684],[0,1735381,0.242073959],[0,1735385,0.6277059252],[0,1735389,0.1188365282],[0,1735397,0.1616752658],[0,1735416,0.5030595718],[0,1735442,0.1390913593],[0,1735461,0.2378127328],[0,1735552,0.1194480148],[0,1735561,0.2217930984],[0,1735563,0.1348357779],[0,1735622,0.132370584],[0,1735637,0.2204735885],[0,1735666,0.1615586766],[0,1735672,0.4843436872],[0,1735705,0.25186875],[0,1735714,0.316970883],[0,1735741,0.1336558638],[0,1735760,0.1332343293],[0,1735765,0.1415336733],[0,1735778,0.1200371078],[0,1735823,0.1522588779],[0,1735827,0.1265850959],[0,1735832,0.1206201878],[0,1735859,0.1166507349],[0,1735860,0.1695771471],[0,1735865,0.1147084994],[0,1735872,0.1982520683],[0,1735877,0.1380659351],[0,1735883,0.1186755022],[0,1735899,0.1206510397],[0,1735910,0.4326366038],[0,1735915,0.1804713186],[0,1735921,0.1353787459],[0,1735933,0.1815236336],[0,1735940,0.1394565768],[0,1735942,0.1183807595],[0,1735954,0.2757948087],[0,1735989,0.2352564394],[0,1736021,0.1179579747],[0,1736041,0.1163449142],[0,1736069,0.1609967386],[0,1736094,0.1163449142],[0,1736095,0.1273340882],[0,1736104,0.1268647491],[0,1736152,0.1169527515],[0,1736157,0.2363083389],[0,1736173,0.118724654],[0,1736174,0.1157135089],[0,1736178,0.3341396592],[0,1736185,0.1183321112],[0,1736189,0.1147084994],[0,1736200,0.115510074],[0,1736232,0.1341994446],[0,1736235,0.1185728636],[0,1736247,0.142402441],[0,1736257,0.1609967386],[0,1736261,0.1251624036],[0,1736290,0.1559333474],[0,1736310,0.1604351808],[0,1736327,0.1159928144],[0,1736357,0.1511307542],[0,1736381,0.1609967386],[0,1736388,0.1353787459],[0,1736395,0.127201744],[0,1736399,0.3086487833],[0,1736410,0.1458554218],[0,1736428,0.1166507349],[0,1736440,0.1309938299],[0,1736446,0.1159909504],[0,1736502,0.2996883283],[0,1736530,0.1231886002],[0,1736532,0.1251624036],[0,1736582,0.1659081096],[0,1736631,0.1156547033],[0,1736636,0.1492418849],[0,1736654,0.134423814],[0,1736658,0.1206201878],[0,1736660,0.1567453532],[0,1736661,0.3532168979],[0,1736662,0.1604351808],[0,1736665,0.1274209601],[0,1736670,0.118086624],[0,1736672,0.1166507349],[0,1736689,0.6239988455],[0,1736693,0.1622356923],[0,1736694,0.1176144086],[0,1736747,0.2678385154],[0,1736757,0.2167975396],[0,1736811,0.1299633864],[0,1736822,0.1205172959],[0,1736829,0.1552136935],[0,1736844,0.2630676594],[0,1736867,0.248065018],[0,1736900,0.1276151635],[0,1736901,0.1183321112],[0,1736923,0.2149773301],[0,1736924,0.1697317259],[0,1736958,0.1736505375],[0,1736971,0.1182926404],[0,1736995,0.163034225],[0,1737000,0.4499691327],[0,1737046,0.17249339],[0,1737062,0.134266301],[0,1737070,0.1526247255],[0,1737074,0.2205958976],[0,1737145,0.1800501006],[0,1737188,0.1242095918],[0,1737192,0.1181190239],[0,1737210,0.2471074721],[0,1737223,0.1609967386],[0,1737229,0.2788846565],[0,1737232,0.1919573906],[0,1737241,0.1161758388],[0,1737251,0.1281236149],[0,1737279,0.2310429789],[0,1737280,0.1535166948],[0,1737299,0.4039149344],[0,1737315,0.2993202499],[0,1737322,0.1177393611],[0,1737335,0.132820478],[0,1737346,0.3637376817],[0,1737368,0.1351477503],[0,1737378,0.1691777226],[0,1737401,0.1225415001],[0,1737420,0.1514065284],[0,1737427,0.1197256547],[0,1737431,0.1178498012],[0,1737438,0.3962823678],[0,1737439,0.5635792252],[0,1737452,0.1209725179],[0,1737456,0.1353787459],[0,1737483,0.1265029835],[0,1737555,0.266851645],[0,1737574,0.1209725179],[0,1737580,0.1255163118],[0,1737584,0.1482521481],[0,1737617,0.3675869708],[0,1737618,0.1318228616],[0,1737626,0.1167102261],[0,1737650,0.1161356683],[0,1737654,0.1183321112],[0,1737655,0.1158423712],[0,1737665,0.1553678182],[0,1737668,0.4059568202],[0,1737701,0.1197256547],[0,1737709,0.3915555693],[0,1737719,0.1290744399],[0,1737720,0.1216183357],[0,1737860,0.1252336189],[0,1737876,0.1609967386],[0,1737877,0.1334157255],[0,1737899,0.1263020229],[0,1737918,0.2520401394],[0,1737952,0.3871896656],[0,1737953,0.1169527515],[0,1738022,0.166097091],[0,1738068,0.1156948492],[0,1738079,0.2460289036],[0,1738094,0.1219863298],[0,1738096,0.1212657177],[0,1738098,0.1212965948],[0,1738104,0.1247047679],[0,1738138,0.1192001784],[0,1738144,0.1252336189],[0,1738145,0.2838534658],[0,1738158,0.2796238119],[0,1738175,0.1794651872],[0,1738192,0.1548372026],[0,1738208,0.1171918379],[0,1738255,0.1176404272],[0,1738273,0.1989273469],[0,1738280,0.1655548561],[0,1738283,0.119241003],[0,1738293,0.1353787459],[0,1738312,0.1687873163],[0,1738322,0.2835079329],[0,1738349,0.1444647755],[0,1738379,0.4716737747],[0,1738409,0.1334009209],[0,1738420,0.1185728636],[0,1738421,0.1156547033],[0,1738425,0.3339385451],[0,1738426,0.122577069],[0,1738435,0.1208249216],[0,1738439,0.1551856323],[0,1738440,0.1164739865],[0,1738450,0.143625844],[0,1738453,0.1621209948],[0,1738495,0.1875449181],[0,1738497,0.1145778874],[0,1738501,0.1356701849],[0,1738505,0.1161236622],[0,1738525,0.2404225543],[0,1738537,0.2617491272],[0,1738598,0.1982001252],[0,1738630,0.8032814714],[0,1738648,0.1709463799],[0,1738670,0.1353787459],[0,1738689,0.164843894],[0,1738700,0.1194802877],[0,1738705,0.1339192499],[0,1738717,0.1615586766],[0,1738718,0.6041802214],[0,1738725,0.1324742451],[0,1738733,0.1337482543],[0,1738736,0.2462791527],[0,1738771,0.1737886769],[0,1738777,0.1318228616],[0,1738824,0.1551185043],[0,1738971,0.392879179],[0,1738978,0.1615586766],[0,1739005,0.1604351808],[0,1739033,0.1609967386],[0,1739037,0.259033501],[0,1739049,0.194843176],[0,1739050,0.1177701005],[0,1739057,0.1548804698],[0,1739075,0.207353007],[0,1739121,0.1206201878],[0,1739122,0.1293765901],[0,1739128,0.2509733576],[0,1739149,0.1177393611],[0,1739153,0.1182523616],[0,1739201,0.1255398539],[0,1739224,0.1156547033],[0,1739226,0.2262817038],[0,1739236,0.1337482543],[0,1739249,0.4395388817],[0,1739253,0.4112064355],[0,1739271,0.1183127814],[0,1739283,0.1303403563],[0,1739286,0.1299340547],[0,1739306,0.2411705054],[0,1739320,0.4018531019],[0,1739396,0.1161356683],[0,1739414,0.1402263324],[0,1739464,0.1535843174],[0,1739466,0.2203404162],[0,1739467,0.1376165273],[0,1739487,0.1924585077],[0,1739522,0.1238719037],[0,1739535,0.1300744908],[0,1739568,0.1164712862],[0,1739588,0.3536055259],[0,1739604,0.1371343587],[0,1739614,0.6781952449],[0,1739616,0.1210754692],[0,1739625,0.114126349],[0,1739639,0.1536243667],[0,1739650,0.1338528699],[0,1739653,0.1332623906],[0,1739658,0.1333902282],[0,1739680,0.4772232839],[0,1739697,0.1210754692],[0,1739702,0.1261120877],[0,1739716,0.2564163767],[0,1739758,0.1206201878],[0,1739760,0.1175926664],[0,1739762,0.3468977477],[0,1739771,0.303251522],[0,1739783,0.1177701005],[0,1739800,0.1849605829],[0,1739813,0.3218727336],[0,1739828,0.6657630308],[0,1739887,0.1261120877],[0,1739888,0.1353787459],[0,1739910,0.1151603733],[0,1739952,0.1378830682],[0,1740012,0.168869335],[0,1740024,0.122577069],[0,1740037,0.1609967386],[0,1740086,0.1252659007],[0,1740206,0.1277701183],[0,1740225,0.212519439],[0,1740229,0.122577069],[0,1740230,0.1353787459],[0,1740241,0.2839486931],[0,1740294,0.2967861913],[0,1740338,0.1194399482],[0,1740348,0.1161356683],[0,1740355,0.1479190876],[0,1740386,0.3476239564],[0,1740440,0.3189333851],[0,1740453,0.1293765901],[0,1740485,0.1206201878],[0,1740488,0.1333902282],[0,1740498,0.1233862012],[0,1740545,0.1273748315],[0,1740589,0.1621209948],[0,1740610,0.1164785825],[0,1740622,0.1206201878],[0,1740625,0.1161945099],[0,1740782,0.1270907599],[0,1740788,0.1293884715],[0,1740838,0.1159974076],[0,1740846,0.5853897853],[0,1740848,0.2955719845],[0,1740923,0.1156547033],[0,1740929,0.1353787459],[0,1740938,0.1167102261],[0,1740951,0.1210462745],[0,1740964,0.1621209948],[0,1740967,0.1273340882],[0,1740999,0.1450001736],[0,1741021,0.1615586766],[0,1741025,0.2915570903],[0,1741031,0.1445763949],[0,1741057,0.5584253028],[0,1741076,0.1238253636],[0,1741111,0.1366472215],[0,1741136,0.1564639547],[0,1741177,0.2152207739],[0,1741183,0.1195450434],[0,1741198,0.4958175873],[0,1741207,0.1172353298],[0,1741252,0.1511836582],[0,1741256,0.1204431496],[0,1741274,0.4264711256],[0,1741471,0.1635607937],[0,1741479,0.1172755566],[0,1741516,0.1791431945],[0,1741523,0.2756282607],[0,1741543,0.7574514323],[0,1741638,0.1366472215],[0,1741705,0.126724428],[0,1741713,0.1876599799],[0,1741725,0.2247651121],[0,1741746,0.1366472215],[0,1741753,0.1951573396],[0,1741765,0.260221044],[0,1741776,0.1257572105],[0,1741779,0.2439079995],[0,1741815,0.1519192267],[0,1741838,0.2036197229],[0,1741842,0.1175010469],[0,1741857,0.1169322945],[0,1741906,0.1655917589],[0,1741934,0.1160152213],[0,1742048,0.2729505727],[0,1742067,0.1310151088],[0,1742128,0.119573714],[0,1742150,0.1281303168],[0,1742161,0.318885991],[0,1742254,0.4565522904],[0,1742264,0.4407017769],[0,1742330,0.1177099555],[0,1742356,0.1351189401],[0,1742380,0.170132791],[0,1742382,0.1935425755],[0,1742385,0.3525273014],[0,1742390,0.1172353298],[0,1742419,0.8131234434],[0,1742423,0.1218653976],[0,1742473,0.1638068416],[0,1742477,0.2637953853],[0,1742522,0.406654229],[0,1742546,0.1504605995],[0,1742630,0.3771929893],[0,1742651,0.1172288625],[0,1742693,0.1246333866],[0,1742700,0.1646224498],[0,1742702,0.1981037935],[0,1742704,0.237034669],[0,1742723,0.2605336856],[0,1742733,0.1453160872],[0,1742735,0.1521911415],[0,1742804,0.1568191234],[0,1742806,0.1173738085],[0,1742819,0.2603850365],[0,1742852,0.1194770786],[0,1742889,0.1272614687],[0,1742932,0.1366472215],[0,1742959,0.1173331932],[0,1742967,0.1159443898],[0,1742981,0.1615586766],[0,1742988,0.4653653336],[0,1742998,0.2448630508],[0,1743093,0.1867789569],[0,1743098,0.2418830697],[0,1743101,0.1477423863],[0,1743103,0.1199434986],[0,1743137,0.2156411862],[0,1743151,0.1424606303],[0,1743157,0.1609967386],[0,1743169,0.1366472215],[0,1743171,0.1615586766],[0,1743211,0.1333007144],[0,1743245,0.1172307294],[0,1743281,0.1827792903],[0,1743289,0.1621209948],[0,1743299,0.117693953],[0,1743317,0.1625685095],[0,1743368,0.6920940464],[0,1743389,0.1580172311],[0,1743417,0.2454589938],[0,1743420,0.2025038112],[0,1743423,0.3072393452],[0,1743431,0.2933145584],[0,1743438,0.1190108173],[0,1743459,0.1187169844],[0,1743470,0.1306873596],[0,1743485,0.1859263015],[0,1743486,0.3077772565],[0,1743489,0.1178896871],[0,1743525,0.1766940655],[0,1743558,0.1608802446],[0,1743578,0.1190511348],[0,1743591,0.1387287487],[0,1743635,0.1365341439],[0,1743653,0.1238253636],[0,1743666,0.1181921798],[0,1743734,0.1603187655],[0,1743744,0.135006077],[0,1743770,0.1175833843],[0,1743796,0.1839611928],[0,1743798,0.1543389015],[0,1743814,0.1784747599],[0,1743840,0.1990568096],[0,1743841,0.1168920854],[0,1743852,0.1603187655],[0,1743853,0.1365341439],[0,1743865,0.2007805306],[0,1743866,0.2030446055],[0,1743877,0.448729285],[0,1743890,0.207718587],[0,1743891,0.1203161388],[0,1743911,0.1468127811],[0,1743913,0.1281279099],[0,1743915,0.334914545],[0,1743926,0.1349013111],[0,1743931,0.1345427709],[0,1743939,0.2636668462],[0,1743957,0.1195341809],[0,1743964,0.3821708288],[0,1743972,0.1510700503],[0,1743982,0.8501684746],[0,1744001,0.154313941],[0,1744029,0.4181846846],[0,1744040,0.1300697813],[0,1744073,0.2302144914],[0,1744087,0.1568154164],[0,1744100,0.1349013111],[0,1744108,0.7865205601],[0,1744109,0.1356840874],[0,1744112,0.2317852185],[0,1744122,0.1810205684],[0,1744129,0.1365341439],[0,1744133,0.2844239149],[0,1744134,0.1588078298],[0,1744185,0.1175833843],[0,1744237,0.1272732347],[0,1744239,0.3169371472],[0,1744270,0.123391191],[0,1744288,0.1320526806],[0,1744351,0.2412707189],[0,1744357,0.1365341439],[0,1744361,0.4090438977],[0,1744402,0.1267671448],[0,1744417,0.2324854666],[0,1744460,0.1630575733],[0,1744463,0.1200283922],[0,1744486,0.178087648],[0,1744488,0.1365341439],[0,1744494,0.2731720915],[0,1744500,0.2560000053],[0,1744504,0.1873701574],[0,1744508,0.1623976089],[0,1744525,0.1807073154],[0,1744529,0.5907148032],[0,1744557,0.1199668756],[0,1744592,0.3230096367],[0,1744628,0.3385528506],[0,1744629,0.152162882],[0,1744649,0.1844496041],[0,1744650,0.1231342108],[0,1744701,0.16001108],[0,1744709,0.1332701694],[0,1744738,0.1786066388],[0,1744748,0.1273659551],[0,1744768,0.1604690376],[0,1744793,0.1444860674],[0,1744830,0.1462398538],[0,1744842,0.1251134863],[0,1744847,0.1365341439],[0,1744858,0.2720163527],[0,1744864,0.1194770786],[0,1744877,0.1832705718],[0,1744881,0.1457351011],[0,1744891,0.4176040508],[0,1744896,0.117693953],[0,1744901,0.3000523586],[0,1744925,0.1484637695],[0,1744943,0.1365341439],[0,1744975,0.1614421037],[0,1744997,0.1320667201],[0,1745011,0.1620043431],[0,1745059,0.2059467268],[0,1745092,0.3890612768],[0,1745114,0.1603187655],[0,1745117,0.1377903266],[0,1745136,0.1260600173],[0,1745149,0.4065386122],[0,1745152,0.4544891764],[0,1745179,0.1767852205],[0,1745180,0.164600161],[0,1745193,0.1199668756],[0,1745201,0.1492037689],[0,1745219,0.1794494961],[0,1745220,0.1516159965],[0,1745229,0.1422645332],[0,1745230,0.1365341439],[0,1745246,0.1172353298],[0,1745258,0.2113676221],[0,1745291,0.1752162792],[0,1745306,0.1195450434],[0,1745349,0.3642163971],[0,1745352,0.248055848],[0,1745353,0.1178151867],[0,1745368,0.2340831632],[0,1745378,0.1417270268],[0,1745401,0.1168920854],[0,1745405,0.3043203144],[0,1745409,0.1420916601],[0,1745460,0.1192063093],[0,1745464,0.1177502065],[0,1745514,0.6196507224],[0,1745528,0.1608802446],[0,1745546,0.1209694543],[0,1745549,0.2328407236],[0,1745558,0.1236220213],[0,1745560,0.3171466068],[0,1745566,0.1898641378],[0,1745568,0.165613768],[0,1745571,0.2234466665],[0,1745591,0.1159443898],[0,1745600,0.118723464],[0,1745619,0.1286706784],[0,1745631,0.1193316279],[0,1745639,0.1365341439],[0,1745678,0.3473983464],[0,1745712,0.119641192],[0,1745720,0.2406449034],[0,1745761,0.1365341439],[0,1745793,0.1620043431],[0,1745795,0.1541570014],[0,1745807,0.2453259557],[0,1745824,0.3858529464],[0,1745829,0.1265184341],[0,1745833,0.2556300344],[0,1745836,0.1218653976],[0,1745880,0.3426017093],[0,1745882,0.1178896871],[0,1745894,0.1182696507],[0,1745916,0.4221685034],[0,1745937,0.120939222],[0,1745942,0.1594154062],[0,1745975,0.1400123621],[0,1746024,0.418202259],[0,1746033,0.1275561897],[0,1746036,0.1397679145],[0,1746055,0.190083273],[0,1746061,0.1301176569],[0,1746108,0.1620867207],[0,1746133,0.175937781],[0,1746144,0.1272732124],[0,1746153,0.3054933303],[0,1746166,0.1606926112],[0,1746211,0.1194770786],[0,1746219,0.1540484562],[0,1746257,0.2009419007],[0,1746263,0.3235934611],[0,1746282,0.1600315424],[0,1746305,0.1579098564],[0,1746322,0.1218653976],[0,1746325,0.1190511348],[0,1746362,0.5963119295],[0,1746365,0.557663752],[0,1746390,0.1468743989],[0,1746391,0.1365341439],[0,1746398,0.1232678354],[0,1746424,0.1246333866],[0,1746438,0.5920278869],[0,1746457,0.116396976],[0,1746483,0.1698242511],[0,1746493,0.336916662],[0,1746501,0.715900375],[0,1746522,0.1886329558],[0,1746524,0.1219367633],[0,1746530,0.1768206798],[0,1746548,0.2994499987],[0,1746549,0.1624625107],[0,1746550,0.13157586],[0,1746557,0.1365341439],[0,1746558,0.1540484562],[0,1746577,0.3422904488],[0,1746578,0.1365341439],[0,1746597,0.1796235127],[0,1746638,0.1474137493],[0,1746639,0.1177099555],[0,1746686,0.1511032135],[0,1746711,0.1222182831],[0,1746719,0.5325517795],[0,1746790,0.1169322945],[0,1746807,0.1160152213],[0,1746865,0.1424796542],[0,1746887,0.2801777538],[0,1746904,0.1184316431],[0,1746951,0.1174327429],[0,1747001,0.1365341439],[0,1747018,0.1826433894],[0,1747021,0.1359776073],[0,1747023,0.1365341439],[0,1747033,0.6463333346],[0,1747093,0.2218500902],[0,1747095,0.1319368062],[0,1747103,0.1218653976],[0,1747123,0.1365341439],[0,1747143,0.1365341439],[0,1747146,0.3116594701],[0,1747257,0.1813666666],[0,1747266,0.1649181878],[0,1747272,0.182439283],[0,1747280,0.290508513],[0,1747285,0.4000930026],[0,1747289,0.152987757],[0,1747297,0.1601642585],[0,1747303,0.3629869648],[0,1747325,0.1365341439],[0,1747329,0.2330299166],[0,1747358,0.1369924014],[0,1747373,0.1261871605],[0,1747411,0.1546853959],[0,1747418,0.4572292531],[0,1747428,0.1197820956],[0,1747437,0.3046981863],[0,1747452,0.1326378118],[0,1747519,0.1608802446],[0,1747554,0.1833478132],[0,1747559,0.1252902743],[0,1747570,0.1277574671],[0,1747625,0.135006077],[0,1747661,0.1158135719],[0,1747707,0.1207237005],[0,1747714,0.1193278398],[0,1747845,0.3461454533],[0,1747847,0.1296668469],[0,1747882,0.2061169804],[0,1747913,0.1365341439],[0,1747914,0.1236197589],[0,1747951,0.1608802446],[0,1747954,0.1218653976],[0,1747969,0.2021818023],[0,1747976,0.1172120337],[0,1747987,0.1564042389],[0,1748020,0.1223213967],[0,1748023,0.1267928192],[0,1748043,0.7519030919],[0,1748054,0.1159443898],[0,1748062,0.1365341439],[0,1748066,0.6139574037],[0,1748076,0.2053984061],[0,1748122,0.1289145911],[0,1748163,0.1218653976],[0,1748176,0.4043958803],[0,1748214,0.1603187655],[0,1748291,0.1276494067],[0,1748313,0.2013221971],[0,1748341,0.757940143],[0,1748456,0.118471931],[0,1748511,0.1238253636],[0,1748547,0.1218653976],[0,1748582,0.1365341439],[0,1748601,0.1420172855],[0,1748646,0.3214486091],[0,1748653,0.1637443978],[0,1748676,0.1273139525],[0,1748691,0.2003135641],[0,1748695,0.1608802446],[0,1748705,0.5938310173],[0,1748709,0.3664765228],[0,1748711,0.1218653976],[0,1748750,0.1855146512],[0,1748758,0.2980942853],[0,1748778,0.1603187655],[0,1748821,0.8012841446],[0,1748823,0.2871837832],[0,1748835,0.1203161388],[0,1748855,0.1223213967],[0,1748878,0.7078161],[0,1748901,0.1198311244],[0,1748903,0.3446298648],[0,1748964,0.1272732124],[0,1748966,0.2943898044],[0,1748977,0.1365341439],[0,1748989,0.1456501927],[0,1748998,0.1452810258],[0,1749003,0.1620043431],[0,1749037,0.7307248264],[0,1749065,0.1614421037],[0,1749069,0.8754486987],[0,1749076,0.122865119],[0,1749083,0.6106716915],[0,1749110,0.130063454],[0,1749113,0.1521389843],[0,1749120,0.1850869055],[0,1749165,0.1169322945],[0,1749180,0.1261871605],[0,1749192,0.1407125811],[0,1749214,0.1289455895],[0,1749244,0.1800093711],[0,1749250,0.1280464392],[0,1749341,0.2491903825],[0,1749433,0.1611485025],[0,1749472,0.1464645052],[0,1749474,0.316135763],[0,1749485,0.1365341439],[0,1749542,0.1365341439],[0,1749551,0.1620043431],[0,1749568,0.1365341439],[0,1749655,0.1473333614],[0,1749664,0.6164048641],[0,1749673,0.1716418695],[0,1749703,0.2190737269],[0,1750203,0.1715534072],[0,1750330,0.1337304041],[0,1750504,0.1307464204],[0,1750595,0.2154675278],[0,1750599,0.3992180549],[0,1750625,0.1219367633],[0,1750631,0.3515749233],[0,1750635,0.4484744506],[0,1750660,0.1603187655],[0,1750669,0.1315349931],[0,1750693,0.5943830173],[0,1750730,0.118471931],[0,1750802,0.1194938387],[0,1750811,0.3508589512],[0,1750852,0.1910610685],[0,1750865,0.1603187655],[0,1750881,0.2036766936],[0,1751006,0.3305653797],[0,1751009,0.2057636106],[0,1751013,0.2483070628],[0,1751022,0.1345427709],[0,1751046,0.2357233627],[0,1751049,0.1603187655],[0,1751050,0.1571794585],[0,1751064,0.151164703],[0,1751073,0.72998345],[0,1751091,0.1652232003],[0,1751125,0.1286306895],[0,1751152,0.2503804804],[0,1751166,0.1168920854],[0,1751195,0.2758631864],[0,1751196,0.1267881629],[0,1751212,0.1317504525],[0,1751265,0.1347761312],[0,1751283,0.1676249468],[0,1751286,0.12676924],[0,1751298,0.1646355208],[0,1751300,0.220856891],[0,1751329,0.1215280026],[0,1751348,0.2723630259],[0,1751353,0.1830813396],[0,1751365,0.3294350551],[0,1751369,0.7651010722],[0,1751387,0.1839359817],[0,1751394,0.7752877743],[0,1751407,0.1199176463],[0,1751449,0.1619526036],[0,1751489,0.4500083017],[0,1751501,0.2146919048],[0,1751520,0.1721163212],[0,1751522,0.2450650825],[0,1751535,0.1322868279],[0,1751573,0.116396976],[0,1751586,0.1165985058],[0,1751610,0.3988733777],[0,1751614,0.1272092768],[0,1751632,0.165379263],[0,1751663,0.2114268547],[0,1751664,0.5600582944],[0,1751673,0.1416330033],[0,1751681,0.1539103747],[0,1751684,0.3861208421],[0,1751695,0.433197302],[0,1751698,0.1253507001],[0,1751713,0.1365341439],[0,1751722,0.3350035491],[0,1751727,0.1609064516],[0,1751733,0.1747272414],[0,1751746,0.1194770786],[0,1751801,0.1165985058],[0,1751823,0.2417110245],[0,1751845,0.1403154844],[0,1751854,0.1358763377],[0,1751881,0.1889600426],[0,1751883,0.2551170962],[0,1751886,0.1218653976],[0,1751896,0.3711764895],[0,1751898,0.3457265861],[0,1751899,0.2242686468],[0,1751907,0.1577510145],[0,1751916,0.1219367633],[0,1751935,0.3960448493],[0,1751979,0.1264643188],[0,1751998,0.2950619649],[0,1752003,0.119641192],[0,1752008,0.1384702139],[0,1752020,0.1184008779],[0,1752049,0.2568833669],[0,1752064,0.1186428142],[0,1752111,0.1398519809],[0,1752127,0.1238253636],[0,1752128,0.1839223724],[0,1752185,0.1355585121],[0,1752200,0.1591730469],[0,1752211,0.1178896871],[0,1752222,0.2582219794],[0,1752228,0.142389806],[0,1752229,0.3849704946],[0,1752249,0.3697422895],[0,1752268,0.424569952],[0,1752275,0.2866983278],[0,1752280,0.1952131041],[0,1752308,0.1603187655],[0,1752312,0.2826398757],[0,1752358,0.1502661943],[0,1752359,0.126724428],[0,1752361,0.4166949927],[0,1752366,0.1178896871],[0,1752374,0.163469814],[0,1752393,0.1448808532],[0,1752422,0.1757786762],[0,1752428,0.1455396145],[0,1752441,0.1663570522],[0,1752455,0.5586437884],[0,1752457,0.1196815418],[0,1752464,0.1780319334],[0,1752472,0.1218653976],[0,1752484,0.1588815322],[0,1752487,0.1620043431],[0,1752495,0.1417948065],[0,1752511,0.1172307294],[0,1752526,0.2891552872],[0,1752527,0.1345427709],[0,1752528,0.1646384509],[0,1752530,0.174713582],[0,1752555,0.4434834294],[0,1752556,0.2568405959],[0,1752557,0.1949919615],[0,1752574,0.1245164017],[0,1752579,0.8287479229],[0,1752592,0.2529534901],[0,1752595,0.299931296],[0,1752610,0.1304742547],[0,1752668,0.1365341439],[0,1752672,0.1591488411],[0,1752697,0.1452003724],[0,1752700,0.1909254044],[0,1752704,0.3531980043],[0,1752717,0.1173738085],[0,1752751,0.5222675404],[0,1752753,0.1177099555],[0,1752781,0.2085233835],[0,1752784,0.1218653976],[0,1752811,0.1668589554],[0,1752842,0.1477014034],[0,1752847,0.1979667491],[0,1752854,0.134143809],[0,1752861,0.1179895354],[0,1752874,0.2837925508],[0,1752881,0.2634841901],[0,1752900,0.1223213967],[0,1752913,0.1168920854],[0,1752931,0.1793372956],[0,1752957,0.126036652],[0,1752984,0.1223213967],[0,1753007,0.2969361083],[0,1753020,0.1172353298],[0,1753067,0.1639028225],[0,1753136,0.3980326428],[0,1753140,0.1886367897],[0,1753143,0.2493070766],[0,1753152,0.1195341809],[0,1753155,0.3167641013],[0,1753156,0.1626202183],[0,1753210,0.118471931],[0,1753252,0.136527966],[0,1753253,0.1783619652],[0,1753269,0.1179746915],[0,1753271,0.1614718709],[0,1753275,0.4294153462],[0,1753279,0.3066132192],[0,1753299,0.2059550085],[0,1753372,0.297034081],[0,1753417,0.1169569004],[0,1753430,0.143059784],[0,1753441,0.1165039916],[0,1753465,0.1177958516],[0,1753471,0.1175113027],[0,1753474,0.195359389],[0,1753475,0.1587998309],[0,1753492,0.1458947245],[0,1753496,0.124390582],[0,1753502,0.1291561111],[0,1753508,0.1205909045],[0,1753518,0.1228487311],[0,1753541,0.1323198265],[0,1753553,0.1210463771],[0,1753557,0.2783139222],[0,1753562,0.1351881881],[0,1753587,0.120377206],[0,1753612,0.1179746915],[0,1753615,0.176580298],[0,1753631,0.1396522769],[0,1753647,0.1416674554],[0,1753660,0.1198898529],[0,1753665,0.2309404991],[0,1753677,0.1294515448],[0,1753693,0.140339407],[0,1753697,0.3386733063],[0,1753717,0.1532337826],[0,1753747,0.2138871249],[0,1753759,0.3766386377],[0,1753840,0.5843310818],[0,1753841,0.1224292191],[0,1753855,0.1202620064],[0,1753856,0.1165039916],[0,1753858,0.1371084194],[0,1753884,0.1185505947],[0,1753886,0.2249464037],[0,1753918,0.1653246652],[0,1753936,0.1847424429],[0,1753940,0.1312175292],[0,1753943,0.1193138208],[0,1753956,0.1815975542],[0,1753973,0.1261009527],[0,1753980,0.2466259043],[0,1753997,0.242711702],[0,1754001,0.1205059504],[0,1754005,0.1228855431],[0,1754026,0.3136968475],[0,1754035,0.1575464526],[0,1754036,0.1235329244],[0,1754044,0.1986111113],[0,1754059,0.1224292191],[0,1754066,0.1914775353],[0,1754083,0.1210468756],[0,1754102,0.2298682311],[0,1754121,0.1831398504],[0,1754145,0.1250821125],[0,1754164,0.1241848307],[0,1754166,0.1228855431],[0,1754197,0.120184662],[0,1754199,0.2630567517],[0,1754209,0.124390582],[0,1754214,0.1237410076],[0,1754226,0.1422051103],[0,1754262,0.1165039916],[0,1754271,0.1252398149],[0,1754276,0.1224292191],[0,1754294,0.1625967705],[0,1754298,0.1927303891],[0,1754302,0.315943742],[0,1754316,0.1231477023],[0,1754337,0.2103933355],[0,1754347,0.1181441541],[0,1754361,0.2711009065],[0,1754389,0.1780291258],[0,1754391,0.4493278137],[0,1754399,0.4786378298],[0,1754409,0.2486739462],[0,1754419,0.1357730034],[0,1754435,0.1183769213],[0,1754439,0.1568922691],[0,1754461,0.1194238859],[0,1754475,0.2806524769],[0,1754481,0.1743035098],[0,1754501,0.1291273117],[0,1754502,0.1177893797],[0,1754507,0.1476641287],[0,1754530,0.1169569004],[0,1754540,0.1169569004],[0,1754549,0.1371084194],[0,1754569,0.1548723434],[0,1754576,0.1285747576],[0,1754630,0.1388494392],[0,1754636,0.2389813779],[0,1754651,0.1405891165],[0,1754652,0.1742645746],[0,1754656,0.1291317574],[0,1754688,0.4435056068],[0,1754694,0.1366838045],[0,1754695,0.1184506752],[0,1754702,0.1507992778],[0,1754706,0.1282602063],[0,1754707,0.222012107],[0,1754713,0.2100588079],[0,1754745,0.2426306003],[0,1754754,0.2402417261],[0,1754763,0.1431334654],[0,1754771,0.1371084194],[0,1754777,0.1208788562],[0,1754816,0.7200055951],[0,1754881,0.1980854417],[0,1754898,0.2459954585],[0,1754911,0.1266986234],[0,1754924,0.2086123567],[0,1754932,0.1535166948],[0,1754936,0.1937703739],[0,1754960,0.452995917],[0,1754961,0.1165039916],[0,1754974,0.3977587894],[0,1754998,0.2193706031],[0,1755008,0.6372629408],[0,1755012,0.1463105764],[0,1755019,0.3033815657],[0,1755023,0.1355792635],[0,1755131,0.1308660645],[0,1755153,0.1277865674],[0,1755178,0.2292054774],[0,1755213,0.1357730034],[0,1755241,0.1205697393],[0,1755287,0.1666073294],[0,1755296,0.1355792635],[0,1755324,0.1205059504],[0,1755327,0.1176404602],[0,1755353,0.1200963411],[0,1755356,0.1210059575],[0,1755364,0.1456290867],[0,1755380,0.1177912479],[0,1755413,0.1946729816],[0,1755417,0.1283254877],[0,1755420,0.3338056014],[0,1755445,0.1604870416],[0,1755459,0.1727500754],[0,1755486,0.1184506752],[0,1755489,0.1305533871],[0,1755512,0.1609099917],[0,1755536,0.1205909045],[0,1755546,0.4531271986],[0,1755555,0.1202250396],[0,1755558,0.1196524877],[0,1755569,0.1642986226],[0,1755585,0.1615275105],[0,1755628,0.4114364382],[0,1755631,0.14408298],[0,1755644,0.1218260269],[0,1755653,0.1227823561],[0,1755656,0.2817611122],[0,1755660,0.124390582],[0,1755688,0.1224292191],[0,1755698,0.4717036159],[0,1755729,0.124390582],[0,1755732,0.516164307],[0,1755771,0.1289138403],[0,1755795,0.1355792635],[0,1755808,0.7147356555],[0,1755817,0.124390582],[0,1755826,0.1264529305],[0,1755857,0.3220304661],[0,1755861,0.1649447353],[0,1755873,0.1616800588],[0,1755912,0.1182548016],[0,1755915,0.1262401029],[0,1755929,0.169963418],[0,1755940,0.3029789952],[0,1755952,0.124390582],[0,1755978,0.2580923842],[0,1755993,0.3525168643],[0,1756029,0.1211170256],[0,1756082,0.1363030263],[0,1756120,0.1489477064],[0,1756139,0.1390642887],[0,1756160,0.6228037305],[0,1756171,0.1196404553],[0,1756173,0.1694826495],[0,1756178,0.1206517872],[0,1756218,0.1298349084],[0,1756234,0.2041205298],[0,1756259,0.1995961334],[0,1756267,0.1216547747],[0,1756281,0.1233763561],[0,1756303,0.1403271269],[0,1756396,0.2578449945],[0,1756401,0.1371670778],[0,1756450,0.2331113902],[0,1756480,0.1275585254],[0,1756567,0.2153157448],[0,1756580,0.3001140049],[0,1756588,0.1252031125],[0,1756626,0.2432637587],[0,1756637,0.1210766024],[0,1756652,0.18059463],[0,1756661,0.3967620448],[0,1756670,0.1212590666],[0,1756683,0.1403271269],[0,1756698,0.2029676629],[0,1756707,0.1724202219],[0,1756717,0.1320998386],[0,1756739,0.1896261092],[0,1756751,0.122761824],[0,1756762,0.1642237044],[0,1756767,0.1272631927],[0,1756772,0.1214547721],[0,1756811,0.1613452833],[0,1756843,0.1255893337],[0,1756872,0.1432200116],[0,1756887,0.1491086578],[0,1756891,0.1411311176],[0,1756897,0.3885928312],[0,1756900,0.1469957017],[0,1756911,0.1301908982],[0,1756924,0.3086267345],[0,1757117,0.1196404553],[0,1757128,0.50315164],[0,1757241,0.2340791847],[0,1757243,0.170812605],[0,1757267,0.2043918976],[0,1757315,0.5206751782],[0,1757381,0.1255999899],[0,1757401,0.1451585785],[0,1757417,0.3020964035],[0,1757797,0.5215172191],[0,1757810,0.2157254355],[0,1757878,0.1573415382],[0,1757904,0.7809073581],[0,1758035,0.1271727799],[0,1758069,0.1247542979],[0,1758101,0.2092327396],[0,1758123,0.1665653014],[0,1758281,0.1439164974],[0,1758330,0.1896904344],[0,1758990,0.1675746549],[0,1759232,0.1324669735],[0,1759243,0.2890924322],[0,1759289,0.1409164474],[0,1759296,0.1225735746],[0,1759439,0.1230587791],[0,1759484,0.1816259258],[0,1759497,0.2426171834],[0,1759503,0.1642858071],[0,1759508,0.2331647223],[0,1759568,0.1243110301],[0,1759595,0.1323159406],[0,1759604,0.503832634],[0,1759616,0.1804298767],[0,1759631,0.6564920704],[0,1759636,0.2234053629],[0,1759642,0.5365716759],[0,1759643,0.1588377334],[0,1759654,0.12271391],[0,1759655,0.2025823051],[0,1759660,0.1313410926],[0,1759668,0.1286859642],[0,1759684,0.3273182495],[0,1759697,0.1208279563],[0,1759705,0.460636524],[0,1759707,0.1267139862],[0,1759724,0.6316730557],[0,1759737,0.1414726319],[0,1759812,0.2774944437],[0,1759815,0.2864825863],[0,1759864,0.3066448004],[0,1759867,0.1267857893],[0,1759908,0.7758405033],[0,1759944,0.1815345459],[0,1759962,0.1286859642],[0,1760047,0.3119679386],[0,1760164,0.1825526802],[0,1760171,0.3465574204],[0,1760186,0.1295008733],[0,1760211,0.5036906172],[0,1760213,0.1637435674],[0,1760271,0.1244082577],[0,1760304,0.1744971546],[0,1760316,0.1853644315],[0,1760333,0.4332516845],[0,1760401,0.6307817281],[0,1760430,0.1215644493],[0,1760443,0.6998608195],[0,1760470,0.3171000771],[0,1760485,0.1561369498],[0,1760510,0.1243896665],[0,1760525,0.2105324316],[0,1760549,0.4116129912],[0,1760578,0.2136304735],[0,1760647,0.5442936344],[0,1760669,0.2646011334],[0,1760671,0.120727977],[0,1760676,0.2947494258],[0,1760681,0.1207566907],[0,1760685,0.2597895003],[0,1760754,0.1846895469],[0,1760771,0.1286859642],[0,1760776,0.2000080247],[0,1760780,0.2781481994],[0,1760791,0.163596325],[0,1760807,0.1873095162],[0,1760812,0.1522435273],[0,1760850,0.1463577697],[0,1760856,0.1649650737],[0,1760860,0.1251552326],[0,1760862,0.6380182278],[0,1760903,0.3654725482],[0,1760939,0.1539710011],[0,1760946,0.1217694535],[0,1760960,0.1828182946],[0,1760961,0.1475254908],[0,1760968,0.1294989392],[0,1760982,0.1697263707],[0,1761023,0.1729529239],[0,1761032,0.1661654224],[0,1761042,0.1414726319],[0,1761046,0.3020931086],[0,1761059,0.1243684824],[0,1761092,0.1244572813],[0,1761095,0.6312047704],[0,1761119,0.2136496026],[0,1761128,0.2079298738],[0,1761136,0.1317333523],[0,1761142,0.1379768598],[0,1761147,0.1292486101],[0,1761174,0.168839497],[0,1761275,0.1220555424],[0,1761296,0.1654361035],[0,1761305,0.3293159841],[0,1761307,0.1380639032],[0,1761359,0.1333437666],[0,1761368,0.1562130307],[0,1761422,0.1220490355],[0,1761435,0.1940259614],[0,1761448,0.2345978066],[0,1761456,0.178204332],[0,1761457,0.12240573],[0,1761570,0.2533897674],[0,1761583,0.1211090816],[0,1761604,0.1418620793],[0,1761606,0.4094457535],[0,1761646,0.1952386911],[0,1761664,0.1535602298],[0,1761667,0.1827894916],[0,1761741,0.1244082577],[0,1761773,0.1809398386],[0,1761779,0.1286859642],[0,1761878,0.1779403161],[0,1761913,0.1637435674],[0,1761920,0.1637435674],[0,1761970,0.1511013228],[0,1761971,0.1517789043],[0,1762016,0.2871941112],[0,1762045,0.1211090816],[0,1762052,0.1688845106],[0,1762062,0.2006001488],[0,1762083,0.164871543],[0,1762090,0.175658808],[0,1762094,0.5018459546],[0,1762099,0.2362331703],[0,1762106,0.1800858932],[0,1762129,0.1221948698],[0,1762174,0.1739714147],[0,1762220,0.1256393248],[0,1762244,0.12271391],[0,1762245,0.1252830218],[0,1762249,0.1212120507],[0,1762268,0.3246429721],[0,1762288,0.3412596418],[0,1762297,0.233899967],[0,1762322,0.1456425139],[0,1762323,0.1313630087],[0,1762337,0.146604641],[0,1762375,0.1663142264],[0,1762398,0.1212120507],[0,1762404,0.4136140578],[0,1762407,0.1825880812],[0,1762444,0.1695252709],[0,1762503,0.2174628725],[0,1762523,0.1472569196],[0,1762574,0.1266103005],[0,1762594,0.1226397569],[0,1762600,0.159994883],[0,1762659,0.12714336],[0,1762664,0.186629046],[0,1762667,0.1399352],[0,1762677,0.1637435674],[0,1762681,0.1267139862],[0,1762717,0.1212120507],[0,1762756,0.2000596155],[0,1762784,0.1948199249],[0,1762791,0.1643073644],[0,1762796,0.1227738602],[0,1762911,0.2858109943],[0,1762912,0.4931072619],[0,1762972,0.1207566907],[0,1762995,0.3429509504],[0,1763035,0.3344683273],[0,1763037,0.344640896],[0,1763038,0.1664687523],[0,1763047,0.1220960157],[0,1763049,0.1660294043],[0,1763052,0.1637435674],[0,1763098,0.164871543],[0,1763157,0.1414726319],[0,1763161,0.1414726319],[0,1763181,0.1211090816],[0,1763213,0.1654361035],[0,1763231,0.130965568],[0,1763246,0.1246508674],[0,1763249,0.5828105083],[0,1763254,0.1643073644],[0,1763271,0.130965568],[0,1763302,0.1414726319],[0,1763333,0.1588203496],[0,1763367,0.2960594287],[0,1763385,0.1323986528],[0,1763397,0.1267139862],[0,1763422,0.164871543],[0,1763445,0.3811188506],[0,1763454,0.1664441871],[0,1763455,0.1294989392],[0,1763503,0.2015510821],[0,1763668,0.1495060484],[0,1763675,0.1667201691],[0,1763716,0.2740770209],[0,1763732,0.1866045213],[0,1763742,0.1230607745],[0,1763748,0.1225330769],[0,1763752,0.4174102355],[0,1763859,0.4177121272],[0,1763863,0.333359127],[0,1763866,0.3103211284],[0,1763872,0.2634250576],[0,1763881,0.2065098609],[0,1763902,0.1235995558],[0,1763913,0.1230182566],[0,1763915,0.1212120507],[0,1763921,0.1346088345],[0,1763942,0.1399352],[0,1764014,0.133853879],[0,1764015,0.1331523226],[0,1764024,0.1718807962],[0,1764052,0.4425355111],[0,1764064,0.1271727799],[0,1764091,0.1831140743],[0,1764102,0.1564985738],[0,1764114,0.1286859642],[0,1764124,0.1651140374],[0,1764127,0.1234255775],[0,1764166,0.1404910208],[0,1764170,0.1238419112],[0,1764187,0.1481148653],[0,1764192,0.1302206489],[0,1764216,0.1902785269],[0,1764240,0.162617259],[0,1764247,0.1918452658],[0,1764280,0.149513555],[0,1764308,0.4914628945],[0,1764409,0.4626215867],[0,1764411,0.3760874935],[0,1764462,0.4297086012],[0,1764522,0.1671530366],[0,1764536,0.3747802957],[0,1764539,0.6222978926],[0,1764581,0.2017728613],[0,1764590,0.2279189664],[0,1764614,0.1798444901],[0,1764666,0.5169029726],[0,1764700,0.1865783943],[0,1764711,0.1244978775],[0,1764719,0.1251552326],[0,1764730,0.1404060542],[0,1764755,0.1286859642],[0,1764766,0.137743392],[0,1764841,0.1307594743],[0,1764853,0.1503974797],[0,1764862,0.1220490355],[0,1764864,0.2668978597],[0,1764880,0.1284500854],[0,1764882,0.1810123763],[0,1764902,0.1821859986],[0,1764928,0.15568203],[0,1764959,0.1499492569],[0,1764968,0.1637435674],[0,1765005,0.1699888418],[0,1765028,0.4168422304],[0,1765084,0.1221948698],[0,1765085,0.2909326483],[0,1765094,0.494174767],[0,1765095,0.4059325662],[0,1765114,0.296738709],[0,1765128,0.2049015308],[0,1765142,0.2416099472],[0,1765163,0.1350387605],[0,1765166,0.2012301539],[0,1765177,0.1306459932],[0,1765195,0.263575217],[0,1765203,0.2333809682],[0,1765267,0.1951696822],[0,1765317,0.1677856252],[0,1765327,0.2582342177],[0,1765350,0.1672194757],[0,1765358,0.1271181925],[0,1765365,0.145476838],[0,1765376,0.1967664067],[0,1765377,0.1448556106],[0,1765378,0.133051083],[0,1765387,0.336454638],[0,1765527,0.140041873],[0,1765568,0.1278605489],[0,1765725,0.1280074543],[0,1765744,0.2285939529],[0,1765757,0.1466144729],[0,1765789,0.189588283],[0,1765839,0.1684018406],[0,1765846,0.1614043848],[0,1765864,0.168352158],[0,1765868,0.4949478855],[0,1765877,0.3270088603],[0,1765916,0.1350387605],[0,1765919,0.1421477739],[0,1765921,0.1340649392],[0,1765933,0.1331234577],[0,1765936,0.1518264546],[0,1765972,0.1307991629],[0,1765976,0.2099927197],[0,1765991,0.1285367911],[0,1766006,0.1470315285],[0,1766027,0.1404148273],[0,1766052,0.2379134945],[0,1766058,0.1677856252],[0,1766060,0.1307954299],[0,1766064,0.1576332673],[0,1766070,0.3238055823],[0,1766072,0.2076911082],[0,1766111,0.1487047621],[0,1766173,0.1677856252],[0,1766224,0.2171525282],[0,1766232,0.1285367911],[0,1766245,0.2225446855],[0,1766273,0.1423281183],[0,1766274,0.1643380117],[0,1766287,0.2459479651],[0,1766293,0.4385436791],[0,1766295,0.420851788],[0,1766303,0.143611876],[0,1766306,0.1406098397],[0,1766322,0.3012956206],[0,1766341,0.6452493231],[0,1766343,0.1672194757],[0,1766359,0.1285367911],[0,1766368,0.1448556106],[0,1766376,0.2775922184],[0,1766389,0.1608132117],[0,1766404,0.1285367911],[0,1766410,0.1289444174],[0,1766435,0.1599898929],[0,1766443,0.1270463595],[0,1766448,0.1516455602],[0,1766458,0.1307269978],[0,1766514,0.1778512703],[0,1766525,0.2114095733],[0,1766543,0.1294044967],[0,1766545,0.1917624913],[0,1766588,0.1294044967],[0,1766635,0.5243717926],[0,1766645,0.1421313497],[0,1766646,0.1350387605],[0,1766663,0.1318522693],[0,1766686,0.1285303063],[0,1766709,0.1502393938],[0,1766722,0.3644255852],[0,1766739,0.1285367911],[0,1766766,0.1295687794],[0,1766796,0.1380242595],[0,1766817,0.1684308495],[0,1766820,0.1646264686],[0,1766823,0.5945505528],[0,1766847,0.128555756],[0,1766877,0.1462600465],[0,1766882,0.1508690344],[0,1766919,0.189296743],[0,1766981,0.1428436729],[0,1767011,0.3373359257],[0,1767066,0.1428436729],[0,1767075,0.2193839778],[0,1767119,0.3337377563],[0,1767293,0.1862926972],[0,1767367,0.1433117635],[0,1767373,0.3251839963],[0,1767378,0.1290191608],[0,1767388,0.5940216216],[0,1767433,0.1660026534],[0,1767470,0.3788580356],[0,1767481,0.3709781695],[0,1767484,0.1448556106],[0,1767509,0.1311257182],[0,1767522,0.2144562148],[0,1767530,0.1588202741],[0,1767574,0.2089643479],[0,1767614,0.1305185513],[0,1767627,0.1701910033],[0,1767698,0.2246379998],[0,1767703,0.1428436729],[0,1767721,0.1291691119],[0,1767756,0.1677856252],[0,1767780,0.1304814889],[0,1767788,0.1672194757],[0,1767813,0.1387680722],[0,1767831,0.1433117635],[0,1767876,0.4855680582],[0,1767908,0.308085726],[0,1767937,0.1689190742],[0,1767944,0.3724967856],[0,1767953,0.4274934444],[0,1767955,0.1275053447],[0,1767956,0.177922617],[0,1767962,0.1358186539],[0,1767971,0.1476602143],[0,1768020,0.128555756],[0,1768023,0.1491549131],[0,1768185,0.138781007],[0,1768224,0.1314799196],[0,1768229,0.1582289946],[0,1768271,0.2299648934],[0,1768308,0.168352158],[0,1768427,0.2003362878],[0,1768439,0.1569007849],[0,1768452,0.1428436729],[0,1768587,0.1300244285],[0,1768641,0.204449787],[0,1768706,0.1394458955],[0,1768707,0.1439841494],[0,1768778,0.128048232],[0,1768811,0.5603414165],[0,1768834,0.1672194757],[0,1768859,0.1435073647],[0,1768903,0.1549181833],[0,1768960,0.1288442993],[0,1769016,0.1270463595],[0,1769019,0.1283963473],[0,1769228,0.1270463595],[0,1769241,0.1306459932],[0,1769306,0.133051083],[0,1769345,0.1672194757],[0,1769378,0.1335135293],[0,1769423,0.1617691193],[0,1769477,0.1429169309],[0,1769531,0.4533259563],[0,1769592,0.1376835089],[0,1769603,0.1433117635],[0,1769606,0.1306459932],[0,1769648,0.1448556106],[0,1769699,0.1270463595],[0,1769744,0.1672194757],[0,1769760,0.1781689331],[0,1769802,0.2191031224],[1,36,0.3132716086],[1,40,0.7956534421],[1,47,0.9198064719],[1,54,0.735175325],[1,58,0.7431354961],[1,63,0.254455472],[1,70,0.4883961519],[1,71,0.150526344],[1,72,0.735175325],[1,76,0.1574518149],[1,82,0.5511555852],[1,85,0.0808550181],[1,89,0.0616598684],[1,97,0.0674040337],[1,98,0.5098565116],[1,101,0.1430302228],[1,102,0.1621078929],[1,103,0.0547464739],[1,105,0.7931597933],[1,109,0.7272499222],[1,110,0.2416863096],[1,115,0.6952832016],[1,126,0.0897218834],[1,129,0.7334620555],[1,130,0.1871857284],[1,131,0.9275998546],[1,135,0.4090147594],[1,136,0.7420922014],[1,137,0.5768385707],[1,139,0.735175325],[1,141,0.7826309821],[1,150,0.7612832963],[1,151,0.0646288566],[1,162,0.2287454279],[1,173,0.8915495289],[1,176,0.9059728344],[1,177,0.8915495289],[1,178,0.8861008116],[1,181,0.4852880588],[1,182,0.1416676158],[1,183,0.8893678298],[1,184,0.8839264806],[1,185,0.8741783194],[1,188,0.8687882685],[1,190,0.0794631632],[1,191,0.9359824571],[1,193,0.8861008116],[1,197,0.9092802289],[1,198,0.9003059074],[1,199,0.9048718605],[1,200,0.8662485324],[1,202,0.8992087663],[1,205,0.8861008116],[1,209,0.7304080161],[1,210,0.9125943435],[1,213,0.870942101],[1,214,0.7783588959],[1,215,0.89701671],[1,216,0.549689822],[1,217,0.7956534421],[1,219,0.3278654295],[1,223,0.8992087663],[1,226,0.882840418],[1,229,0.9326209184],[1,230,0.0604490189],[1,239,0.6930413845],[1,240,0.8741783194],[1,244,0.9248038116],[1,245,0.8904583103],[1,246,0.0553938211],[1,248,0.8644893371],[1,250,0.8904583103],[1,252,0.8981123674],[1,256,0.8959217937],[1,259,0.7879409092],[1,263,0.0642041514],[1,264,0.934861186],[1,265,0.0855259555],[1,266,0.8981123674],[1,267,0.6013748959],[1,269,0.8644893371],[1,270,0.8493138924],[1,271,0.9103841865],[1,276,0.8861008116],[1,277,0.8450675942],[1,279,0.7832255876],[1,282,0.9114888912],[1,283,0.9048718605],[1,284,0.8992087663],[1,286,0.8621219571],[1,288,0.9371044871],[1,291,0.8839264806],[1,296,0.7372005137],[1,303,0.1104905035],[1,308,0.0801073857],[1,309,0.7362665807],[1,310,0.1194094081],[1,311,0.0915100185],[1,314,0.7793204528],[1,315,0.5592395082],[1,316,0.0839263521],[1,323,0.73725344],[1,330,0.787521806],[1,334,0.1602530373],[1,338,0.7411594909],[1,343,0.7675408334],[1,348,0.860367333],[1,351,0.7352275343],[1,353,0.6515623792],[1,355,0.1642937271],[1,361,0.8055100849],[1,369,0.7540441876],[1,371,0.4378795351],[1,374,0.2174340801],[1,378,0.1241714608],[1,383,0.2934569848],[1,391,0.0974050145],[1,397,0.7129425599],[1,403,0.7352275343],[1,405,0.7334952328],[1,406,0.057195369],[1,411,0.0802778427],[1,413,0.9070745534],[1,417,0.2269982378],[1,418,0.1704765703],[1,419,0.0627788004],[1,420,0.3621285985],[1,424,0.3642134211],[1,435,0.1014409782],[1,440,0.8170396158],[1,448,0.214544384],[1,596,0.5214928322],[1,810,0.1391885091],[1,817,0.5852919569],[1,850,0.1016522807],[1,853,0.8757168814],[1,856,0.2169087937],[1,860,0.1004548855],[1,874,0.614041283],[1,876,0.1110560058],[1,896,0.3264715532],[1,897,0.5772917654],[1,898,0.7138001633],[1,907,0.0519405631],[1,908,0.1045886319],[1,911,0.7474702245],[1,912,0.0850617732],[1,917,0.5131197617],[1,918,0.414516958],[1,928,0.7381880051],[1,932,0.8030231268],[1,934,0.0710075811],[1,939,0.7813531379],[1,940,0.7813531379],[1,941,0.9127107381],[1,942,0.6762976682],[1,943,0.9127107381],[1,944,0.7451191837],[1,948,0.7773442347],[1,949,0.73689266],[1,950,0.7793204528],[1,951,0.1734017265],[1,952,0.7081840127],[1,953,0.7537676554],[1,955,0.8760433912],[1,956,0.9127107381],[1,957,0.7612221327],[1,958,0.0697910409],[1,961,0.1085372436],[1,963,0.8609794968],[1,964,0.8609794968],[1,967,0.7627803658],[1,968,0.9049877051],[1,969,0.7803364515],[1,971,0.5746696068],[1,973,0.7813531379],[1,974,0.7753169705],[1,975,0.7783051412],[1,977,0.7352275343],[1,979,0.075234029],[1,981,0.7745149776],[1,986,0.1303163103],[1,988,0.5984028305],[1,989,0.1746160825],[1,993,0.6434749172],[1,1001,0.6065599528],[1,1003,0.0885876141],[1,1008,0.0902739875],[1,1010,0.2216390499],[1,1013,0.0638897201],[1,1019,0.2904674492],[1,1025,0.1631459659],[1,1031,0.4143779304],[1,1034,0.7261992879],[1,1038,0.8252709454],[1,1039,0.0671420777],[1,1040,0.0666784751],[1,1042,0.765526826],[1,1051,0.0721110505],[1,1052,0.2057685551],[1,1053,0.0555710639],[1,1055,0.4514300166],[1,1070,0.0985556142],[1,1079,0.154101958],[1,1080,0.1010148066],[1,1087,0.1073098836],[1,1088,0.0639238426],[1,1090,0.1756932703],[1,1091,0.1756932703],[1,1104,0.0614910174],[1,1117,0.1378647025],[1,1122,0.1489045623],[1,1123,0.0556321429],[1,1125,0.0651163662],[1,1126,0.1112291258],[1,1128,0.0822875876],[1,1139,0.0792292552],[1,1152,0.1129658763],[1,1153,0.2750011307],[1,1158,0.1787888997],[1,1170,0.0587602891],[1,1177,0.549773588],[1,1183,0.0556321429],[1,1185,0.0805384124],[1,1192,0.0659746251],[1,1218,0.0686465194],[1,1227,0.161790354],[1,1232,0.0616598684],[1,1241,0.3817319052],[1,1245,0.1634912912],[1,1261,0.069917669],[1,1263,0.0697444278],[1,1269,0.0826199059],[1,1279,0.3064283057],[1,1281,0.0588236783],[1,1283,0.0916501667],[1,1292,0.1319637524],[1,1297,0.0556740249],[1,1300,0.0533617814],[1,1305,0.0742167907],[1,1307,0.3429240501],[1,1308,0.0809573446],[1,1311,0.0929516532],[1,1329,0.5458931864],[1,1330,0.0627273592],[1,1332,0.0947614541],[1,1334,0.1619785776],[1,1336,0.5238545811],[1,1341,0.6198934028],[1,1343,0.3623636687],[1,1346,0.0825685063],[1,1352,0.1440427172],[1,1363,0.0591398894],[1,1365,0.4947433353],[1,1370,0.2316949619],[1,1376,0.3919104629],[1,1380,0.2395128193],[1,1381,0.0540096708],[1,1383,0.0601577429],[1,1385,0.0630972943],[1,1397,0.1466156854],[1,1404,0.0773751295],[1,1409,0.0542760063],[1,1418,0.1954768614],[1,1422,0.071455817],[1,1433,0.104775013],[1,1455,0.0708192419],[1,1458,0.0560800457],[1,1465,0.1663876765],[1,1473,0.0926238629],[1,1474,0.0727601229],[1,1478,0.1550248061],[1,1480,0.0758020272],[1,1483,0.2339608231],[1,1485,0.0560239623],[1,1486,0.0780627225],[1,1487,0.0530761741],[1,1490,0.1092457677],[1,1508,0.0556321429],[1,1516,0.2665996408],[1,1517,0.0574431012],[1,1518,0.0563609061],[1,1519,0.0587108407],[1,1520,0.0606849413],[1,1531,0.0806756891],[1,1534,0.7410935033],[1,1535,0.1147127331],[1,1536,0.0533560845],[1,1553,0.0560948953],[1,1567,0.0617875159],[1,1570,0.0722229759],[1,1583,0.0557937139],[1,1588,0.0561689411],[1,1589,0.0634198816],[1,1591,0.1978997494],[1,1599,0.0561649494],[1,1604,0.0569170849],[1,1606,0.7696656417],[1,1629,0.1253528554],[1,1639,0.2461033869],[1,1658,0.0756383989],[1,1674,0.2007320173],[1,1676,0.227067959],[1,1681,0.0668244833],[1,1682,0.0933485772],[1,1683,0.0751392569],[1,1685,0.0555302317],[1,1688,0.0620281173],[1,1709,0.3957241876],[1,1712,0.2188795918],[1,1715,0.0688926767],[1,1723,0.0555230585],[1,1724,0.0605213094],[1,1726,0.0543127935],[1,1730,0.1254681651],[1,1739,0.0524634374],[1,1742,0.0792835263],[1,1746,0.4025128252],[1,1748,0.1553211989],[1,1751,0.4576508136],[1,1761,0.2130764195],[1,1769,0.0593625668],[1,1772,0.0555230585],[1,1775,0.0562829381],[1,1779,0.0947751878],[1,1780,0.0541462117],[1,1786,0.0728793063],[1,1811,0.4769175075],[1,1817,0.1093490675],[1,1818,0.0775414505],[1,1822,0.0603569896],[1,1829,0.0534761143],[1,1831,0.1191906924],[1,1833,0.0817149258],[1,1837,0.0657047407],[1,1838,0.3936689658],[1,1840,0.2092262792],[1,1858,0.0575985772],[1,1865,0.1052572126],[1,1878,0.1459749299],[1,1899,0.8104706825],[1,1920,0.1317494515],[1,1923,0.0599020181],[1,1929,0.0546160755],[1,1937,0.0590544577],[1,1939,0.0653917054],[1,1950,0.0615499383],[1,1954,0.3601516524],[1,1962,0.1851418794],[1,1977,0.2331524315],[1,1988,0.6623026319],[1,1994,0.0878465124],[1,1996,0.0570075608],[1,2008,0.1187371821],[1,2013,0.0702223619],[1,2017,0.0535676073],[1,2019,0.3521510565],[1,2021,0.2598268093],[1,2024,0.1304014105],[1,2025,0.2860158292],[1,2030,0.1132688561],[1,2039,0.169609752],[1,2040,0.0718014673],[1,2041,0.7295331875],[1,2046,0.6125364028],[1,2055,0.5152829758],[1,2057,0.1416248339],[1,2061,0.4702443031],[1,2067,0.3382302719],[1,2070,0.0544847455],[1,2071,0.0740978958],[1,2076,0.2214058609],[1,2077,0.5018593349],[1,2082,0.2749420734],[1,2083,0.0719983716],[1,2092,0.1777500438],[1,2097,0.0555230585],[1,2122,0.3694444088],[1,2134,0.0806119567],[1,2140,0.069607614],[1,2156,0.0712438132],[1,2157,0.0599128054],[1,2158,0.0818486579],[1,2162,0.0534168692],[1,2163,0.0667327096],[1,2164,0.0914296133],[1,2166,0.0647931658],[1,2168,0.0617760185],[1,2185,0.0855216134],[1,2186,0.5540123316],[1,2187,0.0749272727],[1,2206,0.0696779636],[1,2210,0.0828880959],[1,2215,0.0735930696],[1,2224,0.0561294126],[1,2237,0.0798749578],[1,2252,0.0817674939],[1,2254,0.5698672067],[1,2274,0.1774184496],[1,2275,0.38786265],[1,2276,0.0797729183],[1,2277,0.0541240992],[1,2280,0.1822368501],[1,2310,0.0817216643],[1,2318,0.1049313683],[1,2319,0.0977975975],[1,2342,0.5166851679],[1,2350,0.0512960263],[1,2351,0.0939935488],[1,2352,0.5414159954],[1,2358,0.1199354602],[1,2361,0.0882851154],[1,2366,0.1130423531],[1,2370,0.06259841],[1,2373,0.0621636941],[1,2383,0.0639213975],[1,2385,0.1605408548],[1,2395,0.3311583368],[1,2403,0.0810784954],[1,2405,0.0464296388],[1,2427,0.0502238771],[1,2435,0.0648084472],[1,2447,0.4282732944],[1,2459,0.0738210146],[1,2460,0.1410457343],[1,2461,0.0490929422],[1,2464,0.0491259416],[1,2465,0.4741368162],[1,2473,0.7039783423],[1,2475,0.3242897223],[1,2481,0.0472084869],[1,2494,0.6979152927],[1,2508,0.0550431591],[1,2512,0.1117928266],[1,2513,0.8513140222],[1,2518,0.071812555],[1,2519,0.1180340928],[1,2533,0.1397129334],[1,2536,0.0511257295],[1,2538,0.0629815891],[1,2540,0.6602221567],[1,2541,0.1457453021],[1,2543,0.2620870739],[1,2560,0.2236813226],[1,2569,0.1088939752],[1,2585,0.0516846581],[1,2596,0.1660484022],[1,2615,0.347914809],[1,2628,0.0607346563],[1,2635,0.0621725088],[1,2638,0.0493973624],[1,2640,0.048584576],[1,2641,0.4212275689],[1,2643,0.0742192978],[1,2659,0.0727184181],[1,2665,0.0784941833],[1,2667,0.055319562],[1,2673,0.0521860584],[1,2679,0.0806473544],[1,2687,0.1380320732],[1,2694,0.2479260087],[1,2702,0.0777189857],[1,2709,0.3416107037],[1,2712,0.0518328315],[1,2719,0.0707808341],[1,2721,0.0927138315],[1,2723,0.1049097571],[1,2730,0.0625962701],[1,2731,0.09589418],[1,2737,0.1154656281],[1,2744,0.0731561829],[1,2758,0.1030329038],[1,2760,0.1694247555],[1,2763,0.1190468839],[1,2764,0.0750128835],[1,2766,0.0814979218],[1,2770,0.0652086314],[1,2775,0.0610722133],[1,2778,0.1049097571],[1,2782,0.0625962701],[1,2802,0.243986536],[1,2814,0.1043471856],[1,2821,0.056712785],[1,2828,0.3553332827],[1,2834,0.0456423543],[1,2845,0.0487535145],[1,2860,0.0447165143],[1,2872,0.2583156777],[1,2879,0.0726235348],[1,2885,0.0789512577],[1,2892,0.0952262286],[1,2893,0.057432386],[1,2902,0.2656033028],[1,2925,0.3587067638],[1,2940,0.046522877],[1,2955,0.0649854422],[1,2956,0.0571201958],[1,2964,0.0542935506],[1,2967,0.0509908829],[1,2968,0.0763771581],[1,2981,0.1019898416],[1,2982,0.0593162194],[1,2991,0.3395180105],[1,2998,0.2589956029],[1,3000,0.043190309],[1,3001,0.0834359904],[1,3006,0.046599778],[1,3008,0.046599778],[1,3021,0.0471650505],[1,3034,0.0457032063],[1,3035,0.0727110875],[1,3049,0.0677976034],[1,3055,0.0626581887],[1,3066,0.046522877],[1,3073,0.1360914321],[1,3079,0.1797342576],[1,3087,0.8646550937],[1,3101,0.0730285272],[1,3106,0.0523014589],[1,3124,0.4788698699],[1,3126,0.1230872387],[1,3128,0.0660614093],[1,3133,0.050036661],[1,3135,0.061005061],[1,3138,0.0461170148],[1,3140,0.2276141087],[1,3144,0.0576695083],[1,3153,0.0570158693],[1,3161,0.0727171921],[1,3163,0.0700821955],[1,3172,0.0974271412],[1,3185,0.0471554339],[1,3189,0.1869423442],[1,3247,0.1555592724],[1,3248,0.0481303533],[1,3255,0.1600921556],[1,3259,0.1762223125],[1,3265,0.1847753818],[1,3280,0.0752226736],[1,3298,0.07965534],[1,3309,0.1859787895],[1,3311,0.0598116602],[1,3314,0.0450364389],[1,3321,0.043483002],[1,3329,0.0681938385],[1,3333,0.0491540909],[1,3362,0.0539766495],[1,3365,0.0567761805],[1,3370,0.0452367463],[1,3373,0.0519168435],[1,3387,0.1834649374],[1,3388,0.0478276489],[1,3390,0.3049823914],[1,3394,0.0452955169],[1,3397,0.2625086737],[1,3401,0.0625877177],[1,3403,0.0540931348],[1,3408,0.0835351891],[1,3441,0.0460198757],[1,3444,0.072096209],[1,3459,0.0637244228],[1,3470,0.2617074726],[1,3488,0.0489109906],[1,3489,0.0451318424],[1,3493,0.5231912867],[1,3495,0.0542807873],[1,3496,0.0392681327],[1,3498,0.046599778],[1,3500,0.046599778],[1,3507,0.1757960617],[1,3526,0.0454034057],[1,3528,0.1240945378],[1,3532,0.0677617841],[1,3533,0.0418876239],[1,3537,0.1143640361],[1,3548,0.151463324],[1,3567,0.2995434768],[1,3572,0.0890250975],[1,3580,0.1068691322],[1,3594,0.3369114706],[1,3595,0.0700808659],[1,3610,0.1219506431],[1,3621,0.0475267841],[1,3623,0.0610475033],[1,3629,0.0587278435],[1,3643,0.1635339128],[1,3651,0.1014413669],[1,3656,0.5877369787],[1,3664,0.046599778],[1,3675,0.3381035841],[1,3701,0.0735098561],[1,3702,0.041113328],[1,3712,0.5187977119],[1,3718,0.0432577914],[1,3721,0.2559106123],[1,3725,0.062145051],[1,3726,0.0393957271],[1,3731,0.0759493332],[1,3738,0.5216744756],[1,3740,0.0405884338],[1,3756,0.0363073903],[1,3758,0.0517549857],[1,3765,0.1065478252],[1,3783,0.0482330407],[1,3786,0.0470444396],[1,3811,0.3945082531],[1,3812,0.0657944851],[1,3817,0.1813405767],[1,3822,0.0642512133],[1,3824,0.4031169759],[1,3830,0.1382011655],[1,3838,0.0672808424],[1,3857,0.1592073716],[1,3861,0.0496325593],[1,3863,0.1103048329],[1,3874,0.0487555105],[1,3881,0.0448318155],[1,3884,0.6560263713],[1,3888,0.140103812],[1,3891,0.0372777066],[1,3892,0.1285738817],[1,3914,0.4734522323],[1,3915,0.0413277876],[1,3923,0.1190074225],[1,3932,0.0960918423],[1,3961,0.7638632593],[1,3981,0.0895468949],[1,4004,0.0510350394],[1,4013,0.0439348894],[1,4014,0.3102771183],[1,4015,0.1551321597],[1,4022,0.5852128635],[1,4034,0.0557053061],[1,4041,0.0541444327],[1,4047,0.2657873653],[1,4057,0.5261107601],[1,4058,0.0718995586],[1,4059,0.0588117791],[1,4060,0.1486178476],[1,4066,0.1730493434],[1,4083,0.0441817795],[1,4088,0.0575184828],[1,4089,0.0458355361],[1,4101,0.0829576726],[1,4123,0.1922508537],[1,4128,0.1540810222],[1,4134,0.0441259372],[1,4135,0.0404447331],[1,4145,0.0539462649],[1,4151,0.2153203291],[1,4157,0.0808490637],[1,4167,0.2628197018],[1,4170,0.0569961193],[1,4171,0.0410940067],[1,4175,0.0589518938],[1,4176,0.2165400384],[1,4179,0.0407668059],[1,4182,0.0414644315],[1,4190,0.1647241936],[1,4191,0.0403763575],[1,4193,0.0496025869],[1,4196,0.56313228],[1,4201,0.0550318387],[1,4209,0.0681864516],[1,4210,0.063973819],[1,4212,0.6004960488],[1,4216,0.0512330613],[1,4227,0.1681441013],[1,4233,0.0407668059],[1,4236,0.0750214978],[1,4244,0.0418796313],[1,4245,0.0827308811],[1,4247,0.2332169257],[1,4250,0.1304186479],[1,4255,0.1606334241],[1,4258,0.0973728277],[1,4262,0.1008574878],[1,4273,0.3707449658],[1,4286,0.0466955658],[1,4289,0.0590363294],[1,4292,0.0486330803],[1,4294,0.0964770991],[1,4298,0.0755935576],[1,4306,0.0354157574],[1,4315,0.051052625],[1,4322,0.0418796313],[1,4328,0.0474031615],[1,4336,0.0970474881],[1,4342,0.0372308569],[1,4347,0.0666418084],[1,4350,0.0392442135],[1,4353,0.0505706149],[1,4368,0.0483246674],[1,4370,0.0407668059],[1,4374,0.0889053699],[1,4377,0.0407668059],[1,4379,0.0431812249],[1,4384,0.1698019367],[1,4386,0.0425217307],[1,4390,0.0407668059],[1,4395,0.0945364449],[1,4396,0.0457803273],[1,4404,0.0407668059],[1,4427,0.0487417986],[1,4430,0.7564974425],[1,4434,0.0600390607],[1,4438,0.0407668059],[1,4457,0.3956200699],[1,4463,0.040773843],[1,4470,0.1569174277],[1,4482,0.4556892902],[1,4484,0.0415122609],[1,4487,0.0862921373],[1,4502,0.1511953161],[1,4505,0.0504430527],[1,4513,0.0457060118],[1,4519,0.0410876273],[1,4530,0.0394724452],[1,4531,0.3446463955],[1,4540,0.0632384101],[1,4541,0.046153819],[1,4543,0.0684676728],[1,4551,0.2073456093],[1,4555,0.0552887572],[1,4564,0.0544477417],[1,4570,0.1338861519],[1,4574,0.1309627839],[1,4575,0.4504406024],[1,4587,0.0685559433],[1,4589,0.0649649496],[1,4595,0.0430561053],[1,4598,0.2261236816],[1,4604,0.034094178],[1,4608,0.072329043],[1,4609,0.0530489471],[1,4615,0.0556788478],[1,4617,0.0776840085],[1,4634,0.6158799112],[1,4641,0.0676429101],[1,4664,0.1490479041],[1,4676,0.0701934987],[1,4696,0.3862679227],[1,4700,0.1363289947],[1,4716,0.0658980836],[1,4720,0.0463940465],[1,4726,0.0494593236],[1,4729,0.0378513329],[1,4742,0.0504125157],[1,4746,0.0509507842],[1,4748,0.1740953091],[1,4749,0.0372442071],[1,4752,0.0456216153],[1,4765,0.0454265043],[1,4789,0.2068188066],[1,4797,0.0665220619],[1,4801,0.0390359289],[1,4808,0.0388026945],[1,4827,0.5132746748],[1,4831,0.1099196588],[1,4834,0.1091004477],[1,4837,0.1136878722],[1,4847,0.0390359289],[1,4851,0.0724371133],[1,4859,0.0458877656],[1,4861,0.0472810558],[1,4862,0.1009703633],[1,4872,0.1050039574],[1,4907,0.0415212605],[1,4922,0.1506887335],[1,4923,0.4507765603],[1,4924,0.2194529723],[1,4930,0.2605019469],[1,4943,0.0407732278],[1,4948,0.2681602941],[1,4950,0.0397796919],[1,4952,0.0514032847],[1,4961,0.2157642986],[1,4969,0.0376666408],[1,4980,0.0376327303],[1,4983,0.3229455035],[1,4985,0.0396763604],[1,4986,0.0394193843],[1,4989,0.0599846596],[1,4993,0.0495247622],[1,4999,0.0612711085],[1,5001,0.0355547489],[1,5005,0.0473223975],[1,5023,0.0741635862],[1,5039,0.3180312677],[1,5041,0.0983286247],[1,5050,0.0586851198],[1,5054,0.0559593826],[1,5058,0.0369744107],[1,5059,0.0424560814],[1,5080,0.0390359289],[1,5084,0.0366965821],[1,5088,0.0612004723],[1,5090,0.0369744107],[1,5091,0.0457223791],[1,5093,0.0643937419],[1,5094,0.0390359289],[1,5097,0.070891458],[1,5100,0.0389366067],[1,5134,0.0390359289],[1,5146,0.049378808],[1,5147,0.0530549996],[1,5149,0.1582348625],[1,5152,0.0330997063],[1,5160,0.2366884485],[1,5163,0.0731015177],[1,5174,0.035987889],[1,5181,0.0400796732],[1,5182,0.0518454669],[1,5188,0.3112572538],[1,5198,0.0508063088],[1,5213,0.0390359289],[1,5229,0.0496440913],[1,5230,0.0640583686],[1,5231,0.1074604316],[1,5238,0.0609045598],[1,5240,0.1510412303],[1,5245,0.0431519633],[1,5249,0.0527302294],[1,5250,0.0686752734],[1,5251,0.0554767432],[1,5259,0.0521017422],[1,5260,0.115229761],[1,5261,0.0343575552],[1,5276,0.0491802726],[1,5278,0.0459441838],[1,5282,0.0384486177],[1,5311,0.0518134498],[1,5321,0.0396214842],[1,5322,0.0736699946],[1,5337,0.2035043664],[1,5346,0.0392887041],[1,5347,0.0569884854],[1,5373,0.0477480569],[1,5380,0.0379866835],[1,5390,0.0611185216],[1,5392,0.035136423],[1,5394,0.0927410772],[1,5406,0.056893676],[1,5411,0.1168853629],[1,5417,0.03904295],[1,5430,0.0389610541],[1,5432,0.0396294226],[1,5433,0.195492695],[1,5438,0.0694537621],[1,5462,0.1526219478],[1,5479,0.0652446587],[1,5505,0.0395956373],[1,5509,0.0677564529],[1,5514,0.0581171637],[1,5517,0.0467103066],[1,5520,0.0775767899],[1,5521,0.1035918521],[1,5526,0.0390359289],[1,5533,0.0488608064],[1,5548,0.0349251244],[1,5551,0.04635598],[1,5568,0.0341105819],[1,5573,0.1136729694],[1,5588,0.0390359289],[1,5593,0.0329541847],[1,5620,0.0376666408],[1,5634,0.0344187171],[1,5635,0.0390359289],[1,5638,0.0408949678],[1,5650,0.0548075995],[1,5655,0.1348307572],[1,5656,0.1453050125],[1,5661,0.0696657043],[1,5663,0.1793308329],[1,5668,0.0551309625],[1,5674,0.0460300284],[1,5679,0.0604461385],[1,5683,0.0603562834],[1,5693,0.0376350217],[1,5718,0.0395861153],[1,5738,0.0707162605],[1,5744,0.0385714857],[1,5745,0.0670232696],[1,5770,0.0380879295],[1,5772,0.0435846871],[1,5784,0.0625098802],[1,5808,0.0420972994],[1,5816,0.038959785],[1,5828,0.1386659451],[1,5834,0.7817154523],[1,5848,0.0622961322],[1,5851,0.0462540036],[1,5858,0.1325862638],[1,5861,0.083977636],[1,5867,0.0580253857],[1,5868,0.2433309517],[1,5871,0.0346107208],[1,5885,0.0376964415],[1,5909,0.0359275491],[1,5918,0.2031185213],[1,5921,0.0530380003],[1,5928,0.0673272479],[1,5940,0.0429955603],[1,5953,0.071502096],[1,5956,0.0346902753],[1,5962,0.0504229794],[1,5964,0.0361456608],[1,5969,0.043125192],[1,5984,0.0677536952],[1,6002,0.0944405492],[1,6014,0.4406211091],[1,6024,0.0407708811],[1,6045,0.0757178739],[1,6049,0.0390359289],[1,6054,0.0775839432],[1,6055,0.0360412007],[1,6061,0.0490426131],[1,6067,0.0666794852],[1,6073,0.0564923479],[1,6113,0.0404215451],[1,6115,0.0355526347],[1,6116,0.051734541],[1,6119,0.0402962557],[1,6125,0.0485174197],[1,6137,0.0650539812],[1,6138,0.3390797626],[1,6151,0.0462032523],[1,6158,0.0887329039],[1,6162,0.0349233578],[1,6171,0.0534283839],[1,6183,0.0371365084],[1,6185,0.0430838458],[1,6186,0.3119455183],[1,6193,0.0734675538],[1,6197,0.093185288],[1,6207,0.1357001076],[1,6209,0.0365383226],[1,6218,0.0371365084],[1,6219,0.0657825834],[1,6226,0.0551776963],[1,6228,0.0376584763],[1,6240,0.0666249497],[1,6249,0.0445903531],[1,6257,0.0598368772],[1,6264,0.2576381918],[1,6267,0.7249880229],[1,6276,0.5625811531],[1,6328,0.0425069249],[1,6329,0.0371365084],[1,6333,0.2901373936],[1,6336,0.058687834],[1,6380,0.034882443],[1,6381,0.1079206176],[1,6384,0.0358951708],[1,6387,0.0427023261],[1,6397,0.0347290238],[1,6399,0.125005991],[1,6401,0.0501036202],[1,6415,0.05520989],[1,6419,0.0651394447],[1,6436,0.0371365084],[1,6455,0.0405481228],[1,6460,0.0442453397],[1,6461,0.1605586496],[1,6467,0.0393182128],[1,6475,0.0424207954],[1,6494,0.0371365084],[1,6505,0.1622913027],[1,6511,0.0390050924],[1,6537,0.075115436],[1,6538,0.0538130024],[1,6540,0.0317176248],[1,6542,0.0453507106],[1,6543,0.186712678],[1,6559,0.0370618205],[1,6561,0.0909001808],[1,6563,0.0376853213],[1,6584,0.057134453],[1,6593,0.0363611149],[1,6599,0.0411821666],[1,6600,0.0333561371],[1,6602,0.0422451869],[1,6603,0.0429321618],[1,6606,0.0362509781],[1,6610,0.1254567424],[1,6615,0.3117115121],[1,6621,0.0552154491],[1,6628,0.5496963741],[1,6632,0.0444764755],[1,6651,0.0439595819],[1,6677,0.0376948195],[1,6687,0.0625957742],[1,6688,0.0606963957],[1,6690,0.4044726336],[1,6694,0.1200030096],[1,6700,0.329241527],[1,6706,0.0682478083],[1,6741,0.032574605],[1,6743,0.0453616093],[1,6746,0.0392513519],[1,6747,0.0395394336],[1,6753,0.5484211862],[1,6755,0.0350801365],[1,6756,0.054448078],[1,6758,0.0371365084],[1,6799,0.107423919],[1,6806,0.0421811091],[1,6808,0.0552741741],[1,6817,0.0352764373],[1,6818,0.0691722399],[1,6832,0.6665740599],[1,6833,0.4019116557],[1,6836,0.0591572079],[1,6837,0.0336640188],[1,6853,0.760383742],[1,6856,0.4188411977],[1,6863,0.8280714042],[1,6864,0.7984524047],[1,6889,0.7085762516],[1,6897,1.0],[1,6898,0.7965672094],[1,6899,0.8319323531],[1,6901,0.9027878342],[1,6902,0.7780308008],[1,6904,0.8814580273],[1,6905,0.7800617397],[1,6906,0.7283879155],[1,6907,0.7902577596],[1,6921,0.1021828628],[1,6924,0.1316678966],[1,6925,0.5660746723],[1,6926,0.1851417321],[1,6928,0.0373757975],[1,6929,0.0659125302],[1,6930,0.0370605545],[1,6931,0.2101894098],[1,6933,0.042086262],[1,6943,0.4298594417],[1,6947,0.2058545164],[1,6952,0.0918229779],[1,6961,0.0643980617],[1,6970,0.0451010639],[1,6973,0.0427843236],[1,6974,0.6886095015],[1,6975,0.0462337583],[1,6979,0.0405481228],[1,6993,0.0667064077],[1,7002,0.0569287042],[1,7004,0.1259823808],[1,7013,0.0536126605],[1,7015,0.2925226582],[1,7017,0.0954028514],[1,7023,0.0917162434],[1,7027,0.0397273975],[1,7029,0.0633010654],[1,7031,0.1003244725],[1,7038,0.0324812019],[1,7043,0.0382546301],[1,7050,0.1087329793],[1,7051,0.0544441071],[1,7053,0.0360309075],[1,7054,0.0491136683],[1,7056,0.5547181257],[1,7062,0.0663215383],[1,7067,0.0592365463],[1,7071,0.0474912],[1,7075,0.060088069],[1,7082,0.9081770181],[1,7087,0.5719254333],[1,7090,0.0388731745],[1,7093,0.1015873189],[1,7106,0.054549349],[1,7112,0.2286404657],[1,7113,0.0515893284],[1,7116,0.0547662709],[1,7122,0.6646792694],[1,7125,0.6363154901],[1,7139,0.1115211207],[1,7141,0.0451391177],[1,7145,0.2686585172],[1,7148,0.1128121109],[1,7149,0.0439114821],[1,7158,0.0363747152],[1,7163,0.0467666916],[1,7171,0.0603983797],[1,7176,0.3631461683],[1,7181,0.0354964596],[1,7185,0.111613343],[1,7193,0.0470672088],[1,7194,0.3904451504],[1,7215,0.0490577032],[1,7217,0.0949178279],[1,7225,0.0761713309],[1,7226,0.0373504532],[1,7228,0.0910039716],[1,7247,0.1038986831],[1,7261,0.0473558602],[1,7269,0.5857951646],[1,7279,0.0647110551],[1,7284,0.1837818842],[1,7286,0.0638883379],[1,7289,0.6623278283],[1,7291,0.4628511595],[1,7300,0.066404856],[1,7302,0.311933612],[1,7303,0.5350834437],[1,7307,0.0438989467],[1,7309,0.0395689007],[1,7320,0.0957210008],[1,7321,0.0620570063],[1,7324,0.0378609706],[1,7333,0.1935249804],[1,7369,0.0348313329],[1,7372,0.0613435134],[1,7376,0.0481534073],[1,7377,0.0765291513],[1,7378,0.0834435691],[1,7388,0.0365383226],[1,7393,0.4528273147],[1,7395,0.8904583103],[1,7397,0.0465051639],[1,7401,0.1253577165],[1,7402,0.7767942453],[1,7403,0.0656720749],[1,7419,0.0376726886],[1,7422,0.0385301021],[1,7433,0.056728846],[1,7436,0.0371365084],[1,7441,0.0434371089],[1,7447,0.0487867435],[1,7454,0.1197724148],[1,7457,0.1543523599],[1,7465,0.0518259548],[1,7467,0.0471694709],[1,7475,0.0356805741],[1,7489,0.0681147941],[1,7491,0.1775137329],[1,7495,0.3304393712],[1,7519,0.0311194748],[1,7524,0.0399201961],[1,7529,0.0330681811],[1,7562,0.0712499181],[1,7572,0.0503289316],[1,7576,0.0371365084],[1,7579,0.0371365084],[1,7591,0.1844721771],[1,7595,0.5741710773],[1,7609,0.0575755743],[1,7622,0.0599391829],[1,7627,0.0382440361],[1,7630,0.0370618205],[1,7638,0.0454355893],[1,7649,0.0782882948],[1,7655,0.4284254315],[1,7673,0.0561873565],[1,7677,0.0477973874],[1,7684,0.0736041176],[1,7702,0.0914392437],[1,7703,0.0627555786],[1,7722,0.0873748735],[1,7723,0.1544167557],[1,7727,0.220571636],[1,7735,0.0558748282],[1,7746,0.0466856023],[1,7762,0.0908491528],[1,7785,0.0504769732],[1,7792,0.0341492561],[1,7805,0.044904376],[1,7828,0.0368160878],[1,7832,0.0572700667],[1,7854,0.2261559286],[1,7855,0.2168389625],[1,7861,0.0434371089],[1,7872,0.0483442038],[1,7886,0.1016772913],[1,7890,0.1323541728],[1,7893,0.2770390842],[1,7918,0.1188300003],[1,7922,0.0416530101],[1,7930,0.0364904649],[1,7934,0.03713125],[1,7935,0.0497392571],[1,7939,0.0444214069],[1,7947,0.0472741496],[1,7952,0.0364999482],[1,7954,0.2195088826],[1,7968,0.3481245547],[1,7973,0.0325405193],[1,7983,0.5576525452],[1,8013,0.0465821454],[1,8016,0.0359425154],[1,8017,0.0392320667],[1,8022,0.2600264241],[1,8025,0.052697616],[1,8026,0.064121921],[1,8053,0.0350753929],[1,8061,0.0525259305],[1,8064,0.167644762],[1,8074,0.0561739318],[1,8101,0.0349757226],[1,8143,0.2364187417],[1,8144,0.083440066],[1,8155,0.0359425154],[1,8156,0.036685622],[1,8160,0.622243728],[1,8168,0.1121623901],[1,8169,0.0897136284],[1,8174,0.0366585773],[1,8176,0.3028969889],[1,8189,0.0485946345],[1,8216,0.0305321568],[1,8248,0.1168052823],[1,8252,0.0400304394],[1,8259,0.0370196004],[1,8264,0.0673421407],[1,8268,0.1008392169],[1,8277,0.0911069798],[1,8284,0.0353760944],[1,8291,0.0362602253],[1,8301,0.177046076],[1,8326,0.0546697758],[1,8332,0.0396509504],[1,8333,0.0850481113],[1,8344,0.2366029864],[1,8354,0.3373621639],[1,8362,0.0479507848],[1,8363,0.0471307666],[1,8379,0.0675914312],[1,8381,0.0834372204],[1,8400,0.3270202108],[1,8409,0.0401753305],[1,8416,0.0474313632],[1,8421,0.0431258716],[1,8428,0.0449693869],[1,8430,0.1404425027],[1,8438,0.0359425154],[1,8447,0.0508433994],[1,8450,0.0447107397],[1,8453,0.0462363942],[1,8472,0.2654719337],[1,8474,0.0471985149],[1,8499,0.0857696112],[1,8513,0.1115368955],[1,8517,0.3666422169],[1,8520,0.03782997],[1,8522,0.0708083435],[1,8524,0.0500773197],[1,8541,0.4563715662],[1,8544,0.060062949],[1,8550,0.1444851152],[1,8567,0.0625681402],[1,8572,0.0349983702],[1,8573,0.037153357],[1,8576,0.0626376521],[1,8583,0.0581907606],[1,8588,0.1146276274],[1,8591,0.0616583487],[1,8592,0.0359425154],[1,8600,0.03755521],[1,8601,0.4354084116],[1,8604,0.1257215595],[1,8605,0.0447961818],[1,8608,0.3390468205],[1,8613,0.0490460736],[1,8643,0.0366832544],[1,8654,0.0299618901],[1,8663,0.051766669],[1,8672,0.043983643],[1,8682,0.0359425154],[1,8684,0.0389799224],[1,8698,0.7197311623],[1,8700,0.8305135309],[1,8709,0.0413419905],[1,8711,0.6532791116],[1,8716,0.0526076422],[1,8717,0.082477892],[1,8719,0.3822806849],[1,8732,0.06323246],[1,8738,0.0370196004],[1,8760,0.0575316508],[1,8781,0.0397144923],[1,8789,0.1878275416],[1,8797,0.055626346],[1,8802,0.0513908354],[1,8809,0.0347627355],[1,8811,0.0530542498],[1,8813,0.0454336748],[1,8824,0.0345787943],[1,8829,0.0539611522],[1,8840,0.0485477288],[1,8841,0.0737253056],[1,8843,0.0621037625],[1,8858,0.0594108167],[1,8860,0.2736724055],[1,8861,0.0526575867],[1,8865,0.1057420218],[1,8874,0.0525887761],[1,8882,0.0515385961],[1,8886,0.0387828618],[1,8893,0.0547895429],[1,8895,0.2293755458],[1,8897,0.119546738],[1,8914,0.0484205595],[1,8917,0.0697001841],[1,8918,0.0359425154],[1,8920,0.0877299116],[1,8928,0.0516449198],[1,8938,0.0443212605],[1,8953,0.7035793665],[1,8971,0.0632612303],[1,8974,0.4263757382],[1,8986,0.8158788687],[1,8990,0.0359425154],[1,9012,0.053806749],[1,9015,0.1909423463],[1,9021,0.045138703],[1,9029,0.3037707902],[1,9037,0.1160112862],[1,9038,0.034871876],[1,9045,0.0359425154],[1,9098,0.2811881796],[1,9108,0.0771508752],[1,9109,0.0364999482],[1,9130,0.064139026],[1,9147,0.364039172],[1,9156,0.046123054],[1,9169,0.0426560402],[1,9170,0.040535225],[1,9206,0.283548962],[1,9227,0.0353452706],[1,9230,0.0338301249],[1,9251,0.0297165191],[1,9259,0.3250234392],[1,9277,0.0751938773],[1,9280,0.0359425154],[1,9281,0.0364999482],[1,9294,0.0595316259],[1,9298,0.0410883153],[1,9299,0.0326887189],[1,9301,0.0323990196],[1,9310,0.1586535316],[1,9314,0.1093423571],[1,9316,0.0531875007],[1,9336,0.061106067],[1,9339,0.044172316],[1,9346,0.1387013477],[1,9357,0.0305321568],[1,9358,0.055882552],[1,9372,0.0374276888],[1,9396,0.1495794495],[1,9398,0.0502833643],[1,9409,0.0323990196],[1,9412,0.0604970529],[1,9416,0.0650680929],[1,9430,0.0730744807],[1,9431,0.0359815421],[1,9432,0.0467230858],[1,9434,0.0381834296],[1,9438,0.0814876202],[1,9439,0.0289847499],[1,9450,0.0444214069],[1,9451,0.0511906568],[1,9453,0.0365335961],[1,9454,0.0488418913],[1,9456,0.0516318434],[1,9459,0.0370483008],[1,9461,0.0626157338],[1,9470,0.0336968269],[1,9481,0.0304322279],[1,9493,0.0732325304],[1,9501,0.0359425154],[1,9508,0.0600344389],[1,9516,0.5135271592],[1,9517,0.0374518633],[1,9527,0.0355315231],[1,9545,0.2263576728],[1,9547,0.0672082635],[1,9549,0.0414417897],[1,9556,0.0522363704],[1,9582,0.0700056823],[1,9591,0.0467790864],[1,9595,0.7030986048],[1,9607,0.2886119465],[1,9626,0.3729038665],[1,9635,0.0357102292],[1,9642,0.0370483008],[1,9645,0.0338893786],[1,9655,0.1014134353],[1,9669,0.0522943363],[1,9688,0.0581463077],[1,9701,0.0501131781],[1,9703,0.0285812829],[1,9713,0.1043799334],[1,9729,0.4984262199],[1,9732,0.0695811552],[1,9735,0.0846841732],[1,9747,0.1844929043],[1,9749,0.0623317642],[1,9753,0.0433438508],[1,9757,0.245094035],[1,9760,0.2919697121],[1,9775,0.3305259681],[1,9819,0.0368158833],[1,9822,0.4658649547],[1,9830,0.2791275309],[1,9832,0.0654978545],[1,9843,0.2672658116],[1,9847,0.0546268667],[1,9854,0.0500043784],[1,9876,0.0425557234],[1,9892,0.0680053729],[1,9903,0.0600275206],[1,9909,0.0359425154],[1,9929,0.0503729772],[1,9936,0.0529828105],[1,9942,0.4631580556],[1,9943,0.1039726414],[1,9951,0.0471441074],[1,9952,0.0359425154],[1,9959,0.0459831935],[1,9964,0.0384279823],[1,10004,0.0797209378],[1,10021,0.1035077546],[1,10030,0.0307818797],[1,10035,0.0366832544],[1,10053,0.0589488374],[1,10059,0.3676549142],[1,10060,0.065180401],[1,10076,0.3143255385],[1,10079,0.0701983436],[1,10081,0.03713125],[1,10098,0.4658277825],[1,10108,0.1447977158],[1,10112,0.0626481409],[1,10122,0.078454478],[1,10156,0.0359425154],[1,10159,0.0666225707],[1,10167,0.035866681],[1,10173,0.0918613694],[1,10184,0.0875804992],[1,10192,0.0348299705],[1,10201,0.0455253074],[1,10210,0.0366832544],[1,10211,0.0467984538],[1,10214,0.0588769783],[1,10216,0.1206277614],[1,10219,0.2753574352],[1,10234,0.1883230405],[1,10235,0.0634736821],[1,10241,0.0370483008],[1,10275,0.2488357652],[1,10277,0.3562862599],[1,10283,0.1937456555],[1,10285,0.3302198039],[1,10294,0.0375331986],[1,10316,0.2710998919],[1,10327,0.1202241428],[1,10336,0.0639187399],[1,10337,0.0710373816],[1,10359,0.3215768266],[1,10365,0.0392320667],[1,10367,0.0305321568],[1,10368,0.0378461607],[1,10373,0.3297136606],[1,10388,0.1222607956],[1,10394,0.1688712266],[1,10396,0.0444214069],[1,10428,0.0558782673],[1,10430,0.3820394027],[1,10436,0.5463796174],[1,10438,0.047013977],[1,10443,0.323643853],[1,10454,0.0448798828],[1,10457,0.1978819601],[1,10474,0.0537360799],[1,10475,0.2065982697],[1,10513,0.0379137789],[1,10523,0.1387132023],[1,10526,0.0671531677],[1,10528,0.0360442877],[1,10537,0.4012322808],[1,10539,0.0615829248],[1,10555,0.0496590718],[1,10557,0.0472016139],[1,10560,0.0626157338],[1,10580,0.0353400076],[1,10607,0.1908922149],[1,10609,0.0470905998],[1,10616,0.0356225989],[1,10624,0.1992378468],[1,10638,0.0338015015],[1,10656,0.3957089656],[1,10672,0.0851735673],[1,10684,0.0335498544],[1,10690,0.1719576516],[1,10692,0.4250874253],[1,10699,0.0528581481],[1,10722,0.046687807],[1,10727,0.0530024163],[1,10734,0.0852562276],[1,10744,0.0395953054],[1,10757,0.2461463427],[1,10770,0.0928580395],[1,10781,0.1906305729],[1,10793,0.0359425154],[1,10806,0.0359425154],[1,10810,0.0473047866],[1,10815,0.0625275459],[1,10822,0.0432951587],[1,10823,0.3479114663],[1,10828,0.0704294675],[1,10839,0.092126102],[1,10844,0.0407059805],[1,10879,0.0359425154],[1,10885,0.0353919883],[1,10886,0.0460000463],[1,10888,0.032141088],[1,10895,0.2293044797],[1,10908,0.3043575248],[1,10914,0.0525541349],[1,10919,0.0356465273],[1,10929,0.0480034136],[1,10942,0.0605433068],[1,10946,0.4824411434],[1,10951,0.0542264752],[1,10952,0.06612775],[1,10971,0.0619066213],[1,10972,0.0364904649],[1,10981,0.0313921352],[1,10996,0.0701423295],[1,11000,0.0485875829],[1,11024,0.0479089389],[1,11025,0.0622950238],[1,11026,0.1990000519],[1,11048,0.0483366809],[1,11053,0.0455114254],[1,11059,0.0647912893],[1,11080,0.0421508098],[1,11085,0.0349757226],[1,11086,0.0494222127],[1,11092,0.0756548181],[1,11094,0.0671531677],[1,11104,0.0388162956],[1,11105,0.2576731658],[1,11116,0.035866681],[1,11123,0.0480703068],[1,11126,0.0567620759],[1,11133,0.0359425154],[1,11136,0.4033677008],[1,11140,0.1187088384],[1,11155,0.0634736821],[1,11164,0.0423855043],[1,11171,0.034656343],[1,11174,0.0743279684],[1,11185,0.5530953411],[1,11187,0.0350583782],[1,11192,0.1105018614],[1,11206,0.2987735343],[1,11207,0.0497121662],[1,11208,0.0477626703],[1,11211,0.0699828467],[1,11221,0.0354296975],[1,11239,0.0496431354],[1,11243,0.0601010743],[1,11244,0.1171432299],[1,11245,0.2542908107],[1,11246,0.2180923904],[1,11250,0.0496835023],[1,11262,0.2296999133],[1,11267,0.0440816528],[1,11300,0.0389914393],[1,11301,0.0672082635],[1,11306,0.0359425154],[1,11321,0.2961155767],[1,11324,0.0448196373],[1,11327,0.0604143688],[1,11330,0.0577393834],[1,11359,0.0680629924],[1,11377,0.0410724109],[1,11391,0.0399253007],[1,11405,0.3290226686],[1,11415,0.0328724299],[1,11428,0.0398634119],[1,11449,0.2619078653],[1,11454,0.2981006716],[1,11472,0.0702388054],[1,11486,0.5662686344],[1,11516,0.0377738109],[1,11525,0.0374276888],[1,11537,0.1118216948],[1,11543,0.1327289886],[1,11545,0.1104111107],[1,11580,0.2003119626],[1,11585,0.0699944964],[1,11590,0.0350583782],[1,11592,0.0321084568],[1,11599,0.0476691813],[1,11606,0.0359425154],[1,11630,0.0496377513],[1,11641,0.042968853],[1,11650,0.0805737998],[1,11678,0.0479089389],[1,11686,0.0703360773],[1,11702,0.0458994973],[1,11704,0.0503637665],[1,11718,0.1126530182],[1,11735,0.0508817332],[1,11747,0.2956864111],[1,11763,0.0969032436],[1,11779,0.0328700187],[1,11790,0.0503657652],[1,11794,0.0625384046],[1,11798,0.2304643203],[1,11807,0.0517613661],[1,11810,0.0359425154],[1,11811,0.1066512382],[1,11820,0.0359425154],[1,11854,0.0695107447],[1,11868,0.1079040344],[1,11873,0.051314655],[1,11888,0.0634736821],[1,11891,0.0624670871],[1,11897,0.3492194096],[1,11908,0.0496700073],[1,11915,0.035875934],[1,11939,0.0404239024],[1,11940,0.0381834296],[1,11942,0.3668622783],[1,11949,0.0728791165],[1,11960,0.2385700053],[1,11964,0.0384177424],[1,12040,0.0633667313],[1,12049,0.1278954436],[1,12059,0.0338893786],[1,12063,0.0465159566],[1,12064,0.0634736821],[1,12071,0.0650072523],[1,12080,0.067158132],[1,12084,0.4817390885],[1,12089,0.0480711102],[1,12092,0.0351210371],[1,12094,0.0394952621],[1,12097,0.0442548546],[1,12105,0.0362602253],[1,12124,0.0460408075],[1,12151,0.3779121037],[1,12152,0.0291763532],[1,12161,0.0426575134],[1,12163,0.0925748964],[1,12165,0.0360726026],[1,12175,0.0472141191],[1,12181,0.1620723103],[1,12194,0.0363611149],[1,12242,0.0339177498],[1,12251,0.031295714],[1,12257,0.0357102292],[1,12261,0.2601209437],[1,12271,0.0571055384],[1,12280,0.0446040683],[1,12289,0.135925378],[1,12297,0.0601908067],[1,12298,0.048275302],[1,12304,0.096255481],[1,12306,0.3561089843],[1,12322,0.0479282272],[1,12325,0.0364778521],[1,12365,0.0359425154],[1,12372,0.0421632652],[1,12373,0.0651795163],[1,12383,0.0727732004],[1,12390,0.0355557561],[1,12393,0.0338015015],[1,12413,0.0634736821],[1,12418,0.0364904649],[1,12423,0.3473075503],[1,12435,0.0447107398],[1,12440,0.035866681],[1,12447,0.0417290511],[1,12466,0.03400904],[1,12469,0.0504921296],[1,12471,0.2310577852],[1,12509,0.044195199],[1,12524,0.0810339399],[1,12563,0.0345787943],[1,12570,0.0476110142],[1,12585,0.043833371],[1,12597,0.0384177424],[1,12605,0.1788330598],[1,12608,0.0336386927],[1,12612,0.0381834296],[1,12622,0.045540895],[1,12625,0.0891355962],[1,12630,0.0327082279],[1,12634,0.0790816955],[1,12639,0.0554046633],[1,12651,0.046097775],[1,12655,0.0364904649],[1,12662,0.1078526824],[1,12663,0.0359425154],[1,12675,0.0439419481],[1,12679,0.200195312],[1,12694,0.0611681337],[1,12697,0.6832812196],[1,12721,0.0361942628],[1,12739,0.3355827854],[1,12765,0.0906825866],[1,12767,0.0413089343],[1,12770,0.0661139572],[1,12774,0.0359425154],[1,12789,0.089218969],[1,12800,0.2752975301],[1,12803,0.8276286502],[1,12811,0.8007776577],[1,12812,0.9178867888],[1,12813,0.9178867888],[1,12814,0.9178867888],[1,12815,0.8516868784],[1,12816,0.9090355076],[1,12817,0.9178867888],[1,12818,0.9178867888],[1,12820,0.9178867888],[1,12821,0.906830163],[1,12822,0.9294379657],[1,12823,0.9178867888],[1,12824,0.9178867888],[1,12832,0.9178867888],[1,12848,0.9178867888],[1,12860,0.0900749853],[1,12861,0.0900749853],[1,12862,0.8698296147],[1,12867,0.1504196855],[1,12881,0.0666939285],[1,12889,0.0591645768],[1,12891,0.4277578306],[1,12898,0.0427643984],[1,12910,0.0731294911],[1,12933,0.0354417916],[1,12936,0.0381729249],[1,12941,0.0571828228],[1,12943,0.0408872228],[1,12951,0.8657288483],[1,12966,0.2189878231],[1,12967,0.3329478631],[1,12988,0.0334518669],[1,12991,0.291485],[1,12992,0.0608279902],[1,12993,0.0455713416],[1,12994,0.0345450217],[1,12999,0.1320692488],[1,13007,0.5247116289],[1,13009,0.0353919883],[1,13015,0.0491943418],[1,13017,0.039959497],[1,13029,0.0353355402],[1,13035,0.3868064174],[1,13036,0.0359425154],[1,13044,0.5631276171],[1,13048,0.0364778521],[1,13060,0.090185092],[1,13071,0.0370483008],[1,13082,0.0399243025],[1,13083,0.0562841312],[1,13095,0.0951388944],[1,13101,0.0510957302],[1,13118,0.0359425154],[1,13124,0.1000472092],[1,13127,0.037290892],[1,13133,0.1171386097],[1,13143,0.0452849448],[1,13147,0.463581941],[1,13151,0.0794870867],[1,13157,0.0399922979],[1,13167,0.1660614199],[1,13174,0.479166528],[1,13180,0.0337291519],[1,13185,0.0706228561],[1,13211,0.0526890351],[1,13213,0.0494851648],[1,13214,0.0363880386],[1,13218,0.6725838637],[1,13224,0.035867945],[1,13227,0.0541679218],[1,13230,0.0359425154],[1,13234,0.0345179119],[1,13240,0.1307711484],[1,13247,0.0313921352],[1,13856,0.0364778521],[1,14510,0.2360630869],[1,15169,0.0525632078],[1,15205,0.0312945326],[1,15246,0.1074335952],[1,15303,0.0340671748],[1,15304,0.0452849448],[1,15363,0.0364904649],[1,15366,0.2714456709],[1,15371,0.265182508],[1,15407,0.0762052819],[1,15433,0.2545846227],[1,15436,0.043833371],[1,15463,0.0508433994],[1,15469,0.3033685726],[1,15486,0.0350583782],[1,15494,0.0550348473],[1,15503,0.1818378666],[1,15508,0.1336564246],[1,15524,0.051552154],[1,15555,0.0937800366],[1,15560,0.0556470342],[1,15562,0.0760118864],[1,15569,0.7072850078],[1,15572,0.029557898],[1,15580,0.0300809677],[1,15624,0.0338301249],[1,15676,0.4584498959],[1,15690,0.0817640504],[1,15693,0.2569873083],[1,15694,0.0694678685],[1,15731,0.8326155361],[1,15751,0.0994034857],[1,15759,0.0456049714],[1,15799,0.0612669452],[1,15805,0.0370483008],[1,15822,0.0725159317],[1,15824,0.7604505997],[1,15825,0.0477138185],[1,15848,0.0430407607],[1,15872,0.0328542224],[1,15931,0.0914138308],[1,15940,0.0801045803],[1,15964,0.0346974048],[1,15983,0.0854582518],[1,16017,0.7682191398],[1,16031,0.0655569045],[1,16049,0.0504410947],[1,16054,0.0466361091],[1,16097,0.0481574309],[1,16098,0.0443473507],[1,16100,0.0395022155],[1,16106,0.0331729696],[1,16109,0.0309386163],[1,16119,0.0307890535],[1,16150,0.0258386415],[1,16171,0.1855429322],[1,16172,0.059301048],[1,16177,0.1402936481],[1,16204,0.0581306166],[1,16215,0.6014752093],[1,16229,0.0599706152],[1,16256,0.1215140902],[1,16263,0.0448940569],[1,16273,0.5219427693],[1,16275,0.0629660488],[1,16312,0.0309016959],[1,16325,0.0426600015],[1,16328,0.053084779],[1,16362,0.0403101008],[1,16415,0.0345611237],[1,16419,0.0501910597],[1,16431,0.0487568128],[1,16476,0.0553732085],[1,16490,0.0337189165],[1,16492,0.0311273368],[1,16504,0.0379252828],[1,16516,0.0520212185],[1,16517,0.1124680495],[1,16539,0.0626267392],[1,16550,0.0716052393],[1,16560,0.0424386061],[1,16566,0.0627838521],[1,16572,0.0337283651],[1,16579,0.0379190249],[1,16610,0.0563676726],[1,16622,0.0610772469],[1,16695,0.2829332309],[1,16696,0.1614193773],[1,16703,0.0416208719],[1,16715,0.4680838235],[1,16723,0.1350743407],[1,16724,0.1238248341],[1,16736,0.0331729696],[1,16772,0.0331729696],[1,16780,0.0587180798],[1,16791,0.0334314418],[1,16821,0.3722706216],[1,16847,0.1727834787],[1,16851,0.0398805861],[1,16860,0.0530426761],[1,16890,0.0719598347],[1,16898,0.0331729696],[1,16901,0.035593552],[1,16912,0.1242256643],[1,16913,0.0614702429],[1,16936,0.0331729696],[1,16950,0.0317144842],[1,16954,0.3452466159],[1,16960,0.043921331],[1,16962,0.0419607501],[1,17004,0.0554847441],[1,17041,0.4272087711],[1,17074,0.0360737794],[1,17079,0.0534332176],[1,17088,0.0345269154],[1,17090,0.0657737043],[1,17122,0.0392119071],[1,17136,0.0338636483],[1,17148,0.0645598765],[1,17183,0.0331199005],[1,17193,0.0363103497],[1,17200,0.0436823446],[1,17203,0.0540752628],[1,17241,0.0396633526],[1,17269,0.0512352383],[1,17281,0.0328684365],[1,17296,0.0286392205],[1,17348,0.0368086874],[1,17349,0.0931474788],[1,17354,0.0455054589],[1,17366,0.5626136893],[1,17371,0.0600455031],[1,17381,0.0320548745],[1,17399,0.0484132284],[1,17424,0.3086200665],[1,17435,0.1334967573],[1,17441,0.0564443745],[1,17494,0.0561136647],[1,17498,0.0524871836],[1,17499,0.055423173],[1,17523,0.0583380366],[1,17565,0.1506333152],[1,17570,0.1162557064],[1,17582,0.0414133429],[1,17599,0.2511371146],[1,17627,0.1970512136],[1,17639,0.0529888452],[1,17642,0.0326512678],[1,17662,0.0269697286],[1,17684,0.0676685562],[1,17700,0.0345472627],[1,17710,0.3661056697],[1,17711,0.133637876],[1,17719,0.0456777557],[1,17739,0.0636686948],[1,17747,0.0347242214],[1,17751,0.0333289745],[1,17804,0.0374311395],[1,17807,0.0357875481],[1,17829,0.0455126604],[1,17857,0.2034777892],[1,17864,0.0380084839],[1,17868,0.0333626928],[1,17877,0.0493471943],[1,17878,0.0467302655],[1,17930,0.1196744749],[1,17963,0.0335201416],[1,17970,0.0614496761],[1,18007,0.0422784766],[1,18013,0.0515034469],[1,18030,0.0384000607],[1,18041,0.0570218814],[1,18083,0.0810102613],[1,18089,0.088172739],[1,18096,0.0519878292],[1,18100,0.0612162867],[1,18107,0.5169708928],[1,18116,0.1317217434],[1,18118,0.2335268231],[1,18121,0.0343826253],[1,18142,0.0317978974],[1,18149,0.0848912002],[1,18162,0.534775234],[1,18169,0.0304307075],[1,18186,0.0605118791],[1,18192,0.0329548924],[1,18195,0.0633546572],[1,18202,0.0347974569],[1,18212,0.0607568761],[1,18215,0.1168580333],[1,18221,0.1204490235],[1,18222,0.0607568761],[1,18223,0.0329712156],[1,18227,0.0669315498],[1,18249,0.0643955173],[1,18272,0.0841241046],[1,18280,0.1847694295],[1,18283,0.3694255196],[1,18287,0.0388759928],[1,18298,0.0760130894],[1,18310,0.0585231156],[1,18320,0.0390821713],[1,18325,0.2479050186],[1,18335,0.1360881909],[1,18336,0.3614333628],[1,18342,0.0318351549],[1,18343,0.0454487849],[1,18369,0.029091291],[1,18370,0.287688844],[1,18371,0.1315051889],[1,18402,0.0446574],[1,18431,0.1330929621],[1,18443,0.2628774573],[1,18444,0.0447482643],[1,18470,0.0870541009],[1,18480,0.0388759928],[1,18487,0.04628669],[1,18496,0.099044728],[1,18526,0.0644934867],[1,18538,0.0450945307],[1,18543,0.1509336541],[1,18582,0.0345832093],[1,18599,0.0926687753],[1,18615,0.1132060646],[1,18617,0.0869135438],[1,18626,0.1760264101],[1,18642,0.0301740821],[1,18655,0.0325526486],[1,18661,0.0312500563],[1,18700,0.3128682274],[1,18711,0.1200368915],[1,18713,0.0522825532],[1,18720,0.0448770936],[1,18725,0.146663615],[1,18739,0.6583096439],[1,18740,0.1185793575],[1,18748,0.0640357489],[1,18768,0.0648900487],[1,18773,0.0279086477],[1,18780,0.3538625528],[1,18786,0.0372866944],[1,18792,0.4135713387],[1,18814,0.0287949572],[1,18816,0.0471958165],[1,18818,0.0522389369],[1,18830,0.1046144327],[1,18835,0.4223960167],[1,18844,0.0348151499],[1,18868,0.0367380499],[1,18881,0.182133042],[1,18886,0.3539954373],[1,18893,0.0753190631],[1,18896,0.0370527302],[1,18909,0.0338724093],[1,18911,0.1461529089],[1,18922,0.0405693348],[1,18925,0.046443766],[1,18941,0.0415227848],[1,18945,0.0348151499],[1,18947,0.0363222522],[1,18957,0.1354142313],[1,18958,0.1483150901],[1,18959,0.118305444],[1,18961,0.0531768478],[1,18979,0.4481757632],[1,19019,0.0300685698],[1,19031,0.0439709872],[1,19032,0.0502375651],[1,19033,0.6059847317],[1,19037,0.0348151499],[1,19042,0.3540905075],[1,19048,0.3461769229],[1,19079,0.0572218739],[1,19083,0.1818582934],[1,19117,0.0800149942],[1,19125,0.1602097783],[1,19129,0.0348559105],[1,19153,0.1305478858],[1,19188,0.0520749351],[1,19192,0.0522389369],[1,19195,0.0337967934],[1,19201,0.0681520901],[1,19217,0.0608418729],[1,19241,0.4643819541],[1,19267,0.0768745511],[1,19280,0.2997720413],[1,19281,0.0510329838],[1,19284,0.7474894113],[1,19287,0.1713911845],[1,19303,0.0373352191],[1,19307,0.0412220186],[1,19314,0.1020613106],[1,19318,0.0387528158],[1,19328,0.3808568968],[1,19366,0.0338724093],[1,19375,0.5015237148],[1,19383,0.1383540074],[1,19393,0.0450416054],[1,19411,0.1522273977],[1,19416,0.0812259331],[1,19437,0.0348151499],[1,19449,0.0348151499],[1,19459,0.3346491823],[1,19464,0.0825205428],[1,19467,0.6536300883],[1,19474,0.1652070483],[1,19500,0.0744792274],[1,19512,0.0413237049],[1,19531,0.3455559183],[1,19542,0.1451812503],[1,19547,0.2003634942],[1,19564,0.0310290898],[1,19566,0.0306726399],[1,19578,0.0552311002],[1,19595,0.0348151499],[1,19608,0.0500454897],[1,19612,0.065072195],[1,19625,0.0353717534],[1,19626,0.0655098981],[1,19703,0.0355547869],[1,19708,0.3110068284],[1,19722,0.1083698307],[1,19724,0.1264079304],[1,19778,0.0608418729],[1,19780,0.0748421248],[1,19783,0.3488957099],[1,19793,0.0641358621],[1,19804,0.0438186169],[1,19808,0.1242264605],[1,19814,0.1472061102],[1,19843,0.0435798312],[1,19862,0.0301740821],[1,19875,0.1182523357],[1,19893,0.0575446136],[1,19906,0.3435190962],[1,19915,0.070495604],[1,19917,0.0348151499],[1,19951,0.0317899712],[1,19955,0.0997959508],[1,19977,0.2244035036],[1,19979,0.2537386071],[1,19986,0.4122674913],[1,19988,0.0402047196],[1,19995,0.0859249383],[1,19996,0.089506612],[1,20000,0.3131366107],[1,20015,0.0713327996],[1,20048,0.0323003366],[1,20070,0.066840304],[1,20074,0.0348151499],[1,20110,0.0417754523],[1,20122,0.2556891715],[1,20128,0.0345832093],[1,20134,0.0370527302],[1,20138,0.0727462274],[1,20142,0.7462922467],[1,20151,0.0877427031],[1,20153,0.0389422608],[1,20179,0.0592630822],[1,20194,0.0607568761],[1,20198,0.0353496901],[1,20228,0.3177054095],[1,20238,0.0348151499],[1,20240,0.0398200933],[1,20243,0.2973281304],[1,20253,0.0370902816],[1,20276,0.0590754431],[1,20294,0.0358531041],[1,20300,0.0461136916],[1,20302,0.3853044715],[1,20334,0.055593158],[1,20341,0.4129523072],[1,20344,0.0415119131],[1,20347,0.0629923007],[1,20348,0.0704479729],[1,20353,0.1031662578],[1,20355,0.0331379972],[1,20370,0.0372866944],[1,20372,0.0328227345],[1,20385,0.0538991155],[1,20388,0.0380723599],[1,20420,0.0312500563],[1,20430,0.0401224918],[1,20437,0.0312881931],[1,20442,0.0608429307],[1,20456,0.0348151499],[1,20465,0.0348151499],[1,20481,0.0432294796],[1,20503,0.4251400885],[1,20511,0.0334816301],[1,20518,0.0945479956],[1,20530,0.0348151499],[1,20534,0.0548032495],[1,20536,0.0332639202],[1,20537,0.0467373409],[1,20540,0.0740569404],[1,20553,0.0907189963],[1,20566,0.0334534577],[1,20577,0.0773462983],[1,20634,0.4085183479],[1,20638,0.0316581234],[1,20680,0.0346927799],[1,20686,0.0342505912],[1,20688,0.03710889],[1,20699,0.0812375512],[1,20702,0.077234914],[1,20706,0.0430869897],[1,20709,0.0378595378],[1,20721,0.0349373831],[1,20765,0.044582209],[1,20781,0.0378595378],[1,20786,0.1922629461],[1,20792,0.0416351735],[1,20801,0.0641750951],[1,20810,0.0353717534],[1,20811,0.1573012521],[1,20826,0.0607568761],[1,20834,0.0955707516],[1,20870,0.8243861041],[1,20872,0.0353467491],[1,20874,0.0409299507],[1,20885,0.134993092],[1,20940,0.0348151499],[1,20953,0.0345832093],[1,20963,0.0462943843],[1,20979,0.0353717534],[1,21014,0.0624321132],[1,21023,0.0386245005],[1,21049,0.0345196022],[1,21055,0.0635832985],[1,21069,0.0291966662],[1,21076,0.0495584639],[1,21085,0.0531391523],[1,21095,0.041061903],[1,21119,0.0348151499],[1,21126,0.3172822719],[1,21129,0.0342187937],[1,21135,0.0358531041],[1,21166,0.0523369132],[1,21231,0.1084775862],[1,21232,0.0348151499],[1,21236,0.0334534577],[1,21245,0.0574192736],[1,21250,0.0468265306],[1,21255,0.0315889996],[1,21309,0.04339866],[1,21315,0.0315515403],[1,21345,0.1223085377],[1,21364,0.0555465208],[1,21381,0.0314143165],[1,21418,0.0390749534],[1,21433,0.033249366],[1,21442,0.2578075112],[1,21457,0.0254886648],[1,21458,0.0523703362],[1,21460,0.0290822173],[1,21467,0.0290822173],[1,21468,0.0461431143],[1,21475,0.0305779006],[1,21513,0.131239137],[1,21514,0.0438993526],[1,21516,0.0290560549],[1,21527,0.0735874606],[1,21539,0.0270476682],[1,21542,0.0548273878],[1,21581,0.0244763141],[1,21584,0.0282665724],[1,21591,0.0299409947],[1,21605,0.0449544638],[1,21618,0.0298171204],[1,21628,0.1083262736],[1,21641,0.029446854],[1,21655,0.0289065783],[1,21671,0.0323952256],[1,21679,0.2267127766],[1,21707,0.0650339878],[1,21709,0.0976036493],[1,21721,0.3960100688],[1,21733,0.0236162334],[1,21743,0.0682555409],[1,21787,0.164502057],[1,21800,0.2759916282],[1,21810,0.0403851257],[1,21819,0.232899368],[1,21829,0.0387788183],[1,21849,0.0376808396],[1,21853,0.0218961376],[1,21854,0.0209294809],[1,21860,0.0650506821],[1,21872,0.0308451585],[1,21887,0.2702483855],[1,21899,0.1109198062],[1,21916,0.0360111372],[1,21934,0.025943286],[1,21944,0.0246021991],[1,21948,0.1405515454],[1,21964,0.1620643938],[1,21978,0.0320254738],[1,22011,0.1656704856],[1,22102,0.0264845244],[1,22112,0.3015602742],[1,22114,0.0323132138],[1,22143,0.074088294],[1,22151,0.1969065394],[1,22153,0.0275940851],[1,22171,0.0289669006],[1,22203,0.0412099343],[1,22208,0.0275280825],[1,22209,0.061162883],[1,22217,0.0289669006],[1,22219,0.5519891309],[1,22236,0.0288917643],[1,22238,0.4381645757],[1,22268,0.1374269839],[1,22276,0.3708280975],[1,22301,0.1374537941],[1,22310,0.0454567387],[1,22329,0.0277332926],[1,22374,0.2462635995],[1,22381,0.0846979435],[1,22382,0.0815268085],[1,22403,0.0297008208],[1,22428,0.0294142928],[1,22437,0.0715452634],[1,22442,0.0749847566],[1,22449,0.0297031666],[1,22455,0.2943961307],[1,22460,0.0399538746],[1,22470,0.072892169],[1,22472,0.2783714871],[1,22501,0.0300625068],[1,22514,0.2585020519],[1,22550,0.19017562],[1,22554,0.0254293625],[1,22556,0.0750538998],[1,22561,0.302625027],[1,22563,0.57502299],[1,22587,0.067367747],[1,22589,0.1621671566],[1,22608,0.0597767065],[1,22621,0.0442962084],[1,22623,0.0235216719],[1,22633,0.0261148815],[1,22637,0.0542816413],[1,22638,0.028866939],[1,22675,0.0486090516],[1,22712,0.029509806],[1,22718,0.0383065502],[1,22731,0.0313458191],[1,22746,0.0525716004],[1,22749,0.0327437971],[1,22780,0.0272755096],[1,22808,0.0302001857],[1,22816,0.029519202],[1,22827,0.0330435532],[1,22838,0.0325523184],[1,22840,0.2010679935],[1,22860,0.0281302737],[1,22878,0.0582068524],[1,22888,0.0248803449],[1,22936,0.0349848247],[1,22956,0.0622116367],[1,22989,0.3312108748],[1,23002,0.2465108054],[1,23004,0.0290073462],[1,23011,0.0357994804],[1,23027,0.0217574219],[1,23036,0.0671219915],[1,23054,0.3716063167],[1,23062,0.1516772453],[1,23064,0.0247451839],[1,23073,0.0546000728],[1,23081,0.0800592633],[1,23132,0.0263941803],[1,23158,0.0297662676],[1,23177,0.0234945253],[1,23232,0.3569366453],[1,23243,0.028586117],[1,23263,0.0328670546],[1,23264,0.0554271399],[1,23270,0.0304329593],[1,23290,0.0233964729],[1,23309,0.0407851454],[1,23312,0.0280563516],[1,23327,0.1176430967],[1,23354,0.0336773998],[1,23371,0.3190382279],[1,23381,0.02382507],[1,23420,0.0547924572],[1,23432,0.3613760932],[1,23444,0.043608868],[1,23446,0.0543697],[1,23463,0.0432248514],[1,23465,0.0591382789],[1,23491,0.22704809],[1,23516,0.0368547011],[1,23540,0.3598267674],[1,23542,0.0218074253],[1,23547,0.0957168006],[1,23550,0.0554271399],[1,23556,0.1219241562],[1,23564,0.0543697],[1,23567,0.0326153809],[1,23575,0.1575761329],[1,23577,0.5056149024],[1,23580,0.1380312677],[1,23584,0.1258298531],[1,23587,0.0507709566],[1,23622,0.0481438322],[1,23660,0.042572273],[1,23682,0.1561798595],[1,23692,0.1309834156],[1,23709,0.0218074253],[1,23716,0.1642673332],[1,23737,0.031980824],[1,23740,0.0232912936],[1,23754,0.0278809525],[1,23762,0.0280563516],[1,23799,0.0238724496],[1,23801,0.0395315639],[1,23836,0.0588989587],[1,23847,0.0471524146],[1,23871,0.0385359445],[1,23926,0.0400194253],[1,23934,0.0279813064],[1,23940,0.0280563516],[1,23955,0.4152198848],[1,23984,0.0287423496],[1,23987,0.0871370488],[1,24004,0.037911816],[1,24015,0.0306131344],[1,24023,0.0218787073],[1,24027,0.0414694376],[1,24081,0.0605005526],[1,24088,0.0445407992],[1,24093,0.0264359243],[1,24140,0.0512970104],[1,24155,0.023566663],[1,24157,0.0320382849],[1,24175,0.0402921345],[1,24185,0.026883113],[1,24195,0.3028888768],[1,24201,0.1324817396],[1,24202,0.0644812874],[1,24218,0.1478009549],[1,24219,0.0902514085],[1,24221,0.0356062312],[1,24243,0.0243472705],[1,24267,0.02382507],[1,24276,0.0246725983],[1,24289,0.0246725983],[1,24311,0.0350116897],[1,24314,0.037345464],[1,24325,0.0285985986],[1,24328,0.0285832022],[1,24359,0.0432354961],[1,24360,0.0305058187],[1,24370,0.0520029158],[1,24376,0.0559531967],[1,24391,0.0528450369],[1,24393,0.1561259432],[1,24399,0.0483672825],[1,24418,0.02529431],[1,24428,0.0563704062],[1,24441,0.1008353852],[1,24472,0.1628803091],[1,24486,0.0414965081],[1,24501,0.0268541915],[1,24519,0.0280563516],[1,24533,0.0468358587],[1,24543,0.0882958278],[1,24545,0.2040803272],[1,24547,0.1836294239],[1,24552,0.0279550092],[1,24569,0.042238557],[1,24578,0.0433733536],[1,24579,0.0379869045],[1,24582,0.1011664975],[1,24584,0.0298076252],[1,24594,0.1411686018],[1,24621,0.0472485387],[1,24635,0.1858819058],[1,24643,0.0547935069],[1,24648,0.0423530514],[1,24653,0.0380837999],[1,24664,0.039565146],[1,24668,0.0535433622],[1,24670,0.0580609864],[1,24671,0.0378211383],[1,24691,0.4528467652],[1,24723,0.023352171],[1,24768,0.0345084776],[1,24777,0.1830436497],[1,24829,0.0615909545],[1,24878,0.0284088808],[1,24893,0.4785531857],[1,24896,0.0385954685],[1,24954,0.0299253605],[1,24972,0.0338386218],[1,24985,0.0375986213],[1,25032,0.047280987],[1,25057,0.0379532871],[1,25064,0.0308567691],[1,25070,0.027604462],[1,25099,0.042176284],[1,25116,0.0402273663],[1,25124,0.0295923123],[1,25136,0.3549334356],[1,25165,0.2289637886],[1,25166,0.030256965],[1,25174,0.0314298364],[1,25175,0.0303322379],[1,25200,0.0466838185],[1,25205,0.0373870787],[1,25235,0.2516419261],[1,25242,0.071365909],[1,25296,0.048134182],[1,25327,0.0527760129],[1,25328,0.4865681087],[1,25337,0.0492052051],[1,25354,0.0302582196],[1,25423,0.0404650896],[1,25468,0.0213328497],[1,25506,0.0298203347],[1,25522,0.0491708329],[1,25537,0.0645080464],[1,25538,0.0261152515],[1,25541,0.0308684288],[1,25567,0.602737737],[1,25583,0.0536539542],[1,25592,0.1775403779],[1,25598,0.0863982449],[1,25637,0.1154117838],[1,25685,0.0880371063],[1,25732,0.0629307374],[1,25734,0.3288205679],[1,25746,0.0338945925],[1,25787,0.0488995267],[1,25838,0.0379721582],[1,25845,0.0306125574],[1,25857,0.3013689022],[1,25889,0.0321674107],[1,25893,0.1033666966],[1,25898,0.0264968839],[1,25909,0.1714254692],[1,25910,0.0408687892],[1,25924,0.2180411153],[1,25925,0.0317794426],[1,25928,0.6369850315],[1,25932,0.0272798026],[1,25950,0.0273159078],[1,25961,0.054545176],[1,26004,0.0515849425],[1,26007,0.0306813346],[1,26028,0.0401805398],[1,26033,0.2931019707],[1,26037,0.1528730648],[1,26073,0.0274333352],[1,26080,0.1069519532],[1,26087,0.0570867936],[1,26105,0.1698581833],[1,26106,0.0966783853],[1,26112,0.0285176326],[1,26144,0.6368252761],[1,26154,0.1385042208],[1,26164,0.2017802905],[1,26174,0.0981678432],[1,26188,0.0626904785],[1,26193,0.0633535408],[1,26195,0.0306813346],[1,26205,0.0275401168],[1,26241,0.0286998046],[1,26263,0.0281061349],[1,26315,0.1016100994],[1,26319,0.0296478535],[1,26325,0.3303750688],[1,26338,0.2783576713],[1,26368,0.043863683],[1,26378,0.1480941193],[1,26385,0.0428220193],[1,26398,0.0629936962],[1,26409,0.0439409271],[1,26412,0.0280849589],[1,26413,0.0383951992],[1,26417,0.1767270174],[1,26420,0.0455676672],[1,26423,0.0260651149],[1,26429,0.0366376832],[1,26430,0.0275973844],[1,26452,0.2840316839],[1,26467,0.1018758224],[1,26479,0.0525538053],[1,26485,0.0541520964],[1,26519,0.0500758818],[1,26532,0.0306813346],[1,26540,0.0194956656],[1,26559,0.0362974695],[1,26568,0.3777075763],[1,26577,0.0435581193],[1,26602,0.0405149596],[1,26609,0.0446064451],[1,26622,0.0279522927],[1,26629,0.3039235589],[1,26630,0.027847092],[1,26641,0.0424732887],[1,26650,0.0538101696],[1,26668,0.0636896552],[1,26682,0.0326808555],[1,26690,0.0372575292],[1,26696,0.0317794426],[1,26714,0.3625096792],[1,26717,0.5716753543],[1,26745,0.2852208549],[1,26746,0.0306306668],[1,26750,0.1649804383],[1,26753,0.274918711],[1,26754,0.0298481394],[1,26769,0.0435979805],[1,26794,0.02950206],[1,26803,0.04896527],[1,26810,0.0381471918],[1,26811,0.2965563619],[1,26813,0.0771077995],[1,26814,0.0897659736],[1,26848,0.065316766],[1,26880,0.0474824892],[1,26892,0.1833116369],[1,26896,0.0242111012],[1,26899,0.6792712905],[1,26909,0.0359306744],[1,26910,0.135902202],[1,26916,0.0675240965],[1,26921,0.0602659308],[1,26925,0.0570867936],[1,26933,0.038740506],[1,26958,0.0312254797],[1,26972,0.0570867936],[1,27008,0.1206907454],[1,27009,0.0294002493],[1,27019,0.0358697234],[1,27020,0.0204009653],[1,27022,0.4264356709],[1,27032,0.0492404145],[1,27064,0.0559434251],[1,27072,0.0827451057],[1,27075,0.0329066902],[1,27094,0.0321128303],[1,27121,0.069076514],[1,27128,0.0955001986],[1,27153,0.0662984608],[1,27168,0.0570867936],[1,27171,0.0867838115],[1,27172,0.0277176205],[1,27178,0.0640487149],[1,27181,0.0252037104],[1,27190,0.2067677437],[1,27206,0.0848315065],[1,27236,0.0894602616],[1,27237,0.0306060267],[1,27240,0.4350053243],[1,27260,0.0340896224],[1,27262,0.02928586],[1,27300,0.083900037],[1,27322,0.0838823601],[1,27329,0.0293456693],[1,27330,0.0570867936],[1,27332,0.027434974],[1,27342,0.0360868188],[1,27371,0.0306813346],[1,27399,0.0973120911],[1,27412,0.0322136483],[1,27440,0.0329066902],[1,27445,0.1180240403],[1,27454,0.2697083623],[1,27478,0.1107295166],[1,27485,0.2332599038],[1,27494,0.0268673193],[1,27528,0.0306813346],[1,27569,0.0585126559],[1,27582,0.3302175986],[1,27604,0.3781997494],[1,27614,0.0375447693],[1,27620,0.0521529862],[1,27625,0.0306813346],[1,27635,0.1409613922],[1,27656,0.181026534],[1,27672,0.028159259],[1,27693,0.0559388551],[1,27705,0.1293472381],[1,27717,0.0414958877],[1,27734,0.0298033361],[1,27738,0.0271919978],[1,27756,0.0740984186],[1,27764,0.079425905],[1,27765,0.2294400799],[1,27776,0.049100233],[1,27784,0.1360033862],[1,27794,0.3675399467],[1,27840,0.029540147],[1,27871,0.0685474318],[1,27888,0.0786144455],[1,27919,0.0489480807],[1,27920,0.0253361475],[1,27927,0.0280874731],[1,27938,0.1094678332],[1,27942,0.0317249882],[1,27944,0.1511574187],[1,27955,0.156124441],[1,27970,0.2731219615],[1,27973,0.0771669773],[1,27975,0.0514014973],[1,27986,0.4748577008],[1,27991,0.8433318622],[1,28010,0.0318324541],[1,28025,0.0306813346],[1,28031,0.0290077753],[1,28034,0.2717119421],[1,28071,0.0327313277],[1,28115,0.0266798287],[1,28133,0.0462239534],[1,28137,0.030607282],[1,28141,0.0306813346],[1,28148,0.05675438],[1,28156,0.0580384978],[1,28158,0.0601019605],[1,28159,0.031289385],[1,28164,0.0931608827],[1,28168,0.0333984517],[1,28169,0.0298287208],[1,28177,0.0304506612],[1,28216,0.1190049526],[1,28221,0.0230985495],[1,28225,0.0956767204],[1,28229,0.0439705124],[1,28244,0.110622715],[1,28253,0.0979603972],[1,28261,0.1644197588],[1,28283,0.0214922946],[1,28285,0.0462805603],[1,28307,0.0290071261],[1,28310,0.0386059817],[1,28313,0.2489413838],[1,28327,0.0306060267],[1,28334,0.0304506612],[1,28358,0.0353418078],[1,28391,0.0454037382],[1,28419,0.0306813346],[1,28455,0.0801313251],[1,28479,0.0330864435],[1,28488,0.0401630014],[1,28498,0.2390304638],[1,28499,0.0324469553],[1,28520,0.0969200525],[1,28524,0.0978319426],[1,28542,0.0500255766],[1,28546,0.0540954411],[1,28551,0.0518898859],[1,28591,0.304820524],[1,28612,0.0537485352],[1,28615,0.0937127493],[1,28616,0.0301068428],[1,28663,0.1072413196],[1,28677,0.0306813346],[1,28699,0.5581803002],[1,28708,0.0404786675],[1,28722,0.0515638248],[1,28724,0.0539953496],[1,28749,0.0458902941],[1,28758,0.0385376427],[1,28760,0.0766369002],[1,28770,0.1865079971],[1,28817,0.041953539],[1,28831,0.0436683234],[1,28864,0.435710706],[1,28867,0.0664141698],[1,28933,0.1291813509],[1,28941,0.1301043932],[1,28971,0.0302087601],[1,28998,0.0435984021],[1,29001,0.0827121666],[1,29028,0.0567851969],[1,29029,0.1348616875],[1,29060,0.030079072],[1,29061,0.0300246717],[1,29078,0.0294719988],[1,29093,0.2500414271],[1,29126,0.0294719988],[1,29132,0.1057272192],[1,29158,0.2805763379],[1,29166,0.0550071244],[1,29176,0.1867919632],[1,29191,0.0567851969],[1,29209,0.0381421028],[1,29248,0.0288080398],[1,29274,0.1107466968],[1,29293,0.0491439277],[1,29325,0.0271871909],[1,29328,0.0259814185],[1,29382,0.0232831833],[1,29433,0.0388806592],[1,29448,0.0301587811],[1,29501,0.0333370647],[1,29516,0.0601558499],[1,29520,0.0294719988],[1,29525,0.0614431439],[1,29544,0.842273256],[1,29560,0.0549511173],[1,29562,0.0235275682],[1,29574,0.0766658553],[1,29581,0.0294719988],[1,29622,0.0287413113],[1,29650,0.0285954112],[1,29705,0.2438843531],[1,29797,0.0514721208],[1,29813,0.1252593725],[1,29893,0.127244196],[1,29920,0.3390925022],[1,29978,0.0382665857],[1,30001,0.0261808854],[1,30011,0.4552138502],[1,30013,0.5762229702],[1,30014,0.0267055812],[1,30043,0.0904135717],[1,30049,0.0438847337],[1,30066,0.1287369187],[1,30107,0.0297215965],[1,30110,0.3008280735],[1,30133,0.0294719988],[1,30139,0.2756879489],[1,30141,0.0468393457],[1,30150,0.0617054487],[1,30152,0.0294719988],[1,30177,0.1727817374],[1,30186,0.0235295119],[1,30188,0.0491912618],[1,30194,0.3210832507],[1,30198,0.2425340612],[1,30200,0.1049329542],[1,30220,0.0623394394],[1,30247,0.0447816614],[1,30249,0.0600850112],[1,30250,0.0339086822],[1,30262,0.023520093],[1,30265,0.2498001266],[1,30285,0.5510154883],[1,30324,0.3009629895],[1,30326,0.0315583327],[1,30328,0.0375767268],[1,30340,0.1704838704],[1,30341,0.2115721939],[1,30344,0.0225083867],[1,30381,0.3389503384],[1,30386,0.0835713271],[1,30412,0.1224657077],[1,30417,0.0412327859],[1,30428,0.0294719988],[1,30435,0.2896680352],[1,30448,0.2337310329],[1,30458,0.1558062864],[1,30496,0.0292416961],[1,30500,0.0630438265],[1,30504,0.0337939544],[1,30522,0.0452027923],[1,30526,0.0388177618],[1,30537,0.0385367055],[1,30545,0.2398426794],[1,30566,0.0899065355],[1,30581,0.1201503913],[1,30582,0.0292054646],[1,30596,0.1802591832],[1,30601,0.1097226725],[1,30610,0.0718088085],[1,30625,0.0539852734],[1,30635,0.051432295],[1,30650,0.0290151503],[1,30652,0.0378516157],[1,30662,0.123198161],[1,30668,0.0294910538],[1,30673,0.0294719988],[1,30675,0.0279014307],[1,30685,0.0383418693],[1,30715,0.0288984302],[1,30717,0.0300152694],[1,30729,0.0510747486],[1,30732,0.0534925231],[1,30787,0.0752947702],[1,30801,0.0322541115],[1,30818,0.1142437332],[1,30822,0.3705105258],[1,30841,0.0257249015],[1,30847,0.0504475741],[1,30857,0.1146594917],[1,30895,0.0545150675],[1,30896,0.0336159903],[1,30900,0.3059746539],[1,30901,0.0244442272],[1,30918,0.0323257048],[1,30938,0.0293968119],[1,30955,0.5011085303],[1,30965,0.0594420422],[1,30976,0.0246533139],[1,30982,0.0598265101],[1,30985,0.0441269165],[1,30988,0.0244913695],[1,31016,0.0443972652],[1,31021,0.0415987881],[1,31026,0.2004489709],[1,31031,0.1271061444],[1,31035,0.0378552909],[1,31036,0.0269070358],[1,31040,0.0272498166],[1,31046,0.0295539665],[1,31053,0.0334111973],[1,31060,0.0393307492],[1,31076,0.0490752017],[1,31078,0.0429378246],[1,31089,0.0294719988],[1,31094,0.0263734029],[1,31112,0.0285058586],[1,31133,0.0782727433],[1,31148,0.0596815288],[1,31162,0.0303434757],[1,31195,0.0262410394],[1,31203,0.299896381],[1,31259,0.02969777],[1,31269,0.0250446701],[1,31278,0.0265130475],[1,31326,0.0232626319],[1,31332,0.0909961437],[1,31338,0.0402712621],[1,31372,0.2658726436],[1,31386,0.037077957],[1,31390,0.0425594253],[1,31392,0.4617102614],[1,31395,0.0345444486],[1,31412,0.028640079],[1,31419,0.1341533178],[1,31441,0.0558350222],[1,31463,0.0300079954],[1,31466,0.1624235073],[1,31472,0.0288899692],[1,31477,0.0234742587],[1,31478,0.0293542957],[1,31513,0.0274931443],[1,31524,0.343281314],[1,31563,0.1496944473],[1,31578,0.0493902171],[1,31593,0.0300152694],[1,31610,0.0300246717],[1,31612,0.0669144034],[1,31672,0.0535934018],[1,31711,0.0355874845],[1,31725,0.0269076993],[1,31735,0.0561740784],[1,31738,0.0379547002],[1,31747,0.0300152694],[1,31814,0.1056582724],[1,31824,0.0276856913],[1,31878,0.0262152543],[1,31896,0.0269434765],[1,31898,0.0346776912],[1,31928,0.0281199224],[1,31954,0.1329958135],[1,31972,0.0291785381],[1,31995,0.0384784303],[1,32007,0.03150586],[1,32031,0.249064107],[1,32038,0.026618061],[1,32040,0.0294281375],[1,32066,0.0729295946],[1,32076,0.0300246717],[1,32085,0.0331316251],[1,32105,0.0874301307],[1,32125,0.0580991534],[1,32130,0.0255540539],[1,32133,0.0256728868],[1,32140,0.1140750515],[1,32141,0.047588978],[1,32157,0.0318993662],[1,32169,0.0405741104],[1,32255,0.0276856913],[1,32257,0.0745961348],[1,32268,0.0410462299],[1,32285,0.0333547916],[1,32301,0.0259434402],[1,32308,0.0520213233],[1,32327,0.0518365442],[1,32331,0.17695254],[1,32368,0.0411211513],[1,32377,0.0795077318],[1,32380,0.0302796587],[1,32391,0.0258298043],[1,32397,0.044354384],[1,32400,0.0595332771],[1,32410,0.0430720764],[1,32477,0.0620094687],[1,32503,0.0315186785],[1,32519,0.0273308084],[1,32531,0.0513026465],[1,32541,0.0736253973],[1,32559,0.0327118032],[1,32592,0.0318506576],[1,32598,0.0367693563],[1,32631,0.1462227781],[1,32642,0.0479606094],[1,32645,0.1170027076],[1,32660,0.0297515176],[1,32665,0.0425262907],[1,32749,0.4581713929],[1,32750,0.0261993063],[1,32774,0.0337809821],[1,32806,0.0421220541],[1,32814,0.1555311908],[1,32827,0.0793751823],[1,32836,0.3971125086],[1,32843,0.0266267015],[1,32865,0.2197913974],[1,32876,0.0220710259],[1,32896,0.134188318],[1,32917,0.0295705154],[1,32953,0.0202213785],[1,32973,0.0642884069],[1,32981,0.0470591018],[1,32992,0.0256749654],[1,33030,0.0274931443],[1,33031,0.2119791857],[1,33033,0.0880305089],[1,33081,0.0308774524],[1,33123,0.0400939286],[1,33148,0.0521996722],[1,33154,0.0443720399],[1,33188,0.0326571324],[1,33215,0.2157636085],[1,33244,0.0232626319],[1,33282,0.1194056345],[1,33290,0.0358998299],[1,33302,0.0301123451],[1,33332,0.0897441005],[1,33338,0.0274330327],[1,33344,0.0284020349],[1,33370,0.0408955177],[1,33384,0.0406610826],[1,33388,0.0279998199],[1,33392,0.0458285682],[1,33394,0.0323177397],[1,33397,0.0360739421],[1,33456,0.0492200639],[1,33464,0.0236623058],[1,33479,0.0231658346],[1,33485,0.0452565273],[1,33500,0.3671193517],[1,33540,0.0961725778],[1,33561,0.0285577156],[1,33566,0.0761737612],[1,33573,0.0257249015],[1,33591,0.0283906722],[1,33592,0.0227190276],[1,33613,0.0525940394],[1,33640,0.0294719988],[1,33649,0.0298438269],[1,33657,0.0539449467],[1,33670,0.0310419477],[1,33671,0.4422105683],[1,33673,0.0299427265],[1,33680,0.0379431861],[1,33713,0.0274931443],[1,33800,0.2189622347],[1,33808,0.0261698723],[1,33811,0.0296594066],[1,33837,0.0255693482],[1,33847,0.0676143247],[1,33876,0.0236623058],[1,33882,0.330599835],[1,33959,0.024002671],[1,33975,0.0412386191],[1,33979,0.0290547278],[1,33995,0.0324171555],[1,34006,0.0424491894],[1,34009,0.0277872696],[1,34077,0.0237113467],[1,34099,0.0292416961],[1,34114,0.3712087825],[1,34117,0.0294719988],[1,34122,0.0415023468],[1,34171,0.0300152694],[1,34197,0.0614450185],[1,34225,0.0202460448],[1,34228,0.0300246717],[1,34235,0.0439863861],[1,34242,0.0468119995],[1,34253,0.0318506576],[1,34280,0.0365973878],[1,34286,0.3120204003],[1,34347,0.0264682003],[1,34370,0.044433888],[1,34442,0.0249257243],[1,34461,0.0260125643],[1,34474,0.3048476276],[1,34504,0.1556156256],[1,34518,0.0212446824],[1,34551,0.0279798356],[1,34560,0.0395847378],[1,34563,0.0656411844],[1,34625,0.1035950237],[1,34651,0.3392986406],[1,34699,0.0754864482],[1,34709,0.0262944838],[1,34717,0.0220208336],[1,34719,0.0335427317],[1,34727,0.0527983273],[1,34744,0.0264979762],[1,34750,0.0267719267],[1,34780,0.0206185112],[1,34800,0.0891691621],[1,34827,0.2274210877],[1,34872,0.0259940594],[1,34893,0.1091303193],[1,34899,0.1117383166],[1,34912,0.0371813666],[1,34922,0.0319843929],[1,34939,0.0305115172],[1,34973,0.0254953796],[1,34984,0.0273861902],[1,34996,0.028370006],[1,35022,0.0233977885],[1,35058,0.0269262149],[1,35064,0.052913883],[1,35065,0.0532676477],[1,35078,0.0269731332],[1,35105,0.021717554],[1,35123,0.0246807391],[1,35132,0.2538385531],[1,35199,0.1299071161],[1,35208,0.0531169505],[1,35250,0.0788801122],[1,35263,0.03288583],[1,35272,0.0369689455],[1,35281,0.0272089259],[1,35297,0.2206249667],[1,35298,0.067200155],[1,35324,0.1063989765],[1,35349,0.0917450602],[1,35355,0.031271722],[1,35366,0.0864864641],[1,35374,0.0408465012],[1,35401,0.0324836585],[1,35414,0.041274343],[1,35446,0.1066357631],[1,35473,0.3322034221],[1,35478,0.7091262808],[1,35479,0.0723315452],[1,35504,0.053980579],[1,35509,0.027583393],[1,35512,0.0483009651],[1,35538,0.0260669021],[1,35551,0.02477853],[1,35561,0.026219615],[1,35568,0.0321665059],[1,35584,0.2031902098],[1,35593,0.0314234445],[1,35600,0.0286630825],[1,35608,0.0577670272],[1,35609,0.0258219001],[1,35613,0.0368074628],[1,35615,0.0427199101],[1,35638,0.0864419456],[1,35651,0.2708849801],[1,35661,0.0262944838],[1,35662,0.0260125643],[1,35663,0.0289170749],[1,35680,0.0485665749],[1,35685,0.0348024163],[1,35724,0.0628031049],[1,35736,0.0232539069],[1,35743,0.5622531639],[1,35748,0.0292281736],[1,35754,0.0935962129],[1,35768,0.0429664768],[1,35772,0.3764581899],[1,35778,0.0262944838],[1,35780,0.0366492789],[1,35790,0.0293653487],[1,35826,0.1801519073],[1,35835,0.0262944838],[1,35859,0.0400955077],[1,35863,0.0262944838],[1,35865,0.0527912059],[1,35869,0.0234409136],[1,35913,0.0366999029],[1,35920,0.0378841482],[1,35928,0.1638300849],[1,35944,0.0396218687],[1,35951,0.028577722],[1,35958,0.0273861902],[1,35967,0.3126132005],[1,35968,0.066000105],[1,35977,0.0505685395],[1,35980,0.0262944838],[1,35988,0.0262944838],[1,36007,0.0433456275],[1,36050,0.0258048014],[1,36080,0.0918143252],[1,36083,0.3504603432],[1,36084,0.0445655096],[1,36086,0.0321531212],[1,36099,0.2500590972],[1,36113,0.2668788896],[1,36125,0.2577900224],[1,36129,0.200682835],[1,36136,0.0285068664],[1,36152,0.2096307528],[1,36166,0.0993643657],[1,36167,0.2442107956],[1,36200,0.0902764563],[1,36209,0.0563160469],[1,36233,0.0398784658],[1,36286,0.2703966489],[1,36293,0.0390691594],[1,36301,0.0322368799],[1,36318,0.0915035192],[1,36355,0.1338786764],[1,36358,0.0485509533],[1,36372,0.1010052802],[1,36413,0.0737796313],[1,36418,0.0417579062],[1,36419,0.0468850867],[1,36431,0.1291940201],[1,36432,0.0423242766],[1,36439,0.3422984901],[1,36454,0.039936534],[1,36467,0.0199522113],[1,36474,0.119823816],[1,36500,0.1364630198],[1,36504,0.0466532358],[1,36512,0.1076228074],[1,36518,0.0536009638],[1,36522,0.1099455319],[1,36524,0.0916347588],[1,36532,0.3308834237],[1,36539,0.03304428],[1,36540,0.0391329503],[1,36549,0.1800060906],[1,36553,0.0417337151],[1,36564,0.0774230307],[1,36573,0.0435594864],[1,36583,0.0396123923],[1,36592,0.2153347715],[1,36595,0.0537006363],[1,36610,0.0262944838],[1,36639,0.3379975104],[1,36641,0.0239918093],[1,36643,0.0851048411],[1,36648,0.0486270832],[1,36688,0.0751992768],[1,36703,0.042718265],[1,36722,0.3345805358],[1,36756,0.0217344042],[1,36760,0.019141243],[1,36764,0.073236367],[1,36774,0.1750730023],[1,36779,0.0331408489],[1,36783,0.290637725],[1,36810,0.2601270729],[1,36830,0.023409973],[1,36832,0.0376256137],[1,36837,0.0211096416],[1,36861,0.0272333727],[1,36864,0.4355499489],[1,36865,0.0214123753],[1,36883,0.0420340548],[1,36909,0.2389769007],[1,36915,0.2552912239],[1,36916,0.0255868012],[1,36921,0.2846450085],[1,36929,0.2232751167],[1,36937,0.1499317826],[1,36950,0.0191163106],[1,36954,0.0226418806],[1,36969,0.0897130495],[1,36990,0.3475691758],[1,37020,0.0391875914],[1,37026,0.201908816],[1,37027,0.0269621416],[1,37049,0.0217168403],[1,37067,0.0308238935],[1,37072,0.114740584],[1,37075,0.0264302],[1,37080,0.189974242],[1,37084,0.0870876482],[1,37093,0.4504993085],[1,37115,0.024153496],[1,37126,0.0217907994],[1,37136,0.0379998669],[1,37151,0.0458738679],[1,37156,0.024655741],[1,37162,0.0348307086],[1,37187,0.0507297413],[1,37200,0.224736453],[1,37207,0.0227076122],[1,37224,0.023073504],[1,37233,0.0305672166],[1,37243,0.1834291551],[1,37266,0.054001968],[1,37274,0.0385260179],[1,37279,0.1935347052],[1,37299,0.0201240647],[1,37312,0.0586463505],[1,37321,0.1445187673],[1,37322,0.0340689981],[1,37341,0.0303670311],[1,37360,0.0845138275],[1,37370,0.0439049549],[1,37372,0.0239694081],[1,37374,0.0628936853],[1,37391,0.0580862402],[1,37394,0.0241151871],[1,37399,0.0385511156],[1,37403,0.070236956],[1,37426,0.0194216715],[1,37428,0.0626143225],[1,37447,0.0245789011],[1,37452,0.0244013269],[1,37471,0.024153496],[1,37488,0.024153496],[1,37491,0.0593177116],[1,37515,0.6663371702],[1,37517,0.0415435702],[1,37537,0.3836935752],[1,37562,0.0167096174],[1,37572,0.0512932111],[1,37589,0.1145576482],[1,37593,0.0219152491],[1,37598,0.1288169796],[1,37616,0.0631915904],[1,37635,0.0336836356],[1,37656,0.5541108562],[1,37671,0.0512604788],[1,37683,0.0247680909],[1,37685,0.3029772673],[1,37686,0.254839473],[1,37697,0.1135847651],[1,37710,0.024153496],[1,37713,0.0907717806],[1,37717,0.024153496],[1,37734,0.0275688591],[1,37743,0.2401438616],[1,37774,0.212291549],[1,37783,0.0265902146],[1,37835,0.1956698902],[1,37846,0.0225894042],[1,37872,0.2420292948],[1,37879,0.0232307186],[1,37890,0.0300087134],[1,37892,0.3576190217],[1,37982,0.024153496],[1,37985,0.0943288068],[1,37986,0.2003850847],[1,37994,0.1334324977],[1,38001,0.0404517793],[1,38012,0.024153496],[1,38042,0.1791224927],[1,38057,0.0238723833],[1,38061,0.027975463],[1,38083,0.1769540065],[1,38123,0.0302759058],[1,38169,0.4332959583],[1,38180,0.0457604054],[1,38188,0.024153496],[1,38189,0.0252420781],[1,38243,0.4280597033],[1,38251,0.0682550405],[1,38252,0.0880968281],[1,38258,0.0937730636],[1,38261,0.2596393937],[1,38283,0.0434750544],[1,38287,0.0258019365],[1,38312,0.0509812267],[1,38317,0.2741921428],[1,38319,0.0228371815],[1,38331,0.0222751563],[1,38352,0.0213197615],[1,38363,0.2733165693],[1,38378,0.0255868012],[1,38382,0.0364519442],[1,38399,0.2048575772],[1,38401,0.2210761207],[1,38409,0.0411355053],[1,38428,0.0350370369],[1,38442,0.4371332405],[1,38455,0.0272894497],[1,38462,0.0263998179],[1,38478,0.0242438485],[1,38490,0.0272538697],[1,38515,0.1315028975],[1,38535,0.0532337665],[1,38540,0.0554427332],[1,38546,0.0342063208],[1,38554,0.0300924976],[1,38559,0.0311687493],[1,38573,0.0977379161],[1,38590,0.0394466459],[1,38620,0.2224413448],[1,38653,0.0899566268],[1,38686,0.2283683193],[1,38712,0.0954767824],[1,38721,0.0245044227],[1,38731,0.357316778],[1,38744,0.0249268576],[1,38745,0.0249181045],[1,38748,0.0388864828],[1,38753,0.0278245793],[1,38789,0.0646047153],[1,38797,0.0674754805],[1,38818,0.0948870935],[1,38897,0.0239558076],[1,38922,0.0239558076],[1,38924,0.0173106924],[1,38931,0.0468905848],[1,38932,0.0699616315],[1,38934,0.0249268576],[1,38950,0.5666397656],[1,38952,0.0478294317],[1,38959,0.0239558076],[1,38973,0.0239558076],[1,38997,0.0232938621],[1,39031,0.0441167526],[1,39088,0.0403180056],[1,39106,0.0229596044],[1,39117,0.0283444557],[1,39126,0.0233156091],[1,39149,0.0476667452],[1,39169,0.0354610785],[1,39172,0.1088677071],[1,39188,0.0448491903],[1,39208,0.0203178107],[1,39211,0.3747679792],[1,39314,0.4230065464],[1,39349,0.036010072],[1,39354,0.0421659907],[1,39380,0.0375510156],[1,39383,0.1881383581],[1,39384,0.0744197463],[1,39401,0.0331060608],[1,39408,0.1702198465],[1,39409,0.0245044227],[1,39410,0.0255259868],[1,39416,0.0241011514],[1,39425,0.0662612563],[1,39445,0.0228267069],[1,39467,0.0287107894],[1,39490,0.0236154116],[1,39509,0.091772939],[1,39536,0.0245702396],[1,39574,0.1816753214],[1,39578,0.2021022314],[1,39587,0.0365754649],[1,39593,0.0239558076],[1,39615,0.0510684663],[1,39637,0.0908429559],[1,39649,0.0290057164],[1,39655,0.0226289913],[1,39657,0.0520129946],[1,39684,0.1663690218],[1,39706,0.0237039495],[1,39711,0.2172201905],[1,39724,0.0239558076],[1,39731,0.1558088412],[1,39776,0.0363884783],[1,39780,0.0347645346],[1,39782,0.048079351],[1,39785,0.3228328426],[1,39789,0.0231675931],[1,39791,0.0344647002],[1,39827,0.0550462399],[1,39846,0.0239558076],[1,39859,0.1289627821],[1,39875,0.4648552784],[1,39895,0.0642717913],[1,39896,0.1505920015],[1,39930,0.1276403242],[1,39931,0.0517354712],[1,39944,0.0388319498],[1,39960,0.1249692166],[1,39971,0.0433651217],[1,39981,0.0303560781],[1,39994,0.0270400206],[1,40015,0.0253358543],[1,40016,0.0389291982],[1,40077,0.0926501496],[1,40112,0.1017147708],[1,40122,0.047930513],[1,40126,0.0303946749],[1,40127,0.0176333615],[1,40135,0.0262723743],[1,40157,0.0238811728],[1,40163,0.0247915382],[1,40197,0.0242810658],[1,40216,0.0254067331],[1,40218,0.0886702041],[1,40223,0.0264685],[1,40227,0.0238811728],[1,40253,0.0247714193],[1,40257,0.0241110154],[1,40273,0.0255808675],[1,40279,0.0790341527],[1,40287,0.1072170726],[1,40329,0.0262319082],[1,40335,0.0233993514],[1,40376,0.0283444557],[1,40377,0.0468899159],[1,40378,0.025297355],[1,40410,0.033968204],[1,40483,0.1456278331],[1,40496,0.0311100628],[1,40517,0.0719467136],[1,40566,0.0993242421],[1,40592,0.0393808763],[1,40595,0.5444746381],[1,40599,0.0239558076],[1,40603,0.2313025837],[1,40609,0.2043994294],[1,40613,0.0335290903],[1,40616,0.0393879343],[1,40632,0.4220340481],[1,40663,0.2058007766],[1,40665,0.4808964055],[1,40669,0.1035133849],[1,40677,0.4598453425],[1,40678,0.1006792122],[1,40693,0.043489423],[1,40702,0.0637070887],[1,40708,0.0466371529],[1,40717,0.0262319082],[1,40738,0.0390007917],[1,40749,0.0245044227],[1,40751,0.0304072926],[1,40770,0.0339777421],[1,40786,0.0262319082],[1,40788,0.253257879],[1,40801,0.2087804764],[1,40803,0.0249268576],[1,40814,0.3934257557],[1,40842,0.187242163],[1,40851,0.0517272841],[1,40877,0.0329027676],[1,40884,0.2520813785],[1,40905,0.0239558076],[1,40943,0.0239558076],[1,40947,0.0883916844],[1,40970,0.0237289373],[1,40981,0.0516832916],[1,40988,0.0598046813],[1,40998,0.0345325983],[1,41010,0.0327840871],[1,41030,0.2214007001],[1,41041,0.022969968],[1,41052,0.0708675903],[1,41053,0.0338882706],[1,41087,0.0239175087],[1,41104,0.0233156091],[1,41120,0.2303602339],[1,41127,0.1443254984],[1,41129,0.0353811362],[1,41135,0.0250441012],[1,41137,0.0605448035],[1,41183,0.190969724],[1,41191,0.0231700919],[1,41203,0.0756502738],[1,41210,0.0215021564],[1,41246,0.0380634602],[1,41260,0.0372045153],[1,41280,0.0510684663],[1,41299,0.0502581879],[1,41308,0.0505150387],[1,41396,0.0339051186],[1,41427,0.0262319082],[1,41481,0.0341999067],[1,41483,0.0236747693],[1,41487,0.327143343],[1,41488,0.0239558076],[1,41515,0.0239558076],[1,41529,0.0230043481],[1,41553,0.03889466],[1,41577,0.0263918804],[1,41586,0.0555473599],[1,41589,0.0624656595],[1,41590,0.0477591503],[1,41611,0.029392342],[1,41612,0.0245702396],[1,41615,0.0387836088],[1,41621,0.0866233845],[1,41639,0.0361595391],[1,41672,0.2199955464],[1,41709,0.2884323116],[1,41724,0.2155280692],[1,41738,0.0237289373],[1,41746,0.1425814581],[1,41747,0.0245702396],[1,41755,0.350513328],[1,41761,0.015880683],[1,41765,0.0208258389],[1,41796,0.0426228737],[1,41800,0.0347786674],[1,41816,0.0245702396],[1,41831,0.0256038112],[1,41852,0.0239264883],[1,41853,0.0331002522],[1,41887,0.2429161207],[1,41905,0.4418289623],[1,41907,0.0499303202],[1,41924,0.0378311826],[1,41933,0.0184750788],[1,41957,0.2389614004],[1,41969,0.0505457134],[1,41978,0.2174462568],[1,41989,0.0570949815],[1,42022,0.0645104879],[1,42023,0.0239558076],[1,42025,0.156708599],[1,42035,0.0614314405],[1,42048,0.0370872025],[1,42049,0.0249268576],[1,42050,0.0446011022],[1,42057,0.2714444618],[1,42064,0.029386047],[1,42083,0.2920064079],[1,42096,0.0250357853],[1,42137,0.0593471645],[1,42169,0.5228312287],[1,42195,0.0325225626],[1,42213,0.0352127392],[1,42232,0.0754273556],[1,42244,0.1662936883],[1,42249,0.1027216569],[1,42260,0.045640762],[1,42267,0.0252555182],[1,42288,0.0330234082],[1,42296,0.050125268],[1,42301,0.1025667678],[1,42322,0.4955265929],[1,42369,0.0459335657],[1,42393,0.2372418496],[1,42395,0.104459892],[1,42396,0.1071981615],[1,42400,0.1182089606],[1,42403,0.351194798],[1,42409,0.0357606227],[1,42437,0.0333200504],[1,42473,0.0348238841],[1,42477,0.0563238231],[1,42478,0.0342230926],[1,42479,0.021122824],[1,42485,0.2356578875],[1,42511,0.0203148376],[1,42527,0.0236563222],[1,42549,0.0223824582],[1,42588,0.0873972543],[1,42606,0.1020467392],[1,42607,0.0314361374],[1,42609,0.0253887329],[1,42610,0.0216241103],[1,42613,0.1906247276],[1,42633,0.0621305653],[1,42653,0.0260984431],[1,42721,0.0228866993],[1,42724,0.0350499942],[1,42732,0.0367122816],[1,42745,0.1362728627],[1,42764,0.0511769256],[1,42779,0.0213091301],[1,42792,0.0389533225],[1,42806,0.0239558076],[1,42863,0.4723258234],[1,42928,0.0531364313],[1,42940,0.039411622],[1,42961,0.2026498574],[1,42970,0.0249268576],[1,42973,0.0211111578],[1,42976,0.2062511982],[1,43001,0.1094464802],[1,43002,0.0471652844],[1,43012,0.0903280227],[1,43039,0.0305675935],[1,43040,0.0537963696],[1,43041,0.1053034387],[1,43042,0.0537963696],[1,43046,0.024357674],[1,43140,0.0383388121],[1,43179,0.3739552321],[1,43189,0.0510883305],[1,43228,0.3533980895],[1,43272,0.0239558076],[1,43318,0.0615102791],[1,43319,0.0249807819],[1,43379,0.0470719457],[1,43420,0.025575714],[1,43437,0.0563437411],[1,43470,0.0507624883],[1,43489,0.0239558076],[1,43500,0.1095817851],[1,43594,0.0211111578],[1,43644,0.1191152825],[1,43652,0.4358160645],[1,43691,0.0400983042],[1,43702,0.0246588746],[1,43724,0.1496011708],[1,43774,0.1031570525],[1,43793,0.6366236674],[1,43797,0.5733922237],[1,43803,0.0263918804],[1,43814,0.0244204006],[1,43836,0.0245702396],[1,43844,0.0239558076],[1,43867,0.0210185816],[1,43872,0.0210668438],[1,43878,0.0281198055],[1,43884,0.0311428746],[1,43893,0.0263170018],[1,43896,0.0511232372],[1,43922,0.0303915212],[1,43925,0.021122824],[1,43989,0.419770801],[1,44049,0.0362150512],[1,44059,0.1167266033],[1,44072,0.026548916],[1,44073,0.0351456406],[1,44080,0.0614431787],[1,44088,0.0213402867],[1,44108,0.0906348938],[1,44120,0.0237689914],[1,44150,0.0256984923],[1,44184,0.0238736198],[1,44186,0.0264391203],[1,44195,0.0262072537],[1,44204,0.0267478295],[1,44206,0.2639476716],[1,44225,0.0333836251],[1,44244,0.1001908216],[1,44252,0.024242155],[1,44263,0.0365678965],[1,44278,0.0279069336],[1,44308,0.1722852773],[1,44318,0.0339507882],[1,44323,0.0193242885],[1,44345,0.0488279685],[1,44381,0.0205515502],[1,44388,0.036111108],[1,44394,0.0266238178],[1,44397,0.0347402218],[1,44419,0.0290686056],[1,44427,0.2591582914],[1,44429,0.0261770531],[1,44457,0.1261439522],[1,44466,0.0266238178],[1,44474,0.0854621019],[1,44486,0.0433931335],[1,44497,0.0441665315],[1,44515,0.0300778113],[1,44523,0.021467488],[1,44525,0.0266238178],[1,44534,0.0487198061],[1,44551,0.046855995],[1,44570,0.0654632541],[1,44576,0.0538334712],[1,44579,0.0626355606],[1,44580,0.0279248138],[1,44587,0.0767846333],[1,44590,0.0284047528],[1,44639,0.0270999038],[1,44653,0.0274423473],[1,44697,0.0376556436],[1,44700,0.0266994185],[1,44720,0.0237806993],[1,44727,0.070315811],[1,44730,0.0387261547],[1,44732,0.0202787533],[1,44746,0.047842756],[1,44760,0.2683051171],[1,44764,0.0222158735],[1,44767,0.042879016],[1,44777,0.150167194],[1,44793,0.1274289092],[1,44798,0.2338382866],[1,44809,0.031207572],[1,44858,0.1776690748],[1,44862,0.0581693265],[1,44914,0.0282038471],[1,44920,0.0320471942],[1,44921,0.0275711218],[1,44929,0.0255420939],[1,44948,0.0453174024],[1,44987,0.0290241046],[1,45025,0.0944700217],[1,45065,0.1306800026],[1,45073,0.0946649207],[1,45100,0.043369009],[1,45128,0.1190160279],[1,45152,0.0372745439],[1,45162,0.0618442583],[1,45181,0.1536933241],[1,45183,0.3124764115],[1,45199,0.1857478114],[1,45203,0.0597805285],[1,45242,0.030653796],[1,45273,0.0516102018],[1,45278,0.0654290109],[1,45316,0.4037854322],[1,45318,0.0288371905],[1,45319,0.4043679211],[1,45357,0.0282038471],[1,45359,0.0282038471],[1,45373,0.0608086788],[1,45382,0.0570759959],[1,45395,0.0202962768],[1,45406,0.0287462007],[1,45422,0.0259641806],[1,45427,0.5980330156],[1,45472,0.0275980387],[1,45476,0.0272765681],[1,45481,0.0257539614],[1,45482,0.0549452114],[1,45523,0.0287619638],[1,45543,0.0376522685],[1,45601,0.0299409239],[1,45606,0.0577787599],[1,45645,0.4653643637],[1,45675,0.1289197438],[1,45724,0.0440760862],[1,45737,0.048406649],[1,45740,0.0262212631],[1,45746,0.0331270501],[1,45748,0.0376522685],[1,45757,0.0269342004],[1,45761,0.0413976022],[1,45771,0.0653167961],[1,45807,0.5566103209],[1,45842,0.0524947125],[1,45875,0.2686143038],[1,45877,0.0582143836],[1,45904,0.0998205395],[1,45910,0.04048433],[1,45911,0.0201193935],[1,45916,0.1068741998],[1,45917,0.0957637756],[1,45953,0.0279756845],[1,45965,0.0234261062],[1,45967,0.0291515316],[1,45992,0.0292983397],[1,46046,0.0199576031],[1,46072,0.1773446182],[1,46076,0.1197864954],[1,46089,0.0468021939],[1,46107,0.1050383629],[1,46185,0.0282038471],[1,46189,0.033885988],[1,46203,0.2089229942],[1,46212,0.4426767126],[1,46217,0.0742345821],[1,46219,0.0912096854],[1,46252,0.033059794],[1,46257,0.0730729682],[1,46265,0.5963381354],[1,46269,0.3268274526],[1,46277,0.0491379603],[1,46285,0.0362859794],[1,46299,0.2758199894],[1,46326,0.0326174933],[1,46331,0.0258952956],[1,46339,0.0282038471],[1,46401,0.0494237966],[1,46455,0.0364920091],[1,46462,0.0370838686],[1,46470,0.1373772577],[1,46506,0.0253547266],[1,46507,0.0292983397],[1,46529,0.0237258473],[1,46538,0.1542000673],[1,46544,0.0915386336],[1,46584,0.0336599696],[1,46586,0.1116178178],[1,46594,0.2325045188],[1,46597,0.055580019],[1,46603,0.123171432],[1,46610,0.0256194824],[1,46626,0.0477813968],[1,46636,0.0537261895],[1,46650,0.0946618585],[1,46656,0.0541145291],[1,46659,0.0287555871],[1,46693,0.1484171037],[1,46703,0.0472317063],[1,46723,0.1238486396],[1,46733,0.0257903484],[1,46780,0.0282038471],[1,46785,0.0579454569],[1,46812,0.0267478295],[1,46818,0.4013261567],[1,46825,0.0578503393],[1,46836,0.0357829765],[1,46865,0.0258171565],[1,46917,0.0365111195],[1,46940,0.2065159751],[1,46994,0.0363373917],[1,47010,0.0265046786],[1,47056,0.0268803855],[1,47060,0.0513846985],[1,47067,0.0918837601],[1,47127,0.0334011238],[1,47133,0.052348944],[1,47151,0.3863729536],[1,47229,0.0365797274],[1,47241,0.030653796],[1,47252,0.0258885069],[1,47312,0.1605834866],[1,47358,0.0495265709],[1,47372,0.0527052445],[1,47431,0.1550606601],[1,47501,0.0366548052],[1,47541,0.0312715016],[1,47559,0.0479880715],[1,47579,0.209412999],[1,47587,0.0342425061],[1,47614,0.023242659],[1,47632,0.0303426037],[1,47639,0.0237113242],[1,47652,0.0361740233],[1,47656,0.0294418991],[1,47670,0.6885438731],[1,47695,0.0783073025],[1,47729,0.0317297056],[1,47742,0.0441528661],[1,47744,0.0257562312],[1,47745,0.0311851961],[1,47759,0.0353575536],[1,47764,0.0362318327],[1,47833,0.0327552808],[1,47851,0.0289348152],[1,47859,0.039501905],[1,47872,0.0697201937],[1,47886,0.0320087141],[1,47910,0.026109981],[1,47926,0.0479469843],[1,47953,0.1033542703],[1,48002,0.1773544429],[1,48006,0.0899807974],[1,48058,0.0437394729],[1,48085,0.0299619564],[1,48104,0.0404871976],[1,48107,0.0419374345],[1,48124,0.0426826323],[1,48149,0.0338438816],[1,48155,0.0399061565],[1,48180,0.0368212291],[1,48188,0.0460838322],[1,48189,0.533690008],[1,48210,0.0421551634],[1,48211,0.1196330253],[1,48250,0.0392189185],[1,48272,0.0518729721],[1,48286,0.0405864383],[1,48302,0.0449520244],[1,48304,0.0529693258],[1,48307,0.519391009],[1,48308,0.1057233983],[1,48311,0.0367972419],[1,48318,0.0613150237],[1,48372,0.0269527063],[1,48377,0.090503041],[1,48383,0.0527415844],[1,48388,0.0703878749],[1,48390,0.4584897265],[1,48399,0.0429818654],[1,48415,0.0345376073],[1,48417,0.0322840394],[1,48452,0.0296959562],[1,48460,0.0462376518],[1,48488,0.0613059563],[1,48510,0.3472082463],[1,48549,0.0386123964],[1,48570,0.1779549526],[1,48581,0.0349787678],[1,48619,0.0635337109],[1,48630,0.0316654211],[1,48636,0.0328991925],[1,48652,0.0386884491],[1,48677,0.0377125554],[1,48681,0.048546173],[1,48684,0.0576083356],[1,48690,0.0428051856],[1,48725,0.1104008878],[1,48736,0.0290688854],[1,48760,0.0765853661],[1,48787,0.3293779252],[1,48801,0.320043021],[1,48828,0.0309481026],[1,48837,0.1028502254],[1,48861,0.0382344745],[1,48867,0.0681182358],[1,48886,0.510560339],[1,48926,0.0301669312],[1,48969,0.0337737051],[1,49010,0.0480808747],[1,49024,0.053719651],[1,49052,0.0331043728],[1,49055,0.0307107043],[1,49072,0.0475874188],[1,49088,0.0301522647],[1,49164,0.3825690631],[1,49189,0.2173450563],[1,49195,0.348500682],[1,49196,0.0358505813],[1,49200,0.0473302793],[1,49240,0.1738348302],[1,49258,0.0394771914],[1,49263,0.1020322463],[1,49288,0.0282675846],[1,49290,0.0309893356],[1,49313,0.0303714768],[1,49322,0.7501479364],[1,49333,0.2179937991],[1,49364,0.0307107043],[1,49370,0.0607355002],[1,49379,0.0302819128],[1,49391,0.0324619879],[1,49415,0.1742733856],[1,49420,0.0289453456],[1,49421,0.1016026212],[1,49426,0.0494879249],[1,49430,0.0301669312],[1,49437,0.0516783967],[1,49450,0.0325025514],[1,49530,0.1920186106],[1,49535,0.1390373126],[1,49574,0.0306077271],[1,49576,0.0318528079],[1,49586,0.0301669312],[1,49590,0.028485012],[1,49601,0.0300477466],[1,49610,0.0715384886],[1,49626,0.0323907656],[1,49629,0.0291771687],[1,49656,0.1251670975],[1,49694,0.0646358162],[1,49722,0.2928163057],[1,49732,0.0268710654],[1,49739,0.0376369943],[1,49750,0.0301373678],[1,49837,0.1296538544],[1,49866,0.0313173173],[1,49867,0.0326232925],[1,49870,0.1961381435],[1,49874,0.0506527527],[1,49915,0.1976251786],[1,49920,0.1041395642],[1,49923,0.0307201153],[1,49939,0.3215717244],[1,49944,0.1394203014],[1,49988,0.3138616965],[1,50017,0.0287282747],[1,50023,0.0844217943],[1,50037,0.0229716225],[1,50048,0.4663263175],[1,50119,0.0338262406],[1,50156,0.0344985845],[1,50181,0.0265905117],[1,50235,0.266724551],[1,50250,0.0289627095],[1,50264,0.0359468496],[1,50340,0.0480740082],[1,50347,0.1412158497],[1,50353,0.0233056779],[1,50373,0.0624294411],[1,50399,0.0324619879],[1,50430,0.4018503023],[1,50464,0.1081516388],[1,50495,0.0752854497],[1,50498,0.0335691101],[1,50507,0.0273912253],[1,50514,0.1690875291],[1,50529,0.1928233556],[1,50554,0.0301669312],[1,50583,0.0596854198],[1,50592,0.2713642724],[1,50656,0.0272706154],[1,50661,0.0768990539],[1,50664,0.0468675003],[1,50692,0.0280012241],[1,50693,0.0650661957],[1,50697,0.0312462104],[1,50734,0.0565543398],[1,50738,0.0567976146],[1,50765,0.0385229821],[1,50773,0.0526949427],[1,50776,0.0734153189],[1,50816,0.0277106333],[1,50859,0.0299381715],[1,50868,0.1446094654],[1,50901,0.0326232925],[1,50930,0.1464237061],[1,50935,0.0289433464],[1,50945,0.0697382036],[1,50958,0.0666372564],[1,50966,0.0292367152],[1,50972,0.0272706154],[1,50985,0.0339987502],[1,50992,0.0315676116],[1,51004,0.0621473834],[1,51010,0.2893006909],[1,51038,0.0307107043],[1,51049,0.1130171065],[1,51073,0.0698433871],[1,51099,0.0287023441],[1,51119,0.057283561],[1,51122,0.0383269173],[1,51126,0.0419817128],[1,51129,0.025252094],[1,51167,0.0237936998],[1,51195,0.1188667031],[1,51213,0.0418464269],[1,51231,0.0729935818],[1,51236,0.0673343864],[1,51247,0.0593584515],[1,51293,0.0729883622],[1,51300,0.0299931956],[1,51309,0.1895270374],[1,51314,0.062542691],[1,51328,0.2188905424],[1,51357,0.0318286599],[1,51358,0.0485314955],[1,51365,0.0553791637],[1,51372,0.0301747738],[1,51378,0.1536512007],[1,51387,0.0245823288],[1,51454,0.0252057308],[1,51463,0.029631388],[1,51467,0.1419167505],[1,51509,0.0304677133],[1,51522,0.0655157842],[1,51526,0.0352720323],[1,51557,0.1071190551],[1,51627,0.0277376142],[1,51645,0.0739855128],[1,51658,0.044660093],[1,51721,0.029631388],[1,51725,0.042963227],[1,51769,0.0432495095],[1,51792,0.0298357879],[1,51810,0.0290209956],[1,51813,0.029972576],[1,51824,0.140492046],[1,51837,0.2551152391],[1,51847,0.0909065436],[1,51863,0.0450861159],[1,51900,0.0599315438],[1,52000,0.0504739374],[1,52021,0.0345311537],[1,52043,0.031371771],[1,52059,0.0257831098],[1,52066,0.0364285315],[1,52073,0.1689738292],[1,52107,0.0512476123],[1,52108,0.1573969359],[1,52113,0.029631388],[1,52118,0.0997233679],[1,52149,0.0425515549],[1,52193,0.2331482759],[1,52197,0.0431937322],[1,52201,0.029631388],[1,52216,0.0724180695],[1,52217,0.0214948097],[1,52222,0.4219178932],[1,52228,0.0311460685],[1,52253,0.0270877816],[1,52287,0.0291432942],[1,52311,0.0704109507],[1,52312,0.1938548169],[1,52375,0.029631388],[1,52384,0.029631388],[1,52385,0.2402530973],[1,52424,0.029631388],[1,52426,0.0316117904],[1,52471,0.037644815],[1,52484,0.0304532067],[1,52530,0.0928503481],[1,52606,0.0404467217],[1,52609,0.2143974619],[1,52620,0.0516217333],[1,52660,0.0324619879],[1,52663,0.031956297],[1,52669,0.0312243762],[1,52674,0.031371771],[1,52685,0.0343541507],[1,52721,0.0649948748],[1,52747,0.0436632004],[1,52748,0.0395113055],[1,52756,0.0416924413],[1,52759,0.0701244147],[1,52767,0.029631388],[1,52850,0.1083700441],[1,52894,0.040197234],[1,52922,0.0260724749],[1,52972,0.0333111643],[1,52979,0.052129414],[1,52984,0.0469847794],[1,52993,0.1103655338],[1,53044,0.4696060994],[1,53150,0.0371963717],[1,53165,0.0274857871],[1,53168,0.0530416531],[1,53184,0.0502422118],[1,53227,0.0492217376],[1,53230,0.4035875022],[1,53235,0.0328183008],[1,53238,0.1651231912],[1,53250,0.027475839],[1,53269,0.0324619879],[1,53292,0.0272147516],[1,53298,0.0366274592],[1,53300,0.0304532067],[1,53314,0.0301841781],[1,53321,0.0269410261],[1,53328,0.1814933379],[1,53337,0.0710554838],[1,53343,0.0261815345],[1,53360,0.0273481829],[1,53405,0.032386313],[1,53426,0.0407805321],[1,53435,0.0515316578],[1,53464,0.0355807842],[1,53489,0.0787351795],[1,53528,0.0285744024],[1,53553,0.0531521986],[1,53586,0.0254773864],[1,53629,0.1004386219],[1,53636,0.1035424108],[1,53648,0.029631388],[1,53653,0.0316117904],[1,53659,0.0560844845],[1,53669,0.039180157],[1,53675,0.029318275],[1,53723,0.2081305723],[1,53733,0.0570392859],[1,53742,0.0304532067],[1,53746,0.0237918296],[1,53762,0.0202548065],[1,53810,0.0539234232],[1,53812,0.039054278],[1,53881,0.0840378165],[1,53898,0.024751441],[1,53912,0.0311772559],[1,53926,0.0289569996],[1,53943,0.0333819304],[1,53945,0.0725589365],[1,53952,0.029564003],[1,53960,0.0347481183],[1,53980,0.4005807685],[1,53984,0.0266262954],[1,53990,0.0300068364],[1,53995,0.0358570081],[1,54009,0.0301736122],[1,54014,0.0387724294],[1,54021,0.0232183883],[1,54025,0.1185477172],[1,54059,0.1690355971],[1,54062,0.0373647681],[1,54073,0.0946774535],[1,54077,0.0288818643],[1,54122,0.0282022886],[1,54124,0.0511869601],[1,54139,0.1026951575],[1,54190,0.0278125044],[1,54199,0.0235440993],[1,54211,0.0275531508],[1,54214,0.302355533],[1,54215,0.02563323],[1,54229,0.0310724233],[1,54231,0.0361463394],[1,54271,0.0467620645],[1,54309,0.0289569996],[1,54323,0.1810240824],[1,54380,0.2148032179],[1,54396,0.0224058856],[1,54399,0.0235156726],[1,54422,0.0198127595],[1,54440,0.1945410234],[1,54479,0.0549632756],[1,54480,0.2635027777],[1,54481,0.0608690691],[1,54486,0.0521512633],[1,54487,0.145418105],[1,54497,0.0289569996],[1,54518,0.0273937891],[1,54530,0.0226831104],[1,54542,0.0289569996],[1,54547,0.1161214036],[1,54557,0.1322304951],[1,54578,0.1516295159],[1,54589,0.1927532841],[1,54609,0.0289479627],[1,54631,0.0396353173],[1,54656,0.0321410524],[1,54706,0.0528113171],[1,54707,0.3539798906],[1,54716,0.1209913652],[1,54727,0.0489940775],[1,54730,0.1296844574],[1,54733,0.0494826472],[1,54751,0.0241337598],[1,54764,0.0309524315],[1,54780,0.0870464335],[1,54789,0.4856469057],[1,54812,0.0413844737],[1,54827,0.0431932571],[1,54851,0.1206846361],[1,54853,0.0346714468],[1,54882,0.69776337],[1,54886,0.0812431447],[1,54891,0.0558311265],[1,54899,0.0945422423],[1,54908,0.0468231768],[1,54930,0.0249599914],[1,54935,0.0349893207],[1,54938,0.0379432435],[1,54956,0.0446977866],[1,54959,0.0269830127],[1,54989,0.0314373359],[1,55032,0.0531158411],[1,55047,0.1217353165],[1,55076,0.1424208595],[1,55077,0.1760961934],[1,55115,0.1283242943],[1,55121,0.0443984296],[1,55137,0.3428222355],[1,55161,0.1888881417],[1,55174,0.0303548226],[1,55203,0.0342489857],[1,55204,0.0303949492],[1,55229,0.0676142376],[1,55248,0.0267313247],[1,55249,0.0355763597],[1,55265,0.1288822737],[1,55276,0.058375527],[1,55335,0.0301875329],[1,55353,0.2577450491],[1,55384,0.1483327117],[1,55401,0.0605693219],[1,55408,0.1159385928],[1,55429,0.0469474142],[1,55442,0.0606755539],[1,55459,0.0313464726],[1,55490,0.1515523673],[1,55495,0.0468231768],[1,55501,0.2134438432],[1,55531,0.0493641733],[1,55604,0.0263625029],[1,55615,0.0513604727],[1,55643,0.0323284688],[1,55644,0.0504473519],[1,55665,0.0313672043],[1,55678,0.0389140446],[1,55729,0.0316774884],[1,55751,0.2243099769],[1,55782,0.0417636172],[1,55809,0.0272483181],[1,55833,0.1822466339],[1,55847,0.1385450172],[1,55875,0.028168805],[1,55917,0.0979807406],[1,55955,0.0352968042],[1,55997,0.0431619293],[1,56004,0.1584478942],[1,56027,0.0456832614],[1,56051,0.0566726437],[1,56067,0.0322634719],[1,56136,0.1129398077],[1,56166,0.0693984633],[1,56222,0.0652799718],[1,56230,0.0528129371],[1,56234,0.1414533391],[1,56283,0.2535488481],[1,56292,0.0514155074],[1,56308,0.3534623193],[1,56317,0.2652179757],[1,56330,0.1554272287],[1,56365,0.0272652327],[1,56370,0.0482269115],[1,56382,0.2274241122],[1,56384,0.0305210285],[1,56406,0.0305210285],[1,56424,0.2016530816],[1,56444,0.0501373607],[1,56449,0.2337373622],[1,56454,0.1093678209],[1,56460,0.0305210285],[1,56474,0.0313438201],[1,56482,0.2164936389],[1,56486,0.0255958514],[1,56498,0.0413663142],[1,56504,0.0355135722],[1,56528,0.029374149],[1,56552,0.029911093],[1,56557,0.0305210285],[1,56576,0.1443776013],[1,56589,0.1376990868],[1,56619,0.0305210285],[1,56665,0.0392694587],[1,56737,0.1177685857],[1,56749,0.4986545019],[1,56754,0.1534156462],[1,56755,0.0262815903],[1,56762,0.0388149222],[1,56773,0.0535890451],[1,56779,0.0294703584],[1,56818,0.0262238602],[1,56837,0.0461512234],[1,56851,0.2248577457],[1,56876,0.0305119728],[1,56887,0.0408307457],[1,56888,0.3980926975],[1,56915,0.0326082765],[1,56919,0.0618843305],[1,56949,0.0398678784],[1,56953,0.0849011895],[1,56960,0.028185469],[1,56972,0.0252509532],[1,56983,0.252031298],[1,56994,0.0305210285],[1,56996,0.0500579873],[1,57021,0.0290347622],[1,57062,0.0779121072],[1,57063,0.028185469],[1,57087,0.0305210285],[1,57101,0.0412704353],[1,57133,0.0287033058],[1,57142,0.1683145266],[1,57143,0.1322885526],[1,57155,0.0312349576],[1,57163,0.0295908848],[1,57198,0.0618614849],[1,57205,0.0602832855],[1,57221,0.1173399603],[1,57228,0.0391395702],[1,57240,0.04300814],[1,57277,0.243702401],[1,57306,0.3302417824],[1,57323,0.2164098695],[1,57329,0.0252010076],[1,57359,0.0305210285],[1,57453,0.0639829356],[1,57462,0.0402642523],[1,57463,0.0506029268],[1,57477,0.0414797004],[1,57491,0.2207317313],[1,57548,0.0323200447],[1,57557,0.0316774884],[1,57586,0.0373409567],[1,57590,0.0276631058],[1,57597,0.0730831457],[1,57603,0.0322543742],[1,57653,0.0763841395],[1,57659,0.0334858113],[1,57682,0.0493088945],[1,57696,0.1116675164],[1,57709,0.2706180787],[1,57710,0.030503572],[1,57716,0.0397383987],[1,57726,0.0269530669],[1,57785,0.0402148088],[1,57791,0.0399169262],[1,57798,0.0254576689],[1,57849,0.0249856364],[1,57893,0.029506818],[1,57908,0.0366913335],[1,57916,0.4954355718],[1,57926,0.0523710193],[1,57949,0.0508885079],[1,57986,0.1356911463],[1,58103,0.03002546],[1,58109,0.0332932526],[1,58175,0.188215651],[1,58184,0.1669203452],[1,58185,0.221740944],[1,58191,0.0832655955],[1,58202,0.0766649969],[1,58234,0.0277368526],[1,58253,0.029679445],[1,58256,0.0337032332],[1,58277,0.1086653207],[1,58287,0.0332164773],[1,58300,0.093533398],[1,58309,0.0295806204],[1,58336,0.0292073882],[1,58363,0.123279661],[1,58378,0.0328462081],[1,58407,0.1589327927],[1,58408,0.034533781],[1,58417,0.0354126938],[1,58449,0.0830022805],[1,58450,0.0407574185],[1,58456,0.0494056489],[1,58469,0.1349651356],[1,58519,0.1408758208],[1,58543,0.0277680125],[1,58544,0.1548453283],[1,58562,0.0531772411],[1,58575,0.0808711986],[1,58606,0.302840324],[1,58657,0.0307135493],[1,58673,0.4626242903],[1,58700,0.0421961619],[1,58714,0.0974001433],[1,58726,0.0516572952],[1,58736,0.0915834612],[1,58738,0.1068793132],[1,58754,0.0418834143],[1,58766,0.0386673967],[1,58904,0.1213868594],[1,58905,0.0320711995],[1,58914,0.0334227915],[1,58918,0.0691762264],[1,58919,0.0568695759],[1,58938,0.2665060598],[1,58946,0.11705119],[1,58974,0.3070964468],[1,59004,0.1684778457],[1,59135,0.0507527989],[1,59137,0.0332164773],[1,59158,0.0513645171],[1,59161,0.0540586151],[1,59179,0.0350582313],[1,59248,0.0275236736],[1,59266,0.0320711995],[1,59286,0.2760758171],[1,59307,0.0561746056],[1,59383,0.034533781],[1,59409,0.031978947],[1,59414,0.0277651771],[1,59423,0.3271226714],[1,59456,0.5615354622],[1,59469,0.0350456968],[1,59474,0.0325814486],[1,59480,0.2295684428],[1,59508,0.0377144075],[1,59542,0.2546360565],[1,59563,0.0417098826],[1,59578,0.0355563772],[1,59581,0.0302394928],[1,59582,0.039469211],[1,59604,0.0331269677],[1,59608,0.0957746651],[1,59640,0.0306908909],[1,59654,0.0331364089],[1,59708,0.5459387179],[1,59717,0.0321138729],[1,59720,0.4247079065],[1,59757,0.0321180065],[1,59798,0.0859812149],[1,59800,0.0482007138],[1,59826,0.0416641798],[1,59834,0.0676080822],[1,59864,0.0350456968],[1,59865,0.0582668525],[1,59935,0.0585022856],[1,59972,0.0337410757],[1,60016,0.0290037156],[1,60039,0.0446951529],[1,60049,0.0422676084],[1,60069,0.0404371964],[1,60076,0.0584099278],[1,60117,0.0513393968],[1,60120,0.0305888722],[1,60124,0.0350456968],[1,60126,0.042001151],[1,60135,0.0583012195],[1,60141,0.0303450291],[1,60142,0.0366544257],[1,60170,0.0742327309],[1,60244,0.0453399188],[1,60257,0.2014434355],[1,60258,0.0335637297],[1,60260,0.0945453524],[1,60275,0.0311710803],[1,60292,0.0366908386],[1,60308,0.0521640542],[1,60322,0.0418295719],[1,60345,0.0796238006],[1,60352,0.048276427],[1,60363,0.0339049807],[1,60389,0.053425261],[1,60390,0.0609167693],[1,60395,0.03212512],[1,60436,0.0319742745],[1,60452,0.0401615684],[1,60456,0.359943261],[1,60458,0.0390332568],[1,60463,0.0337275015],[1,60472,0.0325814486],[1,60475,0.0538434905],[1,60477,0.0277005896],[1,60623,0.1458428088],[1,60630,0.0438852473],[1,60636,0.0973528851],[1,60639,0.0551740823],[1,60661,0.0590845332],[1,60679,0.0326354055],[1,60697,0.0325903093],[1,60739,0.0513425792],[1,60764,0.0280176643],[1,60782,0.0335918923],[1,60914,0.0331364089],[1,60940,0.0527755887],[1,60969,0.0337952336],[1,60988,0.2179435762],[1,60997,0.0314771039],[1,61011,0.068282442],[1,61025,0.2294945984],[1,61065,0.0325814486],[1,61084,0.0325814486],[1,61113,0.0325814486],[1,61118,0.0689386449],[1,61139,0.0291094541],[1,61178,0.4667844507],[1,61217,0.0514212873],[1,61247,0.1265889915],[1,61260,0.0587729039],[1,61284,0.0323341973],[1,61312,0.1352740926],[1,61313,0.1122938162],[1,61337,0.0524922517],[1,61376,0.0295855704],[1,61383,0.0445576579],[1,61398,0.0295855704],[1,61447,0.0994525757],[1,61458,0.0777384612],[1,61475,0.0371567412],[1,61478,0.3637707392],[1,61489,0.0304557058],[1,61495,0.0329074271],[1,61525,0.0391319688],[1,61543,0.0539165328],[1,61549,0.0314746895],[1,61574,0.0937060569],[1,61600,0.0334696754],[1,61606,0.0499601531],[1,61659,0.0960134821],[1,61667,0.0280996204],[1,61668,0.0605019125],[1,61672,0.0304777529],[1,61723,0.0837194033],[1,61760,0.0230289717],[1,61778,0.0408025959],[1,61792,0.0295855704],[1,61795,0.0555109406],[1,61805,0.0298352059],[1,61807,0.0312425457],[1,61823,0.192522257],[1,61856,0.0337884499],[1,61858,0.1300892477],[1,61864,0.0295855704],[1,61866,0.0320400327],[1,61930,0.1563592776],[1,61970,0.110373241],[1,61980,0.0444739684],[1,61999,0.0673753026],[1,62023,0.0257681467],[1,62024,0.6934745952],[1,62027,0.2410708985],[1,62031,0.0717997882],[1,62040,0.0346405936],[1,62080,0.1050827519],[1,62106,0.0279363047],[1,62121,0.2912320198],[1,62123,0.0266841115],[1,62136,0.0405692321],[1,62147,0.0297664371],[1,62155,0.027240975],[1,62188,0.0339319004],[1,62192,0.0363543799],[1,62237,0.0295855704],[1,62246,0.0713761734],[1,62248,0.0396348498],[1,62257,0.0508401116],[1,62292,0.0673885374],[1,62316,0.0364968362],[1,62319,0.0296710581],[1,62352,0.0898569282],[1,62354,0.1620635223],[1,62359,0.499684067],[1,62366,0.0516524638],[1,62372,0.0349283286],[1,62406,0.0512758872],[1,62430,0.0438875663],[1,62448,0.0283045368],[1,62452,0.0378040748],[1,62476,0.0376512981],[1,62477,0.0244783229],[1,62490,0.0242148045],[1,62494,0.0459452378],[1,62499,0.0512598325],[1,62516,0.0509709534],[1,62536,0.0551689732],[1,62587,0.0323565023],[1,62595,0.0645494357],[1,62624,0.0247877083],[1,62631,0.0907999887],[1,62661,0.2093216335],[1,62708,0.1075361925],[1,62717,0.0266841115],[1,62730,0.0716099075],[1,62731,0.0743174617],[1,62786,0.0352624634],[1,62808,0.0416585994],[1,62813,0.1229216227],[1,62843,0.04289456],[1,62896,0.0291094541],[1,62923,0.0737895438],[1,62929,0.0327861733],[1,62975,0.0295855704],[1,62979,0.0295855704],[1,63018,0.0285486431],[1,63020,0.0582449401],[1,63101,0.0595819837],[1,63110,0.1030944147],[1,63161,0.0340494366],[1,63185,0.3908110642],[1,63186,0.0289547264],[1,63212,0.1943057991],[1,63237,0.0285608244],[1,63259,0.0295855704],[1,63275,0.0298478818],[1,63283,0.0321726865],[1,63287,0.3774493101],[1,63288,0.0320400327],[1,63294,0.0331250454],[1,63345,0.0298623634],[1,63363,0.0803520861],[1,63380,0.5646313065],[1,63406,0.1336041735],[1,63411,0.0365863014],[1,63459,0.0379681126],[1,63467,0.1026659959],[1,63475,0.0301289231],[1,63501,0.0295855704],[1,63511,0.2152908629],[1,63545,0.0271011487],[1,63591,0.0303757358],[1,63666,0.0228146343],[1,63667,0.1148913293],[1,63682,0.0304387113],[1,63726,0.0257681467],[1,63749,0.2498392273],[1,63774,0.0273670427],[1,63803,0.0321525731],[1,63819,0.029309009],[1,63869,0.1055823491],[1,63895,0.0302956915],[1,63898,0.0551689732],[1,63930,0.0284823586],[1,63942,0.2201230092],[1,63955,0.0957869031],[1,63978,0.0307765287],[1,63984,0.0449118159],[1,64008,0.0304297849],[1,64027,0.0473405171],[1,64033,0.0714605958],[1,64088,0.0290319645],[1,64120,0.0703707708],[1,64122,0.0292405455],[1,64145,0.0420486524],[1,64149,0.0295855704],[1,64152,0.1344495017],[1,64168,0.2204302281],[1,64180,0.0253492157],[1,64230,0.0280889408],[1,64289,0.0309332173],[1,64295,0.1524013823],[1,64298,0.0295418659],[1,64323,0.0517463594],[1,64364,0.0320400327],[1,64388,0.0861715001],[1,64391,0.0549696147],[1,64394,0.2198150285],[1,64415,0.0288787252],[1,64497,0.0406449289],[1,64508,0.0291094541],[1,64528,0.0505737911],[1,64530,0.022988075],[1,64536,0.0282214822],[1,64565,0.0255389405],[1,64635,0.04115153],[1,64645,0.0492110947],[1,64659,0.0566982604],[1,64662,0.036182554],[1,64667,0.542205032],[1,64688,0.0348010999],[1,64692,0.029676694],[1,64747,0.2289223431],[1,64796,0.0296524626],[1,64800,0.0339742426],[1,64803,0.0316347926],[1,64815,0.1357929251],[1,64885,0.03401734],[1,64904,0.0451609177],[1,64968,0.0291986433],[1,65018,0.0258230136],[1,65020,0.0295855704],[1,65025,0.0315240851],[1,65042,0.0279777476],[1,65043,0.0397180184],[1,65064,0.0587658459],[1,65070,0.0320725907],[1,65090,0.0291094541],[1,65100,0.0568879371],[1,65103,0.0309332173],[1,65146,0.2721076284],[1,65154,0.0404701762],[1,65176,0.0384395691],[1,65213,0.0259520389],[1,65220,0.0296524626],[1,65232,0.0611816119],[1,65274,0.046788897],[1,65279,0.057603748],[1,65296,0.1991681544],[1,65303,0.2480227994],[1,65316,0.0324197153],[1,65344,0.4576276476],[1,65394,0.170774035],[1,65395,0.1516468069],[1,65398,0.028181753],[1,65413,0.0295712514],[1,65437,0.0339829005],[1,65439,0.0317455689],[1,65459,0.0454615592],[1,65471,0.0295855704],[1,65509,0.0674626821],[1,65512,0.036088592],[1,65525,0.3631230046],[1,65530,0.0525493992],[1,65542,0.0720478795],[1,65622,0.0835556153],[1,65625,0.0295855704],[1,65673,0.0295855704],[1,65689,0.0552814107],[1,65718,0.0295855704],[1,65720,0.1505839671],[1,65723,0.0378669093],[1,65733,0.0320400327],[1,65780,0.0276205565],[1,65792,0.1449998111],[1,65798,0.0335547396],[1,65807,0.0352144465],[1,65846,0.0283440994],[1,65868,0.2586759919],[1,65876,0.0406848692],[1,65901,0.2571029074],[1,65912,0.2765706644],[1,65918,0.0257181117],[1,65983,0.1191459606],[1,65987,0.397738235],[1,65995,0.1296589409],[1,66018,0.0320400327],[1,66020,0.1818324349],[1,66024,0.1853947743],[1,66060,0.0286118711],[1,66064,0.0594044874],[1,66101,0.1256615385],[1,66126,0.1216833462],[1,66177,0.0257181117],[1,66184,0.0551689732],[1,66232,0.029309009],[1,66253,0.0261765825],[1,66273,0.025290564],[1,66288,0.0225237741],[1,66306,0.0551689732],[1,66317,0.0514087423],[1,66356,0.0285486431],[1,66370,0.027603905],[1,66393,0.0492602675],[1,66394,0.0526487269],[1,66400,0.0269157398],[1,66408,0.0615283024],[1,66418,0.0356636577],[1,66419,0.0564017108],[1,66423,0.0542984056],[1,66425,0.0270013578],[1,66439,0.0259122453],[1,66465,0.0682827628],[1,66470,0.0330860208],[1,66474,0.025109747],[1,66506,0.0271928479],[1,66532,0.0476590433],[1,66538,0.0285589633],[1,66550,0.5100958045],[1,66551,0.2102138702],[1,66587,0.0299017623],[1,66628,0.0272155669],[1,66666,0.2847765098],[1,66673,0.110580729],[1,66685,0.0531401356],[1,66704,0.0574555],[1,66880,0.1865006935],[1,66890,0.1865587571],[1,66923,0.1277812233],[1,66934,0.028181753],[1,66944,0.0277572059],[1,66945,0.5584345413],[1,66955,0.0324480166],[1,66979,0.0463514233],[1,67037,0.6652436398],[1,67062,0.0263385964],[1,67071,0.0411659219],[1,67100,0.0271104696],[1,67116,0.0305408609],[1,67119,0.0242954063],[1,67167,0.0599309986],[1,67205,0.0401593621],[1,67213,0.029500495],[1,67266,0.0441979636],[1,67284,0.0229731595],[1,67300,0.0578427993],[1,67315,0.0386026258],[1,67320,0.0253929692],[1,67334,0.0527182779],[1,67356,0.0503020433],[1,67358,0.2990697346],[1,67384,0.0229089532],[1,67393,0.0253929692],[1,67436,0.0276442163],[1,67438,0.0472816249],[1,67449,0.0583664484],[1,67450,0.0363476151],[1,67457,0.0268608159],[1,67460,0.2402475166],[1,67501,0.0282199764],[1,67527,0.0224886982],[1,67570,0.3976831079],[1,67574,0.03877911],[1,67583,0.1820617976],[1,67604,0.0392240776],[1,67607,0.0974144605],[1,67623,0.0476856603],[1,67624,0.0242545436],[1,67671,0.0214759897],[1,67681,0.0410873363],[1,67683,0.3755340672],[1,67684,0.0373611443],[1,67685,0.0262327124],[1,67695,0.024859045],[1,67703,0.1069393914],[1,67733,0.0214759897],[1,67736,0.0853379444],[1,67746,0.0478579566],[1,67784,0.0333851793],[1,67798,0.0199738439],[1,67817,0.0487728522],[1,67838,0.067037103],[1,67841,0.0225264095],[1,67855,0.0221219886],[1,67868,0.0483845098],[1,67876,0.0313445768],[1,67905,0.0346644941],[1,67937,0.0221391812],[1,67940,0.0462403007],[1,67977,0.1252267146],[1,67995,0.2006715635],[1,67999,0.0306504143],[1,68009,0.0129529566],[1,68020,0.0189306548],[1,68022,0.0220906126],[1,68023,0.0430739672],[1,68038,0.045730486],[1,68072,0.0169217372],[1,68106,0.0271143175],[1,68121,0.0933960131],[1,68156,0.0282199135],[1,68161,0.0604728908],[1,68197,0.0484525704],[1,68208,0.0519740646],[1,68215,0.0261237228],[1,68224,0.1665452785],[1,68228,0.0689748327],[1,68245,0.0199317931],[1,68328,0.0577818054],[1,68353,0.0305666383],[1,68376,0.0186078294],[1,68383,0.0191296273],[1,68397,0.018629312],[1,68412,0.167095397],[1,68418,0.0719962458],[1,68434,0.0236632336],[1,68457,0.0303142827],[1,68509,0.0169217372],[1,68510,0.0240181483],[1,68588,0.060722639],[1,68598,0.0196249945],[1,68611,0.1147723655],[1,68627,0.0521345021],[1,68709,0.125517725],[1,68729,0.0185540335],[1,68737,0.0192688596],[1,68761,0.0162747347],[1,68774,0.2074139374],[1,68779,0.0141988064],[1,68832,0.0215831353],[1,68849,0.0303389474],[1,68862,0.3305161672],[1,68879,0.0196172619],[1,68895,0.0906464298],[1,68902,0.0367269581],[1,68910,0.0871487219],[1,68934,0.1118790948],[1,68939,0.078876153],[1,68940,0.0298673614],[1,68946,0.017390126],[1,68959,0.1225227981],[1,68972,0.0274097534],[1,68983,0.0498220059],[1,69020,0.0233988359],[1,69039,0.0976210819],[1,69042,0.0604726088],[1,69063,0.0682638784],[1,69065,0.0337723543],[1,69107,0.0386712056],[1,69125,0.0427070264],[1,69164,0.1095199],[1,69172,0.0219772661],[1,69258,0.047300246],[1,69272,0.0271064239],[1,69281,0.0423672659],[1,69321,0.0476451674],[1,69327,0.0479828922],[1,69352,0.0147233512],[1,69396,0.3211129645],[1,69401,0.0402331679],[1,69404,0.0682250168],[1,69427,0.0171231601],[1,69468,0.0199317931],[1,69469,0.0312235449],[1,69498,0.0250547762],[1,69499,0.0196172619],[1,69500,0.0196172619],[1,69519,0.0190892318],[1,69638,0.044422776],[1,69660,0.0306160109],[1,69727,0.0169217372],[1,69741,0.145598688],[1,69760,0.0353568994],[1,69761,0.0365837936],[1,69762,0.1326781403],[1,69781,0.0202180059],[1,69846,0.4912241143],[1,69856,0.0181575932],[1,69881,0.2991590117],[1,69887,0.1490999833],[1,69900,0.0513629288],[1,69919,0.0196172619],[1,69927,0.065065524],[1,69932,0.0446006548],[1,69937,0.1247060667],[1,69942,0.0251392412],[1,69980,0.0192688596],[1,69997,0.0227295152],[1,70034,0.0192883792],[1,70036,0.017390126],[1,70130,0.0371620264],[1,70133,0.0168424447],[1,70187,0.0306402896],[1,70226,0.4306644048],[1,70253,0.0348424969],[1,70255,0.0342617207],[1,70257,0.0218548399],[1,70271,0.0394702249],[1,70296,0.044422776],[1,70346,0.1423819798],[1,70385,0.1079473452],[1,70414,0.0168559781],[1,70440,0.0162747347],[1,70441,0.0240945182],[1,70452,0.0325685712],[1,70463,0.0250591511],[1,70494,0.0230924384],[1,70508,0.0411597891],[1,70537,0.0235448368],[1,70555,0.0306976345],[1,70556,0.0651139904],[1,70558,0.0178796594],[1,70559,0.0455999034],[1,70562,0.0350612395],[1,70570,0.0255050732],[1,70610,0.0180969968],[1,70626,0.0184215766],[1,70658,0.0479713128],[1,70671,0.0162744233],[1,70708,0.0459784493],[1,70728,0.0198548461],[1,70784,0.0463966361],[1,70801,0.0194324798],[1,70846,0.0392157908],[1,70869,0.2758090615],[1,70878,0.2033089023],[1,70912,0.0625839792],[1,70921,0.2462653219],[1,70922,0.0172689598],[1,70924,0.0211225738],[1,70925,0.0283406217],[1,70932,0.0336087146],[1,70947,0.0738239049],[1,70969,0.0274864302],[1,71023,0.0281745731],[1,71071,0.0334564953],[1,71102,0.0214377436],[1,71113,0.0194324798],[1,71129,0.0279805593],[1,71148,0.0276120396],[1,71211,0.0388634166],[1,71222,0.0244370229],[1,71225,0.2593087185],[1,71286,0.0679985805],[1,71292,0.0611534511],[1,71301,0.0459784493],[1,71303,0.2004430404],[1,71334,0.032970486],[1,71386,0.1001834724],[1,71429,0.0371831634],[1,71442,0.3602159507],[1,71467,0.0395198195],[1,71509,0.0202338297],[1,71529,0.0342994618],[1,71565,0.0320519332],[1,71591,0.0354123481],[1,71598,0.0434571194],[1,71641,0.1971771331],[1,71642,0.017872355],[1,71653,0.0302982794],[1,71680,0.0211807844],[1,71689,0.0623650867],[1,71771,0.0331139251],[1,71825,0.0215691211],[1,71858,0.1244868734],[1,71862,0.0397354056],[1,71889,0.0232778323],[1,71902,0.0261710582],[1,71930,0.0326563159],[1,71937,0.2023237553],[1,71942,0.0169109283],[1,71973,0.0210319535],[1,71992,0.0471870947],[1,72011,0.0276339463],[1,72016,0.0235226131],[1,72038,0.0210319535],[1,72040,0.4347077758],[1,72057,0.018283361],[1,72077,0.1363248962],[1,72080,0.0598431004],[1,72122,0.0239953581],[1,72143,0.0425736341],[1,72157,0.0419136156],[1,72184,0.0297063088],[1,72202,0.0238166239],[1,72207,0.0238576372],[1,72218,0.0210980843],[1,72237,0.0676263371],[1,72258,0.362200048],[1,72287,0.2664910798],[1,72309,0.0454929418],[1,72322,0.0251942685],[1,72357,0.0199413001],[1,72369,0.0834705126],[1,72389,0.0535574835],[1,72392,0.0236959734],[1,72395,0.0393344541],[1,72402,0.0241251569],[1,72473,0.1196209512],[1,72574,0.0602945332],[1,72576,0.1429846552],[1,72579,0.0222074412],[1,72628,0.0237143957],[1,72659,0.0209448279],[1,72704,0.0238088475],[1,72714,0.1205177208],[1,72718,0.0238088475],[1,72781,0.0232296757],[1,72834,0.0248675991],[1,72850,0.2352255973],[1,72863,0.309907194],[1,72913,0.0279600182],[1,72917,0.0350092143],[1,72972,0.0210980843],[1,72980,0.1340126984],[1,72984,0.0262603102],[1,72992,0.0257858346],[1,73002,0.0231206836],[1,73053,0.055278853],[1,73111,0.0215691211],[1,73148,0.1356773736],[1,73166,0.133795079],[1,73172,0.029458932],[1,73186,0.0777860334],[1,73213,0.0469614455],[1,73218,0.0210980843],[1,73272,0.0326336275],[1,73273,0.0238088475],[1,73279,0.0810935402],[1,73337,0.032959414],[1,73353,0.0246662305],[1,73355,0.0737908079],[1,73398,0.015888041],[1,73427,0.0277932008],[1,73430,0.0397103065],[1,73438,0.0889051271],[1,73446,0.049049498],[1,73460,0.0215691211],[1,73553,0.0369173788],[1,73586,0.0281063219],[1,73591,0.021652703],[1,73605,0.0371689426],[1,73622,0.1526722403],[1,73629,0.0335094042],[1,73646,0.3922167487],[1,73660,0.0378078021],[1,73671,0.0364216782],[1,73743,0.1062462211],[1,73748,0.1024317744],[1,73759,0.1986795202],[1,73795,0.0373007678],[1,73803,0.0554780111],[1,73807,0.1163082951],[1,73896,0.1180696849],[1,74130,0.0210980843],[1,74132,0.0364665382],[1,74143,0.0223642596],[1,74155,0.0238088475],[1,74161,0.0223642596],[1,74209,0.1558601879],[1,74288,0.0911236692],[1,74299,0.0410854023],[1,74317,0.0424192236],[1,74324,0.1993677213],[1,74339,0.0804847609],[1,74357,0.2845039495],[1,74383,0.0929196574],[1,74389,0.0731346209],[1,74396,0.0343590734],[1,74435,0.0657862237],[1,74442,0.0554591351],[1,74453,0.032305896],[1,74465,0.3059227706],[1,74487,0.1208372096],[1,74499,0.0438573803],[1,74512,0.0283996964],[1,74554,0.0233646947],[1,74588,0.0234584757],[1,74653,0.0765251891],[1,74658,0.0265974571],[1,74688,0.1022375074],[1,74692,0.0232778323],[1,74693,0.0621121281],[1,74712,0.0742272696],[1,74716,0.045806889],[1,74726,0.0737927674],[1,74731,0.0200069668],[1,74779,0.0238166239],[1,74797,0.2977680352],[1,74798,0.0365526768],[1,74837,0.0829059869],[1,74855,0.1009932552],[1,74876,0.0215691211],[1,74900,0.0317687756],[1,74904,0.1128228028],[1,74946,0.0211799376],[1,75000,0.0349314413],[1,75004,0.3263698513],[1,75059,0.0232778323],[1,75084,0.0373055724],[1,75124,0.105739185],[1,75188,0.0260481743],[1,75196,0.0410560057],[1,75212,0.0175968823],[1,75213,0.0787046303],[1,75241,0.0867779606],[1,75253,0.0389829365],[1,75259,0.1822716352],[1,75277,0.0210319535],[1,75306,0.0380626297],[1,75330,0.0359059655],[1,75359,0.0277679275],[1,75402,0.0490171459],[1,75437,0.037941731],[1,75483,0.2362405492],[1,75485,0.0249431621],[1,75497,0.0260421072],[1,75509,0.0218509004],[1,75512,0.0229047176],[1,75558,0.0210319535],[1,75580,0.028081241],[1,75594,0.0436925641],[1,75608,0.0811275464],[1,75619,0.0475448271],[1,75650,0.0257116906],[1,75662,0.0740254176],[1,75778,0.0437467239],[1,75801,0.1028703227],[1,75812,0.023494088],[1,75827,0.2256563896],[1,75840,0.0307366877],[1,75864,0.0453916238],[1,75869,0.0547117251],[1,75875,0.0404434271],[1,75909,0.0662172776],[1,75925,0.0414736247],[1,75932,0.0434678939],[1,75950,0.0231930794],[1,75959,0.0261283747],[1,75985,0.0529409169],[1,76012,0.0578614641],[1,76080,0.0288011671],[1,76101,0.2314877741],[1,76182,0.0238088475],[1,76246,0.0245476468],[1,76249,0.0212787473],[1,76304,0.0205612571],[1,76307,0.0257858346],[1,76333,0.0377741769],[1,76358,0.0345612816],[1,76399,0.0377086561],[1,76448,0.0236959734],[1,76463,0.1054265542],[1,76465,0.0396335341],[1,76484,0.052340447],[1,76487,0.0238088475],[1,76511,0.0218509004],[1,76525,0.0916115303],[1,76542,0.1201347303],[1,76561,0.3768810544],[1,76569,0.1059507731],[1,76585,0.0313733589],[1,76588,0.1387855097],[1,76633,0.1779719585],[1,76635,0.0282494292],[1,76665,0.0228946238],[1,76666,0.1563163393],[1,76671,0.0589215228],[1,76691,0.046498111],[1,76715,0.0349921215],[1,76721,0.0232778323],[1,76727,0.0523091277],[1,76771,0.0426390606],[1,76804,0.046498111],[1,76852,0.2873105082],[1,76914,0.0188329124],[1,76926,0.307917971],[1,76931,0.0256025572],[1,76937,0.0210980843],[1,76948,0.0469910606],[1,76963,0.2387561999],[1,76984,0.1602276545],[1,77034,0.1495354676],[1,77053,0.0206516408],[1,77059,0.233281019],[1,77098,0.0892046251],[1,77111,0.0302506127],[1,77122,0.0461198933],[1,77129,0.095610399],[1,77133,0.0346088354],[1,77135,0.0243567877],[1,77145,0.0238088475],[1,77167,0.0657843561],[1,77201,0.2263991282],[1,77249,0.0276092759],[1,77251,0.0834747163],[1,77253,0.3212281941],[1,77276,0.0225186854],[1,77290,0.6177510733],[1,77321,0.0338646539],[1,77348,0.0362503785],[1,77359,0.0188872702],[1,77381,0.2525668993],[1,77401,0.2263563981],[1,77436,0.0213866726],[1,77448,0.0175968823],[1,77514,0.046498111],[1,77520,0.0238088475],[1,77531,0.2873745523],[1,77588,0.0463934722],[1,77594,0.024213751],[1,77600,0.0468330964],[1,77613,0.0387163006],[1,77624,0.0238088475],[1,77630,0.0437467239],[1,77636,0.2191802714],[1,77654,0.0241251569],[1,77658,0.0182100767],[1,77684,0.034024335],[1,77701,0.0232778323],[1,77714,0.1684447913],[1,77760,0.023024635],[1,77768,0.0232726188],[1,77774,0.0204474241],[1,77777,0.0516149754],[1,77794,0.0222093547],[1,77806,0.2348072508],[1,77845,0.024254504],[1,77856,0.0234805507],[1,77857,0.0338464061],[1,77906,0.0215691211],[1,77914,0.1015529195],[1,77920,0.0210319535],[1,77968,0.0334724753],[1,77998,0.0513592365],[1,78010,0.132412068],[1,78076,0.1581238162],[1,78081,0.0175930547],[1,78083,0.0932642467],[1,78085,0.0271129585],[1,78106,0.0342407514],[1,78150,0.4143820419],[1,78175,0.0260942948],[1,78176,0.0182100767],[1,78177,0.035782535],[1,78183,0.0234640189],[1,78198,0.1058002538],[1,78219,0.0432127496],[1,78221,0.0215699652],[1,78245,0.0456876452],[1,78246,0.0489914717],[1,78287,0.0274260466],[1,78294,0.0238088475],[1,78310,0.0212653282],[1,78320,0.0168914533],[1,78327,0.1258571864],[1,78359,0.1605360745],[1,78372,0.2598189977],[1,78389,0.0236538094],[1,78431,0.0203280643],[1,78483,0.0208647229],[1,78485,0.0333162924],[1,78561,0.0397103065],[1,78588,0.0181670041],[1,78609,0.0296337002],[1,78623,0.0176886515],[1,78650,0.0219756442],[1,78651,0.0411748908],[1,78663,0.144184767],[1,78666,0.0460537864],[1,78670,0.019708302],[1,78697,0.023219955],[1,78727,0.0795729167],[1,78783,0.0308068937],[1,78842,0.1066179327],[1,78905,0.227158217],[1,78958,0.0474685716],[1,79029,0.0286313394],[1,79041,0.0203280643],[1,79067,0.0213356117],[1,79146,0.0248377146],[1,79199,0.6434487972],[1,79210,0.1650855126],[1,79250,0.022572821],[1,79323,0.0279239579],[1,79424,0.0416610686],[1,79462,0.0224962392],[1,79466,0.0217775735],[1,79596,0.0250382739],[1,79620,0.124431235],[1,79622,0.0183342504],[1,79623,0.0324416946],[1,79635,0.0563409619],[1,79644,0.1249868291],[1,79670,0.0459501153],[1,79682,0.0429818321],[1,79684,0.3137529594],[1,79723,0.1444093374],[1,79742,0.2282245721],[1,79765,0.0223316183],[1,79784,0.0481235026],[1,79791,0.0233387033],[1,79861,0.0397396234],[1,79864,0.0212528642],[1,79872,0.0341957355],[1,79897,0.0255915561],[1,79934,0.0217070075],[1,79936,0.0217070075],[1,79938,0.1931682329],[1,79953,0.0408842202],[1,80017,0.0492388919],[1,80047,0.2445252047],[1,80132,0.0580067835],[1,80139,0.2819465682],[1,80146,0.2263256639],[1,80167,0.022798003],[1,80169,0.0486348272],[1,80181,0.046050309],[1,80219,0.0228002571],[1,80228,0.0255837613],[1,80265,0.0383310916],[1,80276,0.0241357347],[1,80287,0.1269514098],[1,80310,0.0570686151],[1,80341,0.1214318673],[1,80440,0.2346026616],[1,80445,0.0228151074],[1,80447,0.0284794027],[1,80463,0.0185229439],[1,80492,0.0382661671],[1,80524,0.038467316],[1,80534,0.0270674054],[1,80563,0.4196383848],[1,80568,0.0233387033],[1,80570,0.0407271449],[1,80593,0.0652446901],[1,80594,0.0255915561],[1,80616,0.0318739194],[1,80630,0.0432849975],[1,80632,0.0200345518],[1,80633,0.044977185],[1,80648,0.3067533058],[1,80667,0.0392571185],[1,80739,0.2965085231],[1,80741,0.0237722975],[1,80742,0.04857148],[1,80744,0.5456089983],[1,80757,0.1453852044],[1,80765,0.0511995021],[1,80792,0.0243953234],[1,80821,0.0364056798],[1,80826,0.0345331441],[1,80874,0.4601024254],[1,80973,0.0228340249],[1,80991,0.1268954615],[1,81022,0.0197890794],[1,81039,0.1165247009],[1,81060,0.0255837613],[1,81063,0.0230476384],[1,81069,0.2225644751],[1,81113,0.4637772295],[1,81117,0.0272417539],[1,81136,0.0580782725],[1,81183,0.0424664685],[1,81191,0.0217070075],[1,81195,0.1512686075],[1,81213,0.0223316183],[1,81244,0.1220256686],[1,81252,0.0241587142],[1,81255,0.0543525916],[1,81261,0.0809234673],[1,81303,0.0433429624],[1,81327,0.0314596207],[1,81343,0.1381910928],[1,81387,0.0803488835],[1,81409,0.025051482],[1,81424,0.0342462961],[1,81449,0.2463359002],[1,81470,0.0233120068],[1,81476,0.0240003082],[1,81522,0.0382606597],[1,81533,0.1518867467],[1,81608,0.0298638267],[1,81611,0.1172286751],[1,81616,0.0249295308],[1,81621,0.107773718],[1,81678,0.0659176136],[1,81740,0.0339251656],[1,81741,0.0182338363],[1,81745,0.0595604391],[1,81751,0.0538784165],[1,81755,0.023946776],[1,81765,0.0241357347],[1,81806,0.0277546355],[1,81818,0.0216858534],[1,81830,0.0240436895],[1,81850,0.0472608289],[1,81882,0.0390335836],[1,81896,0.0189453627],[1,81901,0.5561908654],[1,81929,0.02454153],[1,81930,0.0831027187],[1,81981,0.0421691767],[1,82012,0.3618615292],[1,82013,0.0324646324],[1,82209,0.1270435147],[1,82216,0.2366967525],[1,82234,0.0517501521],[1,82246,0.0197628821],[1,82255,0.0238738551],[1,82260,0.0187313758],[1,82359,0.0212295731],[1,82370,0.068425772],[1,82405,0.0350115837],[1,82411,0.0237722975],[1,82469,0.0225837322],[1,82484,0.0287890317],[1,82517,0.0473080357],[1,82525,0.0890859243],[1,82568,0.1867278864],[1,82585,0.0230476384],[1,82614,0.0329201084],[1,82630,0.0229530305],[1,82659,0.025121045],[1,82669,0.0644847248],[1,82681,0.0189453627],[1,82789,0.0425003416],[1,82807,0.5627853581],[1,82811,0.0346539294],[1,82817,0.0454986151],[1,82839,0.0963025097],[1,82850,0.0604085718],[1,82902,0.0315673916],[1,82904,0.0181593402],[1,82917,0.0375632074],[1,82918,0.0385299147],[1,82927,0.0242361276],[1,82929,0.268056843],[1,82937,0.0177741496],[1,82952,0.0273624991],[1,82966,0.0288181173],[1,83003,0.0362791575],[1,83078,0.0549104033],[1,83095,0.0530896173],[1,83120,0.0367342891],[1,83133,0.0250269294],[1,83190,0.0344212304],[1,83197,0.4061206488],[1,83224,0.0920017635],[1,83346,0.0695054831],[1,83354,0.0524173024],[1,83361,0.0186640255],[1,83384,0.0239003335],[1,83388,0.0514938909],[1,83399,0.0970881109],[1,83406,0.0264240085],[1,83422,0.0264240085],[1,83429,0.0186640255],[1,83446,0.0238656676],[1,83479,0.0346625599],[1,83503,0.0726861297],[1,83556,0.2368902762],[1,83604,0.0367052755],[1,83652,0.0391399293],[1,83724,0.0220535762],[1,83727,0.1333764034],[1,83801,0.0343620817],[1,83843,0.0242361276],[1,83847,0.0825827978],[1,83887,0.2894547206],[1,83889,0.0372767866],[1,83939,0.0498489011],[1,83955,0.0409548164],[1,83963,0.0247678258],[1,83972,0.368655707],[1,83984,0.0446442971],[1,84015,0.043739684],[1,84033,0.0297158256],[1,84051,0.0293818086],[1,84061,0.0181366742],[1,84105,0.1282999465],[1,84134,0.0205485181],[1,84139,0.0209756393],[1,84159,0.0403554676],[1,84180,0.6101391531],[1,84205,0.0665282814],[1,84217,0.0470299918],[1,84240,0.0247180922],[1,84299,0.0943065973],[1,84301,0.0325930756],[1,84303,0.0352505687],[1,84308,0.0247678258],[1,84336,0.099241117],[1,84356,0.0386729728],[1,84369,0.0454232687],[1,84374,0.0303768124],[1,84433,0.025121045],[1,84439,0.0289966606],[1,84445,0.0750594103],[1,84474,0.0324560386],[1,84489,0.0254890586],[1,84500,0.382285484],[1,84548,0.0208741732],[1,84553,0.0385573697],[1,84557,0.2432946041],[1,84594,0.0400619804],[1,84637,0.0653408896],[1,84719,0.0542451273],[1,84804,0.1027714989],[1,84805,0.0255806167],[1,84822,0.0173702],[1,84846,0.0222979142],[1,84878,0.0376352506],[1,84883,0.0264240085],[1,84895,0.0485031869],[1,84941,0.0301998945],[1,84963,0.021019638],[1,84976,0.0712221296],[1,85001,0.0966301041],[1,85010,0.0231662758],[1,85062,0.2690244994],[1,85092,0.0247756121],[1,85109,0.0243707243],[1,85110,0.0408797648],[1,85111,0.0248537608],[1,85123,0.2779692844],[1,85193,0.0385851317],[1,85224,0.048862341],[1,85288,0.029371596],[1,85300,0.0444580135],[1,85430,0.019418126],[1,85445,0.1651478926],[1,85472,0.1622064368],[1,85515,0.0358421313],[1,85534,0.0214062253],[1,85540,0.0433754394],[1,85566,0.0514003423],[1,85611,0.1378263447],[1,85650,0.0247678258],[1,85671,0.0255806167],[1,85689,0.0215192331],[1,85701,0.0900320884],[1,85718,0.0265714295],[1,85719,0.3180875266],[1,85740,0.0247678258],[1,85749,0.0392903307],[1,85751,0.0219873603],[1,85799,0.1215154801],[1,85800,0.0382748005],[1,85821,0.0244170033],[1,85870,0.0208953042],[1,85910,0.0451630558],[1,85919,0.3425549403],[1,85923,0.3613050564],[1,85951,0.0254401257],[1,85985,0.7528474967],[1,85996,0.0262046206],[1,86002,0.021345068],[1,86031,0.0237139036],[1,86048,0.0339218878],[1,86126,0.0200499805],[1,86143,0.0265294732],[1,86173,0.0422038401],[1,86183,0.0694434386],[1,86184,0.3569962812],[1,86228,0.0398004887],[1,86276,0.4511231995],[1,86306,0.0219851087],[1,86319,0.0395114908],[1,86326,0.0464509683],[1,86349,0.0242361276],[1,86361,0.0212187756],[1,86368,0.0245587125],[1,86454,0.0243600152],[1,86458,0.0389933126],[1,86469,0.1921439449],[1,86508,0.0626693113],[1,86531,0.1029311753],[1,86583,0.0457582055],[1,86598,0.0214062253],[1,86605,0.0216968232],[1,86661,0.0154480811],[1,86697,0.0454219452],[1,86711,0.187303467],[1,86726,0.0307225121],[1,86768,0.4940908266],[1,86845,0.0703004918],[1,86880,0.0537462955],[1,86994,0.0221830954],[1,87012,0.0209211143],[1,87072,0.3427635719],[1,87087,0.0171779678],[1,87123,0.032613943],[1,87156,0.0185028534],[1,87162,0.0397223566],[1,87168,0.6862897001],[1,87187,0.0186547451],[1,87189,0.1137892874],[1,87216,0.3890878704],[1,87243,0.6005310705],[1,87249,0.0186547451],[1,87310,0.0185028534],[1,87315,0.1127962148],[1,87383,0.0180337595],[1,87399,0.020741086],[1,87418,0.035602784],[1,87426,0.0232108853],[1,87433,0.0417371468],[1,87518,0.1010264439],[1,87521,0.0237612755],[1,87546,0.016829761],[1,87606,0.0296122362],[1,87639,0.0225570512],[1,87678,0.0903122527],[1,87697,0.0430362569],[1,87728,0.0234770029],[1,87729,0.3948715837],[1,87752,0.0220771866],[1,87768,0.1847169933],[1,87771,0.1028212811],[1,87816,0.0173948812],[1,87835,0.0322694564],[1,87871,0.0157562937],[1,87887,0.0301477658],[1,87893,0.0229187173],[1,87910,0.0433691791],[1,87943,0.035625038],[1,87958,0.0638185392],[1,88006,0.0998306296],[1,88013,0.0207539195],[1,88056,0.1870137211],[1,88091,0.0432598382],[1,88092,0.0467886562],[1,88100,0.0340992101],[1,88126,0.0185028534],[1,88130,0.0443910083],[1,88194,0.0254318579],[1,88197,0.354601068],[1,88201,0.0209875059],[1,88236,0.018568759],[1,88256,0.0186547451],[1,88268,0.0185609191],[1,88281,0.0363142039],[1,88339,0.0178359733],[1,88343,0.0312065093],[1,88345,0.0265747213],[1,88357,0.0212702933],[1,88376,0.0339570083],[1,88387,0.3368529817],[1,88410,0.0220592647],[1,88421,0.0272949621],[1,88561,0.0565242583],[1,88610,0.0390448754],[1,88616,0.1677033722],[1,88637,0.0212702933],[1,88642,0.0400230992],[1,88654,0.1584589805],[1,88670,0.1512949235],[1,88683,0.062035011],[1,88711,0.1813806571],[1,88788,0.0133302574],[1,88880,0.0542346908],[1,88909,0.2081290109],[1,88988,0.1760618899],[1,89013,0.0474925196],[1,89030,0.0241706636],[1,89042,0.0955418195],[1,89048,0.0267081171],[1,89142,0.0163489771],[1,89187,0.064432714],[1,89188,0.0262241943],[1,89197,0.1241441349],[1,89201,0.0386349043],[1,89218,0.1262622356],[1,89248,0.0719636909],[1,89272,0.0159573023],[1,89291,0.0386095744],[1,89343,0.0549701201],[1,89349,0.0416203103],[1,89356,0.0226119309],[1,89364,0.0313414652],[1,89413,0.0243558647],[1,89416,0.1381147564],[1,89430,0.3107338815],[1,89432,0.0312883234],[1,89438,0.0288686615],[1,89453,0.318495669],[1,89460,0.0141864717],[1,89476,0.0240435837],[1,89487,0.0176776273],[1,89524,0.026319949],[1,89528,0.0306016039],[1,89538,0.1435309484],[1,89541,0.0435303919],[1,89546,0.0254080192],[1,89559,0.0180450635],[1,89569,0.014739645],[1,89585,0.2195202448],[1,89628,0.0370770056],[1,89646,0.0363223052],[1,89648,0.0363333652],[1,89651,0.2734480776],[1,89691,0.0742865506],[1,89737,0.1887474065],[1,89750,0.0328707771],[1,89764,0.0434925761],[1,89801,0.7746649821],[1,89807,0.5860827212],[1,89848,0.0764669188],[1,89881,0.0134645964],[1,89885,0.0547260318],[1,89889,0.1284466422],[1,89892,0.0951805021],[1,89926,0.054370606],[1,89980,0.0180527798],[1,89992,0.1904793667],[1,90005,0.105071313],[1,90011,0.060147848],[1,90048,0.2081458946],[1,90147,0.0747240146],[1,90224,0.0430550534],[1,90249,0.015289635],[1,90279,0.0127901372],[1,90287,0.1554274157],[1,90297,0.1202442904],[1,90342,0.0787965697],[1,90346,0.1859671558],[1,90409,0.8044770598],[1,90411,0.1539198696],[1,90412,0.015289635],[1,90414,0.5800173865],[1,90448,0.0114736227],[1,90453,0.0632293856],[1,90468,0.0423758644],[1,90481,0.0515753573],[1,90538,0.0445351977],[1,90594,0.0231265304],[1,90624,0.0166116423],[1,90664,0.0494766634],[1,90673,0.2983724507],[1,90728,0.1050884577],[1,90788,0.0248334012],[1,90808,0.0157529182],[1,90826,0.0445351977],[1,90832,0.0849688328],[1,90836,0.1077689307],[1,90837,0.0162553187],[1,90847,0.6413164157],[1,90859,0.0246327244],[1,90862,0.3113331576],[1,90878,0.0438262511],[1,90896,0.0152276401],[1,90900,0.015534521],[1,90911,0.0197920554],[1,90938,0.0482528119],[1,90959,0.0164579348],[1,90965,0.7506621735],[1,90998,0.0179957777],[1,91017,0.0434925761],[1,91025,0.040601902],[1,91031,0.2928770959],[1,91043,0.0420084228],[1,91106,0.0146256494],[1,91107,0.016242859],[1,91112,0.0821768288],[1,91115,0.0158226502],[1,91134,0.0412282348],[1,91138,0.0452129529],[1,91163,0.0581222352],[1,91165,0.0444276703],[1,91187,0.0499682522],[1,91202,0.0842536011],[1,91214,0.0203435491],[1,91278,0.9173519724],[1,91288,0.6770853081],[1,91332,0.0153552546],[1,91347,0.1858614745],[1,91373,0.0457447417],[1,91435,0.161375314],[1,91445,0.015289635],[1,91489,0.0467048889],[1,91528,0.0417084615],[1,91545,0.2237612061],[1,91618,0.0150647724],[1,91624,0.0308573366],[1,91661,0.0180450635],[1,91679,0.0438262511],[1,91704,0.0141864717],[1,91718,0.0180450635],[1,91762,0.1683027709],[1,91815,0.0394718825],[1,91879,0.0199772329],[1,91903,0.0328453176],[1,91910,0.1396821481],[1,92008,0.0154408675],[1,92036,0.0272793033],[1,92089,0.2428599954],[1,92099,0.1503179734],[1,92123,0.0335174467],[1,92124,0.0312399702],[1,92141,0.0389167924],[1,92157,0.0180450635],[1,92165,0.0924828999],[1,92186,0.2014950189],[1,92211,0.0620622349],[1,92246,0.2145203136],[1,92260,0.0713520298],[1,92285,0.0367452624],[1,92301,0.016085076],[1,92310,0.0152966464],[1,92318,0.2648810582],[1,92337,0.0413978716],[1,92393,0.0290931229],[1,92438,0.0929018008],[1,92474,0.0970122042],[1,92516,0.0131100295],[1,92535,0.0661771811],[1,92553,0.1024361825],[1,92561,0.0462224729],[1,92583,0.0171700594],[1,92600,0.0429468414],[1,92627,0.0352494679],[1,92726,0.0541607049],[1,92730,0.0187977344],[1,92743,0.3607348829],[1,92747,0.10923341],[1,92751,0.0316222451],[1,92794,0.0322162119],[1,92854,0.0159983992],[1,92874,0.0240539936],[1,92904,0.0594793409],[1,92912,0.048761505],[1,93003,0.0438812688],[1,93019,0.0485514585],[1,93028,0.0194042626],[1,93051,0.0417332326],[1,93083,0.0639052335],[1,93118,0.0158260227],[1,93169,0.0147640113],[1,93177,0.041159416],[1,93188,0.0429468414],[1,93234,0.0130474394],[1,93349,0.0149151359],[1,93419,0.0154968961],[1,93485,0.2268267985],[1,93560,0.1370928697],[1,93561,0.2805401292],[1,93618,0.0375023014],[1,93623,0.0407107802],[1,93630,0.0270094495],[1,93671,0.046725121],[1,93709,0.0429468414],[1,93759,0.0311030816],[1,93785,0.0172327106],[1,93811,0.0432802784],[1,93832,0.1770907971],[1,93850,0.0631722292],[1,93859,0.1402572778],[1,93891,0.0180092854],[1,93920,0.0260517016],[1,93926,0.0124590064],[1,93966,0.0396973854],[1,93980,0.5729478943],[1,93994,0.2102562819],[1,93997,0.0415456518],[1,94043,0.4111887173],[1,94050,0.0267501677],[1,94059,0.0201422529],[1,94071,0.0787762243],[1,94072,0.0241962212],[1,94103,0.0313578069],[1,94126,0.0241962212],[1,94145,0.0152966464],[1,94233,0.1070963228],[1,94426,0.017517475],[1,94427,0.0169536969],[1,94447,0.0344250799],[1,94453,0.0175495223],[1,94492,0.0168661469],[1,94494,0.0169909403],[1,94556,0.4474401324],[1,94637,0.017517475],[1,94658,0.0983947105],[1,94692,0.225065566],[1,94725,0.0153723872],[1,94756,0.0183480607],[1,94771,0.0249131582],[1,94792,0.033232671],[1,94815,0.0128945119],[1,94835,0.0206153131],[1,94840,0.0427542482],[1,94864,0.0357320374],[1,94866,0.0432788132],[1,94911,0.3183223543],[1,94913,0.0148295841],[1,94920,0.0239450194],[1,94938,0.015931478],[1,94965,0.0234444401],[1,94979,0.0136616346],[1,94981,0.0368929352],[1,95094,0.1040503089],[1,95298,0.0168148972],[1,95364,0.1174606109],[1,95365,0.0114563778],[1,95388,0.028045511],[1,95397,0.033479044],[1,95411,0.0122662959],[1,95416,0.6335726586],[1,95418,0.0153675716],[1,95424,0.0488442956],[1,95427,0.0537042524],[1,95464,0.0311006694],[1,95483,0.015931478],[1,95502,0.0249098143],[1,95504,0.0119933158],[1,95555,0.0369242512],[1,95593,0.0381883374],[1,95599,0.0504244462],[1,95603,0.0145268682],[1,95626,0.0439242598],[1,95637,0.0555315292],[1,95660,0.0181576292],[1,95698,0.0314913196],[1,95700,0.0410206204],[1,95723,0.0175116198],[1,95733,0.037733148],[1,95737,0.0147640113],[1,95738,0.0317439923],[1,95740,0.0166782736],[1,95748,0.0141279565],[1,95766,0.0094949844],[1,95779,0.2315094177],[1,95780,0.061028055],[1,95791,0.0214942291],[1,95805,0.0125893498],[1,95811,0.017517475],[1,95844,0.1518476721],[1,95872,0.039730817],[1,95932,0.0460487025],[1,95943,0.1045609639],[1,95958,0.0252163707],[1,96035,0.1193431335],[1,96039,0.0506325379],[1,96045,0.0341626803],[1,96070,0.0134458167],[1,96081,0.0192632211],[1,96151,0.0634427004],[1,96197,0.0685910757],[1,96219,0.021742994],[1,96254,0.1609647125],[1,96268,0.0148217837],[1,96286,0.0703749477],[1,96317,0.1257845722],[1,96348,0.0169909403],[1,96365,0.0400338208],[1,96383,0.0885894925],[1,96491,0.0108025351],[1,96520,0.0259083544],[1,96565,0.0420899677],[1,96595,0.3579278224],[1,96647,0.0328621104],[1,96686,0.0152966464],[1,96702,0.044160188],[1,96703,0.0531681361],[1,96710,0.2247625663],[1,96715,0.2427038718],[1,96718,0.0444581348],[1,96739,0.3278322191],[1,96740,0.016085076],[1,96744,0.0357832593],[1,96786,0.0120808433],[1,96806,0.0282230631],[1,96811,0.0122800612],[1,96835,0.0190504955],[1,96931,0.0635550922],[1,96938,0.0144762956],[1,96944,0.0382829264],[1,96988,0.0181843731],[1,96990,0.0266281951],[1,96998,0.0419000854],[1,97017,0.3738847745],[1,97028,0.0117246664],[1,97033,0.0339926477],[1,97097,0.0396779652],[1,97100,0.0446873458],[1,97108,0.0116674457],[1,97174,0.0497821263],[1,97218,0.0435475699],[1,97244,0.018476406],[1,97270,0.0187174034],[1,97287,0.0283410341],[1,97343,0.0148295841],[1,97346,0.0152966464],[1,97363,0.2932919759],[1,97377,0.0421919361],[1,97388,0.0119933158],[1,97390,0.0199958771],[1,97395,0.0229731912],[1,97405,0.0285912554],[1,97437,0.0215566816],[1,97446,0.2828299403],[1,97588,0.1782374295],[1,97621,0.0152269641],[1,97752,0.0369702992],[1,97766,0.0147640113],[1,97804,0.0420599779],[1,97810,0.0160010929],[1,97819,0.0342560988],[1,97820,0.0415300563],[1,97898,0.0347256704],[1,97916,0.0148217837],[1,97933,0.1407897835],[1,98010,0.1128872148],[1,98019,0.0191575741],[1,98023,0.0138007585],[1,98038,0.3716980565],[1,98076,0.0391719908],[1,98159,0.0097660509],[1,98179,0.0141234793],[1,98190,0.060769239],[1,98226,0.0814154226],[1,98290,0.0670942587],[1,98325,0.0937971211],[1,98338,0.03097917],[1,98353,0.0750862052],[1,98357,0.0596563914],[1,98378,0.0292853883],[1,98445,0.0465602211],[1,98481,0.0162896897],[1,98485,0.1636207213],[1,98516,0.010776937],[1,98613,0.0147830768],[1,98675,0.0592004955],[1,98680,0.0845822409],[1,98707,0.0168157246],[1,98717,0.0232708636],[1,98761,0.0140626468],[1,98781,0.0328139865],[1,98815,0.0314005966],[1,98818,0.015231233],[1,98866,0.0333245865],[1,98896,0.2170636525],[1,98909,0.0201920251],[1,98927,0.0154404564],[1,98929,0.0260408576],[1,98951,0.0175999665],[1,98959,0.0305837643],[1,98964,0.0610858553],[1,99000,0.0434746662],[1,99015,0.0537713365],[1,99018,0.0454050914],[1,99024,0.0405279361],[1,99046,0.018454267],[1,99057,0.1038489105],[1,99076,0.0431544952],[1,99092,0.0102945252],[1,99108,0.018006675],[1,99183,0.0329793441],[1,99224,0.0137774318],[1,99294,0.0623766787],[1,99295,0.0441860121],[1,99329,0.037732653],[1,99503,0.012747931],[1,99509,0.0956261871],[1,99514,0.0343718939],[1,99530,0.0192284746],[1,99621,0.0415312564],[1,99778,0.0102945252],[1,99796,0.0173687869],[1,99839,0.3335794933],[1,99862,0.0939692165],[1,99929,0.1766176833],[1,99961,0.0254672927],[1,99964,0.0262396215],[1,100035,0.3490557335],[1,100070,0.0168157246],[1,100122,0.015819045],[1,100163,0.0411769296],[1,100173,0.0426622078],[1,100198,0.0176072626],[1,100219,0.015231233],[1,100285,0.0350426724],[1,100323,0.0658555541],[1,100335,0.1349987656],[1,100337,0.1457159808],[1,100383,0.0115953003],[1,100388,0.0646463212],[1,100406,0.0158552148],[1,100433,0.1516706787],[1,100489,0.0357241687],[1,100503,0.0133720047],[1,100516,0.0160197116],[1,100560,0.0501084857],[1,100593,0.0208943505],[1,100603,0.1399259616],[1,100626,0.032532845],[1,100630,0.0405262087],[1,100649,0.0465002021],[1,100652,0.1911720508],[1,100653,0.014130385],[1,100743,0.0317442258],[1,100749,0.1285252023],[1,100770,0.0453212861],[1,100849,0.1425903934],[1,100915,0.2396384817],[1,100921,0.0311184809],[1,101024,0.0351211109],[1,101040,0.0218778212],[1,101051,0.1068635498],[1,101059,0.0382641574],[1,101080,0.015231233],[1,101117,0.1065398295],[1,101118,0.0107603803],[1,101136,0.0405236254],[1,101140,0.2273005271],[1,101153,0.1806926746],[1,101166,0.0897590621],[1,101170,0.2800032819],[1,101208,0.0153997494],[1,101259,0.1900787087],[1,101306,0.0220870362],[1,101314,0.0499401822],[1,101331,0.1703930782],[1,101336,0.014043678],[1,101363,0.0105065009],[1,101379,0.014597004],[1,101388,0.0600094729],[1,101417,0.0183497247],[1,101430,0.0160367931],[1,101433,0.0845099925],[1,101436,0.0440234114],[1,101472,0.0145273878],[1,101516,0.0181043049],[1,101546,0.0140648744],[1,101571,0.0434331496],[1,101573,0.0433628999],[1,101619,0.0361281709],[1,101631,0.015231233],[1,101645,0.0281838676],[1,101662,0.019700625],[1,101669,0.4254536575],[1,101671,0.0347753612],[1,101674,0.0423983223],[1,101717,0.0455845763],[1,101725,0.6113426127],[1,101737,0.1073954698],[1,101746,0.1037049773],[1,101792,0.0267083127],[1,101799,0.0231575112],[1,101896,0.0443136322],[1,101969,0.012921061],[1,102000,0.0217083449],[1,102022,0.0381994964],[1,102026,0.0304860202],[1,102061,0.1923583554],[1,102100,0.0492548179],[1,102116,0.0120680489],[1,102127,0.0135224213],[1,102170,0.0414590416],[1,102188,0.028573306],[1,102206,0.5186429959],[1,102238,0.0372263392],[1,102260,0.0357427586],[1,102282,0.0146114816],[1,102309,0.0210228446],[1,102382,0.0298374727],[1,102451,0.0400146838],[1,102457,0.078838168],[1,102520,0.0597405886],[1,102521,0.0955854037],[1,102524,0.2720337874],[1,102534,0.2213907787],[1,102574,0.0529968292],[1,102591,0.0246297259],[1,102605,0.0161716875],[1,102628,0.0717750412],[1,102638,0.026327292],[1,102643,0.0373840777],[1,102644,0.0133837735],[1,102647,0.0364975568],[1,102704,0.4602909984],[1,102769,0.0142909336],[1,102770,0.0547362676],[1,102795,0.0535142684],[1,102802,0.0152336262],[1,102878,0.0329134849],[1,102879,0.0122834625],[1,102898,0.0150398097],[1,102902,0.0315732533],[1,102910,0.0415932221],[1,102926,0.0421390042],[1,102930,0.0465614617],[1,102952,0.4907556752],[1,102956,0.2254738865],[1,102996,0.104809196],[1,103000,0.0366949121],[1,103029,0.0465442631],[1,103052,0.0163341157],[1,103079,0.0140974047],[1,103126,0.1541169648],[1,103129,0.0185425946],[1,103140,0.0562221265],[1,103211,0.0271794514],[1,103228,0.32906386],[1,103238,0.0339069283],[1,103249,0.0134492234],[1,103289,0.0438150005],[1,103301,0.0771029268],[1,103320,0.5386402399],[1,103327,0.0264132469],[1,103338,0.085757904],[1,103342,0.0148505282],[1,103343,0.1872660363],[1,103372,0.0115634536],[1,103388,0.0581584031],[1,103397,0.0335604288],[1,103406,0.041513794],[1,103431,0.0452744745],[1,103465,0.2153895065],[1,103469,0.0885980194],[1,103472,0.040164512],[1,103484,0.0133837735],[1,103533,0.0516846106],[1,103562,0.0458557922],[1,103595,0.0512747371],[1,103615,0.0513817146],[1,103644,0.1932708453],[1,103645,0.0081723332],[1,103701,0.5531780663],[1,103703,0.0852170484],[1,103705,0.0161320777],[1,103737,0.0443717186],[1,103793,0.0443520792],[1,103799,0.0284398663],[1,103901,0.480076343],[1,103914,0.0261287265],[1,103923,0.0157853131],[1,103955,0.1153231462],[1,103983,0.0850534585],[1,103989,0.0177691035],[1,104036,0.0139154106],[1,104098,0.034565933],[1,104132,0.0358650051],[1,104193,0.0222457416],[1,104194,0.0330250374],[1,104198,0.3218161447],[1,104201,0.0164881182],[1,104204,0.0785256346],[1,104257,0.028696414],[1,104265,0.0956692958],[1,104290,0.4817587717],[1,104319,0.036566711],[1,104330,0.010653043],[1,104358,0.0866653744],[1,104367,0.1238214669],[1,104396,0.0162083896],[1,104410,0.0690454503],[1,104424,0.0161320777],[1,104440,0.1242463435],[1,104494,0.0157273319],[1,104496,0.026049976],[1,104519,0.2094619562],[1,104522,0.0133837735],[1,104539,0.5504043423],[1,104551,0.0467520818],[1,104590,0.0149650936],[1,104610,0.0312447956],[1,104639,0.1394858556],[1,104662,0.0612047695],[1,104665,0.1093178569],[1,104675,0.0436658271],[1,104681,0.0247491163],[1,104683,0.0168104292],[1,104728,0.0376900687],[1,104762,0.1507631241],[1,104779,0.0264283159],[1,104819,0.0161320777],[1,104821,0.3808537811],[1,104894,0.0148753371],[1,104901,0.0632195089],[1,104906,0.0320693091],[1,104927,0.0261528702],[1,104945,0.0341503495],[1,104974,0.0628119962],[1,104978,0.0284482871],[1,104987,0.0295763724],[1,105019,0.0382040581],[1,105024,0.0296399949],[1,105050,0.0386785907],[1,105078,0.0092192964],[1,105114,0.0757070931],[1,105142,0.1192307745],[1,105146,0.0346758923],[1,105179,0.0142868272],[1,105244,0.2302475762],[1,105246,0.0161262335],[1,105261,0.0141725581],[1,105263,0.3429811531],[1,105270,0.0303704251],[1,105294,0.1722111894],[1,105344,0.0155672279],[1,105361,0.0577483844],[1,105404,0.2540240419],[1,105428,0.0196661494],[1,105445,0.015876716],[1,105467,0.0134492234],[1,105516,0.0283478731],[1,105544,0.0175891115],[1,105554,0.0161320777],[1,105568,0.0460847327],[1,105581,0.0146110171],[1,105622,0.042724867],[1,105636,0.0475964465],[1,105655,0.0797297809],[1,105660,0.0366891118],[1,105672,0.0128093603],[1,105677,0.0152944488],[1,105752,0.2609086572],[1,105774,0.0217755478],[1,105797,0.1258747696],[1,105823,0.0278861304],[1,105837,0.097152132],[1,105845,0.0932976487],[1,105852,0.0162170187],[1,105865,0.0169467503],[1,105871,0.0423967699],[1,105921,0.0156615353],[1,105940,0.0203074388],[1,105965,0.041513794],[1,106012,0.1363470714],[1,106016,0.0139615149],[1,106025,0.0160829193],[1,106057,0.0779261912],[1,106058,0.0180153298],[1,106064,0.1292160304],[1,106126,0.0133815479],[1,106175,0.2195434316],[1,106180,0.0129179232],[1,106248,0.0167383652],[1,106295,0.0860033655],[1,106302,0.0405646952],[1,106319,0.0105630076],[1,106341,0.0276351738],[1,106355,0.0155783962],[1,106359,0.0831296042],[1,106398,0.0157571729],[1,106408,0.1393856409],[1,106418,0.0152266738],[1,106419,0.1839343341],[1,106430,0.0139308979],[1,106435,0.06245079],[1,106440,0.0161039242],[1,106445,0.1430131782],[1,106449,0.0366707357],[1,106455,0.0130843609],[1,106470,0.0449952283],[1,106471,0.0490749663],[1,106474,0.0255062766],[1,106481,0.1763085049],[1,106557,0.0160547678],[1,106565,0.0160980802],[1,106602,0.0475857978],[1,106639,0.4057273416],[1,106640,0.1123360768],[1,106795,0.016688124],[1,106808,0.05292837],[1,106809,0.0112247057],[1,106815,0.1389568125],[1,106835,0.0161621799],[1,106853,0.016688124],[1,106869,0.0333854621],[1,106904,0.0193388653],[1,106917,0.0164233241],[1,106972,0.017442154],[1,106975,0.0190655018],[1,107048,0.0137598886],[1,107057,0.2228173527],[1,107090,0.032505933],[1,107229,0.0134595238],[1,107282,0.0576062255],[1,107308,0.013937749],[1,107380,0.0399365891],[1,107440,0.0186815583],[1,107448,0.0275800286],[1,107549,0.2487105147],[1,107577,0.012282659],[1,107580,0.0070058975],[1,107655,0.013496557],[1,107664,0.0272639774],[1,107665,0.0191309669],[1,107718,0.0881680421],[1,107772,0.0184717991],[1,107774,0.1393937253],[1,107811,0.0343310439],[1,107826,0.030915313],[1,107889,0.0202686515],[1,107890,0.0387876258],[1,107898,0.0226071151],[1,107951,0.0129728858],[1,107959,0.0149741179],[1,107997,0.0753542252],[1,107998,0.0197338763],[1,108084,0.0399943737],[1,108110,0.0152399288],[1,108130,0.013496557],[1,108165,0.0113312087],[1,108206,0.0164979039],[1,108219,0.0079752184],[1,108223,0.0099516476],[1,108227,0.0074849053],[1,108258,0.1885921987],[1,108307,0.0876319044],[1,108310,0.0119191855],[1,108333,0.0272237278],[1,108355,0.0193915207],[1,108373,0.0119160922],[1,108394,0.0561371691],[1,108414,0.0108155263],[1,108442,0.0698774213],[1,108479,0.0239235431],[1,108555,0.013496557],[1,108559,0.0619862766],[1,108568,0.0387016524],[1,108655,0.0376682968],[1,108691,0.0153570297],[1,108757,0.010758068],[1,108787,0.0137840708],[1,108811,0.0107734014],[1,108834,0.1682878428],[1,108852,0.1216722485],[1,108902,0.0092236889],[1,109014,0.1313210334],[1,109016,0.010758068],[1,109035,0.0743020829],[1,109038,0.0095027796],[1,109061,0.0102842949],[1,109083,0.0098592612],[1,109102,0.0064062635],[1,109106,0.0070147917],[1,109132,0.0087761653],[1,109181,0.0520629126],[1,109188,0.0117362787],[1,109212,0.0453331864],[1,109250,0.0074770158],[1,109267,0.1742841126],[1,109268,0.0240345182],[1,109399,0.028890538],[1,109442,0.0075688971],[1,109478,0.3866476114],[1,109511,0.0275154264],[1,109522,0.0260173166],[1,109541,0.0149621453],[1,109543,0.0163106339],[1,109599,0.0345384095],[1,109624,0.005791726],[1,109658,0.0296286875],[1,109660,0.0075097594],[1,109715,0.2111530922],[1,109717,0.0545453079],[1,109718,0.0959985335],[1,109763,0.0473558461],[1,109875,0.2766038421],[1,109884,0.0087271713],[1,109921,0.0346302196],[1,109922,0.0097275528],[1,109936,0.254638313],[1,109947,0.021919598],[1,109965,0.0053028011],[1,109975,0.4278028866],[1,110045,0.0053028011],[1,110051,0.0272626232],[1,110083,0.0566144897],[1,110092,0.0327304175],[1,110160,0.0077934733],[1,110167,0.005310317],[1,110176,0.0252754564],[1,110180,0.0120203811],[1,110193,0.0098183177],[1,110206,0.0091830014],[1,110207,0.0082104889],[1,110221,0.0046955862],[1,110278,0.0342540876],[1,110289,0.0339245158],[1,110292,0.0267341122],[1,110307,0.0574048472],[1,110329,0.0431246744],[1,110353,0.0137246837],[1,110358,0.046256968],[1,110368,0.0273520279],[1,110398,0.0330564325],[1,110499,0.0331857116],[1,110509,0.0809742533],[1,110542,0.0695929253],[1,110584,0.0176106108],[1,110632,0.025162531],[1,110652,0.6179686902],[1,110653,0.0476002788],[1,110689,0.0413354106],[1,110694,0.0115704492],[1,110711,0.0344474279],[1,110730,0.0700988418],[1,110788,0.0120250056],[1,110800,0.0094061795],[1,110803,0.0049540816],[1,110815,0.0814991894],[1,110854,0.0079559309],[1,110905,0.0091259978],[1,111023,0.0091316399],[1,111062,0.0332169429],[1,111115,0.1717884749],[1,111200,0.029618848],[1,111212,0.3024247215],[1,111214,0.0441724297],[1,111260,0.4472038479],[1,111265,0.0562770004],[1,111266,0.0953205648],[1,111280,0.0805097634],[1,111317,0.0082104889],[1,111319,0.0105463597],[1,111387,0.0316033639],[1,111424,0.2083025452],[1,111469,0.1610735816],[1,111588,0.0089015016],[1,111666,0.2543877504],[1,111733,0.0079559309],[1,111750,0.0053028011],[1,111758,0.2159931395],[1,111761,0.1650976863],[1,111812,0.0220808604],[1,111844,0.0065420564],[1,111904,0.0332224699],[1,111909,0.0223364629],[1,111918,0.0099175141],[1,111928,0.0442769991],[1,111966,0.0696925354],[1,111972,0.0253932019],[1,112037,0.0077934733],[1,112074,0.0052380763],[1,112112,0.0720648562],[1,112147,0.055499818],[1,112194,0.0503419061],[1,112225,0.018395882],[1,112269,0.3222887582],[1,112270,0.0181039296],[1,112272,0.1615854531],[1,112295,0.0303970453],[1,112337,0.0074362056],[1,112342,0.0061871909],[1,112345,0.0112206699],[1,112376,0.0358192499],[1,112501,0.0105340883],[1,112598,0.0315247869],[1,112617,0.0307226985],[1,112623,0.4393933509],[1,112640,0.015916522],[1,112659,0.0072930753],[1,112692,0.0072830637],[1,112695,0.1126728186],[1,112712,0.0073820582],[1,112751,0.0187935078],[1,112773,0.0099456074],[1,112827,0.0327916878],[1,112828,0.0023015001],[1,112881,0.0312639792],[1,112919,0.0161315323],[1,112925,0.0074628009],[1,112931,0.052116956],[1,112934,0.0176612921],[1,112959,0.0073820582],[1,112999,0.019259118],[1,113003,0.0023015001],[1,113076,0.0270509023],[1,113093,0.0560915133],[1,113141,0.0028398482],[1,113142,0.0368293206],[1,113156,0.0117531012],[1,113198,0.0140674728],[1,113222,0.0567920898],[1,113232,0.0075588181],[1,113237,0.0392432091],[1,113290,0.0263942714],[1,113295,0.0106591008],[1,113328,0.0283534783],[1,113353,0.2250552833],[1,113396,0.0249855349],[1,113401,0.0258389529],[1,113491,0.1073821212],[1,113497,0.0892261053],[1,113512,0.0075749515],[1,113560,0.0040927664],[1,113604,0.0597601729],[1,113622,0.0110666347],[1,113677,0.0076725039],[1,113684,0.0605490021],[1,113736,0.0058496065],[1,113743,0.0716178939],[1,113757,0.01868381],[1,113770,0.0404413437],[1,113790,0.0136652302],[1,113808,0.0177384986],[1,113856,0.0109716003],[1,113895,0.0177570319],[1,113898,0.0136904544],[1,113953,0.002894048],[1,113995,0.0043825694],[1,114009,0.0221372698],[1,114028,0.0725110709],[1,114036,0.0141678657],[1,114037,0.017381288],[1,114066,0.0319699465],[1,114097,0.0540430514],[1,114140,0.0211384397],[1,114158,0.0151636402],[1,114173,0.081944082],[1,114186,0.069194104],[1,114198,0.2508722033],[1,114201,0.0009053249],[1,114228,0.2515215109],[1,114242,0.085451828],[1,114264,0.1344023903],[1,114266,0.2594459516],[1,114308,0.2378434712],[1,114310,0.0250635704],[1,114321,0.0279346968],[1,114326,0.0042058982],[1,114374,0.0068627416],[1,114408,0.0037554791],[1,114416,0.0019066317],[1,114435,0.0695532385],[1,114439,0.0131471929],[1,114450,0.4857482959],[1,114475,0.0009053249],[1,114516,0.1111122036],[1,114555,0.0032250484],[1,114562,0.0068627416],[1,114563,0.2720773837],[1,114600,0.0354496115],[1,114643,0.0279947904],[1,114669,0.1798275948],[1,114690,0.0195053205],[1,114698,0.0231160039],[1,114764,0.0066041436],[1,114854,0.0092815339],[1,114864,0.0641192289],[1,114971,0.1528662216],[1,114992,0.1561926251],[1,114998,0.1835747342],[1,115055,0.0099511985],[1,115064,0.0023628908],[1,115074,0.0281917654],[1,115120,0.1750083117],[1,115142,0.0073820582],[1,115191,0.0009053249],[1,115192,0.0115674045],[1,115229,0.0196393264],[1,115246,0.0181059223],[1,115275,0.0206777248],[1,115344,0.0028398482],[1,115399,0.0386098182],[1,115404,0.0009053249],[1,115424,0.1145404086],[1,115465,0.0060160538],[1,115547,0.0028941652],[1,115595,0.0068627416],[1,115606,0.1756548165],[1,115640,0.0231710815],[1,115676,0.0738483217],[1,115684,0.0221167783],[1,115696,0.0245323527],[1,115712,0.0447542402],[1,115783,0.0983859462],[1,115788,0.0046641417],[1,115807,0.0073820582],[1,115854,0.0050324491],[1,115859,0.060582424],[1,115920,0.0163725509],[1,115996,0.0138030409],[1,116033,0.0058794919],[1,116034,0.2923404404],[1,116045,0.0073820582],[1,116052,0.4282042658],[1,116187,0.0074659921],[1,116240,0.7168751093],[1,116252,0.1876102686],[1,116278,0.0030997681],[1,116283,0.0030772293],[1,116427,0.7265051982],[1,116435,0.0324909742],[1,116444,0.1314884108],[1,116453,0.016628785],[1,116455,0.015762715],[1,116497,0.0036363115],[1,116509,0.5101792826],[1,116513,0.0314411326],[1,116549,0.0313174489],[1,116582,0.0108622907],[1,116603,0.037279132],[1,116612,0.0211217607],[1,116625,0.014892256],[1,116723,0.3210734854],[1,116745,0.008951651],[1,116814,0.0313313068],[1,116829,0.0150059336],[1,116838,0.0049019592],[1,116846,0.002074303],[1,116898,0.5333952215],[1,116904,0.0668374176],[1,116962,0.0696463739],[1,116966,0.0229506457],[1,117035,0.0321622392],[1,117040,0.0070809296],[1,117086,0.0145291404],[1,117097,0.0070914703],[1,117111,0.0331894251],[1,117133,0.1684039745],[1,117207,0.0089114855],[1,117210,0.0317385999],[1,117223,0.0127828813],[1,117256,0.0224548113],[1,117281,0.0217424246],[1,117292,0.0044153128],[1,117315,0.0139887231],[1,117338,0.0639495466],[1,117346,0.0154800139],[1,117392,0.0388193612],[1,117422,0.5572705218],[1,117438,0.0069841855],[1,117467,0.0049703489],[1,117471,0.0352877884],[1,117496,0.037374311],[1,117518,0.0233945001],[1,117630,0.0125918645],[1,117634,0.0184367966],[1,117736,0.2737443249],[1,117742,0.0737391778],[1,117752,0.0073457267],[1,117779,0.1507394528],[1,117790,0.0432952516],[1,117791,0.0318733101],[1,117802,0.3334623604],[1,117804,0.0328834606],[1,117807,0.0070914703],[1,117813,0.0043768352],[1,117824,0.4444642724],[1,117830,0.1084627878],[1,117833,0.0295074242],[1,117837,0.0032900039],[1,117844,0.0276385143],[1,117860,0.3007872483],[1,117893,0.8101546853],[1,117897,0.1006205813],[1,117926,0.0189213955],[1,117929,0.0060901499],[1,117933,0.4146630758],[1,117959,0.0666433143],[1,117972,0.0354457691],[1,117978,0.0260506838],[1,117980,0.5105790139],[1,118097,0.0559440168],[1,118114,0.3538818341],[1,118116,0.0044414833],[1,118285,0.251608757],[1,118294,0.0259697688],[1,118339,0.006675417],[1,118492,0.0356003111],[1,118521,0.0356416211],[1,118659,0.0086179924],[1,118667,0.0241800744],[1,118684,0.4440084995],[1,118711,0.0070914703],[1,118717,0.0206534654],[1,118734,0.0273454504],[1,118742,0.0355442278],[1,118752,0.0636532849],[1,118755,0.0321509375],[1,118776,0.1496818147],[1,118786,0.6148925669],[1,118821,0.0108996892],[1,118827,0.0057538655],[1,118852,0.018875159],[1,118856,0.0282914909],[1,118859,0.3641071791],[1,118864,0.0055278386],[1,118876,0.1840252914],[1,118878,0.1987347943],[1,118915,0.076066646],[1,118974,0.836676729],[1,119005,0.2806335957],[1,119006,0.0323690991],[1,119007,0.0566004734],[1,119009,0.0271864768],[1,119052,0.0395766762],[1,119057,0.673697286],[1,119112,0.0055596293],[1,119117,0.0086179924],[1,119165,0.0925826655],[1,119170,0.0800763358],[1,119172,0.0171327332],[1,119179,0.1415443924],[1,119197,0.0082173969],[1,119198,0.079482207],[1,119203,0.0070914703],[1,119222,0.1240719698],[1,119225,0.0029607203],[1,119231,0.0169738249],[1,119239,0.1413482167],[1,119265,0.0065132086],[1,119319,0.0283747207],[1,119341,0.0988533709],[1,119347,0.0090507299],[1,119371,0.0242973455],[1,119414,0.0848974021],[1,119428,0.0792741795],[1,119460,0.4488678996],[1,119463,0.1353781522],[1,119483,0.0516442629],[1,119487,0.0222481462],[1,119520,0.0045539878],[1,119526,0.0505265739],[1,119539,0.0374788794],[1,119585,0.0113597883],[1,119636,0.1044146127],[1,119699,0.0468529131],[1,119795,0.0393410561],[1,119803,0.0148210855],[1,119826,0.024753794],[1,119829,0.0020189407],[1,119838,0.00438594],[1,119847,0.0863178948],[1,119854,0.0066932341],[1,119881,0.054389285],[1,119885,0.6214983411],[1,119921,0.0445985733],[1,120006,0.0070357244],[1,120070,0.0034108676],[1,120071,0.0373363096],[1,120146,0.019146109],[1,120167,0.0026290609],[1,120184,0.1171752356],[1,120185,0.0126647807],[1,120201,0.0253559984],[1,120214,0.1511882461],[1,120220,0.0272234753],[1,120223,0.0292424776],[1,120293,0.0231078493],[1,120299,0.1843956416],[1,120342,0.0065166544],[1,120359,0.0167182943],[1,120370,0.1150388929],[1,120387,0.8068195446],[1,120395,0.0681258676],[1,120400,0.0052518032],[1,120438,0.0991805924],[1,120469,0.0070357244],[1,120492,0.00438594],[1,120518,0.0784891002],[1,120537,0.0037794907],[1,120540,0.1184023597],[1,120646,0.1133343376],[1,120655,0.0217934956],[1,120698,0.033071572],[1,120738,0.1386802793],[1,120753,0.3802140525],[1,120756,0.2014813522],[1,120765,0.0400502879],[1,120766,0.0379943984],[1,120770,0.040778085],[1,120774,0.1697424054],[1,120776,0.0066932341],[1,120801,0.449017341],[1,120811,0.0124585111],[1,120851,0.0268141275],[1,120854,0.0020894514],[1,120866,0.0338293807],[1,120889,0.0203643407],[1,120898,0.0025498457],[1,120913,0.059045969],[1,120955,0.0212996476],[1,120999,0.0118880629],[1,121000,0.1663678672],[1,121057,0.0297134444],[1,121059,0.2437003922],[1,121077,0.0027584423],[1,121116,0.1481501509],[1,121128,0.0365680925],[1,121141,0.0867998941],[1,121197,0.2842490167],[1,121209,0.0085237124],[1,121211,0.0449950576],[1,121244,0.02008601],[1,121268,0.0307341229],[1,121285,0.024354813],[1,121351,0.1047914739],[1,121356,0.1356572727],[1,121384,0.0973999102],[1,121427,0.046383903],[1,121431,0.0032402221],[1,121500,0.0281271228],[1,121513,0.0052610862],[1,121515,0.0690420809],[1,121632,0.2653211285],[1,121639,0.1741713413],[1,121657,0.0233537705],[1,121661,0.2248227191],[1,121704,0.0529639123],[1,121713,0.1387164506],[1,121725,0.5346417363],[1,121730,0.0319667687],[1,121783,0.0256012712],[1,121799,0.0076003638],[1,121806,0.0032531009],[1,121941,0.0060756873],[1,121943,0.2050541477],[1,121949,0.1134564942],[1,121985,0.1225408774],[1,122017,0.0915383483],[1,122049,0.0222571053],[1,122052,0.0221485119],[1,122062,0.0144229877],[1,122071,0.1627313446],[1,122132,0.0129943153],[1,122154,0.0111584404],[1,122210,0.0629142776],[1,122226,0.0235169226],[1,122240,0.0415667002],[1,122259,0.0890995498],[1,122294,0.2375983987],[1,122298,0.0493393546],[1,122305,0.1766257881],[1,122351,0.0317153932],[1,122359,0.0048680623],[1,122362,0.1715868508],[1,122367,0.2129146412],[1,122374,0.7050771184],[1,122412,0.1105796191],[1,122506,0.0236199755],[1,122508,0.3450380453],[1,122509,0.02458332],[1,122511,0.0435415416],[1,122524,0.0119735023],[1,122549,0.011109825],[1,122554,0.1043042401],[1,122576,0.034923376],[1,122590,0.0044960227],[1,122599,0.4660522307],[1,122615,0.0210407486],[1,122620,0.0079355709],[1,122622,0.0043429627],[1,122683,0.2622729996],[1,122716,0.1946730523],[1,122728,0.54805882],[1,122747,0.0396027798],[1,122757,0.0367675077],[1,122764,0.0029765391],[1,122778,0.024240826],[1,122816,0.049566086],[1,122817,0.2637256786],[1,122829,0.020099155],[1,122837,0.4915575311],[1,122844,0.0479033641],[1,122853,0.0269212878],[1,122874,0.0076003638],[1,122897,0.0309474469],[1,122902,0.054709094],[1,122931,0.0043429627],[1,122935,0.0222571053],[1,122966,0.0034489251],[1,122982,0.1180889034],[1,122992,0.0308248409],[1,123011,0.0578764905],[1,123041,0.0274826113],[1,123050,0.0784879963],[1,123074,0.0067333699],[1,123075,0.0123469108],[1,123128,0.3558584917],[1,123133,0.003869483],[1,123160,0.0597243063],[1,123187,0.0043429627],[1,123195,0.0555512559],[1,123198,0.0061065767],[1,123199,0.0099750222],[1,123215,0.0347492796],[1,123218,0.0609775878],[1,123219,0.0273293464],[1,123232,0.0037365494],[1,123258,0.040286206],[1,123271,0.0220817548],[1,123279,0.1005362948],[1,123280,0.0040200684],[1,123296,0.0724645318],[1,123330,0.0583453571],[1,123376,0.0077740116],[1,123482,0.0323837884],[1,123487,0.0263150352],[1,123493,0.1581895159],[1,123539,0.0187811668],[1,123571,0.0396388669],[1,123591,0.018890428],[1,123600,0.1775369423],[1,123644,0.2583107671],[1,123649,0.0336401401],[1,123653,0.0218434545],[1,123677,0.0045449345],[1,123685,0.0115129637],[1,123724,0.0035071268],[1,123725,0.3952092789],[1,123765,0.1058007044],[1,123780,0.1618050415],[1,123786,0.0082005189],[1,123788,0.2093144288],[1,123824,0.042846293],[1,123825,0.2174442786],[1,123826,0.0287589762],[1,123828,0.0501479548],[1,123841,0.0061533438],[1,123848,0.2327660691],[1,123879,0.040776226],[1,123884,0.1219125807],[1,123910,0.0587440428],[1,123924,0.0024994916],[1,123948,0.0043429627],[1,123963,0.0003596752],[1,123992,0.1015803138],[1,124035,0.155739586],[1,124064,0.0048011767],[1,124072,0.1530688272],[1,124082,0.0069769346],[1,124113,0.030720703],[1,124162,0.0328664736],[1,124209,0.0045449345],[1,124256,0.0111081198],[1,124269,0.0042612105],[1,124280,0.0057345004],[1,124290,0.0671008668],[1,124339,0.0518981663],[1,124347,0.0962915107],[1,124488,0.0101270925],[1,124491,0.0614761058],[1,124501,0.0130400999],[1,124512,0.0946979768],[1,124532,0.03349834],[1,124596,0.0233182034],[1,124620,0.0191908002],[1,124637,0.1398798882],[1,124675,0.0910662651],[1,124700,0.2795128533],[1,124729,0.1172463705],[1,124766,0.0511406198],[1,124767,0.016352047],[1,124811,0.013852653],[1,124832,0.287885925],[1,124847,0.0879847353],[1,124879,0.0342274662],[1,124883,0.0316284999],[1,124923,0.0252592956],[1,124939,0.0251898277],[1,124940,0.0299259681],[1,124977,0.0171408159],[1,125052,0.0037365494],[1,125063,0.0037365494],[1,125127,0.0372331733],[1,125142,0.0084772392],[1,125144,0.1099324335],[1,125145,0.0580584289],[1,125152,0.2647954782],[1,125168,0.03271262],[1,125197,0.0410382049],[1,125217,0.0048011767],[1,125322,0.0739555727],[1,125324,0.1750488071],[1,125371,0.0676305891],[1,125376,0.5271771474],[1,125397,0.0010416509],[1,125456,0.0326416102],[1,125478,0.2571208185],[1,125551,0.0033497387],[1,125558,0.0104675184],[1,125582,0.0057345004],[1,125595,0.0312316637],[1,125609,0.1967024523],[1,125655,0.0086999339],[1,125664,0.0691422116],[1,125686,0.0062145153],[1,125730,0.0252693482],[1,125731,0.0299415158],[1,125735,0.4854036973],[1,125745,0.017863817],[1,125747,0.0048011767],[1,125764,0.0309132515],[1,125785,0.055985345],[1,125786,0.3472689415],[1,125798,0.0378008166],[1,125825,0.0198364474],[1,125835,0.0094800162],[1,125911,0.0321203956],[1,125934,0.0048680623],[1,125952,0.071407315],[1,125955,0.0333650574],[1,125995,0.0048680623],[1,126000,0.0221930395],[1,126003,0.005293681],[1,126015,0.0043429627],[1,126029,0.0142797165],[1,126077,0.401637072],[1,126091,0.2094037973],[1,126135,0.0037011848],[1,126138,0.0103773978],[1,126147,0.0169171457],[1,126175,0.0089335183],[1,126185,0.0923410766],[1,126212,0.3087879369],[1,126238,0.0157711686],[1,126250,0.0340614514],[1,126252,0.0732768754],[1,126256,0.2986773175],[1,126299,0.190519141],[1,126303,0.2153658209],[1,126306,0.0180745503],[1,126307,0.2464942203],[1,126338,0.0048011767],[1,126345,0.0415394014],[1,126350,0.016296834],[1,126356,0.0255306147],[1,126361,0.0048680623],[1,126386,0.0074084026],[1,126388,0.0057345004],[1,126413,0.0052610862],[1,126429,0.0365501401],[1,126438,0.0101223641],[1,126447,0.0063910218],[1,126454,0.0126072219],[1,126488,0.0447022052],[1,126503,0.0422218993],[1,126557,0.1174070623],[1,126560,0.0374599544],[1,126583,0.1002429031],[1,126596,0.0002943032],[1,126601,0.0284590933],[1,126603,0.1305918536],[1,126622,0.0383669568],[1,126625,0.0974088255],[1,126626,0.0297180923],[1,126654,0.3353917412],[1,126686,0.0210466561],[1,126691,0.0338862506],[1,126710,0.077233927],[1,126714,0.0099766477],[1,126727,0.006097763],[1,126731,0.0309746355],[1,126736,0.0049524573],[1,126739,0.0056889129],[1,126741,0.3583313856],[1,126797,0.0035632491],[1,126833,0.0017760461],[1,126859,0.0107169328],[1,126871,0.076613875],[1,126880,0.0037159066],[1,126913,0.3756108221],[1,126919,0.0109543261],[1,126934,0.0269712454],[1,126962,0.0031846041],[1,126994,0.027751282],[1,126995,0.0127928645],[1,127014,0.0593253128],[1,127033,0.0219546703],[1,127068,0.0250788529],[1,127070,0.0191627361],[1,127083,0.3758121942],[1,127101,0.0027211011],[1,127108,0.6047546348],[1,127127,0.0029014114],[1,127141,0.164181567],[1,127160,0.0061682899],[1,127168,0.0048680623],[1,127196,0.0037159066],[1,127208,0.0123939151],[1,127240,0.0031100169],[1,127246,0.0107334617],[1,127287,0.0105688099],[1,127300,0.0355460466],[1,127306,0.034336529],[1,127319,0.0232663739],[1,127344,0.0466195036],[1,127361,0.3163911011],[1,127371,0.0246445936],[1,127398,0.0313979025],[1,127420,0.0156024517],[1,127462,0.0082495606],[1,127467,0.3475874973],[1,127524,0.0191214397],[1,127537,0.0072082406],[1,127550,0.0063383508],[1,127614,0.0057103465],[1,127622,0.0003047543],[1,127641,0.2077132322],[1,127658,0.0107111161],[1,127672,0.026646883],[1,127729,0.0088905534],[1,127737,0.0048162754],[1,127745,0.0190282924],[1,127783,0.0169930386],[1,127863,0.057522169],[1,127870,0.0035032065],[1,127871,0.0122181441],[1,127892,0.0376422776],[1,127915,0.0343899363],[1,127931,0.0370033604],[1,127970,0.02139817],[1,127971,0.0072082406],[1,128013,0.015177512],[1,128025,0.0172306149],[1,128027,0.1905435731],[1,128034,0.0190853532],[1,128044,0.0033536558],[1,128058,0.0072082406],[1,128065,0.0076003638],[1,128075,0.0318969838],[1,128117,0.1350788488],[1,128120,0.0081947993],[1,128128,0.0591748311],[1,128138,0.0053155143],[1,128171,0.0623643153],[1,128203,0.0133172105],[1,128209,0.0829267318],[1,128244,0.0044931706],[1,128304,0.0153408564],[1,128332,0.0326393205],[1,128343,0.0216548953],[1,128381,0.0367948751],[1,128437,0.0612588512],[1,128442,0.0348018114],[1,128447,0.3795638152],[1,128450,0.0488313973],[1,128504,0.046918842],[1,128507,0.0048162754],[1,128516,0.0048161593],[1,128560,0.0479405719],[1,128566,0.0343899363],[1,128571,0.3181929713],[1,128574,0.0106387006],[1,128596,0.0167547986],[1,128598,0.0072082406],[1,128602,0.0952439192],[1,128606,0.0066890478],[1,128628,0.0015128117],[1,128630,0.007434245],[1,128631,0.0874516447],[1,128724,0.0411110277],[1,128727,0.1827765122],[1,128729,0.0030308663],[1,128750,0.0391430531],[1,128777,0.0003047543],[1,128800,0.2577890852],[1,128860,0.2065820547],[1,128880,0.0930782719],[1,128908,0.0041888106],[1,128909,0.0961626097],[1,128943,0.0412519627],[1,128954,0.012831828],[1,128963,0.1204404305],[1,128977,0.00620872],[1,128981,0.0052610862],[1,128994,0.0348319361],[1,129083,0.0045915292],[1,129098,0.0636447926],[1,129100,0.0041888106],[1,129150,0.0111288362],[1,129166,0.0044931706],[1,129185,0.0688309028],[1,129198,0.0055413312],[1,129207,0.027171144],[1,129209,0.0162886713],[1,129214,0.0164107685],[1,129223,0.0835459621],[1,129237,0.0042992726],[1,129275,0.4305834156],[1,129307,0.014727139],[1,129317,0.16641389],[1,129318,0.2865256522],[1,129355,0.0029703266],[1,129380,0.0902274422],[1,129383,0.0309279749],[1,129390,0.0110689399],[1,129394,0.0121964463],[1,129422,0.6501575768],[1,129434,0.432336078],[1,129464,0.0054261672],[1,129493,0.0114734978],[1,129526,0.1730839009],[1,129528,0.0003047543],[1,129549,0.0086511261],[1,129580,0.1204724419],[1,129588,0.0616882528],[1,129636,0.094237078],[1,129653,0.1401354162],[1,129658,0.005293681],[1,129660,0.0718455618],[1,129682,0.0047344698],[1,129724,0.2073344158],[1,129731,0.0049660116],[1,129750,0.0108260301],[1,129787,0.0143066933],[1,129791,0.0328138979],[1,129793,0.0933040547],[1,129805,0.0264459734],[1,129806,0.3887602408],[1,129832,0.0287372177],[1,129849,0.014761791],[1,129872,0.036373773],[1,129877,0.3999298668],[1,129887,0.0332695932],[1,129890,0.307668054],[1,129910,0.0356783927],[1,129921,0.0965878394],[1,129985,0.0071757463],[1,129986,0.005274788],[1,129992,0.0115739648],[1,130021,0.0072923285],[1,130068,0.0496305061],[1,130070,0.0524739131],[1,130121,0.031497632],[1,130162,0.0015128117],[1,130198,0.0131131546],[1,130200,0.0017341193],[1,130209,0.0053155143],[1,130220,0.0479405719],[1,130235,0.092954233],[1,130261,0.0152705696],[1,130288,0.018214317],[1,130295,0.0156801522],[1,130312,0.322801723],[1,130313,0.0479497937],[1,130320,0.0617880436],[1,130328,0.0053417172],[1,130332,0.0322035091],[1,130340,0.4501469302],[1,130355,0.146610766],[1,130372,0.0094164929],[1,130378,0.2578949614],[1,130385,0.0223771286],[1,130387,0.0090687193],[1,130396,0.0412568008],[1,130435,0.0344952888],[1,130480,0.0288967838],[1,130507,0.0015128117],[1,130534,0.0068656693],[1,130536,0.0713784491],[1,130538,0.0054445564],[1,130569,0.009075523],[1,130603,0.1193061126],[1,130605,0.1380541347],[1,130670,0.1574439022],[1,130683,0.0090687193],[1,130685,0.1413361402],[1,130692,0.0053417172],[1,130703,0.1612083248],[1,130724,0.0775332991],[1,130727,0.0606249953],[1,130772,0.0243274994],[1,130805,0.0628538518],[1,130833,0.0426232539],[1,130836,0.0014514932],[1,130870,0.0692708243],[1,130872,0.003921655],[1,130903,0.0090687193],[1,130906,0.0307952655],[1,130910,0.4410829655],[1,130926,0.0072082406],[1,130927,0.0393697429],[1,130934,0.0064294192],[1,130996,0.0042094668],[1,131045,0.0132995371],[1,131097,0.0440890881],[1,131232,0.5636905944],[1,131272,0.0241266821],[1,131308,0.0053417172],[1,131332,0.0015528916],[1,131364,0.2645950308],[1,131382,0.0044390897],[1,131387,0.0072082406],[1,131395,0.0134372828],[1,131404,0.193371804],[1,131405,0.0263982699],[1,131413,0.097222328],[1,131437,0.0036571617],[1,131457,0.0050183788],[1,131471,0.0876291203],[1,131472,0.0478701102],[1,131502,0.0907482182],[1,131507,0.0306681633],[1,131516,0.4465617789],[1,131521,0.018933208],[1,131613,0.035589707],[1,131619,0.0625715529],[1,131621,0.0820546727],[1,131626,0.005806758],[1,131642,0.0068656693],[1,131665,0.0058324093],[1,131701,0.2078265936],[1,131816,0.0048162754],[1,131825,0.0035032065],[1,131834,0.0048162754],[1,131901,0.1041828472],[1,131902,0.0780219915],[1,131935,0.0348018114],[1,131981,0.2435267741],[1,131986,0.2338281161],[1,132003,0.0286126175],[1,132077,0.009075523],[1,132085,0.0574046808],[1,132087,0.035420157],[1,132094,0.0855170079],[1,132130,0.0072082406],[1,132141,0.0050183788],[1,132147,0.0410660678],[1,132151,0.0135207453],[1,132164,0.0066890478],[1,132190,0.0434400255],[1,132234,0.0072082406],[1,132262,0.2913984081],[1,132289,0.0348018114],[1,132333,0.003002378],[1,132357,0.0544632479],[1,132391,0.017176202],[1,132399,0.1026118069],[1,132444,0.0131189011],[1,132470,0.0072082406],[1,132484,0.4253658606],[1,132513,0.0061631028],[1,132534,0.0177953882],[1,132557,0.0305400112],[1,132567,0.1488086498],[1,132593,0.4140330481],[1,132607,0.0387098632],[1,132614,0.0797762327],[1,132622,0.0078403613],[1,132633,0.1584164053],[1,132636,0.0212669706],[1,132641,0.0054261672],[1,132688,0.0072082406],[1,132733,0.0920006445],[1,132765,0.0081947993],[1,132813,0.0074266542],[1,132822,0.0301529578],[1,132874,0.0031933566],[1,132937,0.1399854592],[1,132985,0.0550910035],[1,133053,0.0143623651],[1,133063,0.0145580174],[1,133076,0.0242693117],[1,133129,0.0125203224],[1,133164,0.0044931706],[1,133173,0.0045915292],[1,133224,0.0036370251],[1,133244,0.003002378],[1,133333,0.0133806618],[1,133364,0.0095234033],[1,133372,0.1864744393],[1,133378,0.0316413287],[1,133433,0.0084804651],[1,133529,0.1510329124],[1,133535,0.2092215168],[1,133567,0.0035032065],[1,133617,0.0082495606],[1,133631,0.0056434049],[1,133648,0.0356651826],[1,133665,0.0241355513],[1,133680,0.0362496405],[1,133701,0.0713153904],[1,133787,0.03354265],[1,133805,0.0208738361],[1,133807,0.0072158438],[1,133860,0.0075784201],[1,133877,0.2678333003],[1,133911,0.1429616958],[1,133938,0.2730768087],[1,133946,0.0138908607],[1,133990,0.0053417172],[1,134010,0.0346317536],[1,134024,0.0060870906],[1,134032,0.1428392507],[1,134116,0.0286022617],[1,134188,0.0064294192],[1,134201,0.0057531898],[1,134228,0.0015128117],[1,134282,0.0044641731],[1,134408,0.034726381],[1,134431,0.0090620639],[1,134500,0.0281584781],[1,134508,0.0044931706],[1,134552,0.0981217863],[1,134619,0.0213474185],[1,134634,0.2011956121],[1,134650,0.0173063718],[1,134683,0.012686858],[1,134728,0.0109543261],[1,134759,0.0506529106],[1,134830,0.0755946709],[1,134834,0.0035825259],[1,134853,0.0347346646],[1,134867,0.0166307178],[1,134892,0.0145368065],[1,134897,0.0078089682],[1,134914,0.0211603608],[1,134935,0.178942528],[1,134938,0.1265153926],[1,134943,0.008439195],[1,134944,0.0048162754],[1,134976,0.0146690354],[1,135022,0.1984482546],[1,135043,0.0052865043],[1,135067,0.0113571078],[1,135081,0.0400834314],[1,135094,0.0305711364],[1,135210,0.624870029],[1,135233,0.2327226421],[1,135236,0.0631386199],[1,135337,0.0137539645],[1,135375,0.0072082406],[1,135384,0.0073342765],[1,135392,0.0240654117],[1,135396,0.0035052449],[1,135435,0.4906273897],[1,135487,0.0056382412],[1,135496,0.0235912653],[1,135604,0.5043034877],[1,135629,0.0044931706],[1,135639,0.0044931706],[1,135721,0.2238768101],[1,135745,0.0328113026],[1,135754,0.15049264],[1,135755,0.0099748982],[1,135795,0.2576687136],[1,135803,0.0003047543],[1,135817,0.0072082406],[1,135854,0.0167287732],[1,135863,0.0254941145],[1,135867,0.023808732],[1,135945,0.0072082406],[1,135971,0.0076365623],[1,136070,0.078928931],[1,136075,0.0348598186],[1,136114,0.0377924464],[1,136244,0.0050183788],[1,136246,0.0780337184],[1,136250,0.0110106978],[1,136266,0.2649156759],[1,136309,0.0428234629],[1,136378,0.023287705],[1,136434,0.0131585061],[1,136454,0.0044931706],[1,136455,0.1701597291],[1,136516,0.0228092583],[1,136572,0.0469105997],[1,136609,0.1978512069],[1,136621,0.0216253216],[1,136635,0.0109615761],[1,136668,0.0267698676],[1,136729,0.0070461121],[1,136733,0.0215231035],[1,136738,0.1169057241],[1,136746,0.4825616082],[1,136782,0.0208245173],[1,136783,0.0068790062],[1,136794,0.0042452722],[1,136795,0.1697441374],[1,136810,0.2647249009],[1,136817,0.0069940414],[1,136821,0.024531795],[1,136847,0.3958932037],[1,136850,0.0766962065],[1,136852,0.0032522977],[1,136923,0.0061166893],[1,136925,0.0321995466],[1,136942,0.0],[1,136955,0.233897944],[1,136958,0.1935737496],[1,136975,0.004163647],[1,136999,0.0179951182],[1,137024,0.0074484537],[1,137025,0.0846731691],[1,137140,0.0084441799],[1,137142,0.0032958442],[1,137145,0.0309539354],[1,137179,0.353219593],[1,137186,0.0026237445],[1,137282,0.007735313],[1,137315,0.0066142529],[1,137335,0.0368412394],[1,137350,0.0104187306],[1,137363,0.0057662679],[1,137396,0.0066354743],[1,137400,0.1955157989],[1,137411,0.0076145082],[1,137446,0.019022381],[1,137476,0.0359713912],[1,137481,0.0141830902],[1,137523,0.0056047537],[1,137542,0.3802418315],[1,137548,0.0189813029],[1,137581,0.0036184164],[1,137619,0.0066436477],[1,137649,0.0282407254],[1,137712,0.0483125307],[1,137716,0.5263674811],[1,137844,0.0039225373],[1,137877,0.0076212578],[1,137881,0.0008832496],[1,137892,0.0061166893],[1,137901,0.009216066],[1,137948,0.0081723163],[1,137992,0.0061166893],[1,138000,0.0390308787],[1,138020,0.0066354743],[1,138031,0.2537383181],[1,138050,0.043335774],[1,138083,0.0048623173],[1,138089,0.3368569621],[1,138110,0.1563228765],[1,138135,0.032511283],[1,138157,0.3328852179],[1,138177,0.1019889859],[1,138204,0.006331584],[1,138212,0.0148194293],[1,138214,0.0331494506],[1,138239,0.0252785496],[1,138249,0.0030126081],[1,138291,0.1464222806],[1,138350,0.0047704173],[1,138352,0.1150435716],[1,138401,0.0036390564],[1,138403,0.0218500102],[1,138407,0.0237860658],[1,138410,0.0068537163],[1,138439,0.105549598],[1,138450,0.0731937883],[1,138486,0.0042453883],[1,138542,0.1725595605],[1,138550,0.0268591923],[1,138651,0.0039225373],[1,138656,0.0382394035],[1,138687,0.0072039749],[1,138693,0.2176847973],[1,138714,0.0183456728],[1,138765,0.1811046239],[1,138819,0.013365128],[1,138908,0.2461460147],[1,138934,0.2574399118],[1,138949,0.1535476963],[1,139008,0.007735313],[1,139015,0.1587060056],[1,139031,0.0147991966],[1,139038,0.0199923732],[1,139092,0.5946122616],[1,139097,0.0048548009],[1,139107,0.0483583776],[1,139113,0.0047563301],[1,139152,0.2876920223],[1,139168,0.0159953805],[1,139215,0.2999002656],[1,139277,0.0937365323],[1,139297,0.0178541494],[1,139300,0.0331008853],[1,139315,0.0148912044],[1,139316,0.1287263769],[1,139335,0.0242953912],[1,139384,0.003472462],[1,139396,0.3481888502],[1,139431,0.0304948406],[1,139434,0.0447106012],[1,139441,0.0106296042],[1,139473,0.0422234398],[1,139484,0.097615127],[1,139508,0.0224373113],[1,139520,0.0143442349],[1,139524,0.1950525088],[1,139541,0.0118983359],[1,139548,0.0439856252],[1,139554,0.0018232854],[1,139576,0.3055390395],[1,139579,0.006519164],[1,139582,0.0189849347],[1,139585,0.8939029946],[1,139620,0.2746272591],[1,139668,0.0138936408],[1,139670,0.0067788246],[1,139690,0.0028402518],[1,139692,0.1061588693],[1,139707,0.0210472284],[1,139718,0.0139768453],[1,139734,0.0137299766],[1,139746,0.0578632855],[1,139893,0.0083227417],[1,139915,0.0050311318],[1,139923,0.011991777],[1,139957,0.006997546],[1,139972,0.0190557526],[1,139995,0.0427066598],[1,139997,0.0375225546],[1,140009,0.0064662011],[1,140035,0.0281911131],[1,140081,0.0295635904],[1,140085,0.0076945811],[1,140095,0.0103863287],[1,140140,0.4062400516],[1,140191,0.0097288428],[1,140202,0.0064533649],[1,140206,0.1242940446],[1,140208,0.0976865188],[1,140293,0.0375798577],[1,140318,0.2494092943],[1,140332,0.4808594529],[1,140357,0.0126566462],[1,140360,0.0073216931],[1,140362,0.0321581156],[1,140393,0.37433256],[1,140422,0.0049753764],[1,140431,0.1301268694],[1,140472,0.0238271648],[1,140633,0.0257411608],[1,140651,0.0399998722],[1,140654,0.0150678705],[1,140677,0.0420387001],[1,140717,0.0072823393],[1,140751,0.0200260189],[1,140787,0.1045931205],[1,140804,0.2088203777],[1,140819,0.0142502682],[1,140828,0.0064791077],[1,140843,0.0363824838],[1,140846,0.0437518356],[1,140882,0.0419056296],[1,140976,0.0516511039],[1,141000,0.0680584919],[1,141055,0.0427166753],[1,141074,0.0083227417],[1,141092,0.0040796271],[1,141161,0.0251340891],[1,141203,0.0399171388],[1,141238,0.0121118808],[1,141259,0.0083227417],[1,141262,0.00570203],[1,141274,0.0080181475],[1,141297,0.0496154086],[1,141333,0.0331072271],[1,141360,0.0059271198],[1,141400,0.0256886699],[1,141460,0.2411718034],[1,141501,0.0024050296],[1,141505,0.0199848055],[1,141547,0.0410295568],[1,141639,0.0074765012],[1,141640,0.0184536692],[1,141645,0.11516361],[1,141656,0.0341372617],[1,141666,0.0194097572],[1,141733,0.0250965791],[1,141779,0.0083227417],[1,141795,0.0700145966],[1,141798,0.0193096511],[1,141834,0.0331390912],[1,141843,0.0176613792],[1,141882,0.0456562499],[1,141884,0.0110188117],[1,141911,0.177043736],[1,141915,0.4991254413],[1,141916,0.0399998722],[1,141952,0.0025571937],[1,141957,0.35039906],[1,141961,0.2242450298],[1,141974,0.1586429455],[1,142032,0.0706193695],[1,142034,0.038115312],[1,142041,0.0408728679],[1,142058,0.0069191165],[1,142106,0.0254426873],[1,142113,0.023272083],[1,142139,0.0206342809],[1,142142,0.0696021664],[1,142151,0.0079796467],[1,142156,0.0063535384],[1,142178,0.5499067641],[1,142235,0.0184557857],[1,142239,0.0716846816],[1,142256,0.0080415187],[1,142277,0.0052986957],[1,142296,0.0601724721],[1,142348,0.0461008887],[1,142416,0.5645823724],[1,142423,0.0283477864],[1,142424,0.0088116464],[1,142453,0.0173361029],[1,142466,0.0014087018],[1,142528,0.0041630969],[1,142531,0.0059271198],[1,142533,0.2298376518],[1,142618,0.7330270246],[1,142624,0.5261152062],[1,142649,0.0391909642],[1,142655,0.4516276563],[1,142671,0.0076045646],[1,142724,0.0300682337],[1,142858,0.0158878395],[1,142868,0.0219698889],[1,142912,0.006997546],[1,142944,0.0059271198],[1,142974,0.0658572553],[1,142982,0.0209506117],[1,142984,0.0095661731],[1,143000,0.062702647],[1,143001,0.0124839507],[1,143036,0.0076620681],[1,143104,0.0969009565],[1,143109,0.0231179064],[1,143124,0.0066430154],[1,143131,0.0215232962],[1,143148,0.0211746773],[1,143168,0.0123198931],[1,143196,0.0348048277],[1,143240,0.0203495237],[1,143351,0.0084065686],[1,143354,0.2380570433],[1,143357,0.0148327633],[1,143366,0.0364451437],[1,143468,0.0280908642],[1,143480,0.1810234452],[1,143504,0.2744499764],[1,143523,0.0382256762],[1,143554,0.1760225095],[1,143625,0.2446496266],[1,143686,0.0167795448],[1,143702,0.0283376923],[1,143704,0.0430958953],[1,143731,0.0106891275],[1,143787,0.0080828948],[1,143796,0.0266910284],[1,143800,0.0121907763],[1,143820,0.0086246966],[1,143850,0.1431329758],[1,143879,0.1948015495],[1,143894,0.2052883474],[1,143896,0.0089835839],[1,143919,0.0276850159],[1,143926,0.2961935182],[1,143947,0.0166971714],[1,143966,0.0534312299],[1,144003,0.0914121455],[1,144023,0.0103920655],[1,144031,0.0386111865],[1,144123,0.0958626111],[1,144148,0.0319067955],[1,144155,0.3714364881],[1,144166,0.204351444],[1,144195,0.0811150356],[1,144225,0.0101959453],[1,144284,0.0536274501],[1,144290,0.018725672],[1,144324,0.0076383443],[1,144341,0.2086350324],[1,144342,0.0237653338],[1,144350,0.0324325632],[1,144354,0.0130186989],[1,144362,0.0274837533],[1,144369,0.0743151126],[1,144384,0.0090040443],[1,144394,0.0204882175],[1,144411,0.059435011],[1,144415,0.0428295792],[1,144420,0.0161329752],[1,144445,0.0074697484],[1,144530,0.0204778561],[1,144562,0.1207218338],[1,144589,0.0113200253],[1,144614,0.0187280507],[1,144624,0.0094139321],[1,144633,0.0406832793],[1,144636,0.0059406635],[1,144646,0.4845118036],[1,144706,0.0098519701],[1,144736,0.0428295792],[1,144752,0.0430958953],[1,144757,0.0181914454],[1,144788,0.0071641411],[1,144817,0.007818279],[1,144875,0.00469939],[1,144901,0.4919044456],[1,144904,0.0840165474],[1,144937,0.0809196268],[1,144955,0.0302554553],[1,144996,0.0071641411],[1,145055,0.0098519701],[1,145062,0.0044771756],[1,145067,0.0117627631],[1,145138,0.2026355081],[1,145149,0.0071848819],[1,145172,0.0101959453],[1,145198,0.0307976665],[1,145229,0.5532512732],[1,145246,0.263468817],[1,145248,0.0032641673],[1,145267,0.285417966],[1,145272,0.0196570373],[1,145276,0.0095740674],[1,145292,0.055276226],[1,145295,0.0730309785],[1,145296,0.0326027401],[1,145301,0.1170587468],[1,145331,0.0093694888],[1,145359,0.0527285614],[1,145362,0.2686325589],[1,145375,0.0099330804],[1,145384,0.1103991978],[1,145392,0.0106332004],[1,145401,0.0851436131],[1,145488,0.0142821036],[1,145505,0.0096746247],[1,145509,0.0683818452],[1,145524,0.0110497015],[1,145599,0.2696820288],[1,145628,0.005107933],[1,145646,0.2135504035],[1,145657,0.0371064474],[1,145668,0.0449497172],[1,145683,0.4102106816],[1,145769,0.0311746023],[1,145789,0.0104867157],[1,145862,0.0088144756],[1,145899,0.0370738509],[1,145921,0.0102051764],[1,145983,0.0208601073],[1,145989,0.3804795057],[1,146012,0.0586338559],[1,146066,0.0302027053],[1,146088,0.0235686144],[1,146094,0.0272178797],[1,146098,0.0071848819],[1,146121,0.0387280755],[1,146162,0.0314359675],[1,146247,0.0247850991],[1,146255,0.0082216899],[1,146272,0.0835719778],[1,146277,0.0240484631],[1,146279,0.0431120661],[1,146348,0.0117773875],[1,146355,0.2628143988],[1,146387,0.2375451577],[1,146441,0.0091923285],[1,146450,0.0064202065],[1,146458,0.0200612934],[1,146465,0.0099100139],[1,146511,0.0677916857],[1,146626,0.4390518923],[1,146640,0.1603209605],[1,146662,0.0182706873],[1,146693,0.0097119001],[1,146724,0.0085243072],[1,146727,0.0152403576],[1,146775,0.0580996536],[1,146885,0.0278375601],[1,146976,0.0247696868],[1,147042,0.0925138054],[1,147055,0.0725331633],[1,147072,0.3319159083],[1,147084,0.0433613541],[1,147089,0.0993738099],[1,147090,0.0952900565],[1,147240,0.0196705484],[1,147287,0.0033744652],[1,147291,0.1408764771],[1,147315,0.0514066319],[1,147386,0.2763891037],[1,147395,0.047052253],[1,147403,0.0583827558],[1,147467,0.0072750322],[1,147472,0.0490922733],[1,147473,0.3269864801],[1,147477,0.0088609966],[1,147499,0.0649370422],[1,147528,0.0382379816],[1,147551,0.0182326491],[1,147557,0.0201144106],[1,147575,0.0063512783],[1,147576,0.0151940184],[1,147590,0.0110891646],[1,147632,0.017290104],[1,147655,0.1142134017],[1,147879,0.1224856248],[1,147909,0.0093655124],[1,147917,0.0482950952],[1,147920,0.0375061355],[1,147924,0.0473829448],[1,147934,0.0541664391],[1,147935,0.0396038669],[1,147970,0.0081473087],[1,147975,0.0066661703],[1,147984,0.0361426261],[1,148036,0.2856451197],[1,148081,0.2313840774],[1,148097,0.0242325206],[1,148128,0.2480172907],[1,148221,0.055910842],[1,148275,0.0483616993],[1,148285,0.0256026871],[1,148315,0.0488755778],[1,148434,0.0393869623],[1,148463,0.1023367569],[1,148492,0.0438381769],[1,148501,0.0194385348],[1,148516,0.008365626],[1,148604,0.018106183],[1,148627,0.0195554901],[1,148632,0.0061397714],[1,148641,0.0272721672],[1,148662,0.0924288507],[1,148698,0.0031669909],[1,148730,0.0184850457],[1,148765,0.4483092769],[1,148782,0.0034761333],[1,148838,0.0088609966],[1,148845,0.4820460605],[1,148887,0.0082211563],[1,148896,0.1155464106],[1,148945,0.0303396408],[1,148956,0.0048370107],[1,148964,0.0405123771],[1,148978,0.0061397714],[1,149008,0.036909398],[1,149012,0.0383466621],[1,149037,0.012856995],[1,149045,0.0033744652],[1,149079,0.0169420012],[1,149135,0.0403300403],[1,149170,0.0122236379],[1,149193,0.023171917],[1,149251,0.0390242738],[1,149279,0.0068219642],[1,149314,0.0508710767],[1,149338,0.0200078896],[1,149446,0.0241700002],[1,149482,0.0044200753],[1,149502,0.0401361332],[1,149555,0.0262861251],[1,149558,0.0333039626],[1,149573,0.0147221805],[1,149602,0.0420934969],[1,149616,0.0889201321],[1,149667,0.060979033],[1,149694,0.2296896334],[1,149712,0.0369440669],[1,149718,0.0409366338],[1,149727,0.0410301467],[1,149732,0.0138464039],[1,149733,0.0107256931],[1,149768,0.0592147955],[1,149817,0.039775971],[1,149832,0.1748917732],[1,149839,0.0105535984],[1,149872,0.1162874989],[1,149899,0.1596814608],[1,149922,0.0730382715],[1,149955,0.0188621955],[1,149961,0.0283008477],[1,150012,0.0053761922],[1,150090,0.0306898289],[1,150216,0.0219221126],[1,150231,0.0389044274],[1,150277,0.0229850881],[1,150290,0.0437210542],[1,150294,0.1474763517],[1,150345,0.0245942789],[1,150357,0.0051510656],[1,150385,0.019335388],[1,150387,0.0088609966],[1,150416,0.0141275019],[1,150428,0.048945107],[1,150547,0.1249813873],[1,150587,0.0110003847],[1,150595,0.0061397714],[1,150607,0.3213630636],[1,150616,0.0069870351],[1,150628,0.0421317758],[1,150701,0.0456593446],[1,150709,0.0088609966],[1,150817,0.2503925572],[1,150827,0.377443991],[1,150828,0.2412517355],[1,150876,0.1593053193],[1,150892,0.0073597067],[1,150935,0.0330138314],[1,150980,0.0661993068],[1,150986,0.0562744971],[1,151005,0.0480767925],[1,151062,0.0682812764],[1,151067,0.007175093],[1,151098,0.0053145461],[1,151123,0.0112131845],[1,151138,0.0072691819],[1,151145,0.5340218629],[1,151158,0.006856698],[1,151261,0.012330487],[1,151350,0.0093655124],[1,151364,0.0237223853],[1,151416,0.0328585302],[1,151452,0.0221853327],[1,151463,0.0088609966],[1,151465,0.0224295545],[1,151469,0.0085795664],[1,151599,0.4032431095],[1,151642,0.0330167505],[1,151651,0.0433392068],[1,151695,0.0048986163],[1,151699,0.0061397714],[1,151810,0.0299506657],[1,151813,0.0052247866],[1,151886,0.0421388602],[1,151895,0.0061397714],[1,151907,0.0197694525],[1,151922,0.0954380297],[1,151973,0.007175093],[1,151975,0.0085176487],[1,152001,0.0302808737],[1,152007,0.4302965895],[1,152072,0.0290785703],[1,152082,0.0125728065],[1,152086,0.0198491636],[1,152135,0.0175013147],[1,152142,0.1126271979],[1,152144,0.2195182353],[1,152205,0.1269905131],[1,152255,0.0061397714],[1,152265,0.0073597067],[1,152290,0.1417024023],[1,152305,0.038950666],[1,152316,0.0084747599],[1,152361,0.0279383405],[1,152371,0.0229768819],[1,152387,0.0061397714],[1,152451,0.0363193591],[1,152466,0.0085176487],[1,152652,0.0382015986],[1,152663,0.7474728818],[1,152706,0.0211791504],[1,152724,0.0277116654],[1,152737,0.0889816655],[1,152768,0.0265320048],[1,152786,0.1208083811],[1,152787,0.020336185],[1,152814,0.3386498579],[1,152819,0.0081618208],[1,152844,0.4086259862],[1,152857,0.0148046035],[1,152870,0.354807278],[1,152882,0.0179230608],[1,152896,0.1344166923],[1,152919,0.0390332943],[1,152968,0.0054316334],[1,153037,0.0159304325],[1,153067,0.0129201207],[1,153112,0.0226608668],[1,153114,0.0414635616],[1,153180,0.0382298105],[1,153186,0.0429693738],[1,153240,0.011267421],[1,153242,0.0519321772],[1,153266,0.0074772765],[1,153306,0.1874106813],[1,153316,0.6045701813],[1,153317,0.0088609966],[1,153319,0.0290241881],[1,153322,0.0081618208],[1,153344,0.0185490489],[1,153362,0.2200049307],[1,153388,0.0134496761],[1,153390,0.0106107794],[1,153393,0.1323931244],[1,153441,0.0677352639],[1,153476,0.0097068834],[1,153482,0.02619305],[1,153581,0.0583965577],[1,153604,0.0205000466],[1,153623,0.097632378],[1,153662,0.0092021321],[1,153694,0.0092021321],[1,153698,0.0025627147],[1,153707,0.0310137905],[1,153709,0.0086401961],[1,153749,0.0048832275],[1,153753,0.7178488441],[1,153769,0.0085619931],[1,153812,0.7534998456],[1,153824,0.2278996909],[1,153828,0.3675032107],[1,153847,0.3243352379],[1,153854,0.0333768764],[1,153886,0.0614888399],[1,153918,0.0070883424],[1,153958,0.0071968976],[1,154070,0.0103830094],[1,154087,0.0389741444],[1,154090,0.0543827598],[1,154157,0.0525904613],[1,154188,0.0070883424],[1,154208,0.0324170285],[1,154222,0.0686113535],[1,154354,0.0116571649],[1,154366,0.013195425],[1,154374,0.0312423628],[1,154381,0.0037745494],[1,154406,0.0272949446],[1,154410,0.0313125733],[1,154458,0.0395005907],[1,154537,0.3601902246],[1,154642,0.0742715972],[1,154667,0.0097068834],[1,154707,0.0994326392],[1,154751,0.0110676991],[1,154769,0.1861694449],[1,154913,0.0051762674],[1,154954,0.1886136407],[1,154976,0.0167252222],[1,154992,0.0160289128],[1,155021,0.5392261156],[1,155041,0.0437276327],[1,155059,0.0553124307],[1,155071,0.0866741818],[1,155072,0.0547650607],[1,155083,0.0064796364],[1,155086,0.0162037729],[1,155087,0.0088586239],[1,155114,0.0978338983],[1,155180,0.0085026298],[1,155190,0.0623156006],[1,155227,0.1550278566],[1,155244,0.0399905022],[1,155271,0.0410412274],[1,155310,0.0075482886],[1,155319,0.007006281],[1,155343,0.0852733995],[1,155357,0.0229091908],[1,155393,0.0077821919],[1,155436,0.0064796364],[1,155444,0.0212722348],[1,155471,0.012213007],[1,155605,0.0245859645],[1,155628,0.0086621299],[1,155658,0.0372798751],[1,155659,0.0491809407],[1,155753,0.0865675198],[1,155762,0.0274675697],[1,155794,0.0239272629],[1,155827,0.007006281],[1,155836,0.0280226516],[1,155874,0.0125256956],[1,155959,0.261741233],[1,156007,0.0955487043],[1,156011,0.0190103098],[1,156038,0.2640432896],[1,156064,0.0076772683],[1,156066,0.0695409252],[1,156093,0.0585209875],[1,156110,0.0818078837],[1,156134,0.0092021321],[1,156178,0.0122448172],[1,156208,0.0210008547],[1,156210,0.0146948568],[1,156354,0.04587834],[1,156378,0.007006281],[1,156408,0.0441343216],[1,156478,0.1673769003],[1,156530,0.1314499383],[1,156550,0.2101590313],[1,156562,0.0226944625],[1,156582,0.0638104343],[1,156591,0.1303554019],[1,156597,0.0588436619],[1,156636,0.0255570425],[1,156694,0.0037745494],[1,156696,0.0034607746],[1,156705,0.0256963893],[1,156706,0.014162493],[1,156760,0.0104253124],[1,156772,0.0568866203],[1,156774,0.0092021321],[1,156817,0.0087065301],[1,156826,0.0082482039],[1,156888,0.011722451],[1,156900,0.0108084125],[1,156916,0.0514219766],[1,156949,0.0137424189],[1,156972,0.0132284675],[1,157044,0.0164352416],[1,157056,0.0092021321],[1,157069,0.0071621477],[1,157085,0.0370246229],[1,157163,0.0100372782],[1,157178,0.0230560939],[1,157310,0.0858146873],[1,157320,0.0264833712],[1,157429,0.0527878871],[1,157480,0.347316663],[1,157505,0.1964163844],[1,157507,0.2095078656],[1,157524,0.509719676],[1,157538,0.0099246189],[1,157556,0.0515174747],[1,157563,0.0655283662],[1,157580,0.0852965538],[1,157603,0.0092097561],[1,157606,0.0129591857],[1,157626,0.0493428259],[1,157636,0.0732738396],[1,157642,0.0092758561],[1,157645,0.0393058493],[1,157680,0.1773010591],[1,157756,0.0997792275],[1,157820,0.100357797],[1,157939,0.0159909096],[1,157962,0.0089205704],[1,157974,0.0367422121],[1,157979,0.0150357216],[1,158021,0.0987035186],[1,158028,0.0092021321],[1,158032,0.0337204022],[1,158083,0.0092021321],[1,158084,0.0415516576],[1,158140,0.0392066323],[1,158174,0.0064589156],[1,158240,0.007006281],[1,158248,0.0092021321],[1,158259,0.0719540439],[1,158276,0.0092021321],[1,158376,0.0384820683],[1,158377,0.051532282],[1,158378,0.0375815406],[1,158426,0.0474933951],[1,158472,0.034569497],[1,158490,0.0097620461],[1,158534,0.0400580531],[1,158545,0.0086815193],[1,158580,0.2631941204],[1,158629,0.0092021321],[1,158633,0.0089205704],[1,158637,0.0100985969],[1,158644,0.0332327953],[1,158650,0.0490774779],[1,158721,0.0415516576],[1,158725,0.0271076293],[1,158735,0.0452567009],[1,158738,0.0389103615],[1,158745,0.0424327268],[1,158777,0.0107335543],[1,158790,0.011929759],[1,158801,0.0350416335],[1,158857,0.0064796364],[1,158872,0.007006281],[1,158882,0.0120130936],[1,158898,0.7318858592],[1,158939,0.0218950455],[1,158966,0.0782409072],[1,158972,0.5837914783],[1,158987,0.1349034704],[1,159058,0.0095621604],[1,159072,0.5881237367],[1,159075,0.033264517],[1,159089,0.0178460641],[1,159225,0.0815000721],[1,159263,0.0217912005],[1,159283,0.0064796364],[1,159319,0.0152370241],[1,159335,0.0183758383],[1,159351,0.0032967487],[1,159354,0.0245908848],[1,159375,0.0975801925],[1,159379,0.3912695091],[1,159423,0.0121824863],[1,159534,0.2477857001],[1,159573,0.0025627147],[1,159637,0.3953536498],[1,159638,0.0457029629],[1,159689,0.0273285937],[1,159723,0.0156896406],[1,159733,0.0057157007],[1,159744,0.018468889],[1,159829,0.01160968],[1,160024,0.323394762],[1,160028,0.1120532133],[1,160066,0.0204033133],[1,160096,0.0244168369],[1,160121,0.049816535],[1,160140,0.0108670668],[1,160149,0.2058654333],[1,160175,0.0219655379],[1,160185,0.1144901649],[1,160187,0.4313915809],[1,160298,0.0085619931],[1,160326,0.0307334039],[1,160346,0.0535207268],[1,160347,0.0515457082],[1,160349,0.0395765423],[1,160356,0.0865781737],[1,160385,0.0221433113],[1,160388,0.5478179026],[1,160392,0.2823599344],[1,160412,0.0064796364],[1,160426,0.0092021321],[1,160457,0.045340124],[1,160507,0.0086836784],[1,160511,0.0092021321],[1,160513,0.0088433716],[1,160745,0.1827725034],[1,160768,0.006691242],[1,160785,0.0082011898],[1,160828,0.0582876152],[1,160876,0.1266457691],[1,160894,0.0327180265],[1,160944,0.0164993329],[1,160957,0.0561098695],[1,160962,0.0385484603],[1,160966,0.0409072217],[1,160982,0.0338760701],[1,161017,0.1421181176],[1,161018,0.0639792937],[1,161019,0.045340124],[1,161020,0.0302797841],[1,161021,0.0462498519],[1,161023,0.0443132212],[1,161036,0.1313448005],[1,161071,0.2365501731],[1,161104,0.0443506419],[1,161143,0.1974995112],[1,161144,0.1658718002],[1,161145,0.7866109518],[1,161147,0.1922990459],[1,161148,0.7866109518],[1,161185,0.0412679199],[1,161207,0.0058507352],[1,161256,0.0092097561],[1,161277,0.0092021321],[1,161291,0.0613486626],[1,161341,0.0177768424],[1,161389,0.0096936541],[1,161395,0.8062056837],[1,161402,0.0086815193],[1,161415,0.0958627685],[1,161440,0.0333375544],[1,161513,0.008439381],[1,161554,0.0064589156],[1,161584,0.1392517877],[1,161606,0.2349083332],[1,161609,0.1249949409],[1,161624,0.056089065],[1,161766,0.0260531857],[1,161811,0.199681704],[1,161814,0.0085026298],[1,161865,0.0631772511],[1,161904,0.0429499887],[1,161907,0.021378474],[1,161934,0.2829224438],[1,162036,0.0316886489],[1,162071,0.0076154272],[1,162139,0.0418844863],[1,162153,0.0982579059],[1,162156,0.0794895407],[1,162197,0.0085026298],[1,162265,0.0343965552],[1,162274,0.0116908487],[1,162303,0.0887344443],[1,162306,0.0747073955],[1,162319,0.0225053399],[1,162323,0.7256624855],[1,162347,0.0287072879],[1,162395,0.0150705553],[1,162468,0.0436143168],[1,162482,0.238150551],[1,162500,0.0350544095],[1,162525,0.0418844863],[1,162581,0.0158016347],[1,162587,0.0111420345],[1,162615,0.0622642367],[1,162622,0.190205757],[1,162625,0.0443229213],[1,162631,0.2371276273],[1,162642,0.019297842],[1,162643,0.0251124074],[1,162655,0.0025627147],[1,162720,0.0320824566],[1,162727,0.0068628934],[1,162751,0.0070883424],[1,162798,0.3364297703],[1,162818,0.0406980105],[1,162837,0.0857944458],[1,162873,0.006691242],[1,162878,0.0223959951],[1,162885,0.5641875338],[1,162934,0.1029979712],[1,162956,0.0082863575],[1,162991,0.0799463727],[1,163050,0.0585863085],[1,163067,0.026778894],[1,163078,0.0216136973],[1,163093,0.0415516576],[1,163130,0.0025627147],[1,163193,0.0070883424],[1,163200,0.0103715045],[1,163231,0.160838269],[1,163252,0.0087347274],[1,163263,0.3403811773],[1,163294,0.0092021321],[1,163311,0.007006281],[1,163345,0.0346726063],[1,163377,0.1137087906],[1,163390,0.3953533059],[1,163403,0.6713398681],[1,163404,0.0088586239],[1,163419,0.2828321833],[1,163420,0.0040236231],[1,163424,0.0476126358],[1,163442,0.00929965],[1,163446,0.0167959475],[1,163474,0.2317073462],[1,163503,0.0092021321],[1,163507,0.0419674333],[1,163514,0.0352272684],[1,163519,0.024841534],[1,163532,0.2304606868],[1,163533,0.3436417766],[1,163554,0.0146658073],[1,163577,0.0084851409],[1,163628,0.0429499887],[1,163630,0.0315155813],[1,163649,0.0399585335],[1,163662,0.0263043821],[1,163702,0.0419325705],[1,163718,0.0064796364],[1,163723,0.0209141694],[1,163737,0.0302762937],[1,163821,0.0074902149],[1,163835,0.4296836541],[1,163855,0.019297842],[1,163856,0.3159754631],[1,163869,0.010826205],[1,163927,0.0499467603],[1,163941,0.0229406452],[1,163985,0.3605360241],[1,164104,0.6905967239],[1,164127,0.0489389849],[1,164268,0.7708845467],[1,164271,0.0057157007],[1,164292,0.024271341],[1,164396,0.0180050233],[1,164419,0.2071398621],[1,164464,0.0224080407],[1,164478,0.1141743742],[1,164492,0.0385781916],[1,164541,0.0787468938],[1,164555,0.0258425136],[1,164591,0.3955673277],[1,164639,0.0403954119],[1,164669,0.0037745494],[1,164690,0.0167088815],[1,164697,0.1890373712],[1,164716,0.0161701109],[1,164807,0.0097068834],[1,164811,0.0360831328],[1,164814,0.0415516576],[1,164845,0.0210083616],[1,164873,0.0519908015],[1,164963,0.093061613],[1,164967,0.0082774288],[1,165041,0.2236715232],[1,165053,0.0435868697],[1,165056,0.0258899292],[1,165062,0.009758868],[1,165069,0.5059616516],[1,165074,0.0403863704],[1,165097,0.0469384913],[1,165107,0.0700299993],[1,165133,0.0097763873],[1,165158,0.0099555888],[1,165171,0.0264068621],[1,165173,0.0277118176],[1,165178,0.0101330023],[1,165179,0.0370635953],[1,165248,0.0574934131],[1,165279,0.0935137854],[1,165291,0.19920462],[1,165336,0.2654032925],[1,165383,0.0097763873],[1,165420,0.0211308904],[1,165437,0.0104847469],[1,165455,0.0128888565],[1,165462,0.0149610042],[1,165492,0.0233735675],[1,165524,0.0450085527],[1,165548,0.006983685],[1,165549,0.0077498657],[1,165610,0.0947830175],[1,165664,0.5994624403],[1,165670,0.0149190261],[1,165698,0.0049784432],[1,165729,0.0360867696],[1,165731,0.0065059654],[1,165754,0.0977468749],[1,165793,0.1084091814],[1,165827,0.010042685],[1,165829,0.0077498657],[1,165869,0.0217349798],[1,165871,0.2376061439],[1,165873,0.0994439441],[1,165876,0.0428830555],[1,165907,0.0104847469],[1,165994,0.0167513634],[1,166006,0.0293919782],[1,166088,0.0285173928],[1,166122,0.0302914345],[1,166193,0.0109537936],[1,166199,0.0071249316],[1,166215,0.0318915103],[1,166232,0.0101330023],[1,166282,0.0125006568],[1,166331,0.0103166746],[1,166363,0.0428830555],[1,166424,0.0436574236],[1,166446,0.0415367544],[1,166598,0.0412971945],[1,166603,0.0195885325],[1,166658,0.0094923569],[1,166665,0.0979567487],[1,166690,0.0132127795],[1,166709,0.3536814736],[1,166783,0.0304858192],[1,166803,0.0083596333],[1,166843,0.0139895868],[1,166844,0.0604899793],[1,166858,0.5751037141],[1,166872,0.0396366988],[1,166881,0.0482912666],[1,166928,0.0190587357],[1,166942,0.0607501019],[1,166954,0.0185077158],[1,166965,0.0088203817],[1,167016,0.0170094849],[1,167076,0.0077498657],[1,167110,0.1608559879],[1,167130,0.0083596333],[1,167134,0.046375948],[1,167140,0.0198717612],[1,167248,0.0101330023],[1,167294,0.0087622067],[1,167298,0.0323329004],[1,167355,0.0094744216],[1,167393,0.0099555888],[1,167521,0.0077498657],[1,167571,0.0081337911],[1,167625,0.0462933388],[1,167697,0.0215292043],[1,167706,0.0128888565],[1,167717,0.1760136091],[1,167772,0.0082774288],[1,167781,0.0064442235],[1,167785,0.1151013732],[1,167805,0.0920253724],[1,167814,0.0444014501],[1,167815,0.0082774288],[1,167889,0.0377977356],[1,167893,0.0104771096],[1,167903,0.1915273549],[1,167908,0.0115279782],[1,167910,0.0286106939],[1,167920,0.0094336374],[1,167987,0.045903187],[1,167994,0.0554331356],[1,168012,0.0088203817],[1,168017,0.0905075318],[1,168039,0.0099529241],[1,168042,0.0077498657],[1,168074,0.0403561287],[1,168095,0.1061956769],[1,168122,0.0104771096],[1,168125,0.0326180993],[1,168196,0.0065059654],[1,168225,0.2251262539],[1,168262,0.010174126],[1,168297,0.0122456552],[1,168337,0.0109827412],[1,168343,0.0395859757],[1,168360,0.0219391928],[1,168415,0.0083596333],[1,168424,0.1618571655],[1,168442,0.2173181209],[1,168443,0.2477724476],[1,168459,0.0127762375],[1,168483,0.5634317633],[1,168511,0.3032733743],[1,168550,0.0079618404],[1,168552,0.0229174067],[1,168641,0.1268630829],[1,168717,0.0675484238],[1,168759,0.0104771096],[1,168779,0.0565900899],[1,168784,0.0128700232],[1,168787,0.0088876374],[1,168808,0.0223347105],[1,168815,0.024117105],[1,168908,0.0699301838],[1,168958,0.0125213473],[1,169045,0.0192738936],[1,169059,0.1693895418],[1,169062,0.0098358541],[1,169096,0.0498856503],[1,169113,0.0894178706],[1,169115,0.0264884379],[1,169202,0.1297254989],[1,169211,0.1063204624],[1,169271,0.009758868],[1,169318,0.0461061193],[1,169354,0.2012029662],[1,169379,0.0343955798],[1,169407,0.0168672008],[1,169456,0.0050400608],[1,169488,0.0098358541],[1,169535,0.032830941],[1,169551,0.0135086617],[1,169579,0.0161385145],[1,169697,0.0675484238],[1,169746,0.0412871529],[1,169748,0.0740940505],[1,169854,0.3948789463],[1,169868,0.0403863704],[1,169923,0.0428830555],[1,169938,0.4759538259],[1,169986,0.0530108529],[1,170063,0.0624740092],[1,170075,0.0152898473],[1,170098,0.011256827],[1,170100,0.0088876374],[1,170102,0.0527292808],[1,170122,0.0082774288],[1,170174,0.0094744216],[1,170176,0.1071618463],[1,170233,0.0084028414],[1,170362,0.0318417716],[1,170412,0.0559618348],[1,170442,0.0126920998],[1,170454,0.0088203817],[1,170496,0.032318731],[1,170498,0.1054449256],[1,170515,0.0097763873],[1,170541,0.0111936729],[1,170605,0.0416326985],[1,170628,0.0104771096],[1,170651,0.0167397376],[1,170684,0.0099555888],[1,170726,0.009758868],[1,170756,0.0367821],[1,170759,0.0168672008],[1,170764,0.0215488813],[1,170860,0.0356052631],[1,170863,0.0381158904],[1,171038,0.0690636502],[1,171054,0.2737282195],[1,171088,0.0077291087],[1,171122,0.0672980254],[1,171132,0.0117222511],[1,171153,0.0463139748],[1,171193,0.0221237428],[1,171197,0.094882805],[1,171204,0.2163664865],[1,171212,0.0077498657],[1,171222,0.038328962],[1,171276,0.0094336374],[1,171299,0.0287892024],[1,171344,0.458472612],[1,171349,0.0323993125],[1,171404,0.0099806433],[1,171445,0.0152633898],[1,171449,0.0484199961],[1,171476,0.1707228088],[1,171511,0.0436589601],[1,171594,0.3260607667],[1,171697,0.0226280976],[1,171707,0.0104771096],[1,171715,0.3611324587],[1,171722,0.1005293567],[1,171739,0.0152633898],[1,171757,0.0438174051],[1,171803,0.0083596333],[1,171806,0.0419685193],[1,171825,0.0104771096],[1,171898,0.0352469171],[1,171959,0.0312671957],[1,171992,0.3212391614],[1,172000,0.0077498657],[1,172083,0.0080499793],[1,172176,0.0103821833],[1,172245,0.0121939546],[1,172258,0.0091964822],[1,172344,0.1245434527],[1,172365,0.1506161772],[1,172374,0.1878492734],[1,172377,0.9254756371],[1,172378,0.8827351836],[1,172379,0.0726751331],[1,172380,0.072798057],[1,172381,0.8849079015],[1,172382,0.0679940141],[1,172383,0.8404329645],[1,172384,0.8870835614],[1,172385,0.9143678394],[1,172396,0.0504271139],[1,172397,0.8790376358],[1,172398,0.0468780104],[1,172399,0.9334344169],[1,172400,0.888815477],[1,172401,0.9311960787],[1,172402,0.9334344169],[1,172403,0.9200497079],[1,172404,0.888815477],[1,172405,0.888815477],[1,172406,0.8790376358],[1,172407,0.8822903029],[1,172408,0.9334344169],[1,172409,0.9334344169],[1,172410,0.9334344169],[1,172411,0.922272953],[1,172412,0.8899055836],[1,172413,0.8822903029],[1,172415,0.9278442453],[1,172435,0.3346426839],[1,172451,0.8648860276],[1,172476,0.0757525336],[1,172479,0.0669227257],[1,172480,0.0487419951],[1,172482,0.0674598135],[1,172483,0.04471719],[1,172485,0.0457583186],[1,172486,0.0503076507],[1,172487,0.0504271139],[1,172491,0.8565890043],[1,172501,0.0679940141],[1,172546,0.8163100766],[1,172562,0.8648860276],[1,172564,0.834103802],[1,172566,0.8713404051],[1,172572,0.0206313949],[1,172581,0.0083084368],[1,172602,0.8648860276],[1,172604,0.8648860276],[1,172634,0.0091789768],[1,172642,0.9334344169],[1,172643,0.7801468055],[1,172645,0.9278442453],[1,172649,0.0788869143],[1,172669,0.0098772052],[1,172710,0.024242561],[1,172712,0.0978921369],[1,172742,0.0346758797],[1,172747,0.0663396525],[1,172763,0.0083084368],[1,172808,0.0095528146],[1,172836,0.038349018],[1,172861,0.0444337271],[1,172863,0.011517102],[1,172869,0.022998747],[1,172948,0.3890981427],[1,172966,0.0238983432],[1,172990,0.0111408036],[1,173125,0.4669661345],[1,173127,0.0888577397],[1,173166,0.0093755417],[1,173176,0.0091789768],[1,173213,0.0123064843],[1,173302,0.0077808512],[1,173354,0.0431543819],[1,173355,0.0083084368],[1,173434,0.1208559227],[1,173471,0.1479133203],[1,173491,0.0076987119],[1,173519,0.055994947],[1,173528,0.0739838034],[1,173590,0.0228599573],[1,173597,0.0030422778],[1,173669,0.0045041534],[1,173688,0.0426930792],[1,173805,0.0315977548],[1,173865,0.0285969266],[1,173891,0.1902190926],[1,173974,0.0410676728],[1,173984,0.0706420487],[1,173985,0.024995322],[1,174011,0.0095528146],[1,174052,0.0342909494],[1,174097,0.6641384847],[1,174098,0.6641384847],[1,174135,0.0190008498],[1,174227,0.016860242],[1,174229,0.0121024912],[1,174284,0.0102049068],[1,174310,0.0228974479],[1,174353,0.0083932313],[1,174379,0.0880150323],[1,174579,0.0083084368],[1,174590,0.0422769084],[1,174644,0.0723247003],[1,174785,0.0140181658],[1,174804,0.0063324751],[1,174816,0.0323680609],[1,174822,0.0106757484],[1,174830,0.0093755417],[1,174907,0.0150441593],[1,174934,0.0251900388],[1,174948,0.0147055719],[1,174976,0.5954718323],[1,175047,0.0082412344],[1,175055,0.0094005763],[1,175225,0.1920063432],[1,175262,0.0170632813],[1,175271,0.0110210999],[1,175277,0.0421728445],[1,175279,0.4868890054],[1,175382,0.0125613595],[1,175388,0.0365018702],[1,175462,0.0348766714],[1,175519,0.0447131262],[1,175556,0.1025831542],[1,175596,0.0669227257],[1,175598,0.0172002309],[1,175646,0.0071170717],[1,175658,0.0576891308],[1,175721,0.3563499137],[1,175726,0.2760973839],[1,175727,0.0262285612],[1,175797,0.2032304653],[1,175875,0.0422769084],[1,175936,0.03368575],[1,175954,0.0394824364],[1,175969,0.0396133553],[1,175987,0.0121098835],[1,176091,0.0127435564],[1,176194,0.0065420682],[1,176247,0.0093728791],[1,176283,0.0418676505],[1,176290,0.0517031735],[1,176301,0.0260721057],[1,176315,0.104225998],[1,176326,0.0303246756],[1,176334,0.0246762698],[1,176398,0.0095528146],[1,176468,0.0071715669],[1,176496,0.0208103431],[1,176587,0.1903506773],[1,176664,0.0098966491],[1,176686,0.0261588093],[1,176734,0.0111408036],[1,176749,0.1066847558],[1,176754,0.0235270193],[1,176777,0.0040646464],[1,176824,0.0248003292],[1,176830,0.0091789768],[1,176917,0.0342033662],[1,176931,0.0411193936],[1,176933,0.013300079],[1,176938,0.0229647064],[1,176948,0.0054494333],[1,176964,0.3045554792],[1,177046,0.0850815689],[1,177116,0.0104748144],[1,177176,0.0226039404],[1,177197,0.0354252705],[1,177232,0.0071508264],[1,177270,0.0095528146],[1,177291,0.0092642592],[1,177332,0.0097363413],[1,177350,0.0098966491],[1,177366,0.1315869046],[1,177379,0.5971855801],[1,177433,0.045106026],[1,177449,0.0103232687],[1,177473,0.0043243741],[1,177602,0.0220912338],[1,177673,0.0099156922],[1,177704,0.0177170744],[1,177710,0.0094742945],[1,177713,0.0055223258],[1,177778,0.1024546235],[1,177883,0.0058505216],[1,177928,0.0091004968],[1,178013,0.0364392214],[1,178047,0.0297202161],[1,178070,0.0098180921],[1,178072,0.0129838822],[1,178198,0.0511171503],[1,178239,0.505926328],[1,178290,0.0088208177],[1,178351,0.2922882798],[1,178352,0.0720548117],[1,178442,0.0155352263],[1,178489,0.0043859362],[1,178497,0.059458535],[1,178502,0.0954685985],[1,178627,0.0339392523],[1,178632,0.094905979],[1,178635,0.0093220726],[1,178676,0.051783775],[1,178696,0.0207784107],[1,178715,0.1168241806],[1,178725,0.0225127232],[1,178732,0.0646354464],[1,178747,0.0098180921],[1,178843,0.0207156645],[1,178878,0.0105740249],[1,178916,0.0335887848],[1,178929,0.1604447098],[1,178990,0.1583388024],[1,179020,0.0385853515],[1,179063,0.0424297592],[1,179072,0.0098257226],[1,179099,0.0092970406],[1,179117,0.0327985762],[1,179155,0.0147464007],[1,179215,0.0605799456],[1,179266,0.0077025214],[1,179271,0.0070725642],[1,179285,0.0098180921],[1,179290,0.0692837263],[1,179347,0.0156772068],[1,179372,0.0077061324],[1,179374,0.2912829376],[1,179398,0.0121423244],[1,179408,0.0090496202],[1,179592,0.0331640588],[1,179600,0.0820240094],[1,179631,0.1190526886],[1,179727,0.2479643878],[1,179919,0.0272867846],[1,179997,0.0158580691],[1,180096,0.0193477673],[1,180147,0.0748197492],[1,180205,0.0414459379],[1,180284,0.5085472338],[1,180514,0.0611639094],[1,180550,0.039555819],[1,180563,0.1523281134],[1,180576,0.0404750471],[1,180667,0.0301248068],[1,180690,0.0098180921],[1,180707,0.043860967],[1,180722,0.0203888202],[1,180734,0.0098180921],[1,180736,0.0761170304],[1,180773,0.0432993345],[1,180776,0.0389576442],[1,180856,0.3855728652],[1,180898,0.1300028238],[1,180918,0.0444772681],[1,180966,0.0353916485],[1,180968,0.0222293455],[1,181005,0.5721758118],[1,181119,0.0081047325],[1,181190,0.2474955511],[1,181200,0.0341245854],[1,181308,0.0385659113],[1,181316,0.0198358496],[1,181384,0.0398437532],[1,181443,0.0359370482],[1,181449,0.1967009213],[1,181482,0.0425279843],[1,181617,0.0828734529],[1,181627,0.0070933025],[1,181679,0.071221474],[1,181726,0.1915614472],[1,181746,0.0161418397],[1,181757,0.0103196923],[1,181772,0.1296921548],[1,181834,0.0398348419],[1,181855,0.0488534677],[1,181897,0.0800291258],[1,181931,0.1349688599],[1,181949,0.0070933025],[1,181982,0.0330646467],[1,182011,0.0379103928],[1,182032,0.0695930136],[1,182034,0.0431515458],[1,182035,0.0091180004],[1,182075,0.0354242809],[1,182081,0.0070933025],[1,182083,0.107300684],[1,182108,0.0070933025],[1,182191,0.0073050864],[1,182233,0.0861248304],[1,182245,0.0446494483],[1,182321,0.0194383018],[1,182326,0.0770787132],[1,182349,0.0358587168],[1,182356,0.0202878732],[1,182460,0.0076203909],[1,182482,0.0191877625],[1,182505,0.0169122019],[1,182521,0.021140138],[1,182533,0.0640865672],[1,182581,0.1154456967],[1,182585,0.0091004968],[1,182641,0.1131767669],[1,182684,0.0493774561],[1,182720,0.0764431041],[1,182814,0.0534677449],[1,182847,0.3404987657],[1,182865,0.0993897859],[1,182952,0.0144843427],[1,182973,0.0092943783],[1,183026,0.0182360801],[1,183058,0.0089072118],[1,183129,0.0606952734],[1,183176,0.0220839344],[1,183197,0.0169605611],[1,183211,0.0124598313],[1,183224,0.3842966113],[1,183247,0.150312002],[1,183348,0.0360268021],[1,183381,0.0369640443],[1,183384,0.0181701536],[1,183408,0.1343602715],[1,183444,0.0072637544],[1,183464,0.0593804843],[1,183470,0.0205975341],[1,183533,0.0081628552],[1,183567,0.0142500097],[1,183630,0.0117989021],[1,183636,0.0189412985],[1,183649,0.0162024327],[1,183681,0.0077025214],[1,183755,0.0443523108],[1,183866,0.0222330594],[1,183867,0.1702345593],[1,183936,0.028697828],[1,183968,0.2262331832],[1,183996,0.0077025214],[1,184020,0.0701136277],[1,184046,0.0142500097],[1,184115,0.011062113],[1,184117,0.0076203909],[1,184159,0.1182309907],[1,184176,0.008775559],[1,184220,0.0129040272],[1,184231,0.0405124684],[1,184242,0.3844967711],[1,184303,0.0224470092],[1,184315,0.0910706601],[1,184337,0.036096993],[1,184349,0.0098180921],[1,184380,0.0142765446],[1,184591,0.00662185],[1,184596,0.1195829365],[1,184642,0.016685663],[1,184660,0.0109003001],[1,184679,0.0091004968],[1,184739,0.4239129783],[1,184786,0.0421948751],[1,184910,0.0093351548],[1,184966,0.009396454],[1,184984,0.0092970406],[1,185110,0.0162730439],[1,185155,0.0504036848],[1,185184,0.0460669011],[1,185186,0.0078111681],[1,185196,0.1937473847],[1,185218,0.0094742945],[1,185230,0.0249915908],[1,185235,0.0094313495],[1,185251,0.0741886303],[1,185252,0.1101901047],[1,185257,0.0092970406],[1,185326,0.0070933025],[1,185361,0.0091004968],[1,185447,0.0269863881],[1,185449,0.0092970406],[1,185452,0.0113027095],[1,185491,0.0256627636],[1,185516,0.0231819078],[1,185519,0.0098180921],[1,185678,0.0189280136],[1,185808,0.0178414713],[1,185871,0.1783037569],[1,185888,0.0425279843],[1,185917,0.1636297731],[1,185978,0.0603141839],[1,186024,0.1467287927],[1,186028,0.4502074181],[1,186048,0.2394807018],[1,186081,0.0070933025],[1,186091,0.0155671839],[1,186321,0.0248096119],[1,186420,0.0423323184],[1,186518,0.012115151],[1,186549,0.0070933025],[1,186630,0.0098180921],[1,186747,0.0105971078],[1,186775,0.005789282],[1,186826,0.0124825164],[1,186841,0.0367315333],[1,186844,0.2789303698],[1,186872,0.1799199955],[1,186878,0.008775559],[1,186916,0.0204485575],[1,186934,0.0191082697],[1,186935,0.3392595539],[1,186937,0.0938302745],[1,186964,0.0272807613],[1,186975,0.0293630181],[1,187030,0.0098180921],[1,187046,0.1531988],[1,187096,0.0471940764],[1,187100,0.0273676098],[1,187102,0.0292087657],[1,187191,0.0213327356],[1,187194,0.0091004968],[1,187202,0.0234779542],[1,187254,0.3567877303],[1,187266,0.0446494483],[1,187298,0.0091180004],[1,187344,0.0414459379],[1,187389,0.3950148572],[1,187405,0.0091004968],[1,187530,0.0092970406],[1,187533,0.0098180921],[1,187652,0.020179399],[1,187730,0.1811570037],[1,187780,0.0174813196],[1,187838,0.4574994864],[1,187893,0.3153298969],[1,187953,0.0421706452],[1,187994,0.3440319787],[1,188036,0.0091524288],[1,188047,0.0061771191],[1,188170,0.0098945428],[1,188208,0.0206493472],[1,188330,0.0205258445],[1,188331,0.0702733144],[1,188368,0.0122846849],[1,188370,0.2343835825],[1,188423,0.0457147485],[1,188472,0.0076966119],[1,188473,0.0815334356],[1,188507,0.1807617033],[1,188537,0.0071694684],[1,188631,0.0944133126],[1,188646,0.1994966626],[1,188686,0.0422747089],[1,188821,0.0071694684],[1,188822,0.0092537974],[1,188830,0.0087732716],[1,188859,0.0191121492],[1,189089,0.1953562715],[1,189158,0.0208158953],[1,189160,0.0091768725],[1,189172,0.0538642955],[1,189186,0.0443677295],[1,189191,0.0076966119],[1,189213,0.0807981231],[1,189302,0.0288358542],[1,189354,0.2802854835],[1,189399,0.2824596793],[1,189546,0.0098945428],[1,189691,0.0687255333],[1,189709,0.0085023364],[1,189734,0.0378374096],[1,189803,0.5337984762],[1,189818,0.7433751219],[1,189824,0.0102943471],[1,189876,0.0039835659],[1,189883,0.0373676809],[1,189938,0.0186706402],[1,189964,0.0146749104],[1,189967,0.0686907201],[1,189976,0.4830314842],[1,189979,0.0888380359],[1,190030,0.1889810325],[1,190049,0.5364394373],[1,190055,0.0185260905],[1,190078,0.0084732878],[1,190171,0.0458809667],[1,190237,0.0358713722],[1,190274,0.0111386937],[1,190378,0.0077410033],[1,190435,0.0202643683],[1,190437,0.0103961954],[1,190495,0.4052738357],[1,190539,0.3252622866],[1,190543,0.032320001],[1,190619,0.0240087935],[1,190668,0.4808411951],[1,190670,0.0076966119],[1,190760,0.0250634219],[1,190814,0.0231728227],[1,190838,0.0434230426],[1,190839,0.0840419738],[1,190842,0.0886068207],[1,190869,0.0680126814],[1,190878,0.0381489315],[1,190899,0.4503088192],[1,190911,0.0427879594],[1,190916,0.0142500097],[1,190920,0.2654938224],[1,190934,0.0044618193],[1,190966,0.0076966119],[1,191032,0.0409829204],[1,191049,0.0183949521],[1,191099,0.0071694684],[1,191156,0.0151159367],[1,191173,0.0916510883],[1,191181,0.0946550571],[1,191186,0.0099176087],[1,191190,0.0443669153],[1,191207,0.1899977606],[1,191270,0.0103997721],[1,191285,0.0370012063],[1,191325,0.0426110012],[1,191330,0.0383973165],[1,191355,0.0098750989],[1,191377,0.1645625409],[1,191424,0.0238135804],[1,191431,0.1054128912],[1,191441,0.0063430758],[1,191506,0.0233105592],[1,191509,0.3040558462],[1,191598,0.0162189512],[1,191600,0.008686004],[1,191617,0.0121918416],[1,191720,0.0850080698],[1,191758,0.1856266436],[1,191773,0.0111386937],[1,191775,0.03599083],[1,191784,0.1556963583],[1,191830,0.0207270807],[1,191908,0.030987358],[1,191994,0.358263196],[1,192001,0.0349738195],[1,192083,0.00987411],[1,192097,0.0098945428],[1,192171,0.2967256769],[1,192236,0.0098945428],[1,192281,0.0577140393],[1,192342,0.009902174],[1,192469,0.0432083151],[1,192556,0.0091943779],[1,192564,0.0087732716],[1,192567,0.0436743644],[1,192670,0.0098945428],[1,192719,0.0219265681],[1,192750,0.0199050422],[1,192758,0.0214767224],[1,192766,0.0611304589],[1,192769,0.0209642201],[1,192781,0.0118830334],[1,192813,0.1131676212],[1,192819,0.0733890275],[1,192909,0.261251468],[1,192916,0.0054473398],[1,192936,0.0160052654],[1,192943,0.0426908785],[1,192951,0.3996621787],[1,192964,0.7908988638],[1,192969,0.0149650496],[1,192979,0.0397299527],[1,192982,0.0098945428],[1,192997,0.220400995],[1,193005,0.0098945428],[1,193024,0.0407832159],[1,193035,0.021527266],[1,193059,0.2092541106],[1,193106,0.0426761686],[1,193179,0.4109988262],[1,193181,0.0248761475],[1,193200,0.0302806213],[1,193286,0.0092537974],[1,193298,0.0156597769],[1,193334,0.0239516826],[1,193478,0.0098945428],[1,193531,0.0701538912],[1,193544,0.0096965185],[1,193668,0.0422747089],[1,193707,0.0476902773],[1,193733,0.0121918416],[1,193735,0.010208905],[1,193804,0.0373202261],[1,193838,0.1209167229],[1,193857,0.0235233929],[1,193909,0.0368317413],[1,193910,0.0348654319],[1,194002,0.0075530885],[1,194009,0.0755519643],[1,194028,0.0215673947],[1,194064,0.0106736398],[1,194090,0.0867000819],[1,194100,0.0071694684],[1,194223,0.0583986293],[1,194224,0.0872108558],[1,194228,0.0071694684],[1,194299,0.1109413284],[1,194304,0.2845266998],[1,194305,0.0219004101],[1,194374,0.0967927527],[1,194378,0.0543613597],[1,194426,0.0695777153],[1,194440,0.0144993788],[1,194485,0.0273846566],[1,194496,0.1113560822],[1,194514,0.2258699116],[1,194546,0.2977905434],[1,194599,0.0235139675],[1,194608,0.0525777874],[1,194641,0.0295337922],[1,194718,0.0439409748],[1,194747,0.0213354181],[1,194921,0.0288536216],[1,194924,0.0138259527],[1,194929,0.5763492353],[1,194988,0.0098945428],[1,195018,0.0342310925],[1,195050,0.0739260429],[1,195084,0.0093320745],[1,195141,0.0103997721],[1,195184,0.0098945428],[1,195202,0.0232931484],[1,195205,0.0422747089],[1,195245,0.0242003782],[1,195251,0.1044944774],[1,195468,0.0373514675],[1,195481,0.037787235],[1,195499,0.0044002507],[1,195567,0.0365135446],[1,195685,0.1073417636],[1,195711,0.0145604118],[1,195760,0.1559635678],[1,195762,0.0113486888],[1,195844,0.084617042],[1,195912,0.0093412815],[1,195939,0.0343888939],[1,196050,0.0388822621],[1,196052,0.2405421607],[1,196069,0.3083793291],[1,196095,0.0409861349],[1,196107,0.0097200805],[1,196133,0.0115266023],[1,196135,0.0059902627],[1,196149,0.0386576923],[1,196151,0.0893887909],[1,196203,0.066347704],[1,196208,0.1321584709],[1,196227,0.0337892331],[1,196248,0.0196144498],[1,196290,0.0311226143],[1,196324,0.0062204708],[1,196353,0.0916607653],[1,196521,0.0117085425],[1,196700,0.0031176037],[1,196719,0.0406140019],[1,196771,0.0172058668],[1,196795,0.0412800117],[1,196800,0.0068289602],[1,196841,0.0311272568],[1,196950,0.0404472209],[1,197012,0.2680092901],[1,197016,0.0075516037],[1,197020,0.224606263],[1,197029,0.0219027012],[1,197083,0.0085986117],[1,197117,0.2680395479],[1,197121,0.015513316],[1,197145,0.0179570194],[1,197162,0.0049175656],[1,197200,0.0383537358],[1,197209,0.0064320011],[1,197225,0.0192498874],[1,197569,0.3667233994],[1,197600,0.0097200805],[1,197618,0.0080067344],[1,197659,0.0253982416],[1,197733,0.0073558574],[1,197739,0.3788243372],[1,197749,0.0066035914],[1,197827,0.0422124025],[1,197855,0.0089419977],[1,197945,0.0306938167],[1,198040,0.4641364757],[1,198070,0.0035163464],[1,198083,0.0209414636],[1,198119,0.0768867318],[1,198202,0.0073558574],[1,198227,0.0241663795],[1,198240,0.136366602],[1,198365,0.0101845289],[1,198369,0.043773542],[1,198401,0.0278702294],[1,198432,0.0315184681],[1,198450,0.0260897775],[1,198502,0.0351761975],[1,198804,0.0125919408],[1,198818,0.0190270743],[1,198945,0.2563215184],[1,199032,0.092258454],[1,199125,0.0351955793],[1,199262,0.1095183475],[1,199345,0.0643474648],[1,199494,0.0406966744],[1,199517,0.0431421478],[1,199520,0.0111856145],[1,199529,0.0951550166],[1,199531,0.0264808299],[1,199596,0.1597975567],[1,199737,0.1248877144],[1,199884,0.008749406],[1,199911,0.0052189361],[1,199967,0.1183902635],[1,200129,0.0638420054],[1,200134,0.0197090372],[1,200331,0.0861824412],[1,200380,0.0132466609],[1,200392,0.0375077597],[1,200410,0.0221984206],[1,200442,0.0223690438],[1,200508,0.0317102223],[1,200542,0.0164528984],[1,200670,0.0586175938],[1,200681,0.0348074449],[1,200697,0.0195425294],[1,200816,0.0332976053],[1,200998,0.1748297111],[1,201039,0.0896895375],[1,201063,0.0413734272],[1,201074,0.0442611784],[1,201079,0.2101802033],[1,201190,0.0319274196],[1,201195,0.010239477],[1,201232,0.0148583342],[1,201271,0.0087896101],[1,201272,0.0165919386],[1,201358,0.0079888224],[1,201362,0.0097900771],[1,201410,0.0103668764],[1,201422,0.0096847398],[1,201426,0.0069457637],[1,201483,0.0423015437],[1,201486,0.2832204766],[1,201589,0.0100637167],[1,201760,0.0669961881],[1,201765,0.0102869796],[1,201790,0.0228031958],[1,201792,0.0104433844],[1,201867,0.0981707656],[1,201879,0.4616363761],[1,202018,0.1607827137],[1,202086,0.0323392907],[1,202125,0.1299427408],[1,202179,0.012635284],[1,202207,0.0121921634],[1,202242,0.0075604437],[1,202259,0.0415082592],[1,202272,0.0365157775],[1,202276,0.0356892421],[1,202343,0.1429241305],[1,202368,0.0442378597],[1,202371,0.2936541392],[1,202375,0.0092648526],[1,202412,0.0455609258],[1,202418,0.1097700138],[1,202499,0.0065417399],[1,202555,0.0037955512],[1,202559,0.0861004797],[1,202561,0.0237151752],[1,202592,0.0316109909],[1,202608,0.0026450959],[1,202678,0.0192479467],[1,202688,0.1530031464],[1,202727,0.0092928933],[1,202774,0.0765774484],[1,202849,0.133317313],[1,202883,0.0087896101],[1,202931,0.2098451199],[1,202945,0.0264123442],[1,202978,0.0377684293],[1,203085,0.0119250837],[1,203124,0.0867441342],[1,203144,0.0076312369],[1,203246,0.0870139447],[1,203247,0.0085856865],[1,203327,0.0034581385],[1,203353,0.0102869796],[1,203369,0.0653543824],[1,203390,0.0271259898],[1,203425,0.1115524452],[1,203430,0.2343050091],[1,203434,0.2021666533],[1,203441,0.0231466831],[1,203452,0.0041061705],[1,203507,0.0142644229],[1,203595,0.0297702768],[1,203651,0.0297859376],[1,203759,0.0203560577],[1,203770,0.0361055696],[1,203789,0.0151088181],[1,203913,0.0249338484],[1,203928,0.0089417211],[1,203955,0.0682907844],[1,203983,0.1352794661],[1,204044,0.0426391296],[1,204051,0.0225400084],[1,204057,0.6103458588],[1,204086,0.5507927642],[1,204089,0.1779687788],[1,204106,0.0606229671],[1,204109,0.0137627742],[1,204153,0.1021753809],[1,204175,0.0313407822],[1,204215,0.042310384],[1,204234,0.5502605832],[1,204306,0.0723076834],[1,204323,0.0242239081],[1,204350,0.0090036747],[1,204432,0.6651725117],[1,204479,0.0878108195],[1,204519,0.0250985376],[1,204529,0.0222615747],[1,204610,0.0053205873],[1,204653,0.1895558948],[1,204769,0.0359578571],[1,204780,0.0365279072],[1,204785,0.008170053],[1,204892,0.0259984458],[1,205020,0.0320921528],[1,205023,0.0442378597],[1,205261,0.0087645964],[1,205292,0.0038570685],[1,205294,0.0204130408],[1,205302,0.0027001948],[1,205340,0.0364661266],[1,205394,0.1195622185],[1,205456,0.0074213303],[1,205483,0.0468024781],[1,205572,0.0352995073],[1,205651,0.0102869796],[1,205730,0.0344898774],[1,205767,0.2932027952],[1,205829,0.0335771355],[1,205842,0.0038570685],[1,205861,0.0396468698],[1,205915,0.0088633937],[1,205924,0.1375177552],[1,205966,0.1681979956],[1,205973,0.3591350295],[1,205995,0.2709953285],[1,206032,0.0075604437],[1,206044,0.0090557491],[1,206046,0.0209255384],[1,206090,0.008170053],[1,206134,0.0085681957],[1,206170,0.028529234],[1,206221,0.1048840187],[1,206255,0.0220159554],[1,206303,0.0466803296],[1,206310,0.1172948711],[1,206323,0.0369159802],[1,206336,0.0250359206],[1,206343,0.0395510168],[1,206355,0.3081808296],[1,206359,0.0187878577],[1,206389,0.0090036747],[1,206391,0.0347911764],[1,206408,0.0075731565],[1,206424,0.3616642877],[1,206442,0.0070891676],[1,206447,0.0102869796],[1,206455,0.0689469266],[1,206470,0.0065624631],[1,206512,0.0449981084],[1,206541,0.0131051469],[1,206569,0.0280640764],[1,206656,0.0243550581],[1,206764,0.0081649309],[1,206771,0.1015726619],[1,206773,0.0121624508],[1,206779,0.0466989963],[1,206858,0.0820960731],[1,206902,0.0598401811],[1,206910,0.3745489672],[1,206942,0.3306467113],[1,206996,0.1794321719],[1,207053,0.0221407243],[1,207062,0.1148559817],[1,207104,0.0662649796],[1,207143,0.007698383],[1,207148,0.0146739603],[1,207215,0.0067740928],[1,207233,0.0069457637],[1,207244,0.0126784036],[1,207249,0.0121624508],[1,207254,0.0273960081],[1,207338,0.1163759558],[1,207411,0.007698383],[1,207421,0.0661811949],[1,207453,0.6088936736],[1,207545,0.0067740928],[1,207599,0.0761293666],[1,207646,0.0103668764],[1,207674,0.1177918745],[1,207712,0.2759986772],[1,207779,0.0513408844],[1,207847,0.0089417211],[1,207889,0.0071712383],[1,207950,0.0100637167],[1,208009,0.0077831067],[1,208026,0.036660743],[1,208096,0.0409338935],[1,208166,0.4947167865],[1,208168,0.0408577616],[1,208176,0.3410349746],[1,208232,0.0460506136],[1,208316,0.020641612],[1,208360,0.0252500614],[1,208446,0.015135828],[1,208447,0.0925015189],[1,208494,0.0084162873],[1,208500,0.0098980082],[1,208516,0.0049087015],[1,208670,0.1066500184],[1,208696,0.0630746637],[1,208714,0.0091114898],[1,208749,0.0478315391],[1,208773,0.0103073534],[1,208775,0.1825433058],[1,208796,0.0052185865],[1,208827,0.0208085097],[1,208852,0.021928292],[1,208884,0.0078886238],[1,208973,0.143987178],[1,209021,0.0168013799],[1,209052,0.005427859],[1,209129,0.0191201449],[1,209149,0.0118617647],[1,209192,0.0294367293],[1,209208,0.3034033249],[1,209219,0.0152846845],[1,209249,0.0472077571],[1,209256,0.1247335655],[1,209257,0.0088884214],[1,209258,0.031972909],[1,209276,0.0198297576],[1,209306,0.0116199214],[1,209315,0.0097503453],[1,209321,0.0377729402],[1,209342,0.0035569346],[1,209497,0.0097756482],[1,209638,0.0827143005],[1,209653,0.0973067299],[1,209767,0.015702267],[1,209860,0.1313812697],[1,210010,0.0089689136],[1,210018,0.3357477506],[1,210064,0.0269406254],[1,210105,0.0444808044],[1,210134,0.0040349068],[1,210148,0.1139151145],[1,210241,0.0088548992],[1,210471,0.0197440931],[1,210489,0.0444808044],[1,210492,0.0139714898],[1,210658,0.0258862804],[1,210730,0.1232504171],[1,210778,0.1921407431],[1,210877,0.008776581],[1,210995,0.0441468442],[1,211052,0.013749578],[1,211102,0.2857292965],[1,211147,0.0074737859],[1,211162,0.0423771965],[1,211167,0.0453330279],[1,211172,0.0232039808],[1,211262,0.0101999978],[1,211300,0.0140316528],[1,211302,0.0088548992],[1,211448,0.0064759238],[1,211532,0.1024860138],[1,211560,0.0101999978],[1,211588,0.0116059412],[1,211619,0.0381485242],[1,211705,0.0037708507],[1,211707,0.0370144829],[1,211803,0.1977039264],[1,211806,0.2602492785],[1,211820,0.0185155114],[1,211875,0.0455499291],[1,211881,0.0688529754],[1,211921,0.0455499291],[1,211943,0.0491057476],[1,211948,0.0145864574],[1,211966,0.0450026107],[1,212012,0.0173665566],[1,212218,0.0477929442],[1,212233,0.0101999978],[1,212236,0.0076964224],[1,212257,0.0037093407],[1,212292,0.0101999978],[1,212327,0.0121887942],[1,212337,0.0984136134],[1,212462,0.0085582699],[1,212480,0.1634313403],[1,212498,0.0183559194],[1,212506,0.0229565306],[1,212576,0.0445640335],[1,212600,0.363833277],[1,212633,0.0087028062],[1,212656,0.0182413801],[1,212673,0.0153580195],[1,212701,0.0099967965],[1,212726,0.0844312563],[1,212749,0.2090135317],[1,212839,0.0223605463],[1,212842,0.8484715671],[1,212884,0.0120752462],[1,212894,0.027380062],[1,212922,0.0221509326],[1,212957,0.0349530993],[1,213015,0.0069526459],[1,213058,0.0288309537],[1,213075,0.0375490519],[1,213117,0.1038806914],[1,213134,0.0101999978],[1,213135,0.0115042705],[1,213144,0.0239024334],[1,213190,0.0363811363],[1,213233,0.0153580195],[1,213314,0.0554877497],[1,213345,0.0242351849],[1,213367,0.0058470259],[1,213415,0.0359141164],[1,213419,0.0445640335],[1,213421,0.0783693856],[1,213449,0.0179886771],[1,213481,0.1593748255],[1,213568,0.346224764],[1,213579,0.0381402702],[1,213589,0.008776581],[1,213624,0.0076117088],[1,213678,0.0465448778],[1,213915,0.0269465696],[1,213976,0.0678273018],[1,214028,0.0132279376],[1,214145,0.0870410911],[1,214155,0.0058470259],[1,214185,0.2100497032],[1,214302,0.0881597915],[1,214304,0.1443100069],[1,214318,0.2464592071],[1,214340,0.0372228256],[1,214462,0.0266671613],[1,214469,0.0361703957],[1,214488,0.0155162092],[1,214515,0.0240101892],[1,214521,0.2396094708],[1,214527,0.0417878591],[1,214632,0.2110859828],[1,214635,0.0080833227],[1,214648,0.1896079724],[1,214701,0.1973932911],[1,214732,0.0086777955],[1,214922,0.0861800271],[1,214931,0.0189624274],[1,214967,0.0334793309],[1,215071,0.1153581884],[1,215107,0.0997570688],[1,215176,0.0145124984],[1,215248,0.0056503174],[1,215316,0.0076117088],[1,215341,0.0401885423],[1,215402,0.08323119],[1,215437,0.0180702114],[1,215440,0.3394644481],[1,215462,0.0363867306],[1,215605,0.0533797523],[1,215613,0.0227839382],[1,215629,0.0444808044],[1,215707,0.0981676273],[1,215736,0.0444808044],[1,215780,0.0101999978],[1,215838,0.0064552031],[1,215957,0.0125429799],[1,215966,0.0137090977],[1,216063,0.1183855216],[1,216178,0.5681086006],[1,216208,0.0070025658],[1,216296,0.0137090977],[1,216318,0.0688529754],[1,216327,0.0828142274],[1,216396,0.074210771],[1,216528,0.0278846634],[1,216543,0.0080833227],[1,216586,0.0441468442],[1,216602,0.0088548992],[1,216618,0.0199037125],[1,216650,0.0364420232],[1,216693,0.0088896186],[1,216763,0.0087028062],[1,216775,0.0937010904],[1,217033,0.0412168521],[1,217037,0.0499058618],[1,217038,0.004019923],[1,217277,0.0092060296],[1,217295,0.0233925968],[1,217328,0.0237113483],[1,217329,0.1267364338],[1,217349,0.0380818521],[1,217395,0.0141318026],[1,217405,0.0070025658],[1,217436,0.0170894185],[1,217457,0.0146812464],[1,217470,0.0457887476],[1,217471,0.0441468442],[1,217549,0.0162855237],[1,217553,0.3067041617],[1,217555,0.0911076012],[1,217580,0.0259974612],[1,217581,0.0224853562],[1,217589,0.0582873372],[1,217601,0.1010156323],[1,217608,0.0825359362],[1,217687,0.0075975666],[1,217695,0.0455499291],[1,217806,0.1310758502],[1,217837,0.017878923],[1,217855,0.0101999978],[1,217947,0.0439382446],[1,218003,0.4007835662],[1,218017,0.0123412043],[1,218088,0.0064552031],[1,218117,0.0082836528],[1,218138,0.0332928002],[1,218230,0.0449642362],[1,218335,0.0119608818],[1,218429,0.0081567555],[1,218436,0.0364696996],[1,218455,0.0554102633],[1,218487,0.0094318743],[1,218495,0.0366557253],[1,218538,0.003011108],[1,218543,0.0398662032],[1,218606,0.0051130741],[1,218609,0.0830413453],[1,218621,0.0305310205],[1,218628,0.0300735051],[1,218666,0.5500389861],[1,218710,0.1652862531],[1,218748,0.1886720599],[1,218751,0.0029295057],[1,218768,0.013391121],[1,218781,0.1419465631],[1,218803,0.0035352445],[1,218806,0.0843715888],[1,218833,0.289543155],[1,218996,0.1619580881],[1,219010,0.1943198517],[1,219024,0.009514177],[1,219036,0.00399913],[1,219053,0.0291144589],[1,219065,0.0011097284],[1,219080,0.0043271699],[1,219090,0.0803978999],[1,219093,0.1767791117],[1,219127,0.0158461837],[1,219128,0.0301170678],[1,219241,0.1048851395],[1,219258,0.0035352445],[1,219284,0.0082062551],[1,219349,0.0400267924],[1,219374,0.020626928],[1,219440,0.0061090688],[1,219532,0.0225867439],[1,219612,0.012589866],[1,219632,0.1404060925],[1,219663,0.128551215],[1,219710,0.1887278843],[1,219768,0.0649230785],[1,219771,0.0112392092],[1,219793,0.0029295057],[1,219805,0.0728613412],[1,219860,0.2875544288],[1,219874,0.0132473019],[1,219958,0.0633847513],[1,220080,0.0215291894],[1,220100,0.1730385024],[1,220127,0.0021337456],[1,220142,0.0412614469],[1,220185,0.0186620894],[1,220219,0.0044000935],[1,220220,0.0301078763],[1,220386,0.020156362],[1,220436,0.0033980923],[1,220437,0.0273964035],[1,220661,0.1143556047],[1,220672,0.0045953733],[1,220779,0.0044156918],[1,220794,0.0061090688],[1,220812,0.3944621471],[1,220946,0.00399913],[1,221026,0.0434268261],[1,221155,0.0381563037],[1,221207,0.0402684771],[1,221236,0.176185751],[1,221251,0.0061090688],[1,221271,0.0325736638],[1,221273,0.0029244292],[1,221315,0.0045953733],[1,221380,0.0174877973],[1,221384,0.0330317436],[1,221393,0.0184770897],[1,221402,0.0232130457],[1,221433,0.4099250619],[1,221470,0.0033787777],[1,221482,0.031394838],[1,221488,0.0198460554],[1,221638,0.0015722006],[1,221721,0.0033438792],[1,221802,0.0127134584],[1,221817,0.2500831262],[1,221831,0.1470310525],[1,221881,0.246835856],[1,222025,0.0073184812],[1,222070,0.0246445999],[1,222075,0.0502020447],[1,222082,0.3168978321],[1,222110,0.026451495],[1,222158,0.0035352445],[1,222162,0.3891372002],[1,222345,0.0431206491],[1,222353,0.0103294873],[1,222379,0.0241335624],[1,222441,0.0113523656],[1,222557,0.0575755026],[1,222630,0.0134646421],[1,222797,0.0184838102],[1,222945,0.0067881582],[1,222954,0.1278683884],[1,222970,0.1680266509],[1,222980,0.0602550209],[1,223060,0.0536186151],[1,223143,0.0118028136],[1,223181,0.0049065323],[1,223252,0.0055983935],[1,223287,0.0042849039],[1,223406,0.2238484199],[1,223441,0.1309432701],[1,223520,0.041428184],[1,223539,0.0267812551],[1,223544,0.0233625018],[1,223579,0.0031432477],[1,223584,0.0043607438],[1,223585,0.0268531088],[1,223596,0.0080947688],[1,223618,0.0146663613],[1,223772,0.0390350902],[1,223804,0.0045224711],[1,223862,0.001207687],[1,223941,0.0515645793],[1,224014,0.1773839149],[1,224047,0.0045078892],[1,224050,0.0266506791],[1,224053,0.0268307155],[1,224057,0.0232448986],[1,224059,0.0209009469],[1,224066,0.1307825089],[1,224076,0.0603020945],[1,224084,0.0321180702],[1,224107,0.029593504],[1,224223,0.0511886199],[1,224228,0.6124012945],[1,224265,0.0419439877],[1,224357,0.014311313],[1,224358,0.0016841112],[1,224377,0.1061440187],[1,224434,0.0383230795],[1,224463,0.0709615615],[1,224474,0.0047249897],[1,224477,0.1475813879],[1,224504,0.01542621],[1,224559,0.0059185145],[1,224560,0.0073288544],[1,224568,0.0520782682],[1,224580,0.720542128],[1,224625,0.0377080384],[1,224647,0.087734782],[1,224684,0.0176197187],[1,224787,0.031711859],[1,224802,0.005242482],[1,224829,0.005091141],[1,224834,0.0858314313],[1,225056,0.0470550541],[1,225119,0.0188771755],[1,225162,0.0064263171],[1,225242,0.0202891331],[1,225337,0.0065809306],[1,225351,0.2224710565],[1,225399,0.022343358],[1,225463,0.0028546498],[1,225478,0.0065809306],[1,225620,0.0517393412],[1,225625,0.0364132082],[1,225727,0.0210525898],[1,225794,0.0048882497],[1,225914,0.1336184367],[1,226076,0.0091617855],[1,226320,0.0001835685],[1,226331,0.0026030304],[1,226338,0.0062960657],[1,226418,0.0313002223],[1,226445,0.0331114626],[1,226453,0.2620434041],[1,226495,0.247655952],[1,226553,0.0324916946],[1,226554,0.303326885],[1,226704,0.9203534878],[1,226749,0.0103337767],[1,226902,0.0061610442],[1,226950,0.0198741326],[1,226951,0.003634114],[1,226954,0.0481562768],[1,227023,0.0438735217],[1,227061,0.0117143964],[1,227063,0.1230876037],[1,227080,0.0193421318],[1,227120,0.0011449812],[1,227211,0.0084039353],[1,227234,0.004825314],[1,227252,0.0481033347],[1,227281,0.5857587669],[1,227294,0.0019818085],[1,227360,0.3518205469],[1,227364,0.0243840351],[1,227414,0.0015426389],[1,227460,0.0293826742],[1,227473,0.0184949088],[1,227541,0.1136472049],[1,227581,0.8315085168],[1,227663,0.1058505159],[1,227700,0.6518305422],[1,227714,0.0065809306],[1,227807,0.1625095643],[1,227817,0.0107745579],[1,227920,0.0324937439],[1,228073,0.1167340301],[1,228158,0.0038774902],[1,228252,0.0847879203],[1,228273,0.6214038397],[1,228297,0.034140308],[1,228308,0.0503565833],[1,228341,0.006063312],[1,228383,0.0190277187],[1,228396,0.0274774044],[1,228444,0.0060964275],[1,228485,0.0028896221],[1,228727,0.0428120873],[1,228745,0.303953646],[1,228769,0.01153365],[1,228803,0.1880384962],[1,228868,0.0428120873],[1,228872,0.0035740162],[1,228880,0.0212603976],[1,228886,0.139272784],[1,228906,0.0261725949],[1,228948,0.0042741035],[1,229020,0.1213842592],[1,229066,0.0941671655],[1,229105,0.0042208366],[1,229197,0.0163237579],[1,229212,0.0384044508],[1,229213,0.005683397],[1,229272,0.0082056689],[1,229308,0.0124572137],[1,229372,0.1150063088],[1,229379,0.1200689125],[1,229416,0.1741828815],[1,229430,0.0075882774],[1,229595,0.0026401922],[1,229631,0.0218521891],[1,229634,0.1585658124],[1,229705,0.2191425499],[1,229720,0.0779190263],[1,229799,0.0089652208],[1,229872,0.005474051],[1,229947,0.1983742167],[1,229962,0.0233407045],[1,229998,0.0145086427],[1,230185,0.006071506],[1,230186,0.0413100619],[1,230187,0.0922407948],[1,230201,0.0075882774],[1,230219,0.0038774902],[1,230240,0.1571778293],[1,230416,0.273583604],[1,230473,0.057652466],[1,230476,0.0306399842],[1,230497,0.0058932557],[1,230516,0.0832097494],[1,230604,0.2115708875],[1,230634,0.2199268429],[1,230637,0.0162335451],[1,230650,0.1254580952],[1,230729,0.0037810562],[1,230813,0.0054791542],[1,230824,0.0134733306],[1,230865,0.0172964417],[1,230881,0.0898433028],[1,230887,0.0025066097],[1,230917,0.1666538982],[1,230920,0.0110912423],[1,230964,0.0356508525],[1,230989,0.1591320922],[1,231076,0.088830213],[1,231094,0.0060964275],[1,231102,0.0143089463],[1,231109,0.3289163178],[1,231111,0.0327728947],[1,231181,0.3265250318],[1,231192,0.0059904374],[1,231214,0.0739774963],[1,231221,0.0074989326],[1,231223,0.0872333726],[1,231292,0.0042208366],[1,231303,0.0054791542],[1,231308,0.0129943392],[1,231383,0.0048717921],[1,231389,0.0080301524],[1,231469,0.2839936309],[1,231502,0.2266618381],[1,231544,0.0038546382],[1,231654,0.0075882774],[1,231726,0.0209953831],[1,231928,0.0315795188],[1,231956,0.0026401922],[1,232071,0.0276228572],[1,232125,0.011959087],[1,232198,0.0026401922],[1,232201,0.0224905657],[1,232248,0.0072455276],[1,232263,0.0048717921],[1,232268,0.1243244265],[1,232317,0.0235234517],[1,232329,0.0128548893],[1,232330,0.0228345719],[1,232337,0.0021448105],[1,232475,0.0054791542],[1,232492,0.0114033834],[1,232520,0.0312719752],[1,232563,0.1519259118],[1,232572,0.0428120873],[1,232620,0.0071055162],[1,232666,0.1347648834],[1,232709,0.0027231121],[1,232730,0.0372664827],[1,232732,0.0943161925],[1,232743,0.0082749204],[1,232803,0.0167200415],[1,232807,0.1039063971],[1,232841,0.005474051],[1,232859,0.6553386684],[1,232861,0.0299312968],[1,232886,0.0038774902],[1,232897,0.2680685891],[1,232909,0.0066924976],[1,233042,0.0391093369],[1,233060,0.2222643368],[1,233072,0.0042593779],[1,233079,0.0550698092],[1,233137,0.0148745116],[1,233160,0.0295074373],[1,233177,0.1256171971],[1,233229,0.5638457434],[1,233261,0.0914841095],[1,233285,0.0032558731],[1,233317,0.0070932066],[1,233350,0.003604876],[1,233359,0.0539546813],[1,233388,0.033417341],[1,233390,0.0276504693],[1,233471,0.0428120873],[1,233502,0.0501118152],[1,233515,0.0054791542],[1,233518,0.2201832276],[1,233562,0.0411468428],[1,233624,0.0165462311],[1,233660,0.0226770022],[1,233704,0.0000294589],[1,233718,0.1141336358],[1,233864,0.0075882774],[1,233923,0.0251654227],[1,234147,0.1901568717],[1,234155,0.0308056107],[1,234231,0.7371396909],[1,234250,0.0137099069],[1,234265,0.0075882774],[1,234269,0.4161814883],[1,234292,0.400922933],[1,234382,0.1700708332],[1,234406,0.0287659569],[1,234417,0.0080399445],[1,234448,0.0333819952],[1,234502,0.0080094641],[1,234547,0.1860808772],[1,234577,0.0014302519],[1,234580,0.0414140084],[1,234693,0.0075882774],[1,234742,0.0441903548],[1,234775,0.2440337979],[1,234806,0.2039711495],[1,234894,0.0438660728],[1,235139,0.0132671664],[1,235147,0.0037810562],[1,235236,0.048596892],[1,235241,0.0475911711],[1,235243,0.1370633666],[1,235281,0.0075882774],[1,235293,0.0607650512],[1,235297,0.012060505],[1,235319,0.2198429775],[1,235398,0.0090118186],[1,235399,0.0569501016],[1,235411,0.033491359],[1,235424,0.0372642598],[1,235507,0.1108551523],[1,235533,0.0367713813],[1,235547,0.0680491247],[1,235634,0.0297761176],[1,235640,0.0104909011],[1,235748,0.0241571064],[1,235817,0.2591776745],[1,235902,0.0134707651],[1,235917,0.0320075471],[1,235966,0.007761303],[1,235968,0.0131383558],[1,236038,0.0527577571],[1,236089,0.0487245655],[1,236090,0.0100177316],[1,236118,0.0631447294],[1,236160,0.4576860993],[1,236207,0.0412690903],[1,236228,0.0136736832],[1,236230,0.194564571],[1,236275,0.1770506785],[1,236322,0.0941431899],[1,236327,0.0958050216],[1,236337,0.0407985267],[1,236366,0.0476510179],[1,236472,0.026540687],[1,236519,0.010503665],[1,236639,0.0328213816],[1,236644,0.0585980294],[1,236662,0.5177701102],[1,236703,0.5864581183],[1,236848,0.0106293947],[1,236951,0.0093156225],[1,236986,0.3644116933],[1,237000,0.6359269503],[1,237007,0.0465049033],[1,237061,0.0271564293],[1,237087,0.2127222478],[1,237091,0.1558629484],[1,237096,0.0470851276],[1,237165,0.0748036629],[1,237175,0.0136389515],[1,237220,0.0632231535],[1,237233,0.0081714845],[1,237246,0.1066953067],[1,237276,0.0520170773],[1,237347,0.1395157753],[1,237511,0.0796088363],[1,237586,0.0074254022],[1,237598,0.027356574],[1,237711,0.0982752218],[1,237748,0.0087429762],[1,237785,0.0238701416],[1,237798,0.0077096105],[1,237823,0.0502892056],[1,237993,0.0107876629],[1,238257,0.2950438245],[1,238408,0.0480982224],[1,238524,0.0098586979],[1,238643,0.0448078769],[1,238646,0.0209274732],[1,238695,0.0078600582],[1,238709,0.1355850862],[1,238722,0.0134569351],[1,238723,0.0816202681],[1,238731,0.043113774],[1,238793,0.0219591885],[1,238814,0.0779664878],[1,238828,0.0409936931],[1,238835,0.0466904781],[1,238844,0.0078600582],[1,238855,0.0192245923],[1,238908,0.0304509099],[1,239037,0.4078144459],[1,239077,0.3053530546],[1,239088,0.1082873593],[1,239095,0.0130968866],[1,239139,0.04256189],[1,239220,0.048604466],[1,239267,0.0352557084],[1,239269,0.0360744685],[1,239303,0.0189078928],[1,239331,0.0317696065],[1,239332,0.0098971682],[1,239402,0.0089222205],[1,239548,0.0111906244],[1,239563,0.0746383613],[1,239572,0.0082789684],[1,239608,0.2206420796],[1,239618,0.1002598401],[1,239619,0.1367626317],[1,239665,0.0271613448],[1,239724,0.3394768537],[1,239811,0.107213606],[1,239814,0.0108475656],[1,239829,0.0083408658],[1,239832,0.0436695241],[1,239850,0.0322912927],[1,239857,0.2081755196],[1,239893,0.0298729721],[1,239991,0.0457578015],[1,239992,0.0087621241],[1,240051,0.0505459978],[1,240158,0.0187039924],[1,240167,0.0083244523],[1,240176,0.0093416988],[1,240216,0.0120748933],[1,240275,0.0284965144],[1,240482,0.0120748933],[1,240517,0.0421800168],[1,240552,0.2531540816],[1,240555,0.0417902672],[1,240645,0.1856992026],[1,240665,0.0412211078],[1,240708,0.0246531042],[1,240726,0.0212831839],[1,240742,0.024719081],[1,240747,0.0120413327],[1,240777,0.045651246],[1,240778,0.0118932434],[1,240795,0.1069048983],[1,240809,0.0076037945],[1,240810,0.0119248809],[1,240835,0.0993532208],[1,240865,0.0451462886],[1,240962,0.0097143176],[1,240995,0.0724847659],[1,241042,0.0283278366],[1,241053,0.0291067187],[1,241063,0.0245024109],[1,241108,0.1369135742],[1,241133,0.0073534923],[1,241151,0.0088692718],[1,241301,0.010964323],[1,241311,0.0083244523],[1,241316,0.0195699457],[1,241363,0.0083429367],[1,241394,0.0101993973],[1,241533,0.0548436356],[1,241667,0.0061356803],[1,241676,0.0237274528],[1,241687,0.0217234563],[1,241696,0.3134195157],[1,241753,0.0468046543],[1,241922,0.3429304641],[1,241926,0.1403838112],[1,242010,0.0488134734],[1,242024,0.0244230012],[1,242056,0.0116821105],[1,242063,0.1433654387],[1,242068,0.2341845524],[1,242235,0.0346390264],[1,242246,0.0657209353],[1,242364,0.0523021786],[1,242503,0.0430045658],[1,242511,0.0282660731],[1,242540,0.010418199],[1,242562,0.0459858622],[1,242598,0.0910506762],[1,242625,0.5190314299],[1,242647,0.0176044972],[1,242667,0.7101532862],[1,242763,0.3427633532],[1,242797,0.1076931392],[1,242801,0.0297220935],[1,242916,0.0093416988],[1,242947,0.0830224836],[1,242953,0.010964323],[1,243037,0.0464860888],[1,243047,0.0913842653],[1,243094,0.0438464867],[1,243117,0.1117856561],[1,243162,0.0704059011],[1,243168,0.0197136632],[1,243196,0.0470570867],[1,243222,0.0439270419],[1,243240,0.008341281],[1,243243,0.0093416988],[1,243306,0.2927938429],[1,243313,0.0185168995],[1,243423,0.4287841007],[1,243447,0.2539635679],[1,243476,0.0129692323],[1,243758,0.0520811265],[1,243775,0.0530776908],[1,243794,0.2470591775],[1,243801,0.0200475294],[1,243824,0.3455811049],[1,243856,0.0099249773],[1,243868,0.1329492559],[1,243906,0.077940498],[1,243946,0.0099289589],[1,243966,0.0202005686],[1,243987,0.2620234115],[1,243996,0.0279906945],[1,244125,0.0090015102],[1,244191,0.0346430043],[1,244208,0.009304218],[1,244224,0.1996229793],[1,244234,0.056130869],[1,244264,0.0059262042],[1,244351,0.008341281],[1,244381,0.010418199],[1,244399,0.0125248955],[1,244481,0.0098568024],[1,244511,0.0097969807],[1,244584,0.0110783793],[1,244605,0.0382535795],[1,244647,0.1353501734],[1,244687,0.1359825434],[1,244694,0.0091695897],[1,244706,0.4280823004],[1,244728,0.2853111344],[1,244741,0.008341281],[1,244742,0.0140569503],[1,244760,0.0083244523],[1,244773,0.0088692718],[1,244811,0.0853604887],[1,244868,0.0677067344],[1,244872,0.008704741],[1,244888,0.0377686855],[1,244920,0.0712944211],[1,244924,0.0134864038],[1,244954,0.1070213952],[1,244965,0.0569124123],[1,245015,0.0317674794],[1,245028,0.0097143176],[1,245134,0.0303188317],[1,245237,0.033573178],[1,245249,0.2374008865],[1,245282,0.0093416988],[1,245292,0.0454632164],[1,245329,0.0120748933],[1,245332,0.0491384182],[1,245342,0.0061356803],[1,245404,0.0093416988],[1,245440,0.0182225012],[1,245443,0.026574515],[1,245451,0.0088692718],[1,245539,0.0229396858],[1,245627,0.082588269],[1,245676,0.0108928903],[1,245698,0.0474321776],[1,245705,0.0194189804],[1,245712,0.0125248955],[1,245801,0.0973937181],[1,245819,0.0750012105],[1,245868,0.0093416988],[1,245877,0.0429755365],[1,245901,0.0120748933],[1,245977,0.0120748933],[1,245991,0.0217590394],[1,245997,0.0966173912],[1,246038,0.0469053258],[1,246082,0.1480639311],[1,246111,0.032160677],[1,246185,0.0244276212],[1,246198,0.010418199],[1,246221,0.3415292123],[1,246293,0.033354465],[1,246401,0.0380525531],[1,246449,0.1416505668],[1,246540,0.0556761825],[1,246548,0.0088692718],[1,246619,0.0456232598],[1,246652,0.0423662271],[1,246737,0.0455523832],[1,246750,0.0773795175],[1,246773,0.2050464742],[1,246807,0.0130821802],[1,246808,0.1850820957],[1,246819,0.1438688883],[1,246891,0.0099622503],[1,246979,0.0204149701],[1,246995,0.0465400755],[1,247041,0.1592213017],[1,247179,0.0284093139],[1,247192,0.010964323],[1,247272,0.1255832122],[1,247273,0.0437711656],[1,247280,0.0124626434],[1,247323,0.1479568801],[1,247341,0.0087621241],[1,247397,0.014686483],[1,247429,0.0430933362],[1,247459,0.0790987587],[1,247524,0.064464088],[1,247615,0.1435315371],[1,247621,0.0520213192],[1,247685,0.0171205979],[1,247711,0.0238138492],[1,247830,0.1413481227],[1,247895,0.0162513756],[1,248121,0.0259242805],[1,248268,0.013730865],[1,248336,0.1671751911],[1,248341,0.1236413317],[1,248401,0.0366038765],[1,248407,0.0518106308],[1,248424,0.0102484384],[1,248496,0.025846883],[1,248567,0.0678468584],[1,248572,0.0236005098],[1,248616,0.0152282191],[1,248654,0.0499872548],[1,248771,0.0336651624],[1,248783,0.1713123013],[1,248812,0.0989225721],[1,248829,0.1320829503],[1,248880,0.0538693934],[1,248920,0.0434862556],[1,249069,0.1088842727],[1,249123,0.0360700671],[1,249168,0.2320565247],[1,249172,0.016221206],[1,249207,0.0441399117],[1,249225,0.0138736046],[1,249226,0.016221206],[1,249229,0.0141590179],[1,249246,0.1502885667],[1,249249,0.0641005618],[1,249473,0.0652891667],[1,249606,0.5855527449],[1,249695,0.0210102277],[1,249700,0.0149478535],[1,249748,0.0348613869],[1,249780,0.1007737081],[1,249796,0.257745695],[1,249857,0.0195803727],[1,250010,0.0528714709],[1,250011,0.015573163],[1,250030,0.7185921846],[1,250064,0.0150873175],[1,250090,0.0531308903],[1,250221,0.0808716636],[1,250410,0.0151033544],[1,250429,0.1778615424],[1,250520,0.0282070477],[1,250522,0.0515358005],[1,250555,0.0378703464],[1,250568,0.0282070477],[1,250592,0.1185291986],[1,250593,0.0461034254],[1,250621,0.0537600299],[1,250793,0.0332432352],[1,250822,0.016582919],[1,250833,0.0361025973],[1,250873,0.0490228228],[1,250878,0.0467021153],[1,250884,0.0504485372],[1,250917,0.1183454418],[1,250955,0.1784854886],[1,251012,0.0733686644],[1,251013,0.0180927276],[1,251114,0.0349156932],[1,251183,0.1895877103],[1,251263,0.1755163588],[1,251291,0.1441962062],[1,251309,0.015573163],[1,251321,0.3249838536],[1,251387,0.0177020044],[1,251509,0.0502483304],[1,251512,0.0340930918],[1,251567,0.0228897044],[1,251573,0.0399062958],[1,251644,0.016582919],[1,251654,0.0518729823],[1,251696,0.0562577078],[1,251716,0.0328367638],[1,251726,0.0943901058],[1,251747,0.0129444026],[1,251823,0.8681793926],[1,251844,0.823178872],[1,251847,0.8523959703],[1,251850,0.0421826754],[1,251951,0.0437458003],[1,251974,0.2285303589],[1,251984,0.0293627021],[1,252038,0.0191711702],[1,252100,0.0314338586],[1,252203,0.0177020044],[1,252227,0.0479199404],[1,252252,0.145944473],[1,252419,0.0518239671],[1,252426,0.0177020044],[1,252489,0.0212837068],[1,252523,0.0514410075],[1,252547,0.0283026119],[1,252549,0.1080911045],[1,252578,0.0508329777],[1,252695,0.0542439368],[1,252743,0.0368462469],[1,252778,0.1742785023],[1,252780,0.0347385105],[1,252800,0.0361941461],[1,252869,0.107704905],[1,252972,0.016582919],[1,253140,0.0305914316],[1,253151,0.0229169778],[1,253163,0.0588299755],[1,253175,0.0172790414],[1,253235,0.0199052543],[1,253283,0.0124619086],[1,253340,0.0839545328],[1,253352,0.034441633],[1,253384,0.0315032473],[1,253459,0.0230223005],[1,253494,0.1817078329],[1,253524,0.1255137071],[1,253530,0.0172790414],[1,253534,0.0288090909],[1,253670,0.0184759987],[1,253676,0.0515144594],[1,253712,0.0172289919],[1,253724,0.0898577476],[1,253762,0.3058664954],[1,253797,0.0531267132],[1,253821,0.0282053074],[1,253877,0.0723188005],[1,253887,0.1342398941],[1,253906,0.0167987449],[1,253949,0.140935388],[1,253985,0.0300401503],[1,254021,0.0381529712],[1,254072,0.0214508521],[1,254073,0.0886844682],[1,254085,0.0135189538],[1,254185,0.0508304421],[1,254276,0.1538058435],[1,254312,0.0486356783],[1,254319,0.0171523462],[1,254327,0.0140313437],[1,254335,0.1034993644],[1,254349,0.0595321288],[1,254430,0.0548079827],[1,254449,0.0182152301],[1,254569,0.053296567],[1,254612,0.1212695919],[1,254647,0.1551608818],[1,254661,0.0262421211],[1,254726,0.0297318578],[1,254736,0.0172790414],[1,254758,0.0396043506],[1,254765,0.0337253145],[1,254776,0.0243954351],[1,254901,0.1397506422],[1,254926,0.0454864364],[1,254974,0.0172289919],[1,255094,0.5696517631],[1,255195,0.0388694086],[1,255203,0.0383590675],[1,255257,0.044815767],[1,255274,0.3362668942],[1,255293,0.0825085969],[1,255372,0.5381095942],[1,255382,0.1301456263],[1,255422,0.0919704774],[1,255464,0.0763305716],[1,255538,0.0744184089],[1,255552,0.0625940448],[1,255577,0.022029192],[1,255732,0.020433866],[1,255755,0.0880461812],[1,255778,0.0172790414],[1,255788,0.1838264031],[1,255831,0.6074404875],[1,255835,0.0337933045],[1,255841,0.016275461],[1,255903,0.0157421186],[1,255904,0.0247052026],[1,255948,0.0510934867],[1,255951,0.017669541],[1,255955,0.0135189538],[1,255969,0.0338242533],[1,256086,0.0172282867],[1,256116,0.0481097528],[1,256170,0.0275655759],[1,256304,0.0140506886],[1,256314,0.016160596],[1,256317,0.0407083468],[1,256335,0.0358559212],[1,256411,0.0280551737],[1,256473,0.0570100054],[1,256492,0.0596777715],[1,256493,0.0473303493],[1,256506,0.2471412843],[1,256538,0.016275461],[1,256584,0.0515650128],[1,256645,0.0242780858],[1,256648,0.0145264656],[1,256662,0.050988222],[1,256675,0.0397232148],[1,256727,0.0153615411],[1,256734,0.0145264656],[1,256735,0.0185573277],[1,256765,0.0140506886],[1,256772,0.2246972869],[1,256806,0.0769105519],[1,256856,0.0302848965],[1,256958,0.5312462989],[1,256967,0.0608275752],[1,257021,0.0210044744],[1,257035,0.0247052026],[1,257080,0.1205366153],[1,257112,0.0690648813],[1,257202,0.0510934867],[1,257212,0.0552479349],[1,257214,0.0172282867],[1,257245,0.115675303],[1,257299,0.0388445338],[1,257359,0.0441872848],[1,257367,0.0694864506],[1,257404,0.0479411057],[1,257434,0.0216183004],[1,257472,0.1177984604],[1,257490,0.0115232246],[1,257519,0.0150962138],[1,257567,0.0315340661],[1,257585,0.0151969443],[1,257662,0.0259725297],[1,257742,0.0435004501],[1,257773,0.0478717166],[1,257882,0.0443642447],[1,257897,0.5132956209],[1,257902,0.0110493881],[1,258086,0.0138124967],[1,258094,0.1074812936],[1,258187,0.110040513],[1,258198,0.3119834026],[1,258338,0.027248169],[1,258667,0.654731899],[1,258698,0.0130666649],[1,258717,0.0224262882],[1,258879,0.0724136753],[1,259035,0.5607339248],[1,259037,0.0114654715],[1,259111,0.1228194611],[1,259114,0.1167713489],[1,259137,0.0467826874],[1,259358,0.0128968246],[1,259393,0.5451256827],[1,259550,0.2549067625],[1,259616,0.0272683474],[1,259683,0.1068287876],[1,259758,0.0388824695],[1,259782,0.0616932739],[1,259805,0.198156585],[1,259879,0.0153261629],[1,259904,0.0202789554],[1,259910,0.0446516291],[1,259979,0.0626428264],[1,260004,0.0116551169],[1,260019,0.0580263017],[1,260029,0.1646351592],[1,260146,0.0128968246],[1,260210,0.0995759472],[1,260213,0.0395511296],[1,260237,0.271736676],[1,260292,0.0527719004],[1,260316,0.0130666649],[1,260553,0.1031645222],[1,260593,0.0101605691],[1,260612,0.0155051897],[1,260701,0.0271849129],[1,260722,0.4970130594],[1,260769,0.3868648621],[1,260832,0.0649155227],[1,260946,0.0437541368],[1,261045,0.5865370205],[1,261199,0.0262826286],[1,261268,0.0085584229],[1,261331,0.2059175978],[1,261373,0.0403707082],[1,261409,0.4259678973],[1,261466,0.0468457721],[1,261490,0.0129268583],[1,261495,0.0075710887],[1,261526,0.0305832608],[1,261562,0.0257232304],[1,261663,0.3192431535],[1,261684,0.0409544228],[1,261808,0.0365869185],[1,261813,0.0114667626],[1,261868,0.0128968246],[1,261924,0.0425349394],[1,261969,0.0121195084],[1,262029,0.1736573924],[1,262085,0.0134512489],[1,262099,0.0528728477],[1,262123,0.0143274504],[1,262136,0.0157441412],[1,262195,0.0565642795],[1,262211,0.0434018408],[1,262356,0.1309050397],[1,262358,0.0124452082],[1,262393,0.017610985],[1,262459,0.0969669811],[1,262597,0.0151976904],[1,262670,0.5732185765],[1,262690,0.0146645143],[1,262714,0.0165897348],[1,262733,0.0504097235],[1,262770,0.0215272639],[1,262785,0.441168974],[1,262796,0.013852272],[1,262835,0.0896206549],[1,262851,0.0140752882],[1,262893,0.1077716818],[1,262903,0.2170640422],[1,262945,0.0140752882],[1,262955,0.0468528825],[1,263046,0.1119925152],[1,263112,0.0231067409],[1,263128,0.0692904096],[1,263137,0.9004754065],[1,263441,0.5618244858],[1,263454,0.0140752882],[1,263554,0.7849278048],[1,263572,0.3062289548],[1,263610,0.0448556015],[1,263632,0.0150829931],[1,263646,0.2584485239],[1,263681,0.015674209],[1,263721,0.0147008726],[1,263888,0.1183481803],[1,263890,0.0395736384],[1,263911,0.0383810246],[1,263963,0.0148928253],[1,264007,0.0236151229],[1,264010,0.0183856126],[1,264059,0.161506337],[1,264071,0.0383492355],[1,264121,0.0285504229],[1,264183,0.0134512489],[1,264197,0.0354722974],[1,264203,0.3648270036],[1,264216,0.0824495494],[1,264269,0.0270890203],[1,264285,0.0358286795],[1,264302,0.0506177467],[1,264320,0.0319276693],[1,264343,0.0915919097],[1,264399,0.0372042151],[1,264420,0.0251827332],[1,264421,0.2245056217],[1,264466,0.0308245691],[1,264477,0.0290975479],[1,264549,0.0292368668],[1,264582,0.1865948464],[1,264585,0.0150829931],[1,264595,0.0527124659],[1,264703,0.049964875],[1,264789,0.0156502728],[1,264791,0.0695060704],[1,264800,0.0765628365],[1,264931,0.0175941004],[1,264951,0.0190079461],[1,264977,0.0143643715],[1,265024,0.0203435166],[1,265088,0.0688370164],[1,265154,0.0139918006],[1,265160,0.0141365897],[1,265224,0.0305864888],[1,265230,0.0517600925],[1,265233,0.0176243261],[1,265527,0.0364301894],[1,265667,0.0169019807],[1,265674,0.01383241],[1,265681,0.0846371287],[1,265713,0.0722026056],[1,266130,0.0666769946],[1,266142,0.2922510191],[1,266187,0.1141961043],[1,266190,0.0213603186],[1,266353,0.051528663],[1,266413,0.1328133488],[1,266496,0.0175941004],[1,266582,0.037360086],[1,266588,0.2016370307],[1,266621,0.1717969535],[1,266655,0.0645962001],[1,266656,0.0836451404],[1,266664,0.0795270669],[1,266681,0.0938487064],[1,266687,0.0293550144],[1,266701,0.4690947057],[1,266711,0.0437558023],[1,266723,0.3043352727],[1,266843,0.0866156528],[1,266880,0.3165117355],[1,266934,0.0164751783],[1,266948,0.0300191705],[1,267004,0.0171983638],[1,267011,0.1182211988],[1,267081,0.0468808345],[1,267082,0.0471812034],[1,267100,0.2818008524],[1,267132,0.5181448403],[1,267143,0.0175941004],[1,267190,0.0143643715],[1,267212,0.0156502728],[1,267218,0.1748601928],[1,267222,0.0382948433],[1,267236,0.0528393729],[1,267282,0.0592915339],[1,267283,0.1082472447],[1,267309,0.1501230463],[1,267400,0.3734107428],[1,267453,0.2337051853],[1,267514,0.0154065475],[1,267558,0.1090309874],[1,267562,0.0175941004],[1,267571,0.2591684215],[1,267674,0.0333373008],[1,267693,0.0200389701],[1,267716,0.126400645],[1,267788,0.0260590361],[1,267856,0.103605],[1,267879,0.0134395401],[1,267920,0.0514229599],[1,268014,0.0146071369],[1,268035,0.0176243261],[1,268074,0.1326835611],[1,268075,0.0906008283],[1,268078,0.0814999479],[1,268086,0.0370952174],[1,268094,0.0175440296],[1,268138,0.0175941004],[1,268163,0.1562387337],[1,268222,0.1430462172],[1,268238,0.0173170422],[1,268239,0.143393776],[1,268257,0.0777408194],[1,268325,0.0231186022],[1,268328,0.0179847666],[1,268331,0.0181893461],[1,268365,0.0303337517],[1,268375,0.0280973566],[1,268428,0.0156689426],[1,268576,0.0188628869],[1,268587,0.0143643715],[1,268618,0.0273719141],[1,268637,0.0474457819],[1,268670,0.0533082398],[1,268914,0.0175941004],[1,268942,0.0168118146],[1,268999,0.0166398688],[1,269068,0.0179446457],[1,269116,0.0259449838],[1,269146,0.0421056366],[1,269186,0.0454275527],[1,269230,0.0485322693],[1,269249,0.0307556227],[1,269262,0.0599524185],[1,269271,0.0374386898],[1,269282,0.0257966031],[1,269393,0.097123579],[1,269407,0.0370641421],[1,269439,0.0538011703],[1,269441,0.0209192052],[1,269619,0.065905318],[1,269648,0.0866491367],[1,269649,0.0641372023],[1,269650,0.0445183465],[1,269818,0.0311505538],[1,269824,0.0300101376],[1,269865,0.3546080745],[1,269896,0.0180474868],[1,269929,0.0286567995],[1,269943,0.01383241],[1,269954,0.0164751783],[1,269963,0.4055087858],[1,270037,0.0298975897],[1,270082,0.0423778355],[1,270122,0.0398389573],[1,270129,0.0606395316],[1,270156,0.0146695708],[1,270167,0.0812005538],[1,270176,0.03207859],[1,270283,0.0152421321],[1,270288,0.0399443066],[1,270302,0.0179847666],[1,270314,0.1303345245],[1,270318,0.0180474868],[1,270321,0.0344743564],[1,270470,0.0148403513],[1,270507,0.0176243261],[1,270529,0.085164633],[1,270537,0.0454193708],[1,270546,0.1514390795],[1,270584,0.0516215913],[1,270613,0.0293550144],[1,270679,0.2164338551],[1,270726,0.2809573927],[1,270731,0.0619994722],[1,270732,0.0680646633],[1,270823,0.4136547172],[1,270844,0.0143643715],[1,270881,0.0530998217],[1,270905,0.0175941004],[1,270958,0.0156502728],[1,270988,0.0942137983],[1,271004,0.0730311645],[1,271032,0.0468090329],[1,271087,0.0231186022],[1,271203,0.0596518451],[1,271207,0.0202897277],[1,271253,0.4697816396],[1,271276,0.0528457555],[1,271278,0.3430705997],[1,271366,0.0513785902],[1,271394,0.2464070834],[1,271403,0.0204149802],[1,271425,0.0277643449],[1,271463,0.0401051113],[1,271478,0.1968363999],[1,271518,0.014943472],[1,271587,0.7113120454],[1,271605,0.0150828686],[1,271690,0.0403058561],[1,271695,0.0375781819],[1,271719,0.0523370818],[1,271756,0.0404531334],[1,271770,0.3594700273],[1,271841,0.0190943379],[1,271905,0.0326909862],[1,271971,0.035197303],[1,271973,0.1509414341],[1,272004,0.0171330874],[1,272007,0.1613704391],[1,272014,0.0687621882],[1,272071,0.0309818991],[1,272155,0.0590985457],[1,272190,0.046599553],[1,272193,0.0517939129],[1,272201,0.0474583446],[1,272203,0.1682336254],[1,272232,0.1096850627],[1,272261,0.094764171],[1,272419,0.0167302759],[1,272422,0.016253075],[1,272475,0.0352321668],[1,272490,0.0164305508],[1,272518,0.0167302759],[1,272519,0.0170510197],[1,272607,0.3097194201],[1,272622,0.0923805974],[1,272695,0.0269978151],[1,272698,0.016253075],[1,272834,0.0194910898],[1,272868,0.0194910898],[1,272887,0.0180481927],[1,272916,0.0718664199],[1,272917,0.1219057527],[1,273004,0.0167302759],[1,273018,0.0209027529],[1,273038,0.0267719458],[1,273074,0.1219920777],[1,273117,0.1143247615],[1,273306,0.1115669946],[1,273320,0.1118240277],[1,273324,0.013956534],[1,273348,0.0930418824],[1,273369,0.0323416691],[1,273410,0.0289743876],[1,273423,0.1967347077],[1,273458,0.0975464856],[1,273556,0.0176687798],[1,273596,0.0245430454],[1,273612,0.1352272174],[1,273629,0.0422279328],[1,273687,0.0260069408],[1,273759,0.0184481624],[1,273768,0.0149646453],[1,273797,0.014793836],[1,273876,0.0389142226],[1,273912,0.0221737918],[1,273956,0.0187213065],[1,273964,0.0287879614],[1,273980,0.0366160853],[1,274007,0.0278488613],[1,274014,0.020344343],[1,274015,0.017718859],[1,274036,0.6923387513],[1,274113,0.1268313926],[1,274130,0.0271861246],[1,274153,0.2204603253],[1,274189,0.0281023049],[1,274228,0.2043871414],[1,274291,0.0355216352],[1,274311,0.0173299607],[1,274357,0.1490368605],[1,274447,0.0181095911],[1,274467,0.0183142052],[1,274480,0.0247799558],[1,274486,0.0343077428],[1,274510,0.0161810217],[1,274570,0.1847443773],[1,274607,0.2002981193],[1,274614,0.5001632577],[1,274663,0.0177490898],[1,274716,0.0165383046],[1,274725,0.032209087],[1,274747,0.1452010404],[1,274767,0.0527631654],[1,274794,0.1286802835],[1,274813,0.0194423126],[1,274817,0.0299127739],[1,274840,0.0349098764],[1,274883,0.0517520912],[1,274900,0.0151909186],[1,274978,0.0640048116],[1,275036,0.0959659981],[1,275127,0.0199903663],[1,275149,0.0510363379],[1,275222,0.0397187489],[1,275238,0.7257024778],[1,275319,0.0518689822],[1,275367,0.0609442496],[1,275556,0.4750077155],[1,275662,0.2842835092],[1,275733,0.0194071088],[1,275783,0.0286088662],[1,275804,0.0333733097],[1,275807,0.0248826711],[1,275825,0.0717050842],[1,275876,0.2375562027],[1,275943,0.1689206688],[1,275972,0.321002289],[1,276039,0.1580979982],[1,276189,0.1094854345],[1,276201,0.0259690169],[1,276221,0.2679821517],[1,276243,0.0186050874],[1,276302,0.028157418],[1,276372,0.0468124798],[1,276495,0.0091336203],[1,276499,0.0196003947],[1,276584,0.0135455304],[1,276649,0.0496613207],[1,276795,0.0532500876],[1,276941,0.0108703148],[1,276942,0.0153185658],[1,276949,0.5015182721],[1,276954,0.0150103578],[1,276975,0.0277242988],[1,277296,0.1370796471],[1,277328,0.0151395843],[1,277371,0.1474211408],[1,277378,0.0171457846],[1,277383,0.2494367493],[1,277464,0.0668583527],[1,277587,0.1711017471],[1,277615,0.0151806868],[1,277636,0.1150136178],[1,277694,0.0135138409],[1,277698,0.0505491199],[1,277751,0.3432543612],[1,277765,0.1189532431],[1,277803,0.0150103578],[1,277807,0.0472275662],[1,277876,0.0241983454],[1,278049,0.3214138206],[1,278065,0.0462165699],[1,278148,0.2275258471],[1,278189,0.0338138712],[1,278265,0.0134781527],[1,278302,0.0174466844],[1,278552,0.0274410569],[1,278632,0.0677894817],[1,278668,0.024234474],[1,278805,0.091702747],[1,278839,0.0391330705],[1,278898,0.0106287812],[1,278905,0.1306864474],[1,279008,0.0120382658],[1,279018,0.0138329773],[1,279043,0.0233805415],[1,279089,0.053712001],[1,279118,0.0122662311],[1,279209,0.4078580395],[1,279270,0.0786344542],[1,279290,0.1384315627],[1,279330,0.0316436049],[1,279332,0.0156584654],[1,279386,0.1588804536],[1,279394,0.0235309853],[1,279412,0.0616117668],[1,279456,0.0546126375],[1,279566,0.0779422708],[1,279647,0.0299230827],[1,279740,0.4500033118],[1,279846,0.0106287812],[1,279853,0.1069660091],[1,280113,0.0791621196],[1,280115,0.0337357056],[1,280124,0.0302153858],[1,280138,0.0270288515],[1,280144,0.0850708733],[1,280149,0.014616004],[1,280226,0.0394444108],[1,280332,0.0634179566],[1,280347,0.0410847393],[1,280413,0.0212578567],[1,280415,0.0117243916],[1,280423,0.0141193085],[1,280523,0.58545942],[1,280537,0.0449143377],[1,280571,0.0545800106],[1,280575,0.0406266456],[1,280611,0.2133415887],[1,280623,0.1010574157],[1,280720,0.053712001],[1,280747,0.0536360464],[1,280805,0.1657521972],[1,280850,0.0902656382],[1,280868,0.0452527823],[1,280881,0.0117919145],[1,281006,0.2499929849],[1,281037,0.3314470381],[1,281069,0.039535561],[1,281093,0.0521364966],[1,281192,0.0117919145],[1,281282,0.0963399368],[1,281319,0.0117919145],[1,281324,0.0402011272],[1,281389,0.1854485214],[1,281391,0.046887509],[1,281457,0.1791292879],[1,281511,0.0139970784],[1,281533,0.0536627826],[1,281586,0.0888526183],[1,281589,0.0485626565],[1,281646,0.0984471501],[1,281658,0.3186752708],[1,281672,0.0924359648],[1,281733,0.0134781527],[1,281842,0.0117919145],[1,281869,0.0120338316],[1,281933,0.0819894024],[1,281939,0.013738317],[1,281948,0.1381024046],[1,281982,0.0556314194],[1,282002,0.0122662311],[1,282062,0.0142308056],[1,282067,0.0346181646],[1,282083,0.0119652042],[1,282137,0.0117919145],[1,282143,0.0251032601],[1,282174,0.0156750196],[1,282179,0.0330463155],[1,282379,0.5624278724],[1,282436,0.0133860628],[1,282564,0.0173889518],[1,282614,0.0391178857],[1,282628,0.0440449373],[1,282687,0.0341430163],[1,282688,0.0655635674],[1,282736,0.0482767869],[1,282808,0.1076357381],[1,282815,0.3480627999],[1,282837,0.028278828],[1,282842,0.0610384034],[1,282892,0.034027393],[1,282907,0.1627721544],[1,282919,0.013656303],[1,283072,0.2207408117],[1,283106,0.0442062877],[1,283131,0.0378369153],[1,283168,0.0122662311],[1,283172,0.0151395843],[1,283176,0.1836038109],[1,283209,0.0106287812],[1,283263,0.1910884296],[1,283265,0.0313928427],[1,283333,0.0552746386],[1,283425,0.1218838495],[1,283433,0.0422726752],[1,283444,0.0717050298],[1,283488,0.0266847996],[1,283529,0.5754305927],[1,283540,0.0533740178],[1,283592,0.0547940047],[1,283683,0.0122662311],[1,283695,0.0332044188],[1,283709,0.0396837889],[1,283752,0.1828975364],[1,283778,0.0228992463],[1,283785,0.1220356546],[1,283975,0.0250299231],[1,284048,0.0341912983],[1,284100,0.0378176453],[1,284217,0.0387234161],[1,284252,0.0705205255],[1,284260,0.0476186855],[1,284291,0.0123393591],[1,284316,0.1076025826],[1,284346,0.0323619193],[1,284347,0.034238476],[1,284350,0.0344568214],[1,284436,0.0181697994],[1,284438,0.0141833622],[1,284570,0.0268083553],[1,284592,0.1603354197],[1,284615,0.3265470419],[1,284624,0.0135514013],[1,284627,0.0135514013],[1,284725,0.0175203285],[1,284813,0.624021654],[1,284955,0.0150837591],[1,285038,0.0413515665],[1,285048,0.0579836958],[1,285054,0.0363280476],[1,285183,0.0965245701],[1,285262,0.0123880987],[1,285393,0.0564824003],[1,285403,0.0143041292],[1,285531,0.0150837591],[1,285561,0.018798127],[1,285568,0.0129624547],[1,285649,0.1135096459],[1,285690,0.0360768116],[1,285706,0.0485913571],[1,285725,0.0552113591],[1,285806,0.036274085],[1,285867,0.5218801408],[1,285875,0.0201424257],[1,285895,0.019819801],[1,285908,0.0240475398],[1,285919,0.0304538099],[1,285968,0.1710739359],[1,286017,0.1204819937],[1,286021,0.0394854589],[1,286104,0.0146280814],[1,286114,0.0272789287],[1,286148,0.0150837591],[1,286157,0.0248344975],[1,286226,0.0370781105],[1,286236,0.0127397757],[1,286244,0.2599259078],[1,286282,0.0490595141],[1,286294,0.017823841],[1,286333,0.058909948],[1,286422,0.6401713487],[1,286498,0.0874674393],[1,286536,0.0143120886],[1,286597,0.0150837591],[1,286689,0.0155356063],[1,286826,0.0798416576],[1,286940,0.0114751179],[1,286949,0.0528147386],[1,286959,0.0241556582],[1,287021,0.0135870931],[1,287090,0.3564376966],[1,287245,0.014689366],[1,287258,0.0541924153],[1,287369,0.0123393591],[1,287511,0.0502916168],[1,287560,0.0345347781],[1,287682,0.0226620309],[1,287754,0.0271840768],[1,287755,0.0177021717],[1,287766,0.0803297283],[1,287774,0.0223764963],[1,287816,0.0482432679],[1,287835,0.0225906795],[1,287850,0.0392693436],[1,287862,0.0140831596],[1,287901,0.0585873543],[1,287905,0.1923627807],[1,288047,0.1375107668],[1,288130,0.0150837591],[1,288155,0.0118649952],[1,288171,0.1624443153],[1,288183,0.06179568],[1,288202,0.0117974655],[1,288257,0.0146280814],[1,288307,0.0428382804],[1,288313,0.0324348559],[1,288316,0.0139686358],[1,288343,0.1032128595],[1,288436,0.022470285],[1,288466,0.0490485332],[1,288502,0.0243988606],[1,288541,0.0521577141],[1,288547,0.015676984],[1,288600,0.0351603005],[1,288653,0.0610584832],[1,288762,0.0817208603],[1,288765,0.088765311],[1,288791,0.2638549071],[1,288817,0.0441131967],[1,288868,0.0418253058],[1,289114,0.015473099],[1,289129,0.1610479057],[1,289190,0.0251164365],[1,289279,0.1146930351],[1,289570,0.0163031949],[1,289631,0.0135870931],[1,289681,0.1756423919],[1,289768,0.0116379868],[1,289774,0.015473099],[1,289775,0.0283354146],[1,289896,0.0257684777],[1,289967,0.0240725623],[1,290014,0.2774691887],[1,290016,0.0248333364],[1,290056,0.2433637861],[1,290089,0.7129492303],[1,290127,0.0146280814],[1,290129,0.0843731112],[1,290136,0.0158104639],[1,290163,0.0135731661],[1,290317,0.023301307],[1,290340,0.0562856784],[1,290353,0.1735202574],[1,290472,0.0256832503],[1,290534,0.0241224079],[1,290535,0.0856064931],[1,290539,0.0525614863],[1,290541,0.0528991153],[1,290600,0.0369505053],[1,290627,0.0671521697],[1,290654,0.014192621],[1,290711,0.0443554991],[1,290765,0.0406555367],[1,290772,0.0352098653],[1,290773,0.0139686358],[1,290828,0.1415073719],[1,290893,0.0287050184],[1,291135,0.0393213731],[1,291137,0.0265975093],[1,291175,0.0808766334],[1,291194,0.0218978367],[1,291307,0.0386762356],[1,291318,0.0287715963],[1,291328,0.0728003016],[1,291410,0.0494008921],[1,291439,0.1711445067],[1,291476,0.1019434929],[1,291536,0.0212438274],[1,291616,0.3645638668],[1,291626,0.0123393591],[1,291639,0.0498229108],[1,291650,0.0107017461],[1,291703,0.0172778547],[1,291789,0.062819211],[1,291791,0.0113348398],[1,291928,0.0372525533],[1,291961,0.0160951758],[1,291991,0.0546163713],[1,292025,0.1491915829],[1,292050,0.0362088753],[1,292089,0.0178329294],[1,292151,0.0209365984],[1,292197,0.0177021717],[1,292233,0.3851859169],[1,292238,0.0627991153],[1,292261,0.0241550053],[1,292295,0.0464103688],[1,292313,0.0534266539],[1,292365,0.038052313],[1,292460,0.1087591524],[1,292527,0.0409635131],[1,292530,0.0432464429],[1,292607,0.1584805607],[1,292611,0.0201538996],[1,292730,0.0593088807],[1,292735,0.0146540962],[1,292768,0.0521668433],[1,292841,0.2141848932],[1,292847,0.5271601857],[1,292860,0.2087859801],[1,292935,0.0293718481],[1,292997,0.0798297414],[1,293038,0.0143041292],[1,293079,0.0981114934],[1,293112,0.1089746619],[1,293126,0.0189336451],[1,293153,0.0148076415],[1,293200,0.035834095],[1,293219,0.0528991153],[1,293316,0.1708794376],[1,293444,0.0113348398],[1,293460,0.0302755264],[1,293462,0.0300789011],[1,293471,0.0129624547],[1,293579,0.0118649952],[1,293629,0.1820000161],[1,293645,0.0150837591],[1,293700,0.0833868311],[1,293749,0.0175203285],[1,293857,0.0786665761],[1,293866,0.0938464216],[1,293869,0.168488423],[1,293959,0.335027356],[1,294035,0.0429778991],[1,294047,0.0174625901],[1,294060,0.0123393591],[1,294091,0.0424983622],[1,294108,0.0556693961],[1,294173,0.0301397154],[1,294182,0.0507065716],[1,294192,0.0379025644],[1,294237,0.1211833129],[1,294238,0.6028639745],[1,294327,0.0401672588],[1,294328,0.4241527236],[1,294451,0.0877052877],[1,294463,0.0362508921],[1,294493,0.2157863857],[1,294594,0.3886298247],[1,294612,0.0137787555],[1,294616,0.1222461259],[1,294771,0.0506782139],[1,294841,0.4363976112],[1,294848,0.0133754727],[1,294857,0.0134593023],[1,294896,0.1416645129],[1,294918,0.0121069364],[1,294932,0.110482447],[1,294941,0.0399939726],[1,294945,0.015254105],[1,294963,0.0491780279],[1,295081,0.0452716687],[1,295113,0.0298478828],[1,295125,0.055070795],[1,295398,0.0525614863],[1,295408,0.5688313144],[1,295514,0.0150837591],[1,295520,0.1120629082],[1,295612,0.0139074112],[1,295622,0.0240725623],[1,295624,0.0114936892],[1,295632,0.0190106957],[1,295697,0.0528991153],[1,295778,0.0226865917],[1,295832,0.4126244666],[1,295866,0.0229079192],[1,295878,0.0409323409],[1,296259,0.0514920757],[1,296291,0.2145697801],[1,296334,0.0135514013],[1,296382,0.0771162798],[1,296420,0.0543985029],[1,296422,0.0150837591],[1,296462,0.0135915367],[1,296470,0.0124211136],[1,296483,0.0136185318],[1,296503,0.036591001],[1,296647,0.0150837591],[1,296655,0.0324644022],[1,296679,0.2257665534],[1,296735,0.0528991153],[1,296762,0.1439866621],[1,296824,0.1884288236],[1,296877,0.0176998881],[1,296901,0.2299700791],[1,296913,0.2289952518],[1,296988,0.0563953533],[1,296997,0.0109660163],[1,297008,0.0314065374],[1,297044,0.0160826348],[1,297151,0.0345993765],[1,297160,0.0150837591],[1,297165,0.0400918387],[1,297192,0.0258590291],[1,297247,0.2859396492],[1,297269,0.1316967055],[1,297351,0.0560681144],[1,297424,0.0528991153],[1,297431,0.0262386382],[1,297474,0.035433069],[1,297510,0.095856532],[1,297552,0.1299118581],[1,297572,0.0239853698],[1,297671,0.0955098019],[1,297674,0.0389487273],[1,297690,0.2327069048],[1,297734,0.0123393591],[1,297814,0.0333613908],[1,297978,0.0304015883],[1,298000,0.0283046877],[1,298033,0.047625117],[1,298037,0.0177021717],[1,298045,0.0400009247],[1,298046,0.020300596],[1,298053,0.1133366911],[1,298154,0.0355731557],[1,298155,0.1102197159],[1,298364,0.0701104176],[1,298367,0.1996418249],[1,298377,0.0195825969],[1,298458,0.015254105],[1,298464,0.0126922093],[1,298523,0.015473099],[1,298590,0.0158410375],[1,298684,0.0528991153],[1,298732,0.0155362448],[1,298953,0.0214218907],[1,299228,0.0118649952],[1,299233,0.3326418427],[1,299335,0.1108268401],[1,299375,0.0135641471],[1,299543,0.1343070709],[1,299550,0.0261405009],[1,299606,0.2320775513],[1,299648,0.0493062524],[1,299704,0.0190106957],[1,299710,0.0223764963],[1,299812,0.0150338583],[1,299925,0.016272188],[1,300067,0.06508931],[1,300082,0.0149529885],[1,300148,0.020952028],[1,300192,0.0139074112],[1,300278,0.014689366],[1,300283,0.0116858298],[1,300506,0.0176998881],[1,300573,0.0139074112],[1,300578,0.0423904348],[1,300644,0.05605887],[1,300698,0.30999799],[1,300801,0.0620212716],[1,300826,0.3088859699],[1,300921,0.1109604509],[1,300971,0.0184174833],[1,300975,0.0239119987],[1,301054,0.0140923025],[1,301060,0.3153430823],[1,301103,0.0892624008],[1,301213,0.2006344796],[1,301237,0.0182319806],[1,301293,0.2131645749],[1,301319,0.0151138822],[1,301327,0.2228957159],[1,301354,0.0128005423],[1,301416,0.4451019844],[1,301433,0.0347933102],[1,301453,0.0732730197],[1,301506,0.0416856814],[1,301569,0.1972529544],[1,301575,0.0287313002],[1,301597,0.0463815605],[1,301642,0.0123393591],[1,301649,0.1125005633],[1,301658,0.0373852672],[1,301685,0.032617073],[1,301829,0.0544991214],[1,301869,0.0118649952],[1,301923,0.1338098476],[1,302000,0.0109433037],[1,302594,0.0469867599],[1,302624,0.0711891846],[1,302678,0.1242825867],[1,302688,0.0118649952],[1,302703,0.0150837591],[1,302782,0.8982816373],[1,302811,0.1710880779],[1,302862,0.135437052],[1,303269,0.0150837591],[1,303293,0.0449316263],[1,303300,0.0548713692],[1,303333,0.0747402152],[1,303418,0.0785440514],[1,303568,0.0396878397],[1,303974,0.057615816],[1,303984,0.0189075926],[1,304006,0.1059410322],[1,304014,0.0129236262],[1,304143,0.0610300838],[1,304150,0.0148899599],[1,304225,0.044987043],[1,304338,0.0236749502],[1,304454,0.0166402757],[1,304564,0.0558184772],[1,304575,0.1875532186],[1,304576,0.0949009301],[1,304631,0.0930222301],[1,304731,0.0126336307],[1,304738,0.0271449472],[1,304757,0.0409558784],[1,304842,0.0533404549],[1,304909,0.1305283806],[1,304917,0.1079476298],[1,304944,0.3288072785],[1,305007,0.0188617754],[1,305114,0.0439922359],[1,305126,0.0171519724],[1,305295,0.0209297747],[1,305300,0.0175468259],[1,305324,0.035230587],[1,305379,0.0815383885],[1,305381,0.0630295939],[1,305406,0.0438544032],[1,305410,0.0183254645],[1,305440,0.18205552],[1,305549,0.0554950763],[1,305593,0.0531828469],[1,305694,0.0430070841],[1,305754,0.0197821681],[1,305903,0.0696634002],[1,306000,0.0377610906],[1,306088,0.104390735],[1,306288,0.0187333222],[1,306292,0.056570272],[1,306295,0.161722118],[1,306383,0.5583347956],[1,306405,0.0242203997],[1,306428,0.0766190273],[1,306430,0.189063006],[1,306633,0.0213275095],[1,306664,0.2054490792],[1,306680,0.1077480452],[1,306736,0.0636800194],[1,306760,0.1680951884],[1,306764,0.2025406778],[1,306778,0.1688774783],[1,306800,0.0193926313],[1,306801,0.1862782843],[1,306977,0.1083491896],[1,307075,0.015728365],[1,307078,0.1141564608],[1,307158,0.0319845378],[1,307313,0.0181050496],[1,307384,0.3829802791],[1,307430,0.1462793873],[1,307543,0.1485028841],[1,307626,0.0883447804],[1,308100,0.0841353214],[1,308106,0.0589265805],[1,308158,0.0615352794],[1,308309,0.0286812981],[1,308468,0.7563717061],[1,308528,0.0352123913],[1,308547,0.0381939691],[1,308640,0.0169083681],[1,308650,0.1395428825],[1,308653,0.0217201481],[1,308655,0.0854445102],[1,308661,0.4720269971],[1,308663,0.0244389853],[1,308673,0.1289139489],[1,308711,0.0262016106],[1,308768,0.0411784779],[1,308796,0.0197821681],[1,308797,0.3540103014],[1,308803,0.0336086124],[1,308811,0.2476800121],[1,308834,0.0183882136],[1,308876,0.4497426808],[1,308913,0.0674217961],[1,308925,0.0410943098],[1,308939,0.057079643],[1,308961,0.0160392892],[1,308967,0.6970467353],[1,308986,0.0815270149],[1,309125,0.0191063139],[1,309141,0.015412568],[1,309142,0.0493667256],[1,309173,0.0606115322],[1,309174,0.0224917694],[1,309186,0.345162454],[1,309204,0.0538384587],[1,309217,0.0568577671],[1,309226,0.1574390246],[1,309246,0.2067330491],[1,309258,0.0154804304],[1,309262,0.344308686],[1,309293,0.0267585198],[1,309324,0.0527681374],[1,309343,0.1902515494],[1,309361,0.0180571058],[1,309404,0.2946532883],[1,309544,0.1822295982],[1,309581,0.0187150555],[1,309588,0.0330556522],[1,309646,0.0179315838],[1,309674,0.0149476625],[1,309734,0.2131637927],[1,309777,0.0552931515],[1,309856,0.0159571318],[1,309892,0.0602705421],[1,309903,0.0357971494],[1,309940,0.0172110143],[1,310030,0.0514630928],[1,310052,0.0187150555],[1,310143,0.0422688478],[1,310145,0.1215591275],[1,310245,0.0161018033],[1,310264,0.098180902],[1,310307,0.1007698938],[1,310333,0.0187150555],[1,310335,0.0186233214],[1,310480,0.0255892451],[1,310490,0.1177129929],[1,310543,0.0522545682],[1,310560,0.0854967364],[1,310567,0.2145985794],[1,310619,0.3937554861],[1,310638,0.0425742805],[1,310725,0.1264167887],[1,310739,0.0731712352],[1,310766,0.0291914908],[1,310814,0.0642240984],[1,310841,0.0674313384],[1,310891,0.0182578385],[1,310936,0.0143114491],[1,311027,0.0884202695],[1,311076,0.0423584312],[1,311108,0.0336586842],[1,311177,0.13500877],[1,311178,0.0242843759],[1,311200,0.0261638133],[1,311256,0.0538928295],[1,311261,0.0245836155],[1,311341,0.1454174803],[1,311567,0.0143114491],[1,311591,0.1258069734],[1,311592,0.0742578536],[1,311684,0.0427684868],[1,311689,0.0159571318],[1,311761,0.0243007854],[1,311825,0.5597582251],[1,311854,0.0177187133],[1,311894,0.0192054966],[1,311910,0.0999580916],[1,311931,0.1973470704],[1,312005,0.0176966815],[1,312020,0.0143114491],[1,312254,0.3693748129],[1,312333,0.2486735389],[1,312616,0.0933085043],[1,312697,0.109911015],[1,312706,0.0186355626],[1,312766,0.544604868],[1,312943,0.0187150555],[1,313027,0.0197319623],[1,313082,0.0423983981],[1,313199,0.0607206337],[1,313213,0.300543592],[1,313218,0.0349723385],[1,313280,0.0367041654],[1,313590,0.0155680719],[1,313664,0.3373076361],[1,313816,0.0885419814],[1,313859,0.0594320129],[1,313869,0.3022110874],[1,313946,0.0314212801],[1,314025,0.0944239454],[1,314033,0.0183119192],[1,314073,0.224983606],[1,314242,0.0180454548],[1,314335,0.0369798218],[1,314378,0.0291618191],[1,314453,0.1021751996],[1,314483,0.0162094354],[1,314552,0.1457550919],[1,314554,0.0477869026],[1,314814,0.0388197624],[1,314901,0.0136891928],[1,315014,0.1219528014],[1,315056,0.0734211798],[1,315086,0.241860554],[1,315261,0.1333565613],[1,315358,0.0409353694],[1,315431,0.017530129],[1,315478,0.0166098294],[1,315603,0.0232956722],[1,315613,0.122205374],[1,315695,0.0765570831],[1,315734,0.0167393365],[1,315786,0.0169999756],[1,315840,0.0133843998],[1,315867,0.0794126621],[1,315948,0.0218402606],[1,315971,0.0221269774],[1,316009,0.0391481069],[1,316058,0.0133843998],[1,316061,0.0417806234],[1,316153,0.0248731025],[1,316169,0.0859788895],[1,316186,0.408775262],[1,316348,0.0405907964],[1,316392,0.0150742983],[1,316398,0.0583681932],[1,316438,0.0551935898],[1,316510,0.027504959],[1,316588,0.0150068799],[1,316731,0.0192336645],[1,316762,0.0181381559],[1,317047,0.0397372832],[1,317082,0.0270141303],[1,317095,0.071207584],[1,317193,0.0303902022],[1,317202,0.1652080159],[1,317229,0.0265160738],[1,317267,0.037097674],[1,317305,0.0427149675],[1,317364,0.0558743586],[1,317385,0.064195629],[1,317415,0.0275752821],[1,317416,0.8410520582],[1,317445,0.3985751709],[1,317467,0.3775517798],[1,317507,0.0857937261],[1,317531,0.0136891928],[1,317534,0.0179709228],[1,317746,0.0256173856],[1,317810,0.0730038233],[1,318183,0.017109614],[1,318206,0.0410515335],[1,318370,0.3500394676],[1,318476,0.0166818132],[1,318628,0.4717951053],[1,318660,0.2469432716],[1,318669,0.1395261086],[1,318689,0.0170023245],[1,318737,0.4483530675],[1,318744,0.0171173205],[1,318747,0.2330318621],[1,318770,0.0993027728],[1,318799,0.0507917163],[1,318834,0.1771204753],[1,318895,0.2157881199],[1,319059,0.0214755271],[1,319061,0.1110250432],[1,319086,0.0249065144],[1,319104,0.017109614],[1,319122,0.0273441332],[1,319240,0.0169368725],[1,319316,0.2647714404],[1,319380,0.0422043352],[1,319520,0.5005947795],[1,319530,0.0548003731],[1,319565,0.3587820553],[1,319597,0.0324070884],[1,319639,0.0222753405],[1,319691,0.6656823329],[1,319725,0.0230870327],[1,319870,0.7890220857],[1,319918,0.0143576693],[1,320051,0.1515046808],[1,320062,0.0340932657],[1,320136,0.0563339232],[1,320141,0.06925477],[1,320253,0.0718370667],[1,320260,0.0616281805],[1,320264,0.071965464],[1,320277,0.0176833689],[1,320326,0.0654107645],[1,320409,0.1835845824],[1,320413,0.0164079452],[1,320638,0.0297588501],[1,320667,0.3230832977],[1,320767,0.0570050253],[1,320798,0.0137219653],[1,320832,0.0252827765],[1,320839,0.0259408116],[1,320892,0.0891299048],[1,320938,0.3956985374],[1,320955,0.0243801302],[1,321035,0.050516138],[1,321093,0.0148900106],[1,321139,0.0428632644],[1,321151,0.0608764023],[1,321222,0.1124313884],[1,321245,0.0148900106],[1,321339,0.017109614],[1,321400,0.0170023245],[1,321476,0.076523465],[1,321534,0.0738288208],[1,321617,0.0618531168],[1,321629,0.0327197827],[1,321654,0.0185129906],[1,321665,0.055382946],[1,321732,0.0382062359],[1,321740,0.0563226132],[1,321761,0.561543252],[1,321789,0.0929111609],[1,321829,0.0336232523],[1,321841,0.0258008364],[1,321939,0.0490146397],[1,321963,0.0904149279],[1,321980,0.0185129906],[1,322087,0.0176651946],[1,322134,0.0143576693],[1,322158,0.359748904],[1,322248,0.1034588561],[1,322295,0.0422997906],[1,322335,0.017664489],[1,322366,0.5977873409],[1,322369,0.0570579485],[1,322400,0.0173609937],[1,322689,0.0963585829],[1,322709,0.0798173639],[1,322725,0.037647096],[1,322769,0.0277186183],[1,322818,0.0157683978],[1,322895,0.0195920457],[1,322926,0.0533453351],[1,322928,0.0207512538],[1,323092,0.1047824958],[1,323187,0.0223074906],[1,323211,0.0399609971],[1,323234,0.0237710548],[1,323295,0.0743030817],[1,323322,0.41084427],[1,323427,0.059120908],[1,323689,0.0176651946],[1,323815,0.04016943],[1,323897,0.0378471759],[1,323960,0.0587434294],[1,323990,0.0165833698],[1,323993,0.0539580219],[1,324032,0.0143576693],[1,324058,0.1300569226],[1,324079,0.0424400194],[1,324311,0.0337837093],[1,324351,0.01617683],[1,324367,0.0481311439],[1,324393,0.0528062828],[1,324461,0.0785899069],[1,324478,0.7687580019],[1,324563,0.0719817713],[1,324692,0.0335182231],[1,324695,0.2292602124],[1,324744,0.5060111672],[1,324813,0.017109614],[1,324857,0.031672388],[1,324864,0.065245263],[1,324897,0.03166246],[1,324911,0.1797190049],[1,324943,0.0723881768],[1,325045,0.0195777214],[1,325062,0.1050107154],[1,325068,0.0571919829],[1,325087,0.0581941233],[1,325138,0.0335815673],[1,325142,0.036522454],[1,325177,0.0226804723],[1,325231,0.1523844588],[1,325232,0.0173392012],[1,325234,0.2194119331],[1,325311,0.0413656129],[1,325330,0.0997469293],[1,325335,0.4629743316],[1,325386,0.0416614278],[1,325406,0.1698510131],[1,325453,0.1445608065],[1,325458,0.1426754304],[1,325581,0.0569204844],[1,325582,0.1343080957],[1,325614,0.0901151615],[1,325615,0.0213014451],[1,325641,0.2902588125],[1,325678,0.0185129906],[1,325757,0.0613617984],[1,325794,0.0991595365],[1,325822,0.0440678959],[1,325897,0.0363199861],[1,325913,0.0355822927],[1,325969,0.0148222025],[1,326051,0.3258520964],[1,326250,0.0571919829],[1,326263,0.0183320564],[1,326294,0.0170023245],[1,326485,0.0181522928],[1,326486,0.0570070409],[1,326527,0.0177260263],[1,326568,0.0898989456],[1,326814,0.2332119129],[1,326852,0.0707205743],[1,326906,0.0562062251],[1,326939,0.0190244485],[1,326970,0.0236869061],[1,327031,0.0231484174],[1,327144,0.0387335517],[1,327290,0.1517923659],[1,327295,0.2935099331],[1,327315,0.0177260263],[1,327348,0.0477835563],[1,327363,0.0416047822],[1,327420,0.0275437087],[1,327427,0.01617683],[1,327599,0.0173392012],[1,327643,0.0363681631],[1,327644,0.7602597328],[1,327659,0.0275150358],[1,327712,0.0309815448],[1,327743,0.0264482171],[1,327774,0.0812018642],[1,327833,0.0148222025],[1,327850,0.0330651559],[1,327876,0.0198176878],[1,327904,0.0225352862],[1,328195,0.017109614],[1,328275,0.0193643772],[1,328296,0.3900720668],[1,328388,0.0148900106],[1,328404,0.2453964965],[1,328486,0.0219179675],[1,328564,0.0185129906],[1,328583,0.0646995364],[1,328625,0.1766955201],[1,328639,0.025521669],[1,328769,0.0221832353],[1,328802,0.1067885995],[1,328832,0.0241965466],[1,328902,0.1705423847],[1,328920,0.0165833698],[1,328960,0.1437280422],[1,329060,0.0474788163],[1,329087,0.0324745485],[1,329121,0.1472228466],[1,329152,0.0148900106],[1,329175,0.017876331],[1,329205,0.5411415435],[1,329235,0.0315500022],[1,329265,0.141723086],[1,329344,0.0137219653],[1,329409,0.0538172771],[1,329442,0.0222020787],[1,329607,0.5410680453],[1,329705,0.0272874315],[1,329773,0.2216389498],[1,329832,0.0167283376],[1,329937,0.054340848],[1,329943,0.0508168752],[1,329983,0.0345221698],[1,330124,0.4183290714],[1,330142,0.0390079882],[1,330213,0.6053677827],[1,330214,0.0617095867],[1,330245,0.4954171728],[1,330274,0.0630473029],[1,330297,0.1184092029],[1,330377,0.0206836878],[1,330439,0.0295576001],[1,330508,0.0219120429],[1,330627,0.1119466097],[1,330628,0.0390278182],[1,330630,0.0819374003],[1,330689,0.0212412467],[1,330874,0.6338304795],[1,330937,0.128178685],[1,331003,0.0298954357],[1,331022,0.0762134852],[1,331027,0.0604272618],[1,331035,0.4033206761],[1,331127,0.0226723033],[1,331154,0.0245527671],[1,331190,0.0418200142],[1,331191,0.0220938522],[1,331224,0.0206899564],[1,331684,0.0424394447],[1,331731,0.0179184325],[1,331774,0.0203108357],[1,331787,0.2212070076],[1,331850,0.7160761979],[1,331888,0.0408652204],[1,331901,0.6961474947],[1,331937,0.0179184325],[1,332014,0.0343580601],[1,332106,0.1311932803],[1,332176,0.1893549328],[1,332211,0.5130878373],[1,332278,0.0553756124],[1,332323,0.0414164953],[1,332343,0.0272927929],[1,332351,0.1012535036],[1,332396,0.1614862479],[1,332404,0.026662442],[1,332463,0.1620777631],[1,332610,0.1280554701],[1,332632,0.3763736367],[1,332681,0.1817629008],[1,332704,0.0320902103],[1,332761,0.1468232882],[1,332770,0.0635648695],[1,332833,0.0317649355],[1,332846,0.1046127865],[1,332933,0.0370471314],[1,333193,0.0293294059],[1,333251,0.0202515219],[1,333284,0.0197153064],[1,333344,0.0238444487],[1,333356,0.0219572046],[1,333401,0.1134899841],[1,333422,0.4094493232],[1,333463,0.1240439754],[1,333465,0.0832212347],[1,333566,0.0717918068],[1,333616,0.0388809223],[1,333733,0.0482882413],[1,333781,0.047680926],[1,333797,0.0248190842],[1,333867,0.023088446],[1,333905,0.090918858],[1,333982,0.0417445984],[1,333984,0.1161439129],[1,334027,0.1513271043],[1,334076,0.4529452579],[1,334078,0.1237221116],[1,334126,0.02112036],[1,334128,0.0224901797],[1,334163,0.2325587883],[1,334182,0.0189284179],[1,334195,0.0519469464],[1,334249,0.0480228272],[1,334260,0.0367322882],[1,334313,0.0242726158],[1,334319,0.1135331703],[1,334351,0.0182080242],[1,334354,0.0480739812],[1,334508,0.0179858828],[1,334516,0.6818215826],[1,334545,0.0609524113],[1,334573,0.1230310657],[1,334719,0.1120621789],[1,334721,0.1457070359],[1,334728,0.0152229878],[1,334749,0.0179781673],[1,334787,0.0538009318],[1,334807,0.0681976697],[1,334837,0.0544176366],[1,334860,0.1065751574],[1,334896,0.0156880671],[1,334983,0.0547300786],[1,335000,0.449825005],[1,335005,0.0179781673],[1,335009,0.1039252127],[1,335092,0.0634898143],[1,335108,0.1086697492],[1,335130,0.0181500316],[1,335138,0.0430193437],[1,335139,0.0483925995],[1,335163,0.0286860044],[1,335201,0.0274888637],[1,335253,0.0153907321],[1,335266,0.0177136914],[1,335274,0.2224954194],[1,335337,0.1472338099],[1,335355,0.4716224435],[1,335411,0.0254488467],[1,335465,0.0448658883],[1,335489,0.3441017361],[1,335884,0.0157559549],[1,335885,0.0302263429],[1,335978,0.1059908822],[1,336076,0.0227390666],[1,336139,0.0134447428],[1,336179,0.0134857507],[1,336303,0.0166548743],[1,336370,0.4461162122],[1,336371,0.0585633692],[1,336411,0.01457447],[1,336434,0.0151274414],[1,336489,0.0134857507],[1,336584,0.1016231023],[1,336667,0.1154812146],[1,337253,0.0144051318],[1,337337,0.0208695649],[1,337361,0.0123624816],[1,337448,0.0383548327],[1,337456,0.2722462036],[1,337493,0.0156968744],[1,337697,0.1237531187],[1,337709,0.0291411531],[1,337736,0.0134447428],[1,337785,0.0459499988],[1,337799,0.0233753695],[1,337834,0.2522065852],[1,337842,0.013677789],[1,337999,0.0818680531],[1,338008,0.0128327732],[1,338096,0.0648617072],[1,338131,0.0164271922],[1,338164,0.0163200018],[1,338177,0.0296082807],[1,338223,0.0687938712],[1,338287,0.0312394033],[1,338332,0.3566211664],[1,338342,0.1206014283],[1,338343,0.0361515507],[1,338344,0.0206832334],[1,338366,0.3333370205],[1,338373,0.0720020625],[1,338389,0.0153357338],[1,338402,0.0254165742],[1,338433,0.1014457196],[1,338436,0.0544845954],[1,338501,0.0854659486],[1,338517,0.1711513626],[1,338558,0.0521300601],[1,338649,0.0866420875],[1,338671,0.0255909873],[1,338774,0.0162585815],[1,338810,0.0152770706],[1,338829,0.0178780102],[1,338849,0.1242994634],[1,338897,0.2380037921],[1,339011,0.2166194084],[1,339023,0.0113947599],[1,339090,0.0163200018],[1,339095,0.0721517106],[1,339327,0.0348512208],[1,339443,0.0170430353],[1,339484,0.1160933631],[1,339500,0.0324341114],[1,339588,0.013677789],[1,339633,0.0179106824],[1,339660,0.0937737516],[1,339709,0.2743932074],[1,339726,0.0903006308],[1,339750,0.013677789],[1,339799,0.4262754946],[1,339838,0.0482038917],[1,339947,0.0177883588],[1,340005,0.1342647264],[1,340031,0.1478485356],[1,340049,0.0428407639],[1,340069,0.0466628303],[1,340070,0.0599300691],[1,340083,0.4525181054],[1,340111,0.0231565474],[1,340129,0.0767453026],[1,340141,0.0587484835],[1,340170,0.0150872147],[1,340195,0.03076028],[1,340201,0.0450958103],[1,340232,0.0765555881],[1,340257,0.0467828347],[1,340278,0.1055867942],[1,340282,0.0124051866],[1,340289,0.3758497708],[1,340305,0.1383527446],[1,340415,0.0597782354],[1,340419,0.0364833762],[1,340463,0.2319574968],[1,340478,0.2622400629],[1,340518,0.0833003222],[1,340526,0.0164271922],[1,340531,0.0174689081],[1,340603,0.1654979439],[1,340633,0.2638403423],[1,340687,0.021514234],[1,340696,0.0365498639],[1,340722,0.086809929],[1,340807,0.0252508363],[1,340868,0.0164582688],[1,340926,0.2572546958],[1,341141,0.013677789],[1,341147,0.1567621729],[1,341251,0.0168525342],[1,341285,0.1414758567],[1,341292,0.0849046001],[1,341329,0.0617839637],[1,341333,0.0164271922],[1,341385,0.0242408594],[1,341411,0.192378622],[1,341462,0.014991397],[1,341472,0.1169256284],[1,341579,0.4469915468],[1,341586,0.0327029871],[1,341603,0.0192158341],[1,341789,0.1498564362],[1,341816,0.0541219184],[1,341833,0.020197103],[1,341889,0.0170430353],[1,341954,0.0739199072],[1,341972,0.0589146226],[1,342006,0.0886614077],[1,342077,0.3015425697],[1,342118,0.132512291],[1,342139,0.0259179498],[1,342194,0.0879240895],[1,342276,0.0214265391],[1,342321,0.0395151144],[1,342322,0.0561432756],[1,342336,0.0164271922],[1,342357,0.0152877371],[1,342606,0.2599235847],[1,342719,0.0639825673],[1,342754,0.0252774853],[1,342770,0.1448900684],[1,342772,0.0588270353],[1,342864,0.164820073],[1,342938,0.027217222],[1,342949,0.0194813039],[1,342993,0.0167205265],[1,343063,0.0183600977],[1,343090,0.016854318],[1,343117,0.0194813039],[1,343212,0.0208481015],[1,343282,0.1197855619],[1,343315,0.0371806498],[1,343334,0.0447076911],[1,343383,0.0224271632],[1,343408,0.0167205265],[1,343451,0.0711136273],[1,343701,0.0557470925],[1,343712,0.0449819763],[1,343740,0.4303719308],[1,343933,0.0874221818],[1,344023,0.0174205492],[1,344068,0.1385827042],[1,344164,0.4146755505],[1,344197,0.0446464282],[1,344210,0.0695957589],[1,344224,0.0461091923],[1,344242,0.016854318],[1,344263,0.0709418918],[1,344349,0.0160827822],[1,344491,0.0158720149],[1,344560,0.0189533706],[1,344636,0.0363010043],[1,344670,0.0222814822],[1,344746,0.0181357829],[1,344780,0.0532220977],[1,344816,0.050721823],[1,344817,0.0194813039],[1,344849,0.0582504671],[1,344858,0.0731635185],[1,344985,0.0194813039],[1,345012,0.0183853301],[1,345050,0.0280839155],[1,345075,0.0848779587],[1,345079,0.0727114494],[1,345157,0.0291623095],[1,345218,0.1397818305],[1,345231,0.0392558596],[1,345241,0.0194813039],[1,345313,0.0452728192],[1,345323,0.062029055],[1,345469,0.0324648805],[1,345492,0.1433269643],[1,345536,0.0245013328],[1,345590,0.0255540046],[1,345620,0.0485675041],[1,345708,0.0564924543],[1,345711,0.0295747974],[1,345729,0.0215755704],[1,345804,0.018344049],[1,345811,0.0237848459],[1,345885,0.0388883212],[1,345890,0.0203648658],[1,345906,0.0264729981],[1,346026,0.1145028293],[1,346113,0.0657277541],[1,346151,0.0207248231],[1,346158,0.0977838796],[1,346180,0.026277969],[1,346203,0.0288754696],[1,346280,0.0181357829],[1,346627,0.0509063113],[1,346696,0.0225017876],[1,346795,0.0551865189],[1,346805,0.3174066301],[1,346946,0.1295454834],[1,347043,0.3829022302],[1,347059,0.0511133193],[1,347076,0.0840635292],[1,347142,0.0167205265],[1,347172,0.0194813039],[1,347195,0.0164896709],[1,347215,0.0212175992],[1,347268,0.0194982452],[1,347329,0.0617439535],[1,347515,0.0197116279],[1,347618,0.0383565522],[1,347667,0.1342372139],[1,347738,0.7185305699],[1,347755,0.156253151],[1,347770,0.0621358235],[1,347831,0.0181357829],[1,347839,0.0678766774],[1,347901,0.0593056816],[1,347970,0.1826407144],[1,348014,0.0303193613],[1,348020,0.1991499753],[1,348045,0.0153862481],[1,348170,0.2635425934],[1,348196,0.1801291639],[1,348229,0.0552930957],[1,348250,0.0194813039],[1,348276,0.032054291],[1,348316,0.01937367],[1,348354,0.1126563763],[1,348402,0.024371474],[1,348426,0.0208891847],[1,348427,0.0183600977],[1,348463,0.0167205265],[1,348497,0.0167205265],[1,348549,0.0181357829],[1,348609,0.3638026432],[1,348656,0.1166254331],[1,348793,0.06267994],[1,348795,0.0216127912],[1,348832,0.0220095741],[1,348851,0.0260527625],[1,348916,0.0290287462],[1,348993,0.2998429695],[1,349004,0.287796182],[1,349022,0.0928534231],[1,349100,0.0618044459],[1,349261,0.161067697],[1,349388,0.1166037416],[1,349457,0.01694217],[1,349473,0.0752403485],[1,349535,0.022418708],[1,349542,0.1895054028],[1,349605,0.0658882264],[1,349667,0.1229047113],[1,349775,0.1814640455],[1,349812,0.0594716628],[1,349975,0.0618044459],[1,349979,0.3079397591],[1,349994,0.01694217],[1,350014,0.4530807902],[1,350028,0.016874173],[1,350048,0.0662992897],[1,350057,0.0273879717],[1,350211,0.0449485695],[1,350213,0.017823004],[1,350231,0.0283399487],[1,350241,0.0205341761],[1,350249,0.0321366557],[1,350264,0.0660210152],[1,350265,0.1409046256],[1,350376,0.0433694737],[1,350404,0.0626183115],[1,350407,0.0385449514],[1,350433,0.0656294813],[1,350447,0.0652152147],[1,350457,0.0190603678],[1,350507,0.0191679562],[1,350557,0.1236320494],[1,350583,0.0674118227],[1,350599,0.0540933829],[1,350613,0.0439369441],[1,350652,0.0301988365],[1,350723,0.0425470272],[1,350808,0.3485388166],[1,350919,0.0181890284],[1,350922,0.021862382],[1,351153,0.0193203086],[1,351175,0.0664475998],[1,351202,0.0513496348],[1,351236,0.020858757],[1,351259,0.0693995661],[1,351273,0.1062630598],[1,351277,0.0447221218],[1,351306,0.1501428011],[1,351489,0.0428771528],[1,351505,0.0798507737],[1,351553,0.0383075229],[1,351566,0.0421683253],[1,351583,0.2721200058],[1,351584,0.4247572883],[1,351589,0.0622069913],[1,351594,0.1457563093],[1,351595,0.0209552952],[1,351640,0.0322367659],[1,351679,0.0218123754],[1,351730,0.0695696073],[1,351744,0.1525650563],[1,351809,0.0202356922],[1,351823,0.1888818883],[1,351899,0.0291302002],[1,351950,0.0372832633],[1,351954,0.0166623948],[1,351993,0.0184658138],[1,352012,0.0670933507],[1,352319,0.0782493395],[1,352337,0.2713959311],[1,352462,0.0254831258],[1,352561,0.0219147887],[1,352569,0.0732048649],[1,352581,0.0779131223],[1,352590,0.0561771759],[1,352606,0.5700762124],[1,352688,0.0173601346],[1,352689,0.4072311743],[1,352709,0.3092171424],[1,352744,0.0245133857],[1,352757,0.0304234315],[1,352776,0.0764808691],[1,352831,0.0659250309],[1,352885,0.4534724195],[1,352953,0.0338986741],[1,352976,0.0653253532],[1,353010,0.0214102539],[1,353031,0.0221382734],[1,353083,0.0640371107],[1,353138,0.0478411307],[1,353345,0.0209552952],[1,353449,0.0298324535],[1,353777,0.0966956579],[1,353835,0.3193289522],[1,353860,0.0237231223],[1,353900,0.0389992484],[1,354021,0.0805629028],[1,354026,0.0291718103],[1,354053,0.0938800488],[1,354144,0.02384442],[1,354181,0.0265763843],[1,354454,0.0271675064],[1,354500,0.0458818791],[1,354542,0.0195142446],[1,354633,0.0384903895],[1,354647,0.0853739204],[1,354677,0.0514855962],[1,354694,0.4467902821],[1,354698,0.029986516],[1,354795,0.0407691096],[1,354825,0.0195142446],[1,354850,0.0315436014],[1,354904,0.5056150061],[1,354943,0.0679690631],[1,355131,0.0434319078],[1,355145,0.0628111285],[1,355224,0.0313599813],[1,355300,0.0211251735],[1,355304,0.0181274561],[1,355334,0.2837272155],[1,355526,0.0612237964],[1,355530,0.335836234],[1,355576,0.0183388677],[1,355619,0.0195142446],[1,355660,0.0601380889],[1,355708,0.0206231459],[1,355732,0.071048767],[1,355762,0.019168859],[1,355772,0.6635748597],[1,355809,0.0423986178],[1,355813,0.0665368157],[1,355962,0.1664906545],[1,355969,0.0674913281],[1,356038,0.0227970036],[1,356084,0.2316403369],[1,356116,0.0672134141],[1,356191,0.0219387886],[1,356195,0.0915013563],[1,356204,0.024958259],[1,356250,0.0231599654],[1,356253,0.1939395169],[1,356257,0.0219788083],[1,356388,0.3429054965],[1,356395,0.0211514177],[1,356409,0.0219387886],[1,356436,0.0214091051],[1,356455,0.0195569011],[1,356463,0.0515034574],[1,356485,0.3078305872],[1,356504,0.6144574863],[1,356516,0.0368586403],[1,356643,0.1138707159],[1,356644,0.0344357451],[1,356695,0.0223680615],[1,356719,0.0194460109],[1,356755,0.0932481322],[1,356775,0.2290571791],[1,356795,0.0219387886],[1,356802,0.1695901489],[1,356817,0.1309569996],[1,356846,0.0888282181],[1,356881,0.044431207],[1,356910,0.0506683807],[1,356912,0.0316936959],[1,357022,0.035492376],[1,357026,0.0238588718],[1,357070,0.0877349754],[1,357073,0.1659973529],[1,357081,0.1132411857],[1,357099,0.0471879803],[1,357208,0.0292139404],[1,357234,0.1297134065],[1,357236,0.1391368915],[1,357245,0.066120642],[1,357257,0.0359613815],[1,357397,0.1694717418],[1,357403,0.0307846663],[1,357432,0.0431860991],[1,357483,0.4453757109],[1,357615,0.0552979202],[1,357635,0.0777702577],[1,357661,0.0286918756],[1,357679,0.0493187152],[1,357735,0.1961108448],[1,357737,0.0267790323],[1,357754,0.0241512597],[1,357804,0.067439371],[1,357807,0.0306298993],[1,357941,0.1435399673],[1,358059,0.0729245869],[1,358129,0.0256153252],[1,358207,0.0787259394],[1,358249,0.0936796002],[1,358281,0.0238588718],[1,358287,0.0846014449],[1,358307,0.024541911],[1,358324,0.1286961112],[1,358381,0.0267553283],[1,358392,0.2124426975],[1,358499,0.4533058117],[1,358552,0.0343207534],[1,358771,0.0669700341],[1,358812,0.0476550951],[1,358813,0.0519431317],[1,358817,0.0269398141],[1,358876,0.2363346715],[1,358889,0.1938797045],[1,359002,0.0422064367],[1,359050,0.0595566642],[1,359133,0.1529975304],[1,359166,0.0807633813],[1,359232,0.0269398141],[1,359242,0.0591865586],[1,359305,0.1166968981],[1,359314,0.028020879],[1,359335,0.0305977797],[1,359344,0.1275352286],[1,359360,0.0252282985],[1,359426,0.0250364452],[1,359467,0.2779253402],[1,359593,0.0537113644],[1,359623,0.6927810532],[1,359633,0.1418149498],[1,359641,0.0673228787],[1,359650,0.0327816244],[1,359659,0.0337509648],[1,359788,0.0264058653],[1,359799,0.1728884138],[1,359831,0.2624081558],[1,359934,0.0548357876],[1,359954,0.1898914273],[1,359960,0.0405935284],[1,359970,0.0761476257],[1,360094,0.3374461922],[1,360143,0.2057686227],[1,360152,0.4727089722],[1,360171,0.0369857411],[1,360181,0.0740333234],[1,360284,0.026914563],[1,360293,0.048017062],[1,360343,0.0277541707],[1,360377,0.0312447725],[1,360417,0.0281035581],[1,360424,0.0317915189],[1,360431,0.0489110935],[1,360474,0.0428880485],[1,360480,0.0267007019],[1,360483,0.0388369615],[1,360494,0.0589326424],[1,360519,0.0394727113],[1,360520,0.0729674302],[1,360521,0.0301258784],[1,360605,0.0269576453],[1,360747,0.7719365184],[1,360751,0.0424185757],[1,360759,0.0422281044],[1,360779,0.0574422682],[1,360834,0.0305212929],[1,360892,0.1149570472],[1,361007,0.0868409697],[1,361008,0.1824427332],[1,361063,0.0303708164],[1,361080,0.0869134764],[1,361223,0.0449153811],[1,361266,0.0301258784],[1,361286,0.0301258784],[1,361339,0.0327354244],[1,361372,0.0917699905],[1,361389,0.152353938],[1,361413,0.0596845461],[1,361418,0.341887308],[1,361433,0.0589973574],[1,361440,0.0280345338],[1,361461,0.2948112087],[1,361464,0.1328843319],[1,361549,0.0337228724],[1,361559,0.0591692907],[1,361645,0.0697556559],[1,361662,0.497162331],[1,361668,0.0763553796],[1,361679,0.026914563],[1,361681,0.0632078222],[1,361894,0.1606744583],[1,361921,0.0334240859],[1,361925,0.0815644752],[1,361970,0.0690755002],[1,362024,0.0307378485],[1,362094,0.0343702019],[1,362114,0.0406231647],[1,362244,0.0742790635],[1,362294,0.1722023061],[1,362325,0.1043180107],[1,362329,0.1102144807],[1,362377,0.0783155287],[1,362424,0.0313448939],[1,362520,0.0305824671],[1,362619,0.0573320861],[1,362621,0.0277541707],[1,362752,0.6199772913],[1,362770,0.0331108493],[1,362776,0.3390760699],[1,362803,0.1650665074],[1,362812,0.0519440666],[1,362829,0.0277541707],[1,362856,0.0328413031],[1,362931,0.0326935836],[1,362947,0.070555361],[1,363059,0.0636824842],[1,363093,0.0355602448],[1,363135,0.139362634],[1,363293,0.0343702019],[1,363309,0.1480598594],[1,363383,0.0516432915],[1,363502,0.1102104946],[1,363612,0.1659097117],[1,363654,0.2767867786],[1,363674,0.0754646239],[1,363717,0.0375101169],[1,363801,0.0319851074],[1,364035,0.0673891601],[1,364075,0.4379931593],[1,364145,0.0317378251],[1,364186,0.1178555033],[1,364197,0.0311579602],[1,364202,0.1142715466],[1,364205,0.0274890772],[1,364207,0.0918642266],[1,364303,0.1723188461],[1,364401,0.4366938721],[1,364422,0.030677248],[1,364453,0.0528023845],[1,364563,0.0281927621],[1,364595,0.0514382046],[1,364615,0.0667223197],[1,364628,0.0307242017],[1,364632,0.3599961483],[1,364708,0.0259432578],[1,364748,0.0659919861],[1,364798,0.1066075736],[1,364819,0.0298599461],[1,364862,0.0773806082],[1,364868,0.5300745719],[1,364938,0.0285435621],[1,364980,0.0337722025],[1,365029,0.0329927965],[1,365277,0.0285435621],[1,365282,0.0504048514],[1,365345,0.0706338356],[1,365388,0.025186299],[1,365401,0.0875093368],[1,365433,0.0274890772],[1,365439,0.0315249975],[1,365462,0.0493875812],[1,365478,0.0558271839],[1,365485,0.094370592],[1,365527,0.0501149815],[1,365550,0.0298599461],[1,365622,0.0930588281],[1,365623,0.2947607266],[1,365626,0.0768372316],[1,365663,0.0315418297],[1,365710,0.0306249155],[1,365747,0.0266497664],[1,365804,0.2631192545],[1,365965,0.0470251127],[1,365974,0.0305598822],[1,366009,0.0286025418],[1,366174,0.0391310512],[1,366208,0.2534662096],[1,366227,0.0702732901],[1,366253,0.0279450356],[1,366262,0.0718242742],[1,366286,0.2640378312],[1,366288,0.0272966433],[1,366329,0.0315418297],[1,366343,0.0554535922],[1,366403,0.0724397634],[1,366456,0.0540767236],[1,366480,0.1248045073],[1,366484,0.0274513188],[1,366493,0.0301286075],[1,366635,0.0755301489],[1,366686,0.6942674587],[1,366709,0.0315888657],[1,366719,0.2567227123],[1,366790,0.0487979129],[1,366793,0.1445500663],[1,366830,0.0278383409],[1,366852,0.1127900384],[1,366939,0.0831597865],[1,366976,0.0272966433],[1,366991,0.3842758274],[1,367004,0.0315390723],[1,367073,0.2901102446],[1,367077,0.2909221517],[1,367199,0.1004365226],[1,367205,0.0670169901],[1,367292,0.0234557478],[1,367309,0.2649085067],[1,367348,0.0236510948],[1,367349,0.0530339157],[1,367363,0.1743524602],[1,367367,0.1152994617],[1,367373,0.0384845314],[1,367452,0.0226209548],[1,367465,0.0674185781],[1,367476,0.2328863933],[1,367480,0.0249184482],[1,367516,0.0728618848],[1,367519,0.0704829688],[1,367640,0.0365246514],[1,367672,0.0268347218],[1,367683,0.1411000579],[1,367782,0.0710947999],[1,367796,0.0725654621],[1,367829,0.0268347218],[1,367864,0.0668769668],[1,367879,0.0741613152],[1,367883,0.0728618848],[1,367909,0.0775467473],[1,367964,0.0272432932],[1,368209,0.1469524586],[1,368339,0.025218479],[1,368368,0.110908735],[1,368410,0.1220566241],[1,368439,0.0271216795],[1,368443,0.0441277193],[1,368471,0.1123268921],[1,368570,0.0268347218],[1,368637,0.0750133458],[1,368766,0.1660830629],[1,368799,0.0275411545],[1,368870,0.0452495174],[1,368938,0.0313930549],[1,368976,0.0629441289],[1,368993,0.025218479],[1,369002,0.0248212528],[1,369016,0.0245611976],[1,369017,0.2098729631],[1,369058,0.1033916229],[1,369198,0.1286957299],[1,369206,0.0306511289],[1,369222,0.0324022677],[1,369254,0.0273062698],[1,369263,0.1122819877],[1,369269,0.029784287],[1,369300,0.1697843335],[1,369336,0.1222614193],[1,369420,0.1198104351],[1,369425,0.0528182672],[1,369442,0.0518111057],[1,369456,0.0294649115],[1,369472,0.0556077535],[1,369502,0.1308341479],[1,369697,0.0681409883],[1,369699,0.2204316039],[1,369884,0.0255963393],[1,369922,0.0624152391],[1,369931,0.1224305724],[1,370020,0.1240918904],[1,370113,0.151137434],[1,370133,0.1239789859],[1,370159,0.0550567144],[1,370226,0.0258979565],[1,370253,0.0290910628],[1,370303,0.0297688632],[1,370321,0.0432756183],[1,370333,0.0391901379],[1,370357,0.2934606148],[1,370367,0.047763174],[1,370479,0.028838846],[1,370579,0.0578276946],[1,370636,0.029571585],[1,370659,0.0711211761],[1,370756,0.2878854773],[1,370759,0.0309745874],[1,370871,0.0250562617],[1,370873,0.0351670558],[1,370999,0.0289120407],[1,371135,0.0768139596],[1,371165,0.6010764576],[1,371182,0.0286721622],[1,371216,0.1029468506],[1,371230,0.0250562617],[1,371236,0.1977848759],[1,371256,0.0381890172],[1,371274,0.4781722331],[1,371312,0.1340289117],[1,371342,0.0720759248],[1,371371,0.1464406267],[1,371437,0.0957222299],[1,371492,0.0322756386],[1,371662,0.0690029715],[1,371725,0.0634461365],[1,371731,0.0284015001],[1,371737,0.1174304633],[1,371803,0.0400689506],[1,371846,0.1271416154],[1,371851,0.0404964787],[1,371865,0.064112268],[1,371887,0.0733084208],[1,371985,0.0292719703],[1,372088,0.0471387989],[1,372111,0.0272806285],[1,372167,0.1829268619],[1,372184,0.0367793524],[1,372216,0.0298139403],[1,372233,0.0952677474],[1,372238,0.041467305],[1,372292,0.0334565865],[1,372300,0.1610716764],[1,372311,0.0492836825],[1,372324,0.0272806285],[1,372393,0.0367479892],[1,372417,0.032115384],[1,372424,0.0321211654],[1,372532,0.0679860941],[1,372568,0.029246262],[1,372569,0.0783688534],[1,372841,0.0741830944],[1,372843,0.0519974213],[1,372977,0.0372081911],[1,373007,0.0298438764],[1,373050,0.0329924206],[1,373098,0.0800255954],[1,373100,0.0713225333],[1,373131,0.1983042608],[1,373140,0.0330316175],[1,373168,0.0573421313],[1,373195,0.0338074886],[1,373197,0.1114516032],[1,373232,0.0844292161],[1,373258,0.1604898015],[1,373284,0.1786786476],[1,373434,0.0781770642],[1,373443,0.0320129504],[1,373462,0.2662304611],[1,373481,0.1185449348],[1,373523,0.5174830244],[1,373535,0.0748334825],[1,373561,0.0614639741],[1,373577,0.0840027151],[1,373585,0.1888671132],[1,373734,0.0595865088],[1,373744,0.1021469315],[1,373766,0.0405665324],[1,373793,0.0660299253],[1,373803,0.0764872454],[1,373822,0.0296203463],[1,373833,0.0284863083],[1,373955,0.0298438764],[1,373988,0.2622200174],[1,374054,0.0471492495],[1,374060,0.0278223145],[1,374080,0.0545635415],[1,374093,0.0524297731],[1,374125,0.4601122868],[1,374165,0.0353432406],[1,374405,0.0917160215],[1,374453,0.0285553742],[1,374458,0.0473226532],[1,374464,0.1687393811],[1,374471,0.0332797454],[1,374518,0.0752379125],[1,374536,0.0298438764],[1,374538,0.0451955113],[1,374632,0.0315088922],[1,374634,0.0624781948],[1,374667,0.0774758507],[1,374682,0.1823228774],[1,374711,0.1204402062],[1,374754,0.2085135097],[1,374856,0.0367478444],[1,374858,0.0416520654],[1,374863,0.3219040576],[1,374901,0.0832609117],[1,374914,0.0278223145],[1,374974,0.0271106515],[1,375142,0.2506925617],[1,375233,0.1215589666],[1,375241,0.1472784908],[1,375301,0.0919501906],[1,375349,0.0782311754],[1,375372,0.058862396],[1,375386,0.22526323],[1,375407,0.0459106262],[1,375430,0.0273656628],[1,375478,0.0572194889],[1,375494,0.6298564634],[1,375506,0.0826585657],[1,375514,0.1094866378],[1,375532,0.031928852],[1,375545,0.3339349378],[1,375572,0.1396849906],[1,375722,0.0558711538],[1,375723,0.0359340699],[1,375750,0.1854736783],[1,375762,0.0340913682],[1,375799,0.0278223145],[1,375801,0.0367478444],[1,375806,0.3226696776],[1,375811,0.0341502215],[1,375882,0.0285553742],[1,375893,0.0274730582],[1,375899,0.0778243422],[1,375977,0.1450479079],[1,375990,0.1002179188],[1,376002,0.0298438764],[1,376035,0.0274147035],[1,376131,0.0285204079],[1,376139,0.0559363294],[1,376211,0.0550214623],[1,376269,0.109631416],[1,376282,0.0532964721],[1,376301,0.0359559247],[1,376331,0.0298438764],[1,376346,0.0486110998],[1,376348,0.0454763537],[1,376402,0.0876676071],[1,376403,0.0305904342],[1,376468,0.085537563],[1,376508,0.0842517565],[1,376591,0.1256774948],[1,376875,0.0857667038],[1,376886,0.0298438764],[1,376934,0.0923240496],[1,376950,0.138206741],[1,376991,0.0774758507],[1,377589,0.0781006831],[1,377713,0.0274730582],[1,377786,0.4250983403],[1,377791,0.0270427153],[1,377812,0.0298438764],[1,377863,0.0604387939],[1,377914,0.0305904342],[1,377969,0.0457355803],[1,378021,0.0308239985],[1,378030,0.1360730375],[1,378073,0.0791351857],[1,378085,0.3258926895],[1,378147,0.1355083853],[1,378205,0.0317024081],[1,378232,0.0745277383],[1,378236,0.0546179496],[1,378309,0.0806446917],[1,378326,0.2390152806],[1,378356,0.0310499948],[1,378366,0.2171614078],[1,378394,0.1214342897],[1,378429,0.2715519269],[1,378443,0.0783688534],[1,378518,0.0406773739],[1,378617,0.0749198855],[1,378632,0.0777023188],[1,378642,0.0278223145],[1,378687,0.1126124863],[1,378694,0.0322455569],[1,378798,0.033580332],[1,378803,0.0297384104],[1,378817,0.0296878132],[1,378833,0.0527231399],[1,378867,0.0561794517],[1,378912,0.0794761706],[1,379002,0.0319702074],[1,379048,0.5028074632],[1,379147,0.074949756],[1,379181,0.1076550409],[1,379188,0.0295453989],[1,379233,0.0302519666],[1,379313,0.1599478111],[1,379342,0.1758830599],[1,379353,0.0418299403],[1,379391,0.182544961],[1,379481,0.0497386401],[1,379671,0.1229491577],[1,379742,0.0283575875],[1,379767,0.0698215976],[1,379821,0.0562866234],[1,379842,0.0843975746],[1,379893,0.0538043858],[1,379902,0.3182307894],[1,379907,0.0304134312],[1,379912,0.0800584909],[1,379992,0.0376389268],[1,380204,0.0856826523],[1,380300,0.0797090258],[1,380319,0.039318561],[1,380322,0.0315548192],[1,380609,0.0291583333],[1,380772,0.0328814917],[1,380824,0.0958997807],[1,380862,0.0485166281],[1,380883,0.0267322143],[1,380901,0.19563885],[1,380913,0.0807811532],[1,381001,0.0267322143],[1,381065,0.033091524],[1,381131,0.0859903042],[1,381158,0.1015443853],[1,381166,0.0329873988],[1,381340,0.0711507097],[1,381346,0.7730671474],[1,381365,0.0278519123],[1,381378,0.0334216643],[1,381395,0.0407194709],[1,381398,0.0314146435],[1,381420,0.0356447652],[1,381448,0.0660520864],[1,381711,0.0356447652],[1,381734,0.4329300211],[1,381761,0.0338708887],[1,381816,0.4957073629],[1,381818,0.194368223],[1,381891,0.1140176257],[1,381942,0.0756901021],[1,382003,0.3457713335],[1,382034,0.0432879628],[1,382075,0.0293642371],[1,382179,0.0343579852],[1,382237,0.3730421061],[1,382267,0.0531635537],[1,382292,0.1739852232],[1,382309,0.0338969096],[1,382313,0.099255282],[1,382356,0.0612557824],[1,382365,0.2430458314],[1,382391,0.0269949317],[1,382418,0.0659084501],[1,382424,0.0293642371],[1,382439,0.2953369009],[1,382476,0.0273439652],[1,382508,0.0369436527],[1,382551,0.3093067108],[1,382707,0.0444815272],[1,382740,0.0831308325],[1,382741,0.045929335],[1,382756,0.0652986633],[1,382768,0.1000506019],[1,382789,0.0728381738],[1,382814,0.0527570255],[1,383097,0.0347590648],[1,383216,0.0744627578],[1,383246,0.0422732992],[1,383267,0.0323417205],[1,383299,0.0510267218],[1,383312,0.0303715388],[1,383335,0.0803030321],[1,383347,0.2057644173],[1,383408,0.0283259746],[1,383522,0.5143426555],[1,383555,0.0283259746],[1,383596,0.0923484517],[1,383613,0.0458357896],[1,383656,0.0815274072],[1,383687,0.0347590648],[1,383693,0.0380255522],[1,383731,0.0294325184],[1,383736,0.0346331201],[1,383750,0.0292203815],[1,383799,0.1256136927],[1,383814,0.0522616874],[1,383821,0.0806308797],[1,383864,0.0287141248],[1,383885,0.0800962135],[1,383916,0.121105661],[1,383942,0.0322755955],[1,384025,0.0408442856],[1,384052,0.0370523302],[1,384062,0.054967708],[1,384094,0.1249911231],[1,384096,0.1730492539],[1,384116,0.5080245465],[1,384161,0.0290157333],[1,384246,0.0296140829],[1,384385,0.0296140829],[1,384392,0.0361063746],[1,384410,0.0272439895],[1,384504,0.0469969795],[1,384543,0.0789305761],[1,384562,0.1380045844],[1,384591,0.0855302796],[1,384660,0.0891929583],[1,384701,0.0503361909],[1,384742,0.1144546115],[1,384772,0.0325116265],[1,384780,0.083140661],[1,384885,0.0763537306],[1,385070,0.0602181801],[1,385134,0.0296140829],[1,385135,0.0320150292],[1,385142,0.0357700649],[1,385325,0.0695785868],[1,385335,0.0902640783],[1,385356,0.5034601708],[1,385373,0.0314720464],[1,385396,0.0385473195],[1,385397,0.1979337921],[1,385422,0.0423574909],[1,385565,0.0434686925],[1,385614,0.0441762563],[1,385671,0.0818331535],[1,385695,0.2161559579],[1,385739,0.0434178204],[1,385823,0.030284773],[1,385908,0.0663394371],[1,385985,0.0821134679],[1,385998,0.2736466074],[1,386006,0.0300586434],[1,386208,0.0527597845],[1,386233,0.0293126133],[1,386240,0.0288108954],[1,386246,0.0931907416],[1,386398,0.0347590648],[1,386464,0.6599050581],[1,386531,0.026751177],[1,386540,0.0268851573],[1,386552,0.0311636549],[1,386567,0.0747751437],[1,386571,0.0292072218],[1,386614,0.114617406],[1,386630,0.0577426204],[1,386713,0.0885093845],[1,386721,0.2725256731],[1,386793,0.0344077208],[1,386846,0.0302290731],[1,386860,0.0523107709],[1,386870,0.0747639937],[1,386885,0.0579799494],[1,386976,0.4419222792],[1,387008,0.0689365096],[1,387097,0.0644889704],[1,387123,0.034992025],[1,387143,0.083140661],[1,387146,0.0606102763],[1,387211,0.1023300843],[1,387226,0.0268361512],[1,387284,0.0269434707],[1,387297,0.03344636],[1,387370,0.0521327973],[1,387435,0.1507298979],[1,387485,0.0709745902],[1,387616,0.0441441323],[1,387642,0.1220903098],[1,387746,0.0324492251],[1,387754,0.0603046209],[1,387776,0.0408273602],[1,387819,0.0540385845],[1,387868,0.2053236224],[1,387886,0.0274892929],[1,387937,0.2656544614],[1,388008,0.0316265757],[1,388048,0.0498221838],[1,388342,0.2147105795],[1,388351,0.2363022238],[1,388366,0.0674201169],[1,388388,0.0320608543],[1,388485,0.0804412598],[1,388578,0.0804481226],[1,388652,0.0298871314],[1,388728,0.3131059219],[1,388752,0.0817723344],[1,388855,0.0396844026],[1,388886,0.2183150888],[1,388892,0.0455669574],[1,388948,0.1617397521],[1,389040,0.0280311299],[1,389054,0.0300532549],[1,389090,0.148940199],[1,389176,0.1096911682],[1,389204,0.0379442033],[1,389212,0.1017925885],[1,389294,0.0426805058],[1,389339,0.0448263244],[1,389426,0.0276234054],[1,389505,0.0273587057],[1,389546,0.0800620833],[1,389616,0.0511960725],[1,389639,0.0477339699],[1,389687,0.0518896978],[1,389700,0.4692583546],[1,389706,0.0323646024],[1,389754,0.5550199124],[1,389788,0.081777747],[1,389835,0.0851293586],[1,389862,0.0408268448],[1,389883,0.4404656882],[1,389942,0.4155748822],[1,389958,0.060347498],[1,389976,0.0692688998],[1,389980,0.5301605925],[1,390163,0.0376752575],[1,390170,0.0315372377],[1,390279,0.2123709083],[1,390377,0.0760161267],[1,390386,0.087751918],[1,390405,0.0498054022],[1,390415,0.0389855735],[1,390489,0.0544402127],[1,390501,0.0305034157],[1,390507,0.0509009275],[1,390510,0.3786347619],[1,390651,0.0814221224],[1,390665,0.0817723344],[1,390681,0.0611028457],[1,390690,0.0850254492],[1,390708,0.2426192134],[1,390760,0.0287643937],[1,390822,0.029582076],[1,390892,0.0324556043],[1,390946,0.0514180347],[1,390964,0.140431226],[1,390987,0.0262198541],[1,391023,0.0332270087],[1,391030,0.0526165408],[1,391166,0.0822344407],[1,391285,0.0814221224],[1,391292,0.2455336098],[1,391351,0.0828637918],[1,391390,0.2059118756],[1,391399,0.0951860845],[1,391440,0.0300532549],[1,391473,0.0431805352],[1,391503,0.3528094609],[1,391518,0.3606217495],[1,391532,0.055685003],[1,391569,0.0308102208],[1,391600,0.0814221224],[1,391613,0.0309581734],[1,391622,0.0389827985],[1,391650,0.0301915439],[1,391657,0.0837462059],[1,391673,0.0300532549],[1,391757,0.0471082568],[1,391769,0.0596382787],[1,391798,0.0311588416],[1,391818,0.3808611274],[1,391821,0.0642374636],[1,391845,0.0489051353],[1,391855,0.0488663368],[1,391870,0.0437227098],[1,391934,0.0555415109],[1,391955,0.0691053206],[1,391958,0.0706291108],[1,391996,0.125251768],[1,392021,0.1306581025],[1,392128,0.0285100728],[1,392145,0.0314899826],[1,392162,0.1252789143],[1,392264,0.1286394316],[1,392334,0.3545736289],[1,392435,0.0299477595],[1,392444,0.0273587057],[1,392472,0.038289556],[1,392529,0.0287643937],[1,392572,0.0320608543],[1,392631,0.029392456],[1,392647,0.0224635765],[1,392754,0.239896907],[1,392764,0.027754259],[1,392862,0.0777371026],[1,392886,0.0770621262],[1,392924,0.0351624323],[1,392928,0.0791050824],[1,393213,0.0255785627],[1,393357,0.1891336577],[1,393419,0.0777371026],[1,393519,0.0738735956],[1,393545,0.0482663848],[1,393608,0.080973882],[1,393629,0.0363711438],[1,393699,0.1922573164],[1,393716,0.1019788503],[1,393733,0.0279348445],[1,393748,0.0224635765],[1,393749,0.5752830218],[1,393751,0.0987556188],[1,393772,0.0409355434],[1,393828,0.0773886491],[1,393860,0.0249266006],[1,393867,0.0238112718],[1,393871,0.0492061605],[1,393875,0.026277728],[1,393893,0.0471352485],[1,393908,0.1106441814],[1,393953,0.0660262906],[1,393997,0.1039596326],[1,394009,0.087258093],[1,394096,0.0400295646],[1,394103,0.0261727623],[1,394197,0.026277728],[1,394209,0.0237266409],[1,394236,0.0509454286],[1,394237,0.0414644458],[1,394244,0.5558436938],[1,394427,0.0235574703],[1,394454,0.0235967093],[1,394455,0.0526747067],[1,394528,0.0595177885],[1,394642,0.0239181577],[1,394647,0.0773886491],[1,394654,0.2099932919],[1,394683,0.2714209809],[1,394689,0.105207062],[1,394789,0.0261587967],[1,394813,0.1692874884],[1,394827,0.0253116672],[1,394859,0.0797010624],[1,394867,0.0252736882],[1,394895,0.2370319997],[1,394970,0.02851382],[1,395000,0.0237266409],[1,395085,0.0884292691],[1,395199,0.0827150919],[1,395290,0.0696646128],[1,395294,0.138943281],[1,395347,0.0224635765],[1,395432,0.0253817656],[1,395442,0.1636410486],[1,395448,0.0358860926],[1,395510,0.0544590804],[1,395511,0.026277728],[1,395523,0.026277728],[1,395565,0.6148691516],[1,395642,0.0264666176],[1,395740,0.024265757],[1,395755,0.026277728],[1,395833,0.1467322094],[1,395857,0.0239181577],[1,395896,0.0385600354],[1,395902,0.0248876846],[1,395953,0.0279348445],[1,395997,0.0336910762],[1,396024,0.2263411651],[1,396039,0.086595897],[1,396091,0.0543498827],[1,396133,0.0875341772],[1,396215,0.0276978669],[1,396255,0.0750552861],[1,396288,0.0850681134],[1,396418,0.0249953388],[1,396429,0.0773886491],[1,396517,0.1110191324],[1,396587,0.1857323464],[1,396599,0.12742919],[1,396669,0.024265757],[1,396716,0.0685372579],[1,396767,0.0964667955],[1,396799,0.0266838821],[1,396855,0.3864149391],[1,396877,0.0824359558],[1,396897,0.0391860479],[1,396958,0.0829563888],[1,396965,0.0802315311],[1,396980,0.0301360118],[1,396996,0.0339349134],[1,397016,0.2489979738],[1,397022,0.0440530902],[1,397040,0.0799862709],[1,397048,0.0347137264],[1,397100,0.0907202974],[1,397120,0.037205523],[1,397138,0.0783266195],[1,397152,0.4118871714],[1,397215,0.366595751],[1,397343,0.0249953388],[1,397381,0.0409854033],[1,397387,0.05208946],[1,397450,0.027190485],[1,397469,0.0674688254],[1,397541,0.2714644161],[1,397644,0.0352454995],[1,397672,0.0352454995],[1,397706,0.1517033916],[1,397730,0.0428033069],[1,397733,0.0886149752],[1,397761,0.114134412],[1,397785,0.0485958195],[1,397813,0.268910608],[1,397854,0.0249266006],[1,397864,0.0278694612],[1,397899,0.0274781242],[1,397908,0.046212456],[1,397935,0.0792946644],[1,397998,0.0477165876],[1,398003,0.0337424659],[1,398038,0.0454102276],[1,398196,0.026032575],[1,398198,0.0787394269],[1,398302,0.0937990468],[1,398339,0.0590199329],[1,398365,0.173568703],[1,398494,0.0409198124],[1,398497,0.0296344306],[1,398523,0.0791050824],[1,398538,0.0259806666],[1,398555,0.0237266409],[1,398560,0.0423964157],[1,398631,0.0339159689],[1,398698,0.0658062791],[1,398704,0.0235574703],[1,398715,0.2300154686],[1,398838,0.0360196648],[1,398850,0.1578842706],[1,398971,0.1407692812],[1,399039,0.0777371026],[1,399057,0.0527275948],[1,399072,0.0273765878],[1,399094,0.525762777],[1,399196,0.1621033442],[1,399250,0.1523892046],[1,399340,0.0393819585],[1,399386,0.1281107571],[1,399387,0.0434392808],[1,399392,0.0676192372],[1,399443,0.033543995],[1,399451,0.0371187182],[1,399462,0.0274781242],[1,399490,0.1149029871],[1,399521,0.1720825505],[1,399544,0.0376600973],[1,399687,0.0349812921],[1,399764,0.0459686407],[1,399786,0.0283376666],[1,399838,0.0317021738],[1,399844,0.1007127331],[1,399869,0.180996497],[1,399883,0.0261727623],[1,399931,0.0257129739],[1,399997,0.0377619145],[1,400071,0.3792398706],[1,400072,0.5821556163],[1,400082,0.0599291747],[1,400086,0.0456379574],[1,400103,0.0730932271],[1,400112,0.1234768847],[1,400342,0.1452876704],[1,400387,0.0369485965],[1,400439,0.0635296205],[1,400605,0.0632854348],[1,400671,0.0773886491],[1,400701,0.0554678657],[1,400750,0.0889206712],[1,400765,0.0339349134],[1,400912,0.1248016294],[1,400937,0.0827845992],[1,401054,0.0284026781],[1,401097,0.0517491639],[1,401174,0.2995979986],[1,401209,0.0274496709],[1,401232,0.0453233066],[1,401266,0.0371593393],[1,401276,0.0294712154],[1,401334,0.0367427266],[1,401339,0.0927472036],[1,401369,0.026277728],[1,401422,0.0900817],[1,401428,0.0346739504],[1,401455,0.0519063073],[1,401557,0.091693528],[1,401658,0.0309331656],[1,401719,0.1586350859],[1,401739,0.026277728],[1,401812,0.0235574703],[1,401824,0.1086661565],[1,401911,0.0529217679],[1,401915,0.0300205489],[1,401923,0.060882703],[1,401927,0.0297797514],[1,402001,0.0773886491],[1,402002,0.0272864694],[1,402051,0.0292607475],[1,402133,0.0521623174],[1,402247,0.1307187712],[1,402280,0.033543995],[1,402341,0.0261727623],[1,402422,0.0261820707],[1,402559,0.046490938],[1,402634,0.2038916267],[1,402658,0.0504312712],[1,402673,0.0238600799],[1,402679,0.565471789],[1,402700,0.026277728],[1,402788,0.0289327579],[1,402840,0.0356431286],[1,402888,0.1116866298],[1,402895,0.026277728],[1,402937,0.0625151076],[1,402981,0.0675280121],[1,402982,0.0261727623],[1,403094,0.0421807028],[1,403130,0.0782815543],[1,403210,0.0398252937],[1,403365,0.026277728],[1,403370,0.6817461516],[1,403373,0.1649098468],[1,403410,0.0340882972],[1,403493,0.0380403295],[1,403527,0.0280913838],[1,403637,0.0387427939],[1,403648,0.0534070906],[1,403671,0.0490154457],[1,403711,0.024265757],[1,403793,0.026277728],[1,403795,0.0934965945],[1,403817,0.4726211377],[1,403915,0.0515923078],[1,403941,0.0754968383],[1,404008,0.0340440095],[1,404015,0.0383193536],[1,404095,0.0559161044],[1,404255,0.3593899778],[1,404288,0.040925416],[1,404349,0.0736001686],[1,404483,0.2941297826],[1,404630,0.1218229484],[1,404667,0.022260012],[1,404739,0.06049086],[1,404928,0.040121034],[1,405282,0.0242665741],[1,405296,0.0511714236],[1,405344,0.0271931729],[1,405375,0.1226956081],[1,405376,0.1143192702],[1,405382,0.1328667839],[1,405428,0.0633798867],[1,405435,0.3000068694],[1,405498,0.1291485537],[1,405543,0.2601716296],[1,405545,0.0255929144],[1,405560,0.1411595971],[1,405576,0.0274391607],[1,405592,0.0652461751],[1,405593,0.0336965445],[1,405677,0.0966445646],[1,405687,0.067836015],[1,405694,0.0779763241],[1,405696,0.2258825174],[1,405713,0.1149579317],[1,405729,0.0729752572],[1,405732,0.0630599062],[1,405752,0.0833817517],[1,405762,0.0555097427],[1,405915,0.1968328165],[1,406007,0.0242665741],[1,406065,0.2596059141],[1,406314,0.519511331],[1,406317,0.1280695224],[1,406319,0.1549200323],[1,406327,0.0217223452],[1,406371,0.0743755655],[1,406388,0.0463511931],[1,406393,0.053691605],[1,406498,0.0728688547],[1,406531,0.0289839855],[1,406621,0.0217223452],[1,406634,0.050039236],[1,406664,0.022260012],[1,406668,0.0752400919],[1,406679,0.0242665741],[1,406715,0.0519901221],[1,406764,0.0380761894],[1,406813,0.1350213731],[1,406903,0.0800789648],[1,406919,0.0536689249],[1,406920,0.0367233079],[1,406945,0.1800717643],[1,407049,0.0467154475],[1,407075,0.0664546204],[1,407157,0.0223609152],[1,407171,0.0766342837],[1,407189,0.032244673],[1,407204,0.1008406512],[1,407220,0.0867040269],[1,407246,0.3926913271],[1,407275,0.1004766247],[1,407278,0.1599858557],[1,407309,0.0219111016],[1,407344,0.0732009664],[1,407370,0.1548988287],[1,407415,0.3919965995],[1,407427,0.0666426411],[1,407469,0.0293321105],[1,407487,0.048657663],[1,407499,0.054582658],[1,407532,0.0231588997],[1,407534,0.0246935359],[1,407572,0.6695912662],[1,407615,0.0648862153],[1,407624,0.0560122721],[1,407681,0.0498156561],[1,407696,0.062983743],[1,408015,0.1871332141],[1,408036,0.035902051],[1,408040,0.0223389622],[1,408199,0.0559211736],[1,408301,0.0246195298],[1,408377,0.0243971035],[1,408621,0.098744768],[1,408634,0.0907052667],[1,408643,0.0540555777],[1,408674,0.0289071971],[1,408679,0.0756740757],[1,408695,0.1081714666],[1,408712,0.4783789336],[1,408746,0.1146291245],[1,408764,0.0426367157],[1,408768,0.0222964663],[1,408850,0.0583282086],[1,408912,0.0932772969],[1,408942,0.2074336814],[1,408946,0.1714160453],[1,408977,0.0406715237],[1,409017,0.1094645029],[1,409026,0.221408775],[1,409027,0.0348595943],[1,409071,0.1948314144],[1,409072,0.0414873812],[1,409096,0.0246935359],[1,409120,0.0539936639],[1,409150,0.1602742471],[1,409204,0.1322909353],[1,409359,0.0357451376],[1,409375,0.2251854122],[1,409526,0.0760439396],[1,409714,0.0255398405],[1,409780,0.055980125],[1,409859,0.2653428844],[1,409893,0.0244311781],[1,409906,0.0764200983],[1,409907,0.0638105182],[1,409925,0.0270053599],[1,409961,0.1124610308],[1,410060,0.0296804041],[1,410180,0.023788978],[1,410316,0.0484445741],[1,410416,0.1454638543],[1,410960,0.0433521781],[1,411018,0.0280696557],[1,411201,0.0409960547],[1,411202,0.023767253],[1,411216,0.2817188312],[1,411219,0.0231274762],[1,411245,0.1720475446],[1,411371,0.0292042509],[1,411421,0.0319250394],[1,411590,0.0277029652],[1,411658,0.0255716318],[1,411661,0.3023126134],[1,411705,0.032241135],[1,411846,0.0731275178],[1,411852,0.0918379302],[1,411897,0.0444584851],[1,411941,0.0243928721],[1,411961,0.5713159631],[1,411975,0.0423579747],[1,411987,0.2339210627],[1,412012,0.1127614024],[1,412018,0.02273833],[1,412067,0.0502256101],[1,412069,0.0257975127],[1,412100,0.0301527536],[1,412155,0.0256765131],[1,412165,0.2837544098],[1,412177,0.6890866232],[1,412194,0.1169097921],[1,412321,0.0336696887],[1,412341,0.026981009],[1,412353,0.0555559577],[1,412356,0.0247812708],[1,412357,0.0367768212],[1,412390,0.0354914445],[1,412414,0.0247812708],[1,412418,0.037689304],[1,412452,0.072516044],[1,412590,0.194400793],[1,412615,0.2477933567],[1,412621,0.0766408752],[1,412665,0.0408197415],[1,412776,0.0375567886],[1,413163,0.0413012536],[1,413206,0.0638518951],[1,413237,0.1560099266],[1,413282,0.1011918553],[1,413297,0.0286210001],[1,413346,0.2792335322],[1,413513,0.0339145437],[1,413517,0.0553938897],[1,413518,0.0269142286],[1,413560,0.046180013],[1,413614,0.024785249],[1,413747,0.1064208909],[1,413776,0.0256765131],[1,413807,0.1242411981],[1,413855,0.0419607321],[1,413900,0.0353389639],[1,413960,0.1147642193],[1,414062,0.1897876578],[1,414087,0.0794969215],[1,414101,0.0466058947],[1,414111,0.0746905787],[1,414154,0.0256765131],[1,414174,0.5106626456],[1,414197,0.1571530811],[1,414210,0.029061876],[1,414211,0.0812789658],[1,414215,0.0872424741],[1,414230,0.0894364957],[1,414302,0.0284149181],[1,414411,0.0471505729],[1,414435,0.0569566483],[1,414450,0.0273322979],[1,414486,0.0702688145],[1,414501,0.6216862676],[1,414634,0.0354890362],[1,414638,0.027433889],[1,414651,0.0265397571],[1,414679,0.027433889],[1,414725,0.0246915239],[1,414732,0.0297286377],[1,414768,0.0256765131],[1,414810,0.0287170808],[1,414838,0.1959434503],[1,414854,0.2848367278],[1,414920,0.0767463588],[1,414959,0.2971018282],[1,414962,0.057381642],[1,415010,0.0256765131],[1,415013,0.1021353444],[1,415098,0.0243951545],[1,415126,0.037079273],[1,415183,0.0513424271],[1,415332,0.2185368362],[1,415344,0.2224948485],[1,415385,0.0529900409],[1,415407,0.0255716318],[1,415426,0.0270053599],[1,415434,0.0278872673],[1,415452,0.0243951545],[1,415470,0.1148662998],[1,415506,0.1196339562],[1,415521,0.0684555528],[1,415524,0.0605826293],[1,415594,0.0264482461],[1,415623,0.5902702119],[1,415697,0.0229361625],[1,415703,0.023474707],[1,415717,0.0370935107],[1,415720,0.0765412751],[1,415839,0.0233263171],[1,415856,0.0478652914],[1,415926,0.0273942489],[1,415995,0.0747748525],[1,416020,0.0717480884],[1,416114,0.1278528776],[1,416267,0.0865725434],[1,416278,0.0251877983],[1,416290,0.1994525391],[1,416324,0.11487149],[1,416333,0.0356334774],[1,416335,0.0629064522],[1,416353,0.0292268161],[1,416354,0.1461807995],[1,416381,0.023474707],[1,416395,0.0242395296],[1,416427,0.1185733277],[1,416513,0.0356411718],[1,416520,0.0230849353],[1,416567,0.0265348467],[1,416605,0.326624851],[1,416624,0.2504497764],[1,416711,0.0388727416],[1,416736,0.0248614727],[1,416755,0.3361577765],[1,416778,0.0342619796],[1,416786,0.0292268161],[1,416979,0.0438443926],[1,417015,0.0254845448],[1,417055,0.052100625],[1,417063,0.0431657543],[1,417076,0.6133935692],[1,417127,0.0765202863],[1,417173,0.2001436037],[1,417197,0.0573456093],[1,417245,0.1268068273],[1,417254,0.0222882882],[1,417295,0.0377067318],[1,417320,0.0612109733],[1,417354,0.0410591287],[1,417356,0.2169353629],[1,417369,0.0856336712],[1,417410,0.5540467985],[1,417411,0.178031045],[1,417480,0.0328650019],[1,417481,0.0355633783],[1,417484,0.44882686],[1,417598,0.1030496601],[1,417647,0.060674089],[1,417671,0.0231274762],[1,417672,0.0765412751],[1,417726,0.0272414693],[1,417784,0.0916543244],[1,417905,0.031761764],[1,417972,0.0966124581],[1,417977,0.0755538257],[1,417978,0.0374035203],[1,418018,0.02273833],[1,418068,0.4546685905],[1,418095,0.5649936061],[1,418317,0.07936069],[1,418426,0.3847906275],[1,418529,0.0523431884],[1,418631,0.0394686374],[1,418638,0.2026637068],[1,418684,0.0915980571],[1,418693,0.0722910157],[1,418754,0.0538021838],[1,418782,0.073241951],[1,418909,0.0242035152],[1,418925,0.4664627033],[1,418981,0.025373601],[1,418994,0.0238685216],[1,419005,0.084694506],[1,419018,0.1467615575],[1,419141,0.0241398562],[1,419350,0.0229361625],[1,419357,0.1281047449],[1,419396,0.0938852248],[1,419700,0.0394177685],[1,419746,0.0254845448],[1,419753,0.0270898656],[1,419811,0.6262514343],[1,419835,0.0229361625],[1,419851,0.5888003305],[1,419946,0.072013262],[1,420067,0.1479901958],[1,420202,0.0389644703],[1,420208,0.0242035152],[1,420232,0.0956809918],[1,420248,0.0280623278],[1,420272,0.0981744808],[1,420302,0.1991363143],[1,420348,0.3862432329],[1,420353,0.3258573457],[1,420361,0.0754390595],[1,420408,0.0566890612],[1,420464,0.0403524094],[1,420476,0.1307654879],[1,420491,0.3522120164],[1,420568,0.0293974336],[1,420619,0.1448087887],[1,420643,0.0242035152],[1,420738,0.0885420301],[1,420766,0.0518886915],[1,420779,0.0271399043],[1,420904,0.5651864605],[1,420942,0.0369501442],[1,420967,0.1352748226],[1,420997,0.4482596762],[1,421041,0.0254845448],[1,421125,0.0289892787],[1,421130,0.035820579],[1,421141,0.0607059432],[1,421165,0.1849576604],[1,421202,0.0457553508],[1,421205,0.2926500201],[1,421240,0.0320736719],[1,421273,0.124054834],[1,421283,0.1102837486],[1,421320,0.0781986699],[1,421327,0.0242395296],[1,421334,0.5615263627],[1,421377,0.0278099222],[1,421435,0.0387996687],[1,421570,0.0264589826],[1,421628,0.069952189],[1,421689,0.0456234763],[1,421703,0.0404792391],[1,421707,0.0379019503],[1,421708,0.3097397547],[1,421735,0.026142169],[1,421773,0.0873883671],[1,421822,0.0960559264],[1,421913,0.0327885742],[1,422000,0.0256496226],[1,422074,0.0589981248],[1,422143,0.0279775008],[1,422160,0.0668138814],[1,422197,0.0755538257],[1,422207,0.0812304262],[1,422333,0.0335651961],[1,422359,0.2984457085],[1,422473,0.0254845448],[1,422479,0.0290635748],[1,422553,0.023474707],[1,422633,0.0367573444],[1,422673,0.0595234703],[1,422688,0.0245472088],[1,422799,0.0255267848],[1,422805,0.0267482158],[1,422813,0.0253796143],[1,422939,0.3114527671],[1,422960,0.0401428535],[1,422983,0.0555177338],[1,423088,0.0303218185],[1,423202,0.0272414693],[1,423278,0.0267887056],[1,423309,0.0773061679],[1,423399,0.0254845448],[1,423455,0.0224818543],[1,423466,0.0837040062],[1,423517,0.6975300475],[1,423640,0.0717993572],[1,423678,0.1179954697],[1,423703,0.0987023617],[1,423791,0.0231274762],[1,423805,0.0361236537],[1,424060,0.0245895324],[1,424093,0.0540110517],[1,424151,0.7032633296],[1,424173,0.025867789],[1,424209,0.0637486433],[1,424225,0.1895450651],[1,424279,0.0452343673],[1,424288,0.0414221904],[1,424313,0.0242395296],[1,424444,0.0765787971],[1,424479,0.2654197604],[1,424593,0.0254845448],[1,424662,0.0620320567],[1,424685,0.0326445776],[1,424698,0.0358001257],[1,424788,0.2032803455],[1,424878,0.1948675333],[1,424884,0.0768893591],[1,424885,0.0903675058],[1,424955,0.0267887056],[1,424994,0.0231274762],[1,425045,0.1721236797],[1,425046,0.0571708766],[1,425075,0.1036613509],[1,425082,0.2333308213],[1,425136,0.0527049695],[1,425176,0.3600668465],[1,425277,0.0254845448],[1,425279,0.0912229437],[1,425343,0.2377906771],[1,425362,0.0242395296],[1,425505,0.1399919605],[1,425537,0.0293798812],[1,425652,0.0576274759],[1,425765,0.0585970744],[1,425915,0.0706675948],[1,425923,0.0274551649],[1,425932,0.023365365],[1,426034,0.0348056121],[1,426049,0.1144170267],[1,426142,0.0789448135],[1,426145,0.0556369101],[1,426158,0.023474707],[1,426184,0.5650829488],[1,426270,0.0271602355],[1,426290,0.1002086213],[1,426310,0.0277226837],[1,426350,0.0770445032],[1,426379,0.0527005861],[1,426393,0.0497632644],[1,426405,0.0712890245],[1,426446,0.0648235297],[1,426473,0.027979861],[1,426611,0.0648404479],[1,426622,0.0259390557],[1,426625,0.1479769258],[1,426655,0.0674824585],[1,426816,0.0709076981],[1,426849,0.027979861],[1,426872,0.025501677],[1,426946,0.1113466482],[1,426949,0.1872971789],[1,427046,0.0312723912],[1,427087,0.0388663499],[1,427189,0.0736578506],[1,427226,0.3501413125],[1,427465,0.1556864278],[1,427514,0.120423926],[1,427562,0.1516493254],[1,427790,0.0795563192],[1,427791,0.0259633122],[1,427803,0.0293128024],[1,427843,0.0556593772],[1,427928,0.3177957435],[1,427933,0.027979861],[1,428016,0.1948747929],[1,428058,0.027979861],[1,428082,0.0585405369],[1,428123,0.0316157967],[1,428140,0.0890274353],[1,428150,0.0734552274],[1,428169,0.2044373934],[1,428170,0.060227157],[1,428177,0.0713793075],[1,428230,0.0422901176],[1,428250,0.0773760002],[1,428282,0.0458370651],[1,428402,0.0670451279],[1,428411,0.026694554],[1,428422,0.027979861],[1,428497,0.1080279122],[1,428545,0.0593364485],[1,428553,0.0278746565],[1,428576,0.2083596186],[1,428622,0.027979861],[1,428698,0.1464905206],[1,428857,0.027979861],[1,428907,0.0779818631],[1,428910,0.028428703],[1,428982,0.0612559342],[1,429106,0.1281897961],[1,429153,0.0473279008],[1,429162,0.2004986388],[1,429220,0.0326497131],[1,429334,0.1273734069],[1,429336,0.0745636977],[1,429428,0.0261940756],[1,429455,0.1253975155],[1,429512,0.0504128221],[1,429561,0.1331499095],[1,429579,0.0355748736],[1,429634,0.0313342963],[1,429636,0.0283336452],[1,429656,0.0773622356],[1,429694,0.025614922],[1,429697,0.0934986125],[1,429724,0.1983367748],[1,429772,0.0423667143],[1,429940,0.1222045839],[1,430178,0.2057145288],[1,430203,0.0632530852],[1,430240,0.0539669654],[1,430244,0.0395697924],[1,430329,0.0259633122],[1,430338,0.0556251073],[1,430340,0.025614922],[1,430472,0.1098404153],[1,430523,0.1918871619],[1,430531,0.040239968],[1,430538,0.2294065331],[1,430579,0.140030412],[1,430605,0.0863187061],[1,430621,0.0779101714],[1,430655,0.0709816812],[1,430692,0.0304811411],[1,430696,0.659561455],[1,430718,0.0947644063],[1,430765,0.108238538],[1,430779,0.0293128024],[1,430845,0.0371729557],[1,430884,0.2013468298],[1,430899,0.0276821236],[1,430905,0.0619314946],[1,430987,0.0384838424],[1,431037,0.0325517916],[1,431109,0.0328702771],[1,431159,0.0243314431],[1,431199,0.0289083187],[1,431242,0.0290583704],[1,431266,0.0246546324],[1,431304,0.054344053],[1,431341,0.0792200481],[1,431359,0.7022369012],[1,431387,0.0663012044],[1,431395,0.1068217291],[1,431506,0.0249598197],[1,431527,0.0536737546],[1,431807,0.0746147068],[1,431842,0.0768988641],[1,431849,0.7283301042],[1,431866,0.3870829225],[1,431875,0.0247214219],[1,431968,0.0270835338],[1,432013,0.0463035501],[1,432038,0.0254784598],[1,432128,0.030971975],[1,432140,0.0480060412],[1,432160,0.0790055639],[1,432348,0.0513840015],[1,432377,0.339710267],[1,432419,0.0309872049],[1,432516,0.0250693957],[1,432575,0.331911028],[1,432591,0.0600778982],[1,432640,0.0933368831],[1,432707,0.2713837258],[1,432723,0.2700596413],[1,432733,0.1328208666],[1,432738,0.0250693957],[1,432740,0.0238804383],[1,432771,0.0273362961],[1,432780,0.1049911466],[1,432811,0.0458881648],[1,432831,0.0799104492],[1,432843,0.0270835338],[1,432862,0.1815941175],[1,432923,0.0422834173],[1,433008,0.0280091901],[1,433146,0.0247191679],[1,433205,0.0543662491],[1,433234,0.0336866194],[1,433244,0.5832450083],[1,433295,0.0497141561],[1,433340,0.0247214219],[1,433392,0.0995190046],[1,433461,0.0430324127],[1,433478,0.0495446765],[1,433571,0.0270835338],[1,433636,0.0812510193],[1,433647,0.0427915224],[1,433704,0.1508594211],[1,433729,0.0301163485],[1,433798,0.0698238264],[1,433840,0.193078595],[1,433878,0.1620711151],[1,433975,0.0413705516],[1,434107,0.0250693957],[1,434150,0.1522908934],[1,434170,0.0249598197],[1,434238,0.0275559624],[1,434274,0.0247214219],[1,434284,0.0378912522],[1,434325,0.1143146665],[1,434387,0.0280091901],[1,434434,0.3021339063],[1,434486,0.0270835338],[1,434490,0.1357961717],[1,434492,0.0269189134],[1,434568,0.053558489],[1,434592,0.0611119053],[1,434669,0.0270835338],[1,434722,0.0470567983],[1,434725,0.0351391046],[1,434756,0.0728423513],[1,434763,0.1451244538],[1,434794,0.1154490615],[1,434811,0.0270835338],[1,434819,0.0744957841],[1,434823,0.0791433751],[1,434870,0.0247214219],[1,435024,0.0402635359],[1,435043,0.0417043652],[1,435073,0.2810761397],[1,435174,0.086909083],[1,435214,0.0306249107],[1,435227,0.0273260241],[1,435229,0.0390424433],[1,435282,0.0264008483],[1,435283,0.2722655511],[1,435293,0.1350288378],[1,435337,0.0282549221],[1,435346,0.0782495081],[1,435415,0.0270835338],[1,435507,0.4811714762],[1,435535,0.0321737146],[1,435551,0.0279827036],[1,435754,0.2023785309],[1,435755,0.0808324578],[1,435812,0.0347282888],[1,435880,0.0791603635],[1,435989,0.4594537133],[1,435991,0.0247214219],[1,436154,0.0526682722],[1,436371,0.0284148819],[1,436373,0.0346379907],[1,436435,0.0658516099],[1,436439,0.1861611964],[1,436522,0.0681009227],[1,436540,0.0499106456],[1,436618,0.0952170282],[1,436717,0.1479528624],[1,436746,0.0249598197],[1,436767,0.2630521312],[1,436838,0.0719708915],[1,436854,0.035268683],[1,437135,0.0270835338],[1,437155,0.1112479464],[1,437203,0.08842125],[1,437243,0.0249598197],[1,437263,0.0410571361],[1,437323,0.4489088859],[1,437337,0.0587162608],[1,437409,0.0534941233],[1,437501,0.0475243997],[1,437526,0.0283904852],[1,437596,0.0242060724],[1,437701,0.0376952414],[1,437707,0.0273971714],[1,437713,0.0532068321],[1,437945,0.0247214219],[1,438008,0.2488022069],[1,438079,0.0589129541],[1,438161,0.2550296535],[1,438164,0.0334651216],[1,438215,0.1026728174],[1,438327,0.0280600567],[1,438378,0.3309992279],[1,438387,0.0270835338],[1,438402,0.0246083124],[1,438414,0.030205518],[1,438440,0.0535711251],[1,438462,0.3202476112],[1,438521,0.1923481667],[1,438712,0.0244643707],[1,438794,0.0267861524],[1,438882,0.1694637796],[1,438889,0.02658008],[1,438890,0.1744538091],[1,438939,0.0948277602],[1,438957,0.1076597181],[1,439000,0.0424208805],[1,439003,0.0266162179],[1,439010,0.351480314],[1,439040,0.065709175],[1,439094,0.4436658061],[1,439134,0.028555412],[1,439141,0.0554118564],[1,439194,0.0882733638],[1,439202,0.053403423],[1,439220,0.0366952984],[1,439354,0.0257997633],[1,439373,0.0299996716],[1,439424,0.0469480593],[1,439576,0.0341102325],[1,439618,0.0281231527],[1,439627,0.0418521431],[1,439744,0.4578712965],[1,439756,0.2204945225],[1,439784,0.0270835338],[1,439813,0.265400948],[1,440006,0.0270835338],[1,440031,0.0292854489],[1,440054,0.0493948222],[1,440128,0.0416182031],[1,440186,0.1864348768],[1,440214,0.0627848355],[1,440224,0.0301163485],[1,440253,0.5489014692],[1,440260,0.0270835338],[1,440279,0.0270835338],[1,440288,0.0390245187],[1,440375,0.1784177777],[1,440396,0.0465716047],[1,440413,0.0329491365],[1,440488,0.0510202704],[1,440493,0.0261866064],[1,440584,0.0267861524],[1,440652,0.0480459954],[1,440835,0.0647508981],[1,440839,0.1293660379],[1,440876,0.075565285],[1,441086,0.0485769055],[1,441097,0.0247214219],[1,441101,0.0678632217],[1,441103,0.161135109],[1,441131,0.1638189928],[1,441141,0.1600480144],[1,441155,0.168961341],[1,441174,0.0536587461],[1,441287,0.0858115142],[1,441302,0.0782495081],[1,441339,0.0677902761],[1,441476,0.2018993879],[1,441490,0.041603755],[1,441499,0.0739535033],[1,441520,0.03645986],[1,441526,0.038519261],[1,441619,0.0661458119],[1,441630,0.0582096666],[1,441673,0.1121956111],[1,441844,0.0735176308],[1,441848,0.0876441092],[1,441953,0.0254053002],[1,441989,0.025020604],[1,441997,0.0270944781],[1,442011,0.032104616],[1,442050,0.0795874348],[1,442056,0.0293392503],[1,442100,0.0279318774],[1,442160,0.0545304009],[1,442196,0.0326237982],[1,442247,0.0271216351],[1,442271,0.0465396459],[1,442273,0.0361215898],[1,442284,0.0837459635],[1,442296,0.0774939401],[1,442297,0.0272436183],[1,442319,0.0311024121],[1,442383,0.3181907706],[1,442460,0.0292636197],[1,442640,0.0280627012],[1,442771,0.0274181556],[1,442787,0.0831438629],[1,442852,0.0272526872],[1,442900,0.11491779],[1,442920,0.0941330665],[1,442952,0.2380625439],[1,442973,0.0959624023],[1,442979,0.3190842806],[1,442986,0.0556851765],[1,443036,0.0460873673],[1,443144,0.1193249671],[1,443191,0.055556397],[1,443216,0.7634911774],[1,443289,0.1376760153],[1,443315,0.0272526872],[1,443326,0.2481399028],[1,443349,0.0272526872],[1,443460,0.0365856665],[1,443472,0.0367919956],[1,443481,0.0339793328],[1,443511,0.2155449018],[1,443524,0.0263555572],[1,443539,0.0444753545],[1,443613,0.045920924],[1,443623,0.0311420066],[1,443663,0.0246982754],[1,443696,0.0248900418],[1,443737,0.0745401266],[1,443746,0.0259686267],[1,443770,0.0281161455],[1,443775,0.0449987635],[1,443802,0.0425810204],[1,443809,0.2400324363],[1,443818,0.0246982754],[1,443840,0.0977432542],[1,443853,0.0367919956],[1,443860,0.0444824594],[1,443874,0.0944997783],[1,443921,0.3551266844],[1,443933,0.7003447661],[1,443940,0.0272526872],[1,443962,0.0665922165],[1,443981,0.0485121966],[1,444020,0.0377861955],[1,444021,0.038644147],[1,444119,0.1271955428],[1,444121,0.0691906293],[1,444133,0.0272526872],[1,444169,0.0272526872],[1,444172,0.0271475847],[1,444256,0.0619843503],[1,444275,0.0284594276],[1,444288,0.0732007583],[1,444299,0.0663036974],[1,444312,0.2426231858],[1,444403,0.0714233428],[1,444405,0.0473767765],[1,444532,0.0629797964],[1,444534,0.0496684877],[1,444614,0.0252380941],[1,444654,0.3760249756],[1,444658,0.3550114311],[1,444673,0.26355906],[1,444715,0.0489790345],[1,444728,0.2681387801],[1,444751,0.0544628865],[1,444756,0.073626822],[1,444813,0.0277584527],[1,444823,0.0272526872],[1,444847,0.1392280965],[1,444872,0.0404060879],[1,444912,0.0472508707],[1,444913,0.0669045954],[1,445012,0.3131940177],[1,445032,0.0958014423],[1,445154,0.0657507034],[1,445233,0.0562017292],[1,445242,0.0295735311],[1,445269,0.045901644],[1,445276,0.0269652852],[1,445321,0.0254686337],[1,445367,0.5700362273],[1,445414,0.0239904103],[1,445459,0.0383017675],[1,445501,0.0718183395],[1,445600,0.0591683477],[1,445622,0.0240488682],[1,445744,0.026284537],[1,445801,0.0401598774],[1,445874,0.0661242629],[1,445889,0.0784302181],[1,445896,0.1526139437],[1,445929,0.1153472676],[1,445953,0.2828502289],[1,446157,0.0262634044],[1,446277,0.2120766041],[1,446311,0.0792779124],[1,446325,0.0269652852],[1,446327,0.0771314878],[1,446342,0.0518760493],[1,446359,0.0538505419],[1,446413,0.0404310417],[1,446418,0.0272526872],[1,446462,0.0415490655],[1,446505,0.043602998],[1,446508,0.0944114917],[1,446633,0.0291825497],[1,446674,0.059321301],[1,446721,0.03115724],[1,446802,0.0772263858],[1,446855,0.176113689],[1,446860,0.0551138838],[1,446911,0.0298365693],[1,447037,0.0886396652],[1,447062,0.0522426811],[1,447082,0.0267735353],[1,447098,0.0517814471],[1,447184,0.2362067102],[1,447194,0.0500269079],[1,447265,0.1430200941],[1,447350,0.0366504255],[1,447355,0.1467578704],[1,447436,0.0382224677],[1,447500,0.091534669],[1,447514,0.0533818859],[1,447578,0.0306038693],[1,447587,0.2355466766],[1,447638,0.0307878635],[1,447652,0.0290137686],[1,447685,0.0368037472],[1,447700,0.7293136784],[1,447736,0.0540790668],[1,447744,0.1199307579],[1,447766,0.0936787594],[1,447935,0.0619292501],[1,447970,0.0821329261],[1,447992,0.0781032697],[1,447993,0.03173571],[1,448000,0.0976152195],[1,448044,0.0269552386],[1,448113,0.0387151121],[1,448232,0.0611439811],[1,448251,0.0524345722],[1,448266,0.0734601278],[1,448399,0.028101893],[1,448466,0.0452677577],[1,448500,0.4283534675],[1,448526,0.0446947573],[1,448723,0.0252380941],[1,448760,0.0612797471],[1,448817,0.0365524813],[1,448895,0.0752373102],[1,449132,0.0348587434],[1,449195,0.2546411192],[1,449250,0.0333709963],[1,449254,0.0246738223],[1,449263,0.0238569583],[1,449308,0.0466225944],[1,449399,0.0231844708],[1,449438,0.1163733973],[1,449439,0.0275551857],[1,449523,0.0280064413],[1,449622,0.023349113],[1,449631,0.0313177071],[1,449732,0.1476385213],[1,449929,0.0203152513],[1,449931,0.024243799],[1,450072,0.023349113],[1,450102,0.6902850838],[1,450163,0.0764525297],[1,450198,0.1171158395],[1,450219,0.0358073866],[1,450247,0.0759696631],[1,450268,0.1419018364],[1,450437,0.0407073981],[1,450457,0.0664232964],[1,450497,0.0337175076],[1,450531,0.0360674548],[1,450537,0.2832358029],[1,450546,0.3733736931],[1,450571,0.0218514564],[1,450624,0.0355776084],[1,450798,0.3915970989],[1,450810,0.3265915231],[1,450836,0.2257598314],[1,450873,0.037182744],[1,450928,0.0241938935],[1,450993,0.0246450878],[1,451126,0.0933449744],[1,451148,0.0823032215],[1,451153,0.0448817353],[1,451188,0.4288433334],[1,451253,0.2349280289],[1,451256,0.1431247586],[1,451347,0.0243158486],[1,451381,0.0486769865],[1,451402,0.0209987797],[1,451444,0.0569661022],[1,451458,0.0403073358],[1,451528,0.068681839],[1,451553,0.0794461005],[1,451700,0.0728459038],[1,451770,0.0708114442],[1,451774,0.023349113],[1,451881,0.0617337149],[1,451888,0.1233399436],[1,451896,0.034743899],[1,451899,0.0501075612],[1,451923,0.2428364258],[1,451940,0.0643213237],[1,451983,0.0819965773],[1,452009,0.3436459918],[1,452047,0.0779268425],[1,452095,0.2792159241],[1,452112,0.0256746496],[1,452118,0.122796844],[1,452131,0.5323668588],[1,452184,0.1318599597],[1,452186,0.1568720665],[1,452224,0.4331488942],[1,452235,0.0209987797],[1,452252,0.3239246553],[1,452262,0.0221761342],[1,452271,0.2948394793],[1,452277,0.0468318146],[1,452286,0.3033929149],[1,452306,0.023349113],[1,452323,0.1914237405],[1,452362,0.0230532144],[1,452454,0.0232445582],[1,452493,0.023349113],[1,452554,0.1701621894],[1,452557,0.1342565458],[1,452566,0.0523605613],[1,452608,0.0781726615],[1,452628,0.0502136852],[1,452658,0.0337475227],[1,452718,0.0376938536],[1,452755,0.0496581294],[1,452761,0.0412833232],[1,452766,0.1556480541],[1,452778,0.0909031912],[1,452789,0.0234517003],[1,452901,0.5886311645],[1,452909,0.0298932954],[1,452925,0.1185947334],[1,452947,0.0331062439],[1,452979,0.0252689186],[1,453068,0.0534198102],[1,453090,0.0262282443],[1,453110,0.0245708962],[1,453160,0.0233447329],[1,453166,0.0241496645],[1,453182,0.0199433057],[1,453284,0.6057195245],[1,453296,0.0741883983],[1,453322,0.2247375783],[1,453364,0.0279357112],[1,453409,0.020153256],[1,453491,0.1014478131],[1,453544,0.0237424343],[1,453547,0.0300957766],[1,453565,0.0593787453],[1,453588,0.0281515226],[1,453683,0.0201510158],[1,453686,0.0420952338],[1,453697,0.0777148639],[1,453770,0.020153256],[1,453805,0.3139233893],[1,453869,0.0323646617],[1,453886,0.0451388876],[1,453962,0.165858662],[1,453986,0.022500914],[1,454033,0.1132708318],[1,454065,0.0265857859],[1,454066,0.0554575449],[1,454101,0.0234714614],[1,454131,0.0204991005],[1,454141,0.3690842885],[1,454239,0.0927391434],[1,454242,0.0585833563],[1,454248,0.1516327328],[1,454331,0.0234349277],[1,454352,0.0403306221],[1,454377,0.0687275459],[1,454396,0.075849277],[1,454460,0.0337627445],[1,454473,0.0396293092],[1,454539,0.0353729923],[1,454577,0.2451606699],[1,454610,0.0239798816],[1,454708,0.028121195],[1,454869,0.0353729923],[1,454870,0.1117104607],[1,454902,0.0258861562],[1,454919,0.1585015089],[1,454952,0.0470463548],[1,455079,0.0301121325],[1,455089,0.050741564],[1,455100,0.352595832],[1,455126,0.0462024812],[1,455153,0.0273204877],[1,455326,0.0767560211],[1,455400,0.1466700158],[1,455414,0.251467285],[1,455462,0.027345565],[1,455480,0.1337457825],[1,455523,0.3589448798],[1,455530,0.0645686885],[1,455593,0.1081141122],[1,455619,0.435283197],[1,455630,0.0238704655],[1,455687,0.0396561498],[1,455729,0.0276475632],[1,455755,0.0317693836],[1,455763,0.0374720242],[1,455819,0.0705619444],[1,455873,0.0248139441],[1,455874,0.0579041342],[1,455929,0.13298329],[1,455938,0.0369246681],[1,455966,0.025694134],[1,455971,0.0473508689],[1,455999,0.2380795336],[1,456075,0.0364551991],[1,456090,0.1583407721],[1,456099,0.1539933519],[1,456118,0.0347744659],[1,456327,0.0448168207],[1,456409,0.0714969878],[1,456444,0.0893056809],[1,456449,0.0277985834],[1,456476,0.0396707561],[1,456561,0.0286810463],[1,456592,0.5281190652],[1,456650,0.0357788091],[1,456687,0.0690053671],[1,456707,0.0685749227],[1,456734,0.1203755658],[1,456747,0.1274010765],[1,456771,0.0530232388],[1,456780,0.0542625883],[1,456786,0.0813329277],[1,456791,0.0571972809],[1,456803,0.033843254],[1,456832,0.1084371443],[1,456891,0.1347246337],[1,456917,0.0299697643],[1,457065,0.0508829006],[1,457076,0.0287863639],[1,457148,0.0299697643],[1,457203,0.051040678],[1,457330,0.0466897746],[1,457362,0.1044553965],[1,457384,0.0521126869],[1,457498,0.2102888235],[1,457528,0.0366667407],[1,457541,0.1151916307],[1,457548,0.4617342737],[1,457598,0.1146797437],[1,457619,0.0279478639],[1,457633,0.0317372336],[1,457634,0.2839336919],[1,457641,0.0309500506],[1,457645,0.0447085076],[1,457704,0.3300912692],[1,457715,0.0318493467],[1,457736,0.0513816965],[1,457863,0.157070371],[1,457885,0.368086382],[1,457918,0.0312817525],[1,457934,0.0530670202],[1,457946,0.0356924903],[1,457956,0.058877983],[1,458011,0.2481753323],[1,458169,0.133823734],[1,458248,0.4407447536],[1,458410,0.0360384654],[1,458506,0.1819875854],[1,458512,0.0273405071],[1,458572,0.0570859575],[1,458622,0.0603675237],[1,458715,0.1956536674],[1,458723,0.0389739162],[1,458732,0.0529839669],[1,458735,0.0270454198],[1,458747,0.1326219114],[1,458757,0.0820552664],[1,458801,0.0659461545],[1,458969,0.0282717172],[1,458970,0.0284644015],[1,458974,0.0956933895],[1,459126,0.0627389769],[1,459265,0.0864358346],[1,459289,0.0825317442],[1,459291,0.0929838188],[1,459306,0.0330161181],[1,459324,0.1813346162],[1,459357,0.0989790046],[1,459369,0.0352982286],[1,459401,0.0355989914],[1,459425,0.0307327508],[1,459553,0.0358579725],[1,459582,0.094849311],[1,459602,0.2451055299],[1,459674,0.0542405632],[1,459723,0.4547238546],[1,459766,0.0648656245],[1,459853,0.0332100212],[1,459873,0.1323961228],[1,459955,0.0326866997],[1,460003,0.3827959263],[1,460150,0.5376506049],[1,460173,0.064637529],[1,460238,0.0793851298],[1,460256,0.0295481494],[1,460281,0.0568136235],[1,460334,0.0556015224],[1,460385,0.042286037],[1,460420,0.0323037086],[1,460441,0.0387455882],[1,460567,0.0427611344],[1,460600,0.0337691148],[1,460705,0.0605750767],[1,460706,0.0668761203],[1,460749,0.0956445649],[1,460811,0.0459552352],[1,460895,0.0288141199],[1,460927,0.0307776565],[1,461169,0.0308383564],[1,461193,0.029936932],[1,461746,0.053143263],[1,462108,0.053939191],[1,462322,0.0819323491],[1,462714,0.0295062715],[1,462813,0.2906131698],[1,463061,0.2904688622],[1,463073,0.0469364273],[1,463309,0.2498777459],[1,463485,0.0997681049],[1,463649,0.0316927756],[1,463758,0.0592904379],[1,463790,0.0284621362],[1,463791,0.081698707],[1,463793,0.0344114073],[1,463897,0.0871008213],[1,463927,0.1057215531],[1,464092,0.0393771259],[1,464167,0.0527166331],[1,464172,0.0308383564],[1,464224,0.0284621362],[1,464285,0.0303244875],[1,464286,0.1910594111],[1,464326,0.0349689842],[1,464390,0.0558195489],[1,464431,0.0308383564],[1,464445,0.2245420039],[1,464502,0.3330760535],[1,464511,0.0404112205],[1,464718,0.0581222885],[1,464747,0.0326866997],[1,464816,0.034150079],[1,464858,0.0869230712],[1,464879,0.0283452495],[1,464901,0.0296535887],[1,464943,0.0758732317],[1,464945,0.0612228471],[1,464960,0.0355424923],[1,465044,0.0295844218],[1,465149,0.0298581547],[1,465232,0.117798151],[1,465249,0.0401940716],[1,465270,0.0284644015],[1,465277,0.0424947107],[1,465322,0.091608761],[1,465328,0.0496778678],[1,465685,0.0298170769],[1,465696,0.5440928273],[1,465762,0.0842781167],[1,465769,0.0897404144],[1,465941,0.0844754791],[1,465946,0.0413364115],[1,465963,0.0606324521],[1,466190,0.0308383564],[1,466225,0.0813887211],[1,466229,0.0420937756],[1,466257,0.0326866997],[1,466271,0.1427215949],[1,466322,0.0430512812],[1,466354,0.6195198877],[1,466357,0.0839035676],[1,466402,0.2975153786],[1,466424,0.2360790123],[1,466493,0.4995385355],[1,466500,0.0308383564],[1,466534,0.0308383564],[1,466540,0.0490447412],[1,466658,0.2456573328],[1,466682,0.1282277016],[1,466715,0.0484733185],[1,466892,0.0616844476],[1,466902,0.2471739708],[1,466930,0.0330161181],[1,467015,0.0513317854],[1,467089,0.0457164571],[1,467094,0.0364489136],[1,467125,0.038049235],[1,467151,0.1781357409],[1,467255,0.2653654549],[1,467290,0.0284644015],[1,467352,0.0719200405],[1,467374,0.1476394292],[1,467398,0.0533108733],[1,467429,0.0973570951],[1,467542,0.0895158122],[1,467552,0.0586616746],[1,467587,0.0497511625],[1,467612,0.1301904491],[1,467617,0.4064094032],[1,467654,0.0308383564],[1,467662,0.0653800312],[1,467753,0.1251652763],[1,467756,0.0418942371],[1,467786,0.0620784807],[1,467822,0.4045887675],[1,467836,0.0321514778],[1,467840,0.045148804],[1,467910,0.0468500009],[1,467926,0.0371181386],[1,467970,0.0377170384],[1,467984,0.689961823],[1,467986,0.0465497381],[1,468094,0.0708018162],[1,468135,0.2369580206],[1,468141,0.0619291394],[1,468219,0.0345125401],[1,468245,0.0301151844],[1,468266,0.0468719692],[1,468313,0.0736829342],[1,468380,0.2911405662],[1,468523,0.0872431091],[1,468551,0.1621538522],[1,468714,0.0910589002],[1,468751,0.1868131987],[1,468757,0.0890888096],[1,468790,0.0378200377],[1,468796,0.0347278861],[1,468842,0.0822857705],[1,468884,0.1975154062],[1,468897,0.1264416787],[1,468915,0.0776477882],[1,468937,0.0880127955],[1,468943,0.0916318097],[1,468957,0.1191270641],[1,468985,0.0499422876],[1,468997,0.0540091548],[1,469008,0.6083447563],[1,469117,0.035419451],[1,469124,0.0629700959],[1,469192,0.0319260539],[1,469249,0.1028430369],[1,469285,0.0602588349],[1,469302,0.0566948553],[1,469323,0.0345394082],[1,469366,0.035419451],[1,469399,0.1857561825],[1,469465,0.1702037611],[1,469467,0.426487334],[1,469479,0.0636210454],[1,469509,0.1640853363],[1,469540,0.0625939342],[1,469562,0.1560337849],[1,469585,0.035419451],[1,469596,0.0364513596],[1,469707,0.1094016364],[1,469734,0.0634307497],[1,469776,0.0407582079],[1,469832,0.0362779152],[1,469906,0.0880587568],[1,469971,0.0452797985],[1,470093,0.0979946365],[1,470166,0.0880587568],[1,470344,0.0365016841],[1,470347,0.0555410956],[1,470523,0.1924113203],[1,470587,0.0619847575],[1,470653,0.0487231733],[1,470654,0.1555286176],[1,470716,0.0342757387],[1,470743,0.0305394991],[1,470759,0.0533248312],[1,470818,0.1157811113],[1,470829,0.0841272341],[1,470831,0.0491354525],[1,470837,0.0365016841],[1,470862,0.1226974318],[1,470947,0.0365016841],[1,470969,0.4160647439],[1,471054,0.0447181681],[1,471065,0.033545024],[1,471149,0.1391774072],[1,471162,0.0574007209],[1,471211,0.0352383144],[1,471212,0.0344622165],[1,471267,0.1568030225],[1,471303,0.1161761774],[1,471320,0.6173756792],[1,471495,0.1289721877],[1,471661,0.4344644838],[1,471776,0.0341098667],[1,471843,0.1705260523],[1,471893,0.0365016841],[1,471933,0.0654829101],[1,471936,0.0381394152],[1,472018,0.087717337],[1,472069,0.0823578754],[1,472101,0.3644285839],[1,472237,0.0832507854],[1,472319,0.0408288161],[1,472386,0.0681965028],[1,472395,0.0396968611],[1,472412,0.5156750668],[1,472419,0.1185360185],[1,472438,0.0390687879],[1,472470,0.0695545772],[1,472536,0.2473986888],[1,472563,0.0607845876],[1,472619,0.0377790197],[1,472719,0.0603603592],[1,472728,0.2155606115],[1,472781,0.133484024],[1,472849,0.0423107191],[1,472929,0.0546930103],[1,472936,0.2518708695],[1,472983,0.0424349658],[1,473023,0.0469871055],[1,473030,0.058308304],[1,473067,0.0434903427],[1,473163,0.043320158],[1,473169,0.3338518678],[1,473216,0.1518250125],[1,473339,0.0464329004],[1,473453,0.1028276737],[1,473496,0.0497324483],[1,473527,0.0730683384],[1,473573,0.0740858429],[1,473577,0.0358008685],[1,473643,0.1862109484],[1,473652,0.2212241075],[1,473690,0.1371561415],[1,473769,0.0397324175],[1,473811,0.0416923941],[1,473880,0.6868996119],[1,473899,0.1670708549],[1,473925,0.0386150486],[1,473969,0.0930127365],[1,473975,0.0411860278],[1,473979,0.4327594558],[1,474005,0.090972959],[1,474069,0.0520489051],[1,474075,0.0409136295],[1,474090,0.0573376363],[1,474185,0.2139362669],[1,474212,0.0711060582],[1,474283,0.0372877977],[1,474289,0.0357985811],[1,474320,0.1348541845],[1,474391,0.1228924161],[1,474426,0.3925152339],[1,474506,0.0358008685],[1,474532,0.3869278048],[1,474559,0.0675379131],[1,474583,0.2068758324],[1,474604,0.0816709991],[1,474665,0.2744027576],[1,474900,0.0431125494],[1,474975,0.0393615015],[1,475021,0.0445858573],[1,475032,0.260922616],[1,475046,0.0940999061],[1,475089,0.0837142824],[1,475123,0.065780782],[1,475129,0.2032627407],[1,475199,0.0404273925],[1,475293,0.0358008685],[1,475324,0.2478318757],[1,475476,0.0519888288],[1,475581,0.0359628616],[1,475636,0.0356063001],[1,475666,0.0358008685],[1,475747,0.0357985811],[1,475877,0.0500001503],[1,475894,0.070659348],[1,475935,0.0482957421],[1,475938,0.0357985811],[1,475990,0.0381980364],[1,476021,0.3154444166],[1,476034,0.1357216071],[1,476046,0.3812519372],[1,476105,0.0734644276],[1,476118,0.0356063001],[1,476134,0.5924865005],[1,476175,0.0358008685],[1,476224,0.1458396976],[1,476327,0.1120371465],[1,476342,0.2680767685],[1,476427,0.0348014097],[1,476453,0.0348014097],[1,476524,0.0391661437],[1,476685,0.0537654094],[1,476686,0.2726484626],[1,476708,0.035894311],[1,476758,0.0627272355],[1,476917,0.052576091],[1,477059,0.0401566556],[1,477156,0.1682125442],[1,477265,0.0472462438],[1,477285,0.1986579304],[1,477343,0.1135168494],[1,477344,0.2700617901],[1,477345,0.035894311],[1,477477,0.0371954151],[1,477481,0.2798237738],[1,477497,0.0469320691],[1,477547,0.0349877852],[1,477592,0.0532531252],[1,477611,0.0777692389],[1,477638,0.0318779289],[1,477640,0.3458601677],[1,477643,0.0912954086],[1,477646,0.0332299436],[1,477670,0.6581881693],[1,477684,0.0846650852],[1,477687,0.0464855284],[1,477733,0.0385116484],[1,477740,0.0394172445],[1,477939,0.0332299436],[1,478019,0.0427215553],[1,478030,0.0496957575],[1,478075,0.0639689416],[1,478230,0.0810012132],[1,478267,0.1400886567],[1,478291,0.128547044],[1,478346,0.0555993493],[1,478373,0.5816017096],[1,478443,0.0365878607],[1,478445,0.0771196717],[1,478447,0.151881294],[1,478523,0.1456914133],[1,478527,0.0355913827],[1,478557,0.0343593609],[1,478571,0.1726164613],[1,478608,0.2394101077],[1,478652,0.0570066615],[1,478714,0.4913818887],[1,478730,0.081208712],[1,478857,0.6442755439],[1,478882,0.0574221769],[1,478914,0.0456961838],[1,478953,0.0871368429],[1,479038,0.4041406857],[1,479064,0.137861059],[1,479115,0.0488474719],[1,479134,0.5396968996],[1,479140,0.0324184383],[1,479188,0.0618567604],[1,479210,0.0568334784],[1,479217,0.1021633406],[1,479232,0.0328737255],[1,479276,0.0376332295],[1,479283,0.0945535672],[1,479330,0.3018115961],[1,479365,0.0465914629],[1,479366,0.1412775374],[1,479368,0.0664302169],[1,479374,0.3261216045],[1,479379,0.0565048347],[1,479392,0.1926144059],[1,479399,0.059919678],[1,479402,0.0389391327],[1,479598,0.0628860874],[1,479641,0.1220384186],[1,479642,0.0610810761],[1,479646,0.0411873897],[1,479661,0.0472394626],[1,479678,0.0564784443],[1,479686,0.0426613512],[1,479720,0.1448501315],[1,479725,0.1264832158],[1,479787,0.0389758584],[1,479854,0.0403528435],[1,479893,0.0485025132],[1,479905,0.1258888782],[1,479910,0.0524603253],[1,479960,0.0402454652],[1,479967,0.0825483514],[1,479970,0.2513541271],[1,480006,0.1369288412],[1,480050,0.2976072733],[1,480124,0.0802753132],[1,480200,0.4828948259],[1,480341,0.1111707263],[1,480357,0.037839546],[1,480380,0.0429430814],[1,480392,0.0389391327],[1,480420,0.1284421609],[1,480428,0.1880176544],[1,480432,0.075204624],[1,480471,0.1356984417],[1,480476,0.2229831439],[1,480508,0.037839546],[1,480526,0.0661224245],[1,480553,0.4694460257],[1,480579,0.0637522567],[1,480616,0.1293189233],[1,480629,0.0510010804],[1,480694,0.037839546],[1,480717,0.0393529284],[1,480726,0.1057192025],[1,480779,0.051840685],[1,481685,0.0495913532],[1,481812,0.2370388524],[1,481823,0.0428794458],[1,481929,0.0395494929],[1,481934,0.2044722918],[1,482038,0.053682403],[1,482060,0.2366143388],[1,482109,0.2337337051],[1,482151,0.3160695675],[1,482195,0.0499802532],[1,482266,0.037839546],[1,482362,0.050204211],[1,482431,0.0466875814],[1,482445,0.0670242685],[1,482489,0.0409868781],[1,482497,0.0942838744],[1,482522,0.0683507105],[1,482528,0.042224113],[1,482605,0.0564404524],[1,482643,0.0926079804],[1,482659,0.0426613512],[1,482669,0.0378277847],[1,482678,0.0389758584],[1,482752,0.2899587279],[1,482893,0.0402454652],[1,482912,0.1045851423],[1,482926,0.1372887733],[1,482937,0.0426613512],[1,482940,0.0412391502],[1,482962,0.0922509737],[1,483008,0.4014341562],[1,483042,0.0402454652],[1,483067,0.0590678963],[1,483074,0.0421992672],[1,483097,0.0383407065],[1,483282,0.0859075782],[1,483321,0.0712187688],[1,483323,0.1425113454],[1,483367,0.1156669818],[1,483428,0.0394676867],[1,483429,0.0637215846],[1,483431,0.0399428574],[1,483434,0.7213502459],[1,483446,0.1881421779],[1,483476,0.05206433],[1,483637,0.0922530461],[1,483643,0.3061342248],[1,483656,0.2066205104],[1,483666,0.1046170852],[1,483728,0.0402860161],[1,483755,0.037839546],[1,483787,0.0945714264],[1,483856,0.0612268432],[1,483984,0.0961916459],[1,483985,0.1154057137],[1,483987,0.0389758584],[1,484002,0.0922530461],[1,484030,0.1593978829],[1,484046,0.0945535672],[1,484050,0.4523220999],[1,484080,0.085335226],[1,484121,0.3113039863],[1,484276,0.0453345796],[1,484278,0.0749180835],[1,484280,0.1454877237],[1,484358,0.0417839798],[1,484362,0.0756073073],[1,484379,0.0964584126],[1,484532,0.0453150386],[1,484553,0.038195929],[1,484585,0.0639045799],[1,484624,0.0451986977],[1,484639,0.0537918105],[1,484675,0.1102185242],[1,484682,0.1937805122],[1,484705,0.4540329954],[1,484710,0.0886090553],[1,484731,0.0590495704],[1,484834,0.3309677639],[1,484875,0.0525821088],[1,484922,0.1669703227],[1,485002,0.1233119893],[1,485092,0.050385853],[1,485172,0.0393111597],[1,485175,0.062123561],[1,485205,0.0723848288],[1,485217,0.0377157594],[1,485245,0.0402454652],[1,485291,0.0731688866],[1,485336,0.106742874],[1,485352,0.3578448484],[1,485379,0.0634206929],[1,485442,0.4979050742],[1,485527,0.0926079804],[1,485573,0.0457928315],[1,485630,0.0389391327],[1,485655,0.0999627873],[1,485752,0.0491749903],[1,485753,0.3680045311],[1,485770,0.044343142],[1,485771,0.5756330769],[1,485819,0.0688442386],[1,485858,0.6320159888],[1,485865,0.0710048961],[1,485907,0.0608699713],[1,485930,0.1389596867],[1,486011,0.0873843869],[1,486013,0.0399428574],[1,486018,0.1399141407],[1,486068,0.0736100564],[1,486090,0.0381992682],[1,486093,0.3028097719],[1,486127,0.0621596208],[1,486149,0.0982514163],[1,486154,0.0956300374],[1,486168,0.0650058672],[1,486194,0.0509278522],[1,486215,0.0607347976],[1,486239,0.3110914596],[1,486246,0.0906258616],[1,486251,0.0558659921],[1,486254,0.0389758584],[1,486260,0.2785826352],[1,486296,0.0390458899],[1,486310,0.2509850243],[1,486318,0.0559214026],[1,486319,0.0746225911],[1,486330,0.0433051644],[1,486333,0.0379722004],[1,486386,0.1944518358],[1,486414,0.0944951831],[1,486419,0.0425329007],[1,486430,0.0945535672],[1,486443,0.0376467471],[1,486500,0.0751169281],[1,486509,0.074470114],[1,486522,0.1353719797],[1,486553,0.0544282187],[1,486567,0.0922530461],[1,486685,0.0896190967],[1,486695,0.038195929],[1,486696,0.0504359167],[1,486706,0.0706004088],[1,486779,0.5871926819],[1,486802,0.0933175939],[1,486819,0.5964771677],[1,486855,0.0434784582],[1,487080,0.0466538882],[1,487152,0.0442488526],[1,487156,0.041521385],[1,487180,0.1342878443],[1,487288,0.2317136097],[1,487294,0.3266801162],[1,487348,0.0502314762],[1,487349,0.1436950413],[1,487393,0.0407905631],[1,487516,0.2928177454],[1,487599,0.0568309586],[1,487649,0.0939235407],[1,487705,0.0356890311],[1,487738,0.0478773591],[1,487741,0.045233315],[1,487758,0.0800185848],[1,487771,0.1407955341],[1,487810,0.1474566124],[1,487816,0.0371149137],[1,487853,0.4984666186],[1,487960,0.0935532352],[1,488021,0.0653079274],[1,488033,0.528092499],[1,488038,0.0436513985],[1,488045,0.064278795],[1,488117,0.0625162869],[1,488260,0.050281518],[1,488273,0.0567586661],[1,488323,0.047162161],[1,488342,0.0891851294],[1,488403,0.062245827],[1,488414,0.0732512149],[1,488535,0.1519562256],[1,488542,0.0397173847],[1,488603,0.0405439582],[1,488639,0.0346303669],[1,488649,0.1384128454],[1,488666,0.0335753192],[1,488708,0.0401909287],[1,488723,0.0877893587],[1,488740,0.0852231766],[1,488745,0.0554035988],[1,488853,0.037610919],[1,488902,0.0598580878],[1,488928,0.0349277415],[1,488946,0.0329577349],[1,488967,0.036595081],[1,489020,0.0531448756],[1,489110,0.0419793112],[1,489111,0.1193320231],[1,489117,0.6938135741],[1,489126,0.0432044907],[1,489132,0.0325322899],[1,489167,0.0608203642],[1,489252,0.250918423],[1,489330,0.0852231766],[1,489341,0.0301821219],[1,489356,0.0558042184],[1,489391,0.0994685353],[1,489475,0.0962602225],[1,489486,0.2671880862],[1,489523,0.1462884476],[1,489636,0.0460912682],[1,489656,0.0684769242],[1,489667,0.041731],[1,489687,0.2839460408],[1,489776,0.2425274017],[1,489788,0.0549666001],[1,489799,0.0370122374],[1,489888,0.0402496412],[1,489917,0.1977103279],[1,489951,0.1430564832],[1,489984,0.0335814372],[1,489986,0.0666441535],[1,490004,0.1922487068],[1,490006,0.0450300304],[1,490022,0.0646614809],[1,490037,0.0342177778],[1,490236,0.1355588656],[1,490248,0.0385377947],[1,490410,0.0314253138],[1,490425,0.0328852817],[1,490430,0.0757826001],[1,490493,0.0333354537],[1,490514,0.0852231766],[1,490536,0.0333354537],[1,490667,0.0324310387],[1,490703,0.0348966273],[1,490710,0.0419621108],[1,490716,0.077411581],[1,490745,0.0848714606],[1,490783,0.0828956645],[1,490817,0.0741739441],[1,490823,0.0857952249],[1,490832,0.0314131469],[1,490958,0.0432008379],[1,491086,0.403081605],[1,491147,0.0411263546],[1,491208,0.0870665996],[1,491231,0.1449407049],[1,491291,0.2057187779],[1,491358,0.4642155555],[1,491396,0.116806012],[1,491471,0.1074954177],[1,491501,0.0441124132],[1,491607,0.0974464573],[1,491698,0.1032737816],[1,491701,0.0778118545],[1,491765,0.1570446444],[1,491781,0.0814143032],[1,491783,0.0355768162],[1,491891,0.0403187627],[1,491895,0.0302159496],[1,491961,0.0584258453],[1,491967,0.0342595337],[1,491968,0.687916885],[1,491986,0.0313045014],[1,492001,0.0857727258],[1,492020,0.0379167211],[1,492037,0.0355768162],[1,492070,0.0598923877],[1,492096,0.0323264908],[1,492156,0.0313408586],[1,492175,0.1384312526],[1,492198,0.0314253138],[1,492283,0.0348106532],[1,492313,0.0333354537],[1,492323,0.0852231766],[1,492333,0.0333354537],[1,492350,0.3957365852],[1,492386,0.0574749864],[1,492430,0.0852231766],[1,492511,0.05327838],[1,492512,0.0947391584],[1,492557,0.4851601099],[1,492591,0.035626718],[1,492602,0.1052415836],[1,492635,0.0324310387],[1,492910,0.047135605],[1,492926,0.2866710712],[1,492944,0.0855993713],[1,492994,0.1343077658],[1,493054,0.0564595043],[1,493086,0.144606093],[1,493087,0.0693062335],[1,493107,0.0353608043],[1,493254,0.0670329654],[1,493255,0.0569099434],[1,493352,0.0333354537],[1,493479,0.1304314293],[1,493510,0.1749769362],[1,493535,0.0334551908],[1,493537,0.0307602992],[1,493569,0.1683452267],[1,493610,0.0367010625],[1,493676,0.0860167443],[1,493678,0.0735769391],[1,493682,0.0362046263],[1,493683,0.0351015613],[1,493686,0.0749453575],[1,493745,0.0371316487],[1,493751,0.0371316487],[1,493762,0.1569792259],[1,493774,0.0322055674],[1,493777,0.2049300219],[1,493797,0.6039591468],[1,493908,0.2577614706],[1,493911,0.0415057628],[1,493944,0.0860167443],[1,493966,0.0526795113],[1,493976,0.0760562633],[1,493995,0.0428414306],[1,494019,0.03867717],[1,494036,0.1119589006],[1,494045,0.1848691365],[1,494086,0.1003437037],[1,494098,0.2473573013],[1,494099,0.0759739553],[1,494112,0.1379094681],[1,494143,0.1220045683],[1,494223,0.6465453553],[1,494333,0.1936819062],[1,494401,0.0863306557],[1,494429,0.3572757104],[1,494459,0.0956919365],[1,494476,0.1123845926],[1,494482,0.0418361683],[1,494495,0.1182595511],[1,494514,0.0878886739],[1,494564,0.1043805753],[1,494574,0.0475562307],[1,494575,0.0479669299],[1,494586,0.0330658593],[1,494589,0.3956358326],[1,494591,0.0393870257],[1,494606,0.1602445022],[1,494661,0.0608733288],[1,494694,0.0549190675],[1,494735,0.1190688277],[1,494750,0.2239855365],[1,494800,0.0676831473],[1,494801,0.03271416],[1,494823,0.0855568181],[1,494828,0.0320474432],[1,494830,0.0375932129],[1,494837,0.1545731166],[1,494842,0.1473626708],[1,494910,0.2509802727],[1,494913,0.0496551671],[1,494924,0.0422339731],[1,494930,0.0815568969],[1,495028,0.4841851613],[1,495031,0.3662162728],[1,495043,0.144092682],[1,495066,0.1088908439],[1,495106,0.0542587581],[1,495108,0.0879465855],[1,495168,0.0311157132],[1,495196,0.1332206192],[1,495223,0.0851185302],[1,495262,0.0333597033],[1,495282,0.1686736578],[1,495342,0.0633688393],[1,495348,0.4659477979],[1,495392,0.03867717],[1,495397,0.0446806795],[1,495439,0.5515391409],[1,495459,0.1330915654],[1,495465,0.4922414705],[1,495493,0.3138408758],[1,495510,0.03271416],[1,495537,0.2104273975],[1,495584,0.4318021837],[1,495616,0.0371004438],[1,495640,0.0389485269],[1,495642,0.5330438358],[1,495674,0.0352193721],[1,495681,0.0900765884],[1,495694,0.0687202352],[1,495728,0.1370469058],[1,495748,0.0618631989],[1,495927,0.0603646194],[1,495934,0.2524708608],[1,495962,0.0944901888],[1,495998,0.2166294826],[1,496005,0.0330658593],[1,496008,0.0931255109],[1,496017,0.1713718999],[1,496030,0.0624414447],[1,496086,0.3249148871],[1,496135,0.1051788023],[1,496171,0.2591985522],[1,496245,0.0335379468],[1,496261,0.0598579449],[1,496262,0.2547304074],[1,496283,0.0455693968],[1,496319,0.1834829126],[1,496361,0.0334577422],[1,496371,0.226227841],[1,496417,0.0828383101],[1,496428,0.0640784142],[1,496456,0.1290108422],[1,496508,0.0381627981],[1,496557,0.0889850992],[1,496559,0.0347735272],[1,496579,0.2603243568],[1,496589,0.0791368598],[1,496600,0.0541884999],[1,496620,0.0861301078],[1,496639,0.069617586],[1,496664,0.1424294961],[1,496672,0.0331717922],[1,496729,0.0855849521],[1,496763,0.0306064555],[1,496777,0.0417827856],[1,496814,0.0331717922],[1,496921,0.086854882],[1,496980,0.5183732106],[1,496987,0.0332082395],[1,497013,0.1756761428],[1,497020,0.0397128562],[1,497106,0.0330092504],[1,497162,0.0850504269],[1,497190,0.5629724293],[1,497394,0.0357450906],[1,497420,0.1131977976],[1,497428,0.0454190043],[1,497444,0.1851269633],[1,497460,0.0618519702],[1,497660,0.1037231259],[1,497671,0.0427795506],[1,497729,0.1010816407],[1,497746,0.0372547523],[1,497846,0.4877789386],[1,497873,0.0831022661],[1,497885,0.1516022677],[1,497925,0.1712493234],[1,497950,0.0366633743],[1,497957,0.055842619],[1,497968,0.0329978699],[1,497983,0.1597872242],[1,497990,0.0679263777],[1,498036,0.0439344216],[1,498115,0.1594320245],[1,498151,0.3361501093],[1,498211,0.1711677149],[1,498236,0.243184864],[1,498277,0.0401384301],[1,498304,0.1243261872],[1,498308,0.0360785655],[1,498319,0.0363213952],[1,498365,0.1256436521],[1,498375,0.1724242736],[1,498405,0.0904846458],[1,498411,0.0993099544],[1,498476,0.0981223233],[1,498504,0.0381959429],[1,498522,0.0553087163],[1,498542,0.0600051365],[1,498552,0.0820843032],[1,498601,0.0890719335],[1,498657,0.0735071799],[1,498703,0.0787125042],[1,498751,0.0608258545],[1,498808,0.0341232762],[1,498819,0.0351806644],[1,498835,0.2247797818],[1,498836,0.1319594133],[1,498865,0.0371601075],[1,498866,0.0363213952],[1,498881,0.1302992984],[1,498894,0.0773720668],[1,498960,0.0315205238],[1,498962,0.0339301465],[1,499000,0.0366986792],[1,499013,0.0584427534],[1,499046,0.0365600442],[1,499049,0.0515078034],[1,499062,0.1834348012],[1,499072,0.0861535691],[1,499101,0.0337787257],[1,499113,0.0673575216],[1,499128,0.0469026269],[1,499193,0.1851473123],[1,499289,0.0734490163],[1,499352,0.0592177187],[1,499426,0.0908385066],[1,499449,0.0596626884],[1,499455,0.0381959429],[1,499487,0.0900938517],[1,499499,0.166348078],[1,499522,0.0596241065],[1,499528,0.0340481838],[1,499619,0.0765769157],[1,499691,0.3661176795],[1,499776,0.1158702169],[1,499790,0.0822811713],[1,499813,0.0383045886],[1,499817,0.2184070523],[1,499879,0.0380653908],[1,499916,0.0686716411],[1,500025,0.0480367723],[1,500145,0.1222992202],[1,500190,0.2711860421],[1,500213,0.1098860005],[1,500243,0.0376081548],[1,500312,0.0993254322],[1,500437,0.0386840692],[1,500454,0.0409652523],[1,500515,0.0363917531],[1,500523,0.2573725123],[1,500560,0.2772734604],[1,500578,0.1193637856],[1,500585,0.0682611203],[1,500646,0.0562335864],[1,500676,0.0394289441],[1,500715,0.0362890084],[1,500744,0.0346550973],[1,500802,0.0382301103],[1,500858,0.0585675708],[1,500880,0.1345664234],[1,500888,0.040848538],[1,500929,0.0899289416],[1,500995,0.1853619777],[1,501017,0.0447270388],[1,501039,0.0362890084],[1,501046,0.0521044819],[1,501059,0.0432056217],[1,501089,0.0390661822],[1,501111,0.1664011201],[1,501114,0.0368372085],[1,501156,0.0341990561],[1,501182,0.3103217594],[1,501189,0.0471684773],[1,501223,0.0495390379],[1,501296,0.4935776702],[1,501299,0.1225743774],[1,501319,0.0391040625],[1,501334,0.1465733133],[1,501351,0.0412680339],[1,501417,0.1126012248],[1,501448,0.3004713984],[1,501505,0.044799568],[1,501522,0.0986472476],[1,501752,0.0393099654],[1,501833,0.0564794964],[1,501904,0.5902389607],[1,501922,0.0586600306],[1,501928,0.5401491472],[1,501960,0.0428730616],[1,501974,0.0364599237],[1,501999,0.0419453968],[1,502006,0.096730803],[1,502062,0.1356454414],[1,502105,0.1576922248],[1,502126,0.039875223],[1,502177,0.1989971296],[1,502210,0.0491372521],[1,502271,0.0446917279],[1,502275,0.0624447625],[1,502335,0.0368372085],[1,502355,0.0368738327],[1,502361,0.0368372085],[1,502383,0.1095553948],[1,502385,0.0393455536],[1,502396,0.0362890084],[1,502411,0.0838315208],[1,502432,0.0361989422],[1,502447,0.0357406599],[1,502565,0.0526727528],[1,502573,0.298669569],[1,502590,0.055221643],[1,502618,0.0425384167],[1,502619,0.1399487212],[1,502734,0.1458201224],[1,502737,0.2657277949],[1,502872,0.0346534861],[1,502919,0.3372572748],[1,502923,0.0835010529],[1,502954,0.0550908165],[1,502967,0.0843406862],[1,503048,0.0384894311],[1,503054,0.0619594523],[1,503082,0.0350833198],[1,503134,0.041522849],[1,503159,0.0387840132],[1,503165,0.0382301103],[1,503232,0.3351713089],[1,503337,0.0386877207],[1,503360,0.2243688725],[1,503433,0.6472888024],[1,503482,0.0394201691],[1,503499,0.0732832324],[1,503621,0.0416643161],[1,503622,0.0396134736],[1,503624,0.1703761407],[1,503634,0.0868928081],[1,503687,0.0405681192],[1,503713,0.0354569582],[1,503733,0.1112659547],[1,503769,0.0607650197],[1,503827,0.0923976554],[1,503830,0.130154729],[1,503875,0.0914416804],[1,503877,0.0528292509],[1,503925,0.0910477221],[1,503929,0.058947981],[1,503946,0.0575364789],[1,503955,0.3355633176],[1,503956,0.0362890084],[1,503958,0.0457784455],[1,504055,0.0378483525],[1,504056,0.1587649184],[1,504063,0.0356624116],[1,504080,0.1045738937],[1,504092,0.0798071244],[1,504106,0.4044013983],[1,504140,0.3139065738],[1,504225,0.060693923],[1,504243,0.0615706544],[1,504248,0.4385825205],[1,504253,0.0926338145],[1,504277,0.0375148388],[1,504296,0.1467328931],[1,504312,0.0388243311],[1,504335,0.0368372085],[1,504339,0.0712263177],[1,504342,0.3780486454],[1,504361,0.3684456717],[1,504362,0.0587341892],[1,504425,0.0530740035],[1,504430,0.0368372085],[1,504474,0.0362890084],[1,504477,0.2166746563],[1,504497,0.4332124292],[1,504511,0.30208808],[1,504555,0.034267342],[1,504627,0.1315158821],[1,504634,0.0456682643],[1,504641,0.0593360848],[1,504686,0.1153589258],[1,504688,0.1378696707],[1,504721,0.1795528687],[1,504728,0.182984486],[1,504735,0.057505587],[1,504758,0.0496301172],[1,504762,0.1456744438],[1,504774,0.275115583],[1,504780,0.1204010439],[1,504800,0.4373094126],[1,504807,0.046577624],[1,504830,0.0614121902],[1,504831,0.0338269845],[1,504906,0.2959228277],[1,504939,0.036079055],[1,504960,0.1086148952],[1,504999,0.2451461453],[1,505005,0.0385810138],[1,505009,0.1332692664],[1,505052,0.039466178],[1,505079,0.1447865397],[1,505150,0.3833794995],[1,505168,0.4906282983],[1,505185,0.0591754115],[1,505221,0.0895751748],[1,505259,0.0936548132],[1,505280,0.2763767532],[1,505288,0.1215018989],[1,505340,0.2592069052],[1,505420,0.0666001874],[1,505456,0.0926679388],[1,505478,0.0828354202],[1,505514,0.0380652823],[1,505536,0.0915351493],[1,505573,0.0338718648],[1,505594,0.1416292135],[1,505600,0.0456982269],[1,505606,0.0887525305],[1,505621,0.0615874303],[1,505630,0.4082126799],[1,505640,0.113601295],[1,505692,0.0378483525],[1,505699,0.0628421267],[1,505713,0.0404199912],[1,505734,0.037916669],[1,505765,0.2861365171],[1,505814,0.1014729397],[1,505818,0.036079055],[1,505839,0.0589760889],[1,505889,0.0779666761],[1,505943,0.5609124392],[1,505950,0.332360443],[1,505965,0.2365734037],[1,505981,0.0887525305],[1,505990,0.0355203887],[1,506000,0.1383577963],[1,506008,0.349298946],[1,506017,0.7832524941],[1,506058,0.0891059386],[1,506085,0.1175356347],[1,506111,0.0445184921],[1,506115,0.0902073566],[1,506117,0.0385645917],[1,506130,0.0368716325],[1,506149,0.7924452936],[1,506153,0.0448589097],[1,506161,0.0478397749],[1,506267,0.1426389158],[1,506389,0.0591587724],[1,506462,0.0497670531],[1,506476,0.036301949],[1,506481,0.0980036756],[1,506484,0.0465118708],[1,506513,0.0530740035],[1,506526,0.1321295066],[1,506609,0.1496186012],[1,506625,0.0366235894],[1,506675,0.037916669],[1,506692,0.0400665971],[1,506719,0.2211890564],[1,506726,0.0355203887],[1,506802,0.069371919],[1,506818,0.036068033],[1,506871,0.0486698931],[1,506872,0.0360577324],[1,506926,0.0414259922],[1,506931,0.0891059386],[1,506947,0.0393215273],[1,506988,0.039466178],[1,507064,0.0804603141],[1,507068,0.0884832555],[1,507080,0.1702515941],[1,507086,0.0819631907],[1,507090,0.1332152086],[1,507100,0.1120906003],[1,507113,0.1080931306],[1,507142,0.0699427712],[1,507150,0.0646687676],[1,507165,0.1568371006],[1,507219,0.0406751049],[1,507346,0.329512187],[1,507351,0.0394289441],[1,507367,0.0857385621],[1,507370,0.1495248529],[1,507371,0.0594587057],[1,507387,0.0386478967],[1,507406,0.1082920847],[1,507430,0.0447095978],[1,507433,0.0374607952],[1,507486,0.0350833198],[1,507506,0.3719620075],[1,507509,0.2003183571],[1,507510,0.0463708319],[1,507512,0.0415372506],[1,507552,0.0523333443],[1,507570,0.0424089365],[1,507572,0.0398114962],[1,507586,0.0389746053],[1,507612,0.0376138542],[1,507624,0.0533207126],[1,507660,0.036068033],[1,507667,0.055964527],[1,507674,0.0400717714],[1,507819,0.036068033],[1,507825,0.3792465714],[1,507867,0.0887525305],[1,507935,0.1529148754],[1,507967,0.036068033],[1,508013,0.2524692488],[1,508084,0.0364399046],[1,508117,0.102689053],[1,508148,0.0662871146],[1,508236,0.0703953079],[1,508250,0.2405938646],[1,508253,0.034267342],[1,508449,0.1809249831],[1,508543,0.0425459953],[1,508547,0.036068033],[1,508554,0.0486698931],[1,508556,0.1469403028],[1,508615,0.0583097643],[1,508639,0.137122542],[1,508669,0.1975389902],[1,508752,0.1444218006],[1,508764,0.0746322554],[1,508780,0.0671341816],[1,508800,0.3916365582],[1,508824,0.059900859],[1,508940,0.0463179364],[1,508963,0.1164646839],[1,508990,0.0990385417],[1,508998,0.037916669],[1,509048,0.0461304033],[1,509120,0.0431840312],[1,509146,0.038890238],[1,509155,0.0654902742],[1,509191,0.0393108753],[1,509194,0.0394403191],[1,509248,0.1570768184],[1,509292,0.041062548],[1,509297,0.0383502819],[1,509301,0.0381496201],[1,509338,0.0453740298],[1,509343,0.037120726],[1,509361,0.1481144107],[1,509372,0.1650387262],[1,509383,0.0610504007],[1,509385,0.0371354083],[1,509406,0.0845469919],[1,509407,0.0692642829],[1,509411,0.0365973006],[1,509466,0.0365973006],[1,509523,0.0583931332],[1,509586,0.0595652762],[1,509616,0.0397091988],[1,509625,0.0433574825],[1,509628,0.0614235332],[1,509751,0.3785536683],[1,509784,0.0389969883],[1,509839,0.0998674792],[1,509844,0.0423046751],[1,509854,0.0456081989],[1,509861,0.1056617433],[1,509940,0.0612795226],[1,509976,0.0506174265],[1,510027,0.0367317068],[1,510033,0.0359423924],[1,510055,0.0393266981],[1,510073,0.0653736953],[1,510100,0.1461749862],[1,510185,0.0899051361],[1,510244,0.0400877402],[1,510256,0.1234969419],[1,510320,0.0902590467],[1,510325,0.0452799334],[1,510336,0.0474563875],[1,510360,0.1116083332],[1,510365,0.0358558356],[1,510367,0.0619722166],[1,510373,0.0371902545],[1,510386,0.0567740372],[1,510423,0.036877242],[1,510458,0.095489506],[1,510469,0.0819707452],[1,510538,0.1324657193],[1,510570,0.0344842401],[1,510607,0.0389129508],[1,510610,0.0353096968],[1,510625,0.1967130262],[1,510648,0.0421916632],[1,510666,0.0416762564],[1,510670,0.4767500045],[1,510699,0.2558735099],[1,510728,0.0383843172],[1,510740,0.6345694454],[1,510754,0.0455292627],[1,510756,0.277931205],[1,510805,0.0350511575],[1,510838,0.1895692649],[1,510913,0.0803348304],[1,510918,0.0744802765],[1,510924,0.0573444256],[1,510940,0.0369926681],[1,510943,0.0405396779],[1,510946,0.0502760307],[1,510997,0.0344842401],[1,511004,0.0829323821],[1,511092,0.1912415811],[1,511098,0.0392577824],[1,511116,0.3453941839],[1,511125,0.0695026316],[1,511148,0.0921696619],[1,511151,0.1626770408],[1,511186,0.0919132964],[1,511207,0.1485777199],[1,511219,0.1888838042],[1,511268,0.0821931069],[1,511283,0.0328379716],[1,511306,0.0360814729],[1,511317,0.1095565023],[1,511320,0.167652477],[1,511342,0.1374428355],[1,511354,0.0483480702],[1,511360,0.0467121314],[1,511393,0.0478673089],[1,511403,0.0919499406],[1,511407,0.2115245819],[1,511441,0.0376403998],[1,511472,0.6477809471],[1,511531,0.0367707892],[1,511553,0.0344842401],[1,511560,0.0370389944],[1,511573,0.0609479466],[1,511584,0.0650677484],[1,511588,0.0465731379],[1,511593,0.0350311351],[1,511615,0.0944946609],[1,511621,0.0409291177],[1,511629,0.0980831357],[1,511634,0.1671596337],[1,511650,0.0619917101],[1,511659,0.1676477718],[1,511674,0.0344240099],[1,511686,0.0358358325],[1,511821,0.036877242],[1,511824,0.0387566163],[1,511831,0.037796342],[1,511836,0.0344240099],[1,511845,0.037958827],[1,511851,0.0328379716],[1,511859,0.1601841521],[1,511880,0.0393514291],[1,511895,0.1070989522],[1,511902,0.0719136124],[1,511930,0.036877242],[1,511940,0.0602869708],[1,512103,0.0432507706],[1,512132,0.0957616227],[1,512143,0.0440442503],[1,512145,0.0526790457],[1,512150,0.0485808717],[1,512185,0.1350724781],[1,512208,0.0574377272],[1,512212,0.3357423841],[1,512260,0.0656305358],[1,512346,0.0441888362],[1,512370,0.0601242562],[1,512428,0.0894142063],[1,512513,0.4812960052],[1,512529,0.0369532285],[1,512652,0.4250103648],[1,512667,0.036991424],[1,512669,0.1474689834],[1,512672,0.2427809958],[1,512674,0.2800024676],[1,512679,0.0606807958],[1,512689,0.0460952676],[1,512699,0.0350311351],[1,512788,0.0350311351],[1,512790,0.1102645353],[1,512809,0.0460555637],[1,512914,0.1094362782],[1,512940,0.2411677685],[1,512979,0.0573661354],[1,513028,0.0397902083],[1,513031,0.0373193376],[1,513041,0.0467455951],[1,513096,0.0513711768],[1,513107,0.0336650006],[1,513123,0.0613939317],[1,513169,0.2635835373],[1,513250,0.5368408588],[1,513268,0.0425686487],[1,513281,0.0605986855],[1,513301,0.0405824917],[1,513316,0.1042594619],[1,513318,0.1731631834],[1,513331,0.1895693402],[1,513346,0.0898468238],[1,513431,0.0600751019],[1,513463,0.0383843172],[1,513469,0.0370389944],[1,513480,0.0336650006],[1,513488,0.0550653297],[1,513588,0.0335547086],[1,513603,0.0344842401],[1,513640,0.0387264046],[1,513656,0.0387566163],[1,513681,0.0630095351],[1,513718,0.0415441951],[1,513792,0.1722696116],[1,513812,0.2738588875],[1,513847,0.1211573799],[1,513862,0.1197008382],[1,514075,0.3324524749],[1,514202,0.0611243583],[1,514223,0.0538213392],[1,514278,0.0483308925],[1,514290,0.0700503283],[1,514300,0.0394447013],[1,514452,0.0365606139],[1,514469,0.4125961008],[1,514517,0.0482480998],[1,514532,0.0474615191],[1,514541,0.0983727866],[1,514588,0.0465547648],[1,514663,0.036004541],[1,514702,0.0390061693],[1,514705,0.0946814952],[1,514759,0.0606015769],[1,514761,0.0480363965],[1,514808,0.2397657278],[1,514845,0.1483792843],[1,514849,0.0394447013],[1,514899,0.036004541],[1,514953,0.3384556677],[1,514973,0.0750419193],[1,514983,0.0825743102],[1,514998,0.0611435079],[1,515022,0.0571142641],[1,515055,0.0626025084],[1,515056,0.0529278339],[1,515060,0.039880349],[1,515144,0.0987804865],[1,515205,0.0386893924],[1,515230,0.2562759124],[1,515282,0.0425686487],[1,515295,0.092360478],[1,515301,0.0422461466],[1,515316,0.0958823285],[1,515329,0.0526045295],[1,515406,0.0554939583],[1,515408,0.0905987603],[1,515428,0.0621479353],[1,515453,0.2850908648],[1,515517,0.0575967486],[1,515567,0.2179097734],[1,515575,0.0384023533],[1,515664,0.0763921186],[1,515681,0.0840927513],[1,515687,0.0460151835],[1,515759,0.0755803123],[1,515772,0.1401971736],[1,515775,0.0585736634],[1,515788,0.1054318452],[1,515823,0.0587247361],[1,515831,0.0384023533],[1,515841,0.1171898841],[1,515944,0.0629311425],[1,515952,0.0373197787],[1,516035,0.0561386249],[1,516054,0.036004541],[1,516063,0.0567313062],[1,516081,0.096870117],[1,516083,0.0438795425],[1,516085,0.0394447013],[1,516138,0.2781970518],[1,516165,0.0382956864],[1,516198,0.0560896686],[1,516199,0.1185496944],[1,516242,0.0406592554],[1,516243,0.2168667888],[1,516292,0.1105303105],[1,516346,0.091806446],[1,516353,0.605685986],[1,516390,0.0373197787],[1,516409,0.2102435176],[1,516420,0.0889423742],[1,516465,0.0619576332],[1,516483,0.03355128],[1,516489,0.3400556124],[1,516512,0.0702893701],[1,516536,0.0405495267],[1,516543,0.0858146326],[1,516554,0.1418857528],[1,516566,0.2877563015],[1,516589,0.0623719861],[1,516607,0.0559389391],[1,516638,0.1050354512],[1,516679,0.0558992166],[1,516749,0.0719183945],[1,516751,0.0710490841],[1,516770,0.0859426972],[1,516778,0.03944055],[1,516791,0.3056766173],[1,516853,0.2016567083],[1,516859,0.0551927974],[1,516880,0.1144873682],[1,516933,0.0326935266],[1,516976,0.0615791752],[1,516985,0.1271373897],[1,517058,0.034642111],[1,517079,0.1151703717],[1,517082,0.0499609114],[1,517129,0.0369291374],[1,517158,0.0314812345],[1,517189,0.0412632566],[1,517196,0.9137005441],[1,517261,0.0463365498],[1,517264,0.0436781362],[1,517276,0.0326935266],[1,517279,0.0523809995],[1,517311,0.1135699039],[1,517319,0.0455640139],[1,517377,0.0335035187],[1,517413,0.0366728807],[1,517442,0.1467552441],[1,517446,0.105729385],[1,517468,0.0807102128],[1,517529,0.0353381601],[1,517563,0.0899897476],[1,517583,0.1117659837],[1,517605,0.0404268391],[1,517638,0.0356721601],[1,517642,0.0877179413],[1,517706,0.034331277],[1,517707,0.1331216196],[1,517727,0.165815337],[1,517741,0.1914746416],[1,517743,0.0617768533],[1,517831,0.223178587],[1,517834,0.2960365515],[1,517861,0.0915383508],[1,517871,0.061714437],[1,517872,0.0640358226],[1,517885,0.0395520322],[1,517887,0.1444400744],[1,517889,0.2074338356],[1,517894,0.0365066406],[1,517966,0.1075122861],[1,517968,0.0392350281],[1,517975,0.0798449861],[1,518015,0.1958051746],[1,518023,0.0455640139],[1,518027,0.1722605349],[1,518028,0.8623442304],[1,518045,0.0873060103],[1,518072,0.1582368574],[1,518093,0.0457028869],[1,518130,0.1000836111],[1,518142,0.2224029219],[1,518144,0.2804414092],[1,518151,0.0362479329],[1,518154,0.0617347661],[1,518172,0.0642924706],[1,518179,0.0390728076],[1,518185,0.2007582287],[1,518203,0.0366728807],[1,518207,0.0404989722],[1,518208,0.0385475389],[1,518213,0.4696186311],[1,518233,0.0355355305],[1,518242,0.0403655453],[1,518262,0.0342174561],[1,518263,0.1750120147],[1,518271,0.0395161825],[1,518280,0.1115950249],[1,518303,0.1102345126],[1,518337,0.0504048807],[1,518340,0.0409130295],[1,518408,0.1359679332],[1,518456,0.0927766136],[1,518465,0.0899897476],[1,518548,0.0377227287],[1,518593,0.098968471],[1,518600,0.2039597594],[1,518652,0.0406579233],[1,518677,0.0419917091],[1,518691,0.0916184621],[1,518705,0.0583140951],[1,518715,0.0914537452],[1,518728,0.041550312],[1,518742,0.0976913378],[1,518747,0.0355366413],[1,518753,0.1007633258],[1,518792,0.1042603009],[1,518794,0.3595382411],[1,518866,0.0780562298],[1,518905,0.0431499022],[1,518914,0.0448991273],[1,518922,0.062958996],[1,518935,0.0899897476],[1,518949,0.4049810802],[1,518970,0.1096557186],[1,519010,0.3519967034],[1,519061,0.0402566658],[1,519076,0.0638913026],[1,519091,0.0801461526],[1,519104,0.0357546251],[1,519105,0.1070874],[1,519110,0.1081191062],[1,519128,0.0494312513],[1,519226,0.1153922947],[1,519240,0.2393468064],[1,519257,0.0350992504],[1,519299,0.0854147977],[1,519306,0.1110386165],[1,519313,0.426954521],[1,519325,0.0438996842],[1,519330,0.0352916872],[1,519331,0.0937268981],[1,519346,0.3500039866],[1,519348,0.147009821],[1,519351,0.1671777859],[1,519405,0.0606595529],[1,519425,0.1805752574],[1,519456,0.085123929],[1,519457,0.5306884044],[1,519474,0.1088487437],[1,519487,0.0392183058],[1,519512,0.1570558236],[1,519527,0.1849797433],[1,519533,0.3281267496],[1,519538,0.0974085639],[1,519555,0.0389744879],[1,519556,0.0587892326],[1,519569,0.0401160749],[1,519633,0.1043836999],[1,519669,0.0502589152],[1,519677,0.0925031151],[1,519698,0.060897884],[1,519741,0.1251287433],[1,519744,0.7862227989],[1,519748,0.6195296434],[1,519758,0.5283186203],[1,519781,0.0764548881],[1,519822,0.6863262384],[1,519909,0.0619126198],[1,519920,0.1045616951],[1,519939,0.0401160749],[1,519975,0.0496985243],[1,520003,0.3241470965],[1,520004,0.1242576112],[1,520031,0.12501033],[1,520070,0.2211546113],[1,520095,0.1392657399],[1,520101,0.0395471817],[1,520121,0.0347190652],[1,520173,0.0575117015],[1,520190,0.0466455192],[1,520198,0.0813313428],[1,520199,0.1325559403],[1,520253,0.0423647386],[1,520273,0.0783491131],[1,520295,0.464228041],[1,520296,0.0497618132],[1,520298,0.0397822998],[1,520304,0.0338508089],[1,520327,0.1170722296],[1,520328,0.0584888677],[1,520349,0.2101555775],[1,520356,0.0344044859],[1,520360,0.1051570391],[1,520396,0.0492092031],[1,520399,0.0358023821],[1,520402,0.0816073301],[1,520434,0.041179673],[1,520455,0.1435525192],[1,520472,0.1167486235],[1,520482,0.4116250452],[1,520490,0.0524959862],[1,520502,0.0386350235],[1,520542,0.0420832684],[1,520582,0.1037189143],[1,520634,0.1096881074],[1,520635,0.0409289473],[1,520730,0.0562964135],[1,520731,0.0609021619],[1,520759,0.0423868107],[1,520774,0.0463109411],[1,520785,0.0475972569],[1,520794,0.0646188376],[1,520847,0.0378137763],[1,520980,0.0722713872],[1,520985,0.1528211864],[1,521053,0.1390825743],[1,521058,0.0816474812],[1,521066,0.1952513117],[1,521067,0.0394771367],[1,521099,0.1083872659],[1,521123,0.0497338842],[1,521136,0.0608205425],[1,521220,0.1658364604],[1,521248,0.0497338842],[1,521357,0.1809793687],[1,521365,0.0395333357],[1,521368,0.2639763031],[1,521369,0.5802621487],[1,521388,0.0532346312],[1,521408,0.0493016276],[1,521415,0.0733606947],[1,521442,0.0743348718],[1,521453,0.0550785714],[1,521471,0.0625777279],[1,521501,0.1966653738],[1,521513,0.0367845165],[1,521526,0.1049430856],[1,521529,0.0596841371],[1,521544,0.1849387573],[1,521555,0.048041543],[1,521557,0.0769160602],[1,521559,0.296087387],[1,521587,0.0782145488],[1,521596,0.1276570737],[1,521601,0.0339814706],[1,521615,0.0505888507],[1,521622,0.0446466938],[1,521630,0.5102747333],[1,521655,0.1538518926],[1,521660,0.2864370414],[1,521666,0.0702445129],[1,521673,0.0420832684],[1,521676,0.2177485463],[1,521677,0.2021633057],[1,521679,0.0938103335],[1,521688,0.0637478847],[1,521720,0.2540100085],[1,521753,0.0606214629],[1,521765,0.0390955883],[1,521812,0.039638448],[1,521852,0.1621833435],[1,521863,0.0408122289],[1,521881,0.0550042158],[1,521882,0.0362340015],[1,521884,0.3832169578],[1,521911,0.0472467088],[1,521921,0.0343080835],[1,521924,0.1267870548],[1,521928,0.0573843379],[1,521933,0.039164017],[1,521965,0.039164017],[1,521967,0.1565857409],[1,521968,0.0564371511],[1,521997,0.0652330133],[1,522002,0.03619801],[1,522028,0.0368812799],[1,522047,0.2923072624],[1,522059,0.03392417],[1,522075,0.0816801606],[1,522100,0.2223549397],[1,522109,0.0610012072],[1,522141,0.0596244742],[1,522142,0.050493962],[1,522163,0.2122150143],[1,522175,0.1338030396],[1,522180,0.045245333],[1,522189,0.2142931012],[1,522210,0.1969451049],[1,522220,0.0474194408],[1,522251,0.035836844],[1,522293,0.0384612798],[1,522322,0.0496893102],[1,522324,0.0370263591],[1,522396,0.1524279321],[1,522409,0.0597465656],[1,522428,0.0387162017],[1,522461,0.1640258302],[1,522536,0.037194294],[1,522543,0.4535544379],[1,522560,0.0899544836],[1,522569,0.1400874281],[1,522608,0.1599468248],[1,522616,0.054807563],[1,522619,0.1315091539],[1,522643,0.1123556523],[1,522651,0.0350010154],[1,522653,0.0482928705],[1,522699,0.3891706593],[1,522702,0.0706343282],[1,522719,0.1101113534],[1,522727,0.0325400771],[1,522740,0.0364434075],[1,522743,0.0894870524],[1,522765,0.0933687105],[1,522794,0.0349330433],[1,522800,0.0495729132],[1,522808,0.056432137],[1,522834,0.0939140059],[1,522837,0.0336346814],[1,522899,0.0338351783],[1,522921,0.0350717585],[1,522956,0.2218183758],[1,522992,0.4701841193],[1,523025,0.1007717086],[1,523029,0.124413712],[1,523036,0.109994565],[1,523037,0.0353044886],[1,523041,0.0348329373],[1,523066,0.5929199525],[1,523079,0.2969347612],[1,523096,0.4802882674],[1,523114,0.1115953022],[1,523115,0.0370430332],[1,523140,0.1222536374],[1,523143,0.0381821185],[1,523230,0.0432687955],[1,523244,0.5574107026],[1,523248,0.0374838878],[1,523270,0.0678015796],[1,523272,0.1601029235],[1,523332,0.1025880839],[1,523349,0.0313280316],[1,523384,0.0345030655],[1,523405,0.0452242908],[1,523407,0.0335726545],[1,523411,0.2156905867],[1,523414,0.0368812799],[1,523422,0.0885816421],[1,523445,0.0889349757],[1,523451,0.0482718427],[1,523452,0.1174704783],[1,523468,0.0331573889],[1,523487,0.4435884708],[1,523543,0.0481256694],[1,523548,0.095234395],[1,523566,0.2701730801],[1,523577,0.216482387],[1,523632,0.7312673034],[1,523693,0.0383563185],[1,523696,0.0358398648],[1,523735,0.0921925178],[1,523741,0.3822955854],[1,523744,0.0730680243],[1,523760,0.0632580814],[1,523781,0.0391345248],[1,523786,0.0389628022],[1,523825,0.1023000736],[1,523839,0.1127092428],[1,523859,0.0466251542],[1,523870,0.0402039213],[1,523883,0.0389628022],[1,523908,0.5273276459],[1,523928,0.0468598862],[1,523933,0.0433258089],[1,523951,0.0367748265],[1,523954,0.0467121814],[1,523981,0.0320399127],[1,524001,0.0313280316],[1,524035,0.0654558014],[1,524057,0.0343204044],[1,524067,0.1172420555],[1,524119,0.0379215425],[1,524149,0.2238136307],[1,524212,0.1011080616],[1,524221,0.1663385446],[1,524231,0.0740121451],[1,524233,0.0344882653],[1,524271,0.1066341227],[1,524285,0.0367748265],[1,524322,0.1095945319],[1,524340,0.1343654917],[1,524350,0.0395876327],[1,524369,0.056342291],[1,524375,0.0669946536],[1,524408,0.0768940656],[1,524442,0.074009537],[1,524456,0.4003718632],[1,524478,0.0374315312],[1,524501,0.144741069],[1,524538,0.4934310227],[1,524572,0.150054031],[1,524575,0.0634029023],[1,524578,0.1542375003],[1,524580,0.0521398798],[1,524596,0.043352452],[1,524625,0.0342610557],[1,524651,0.0807575684],[1,524722,0.2589715632],[1,524723,0.1285320124],[1,524824,0.0401565561],[1,524830,0.0810628494],[1,524883,0.0394271682],[1,524936,0.0394271682],[1,524944,0.0925848538],[1,524984,0.043352452],[1,524987,0.0360581707],[1,524990,0.0323674307],[1,524993,0.0451837658],[1,524996,0.0366012109],[1,525011,0.0584502458],[1,525013,0.0366022972],[1,525018,0.037588809],[1,525057,0.0700747496],[1,525092,0.0411755191],[1,525099,0.093592308],[1,525122,0.0541647016],[1,525156,0.0362375217],[1,525160,0.12252315],[1,525174,0.1094700265],[1,525180,0.1642919174],[1,525224,0.0407176186],[1,525288,0.1369065061],[1,525320,0.1547465077],[1,525346,0.117850228],[1,525352,0.085748525],[1,525369,0.0543162821],[1,525399,0.0606262877],[1,525436,0.1075324711],[1,525437,0.0399157583],[1,525457,0.036415647],[1,525513,0.0371465565],[1,525514,0.038899329],[1,525559,0.0347527051],[1,525588,0.5630419285],[1,525593,0.0417903539],[1,525623,0.0971655281],[1,525627,0.0561104659],[1,525690,0.0526762132],[1,525701,0.0671712218],[1,525709,0.5297337052],[1,525710,0.5530154816],[1,525711,0.0319777495],[1,525725,0.0944192897],[1,525730,0.6183151559],[1,525758,0.0379963593],[1,525776,0.040933457],[1,525777,0.0616267293],[1,525790,0.3159820774],[1,525801,0.0379687135],[1,525803,0.0376197297],[1,525804,0.0558973911],[1,525846,0.1503650668],[1,525976,0.1996670135],[1,525978,0.4289578496],[1,526016,0.0620931934],[1,526019,0.0842500448],[1,526098,0.2089613383],[1,526122,0.0962482669],[1,526138,0.1180653597],[1,526168,0.0373083663],[1,526181,0.0495110803],[1,526280,0.0934113874],[1,526281,0.1720559652],[1,526304,0.1321631643],[1,526332,0.1580044251],[1,526388,0.0585223945],[1,526455,0.0377476644],[1,526476,0.090660655],[1,526478,0.0315003752],[1,526485,0.0897125325],[1,526496,0.1804871465],[1,526505,0.0534519768],[1,526587,0.1667132878],[1,526592,0.0720418763],[1,526609,0.0342720083],[1,526623,0.3691781575],[1,526644,0.0371465565],[1,526661,0.0550010945],[1,526662,0.1036205068],[1,526689,0.0366385992],[1,526690,0.6272023901],[1,526750,0.2225113658],[1,526832,0.1405719889],[1,526867,0.0318708018],[1,526880,0.0313267314],[1,526931,0.378014171],[1,526940,0.0347527051],[1,526952,0.0389821198],[1,527000,0.0905019944],[1,527058,0.2730337302],[1,527101,0.1316674974],[1,527147,0.1203478272],[1,527173,0.0621302909],[1,527176,0.4127107725],[1,527254,0.2772768935],[1,527278,0.0698970829],[1,527279,0.1277899325],[1,527308,0.1528236932],[1,527323,0.1183130723],[1,527324,0.063790952],[1,527327,0.3970642696],[1,527340,0.065111782],[1,527347,0.3177977769],[1,527348,0.3407043019],[1,527403,0.1381987331],[1,527441,0.0409119272],[1,527451,0.1319926141],[1,527488,0.1900792077],[1,527491,0.0381977801],[1,527493,0.0438021615],[1,527510,0.0376197297],[1,527521,0.0688940865],[1,527585,0.1026652883],[1,527603,0.3445562339],[1,527624,0.0445959304],[1,527653,0.0629013315],[1,527663,0.0367519707],[1,527698,0.1285219009],[1,527714,0.0606262877],[1,527717,0.0387740177],[1,527751,0.3147408556],[1,527784,0.0963759779],[1,527819,0.2529961005],[1,527833,0.0371465565],[1,527834,0.1000447251],[1,527855,0.0711119121],[1,527869,0.0422958361],[1,527871,0.0387886858],[1,527872,0.049921964],[1,527873,0.2718283408],[1,527875,0.0368710989],[1,527878,0.0622679484],[1,527886,0.0882915752],[1,527887,0.0928599956],[1,527889,0.0557069656],[1,527900,0.093851237],[1,527929,0.4542829327],[1,527930,0.0366022972],[1,527931,0.630542997],[1,527939,0.0965712991],[1,527966,0.0751808333],[1,527968,0.1135436762],[1,527969,0.1184941616],[1,527972,0.0874220447],[1,527973,0.040053215],[1,527974,0.0925049515],[1,527976,0.0779014463],[1,527977,0.0368710989],[1,527978,0.0368710989],[1,527982,0.0593245147],[1,527983,0.1263514835],[1,527986,0.066849287],[1,527988,0.0441453712],[1,527997,0.1265853649],[1,527999,0.0382872783],[1,528003,0.0361047773],[1,528008,0.0484431701],[1,528010,0.0534118265],[1,528013,0.0342720083],[1,528015,0.0500421252],[1,528016,0.0686590493],[1,528018,0.1386919077],[1,528020,0.0319329755],[1,528021,0.0373052738],[1,528022,0.0372523655],[1,528023,0.1355293913],[1,528024,0.1108489823],[1,528025,0.0459642829],[1,528027,0.3242084402],[1,528028,0.4882183707],[1,528031,0.0333596793],[1,528034,0.0368710989],[1,528036,0.0391160955],[1,528037,0.0372443664],[1,528039,0.2582409215],[1,528041,0.0668195809],[1,528042,0.0366176038],[1,528043,0.1212245838],[1,528044,0.0885828728],[1,528050,0.0391472234],[1,528051,0.0402316279],[1,528052,0.083264191],[1,528055,0.1713867862],[1,528056,0.042591883],[1,528057,0.1078502775],[1,528059,0.1672187095],[1,528062,0.0913247638],[1,528063,0.0373052738],[1,528065,0.3379820902],[1,528068,0.0443997156],[1,528069,0.241139693],[1,528071,0.0939076997],[1,528072,0.0830250833],[1,528073,0.0410477716],[1,528074,0.2010678607],[1,528075,0.1106614021],[1,528077,0.0499844644],[1,528078,0.1081689434],[1,528079,0.0700784659],[1,528082,0.2990629316],[1,528094,0.0668873499],[1,528095,0.147585162],[1,528101,0.1286860305],[1,528102,0.1516620415],[1,528103,0.1429817593],[1,528105,0.0347527051],[1,528107,0.0928599956],[1,528108,0.0355306576],[1,528112,0.2143942809],[1,528113,0.0645449888],[1,528115,0.049336757],[1,528116,0.2604705562],[1,528117,0.0822951098],[1,528122,0.0367433118],[1,528123,0.1091819108],[1,528124,0.063865081],[1,528128,0.1468167731],[1,528129,0.1190531835],[1,528132,0.115471751],[1,528133,0.0711982138],[1,528134,0.3770011932],[1,528136,0.0387387451],[1,528142,0.0422958361],[1,528143,0.0327464341],[1,528147,0.0389821198],[1,528148,0.0846946012],[1,528149,0.0460304692],[1,528151,0.0391434616],[1,528152,0.0511217146],[1,528153,0.2213000605],[1,528156,0.0798998323],[1,528157,0.1434307868],[1,528158,0.6095483336],[1,528160,0.116238074],[1,528161,0.1609431032],[1,528173,0.0617327386],[1,528180,0.1548343593],[1,528184,0.043472821],[1,528185,0.0333518092],[1,528186,0.03576316],[1,528187,0.0373052738],[1,528188,0.2846671511],[1,528189,0.0554231221],[1,528190,0.1740758721],[1,528199,0.0429028917],[1,528200,0.09943255],[1,528203,0.0319808479],[1,528204,0.1075984093],[1,528205,0.0688466177],[1,528210,0.0780886017],[1,528217,0.1007012261],[1,528218,0.0422253103],[1,528219,0.0541173091],[1,528221,0.2572160895],[1,528222,0.0319808479],[1,528223,0.0618031901],[1,528224,0.0920978734],[1,528225,0.0371465565],[1,528228,0.1583917788],[1,528232,0.1031759508],[1,528238,0.1264341088],[1,528239,0.0335739011],[1,528247,0.0412747978],[1,528248,0.2791002081],[1,528249,0.0305585786],[1,528250,0.1065759781],[1,528251,0.0673883709],[1,528252,0.0329366651],[1,528253,0.040053215],[1,528254,0.0509549775],[1,528255,0.0925049515],[1,528258,0.2172632513],[1,528262,0.0652455718],[1,528263,0.2530575465],[1,528267,0.4165910386],[1,528270,0.0372359856],[1,528271,0.0364340494],[1,528272,0.0501301114],[1,528273,0.0958023677],[1,528274,0.1023722261],[1,528275,0.0367519707],[1,528276,0.0605632783],[1,528277,0.0364340494],[1,528278,0.0925049515],[1,528280,0.0535076266],[1,528281,0.1955861097],[1,528282,0.04026288],[1,528286,0.0379963593],[1,528289,0.0443997156],[1,528290,0.0387387451],[1,528292,0.5130964831],[1,528293,0.0534325484],[1,528294,0.1311333343],[1,528295,0.1103150082],[1,528296,0.2958223034],[1,528297,0.070604212],[1,528298,0.0422958361],[1,528303,0.0367433118],[1,528309,0.0379963593],[1,528310,0.0333159546],[1,528311,0.153480876],[1,528313,0.0387387451],[1,528315,0.0599063031],[1,528316,0.0929031876],[1,528317,0.0845547477],[1,528318,0.0499309749],[1,528320,0.0697072728],[1,528321,0.0394005892],[1,528328,0.045073992],[1,528331,0.1625593446],[1,528332,0.1646973607],[1,528333,0.0513787229],[1,528334,0.1279508648],[1,528335,0.0467167473],[1,528336,0.0377599664],[1,528343,0.0378343717],[1,528344,0.0799738427],[1,528347,0.5719553497],[1,528348,0.0654524641],[1,528351,0.1347774027],[1,528352,0.1733627701],[1,528353,0.0532671989],[1,528354,0.0928599956],[1,528355,0.3014896798],[1,528359,0.0974291437],[1,528363,0.0367519707],[1,528364,0.0771823978],[1,528368,0.0348700063],[1,528369,0.1373893673],[1,528370,0.0319329755],[1,528372,0.0414499612],[1,528373,0.0815726598],[1,528375,0.0455579325],[1,528378,0.1241651177],[1,528379,0.2121857677],[1,528381,0.0617770853],[1,528382,0.5432769708],[1,528388,0.040053215],[1,528389,0.0455175225],[1,528390,0.1134249149],[1,528392,0.0397041101],[1,528393,0.0517754868],[1,528396,0.1455302576],[1,528397,0.112790977],[1,528398,0.040053215],[1,528400,0.0542141603],[1,528402,0.0537726688],[1,528414,0.6341305787],[1,528415,0.6510146588],[1,528416,0.2228697042],[1,528422,0.3261176186],[1,528423,0.0337792939],[1,528424,0.0385099451],[1,528426,0.0333518092],[1,528428,0.170359971],[1,528429,0.0422753831],[1,528430,0.0577166981],[1,528432,0.0521832062],[1,528433,0.3221322808],[1,528434,0.0761757953],[1,528439,0.0478861647],[1,528441,0.1120542195],[1,528442,0.0362577608],[1,528443,0.218668988],[1,528444,0.0621913794],[1,528453,0.093414745],[1,528454,0.0446368746],[1,528456,0.0811998356],[1,528457,0.0367519707],[1,528459,0.0319808479],[1,528461,0.0696090965],[1,528462,0.1717139916],[1,528465,0.0903718449],[1,528466,0.1658579645],[1,528471,0.0319329755],[1,528476,0.0975097684],[1,528477,0.121719325],[1,528479,0.0673355884],[1,528480,0.3970131571],[1,528486,0.1221080944],[1,528487,0.0348342326],[1,528488,0.0775506026],[1,528489,0.1796092111],[1,528490,0.0384478498],[1,528496,0.0595676926],[1,528502,0.6271913218],[1,528503,0.0999666002],[1,528505,0.1213810594],[1,528506,0.1292587899],[1,528507,0.0449438351],[1,528510,0.11369264],[1,528511,0.1668446614],[1,528512,0.0638361576],[1,528513,0.059071366],[1,528516,0.1199707378],[1,528523,0.0770017077],[1,528524,0.0451311622],[1,528526,0.043462497],[1,528527,0.0925049515],[1,528530,0.0444930187],[1,528533,0.0928599956],[1,528534,0.2645467779],[1,528536,0.0364340494],[1,528539,0.5046554692],[1,528544,0.0790005864],[1,528547,0.0355947568],[1,528549,0.5916973331],[1,528551,0.0951502194],[1,528552,0.0870981391],[1,528555,0.0953004518],[1,528561,0.1653172699],[1,528562,0.158969347],[1,528563,0.061464057],[1,528566,0.1454741293],[1,528567,0.0518023333],[1,528569,0.1206345074],[1,528570,0.1641708042],[1,528572,0.0432124195],[1,528575,0.0442856637],[1,528577,0.0373052738],[1,528583,0.040053215],[1,528584,0.0432679689],[1,528585,0.4156769497],[1,528587,0.0472100592],[1,528588,0.0315435188],[1,528590,0.1343200233],[1,528591,0.1720918275],[1,528592,0.0854770008],[1,528594,0.0397125586],[1,528595,0.0589477307],[1,528596,0.0483705866],[1,528599,0.1903297801],[1,528601,0.1953132123],[1,528602,0.0503687285],[1,528606,0.0333997717],[1,528607,0.036405181],[1,528611,0.171862027],[1,528616,0.0435504025],[1,528617,0.1556290083],[1,528618,0.0563215398],[1,528619,0.4909785841],[1,528620,0.0319329755],[1,528622,0.0380881172],[1,528623,0.0455786518],[1,528624,0.0587766804],[1,528628,0.1409517158],[1,528633,0.0679502353],[1,528634,0.2892103561],[1,528635,0.0469521934],[1,528638,0.0829961798],[1,528639,0.0314366327],[1,528643,0.0836280105],[1,528645,0.0367014453],[1,528647,0.1607126557],[1,528652,0.0355306576],[1,528656,0.0569572732],[1,528657,0.041000437],[1,528658,0.0644890022],[1,528660,0.369540009],[1,528661,0.1056212179],[1,528663,0.3375123951],[1,528664,0.0587414754],[1,528665,0.2602619729],[1,528667,0.1042933469],[1,528668,0.0774244461],[1,528669,0.0422958361],[1,528670,0.0373052738],[1,528675,0.0387387451],[1,528683,0.0594605487],[1,528685,0.3676234655],[1,528688,0.0824471299],[1,528691,0.0372869139],[1,528692,0.0774497694],[1,528693,0.4967397095],[1,528694,0.2036724836],[1,528695,0.3179798655],[1,528696,0.128498478],[1,528697,0.0937437394],[1,528704,0.0447718914],[1,528705,0.0517678874],[1,528706,0.040053215],[1,528707,0.1391365647],[1,528708,0.0416744351],[1,528711,0.21553278],[1,528713,0.132821682],[1,528714,0.1131174723],[1,528715,0.4284338103],[1,528716,0.0921288631],[1,528718,0.2705325229],[1,528719,0.0441058859],[1,528731,0.0422958361],[1,528732,0.0389821198],[1,528733,0.0319329755],[1,528734,0.0342239904],[1,528735,0.0353839344],[1,528736,0.0889327362],[1,528737,0.093414745],[1,528739,0.0887098774],[1,528740,0.0422103679],[1,528749,0.0499209767],[1,528751,0.0329246525],[1,528752,0.0544221927],[1,528753,0.0569740976],[1,528754,0.1021414707],[1,528755,0.1421708959],[1,528756,0.0348810855],[1,528757,0.0378082088],[1,528758,0.039024552],[1,528760,0.034726926],[1,528761,0.0960074569],[1,528763,0.0337407588],[1,528766,0.0408646478],[1,528767,0.1444706151],[1,528769,0.3864271861],[1,528770,0.3930774225],[1,528778,0.347659765],[1,528779,0.1386206815],[1,528780,0.0352997943],[1,528781,0.0565650218],[1,528783,0.0374700725],[1,528784,0.0314844736],[1,528786,0.0494605945],[1,528790,0.0617472443],[1,528791,0.0389821198],[1,528792,0.0422958361],[1,528794,0.0382992278],[1,528797,0.0595676926],[1,528801,0.1138803416],[1,528803,0.0373052738],[1,528804,0.0576448009],[1,528805,0.0663769277],[1,528806,0.0504533117],[1,528807,0.0600563623],[1,528808,0.0373052738],[1,528809,0.1000827293],[1,528810,0.0490490139],[1,528811,0.0617624039],[1,528812,0.0641382967],[1,528815,0.1786011059],[1,528816,0.6037818148],[1,528817,0.1492070675],[1,528818,0.0692525595],[1,528819,0.0496874315],[1,528830,0.0666192696],[1,528831,0.0739365837],[1,528832,0.0391923726],[1,528834,0.0457306134],[1,528835,0.1804273427],[1,528836,0.0719156447],[1,528837,0.1058104568],[1,528838,0.0347527051],[1,528839,0.0447470002],[1,528840,0.0595676926],[1,528841,0.0956921043],[1,528842,0.0820103082],[1,528846,0.0612907978],[1,528847,0.0422958361],[1,528848,0.0925049515],[1,528849,0.1320107902],[1,528853,0.040053215],[1,528857,0.0423338192],[1,528858,0.15667234],[1,528859,0.0576485023],[1,528860,0.054260671],[1,528861,0.1077205723],[1,528865,0.0935257886],[1,528866,0.0348225131],[1,528868,0.1572355799],[1,528869,0.1119985789],[1,528870,0.0556128251],[1,528872,0.0432124195],[1,528873,0.0442413975],[1,528875,0.0892894789],[1,528878,0.0455962745],[1,528883,0.0953004518],[1,528884,0.0538286317],[1,528886,0.2111637152],[1,528887,0.0379687135],[1,528888,0.0870724848],[1,528889,0.0412909474],[1,528890,0.0455393839],[1,528891,0.1213877941],[1,528892,0.0378082088],[1,528893,0.058021407],[1,528897,0.1296420233],[1,528899,0.0499044885],[1,528900,0.0950525581],[1,528901,0.0420294556],[1,528902,0.1575775259],[1,528903,0.0370400658],[1,528909,0.0526144236],[1,528911,0.2054160544],[1,528918,0.040053215],[1,528921,0.0887552876],[1,528922,0.0519028545],[1,528923,0.0925527574],[1,528927,0.0635028656],[1,528940,0.0357256417],[1,528941,0.1071258691],[1,528942,0.0417723372],[1,528943,0.1408700998],[1,528944,0.043537986],[1,528946,0.0672764451],[1,528947,0.4761869991],[1,528949,0.0687247275],[1,528952,0.5003996569],[1,528955,0.3089753376],[1,528957,0.0555510559],[1,528959,0.162754555],[1,528968,0.24220526],[1,528969,0.0501423832],[1,528970,0.1299804247],[1,528972,0.0417571939],[1,528973,0.2687097572],[1,528974,0.0458653869],[1,528977,0.1301582126],[1,528978,0.0348764335],[1,528979,0.0924845572],[1,528982,0.1946340396],[1,528983,0.0363362806],[1,528984,0.0363362806],[1,528987,0.1480862109],[1,528994,0.5813366858],[1,529001,0.0309056453],[1,529002,0.0981748702],[1,529003,0.0314016382],[1,529004,0.1373779925],[1,529006,0.0389821198],[1,529009,0.1756636665],[1,529015,0.0600413715],[1,529021,0.0333997717],[1,529024,0.0363362806],[1,529028,0.0382026103],[1,529030,0.0420635647],[1,529040,0.0529858329],[1,529042,0.091930916],[1,529043,0.0417571939],[1,529044,0.0394644017],[1,529045,0.0385625712],[1,529049,0.0492292138],[1,529056,0.0342193802],[1,529057,0.0443997156],[1,529058,0.1248698333],[1,529062,0.1031878114],[1,529064,0.094960608],[1,529066,0.1040469036],[1,529067,0.2110567742],[1,529069,0.0363362806],[1,529070,0.0379963593],[1,529073,0.0903058392],[1,529074,0.0556311823],[1,529079,0.0808147407],[1,529083,0.2588602057],[1,529084,0.0922857098],[1,529086,0.0697072728],[1,529099,0.3049332052],[1,529103,0.0507737035],[1,529112,0.0536070052],[1,529115,0.0417677609],[1,529116,0.0363362806],[1,529126,0.060433125],[1,529128,0.0579286449],[1,529131,0.0358995392],[1,529132,0.0687363923],[1,529133,0.0806130702],[1,529144,0.0314016382],[1,529145,0.0399734252],[1,529146,0.116276218],[1,529147,0.091930916],[1,529151,0.04674641],[1,529154,0.0622716212],[1,529155,0.1081001366],[1,529157,0.0314016382],[1,529159,0.1224627617],[1,529161,0.0667613943],[1,529164,0.0371180924],[1,529166,0.0396843328],[1,529167,0.0493475631],[1,529168,0.2199026595],[1,529169,0.0387289114],[1,529171,0.0456275869],[1,529172,0.0389821198],[1,529179,0.0417571939],[1,529180,0.0432890492],[1,529181,0.17603993],[1,529182,0.0587025301],[1,529186,0.04121147],[1,529191,0.298977064],[1,529192,0.0450918958],[1,529194,0.0861400721],[1,529196,0.0652438753],[1,529202,0.5642259312],[1,529204,0.0552643673],[1,529210,0.0361239816],[1,529217,0.0413340046],[1,529219,0.0740200709],[1,529220,0.0527252996],[1,529221,0.0922857098],[1,529222,0.04121147],[1,529228,0.0544664656],[1,529231,0.0429031402],[1,529234,0.4980234213],[1,529235,0.091930916],[1,529238,0.0432268567],[1,529240,0.0363362806],[1,529244,0.0382026103],[1,529248,0.0328194718],[1,529252,0.2561164645],[1,529383,0.1269348923],[1,529387,0.5059448323],[1,529391,0.0559949926],[1,529400,0.0689114531],[1,529403,0.0366115441],[1,529406,0.3705513352],[1,529412,0.0417571939],[1,529436,0.0521559442],[1,529437,0.0314016382],[1,529439,0.0905797179],[1,529441,0.04577619],[1,529452,0.4921633307],[1,529456,0.0518124394],[1,529458,0.0520298883],[1,529459,0.0363362806],[1,529461,0.0704056878],[1,529462,0.651902119],[1,529463,0.6024300809],[1,529466,0.4610729539],[1,529468,0.2391947365],[1,529469,0.1093671222],[1,529470,0.1053789384],[1,529472,0.0595959928],[1,529473,0.1274861629],[1,529475,0.0385625712],[1,529476,0.0887857937],[1,529478,0.0342402663],[1,529480,0.0717026117],[1,529482,0.0590730671],[1,529483,0.0606314568],[1,529489,0.4539572662],[1,529490,0.1722517824],[1,529496,0.0685225179],[1,529498,0.6094121399],[1,529508,0.089380334],[1,529509,0.0515297827],[1,529510,0.1828590965],[1,529522,0.0809655639],[1,529524,0.1194240775],[1,529525,0.0362706214],[1,529526,0.0542229321],[1,529529,0.0314016382],[1,529530,0.2547197436],[1,529531,0.0534118265],[1,529532,0.3429873261],[1,529533,0.0588015232],[1,529534,0.0979671974],[1,529535,0.2558051581],[1,529537,0.257013637],[1,529538,0.1279074667],[1,529539,0.1353868528],[1,529543,0.0350553218],[1,529544,0.2101721919],[1,529546,0.0685497934],[1,529553,0.0385003459],[1,529554,0.091930916],[1,529556,0.0492954968],[1,529557,0.0716202571],[1,529558,0.0417571939],[1,529562,0.2256892952],[1,529563,0.2432937935],[1,529564,0.0314016382],[1,529569,0.1195263161],[1,529570,0.1468326895],[1,529571,0.1373218513],[1,529574,0.0552335655],[1,529575,0.1629098649],[1,529581,0.0443277611],[1,529582,0.226678868],[1,529583,0.0351551672],[1,529589,0.0596263515],[1,529590,0.0824978688],[1,529593,0.0573610817],[1,529595,0.0933914466],[1,529598,0.0314016382],[1,529599,0.1556127114],[1,529600,0.0355704992],[1,529601,0.1007647373],[1,529602,0.104367333],[1,529607,0.3089577615],[1,529608,0.04121147],[1,529609,0.0366115441],[1,529610,0.0900350864],[1,529614,0.0328194718],[1,529618,0.0828973961],[1,529619,0.0474931115],[1,529621,0.0460312151],[1,529622,0.1430572989],[1,529630,0.0372886195],[1,529631,0.0739419496],[1,529633,0.0794779816],[1,529634,0.0314016382],[1,529635,0.0948193577],[1,529636,0.0575300337],[1,529637,0.0319808479],[1,529638,0.0433398204],[1,529641,0.0935069894],[1,529642,0.0455175225],[1,529644,0.0377634029],[1,529647,0.0417571939],[1,529649,0.0859619482],[1,529652,0.1721587696],[1,529655,0.1026208182],[1,529657,0.0366115441],[1,529658,0.0367701495],[1,529659,0.041715973],[1,529660,0.0455393839],[1,529664,0.1363185532],[1,529666,0.0753330596],[1,529668,0.0355787584],[1,529669,0.082775496],[1,529670,0.0382026103],[1,529680,0.0523250633],[1,529681,0.0527788152],[1,529682,0.0796726041],[1,529684,0.0433398204],[1,529685,0.0357670303],[1,529692,0.091930916],[1,529694,0.1379692153],[1,529695,0.3961027106],[1,529696,0.0615192034],[1,529697,0.0417340825],[1,529698,0.0417571939],[1,529701,0.0781295836],[1,529706,0.2778498152],[1,529709,0.0367701495],[1,529713,0.045627817],[1,529714,0.4784045756],[1,529716,0.0423450785],[1,529717,0.0464210138],[1,529718,0.0329803678],[1,529720,0.0417293103],[1,529724,0.3892235192],[1,529730,0.0395161536],[1,529731,0.0499209767],[1,529747,0.0365382941],[1,529748,0.0397125586],[1,529751,0.0518075082],[1,529752,0.0348700063],[1,529753,0.1090635239],[1,529758,0.0362085836],[1,529759,0.2033750888],[1,529760,0.0462500361],[1,529761,0.0805575732],[1,529764,0.1065794807],[1,529767,0.0617472443],[1,529768,0.0526796424],[1,529776,0.5397346559],[1,529777,0.2228051439],[1,529778,0.0375704676],[1,529780,0.0412909474],[1,529781,0.0544866057],[1,529790,0.3811717292],[1,529791,0.5069852503],[1,529792,0.0363362806],[1,529795,0.1145331267],[1,529796,0.0333997717],[1,529799,0.1324379788],[1,529800,0.0416744351],[1,529805,0.0363362806],[1,529808,0.0385625712],[1,529816,0.0861819691],[1,529821,0.1977112494],[1,529822,0.119132012],[1,529823,0.068776723],[1,529828,0.0367701495],[1,529833,0.3134735134],[1,529834,0.0994970241],[1,529835,0.1572540865],[1,529840,0.0319808479],[1,529847,0.0741090216],[1,529848,0.0480926056],[1,529849,0.0319808479],[1,529851,0.0399200309],[1,529852,0.0314016382],[1,529853,0.0693290463],[1,529854,0.0358995392],[1,529856,0.2128709062],[1,529857,0.335205666],[1,529860,0.098809451],[1,529863,0.1540546142],[1,529866,0.0331824654],[1,529869,0.0696899401],[1,529870,0.0612291942],[1,529871,0.0417571939],[1,529874,0.1391298739],[1,529875,0.0483383955],[1,529876,0.1363816024],[1,529877,0.061066724],[1,529880,0.0951441887],[1,529881,0.7075301749],[1,529883,0.1741509558],[1,529889,0.0342720083],[1,529892,0.1309777841],[1,529893,0.0442413975],[1,529894,0.0441091142],[1,529895,0.0417571939],[1,529902,0.0493124502],[1,529907,0.0441453712],[1,529908,0.0327836424],[1,529909,0.6864363134],[1,529912,0.0612052059],[1,529914,0.0365051285],[1,529917,0.1352688403],[1,529918,0.2431579924],[1,529923,0.0516166342],[1,529934,0.0628323455],[1,529936,0.0382026103],[1,529938,0.1079704448],[1,529939,0.2936060876],[1,529940,0.2639076534],[1,529941,0.0663344817],[1,529945,0.479534267],[1,529946,0.0363362806],[1,529947,0.7112901794],[1,529950,0.2176833195],[1,529951,0.2972266511],[1,529953,0.0630624185],[1,529954,0.1518498943],[1,529960,0.2854994073],[1,529963,0.0367701495],[1,529964,0.2742546899],[1,529965,0.0438228496],[1,529966,0.2408925852],[1,529970,0.0331813752],[1,529977,0.0362324987],[1,529983,0.2605985092],[1,529984,0.2174771178],[1,529985,0.04643964],[1,529986,0.2442304449],[1,529987,0.1369886761],[1,529988,0.1078335198],[1,529989,0.0609058261],[1,529990,0.163171871],[1,529992,0.2111105781],[1,529993,0.0983682905],[1,529996,0.2670912173],[1,529999,0.1036249965],[1,530008,0.2206505614],[1,530009,0.3304703065],[1,530011,0.0412346186],[1,530012,0.050102532],[1,530013,0.046100105],[1,530014,0.2195883909],[1,530015,0.2804172757],[1,530016,0.0417571939],[1,530018,0.0819669031],[1,530020,0.049982613],[1,530023,0.0619216717],[1,530024,0.0395161536],[1,530025,0.2580599339],[1,530026,0.0414499612],[1,530027,0.0625232415],[1,530028,0.0367701495],[1,530029,0.0881657948],[1,530030,0.1791642222],[1,530032,0.1777118984],[1,530033,0.0447630187],[1,530040,0.0842922926],[1,530041,0.1529835111],[1,530042,0.1343177866],[1,530043,0.2107522324],[1,530044,0.2718149874],[1,530045,0.0551082154],[1,530047,0.1185019736],[1,530048,0.0377514617],[1,530075,0.0638465349],[1,530076,0.0453587002],[1,530097,0.0554828426],[1,530101,0.0395161536],[1,530102,0.0419722113],[1,530103,0.2997206094],[1,530104,0.2416982901],[1,530105,0.0915645521],[1,530106,0.0550491657],[1,530107,0.0394856012],[1,530108,0.0995570945],[1,530115,0.2304114634],[1,530119,0.0680797596],[1,530120,0.0763470366],[1,530121,0.0369138988],[1,530122,0.0568598372],[1,530124,0.3515613824],[1,530126,0.0594223872],[1,530127,0.0358995392],[1,530134,0.0701625636],[1,530135,0.06180498],[1,530136,0.0382026103],[1,530137,0.2628473895],[1,530138,0.0728235426],[1,530140,0.0757599631],[1,530143,0.2254088901],[1,530144,0.0364213851],[1,530145,0.0314016382],[1,530146,0.2574531132],[1,530149,0.0730123837],[1,530150,0.0327836424],[1,530151,0.0928400682],[1,530155,0.045627817],[1,530158,0.091930916],[1,530160,0.0488090517],[1,530161,0.0631367708],[1,530163,0.3838195691],[1,530164,0.4268157327],[1,530165,0.0720866569],[1,530166,0.0314016382],[1,530169,0.057375307],[1,530170,0.1100121313],[1,530171,0.4207905338],[1,530172,0.0923642627],[1,530173,0.1104338311],[1,530174,0.0358811498],[1,530176,0.1240506434],[1,530180,0.1129931872],[1,530181,0.2075482479],[1,530182,0.0493176007],[1,530185,0.0553891449],[1,530191,0.1937472321],[1,530192,0.0582340194],[1,530193,0.0389821198],[1,530196,0.0395161536],[1,530198,0.3829560094],[1,530201,0.0374331215],[1,530203,0.1517503114],[1,530213,0.2952402414],[1,530214,0.223042909],[1,530215,0.0813807447],[1,530216,0.0331813752],[1,530217,0.0991916499],[1,530218,0.0481327463],[1,530219,0.0436431804],[1,530227,0.0717231267],[1,530228,0.0456086675],[1,530230,0.0351160535],[1,530231,0.0534075958],[1,530232,0.3073832885],[1,530235,0.1053907413],[1,530239,0.0899293708],[1,530240,0.0459768052],[1,530242,0.0435410871],[1,530243,0.0571831132],[1,530245,0.0417113969],[1,530255,0.0672915387],[1,530258,0.3998606123],[1,530259,0.0898708596],[1,530261,0.0549679593],[1,530262,0.0599526124],[1,530263,0.0400295209],[1,530266,0.0394005892],[1,530267,0.0613666085],[1,530271,0.0595107278],[1,530277,0.4847911864],[1,530280,0.1324267808],[1,530281,0.091930916],[1,530282,0.0537232987],[1,530283,0.1231389244],[1,530284,0.0314016382],[1,530285,0.2645107107],[1,530287,0.0677291893],[1,530288,0.0949926159],[1,530289,0.1104907041],[1,530292,0.4462191706],[1,530294,0.3401617346],[1,530297,0.1455605887],[1,530305,0.0342193802],[1,530306,0.0382026103],[1,530307,0.0358995392],[1,530308,0.0619160721],[1,530309,0.2213052532],[1,530310,0.2381569028],[1,530317,0.0395161536],[1,530318,0.0336910382],[1,530324,0.0585482628],[1,530335,0.0363362806],[1,530341,0.0314016382],[1,530342,0.1250168718],[1,530343,0.0342193802],[1,530344,0.0941360721],[1,530346,0.0704787264],[1,530351,0.0363362806],[1,530353,0.0995328246],[1,530354,0.160603043],[1,530357,0.0651092047],[1,530359,0.045627817],[1,530362,0.0662891103],[1,530376,0.0566578796],[1,530377,0.0585866535],[1,530378,0.2125285193],[1,530379,0.0760931559],[1,530380,0.0465234372],[1,530381,0.3186010082],[1,530388,0.5141916789],[1,530391,0.0389821198],[1,530392,0.0569388281],[1,530393,0.0370620717],[1,530394,0.0385625712],[1,530395,0.0571427149],[1,530399,0.0362706214],[1,530402,0.0333863086],[1,530404,0.1497933289],[1,530405,0.1715146165],[1,530408,0.0449181888],[1,530409,0.0329246525],[1,530414,0.3620357746],[1,530415,0.0548459492],[1,530416,0.4311331864],[1,530418,0.0532671989],[1,530419,0.0928400682],[1,530420,0.0503097499],[1,530421,0.065978603],[1,530423,0.1257671418],[1,530424,0.0371562006],[1,530427,0.0313907786],[1,530428,0.0928400682],[1,530430,0.4280207874],[1,530434,0.0993065113],[1,530437,0.0377514617],[1,530441,0.0363362806],[1,530450,0.0552165405],[1,530451,0.158754911],[1,530454,0.1384400177],[1,530455,0.0578705499],[1,530456,0.0342193802],[1,530457,0.0315913665],[1,530461,0.0646494223],[1,530463,0.0546052451],[1,530464,0.1185528252],[1,530468,0.3519333864],[1,530469,0.0385003459],[1,530477,0.0578344859],[1,530480,0.0569880121],[1,530482,0.1312529765],[1,530488,0.2496626849],[1,530492,0.4405909728],[1,530499,0.0545794933],[1,530501,0.0385003459],[1,530502,0.036634185],[1,530503,0.1010432176],[1,530504,0.0342193802],[1,530505,0.0946098764],[1,530506,0.0348764335],[1,530509,0.0618099485],[1,530523,0.1009668956],[1,530525,0.0359598111],[1,530526,0.0577894042],[1,530527,0.0329846012],[1,530528,0.0553911603],[1,530529,0.3109789943],[1,530530,0.0355787584],[1,530531,0.0367732398],[1,530532,0.0563144373],[1,530533,0.1521855195],[1,530537,0.0414499612],[1,530542,0.1584268039],[1,530543,0.0818787221],[1,530546,0.0468331748],[1,530547,0.3592062801],[1,530551,0.0462167858],[1,530552,0.0363362806],[1,530553,0.0922857098],[1,530555,0.0395161536],[1,530556,0.2129212875],[1,530557,0.2084662131],[1,530558,0.0367701495],[1,530561,0.0493093873],[1,530564,0.0566954492],[1,530565,0.4192235448],[1,530566,0.0595676926],[1,530568,0.0540004621],[1,530569,0.2608825091],[1,530570,0.0617993291],[1,530571,0.3362659423],[1,530572,0.0607900204],[1,530573,0.0985399348],[1,530574,0.0342193802],[1,530575,0.0600413715],[1,530576,0.2278160371],[1,530577,0.0612292827],[1,530578,0.2693670132],[1,530580,0.2324648251],[1,530581,0.072189421],[1,530585,0.0539061203],[1,530588,0.1337093997],[1,530591,0.0381486552],[1,530592,0.1019569036],[1,530593,0.0427073245],[1,530594,0.040714478],[1,530595,0.0342193802],[1,530597,0.3001234314],[1,530598,0.031337786],[1,530599,0.091930916],[1,530608,0.0578665648],[1,530609,0.0504784906],[1,530610,0.0374994832],[1,530613,0.120026225],[1,530614,0.1174571621],[1,530615,0.1219328966],[1,530619,0.0374331215],[1,530620,0.1428921379],[1,530621,0.0621787655],[1,530624,0.0487881017],[1,530631,0.0860183394],[1,530636,0.6585000753],[1,530639,0.2395428583],[1,530640,0.1770907492],[1,530642,0.2247488385],[1,530644,0.0422103679],[1,530650,0.0697498504],[1,530651,0.0328194718],[1,530652,0.0358995392],[1,530656,0.0553891449],[1,530658,0.1445891854],[1,530659,0.154801116],[1,530660,0.0347660837],[1,530662,0.4653787491],[1,530663,0.0864119455],[1,530667,0.0442413975],[1,530671,0.0541124353],[1,530674,0.1097667974],[1,530675,0.0360207501],[1,530676,0.0381906176],[1,530678,0.0372642576],[1,530679,0.3590772226],[1,530682,0.137756763],[1,530684,0.0322242241],[1,530685,0.1992127207],[1,530689,0.0360630172],[1,530692,0.0415062731],[1,530693,0.0415062731],[1,530695,0.081121448],[1,530697,0.5666692674],[1,530698,0.0363623142],[1,530699,0.3321224774],[1,530707,0.5787816332],[1,530709,0.0398835326],[1,530717,0.0327365632],[1,530718,0.1378943956],[1,530719,0.0331578571],[1,530720,0.0372076283],[1,530721,0.0916635076],[1,530722,0.1037497505],[1,530726,0.0563586849],[1,530727,0.0332001555],[1,530729,0.0612957714],[1,530730,0.033442768],[1,530736,0.2606540095],[1,530746,0.0339709364],[1,530747,0.4096161245],[1,530748,0.039040073],[1,530751,0.0368064037],[1,530752,0.0317331397],[1,530755,0.0474724427],[1,530757,0.0916635076],[1,530759,0.0408518925],[1,530760,0.057859002],[1,530765,0.0544633746],[1,530772,0.0327365632],[1,530775,0.0484566162],[1,530776,0.0377584833],[1,530777,0.1154402951],[1,530780,0.151035163],[1,530781,0.0965436545],[1,530782,0.0325043459],[1,530783,0.0360871412],[1,530788,0.0710955641],[1,530793,0.1237881013],[1,530794,0.0356505433],[1,530796,0.2317706018],[1,530798,0.0383127002],[1,530800,0.0585462786],[1,530802,0.0360871412],[1,530807,0.1073814974],[1,530815,0.0369750903],[1,530816,0.4910304843],[1,530817,0.0345174603],[1,530818,0.0642885721],[1,530819,0.0488020785],[1,530823,0.0365208675],[1,530827,0.0400400044],[1,530828,0.033442768],[1,530829,0.1081209563],[1,530834,0.0454113664],[1,530836,0.0538711992],[1,530837,0.0627147138],[1,530838,0.1097495617],[1,530839,0.0659255351],[1,530840,0.0345174603],[1,530843,0.0345174603],[1,530846,0.0379663653],[1,530862,0.2921767191],[1,530865,0.0746365715],[1,530866,0.200354209],[1,530867,0.1795889929],[1,530870,0.0366751142],[1,530872,0.0674222312],[1,530873,0.0499027473],[1,530878,0.1706193409],[1,530883,0.0339709364],[1,530885,0.0752953253],[1,530895,0.0492889395],[1,530896,0.1433846301],[1,530897,0.0317331397],[1,530901,0.0577075709],[1,530902,0.0362559336],[1,530906,0.0418915935],[1,530919,0.0464399941],[1,530920,0.1663377324],[1,530925,0.0549662779],[1,530927,0.1660471268],[1,530930,0.0473241407],[1,530933,0.1413722177],[1,530934,0.0730123837],[1,530936,0.0486779721],[1,530937,0.0590489668],[1,530938,0.0920181849],[1,530939,0.0415062731],[1,530942,0.0795264286],[1,530943,0.0529974701],[1,530944,0.0652237492],[1,530947,0.0535388518],[1,530948,0.0332001555],[1,530949,0.0379528576],[1,530950,0.0622200068],[1,530951,0.5354263722],[1,530952,0.0311541204],[1,530953,0.0503351658],[1,530961,0.2850077199],[1,530963,0.0340991842],[1,530965,0.0553497367],[1,530975,0.264844213],[1,530976,0.0917732598],[1,530978,0.0499481917],[1,530979,0.1922192512],[1,530980,0.0529301731],[1,530981,0.1811706544],[1,530983,0.1077493699],[1,530984,0.0630127678],[1,530986,0.2381328634],[1,530995,0.0526472412],[1,530997,0.05135132],[1,530998,0.1171502211],[1,531000,0.0795615935],[1,531001,0.1142762661],[1,531003,0.0920181849],[1,531004,0.0429581265],[1,531006,0.1557060103],[1,531010,0.2869867845],[1,531013,0.3069776493],[1,531014,0.1142164052],[1,531015,0.0680054174],[1,531017,0.0436502233],[1,531018,0.5089899788],[1,531019,0.0916635076],[1,531026,0.1589117145],[1,531027,0.0415062731],[1,531028,0.0365208675],[1,531029,0.0825438973],[1,531030,0.5081774856],[1,531033,0.1133745515],[1,531036,0.033442768],[1,531047,0.1003281368],[1,531050,0.0353216114],[1,531051,0.0769014517],[1,531054,0.0709771064],[1,531055,0.0650637179],[1,531057,0.0916635076],[1,531064,0.0845794152],[1,531065,0.0422790387],[1,531066,0.0450794891],[1,531067,0.1240930601],[1,531068,0.0596053554],[1,531070,0.0401857599],[1,531071,0.110392999],[1,531072,0.1219868505],[1,531073,0.1044954845],[1,531074,0.2030455831],[1,531075,0.0345174603],[1,531077,0.0369750903],[1,531079,0.0315325141],[1,531080,0.0360871412],[1,531093,0.0441479265],[1,531094,0.1595606585],[1,531098,0.046419924],[1,531099,0.1197923722],[1,531101,0.1119304941],[1,531104,0.0556298155],[1,531107,0.0423026142],[1,531108,0.0366639188],[1,531109,0.0610358937],[1,531110,0.0651860147],[1,531112,0.073612156],[1,531113,0.0345174603],[1,531114,0.1436894345],[1,531121,0.3501250207],[1,531123,0.0542087467],[1,531124,0.0355875146],[1,531126,0.0418115035],[1,531128,0.109044343],[1,531129,0.0642528251],[1,531130,0.0405463113],[1,531131,0.054452819],[1,531136,0.0430988903],[1,531138,0.1087102486],[1,531139,0.0334941171],[1,531145,0.0373057167],[1,531146,0.087037359],[1,531147,0.1149809004],[1,531149,0.0916635076],[1,531151,0.0707237196],[1,531152,0.1860372283],[1,531153,0.0363603967],[1,531154,0.0978610039],[1,531155,0.0391157996],[1,531157,0.0980977193],[1,531162,0.3925948933],[1,531164,0.0417998799],[1,531165,0.0473241407],[1,531166,0.0890683158],[1,531168,0.0345269064],[1,531171,0.065935855],[1,531173,0.0339709364],[1,531180,0.0613743699],[1,531181,0.0311541204],[1,531183,0.0388179874],[1,531184,0.0497308401],[1,531185,0.0339918157],[1,531187,0.1351879617],[1,531189,0.0360871412],[1,531190,0.0597332159],[1,531191,0.1041620124],[1,531193,0.036184004],[1,531199,0.0671102312],[1,531203,0.0379663653],[1,531204,0.0365208675],[1,531206,0.0642011836],[1,531216,0.1038833122],[1,531217,0.0581599869],[1,531221,0.0916635076],[1,531223,0.0317331397],[1,531224,0.0436293296],[1,531225,0.1077871445],[1,531230,0.0319667385],[1,531231,0.0345174603],[1,531232,0.0359959871],[1,531242,0.0988320301],[1,531245,0.0376009385],[1,531247,0.0375056486],[1,531248,0.2256141111],[1,531249,0.0360871412],[1,531251,0.0881539225],[1,531257,0.1411965788],[1,531258,0.0365208675],[1,531260,0.0354885511],[1,531261,0.0359681362],[1,531262,0.1191118799],[1,531268,0.0536531805],[1,531280,0.0714623589],[1,531281,0.0379528576],[1,531286,0.0903021191],[1,531295,0.0345802999],[1,531307,0.1265290246],[1,531308,0.0417726638],[1,531312,0.0875832949],[1,531313,0.1064542761],[1,531325,0.0411991414],[1,531351,0.054394803],[1,531359,0.2341803761],[1,531361,0.0454403919],[1,531362,0.1355612635],[1,531365,0.1019962679],[1,531366,0.0482384974],[1,531368,0.0833711201],[1,531372,0.0629196796],[1,531383,0.0816798936],[1,531390,0.1146246177],[1,531395,0.0433312743],[1,531415,0.0667977583],[1,531416,0.0922590149],[1,531417,0.0926139518],[1,531424,0.0926139518],[1,531425,0.2961765486],[1,531426,0.0590148522],[1,531428,0.0899602874],[1,531436,0.0365762786],[1,531437,0.0381406594],[1,531440,0.0381406594],[1,531441,0.0339956553],[1,531447,0.3694854792],[1,531448,0.0922590149],[1,531452,0.3097843173],[1,531453,0.047556567],[1,531457,0.3331265208],[1,531469,0.2508119466],[1,531472,0.0605542996],[1,531475,0.1678605409],[1,531476,0.1441868159],[1,531478,0.2350652819],[1,531487,0.0620073873],[1,531488,0.1005068814],[1,531491,0.2118261845],[1,531492,0.0515465249],[1,531493,0.1505102442],[1,531500,0.129250106],[1,531503,0.0398231194],[1,531505,0.3694640571],[1,531509,0.078194973],[1,531513,0.0369173387],[1,531514,0.0632424861],[1,531515,0.0414831539],[1,531518,0.0366419643],[1,531519,0.0659061305],[1,531520,0.0473090361],[1,531521,0.1137293175],[1,531522,0.4421258971],[1,531523,0.0718917664],[1,531530,0.0579098665],[1,531536,0.5595416339],[1,531538,0.1457420549],[1,531542,0.1155494556],[1,531544,0.0420650632],[1,531546,0.0536370092],[1,531548,0.2286577295],[1,531551,0.1990875413],[1,531553,0.0414076721],[1,531557,0.3909269275],[1,531558,0.0366419643],[1,531559,0.3546512336],[1,531564,0.0789483486],[1,531565,0.0843095093],[1,531567,0.2942866884],[1,531570,0.1874762449],[1,531579,0.0771874059],[1,531583,0.1604534931],[1,531584,0.0455322915],[1,531585,0.1187303248],[1,531586,0.1286795771],[1,531589,0.0370760081],[1,531590,0.0926139518],[1,531591,0.3432931753],[1,531593,0.173292776],[1,531598,0.0753545129],[1,531600,0.3619335791],[1,531601,0.128343822],[1,531602,0.0345451049],[1,531605,0.0345242103],[1,531606,0.0370760081],[1,531607,0.1883291065],[1,531608,0.2790644037],[1,531609,0.0490073337],[1,531610,0.0475190106],[1,531611,0.1853963308],[1,531612,0.0687105378],[1,531614,0.0702977779],[1,531615,0.5359245039],[1,531616,0.0692674529],[1,531631,0.1548950496],[1,531632,0.0539512991],[1,531633,0.0398231194],[1,531634,0.2435603055],[1,531637,0.3544591193],[1,531638,0.0711378312],[1,531642,0.0402271595],[1,531648,0.1766309137],[1,531650,0.2994272292],[1,531651,0.0936120541],[1,531652,0.4002351022],[1,531653,0.0803463802],[1,531654,0.0578887346],[1,531656,0.1594428393],[1,531658,0.1220668366],[1,531659,0.1715488315],[1,531692,0.0897074045],[1,531705,0.0370760081],[1,531709,0.0926139518],[1,531710,0.5026466512],[1,531711,0.0317053323],[1,531713,0.1047199617],[1,531716,0.0606461291],[1,531743,0.6064681727],[1,531744,0.0425404961],[1,531745,0.0341737865],[1,531746,0.1624580724],[1,531748,0.0714527334],[1,531749,0.1774058238],[1,531750,0.0522179892],[1,531751,0.0860509036],[1,531752,0.2752734728],[1,531753,0.1584303436],[1,531754,0.038644405],[1,531756,0.0494958262],[1,531758,0.0339956553],[1,531760,0.2526290357],[1,531768,0.2223248081],[1,531769,0.0376273787],[1,531771,0.0398231194],[1,531773,0.2213975841],[1,531774,0.1597978862],[1,531777,0.0479653417],[1,531786,0.287844491],[1,531788,0.0625432451],[1,531789,0.0553893118],[1,531790,0.0375305633],[1,531791,0.0366419643],[1,531792,0.2651166828],[1,531793,0.1956116216],[1,531794,0.2870067889],[1,531797,0.0926139518],[1,531798,0.0385090464],[1,531799,0.0788287177],[1,531800,0.1962560088],[1,531802,0.1422289869],[1,531803,0.0502937629],[1,531804,0.2797283518],[1,531809,0.0549734844],[1,531811,0.0424424546],[1,531813,0.0370760081],[1,531814,0.0433907604],[1,531816,0.0434591252],[1,531822,0.0488105189],[1,531824,0.0366419643],[1,531827,0.0356063636],[1,531829,0.1074419464],[1,531831,0.048297934],[1,531834,0.0376273787],[1,531837,0.1800036241],[1,531842,0.0511518364],[1,531843,0.0670793451],[1,531848,0.097664573],[1,531849,0.0404377009],[1,531853,0.050422658],[1,531856,0.2431329376],[1,531857,0.0332889335],[1,531859,0.0362050468],[1,531860,0.0904618789],[1,531861,0.0968813864],[1,531863,0.0331237375],[1,531869,0.2564472635],[1,531870,0.1352524357],[1,531871,0.1140061407],[1,531882,0.0547891154],[1,531883,0.0339686172],[1,531889,0.0531798241],[1,531890,0.1143526523],[1,531894,0.0447086504],[1,531899,0.5245059773],[1,531900,0.1263498309],[1,531902,0.1662004428],[1,531903,0.0493536662],[1,531904,0.0381568697],[1,531905,0.0322847755],[1,531909,0.0312091394],[1,531910,0.5310080736],[1,531911,0.0370760081],[1,531915,0.2463631929],[1,531916,0.306756644],[1,531917,0.0406424312],[1,531918,0.103446129],[1,531919,0.0912432884],[1,531920,0.0397075083],[1,531922,0.2001757807],[1,531923,0.0918387492],[1,531926,0.598136849],[1,531929,0.045657676],[1,531930,0.040730379],[1,531939,0.1002445765],[1,531941,0.0739709838],[1,531943,0.0494492085],[1,531944,0.2612742408],[1,531945,0.5840556047],[1,531947,0.0580154935],[1,531948,0.1751746966],[1,531950,0.0774626586],[1,531951,0.1255409451],[1,531952,0.2611643543],[1,531955,0.0883943025],[1,531956,0.044932759],[1,531969,0.6178820485],[1,531970,0.0532393653],[1,531971,0.1987749506],[1,531972,0.0418796232],[1,531973,0.046091234],[1,531974,0.0356963611],[1,531976,0.0396620997],[1,531977,0.2834084481],[1,531979,0.3766031346],[1,531983,0.0687721887],[1,531989,0.2215166941],[1,531990,0.0624664763],[1,531991,0.1068730374],[1,531992,0.0938626159],[1,531994,0.0664464926],[1,531995,0.0446681559],[1,531996,0.2633693285],[1,531997,0.2199041713],[1,532001,0.0809390505],[1,532002,0.0448758922],[1,532003,0.0367221227],[1,532004,0.2337744972],[1,532005,0.0389422419],[1,532018,0.0424500457],[1,532019,0.0462559636],[1,532020,0.0402123732],[1,532022,0.0616984353],[1,532023,0.0394350907],[1,532024,0.1058308958],[1,532026,0.1499516389],[1,532033,0.3437657113],[1,532034,0.287903857],[1,532036,0.0899838374],[1,532039,0.0456610008],[1,532040,0.0370787413],[1,532042,0.1272579762],[1,532044,0.0985117242],[1,532045,0.0868284644],[1,532046,0.0406484602],[1,532047,0.0598233689],[1,532054,0.0406484602],[1,532055,0.0642798012],[1,532056,0.043292348],[1,532057,0.0645331879],[1,532058,0.6068481842],[1,532061,0.0500261151],[1,532062,0.0841754308],[1,532068,0.2642516871],[1,532071,0.096447843],[1,532076,0.0420882445],[1,532077,0.0707930349],[1,532079,0.0537694576],[1,532082,0.0468088462],[1,532083,0.0434085033],[1,532084,0.0719983253],[1,532092,0.04244701],[1,532094,0.598793766],[1,532096,0.0456610008],[1,532097,0.0523734938],[1,532103,0.0471752629],[1,532104,0.1441425194],[1,532105,0.0791922847],[1,532106,0.073774405],[1,532108,0.1471599305],[1,532109,0.1611950015],[1,532111,0.1254920828],[1,532112,0.3257441743],[1,532116,0.0401463783],[1,532117,0.061904871],[1,532118,0.2163252071],[1,532119,0.1042140067],[1,532124,0.0716965574],[1,532125,0.0456610008],[1,532127,0.0606157742],[1,532134,0.037553607],[1,532137,0.1697711924],[1,532138,0.0789521745],[1,532139,0.3401902242],[1,532140,0.1311747963],[1,532142,0.0960912353],[1,532144,0.0597344588],[1,532145,0.1853123894],[1,532152,0.0803780125],[1,532153,0.1718225616],[1,532156,0.0621993126],[1,532157,0.0406484602],[1,532158,0.3279547327],[1,532159,0.046057311],[1,532160,0.0721634557],[1,532161,0.1288792989],[1,532162,0.0456610008],[1,532164,0.0818732252],[1,532166,0.0563113958],[1,532167,0.0592857291],[1,532168,0.0614350139],[1,532170,0.039773399],[1,532171,0.1108292217],[1,532173,0.0483170324],[1,532176,0.0434085033],[1,532178,0.045397738],[1,532186,0.0605813114],[1,532189,0.0960912353],[1,532190,0.0531267434],[1,532191,0.0539733192],[1,532192,0.2554360447],[1,532195,0.0809094325],[1,532196,0.5011903447],[1,532197,0.0439803522],[1,532200,0.2641651889],[1,532205,0.1842813328],[1,532206,0.2736445545],[1,532207,0.3664931735],[1,532209,0.1361866461],[1,532211,0.0483170324],[1,532212,0.0367969931],[1,532213,0.1678137],[1,532214,0.0347539739],[1,532215,0.1193208658],[1,532217,0.229610599],[1,532218,0.1033558553],[1,532222,0.0936240487],[1,532223,0.1820823595],[1,532224,0.0592372295],[1,532232,0.332401512],[1,532236,0.2425020256],[1,532237,0.0372608515],[1,532238,0.0795671171],[1,532240,0.0490522185],[1,532241,0.0352525026],[1,532245,0.1040515354],[1,532247,0.4863864437],[1,532254,0.1258416303],[1,532255,0.0402123732],[1,532257,0.0362607101],[1,532258,0.0651039139],[1,532259,0.0400840234],[1,532260,0.105723836],[1,532261,0.1101399772],[1,532262,0.0821189595],[1,532263,0.045112487],[1,532266,0.0925817136],[1,532272,0.0352525026],[1,532273,0.2704937033],[1,532274,0.1346573829],[1,532276,0.0397549156],[1,532278,0.6366131614],[1,532279,0.29012091],[1,532280,0.0418548379],[1,532282,0.0532362871],[1,532283,0.1025594198],[1,532284,0.1672954136],[1,532286,0.0395424086],[1,532287,0.0519826451],[1,532289,0.0436506857],[1,532294,0.2176564538],[1,532297,0.0656273243],[1,532298,0.054189168],[1,532299,0.0395024685],[1,532300,0.083315059],[1,532302,0.1230429505],[1,532304,0.0531267434],[1,532305,0.0886328456],[1,532306,0.0751420811],[1,532307,0.2727107289],[1,532311,0.2557896124],[1,532315,0.0434085033],[1,532319,0.0596945346],[1,532320,0.0688985577],[1,532323,0.0707381552],[1,532324,0.1661953261],[1,532325,0.0540372969],[1,532327,0.3680844059],[1,532328,0.0505232169],[1,532330,0.0388660287],[1,532331,0.6293075248],[1,532334,0.2188954218],[1,532337,0.0672916218],[1,532338,0.2508822101],[1,532343,0.0618707547],[1,532350,0.0764832792],[1,532351,0.0352525026],[1,532353,0.0389859078],[1,532358,0.1972641128],[1,532360,0.0657270945],[1,532361,0.1109848345],[1,532362,0.0861044453],[1,532363,0.1485591148],[1,532371,0.3959147804],[1,532373,0.0926129519],[1,532375,0.4991360502],[1,532376,0.3261117203],[1,532377,0.0424500457],[1,532381,0.074781862],[1,532383,0.079196892],[1,532384,0.1888388767],[1,532386,0.0402123732],[1,532393,0.1255526505],[1,532397,0.045112487],[1,532398,0.3002359193],[1,532399,0.0757801341],[1,532400,0.0604861328],[1,532402,0.1418045282],[1,532403,0.0673845793],[1,532404,0.4385172391],[1,532405,0.3537539731],[1,532407,0.0955837187],[1,532408,0.0481621349],[1,532409,0.0557238951],[1,532413,0.177197893],[1,532416,0.0406484602],[1,532420,0.0716059943],[1,532424,0.0401456208],[1,532425,0.0643586799],[1,532426,0.0684337837],[1,532428,0.0745481676],[1,532432,0.0865025809],[1,532433,0.0785565613],[1,532436,0.1304645358],[1,532439,0.0611363931],[1,532441,0.0437474739],[1,532442,0.0402123732],[1,532443,0.1244125749],[1,532444,0.0765795826],[1,532445,0.1003286368],[1,532446,0.0406484602],[1,532449,0.094752123],[1,532450,0.1664736601],[1,532451,0.0572903129],[1,532452,0.1341857276],[1,532453,0.7423918727],[1,532454,0.0870377335],[1,532455,0.0684714939],[1,532456,0.0406484602],[1,532460,0.1883539307],[1,532461,0.0893842909],[1,532462,0.096447843],[1,532463,0.0977984055],[1,532464,0.3032774176],[1,532465,0.0968752496],[1,532466,0.1519053991],[1,532475,0.1115916477],[1,532476,0.0523976908],[1,532477,0.303806926],[1,532478,0.169933187],[1,532479,0.1334789115],[1,532480,0.0380306229],[1,532481,0.069824415],[1,532484,0.0388808798],[1,532490,0.0866791468],[1,532491,0.0352525026],[1,532493,0.096447843],[1,532494,0.5263194611],[1,532495,0.0825307428],[1,532499,0.1415770355],[1,532500,0.0724045111],[1,532505,0.0533544063],[1,532506,0.0499934067],[1,532508,0.2251674857],[1,532509,0.0494625272],[1,532510,0.0653213098],[1,532514,0.0445592467],[1,532518,0.0427021609],[1,532519,0.0426864304],[1,532524,0.0400840234],[1,532525,0.0402123732],[1,532526,0.0352525026],[1,532531,0.1112569269],[1,532533,0.0434085033],[1,532537,0.2254817187],[1,532541,0.0383056672],[1,532547,0.0747009216],[1,532549,0.0574691376],[1,532557,0.2865411809],[1,532558,0.1642755611],[1,532559,0.1262635536],[1,532560,0.0867486649],[1,532563,0.139449469],[1,532566,0.0604827771],[1,532570,0.0404797498],[1,532574,0.505181464],[1,532575,0.0373231812],[1,532576,0.0431756136],[1,532577,0.5835787752],[1,532578,0.039773399],[1,532579,0.0484370601],[1,532580,0.0911344053],[1,532587,0.1101425814],[1,532593,0.3021392027],[1,532595,0.1027389213],[1,532597,0.039773399],[1,532598,0.0739835754],[1,532603,0.0388660287],[1,532606,0.6630107991],[1,532607,0.061536737],[1,532613,0.1094153161],[1,532614,0.0948811028],[1,532615,0.1128975708],[1,532616,0.0996498408],[1,532617,0.0386341487],[1,532618,0.0402123732],[1,532619,0.1191305498],[1,532620,0.0551645987],[1,532621,0.0401456208],[1,532622,0.0970872238],[1,532623,0.0926122541],[1,532624,0.0402123732],[1,532627,0.0472939082],[1,532628,0.0380846502],[1,532634,0.039773399],[1,532636,0.0490160106],[1,532639,0.0849813112],[1,532640,0.040489044],[1,532643,0.0703468874],[1,532648,0.0420882445],[1,532652,0.0855966525],[1,532655,0.0781193918],[1,532663,0.040489044],[1,532664,0.0598393594],[1,532665,0.1286323059],[1,532666,0.1199028682],[1,532667,0.2004661856],[1,532668,0.1151337405],[1,532670,0.0400840234],[1,532671,0.1563119835],[1,532674,0.0370787413],[1,532675,0.0547594093],[1,532676,0.1370968333],[1,532677,0.0346435886],[1,532680,0.0456610008],[1,532686,0.1089737189],[1,532687,0.0367409961],[1,532689,0.0434085033],[1,532691,0.0607884731],[1,532696,0.4791576063],[1,532698,0.0500261943],[1,532699,0.1478233664],[1,532700,0.0557116398],[1,532702,0.0618998726],[1,532706,0.038702646],[1,532707,0.0783090204],[1,532708,0.040489044],[1,532709,0.184219391],[1,532714,0.0494625272],[1,532719,0.0434085033],[1,532724,0.0380122522],[1,532725,0.1182364921],[1,532726,0.6587599984],[1,532729,0.2318278516],[1,532730,0.1934062746],[1,532731,0.0707825981],[1,532732,0.0527744591],[1,532733,0.0483170324],[1,532734,0.0463925907],[1,532736,0.1169138111],[1,532744,0.0629435925],[1,532748,0.292036574],[1,532750,0.0358346734],[1,532751,0.0662691708],[1,532753,0.053403785],[1,532754,0.0931716542],[1,532757,0.0530814929],[1,532758,0.0961090231],[1,532764,0.2457588998],[1,532770,0.0578206843],[1,532771,0.0434085033],[1,532772,0.0520670469],[1,532775,0.0413148217],[1,532777,0.040489044],[1,532779,0.0383056672],[1,532782,0.0448833498],[1,532783,0.0804528263],[1,532786,0.3251852757],[1,532793,0.0661709586],[1,532796,0.1161342235],[1,532797,0.1502027663],[1,532798,0.042037931],[1,532799,0.0584157012],[1,532800,0.1002689764],[1,532808,0.0591464435],[1,532813,0.051389805],[1,532815,0.1169480033],[1,532816,0.0386899275],[1,532821,0.0436723179],[1,532823,0.0391265371],[1,532824,0.0491456843],[1,532826,0.513366408],[1,532832,0.0808879193],[1,532836,0.122210763],[1,532843,0.1850064628],[1,532844,0.03938274],[1,532845,0.0480079014],[1,532846,0.4213974957],[1,532848,0.055905049],[1,532853,0.1092171643],[1,532854,0.0468829596],[1,532855,0.0586465053],[1,532856,0.1044296442],[1,532857,0.0529423982],[1,532858,0.1139098658],[1,532860,0.248396053],[1,532864,0.0428383029],[1,532869,0.1074900337],[1,532870,0.0401214802],[1,532871,0.0468829596],[1,532872,0.0436723179],[1,532873,0.0468829596],[1,532874,0.0564840566],[1,532883,0.06268336],[1,532890,0.040436532],[1,532891,0.3420050633],[1,532892,0.0577771425],[1,532893,0.0983934967],[1,532894,0.0754669399],[1,532901,0.0439932741],[1,532902,0.1095368387],[1,532903,0.1168040754],[1,532905,0.2340920995],[1,532909,0.1317381983],[1,532913,0.0613489255],[1,532914,0.0939553128],[1,532915,0.0490387387],[1,532917,0.4228711969],[1,532918,0.1110590678],[1,532919,0.4529301019],[1,532921,0.1366774716],[1,532922,0.0429074659],[1,532923,0.0539060347],[1,532924,0.3672724822],[1,532928,0.1017310688],[1,532931,0.1183783423],[1,532932,0.0587881237],[1,532935,0.1104581197],[1,532936,0.2778576304],[1,532937,0.0446661311],[1,532942,0.1798514264],[1,532944,0.0440918604],[1,532945,0.0425902882],[1,532947,0.1198611732],[1,532953,0.0461576082],[1,532955,0.5673280178],[1,532956,0.0457794513],[1,532960,0.105840796],[1,532961,0.0468829596],[1,532963,0.0448096623],[1,532966,0.3720780321],[1,532967,0.0662818726],[1,532968,0.0445691535],[1,532971,0.0631443431],[1,532976,0.3016590388],[1,532977,0.0404954999],[1,532978,0.1307132116],[1,532980,0.0611400921],[1,532981,0.0386899275],[1,532984,0.0480389278],[1,532986,0.0502794951],[1,532990,0.042037931],[1,532995,0.0445691535],[1,532997,0.0607198456],[1,532998,0.0551049372],[1,532999,0.0386899275],[1,533002,0.2863136211],[1,533008,0.0436157618],[1,533012,0.0445863022],[1,533013,0.0429821034],[1,533015,0.0939898495],[1,533016,0.1395732863],[1,533023,0.0468829596],[1,533024,0.0511064621],[1,533025,0.0593163688],[1,533028,0.0606746029],[1,533032,0.1480784358],[1,533033,0.0936757485],[1,533034,0.0461576082],[1,533037,0.0842635317],[1,533042,0.1676763912],[1,533043,0.2042672391],[1,533044,0.0387896199],[1,533046,0.0425825401],[1,533048,0.061407373],[1,533052,0.0661759203],[1,533053,0.0857140137],[1,533057,0.0630726484],[1,533058,0.0709605655],[1,533067,0.0468829596],[1,533069,0.0800499882],[1,533070,0.0997797754],[1,533071,0.0509621933],[1,533087,0.0719941131],[1,533088,0.0386899275],[1,533089,0.042037931],[1,533090,0.0425902882],[1,533091,0.0386899275],[1,533092,0.1470067817],[1,533094,0.078261082],[1,533097,0.0386899275],[1,533099,0.0410019268],[1,533158,0.0444142805],[1,533167,0.3243715621],[1,533168,0.3719547452],[1,533169,0.0542375209],[1,533178,0.0848566541],[1,533183,0.0757810842],[1,533186,0.0987511082],[1,533194,0.1222888434],[1,533195,0.0436599756],[1,533203,0.2167821429],[1,533211,0.0657870751],[1,533214,0.0993098691],[1,533215,0.0779062468],[1,533220,0.0663563137],[1,533221,0.0525522995],[1,533223,0.3281777757],[1,533228,0.6980556525],[1,533229,0.0432313505],[1,533230,0.0445691535],[1,533233,0.1672926628],[1,533234,0.0436723179],[1,533241,0.1214125109],[1,533249,0.0900894353],[1,533252,0.0983934967],[1,533256,0.0548850924],[1,533258,0.0623643364],[1,533260,0.0566453242],[1,533266,0.088844165],[1,533267,0.0931636507],[1,533268,0.0733523708],[1,533270,0.2529876695],[1,533271,0.0770894517],[1,533272,0.0983934967],[1,533273,0.0946319791],[1,533274,0.0626177581],[1,533275,0.0441103849],[1,533278,0.050713581],[1,533281,0.2845885558],[1,533282,0.2015031998],[1,533283,0.146556551],[1,533284,0.0478750734],[1,533285,0.0440477056],[1,533286,0.0436723179],[1,533287,0.1168984173],[1,533293,0.0726608936],[1,533294,0.038335857],[1,533297,0.5404972794],[1,533298,0.0523876124],[1,533299,0.0496057813],[1,533301,0.1242461209],[1,533304,0.1050062627],[1,533305,0.0436723179],[1,533307,0.0643674564],[1,533311,0.053139687],[1,533316,0.2365429743],[1,533317,0.0677002121],[1,533321,0.2738582558],[1,533324,0.0410169274],[1,533325,0.0386899275],[1,533326,0.0987511082],[1,533329,0.1644697989],[1,533333,0.0563811144],[1,533337,0.2802670411],[1,533338,0.0545702182],[1,533339,0.1402664575],[1,533342,0.1026753731],[1,533343,0.1065965832],[1,533345,0.0531704673],[1,533348,0.0699949825],[1,533350,0.0828886],[1,533354,0.4838366088],[1,533355,0.0601368053],[1,533356,0.2758739727],[1,533357,0.0699986949],[1,533362,0.0962322861],[1,533369,0.0476685277],[1,533372,0.1163915699],[1,533373,0.1354684703],[1,533378,0.0532496584],[1,533379,0.1059334656],[1,533380,0.0688688023],[1,533381,0.0623727517],[1,533385,0.1141648665],[1,533387,0.0914879674],[1,533388,0.0547038239],[1,533395,0.0695702313],[1,533396,0.0594332348],[1,533398,0.1525543252],[1,533399,0.2973120553],[1,533401,0.0508959302],[1,533402,0.1000773192],[1,533404,0.0606623095],[1,533409,0.5678087834],[1,533411,0.135759673],[1,533414,0.1030348514],[1,533415,0.4832925038],[1,533416,0.0660835736],[1,533419,0.2653983839],[1,533425,0.1731696324],[1,533426,0.0517690534],[1,533427,0.0476685277],[1,533428,0.12455782],[1,533429,0.0476011221],[1,533430,0.0440627882],[1,533434,0.0621748256],[1,533435,0.0714951801],[1,533441,0.061681424],[1,533442,0.0437784471],[1,533444,0.0460981287],[1,533445,0.0649879431],[1,533446,0.050590815],[1,533448,0.1608849209],[1,533452,0.0481088816],[1,533455,0.1774661785],[1,533456,0.0481088816],[1,533460,0.0740605862],[1,533468,0.1026753731],[1,533469,0.0587574245],[1,533472,0.0484620252],[1,533476,0.0858063658],[1,533477,0.2732370962],[1,533483,0.0732504445],[1,533486,0.0545347437],[1,533487,0.0472252583],[1,533489,0.0776112586],[1,533494,0.3842261348],[1,533496,0.0478105121],[1,533497,0.0938461875],[1,533498,0.0462489352],[1,533501,0.0724071566],[1,533503,0.115783634],[1,533505,0.0449837468],[1,533506,0.3682996765],[1,533512,0.1030348514],[1,533514,0.2922778819],[1,533516,0.1394705451],[1,533518,0.0704659021],[1,533519,0.135524153],[1,533522,0.3932660895],[1,533524,0.1030348514],[1,533526,0.3130220318],[1,533529,0.0807093501],[1,533539,0.0588651941],[1,533540,0.0481088816],[1,533542,0.0466638023],[1,533546,0.1741341365],[1,533547,0.0525623537],[1,533548,0.0530903729],[1,533549,0.0436781008],[1,533550,0.0998253526],[1,533552,0.6679150753],[1,533553,0.1497189836],[1,533554,0.1272101657],[1,533556,0.1290690048],[1,533558,0.1538227796],[1,533559,0.0601368053],[1,533560,0.3134711636],[1,533562,0.0824764779],[1,533564,0.0818978826],[1,533565,0.0525623537],[1,533573,0.054687648],[1,533574,0.0476685277],[1,533575,0.0460256086],[1,533576,0.0572107337],[1,533578,0.0903088233],[1,533579,0.0508959302],[1,533580,0.0510952708],[1,533581,0.1804023759],[1,533582,0.0609552784],[1,533584,0.1026753731],[1,533590,0.2738623911],[1,533592,0.1122635267],[1,533594,0.060110512],[1,533595,0.0713738237],[1,533597,0.0476685277],[1,533598,0.356085527],[1,533601,0.0491169811],[1,533602,0.0483896435],[1,533604,0.0704250059],[1,533613,0.0498615682],[1,533614,0.053561759],[1,533615,0.0436781008],[1,533616,0.0723097664],[1,533617,0.0481088816],[1,533618,0.0732241545],[1,533625,0.0558524868],[1,533626,0.2088390264],[1,533630,0.0683865546],[1,533632,0.1026753731],[1,533633,0.091624605],[1,533634,0.0463090098],[1,533635,0.2504373456],[1,533638,0.0621589884],[1,533642,0.1833290612],[1,533643,0.0486193487],[1,533644,0.0988835146],[1,533647,0.2698388654],[1,533652,0.0430990161],[1,533653,0.0648613359],[1,533654,0.1948851446],[1,533655,0.0442325863],[1,533656,0.4170233547],[1,533657,0.2543350676],[1,533659,0.2692325463],[1,533660,0.0476685277],[1,533661,0.114966247],[1,533663,0.0483622118],[1,533671,0.5714965824],[1,533674,0.0520927865],[1,533676,0.0497565307],[1,533677,0.0460256086],[1,533682,0.0666429835],[1,533683,0.1073524499],[1,533684,0.094746268],[1,533685,0.1074517542],[1,533686,0.1416550638],[1,533687,0.2468611424],[1,533688,0.7392532412],[1,533689,0.3341482015],[1,533692,0.1677776831],[1,533693,0.1026753731],[1,533698,0.0482172979],[1,533704,0.0866165329],[1,533705,0.1453481887],[1,533707,0.0436781008],[1,533711,0.0495627534],[1,533716,0.0711009751],[1,533721,0.2173455168],[1,533722,0.2079697364],[1,533723,0.0491893212],[1,533725,0.1612984479],[1,533726,0.3822313813],[1,533730,0.0464300619],[1,533731,0.0476685277],[1,533732,0.0460256086],[1,533733,0.0495627534],[1,533734,0.6575168225],[1,533735,0.1089224352],[1,533737,0.0425363693],[1,533742,0.2112830688],[1,533743,0.0479842415],[1,533744,0.0486481695],[1,533745,0.0527332335],[1,533746,0.0474285394],[1,533747,0.2808932389],[1,533748,0.0703858556],[1,533754,0.0775013445],[1,533756,0.0474765643],[1,533757,0.0450121342],[1,533759,0.0772759732],[1,533760,0.0904782231],[1,533762,0.1895651392],[1,533772,0.057242734],[1,533776,0.0439751622],[1,533777,0.0473061774],[1,533778,0.5750090592],[1,533779,0.0435667706],[1,533783,0.0473061774],[1,533784,0.0809790279],[1,533785,0.6150241806],[1,533788,0.0424248485],[1,533789,0.0530450079],[1,533790,0.2516765217],[1,533792,0.2045739274],[1,533794,0.0477489587],[1,533795,0.1455288018],[1,533796,0.1377006036],[1,533797,0.0883917352],[1,533801,0.0530501277],[1,533809,0.1423436758],[1,533810,0.1173460156],[1,533811,0.0530450079],[1,533814,0.0765366148],[1,533816,0.086069752],[1,533820,0.1524229244],[1,533826,0.0808070165],[1,533829,0.0623096723],[1,533833,0.0655924708],[1,533841,0.0439751622],[1,533842,0.0608380493],[1,533843,0.0512056673],[1,533845,0.1915958569],[1,533846,0.2895505319],[1,533847,0.0718241902],[1,533848,0.0598465117],[1,533849,0.321253104],[1,533851,0.0567849023],[1,533852,0.0490790008],[1,533853,0.0553929219],[1,533855,0.0489800602],[1,533863,0.049437878],[1,533864,0.4426065656],[1,533865,0.1031443771],[1,533866,0.4122059174],[1,533867,0.1948670695],[1,533868,0.1127229879],[1,533869,0.1134842736],[1,533870,0.076437932],[1,533871,0.5194459352],[1,533874,0.1236657428],[1,533884,0.0471007613],[1,533889,0.0721605949],[1,533890,0.0464564567],[1,533892,0.1527524893],[1,533893,0.1513099588],[1,533894,0.0592645814],[1,533895,0.0786376028],[1,533896,0.0693954599],[1,533897,0.0564526044],[1,533903,0.0691735937],[1,533905,0.1292305772],[1,533906,0.0439388032],[1,533907,0.0459013058],[1,533908,0.1089576914],[1,533909,0.1495371673],[1,533910,0.5317705501],[1,533911,0.0431241412],[1,533912,0.213676043],[1,533913,0.0425363693],[1,533915,0.0656896376],[1,533919,0.0513481897],[1,533920,0.1218912831],[1,533921,0.0552872414],[1,533925,0.1355914243],[1,533927,0.0681149552],[1,533928,0.0930212538],[1,533929,0.1205680872],[1,533930,0.0720835045],[1,533931,0.3166410803],[1,533936,0.0641600876],[1,533939,0.2142880285],[1,533940,0.1479727938],[1,533947,0.050770839],[1,533949,0.0425363693],[1,533951,0.0671470373],[1,533952,0.0425363693],[1,533955,0.0724080185],[1,533956,0.1485522083],[1,533957,0.1322841593],[1,533959,0.0481783419],[1,533966,0.049437878],[1,533967,0.0766580692],[1,533969,0.1071080254],[1,533970,0.1197788584],[1,533977,0.0567249526],[1,533990,0.2060637519],[1,533991,0.3622177326],[1,533992,0.2918001203],[1,533993,0.072230027],[1,533994,0.0488105776],[1,533995,0.0803969019],[1,533996,0.1372872801],[1,533997,0.0400245517],[1,533998,0.049437878],[1,534003,0.0661889505],[1,534004,0.0425363693],[1,534005,0.2156055363],[1,534007,0.1025418996],[1,534008,0.0806169801],[1,534009,0.077434759],[1,534017,0.0552765468],[1,534019,0.0425363693],[1,534021,0.0479842415],[1,534022,0.049437878],[1,534024,0.3099889661],[1,534025,0.0993267526],[1,534026,0.0545731007],[1,534027,0.0683537249],[1,534029,0.0492789397],[1,534032,0.1171422772],[1,534036,0.0534850526],[1,534037,0.0638830082],[1,534043,0.0861315396],[1,534044,0.272826462],[1,534049,0.1046500893],[1,534051,0.061806779],[1,534054,0.0629852616],[1,534057,0.1434394455],[1,534060,0.0535897383],[1,534061,0.0464564567],[1,534062,0.0681117247],[1,534063,0.0425363693],[1,534066,0.0459013058],[1,534067,0.0786360188],[1,534068,0.1608189928],[1,534069,0.0658473079],[1,534070,0.0439953315],[1,534076,0.1483697483],[1,534077,0.1046930485],[1,534078,0.0727218771],[1,534079,0.0425363693],[1,534080,0.1128241612],[1,534081,0.0445640405],[1,534083,0.1649307451],[1,534085,0.0622458026],[1,534091,0.0663501771],[1,534092,0.0662099061],[1,534093,0.0581500853],[1,534095,0.3478663521],[1,534099,0.2457116309],[1,534101,0.0484625659],[1,534103,0.0530450079],[1,534111,0.0762041125],[1,534112,0.0522585788],[1,534113,0.0578322175],[1,534114,0.1513661204],[1,534115,0.0502289107],[1,534116,0.078564407],[1,534117,0.0419845997],[1,534118,0.1082909485],[1,534123,0.0530450079],[1,534134,0.3143095853],[1,534136,0.6741466571],[1,534138,0.272490698],[1,534139,0.0464386044],[1,534140,0.1644841236],[1,534143,0.0541921868],[1,534144,0.2937956621],[1,534146,0.0486872374],[1,534148,0.0720918428],[1,534150,0.0475439589],[1,534152,0.0954338977],[1,534154,0.0530450079],[1,534156,0.0479842415],[1,534160,0.1270613324],[1,534164,0.083728107],[1,534165,0.0453032463],[1,534166,0.0844145675],[1,534168,0.0610971049],[1,534169,0.0475439589],[1,534170,0.0561769354],[1,534171,0.0475439589],[1,534172,0.1427657578],[1,534173,0.0790446875],[1,534174,0.1106431598],[1,534175,0.0701592463],[1,534176,0.0479212452],[1,534177,0.0479212452],[1,534188,0.1042168669],[1,534189,0.0828577179],[1,534191,0.0563081478],[1,534192,0.0563081478],[1,534197,0.0717162006],[1,534202,0.3980127876],[1,534203,0.0485108205],[1,534206,0.0718358815],[1,534209,0.0906314374],[1,534210,0.0500337032],[1,534211,0.0530581402],[1,534216,0.1363677112],[1,534226,0.1150308871],[1,534227,0.2522974124],[1,534228,0.0683886391],[1,534229,0.058122086],[1,534230,0.0535015784],[1,534231,0.0570296956],[1,534235,0.0501202448],[1,534248,0.0519569693],[1,534249,0.096551213],[1,534251,0.0738150789],[1,534253,0.0600614049],[1,534255,0.0590486576],[1,534256,0.0641905133],[1,534257,0.0520001251],[1,534258,0.0484566233],[1,534259,0.4952315902],[1,534260,0.0623805312],[1,534261,0.0930856132],[1,534262,0.3089545923],[1,534271,0.0540156209],[1,534273,0.0815488248],[1,534275,0.0705327281],[1,534278,0.0740799096],[1,534283,0.0534381305],[1,534287,0.0652983646],[1,534288,0.0988884037],[1,534289,0.0519628437],[1,534290,0.0695551354],[1,534292,0.093412921],[1,534293,0.054006867],[1,534294,0.051403714],[1,534295,0.0653447341],[1,534296,0.0927925057],[1,534298,0.0572320319],[1,534299,0.0519448635],[1,534305,0.1980959335],[1,534307,0.0714013156],[1,534309,0.0730642595],[1,534314,0.0516891002],[1,534315,0.0480146605],[1,534316,0.0766235324],[1,534318,0.051403714],[1,534320,0.0530009429],[1,534321,0.048606645],[1,534322,0.051403714],[1,534325,0.1202998272],[1,534328,0.2347660639],[1,534329,0.1142872164],[1,534332,0.2657063209],[1,534340,0.0563081478],[1,534342,0.0480146605],[1,534343,0.049039764],[1,534345,0.0523337051],[1,534349,0.3616318288],[1,534356,0.1319944952],[1,534358,0.0625168704],[1,534362,0.345849745],[1,534363,0.1255224627],[1,534365,0.051403714],[1,534368,0.1204901292],[1,534370,0.0521359785],[1,534371,0.0725046666],[1,534376,0.221942466],[1,534380,0.1279233002],[1,534381,0.0625736383],[1,534382,0.2154258247],[1,534383,0.1014352117],[1,534388,0.1044582471],[1,534390,0.0914504553],[1,534391,0.0519448635],[1,534392,0.051403714],[1,534396,0.0747985622],[1,534398,0.0499101903],[1,534399,0.0516891002],[1,534403,0.1052511043],[1,534404,0.0519702897],[1,534407,0.1584867027],[1,534409,0.0535015784],[1,534411,0.0535015784],[1,534413,0.2111686149],[1,534414,0.0557623354],[1,534416,0.0523704627],[1,534422,0.0480146605],[1,534426,0.048606645],[1,534427,0.0577254988],[1,534429,0.054628199],[1,534430,0.0563081478],[1,534431,0.0549656332],[1,534436,0.057513387],[1,534437,0.1134867611],[1,534438,0.0561900345],[1,534439,0.3304679134],[1,534442,0.0561900345],[1,534450,0.0608003684],[1,534451,0.0763542384],[1,534452,0.0797639038],[1,534453,0.2828604742],[1,534454,0.2425266565],[1,534455,0.1656200857],[1,534456,0.1813418281],[1,534459,0.051403714],[1,534460,0.0700497651],[1,534462,0.139262582],[1,534472,0.0518110001],[1,534473,0.129429374],[1,534474,0.0711295997],[1,534476,0.0503545549],[1,534485,0.11537539],[1,534486,0.0563081478],[1,534487,0.1293502786],[1,534488,0.1737495512],[1,534489,0.054006867],[1,534490,0.0523704627],[1,534491,0.0519599055],[1,534495,0.0759711541],[1,534496,0.0534475413],[1,534497,0.0723091282],[1,534510,0.0585986159],[1,534512,0.0481155752],[1,534513,0.0613735616],[1,534515,0.052258656],[1,534516,0.375796213],[1,534524,0.1551250638],[1,534525,0.0530581402],[1,534527,0.0533671266],[1,534528,0.0563081478],[1,534529,0.2310115535],[1,534533,0.2620904203],[1,534534,0.0919689524],[1,534536,0.2453873593],[1,534538,0.0535015784],[1,534539,0.0661902094],[1,534540,0.051403714],[1,534541,0.0639992926],[1,534549,0.0524938424],[1,534553,0.0906497947],[1,534555,0.0653513763],[1,534557,0.1736025274],[1,534561,0.0633690246],[1,534562,0.0584232412],[1,534566,0.3920997854],[1,534567,0.051403714],[1,534569,0.1434899527],[1,534570,0.1042168669],[1,534571,0.0637403377],[1,534575,0.1087367051],[1,534576,0.3044136625],[1,534587,0.0594436727],[1,534589,0.1852726493],[1,534590,0.1599518557],[1,534591,0.1374882947],[1,534600,0.0811687781],[1,534601,0.0526117661],[1,534602,0.0535015784],[1,534603,0.1041043673],[1,534605,0.2072865138],[1,534608,0.0970096223],[1,534609,0.288398654],[1,534613,0.066962522],[1,534615,0.052014085],[1,534616,0.3347399098],[1,534621,0.2870683619],[1,534623,0.0561943569],[1,534624,0.0553097233],[1,534631,0.1175207234],[1,534633,0.0949917606],[1,534634,0.0535015784],[1,534635,0.0563081478],[1,534636,0.0561900345],[1,534637,0.1133676073],[1,534638,0.0553097233],[1,534643,0.0765858647],[1,534644,0.1220222721],[1,534645,0.16111567],[1,534647,0.1371817395],[1,534648,0.1584023119],[1,534655,0.0662396265],[1,534658,0.0571776977],[1,534659,0.2812455475],[1,534669,0.1813359791],[1,534672,0.1129418145],[1,534673,0.104068027],[1,534674,0.1215979035],[1,534679,0.2149284826],[1,534681,0.1081749478],[1,534685,0.0541074019],[1,534686,0.0573618102],[1,534689,0.0610516906],[1,534690,0.051102062],[1,534691,0.0541074019],[1,534692,0.1231565498],[1,534694,0.0975111622],[1,534696,0.0595239385],[1,534697,0.0730035775],[1,534705,0.0673279684],[1,534707,0.0652760721],[1,534708,0.0614933594],[1,534709,0.0625310134],[1,534710,0.0560174778],[1,534711,0.0550661791],[1,534713,0.1001310107],[1,534715,0.0565561153],[1,534722,0.0667951389],[1,534723,0.0729209054],[1,534726,0.0722158165],[1,534727,0.1005479672],[1,534728,0.0524507354],[1,534732,0.2304108577],[1,534736,0.0836473435],[1,534737,0.0592066189],[1,534739,0.0563320164],[1,534742,0.092020528],[1,534743,0.0573618102],[1,534754,0.2286531611],[1,534757,0.2580747668],[1,534766,0.05050816],[1,534769,0.0532849766],[1,534772,0.2766795861],[1,534774,0.4403330783],[1,534776,0.0580084044],[1,534778,0.0541525772],[1,534779,0.1571234248],[1,534807,0.5527660883],[1,534812,0.4393171656],[1,534817,0.049649879],[1,534822,0.1235071749],[1,534826,0.0659833314],[1,534828,0.0750240935],[1,534829,0.0760824458],[1,534831,0.1252939722],[1,534834,0.0533321236],[1,534839,0.0596553799],[1,534840,0.1632356353],[1,534841,0.0618018746],[1,534846,0.0535248085],[1,534855,0.0524507354],[1,534856,0.2233645017],[1,534857,0.1023357328],[1,534859,0.1010187566],[1,534860,0.0524900196],[1,534862,0.0969329103],[1,534864,0.0804394941],[1,534867,0.1258304401],[1,534868,0.0688425029],[1,534869,0.1336000841],[1,534877,0.0680893562],[1,534879,0.1737491407],[1,534883,0.1056960402],[1,534884,0.0524507354],[1,534887,0.105335402],[1,534888,0.2948340921],[1,534889,0.1624048832],[1,534891,0.3793761266],[1,534892,0.052736508],[1,534893,0.0701366067],[1,534894,0.105335402],[1,534901,0.7192202708],[1,534902,0.1056960402],[1,534904,0.0545514405],[1,534907,0.0545514405],[1,534908,0.2749497366],[1,534909,0.1476933153],[1,534911,0.1030239073],[1,534912,0.1811701033],[1,534915,0.3546285587],[1,534918,0.1019759813],[1,534921,0.2928010703],[1,534923,0.0640534496],[1,534933,0.1113065237],[1,534934,0.0659280122],[1,534935,0.2142541015],[1,534936,0.052736508],[1,534937,0.2416151034],[1,534943,0.0955697404],[1,534947,0.123119518],[1,534954,0.0666679353],[1,534960,0.0568812852],[1,534963,0.0907220097],[1,534965,0.0524507354],[1,534968,0.0915297354],[1,534969,0.0712546466],[1,534974,0.1502118247],[1,534977,0.0734123678],[1,534982,0.0714116912],[1,534989,0.0556138317],[1,534990,0.0722108108],[1,534992,0.0940516075],[1,534993,0.0637407299],[1,534995,0.0488979113],[1,534996,0.0761713154],[1,534997,0.0560174778],[1,534998,0.0524507354],[1,535003,0.0979888832],[1,535004,0.0807779117],[1,535005,0.1431340682],[1,535006,0.1489081623],[1,535007,0.0534824498],[1,535008,0.0572930813],[1,535011,0.0513802443],[1,535014,0.5481979961],[1,535018,0.0545514405],[1,535019,0.0622278758],[1,535021,0.0650428722],[1,535024,0.0740014828],[1,535026,0.0729150823],[1,535028,0.0588125978],[1,535033,0.0649633476],[1,535035,0.5625257015],[1,535043,0.0560174778],[1,535045,0.1321937701],[1,535046,0.0530106222],[1,535047,0.1141434264],[1,535051,0.0840151531],[1,535059,0.0596553799],[1,535060,0.4845530748],[1,535061,0.0817161786],[1,535064,0.0550661791],[1,535066,0.0736050933],[1,535068,0.0544879068],[1,535069,0.3600060107],[1,535070,0.3974493793],[1,535073,0.1569844933],[1,535077,0.0684084056],[1,535079,0.0822039929],[1,535081,0.0601751327],[1,535082,0.3010431848],[1,535083,0.6437101601],[1,535085,0.0596553799],[1,535087,0.1886880511],[1,535091,0.0662730251],[1,535092,0.0529960895],[1,535094,0.2381843507],[1,535097,0.3533302511],[1,535102,0.0927302123],[1,535106,0.1025970457],[1,535109,0.0714122058],[1,535110,0.0596609749],[1,535111,0.1052196437],[1,535114,0.5298671162],[1,535116,0.0729987933],[1,535118,0.5046856567],[1,535119,0.0596553799],[1,535121,0.0585686815],[1,535122,0.0694172568],[1,535123,0.1138522818],[1,535125,0.0832302345],[1,535129,0.4478590496],[1,535135,0.0544122891],[1,535136,0.1935175703],[1,535137,0.3070956482],[1,535140,0.0523471453],[1,535143,0.0691820939],[1,535149,0.2016935148],[1,535150,0.0758862997],[1,535151,0.0525701463],[1,535152,0.0780773085],[1,535153,0.0526759307],[1,535154,0.0490570928],[1,535155,0.0490570928],[1,535157,0.0648537573],[1,535160,0.4277756125],[1,535169,0.0915453027],[1,535171,0.1531987521],[1,535172,0.1691598091],[1,535174,0.172391141],[1,535176,0.105335402],[1,535177,0.0672894728],[1,535179,0.6834365237],[1,535180,0.243501306],[1,535181,0.0508796621],[1,535190,0.1706533696],[1,535191,0.0712729067],[1,535194,0.0752825592],[1,535195,0.1573601276],[1,535196,0.0634478367],[1,535197,0.2546931488],[1,535198,0.206485621],[1,535199,0.0601418989],[1,535203,0.5781688015],[1,535206,0.4344909417],[1,535207,0.2105136111],[1,535208,0.0541074019],[1,535210,0.0919689524],[1,535211,0.0510949031],[1,535213,0.1087708916],[1,535214,0.0720995951],[1,535215,0.093771426],[1,535216,0.1250265458],[1,535218,0.4469280793],[1,535219,0.05050816],[1,535221,0.0659833314],[1,535222,0.6017432903],[1,535223,0.0805465836],[1,535224,0.0550338454],[1,535227,0.0573170744],[1,535228,0.0909632987],[1,535231,0.1368353728],[1,535232,0.1225668062],[1,535235,0.06818538],[1,535247,0.2201573912],[1,535248,0.0525701463],[1,535249,0.084163425],[1,535250,0.0514006087],[1,535251,0.0548963563],[1,535252,0.0881351989],[1,535258,0.0524900196],[1,535263,0.1260470333],[1,535266,0.0742622596],[1,535268,0.1838415991],[1,535270,0.1450535801],[1,535271,0.0912920427],[1,535272,0.6147445119],[1,535273,0.0551313349],[1,535274,0.0490570928],[1,535279,0.0541074019],[1,535280,0.0544879068],[1,535283,0.2147158558],[1,535288,0.052736508],[1,535290,0.0540501271],[1,535291,0.5158715875],[1,535292,0.0663314387],[1,535295,0.4592570229],[1,535297,0.067547299],[1,535298,0.0616221028],[1,535299,0.0941459445],[1,535300,0.1680818569],[1,535301,0.4743455972],[1,535306,0.0666800099],[1,535307,0.0641250097],[1,535309,0.1303564114],[1,535310,0.2679943074],[1,535311,0.0560174778],[1,535312,0.1703586078],[1,535316,0.1633170314],[1,535320,0.1419696104],[1,535325,0.0594847466],[1,535326,0.0540501271],[1,535328,0.0593409458],[1,535329,0.0545514405],[1,535330,0.05067716],[1,535332,0.1701268677],[1,535334,0.0524507354],[1,535341,0.0637379501],[1,535342,0.2270898971],[1,535343,0.0544168067],[1,535344,0.0545565958],[1,535346,0.2847566878],[1,535351,0.0703713685],[1,535352,0.0861816397],[1,535353,0.132210872],[1,535354,0.0572854652],[1,535355,0.2452339201],[1,535358,0.1020496739],[1,535359,0.076726639],[1,535360,0.2036208618],[1,535361,0.052736508],[1,535363,0.7214332737],[1,535364,0.1438620947],[1,535367,0.3783329204],[1,535369,0.1331482541],[1,535373,0.0512755884],[1,535374,0.0541525772],[1,535375,0.0748984968],[1,535376,0.5951181506],[1,535377,0.057243537],[1,535378,0.0607862546],[1,535380,0.1833482804],[1,535381,0.0545514405],[1,535382,0.2699611684],[1,535384,0.0603132475],[1,535389,0.0712573212],[1,535390,0.2528255738],[1,535392,0.0725912718],[1,535393,0.2898179386],[1,535397,0.0530003942],[1,535398,0.0579401083],[1,535399,0.0490570928],[1,535400,0.1217966],[1,535401,0.085988963],[1,535402,0.0635262235],[1,535404,0.1056960402],[1,535405,0.1446134913],[1,535408,0.0883579474],[1,535409,0.0810080325],[1,535410,0.1504594974],[1,535412,0.0634137358],[1,535413,0.2763036342],[1,535420,0.0751030002],[1,535422,0.0889668543],[1,535423,0.0536604234],[1,535424,0.0740799096],[1,535425,0.0685753626],[1,535426,0.1676257579],[1,535427,0.0947916687],[1,535429,0.0505285014],[1,535430,0.1610811741],[1,535431,0.2802597159],[1,535432,0.4946479567],[1,535433,0.1083500167],[1,535434,0.0644652742],[1,535437,0.6177670703],[1,535438,0.4659485234],[1,535439,0.1052227501],[1,535448,0.1039770913],[1,535449,0.1449129386],[1,535450,0.0620033601],[1,535451,0.0850533936],[1,535452,0.2996100042],[1,535455,0.0541074019],[1,535456,0.0628585792],[1,535457,0.0536604234],[1,535459,0.2705840896],[1,535462,0.0545900176],[1,535464,0.0620330983],[1,535465,0.0536604234],[1,535466,0.1683201255],[1,535467,0.1230332073],[1,535468,0.0575339315],[1,535472,0.0743974357],[1,535479,0.0586228483],[1,535480,0.105335402],[1,535483,0.0658327324],[1,535484,0.0569070316],[1,535485,0.2924708134],[1,535486,0.0766163519],[1,535488,0.061785031],[1,535489,0.0728390072],[1,535490,0.2216764035],[1,535491,0.0550338454],[1,535492,0.0708792261],[1,535503,0.449247727],[1,535504,0.212727342],[1,535505,0.071798963],[1,535506,0.2766154324],[1,535508,0.2168983527],[1,535510,0.3382288376],[1,535511,0.1400369609],[1,535512,0.0541074019],[1,535513,0.0652533814],[1,535514,0.105335402],[1,535517,0.2893561977],[1,535518,0.0751710371],[1,535520,0.0551147625],[1,535530,0.3060790959],[1,535533,0.3389637864],[1,535534,0.0490570928],[1,535535,0.2167676133],[1,535537,0.0685437365],[1,535538,0.0573618102],[1,535539,0.0524507354],[1,535540,0.1111448223],[1,535542,0.1056960402],[1,535543,0.0766026087],[1,535544,0.107658148],[1,535546,0.0927960371],[1,535548,0.0653977583],[1,535559,0.150004743],[1,535561,0.0524507354],[1,535562,0.1443527804],[1,535564,0.1282172081],[1,535571,0.1021122005],[1,535572,0.0524507354],[1,535573,0.4876348291],[1,535575,0.0736144562],[1,535581,0.2766833714],[1,535587,0.0661010455],[1,535588,0.0793900772],[1,535602,0.0555557545],[1,535603,0.216752452],[1,535604,0.5692930019],[1,535606,0.0541199079],[1,535607,0.0525701463],[1,535609,0.0524507354],[1,535610,0.0524507354],[1,535611,0.2265957528],[1,535612,0.0541074019],[1,535615,0.0650811469],[1,535617,0.1837233798],[1,535629,0.0641516251],[1,535630,0.0560174778],[1,535631,0.0545514405],[1,535632,0.0544879068],[1,535634,0.089153392],[1,535635,0.0524507354],[1,535636,0.0541074019],[1,535638,0.0841107258],[1,535640,0.1751956972],[1,535641,0.0618519458],[1,535649,0.5664401328],[1,535650,0.0545514405],[1,535651,0.0545565958],[1,535652,0.0610815433],[1,535653,0.0530106222],[1,535654,0.0541074019],[1,535657,0.1331635742],[1,535659,0.0872010585],[1,535661,0.0806760743],[1,535663,0.0957308074],[1,535667,0.0555051971],[1,535675,0.4560626837],[1,535676,0.0738150789],[1,535678,0.1324034582],[1,535681,0.1192412647],[1,535682,0.192340091],[1,535683,0.0550722119],[1,535686,0.0586228483],[1,535688,0.3864373423],[1,535694,0.4202314635],[1,535695,0.0569780615],[1,535698,0.5018850221],[1,535699,0.1348281179],[1,535703,0.2524514616],[1,535704,0.0630400551],[1,535705,0.0863674061],[1,535707,0.125471544],[1,535712,0.1216913094],[1,535713,0.0665427636],[1,535714,0.074410789],[1,535715,0.1026908065],[1,535716,0.0724918769],[1,535718,0.0536604234],[1,535720,0.0524507354],[1,535721,0.0649964873],[1,535724,0.0759316104],[1,535725,0.0524507354],[1,535727,0.3234233735],[1,535729,0.2046960563],[1,535732,0.1702034957],[1,535735,0.0821340656],[1,535736,0.0823718472],[1,535739,0.0536604234],[1,535741,0.0830471705],[1,535744,0.0758862997],[1,535745,0.0673216259],[1,535751,0.0524507354],[1,535755,0.2605652446],[1,535758,0.0563187931],[1,535763,0.1305485859],[1,535765,0.3882575129],[1,535766,0.0564127147],[1,535768,0.0536604234],[1,535769,0.0791111496],[1,535772,0.0550227577],[1,535773,0.1386790397],[1,535774,0.0672704102],[1,535776,0.0807020093],[1,535779,0.0579664397],[1,535780,0.1022949463],[1,535789,0.0662166352],[1,535790,0.0524507354],[1,535795,0.0580142626],[1,535796,0.0808522903],[1,535797,0.051122419],[1,535798,0.156221068],[1,535799,0.5170848668],[1,535800,0.0545514405],[1,535802,0.0608527968],[1,535805,0.0549008566],[1,535811,0.1790345762],[1,535818,0.1397466784],[1,535820,0.0593409458],[1,535821,0.1855441539],[1,535823,0.1429526219],[1,535824,0.0601842984],[1,535825,0.0541074019],[1,535826,0.0834946787],[1,535831,0.1077574518],[1,535844,0.1095783584],[1,535845,0.1293205891],[1,535846,0.1965513139],[1,535847,0.0836038901],[1,535849,0.0571534396],[1,535851,0.0933098254],[1,535853,0.0544879068],[1,535854,0.0852831901],[1,535855,0.1466646867],[1,535856,0.1802913127],[1,535858,0.080323365],[1,535860,0.0849232204],[1,535861,0.291322595],[1,535864,0.0654141593],[1,535865,0.071587068],[1,535869,0.0595706813],[1,535871,0.0679191845],[1,535872,0.0565341588],[1,535873,0.1246051918],[1,535875,0.0545514405],[1,535876,0.3066899418],[1,535878,0.2647803131],[1,535886,0.3143213979],[1,535889,0.0548068906],[1,535895,0.0482767361],[1,535903,0.0539855674],[1,535904,0.1160508163],[1,535905,0.0523713562],[1,535906,0.1802836505],[1,535907,0.1261526239],[1,535912,0.0578891764],[1,535920,0.0490570928],[1,535921,0.0575255555],[1,535922,0.0657858571],[1,535923,0.0571545426],[1,535924,0.0821243005],[1,535925,0.0752477357],[1,535926,0.1943930139],[1,535928,0.088725839],[1,535932,0.2348749211],[1,535935,0.0748475368],[1,535938,0.0524507354],[1,535939,0.0552299524],[1,535942,0.076147706],[1,535943,0.0524507354],[1,535944,0.150624928],[1,535946,0.1485535767],[1,535948,0.081228237],[1,535953,0.0533845527],[1,535958,0.0713955583],[1,535959,0.0813129896],[1,535962,0.0832302345],[1,535963,0.0622998033],[1,535964,0.0759433517],[1,535969,0.1035600106],[1,535972,0.2454690155],[1,535983,0.0520342535],[1,535986,0.1562051537],[1,535988,0.1691165039],[1,535990,0.0601842984],[1,535992,0.0816559814],[1,535993,0.1581954916],[1,535994,0.2795508499],[1,535996,0.1996072426],[1,535998,0.148862913],[1,536000,0.112468101],[1,536002,0.197576335],[1,536003,0.0720318341],[1,536004,0.1120593257],[1,536008,0.0672220596],[1,536009,0.051122419],[1,536012,0.1626137194],[1,536017,0.0666928357],[1,536021,0.0610216028],[1,536022,0.0593211575],[1,536023,0.0897671786],[1,536024,0.0764987146],[1,536025,0.0680222033],[1,536030,0.0601842984],[1,536033,0.0525701463],[1,536042,0.1053802764],[1,536043,0.0681382715],[1,536044,0.0585335364],[1,536045,0.1963197317],[1,536047,0.1022815943],[1,536049,0.0678442187],[1,536051,0.105335402],[1,536052,0.1467384799],[1,536064,0.090603126],[1,536065,0.053595896],[1,536067,0.0519234869],[1,536068,0.0645488239],[1,536070,0.056595044],[1,536071,0.0524507354],[1,536073,0.0524507354],[1,536076,0.1411378717],[1,536077,0.0537635299],[1,536083,0.1839878151],[1,536097,0.1099419976],[1,536099,0.071782993],[1,536102,0.0555391045],[1,536103,0.0528959417],[1,536104,0.1058078922],[1,536106,0.103662668],[1,536125,0.0551580844],[1,536126,0.2100135535],[1,536128,0.0534991746],[1,536129,0.0501009367],[1,536131,0.077145327],[1,536136,0.1073347587],[1,536140,0.0566891558],[1,536145,0.0696722978],[1,536146,0.1078182073],[1,536148,0.0501009367],[1,536149,0.1080680157],[1,536151,0.1046344106],[1,536152,0.2000933229],[1,536155,0.0534991746],[1,536156,0.0925152037],[1,536159,0.0555391045],[1,536161,0.1035632825],[1,536172,0.054431686],[1,536173,0.0556027242],[1,536174,0.0537103983],[1,536175,0.0818163151],[1,536177,0.0737917231],[1,536179,0.1064554517],[1,536180,0.0671328425],[1,536181,0.0578780185],[1,536185,0.1309382679],[1,536186,0.0590505952],[1,536187,0.0958565091],[1,536190,0.0875316373],[1,536191,0.0534991746],[1,536198,0.2186715727],[1,536199,0.0534991746],[1,536200,0.0894870987],[1,536201,0.0540598195],[1,536203,0.1246835059],[1,536204,0.0549071055],[1,536213,0.1001697716],[1,536216,0.1085536927],[1,536217,0.2398661334],[1,536218,0.0938631345],[1,536219,0.0565948671],[1,536221,0.1512356371],[1,536233,0.1392011226],[1,536244,0.0537246749],[1,536245,0.062087491],[1,536246,0.1568017798],[1,536247,0.0797226956],[1,536248,0.0848198021],[1,536249,0.0618539981],[1,536250,0.0594416414],[1,536254,0.0846945134],[1,536256,0.1239388008],[1,536257,0.0501009367],[1,536258,0.0540598195],[1,536259,0.0696129598],[1,536261,0.0575286495],[1,536263,0.073264702],[1,536264,0.0560099514],[1,536273,0.0628227357],[1,536276,0.0556027242],[1,536280,0.0679835586],[1,536282,0.0501009367],[1,536283,0.0540598195],[1,536288,0.0759773322],[1,536289,0.0625811404],[1,536290,0.071956154],[1,536297,0.107465932],[1,536298,0.0641676669],[1,536299,0.0793716462],[1,536304,0.1514171659],[1,536305,0.0709866508],[1,536306,0.0534991746],[1,536314,0.0540672858],[1,536316,0.0913425233],[1,536318,0.0506945256],[1,536329,0.058763411],[1,536332,0.165864903],[1,536333,0.0556027242],[1,536335,0.2128124051],[1,536337,0.0618132243],[1,536338,0.0675085273],[1,536339,0.0534991746],[1,536341,0.0615192268],[1,536346,0.0575286495],[1,536356,0.0534991746],[1,536361,0.0671481413],[1,536362,0.3452124991],[1,536365,0.1475223777],[1,536371,0.0612432095],[1,536373,0.1249060858],[1,536374,0.1485263485],[1,536376,0.0637084163],[1,536377,0.0534991746],[1,536380,0.0618310483],[1,536381,0.0505984413],[1,536384,0.0743902614],[1,536386,0.0549562442],[1,536387,0.0589449797],[1,536398,0.0629151702],[1,536399,0.1221839135],[1,536402,0.0562732228],[1,536403,0.0582093512],[1,536406,0.0668237593],[1,536408,0.0534991746],[1,536418,0.0534991746],[2,13,0.0604203229],[2,20,0.2753177619],[2,52,0.7403933826],[2,110,0.0722086962],[2,126,0.1722227286],[2,152,0.0722086962],[2,182,0.9838358412],[2,196,0.2535191499],[2,235,0.1340044631],[2,267,0.259057605],[2,425,0.7379990445],[2,430,0.3259718645],[2,433,0.195839279],[2,457,0.7977208283],[2,477,0.0716765475],[2,478,0.063855985],[2,480,0.1716722593],[2,500,0.1800702919],[2,501,0.2694559185],[2,503,0.5402547763],[2,504,0.0642581101],[2,505,0.4153502154],[2,509,0.0716765475],[2,519,0.0971201132],[2,520,0.0635098863],[2,521,0.9758109972],[2,531,0.0698732383],[2,542,0.0726622082],[2,551,0.0722086962],[2,555,0.0738899932],[2,557,0.0656916331],[2,558,0.063855985],[2,567,0.0644107534],[2,572,0.7125965497],[2,579,0.1483781505],[2,580,0.1928364232],[2,581,0.0771279813],[2,596,0.0722086962],[2,600,0.1716700183],[2,605,0.1256135462],[2,618,0.0766171649],[2,621,0.0716765475],[2,637,0.0644107534],[2,646,0.3266761241],[2,649,0.0716765475],[2,658,0.0883155346],[2,660,0.0716765475],[2,669,0.1212997592],[2,675,0.0978236084],[2,691,0.0541556254],[2,693,0.9118615029],[2,696,0.2349189739],[2,700,0.5274959725],[2,704,0.5064336848],[2,730,0.6143205944],[2,741,0.0716765475],[2,747,0.5737300207],[2,751,0.4107177386],[2,758,0.7423635419],[2,775,0.0722086962],[2,777,0.7480212723],[2,791,0.0766171649],[2,797,0.1921675385],[2,799,0.0644107534],[2,800,0.0644107534],[2,810,0.0772311308],[2,812,0.0539623929],[2,816,0.0716765475],[2,818,0.063855985],[2,829,0.39945866],[2,830,0.4401923171],[2,841,0.069758409],[2,842,0.108011874],[2,859,0.2247137233],[2,869,0.0614317106],[2,872,0.0701952767],[2,880,0.0720179138],[2,882,0.5971497914],[2,892,0.2505042077],[2,895,0.6848619735],[2,904,0.3457020726],[2,912,0.0692279175],[2,930,0.069758409],[2,942,0.0832832968],[2,951,0.0730039819],[2,954,0.0619847513],[2,958,0.2939509055],[2,959,0.6851525051],[2,967,0.0609878826],[2,971,0.0692279175],[2,976,0.0619847513],[2,978,0.7185457955],[2,979,0.0741531484],[2,982,0.0791391975],[2,988,0.0646226055],[2,1009,0.0614317106],[2,1010,0.0942483672],[2,1023,0.0724712953],[2,1032,0.0692279175],[2,1046,0.0730039819],[2,1048,0.0619847513],[2,1049,0.7431972966],[2,1054,0.0692279175],[2,1079,0.0730039819],[2,1088,0.311041893],[2,1095,0.2506947716],[2,1111,0.0515689301],[2,1114,0.0569915315],[2,1119,0.2946249277],[2,1125,0.069758409],[2,1135,0.4614500765],[2,1147,0.069758409],[2,1149,0.069758409],[2,1156,0.239281668],[2,1162,0.7404833045],[2,1164,0.5221549238],[2,1170,0.0951622656],[2,1177,0.1870166181],[2,1287,0.1401973893],[2,1288,0.2490207702],[2,1472,0.3595868419],[2,1480,0.1978926558],[2,1489,0.0503873499],[2,1501,0.291987369],[2,1503,0.5479920332],[2,1505,0.1904872653],[2,1515,0.4491260192],[2,1522,0.1950914267],[2,1564,0.2072679832],[2,1612,0.0919741382],[2,1673,0.3318739391],[2,1689,0.0743356248],[2,1717,0.2077329933],[2,1732,0.2532397776],[2,1784,0.0494644845],[2,1827,0.7212839288],[2,1877,0.1105350711],[2,1910,0.2589088883],[2,1935,0.2133432374],[2,2003,0.6044272212],[2,2011,0.1006546429],[2,2058,0.051557465],[2,2082,0.1608831066],[2,2096,0.2406766175],[2,2132,0.3982943226],[2,2144,0.0830032863],[2,2154,0.410082939],[2,2189,0.3479020697],[2,2192,0.1112184774],[2,2286,0.0502621336],[2,2315,0.0755941561],[2,2359,0.0535734765],[2,2360,0.1886404408],[2,2451,0.2404406406],[2,2625,0.2517729233],[2,2635,0.8123372288],[2,2660,0.2564001877],[2,2722,0.2559009868],[2,2740,0.0609018932],[2,2825,0.0930118577],[2,2828,0.0634665366],[2,2830,0.0597994417],[2,2831,0.1531581557],[2,2838,0.1247124058],[2,2842,0.1206159547],[2,2846,0.1213756095],[2,2863,0.451668357],[2,2867,0.0523882375],[2,2897,0.0970756296],[2,2945,0.7313826467],[2,2954,0.1769112158],[2,2958,0.1256243996],[2,2972,0.164602218],[2,3025,0.0638116551],[2,3029,0.7561482568],[2,3032,0.1625014925],[2,3063,0.3075584575],[2,3087,0.0812057074],[2,3094,0.1225114496],[2,3118,0.2062511752],[2,3131,0.5017462231],[2,3148,0.1154661841],[2,3182,0.0647824352],[2,3232,0.1470730674],[2,3238,0.1407572313],[2,3280,0.5300379376],[2,3288,0.2735265965],[2,3289,0.3388433449],[2,3303,0.0861623031],[2,3307,0.281778644],[2,3320,0.616053432],[2,3330,0.0797655782],[2,3332,0.1533159245],[2,3351,0.5297082952],[2,3366,0.258115332],[2,3371,0.1878128932],[2,3377,0.0642803756],[2,3380,0.0489048197],[2,3383,0.4880870786],[2,3392,0.0455218551],[2,3402,0.0480510866],[2,3410,0.3956034687],[2,3473,0.2894792098],[2,3484,0.0522471039],[2,3506,0.4206030579],[2,3515,0.0659605197],[2,3518,0.3357245577],[2,3533,0.0627896497],[2,3545,0.2688589341],[2,3566,0.0528796118],[2,3647,0.0699810683],[2,3678,0.3117985314],[2,3679,0.0831475226],[2,3683,0.0662391694],[2,3700,0.3402595986],[2,3704,0.2327285031],[2,3716,0.332951542],[2,3769,0.6993167917],[2,3775,0.2189391511],[2,3778,0.2199434317],[2,3788,0.389424558],[2,3792,0.1637965796],[2,3811,0.0928311966],[2,3824,0.0673520193],[2,3861,0.0761471665],[2,3889,0.4343769369],[2,3903,0.0472641342],[2,3921,0.0498567945],[2,3931,0.2428514359],[2,3948,0.4527301847],[2,3965,0.1664351461],[2,3978,0.128789922],[2,3995,0.1131244133],[2,4024,0.0560459047],[2,4046,0.2607005129],[2,4104,0.6514998967],[2,4117,0.3913084901],[2,4119,0.1000922419],[2,4120,0.0662391694],[2,4127,0.1196368109],[2,4135,0.0668969631],[2,4191,0.7755301994],[2,4199,0.4554424398],[2,4229,0.0662391694],[2,4245,0.5154170866],[2,4260,0.0818193896],[2,4270,0.6574054863],[2,4308,0.0554690489],[2,4316,0.109009819],[2,4329,0.3460243439],[2,4356,0.0710242967],[2,4386,0.5687067545],[2,4389,0.0596700007],[2,4415,0.3937247884],[2,4436,0.4500662036],[2,4518,0.154760433],[2,4521,0.5370890602],[2,4592,0.1302872492],[2,4593,0.4587564258],[2,4618,0.4521842954],[2,4677,0.0515584666],[2,4730,0.3512952392],[2,4779,0.0953361683],[2,4787,0.1985721785],[2,4851,0.0553678615],[2,4873,0.0631676438],[2,4886,0.2803025943],[2,4890,0.2012191378],[2,4892,0.0815326645],[2,4952,0.0714770654],[2,4955,0.4150350837],[2,4990,0.0644167728],[2,4992,0.2208295812],[2,5007,0.0917721822],[2,5016,0.8124847993],[2,5350,0.063463025],[2,5407,0.343587889],[2,5479,0.3680663409],[2,5996,0.2109990484],[2,6024,0.0692614881],[2,6029,0.0361552184],[2,6083,0.2883353014],[2,6102,0.0870351437],[2,6123,0.2904263513],[2,6133,0.1978011103],[2,6146,0.3632188752],[2,6178,0.0655039862],[2,6199,0.0515712098],[2,6234,0.2685739875],[2,6241,0.1418595328],[2,6248,0.6426394427],[2,6250,0.246808016],[2,6257,0.1560927976],[2,6262,0.1116289565],[2,6284,0.1838805156],[2,6285,0.1053818267],[2,6296,0.437482679],[2,6303,0.1528344709],[2,6308,0.3166319026],[2,6312,0.4704030428],[2,6318,0.0842411793],[2,6344,0.5319874226],[2,6350,0.2001818339],[2,6379,0.0443672528],[2,6397,0.1136037914],[2,6398,0.1080839311],[2,6412,0.5900721476],[2,6421,0.5003938714],[2,6431,0.0899377835],[2,6453,0.1792610388],[2,6470,0.0432096107],[2,6475,0.0553648131],[2,6476,0.27133936],[2,6497,0.1565002495],[2,6530,0.1111979368],[2,6536,0.1043232809],[2,6548,0.1972107934],[2,6618,0.1005268898],[2,6632,0.2337127302],[2,6637,0.4230467915],[2,6639,0.5321021499],[2,6653,0.0447180498],[2,6660,0.0408006261],[2,6662,0.1512615109],[2,6677,0.0519006275],[2,6694,0.1356313008],[2,6697,0.5360990152],[2,6701,0.5781078735],[2,6703,0.0520115018],[2,6733,0.376248838],[2,6735,0.0365925866],[2,6752,0.0383966613],[2,6755,0.076032859],[2,6763,0.0815116843],[2,6765,0.0447169207],[2,6767,0.3765752023],[2,6768,0.1933516366],[2,6785,0.0370785997],[2,6801,0.0957335195],[2,6824,0.0376142846],[2,6839,0.0504807816],[2,6842,0.4646603007],[2,6847,0.0365925866],[2,6849,0.1044900252],[2,6856,0.0626889886],[2,6857,0.0766160965],[2,6864,0.1037175422],[2,6866,0.207613489],[2,6872,0.2914610696],[2,6910,0.0477856913],[2,6911,0.06175759],[2,6936,0.1415069092],[2,6962,0.0379729494],[2,6969,0.2341010079],[2,6971,0.0553648131],[2,6982,0.0618683474],[2,6985,0.2467613271],[2,6995,0.0553648131],[2,7005,0.0639574106],[2,7044,0.1947188243],[2,7047,0.2527213077],[2,7069,0.1676399356],[2,7082,0.1697411345],[2,7087,0.0789515971],[2,7093,0.0561753612],[2,7109,0.0352542714],[2,7112,0.2660375985],[2,7120,0.0369389728],[2,7121,0.053647024],[2,7138,0.1589422502],[2,7140,0.0442090665],[2,7147,0.6980172994],[2,7155,0.6190716893],[2,7160,0.5041004661],[2,7167,0.1767044339],[2,7187,0.0537149194],[2,7216,0.0533591488],[2,7217,0.0617788992],[2,7219,0.0726378077],[2,7223,0.3952711014],[2,7228,0.045015256],[2,7235,0.1979505769],[2,7252,0.0821584297],[2,7253,0.1606405696],[2,7259,0.3919653947],[2,7263,0.1259677442],[2,7270,0.0435230848],[2,7271,0.0911591872],[2,7290,0.214422816],[2,7295,0.3731572678],[2,7313,0.1214635639],[2,7317,0.40410792],[2,7325,0.3509937006],[2,7336,0.271368632],[2,7348,0.4850331085],[2,7351,0.2052151157],[2,7362,0.2804463444],[2,7387,0.0452240773],[2,7391,0.3784789652],[2,7398,0.1908026296],[2,7423,0.0506780711],[2,7437,0.1118876265],[2,7451,0.0461780656],[2,7466,0.1048332723],[2,7468,0.205096945],[2,7479,0.3526884448],[2,7480,0.3085142684],[2,7485,0.1720989127],[2,7504,0.1532396862],[2,7505,0.2303790459],[2,7506,0.2074602456],[2,7509,0.1652609414],[2,7513,0.1522906518],[2,7521,0.0598687512],[2,7529,0.3434665077],[2,7545,0.3165266556],[2,7549,0.1656145263],[2,7573,0.0498813079],[2,7579,0.0404614373],[2,7588,0.279664812],[2,7602,0.0357799554],[2,7609,0.414031976],[2,7610,0.3063246062],[2,7629,0.1858964497],[2,7644,0.2974730773],[2,7652,0.1468073737],[2,7656,0.0697021577],[2,7657,0.6055442186],[2,7677,0.2113540386],[2,7686,0.2870968196],[2,7701,0.3380773008],[2,7708,0.1166565671],[2,7730,0.0440580478],[2,7752,0.0988820607],[2,7792,0.0735569854],[2,7802,0.8630885657],[2,7805,0.0365925866],[2,7806,0.3134636712],[2,7814,0.4040286266],[2,7823,0.0365925866],[2,7834,0.0518358082],[2,7858,0.0433328314],[2,7872,0.1012083051],[2,7879,0.1813939951],[2,7894,0.1365216621],[2,7896,0.0820897542],[2,7902,0.305968944],[2,7941,0.1245092926],[2,7952,0.0970613657],[2,7956,0.0792917952],[2,7964,0.075586144],[2,7983,0.132670358],[2,7998,0.2014939259],[2,8008,0.1212164124],[2,8019,0.0546107927],[2,8036,0.0500503105],[2,8039,0.0541548722],[2,8064,0.0748852385],[2,8075,0.3638282594],[2,8092,0.2096280901],[2,8094,0.3975626206],[2,8099,0.2569322278],[2,8108,0.1803285533],[2,8141,0.2036325105],[2,8147,0.1692776351],[2,8148,0.0394578971],[2,8195,0.6933970403],[2,8204,0.0371884744],[2,8210,0.0388296568],[2,8227,0.0584248967],[2,8235,0.0486694839],[2,8242,0.0547181644],[2,8243,0.0531471828],[2,8276,0.5202568218],[2,8292,0.2856395026],[2,8293,0.1294035528],[2,8298,0.1332823749],[2,8310,0.0452225789],[2,8356,0.4982508586],[2,8369,0.2159595353],[2,8392,0.2510932773],[2,8422,0.0599413134],[2,8446,0.3281729258],[2,8449,0.2734334566],[2,8475,0.3161981474],[2,8492,0.3705084823],[2,8527,0.0782020455],[2,8537,0.3174156072],[2,8539,0.0424834453],[2,8542,0.1590112971],[2,8548,0.2854458108],[2,8568,0.043743722],[2,8573,0.2506865373],[2,8594,0.2922304256],[2,8600,0.0288996617],[2,8610,0.0503020227],[2,8616,0.086568141],[2,8617,0.0345534864],[2,8654,0.0362129541],[2,8681,0.0354482042],[2,8685,0.1749458673],[2,8716,0.0660329172],[2,8764,0.039622706],[2,8789,0.2714862435],[2,8839,0.0380466621],[2,8884,0.2230048437],[2,8890,0.0755212007],[2,8894,0.1694532469],[2,8907,0.1325737856],[2,8914,0.2935337142],[2,9006,0.3788554914],[2,9029,0.5316823884],[2,9041,0.0520932178],[2,9053,0.0639425692],[2,9141,0.026947341],[2,9142,0.0508900852],[2,9165,0.0972321083],[2,9210,0.0284010608],[2,9218,0.0305642905],[2,9252,0.1557141735],[2,9296,0.0635009983],[2,9329,0.047376331],[2,9349,0.5575955104],[2,9354,0.0519716748],[2,9356,0.036428047],[2,9373,0.0343594498],[2,9388,0.228885272],[2,9409,0.0234985853],[2,9433,0.1418232072],[2,9461,0.0263353544],[2,9480,0.174958832],[2,9492,0.2200896909],[2,9509,0.1026682113],[2,9546,0.0496409313],[2,9561,0.0246386326],[2,9566,0.0625202838],[2,9583,0.0246386326],[2,9599,0.0262970511],[2,9606,0.0649202405],[2,9611,0.2123464881],[2,9629,0.9329045691],[2,9636,0.0276269184],[2,9678,0.0236806559],[2,9687,0.0265979628],[2,9689,0.1331360702],[2,9702,0.0526545962],[2,9713,0.0439332634],[2,9728,0.6992864161],[2,9746,0.0227935081],[2,9780,0.7280272413],[2,9787,0.1128987398],[2,9795,0.0395326012],[2,9798,0.0662291817],[2,9814,0.6341069957],[2,9818,0.0407942193],[2,9821,0.1956099414],[2,9838,0.1077201524],[2,9851,0.0944211927],[2,9859,0.4388065771],[2,9961,0.5547206564],[2,9962,0.0353654466],[2,9968,0.3672907302],[2,10006,0.0316577324],[2,10007,0.0832814252],[2,10009,0.0677001265],[2,10034,0.1090606998],[2,10045,0.2758020156],[2,10188,0.0347710039],[2,10224,0.5570413832],[2,10227,0.026102387],[2,10234,0.0251984993],[2,10237,0.0452441834],[2,10257,0.0580072941],[2,10284,0.0854726629],[2,10313,0.0520263143],[2,10349,0.0767744074],[2,10357,0.1682408437],[2,10397,0.3022895793],[2,10438,0.2288990157],[2,10447,0.0375274448],[2,10451,0.0457584423],[2,10472,0.1654955266],[2,10473,0.4033023172],[2,10474,0.048408044],[2,10476,0.026102387],[2,10489,0.0454185064],[2,10504,0.028870884],[2,10546,0.0322808438],[2,10547,0.0903891145],[2,10548,0.0248390281],[2,10554,0.0386989068],[2,10570,0.024170455],[2,10596,0.0356888188],[2,10641,0.0538468145],[2,10654,0.0272135604],[2,10672,0.0390134804],[2,10675,0.0906900254],[2,10682,0.0355306158],[2,10684,0.0452998863],[2,10734,0.2853914846],[2,10740,0.0250839421],[2,10755,0.0911574386],[2,10801,0.0237249186],[2,10807,0.1107278299],[2,10853,0.0584138125],[2,10866,0.1247199514],[2,10875,0.0321299733],[2,10878,0.0304613734],[2,10894,0.057536066],[2,10910,0.0970266309],[2,10922,0.0814521576],[2,10926,0.1577734176],[2,10932,0.0263973326],[2,10992,0.1400162868],[2,10995,0.0968596633],[2,11008,0.046252461],[2,11011,0.046252461],[2,11040,0.0489395564],[2,11058,0.0342912158],[2,11071,0.0426418792],[2,11094,0.0353841211],[2,11123,0.2614278616],[2,11126,0.023555481],[2,11153,0.0454692863],[2,11161,0.0614461111],[2,11172,0.0537186977],[2,11188,0.1227612307],[2,11191,0.1473572785],[2,11301,0.1768619322],[2,11321,0.045401791],[2,11332,0.0809097457],[2,11338,0.0368985778],[2,11369,0.0250279404],[2,11370,0.0250279404],[2,11412,0.0252138979],[2,11439,0.156027695],[2,11451,0.2382650634],[2,11454,0.0456782744],[2,11456,0.1426288033],[2,11486,0.0253264764],[2,11495,0.0690972229],[2,11528,0.0381555877],[2,11550,0.4548499911],[2,11555,0.0554911025],[2,11562,0.1181398424],[2,11579,0.0403675215],[2,11581,0.3043998757],[2,11607,0.0427069433],[2,11618,0.0421947483],[2,11621,0.0285396307],[2,11629,0.0231187352],[2,11705,0.0323474088],[2,11706,0.4044337016],[2,11709,0.0393224222],[2,11712,0.6824206457],[2,11739,0.0379535438],[2,11741,0.0373423406],[2,11756,0.0522986438],[2,11773,0.0792073154],[2,11802,0.0314883085],[2,11875,0.1381036278],[2,11916,0.0179214431],[2,11934,0.1158370907],[2,11935,0.0153400296],[2,11952,0.0177347357],[2,11964,0.0346021756],[2,11966,0.030281413],[2,11981,0.0382212995],[2,12031,0.0332821115],[2,12045,0.0250355531],[2,12050,0.1866895027],[2,12051,0.0740410931],[2,12052,0.0414148307],[2,12067,0.2479331639],[2,12078,0.5062823169],[2,12079,0.1495066925],[2,12085,0.5900219884],[2,12090,0.1142499169],[2,12094,0.4490457947],[2,12097,0.0188963781],[2,12128,0.0203288203],[2,12129,0.0372501207],[2,12131,0.1306834177],[2,12165,0.0421813723],[2,12170,0.2296734844],[2,12190,0.3225721152],[2,12196,0.409908454],[2,12216,0.1390581589],[2,12245,0.0594422005],[2,12264,0.2852183197],[2,12292,0.626004867],[2,12299,0.0277242848],[2,12331,0.1709241382],[2,12336,0.1072632677],[2,12338,0.0455778563],[2,12352,0.1859284594],[2,12360,0.0379535438],[2,12373,0.0209583669],[2,12401,0.6837038579],[2,12403,0.3992832767],[2,12405,0.5648837311],[2,12411,0.0907980971],[2,12417,0.0357603743],[2,12418,0.0282787207],[2,12426,0.02856675],[2,12429,0.1637152842],[2,12430,0.2529990051],[2,12438,0.0566280197],[2,12445,0.0958130212],[2,12460,0.0379535438],[2,12483,0.1146755266],[2,12490,0.7612840474],[2,12493,0.0407084596],[2,12495,0.2692806874],[2,12501,0.1568857151],[2,12502,0.1999364507],[2,12514,0.0282632314],[2,12523,0.1492204617],[2,12529,0.3999340142],[2,12531,0.0195056374],[2,12535,0.058969081],[2,12536,0.2312276883],[2,12537,0.2351274535],[2,12539,0.4986623425],[2,12545,0.3996274338],[2,12551,0.0179330434],[2,12554,0.4561267123],[2,12555,0.1626016981],[2,12557,0.0281514328],[2,12564,0.0454173591],[2,12565,0.3477217789],[2,12569,0.0184992593],[2,12573,0.1105784527],[2,12577,0.5491457902],[2,12579,0.024897804],[2,12580,0.0374445639],[2,12588,0.3630621649],[2,12589,0.4574664295],[2,12593,0.1774219223],[2,12595,0.0662393937],[2,12603,0.0273304778],[2,12610,0.0317869234],[2,12619,0.0443191817],[2,12620,0.0639586287],[2,12625,0.177685938],[2,12648,0.0463166839],[2,12654,0.6466921518],[2,12656,0.0530683236],[2,12663,0.335841273],[2,12668,0.2142581096],[2,12673,0.1820462208],[2,12691,0.0379535438],[2,12703,0.4144809347],[2,12704,0.0679459664],[2,12714,0.2388118217],[2,12716,0.5350325925],[2,12719,0.0356222729],[2,12727,0.1010732381],[2,12730,0.1038279494],[2,12760,0.0893932962],[2,12765,0.1436598383],[2,12769,0.1127209768],[2,12782,0.1566178362],[2,12789,0.0849742383],[2,12801,0.3439234279],[2,12842,0.0986678805],[2,12848,0.0183149896],[2,12852,0.0179330434],[2,12853,0.0350581552],[2,12855,0.0855430566],[2,12867,0.1835683865],[2,12869,0.07787956],[2,12875,0.2310448272],[2,12880,0.5196013069],[2,12883,0.1378884823],[2,12896,0.0262304706],[2,12912,0.0895283521],[2,12918,0.0291447166],[2,12920,0.860955155],[2,12929,0.0351437537],[2,12965,0.0379535438],[2,12966,0.0815631676],[2,13000,0.0499295658],[2,13018,0.0556277712],[2,13033,0.1908351754],[2,13075,0.0359480045],[2,13084,0.0211799887],[2,13100,0.4463641954],[2,13108,0.0219000541],[2,13158,0.2280290498],[2,13162,0.8135246318],[2,13167,0.0374445639],[2,13169,0.0197095354],[2,13174,0.3595289537],[2,13184,0.0506269848],[2,13188,0.2661287348],[2,13227,0.0322022683],[2,13241,0.0630333924],[2,13242,0.0189419291],[2,13245,0.038235309],[2,13250,0.023531207],[2,13266,0.0488741172],[2,13305,0.066189325],[2,13310,0.4610357443],[2,13335,0.0165803491],[2,13344,0.0153332316],[2,13347,0.4522893808],[2,13366,0.1849053487],[2,13383,0.4583151459],[2,13400,0.0528674324],[2,13405,0.0589979842],[2,13414,0.1329440102],[2,13433,0.0508301726],[2,13458,0.2513787019],[2,13459,0.0153332316],[2,13461,0.0238346144],[2,13472,0.0167641826],[2,13501,0.0235076944],[2,13520,0.0662198685],[2,13527,0.0221387522],[2,13546,0.1742980383],[2,13572,0.0502370934],[2,13676,0.2628941877],[2,13688,0.0576939777],[2,13690,0.0428935501],[2,13701,0.0272565725],[2,13718,0.2602311452],[2,13728,0.2644951721],[2,13773,0.1012196831],[2,13851,0.6664301881],[2,13862,0.0705084844],[2,13879,0.0225966957],[2,13897,0.2810770572],[2,13923,0.2780122191],[2,13938,0.0193560511],[2,13965,0.0666064347],[2,13973,0.1223996165],[2,13975,0.0350147096],[2,13981,0.0136162678],[2,13995,0.230929951],[2,14038,0.0104780816],[2,14045,0.076830561],[2,14046,0.591840913],[2,14088,0.1680913551],[2,14100,0.0117169036],[2,14120,0.5583816588],[2,14136,0.0233305886],[2,14146,0.2911121892],[2,14152,0.5765467562],[2,14163,0.1726407716],[2,14168,0.1744092047],[2,14181,0.1090260564],[2,14195,0.0432153348],[2,14227,0.0740730577],[2,14258,0.0681801682],[2,14267,0.1297237231],[2,14281,0.0175073306],[2,14287,0.6483554581],[2,14360,0.0202086358],[2,14368,0.0705302725],[2,14389,0.2072318768],[2,14395,0.3776108532],[2,14397,0.0136162678],[2,14414,0.2622803187],[2,14418,0.1218833484],[2,14447,0.0153324038],[2,14471,0.0130538234],[2,14488,0.1365182702],[2,14498,0.1258601699],[2,14501,0.6733033322],[2,15192,0.0151367324],[2,15244,0.0332923965],[2,15251,0.1085694043],[2,15289,0.0442869447],[2,15290,0.0731233656],[2,15308,0.2729578562],[2,15336,0.3910676675],[2,15351,0.384638125],[2,15369,0.0163965784],[2,15398,0.013030266],[2,15704,0.0116655803],[2,15710,0.0495175056],[2,15728,0.2817158235],[2,15733,0.0596153296],[2,15814,0.1529084786],[2,15837,0.4711084701],[2,15867,0.0103426912],[2,15948,0.0741513723],[2,15958,0.0122384616],[2,15960,0.0361962202],[2,15966,0.3893752763],[2,15990,0.1668528923],[2,15998,0.0302593119],[2,16005,0.111133857],[2,16026,0.0558036095],[2,16053,0.1567220731],[2,16054,0.0308837735],[2,16062,0.0118596822],[2,16083,0.0122384616],[2,16093,0.0958310852],[2,16102,0.2452592636],[2,16121,0.1427320913],[2,16139,0.0140279813],[2,16141,0.0405585673],[2,16143,0.3522838871],[2,16144,0.4102392444],[2,16147,0.2126161403],[2,16148,0.1538257666],[2,16158,0.2766809211],[2,16167,0.0545743986],[2,16195,0.0251175428],[2,16203,0.1927555413],[2,16207,0.3708669978],[2,16213,0.0668538616],[2,16214,0.2115976873],[2,16215,0.1271765114],[2,16226,0.0164598627],[2,16231,0.277064783],[2,16233,0.2292252112],[2,16234,0.0187349089],[2,16241,0.0141486338],[2,16249,0.0786722446],[2,16258,0.1954591861],[2,16259,0.1105672696],[2,16260,0.0585050668],[2,16265,0.1613130862],[2,16266,0.0381088528],[2,16270,0.0144027317],[2,16281,0.3617790758],[2,16282,0.0112979619],[2,16288,0.0570954434],[2,16294,0.0635561809],[2,16296,0.4743362222],[2,16298,0.0164598627],[2,16307,0.131029732],[2,16308,0.0624700578],[2,16311,0.5143675452],[2,16312,0.2702060156],[2,16322,0.0440281824],[2,16327,0.0826328232],[2,16331,0.0828757382],[2,16336,0.4562521651],[2,16347,0.2531762277],[2,16348,0.0112979619],[2,16349,0.1254720352],[2,16363,0.4542664002],[2,16367,0.015072068],[2,16368,0.0389054417],[2,16378,0.0573954943],[2,16379,0.022839976],[2,16399,0.0400541511],[2,16403,0.0142946253],[2,16406,0.0427827721],[2,16409,0.0116770814],[2,16411,0.0217627278],[2,16417,0.0190722931],[2,16419,0.0353882797],[2,16422,0.0383284345],[2,16430,0.0197639956],[2,16436,0.2059820905],[2,16453,0.0219344022],[2,16454,0.0929026907],[2,16456,0.3648016998],[2,16467,0.0201952097],[2,16469,0.0445748321],[2,16477,0.0223636112],[2,16480,0.0201780715],[2,16481,0.0636017154],[2,16484,0.0131412009],[2,16488,0.258774048],[2,16496,0.0389538711],[2,16501,0.6303102529],[2,16504,0.0699352808],[2,16512,0.0920887308],[2,16513,0.1316632778],[2,16516,0.4907376222],[2,16517,0.165120669],[2,16519,0.0866413046],[2,16521,0.0860536108],[2,16526,0.0405498139],[2,16529,0.0805530446],[2,16537,0.0394889558],[2,16540,0.0271917908],[2,16542,0.0933956542],[2,16548,0.2751463187],[2,16549,0.0244126708],[2,16560,0.1179435642],[2,16561,0.5374446106],[2,16567,0.0641777805],[2,16574,0.4605174034],[2,16576,0.2589726094],[2,16580,0.4150105643],[2,16583,0.095102982],[2,16588,0.0406130775],[2,16590,0.0536345795],[2,16600,0.4151012366],[2,16605,0.0620986822],[2,16606,0.0508282027],[2,16616,0.0403411531],[2,16623,0.021749269],[2,16624,0.0554667929],[2,16631,0.0103426912],[2,16638,0.0501720978],[2,16651,0.3360937996],[2,16653,0.0975450173],[2,16654,0.2203866181],[2,16657,0.0280777534],[2,16659,0.3421411926],[2,16662,0.013030266],[2,16664,0.1391054035],[2,16686,0.1222696057],[2,16725,0.124459872],[2,16732,0.0311638602],[2,16735,0.0716068132],[2,16748,0.1166337865],[2,16760,0.0263381046],[2,16771,0.0235733966],[2,16777,0.5860094562],[2,16784,0.0460401461],[2,16790,0.1970642368],[2,16801,0.0304742881],[2,16808,0.0216409735],[2,16816,0.0138157543],[2,16823,0.308991725],[2,16852,0.0370534732],[2,16859,0.0415292823],[2,16864,0.0131636827],[2,16873,0.0090179475],[2,16875,0.0223145583],[2,16882,0.1938675261],[2,16885,0.3778479281],[2,16888,0.0160479462],[2,16908,0.0122863146],[2,16913,0.0406161575],[2,16916,0.0102542747],[2,16927,0.1198355516],[2,16930,0.1396767811],[2,16933,0.1260532689],[2,16935,0.2918228487],[2,16936,0.1132963527],[2,16940,0.0555364989],[2,16947,0.1929667425],[2,16951,0.0164869174],[2,16959,0.0255832007],[2,16969,0.3335746711],[2,16978,0.0180843226],[2,16979,0.0129570304],[2,16987,0.0162043214],[2,16994,0.0487927871],[2,17002,0.0157532897],[2,17011,0.0371793669],[2,17015,0.1483073373],[2,17019,0.1372595801],[2,17022,0.1082562622],[2,17029,0.1830422615],[2,17032,0.2762663466],[2,17055,0.6815308841],[2,17056,0.2568002924],[2,17069,0.0089231292],[2,17071,0.1696518776],[2,17079,0.0187374117],[2,17098,0.0291353082],[2,17099,0.163581992],[2,17120,0.0819238921],[2,17124,0.0106727224],[2,17128,0.0195036575],[2,17130,0.012038909],[2,17135,0.0199237456],[2,17136,0.0330236265],[2,17154,0.0548655253],[2,17156,0.0547862827],[2,17157,0.0169610636],[2,17169,0.2371850595],[2,17176,0.0937100613],[2,17188,0.2155415754],[2,17201,0.0502526975],[2,17224,0.0154619767],[2,17228,0.0292520554],[2,17234,0.0171578405],[2,17249,0.0570809694],[2,17256,0.1145720708],[2,17267,0.1143863952],[2,17268,0.0119962138],[2,17276,0.01706945],[2,17277,0.0163107259],[2,17281,0.1392214636],[2,17285,0.0100720774],[2,17297,0.027837651],[2,17303,0.100751272],[2,17315,0.0089231292],[2,17323,0.1074476828],[2,17324,0.3454178701],[2,17325,0.0159387976],[2,17338,0.0363727955],[2,17342,0.012917935],[2,17343,0.0739977339],[2,17345,0.0166465388],[2,17349,0.0172288639],[2,17350,0.0201806028],[2,17360,0.0122863146],[2,17379,0.0741285324],[2,17384,0.1674244372],[2,17419,0.0071305772],[2,17432,0.2416419101],[2,17440,0.1073008619],[2,17448,0.0199237456],[2,17449,0.0102542747],[2,17465,0.3467624895],[2,17466,0.1358429626],[2,17482,0.4701279133],[2,17483,0.1710354947],[2,17486,0.0402911976],[2,17490,0.0557412289],[2,17509,0.0215961349],[2,17514,0.0250077367],[2,17517,0.0246908704],[2,17550,0.02869687],[2,17565,0.1072994251],[2,17584,0.0325842999],[2,17588,0.2093700623],[2,17592,0.0251579859],[2,17595,0.017872416],[2,17606,0.0343154268],[2,17639,0.0289864996],[2,17655,0.0282776152],[2,17656,0.0643714866],[2,17657,0.022138159],[2,17660,0.0320633588],[2,17663,0.1820285557],[2,17668,0.1618912569],[2,17672,0.0400274301],[2,17673,0.0256592725],[2,17677,0.1677311029],[2,17703,0.3844294239],[2,17707,0.0206766859],[2,17709,0.0073120348],[2,17713,0.0670202227],[2,17717,0.0160010141],[2,17735,0.0154821101],[2,17738,0.0083636796],[2,17748,0.043088709],[2,17754,0.0303058333],[2,17755,0.0122863146],[2,17764,0.0199237456],[2,17767,0.0289864996],[2,17774,0.020832555],[2,17778,0.0669419133],[2,17784,0.2902025868],[2,17786,0.1785151062],[2,17794,0.0356175476],[2,17797,0.208656467],[2,17810,0.1351502761],[2,17821,0.0310330639],[2,17829,0.0679536872],[2,17849,0.018567522],[2,17852,0.0710033909],[2,17861,0.0166927329],[2,17868,0.0197127077],[2,17872,0.016716079],[2,17882,0.0081961933],[2,17920,0.0346074173],[2,17923,0.0253542945],[2,17924,0.0441797503],[2,17936,0.0361355902],[2,17953,0.018567522],[2,17955,0.014477739],[2,17965,0.3820035104],[2,17969,0.3399795638],[2,17972,0.0545055947],[2,17979,0.242865613],[2,17989,0.0296337381],[2,17995,0.0195689961],[2,18000,0.1846582083],[2,18007,0.1351963575],[2,18009,0.0176956712],[2,18028,0.020782668],[2,18031,0.1408210979],[2,18054,0.0127287012],[2,18062,0.200154441],[2,18076,0.0397802988],[2,18083,0.0213226341],[2,18084,0.0067820065],[2,18088,0.392722261],[2,18103,0.4111625013],[2,18106,0.0470357227],[2,18121,0.0148968434],[2,18129,0.0207294383],[2,18131,0.008391918],[2,18137,0.0366384765],[2,18142,0.0094330411],[2,18150,0.3727781888],[2,18164,0.2144851604],[2,18175,0.0103762759],[2,18176,0.1128527606],[2,18179,0.0656685245],[2,18197,0.040004067],[2,18205,0.1513288556],[2,18210,0.0460845011],[2,18243,0.0463091084],[2,18244,0.0137754229],[2,18260,0.0195689961],[2,18290,0.0073329383],[2,18299,0.0132748842],[2,18304,0.0163023003],[2,18307,0.019215643],[2,18324,0.0330044284],[2,18325,0.2082358428],[2,18332,0.0469594842],[2,18351,0.4435017854],[2,18353,0.0145686445],[2,18384,0.1346767727],[2,18385,0.0641678898],[2,18388,0.0836010448],[2,18398,0.1086078479],[2,18401,0.0063591479],[2,18403,0.0335186647],[2,18410,0.0983358451],[2,18426,0.0853107587],[2,18430,0.3279713147],[2,18434,0.0085629557],[2,18449,0.0127264714],[2,18470,0.033696335],[2,18474,0.0610321417],[2,18503,0.0085266289],[2,18511,0.0303040448],[2,18512,0.0434870915],[2,18513,0.0505285665],[2,18516,0.0077725057],[2,18518,0.0216335303],[2,18519,0.0576968283],[2,18522,0.0570643276],[2,18549,0.0179984879],[2,18559,0.0230889043],[2,18562,0.7622516604],[2,18563,0.0308574529],[2,18568,0.0066405402],[2,18575,0.008344866],[2,18578,0.029848766],[2,18592,0.0683865523],[2,18600,0.3641778637],[2,18607,0.008344866],[2,18609,0.0127020998],[2,18614,0.0079566733],[2,18619,0.2085474141],[2,18628,0.1275434681],[2,18634,0.0065404116],[2,18636,0.0266958916],[2,18641,0.0075909323],[2,18644,0.0214224426],[2,18648,0.0069056205],[2,18656,0.2745537912],[2,18661,0.0231792852],[2,18669,0.0181730525],[2,18680,0.0312526796],[2,18682,0.0065404116],[2,18687,0.0176465607],[2,18695,0.0188382606],[2,18697,0.4594356334],[2,18708,0.0650647185],[2,18713,0.006724265],[2,18725,0.4474368371],[2,18728,0.0127020998],[2,18738,0.0223081869],[2,18743,0.0178347669],[2,18748,0.1003894433],[2,18755,0.0085629557],[2,18788,0.0063591479],[2,18807,0.0083340477],[2,18819,0.0477773364],[2,18830,0.0181725343],[2,18845,0.0319091631],[2,18850,0.0109846646],[2,18851,0.0169873161],[2,18852,0.0806397892],[2,18861,0.0209804559],[2,18882,0.0736354867],[2,18888,0.0936182167],[2,18893,0.0600943333],[2,18905,0.008344866],[2,18921,0.1217052703],[2,18943,0.0103070096],[2,18967,0.062616153],[2,18997,0.0505070503],[2,19007,0.0085158079],[2,19010,0.0369614251],[2,19014,0.210375351],[2,19018,0.0339749959],[2,19068,0.037926802],[2,19071,0.0201263919],[2,19074,0.037145832],[2,19077,0.0272142053],[2,19086,0.0255807811],[2,19089,0.0719297928],[2,19096,0.0068218746],[2,19098,0.1650157056],[2,19114,0.1808673001],[2,19134,0.0982252004],[2,19139,0.0073329383],[2,19147,0.3251618551],[2,19158,0.1887379083],[2,19163,0.0082842429],[2,19173,0.0085266289],[2,19177,0.1235334473],[2,19182,0.2530663078],[2,19192,0.019052367],[2,19203,0.0072932485],[2,19207,0.0127264714],[2,19217,0.0753614993],[2,19227,0.2559565961],[2,19249,0.0135694318],[2,19259,0.0147243752],[2,19266,0.4974694612],[2,19267,0.0668630718],[2,19276,0.0246431213],[2,19310,0.040444227],[2,19313,0.0643167567],[2,19318,0.036232156],[2,19330,0.035555881],[2,19337,0.006724265],[2,19364,0.0540572735],[2,19390,0.0342138433],[2,19393,0.3989806342],[2,19395,0.008344866],[2,19397,0.0286997598],[2,19404,0.0108569287],[2,19432,0.0085266289],[2,19435,0.0204677271],[2,19461,0.0106798461],[2,19463,0.0156482772],[2,19473,0.0186000696],[2,19476,0.1727021714],[2,19493,0.0192593857],[2,19500,0.1132788181],[2,19501,0.1491985815],[2,19511,0.0066405402],[2,19517,0.3215614177],[2,19529,0.0071117956],[2,19537,0.0892746066],[2,19546,0.0085629557],[2,19563,0.0411160654],[2,19564,0.0485235441],[2,19572,0.0729740687],[2,19577,0.0137525084],[2,19578,0.0631631185],[2,19584,0.0301115199],[2,19591,0.0249201188],[2,19592,0.0593478786],[2,19640,0.0104374074],[2,19663,0.0079566733],[2,19664,0.0095963476],[2,19671,0.1102995859],[2,19688,0.0334223152],[2,19694,0.0421015835],[2,19697,0.0195118259],[2,19712,0.008344866],[2,19720,0.0873587523],[2,19724,0.0168899745],[2,19731,0.013273873],[2,19733,0.0317494828],[2,19745,0.0105538255],[2,19751,0.1223990238],[2,19759,0.2672616674],[2,19765,0.0085266289],[2,19778,0.0272142053],[2,19788,0.0070172272],[2,19795,0.1541161297],[2,19809,0.0271278816],[2,19829,0.0814880962],[2,19838,0.0156722288],[2,19841,0.0103070096],[2,19843,0.2480528213],[2,19855,0.0085266289],[2,19860,0.0699635955],[2,19862,0.005591402],[2,19863,0.097283385],[2,19879,0.1792729258],[2,19880,0.0109299528],[2,19901,0.0175209219],[2,19903,0.2005449522],[2,19909,0.2035782648],[2,19921,0.0085266289],[2,19940,0.0168834901],[2,19949,0.0332383439],[2,19953,0.0068218746],[2,19956,0.120121124],[2,19957,0.028218668],[2,19962,0.0237707204],[2,19966,0.0247365686],[2,19985,0.0556409727],[2,19986,0.0188382606],[2,20008,0.0923087861],[2,20009,0.0421348884],[2,20023,0.0177210856],[2,20042,0.0125144651],[2,20051,0.0149036571],[2,20055,0.4088107124],[2,20086,0.1423159519],[2,20092,0.082685452],[2,20105,0.0085266289],[2,20111,0.0192593857],[2,20114,0.0166640631],[2,20128,0.1463417758],[2,20138,0.0411444626],[2,20154,0.008344866],[2,20179,0.0313914993],[2,20182,0.012909336],[2,20188,0.0089440793],[2,20201,0.0159668126],[2,20206,0.0330165509],[2,20207,0.0249201188],[2,20209,0.0121224563],[2,20216,0.2763511381],[2,20225,0.0566943065],[2,20227,0.0124272378],[2,20229,0.148923531],[2,20238,0.0668652941],[2,20248,0.0627444829],[2,20251,0.0571440564],[2,20271,0.0145686445],[2,20273,0.0077725057],[2,20274,0.008344866],[2,20288,0.0183154054],[2,20304,0.0093143896],[2,20305,0.0121224563],[2,20308,0.0163427308],[2,20332,0.0260243185],[2,20367,0.0595545479],[2,20370,0.0181685245],[2,20380,0.0174243977],[2,20390,0.030791337],[2,20395,0.0440494658],[2,20399,0.0181730525],[2,20407,0.0280305853],[2,20408,0.0085663865],[2,20444,0.0181455796],[2,20451,0.0109299528],[2,20457,0.4551438065],[2,20467,0.3246175625],[2,20476,0.005591402],[2,20479,0.0115701384],[2,20513,0.0191699563],[2,20514,0.0066405402],[2,20548,0.0070172272],[2,20564,0.0315142927],[2,20579,0.0342198384],[2,20584,0.0272142053],[2,20589,0.0063591479],[2,20611,0.0066405402],[2,20622,0.0177627223],[2,20645,0.4826033766],[2,20646,0.0113457642],[2,20657,0.0116812333],[2,20667,0.0070172272],[2,20676,0.0186134217],[2,20705,0.0375492108],[2,20706,0.006724265],[2,20734,0.010331301],[2,20743,0.1142295353],[2,20744,0.0190969186],[2,20752,0.0217282511],[2,20754,0.0519529846],[2,20757,0.0234900693],[2,20765,0.0131943336],[2,20775,0.2267700738],[2,20791,0.1794454866],[2,20794,0.0478645139],[2,20821,0.0085663865],[2,20823,0.0367461863],[2,20846,0.3018669509],[2,20850,0.0199233561],[2,20851,0.0113457642],[2,20855,0.0148311069],[2,20880,0.0054103768],[2,20888,0.0143420414],[2,20891,0.2565963944],[2,20900,0.0158036154],[2,20915,0.0153255602],[2,20941,0.0463439786],[2,20943,0.0063591479],[2,20948,0.008344866],[2,20961,0.0129282488],[2,20965,0.164141567],[2,20973,0.2360408288],[2,20987,0.0271278816],[2,21017,0.0138922685],[2,21018,0.0163032897],[2,21019,0.010331301],[2,21049,0.0071117956],[2,21050,0.1187318246],[2,21059,0.0235018323],[2,21081,0.1550777762],[2,21085,0.0625423476],[2,21098,0.0272142053],[2,21101,0.008344866],[2,21102,0.1525049889],[2,21106,0.0166995627],[2,21107,0.1503786533],[2,21136,0.2089737958],[2,21138,0.1443986946],[2,21142,0.093027896],[2,21178,0.3511574117],[2,21233,0.0475940497],[2,21242,0.0373344788],[2,21243,0.0065404116],[2,21374,0.0951473352],[2,21518,0.0103070096],[2,21522,0.0072932485],[2,21559,0.0530472487],[2,21563,0.0564384591],[2,21567,0.0272142053],[2,21625,0.0311936434],[2,21649,0.2539633829],[2,21660,0.0063591479],[2,21662,0.0085629557],[2,21681,0.0106378137],[2,21689,0.015855834],[2,21695,0.0085266289],[2,21698,0.1786490607],[2,21703,0.1453741709],[2,21705,0.014544809],[2,21718,0.0072932485],[2,21739,0.0054103768],[2,21740,0.0119890897],[2,21753,0.0119061174],[2,21755,0.0274463373],[2,21765,0.0123026809],[2,21768,0.0218844872],[2,21781,0.0277162612],[2,21788,0.0222283634],[2,21804,0.0142612908],[2,21816,0.0127264714],[2,21826,0.0066405402],[2,21833,0.0127020998],[2,21847,0.1599006452],[2,21848,0.1461044884],[2,21855,0.017744582],[2,21867,0.0066405402],[2,21882,0.0243080745],[2,21891,0.1187773882],[2,21896,0.126047293],[2,21897,0.0381051789],[2,21901,0.0195221868],[2,21909,0.3495798607],[2,21931,0.1892074672],[2,21942,0.0105538255],[2,21944,0.0214409443],[2,21959,0.0181730525],[2,21983,0.008344866],[2,22020,0.012909336],[2,22041,0.0366059049],[2,22047,0.021760041],[2,22051,0.0152482291],[2,22052,0.2602067651],[2,22053,0.3576470682],[2,22061,0.3712681722],[2,22075,0.0066405402],[2,22076,0.0536002109],[2,22083,0.0181730525],[2,22089,0.5260561322],[2,22099,0.0600287938],[2,22103,0.1183950302],[2,22117,0.008344866],[2,22118,0.0807329097],[2,22121,0.1217134649],[2,22126,0.2828356582],[2,22129,0.0066405402],[2,22131,0.0105512233],[2,22137,0.462151035],[2,22142,0.0176465607],[2,22143,0.1755609388],[2,22146,0.2181421182],[2,22152,0.4437408036],[2,22169,0.0735489582],[2,22174,0.3805575824],[2,22178,0.0544540201],[2,22186,0.2542125373],[2,22190,0.01541374],[2,22194,0.0346427261],[2,22196,0.0070172272],[2,22206,0.0390289095],[2,22223,0.008344866],[2,22229,0.0577479694],[2,22231,0.0105538255],[2,22245,0.105656118],[2,22275,0.0103127267],[2,22293,0.1388703422],[2,22298,0.017119662],[2,22308,0.0093143896],[2,22311,0.0169895463],[2,22318,0.2138038862],[2,22333,0.0173901753],[2,22340,0.2118414458],[2,22371,0.1975405757],[2,22376,0.008344866],[2,22389,0.1017916637],[2,22400,0.017224252],[2,22420,0.0177539657],[2,22430,0.4117713614],[2,22437,0.0078007415],[2,22481,0.0192697743],[2,22486,0.0267491777],[2,22496,0.1547547388],[2,22533,0.6082300648],[2,22543,0.2742898837],[2,22564,0.0279356117],[2,22579,0.3134844837],[2,22583,0.010764736],[2,22586,0.2524776306],[2,22594,0.0967507284],[2,22609,0.0070988856],[2,22611,0.0175865598],[2,22619,0.2445924055],[2,22620,0.3551564171],[2,22632,0.4339039542],[2,22634,0.0362174723],[2,22637,0.1253739893],[2,22638,0.288872805],[2,22643,0.3290050393],[2,22646,0.0139850368],[2,22650,0.2479729083],[2,22651,0.2826128741],[2,22657,0.2122571656],[2,22659,0.0565296956],[2,22665,0.0767917555],[2,22669,0.3803288444],[2,22670,0.1025074587],[2,22672,0.0293147125],[2,22700,0.0552037536],[2,22724,0.0463567287],[2,22733,0.2023780945],[2,22754,0.0158393564],[2,22800,0.2406473778],[2,22808,0.0317172432],[2,22858,0.017959776],[2,22948,0.4003189507],[2,22963,0.0112997033],[2,22991,0.0592216578],[2,22992,0.664366102],[2,23000,0.0545757983],[2,23001,0.0205232046],[2,23012,0.1284397567],[2,23014,0.1265930168],[2,23023,0.521769961],[2,23026,0.0097268128],[2,23035,0.0097325254],[2,23042,0.0079478572],[2,23052,0.3712560155],[2,23053,0.2550821856],[2,23056,0.2786943297],[2,23057,0.0103492574],[2,23060,0.0079841549],[2,23061,0.0083649735],[2,23064,0.0236888235],[2,23067,0.1918330997],[2,23071,0.13887732],[2,23077,0.6388730843],[2,23081,0.4444465826],[2,23082,0.0124666072],[2,23083,0.031638878],[2,23085,0.1082178414],[2,23087,0.4773058555],[2,23091,0.3906481766],[2,23101,0.3597135622],[2,23102,0.0205251884],[2,23103,0.0633956267],[2,23117,0.016305108],[2,23119,0.0250744216],[2,23124,0.013758615],[2,23126,0.008418663],[2,23139,0.1170412543],[2,23146,0.0721464807],[2,23147,0.1065356687],[2,23148,0.2236361728],[2,23155,0.1701418107],[2,23159,0.0583753041],[2,23161,0.041898751],[2,23171,0.067419036],[2,23176,0.0201088965],[2,23195,0.0077662397],[2,23197,0.008325227],[2,23212,0.0968208102],[2,23213,0.1592571865],[2,23214,0.1828404074],[2,23228,0.2274520329],[2,23233,0.3231533472],[2,23234,0.0144751062],[2,23240,0.2089931311],[2,23253,0.2221809564],[2,23270,0.0227095312],[2,23272,0.5485771821],[2,23277,0.0100993509],[2,23283,0.0271221301],[2,23287,0.0063281462],[2,23317,0.0183263756],[2,23322,0.0136893721],[2,23335,0.0580912741],[2,23344,0.0077056651],[2,23353,0.1194992552],[2,23367,0.0479679799],[2,23370,0.0088971466],[2,23374,0.0263154989],[2,23384,0.0060632781],[2,23388,0.006755122],[2,23392,0.006808692],[2,23413,0.0507247184],[2,23418,0.0136554259],[2,23424,0.0519308384],[2,23435,0.1003359587],[2,23440,0.0556875167],[2,23461,0.05091512],[2,23466,0.0892131492],[2,23508,0.0825501897],[2,23548,0.2666450578],[2,23549,0.2277761531],[2,23551,0.0048340993],[2,23562,0.0077662397],[2,23570,0.5775981794],[2,23595,0.1486188537],[2,23605,0.245998531],[2,23608,0.0727632497],[2,23609,0.21248366],[2,23616,0.0188722456],[2,23618,0.09977703],[2,23621,0.2726933693],[2,23637,0.0175865598],[2,23640,0.0282908039],[2,23667,0.0114010839],[2,23691,0.0183594139],[2,23701,0.3932324589],[2,23710,0.2151791376],[2,23718,0.010836447],[2,23727,0.1798884099],[2,23745,0.0351268668],[2,23752,0.0180570643],[2,23755,0.1548179176],[2,23794,0.0633120859],[2,23805,0.0063148188],[2,23816,0.0063148188],[2,23817,0.0070674202],[2,23822,0.0166943775],[2,23828,0.4433117954],[2,23854,0.0109234896],[2,23895,0.1237587729],[2,23922,0.3814537879],[2,23930,0.2110682103],[2,23932,0.0487581392],[2,23934,0.0093412717],[2,23938,0.0167749377],[2,23954,0.0138526313],[2,23976,0.0239648401],[2,23977,0.0120427333],[2,24027,0.0973455319],[2,24052,0.0815738681],[2,24056,0.0152278542],[2,24061,0.0597094615],[2,24063,0.5806689556],[2,24064,0.0275281884],[2,24076,0.0180582129],[2,24181,0.0200447835],[2,24200,0.0257172855],[2,24204,0.0098808665],[2,24214,0.0249757004],[2,24231,0.0978471482],[2,24235,0.208154087],[2,24261,0.2415648947],[2,24321,0.049457765],[2,24343,0.1165903574],[2,24357,0.0166943775],[2,24360,0.4155718543],[2,24363,0.1549186067],[2,24369,0.1883097324],[2,24375,0.8450462628],[2,24377,0.3651489719],[2,24379,0.2469818577],[2,24386,0.0321800401],[2,24390,0.4123265011],[2,24393,0.0465705855],[2,24395,0.0201478721],[2,24403,0.0166943775],[2,24407,0.5657140478],[2,24415,0.5631890425],[2,24417,0.0344655765],[2,24418,0.0055610643],[2,24423,0.0560280853],[2,24424,0.4048637252],[2,24429,0.1927289629],[2,24433,0.1218018048],[2,24435,0.0078978646],[2,24448,0.021575545],[2,24449,0.5293468179],[2,24457,0.0185393981],[2,24463,0.1290227337],[2,24464,0.0288672877],[2,24472,0.021624411],[2,24478,0.0112361199],[2,24534,0.2711604928],[2,24541,0.117692037],[2,24560,0.2295978961],[2,24570,0.3364174032],[2,24600,0.1571977248],[2,24626,0.3283246684],[2,24657,0.059952602],[2,24709,0.2000900374],[2,24727,0.0057816145],[2,24750,0.0072693166],[2,24758,0.0908281445],[2,24765,0.1766656328],[2,24788,0.065444446],[2,24790,0.0163254164],[2,24821,0.0192236718],[2,24825,0.5995720875],[2,24827,0.5222818294],[2,24828,0.0213751862],[2,24932,0.0450983147],[2,24951,0.1999938577],[2,25001,0.0229645929],[2,25002,0.1328596964],[2,25009,0.0054409506],[2,25011,0.0067033164],[2,25017,0.0087922945],[2,25019,0.0830950162],[2,25021,0.3680411987],[2,25029,0.0198520382],[2,25039,0.382195684],[2,25040,0.0032668868],[2,25043,0.0390396323],[2,25063,0.2104874539],[2,25071,0.022207955],[2,25075,0.0239157581],[2,25087,0.0884779215],[2,25091,0.0974758596],[2,25099,0.1274377755],[2,25117,0.0306781767],[2,25127,0.0140963451],[2,25149,0.015826705],[2,25170,0.0099589837],[2,25300,0.0451472176],[2,25311,0.0388802297],[2,25344,0.0216143098],[2,25351,0.7415887833],[2,25356,0.0153742292],[2,25438,0.0930603876],[2,25462,0.0048689534],[2,25465,0.0170287625],[2,25479,0.0258445619],[2,25480,0.0360848922],[2,25496,0.106148089],[2,25497,0.2738887962],[2,25504,0.0200219261],[2,25505,0.4682311039],[2,25511,0.0067033164],[2,25515,0.2856930927],[2,25523,0.549449795],[2,25531,0.0177925072],[2,25538,0.0056219834],[2,25550,0.1247426001],[2,25570,0.0493620422],[2,25617,0.1708384572],[2,25621,0.0209512387],[2,25624,0.0530765939],[2,25664,0.0474084322],[2,25671,0.1561102184],[2,25677,0.0050498424],[2,25679,0.0601237626],[2,25718,0.0177366593],[2,25730,0.0234073633],[2,25731,0.0155643716],[2,25736,0.0067033164],[2,25742,0.0067033164],[2,25756,0.3746582009],[2,25758,0.0045768635],[2,25783,0.859864323],[2,25818,0.3416467761],[2,25822,0.0091579423],[2,25834,0.0042128334],[2,25837,0.2678400289],[2,25838,0.3170822408],[2,25853,0.3579296932],[2,25860,0.116576505],[2,25874,0.1854996173],[2,25881,0.0396003227],[2,25888,0.038375353],[2,25892,0.04315765],[2,25901,0.1184488569],[2,25906,0.2505234271],[2,25918,0.2892771651],[2,25919,0.2802431859],[2,25923,0.8502414022],[2,25925,0.5052874663],[2,25927,0.0192117894],[2,25928,0.0317693305],[2,25930,0.1670292689],[2,25945,0.0216545895],[2,26026,0.2817533444],[2,26049,0.0084202341],[2,26063,0.001800383],[2,26068,0.0840968502],[2,26077,0.3035409268],[2,26087,0.0052297921],[2,26170,0.0142540679],[2,26182,0.0019805005],[2,26184,0.0066300843],[2,26206,0.6361665193],[2,26207,0.0359196029],[2,26239,0.7333826117],[2,26242,0.4715549704],[2,26244,0.0514148135],[2,26258,0.0164549061],[2,26272,0.0318353268],[2,26293,0.0198716884],[2,26327,0.3441448621],[2,26339,0.0058567194],[2,26368,0.0237462364],[2,26371,0.0148322349],[2,26392,0.0154944102],[2,26405,0.199450081],[2,26425,0.0061153536],[2,26429,0.0033991766],[2,26465,0.0152852956],[2,26478,0.010776046],[2,26488,0.0090108294],[2,26492,0.0402702846],[2,26495,0.3157105586],[2,26511,0.0291490768],[2,26514,0.0181438676],[2,26516,0.1451479099],[2,26517,0.0536391503],[2,26536,0.0174606038],[2,26541,0.2097520729],[2,26550,0.0621554131],[2,26553,0.0037888533],[2,26554,0.0108761635],[2,26560,0.418667094],[2,26561,0.1434933232],[2,26571,0.0355369809],[2,26574,0.6337686081],[2,26579,0.2354829848],[2,26581,0.2866516764],[2,26589,0.2720857635],[2,26590,0.0475281893],[2,26592,0.189487376],[2,26597,0.0282121627],[2,26620,0.6795559249],[2,26624,0.045110249],[2,26626,0.2687085241],[2,26629,0.0322665811],[2,26638,0.5239781159],[2,26645,0.3913938214],[2,26651,0.0005460662],[2,26653,0.0353978832],[2,26662,0.284512909],[2,26665,0.0634481862],[2,26680,0.3524491396],[2,26687,0.1368676334],[2,26694,0.0242908174],[2,26703,0.4807291193],[2,26704,0.0686305591],[2,26714,0.0745364875],[2,26717,0.0140727449],[2,26726,0.1875369939],[2,26733,0.0037888533],[2,26735,0.1635070692],[2,26739,0.0598575357],[2,26749,0.006513381],[2,26754,0.0610321176],[2,26756,0.0430678193],[2,26761,0.052860849],[2,26771,0.056338317],[2,26773,0.1700519473],[2,26778,0.0118650266],[2,26781,0.0105279661],[2,26782,0.1682470983],[2,26783,0.0130687224],[2,26801,0.0537815612],[2,26803,0.0314259042],[2,26813,0.0165366308],[2,26814,0.4131197272],[2,26826,0.2390668013],[2,26828,0.0384922054],[2,26865,0.0842179658],[2,26866,0.0965871114],[2,26876,0.061422555],[2,26882,0.0379632879],[2,26889,0.1263404957],[2,26891,0.1175626224],[2,26913,0.1152418689],[2,26937,0.0151018338],[2,26950,0.0407764654],[2,26955,0.0148122413],[2,26962,0.0484543784],[2,26967,0.0156447692],[2,26972,0.0201393176],[2,26978,0.0037888533],[2,26984,0.0555680698],[2,26987,0.0099549193],[2,27005,0.0007258683],[2,27006,0.1514232617],[2,27012,0.0098379395],[2,27013,0.0039694707],[2,27014,0.0014884273],[2,27015,0.106279811],[2,27017,0.003948645],[2,27038,0.0433985321],[2,27050,0.0095345371],[2,27064,0.0081152965],[2,27069,0.0247908947],[2,27074,0.0073410638],[2,27075,0.1557774508],[2,27080,0.6866749009],[2,27083,0.0152852956],[2,27109,0.013253928],[2,27115,0.0146657141],[2,27118,0.0996602685],[2,27125,0.0018510776],[2,27144,0.4232675337],[2,27149,0.0030751596],[2,27159,0.168127656],[2,27171,0.028936312],[2,27173,0.0219883157],[2,27174,0.0014884273],[2,27181,0.0849103742],[2,27193,0.0486986114],[2,27195,0.2334335415],[2,27207,0.073780913],[2,27212,0.2709639255],[2,27223,0.0297888909],[2,27224,0.0292766042],[2,27233,0.4289590643],[2,27243,0.0074245221],[2,27244,0.0036305167],[2,27273,0.0315342727],[2,27275,0.0669090121],[2,27297,0.0855298747],[2,27320,0.0063765159],[2,27323,0.0242048339],[2,27324,0.0037888533],[2,27328,0.0164549061],[2,27333,0.2009597766],[2,27348,0.0462050482],[2,27351,0.0146419473],[2,27352,0.0106560502],[2,27353,0.0142354677],[2,27361,0.0023222639],[2,27378,0.0467776877],[2,27382,0.0021420605],[2,27383,0.1408325207],[2,27400,0.0069318384],[2,27405,0.3540831703],[2,27409,0.0152852956],[2,27413,0.1149800828],[2,27425,0.0149304865],[2,27426,0.0158221904],[2,27438,0.0149483432],[2,27459,0.0408254215],[2,27460,0.1331857413],[2,27463,0.0028922363],[2,27494,0.0016684663],[2,27518,0.0082026053],[2,27524,0.0058266563],[2,27525,0.0131348833],[2,27526,0.0039694707],[2,27533,0.5189127714],[2,27543,0.2982272274],[2,27549,0.0027438634],[2,27552,0.0451963845],[2,27561,0.2338275487],[2,27565,0.0334085953],[2,27614,0.2086839108],[2,27626,0.042659117],[2,27670,0.024478462],[2,27704,0.0030751596],[2,27716,0.0966367921],[2,27728,0.0420484101],[2,27736,0.0147067694],[2,27740,0.3499213079],[2,27745,0.2096629889],[2,27771,0.028754589],[2,27773,0.0048300499],[2,27780,0.0306563644],[2,27782,0.0057289617],[2,27786,0.0152852956],[2,27789,0.1405614542],[2,27795,0.0407499465],[2,27797,0.0287706394],[2,27799,0.0072493037],[2,27806,0.0966390394],[2,27818,0.0021420605],[2,27845,0.0215062254],[2,27848,0.0016684663],[2,27857,0.127189232],[2,27860,0.067569703],[2,27867,0.0514057522],[2,27873,0.0592730362],[2,27874,0.322079993],[2,27891,0.0055391336],[2,27912,0.0700284182],[2,27915,0.0223204342],[2,27922,0.1420797006],[2,27927,0.1993527882],[2,27928,0.0700897466],[2,27953,0.0041461293],[2,27959,0.0518709217],[2,27967,0.0041461293],[2,27996,0.1201570497],[2,28001,0.0251155349],[2,28016,0.0139409048],[2,28026,0.1309511505],[2,28029,0.2503561576],[2,28033,0.0147932278],[2,28059,0.0126344734],[2,28065,0.0137680702],[2,28087,0.2517180931],[2,28088,0.0103253308],[2,28124,0.0974298749],[2,28139,0.2747215931],[2,28145,0.0296304726],[2,28155,0.0076349051],[2,28168,0.0048706598],[2,28176,0.3498556291],[2,28177,0.0816720718],[2,28179,0.263613028],[2,28192,0.0143001763],[2,28194,0.0586220186],[2,28210,0.0206162689],[2,28212,0.1448839618],[2,28216,0.0180231274],[2,28222,0.0193380576],[2,28232,0.0216352085],[2,28245,0.0093512171],[2,28253,0.0132762096],[2,28278,0.0333543288],[2,28289,0.1440341994],[2,28293,0.0157353413],[2,28299,0.0595942625],[2,28310,0.0095392338],[2,28348,0.0192920951],[2,28350,0.0479054886],[2,28377,0.0183615],[2,28380,0.0673475594],[2,28394,0.0094589953],[2,28430,0.0444709638],[2,28438,0.0445731929],[2,28448,0.0106524435],[2,28475,0.124226554],[2,28501,0.0378013359],[2,28506,0.3759933892],[2,28520,0.0160847668],[2,28526,0.0092837365],[2,28529,0.0429546507],[2,28540,0.2209750865],[2,28548,0.0183615],[2,28561,0.006877269],[2,28563,0.0206477617],[2,28565,0.0183615],[2,28568,0.0513095111],[2,28580,0.1143371306],[2,28585,0.017739307],[2,28587,0.1213997054],[2,28596,0.0057350999],[2,28626,0.0947162266],[2,28644,0.0069979814],[2,28656,0.0067642025],[2,28658,0.027404978],[2,28668,0.0183615],[2,28671,0.0361832254],[2,28674,0.0282920982],[2,28696,0.3591501345],[2,28703,0.0189960441],[2,28742,0.0181157877],[2,28750,0.0177805354],[2,28764,0.3867040226],[2,28794,0.003740708],[2,28824,0.043785819],[2,28828,0.011161281],[2,28837,0.3454207225],[2,28846,0.0084458997],[2,28853,0.027907163],[2,28884,0.0105408091],[2,28886,0.0388475935],[2,28899,0.1790266393],[2,28914,0.006111314],[2,28943,0.0186393511],[2,28950,0.0051628691],[2,28980,0.0448020903],[2,28982,0.0064763056],[2,28998,0.0069979814],[2,29002,0.0201619284],[2,29007,0.0183615],[2,29014,0.0183615],[2,29035,0.1828924832],[2,29039,0.0711212635],[2,29047,0.0094151717],[2,29051,0.0105130739],[2,29072,0.0223613042],[2,29080,0.0099379898],[2,29109,0.016181045],[2,29114,0.0187839681],[2,29123,0.0183615],[2,29134,0.0753150644],[2,29148,0.0111165946],[2,29151,0.0059162067],[2,29169,0.0251026826],[2,29171,0.0671776692],[2,29175,0.0059162067],[2,29176,0.1852790229],[2,29178,0.0046872789],[2,29182,0.1368132306],[2,29203,0.0158547131],[2,29221,0.0445660313],[2,29225,0.0047871501],[2,29235,0.4421423732],[2,29257,0.1341471375],[2,29258,0.016701195],[2,29316,0.0049680185],[2,29323,0.0047871501],[2,29341,0.1272429245],[2,29342,0.163964346],[2,29355,0.0242101292],[2,29384,0.0108764502],[2,29393,0.005531244],[2,29400,0.1541943829],[2,29431,0.174750452],[2,29446,0.1504396984],[2,29450,0.0046872789],[2,29452,0.4764030094],[2,29462,0.0212990926],[2,29470,0.0658922548],[2,29483,0.2001321361],[2,29484,0.027907163],[2,29498,0.0264464436],[2,29502,0.0161800065],[2,29532,0.0697095801],[2,29538,0.0382968302],[2,29545,0.0406388809],[2,29548,0.0060444459],[2,29562,0.006429683],[2,29563,0.0284684568],[2,29575,0.0202131704],[2,29579,0.2145569966],[2,29589,0.0216054814],[2,29598,0.4351962498],[2,29607,0.2670010762],[2,29612,0.0238270187],[2,29624,0.0847689696],[2,29634,0.007401979],[2,29647,0.0091066566],[2,29677,0.0499569925],[2,29707,0.0062484471],[2,29713,0.0242402547],[2,29721,0.0158538091],[2,29729,0.0441308127],[2,29745,0.012054976],[2,29750,0.0188838456],[2,29758,0.0173962745],[2,29764,0.0860415751],[2,29765,0.0075122287],[2,29768,0.0160306384],[2,29770,0.0363056066],[2,29773,0.0850792143],[2,29783,0.0056758084],[2,29809,0.0072756926],[2,29816,0.0398049627],[2,29819,0.090884965],[2,29824,0.0075122287],[2,29867,0.0107413459],[2,29871,0.0191165915],[2,29877,0.3798768609],[2,29880,0.098679561],[2,29882,0.0116581129],[2,29903,0.2258993161],[2,29920,0.144791563],[2,29932,0.0187295595],[2,29957,0.0279337687],[2,30055,0.0052998217],[2,30059,0.0056758084],[2,30087,0.0098421138],[2,30094,0.0080292856],[2,30096,0.16227465],[2,30105,0.0308766295],[2,30120,0.0969493571],[2,30124,0.0040719452],[2,30141,0.0467826864],[2,30145,0.0188838456],[2,30150,0.0163435332],[2,30197,0.0117662345],[2,30200,0.3172724711],[2,30202,0.0182373253],[2,30223,0.00869277],[2,30225,0.0098232932],[2,30257,0.0262850336],[2,30287,0.0448996315],[2,30340,0.0342958739],[2,30343,0.0102230975],[2,30363,0.0120740959],[2,30382,0.0072782831],[2,30391,0.0087942655],[2,30399,0.1584031539],[2,30413,0.0581278154],[2,30415,0.0599892038],[2,30427,0.1175079971],[2,30431,0.0363477138],[2,30432,0.0187527859],[2,30449,0.0066249293],[2,30457,0.0052998217],[2,30472,0.0098421138],[2,30494,0.5540527139],[2,30511,0.013518014],[2,30516,0.15491326],[2,30528,0.1467654142],[2,30555,0.0182497363],[2,30560,0.0190899268],[2,30565,0.0090888548],[2,30570,0.3449844004],[2,30577,0.0082333201],[2,30620,0.0070462164],[2,30643,0.0483736677],[2,30654,0.0067464673],[2,30670,0.0070318922],[2,30693,0.2989624242],[2,30694,0.1117771397],[2,30713,0.5309839924],[2,30742,0.0511143358],[2,30774,0.088753129],[2,30785,0.7724622761],[2,30799,0.0018506605],[2,30810,0.1804622654],[2,30850,0.0627930669],[2,30878,0.1087044202],[2,30892,0.0599773958],[2,30896,0.0094949849],[2,30951,0.4000978921],[2,30962,0.8292093231],[2,30968,0.002926331],[2,30969,0.9396061451],[2,30971,0.002130293],[2,30989,0.0180609459],[2,31026,0.0156545082],[2,31037,0.300389678],[2,31074,0.1075654423],[2,31089,0.0689959761],[2,31100,0.0743781527],[2,31140,0.0141489886],[2,31142,0.0029156099],[2,31169,0.0196532239],[2,31173,0.0084921431],[2,31180,0.2428633657],[2,31183,0.116350174],[2,31189,0.00568192],[2,31224,0.046037693],[2,31229,0.0127665165],[2,31245,0.003629145],[2,31290,0.0284408212],[2,31326,0.0156541832],[2,31377,0.0064581037],[2,31380,0.134828487],[2,31388,0.0144012869],[2,31394,0.0028323272],[2,31398,0.3234003107],[2,31412,0.3913742698],[2,31428,0.1752822499],[2,31429,0.1833450146],[2,31432,0.0319326076],[2,31443,0.0028323272],[2,31462,0.007304605],[2,31466,0.0097463967],[2,31467,0.3421650672],[2,31490,0.0199230926],[2,31500,0.0038097222],[2,31506,0.0782590979],[2,31507,0.0209271682],[2,31534,0.0239179154],[2,31541,0.1433964081],[2,31545,0.0692636308],[2,31554,0.0173922838],[2,31564,0.014898464],[2,31571,0.3032892337],[2,31575,0.0475181574],[2,31589,0.0139488566],[2,31592,0.0118457319],[2,31594,0.0156387984],[2,31597,0.1184090252],[2,31598,0.014898464],[2,31600,0.4117049763],[2,31641,0.0374102392],[2,31688,0.0091891368],[2,31697,0.0205054888],[2,31702,0.2244001213],[2,31711,0.017574718],[2,31724,0.042911749],[2,31728,0.1186281405],[2,31732,0.0025524208],[2,31734,0.0027327273],[2,31747,0.1356898773],[2,31751,0.0045168925],[2,31755,0.0048040071],[2,31775,0.0144416262],[2,31790,0.0121401489],[2,31802,0.021721845],[2,31814,0.0177588016],[2,31838,0.003777701],[2,31839,0.0048562649],[2,31840,0.0230264142],[2,31842,0.0163491689],[2,31853,0.0105423306],[2,31857,0.0816100975],[2,31866,0.1036281636],[2,31925,0.0256379927],[2,31927,0.1573936187],[2,31931,0.0016086594],[2,31935,0.0077795167],[2,32013,0.0073161025],[2,32021,0.4085498513],[2,32026,0.003574399],[2,32029,0.0743523047],[2,32033,0.0499871102],[2,32040,0.2501063988],[2,32044,0.0092765753],[2,32052,0.0025524208],[2,32074,0.035430206],[2,32075,0.0046969219],[2,32093,0.112237837],[2,32125,0.0074884877],[2,32138,0.0867193052],[2,32158,0.0208122284],[2,32174,0.1976018936],[2,32195,0.0174492393],[2,32196,0.0048562649],[2,32212,0.0087173843],[2,32239,0.0441044116],[2,32249,0.0025524208],[2,32264,0.0110223503],[2,32279,0.0263305056],[2,32284,0.0222210642],[2,32311,0.021993108],[2,32315,0.2339699734],[2,32333,0.0110021177],[2,32356,0.0163697897],[2,32357,0.046594324],[2,32363,0.0032070252],[2,32371,0.0032070252],[2,32390,0.0525181114],[2,32391,0.0295627991],[2,32423,0.040893992],[2,32428,0.0281071104],[2,32436,0.0126978844],[2,32437,0.0388551014],[2,32439,0.6514072916],[2,32459,0.0652239175],[2,32470,0.1660803355],[2,32475,0.0430117343],[2,32484,0.0126978844],[2,32485,0.0167784816],[2,32503,0.1704783474],[2,32515,0.1692926168],[2,32546,0.2499925637],[2,32557,0.0066261136],[2,32559,0.1122876914],[2,32560,0.0140523563],[2,32585,0.0093405728],[2,32614,0.0016086594],[2,32620,0.0158246091],[2,32629,0.0099201344],[2,32684,0.0028323272],[2,32691,0.0421739058],[2,32697,0.0074477725],[2,32721,0.0447675591],[2,32722,0.0260358953],[2,32734,0.0169136686],[2,32736,0.0091891368],[2,32740,0.0138404101],[2,32763,0.0093271294],[2,32769,0.0137173337],[2,32818,0.008681682],[2,32837,0.0146471969],[2,32884,0.0280568386],[2,32885,0.1233210942],[2,32916,0.0051771519],[2,32929,0.0025524208],[2,32930,0.0767249564],[2,32945,0.0425558125],[2,32952,0.0326226629],[2,32960,0.0110223503],[2,32967,0.0025524208],[2,32996,0.0056723436],[2,33009,0.0375199458],[2,33012,0.0039583157],[2,33020,0.0418373806],[2,33047,0.0279472682],[2,33048,0.0048562649],[2,33050,0.0532492105],[2,33055,0.0048562649],[2,33072,0.4044014375],[2,33074,0.0730002811],[2,33090,0.0395888849],[2,33111,0.0107060684],[2,33115,0.0060151794],[2,33132,0.6024772276],[2,33145,0.0075431836],[2,33150,0.0633269628],[2,33152,0.0035604772],[2,33155,0.0119283926],[2,33169,0.1692830515],[2,33190,0.0147279746],[2,33195,0.0354355469],[2,33207,0.0],[2,33225,0.0228520412],[2,33239,0.593618596],[2,33247,0.4910022199],[2,33274,0.0084799375],[2,33281,0.0493638671],[2,33336,0.0032403109],[2,33337,0.0025385187],[2,33341,0.0974171114],[2,33351,0.0021641697],[2,33369,0.0271842804],[2,33375,0.0158538726],[2,33376,0.2384768143],[2,33386,0.0105663295],[2,33388,0.0035604772],[2,33389,0.0021641697],[2,33392,0.0025271622],[2,33395,0.0065736232],[2,33402,0.0066955676],[2,33447,0.1379150558],[2,33468,0.0147279746],[2,33476,0.0015947756],[2,33479,0.0918915412],[2,33488,0.3145712385],[2,33496,0.0],[2,33513,0.0134703689],[2,33515,0.0162190878],[2,33523,0.0125572959],[2,33524,0.0021641697],[2,33531,0.0012209192],[2,33537,0.0188057507],[2,33557,0.0084792558],[2,33569,0.0075189857],[2,33574,0.0118109956],[2,33590,0.0067051577],[2,33606,0.0850669702],[2,33612,0.0147279746],[2,33614,0.2806123914],[2,33645,0.0107139267],[2,33651,0.0225365476],[2,33653,0.0106958355],[2,33661,0.0147279746],[2,33668,0.0032403109],[2,33697,0.0598895046],[2,33715,0.0239144433],[2,33717,0.0012209192],[2,33734,0.0009416415],[2,33739,0.001304015],[2,33746,0.0072835562],[2,33754,0.0081194588],[2,33776,0.0032403109],[2,33798,0.011467424],[2,33814,0.0057374742],[2,33828,0.0153900552],[2,33840,0.1029456176],[2,33857,0.0120932186],[2,33882,0.0118643062],[2,33930,0.0102797301],[2,33950,0.0431234451],[2,33960,0.0319800205],[2,34006,0.6077287741],[2,34028,0.0029017008],[2,34032,0.0231776113],[2,34049,0.0245295656],[2,34055,0.0023443786],[2,34056,0.0032403109],[2,34141,0.0021641697],[2,34220,0.1940251399],[2,34288,0.0034207905],[2,34297,0.001304015],[2,34311,0.0025385187],[2,34331,0.0021961189],[2,34364,0.0147279746],[2,34376,0.1102176297],[2,34441,0.048671887],[2,34443,0.0012209192],[2,34447,0.2447778396],[2,34455,0.0045946427],[2,34462,0.0693234364],[2,34481,0.0244651123],[2,34518,0.034556858],[2,34534,0.0145446529],[2,34557,0.0196178237],[2,34580,0.0077145464],[2,34604,0.1719720892],[2,34609,0.0017748414],[2,34615,0.0001796648],[2,34620,0.0157481721],[2,34624,0.0012209192],[2,34625,0.0203501087],[2,34628,0.4977318606],[2,34635,0.0037410372],[2,34636,0.0154361122],[2,34656,0.0023443786],[2,34664,0.0058259976],[2,34678,0.0021641697],[2,34679,0.6411963833],[2,34683,0.0943898501],[2,34690,0.0280879873],[2,34709,0.0113308733],[2,34711,0.0088937431],[2,34722,0.0141276413],[2,34724,0.3303793993],[2,34729,0.1776286839],[2,34734,0.1277282109],[2,34741,0.2168206549],[2,34747,0.1779349793],[2,34756,0.0090857662],[2,34757,0.1533880556],[2,34761,0.0620115413],[2,34768,0.0032403109],[2,34804,0.0015947756],[2,34805,0.014920061],[2,34832,0.0015947756],[2,34838,0.0088937431],[2,34848,0.0132865677],[2,34860,0.0976260112],[2,34870,0.0230041392],[2,34924,0.0178080287],[2,34930,0.0147279746],[2,34939,0.1734752099],[2,34950,0.1253033062],[2,34958,0.0052635602],[2,34993,0.0569872278],[2,34995,0.1886904305],[2,35000,0.1474587462],[2,35010,0.3135107626],[2,35042,0.0147905073],[2,35045,0.0096799592],[2,35050,0.0643342715],[2,35068,0.012374512],[2,35079,0.0064945251],[2,35111,0.3293615622],[2,35124,0.0021641697],[2,35189,0.1959813209],[2,35193,0.2995015032],[2,35203,0.0],[2,35240,0.0610874862],[2,35264,0.023486644],[2,35272,0.0415672111],[2,35275,0.2295765322],[2,35285,0.0178327046],[2,35313,0.0034207905],[2,35363,0.0075634505],[2,35371,0.0485864502],[2,35375,0.0672710204],[2,35404,0.2011975624],[2,35406,0.0305571502],[2,35415,0.6422168285],[2,35420,0.027786158],[2,35428,0.0012209192],[2,35441,0.0342530826],[2,35442,0.0235803873],[2,35469,0.0043041264],[2,35477,0.4634177549],[2,35480,0.0012209192],[2,35491,0.176900987],[2,35495,0.0151802339],[2,35505,0.2448274556],[2,35513,0.5118660645],[2,35563,0.0012209192],[2,35580,0.008234887],[2,35610,0.012987576],[2,35622,0.0151958225],[2,35635,0.0084315926],[2,35644,0.1346127281],[2,35658,0.0012209192],[2,35662,0.0056831517],[2,35665,0.0880413264],[2,35695,0.0534927014],[2,35701,0.0135500327],[2,35718,0.0147279746],[2,35746,0.0359336904],[2,35755,0.0067245077],[2,35756,0.1657581877],[2,35779,0.3710485657],[2,35786,0.0433614347],[2,35787,0.0410459091],[2,35794,0.0545336328],[2,35802,0.1270545738],[2,35814,0.0001796648],[2,35825,0.0610705115],[2,35840,0.0120193691],[2,35875,0.0103690081],[2,35879,0.0478506515],[2,35886,0.1405179448],[2,35959,0.0070519593],[2,35972,0.0147279746],[2,35977,0.0147279746],[2,35986,0.0009416415],[2,36013,0.2234798659],[2,36020,0.0151525535],[2,36043,0.0138132615],[2,36047,0.0032403109],[2,36053,0.0126974596],[2,36070,0.0305312128],[2,36115,0.0153811402],[2,36126,0.0039523328],[2,36130,0.0459251728],[2,36143,0.0132599537],[2,36147,0.0223139653],[2,36149,0.0174169794],[2,36163,0.0118764745],[2,36168,0.0075189857],[2,36173,0.0012209192],[2,36191,0.0295104573],[2,36199,0.0043200153],[2,36217,0.0080569888],[2,36219,0.0008949407],[2,36230,0.0211845746],[2,36236,0.0356491226],[2,36238,0.0144842894],[2,36245,0.1275113966],[2,36252,0.0599363032],[2,36276,0.0026720049],[2,36280,0.0089058225],[2,36286,0.0030938101],[2,36299,0.0156413607],[2,36304,0.0245378295],[2,36309,0.0279494424],[2,36321,0.0830770013],[2,36385,0.1228376057],[2,36388,0.0795632408],[2,36394,0.0936020742],[2,36405,0.217051494],[2,36411,0.002117389],[2,36415,0.0239278903],[2,36453,0.0337020493],[2,36465,0.0043200153],[2,36474,0.0018377615],[2,36475,0.0041393098],[2,36479,0.0032113877],[2,36486,0.0382912177],[2,36519,0.0034252679],[2,36520,0.024427722],[2,36562,0.0238042197],[2,36567,0.0029134127],[2,36571,0.1130284393],[2,36582,0.0268643411],[2,36596,0.002117389],[2,36615,0.0282755801],[2,36734,0.0042652168],[2,36738,0.0057395522],[2,36749,0.0088400698],[2,36755,0.0249776731],[2,36762,0.024427722],[2,36795,0.0165427461],[2,36807,0.067474565],[2,36817,0.0614083697],[2,36818,0.0559697506],[2,36851,0.0008949407],[2,36870,0.0959128903],[2,36874,0.0043200153],[2,36887,0.0022975862],[2,36898,0.0481384281],[2,36900,0.0177854035],[2,36905,0.0050874414],[2,36911,0.0573549272],[2,36917,0.1832764464],[2,36924,0.0153042439],[2,36930,0.0353945895],[2,36932,0.0029134127],[2,36944,0.082178416],[2,36972,0.0149110979],[2,36989,0.0322025603],[2,36993,0.0236313314],[2,37000,0.0106723991],[2,37004,0.002117389],[2,37015,0.0753734526],[2,37029,0.0117867542],[2,37034,0.0348293906],[2,37055,0.0780167222],[2,37056,0.0728678291],[2,37062,0.0390722963],[2,37064,0.038872825],[2,37067,0.0156413607],[2,37072,0.0041393098],[2,37082,0.0088426802],[2,37090,0.0150402755],[2,37095,0.0106723991],[2,37118,0.3336860649],[2,37152,0.011276935],[2,37183,0.0546702433],[2,37185,0.0049641421],[2,37187,0.0030938101],[2,37198,0.0132465817],[2,37219,0.0038002756],[2,37232,0.0169986978],[2,37243,0.0144619435],[2,37258,0.004459877],[2,37264,0.0041393098],[2,37372,0.2191407762],[2,37389,0.00658521],[2,37404,0.0286980743],[2,37419,0.0041393098],[2,37468,0.0156413607],[2,37472,0.0065972873],[2,37479,0.0075009933],[2,37480,0.0042102531],[2,37505,0.0207797701],[2,37520,0.0066396005],[2,37530,0.0164958637],[2,37533,0.0030618208],[2,37600,0.0235018271],[2,37608,0.0156413607],[2,37639,0.1079530213],[2,37673,0.0030938101],[2,37675,0.0300625393],[2,37718,0.1491117689],[2,37733,0.0041393098],[2,37774,0.0135973464],[2,37775,0.0022005888],[2,37810,0.2251474054],[2,37829,0.0901197992],[2,37840,0.0156413607],[2,37857,0.0220490071],[2,37858,0.0057395522],[2,37870,0.0274458371],[2,37871,0.0022005888],[2,37875,0.1848596542],[2,37877,0.0068268657],[2,37891,0.0288052603],[2,37892,0.0279797259],[2,37896,0.1374808888],[2,37917,0.0043200153],[2,37924,0.0038002756],[2,37993,0.0128382725],[2,38006,0.0328942007],[2,38011,0.0092311635],[2,38014,0.5332886794],[2,38020,0.1237528646],[2,38025,0.0024917136],[2,38038,0.0024917136],[2,38054,0.0030618208],[2,38079,0.0156413607],[2,38082,0.0323213074],[2,38123,0.0273558157],[2,38143,0.1357529309],[2,38145,0.014233213],[2,38209,0.0066208631],[2,38220,0.0030618208],[2,38229,0.0358039332],[2,38244,0.0927729778],[2,38247,0.0167989424],[2,38248,0.0882064804],[2,38288,0.1253839204],[2,38324,0.0038002756],[2,38341,0.0041393098],[2,38344,0.232264645],[2,38355,0.0551026883],[2,38363,0.0034252679],[2,38373,0.4839208551],[2,38374,0.7225801496],[2,38398,0.022076226],[2,38447,0.2416875867],[2,38453,0.0055031583],[2,38466,0.1848994793],[2,38483,0.567738835],[2,38503,0.0049810602],[2,38509,0.0043200153],[2,38540,0.0065972873],[2,38567,0.0513103851],[2,38637,0.088150556],[2,38659,0.2489979633],[2,38665,0.0156413607],[2,38706,0.004084525],[2,38749,0.024766123],[2,38789,0.376640502],[2,38836,0.1352703409],[2,38853,0.1336632572],[2,38871,0.0137330508],[2,38936,0.0030618208],[2,38939,0.0972201551],[2,38977,0.014815852],[2,38980,0.2029669402],[2,38983,0.0189375226],[2,38991,0.0038002756],[2,38998,0.0066396005],[2,39016,0.0422017507],[2,39029,0.0365986003],[2,39032,0.0500312157],[2,39040,0.317736986],[2,39046,0.1223908348],[2,39064,0.0239278903],[2,39069,0.0541758539],[2,39104,0.007627278],[2,39115,0.013008929],[2,39122,0.0077900738],[2,39124,0.060978683],[2,39128,0.225709834],[2,39130,0.0057395522],[2,39138,0.0030938101],[2,39147,0.0066838213],[2,39173,0.0447068007],[2,39177,0.0066838213],[2,39180,0.0084233432],[2,39191,0.1924557485],[2,39200,0.0041393098],[2,39203,0.0264071369],[2,39229,0.0865116388],[2,39248,0.0156413607],[2,39250,0.089276011],[2,39299,0.0202709717],[2,39305,0.0312794644],[2,39332,0.0024917136],[2,39335,0.0141715012],[2,39358,0.1595805806],[2,39370,0.0814413938],[2,39378,0.2287045866],[2,39382,0.0008949407],[2,39386,0.0418410573],[2,39403,0.019006679],[2,39437,0.0039808959],[2,39477,0.002117389],[2,39502,0.2242763834],[2,39505,0.0008949407],[2,39535,0.0475738986],[2,39569,0.0123767266],[2,39581,0.0176524487],[2,39610,0.0043200153],[2,39614,0.2462621187],[2,39626,0.0008949407],[2,39629,0.004459877],[2,39640,0.0086191488],[2,39647,0.0764776163],[2,39725,0.1403472412],[2,39733,0.0043200153],[2,39774,0.0131145017],[2,39788,0.0041393098],[2,39837,0.0079544754],[2,39846,0.1545157974],[2,39886,0.0194287966],[2,39896,0.0200050938],[2,39897,0.0316044722],[2,39905,0.1751077223],[2,39939,0.0156413607],[2,39945,0.0612131868],[2,39946,0.0041393098],[2,39987,0.1002981555],[2,39996,0.0043200153],[2,40001,0.0024917136],[2,40010,0.0156413607],[2,40011,0.0257459427],[2,40018,0.0041393098],[2,40019,0.055024918],[2,40059,0.2916399235],[2,40071,0.0875498823],[2,40163,0.0983964202],[2,40170,0.021614289],[2,40171,0.0050424562],[2,40183,0.0171111431],[2,40187,0.016499325],[2,40193,0.0020178884],[2,40225,0.0230125472],[2,40245,0.0163325133],[2,40250,0.0034366386],[2,40300,0.0256609344],[2,40308,0.0242419621],[2,40318,0.0274224409],[2,40324,0.0327064538],[2,40335,0.0713273876],[2,40354,0.0485988619],[2,40355,0.0075009933],[2,40373,0.0167225277],[2,40384,0.0043200153],[2,40476,0.0038002756],[2,40507,0.0160491921],[2,40546,0.0086191488],[2,40577,0.0212654531],[2,40632,0.0222631174],[2,40762,0.0023808069],[2,40776,0.0152237219],[2,40844,0.0026556275],[2,40862,0.019826001],[2,40893,0.022695759],[2,40917,0.0050547293],[2,40922,0.0026556275],[2,40938,0.0400257217],[2,40951,0.0277852493],[2,40952,0.0499260864],[2,40970,0.0408338525],[2,40973,0.0264133138],[2,40983,0.0233146007],[2,41016,0.0287525955],[2,41086,0.3438239251],[2,41154,0.0272831467],[2,41213,0.0021239295],[2,41231,0.031179327],[2,41239,0.0039040463],[2,41243,0.0033484777],[2,41246,0.0066917616],[2,41275,0.0029850694],[2,41300,0.3310232087],[2,41301,0.0040624432],[2,41302,0.0083702441],[2,41322,0.0177268046],[2,41338,0.0277852493],[2,41347,0.0123443842],[2,41371,0.0189954537],[2,41397,0.158912481],[2,41402,0.050674694],[2,41406,0.3281102741],[2,41420,0.0452859034],[2,41463,0.0040624432],[2,41464,0.0177268046],[2,41474,0.2051621713],[2,41602,0.3387900074],[2,41623,0.0024150232],[2,41628,0.0540329295],[2,41643,0.0066066826],[2,41654,0.0029850694],[2,41692,0.0024150232],[2,41702,0.0040624432],[2,41705,0.0092081555],[2,41730,0.0169889371],[2,41741,0.0438832761],[2,41819,0.0188431029],[2,41827,0.0041909161],[2,41836,0.005723101],[2,41838,0.017330319],[2,41847,0.0631304513],[2,41855,0.1707829966],[2,41858,0.0217215568],[2,41875,0.0177331957],[2,41883,0.0008184211],[2,41912,0.0028366772],[2,41964,0.1352142357],[2,41984,0.0029850694],[2,41987,0.0040624432],[2,41990,0.0619425661],[2,41996,0.0029850694],[2,42013,0.0183301031],[2,42016,0.0179109264],[2,42019,0.0040624432],[2,42027,0.4369879396],[2,42029,0.0042431294],[2,42038,0.0025952952],[2,42039,0.0802826447],[2,42040,0.0113968524],[2,42064,0.2415746717],[2,42085,0.0376904687],[2,42128,0.0082067224],[2,42142,0.0996445468],[2,42171,0.0028366772],[2,42192,0.0035403567],[2,42206,0.3728149824],[2,42214,0.0179109264],[2,42228,0.0042431294],[2,42243,0.061146079],[2,42247,0.0269484728],[2,42257,0.0632241589],[2,42318,0.0083460184],[2,42324,0.0272831467],[2,42327,0.0043829761],[2,42376,0.0371438027],[2,42387,0.0183301031],[2,42388,0.0563693784],[2,42414,0.0179109264],[2,42418,0.0843249379],[2,42446,0.0183301031],[2,42473,0.0157684121],[2,42487,0.1182991508],[2,42491,0.1602142675],[2,42495,0.0290529047],[2,42497,0.0087653106],[2,42518,0.0042431294],[2,42558,0.0037234453],[2,42566,0.0009982917],[2,42580,0.0008184211],[2,42602,0.0257963039],[2,42633,0.1100093617],[2,42674,0.086994387],[2,42721,0.0243659439],[2,42729,0.0158830327],[2,42731,0.0657407572],[2,42735,0.0021239295],[2,42792,0.0185728771],[2,42811,0.0019412487],[2,42829,0.0033484777],[2,42847,0.0855779799],[2,42873,0.0115265068],[2,42875,0.6768809329],[2,42876,0.1082880988],[2,42881,0.1390269813],[2,42916,0.004010243],[2,42943,0.0984460213],[2,42960,0.0033484777],[2,42974,0.0419082804],[2,43010,0.0042431294],[2,43035,0.0022209166],[2,43037,0.0033484777],[2,43053,0.0183301031],[2,43055,0.0539121913],[2,43122,0.0037234453],[2,43132,0.0029850694],[2,43186,0.0091207253],[2,43219,0.0853928285],[2,43225,0.1146044669],[2,43273,0.1100033424],[2,43309,0.0084239934],[2,43392,0.0137853155],[2,43397,0.0126046832],[2,43535,0.0093148427],[2,43567,0.0218850171],[2,43570,0.3218708088],[2,43578,0.0093148427],[2,43592,0.0352029353],[2,43628,0.8436158384],[2,43662,0.2475804765],[2,43663,0.0093148427],[2,43666,0.0080157145],[2,43693,0.0822823351],[2,43730,0.0093148427],[2,43760,0.0176769967],[2,43805,0.2141221031],[2,43812,0.0128312172],[2,43828,0.0089032102],[2,43832,0.1001319106],[2,43841,0.0093148427],[2,43855,0.0241947702],[2,43856,0.0884933912],[2,43927,0.1305713947],[2,44050,0.0183493395],[2,44051,0.0081999441],[2,44087,0.6351119082],[2,44114,0.1308050918],[2,44208,0.0304843568],[2,44307,0.0160336134],[2,44323,0.0278922317],[2,44331,0.0304430746],[2,44345,0.1080715685],[2,44353,0.1129352229],[2,44382,0.0244833006],[2,44392,0.0110584314],[2,44420,0.0327461476],[2,44428,0.0238684659],[2,44445,0.0148168129],[2,44510,0.0091328816],[2,44591,0.0087479117],[2,44598,0.0090555277],[2,44610,0.1535169734],[2,44652,0.0085660931],[2,44700,0.0087479117],[2,44717,0.0079706684],[2,44737,0.0220564192],[2,44836,0.0229231671],[2,44904,0.0074819675],[2,44939,0.0951762911],[2,44940,0.0115181585],[2,45039,0.0461089972],[2,45069,0.0324375679],[2,45244,0.7094822244],[2,45246,0.0251067986],[2,45358,0.0319323187],[2,45365,0.0079706684],[2,45393,0.0317869905],[2,45510,0.0493205392],[2,45602,0.0238523949],[2,45714,0.1311367087],[2,45722,0.0673665324],[2,45887,0.0970104051],[2,45931,0.0945044225],[2,46478,0.8620214995],[2,46500,0.0300768603],[2,46551,0.0085660931],[2,46611,0.0764213756],[2,46616,0.0087479117],[2,46641,0.006250369],[2,46673,0.2477287933],[2,46680,0.0215737123],[2,46769,0.0188648368],[2,46806,0.1127631717],[2,46818,0.0071035453],[2,46825,0.031838417],[2,46836,0.0508114742],[2,46887,0.011316196],[2,46905,0.0146207532],[2,47031,0.0239902882],[2,47172,0.0330463788],[2,47175,0.0724268605],[2,47179,0.059381657],[2,47230,0.0147719325],[2,47262,0.0173776417],[2,47284,0.0383504592],[2,47298,0.17838372],[2,47315,0.0908237038],[2,47349,0.0111844156],[2,47366,0.0087479117],[2,47370,0.0246388728],[2,47383,0.3858090614],[2,47397,0.1058290738],[2,47412,0.0946122842],[2,47448,0.0328505303],[2,47545,0.0057889662],[2,47579,0.0067382362],[2,47623,0.0177591048],[2,47639,0.0319323187],[2,47672,0.0461394053],[2,47678,0.0474072639],[2,47705,0.0087479117],[2,47762,0.0080028765],[2,47812,0.0245922352],[2,47828,0.0542825455],[2,47835,0.0070197764],[2,47858,0.0213045879],[2,47875,0.0758807382],[2,47892,0.0080028765],[2,47901,0.219206568],[2,47947,0.0171035167],[2,47997,0.0379555694],[2,48061,0.0072012062],[2,48071,0.0267353136],[2,48081,0.0070882121],[2,48109,0.1359970462],[2,48140,0.0334049271],[2,48143,0.0098283219],[2,48265,0.0093383647],[2,48364,0.0227036471],[2,48383,0.0463290213],[2,48419,0.0061701457],[2,48436,0.0203759653],[2,48463,0.0105703851],[2,48547,0.0559765439],[2,48560,0.0121517201],[2,48587,0.0061701457],[2,48601,0.1987918028],[2,48627,0.0118936897],[2,48704,0.004705091],[2,48707,0.0294248203],[2,48791,0.0190466127],[2,48817,0.1076971561],[2,48901,0.165918873],[2,48904,0.0108027611],[2,48947,0.1220999332],[2,48960,0.0111087611],[2,49046,0.0151441603],[2,49135,0.0202948837],[2,49154,0.0351219631],[2,49235,0.0307849911],[2,49286,0.0587410195],[2,49319,0.0202948837],[2,49334,0.1065994047],[2,49385,0.0987040005],[2,49390,0.009390682],[2,49442,0.8965282688],[2,49448,0.0559363777],[2,49456,0.2313185321],[2,49460,0.0088146228],[2,49471,0.0343173148],[2,49494,0.0101347234],[2,49626,0.0356580653],[2,49632,0.4460526589],[2,49686,0.3109233606],[2,49714,0.0729257053],[2,49749,0.0054258859],[2,49838,0.0303320618],[2,49933,0.0066560758],[2,49948,0.0269718399],[2,49967,0.0231117887],[2,49993,0.0197325921],[2,50007,0.0222381175],[2,50023,0.4262883712],[2,50032,0.0054258859],[2,50078,0.0230503509],[2,50091,0.0702963238],[2,50101,0.0101347234],[2,50109,0.0512825395],[2,50128,0.0237199867],[2,50144,0.0718715494],[2,50151,0.5024382436],[2,50158,0.0237361589],[2,50165,0.03037122],[2,50171,0.0189563951],[2,50207,0.0088726509],[2,50268,0.3312387337],[2,50305,0.164956502],[2,50339,0.0230503509],[2,50388,0.0088726509],[2,50419,0.0828680511],[2,50445,0.1087230623],[2,50458,0.0054258859],[2,50498,0.0161655826],[2,50553,0.1759443173],[2,50567,0.1282900548],[2,50591,0.3784876109],[2,50592,0.0096619737],[2,50661,0.0343949128],[2,50670,0.0365519977],[2,50674,0.0969480957],[2,50706,0.3764344308],[2,50730,0.0798269131],[2,50740,0.2354317484],[2,50772,0.0226624586],[2,50777,0.0689439045],[2,50786,0.3284114647],[2,50795,0.1196260341],[2,50813,0.0213742235],[2,50839,0.0287564004],[2,50848,0.0103803654],[2,50849,0.0233375736],[2,50860,0.0302071994],[2,50915,0.0699326348],[2,50929,0.1347912628],[2,50966,0.0071612749],[2,51000,0.1796141375],[2,51046,0.0238551423],[2,51067,0.0546977841],[2,51077,0.0146245154],[2,51174,0.0586307962],[2,51213,0.0508796132],[2,51225,0.0510712433],[2,51232,0.0238551423],[2,51272,0.0327886962],[2,51296,0.0119544717],[2,51314,0.0228478214],[2,51318,0.0187421387],[2,51345,0.059500921],[2,51425,0.0886479183],[2,51588,0.0096619737],[2,51596,0.0306048991],[2,51610,0.0360494681],[2,51643,0.0062114481],[2,51647,0.0096038823],[2,51656,0.0096038823],[2,51717,0.0096619737],[2,51724,0.0318781096],[2,51729,0.0283355032],[2,51734,0.1209596546],[2,51759,0.1704177163],[2,51762,0.0206280742],[2,51786,0.0235039648],[2,51791,0.073162407],[2,51798,0.1885728613],[2,51830,0.0096619737],[2,51844,0.0328756787],[2,51863,0.0074429802],[2,51868,0.0211130903],[2,51884,0.1622598163],[2,51969,0.0145005175],[2,52002,0.0266673722],[2,52006,0.0098942773],[2,52083,0.0088081605],[2,52135,0.0078561664],[2,52169,0.0100183051],[2,52210,0.0092743833],[2,52296,0.0201769527],[2,52304,0.0143324543],[2,52318,0.0589510823],[2,52325,0.0066239296],[2,52359,0.0098942773],[2,52364,0.0100183051],[2,52370,0.0971296024],[2,52391,0.0612305558],[2,52461,0.0086585641],[2,52480,0.0252873435],[2,52513,0.0091745179],[2,52542,0.0100183051],[2,52568,0.0339752007],[2,52594,0.0098942773],[2,52659,0.0106585765],[2,52707,0.1449041095],[2,52718,0.0158579474],[2,52775,0.0242777206],[2,52777,0.0066239296],[2,52800,0.1284288551],[2,52897,0.0236843021],[2,52928,0.0100183051],[2,52939,0.0100764298],[2,52950,0.0190766088],[2,52955,0.0626802028],[2,52960,0.2327825013],[2,53214,0.072534285],[2,53228,0.0111555722],[2,53234,0.0100764298],[2,53258,0.3511158205],[2,53294,0.0249484707],[2,53493,0.3859219079],[2,53498,0.0111555722],[2,53603,0.0743647751],[2,53605,0.0474803672],[2,53608,0.0593841568],[2,53610,0.0214845674],[2,53616,0.0409967738],[2,53635,0.5062037958],[2,53685,0.1555812661],[2,53724,0.1378514973],[2,53742,0.2259982083],[2,53747,0.196096653],[2,53814,0.0654416636],[2,53834,0.0184904027],[2,53849,0.0059704458],[2,54178,0.1952591125],[2,54202,0.0343237135],[2,54212,0.0333515459],[2,54225,0.0069323964],[2,54226,0.0107284557],[2,54299,0.0606924877],[2,54340,0.0172284973],[2,54345,0.0040482818],[2,54411,0.0828178229],[2,54414,0.0077258868],[2,54438,0.0145161343],[2,54443,0.0052761179],[2,54519,0.0120193535],[2,54555,0.0217611307],[2,54561,0.4477327259],[2,54626,0.0054571093],[2,54676,0.0252226045],[2,54683,0.0054571093],[2,54692,0.0486509187],[2,54701,0.0193377819],[2,54767,0.0031025519],[2,54778,0.0230952115],[2,54791,0.015605671],[2,54816,0.0602768348],[2,54822,0.0474010372],[2,54887,0.0079074941],[2,54900,0.0160310312],[2,54935,0.0431777127],[2,54991,0.0031025519],[2,54994,0.0244059354],[2,55030,0.1803157063],[2,55110,0.0056406865],[2,55129,0.0314255805],[2,55154,0.1364766393],[2,55155,0.0205390584],[2,55156,0.0052761179],[2,55207,0.0031025519],[2,55224,0.0200029605],[2,55260,0.3808341446],[2,55307,0.0064803539],[2,55316,0.0052761179],[2,55328,0.0206045453],[2,55345,0.0150073758],[2,55358,0.0109406811],[2,55493,0.0314255805],[2,55513,0.0100213573],[2,55640,0.021548298],[2,55706,0.0233174164],[2,55774,0.0189859529],[2,55778,0.0189859529],[2,55858,0.0225913249],[2,55994,0.0217520874],[2,56045,0.0056406865],[2,56093,0.0134050351],[2,56129,0.0052761179],[2,56147,0.3577642627],[2,56193,0.0056406865],[2,56214,0.0318504051],[2,56241,0.0082825622],[2,56280,0.0189859529],[2,56285,0.0054571093],[2,56376,0.0457417075],[2,56406,0.0498204629],[2,56415,0.0052761179],[2,56445,0.0124976326],[2,56458,0.0261401718],[2,56460,0.0189859529],[2,56464,0.008459465],[2,56466,0.0189859529],[2,56531,0.1692395103],[2,56550,0.0204005244],[2,56559,0.0794636691],[2,56606,0.0918402183],[2,56645,0.0033662996],[2,56655,0.0033662996],[2,56742,0.0258554608],[2,56886,0.0135833626],[2,56933,0.0179214069],[2,56960,0.0160855869],[2,57011,0.0313117002],[2,57054,0.0186381508],[2,57086,0.0176229101],[2,57120,0.0045929704],[2,57156,0.004616783],[2,57189,0.0081843259],[2,57217,0.0031858338],[2,57220,0.0099538932],[2,57284,0.0120633512],[2,57324,0.0071596452],[2,57345,0.016024465],[2,57390,0.0029780303],[2,57397,0.0300018994],[2,57432,0.0175860118],[2,57478,0.0019130233],[2,57514,0.0040829868],[2,57554,0.0168497164],[2,57600,0.0072352089],[2,57675,0.1249997696],[2,57747,0.0894164861],[2,57769,0.0206615077],[2,57794,0.0017329228],[2,57821,0.0571072454],[2,57849,0.541409868],[2,58163,0.00663924],[2,58251,0.0031851918],[2,58258,0.0044115533],[2,58271,0.2491125887],[2,58275,0.1340412136],[2,58441,0.0965990956],[2,58486,0.024661088],[2,58870,0.0099532452],[2,58882,0.0406027807],[2,58900,0.1321838421],[2,58981,0.0242418995],[2,59011,0.0181049223],[2,59022,0.0103857306],[2,59075,0.0035487015],[2,59181,0.0244811896],[2,59186,0.0261427937],[2,59193,0.3333837485],[2,59196,0.0147152106],[2,59199,0.0045923273],[2,59205,0.0405758844],[2,59237,0.3678018812],[2,59256,0.0227511682],[2,59361,0.0125062671],[2,59409,0.0068229208],[2,59439,0.0107440272],[2,59455,0.0093305563],[2,59465,0.0183738615],[2,59473,0.0355465426],[2,59501,0.0036291865],[2,59527,0.0163913055],[2,59539,0.0025037137],[2,59541,0.0057217785],[2,59561,0.420852103],[2,59564,0.0034486546],[2,59575,0.0831451772],[2,59585,0.1234916821],[2,59586,0.0232650621],[2,59621,0.0048563065],[2,59633,0.0136466825],[2,59654,0.0183738615],[2,59669,0.0048563065],[2,59697,0.0046754661],[2,59700,0.0097484681],[2,59750,0.2911693105],[2,59780,0.0183738615],[2,59875,0.0146238982],[2,59991,0.0183738615],[2,59998,0.0039095132],[2,60089,0.066705997],[2,60092,0.0159122561],[2,60095,0.0232505218],[2,60136,0.0218164543],[2,60241,0.0202508588],[2,60324,0.0194866231],[2,60336,0.0224927671],[2,60362,0.0232234153],[2,60375,0.0988895304],[2,60418,0.0100011272],[2,60668,0.0224311806],[2,60678,0.0110070784],[2,60751,0.0054640761],[2,60770,0.0264681445],[2,60779,0.0740007285],[2,60980,0.1348926028],[2,61032,0.0160771393],[2,61070,0.056170182],[2,61166,0.0365217577],[2,61167,0.008767904],[2,61172,0.0080345357],[2,61185,0.0266556224],[2,61338,0.0034359708],[2,61372,0.0795656994],[2,61398,0.182330678],[2,61401,0.0056105466],[2,61410,0.1108919807],[2,61425,0.0059752846],[2,61437,0.0189070043],[2,61449,0.1674462272],[2,61582,0.0193267509],[2,61643,0.0062266019],[2,61801,0.1821778733],[2,61838,0.0189070043],[2,61861,0.6120419842],[2,61872,0.8235654659],[2,61973,0.1166399621],[2,62161,0.0070033725],[2,62162,0.5629379668],[2,62215,0.0774424704],[2,62223,0.0200017226],[2,62252,0.0125500823],[2,62255,0.005635694],[2,62266,0.0052949629],[2,62480,0.0256658696],[2,62628,0.0615194654],[2,62657,0.005635694],[2,62659,0.005635694],[2,62723,0.0997801592],[2,62913,0.0088770055],[2,63094,0.0191678931],[2,63103,0.0054546577],[2,63111,0.2933769048],[2,63119,0.1240548716],[2,63317,0.0191678931],[2,63327,0.0443188297],[2,63341,0.0058406288],[2,63372,0.0135598117],[2,63382,0.1892447338],[2,63389,0.0045905541],[2,63582,0.0181961596],[2,63634,0.0206705038],[2,63638,0.020914053],[2,63716,0.0181864117],[2,63739,0.069471869],[2,63759,0.0071105926],[2,63766,0.1674431593],[2,63821,0.0810899962],[2,63842,0.045164247],[2,63850,0.2458920503],[2,63856,0.0092439441],[2,63906,0.0450808304],[2,63992,0.0285187978],[2,64051,0.1474260356],[2,64094,0.0208891419],[2,64246,0.0257389274],[2,64362,0.0577840496],[2,64430,0.1586019555],[2,64449,0.1486274776],[2,64458,0.1162965423],[2,64545,0.0059796643],[2,64650,0.0297066436],[2,64677,0.0126132131],[2,64760,0.0380484234],[2,64779,0.0052113659],[2,64789,0.1548228454],[2,64833,0.013116505],[2,64857,0.0042641052],[2,64870,0.0044448422],[2,64914,0.0894625478],[2,64928,0.0069291856],[2,65044,0.0381311879],[2,65105,0.0217806964],[2,65116,0.0563301486],[2,65193,0.0508603092],[2,65422,0.0206705038],[2,65453,0.0196190752],[2,65506,0.0215301046],[2,65525,0.0254374896],[2,65562,0.0204856883],[2,65625,0.4339464347],[2,65637,0.1495696757],[2,65932,0.2452135286],[2,66170,0.0144313166],[2,66237,0.0488817487],[2,66273,0.0249883556],[2,66303,0.0386395909],[2,66313,0.2055981632],[2,66498,0.0115816564],[2,66553,0.0333923722],[2,66558,0.0300321801],[2,66646,0.0249883556],[2,66688,0.1007952379],[2,66735,0.0249883556],[2,66737,0.386143395],[2,66767,0.2924595032],[2,66809,0.0341027662],[2,66814,0.0277035884],[2,66845,0.0221224764],[2,66938,0.0117642332],[2,67019,0.0131873777],[2,67132,0.0345743415],[2,67175,0.0478346123],[2,67180,0.0274665127],[2,67190,0.0115816564],[2,67278,0.016404787],[2,67319,0.0252333351],[2,67398,0.0257340503],[2,67403,0.0117642332],[2,67448,0.0924860695],[2,67479,0.1455350812],[2,67486,0.2552197846],[2,67502,0.1883999847],[2,67508,0.0951263812],[2,67513,0.9118615029],[2,67516,0.2296321523],[2,67572,0.0809737182],[2,67625,0.0185402211],[2,67683,0.0248024547],[2,67710,0.042316011],[2,67787,0.0546788405],[2,67798,0.0130044432],[2,67803,0.0288768629],[2,67844,0.0249883556],[2,67953,0.2219385205],[2,68064,0.2600865886],[2,68098,0.0298890192],[2,68167,0.0115816564],[2,68238,0.0231735641],[2,68331,0.4619885732],[2,68376,0.0248024547],[2,68459,0.0306134999],[2,68461,0.5648818497],[2,68469,0.0991828198],[2,68488,0.135050032],[2,68503,0.010278469],[2,68505,0.0115816564],[2,68632,0.0249883556],[2,68650,0.3359151347],[2,68651,0.7663711646],[2,68683,0.0137262096],[2,68699,0.0115816564],[2,68774,0.0115816564],[2,68802,0.0128041136],[2,68810,0.0254479533],[2,68877,0.0268362207],[2,69042,0.0732022874],[2,69112,0.0264044067],[2,69156,0.0316806907],[2,69179,0.0770774774],[2,69215,0.5393192547],[2,69229,0.1742413921],[2,69241,0.0217966984],[2,69271,0.0063817051],[2,69331,0.3338567498],[2,69439,0.0369998424],[2,69467,0.5054557605],[2,69524,0.0169982266],[2,69631,0.0136571715],[2,69640,0.022642195],[2,69642,0.7442514549],[2,69767,0.0136571715],[2,69798,0.0092780266],[2,69802,0.0059570524],[2,69814,0.0094600242],[2,69857,0.0232807551],[2,69890,0.2820027254],[2,69976,0.1555677953],[2,70048,0.0919592919],[2,70054,0.0286928072],[2,70099,0.0530697531],[2,70132,0.36661055],[2,70137,0.0705693878],[2,70164,0.0492937764],[2,70166,0.0243275907],[2,70234,0.0243789255],[2,70237,0.6539076426],[2,70280,0.0132113868],[2,70296,0.2544845289],[2,70304,0.1396242331],[2,70346,0.3477539673],[2,70444,0.3711720486],[2,70541,0.0115882735],[2,70637,0.0403539677],[2,70715,0.0090272875],[2,70761,0.0368340373],[2,70767,0.3411568821],[2,70771,0.0339518327],[2,70810,0.0222696966],[2,70832,0.0241056335],[2,70939,0.0415741338],[2,71073,0.0837544701],[2,71098,0.0277464286],[2,71103,0.0648464813],[2,71162,0.0124403361],[2,71172,0.0138654018],[2,71201,0.6097890245],[2,71290,0.3934940744],[2,71372,0.0518011563],[2,71481,0.4098836279],[2,71521,0.0689643977],[2,71527,0.4549525357],[2,71529,0.0786892214],[2,71551,0.1097924624],[2,71608,0.0104503284],[2,71630,0.0276432954],[2,71660,0.2238884431],[2,71722,0.0486331184],[2,71771,0.13971058],[2,71778,0.1883262724],[2,71841,0.0407283331],[2,71867,0.0232475069],[2,71880,0.0296998813],[2,71916,0.0423211617],[2,71937,0.0266809711],[2,72058,0.4480065818],[2,72127,0.0265451479],[2,72137,0.0094824683],[2,72138,0.0094824683],[2,72147,0.7286799034],[2,72166,0.0285850115],[2,72230,0.0118820099],[2,72260,0.0283818232],[2,72278,0.0499493717],[2,72313,0.0242828706],[2,72385,0.0130025793],[2,72413,0.0262492476],[2,72414,0.0274287837],[2,72492,0.0305889903],[2,72555,0.4384453056],[2,72596,0.6351018327],[2,72610,0.0515345683],[2,72622,0.141566356],[2,72678,0.0534967046],[2,72690,0.0094824683],[2,73479,0.093680695],[2,73486,0.2136566349],[2,73501,0.0094824683],[2,73507,0.1143972193],[2,73511,0.0430936417],[2,73517,0.0260630298],[2,73557,0.1176328534],[2,73597,0.0292195965],[2,73611,0.0217368203],[2,73644,0.3572053309],[2,73663,0.0419034023],[2,73673,0.0154878279],[2,73767,0.1287795433],[2,73800,0.2136008182],[2,74290,0.0104366054],[2,74297,0.0186984482],[2,74307,0.0162668863],[2,74329,0.0094824683],[2,74348,0.0146803688],[2,74434,0.0160894561],[2,74508,0.3025543005],[2,74510,0.7441052392],[2,74640,0.0285026116],[2,74743,0.0258255417],[2,74870,0.0262492476],[2,74882,0.0172202623],[2,74968,0.0866081116],[2,75135,0.4005963585],[2,75148,0.0134824357],[2,75154,0.0417574206],[2,75186,0.2910083615],[2,75430,0.0158872551],[2,75598,0.0130025793],[2,75643,0.0168850135],[2,75646,0.2795225933],[2,75662,0.11432335],[2,75675,0.2429696174],[2,75711,0.0128196912],[2,75770,0.0104366054],[2,75812,0.0129039362],[2,75868,0.0262492476],[2,75947,0.0281889772],[2,76194,0.017828444],[2,76241,0.7548039062],[2,76309,0.0217013892],[2,76388,0.0552359602],[2,76602,0.0509830403],[2,76634,0.0118820099],[2,76661,0.5075577567],[2,77130,0.0430936417],[2,77186,0.1608704421],[2,77313,0.0264946448],[2,77590,0.3886402654],[2,77638,0.0184785371],[2,77641,0.0589077015],[2,77669,0.0430936417],[2,77715,0.017921067],[2,77726,0.0227576797],[2,77769,0.0306878719],[2,77790,0.0106188943],[2,77798,0.0118624179],[2,77839,0.0128196912],[2,77856,0.0480551419],[2,77868,0.0342262829],[2,77922,0.1335833358],[2,77952,0.1412490581],[2,77995,0.0371061387],[2,78028,0.0576803187],[2,78035,0.0127123656],[2,78105,0.3712221671],[2,78128,0.0604322429],[2,78178,0.1289961403],[2,78287,0.0173966969],[2,78309,0.0296163088],[2,78342,0.0106188943],[2,78581,0.0492098573],[2,78629,0.0094824683],[2,78662,0.0269110938],[2,78664,0.0260630298],[2,78669,0.0258255417],[2,78762,0.0326719754],[2,78776,0.0281951271],[2,78806,0.0244009353],[2,78825,0.0226925064],[2,78868,0.0341452173],[2,78877,0.0258939103],[2,78918,0.163755395],[2,78942,0.0312516122],[2,79005,0.0243593142],[2,79051,0.0266809711],[2,79068,0.0127123656],[2,79126,0.0146386994],[2,79191,0.1898427091],[2,79225,0.1000054993],[2,79261,0.03713598],[2,79346,0.1474106692],[2,79522,0.0356630636],[2,79535,0.0138437105],[2,79572,0.1096596094],[2,79600,0.0282248198],[2,79607,0.0104366054],[2,79740,0.0320612561],[2,79782,0.0289336304],[2,79987,0.1630349933],[2,80037,0.0461423751],[2,80083,0.0323987175],[2,80090,0.0106188943],[2,80095,0.0182409244],[2,80100,0.0231144453],[2,80188,0.0128196912],[2,80209,0.0094824683],[2,80256,0.0406162579],[2,80259,0.0190574413],[2,80429,0.0182431373],[2,80461,0.0120450652],[2,80681,0.0152951321],[2,80707,0.1350005176],[2,80720,0.4980404444],[2,80815,0.0241484194],[2,80816,0.0130025793],[2,80833,0.0136707589],[2,80835,0.029867447],[2,80838,0.032945509],[2,80881,0.0262492476],[2,80903,0.0458692969],[2,80970,0.044003719],[2,80971,0.0184675415],[2,80981,0.0175375734],[2,80996,0.0262492476],[2,81106,0.0128196912],[2,81128,0.2503178607],[2,81147,0.079588357],[2,81180,0.0289336304],[2,81267,0.0262492476],[2,81417,0.013853861],[2,81422,0.0352860722],[2,81683,0.0388049038],[2,81725,0.0173904387],[2,81785,0.0537242161],[2,81843,0.4148244234],[2,81869,0.0150763716],[2,81909,0.0115462273],[2,81933,0.5046284809],[2,81993,0.2323867948],[2,82076,0.030620583],[2,82232,0.0147853307],[2,82313,0.0160271574],[2,82328,0.0162108519],[2,82538,0.2464307002],[2,82644,0.1524115307],[2,82659,0.0648521255],[2,82712,0.1779354844],[2,82842,0.0141855499],[2,82893,0.0162108519],[2,82988,0.0170658873],[2,83093,0.0484512404],[2,83095,0.0177451586],[2,83101,0.0160271574],[2,83105,0.6529158086],[2,83135,0.1207074512],[2,83139,0.0410428159],[2,83276,0.0289034254],[2,83329,0.1760282805],[2,83383,0.0500196398],[2,83680,0.0464346012],[2,83705,0.0322757418],[2,83717,0.0273328249],[2,83729,0.0180114222],[2,83744,0.0220866056],[2,83784,0.030939665],[2,83834,0.0470464326],[2,83903,0.0854396773],[2,83968,0.0179265858],[2,83978,0.0270643365],[2,83985,0.0179265858],[2,83997,0.4908850009],[2,84059,0.0236673378],[2,84107,0.2585571837],[2,84153,0.1350551787],[2,84170,0.0179265858],[2,84171,0.1862650417],[2,84172,0.0155267688],[2,84181,0.0397235772],[2,84270,0.0297130027],[2,84272,0.2744163584],[2,84495,0.0489745033],[2,84553,0.01774246],[2,84630,0.0387303176],[2,84637,0.0295159315],[2,84713,0.9107614488],[2,84733,0.0461948931],[2,84764,0.07235527],[2,84886,0.0272853296],[2,84996,0.0698142894],[2,85014,0.020180194],[2,85083,0.277030283],[2,85155,0.4394036387],[2,85157,0.3665692166],[2,85217,0.0414752998],[2,85220,0.0295159315],[2,85464,0.1801728751],[2,85513,0.0578567634],[2,85527,0.0263164568],[2,85541,0.0205524321],[2,85567,0.0270592623],[2,85674,0.0406811545],[2,85987,0.0251046977],[2,86002,0.1899928137],[2,86007,0.032849558],[2,86019,0.0440005777],[2,86020,0.0339110462],[2,86022,0.0320185883],[2,86030,0.7916243613],[2,86032,0.3044848103],[2,86033,0.1751141121],[2,86037,0.0440005777],[2,86052,0.1693612118],[2,86054,0.0178989999],[2,86055,0.0338176173],[2,86071,0.0255095653],[2,86074,0.0410752747],[2,86079,0.0440005777],[2,86081,0.0155081795],[2,86139,0.1731099602],[2,86271,0.4473527452],[2,86299,0.2542394288],[2,86409,0.0153246617],[2,86490,0.334308599],[2,86594,0.0119759461],[2,86634,0.0867842291],[2,86645,0.3329973864],[2,86646,0.0780905333],[2,86706,0.0320023491],[2,86767,0.1154226084],[2,86800,0.4221868154],[2,86801,0.0160984549],[2,86804,0.5004820599],[2,86811,0.0289679608],[2,86850,0.4214813068],[2,86895,0.0356258696],[2,86950,0.1621874913],[2,87111,0.0464535904],[2,87124,0.0650308109],[2,87154,0.0802554663],[2,87181,0.231593914],[2,87209,0.1224873509],[2,87212,0.1181717731],[2,87215,0.0260935809],[2,87315,0.1583342606],[2,87359,0.0153246617],[2,87404,0.0963323204],[2,87440,0.0153246617],[2,87467,0.0155081795],[2,87498,0.0119759461],[2,87596,0.0168858639],[2,87620,0.0376124528],[2,87755,0.0299873467],[2,87773,0.0145473681],[2,87887,0.0153246617],[2,87901,0.0371613391],[2,88005,0.0270592623],[2,88040,0.0153246617],[2,88217,0.0155081795],[2,88252,0.0266675475],[2,88378,0.5651959286],[2,88515,0.17238994],[2,88516,0.0228392183],[2,88565,0.0270592623],[2,88602,0.055124326],[2,88723,0.0301278498],[2,88796,0.0354467139],[2,88810,0.0543325382],[2,88860,0.0152169664],[2,88869,0.3058804578],[2,88935,0.385032652],[2,88961,0.6691627339],[2,89002,0.1263669135],[2,89045,0.0327110499],[2,89066,0.0154004572],[2,89096,0.0129333691],[2,89122,0.0155081795],[2,89124,0.0182409061],[2,89145,0.1335505349],[2,89203,0.0949694874],[2,89208,0.0214531403],[2,89212,0.628655558],[2,89254,0.044435438],[2,89276,0.0324026533],[2,89354,0.02766],[2,89394,0.0516648091],[2,89397,0.0440598344],[2,89422,0.075286134],[2,89649,0.0181422481],[2,89729,0.0433511076],[2,89799,0.0621200115],[2,89801,0.0234363253],[2,89915,0.0183290595],[2,89943,0.1414922239],[2,89950,0.038004141],[2,90002,0.0488030388],[2,90032,0.0300473845],[2,90065,0.0151632513],[2,90080,0.0267088769],[2,90108,0.0176922851],[2,90198,0.0281083261],[2,90241,0.2734648833],[2,90246,0.0490099563],[2,90275,0.0477065249],[2,90282,0.0149798202],[2,90301,0.3591600301],[2,90434,0.1105929617],[2,90506,0.0267088769],[2,90509,0.0707332216],[2,90512,0.0151632513],[2,90584,0.0127769542],[2,90595,0.1879604777],[2,90607,0.0269544263],[2,90637,0.0350098824],[2,90640,0.0162279652],[2,90890,0.023021089],[2,90935,0.0339512277],[2,90972,0.0401083082],[2,91005,0.0162279652],[2,91013,0.0162279652],[2,91131,0.1392667925],[2,91234,0.0164588696],[2,91310,0.0152119594],[2,91349,0.3930656326],[2,93153,0.4547176149],[2,96874,0.0154503926],[2,96903,0.1069703785],[2,96907,0.1379165282],[2,97098,0.0410717798],[2,97164,0.139133163],[2,97239,0.4212815513],[2,97539,0.0336425807],[2,97683,0.0155517584],[2,97690,0.0156365177],[2,97694,0.1140060187],[2,97787,0.0282676133],[2,97829,0.4638296163],[2,97857,0.0871784387],[2,97961,0.0290166145],[2,98041,0.1596678247],[2,98166,0.3693713091],[2,98272,0.1394096859],[2,98377,0.8004678934],[2,98410,0.2900840637],[2,98807,0.1219088265],[2,98972,0.2175676858],[2,98977,0.0606658941],[2,99035,0.1696652757],[2,99080,0.0817667209],[2,99156,0.0366932785],[2,99242,0.1368859186],[2,99454,0.056806736],[2,99532,0.0175578113],[2,99546,0.0282676133],[2,99606,0.0152668893],[2,99669,0.0345313907],[2,99737,0.0614903283],[2,99790,0.0289675992],[2,99791,0.0544512338],[2,99800,0.1516053931],[2,99818,0.1263378672],[2,99828,0.0848915558],[2,99881,0.1983102885],[2,99890,0.1691293823],[2,99981,0.0168807106],[2,99996,0.0262977325],[2,100033,0.062981855],[2,100038,0.0455390886],[2,100097,0.1723582745],[2,100132,0.0358779673],[2,100135,0.0168807106],[2,100151,0.1378786004],[2,100172,0.0927516915],[2,100223,0.0322349352],[2,100294,0.0344340153],[2,100406,0.0268511571],[2,100517,0.1858612226],[2,100611,0.029576759],[2,100742,0.4018206747],[2,100764,0.0972209683],[2,100813,0.0797435547],[2,100838,0.0168807106],[2,100864,0.0484148496],[2,100963,0.0286667971],[2,100982,0.0278497092],[2,101038,0.3115454241],[2,101043,0.2226596342],[2,101212,0.0607337364],[2,101253,0.0286403278],[2,101307,0.1362561097],[2,101433,0.0156330808],[2,101479,0.1904774175],[2,101555,0.035299869],[2,101558,0.777345455],[2,101563,0.7146966109],[2,101600,0.3237067601],[2,101621,0.4256919482],[2,101713,0.0605631112],[2,101719,0.0576614172],[2,101721,0.1364281519],[2,101736,0.1987105721],[2,101751,0.038456469],[2,101761,0.0319190003],[2,101843,0.0303265552],[2,101895,0.0804600639],[2,102015,0.0349468973],[2,102091,0.0481757468],[2,102141,0.0287375308],[2,102146,0.0214732302],[2,102237,0.0853006633],[2,102259,0.0157285826],[2,102269,0.0750455183],[2,102395,0.1525400382],[2,102528,0.4000729438],[2,102641,0.0326206615],[2,102717,0.1211140957],[2,102730,0.0552236375],[2,102955,0.0424634911],[2,102990,0.0434395247],[2,103089,0.0552031284],[2,103239,0.2389628185],[2,103269,0.0390645149],[2,103421,0.1456787363],[2,103436,0.0248112165],[2,103442,0.5617213555],[2,103461,0.0373741108],[2,103479,0.0293614933],[2,103538,0.0240443135],[2,103830,0.0199568254],[2,103845,0.0328682218],[2,103899,0.0460031914],[2,103961,0.0294568663],[2,104098,0.030515691],[2,104143,0.017554853],[2,104157,0.1049250904],[2,104177,0.0197721893],[2,104180,0.0369758764],[2,104187,0.4598461241],[2,104235,0.0334945748],[2,104344,0.0521604242],[2,104396,0.0367449902],[2,104475,0.8781035471],[2,104493,0.3914474655],[2,104580,0.2662305357],[2,104653,0.1797966719],[2,104809,0.0340161712],[2,104847,0.0394243307],[2,104853,0.1445193682],[2,104857,0.2101514675],[2,104888,0.027071302],[2,104900,0.0850145663],[2,105019,0.0200734668],[2,105083,0.2557868093],[2,105137,0.0569484288],[2,105197,0.0220292474],[2,105207,0.0350369723],[2,105347,0.0264884653],[2,105388,0.1287088132],[2,105395,0.7479462808],[2,105430,0.1238340745],[2,105436,0.3967669691],[2,105470,0.0347709686],[2,105765,0.1469207368],[2,105831,0.2281766913],[2,106059,0.0293895455],[2,106266,0.0342641412],[2,106434,0.0213286834],[2,106479,0.693552842],[2,106543,0.0627641912],[2,106565,0.0191066473],[2,106575,0.0339548114],[2,106818,0.1811630625],[2,107030,0.0535924275],[2,107250,0.1915032833],[2,107261,0.2950486675],[2,107262,0.4138512363],[2,107278,0.0217013818],[2,107299,0.0535924275],[2,107302,0.0535924275],[2,107362,0.0535924275],[2,107523,0.0280355778],[2,107616,0.0293894298],[2,107918,0.0535924275],[2,107934,0.2026297513],[2,107956,0.0198888014],[2,107972,0.046675633],[2,107976,0.0352016811],[2,108025,0.0340161712],[2,108039,0.0213286834],[2,108040,0.0335712315],[2,108041,0.3545903595],[2,108062,0.4810062704],[2,108078,0.1381413183],[2,108096,0.0213286834],[2,108099,0.0483216012],[2,108140,0.0982194401],[2,108147,0.0340161712],[2,108176,0.0211437024],[2,108238,0.0239599901],[2,108247,0.0319204608],[2,108297,0.0211437024],[2,108379,0.0394581508],[2,108407,0.0872081252],[2,108455,0.030771789],[2,108487,0.0834561462],[2,108550,0.0575305905],[2,108793,0.0381111397],[2,108871,0.0381111397],[2,108949,0.0493528739],[2,109009,0.1083870633],[2,109681,0.326893579],[2,109825,0.0282578348],[2,109925,0.0330630983],[2,109966,0.0411938809],[2,109987,0.0253542486],[2,109992,0.0552252412],[2,110091,0.0385497759],[2,110110,0.0740553936],[2,110147,0.0652472958],[2,110392,0.0424354996],[2,110466,0.0440152041],[2,110634,0.0379883127],[2,110644,0.0909705709],[2,110653,0.0258429861],[2,110768,0.159587411],[2,110920,0.0278773145],[2,111011,0.0234217656],[2,111126,0.2737837263],[2,111277,0.0293972533],[2,111278,0.0460209397],[2,111350,0.2515066902],[2,111353,0.0386083033],[2,111411,0.0557337089],[2,111469,0.0828913596],[2,111516,0.0386083033],[2,111573,0.0930982393],[2,111578,0.0586959732],[2,111604,0.0442132695],[2,111744,0.0593706479],[2,111880,0.4211466012],[2,112003,0.3357972951],[2,112023,0.0376111225],[2,112140,0.0358574517],[2,112242,0.4089763418],[2,112508,0.1694769206],[2,112585,0.1301885739],[2,112639,0.0388941038],[2,112669,0.2954668044],[2,112814,0.0387247539],[2,112823,0.2218988852],[2,113005,0.0349971071],[2,113325,0.1273015015],[2,113326,0.7095501654],[2,113331,0.036397899],[2,113357,0.0413593561],[2,113401,0.0234217656],[2,113629,0.6104755146],[2,113750,0.0256568704],[2,113990,0.3019506795],[2,114017,0.0987347783],[2,114095,0.0439017703],[2,114143,0.0624403758],[2,114163,0.0382823172],[2,114174,0.3005219837],[2,114329,0.0517047404],[2,114423,0.0567365593],[2,114458,0.0323624328],[2,114501,0.3085048944],[2,114517,0.0386083033],[2,114563,0.0670577],[2,114584,0.0483635743],[2,114723,0.1223137961],[2,114764,0.1571622424],[2,114774,0.0251159005],[2,114882,0.4171708155],[2,114899,0.0683315083],[2,114908,0.0871756937],[2,114951,0.0264657489],[2,114953,0.133000651],[2,115030,0.0271482312],[2,115120,0.0238542242],[2,115124,0.0342089317],[2,115142,0.906010061],[2,115156,0.0401386941],[2,115202,0.038521589],[2,115275,0.1327907338],[2,115380,0.0734104409],[2,115458,0.0920166335],[2,115714,0.0284715964],[2,116087,0.7210989807],[2,116128,0.0332001713],[2,116280,0.0240398866],[2,116369,0.056185102],[2,116421,0.1206448475],[2,116434,0.1707148071],[2,116624,0.1444670094],[2,116639,0.0332246522],[2,116644,0.0496902333],[2,116727,0.4644342665],[2,116764,0.06624901],[2,116792,0.0612433192],[2,116874,0.3853674623],[2,116876,0.1066979602],[2,116887,0.0370121725],[2,116959,0.1222427662],[2,116961,0.1201919125],[2,116999,0.0845109216],[2,117012,0.0459150075],[2,117050,0.8835935553],[2,117052,0.1852859009],[2,117270,0.1504902576],[2,117606,0.0369375555],[2,117897,0.1547483021],[2,118092,0.0380008826],[2,118122,0.1347453084],[2,118192,0.0359485424],[2,118198,0.6414441062],[2,118264,0.052419581],[2,118309,0.1419029696],[2,118347,0.0876262246],[2,118377,0.0414840369],[2,118399,0.1422067166],[2,118419,0.4023408556],[2,118506,0.0601231143],[2,118507,0.0414952961],[2,118546,0.0513928561],[2,121262,0.075557929],[2,122515,0.7817806678],[2,123979,0.0987930344],[2,124001,0.0472070399],[2,124861,0.2624526337],[2,124883,0.0759061611],[2,124953,0.0445279141],[2,125023,0.0477880945],[2,125043,0.0447187745],[2,125071,0.0438022436],[2,125120,0.0447187745],[2,125169,0.0494681605],[2,125306,0.0885796984],[2,125328,0.0478432436],[2,125331,0.1233579933],[2,125341,0.1396347258],[2,125480,0.0449123617],[2,125658,0.0827864974],[2,125666,0.2767684044],[2,125700,0.0445279141],[2,125728,0.3541262621],[2,125731,0.4003419304],[2,125746,0.1055352655],[2,125804,0.083974401],[2,125835,0.0760398252],[2,126002,0.4881907877],[2,126077,0.0753715116],[2,126101,0.345119373],[2,126122,0.3543883905],[2,138916,0.22277384],[2,140057,0.2651148808],[2,140841,0.1590248021],[2,140870,0.1350752949],[2,140898,0.0670810833],[2,141030,0.0529248339],[2,141053,0.1168504883],[2,141066,0.0461237602],[2,141114,0.0640713856],[2,141118,0.0484276203],[2,141362,0.0649142975],[2,141578,0.0826486478],[2,141722,0.2307043792],[2,141732,0.0688882048],[2,141761,0.0531122997],[2,141793,0.0450403859],[2,141806,0.161737208],[2,141817,0.0465408126],[2,141877,0.0426655815],[2,141909,0.0826550368],[2,141918,0.2158112679],[2,141949,0.1280409454],[2,142167,0.061061548],[2,142181,0.061323397],[2,142524,0.0485625654],[2,142572,0.0525565133],[2,142609,0.3995355559],[2,142610,0.0492176267],[2,142625,0.0642506825],[2,142839,0.0505982151],[2,142854,0.0637928928],[2,142927,0.8205636234],[2,155014,0.0449452576],[2,155319,0.0423446657],[2,156904,0.0474024368],[2,157089,0.0342237385],[2,157961,0.0362718895],[2,157982,0.0342237385],[2,157995,0.1871712684],[2,158372,0.0630135975],[2,158466,0.0960246418],[2,159393,0.5105214834],[2,159884,0.0406831937],[2,159885,0.1423815291],[2,160042,0.2320826718],[2,160077,0.1852604942],[2,160130,0.0365443423],[2,160186,0.3111603204],[2,160233,0.066928973],[2,160302,0.0669060606],[2,160324,0.0561281842],[2,160474,0.1655302306],[2,160495,0.0323752351],[2,160606,0.0335475918],[2,160626,0.0637968416],[2,161072,0.0432519142],[2,161080,0.0383921887],[2,161488,0.0358487815],[2,162112,0.0518634922],[2,162201,0.074602344],[2,162237,0.1786372338],[2,162246,0.0358487815],[2,162262,0.2321993333],[2,162263,0.2393695105],[2,162265,0.1692711006],[2,162267,0.2081731919],[2,162268,0.2026727931],[2,162274,0.2251333796],[2,162278,0.2294017581],[2,162286,0.3480684004],[2,162289,0.0884794049],[2,162294,0.2437315833],[2,162303,0.2202611209],[2,162322,0.2221117426],[2,162324,0.0379220292],[2,162341,0.3393630519],[2,162458,0.0423890179],[2,162552,0.0527234971],[2,162678,0.0360374597],[2,162740,0.0334998785],[2,162880,0.2430885843],[2,163032,0.3877719269],[2,163232,0.4966026531],[2,163288,0.0330764569],[2,163387,0.0431823205],[2,163416,0.0774359356],[2,163443,0.0423890179],[2,163493,0.0501877412],[2,163505,0.0729209887],[2,163528,0.062412165],[2,163955,0.1139704039],[2,164935,0.5241022449],[2,164988,0.0360374597],[2,165078,0.1346473789],[2,165404,0.3337498471],[2,166521,0.0398072325],[2,167733,0.0412323639],[2,167805,0.0473847301],[2,167827,0.0492095982],[2,168015,0.0482342471],[2,168163,0.0667765766],[2,168178,0.0847895172],[2,168498,0.0823527029],[2,168502,0.2125629309],[2,168549,0.0399969059],[2,168561,0.1377109715],[2,168607,0.2365494819],[2,168616,0.1382139378],[2,168776,0.048363858],[2,168950,0.1173267394],[2,168962,0.284439361],[2,169196,0.0469828868],[2,169199,0.0381813214],[2,169315,0.1624449904],[2,169327,0.1754787896],[2,169414,0.0546302884],[2,169427,0.3827573169],[2,169482,0.0473847301],[2,169592,0.0808294762],[2,169700,0.0861159363],[2,169746,0.0520943486],[2,169762,0.0433009935],[2,170197,0.1836247786],[2,170238,0.0641241281],[2,170239,0.049017609],[2,170258,0.0748770771],[2,170263,0.0603841389],[2,170349,0.0980620512],[2,170401,0.1967620556],[2,170402,0.0545020332],[2,170418,0.2944670224],[2,170435,0.1466575169],[2,170559,0.217610908],[2,170577,0.0402474432],[2,170629,0.0374459392],[2,170915,0.0554754312],[2,170957,0.0384369254],[2,170973,0.0683978743],[2,170979,0.0885817083],[2,171073,0.090065334],[2,171074,0.0420220282],[2,171113,0.0375538503],[2,171217,0.0398858146],[2,171385,0.4752192388],[2,171495,0.595797668],[2,171505,0.0393492828],[2,171524,0.1418829478],[2,171577,0.6721653531],[2,171994,0.1298251746],[2,172025,0.0441729038],[2,172056,0.1508111378],[2,172066,0.0758098013],[2,172067,0.039089231],[2,172122,0.1972457866],[2,172248,0.2402132371],[2,172386,0.0439195334],[2,172436,0.0559459592],[2,174634,0.0414556841],[2,174954,0.805105799],[2,175021,0.0392787239],[2,175112,0.2850034359],[2,175326,0.0414556841],[2,175377,0.0386626457],[2,175427,0.0423913398],[2,175434,0.0644010047],[2,175576,0.0889609332],[2,175578,0.1254862552],[2,175586,0.0380973862],[2,175636,0.1368179448],[2,175742,0.089623656],[2,175848,0.0414686824],[2,176048,0.0685098007],[2,176110,0.0431074685],[2,176209,0.0421932125],[2,176324,0.1070030514],[2,176427,0.085895394],[2,176674,0.0568338748],[2,176708,0.0402996163],[2,176750,0.0756062141],[2,176756,0.041673313],[2,177081,0.3559417716],[2,177090,0.2550948248],[2,177222,0.2120962202],[2,177271,0.0549980425],[2,177354,0.5459751277],[2,177431,0.0432985456],[2,177476,0.0451804627],[2,177482,0.0692047974],[2,177552,0.0404800324],[2,177620,0.0402370464],[2,177633,0.050981191],[2,177696,0.0406312206],[2,177768,0.0397424045],[2,178014,0.0465699051],[2,178101,0.0428214923],[2,178144,0.1022124606],[2,178146,0.1446332372],[2,178391,0.1216879639],[2,178527,0.0399042191],[2,178550,0.0762330506],[2,178617,0.0670720227],[2,178844,0.0423350679],[2,178849,0.0457122735],[2,179001,0.0786986394],[2,179022,0.2408565921],[2,179044,0.0456063841],[2,179258,0.0998569667],[2,179282,0.2623569506],[2,179319,0.1795652091],[2,179438,0.0754663566],[2,179502,0.0607219747],[2,179520,0.3987523292],[2,179718,0.048836049],[2,179759,0.2281992368],[2,179777,0.0600600896],[2,180050,0.0416006562],[2,180058,0.0484652438],[2,180634,0.0499491002],[2,180751,0.0434975159],[2,180797,0.0563495782],[2,180973,0.7111187691],[2,181249,0.2221904021],[2,181339,0.1984293164],[2,181560,0.0481553861],[2,181677,0.17744281],[2,181915,0.5376353613],[2,181955,0.0741032943],[2,181971,0.0411243296],[2,181991,0.7833610017],[2,182117,0.4936470541],[2,182131,0.0506709745],[2,182255,0.0441317558],[2,182508,0.0731202247],[2,182516,0.0775422816],[2,182581,0.0411243296],[2,182609,0.1544152515],[2,182618,0.0746523773],[2,182675,0.0393025894],[2,182676,0.9271102908],[2,182699,0.5943928523],[2,182772,0.0411243296],[2,182935,0.0485165211],[2,187531,0.0378630435],[2,187797,0.0492098252],[2,187798,0.4131289563],[2,187923,0.0509370942],[2,187930,0.0578573077],[2,187949,0.1582855256],[2,188127,0.13454713],[2,188149,0.4536068603],[2,188717,0.3834583835],[2,188804,0.2223335473],[2,189026,0.0922980374],[2,189042,0.0404154251],[2,189063,0.047671958],[2,189302,0.1705899625],[2,189357,0.1524762027],[2,189367,0.1512450007],[2,189371,0.0402256465],[2,189415,0.188643116],[2,189508,0.039716466],[2,189872,0.0715814816],[2,189902,0.0489520823],[2,189913,0.1632548853],[2,189935,0.2714349103],[2,189963,0.2822937064],[2,189979,0.6024983548],[2,190081,0.5380164042],[2,190092,0.0404154251],[2,190093,0.1455353559],[2,190134,0.0468040289],[2,190208,0.1381976955],[2,190276,0.1009686756],[2,190326,0.0971204222],[2,190625,0.0679662877],[2,190670,0.6190629993],[2,190687,0.1635058562],[2,190694,0.0960648982],[2,190715,0.1149686637],[2,190719,0.2763220148],[2,190739,0.0505935447],[2,191003,0.0468040289],[2,191080,0.3001092],[2,191208,0.1112404936],[2,191210,0.0855451928],[2,191377,0.6790512709],[2,191410,0.0465153197],[2,191434,0.8482151379],[2,191507,0.0379015562],[2,191528,0.063537009],[2,191539,0.0732235734],[2,191541,0.047671958],[2,191554,0.1307069966],[2,191571,0.0404223176],[2,191681,0.0468040289],[2,191813,0.0397994416],[2,191816,0.127389127],[2,191902,0.0452811948],[2,191987,0.0426288931],[2,192071,0.0412774999],[2,192097,0.3778826919],[2,192103,0.0510632423],[2,192130,0.041030272],[2,192167,0.0602531192],[2,192200,0.0468040289],[2,192224,0.0412651531],[2,192353,0.2576999135],[2,192428,0.0389116045],[2,192486,0.344887548],[2,192609,0.3443255364],[2,192666,0.0493008418],[2,192681,0.0381150166],[2,192715,0.0383968369],[2,192800,0.1982681091],[2,192859,0.0697197508],[2,192996,0.1327307954],[2,193041,0.0359860904],[2,193158,0.0530846202],[2,193220,0.0374059034],[2,193268,0.0543728335],[2,193277,0.0383042646],[2,193280,0.0576306041],[2,193308,0.0371667659],[2,193384,0.0383125268],[2,193410,0.0383042646],[2,193577,0.1652064928],[2,193682,0.1594122799],[2,193710,0.0434887987],[2,193864,0.0613541591],[2,193984,0.2516224837],[2,194100,0.0447861929],[2,194155,0.0437799958],[2,194180,0.0415027722],[2,194222,0.0940854585],[2,194242,0.0455119834],[2,194342,0.0375949731],[2,194417,0.1410589205],[2,194419,0.1024055523],[2,194472,0.0444676521],[2,194488,0.0378570038],[2,194511,0.0437799958],[2,194932,0.3447186816],[2,195012,0.0437812045],[2,195027,0.2789249617],[2,195034,0.0506426795],[2,195041,0.0335957461],[2,195097,0.0497745897],[2,195165,0.0670412692],[2,195223,0.056276438],[2,195244,0.0415548333],[2,195339,0.069340313],[2,195388,0.0483352524],[2,195507,0.0870687746],[2,195580,0.453501308],[2,195584,0.0973237266],[2,195633,0.0961162156],[2,195790,0.34162977],[2,195888,0.0381986285],[2,195890,0.0472364879],[2,195955,0.0381986285],[2,196058,0.0429164983],[2,196074,0.8176883379],[2,196095,0.1311710581],[2,196126,0.0530020203],[2,196142,0.1604339313],[2,196184,0.0376568659],[2,196265,0.0856753845],[2,196400,0.039181489],[2,196452,0.0712796409],[2,196514,0.0663858146],[2,196522,0.0358807796],[2,196619,0.0429164983],[2,196658,0.2179174329],[2,196830,0.0352077981],[2,196877,0.127129443],[2,197011,0.275132671],[2,197032,0.0536687166],[2,197034,0.0373003934],[2,197214,0.127538985],[2,197234,0.0407604376],[2,197494,0.0695693962],[2,197495,0.0453467196],[2,197544,0.0498832575],[2,197603,0.0447861579],[2,197707,0.0420000525],[2,197747,0.0429164983],[2,197748,0.0425912251],[2,197863,0.041339498],[2,197890,0.3678583836],[2,197913,0.0381986285],[2,197962,0.065358242],[2,197992,0.0445398018],[2,197996,0.0793507319],[2,198076,0.0429164983],[2,198078,0.124965853],[2,198120,0.0622436889],[2,198136,0.0470667476],[2,198196,0.1722589517],[2,198220,0.4939302738],[2,198235,0.0459099662],[2,198238,0.3523649096],[2,198457,0.177961737],[2,198476,0.038418061],[2,198493,0.3073432746],[2,198563,0.0349591902],[2,198615,0.375079913],[2,198621,0.203758171],[2,198638,0.0892240734],[2,198642,0.0393710052],[2,198667,0.1592893373],[2,198697,0.0355610485],[2,198713,0.0415560008],[2,198749,0.0373126754],[2,199197,0.0404467149],[2,199219,0.3227139749],[2,199262,0.2133233563],[2,199286,0.6248217262],[2,199302,0.052120644],[2,199332,0.1032797101],[2,199336,0.1324768236],[2,199341,0.4987039976],[2,199373,0.0416344042],[2,199699,0.0357510775],[2,199759,0.0404467149],[2,199979,0.661873416],[2,199981,0.0348611631],[2,200021,0.0510598317],[2,200087,0.0420647257],[2,200150,0.0355556231],[2,200266,0.0404467149],[2,200321,0.0334378341],[2,200455,0.0754830728],[2,200477,0.0404467149],[2,200776,0.0348611631],[2,200818,0.3356755858],[2,200837,0.0459020737],[2,200873,0.0872847116],[2,200984,0.1366935303],[2,201018,0.1007898929],[2,201052,0.0802325077],[2,201071,0.0360346851],[2,201251,0.0824968514],[2,201272,0.033687928],[2,201286,0.1076712645],[2,201345,0.0471405352],[2,201414,0.1094732091],[2,201418,0.0763980665],[2,201481,0.1994670546],[2,201555,0.3411435055],[2,201617,0.2730240261],[2,201651,0.0597962925],[2,201778,0.0325153514],[2,201791,0.088134374],[2,201813,0.0357442276],[2,201852,0.0358487433],[2,201880,0.0558495729],[2,201897,0.0518090307],[2,201901,0.1462666721],[2,201924,0.0662541128],[2,201929,0.0360374215],[2,201961,0.0936687047],[2,201964,0.0337817186],[2,202000,0.041307397],[2,202619,0.0904389478],[2,202711,0.224270073],[2,202719,0.0457791506],[2,202760,0.0365627095],[2,202896,0.1593258823],[2,202911,0.0675177868],[2,203100,0.0461160924],[2,203195,0.0653685478],[2,203252,0.0905367295],[2,203291,0.1126599377],[2,203443,0.1562347853],[2,203457,0.0420168947],[2,203573,0.1089304343],[2,203726,0.0362981526],[2,203825,0.042996345],[2,203892,0.1580899462],[2,203930,0.0760098022],[2,203937,0.0645701574],[2,203966,0.0367419726],[2,204098,0.0403691707],[2,204169,0.0535638255],[2,204199,0.5443732885],[2,204211,0.3789360272],[2,204247,0.0443309586],[2,204411,0.0365531173],[2,204425,0.0570241777],[2,204502,0.0350749727],[2,204780,0.1130971449],[2,204875,0.042930313],[2,204982,0.0364485035],[2,205074,0.0789463975],[2,205159,0.0420168947],[2,205331,0.0504597198],[2,205424,0.0427749343],[2,205662,0.7381350719],[2,205671,0.4144670337],[2,205677,0.2660847656],[2,205686,0.0364485035],[2,205692,0.0472942781],[2,205790,0.0693487116],[2,205820,0.0332165966],[2,205899,0.0356253213],[2,205978,0.0418267135],[2,206004,0.0411554047],[2,206053,0.3299943618],[2,206269,0.0501926151],[2,206323,0.0411554047],[2,206402,0.0346380351],[2,206471,0.0364485035],[2,206513,0.1302726308],[2,206685,0.0367392336],[2,206691,0.0420168947],[2,206700,0.0485867224],[2,206709,0.2664848861],[2,206842,0.2664760734],[2,206843,0.0478320603],[2,206930,0.1929684078],[2,206955,0.2456046082],[2,206959,0.0434922142],[2,207065,0.0432875219],[2,207155,0.1662772671],[2,207188,0.1380331537],[2,207192,0.7683777365],[2,207227,0.2144937436],[2,207245,0.0980407666],[2,207379,0.0344986814],[2,207428,0.3620512057],[2,207447,0.0370465124],[2,207480,0.0424220939],[2,207638,0.2555455625],[2,207682,0.0424220939],[2,207690,0.8174386076],[2,207776,0.0419280902],[2,208168,0.0415665889],[2,208172,0.0350559238],[2,208269,0.2977010323],[2,208324,0.0342265529],[2,208342,0.0446318905],[2,208343,0.0355431088],[2,208369,0.040895511],[2,208394,0.0549884523],[2,208412,0.3212355986],[2,208451,0.0385558439],[2,208454,0.0372485529],[2,208523,0.053805099],[2,208525,0.2481031313],[2,208546,0.2274329102],[2,208611,0.1939765309],[2,208633,0.0533989649],[2,208735,0.0382894179],[2,208774,0.0412519521],[2,208838,0.0838255524],[2,208889,0.0417567047],[2,208928,0.0509005403],[2,208986,0.040895511],[2,209058,0.0361902285],[2,209176,0.0336521308],[2,209188,0.0672488272],[2,209229,0.0375484569],[2,209244,0.0606848701],[2,209290,0.5520802937],[2,209337,0.1401209797],[2,209353,0.0796155389],[2,209385,0.0592164354],[2,209386,0.0607722443],[2,209397,0.0430268949],[2,209584,0.0386093995],[2,209612,0.2351055112],[2,209641,0.0660852257],[2,209858,0.0361296012],[2,209975,1.0],[2,209981,0.750240254],[2,210035,0.038308107],[2,210128,0.2450488485],[2,210547,0.0350728519],[2,210726,0.047285692],[2,210735,0.2475292929],[2,210766,0.0395132815],[2,210771,0.0415117623],[2,210823,0.0366094641],[2,210879,0.0294925104],[2,210971,0.0926285792],[2,210989,0.0350629663],[2,211032,0.095333401],[2,211034,0.0327083221],[2,211157,0.0793104481],[2,211225,0.0639538307],[2,211318,0.0590487531],[2,211333,0.0328124149],[2,211359,0.0639538307],[2,211413,0.5889309323],[2,211472,0.2020346775],[2,211489,0.1332583794],[2,211521,0.0387459478],[2,211526,0.0327083221],[2,211575,0.0351946291],[2,211578,0.658440317],[2,211594,0.0423870145],[2,211769,0.036839809],[2,211770,0.0325204807],[2,211773,0.1778866383],[2,211876,0.1388265481],[2,211912,0.0397028811],[2,212077,0.0915882358],[2,212080,0.0617871104],[2,212087,0.0325138009],[2,212288,0.0458035359],[2,212291,0.0546589433],[2,212314,0.1387602313],[2,212429,0.0398906186],[2,212468,0.1158651337],[2,212625,0.1538363685],[2,212626,0.0770298183],[2,212739,0.2666186114],[2,212779,0.1955191751],[2,213450,0.7857240264],[2,213564,0.0363259626],[2,213566,0.2504501394],[2,213633,0.0404158673],[2,213658,0.0362213805],[2,213855,0.0404158673],[2,214038,0.1027337727],[2,214063,0.0668401849],[2,214204,0.0722278305],[2,214285,0.0365230034],[2,214323,0.086164654],[2,214544,0.3890360161],[2,214557,0.0341637732],[2,214586,0.033975566],[2,214638,0.0336831775],[2,214662,0.0988627998],[2,214668,0.2332275702],[2,214680,0.1583668756],[2,214695,0.0474040129],[2,214729,0.0716981199],[2,214911,0.3534351727],[2,214940,0.0439433448],[2,214962,0.0360326561],[2,215008,0.2200722112],[2,215158,0.0360326561],[2,215321,0.066752452],[2,215409,0.0579766595],[2,215476,0.0360326561],[2,215731,0.1052725819],[2,215742,0.0336831775],[2,219704,0.0358277611],[2,220354,0.0380791325],[2,220563,0.0364931605],[2,220826,0.0669264247],[2,220979,0.081327213],[2,220996,0.0452230079],[2,221323,0.0500529006],[2,221475,0.2881456712],[2,221516,0.2459551184],[2,221887,0.0375723207],[2,221980,0.4200357676],[2,222017,0.5416657326],[2,222034,0.0366347249],[2,222219,0.1647309573],[2,222282,0.4399459939],[2,222288,0.0388143178],[2,222423,0.0343304897],[2,222504,0.0649712448],[2,222648,0.0354924798],[2,222675,0.2269915674],[2,222878,0.0526725363],[2,223112,0.0349605644],[2,223184,0.0419753428],[2,223263,0.034734505],[2,223302,0.0595324733],[2,223332,0.1283759956],[2,223511,0.0425885049],[2,223581,0.3071840951],[2,223814,0.1148814289],[2,223862,0.0604325802],[2,224025,0.034967338],[2,224026,0.029882269],[2,224210,0.0307594669],[2,224211,0.0673252668],[2,224363,0.0481338256],[2,224404,0.040969374],[2,224481,0.0673252668],[2,224487,0.0376100558],[2,224712,0.0467535233],[2,224862,0.1049107003],[2,224995,0.1878137994],[2,225229,0.169510808],[2,225359,0.0312263152],[2,225407,0.0414339621],[2,225432,0.0588859653],[2,225549,0.0315586833],[2,225587,0.2904215745],[2,225651,0.0385010764],[2,225710,0.3567197405],[2,225716,0.0411125821],[2,225753,0.0414339621],[2,225775,0.0303841],[2,225988,0.035349363],[2,226049,0.0282443479],[2,226085,0.1083958669],[2,226338,0.0315432087],[2,226407,0.0858989075],[2,226408,0.0393070596],[2,226446,0.0464399498],[2,226449,0.3631920344],[2,226491,0.1726362562],[2,226523,0.1260953799],[2,226527,0.1608823639],[2,226687,0.0364794833],[2,226847,0.0303244482],[2,226879,0.0378578844],[2,226887,0.0894267688],[2,226925,0.0295783444],[2,226932,0.100620336],[2,227068,0.0890064737],[2,227114,0.0387156154],[2,227195,0.0301244026],[2,227219,0.452750129],[2,227328,0.0298335184],[2,227413,0.1056263238],[2,227415,0.0892142294],[2,227525,0.5483732298],[2,227539,0.0888067091],[2,227542,0.2462327529],[2,227547,0.3560499069],[2,227550,0.3998830173],[2,227562,0.0300206842],[2,227614,0.0301372061],[2,227633,0.0279800703],[2,227648,0.0469092736],[2,227726,0.1329195648],[2,227825,0.0277318841],[2,227867,0.0412087889],[2,227935,0.4131418251],[2,227941,0.0396599362],[2,227952,0.0448609652],[2,227994,0.0535054509],[2,227997,0.0463623148],[2,228009,0.0610313122],[2,228022,0.0429987502],[2,228122,0.0698774605],[2,228174,0.0300206842],[2,228219,0.0301372061],[2,228411,0.0394703474],[2,228497,0.8952126857],[2,228656,0.1112934934],[2,228712,0.0295783444],[2,228802,0.2883797121],[2,228814,0.071549181],[2,228908,0.0312335056],[2,228938,0.2207933228],[2,228940,0.0385855413],[2,229103,0.2303285367],[2,229239,0.275459708],[2,229267,0.0354331899],[2,229340,0.0378578844],[2,229346,0.0307988439],[2,229445,0.0365761811],[2,229545,0.6432894119],[2,229596,0.070968246],[2,229628,0.0394703474],[2,229672,0.0336907395],[2,229723,0.0394703474],[2,229742,0.0285250736],[2,229887,0.1763727992],[2,230019,0.1595843985],[2,230112,0.0417222486],[2,230138,0.0408638385],[2,230303,0.1874943635],[2,230308,0.0373064226],[2,230320,0.0629431506],[2,230333,0.6466344368],[2,230453,0.1240300336],[2,230786,0.4455021413],[2,230892,0.0390328138],[2,230922,0.6757475],[2,231050,0.0561061276],[2,231059,0.2822553837],[2,231158,0.0417222486],[2,231160,0.2984670379],[2,231318,0.042278435],[2,231348,0.097859167],[2,231393,0.1946180228],[2,231394,0.3227433266],[2,231443,0.0583915029],[2,231502,0.1050512938],[2,231523,0.3543529412],[2,231686,0.0359693429],[2,231694,0.0340259589],[2,231701,0.034901632],[2,231786,0.0349173767],[2,231854,0.0315724472],[2,231891,0.0481591211],[2,231899,0.0347972482],[2,231904,0.1729255051],[2,232015,0.034901632],[2,232110,0.0322638598],[2,232137,0.720439861],[2,232141,0.0347972482],[2,232154,0.0353292991],[2,232205,0.0557131126],[2,232240,0.1054983938],[2,232241,0.0905461945],[2,232244,0.0698403116],[2,232311,0.0404426421],[2,232328,0.0552723719],[2,232357,0.1095956044],[2,232534,0.0426847237],[2,232594,0.0347972482],[2,232749,0.1140762925],[2,232781,0.0372241348],[2,232861,0.1017579055],[2,232866,0.0902542407],[2,232868,0.0435479571],[2,232936,0.0444983356],[2,232937,0.0387955138],[2,233077,0.0426847237],[2,233114,0.0914252203],[2,233253,0.0345403727],[2,233317,0.0426847237],[2,233369,0.0325556937],[2,233451,0.1351926121],[2,233526,0.0657856657],[2,233605,0.0388980792],[2,233611,0.2717573223],[2,233612,0.1244070537],[2,233653,0.7055853873],[2,233686,0.1331439649],[2,233703,0.1681153746],[2,233705,0.0416323801],[2,233716,0.8669768377],[2,233737,0.0315174261],[2,233900,0.0331863888],[2,234022,0.0416323801],[2,234171,0.033338609],[2,234185,0.0840087059],[2,234209,0.7377457928],[2,234272,0.0333889922],[2,234284,0.6778861279],[2,234534,0.0628507605],[2,234595,0.0424944139],[2,234606,0.0346347855],[2,234770,0.1828542761],[2,235239,0.0861074014],[2,235258,0.0334612037],[2,235275,0.5719246278],[2,235289,0.1937265864],[2,235311,0.26322984],[2,235328,0.0393445282],[2,235379,0.0339661616],[2,235399,0.0893337766],[2,235402,0.1803029488],[2,235491,0.0391149936],[2,235510,0.0415425649],[2,235526,0.1584024049],[2,235626,0.1161212495],[2,235628,0.1244951555],[2,235633,0.6154619141],[2,235641,0.0339661616],[2,235694,0.1362732806],[2,235698,0.0339661616],[2,235742,0.0332731731],[2,235753,0.2040843498],[2,235804,0.036513567],[2,235857,0.7529273035],[2,235938,0.2236696034],[2,235949,0.1588890616],[2,236031,0.3513897664],[2,236052,0.6424636726],[2,236102,0.0684354587],[2,236138,0.1398548902],[2,236213,0.0354762975],[2,236402,0.2311596327],[2,236624,0.0339661616],[2,236640,0.0328016872],[2,236721,0.0842530819],[2,236730,0.1166272035],[2,236748,0.1176386836],[2,236756,0.0428573308],[2,236757,0.0407301506],[2,236813,0.5503039794],[2,236829,0.0335845864],[2,236842,0.0624557456],[2,236894,0.0414313883],[2,237040,0.387034986],[2,237195,0.0438394886],[2,237247,0.0341543665],[2,237287,0.0451920166],[2,237290,0.0332731731],[2,237293,0.0339661616],[2,237310,0.0394742303],[2,237337,0.3326335555],[2,237342,0.0381588362],[2,237394,0.7138529478],[2,237427,0.0472512955],[2,237461,0.2632560733],[2,237487,0.0586161893],[2,237652,0.0325852691],[2,237681,0.0349313003],[2,237786,0.0348269124],[2,237810,0.2129502572],[2,237841,0.1992076137],[2,237904,0.0325852691],[2,237927,0.1486402395],[2,237929,0.0458703418],[2,237938,0.0404620931],[2,237973,0.1387571014],[2,237976,0.051634939],[2,238020,0.3399818471],[2,238022,0.0348269124],[2,238086,0.0325852691],[2,238151,0.4491059094],[2,238336,0.2623336406],[2,238493,0.033411944],[2,238506,0.1910551799],[2,238530,0.5766400041],[2,238656,0.049062776],[2,238659,0.2090951187],[2,238680,0.0582370664],[2,241120,0.0583021093],[2,242994,0.043589824],[2,243736,0.6109879976],[2,243768,0.2298240506],[2,243854,0.0608553453],[2,243908,0.0376888478],[2,244222,0.0569888767],[2,244254,0.1447747235],[2,244343,0.4942152718],[2,244452,0.0578731996],[2,244607,0.0365735275],[2,244622,0.0388271364],[2,245079,0.039919258],[2,245158,0.0389320815],[2,245283,0.0407860219],[2,245514,0.0445603364],[2,245571,0.1434889027],[2,245910,0.1399520031],[2,246211,0.053149706],[2,246333,0.0672259758],[2,246382,0.1606377155],[2,246658,0.0947932567],[2,246766,0.0518542445],[2,246809,0.0411619192],[2,246890,0.1122186761],[2,246916,0.037601459],[2,247043,0.0530882935],[2,247077,0.5835300056],[2,247095,0.0396294688],[2,247302,0.0886531261],[2,247328,0.1071595359],[2,247361,0.0359991037],[2,248346,0.1463460978],[2,249067,0.1029406338],[2,250298,0.6010944077],[2,250300,0.7439953444],[2,252985,0.4488548902],[2,253089,0.0361960587],[2,253211,0.2731634055],[2,253215,0.0644244902],[2,253297,0.3634761187],[2,253308,0.3029018041],[2,254025,0.073586494],[2,254279,0.0377485566],[2,254602,0.1409677721],[2,254992,0.0315647863],[2,255033,0.0381963358],[2,255070,0.0469827292],[2,255311,0.0371282242],[2,255500,0.6828676059],[2,255549,0.0275652798],[2,255554,0.0497934229],[2,255555,0.0470886874],[2,255578,0.0925534619],[2,255940,0.645676505],[2,256001,0.036943502],[2,256165,0.0298666906],[2,256315,0.3414019769],[2,256324,0.037263109],[2,256347,0.0346097278],[2,256614,0.0606516964],[2,256696,0.0305568681],[2,256716,0.0308480349],[2,256832,0.3316065954],[2,256905,0.8835006016],[2,257388,0.371032236],[2,257444,0.0644311218],[2,257467,0.685091134],[2,257800,0.0287812219],[2,257841,0.3419209693],[2,257920,0.2971317527],[2,257959,0.0652082066],[2,258160,0.0631937301],[2,258161,0.1220135911],[2,258176,0.0974934387],[2,258332,0.0301665927],[2,258666,0.0341871989],[2,258989,0.0290716949],[2,259018,0.0390723217],[2,259368,0.5460674268],[2,259441,0.6357192983],[2,259515,0.7936967903],[2,259594,0.2115314291],[2,259721,0.0852703805],[2,259767,0.2626855785],[2,259793,0.030237692],[2,260015,0.0378561084],[2,260221,0.0311153087],[2,260247,0.0342906077],[2,260306,0.0693074476],[2,260339,0.2865054989],[2,260340,0.0428892417],[2,260344,0.5807162979],[2,260386,0.2570208255],[2,260388,0.2972776813],[2,260457,0.3658469119],[2,260550,0.1056014809],[2,260748,0.0386403471],[2,260816,0.0316628025],[2,260880,0.0616071432],[2,261010,0.1887587627],[2,261011,0.0321447343],[2,261078,0.02845149],[2,261098,0.1010306114],[2,261107,0.0358985155],[2,261160,0.0312379248],[2,261176,0.0304636894],[2,261190,0.0317667496],[2,261426,0.2733771378],[2,261470,0.0361772923],[2,261511,0.1339315808],[2,261539,0.3768067982],[2,261542,0.0343147016],[2,261629,0.0290095054],[2,261690,0.0324702418],[2,261858,0.1164417188],[2,261954,0.4179268603],[2,261981,0.2624751437],[2,262043,0.1667649612],[2,262060,0.5131556267],[2,262069,0.0355722911],[2,262077,0.0361772923],[2,262209,0.0379753191],[2,262211,0.3833679374],[2,262234,0.0631842108],[2,262323,0.2890668852],[2,262354,0.1098300048],[2,262451,0.0666299029],[2,262537,0.0635240298],[2,262741,0.1337380679],[2,262776,0.0401731777],[2,262866,0.0951411937],[2,262903,0.0347804367],[2,262906,0.2921864687],[2,263118,0.1597848285],[2,263156,0.0345420863],[2,263352,0.3026581718],[2,263628,0.047692941],[2,263707,0.034775623],[2,263756,0.3147876799],[2,263759,0.2684103238],[2,263774,0.0395033365],[2,263775,0.0340866943],[2,263780,0.1291449804],[2,263793,0.2223015805],[2,263801,0.0342749295],[2,263820,0.0564045879],[2,263857,0.0349688463],[2,264057,0.0875551984],[2,264075,0.2377014365],[2,264080,0.0385836649],[2,264176,0.041309329],[2,264340,0.0412688332],[2,264357,0.475001554],[2,264471,0.0350732539],[2,264496,0.3377195862],[2,264503,0.0324954745],[2,264577,0.0320093048],[2,264729,0.0857941782],[2,264758,0.0344315777],[2,264875,0.563650642],[2,264940,0.0293164553],[2,265036,0.0703534929],[2,265038,0.0630944522],[2,265188,0.0353674367],[2,265229,0.0325315061],[2,265309,0.6475721691],[2,265343,0.3608191582],[2,265349,0.2780538302],[2,265356,0.0989675107],[2,265410,0.0483218661],[2,265436,0.5032669295],[2,270593,0.1921908335],[2,271163,0.0390541686],[2,271197,0.6320885438],[2,271217,0.0299216085],[2,271440,0.0374219961],[2,271511,0.4604719184],[2,271633,0.0348047614],[2,271636,0.0377301891],[2,271842,0.1174700465],[2,272074,0.0727106983],[2,272133,0.0356310219],[2,272148,0.0949084634],[2,272194,0.5101934393],[2,272251,0.0799377958],[2,272318,0.034577319],[2,272324,0.1802549109],[2,272332,0.0325236065],[2,272382,0.094726681],[2,272485,0.0350661027],[2,272498,0.0871620778],[2,272512,0.0995460236],[2,272586,0.2398606165],[2,272629,0.1564069439],[2,272649,0.0460242474],[2,272748,0.0284269399],[2,272800,0.0321270995],[2,272828,0.3797379459],[2,272938,0.0377770708],[2,272944,0.0390405776],[2,272983,0.031326693],[2,273002,0.0369204084],[2,273247,0.0822548676],[2,273329,0.0327114487],[2,273369,0.0618090887],[2,273418,0.0369204084],[2,273453,0.1034744446],[2,273489,0.1183576052],[2,273646,0.2728353652],[2,273710,0.0278205305],[2,273742,0.3089560315],[2,273974,0.0363181327],[2,273978,0.037927296],[2,273980,0.0360894318],[2,273989,0.0341054471],[2,274119,0.3094566006],[2,274248,0.0332665294],[2,274365,0.0429452273],[2,274477,0.0307289002],[2,274571,0.034092579],[2,274596,0.3015133863],[2,274614,0.0800593251],[2,274643,0.0650819244],[2,274664,0.204194566],[2,274675,0.0335846025],[2,274749,0.0565073857],[2,274825,0.0373141716],[2,274862,0.0304441619],[2,274930,0.0346625861],[2,274983,0.0638710541],[2,275046,0.120976064],[2,275073,0.1292723971],[2,275537,0.2105648485],[2,275721,0.2291967147],[2,275738,0.031047131],[2,275747,0.245548557],[2,275748,0.033468841],[2,275774,0.288046882],[2,275816,0.0598533389],[2,275883,0.171792927],[2,275971,0.0376549345],[2,275974,0.037927296],[2,276205,0.8831094321],[2,276208,0.0711466485],[2,276209,0.5088937049],[2,276241,0.4907264967],[2,276289,0.7727620783],[2,276352,0.1721115753],[2,276381,0.0381164968],[2,276500,0.0453788178],[2,276567,0.0656165757],[2,276860,0.0342646604],[2,277004,0.0361169837],[2,277011,0.0691492411],[2,277073,0.3022739311],[2,277090,0.0985124073],[2,277131,0.0367838169],[2,277148,0.22099937],[2,277412,0.0317248824],[2,277674,0.0319125238],[2,277708,0.0326519763],[2,277711,0.0319125238],[2,277882,0.0367838169],[2,277969,0.0499136456],[2,277983,0.0311386744],[2,277986,0.2137586864],[2,278042,0.4373528675],[2,278065,0.0320165057],[2,278092,0.0391787562],[2,278184,0.1570698681],[2,278202,0.0330843518],[2,278205,0.0369727302],[2,278297,0.0853249396],[2,278301,0.0319194275],[2,278415,0.6772050061],[2,278447,0.0953698651],[2,278461,0.0392740174],[2,278582,0.0302279932],[2,278587,0.1040928292],[2,278647,0.1698340216],[2,278754,0.7273534551],[2,278854,0.0887186526],[2,278855,0.1502938294],[2,278874,0.0331514334],[2,278875,0.1023216818],[2,278895,0.0315640362],[2,279049,0.0527235445],[2,279099,0.2055138207],[2,279140,0.0479125644],[2,279197,0.0318889756],[2,279235,0.0331514334],[2,279339,0.1159600325],[2,279370,0.3306793039],[2,279387,0.0452039043],[2,279400,0.2630199475],[2,279527,0.8535109108],[2,279532,0.0815607783],[2,279552,0.0319125238],[2,279676,0.0393523221],[2,279700,0.0591792755],[2,279807,0.0433190444],[2,279881,0.0432723228],[2,279975,0.626003921],[2,280031,0.1465301071],[2,280144,0.0343504449],[2,280151,0.1336616406],[2,280187,0.2749331634],[2,280252,0.0291968449],[2,280280,0.413445044],[2,280287,0.0299686792],[2,280305,0.4729848865],[2,280314,0.1803365468],[2,280326,0.1061565019],[2,280338,0.0583884376],[2,280374,0.041426983],[2,280386,0.1158172987],[2,280411,0.6506414366],[2,280436,0.1178349428],[2,280451,0.1557143729],[2,280465,0.3871372858],[2,280529,0.0343504449],[2,280530,0.0351657506],[2,280590,0.0313358076],[2,280610,0.0683795773],[2,280612,0.0327944077],[2,280656,0.2024871964],[2,280692,0.3384567715],[2,280806,0.0348457262],[2,280919,0.0391528859],[2,280942,0.4928829152],[2,281225,0.0352041777],[2,281241,0.0284047784],[2,281260,0.042522045],[2,281278,0.0282180188],[2,281324,0.0282180188],[2,281332,0.0359553915],[2,281371,0.043636838],[2,281516,0.6597573862],[2,281518,0.0449536717],[2,281899,0.0299686792],[2,282015,0.0359553915],[2,282029,0.0302596162],[2,282042,0.3243984955],[2,282066,0.0523407285],[2,282105,0.0418749677],[2,282149,0.0370875602],[2,282202,0.3726662566],[2,282209,0.0484091817],[2,282245,0.2266335443],[2,282278,0.0343504449],[2,282303,0.0737834831],[2,282308,0.0419962859],[2,282315,0.0714311969],[2,282350,0.0948223303],[2,282456,0.0723854899],[2,282462,0.027738721],[2,282472,0.0888774143],[2,282490,0.0559654634],[2,282492,0.2584266762],[2,282523,0.0514754607],[2,282616,0.1155464562],[2,282638,0.0289298771],[2,282686,0.3152889383],[2,282690,0.5655084704],[2,282693,0.1602043089],[2,282717,0.0304575318],[2,282733,0.2092547708],[2,282796,0.0756638248],[2,282875,0.0289405499],[2,283007,0.2455703303],[2,283017,0.0407334311],[2,283084,0.0296828206],[2,283199,0.0388279788],[2,283221,0.0277761787],[2,283257,0.0276725374],[2,283277,0.0517937433],[2,283388,0.0303691297],[2,283444,0.0901383571],[2,283460,0.0274859618],[2,283915,0.1149925071],[2,283934,0.0277759287],[2,283937,0.0291415178],[2,283986,0.0460363507],[2,283987,0.0324336745],[2,284029,0.0269030834],[2,284030,0.2828196812],[2,284055,0.0618281379],[2,284129,0.2392133949],[2,284160,0.0329511342],[2,284188,0.0355323511],[2,284192,0.0532360068],[2,284235,0.0288916607],[2,284253,0.2287982573],[2,284257,0.0406904864],[2,284293,0.0267167012],[2,284306,0.0445173421],[2,284318,0.0419462915],[2,284358,0.0324336745],[2,284414,0.0489699707],[2,284420,0.0345530821],[2,284421,0.112093281],[2,284488,0.0400493948],[2,284598,0.0898247915],[2,284653,0.1283647088],[2,284709,0.4909079551],[2,284842,0.6246475298],[2,284844,0.087989006],[2,284858,0.2379995577],[2,284863,0.2879325864],[2,284964,0.0274859618],[2,284966,0.0274859618],[2,284973,0.0329511342],[2,284977,0.0274859618],[2,284979,0.0264271481],[2,284984,0.0264271481],[2,284987,0.0264271481],[2,285018,0.0301985246],[2,285023,0.1499291532],[2,285037,0.2350232418],[2,285066,0.1486974553],[2,285203,0.0274859618],[2,285322,0.0274859618],[2,285371,0.7415202867],[2,285378,0.737934469],[2,285422,0.7106592638],[2,285427,0.2082677993],[2,285434,0.0274859618],[2,285452,0.0809023272],[2,285468,0.0327696794],[2,285481,0.817850447],[2,285582,0.5760818395],[2,285593,0.121419678],[2,285595,0.1854694936],[2,285605,0.0351607582],[2,285625,0.0564261245],[2,285638,0.2136674529],[2,285639,0.0753807546],[2,285666,0.0274859618],[2,285697,0.0274859618],[2,285740,0.178748475],[2,285747,0.0274859618],[2,285752,0.0415531239],[2,285761,0.0329511342],[2,285841,0.4356452125],[2,286080,0.0850565259],[2,286161,0.2557916161],[2,286349,0.035060105],[2,286387,0.0322525775],[2,286397,0.0336151552],[2,286407,0.3195702896],[2,286440,0.2833586434],[2,286510,0.0853610471],[2,286554,0.0267989725],[2,286641,0.064621209],[2,286643,0.0274859618],[2,286778,0.0378241743],[2,286829,0.1577615061],[2,286843,0.0856717434],[2,286844,0.096665506],[2,286887,0.4584822566],[2,286912,0.525258888],[2,286916,0.0568101885],[2,286961,0.0254170069],[2,287002,0.0268758633],[2,287035,0.2595712887],[2,287067,0.0298256067],[2,287130,0.0252309983],[2,287145,0.3098523017],[2,287150,0.0263809833],[2,287159,0.0489269553],[2,287164,0.1036544807],[2,287207,0.0506779565],[2,287332,0.2143181844],[2,287336,0.1178357807],[2,287395,0.0257061653],[2,287446,0.7377978595],[2,287552,0.0724624661],[2,287557,0.1586188761],[2,287590,0.037374038],[2,287646,0.1047572254],[2,287648,0.3536688437],[2,287717,0.024175402],[2,287719,0.0242248819],[2,287730,0.0246775846],[2,287752,0.0601445351],[2,287771,0.302137198],[2,287776,0.5876632049],[2,287809,0.8343678276],[2,287823,0.0272128859],[2,287836,0.3164593838],[2,287870,0.2309314787],[2,287902,0.0334546895],[2,287910,0.167200223],[2,287921,0.3424103651],[2,287932,0.3702507951],[2,288008,0.2293295977],[2,288036,0.4239794066],[2,288054,0.0950655502],[2,288059,0.8075172456],[2,288067,0.1040780991],[2,288089,0.2618090961],[2,288211,0.0254170069],[2,288231,0.1207492418],[2,288270,0.0327908109],[2,288276,0.0731825814],[2,288316,0.0254170069],[2,288337,0.0310772199],[2,288339,0.0301022404],[2,288348,0.4848910259],[2,288384,0.4702265385],[2,288386,0.2336405804],[2,288390,0.335755174],[2,288399,0.025520084],[2,288406,0.1449763768],[2,288415,0.2857952665],[2,288429,0.579366421],[2,288433,0.1598678191],[2,288460,0.6366734515],[2,288469,0.0327908109],[2,288472,0.5079183873],[2,288507,0.0327908109],[2,288609,0.0274397311],[2,288731,0.7243791956],[2,288748,0.024175402],[2,288754,0.0968863915],[2,288760,0.4427390792],[2,288857,0.0304370109],[2,288882,0.1944505676],[2,288952,0.5585808314],[2,288963,0.2212547381],[2,288978,0.0864996551],[2,289029,0.3148578312],[2,289072,0.0315543517],[2,289089,0.4945808961],[2,289092,0.1853408023],[2,289100,0.0332497793],[2,289124,0.0897882044],[2,289205,0.1167134244],[2,289217,0.0555624675],[2,289263,0.1504069086],[2,289286,0.3614356974],[2,289303,0.0246758097],[2,289315,0.5190717878],[2,289416,0.3040497929],[2,289447,0.2453489914],[2,289486,0.0461687329],[2,289580,0.0301039852],[2,289611,0.2123183863],[2,289679,0.1188290059],[2,289723,0.0401120067],[2,289745,0.1675680937],[2,289805,0.0271725111],[2,289824,0.2250972438],[2,289837,0.328795427],[2,290033,0.0261947237],[2,290094,0.4395067334],[2,290126,0.088041546],[2,290164,0.0800397013],[2,290188,0.1827530906],[2,290247,0.3240053843],[2,290259,0.2120204305],[2,290288,0.0287352282],[2,290291,0.3852393484],[2,290417,0.0333942243],[2,291000,0.7646943813],[2,291034,0.0783861814],[2,291054,0.0348276738],[2,291075,0.7890865606],[2,291098,0.0308643675],[2,291176,0.2816535045],[2,291227,0.0327193569],[2,291329,0.2034487009],[2,291405,0.0379070165],[2,291428,0.2471416435],[2,291447,0.1480084038],[2,291477,0.0253462579],[2,291501,0.3435478293],[2,291532,0.8684347396],[2,291569,0.0232744878],[2,291613,0.1592779554],[2,291637,0.1555741981],[2,291641,0.0397317449],[2,291653,0.0223243152],[2,291655,0.0232744878],[2,291671,0.0616933838],[2,291677,0.1208758706],[2,291689,0.1784604735],[2,291691,0.1254303033],[2,291701,0.0309994338],[2,291710,0.3451253232],[2,291717,0.0230890178],[2,291755,0.3696486944],[2,291757,0.1838310946],[2,291766,0.3504542602],[2,291785,0.0726279686],[2,291787,0.0232744878],[2,291804,0.0312888988],[2,291808,0.6837567354],[2,291810,0.1066943486],[2,291816,0.7497046759],[2,291822,0.0901814603],[2,291828,0.0946128912],[2,291843,0.0333475703],[2,291854,0.2226495228],[2,291861,0.0224060991],[2,291863,0.0259664779],[2,291870,0.1411440509],[2,291871,0.0817343011],[2,291873,0.0306269423],[2,291874,0.0320412893],[2,291877,0.0471022725],[2,291879,0.5301298113],[2,291900,0.127992509],[2,291904,0.2283491819],[2,291908,0.026701966],[2,291923,0.1292335145],[2,291925,0.104461169],[2,291935,0.482650472],[2,291944,0.1688438502],[2,291954,0.0818611627],[2,291999,0.04614884],[2,292008,0.0645012582],[2,292015,0.0247291204],[2,292016,0.1820433484],[2,292036,0.0232744878],[2,292042,0.0852022381],[2,292045,0.0220364778],[2,292053,0.084979825],[2,292081,0.1516226058],[2,292091,0.1318908019],[2,292093,0.2506890863],[2,292095,0.2026293891],[2,292096,0.3172952853],[2,292097,0.3386457435],[2,292098,0.4970808203],[2,292100,0.081849113],[2,292108,0.7197515258],[2,292110,0.2545254113],[2,292124,0.669268551],[2,292126,0.1535915727],[2,292127,0.0758315393],[2,292128,0.27473863],[2,292139,0.0316139674],[2,292143,0.0267140317],[2,292145,0.0354924898],[2,292146,0.0254693968],[2,292151,0.0393524454],[2,292158,0.1314687873],[2,292161,0.0228237261],[2,292166,0.0205746182],[2,292171,0.2931006413],[2,292194,0.1711470149],[2,292207,0.2342877916],[2,292214,0.0823537243],[2,292216,0.6679149839],[2,292226,0.0748813114],[2,292228,0.0766472336],[2,292231,0.026618466],[2,292241,0.6178032049],[2,292254,0.021659796],[2,292259,0.3889223591],[2,292270,0.0220364778],[2,292280,0.0495516886],[2,292281,0.3510560938],[2,292285,0.024734764],[2,292287,0.1893571821],[2,292289,0.0347537893],[2,292292,0.2457492382],[2,292295,0.4024797928],[2,292298,0.0382408581],[2,292313,0.0222216832],[2,292315,0.0255994066],[2,292332,0.0205746182],[2,292335,0.0633995894],[2,292345,0.0252404637],[2,292366,0.0510806498],[2,292376,0.0324431526],[2,292394,0.1897378271],[2,292413,0.2451204582],[2,292423,0.0213545336],[2,292426,0.0411114596],[2,292433,0.5109237529],[2,292442,0.0306269423],[2,292443,0.3539179405],[2,292460,0.8636421757],[2,292485,0.0447360731],[2,292493,0.0474512608],[2,292497,0.1548078572],[2,292498,0.0481527907],[2,292504,0.0402909095],[2,292518,0.2369372185],[2,292521,0.0306269423],[2,292529,0.2326535441],[2,292532,0.0643915039],[2,292545,0.0437214289],[2,292551,0.2062683308],[2,292570,0.0294370158],[2,292578,0.0285925553],[2,292594,0.198805172],[2,292603,0.0279161921],[2,292609,0.2523002908],[2,292616,0.0285925553],[2,292619,0.0484671083],[2,292652,0.024427551],[2,292655,0.0306892295],[2,292660,0.5175600079],[2,292676,0.0393197495],[2,292686,0.1833846958],[2,292721,0.0767870631],[2,292726,0.2136953444],[2,292728,0.0285925553],[2,292777,0.030372417],[2,292798,0.1507268945],[2,292808,0.0266496884],[2,292811,0.031752643],[2,292821,0.0619752251],[2,292851,0.1010541684],[2,292852,0.027987254],[2,292870,0.0310154759],[2,292888,0.0197579199],[2,292904,0.0219211366],[2,292914,0.0245453883],[2,292936,0.0551604993],[2,292941,0.0264642764],[2,292984,0.2712226958],[2,293027,0.0527226101],[2,293045,0.0219453728],[2,293055,0.0596813413],[2,293057,0.1447798019],[2,293063,0.0380646944],[2,293064,0.2381264698],[2,293075,0.184271113],[2,293115,0.2596713769],[2,293126,0.0334991078],[2,293137,0.0251481031],[2,293141,0.1947103304],[2,293168,0.0948554643],[2,293173,0.0219211366],[2,293192,0.0285925553],[2,293194,0.0220237267],[2,293198,0.0285925553],[2,293200,0.1196105037],[2,293226,0.1111738196],[2,293243,0.046664578],[2,293246,0.0285925553],[2,293248,0.0285925553],[2,293259,0.2551380666],[2,293272,0.0352412381],[2,293278,0.1401727171],[2,293293,0.262611092],[2,293319,0.1687869518],[2,293335,0.2816848314],[2,293363,0.236116559],[2,293399,0.0341674765],[2,293412,0.0330073389],[2,293424,0.2338335227],[2,293447,0.0481347133],[2,293464,0.0252493405],[2,293468,0.2605696149],[2,293499,0.0219453728],[2,293508,0.0299282142],[2,293516,0.0460535651],[2,293681,0.0417130514],[2,293694,0.0285925553],[2,293709,0.0447738769],[2,293713,0.2275430193],[2,293723,0.548153565],[2,293732,0.022880559],[2,293735,0.061339261],[2,293767,0.03805144],[2,293785,0.3216185449],[2,293794,0.0559186252],[2,293796,0.0285925553],[2,293839,0.0325661791],[2,293873,0.0465086287],[2,293875,0.2168879324],[2,293877,0.2903402774],[2,293894,0.0358457898],[2,293896,0.0219211366],[2,293902,0.0219211366],[2,293907,0.0215372413],[2,293914,0.0219211366],[2,293922,0.0209727068],[2,293923,0.0211576447],[2,293929,0.0214712016],[2,293951,0.0252955133],[2,293956,0.2279324406],[2,293963,0.0464045931],[2,293971,0.0283379396],[2,293974,0.2424832133],[2,293981,0.0214712016],[2,294007,0.1927508599],[2,294029,0.2248195229],[2,294043,0.0210210093],[2,294056,0.4575887058],[2,294099,0.0227547075],[2,294131,0.1395928088],[2,294138,0.0223621005],[2,294142,0.0248966001],[2,294157,0.0339901769],[2,294172,0.0320340667],[2,294179,0.0334231552],[2,294195,0.0390415919],[2,294228,0.1348266263],[2,294234,0.0264146817],[2,294236,0.0317203171],[2,294246,0.0397301849],[2,294251,0.0487848072],[2,294256,0.0449307201],[2,294259,0.0510572564],[2,294270,0.1825439935],[2,294275,0.3227735691],[2,294299,0.0355720684],[2,294325,0.2885589558],[2,294328,0.2593363794],[2,294330,0.0294336769],[2,294347,0.3947722668],[2,294351,0.0851700046],[2,294352,0.0702492706],[2,294360,0.0225693681],[2,294370,0.0530659534],[2,294372,0.0215175696],[2,294388,0.0213371796],[2,294407,0.0215175696],[2,294414,0.0215175696],[2,294416,0.0218052043],[2,294422,0.0449307201],[2,294432,0.0497932246],[2,294448,0.0337941586],[2,294460,0.0215175696],[2,294473,0.0539903318],[2,294484,0.0210210093],[2,294487,0.0225693681],[2,294497,0.0266787065],[2,294498,0.0294336769],[2,294507,0.1821864441],[2,294524,0.0376767022],[2,294531,0.1389458305],[2,294537,0.0210210093],[2,294564,0.0348560613],[2,294570,0.0913416456],[2,294585,0.0307881891],[2,294594,0.0215175696],[2,294596,0.0223069384],[2,294609,0.0294336769],[2,294625,0.0210210093],[2,294643,0.0597588007],[2,294659,0.2370740015],[2,294663,0.0283307354],[2,294676,0.0294336769],[2,294680,0.0215175696],[2,294681,0.0215175696],[2,294683,0.0215175696],[2,294711,0.0300945734],[2,294741,0.6230889318],[2,294750,0.0228574137],[2,294759,0.0225693681],[2,294760,0.0223069384],[2,294761,0.0218052043],[2,294762,0.0215175696],[2,294763,0.0294336769],[2,294774,0.063307712],[2,294788,0.1507838349],[2,294813,0.0447026554],[2,294814,0.027096193],[2,294817,0.3329423982],[2,294829,0.026272902],[2,294851,0.1114475302],[2,294852,0.0215175696],[2,294854,0.0294336769],[2,294864,0.0646336202],[2,294871,0.0439315221],[2,294876,0.1041629405],[2,294895,0.0454231204],[2,294903,0.0294336769],[2,294944,0.0464809563],[2,294950,0.2836866143],[2,294952,0.1254301321],[2,294956,0.122546888],[2,294997,0.0210210093],[2,295018,0.0302818049],[2,295048,0.0223621005],[2,295051,0.0219903515],[2,295074,0.0294336769],[2,295085,0.0218052043],[2,295095,0.0227547075],[2,295096,0.0895254579],[2,295103,0.0334863756],[2,295119,0.100366004],[2,295137,0.1502923291],[2,295138,0.0243931027],[2,295142,0.0294336769],[2,295147,0.2167820617],[2,295155,0.0449307201],[2,295162,0.1976294067],[2,295168,0.0223069384],[2,295185,0.0676522996],[2,295204,0.0225693681],[2,295210,0.1269359549],[2,295219,0.0239008433],[2,295251,0.0302818049],[2,295256,0.2985494687],[2,295258,0.0328518707],[2,295261,0.0223703777],[2,295284,0.0396473728],[2,295292,0.4394773364],[2,295295,0.0257908466],[2,295318,0.6360383775],[2,295323,0.0223069384],[2,295327,0.4053456225],[2,295332,0.5611671777],[2,295347,0.7602702657],[2,295349,0.2612072056],[2,295353,0.0573297487],[2,295394,0.0208361057],[2,295421,0.028164557],[2,295427,0.02544901],[2,295434,0.0293864255],[2,295435,0.0431467637],[2,295439,0.0243931027],[2,295442,0.0227547075],[2,295445,0.0306162647],[2,295446,0.2243462071],[2,295452,0.0253670144],[2,295466,0.160026338],[2,295471,0.0294336769],[2,295492,0.0247173304],[2,295502,0.0999135982],[2,295504,0.0242329938],[2,295505,0.0218052043],[2,295508,0.0294336769],[2,295514,0.0227547075],[2,295526,0.0219903515],[2,295528,0.0316706138],[2,295540,0.2918319643],[2,295551,0.0312155531],[2,295557,0.7443081178],[2,295581,0.4741784921],[2,295585,0.1614367723],[2,295589,0.5905031473],[2,295593,0.1158908752],[2,295596,0.1745876556],[2,295597,0.0622808693],[2,295599,0.4228591349],[2,295602,0.042073407],[2,295613,0.2312969474],[2,295617,0.0746946854],[2,295620,0.0294336769],[2,295629,0.0492429502],[2,295631,0.0314572011],[2,295639,0.1248277801],[2,295654,0.3239284512],[2,295658,0.0294336769],[2,295664,0.0294336769],[2,295668,0.3042737638],[2,295669,0.0237152158],[2,295675,0.031929903],[2,295680,0.0237152158],[2,295686,0.0323376967],[2,295687,0.0294336769],[2,295701,0.2622345863],[2,295704,0.8503909517],[2,295712,0.0219903515],[2,295718,0.0218052043],[2,295734,0.0278189862],[2,295752,0.2351658467],[2,295780,0.0302818049],[2,295782,0.0225693681],[2,295790,0.0294336769],[2,295795,0.0412213421],[2,295797,0.0503066449],[2,295798,0.0227547075],[2,295799,0.026272902],[2,295806,0.0985171644],[2,295808,0.0294336769],[2,295815,0.0239391084],[2,295819,0.1079440739],[2,295835,0.0343457826],[2,295839,0.0294336769],[2,295847,0.0530548282],[2,295848,0.2513976597],[2,295849,0.0439006081],[2,295850,0.0496816894],[2,295861,0.0225693681],[2,295870,0.0278658474],[2,295881,0.0294336769],[2,295883,0.0403521067],[2,295903,0.2629143629],[2,295910,0.1980319333],[2,295916,0.0303251884],[2,295918,0.0294336769],[2,295934,0.196066471],[2,295955,0.1042460494],[2,295972,0.0218052043],[2,295978,0.2552460951],[2,295981,0.0730176517],[2,295985,0.1254499288],[2,295998,0.3669936104],[2,296014,0.0575280884],[2,296018,0.0690472693],[2,296023,0.0225693681],[2,296032,0.0334552571],[2,296038,0.0998729111],[2,296041,0.0511581442],[2,296047,0.4109506112],[2,296065,0.126118695],[2,296123,0.282866554],[2,296134,0.0249032098],[2,296178,0.0294336769],[2,296180,0.0394611724],[2,296185,0.4192523057],[2,296186,0.4948961893],[2,296187,0.2636144422],[2,296202,0.0294336769],[2,296225,0.3339255746],[2,296240,0.0525392804],[2,296242,0.1480139307],[2,296247,0.3240750033],[2,296249,0.287779805],[2,296252,0.0228574137],[2,296268,0.3561042007],[2,296275,0.1187813002],[2,296278,0.4553329996],[2,296290,0.0838775284],[2,296460,0.1791557955],[2,296507,0.4569659865],[2,296531,0.0294336769],[2,296546,0.0242083153],[2,296568,0.0294336769],[2,296570,0.3292433405],[2,296574,0.0357097895],[2,296605,0.0425191164],[2,296614,0.0227547075],[2,296627,0.1769277525],[2,296635,0.2779777134],[2,296657,0.0228889264],[2,296691,0.0220720983],[2,296699,0.228402383],[2,296715,0.0294336769],[2,296717,0.0901202192],[2,296731,0.0791577951],[2,296747,0.0271254025],[2,296751,0.0498662877],[2,296759,0.10825854],[2,296762,0.0446191502],[2,296769,0.0296342503],[2,296776,0.2054524776],[2,296788,0.0748025613],[2,296790,0.0247701545],[2,296801,0.0324071284],[2,296804,0.0266555408],[2,296809,0.0218052043],[2,296814,0.3060171244],[2,296825,0.0589601735],[2,296829,0.0225693681],[2,296842,0.0294336769],[2,296868,0.0225693681],[2,296886,0.0802107995],[2,296897,0.0294336769],[2,296903,0.0300802913],[2,296914,0.0606365416],[2,296918,0.0294336769],[2,296921,0.0498631981],[2,296968,0.1802858413],[2,296996,0.0406923181],[2,297044,0.0225693681],[2,297047,0.0236089488],[2,297083,0.0294336769],[2,297094,0.1891355621],[2,297170,0.118904569],[2,297173,0.128812188],[2,297177,0.0227547075],[2,297180,0.2400337793],[2,297247,0.2300856384],[2,297275,0.0260059822],[2,297296,0.0334682313],[2,297315,0.0254523913],[2,297317,0.0684271997],[2,297325,0.0273618844],[2,297332,0.0403504613],[2,297363,0.0854837406],[2,297365,0.0327144026],[2,297367,0.1787098383],[2,297381,0.0260059822],[2,297386,0.1937367616],[2,297402,0.0327144026],[2,297407,0.0279298453],[2,297409,0.075735727],[2,297425,0.0258198255],[2,297429,0.5019089117],[2,297445,0.1418499015],[2,297459,0.1808202153],[2,297469,0.1550477911],[2,297479,0.0335662704],[2,297493,0.0329929998],[2,297515,0.04831973],[2,297520,0.204580383],[2,297522,0.0511987138],[2,297525,0.0260064984],[2,297526,0.2550528066],[2,297533,0.0382648752],[2,297560,0.0267837992],[2,297567,0.0246601683],[2,297581,0.0257172227],[2,297594,0.2541481751],[2,297607,0.0679178923],[2,297614,0.3917259407],[2,297617,0.0270825707],[2,297625,0.0280498384],[2,297637,0.0482797802],[2,297643,0.0267198319],[2,297645,0.0469528437],[2,297650,0.0277407675],[2,297661,0.0265775849],[2,297670,0.0283493612],[2,297684,0.107994247],[2,297688,0.0327144026],[2,297692,0.3557657117],[2,297703,0.263019152],[2,297728,0.1546878005],[2,297745,0.0257172227],[2,297758,0.1811932578],[2,297765,0.191314166],[2,297773,0.2324745697],[2,297777,0.0257172227],[2,297779,0.0464708317],[2,297781,0.5574477937],[2,297787,0.0257172227],[2,297789,0.2395129653],[2,297797,0.0245572109],[2,297812,0.0297209557],[2,297835,0.0473391652],[2,297843,0.2609755264],[2,297851,0.216295037],[2,297853,0.0482797802],[2,297863,0.0554264163],[2,297870,0.06938598],[2,297896,0.034504136],[2,297898,0.0255311386],[2,297901,0.0255311386],[2,297903,0.1860297761],[2,297915,0.0523868316],[2,297925,0.7669691249],[2,297937,0.0265775849],[2,297939,0.2944559102],[2,297955,0.0251638476],[2,297959,0.023872933],[2,297978,0.0246601683],[2,297990,0.5350477948],[2,298010,0.0265775849],[2,298014,0.1734158712],[2,298023,0.0762849644],[2,298030,0.0265775849],[2,298036,0.0327144026],[2,298039,0.068677759],[2,298058,0.0773885277],[2,298069,0.0341549399],[2,298074,0.0978788016],[2,298075,0.1161743598],[2,298078,0.0248460333],[2,298086,0.0335662704],[2,298088,0.0744865583],[2,298091,0.0335662704],[2,298093,0.0640262526],[2,298094,0.0289582416],[2,298096,0.604000208],[2,298100,0.0837049827],[2,298101,0.0441870586],[2,298103,0.0238159626],[2,298118,0.0259177333],[2,298121,0.1086111498],[2,298132,0.1421379381],[2,298133,0.4561697061],[2,298159,0.028172189],[2,298166,0.0355531102],[2,298171,0.1934506514],[2,298174,0.0247446891],[2,298212,0.0404693976],[2,298213,0.1685664682],[2,298219,0.0659944867],[2,298222,0.0430093332],[2,298227,0.1914399088],[2,298242,0.0257172227],[2,298257,0.0255311386],[2,298266,0.0265775849],[2,298275,0.0246601683],[2,298277,0.0415425923],[2,298305,0.025076813],[2,298318,0.025076813],[2,298343,0.0966223208],[2,298344,0.025076813],[2,298353,0.0488160589],[2,298355,0.0332397391],[2,298366,0.0260514254],[2,298379,0.0387100993],[2,298383,0.0596079827],[2,298384,0.0332397391],[2,298390,0.1137330624],[2,298396,0.0837456874],[2,298427,0.0269188037],[2,298428,0.3268145387],[2,298438,0.0253657642],[2,298441,0.3006812404],[2,298445,0.1009485432],[2,298450,0.1910857637],[2,298451,0.505478807],[2,298458,0.1370928104],[2,298465,0.1620241043],[2,298500,0.0262376403],[2,298530,0.1240110814],[2,298560,0.0340922058],[2,298561,0.0262376403],[2,298565,0.1344893705],[2,298573,0.1920119616],[2,298579,0.1920824328],[2,298580,0.0742802996],[2,298581,0.0633283388],[2,298589,0.0332397391],[2,298604,0.2892010696],[2,298609,0.0294809376],[2,298619,0.0319559497],[2,298673,0.1520185328],[2,298686,0.0353163525],[2,298699,0.5610980224],[2,298701,0.204369144],[2,298706,0.0374840175],[2,298711,0.0884900993],[2,298744,0.0808821057],[2,298750,0.3255041177],[2,298764,0.2265633697],[2,298768,0.178665372],[2,298778,0.4018324214],[2,298785,0.0260514254],[2,298796,0.0722498711],[2,298814,0.2067299753],[2,298818,0.2212530933],[2,298840,0.1639544233],[2,298848,0.0569591658],[2,298852,0.0394677589],[2,298853,0.0572199545],[2,298856,0.0524377466],[2,298864,0.0313729237],[2,298867,0.0899021957],[2,298874,0.0332397391],[2,298881,0.0534322977],[2,298894,0.0237833591],[2,298901,0.4341950071],[2,298905,0.0262376403],[2,298921,0.0387175596],[2,298923,0.0402333366],[2,298966,0.030524422],[2,298977,0.0488160589],[2,298978,0.0883492325],[2,298981,0.0251798427],[2,299011,0.3077518843],[2,299019,0.0251798427],[2,299037,0.0456602481],[2,299061,0.0481312044],[2,299066,0.0378878771],[2,299084,0.0336255197],[2,299089,0.0296680147],[2,299090,0.0350307306],[2,299093,0.0457073923],[2,299097,0.1308825512],[2,299110,0.0502044351],[2,299118,0.0517051065],[2,299129,0.0303561783],[2,299142,0.026579343],[2,299151,0.03363044],[2,299154,0.0248955508],[2,299166,0.035023732],[2,299168,0.0376195291],[2,299185,0.0265431247],[2,299192,0.179997443],[2,299246,0.026579343],[2,299255,0.0268871846],[2,299277,0.4764461682],[2,299295,0.6177065178],[2,299300,0.0364179326],[2,299306,0.6293037729],[2,299309,0.0260902292],[2,299326,0.026442665],[2,299328,0.6520496277],[2,299346,0.0257971858],[2,299349,0.2510495786],[2,299363,0.5174132006],[2,299365,0.1912832962],[2,299374,0.0279886338],[2,299376,0.0867660519],[2,299380,0.0570459812],[2,299382,0.0360577849],[2,299388,0.4435109454],[2,299404,0.0240700628],[2,299410,0.0389037115],[2,299413,0.0827464371],[2,299418,0.0272526469],[2,299420,0.1008189905],[2,299433,0.1032750527],[2,299436,0.1658035652],[2,299437,0.0311844373],[2,299444,0.1474711629],[2,299463,0.038514433],[2,299474,0.2527391683],[2,299483,0.0250516499],[2,299513,0.6461603813],[2,299520,0.2053053209],[2,299524,0.090744217],[2,299531,0.0270661769],[2,299540,0.3075106969],[2,299541,0.026442665],[2,299546,0.0629361468],[2,299553,0.0272526469],[2,299554,0.0250434229],[2,299564,0.0432152349],[2,299570,0.0327251964],[2,299575,0.0432133568],[2,299592,0.1818017425],[2,299599,0.0231126799],[2,299601,0.027653652],[2,299602,0.0245733401],[2,299637,0.0297772052],[2,299639,0.1584218182],[2,299641,0.2599246667],[2,299643,0.0960314386],[2,299647,0.0953457122],[2,299651,0.3950208718],[2,299661,0.2955969707],[2,299674,0.0358691016],[2,299677,0.3434204251],[2,299681,0.3771925265],[2,299695,0.1304338566],[2,299699,0.0366946685],[2,299706,0.421893832],[2,299714,0.05349319],[2,299719,0.0518285961],[2,299723,0.2298924371],[2,299744,0.0654774155],[2,299753,0.3435784589],[2,299759,0.2264279941],[2,299782,0.1160405274],[2,299794,0.0272526469],[2,299806,0.1066853347],[2,299810,0.2498174811],[2,299831,0.0270661769],[2,299848,0.2741789233],[2,299850,0.0275604629],[2,299865,0.3019855122],[2,299867,0.4479852776],[2,299871,0.2094470025],[2,299873,0.2677002425],[2,299880,0.0240700628],[2,299889,0.2974436649],[2,299900,0.0272526469],[2,299904,0.0291067404],[2,299906,0.3531642524],[2,299910,0.5735163927],[2,299916,0.4966022077],[2,299919,0.0449790232],[2,299922,0.0270661769],[2,299924,0.1714259848],[2,299925,0.0816443918],[2,299930,0.0270661769],[2,299931,0.1447990424],[2,299936,0.4400710349],[2,299948,0.0416538817],[2,299951,0.1367013307],[2,299954,0.0240700628],[2,299959,0.0390391987],[2,299965,0.0271482774],[2,299969,0.0272526469],[2,299971,0.0300130905],[2,299973,0.0838968168],[2,299980,0.222781825],[2,299982,0.1652036509],[2,299984,0.6123171014],[2,299988,0.1937632488],[2,300006,0.0503812094],[2,300009,0.3121053368],[2,300011,0.2484341687],[2,300017,0.0271041543],[2,300023,0.6447215908],[2,300040,0.7931677158],[2,300041,0.788881775],[2,300056,0.0311125323],[2,300061,0.2620767932],[2,300066,0.2505077425],[2,300068,0.4079266365],[2,300080,0.0252293843],[2,300085,0.2442741363],[2,300109,0.04196049],[2,300119,0.0245733401],[2,300126,0.2102420354],[2,300129,0.2551558449],[2,300141,0.0351179741],[2,300145,0.026390451],[2,300149,0.0384898449],[2,300151,0.0297772052],[2,300153,0.0267727474],[2,300160,0.1194428967],[2,300180,0.1247859958],[2,300186,0.0272526469],[2,300216,0.2235280207],[2,300217,0.0369772603],[2,300223,0.0412137569],[2,300225,0.0270661769],[2,300230,0.0353527351],[2,300244,0.0284093302],[2,300247,0.1197715464],[2,300250,0.1976215858],[2,300258,0.1592313811],[2,300261,0.0342643395],[2,300270,0.0242557795],[2,300309,0.0821550601],[2,300320,0.0393177675],[2,300325,0.0342643395],[2,300380,0.0242557795],[2,300381,0.265752563],[2,300389,0.0254153182],[2,300417,0.5046108759],[2,300433,0.0437875508],[2,300440,0.0546158948],[2,300451,0.2702776784],[2,300453,0.4835695154],[2,300456,0.225478676],[2,300468,0.5399716239],[2,300477,0.5033180556],[2,300482,0.2455813986],[2,300496,0.132472161],[2,300499,0.0297772052],[2,300500,0.0274670812],[2,300509,0.0482524618],[2,300523,0.5411012168],[2,300527,0.2449824922],[2,300539,0.3149801608],[2,300565,0.2661916587],[2,300572,0.7490169564],[2,300585,0.0624347593],[2,300589,0.0245832678],[2,300593,0.1546432131],[2,300598,0.0255363531],[2,300619,0.0253503145],[2,300624,0.0483936327],[2,300644,0.2737707511],[2,300657,0.2128612438],[2,300661,0.027191873],[2,300664,0.0524915636],[2,300665,0.0649226671],[2,300673,0.0268256373],[2,300674,0.0346981877],[2,300683,0.0435908332],[2,300687,0.024294548],[2,300691,0.0264564004],[2,300692,0.0560458311],[2,300694,0.0325318234],[2,300702,0.0275594035],[2,300703,0.0503774362],[2,300751,0.1266093382],[2,300752,0.1490601923],[2,300769,0.1248136694],[2,300778,0.1013348014],[2,300781,0.0245640571],[2,300785,0.2317771112],[2,300789,0.0245832678],[2,300801,0.0286137367],[2,300804,0.1333086034],[2,300805,0.6136611584],[2,300810,0.0325318234],[2,300813,0.0316378595],[2,300817,0.0321026988],[2,300819,0.4790681457],[2,300828,0.0325318234],[2,300832,0.0286032704],[2,300838,0.0308260316],[2,300842,0.5750758016],[2,300851,0.048093398],[2,300852,0.0325318234],[2,300856,0.0884649386],[2,300861,0.0255363531],[2,300870,0.033195472],[2,300881,0.0245832678],[2,300892,0.1150582687],[2,300894,0.2178746179],[2,300899,0.0251505734],[2,300901,0.0255363531],[2,300909,0.0326160927],[2,300915,0.0253503145],[2,300916,0.0378377844],[2,300920,0.0263938172],[2,300925,0.1450602231],[2,300950,0.0498680857],[2,300952,0.0651970336],[2,300955,0.0245832678],[2,300957,0.2487217238],[2,300958,0.033195472],[2,300979,0.2336509532],[2,300987,0.0333834831],[2,301009,0.1036031897],[2,301023,0.0263208264],[2,301027,0.0339017462],[2,301046,0.2527773968],[2,301052,0.7300490377],[2,301065,0.0242992051],[2,301073,0.0343211195],[2,301078,0.0236105133],[2,301085,0.0249920191],[2,301086,0.0307204616],[2,301090,0.0237961144],[2,301091,0.0245832678],[2,301097,0.2758161664],[2,301098,0.2516404264],[2,301103,0.7211779508],[2,301105,0.0325318234],[2,301109,0.2730325011],[2,301118,0.0385493305],[2,301127,0.0236105133],[2,301132,0.131432029],[2,301134,0.0411709116],[2,301137,0.0277638842],[2,301145,0.1063855533],[2,301149,0.0255363531],[2,301162,0.0259816394],[2,301166,0.0472783896],[2,301179,0.1171933047],[2,301193,0.0303762436],[2,301198,0.3468617962],[2,301210,0.8449839551],[2,301218,0.0275266463],[2,301223,0.0294540336],[2,301231,0.1590017919],[2,301237,0.1643466372],[2,301240,0.5420296588],[2,301242,0.0259816394],[2,301243,0.5834378194],[2,301246,0.0325318234],[2,301252,0.2577073107],[2,301259,0.079241904],[2,301272,0.048093398],[2,301278,0.0474154874],[2,301280,0.0367539734],[2,301303,0.0272546414],[2,301311,0.028880894],[2,301346,0.0325318234],[2,301365,0.0251505734],[2,301370,0.0335630459],[2,301373,0.0280550706],[2,301380,0.5810720141],[2,301408,0.0346396113],[2,301410,0.0255363531],[2,301423,0.0349700785],[2,301424,0.022653726],[2,301425,0.0241134775],[2,301427,0.0361715477],[2,301431,0.026779369],[2,301436,0.0244803211],[2,301439,0.0356138538],[2,301442,0.2810765792],[2,301458,0.4192149451],[2,301461,0.2750286319],[2,301466,0.3056940869],[2,301467,0.0249319972],[2,301471,0.0319217553],[2,301474,0.0319217553],[2,301485,0.0250354883],[2,301498,0.0319217553],[2,301501,0.0319217553],[2,301503,0.0224678856],[2,301511,0.0318297256],[2,301519,0.0414423006],[2,301530,0.0219660419],[2,301538,0.0371716361],[2,301543,0.0226485528],[2,301578,0.0255621085],[2,301583,0.3376444912],[2,301584,0.1788732141],[2,301594,0.0226485528],[2,301597,0.1538368193],[2,301606,0.0245880658],[2,301614,0.0779776014],[2,301615,0.0319217553],[2,301621,0.0391611743],[2,301624,0.4132290226],[2,301632,0.0249319972],[2,301633,0.381101035],[2,301647,0.0647528719],[2,301662,0.0254590255],[2,301663,0.3890527456],[2,301672,0.0252809541],[2,301674,0.0337584132],[2,301692,0.0308246193],[2,301720,0.0249319972],[2,301723,0.0254590255],[2,301753,0.0256325624],[2,301754,0.0228717028],[2,301756,0.0319217553],[2,301762,0.0943024095],[2,301763,0.0277908276],[2,301769,0.7802778436],[2,301779,0.0508021731],[2,301780,0.1405739527],[2,301803,0.025353384],[2,301807,0.0383306383],[2,301810,0.0319217553],[2,301811,0.2581851199],[2,301819,0.0219660419],[2,301834,0.501239722],[2,301837,0.0319217553],[2,301840,0.0301153022],[2,301845,0.5209774477],[2,301848,0.0257144784],[2,301854,0.0581594718],[2,301856,0.2916131358],[2,301865,0.0249319972],[2,301870,0.2299498978],[2,301879,0.0234391344],[2,301894,0.0337095903],[2,301902,0.2687112692],[2,301906,0.0419138358],[2,301922,0.0317865089],[2,301929,0.0306963138],[2,301930,0.025353384],[2,301935,0.0319217553],[2,301949,0.1147917402],[2,301963,0.0245103079],[2,301966,0.0453162241],[2,301971,0.027709311],[2,301972,0.1716342517],[2,301982,0.1707967175],[2,301992,0.0323619451],[2,302006,0.0491779757],[2,302007,0.3849336492],[2,302016,0.2788226887],[2,302044,0.0334774002],[2,302047,0.0639678652],[2,302052,0.0239092766],[2,302057,0.0851441442],[2,302073,0.0446933037],[2,302074,0.1870818482],[2,302077,0.2251416806],[2,302085,0.0233436874],[2,302096,0.0387484994],[2,302097,0.0262599903],[2,302109,0.0239092766],[2,302116,0.4065249445],[2,302130,0.1706847513],[2,302131,0.0256292849],[2,302136,0.0353490427],[2,302139,0.0336886123],[2,302150,0.1035641978],[2,302158,0.5268158366],[2,302162,0.1576791897],[2,302170,0.1704453053],[2,302172,0.5146355048],[2,302178,0.1284048928],[2,302185,0.0264462576],[2,302191,0.3269622642],[2,302200,0.1954656926],[2,302206,0.0474706227],[2,302209,0.0277820125],[2,302214,0.024438587],[2,302215,0.0254711936],[2,302218,0.0248183648],[2,302227,0.2855618655],[2,302238,0.0342268839],[2,302243,0.028368307],[2,302258,0.1511049948],[2,302259,0.0237062892],[2,302263,0.048209541],[2,302267,0.0309002089],[2,302276,0.0304854947],[2,302281,0.0400964727],[2,302284,0.0475527247],[2,302290,0.034600804],[2,302292,0.0363893865],[2,302303,0.0747559688],[2,302316,0.1340519447],[2,302329,0.4913817356],[2,302333,0.025752713],[2,302346,0.1129232321],[2,302351,0.0383836545],[2,302353,0.0256292849],[2,302362,0.135371981],[2,302375,0.0268004179],[2,302377,0.0441762006],[2,302382,0.2026839319],[2,302387,0.7460763395],[2,302399,0.0269868211],[2,302400,0.0256292849],[2,302404,0.034629497],[2,302408,0.033158145],[2,302415,0.120401278],[2,302422,0.0252850291],[2,302425,0.0352023186],[2,302434,0.2214971087],[2,302440,0.1016942988],[2,302445,0.2238982204],[2,302458,0.0238176422],[2,302463,0.0233436874],[2,302471,0.2128608016],[2,302474,0.092656503],[2,302483,0.0384524276],[2,302484,0.0347336861],[2,302489,0.4701931363],[2,302494,0.2542626992],[2,302508,0.0537255911],[2,302510,0.023640247],[2,302517,0.0628728717],[2,302520,0.0261568101],[2,302527,0.0240399982],[2,302532,0.2415886942],[2,302539,0.0662520593],[2,302542,0.2749021913],[2,302548,0.0235373465],[2,302556,0.1572733909],[2,302561,0.0495938535],[2,302564,0.0254432229],[2,302566,0.0414192688],[2,302571,0.2765289701],[2,302574,0.023454685],[2,302579,0.8545095287],[2,302581,0.341419873],[2,302584,0.7667198113],[2,302590,0.0256292849],[2,302607,0.0287811704],[2,302608,0.0538618525],[2,302609,0.1471366737],[2,302625,0.2316011618],[2,302627,0.1068110055],[2,302629,0.0326256336],[2,302646,0.193487518],[2,302655,0.437626073],[2,302665,0.0254432229],[2,302677,0.0277820125],[2,302684,0.0472790706],[2,302687,0.4535825629],[2,302693,0.0465596221],[2,302696,0.157093153],[2,302702,0.0358991686],[2,302721,0.220030282],[2,302726,0.0423353301],[2,302728,0.2217277178],[2,302738,0.025553401],[2,302740,0.1124428218],[2,302778,0.29886957],[2,302782,0.0553954838],[2,302786,0.0264462576],[2,302787,0.0756222393],[2,302803,0.0256292849],[2,302818,0.3730247653],[2,302825,0.3048693276],[2,302827,0.3396060094],[2,302848,0.0968075886],[2,302852,0.0277130652],[2,302864,0.1543389276],[2,302874,0.2321707239],[2,302875,0.0415323439],[2,302883,0.0275264794],[2,302886,0.3774034409],[2,302890,0.4699311925],[2,302896,0.1164256333],[2,302900,0.0447018918],[2,302902,0.0323030471],[2,302903,0.0591179621],[2,302910,0.0459659825],[2,302912,0.0487641409],[2,302913,0.1784994138],[2,302927,0.0344970511],[2,302931,0.1242784632],[2,302939,0.0283794339],[2,302943,0.0264462576],[2,302990,0.0292206561],[2,302999,0.3355130272],[2,303006,0.0259706155],[2,303008,0.0343587578],[2,303017,0.0334774002],[2,303020,0.1222381719],[2,303029,0.1058779212],[2,303036,0.0600537307],[2,303047,0.0332893655],[2,303057,0.023454685],[2,303065,0.0317826957],[2,303070,0.1003702242],[2,303073,0.0235373465],[2,303078,0.1657733529],[2,303082,0.0233436874],[2,303083,0.0326256336],[2,303084,0.0256066079],[2,303092,0.0277130652],[2,303093,0.0549514354],[2,303094,0.189454658],[2,303105,0.0334774002],[2,303106,0.0264602552],[2,303112,0.1753567896],[2,303129,0.0740309956],[2,303139,0.2288825464],[2,303168,0.0354277384],[2,303170,0.1733548985],[2,303184,0.0401750761],[2,303187,0.0414252634],[2,303195,0.0332893655],[2,303203,0.0606199995],[2,303210,0.0262599903],[2,303215,0.1634721942],[2,303221,0.1790346275],[2,303252,0.0254432229],[2,303271,0.0347336861],[2,303280,0.1039787076],[2,303286,0.1392772625],[2,303325,0.4536253914],[2,303328,0.1103930827],[2,303338,0.0471684873],[2,303339,0.0334774002],[2,303349,0.3247844934],[2,303353,0.0326256336],[2,303401,0.0818346688],[2,303403,0.1849316533],[2,303404,0.0326256336],[2,303410,0.0362037249],[2,303434,0.0259706155],[2,303448,0.0238176422],[2,303464,0.0256292849],[2,303468,0.114205509],[2,303483,0.1072095818],[2,303485,0.1546724022],[2,303488,0.0593529428],[2,303489,0.0256292849],[2,303495,0.0334774002],[2,303499,0.0228458952],[2,303500,0.0334774002],[2,303507,0.0232156758],[2,303511,0.6346957497],[2,303512,0.0277358212],[2,303513,0.1181904323],[2,303526,0.075234824],[2,303534,0.0334774002],[2,303536,0.0268004179],[2,303540,0.0326256336],[2,303542,0.0235126126],[2,303553,0.1102291083],[2,303555,0.1201258257],[2,303575,0.0236320356],[2,303587,0.725519745],[2,303590,0.0970953542],[2,303601,0.0326256336],[2,303602,0.0398979399],[2,303612,0.059198492],[2,303616,0.2359249753],[2,303629,0.1149757554],[2,303634,0.0307603512],[2,303649,0.1194035746],[2,303673,0.2750944676],[2,303676,0.0264602552],[2,303684,0.0312172074],[2,303698,0.0236320356],[2,303703,0.0256292849],[2,303724,0.0275264794],[2,303727,0.1367511746],[2,303728,0.3877919336],[2,303734,0.0326256336],[2,303735,0.0347690568],[2,303738,0.1529893432],[2,303740,0.0236320356],[2,303768,0.0347336861],[2,303779,0.0335404409],[2,303781,0.0580963062],[2,303824,0.0401498488],[2,303825,0.0402762886],[2,303830,0.0310705993],[2,303834,0.0395168406],[2,303837,0.1225002003],[2,303840,0.0595800431],[2,303843,0.0330827592],[2,303846,0.0388475608],[2,303851,0.0388475608],[2,303855,0.2331359887],[2,303868,0.1127937005],[2,303875,0.0332541872],[2,303881,0.0309667491],[2,303887,0.0399508829],[2,303888,0.3728337149],[2,303894,0.0316991774],[2,303898,0.0779574394],[2,303904,0.0304357984],[2,303910,0.0826086005],[2,303918,0.0424602259],[2,303929,0.0369626055],[2,303948,0.7584886422],[2,303952,0.2867180623],[2,303953,0.6171963293],[2,303954,0.1820887606],[2,303960,0.0324233595],[2,303970,0.0284255801],[2,303971,0.0445804091],[2,303978,0.0309847541],[2,303980,0.0349702337],[2,303988,0.0302485283],[2,303990,0.027447769],[2,303992,0.0330046432],[2,303993,0.1463707454],[2,304007,0.03704745],[2,304020,0.037337716],[2,304022,0.03704745],[2,304023,0.2335533102],[2,304033,0.0340434281],[2,304039,0.0296196126],[2,304052,0.0415133543],[2,304065,0.0372961432],[2,304070,0.027447769],[2,304076,0.5072251466],[2,304085,0.2539659736],[2,304092,0.1905444347],[2,304097,0.0309667491],[2,304101,0.2334103997],[2,304107,0.0386739073],[2,304117,0.042347096],[2,304121,0.0312580761],[2,304122,0.0543969928],[2,304128,0.1809952522],[2,304144,0.0423400029],[2,304155,0.0519802892],[2,304158,0.0779772747],[2,304165,0.0467235963],[2,304173,0.0943797707],[2,304175,0.0309847541],[2,304182,0.0907252048],[2,304205,0.0302841762],[2,304207,0.4329338485],[2,304209,0.0310626591],[2,304211,0.026673699],[2,304227,0.0302485283],[2,304235,0.0372961432],[2,304237,0.1266155144],[2,304254,0.03704745],[2,304258,0.0484635739],[2,304260,0.0302485283],[2,304268,0.0290735318],[2,304274,0.0326340826],[2,304291,0.0309667491],[2,304299,0.0386581763],[2,304303,0.1986046094],[2,304306,0.03704745],[2,304322,0.0294325476],[2,304331,0.0336449205],[2,304342,0.0300893073],[2,304364,0.0727547375],[2,304366,0.0525347727],[2,304367,0.0465427368],[2,304373,0.0369572424],[2,304384,0.1085039712],[2,304389,0.0296663823],[2,304391,0.027447769],[2,304393,0.03704745],[2,304397,0.5077084057],[2,304406,0.0304357984],[2,304414,0.0890756294],[2,304427,0.0303527634],[2,304433,0.0357247064],[2,304440,0.0836326475],[2,304453,0.0301998308],[2,304468,0.0892073433],[2,304482,0.0478706781],[2,304485,0.0429124328],[2,304525,0.1218677575],[2,304534,0.2574286609],[2,304548,0.0424275022],[2,304554,0.0383337325],[2,304570,0.1768553272],[2,304576,0.0310626978],[2,304580,0.1168769762],[2,304602,0.0409418082],[2,304621,0.0337668659],[2,304645,0.0595988383],[2,304909,0.0398062093],[2,304921,0.0391366937],[2,304926,0.0336886784],[2,304927,0.0353898483],[2,304932,0.3490225583],[2,304945,0.0391366937],[2,304958,0.0316238352],[2,304966,0.210129331],[2,304977,0.0394241184],[2,304988,0.0344997499],[2,304992,0.0303870887],[2,304993,0.0367098407],[2,304994,0.0733209326],[2,304997,0.0399958825],[2,305000,0.1433401475],[2,305014,0.0409418082],[2,305026,0.0321592099],[2,305047,0.1377898179],[2,305050,0.0615585785],[2,305243,0.0541110407],[2,305286,0.036037546],[2,305387,0.0330390907],[2,305392,0.0328511662],[2,305395,0.0541110407],[2,305398,0.0391366937],[2,305408,0.0450437136],[2,305412,0.0424702763],[2,305413,0.0464643164],[2,305420,0.0399958825],[2,305431,0.6822910578],[2,305436,0.0299588772],[2,305437,0.0421195608],[2,305442,0.0325066642],[2,305454,0.0350826494],[2,305458,0.0391366937],[2,305460,0.0325066642],[2,305469,0.0464774862],[2,305473,0.2042033818],[2,305477,0.037957683],[2,305489,0.1687535774],[2,305509,0.0457964396],[2,305510,0.0696373046],[2,305513,0.6527875473],[2,305519,0.0323188735],[2,305524,0.2748697567],[2,305534,0.2075527],[2,305544,0.0379346877],[2,305546,0.2650798241],[2,305548,0.0297177583],[2,305550,0.3751837265],[2,305552,0.0452475605],[2,305558,0.0353898483],[2,305564,0.0920753466],[2,305569,0.089156698],[2,305570,0.1135254526],[2,305586,0.4388165979],[2,305588,0.0413522033],[2,305589,0.0432401454],[2,305591,0.1180415817],[2,305593,0.0588503796],[2,305596,0.2609639358],[2,305599,0.0391366937],[2,305602,0.0479585005],[2,305607,0.2008504802],[2,305612,0.0306604592],[2,305616,0.0541110407],[2,305618,0.3971062781],[2,305626,0.035569889],[2,305628,0.0304908581],[2,305635,0.4261177871],[2,305636,0.0391366937],[2,305641,0.0546733199],[2,305650,0.0328511662],[2,305654,0.0851238968],[2,305656,0.032922314],[2,305657,0.0426209013],[2,305661,0.0330390907],[2,305670,0.0540161917],[2,305680,0.2933241757],[2,305687,0.1124124572],[2,305698,0.0391366937],[2,305704,0.3439747177],[2,305705,0.4537417571],[2,305713,0.0321592099],[2,305715,0.0325066642],[2,305719,0.0988729315],[2,305724,0.0453638404],[2,305726,0.0353031574],[2,305728,0.059188323],[2,305736,0.107381332],[2,305738,0.1610803734],[2,305741,0.0323469604],[2,305755,0.0325066642],[2,306285,0.2272782289],[2,306294,0.2717914104],[2,306296,0.602155211],[2,306299,0.0383639694],[2,306306,0.0752195635],[2,306314,0.0330390907],[2,306319,0.1533344873],[2,306330,0.0296974136],[2,306335,0.2788838262],[2,306358,0.0418903238],[2,306365,0.0389527847],[2,306368,0.0628683535],[2,306375,0.1934414053],[2,306398,0.0577362621],[2,306399,0.0344997499],[2,306403,0.0391366937],[2,306422,0.3882997825],[2,306450,0.0519306356],[2,306455,0.1544971879],[2,306464,0.0441904782],[2,306468,0.0391366937],[2,306472,0.0399958825],[2,306478,0.0474506646],[2,306485,0.0399958825],[2,306487,0.0306781892],[2,306491,0.0391366937],[2,306494,0.0407518972],[2,306496,0.055734275],[2,306512,0.083925816],[2,306523,0.1319886025],[2,306573,0.0295103291],[2,306580,0.0608729371],[2,306585,0.0398062093],[2,306587,0.0999391666],[2,306588,0.0391366937],[2,306593,0.0391366937],[2,306616,0.040601836],[2,306632,0.0324250077],[2,306638,0.0468836313],[2,306643,0.0949107768],[2,306644,0.0391366937],[2,306651,0.0426209013],[2,306671,0.0325066642],[2,306680,0.1996929694],[2,306683,0.084104208],[2,306688,0.0727177122],[2,306693,0.0535627223],[2,306694,0.0391366937],[2,306702,0.0303118597],[2,306704,0.0433827005],[2,306707,0.0286556533],[2,306708,0.0544734423],[2,306709,0.1537979038],[2,306715,0.1598256738],[2,306727,0.1109361564],[2,306733,0.05639533],[2,306738,0.0744829261],[2,306748,0.1752872533],[2,306753,0.7881389068],[2,306781,0.0377188821],[2,306783,0.0324837765],[2,306784,0.0323188735],[2,306792,0.0462628046],[2,306793,0.2870424866],[2,306798,0.0391366937],[2,306803,0.0556983177],[2,306822,0.0409418082],[2,306824,0.0391366937],[2,306825,0.2309043227],[2,306827,0.0334235068],[2,306833,0.2548045991],[2,306840,0.0390254457],[2,306842,0.0391366937],[2,306843,0.2574475158],[2,306846,0.1744922347],[2,306848,0.0391366937],[2,306856,0.0317101007],[2,306860,0.3014380349],[2,306868,0.0726610538],[2,306875,0.0479477258],[2,306890,0.0349389829],[2,306908,0.0481875554],[2,306912,0.0835259138],[2,306919,0.0565133073],[2,306936,0.0313790041],[2,306946,0.1149725512],[2,306948,0.0346538914],[2,306951,0.049966966],[2,306972,0.0297751841],[2,306974,0.0403756145],[2,306981,0.0405065893],[2,306989,0.0411428771],[2,306993,0.2128498848],[2,307004,0.0970096273],[2,307009,0.1172853634],[2,307011,0.039646819],[2,307017,0.0632022959],[2,307052,0.0494901097],[2,307072,0.0327155808],[2,307076,0.0599473602],[2,307091,0.0310739154],[2,307108,0.0468131305],[2,307110,0.266593653],[2,307113,0.0325915302],[2,307118,0.039646819],[2,307119,0.3220099277],[2,307135,0.0788752297],[2,307137,0.0326115015],[2,307142,0.0325847282],[2,307148,0.178541076],[2,307150,0.1776283936],[2,307155,0.0325847282],[2,307156,0.1042641656],[2,307160,0.0552021074],[2,307165,0.225512097],[2,307167,0.0405065893],[2,307170,0.0369383728],[2,307171,0.0956871842],[2,307172,0.0290847581],[2,307182,0.2238247261],[2,307190,0.2346798179],[2,307194,0.033993106],[2,307205,0.0326892902],[2,307217,0.0793092714],[2,307219,0.0332925431],[2,307228,0.0843968779],[2,307231,0.0409418082],[2,307235,0.0319961612],[2,307239,0.3226673064],[2,307240,0.0414531551],[2,307262,0.0809652204],[2,307268,0.1807231373],[2,307272,0.0642800877],[2,307274,0.0419919576],[2,307292,0.0290847581],[2,307293,0.1384497855],[2,307298,0.1488651035],[2,307301,0.0499117896],[2,307310,0.2553831947],[2,307319,0.0310739154],[2,307320,0.0331172103],[2,307324,0.0335417051],[2,307340,0.1022503923],[2,307342,0.0324236844],[2,307355,0.0392652478],[2,307371,0.0318771303],[2,307374,0.0989884266],[2,307376,0.1272017747],[2,307383,0.0307560621],[2,307385,0.033811631],[2,307401,0.0324236844],[2,307403,0.039646819],[2,307408,0.3283440442],[2,307418,0.0454336294],[2,307423,0.039646819],[2,307429,0.0391366937],[2,307434,0.0320793794],[2,307438,0.2398887605],[2,307439,0.039646819],[2,307451,0.095903126],[2,307454,0.0427599469],[2,307456,0.134112723],[2,307459,0.1370601569],[2,307467,0.0622025102],[2,307482,0.0325847282],[2,307496,0.0351274323],[2,307498,0.0945916341],[2,307509,0.0403473647],[2,307513,0.0326115015],[2,307516,0.0399345688],[2,307518,0.1538774257],[2,307524,0.2351874252],[2,307526,0.1804006479],[2,307528,0.334840969],[2,307536,0.0297738654],[2,307550,0.0369779058],[2,307576,0.0427271579],[2,307585,0.1280536772],[2,307591,0.0307560621],[2,307593,0.0325847282],[2,307602,0.0872657751],[2,307604,0.039646819],[2,307610,0.039646819],[2,307621,0.0307560621],[2,307628,0.1154954104],[2,307641,0.0318114511],[2,307647,0.0325847282],[2,307652,0.0339041761],[2,307655,0.0496645779],[2,307660,0.0297738654],[2,307683,0.0544504374],[2,307687,0.039646819],[2,307691,0.1000223323],[2,307695,0.04165261],[2,307699,0.0318916962],[2,307702,0.0315866906],[2,307715,0.0290847581],[2,307724,0.0403356624],[2,307730,0.0290847581],[2,307740,0.2520815392],[2,307769,0.0404661722],[2,307779,0.2131753703],[2,307800,0.0325847282],[2,307809,0.0396974835],[2,307811,0.0325847282],[2,307821,0.302789261],[2,307827,0.230786104],[2,307834,0.1705459966],[2,307865,0.1511197103],[2,307866,0.039646819],[2,307891,0.0290847581],[2,307893,0.0504858419],[2,307918,0.0347662116],[2,307925,0.0346008047],[2,307963,0.3752174896],[2,307966,0.0317836796],[2,307984,0.0373279214],[2,307992,0.0769746107],[2,307993,0.1097444838],[2,308017,0.0852590657],[2,308019,0.1538588189],[2,308023,0.0350029586],[2,308033,0.0399652002],[2,308047,0.1287541916],[2,308048,0.1361037304],[2,308062,0.2303693466],[2,308067,0.038280108],[2,308068,0.1007041284],[2,308072,0.0566459897],[2,308073,0.0392233316],[2,308076,0.0388950839],[2,308077,0.0409649501],[2,308092,0.0461016005],[2,308093,0.0389790711],[2,308094,0.1010764319],[2,308112,0.0469687289],[2,308117,0.0721527103],[2,308120,0.0399373373],[2,308126,0.0389790711],[2,308134,0.0461016005],[2,308145,0.0845320406],[2,308148,0.0474076725],[2,308163,0.0389790711],[2,308177,0.1816003978],[2,308179,0.0388181286],[2,308180,0.0359567771],[2,308185,0.0461016005],[2,308189,0.0591415057],[2,308199,0.1162979479],[2,308205,0.0580764164],[2,308227,0.0389790711],[2,308229,0.0574229367],[2,308230,0.1074584317],[2,308231,0.0601833379],[3,24,0.097082613],[3,31,0.0211943977],[3,32,0.1194749972],[3,42,0.1297162927],[3,44,0.0263660206],[3,45,0.0911717786],[3,46,0.2204868156],[3,52,0.3341148371],[3,56,0.2727488803],[3,58,0.5659572969],[3,63,0.9605982416],[3,64,0.3919680279],[3,66,0.5172804227],[3,69,0.2453458243],[3,70,0.2771087678],[3,80,0.3305578375],[3,81,0.0655564142],[3,83,0.1373042045],[3,87,0.4932527966],[3,90,0.2384040352],[3,93,0.7884497132],[3,95,0.313278927],[3,96,0.1793753011],[3,99,0.2421558848],[3,104,0.113077968],[3,105,0.0462073409],[3,108,0.3384335784],[3,114,0.2431393911],[3,115,0.1963493514],[3,117,0.6584312673],[3,118,0.6428583479],[3,119,0.3263495139],[3,122,0.0788353953],[3,129,0.503799976],[3,133,0.2613081826],[3,137,0.1766629092],[3,138,0.8919112872],[3,142,0.1111863079],[3,143,0.232914976],[3,144,0.2506047197],[3,146,0.085931686],[3,152,0.0241750935],[3,155,0.881880525],[3,157,0.448334619],[3,159,0.0938825923],[3,160,0.0567317685],[3,162,0.8546732659],[3,163,0.1667203755],[3,165,0.288639717],[3,167,0.2799558959],[3,174,0.0263660206],[3,175,0.7813789356],[3,178,0.1448415379],[3,179,0.2001988407],[3,183,0.2298930067],[3,184,0.1851019357],[3,186,0.2915941964],[3,188,0.5533487571],[3,191,0.7054197201],[3,194,0.142429847],[3,195,0.2192385937],[3,202,0.4665249284],[3,211,0.0583975572],[3,212,0.6765186319],[3,215,0.0810013125],[3,216,0.3281203781],[3,221,0.64429701],[3,222,0.0576350613],[3,223,0.0717672255],[3,227,0.5302615281],[3,236,0.786462595],[3,237,0.2107525542],[3,238,0.0772463555],[3,242,0.2339583295],[3,243,0.141662043],[3,245,0.0263150537],[3,247,0.3380589724],[3,248,0.7702841965],[3,257,0.1557082484],[3,259,0.6339742388],[3,261,0.4125799003],[3,263,0.0458649679],[3,265,0.0835701177],[3,268,0.2467708962],[3,270,0.1074402599],[3,271,0.1180250706],[3,274,0.1355282237],[3,277,0.2227574275],[3,282,0.4868624107],[3,287,0.3249088183],[3,296,0.077134109],[3,297,0.1044339167],[3,299,0.7047898708],[3,300,0.1378625748],[3,307,0.1818398814],[3,309,0.4093439343],[3,311,0.2998686138],[3,314,0.2167812507],[3,323,0.2253253246],[3,325,0.2318787875],[3,326,0.0958397914],[3,327,0.3780462834],[3,329,0.109848933],[3,333,0.1955609719],[3,334,0.4287430529],[3,337,0.3249161348],[3,340,0.2993565144],[3,344,0.1726390613],[3,347,0.2384387709],[3,355,0.2480763228],[3,358,0.5728895761],[3,359,0.4531868177],[3,363,0.2134342658],[3,369,0.2015253066],[3,373,0.3760716768],[3,377,0.0208072284],[3,387,0.1769486085],[3,389,0.1752240271],[3,396,0.2917950438],[3,399,0.2876319436],[3,400,0.2044838879],[3,401,0.4479582175],[3,404,0.0823470576],[3,405,0.0923297842],[3,408,0.3686234653],[3,410,0.3947882559],[3,412,0.5935982934],[3,415,0.6052449711],[3,417,0.0817117198],[3,424,0.1006111198],[3,425,0.5780762228],[3,427,0.370601113],[3,433,0.3423883317],[3,434,0.5724607062],[3,438,0.3691840645],[3,441,0.3804711542],[3,448,0.4444701139],[3,453,0.214796476],[3,458,0.1488622375],[3,473,0.9520491686],[3,477,0.4195697404],[3,478,0.3063491716],[3,479,0.3439338303],[3,480,0.1574994219],[3,481,0.1260044133],[3,486,0.1725587956],[3,487,0.0714208558],[3,492,0.1969953704],[3,498,0.1394002902],[3,499,0.1763871587],[3,506,0.1002064766],[3,509,0.1490405438],[3,512,0.1358858018],[3,514,0.1868826738],[3,517,0.1120122401],[3,524,0.2378636026],[3,527,0.519810262],[3,529,0.06687703],[3,536,0.2908509889],[3,544,0.3825661285],[3,546,0.3588091236],[3,548,0.2829851944],[3,549,0.2840252976],[3,551,0.2064432463],[3,554,0.0252974434],[3,557,0.3622443053],[3,559,0.0734004203],[3,560,0.2803903623],[3,562,0.1570336346],[3,563,0.3322841793],[3,570,0.412470208],[3,571,0.2373713099],[3,572,0.4377071161],[3,576,0.4827456744],[3,578,0.0681357503],[3,579,0.7736774751],[3,580,0.3130676677],[3,582,0.1877079506],[3,584,0.6987930238],[3,593,0.4934769616],[3,595,0.1826044289],[3,599,0.0515635907],[3,601,0.2879789599],[3,606,0.2816386414],[3,607,0.196551165],[3,609,0.2298615016],[3,610,0.9640385229],[3,612,0.1214189604],[3,615,0.0404141656],[3,628,0.1907174502],[3,640,0.1835194823],[3,641,0.3647166584],[3,645,0.0989663515],[3,650,0.2253602929],[3,663,0.1578154703],[3,668,0.1267231197],[3,669,0.873438702],[3,670,0.8397092344],[3,675,0.0474066306],[3,681,0.1582211263],[3,695,0.0233637093],[3,699,0.6636637482],[3,701,0.1943271106],[3,704,0.1424800816],[3,705,0.607499102],[3,712,0.3179738157],[3,722,0.083318726],[3,733,0.2006648228],[3,739,0.0590226473],[3,743,0.2766777259],[3,749,0.1491948279],[3,750,0.1833500411],[3,768,0.2306611821],[3,770,0.8491474106],[3,779,0.2352648211],[3,784,0.1012373888],[3,829,0.2805090249],[3,834,0.0517887656],[3,840,0.1336728201],[3,847,0.2631691349],[3,848,0.1158142726],[3,849,0.0629694996],[3,850,0.1896247694],[3,854,0.3903079003],[3,886,0.1483232612],[3,895,0.1587738353],[3,905,0.1439271899],[3,906,0.1539573449],[3,912,0.3169490605],[3,914,0.0809993332],[3,918,0.0263660206],[3,923,0.5017246741],[3,925,0.1563681923],[3,937,0.2485664907],[3,964,0.026690536],[3,966,0.4776005026],[3,973,0.0832089097],[3,981,0.0774410119],[3,1036,0.6295571611],[3,1054,0.1886762667],[3,1056,0.1339496361],[3,1059,0.4144542292],[3,1063,0.2052673621],[3,1073,0.1040999445],[3,1080,0.4272142831],[3,1107,0.3261425659],[3,1114,0.0241489236],[3,1134,0.0918409513],[3,1150,0.1075698444],[3,1158,0.2682315222],[3,1181,0.081290833],[3,1192,0.5590927311],[3,1197,0.2401135128],[3,1202,0.1021594133],[3,1218,0.4536539257],[3,1232,0.2344450492],[3,1257,0.2706805537],[3,1262,0.2460006684],[3,1291,0.2483438546],[3,1309,0.1018372848],[3,1330,0.1614715941],[3,1335,0.0612670468],[3,1349,0.0270808157],[3,1356,0.5983123774],[3,1360,0.2138726868],[3,1366,0.189627269],[3,1380,0.1258485025],[3,1418,0.3149460736],[3,1421,0.3206027659],[3,1429,0.3149252901],[3,1440,0.2350318145],[3,1456,0.1740740221],[3,1465,0.0217234941],[3,1477,0.360556758],[3,1502,0.1595339014],[3,1513,0.5121801831],[3,1526,0.0906817946],[3,1529,0.201639259],[3,1534,0.2777374219],[3,1535,0.4748799859],[3,1542,0.2943849386],[3,1544,0.1991350628],[3,1559,0.1103975623],[3,1579,0.4438302495],[3,1597,0.112782232],[3,1603,0.1609800471],[3,1611,0.3152899752],[3,1613,0.1752592871],[3,1629,0.2938368509],[3,1631,0.3001442177],[3,1636,0.1304885935],[3,1643,0.22027164],[3,1648,0.6308605623],[3,1662,0.5652800292],[3,1663,0.4435606749],[3,1685,0.3745222593],[3,1687,0.277469904],[3,1707,0.6176454183],[3,1716,0.194424964],[3,1720,0.2296052079],[3,1722,0.6334316678],[3,1746,0.2912043266],[3,1749,0.15526958],[3,1759,0.0883838247],[3,1762,0.10175016],[3,1769,0.5865945216],[3,1778,0.1257188639],[3,1780,0.1894302135],[3,1782,0.4812884567],[3,1805,0.6207377273],[3,1823,0.7161012803],[3,1865,0.2971728585],[3,1915,0.1459683973],[3,1935,0.331789012],[3,1938,0.3666041103],[3,1984,0.3990473694],[3,1991,0.3346636661],[3,1995,0.17123333],[3,2023,0.3162636765],[3,2033,0.9229801304],[3,2066,0.1472007195],[3,2072,0.225351367],[3,2104,0.0813061969],[3,2116,0.1106430815],[3,2124,0.1002549322],[3,2131,0.3476236712],[3,2144,0.32694729],[3,2158,0.3124442339],[3,2161,0.2184794412],[3,2170,0.2637024161],[3,2200,0.1537127467],[3,2202,0.1786496992],[3,2307,0.3636772974],[3,2316,0.3785305103],[3,2321,0.1081731506],[3,2331,0.2146673179],[3,2382,0.3816203682],[3,2385,0.1722411124],[3,2387,0.459934895],[3,2424,0.3883234161],[3,2436,0.0989298971],[3,2486,0.8998524542],[3,2493,0.055662135],[3,2524,0.0165575539],[3,2536,0.2060556686],[3,2552,0.1422170132],[3,2595,0.392759028],[3,2610,0.3201722708],[3,2612,0.6433071018],[3,2614,0.3551478019],[3,2615,0.1802601292],[3,2618,0.1732159217],[3,2620,0.103797248],[3,2621,0.0201824962],[3,2622,0.3849471857],[3,2624,0.2628663492],[3,2626,0.3056160871],[3,2627,0.1390335526],[3,2628,0.0557586089],[3,2629,0.4564120947],[3,2633,0.3314605798],[3,2636,0.2511136022],[3,2640,0.1093667954],[3,2642,0.0965289161],[3,2644,0.1932773344],[3,2654,0.0277563746],[3,2655,0.1816799317],[3,2660,0.1862729572],[3,2663,0.0337486197],[3,2667,0.810544456],[3,2674,0.1269467665],[3,2677,0.4904741165],[3,2680,0.0840892216],[3,2687,0.2646295174],[3,2693,0.1097917203],[3,2700,0.0441575087],[3,2703,0.449047353],[3,2705,0.0371756067],[3,2720,0.4099453183],[3,2725,0.2767405946],[3,2731,0.3686600849],[3,2733,0.0426775311],[3,2735,0.0224790967],[3,2736,0.4648315796],[3,2750,0.8761297381],[3,2751,0.020272267],[3,2757,0.2326734605],[3,2761,0.6737002525],[3,2765,0.0198544284],[3,2766,0.6680701993],[3,2771,0.3355388553],[3,2773,0.0313233494],[3,2774,0.2989380924],[3,2775,0.5195380349],[3,2783,0.6344978495],[3,2785,0.0181973626],[3,2792,0.2634817577],[3,2797,0.369609796],[3,2800,0.3227874349],[3,2823,0.1958432453],[3,2824,0.3059616824],[3,2827,0.4375468546],[3,2829,0.1697514899],[3,2836,0.0525416149],[3,2839,0.0826534173],[3,2840,0.0771246723],[3,2843,0.0850635217],[3,2844,0.0181973626],[3,2865,0.1327496987],[3,2866,0.0756101516],[3,2869,0.6284790354],[3,2870,0.1825066314],[3,2878,0.1902243915],[3,2880,0.1193809557],[3,2881,0.1843740689],[3,2883,0.1254496481],[3,2884,0.6818592836],[3,2888,0.2048614417],[3,2889,0.2018043309],[3,2899,0.1861408431],[3,2901,0.1960445725],[3,2902,0.1372063033],[3,2903,0.5802609648],[3,2904,0.2996690718],[3,2905,0.1580258767],[3,2907,0.6636635727],[3,2911,0.1042119201],[3,2912,0.2239140064],[3,2920,0.0173356988],[3,2921,0.1689213856],[3,2925,0.3069650125],[3,2926,0.1372601914],[3,2928,0.1183427076],[3,2929,0.4424564792],[3,2930,0.2450489379],[3,2931,0.3024954119],[3,2932,0.2270039261],[3,2933,0.3151052642],[3,2935,0.240195588],[3,2936,0.1173057532],[3,2937,0.177853457],[3,2939,0.2471175681],[3,2940,0.1630504933],[3,2941,0.3410530708],[3,2942,0.072488628],[3,2943,0.2750527845],[3,2944,0.2099043385],[3,2946,0.0721081697],[3,2949,0.0652179969],[3,2951,0.2082440699],[3,2952,0.2957541191],[3,2954,0.2283931454],[3,2956,0.0816157173],[3,2957,0.8007820847],[3,2959,0.2216184735],[3,2960,0.203239877],[3,2964,0.0198544284],[3,2965,0.1495084398],[3,2966,0.0774494586],[3,2967,0.1231342258],[3,2970,0.3828120085],[3,2971,0.0387947017],[3,2972,0.3160183784],[3,2974,0.0860996358],[3,2975,0.1890513903],[3,2976,0.1880751014],[3,2978,0.1554926152],[3,2979,0.099893724],[3,2981,0.3872076462],[3,2982,0.1551196592],[3,2983,0.1963646848],[3,2985,0.3220738602],[3,2986,0.3507508878],[3,2987,0.2987859444],[3,2988,0.4088810302],[3,2990,0.1919149311],[3,2991,0.4412341104],[3,2992,0.1896202606],[3,2994,0.2475174322],[3,2996,0.0765554573],[3,2998,0.1221785979],[3,2999,0.1457148868],[3,3003,0.2519459334],[3,3004,0.6458071555],[3,3005,0.2336798318],[3,3006,0.1544250036],[3,3008,0.1915118018],[3,3009,0.2014348903],[3,3010,0.2789012261],[3,3013,0.0334545009],[3,3015,0.360880058],[3,3016,0.1400439572],[3,3017,0.3403990858],[3,3018,0.1233840954],[3,3019,0.1779431281],[3,3020,0.343114831],[3,3021,0.1832915843],[3,3022,0.1458450582],[3,3025,0.2215682178],[3,3026,0.140030146],[3,3027,0.1940709723],[3,3029,0.1497761647],[3,3031,0.2298698578],[3,3033,0.1013728025],[3,3034,0.1423912677],[3,3035,0.0777828623],[3,3038,0.1432411938],[3,3043,0.1788454521],[3,3044,0.5505039795],[3,3047,0.2888391416],[3,3049,0.2395616711],[3,3050,0.2125453514],[3,3054,0.2499286139],[3,3056,0.308065827],[3,3058,0.1103009756],[3,3059,0.1788634408],[3,3060,0.1812428515],[3,3062,0.2007505695],[3,3069,0.1808469022],[3,3070,0.2146955209],[3,3071,0.1961499883],[3,3072,0.0573481568],[3,3073,0.1989426767],[3,3076,0.5343472893],[3,3078,0.1576248487],[3,3079,0.43634071],[3,3080,0.5822955591],[3,3082,0.2750770736],[3,3083,0.1552433755],[3,3084,0.1897464557],[3,3085,0.1884561712],[3,3087,0.1810320863],[3,3090,0.2094437336],[3,3091,0.1399984815],[3,3092,0.2252228201],[3,3093,0.2634178958],[3,3095,0.2159519031],[3,3096,0.1036344001],[3,3097,0.1162359861],[3,3098,0.1958633306],[3,3101,0.1465860288],[3,3104,0.2526011913],[3,3109,0.227049833],[3,3110,0.0928924735],[3,3111,0.192184034],[3,3112,0.3757683317],[3,3113,0.38569615],[3,3118,0.143304927],[3,3120,0.4211895543],[3,3123,0.2744414677],[3,3126,0.1716191994],[3,3128,0.28053116],[3,3132,0.5031397817],[3,3133,0.2259299913],[3,3134,0.5342388864],[3,3135,0.1550040267],[3,3136,0.1294479954],[3,3138,0.1693089734],[3,3139,0.1552853079],[3,3140,0.1921710172],[3,3142,0.1712473482],[3,3143,0.9061504172],[3,3144,0.2312337804],[3,3145,0.1355068745],[3,3153,0.2988924733],[3,3154,0.6830432258],[3,3155,0.2210056186],[3,3156,0.1947923506],[3,3157,0.1597928218],[3,3158,0.1865789811],[3,3159,0.156062361],[3,3160,0.1552279301],[3,3161,0.1870837783],[3,3163,0.2009594196],[3,3164,0.2436863205],[3,3168,0.2496794007],[3,3169,0.1323732493],[3,3170,0.2992603181],[3,3172,0.404761006],[3,3175,0.1871220367],[3,3176,0.2119354027],[3,3179,0.5045659275],[3,3180,0.1526451005],[3,3181,0.2421479942],[3,3182,0.2502268321],[3,3186,0.4681237867],[3,3187,0.0610893502],[3,3188,0.3654606304],[3,3191,0.7990595583],[3,3193,0.0449194637],[3,3194,0.0589696273],[3,3196,0.2182142537],[3,3197,0.0675447458],[3,3199,0.3232867165],[3,3206,0.1062541421],[3,3207,0.1847496991],[3,3208,0.2562881186],[3,3209,0.5395234552],[3,3210,0.1418918305],[3,3211,0.292232067],[3,3214,0.3624213463],[3,3215,0.068366603],[3,3216,0.0775306714],[3,3217,0.5154496559],[3,3221,0.4914292901],[3,3222,0.0476137322],[3,3223,0.209612609],[3,3224,0.4705822029],[3,3227,0.0434118767],[3,3228,0.1195688021],[3,3231,0.3023484608],[3,3239,0.2739840435],[3,3241,0.2286238832],[3,3243,0.1815879516],[3,3244,0.4676805347],[3,3247,0.6926299208],[3,3248,0.2117151572],[3,3250,0.1869959596],[3,3251,0.1133313967],[3,3253,0.193384475],[3,3255,0.2062772003],[3,3256,0.0771173154],[3,3262,0.2030596186],[3,3264,0.2845636566],[3,3265,0.4502515309],[3,3266,0.1655480147],[3,3267,0.163494594],[3,3268,0.1004285127],[3,3271,0.3255936299],[3,3272,0.3355249431],[3,3274,0.1649817678],[3,3275,0.6833073525],[3,3276,0.1075478649],[3,3277,0.2981551443],[3,3278,0.2198223188],[3,3281,0.1364096211],[3,3283,0.0352260615],[3,3284,0.1296337352],[3,3285,0.4052191018],[3,3286,0.2278459133],[3,3287,0.045091298],[3,3288,0.4421824431],[3,3290,0.1846871477],[3,3293,0.1100752426],[3,3294,0.3976614875],[3,3296,0.0443712236],[3,3298,0.1408190304],[3,3299,0.1124501121],[3,3305,0.1305651374],[3,3307,0.2063948528],[3,3308,0.5315724907],[3,3311,0.1610929631],[3,3314,0.6323582323],[3,3316,0.1271512185],[3,3318,0.1785027422],[3,3319,0.2109097961],[3,3320,0.3404590901],[3,3322,0.1503162309],[3,3323,0.4285094589],[3,3324,0.5009131112],[3,3325,0.3401671462],[3,3326,0.3385003284],[3,3327,0.1316868965],[3,3328,0.1869802664],[3,3332,0.0195328348],[3,3334,0.4808210862],[3,3335,0.8849918409],[3,3337,0.8985778858],[3,3340,0.3088705022],[3,3341,0.3046778151],[3,3347,0.053277387],[3,3350,0.4061017394],[3,3352,0.2330950894],[3,3354,0.0961510911],[3,3355,0.1268966711],[3,3356,0.2325140386],[3,3359,0.1829427338],[3,3360,0.377575325],[3,3362,0.4562955214],[3,3366,0.1280542035],[3,3368,0.3632086866],[3,3369,0.6231270305],[3,3373,0.1841887736],[3,3374,0.3029784763],[3,3376,0.0508769001],[3,3377,0.1059561077],[3,3378,0.109134347],[3,3379,0.3672909779],[3,3381,0.0565165579],[3,3382,0.4588405804],[3,3383,0.2608327481],[3,3384,0.5198515376],[3,3385,0.1601712682],[3,3387,0.2250189624],[3,3389,0.0611104411],[3,3391,0.3729585631],[3,3392,0.2158504433],[3,3394,0.0453125721],[3,3395,0.170786839],[3,3398,0.4801300662],[3,3400,0.1596384025],[3,3401,0.3208314286],[3,3403,0.426450548],[3,3404,0.2704266346],[3,3405,0.192465768],[3,3406,0.0716746604],[3,3407,0.2427261985],[3,3412,0.1226028399],[3,3415,0.813880244],[3,3423,0.262664372],[3,3426,0.4986725659],[3,3427,0.238823583],[3,3432,0.5640600298],[3,3444,0.5153302422],[3,3445,0.3141155732],[3,3446,0.4711785532],[3,3449,0.126472494],[3,3450,0.1223565874],[3,3452,0.0893086481],[3,3457,0.2383163273],[3,3458,0.4062611298],[3,3461,0.2567486358],[3,3463,0.1805318299],[3,3464,0.062693732],[3,3465,0.3728647906],[3,3466,0.1631586072],[3,3467,0.1757612404],[3,3470,0.0430092069],[3,3473,0.1595391216],[3,3477,0.1797215772],[3,3483,0.2711972429],[3,3487,0.2456432727],[3,3488,0.3907785387],[3,3489,0.3774910436],[3,3493,0.3627856841],[3,3500,0.2807048747],[3,3506,0.2380279408],[3,3509,0.2421934798],[3,3510,0.3492687638],[3,3514,0.1528943549],[3,3516,0.2451114678],[3,3517,0.0885727724],[3,3525,0.6038397396],[3,3529,0.5670506207],[3,3530,0.1200687872],[3,3531,0.1736086002],[3,3532,0.5633290268],[3,3535,0.3204205937],[3,3540,0.1613702437],[3,3541,0.1941577783],[3,3542,0.1059280809],[3,3547,0.2495174952],[3,3553,0.4508644883],[3,3555,0.4798021002],[3,3556,0.7478077872],[3,3557,0.4769481031],[3,3558,0.4078922962],[3,3571,0.197465003],[3,3572,0.2801250847],[3,3573,0.1745342285],[3,3575,0.1135078787],[3,3578,0.336296915],[3,3579,0.7764297502],[3,3580,0.0993992971],[3,3582,0.22541043],[3,3585,0.0431005642],[3,3586,0.0995084384],[3,3590,0.3396426352],[3,3591,0.2851452098],[3,3593,0.0531993493],[3,3597,0.1499705655],[3,3598,0.1970061219],[3,3600,0.3130192316],[3,3603,0.1052251776],[3,3606,0.3228871581],[3,3609,0.0671552214],[3,3610,0.3384823559],[3,3613,0.488610612],[3,3623,0.164202021],[3,3624,0.4905796398],[3,3626,0.2573819057],[3,3631,0.172098777],[3,3632,0.0453125721],[3,3637,0.1496295492],[3,3642,0.1451180008],[3,3645,0.0511625239],[3,3647,0.0632140869],[3,3649,0.1860829934],[3,3654,0.1282440199],[3,3656,0.2307636401],[3,3657,0.3942919969],[3,3662,0.1471393755],[3,3664,0.5069190052],[3,3666,0.1922849236],[3,3668,0.2297329006],[3,3670,0.177918513],[3,3671,0.1503180162],[3,3676,0.1299848619],[3,3677,0.42593022],[3,3678,0.0970932533],[3,3682,0.1793793212],[3,3684,0.1242543615],[3,3685,0.3632309891],[3,3686,0.168346181],[3,3687,0.1717265647],[3,3689,0.2491629345],[3,3691,0.5491214416],[3,3695,0.37423619],[3,3696,0.1994600414],[3,3700,0.1214040128],[3,3701,0.3799436664],[3,3702,0.952245899],[3,3703,0.2296787994],[3,3708,0.1677933522],[3,3709,0.327850997],[3,3712,0.1950190006],[3,3718,0.2311214401],[3,3731,0.1897083277],[3,3734,0.2198824785],[3,3738,0.092824981],[3,3741,0.1841993528],[3,3743,0.1847206628],[3,3744,0.6975472043],[3,3746,0.0414538905],[3,3750,0.075465285],[3,3761,0.1675704103],[3,3766,0.197671722],[3,3767,0.1471977422],[3,3779,0.2034617804],[3,3782,0.1609765544],[3,3791,0.328410536],[3,3797,0.1624975144],[3,3801,0.5042010793],[3,3802,0.1234676365],[3,3806,0.3935605943],[3,3808,0.1450476293],[3,3813,0.1165433524],[3,3819,0.0656177565],[3,3820,0.1766951699],[3,3822,0.2804673517],[3,3825,0.7198886318],[3,3833,0.2890905674],[3,3837,0.1174923024],[3,3844,0.1402665703],[3,3853,0.3190427265],[3,3864,0.0991484094],[3,3874,0.3365413372],[3,3879,0.0523008011],[3,3881,0.3810156549],[3,3892,0.0512282817],[3,3907,0.3278968415],[3,3913,0.2203406607],[3,3915,0.06692401],[3,3918,0.2627559954],[3,3925,0.1353343348],[3,3927,0.5654819466],[3,3929,0.585590428],[3,3931,0.2375266531],[3,3937,0.3086661741],[3,3955,0.5601690118],[3,3960,0.1855810092],[3,3963,0.1816876693],[3,3966,0.1892399771],[3,3974,0.2674955887],[3,3986,0.6611281722],[3,3995,0.1494748274],[3,3996,0.2602735643],[3,3999,0.1437096347],[3,4020,0.2764569832],[3,4024,0.1594501053],[3,4028,0.4617999767],[3,4034,0.0694414104],[3,4044,0.109966889],[3,4071,0.326294277],[3,4072,0.3932223768],[3,4085,0.0805700005],[3,4088,0.2898319272],[3,4101,0.3718151054],[3,4105,0.1769059713],[3,4111,0.3605449882],[3,4152,0.2013634356],[3,4155,0.3772788934],[3,4158,0.0687629923],[3,4182,0.4258342438],[3,4184,0.1497983974],[3,4188,0.0891100382],[3,4237,0.2194610555],[3,4242,0.1385320887],[3,4245,0.1213559064],[3,4246,0.0198544284],[3,4253,0.0474063065],[3,4264,0.1537631935],[3,4265,0.2404887852],[3,4272,0.2799389357],[3,4276,0.3261339393],[3,4287,0.1291768978],[3,4288,0.3011948361],[3,4298,0.1295258264],[3,4299,0.2785845683],[3,4317,0.1216769241],[3,4320,0.1438166707],[3,4321,0.2184008601],[3,4322,0.0630682996],[3,4348,0.6619816576],[3,4351,0.0606953117],[3,4365,0.0195328348],[3,4402,0.0347474627],[3,4464,0.0416259415],[3,4465,0.0495750234],[3,4478,0.1552130712],[3,4493,0.2063692652],[3,4500,0.020272267],[3,4505,0.1719777532],[3,4507,0.1823845355],[3,4534,0.3391996857],[3,4540,0.4168656951],[3,4548,0.0806161795],[3,4570,0.0904133008],[3,4596,0.4458266601],[3,4606,0.567864378],[3,4611,0.1336320821],[3,4617,0.3336698597],[3,4689,0.1106238562],[3,4692,0.0195328348],[3,4693,0.0181973626],[3,4699,0.1381775366],[3,4716,0.2711340924],[3,4721,0.0632358115],[3,4725,0.6590812362],[3,4728,0.0979603622],[3,4794,0.1031629075],[3,4799,0.0523663423],[3,4803,0.4899442432],[3,4804,0.1140422899],[3,4808,0.4335449876],[3,4809,0.1257932636],[3,4813,0.6963841934],[3,4814,0.6378553828],[3,4817,0.1495315189],[3,4818,0.1805536421],[3,4829,0.0922236141],[3,4831,0.0917709101],[3,4832,0.3179918828],[3,4834,0.0449224401],[3,4835,0.1660144473],[3,4836,0.2936825072],[3,4837,0.6016181831],[3,4841,0.1049350168],[3,4842,0.4601654715],[3,4843,0.2634798468],[3,4844,0.1207349391],[3,4845,0.136130662],[3,4846,0.1778800445],[3,4847,0.1860571671],[3,4849,0.2005407501],[3,4850,0.4507140482],[3,4851,0.2972490829],[3,4853,0.1323974724],[3,4858,0.1595516729],[3,4860,0.2334128266],[3,4863,0.1133635152],[3,4864,0.1768211216],[3,4867,0.7044555165],[3,4872,0.0456143795],[3,4878,0.1956335994],[3,4881,0.3587970607],[3,4885,0.6449714645],[3,4888,0.211111373],[3,4897,0.365963909],[3,4899,0.1445254246],[3,4900,0.4489588783],[3,4904,0.2516788699],[3,4906,0.2239675814],[3,4907,0.0390029225],[3,4911,0.0578929933],[3,4912,0.2465661568],[3,4918,0.3048309277],[3,4919,0.1317009306],[3,4920,0.1703570503],[3,4921,0.105908273],[3,4926,0.211079595],[3,4927,0.577553435],[3,4930,0.126162961],[3,4932,0.1573410486],[3,4935,0.2136867646],[3,4937,0.699271169],[3,4938,0.0967530261],[3,4941,0.259631583],[3,4960,0.4307725241],[3,4961,0.1647835804],[3,4977,0.1562308915],[3,4978,0.1978052776],[3,4980,0.080296016],[3,4982,0.1773744633],[3,4983,0.1133796337],[3,4985,0.3616708965],[3,4987,0.0408863219],[3,4992,0.2226058758],[3,4994,0.088425655],[3,4998,0.1715197982],[3,4999,0.4659191824],[3,5001,0.3290220644],[3,5004,0.2683943846],[3,5007,0.4682347846],[3,5014,0.6898736832],[3,5015,0.1159867829],[3,5018,0.1471800205],[3,5022,0.4223095299],[3,5024,0.2960076142],[3,5025,0.3406130274],[3,5027,0.402594335],[3,5029,0.2015373477],[3,5033,0.4818083972],[3,5036,0.1957685979],[3,5038,0.1086801571],[3,5042,0.1489474345],[3,5044,0.3600799115],[3,5045,0.1662831229],[3,5049,0.1011155642],[3,5050,0.0844196323],[3,5051,0.2994619858],[3,5052,0.1030022368],[3,5059,0.0924909235],[3,5061,0.0204791021],[3,5065,0.2713641254],[3,5069,0.1490330161],[3,5071,0.1297248115],[3,5117,0.1557598592],[3,5124,0.0530343035],[3,5137,0.303663719],[3,5148,0.5034661339],[3,5149,0.2286487134],[3,5170,0.0736682641],[3,5186,0.3918899016],[3,5205,0.1899171097],[3,5211,0.3204230658],[3,5234,0.110912527],[3,5235,0.0973392985],[3,5262,0.2069671502],[3,5288,0.1154901219],[3,5332,0.0765763288],[3,5339,0.1788540326],[3,5369,0.0704639617],[3,5379,0.3391617527],[3,7870,0.1736514931],[3,7891,0.0611975156],[3,7942,0.1253242048],[3,7948,0.1771887064],[3,7970,0.1924006265],[3,7992,0.0686952224],[3,8008,0.0865466989],[3,8056,0.1411969632],[3,8080,0.048340979],[3,8104,0.030260923],[3,8120,0.7677042311],[3,8253,0.1484423148],[3,8254,0.0416892125],[3,8263,0.1688087952],[3,8266,0.0951346168],[3,8267,0.2649762835],[3,8269,0.0683521883],[3,8274,0.4729993224],[3,8276,0.0962754379],[3,8280,0.1133637126],[3,8282,0.0928676871],[3,8292,0.0906584138],[3,8311,0.2231260122],[3,8314,0.2488816859],[3,8323,0.073129033],[3,8342,0.0354200371],[3,8349,0.1241394081],[3,8351,0.0504820967],[3,8365,0.4197873214],[3,8366,0.0945241608],[3,8374,0.7421734436],[3,8380,0.1868185425],[3,8381,0.0396411747],[3,8392,0.1373806429],[3,8397,0.1164872587],[3,8401,0.1279126512],[3,8403,0.2851740189],[3,8412,0.1052796066],[3,8424,0.1511742053],[3,8428,0.2006328487],[3,8438,0.1104426189],[3,8451,0.4774232911],[3,8452,0.3886640357],[3,8455,0.0363081469],[3,8463,0.0171783409],[3,8465,0.0671777285],[3,8486,0.1164459769],[3,8506,0.0398977314],[3,8594,0.101810683],[3,8618,0.0572090436],[3,8651,0.1380567139],[3,8654,0.0970290133],[3,8660,0.1951825984],[3,8709,0.0467684071],[3,8739,0.0713072215],[3,8745,0.035311534],[3,8747,0.1390371795],[3,8760,0.1290647452],[3,8783,0.0413308657],[3,8821,0.0438000912],[3,8824,0.2240874904],[3,8827,0.0648470273],[3,8927,0.2079240152],[3,8928,0.0773027104],[3,8946,0.0924895804],[3,8949,0.2951369169],[3,8961,0.3565143721],[3,8962,0.3336787739],[3,8974,0.1893915245],[3,8981,0.0526326168],[3,9002,0.0932385646],[3,9005,0.193449159],[3,9035,0.0670606089],[3,9042,0.099998556],[3,9066,0.1708721047],[3,9100,0.3475665451],[3,9119,0.2664631895],[3,9226,0.0358516201],[3,9231,0.1519827361],[3,9236,0.0493170058],[3,9261,0.0940773732],[3,9287,0.070282997],[3,9299,0.2954977243],[3,9304,0.1155020718],[3,9335,0.0975321323],[3,9346,0.5050413972],[3,9474,0.1799502668],[3,9479,0.2155215519],[3,9486,0.2848267366],[3,9489,0.7316013053],[3,9512,0.4272128906],[3,9521,0.1516360075],[3,9527,0.584150649],[3,9528,0.1944840105],[3,9531,0.3153207679],[3,9539,0.0867994165],[3,9541,0.282637487],[3,9542,0.4999189945],[3,9549,0.153838331],[3,9550,0.1870429512],[3,9560,0.063201482],[3,9567,0.0412861119],[3,9575,0.2996022777],[3,9576,0.4550977967],[3,9587,0.1061966236],[3,9612,0.4473354387],[3,9650,0.053419844],[3,9663,0.3119217224],[3,9691,0.1031085815],[3,9716,0.1386088411],[3,9733,0.0968975948],[3,9759,0.2049439823],[3,9804,0.0707711933],[3,9829,0.10807689],[3,9841,0.190933262],[3,9845,0.3730985385],[3,9881,0.0946533106],[3,9993,0.0472636194],[3,10003,0.406194123],[3,10026,0.013478613],[3,10055,0.1897382373],[3,10065,0.0527611465],[3,10091,0.1444908479],[3,10106,0.0723337454],[3,10127,0.2250098441],[3,10147,0.1636951634],[3,10161,0.2413783165],[3,10208,0.2614164352],[3,10220,0.0148363412],[3,10280,0.0321380093],[3,10282,0.0379494229],[3,10286,0.0151208409],[3,10288,0.0147453136],[3,10290,0.0237846376],[3,10297,0.1635725114],[3,10298,0.0393700939],[3,10299,0.0421810247],[3,10305,0.0322758392],[3,10308,0.0953987961],[3,10310,0.0148363412],[3,10313,0.0629348218],[3,10318,0.0520590545],[3,10321,0.2085267064],[3,10324,0.0259824301],[3,10325,0.0750559825],[3,10326,0.012678697],[3,10327,0.0172899914],[3,10341,0.0620049059],[3,10343,0.0659239282],[3,10344,0.0477031124],[3,10345,0.0531344763],[3,10346,0.1814081892],[3,10349,0.0435856598],[3,10353,0.0747415256],[3,10357,0.0552317875],[3,10358,0.0997153218],[3,10362,0.4002327799],[3,10363,0.0627949625],[3,10373,0.0606085157],[3,10376,0.0383138689],[3,10380,0.3968966922],[3,10383,0.015638573],[3,10385,0.0483947539],[3,10386,0.0229652263],[3,10390,0.0153421279],[3,10396,0.0756839753],[3,10402,0.0411896313],[3,10404,0.1572749539],[3,10407,0.0352331138],[3,10408,0.033895467],[3,10423,0.0120840079],[3,10424,0.0430610697],[3,10438,0.0145395074],[3,10440,0.0134528129],[3,10444,0.0437761562],[3,10448,0.0321663842],[3,10451,0.0678447282],[3,10457,0.03418398],[3,10459,0.1507708945],[3,10460,0.0243825524],[3,10462,0.0374283749],[3,10472,0.1016006077],[3,10474,0.0147798458],[3,10478,0.046715401],[3,10480,0.0348239194],[3,10481,0.6365958078],[3,10484,0.0247785761],[3,10500,0.07235641],[3,10502,0.0471489828],[3,10508,0.0611297541],[3,10511,0.0339009965],[3,10515,0.1561640161],[3,10518,0.117325108],[3,10525,0.0649113079],[3,10526,0.2285703497],[3,10529,0.0392518433],[3,10533,0.085169733],[3,10534,0.0645232441],[3,10535,0.0254879611],[3,10552,0.1833901847],[3,10561,0.1802024545],[3,10565,0.0995575134],[3,10567,0.0662660871],[3,10570,0.0609399856],[3,10590,0.0709390196],[3,10591,0.0620049059],[3,10592,0.0712462388],[3,10594,0.1425932006],[3,10600,0.0324278327],[3,10609,0.6025845192],[3,10629,0.0375845588],[3,10630,0.0435653514],[3,10634,0.0120840079],[3,10641,0.0147798458],[3,10655,0.0642183668],[3,10659,0.0886094502],[3,10664,0.0247785761],[3,10677,0.012678697],[3,10732,0.0143642056],[3,10750,0.0153421279],[3,10752,0.0727475954],[3,10754,0.0544631892],[3,10771,0.0099194978],[3,10772,0.0555193256],[3,10775,0.012678697],[3,10776,0.0582376695],[3,10777,0.012678697],[3,10784,0.0189055176],[3,10790,0.0326407781],[3,10800,0.0342466927],[3,10803,0.0552196712],[3,10807,0.0362778193],[3,10816,0.0315360948],[3,10821,0.0153421279],[3,10822,0.0345216481],[3,10831,0.1965285902],[3,10837,0.0607077804],[3,10849,0.0510667747],[3,10850,0.0702673718],[3,10855,0.012678697],[3,10860,0.1442339709],[3,10869,0.3807067275],[3,10889,0.0118794915],[3,10893,0.0286224683],[3,10900,0.0109395105],[3,10913,0.012678697],[3,10916,0.012678697],[3,10920,0.0144644517],[3,10921,0.0683976873],[3,10969,0.0653963342],[3,10974,0.1584931427],[3,10978,0.0607077804],[3,10982,0.0164550796],[3,10993,0.0263646349],[3,10995,0.1726986157],[3,11002,0.0571665078],[3,11015,0.0668112785],[3,11018,0.058808297],[3,11024,0.0471893408],[3,11025,0.0247785761],[3,11030,0.0305668284],[3,11031,0.0775089023],[3,11033,0.0290805293],[3,11034,0.6409828209],[3,11036,0.0143642056],[3,11044,0.0445770709],[3,11045,0.0667274053],[3,11050,0.1348517831],[3,11051,0.1057218044],[3,11067,0.5641732992],[3,11068,0.1049151266],[3,11070,0.3677974264],[3,11072,0.010301072],[3,11084,0.0409924179],[3,11096,0.0682450261],[3,11103,0.0906966022],[3,11114,0.0341502603],[3,11119,0.0183125817],[3,11123,0.0321638383],[3,11124,0.0887594876],[3,11139,0.0339272625],[3,11142,0.0389417221],[3,11143,0.0510908367],[3,11145,0.0591523611],[3,11146,0.0620608511],[3,11160,0.126280105],[3,11162,0.0499323697],[3,11166,0.0144644517],[3,11172,0.0115549552],[3,11174,0.0247785761],[3,11178,0.043233957],[3,11187,0.012678697],[3,11194,0.0211993521],[3,11197,0.0293014601],[3,11203,0.0226370643],[3,11205,0.0164550796],[3,11217,0.0891858147],[3,11218,0.022420761],[3,11231,0.0120840079],[3,11235,0.0464788218],[3,11241,0.5847316329],[3,11254,0.0114520241],[3,11255,0.012678697],[3,11257,0.0923039855],[3,11263,0.0827897498],[3,11266,0.012678697],[3,11267,0.0643446372],[3,11270,0.037179075],[3,11274,0.0299464518],[3,11277,0.0555014467],[3,11279,0.0417846455],[3,11281,0.0619070927],[3,11283,0.0798769424],[3,11286,0.1002401937],[3,11302,0.0900892187],[3,11303,0.0680917191],[3,11305,0.0528520495],[3,11307,0.0133088398],[3,11308,0.0621440931],[3,11310,0.1031851205],[3,11322,0.013408606],[3,11327,0.0702726782],[3,11331,0.1491468472],[3,11333,0.013478613],[3,11336,0.010301072],[3,11342,0.0143642056],[3,11349,0.012678697],[3,11359,0.010301072],[3,11360,0.0932969213],[3,11363,0.1807583642],[3,11366,0.1237946361],[3,11370,0.0425755572],[3,11376,0.0333754617],[3,11378,0.0148363412],[3,11381,0.0247785761],[3,11384,0.0123678602],[3,11385,0.1069886686],[3,11386,0.0987052907],[3,11389,0.1247672872],[3,11394,0.0456810618],[3,11399,0.0105576084],[3,11401,0.1910063018],[3,11402,0.0308975851],[3,11404,0.012678697],[3,11407,0.0989332258],[3,11411,0.0314143342],[3,11414,0.012678697],[3,11415,0.1409959012],[3,11418,0.0704106436],[3,11420,0.0894129608],[3,11428,0.0304233884],[3,11431,0.0515800671],[3,11434,0.0462553371],[3,11438,0.0351066614],[3,11441,0.1868987969],[3,11447,0.03367767],[3,11453,0.4223222942],[3,11455,0.014565566],[3,11461,0.4979220124],[3,11466,0.046111794],[3,11468,0.075117887],[3,11471,0.018962623],[3,11475,0.015638573],[3,11478,0.2810924397],[3,11480,0.2858437716],[3,11484,0.0259310761],[3,11488,0.0144644517],[3,11492,0.0390767166],[3,11495,0.0621535787],[3,11497,0.0099194978],[3,11501,0.012678697],[3,11508,0.0152106261],[3,11509,0.012678697],[3,11521,0.1048333104],[3,11523,0.1245400903],[3,11524,0.1048001447],[3,11525,0.067742927],[3,11530,0.0224826907],[3,11536,0.0301195282],[3,11537,0.0296795434],[3,11541,0.1268977891],[3,11546,0.039336243],[3,11548,0.0295711703],[3,11550,0.1060796813],[3,11553,0.0358421243],[3,11554,0.0261664531],[3,11561,0.1553933786],[3,11562,0.1010205537],[3,11564,0.0144644517],[3,11565,0.0199684313],[3,11566,0.0099194978],[3,11567,0.0134528129],[3,11568,0.0577354942],[3,11569,0.0693623213],[3,11572,0.2867438666],[3,11580,0.0288288226],[3,11585,0.0618377016],[3,11587,0.0123678602],[3,11588,0.11947912],[3,11589,0.0654604186],[3,11590,0.1636767629],[3,11593,0.0322758392],[3,11597,0.0519880609],[3,11606,0.0402900442],[3,11609,0.0910499562],[3,11612,0.0099194978],[3,11632,0.0247785761],[3,11638,0.124911184],[3,11639,0.1029021292],[3,11643,0.0115691351],[3,11646,0.1432038617],[3,11656,0.0360998219],[3,11670,0.043665573],[3,11676,0.2838969828],[3,11678,0.1763952831],[3,11680,0.9953173166],[3,11683,0.0278646656],[3,11691,0.0458801191],[3,11692,0.5006102246],[3,11693,0.0306179602],[3,11695,0.1055226782],[3,11698,0.0411379117],[3,11699,0.0887893798],[3,11706,0.0363263366],[3,11713,0.0145677995],[3,11714,0.5001141062],[3,11716,0.5292480589],[3,11723,0.0620633557],[3,11724,0.0372883635],[3,11730,0.6556545809],[3,11732,0.039818405],[3,11737,0.0515900682],[3,11738,0.1324807669],[3,11741,0.0203888526],[3,11742,0.0278646656],[3,11744,0.07980182],[3,11746,0.0566789222],[3,11747,0.0697991309],[3,11752,0.0123678602],[3,11755,0.1340498882],[3,11756,0.0365170378],[3,11762,0.0133088398],[3,11769,0.022316327],[3,11771,0.012678697],[3,11773,0.0145677995],[3,11775,0.0774262805],[3,11784,0.1555484162],[3,11789,0.1500397746],[3,11798,0.0147367912],[3,11800,0.0386972176],[3,11805,0.0374196711],[3,11807,0.0307364691],[3,11809,0.0493854729],[3,11810,0.1269666198],[3,11821,0.0559223536],[3,11822,0.0099194978],[3,11825,0.0395701531],[3,11829,0.0438463899],[3,11834,0.0172899914],[3,11835,0.0120840079],[3,11840,0.0120840079],[3,11851,0.0482461924],[3,11853,0.012678697],[3,11869,0.2671419429],[3,11870,0.0147798458],[3,11874,0.0844552534],[3,11881,0.1162345812],[3,11889,0.0467362053],[3,11890,0.0644822907],[3,11896,0.1570566884],[3,11898,0.2823254144],[3,11900,0.2383638969],[3,11916,0.2071207273],[3,11917,0.012678697],[3,11924,0.0114096017],[3,11925,0.1135885389],[3,11928,0.1533474145],[3,11931,0.0857246546],[3,11932,0.0230182895],[3,11939,0.012678697],[3,11940,0.1454274585],[3,11941,0.1095364448],[3,11959,0.4996642717],[3,11960,0.0554637948],[3,11964,0.0501428468],[3,11965,0.086739594],[3,11967,0.0709246454],[3,11968,0.031546364],[3,11972,0.0667037512],[3,11978,0.0766796904],[3,11980,0.0655330515],[3,11982,0.0475889805],[3,11983,0.0120840079],[3,11985,0.0352044214],[3,11988,0.1241538426],[3,11993,0.0305180706],[3,11994,0.1363918741],[3,11997,0.0353763833],[3,12000,0.0469208407],[3,12004,0.0112402459],[3,12006,0.0563031944],[3,12010,0.022316327],[3,12014,0.0130370282],[3,12015,0.0120840079],[3,12016,0.0573733991],[3,12021,0.0411095384],[3,12025,0.012678697],[3,12030,0.0254879611],[3,12041,0.3550361031],[3,12042,0.0641912898],[3,12044,0.0402454133],[3,12045,0.0404924713],[3,12047,0.0606236953],[3,12052,0.0465645672],[3,12058,0.0476899518],[3,12059,0.0455483824],[3,12076,0.012678697],[3,12080,0.0340514733],[3,12081,0.015638573],[3,12082,0.0338094774],[3,12084,0.3917606663],[3,12087,0.0287515239],[3,12089,0.0115691351],[3,12091,0.028018397],[3,12092,0.0634798203],[3,12093,0.056313906],[3,12094,0.1189210534],[3,12101,0.0708795514],[3,12104,0.0115549552],[3,12109,0.0409924179],[3,12110,0.0354689567],[3,12111,0.1307293085],[3,12117,0.012678697],[3,12119,0.0120840079],[3,12120,0.0123678602],[3,12123,0.0506048627],[3,12129,0.012678697],[3,12135,0.0777389504],[3,12138,0.012678697],[3,12141,0.0876952136],[3,12144,0.1924863823],[3,12146,0.0108541314],[3,12147,0.2744376884],[3,12161,0.1009755093],[3,12162,0.0752663167],[3,12166,0.0887903999],[3,12167,0.1570977819],[3,12169,0.324514525],[3,12171,0.0099194978],[3,12178,0.1273512211],[3,12182,0.034608762],[3,12186,0.0153875714],[3,12188,0.0773215624],[3,12197,0.0377439643],[3,12203,0.2368576232],[3,12205,0.0143642056],[3,12208,0.1379079993],[3,12210,0.0503109454],[3,12214,0.012678697],[3,12218,0.0599411369],[3,12222,0.1020136956],[3,12223,0.0701624393],[3,12227,0.1089142298],[3,12231,0.1154929368],[3,12232,0.3673486233],[3,12238,0.0189055176],[3,12244,0.0328380246],[3,12245,0.0190663307],[3,12250,0.0620159821],[3,12251,0.043955005],[3,12252,0.1958345611],[3,12254,0.0596138714],[3,12261,0.2998600687],[3,12262,0.0147798458],[3,12268,0.0954766015],[3,12269,0.0413169517],[3,12280,0.0153421279],[3,12281,0.034263831],[3,12283,0.012678697],[3,12288,0.1714731339],[3,12298,0.0759035284],[3,12303,0.0653947836],[3,12306,0.0750743176],[3,12307,0.0501718323],[3,12309,0.4599872149],[3,12319,0.0943380727],[3,12321,0.0613720297],[3,12322,0.033159531],[3,12332,0.0583058347],[3,12333,0.012678697],[3,12339,0.0669685669],[3,12343,0.0795033374],[3,12345,0.1288056066],[3,12349,0.0386755643],[3,12351,0.0329217236],[3,12352,0.0372605752],[3,12353,0.0860571476],[3,12354,0.0336044246],[3,12356,0.0112402459],[3,12359,0.0118794915],[3,12368,0.0345372698],[3,12370,0.0527050927],[3,12372,0.1147519982],[3,12375,0.0311878201],[3,12376,0.0358948196],[3,12377,0.1694452809],[3,12378,0.0858912004],[3,12381,0.0184140182],[3,12384,0.0364115404],[3,12385,0.015638573],[3,12389,0.1998537794],[3,12391,0.012678697],[3,12392,0.1536445282],[3,12393,0.0717123117],[3,12399,0.0487486393],[3,12400,0.0329160407],[3,12402,0.0717160063],[3,12406,0.0133088398],[3,12410,0.1858399937],[3,12419,0.177591343],[3,12426,0.0379520675],[3,12427,0.0428735475],[3,12439,0.7631323058],[3,12452,0.07332866],[3,12454,0.535791598],[3,12456,0.0613677527],[3,12460,0.0865740247],[3,12463,0.0120840079],[3,12471,0.0330933313],[3,12473,0.012678697],[3,12475,0.1805941646],[3,12477,0.0633046223],[3,12479,0.0144644517],[3,12482,0.0716527217],[3,12487,0.0196637279],[3,12498,0.0099194978],[3,12502,0.0342803424],[3,12506,0.014565566],[3,12510,0.1074253855],[3,12521,0.280206897],[3,12527,0.012678697],[3,12530,0.0403167862],[3,12532,0.0120840079],[3,12533,0.012678697],[3,12536,0.257098468],[3,12538,0.0705416829],[3,12542,0.012678697],[3,12543,0.0484378412],[3,12544,0.0392626855],[3,12549,0.1263159894],[3,12551,0.0336698205],[3,12556,0.0384922169],[3,12562,0.0309680915],[3,12566,0.024792648],[3,12569,0.012678697],[3,12571,0.0233022086],[3,12576,0.041614095],[3,12578,0.0117130898],[3,12579,0.022420761],[3,12582,0.1370498061],[3,12589,0.0120840079],[3,12594,0.1603102982],[3,12595,0.0451889694],[3,12596,0.0462209677],[3,12598,0.2914624231],[3,12600,0.0145677995],[3,12602,0.7051687422],[3,12610,0.0680435826],[3,12611,0.0780261314],[3,12617,0.0267161438],[3,12619,0.0883662931],[3,12621,0.2233757662],[3,12623,0.0379190571],[3,12625,0.0157487559],[3,12629,0.0676018835],[3,12633,0.0501689015],[3,12634,0.0120840079],[3,12635,0.0147453136],[3,12644,0.0433970161],[3,12646,0.010301072],[3,12650,0.0418022378],[3,12653,0.1729232294],[3,12657,0.1466959808],[3,12660,0.1988597993],[3,12662,0.1964561923],[3,12663,0.3018894676],[3,12664,0.012678697],[3,12668,0.0325243887],[3,12672,0.0284480228],[3,12681,0.022420761],[3,12683,0.0689277792],[3,12685,0.012678697],[3,12686,0.050689266],[3,12692,0.0955245688],[3,12693,0.0576205203],[3,12700,0.0355228911],[3,12706,0.0782918113],[3,12707,0.298931834],[3,12708,0.0120955857],[3,12720,0.0115691351],[3,12722,0.0503464841],[3,12729,0.0359788327],[3,12732,0.012678697],[3,12735,0.0744571395],[3,12736,0.0590881702],[3,12745,0.0967266294],[3,12750,0.0720280112],[3,12753,0.1127095621],[3,12756,0.0233022086],[3,12762,0.0280951044],[3,12767,0.010301072],[3,12770,0.0372347125],[3,12771,0.0120840079],[3,12772,0.0543524565],[3,12773,0.0427511551],[3,12776,0.0144644517],[3,12779,0.0516166841],[3,12780,0.1540712515],[3,12789,0.013478613],[3,12791,0.0945545652],[3,12798,0.0120840079],[3,12800,0.0612962955],[3,12805,0.3806522162],[3,12807,0.031710102],[3,12812,0.0099194978],[3,12813,0.012678697],[3,12814,0.0099194978],[3,12815,0.012678697],[3,12817,0.0494337541],[3,12818,0.092033089],[3,12820,0.0133088398],[3,12826,0.029554873],[3,12827,0.0810289256],[3,12830,0.0714357033],[3,12833,0.012678697],[3,12837,0.0658687006],[3,12839,0.0374283749],[3,12844,0.0761834885],[3,12847,0.1226439117],[3,12850,0.1647650697],[3,12851,0.4311445007],[3,12852,0.014353364],[3,12857,0.0484617224],[3,12858,0.0444837851],[3,12859,0.0325268864],[3,12862,0.114518267],[3,12869,0.71903777],[3,12873,0.0144644517],[3,12876,0.0362778193],[3,12884,0.0790990452],[3,12886,0.0545969705],[3,12895,0.104067002],[3,12900,0.0478325304],[3,12902,0.0398758538],[3,12904,0.0173970526],[3,12916,0.1913783975],[3,12917,0.8943392986],[3,12920,0.012678697],[3,12923,0.036817976],[3,12925,0.2986443727],[3,12927,0.012678697],[3,12931,0.0363406547],[3,12933,0.0099194978],[3,12936,0.6281623391],[3,12941,0.0164550796],[3,12942,0.012678697],[3,12943,0.0163186973],[3,12951,0.0157487559],[3,12952,0.06633058],[3,12954,0.0299304389],[3,12964,0.1077844749],[3,12971,0.0268638597],[3,12980,0.0099194978],[3,12982,0.0715972862],[3,12988,0.1995027353],[3,12993,0.0144644517],[3,12995,0.062377046],[3,12996,0.0099194978],[3,12997,0.0393864018],[3,12999,0.0758032401],[3,13008,0.1075399345],[3,13009,0.012678697],[3,13010,0.0118794915],[3,13011,0.0455508027],[3,13022,0.0120840079],[3,13027,0.0474930059],[3,13034,0.0697727361],[3,13035,0.0652632084],[3,13045,0.0692955184],[3,13046,0.0592522124],[3,13050,0.0549980549],[3,13055,0.0892756485],[3,13056,0.0334122781],[3,13059,0.1716927619],[3,13062,0.0434145558],[3,13065,0.013408606],[3,13073,0.0404822653],[3,13084,0.0123678602],[3,13089,0.1115843809],[3,13090,0.0636283819],[3,13094,0.2540870578],[3,13099,0.2353757182],[3,13103,0.0955237957],[3,13106,0.0296015498],[3,13107,0.3972522165],[3,13109,0.3276248098],[3,13112,0.053722002],[3,13115,0.0422216067],[3,13117,0.0164550796],[3,13118,0.0276285969],[3,13129,0.1299440665],[3,13135,0.0442876665],[3,13139,0.1007680585],[3,13141,0.0134346252],[3,13143,0.0458574536],[3,13151,0.1903840423],[3,13154,0.0130370282],[3,13156,0.0448477154],[3,13157,0.0728947757],[3,13164,0.0147367912],[3,13166,0.0381020202],[3,13172,0.0348980524],[3,13173,0.1147580911],[3,13181,0.0762852899],[3,13187,0.0348719378],[3,13188,0.3091187719],[3,13189,0.0693405354],[3,13200,0.4065588302],[3,13210,0.0430610697],[3,13211,0.0976574109],[3,13213,0.0484313938],[3,13215,0.012678697],[3,13221,0.0123678602],[3,13226,0.0836944534],[3,13227,0.0518892399],[3,13240,0.0134251593],[3,13242,0.0746039656],[3,13246,0.0217450909],[3,13249,0.0486226556],[3,13251,0.0152106261],[3,13256,0.0143642056],[3,13257,0.0952831107],[3,13259,0.0622359289],[3,13262,0.1109155259],[3,13263,0.0145395074],[3,13268,0.150112098],[3,13269,0.128838167],[3,13271,0.0618952135],[3,13272,0.0771973485],[3,13273,0.044805255],[3,13277,0.0726582838],[3,13280,0.0427137359],[3,13282,0.012678697],[3,13285,0.0946580278],[3,13293,0.0240069595],[3,13296,0.010301072],[3,13297,0.0729237453],[3,13300,0.5092819997],[3,13301,0.0993737137],[3,13302,0.0360071992],[3,13303,0.0321423629],[3,13305,0.0120840079],[3,13307,0.0269670745],[3,13311,0.0603336045],[3,13315,0.03698746],[3,13318,0.433373078],[3,13319,0.0526335257],[3,13322,0.0306274676],[3,13325,0.1270424569],[3,13327,0.0853888371],[3,13329,0.1841441304],[3,13330,0.1429774883],[3,13332,0.347657438],[3,13333,0.1108555908],[3,13335,0.1491335935],[3,13337,0.017243805],[3,13338,0.1852344495],[3,13345,0.1247268091],[3,13351,0.0269670745],[3,13352,0.2002345239],[3,13353,0.0528416491],[3,13358,0.0224826907],[3,13362,0.0938587601],[3,13363,0.0283360014],[3,13365,0.0317045584],[3,13370,0.0309348418],[3,13371,0.1131182942],[3,13373,0.063642389],[3,13376,0.0319833275],[3,13377,0.0309600678],[3,13382,0.0398832357],[3,13383,0.1405254656],[3,13384,0.0115691351],[3,13396,0.1624368486],[3,13400,0.0688408461],[3,13404,0.2494421355],[3,13406,0.0696373613],[3,13409,0.015638573],[3,13411,0.012678697],[3,13418,0.0479551203],[3,13420,0.1005635242],[3,13422,0.2741275368],[3,13426,0.0799707856],[3,13427,0.0134251593],[3,13429,0.0412423357],[3,13431,0.0115691351],[3,13432,0.012678697],[3,13439,0.0925725122],[3,13448,0.0940467474],[3,13449,0.0360931815],[3,13453,0.1459140977],[3,13455,0.2615481702],[3,13456,0.0341256597],[3,13460,0.57893643],[3,13463,0.012678697],[3,13464,0.1192307449],[3,13465,0.0118794915],[3,13470,0.0331853053],[3,13476,0.1780328639],[3,13485,0.0534159174],[3,13487,0.0717401115],[3,13492,0.0123678602],[3,13496,0.6145248006],[3,13500,0.012678697],[3,13508,0.0513133477],[3,13509,0.0720280112],[3,13516,0.0204305148],[3,13519,0.1993680566],[3,13520,0.0439810863],[3,13521,0.0183125817],[3,13522,0.0375033622],[3,13523,0.0758885651],[3,13524,0.080982934],[3,13525,0.0989042199],[3,13526,0.0314272107],[3,13529,0.0274571452],[3,13538,0.012678697],[3,13541,0.0336164307],[3,13543,0.0265457591],[3,13544,0.0418088745],[3,13547,0.031022698],[3,13548,0.0099194978],[3,13549,0.0130370282],[3,13550,0.0734321845],[3,13551,0.0796798092],[3,13553,0.0362428506],[3,13557,0.0594131818],[3,13561,0.0538230363],[3,13567,0.4320080111],[3,13568,0.0949679324],[3,13572,0.0169601045],[3,13577,0.013408606],[3,13579,0.0115691351],[3,13583,0.055333334],[3,13599,0.1118447337],[3,13603,0.1093450123],[3,13604,0.0501614156],[3,13605,0.0341294898],[3,13610,0.0120840079],[3,13612,0.0886422551],[3,13613,0.0324764918],[3,13617,0.0324724775],[3,13621,0.0620049059],[3,13623,0.012678697],[3,13624,0.0162848707],[3,13625,0.0363709023],[3,13629,0.051894834],[3,13630,0.012678697],[3,13631,0.1341776429],[3,13634,0.0380283497],[3,13635,0.0370357911],[3,13636,0.0385926725],[3,13638,0.0202765797],[3,13642,0.2704684461],[3,13645,0.0640713591],[3,13647,0.0157487559],[3,13650,0.0475577826],[3,13651,0.0145677995],[3,13652,0.1658539215],[3,13655,0.0626160887],[3,13658,0.0626874982],[3,13661,0.4181249117],[3,13662,0.0134251593],[3,13663,0.0640500356],[3,13667,0.0748174434],[3,13668,0.203108388],[3,13671,0.3747625333],[3,13672,0.0773235063],[3,13674,0.0140757566],[3,13675,0.0867847245],[3,13687,0.2039398112],[3,13693,0.0951101593],[3,13695,0.0412135241],[3,13696,0.1778733695],[3,13703,0.0564193869],[3,13711,0.1502595946],[3,13719,0.0115691351],[3,13721,0.0765406099],[3,13723,0.0618546931],[3,13727,0.0411379117],[3,13729,0.0224826907],[3,13730,0.0448962853],[3,13731,0.5976586457],[3,13732,0.1449633003],[3,13734,0.0134251593],[3,13743,0.0420392895],[3,13746,0.0403426809],[3,13747,0.0320310957],[3,13753,0.059029177],[3,13755,0.0163390237],[3,13758,0.0143642056],[3,13759,0.0690199903],[3,13761,0.0282898941],[3,13766,0.0153421279],[3,13767,0.0354793508],[3,13768,0.0756413145],[3,13779,0.0586980987],[3,13780,0.1671578613],[3,13786,0.1537451999],[3,13796,0.1581321495],[3,13799,0.0594807109],[3,13800,0.0235085581],[3,13802,0.0388526387],[3,13807,0.0747492942],[3,13810,0.0194812784],[3,13820,0.0546185371],[3,13827,0.0561315349],[3,13836,0.0693308988],[3,13839,0.0769900068],[3,13840,0.0617463504],[3,13843,0.0703784925],[3,13846,0.0589379355],[3,13853,0.0446568171],[3,13856,0.0120840079],[3,13864,0.0656522633],[3,13875,0.0615808338],[3,13878,0.0309600678],[3,13881,0.0334348826],[3,13886,0.0118794915],[3,13891,0.0118794915],[3,13892,0.0458982132],[3,13893,0.0265151588],[3,13895,0.0388347181],[3,13896,0.1660075452],[3,13905,0.022316327],[3,13908,0.1074341939],[3,13909,0.0642384684],[3,13910,0.012678697],[3,13916,0.0120840079],[3,13918,0.0369956399],[3,13919,0.0233981481],[3,13920,0.0345973881],[3,13923,0.2658869811],[3,13925,0.0247785761],[3,13928,0.0401178107],[3,13929,0.2985606971],[3,13932,0.0723139566],[3,13934,0.0417540511],[3,13939,0.0355604033],[3,13942,0.0436265935],[3,13943,0.0133088398],[3,13945,0.0788788483],[3,13947,0.2015695305],[3,13948,0.0652559832],[3,13951,0.0596225909],[3,13953,0.0115691351],[3,13955,0.0890938095],[3,13957,0.012678697],[3,13960,0.012678697],[3,13961,0.4555709232],[3,13966,0.0983083994],[3,13975,0.2166566896],[3,13978,0.0356664013],[3,13999,0.0512787568],[3,14001,0.0506829473],[3,14002,0.0546528282],[3,14003,0.0537847837],[3,14018,0.111048553],[3,14021,0.0755626479],[3,14022,0.0164550796],[3,14023,0.4499615424],[3,14024,0.0951065281],[3,14040,0.7124195435],[3,14041,0.0504542773],[3,14044,0.0099194978],[3,14045,0.1016453109],[3,14049,0.0688924471],[3,14050,0.012678697],[3,14051,0.0317492192],[3,14053,0.0224826907],[3,14055,0.1547010489],[3,14056,0.015638573],[3,14065,0.1389987045],[3,14070,0.3981273022],[3,14073,0.0686895561],[3,14074,0.0985722681],[3,14075,0.0352772214],[3,14078,0.0161449026],[3,14084,0.0301447917],[3,14089,0.0404332087],[3,14090,0.0621205778],[3,14097,0.0676297243],[3,14103,0.1823756849],[3,14107,0.0439753407],[3,14111,0.646848294],[3,14112,0.012678697],[3,14119,0.0569967447],[3,14120,0.0588367951],[3,14123,0.046719771],[3,14124,0.3659393664],[3,14125,0.141274105],[3,14127,0.0638942505],[3,14130,0.0372505908],[3,14132,0.0664015531],[3,14135,0.0463525313],[3,14149,0.0284301578],[3,14150,0.012678697],[3,14152,0.063019221],[3,14158,0.0709897727],[3,14167,0.0427137359],[3,14168,0.0345404022],[3,14169,0.0134251593],[3,14172,0.2123476407],[3,14174,0.0930133286],[3,14176,0.0108541314],[3,14195,0.0164550796],[3,14199,0.1241671332],[3,14203,0.2478896765],[3,14207,0.0207176991],[3,14209,0.067995812],[3,14211,0.1147781213],[3,14212,0.0463085799],[3,14215,0.0146073281],[3,14217,0.0103302182],[3,14218,0.0272890067],[3,14226,0.0145120951],[3,14228,0.0085974606],[3,14231,0.0445464312],[3,14239,0.0097025917],[3,14242,0.0688402216],[3,14243,0.0345922601],[3,14245,0.0687239624],[3,14247,0.029735519],[3,14249,0.0091102773],[3,14253,0.0598159758],[3,14256,0.0788544121],[3,14257,0.0468074847],[3,14259,0.2889952616],[3,14263,0.274504145],[3,14268,0.0548946919],[3,14275,0.0366648762],[3,14276,0.0914388965],[3,14277,0.0238731698],[3,14278,0.0638249877],[3,14279,0.0948569463],[3,14280,0.1248897287],[3,14290,0.010059492],[3,14291,0.1201060882],[3,14293,0.1308436194],[3,14298,0.3224569671],[3,14304,0.1078923002],[3,14309,0.1638121862],[3,14312,0.0984563097],[3,14320,0.0506200969],[3,14321,0.2155938736],[3,14323,0.0387192747],[3,14325,0.1672292161],[3,14329,0.0388022657],[3,14331,0.0924762725],[3,14334,0.1796904086],[3,14335,0.1955689958],[3,14336,0.0607007328],[3,14341,0.0615285194],[3,14344,0.0483953481],[3,14347,0.0308750137],[3,14353,0.0629565756],[3,14354,0.0097025917],[3,14355,0.0331694454],[3,14358,0.5214289489],[3,14359,0.078888329],[3,14365,0.0987200003],[3,14366,0.0971163768],[3,14367,0.3277299574],[3,14372,0.0302185863],[3,14373,0.0341276755],[3,14376,0.1798837812],[3,14378,0.032754835],[3,14387,0.1387088533],[3,14389,0.0694074461],[3,14392,0.0373295614],[3,14397,0.0097025917],[3,14404,0.0411266076],[3,14409,0.0567357522],[3,14411,0.06391081],[3,14412,0.0650136989],[3,14413,0.0476042447],[3,14414,0.0314433689],[3,14415,0.2361431017],[3,14416,0.3789746962],[3,14421,0.0935635188],[3,14427,0.1299501603],[3,14443,0.0603255905],[3,14447,0.070039495],[3,14449,0.0620937977],[3,14450,0.3101090873],[3,14452,0.03660685],[3,14458,0.2261996962],[3,14459,0.0376050778],[3,14460,0.1633256411],[3,14465,0.1141013131],[3,14466,0.0704122166],[3,14475,0.102212565],[3,14477,0.0784978385],[3,14485,0.0582334481],[3,14489,0.0357575181],[3,14490,0.0519590051],[3,14491,0.5994545831],[3,14492,0.0073344612],[3,14493,0.0496902866],[3,14498,0.7461593779],[3,14499,0.0379494652],[3,14502,0.0939916107],[3,14508,0.0659765108],[3,14510,0.010429586],[3,14513,0.0678436864],[3,14518,0.0316327315],[3,14520,0.0541774605],[3,14521,0.0606835724],[3,14526,0.0447378037],[3,14527,0.0647984343],[3,14535,0.1391590133],[3,14536,0.0581723329],[3,14550,0.037094297],[3,14551,0.0596389444],[3,14552,0.0114812155],[3,14555,0.1062603212],[3,14557,0.0366665547],[3,14559,0.0091102773],[3,14562,0.1401473934],[3,14567,0.0821501825],[3,14568,0.0379651712],[3,14573,0.0553253489],[3,14574,0.0122244102],[3,14581,0.0576246219],[3,14583,0.3788427881],[3,14585,0.2910476224],[3,14589,0.0595452633],[3,14592,0.010059492],[3,14594,0.3230751671],[3,14596,0.0115559715],[3,14597,0.0825610412],[3,14599,0.0079703502],[3,14605,0.0762227594],[3,14608,0.0684047578],[3,14611,0.0368831223],[3,14615,0.0280928577],[3,14616,0.0743785773],[3,14617,0.0525703402],[3,14618,0.0851847398],[3,14619,0.0266355493],[3,14620,0.0943418417],[3,14622,0.1258895797],[3,14625,0.0923243591],[3,14643,0.032503307],[3,14644,0.0668609173],[3,14653,0.0500565268],[3,14660,0.0091102773],[3,14665,0.0383677963],[3,14667,0.0142987808],[3,14672,0.0575436891],[3,14675,0.0637004574],[3,14679,0.1107206759],[3,14682,0.062297697],[3,14687,0.0318772172],[3,14693,0.053337074],[3,14701,0.0860014813],[3,14702,0.0656726834],[3,14704,0.2233406264],[3,14706,0.097852722],[3,14707,0.0084808173],[3,14708,0.0614332261],[3,14711,0.0073344612],[3,14715,0.1568113256],[3,14716,0.0388022657],[3,14717,0.0657391781],[3,14718,0.1903756373],[3,14722,0.0665257224],[3,14728,0.088971245],[3,14734,0.0450637326],[3,14735,0.0804085798],[3,14736,0.073830057],[3,14738,0.0575781063],[3,14741,0.2288652681],[3,14744,0.0381198662],[3,14746,0.0736802601],[3,14750,0.0097025917],[3,14757,0.0771663217],[3,14758,0.0506200969],[3,14760,0.0153139788],[3,14764,0.0302126269],[3,14774,0.1503456291],[3,14787,0.1346552647],[3,14788,0.0323504328],[3,14791,0.0637951018],[3,14795,0.03984487],[3,14796,0.0543882374],[3,14808,0.0419983953],[3,14810,0.1414085752],[3,14814,0.0323075584],[3,14815,0.0937530863],[3,14824,0.1252677462],[3,14827,0.0998709502],[3,14828,0.1966450343],[3,14832,0.1040966088],[3,14835,0.0097025917],[3,14836,0.5946391856],[3,14837,0.0097025917],[3,14845,0.0284191636],[3,14850,0.0582187817],[3,14864,0.0650795474],[3,14867,0.0328246654],[3,14868,0.0286768064],[3,14869,0.0418637404],[3,14873,0.340857563],[3,14874,0.2968686185],[3,14882,0.0486775029],[3,14886,0.0481622583],[3,14888,0.1820967873],[3,14896,0.1445930888],[3,14912,0.0114812155],[3,14913,0.0754467664],[3,14917,0.0372643332],[3,14920,0.1771721689],[3,14922,0.1083366397],[3,14931,0.0278029938],[3,14934,0.0550408311],[3,14935,0.2663711104],[3,14937,0.052604624],[3,14938,0.0710812317],[3,14940,0.0579684723],[3,14941,0.0256319316],[3,14947,0.0262995135],[3,14959,0.0097025917],[3,14965,0.1103084741],[3,14968,0.0845497328],[3,14970,0.1060563591],[3,14973,0.0648443942],[3,14975,0.0312815497],[3,14976,0.088858795],[3,14978,0.0398642323],[3,14979,0.0981985551],[3,14983,0.0097025917],[3,14984,0.0670321156],[3,14985,0.3235193903],[3,14990,0.0573910431],[3,14994,0.0984188803],[3,15005,0.0939702108],[3,15007,0.043478504],[3,15009,0.1719132345],[3,15010,0.0929557209],[3,15013,0.0428957054],[3,15014,0.0514610547],[3,15016,0.1540656817],[3,15020,0.305078243],[3,15028,0.0575061711],[3,15029,0.0103302182],[3,15032,0.0520941165],[3,15034,0.0943366217],[3,15041,0.0478301337],[3,15047,0.3264890902],[3,15049,0.0454655306],[3,15054,0.025663364],[3,15064,0.0657370191],[3,15066,0.0168959089],[3,15069,0.0435555417],[3,15072,0.0777457474],[3,15098,0.052334519],[3,15099,0.013294365],[3,15105,0.2436882732],[3,15109,0.0117953501],[3,15112,0.0280908126],[3,15113,0.0117953501],[3,15120,0.0155095991],[3,15130,0.0602549666],[3,15138,0.0187327812],[3,15140,0.0194057531],[3,15142,0.1893233622],[3,15146,0.0697827311],[3,15150,0.1085968254],[3,15152,0.0274952288],[3,15154,0.0089065776],[3,15155,0.2097112883],[3,15171,0.0773345842],[3,15179,0.2097705057],[3,15180,0.4800637457],[3,15190,0.27857277],[3,15192,0.0097025917],[3,15194,0.1906307178],[3,15197,0.1582929367],[3,15206,0.1029781123],[3,15209,0.0217541528],[3,15221,0.102227349],[3,15222,0.0705918827],[3,15223,0.0114812155],[3,15225,0.032398307],[3,15232,0.0647645408],[3,15233,0.0097025917],[3,15236,0.0648145289],[3,15242,0.1158784612],[3,15243,0.2132296242],[3,15248,0.0420925276],[3,15251,0.0487950086],[3,15264,0.0093929962],[3,15265,0.0505627234],[3,15271,0.1191181765],[3,15273,0.1279073966],[3,15274,0.0126506482],[3,15276,0.066761804],[3,15280,0.3833716083],[3,15286,0.0667029553],[3,15287,0.0942032365],[3,15288,0.0635739548],[3,15289,0.1731282669],[3,15298,0.0097025917],[3,15307,0.1036866474],[3,15308,0.0612149513],[3,15309,0.6432902351],[3,15312,0.0126506482],[3,15317,0.0516498861],[3,15318,0.0718572557],[3,15319,0.0662575934],[3,15321,0.0943418417],[3,15322,0.070032225],[3,15327,0.0722249223],[3,15330,0.0222097495],[3,15335,0.2471508898],[3,15339,0.0887179864],[3,15340,0.0752351264],[3,15347,0.3665051189],[3,15351,0.1215048926],[3,15358,0.0703300771],[3,15359,0.1634112587],[3,15360,0.0114812155],[3,15362,0.2627145435],[3,15363,0.0552494435],[3,15365,0.1141001002],[3,15366,0.0297814179],[3,15370,0.0091102773],[3,15392,0.0122244102],[3,15393,0.4295820487],[3,15395,0.0324790758],[3,15396,0.0618668804],[3,15397,0.2548022914],[3,15402,0.1445845852],[3,15409,0.0427230878],[3,15418,0.095959764],[3,15425,0.0417431839],[3,15426,0.1131513893],[3,15427,0.0463108763],[3,15430,0.1883294613],[3,15443,0.0153139788],[3,15460,0.1327466022],[3,15461,0.0291687278],[3,15465,0.2902869152],[3,15468,0.0277253526],[3,15472,0.1431982641],[3,15479,0.1290227832],[3,15482,0.0841621984],[3,15485,0.0153139788],[3,15487,0.1373097541],[3,15489,0.010059492],[3,15490,0.1310294438],[3,15499,0.0332198556],[3,15500,0.0624147679],[3,15503,0.0721886026],[3,15511,0.1239917699],[3,15514,0.082367763],[3,15515,0.0675678376],[3,15532,0.1111750568],[3,15535,0.0193017362],[3,15538,0.1253379172],[3,15541,0.0114812155],[3,15555,0.0093929962],[3,15557,0.1219793551],[3,15564,0.0488137257],[3,15565,0.0251466238],[3,15566,0.0413300415],[3,15567,0.2471498205],[3,15568,0.1053186341],[3,15570,0.0334997496],[3,15572,0.0118516199],[3,15575,0.0126506482],[3,15585,0.0612199839],[3,15590,0.0579627736],[3,15592,0.4519974077],[3,15594,0.0338809382],[3,15620,0.1514499045],[3,15647,0.1048016347],[3,15657,0.2229295912],[3,15658,0.0536553543],[3,15662,0.0662590623],[3,15674,0.0783638085],[3,15676,0.0622107978],[3,15678,0.0524815708],[3,15679,0.1571277665],[3,15680,0.0948145559],[3,15684,0.0280196495],[3,15697,0.1844772122],[3,15706,0.0552668208],[3,15707,0.0877341392],[3,15711,0.059676199],[3,15713,0.5492063779],[3,15719,0.0357616185],[3,15720,0.2149033975],[3,15721,0.0340864776],[3,15730,0.6122758008],[3,15735,0.2149457266],[3,15745,0.014161748],[3,15749,0.1395795522],[3,15751,0.0778510571],[3,15778,0.0555869898],[3,15783,0.0115559715],[3,15803,0.03399626],[3,15807,0.1822100883],[3,15808,0.0085974606],[3,15809,0.0844988203],[3,15810,0.241229204],[3,15812,0.0200008955],[3,15820,0.1805102876],[3,15829,0.2361481964],[3,15834,0.0453555959],[3,15835,0.0122244102],[3,15836,0.0516498861],[3,15837,0.0538320692],[3,15840,0.0955646106],[3,15847,0.096883814],[3,15851,0.0439943443],[3,15852,0.0565274949],[3,15854,0.0533999129],[3,15868,0.040373732],[3,15869,0.0449255015],[3,15872,0.0123553869],[3,15898,0.0921872381],[3,15899,0.0613693622],[3,15901,0.0265139357],[3,15903,0.1552754434],[3,15910,0.0427137165],[3,15912,0.0117609557],[3,15924,0.0362259173],[3,15937,0.0266355493],[3,15945,0.0638102618],[3,15955,0.0921551948],[3,15978,0.0399636397],[3,15988,0.0399730983],[3,15989,0.0401359937],[3,15996,0.0823573136],[3,15998,0.0123340903],[3,16002,0.0502761502],[3,16005,0.0354215746],[3,16007,0.0668364797],[3,16012,0.0435689451],[3,16017,0.0187327812],[3,16031,0.3076441082],[3,16035,0.0375025253],[3,16044,0.0871413746],[3,16046,0.1617951642],[3,16050,0.1920362846],[3,16051,0.0643940829],[3,16054,0.0104460732],[3,16055,0.0117524674],[3,16057,0.0286701589],[3,16058,0.0297198715],[3,16059,0.0117953501],[3,16066,0.0180051861],[3,16070,0.0656726834],[3,16086,0.178972478],[3,16087,0.0097025917],[3,16106,0.0155200947],[3,16109,0.0122244102],[3,16118,0.0905419274],[3,16119,0.125284857],[3,16122,0.0522968618],[3,16123,0.0671639812],[3,16127,0.0493021231],[3,16131,0.1154487434],[3,16134,0.038236479],[3,16135,0.1473719548],[3,16138,0.0421289671],[3,16146,0.1413496838],[3,16153,0.0312125393],[3,16156,0.1183424206],[3,16157,0.0702699145],[3,16159,0.0796202285],[3,16161,0.0780061506],[3,16162,0.0511520985],[3,16171,0.0181386416],[3,16174,0.0627692868],[3,16177,0.0630033243],[3,16178,0.0665815158],[3,16188,0.2051753993],[3,16190,0.0595100391],[3,16206,0.0097025917],[3,16207,0.2104504491],[3,16208,0.1202832707],[3,16216,0.0194057531],[3,16229,0.1502281221],[3,16230,0.0812979286],[3,16238,0.0696338616],[3,16240,0.0606746686],[3,16256,0.0097025917],[3,16259,0.0479303569],[3,16270,0.1982750929],[3,16271,0.0337888347],[3,16274,0.0437308639],[3,16283,0.0118516199],[3,16284,0.0835296193],[3,16297,0.0679358254],[3,16302,0.0358082586],[3,16304,0.1140278772],[3,16314,0.0765232266],[3,16319,0.0097025917],[3,16320,0.2270201329],[3,16322,0.0217541528],[3,16324,0.2369968528],[3,16325,0.1554283561],[3,16329,0.056119462],[3,16330,0.0613720787],[3,16331,0.1216703814],[3,16334,0.0580631044],[3,16336,0.035483165],[3,16340,0.028758128],[3,16344,0.0675052185],[3,16345,0.0422230773],[3,16350,0.0296332658],[3,16351,0.0097025917],[3,16357,0.0594699075],[3,16373,0.0635739548],[3,16378,0.0097025917],[3,16402,0.0134638943],[3,16403,0.054775904],[3,16408,0.0579147334],[3,16409,0.034721106],[3,16415,0.0293529147],[3,16440,0.062068765],[3,16445,0.0392034911],[3,16454,0.056268791],[3,16459,0.022564149],[3,16467,0.0314490417],[3,16491,0.0899038136],[3,16502,0.4741408467],[3,16503,0.0346764955],[3,16505,0.0374059923],[3,16506,0.0779131614],[3,16517,0.1749610713],[3,16518,0.0535645101],[3,16520,0.1206063962],[3,16521,0.0315014693],[3,16523,0.181413126],[3,16527,0.1098771197],[3,16530,0.0969804591],[3,16531,0.0720762397],[3,16548,0.525256152],[3,16550,0.03189659],[3,16555,0.0702337929],[3,16558,0.3899612921],[3,16559,0.2014892013],[3,16563,0.1854829753],[3,16565,0.0702209539],[3,16573,0.0329426108],[3,16583,0.0386691025],[3,16593,0.0856853579],[3,16596,0.0637820515],[3,16597,0.0303806253],[3,16608,0.0428848057],[3,16613,0.0364996304],[3,16615,0.0653401829],[3,16629,0.0864129035],[3,16644,0.035865612],[3,16649,0.0295537669],[3,16650,0.0365942442],[3,16652,0.0085974606],[3,16653,0.0910957281],[3,16660,0.0908650474],[3,16661,0.0856348552],[3,16664,0.0301811963],[3,16669,0.01737387],[3,16676,0.0653336736],[3,16677,0.0411490897],[3,16680,0.0492194245],[3,16691,0.1054166748],[3,16697,0.1042863762],[3,16698,0.022564149],[3,16706,0.0073344612],[3,16715,0.4925155535],[3,16716,0.0462169417],[3,16719,0.0438455007],[3,16723,0.0131434251],[3,16724,0.1376492508],[3,16725,0.1732651423],[3,16726,0.1129312842],[3,16734,0.0401022843],[3,16761,0.121144299],[3,16770,0.0134638943],[3,16776,0.0097025917],[3,16788,0.0332070683],[3,16791,0.0920500087],[3,16800,0.0274952288],[3,16805,0.0865918666],[3,16808,0.022750991],[3,16810,0.1514170611],[3,16812,0.0664010301],[3,16817,0.0552003229],[3,16819,0.0131434251],[3,16825,0.041498937],[3,16827,0.0560701359],[3,16831,0.0385677689],[3,16836,0.0624143792],[3,16838,0.0661756226],[3,16841,0.0689349757],[3,16846,0.0669191997],[3,16851,0.0348141878],[3,16852,0.0328800322],[3,16865,0.0097025917],[3,16873,0.0279733403],[3,16881,0.0614635809],[3,16882,0.1484405992],[3,16889,0.0652914244],[3,16931,0.0113813696],[3,16939,0.0091102773],[3,16945,0.0091102773],[3,16963,0.0113813696],[3,16978,0.0122244102],[3,16983,0.0662141653],[3,16993,0.1994790967],[3,17004,0.0085974606],[3,17015,0.0093929962],[3,17017,0.0397146368],[3,17029,0.0104993134],[3,17052,0.0097025917],[3,17089,0.067988115],[3,17090,0.0085974606],[3,17092,0.1320397979],[3,17098,0.0073344612],[3,17100,0.0084808173],[3,17109,0.0910980225],[3,17112,0.0318850556],[3,17114,0.1105318416],[3,17115,0.0439710204],[3,17116,0.1693432464],[3,17117,0.3814082443],[3,17123,0.0563634382],[3,17124,0.094383205],[3,17127,0.049743603],[3,17134,0.0686955369],[3,17136,0.0751883474],[3,17138,0.0097025917],[3,17143,0.052823317],[3,17150,0.1672354541],[3,17154,0.0356043831],[3,17164,0.0174438431],[3,17166,0.0208315815],[3,17179,0.0346617741],[3,17183,0.0595579419],[3,17185,0.1197523117],[3,17190,0.0115559715],[3,17192,0.078869582],[3,17193,0.0422212677],[3,17195,0.0475374746],[3,17208,0.0091102773],[3,17213,0.0810921583],[3,17226,0.0085974606],[3,17227,0.117914131],[3,17228,0.0373299956],[3,17230,0.0097025917],[3,17238,0.1986822998],[3,17243,0.0087408404],[3,17245,0.0751017978],[3,17248,0.0127348625],[3,17250,0.0653730506],[3,17254,0.0097025917],[3,17264,0.0358774731],[3,17266,0.110071475],[3,17269,0.1292734095],[3,17271,0.0073344612],[3,17273,0.1154372962],[3,17278,0.0082698847],[3,17283,0.0123553869],[3,17288,0.170241313],[3,17296,0.0123553869],[3,17299,0.0216095511],[3,17305,0.159835407],[3,17306,0.0118516199],[3,17311,0.0312495426],[3,17314,0.0423296729],[3,17316,0.0708187017],[3,17321,0.120621503],[3,17322,0.0103302182],[3,17323,0.0850010489],[3,17324,0.0656619061],[3,17325,0.0575433332],[3,17326,0.0668023434],[3,17334,0.0682242674],[3,17337,0.2917506589],[3,17341,0.1032462318],[3,17342,0.0554354084],[3,17343,0.1005624491],[3,17344,0.2907636402],[3,17345,0.0320747837],[3,17346,0.1858336695],[3,17353,0.0264630828],[3,17354,0.0097025917],[3,17357,0.0407458239],[3,17358,0.1935274576],[3,17361,0.0097025917],[3,17373,0.0194057531],[3,17379,0.0097025917],[3,17390,0.0091102773],[3,17394,0.0181406268],[3,17397,0.0565956241],[3,17402,0.395316175],[3,17408,0.040293113],[3,17409,0.0092632418],[3,17414,0.0103302182],[3,17423,0.060153808],[3,17429,0.1121036196],[3,17434,0.0276882544],[3,17435,0.0421289671],[3,17446,0.0794409924],[3,17464,0.3364410598],[3,17475,0.0365329839],[3,17480,0.0863212253],[3,17483,0.0540291429],[3,17484,0.1238428949],[3,17485,0.0279733403],[3,17488,0.4711354151],[3,17492,0.0325985001],[3,17496,0.2580833428],[3,17503,0.0343751063],[3,17510,0.0431130523],[3,17512,0.0768719648],[3,17527,0.0812443176],[3,17530,0.0955069256],[3,17532,0.0931538807],[3,17533,0.0475170907],[3,17534,0.1908129636],[3,17536,0.0103302182],[3,17541,0.0397754127],[3,17542,0.0720893947],[3,17547,0.0082698847],[3,17549,0.0104460732],[3,17555,0.0664131987],[3,17566,0.5922708348],[3,17571,0.0131434251],[3,17574,0.0894115598],[3,17575,0.2328553654],[3,17586,0.0071591178],[3,17587,0.0071591178],[3,17589,0.0103929676],[3,17590,0.005829231],[3,17591,0.1172274969],[3,17594,0.0557021379],[3,17596,0.0093436374],[3,17597,0.0188707563],[3,17603,0.0205976908],[3,17604,0.0064189252],[3,17606,0.0121060743],[3,17607,0.0565551985],[3,17608,0.0919096617],[3,17610,0.1071905022],[3,17613,0.0258514671],[3,17615,0.1251555431],[3,17618,0.0549171023],[3,17622,0.2225716663],[3,17628,0.0373883016],[3,17629,0.0286781237],[3,17631,0.0071427035],[3,17632,0.0534006906],[3,17636,0.059973287],[3,17643,0.1880828376],[3,17645,0.0064189252],[3,17648,0.0416612185],[3,17653,0.2593378495],[3,17659,0.0557344287],[3,17661,0.0613248842],[3,17663,0.1622676643],[3,17668,0.0434419047],[3,17672,0.0344150778],[3,17673,0.2992801806],[3,17674,0.0440942198],[3,17681,0.2179069522],[3,17684,0.0193624137],[3,17685,0.0333613695],[3,17686,0.0684022493],[3,17691,0.0080902768],[3,17694,0.0064189252],[3,17695,0.0317056256],[3,17698,0.0586853841],[3,17706,0.1904843136],[3,17710,0.0695854811],[3,17712,0.0752168852],[3,17717,0.008502426],[3,17722,0.0104018292],[3,17723,0.0795956312],[3,17724,0.0772597055],[3,17726,0.0341492248],[3,17741,0.018417175],[3,17745,0.0360528496],[3,17753,0.0638361647],[3,17758,0.0540600829],[3,17765,0.0556582747],[3,17776,0.0629265737],[3,17788,0.0285605335],[3,17792,0.2976007833],[3,17795,0.2119376064],[3,17802,0.0351421263],[3,17810,0.0525636629],[3,17816,0.118800123],[3,17819,0.0039197471],[3,17821,0.0084279768],[3,17823,0.1745472918],[3,17826,0.0163814266],[3,17839,0.0678988053],[3,17840,0.0201695151],[3,17841,0.3127387145],[3,17843,0.1461981964],[3,17844,0.0064189252],[3,17852,0.0064189252],[3,17861,0.0485793137],[3,17867,0.005829231],[3,17878,0.070882301],[3,17895,0.0241537397],[3,17902,0.0379421091],[3,17904,0.115460904],[3,17909,0.0194654332],[3,17910,0.2315906653],[3,17912,0.0957691776],[3,17913,0.082409465],[3,17916,0.0484627218],[3,17918,0.074575313],[3,17923,0.0411778087],[3,17928,0.0889908011],[3,17929,0.0357474086],[3,17932,0.0967363066],[3,17933,0.017739975],[3,17934,0.1996239658],[3,17935,0.1624255746],[3,17938,0.0575733778],[3,17946,0.0346200997],[3,17950,0.1463778534],[3,17957,0.1122963866],[3,17970,0.0799753355],[3,17971,0.0850333098],[3,17972,0.005829231],[3,17986,0.0964744082],[3,18007,0.0213400052],[3,18008,0.8491474106],[3,18013,0.0497800709],[3,18022,0.2267341742],[3,18027,0.0647346279],[3,18028,0.0571307187],[3,18031,0.0101420374],[3,18042,0.065922901],[3,18057,0.1309617839],[3,18067,0.4433908243],[3,18068,0.0901636421],[3,18080,0.0344703975],[3,18118,0.1108721779],[3,18150,0.2078861562],[3,18152,0.057274052],[3,18153,0.6985137104],[3,18154,0.2431262338],[3,18169,0.2975091481],[3,18177,0.1155555864],[3,18181,0.0509411757],[3,18194,0.0064189252],[3,18201,0.054417451],[3,18215,0.0093539406],[3,18216,0.0583686618],[3,18218,0.0339356785],[3,18220,0.4494614868],[3,18238,0.1538343147],[3,18250,0.1269750759],[3,18255,0.0778094437],[3,18258,0.0656236146],[3,18264,0.0039197471],[3,18267,0.0366396499],[3,18268,0.0441722522],[3,18278,0.3491988842],[3,18279,0.1154825914],[3,18281,0.0124502831],[3,18283,0.0438600118],[3,18284,0.0579856481],[3,18321,0.1215461638],[3,18327,0.0340534194],[3,18334,0.0605812227],[3,18339,0.1317722457],[3,18341,0.0675003852],[3,18342,0.0511301821],[3,18351,0.0072121225],[3,18359,0.1009938564],[3,18377,0.0642327134],[3,18385,0.1659414446],[3,18391,0.0290523569],[3,18407,0.1892692748],[3,18409,0.0334183964],[3,18412,0.0137438022],[3,18420,0.074079285],[3,18432,0.0622607457],[3,18435,0.1889570851],[3,18439,0.023179321],[3,18442,0.0286486962],[3,18443,0.0536631442],[3,18444,0.0586146079],[3,18453,0.077274501],[3,18456,0.0495609752],[3,18478,0.0792306022],[3,18510,0.0411972146],[3,18514,0.283387154],[3,18523,0.1529297029],[3,18525,0.0812216976],[3,18528,0.2753729459],[3,18532,0.245936575],[3,18534,0.0593862824],[3,18540,0.054044294],[3,18551,0.1535529533],[3,18563,0.0064189252],[3,18564,0.0543433859],[3,18575,0.0169532078],[3,18585,0.0625615124],[3,18588,0.0993470528],[3,18606,0.0344567116],[3,18620,0.0192584027],[3,18623,0.0221948342],[3,18631,0.1690174858],[3,18632,0.008502426],[3,18642,0.1191484661],[3,18652,0.0775191024],[3,18653,0.7121534585],[3,18659,0.0805209776],[3,18689,0.0896791707],[3,18700,0.1153438724],[3,18702,0.0071239955],[3,18707,0.0471249318],[3,18710,0.0056988464],[3,18711,0.0425910277],[3,18726,0.0413119946],[3,18727,0.0371609027],[3,18735,0.1086475913],[3,18748,0.0058639036],[3,18749,0.0510998626],[3,18761,0.0627978673],[3,18765,0.5866706595],[3,18778,0.061768871],[3,18809,0.0071591178],[3,18819,0.0064189252],[3,18848,0.0087065692],[3,18879,0.0064189252],[3,18890,0.0514591343],[3,18891,0.008528274],[3,18892,0.0087065692],[3,18899,0.2381759424],[3,18913,0.0064189252],[3,18914,0.01040252],[3,18915,0.127238355],[3,18916,0.2193273421],[3,18919,0.0082641063],[3,18920,0.0544772295],[3,18929,0.118469743],[3,18930,0.0397289209],[3,18931,0.0291319184],[3,18936,0.0052298232],[3,18946,0.0326788851],[3,18950,0.4546993989],[3,18962,0.1080761412],[3,18966,0.0273887797],[3,18974,0.0219513554],[3,18976,0.0163628232],[3,18982,0.0417388655],[3,18987,0.3738592979],[3,18988,0.0602896103],[3,19001,0.0456621903],[3,19009,0.1049524237],[3,19016,0.0924934734],[3,19025,0.0681966972],[3,19026,0.0978199092],[3,19027,0.068085883],[3,19040,0.1573250843],[3,19058,0.0394421088],[3,19059,0.0614539347],[3,19060,0.0239795103],[3,19064,0.0338095496],[3,19065,0.0188707563],[3,19071,0.0318365025],[3,19097,0.0230282743],[3,19102,0.0203242637],[3,19104,0.0097019034],[3,19108,0.0063483262],[3,19109,0.0735947098],[3,19121,0.1585222486],[3,19122,0.0618250656],[3,19129,0.0995397726],[3,19134,0.0773160876],[3,19138,0.0271423421],[3,19141,0.005829231],[3,19144,0.3158607858],[3,19160,0.0605123605],[3,19162,0.0087065692],[3,19172,0.1516956711],[3,19178,0.0268483987],[3,19179,0.005829231],[3,19188,0.0239913879],[3,19199,0.0596457641],[3,19214,0.0128898018],[3,19217,0.1178215686],[3,19220,0.0322815251],[3,19231,0.1085357463],[3,19247,0.3056891761],[3,19258,0.008502426],[3,19267,0.1191397013],[3,19277,0.0616214034],[3,19278,0.0637753719],[3,19280,0.0500353127],[3,19284,0.0581865007],[3,19287,0.1912031489],[3,19292,0.0064189252],[3,19294,0.0612224991],[3,19304,0.0457411009],[3,19306,0.1302510603],[3,19310,0.0436051493],[3,19324,0.1049118593],[3,19325,0.0064189252],[3,19326,0.0599926235],[3,19332,0.0820454774],[3,19337,0.1530692363],[3,19349,0.0562622013],[3,19354,0.0547096938],[3,19360,0.0494409484],[3,19361,0.1149026827],[3,19369,0.0151736513],[3,19376,0.0391882377],[3,19377,0.0125850866],[3,19380,0.0816333075],[3,19385,0.017615528],[3,19392,0.1575803944],[3,19395,0.0063483262],[3,19407,0.0201695151],[3,19411,0.3220165643],[3,19415,0.1229661198],[3,19422,0.0320928979],[3,19424,0.0442275167],[3,19425,0.1978949566],[3,19433,0.2290962214],[3,19445,0.2084371322],[3,19465,0.1784837091],[3,19467,0.3969808988],[3,19468,0.0595814765],[3,19470,0.0311031728],[3,19472,0.1288663188],[3,19478,0.0978920574],[3,19483,0.2681493628],[3,19487,0.0245797541],[3,19488,0.0687136945],[3,19504,0.0522701056],[3,19514,0.0256632332],[3,19515,0.1829811513],[3,19521,0.0142305382],[3,19522,0.189122003],[3,19524,0.0058639036],[3,19532,0.0351706938],[3,19540,0.108635],[3,19541,0.1454759995],[3,19547,0.0141931415],[3,19551,0.038535244],[3,19552,0.0117773442],[3,19553,0.1215831608],[3,19586,0.0591041488],[3,19601,0.059443583],[3,19607,0.0118540339],[3,19611,0.0348246363],[3,19619,0.005829231],[3,19650,0.0513291772],[3,19659,0.0641524695],[3,19665,0.0169532078],[3,19670,0.0064189252],[3,19689,0.0583686284],[3,19691,0.0725183745],[3,19697,0.0812056011],[3,19716,0.1145609552],[3,19718,0.0703494549],[3,19729,0.0721258411],[3,19746,0.0891334393],[3,19747,0.0979889541],[3,19759,0.0064189252],[3,19760,0.0443120333],[3,19766,0.0439911958],[3,19771,0.2326518445],[3,19788,0.1264526812],[3,19793,0.0960037863],[3,19799,0.0581400558],[3,19802,0.015444278],[3,19818,0.0643739517],[3,19840,0.0072226616],[3,19849,0.0389800443],[3,19856,0.1004935883],[3,19867,0.0093539406],[3,19868,0.1836774077],[3,19927,0.0666551667],[3,19930,0.1136575729],[3,19931,0.0056988464],[3,19932,0.0064189252],[3,19943,0.1440308434],[3,19966,0.2383965027],[3,19973,0.0587507548],[3,19986,0.0463036214],[3,19991,0.1309483064],[3,19993,0.0824409675],[3,20011,0.0683827771],[3,20017,0.0671479044],[3,20018,0.1842541973],[3,20047,0.1571905765],[3,20052,0.0740799826],[3,20066,0.0643608382],[3,20074,0.0244063899],[3,20081,0.1269605303],[3,20090,0.0237383187],[3,20092,0.0780253597],[3,20093,0.0064189252],[3,20107,0.008502426],[3,20113,0.0049315178],[3,20118,0.0642516697],[3,20193,0.0791099502],[3,20208,0.0222732635],[3,20210,0.0085304894],[3,20218,0.0064189252],[3,20221,0.0064189252],[3,20225,0.0337586321],[3,20227,0.1150376615],[3,20229,0.3906784992],[3,20231,0.1296783993],[3,20235,0.0286332686],[3,20237,0.0174730335],[3,20245,0.3596119198],[3,20255,0.005556055],[3,20262,0.0063483262],[3,20266,0.1265043266],[3,20289,0.0235201271],[3,20292,0.0574614747],[3,20296,0.2375556765],[3,20301,0.0606078092],[3,20315,0.0085304894],[3,20330,0.0286705658],[3,20337,0.0651124628],[3,20341,0.0271675365],[3,20343,0.0443950188],[3,20350,0.0717700448],[3,20383,0.2126561348],[3,20407,0.0093539406],[3,20425,0.0671180599],[3,20433,0.0084279768],[3,20442,0.1793145039],[3,20447,0.071451084],[3,20458,0.0646777269],[3,20459,0.0107190386],[3,20466,0.1871737398],[3,20479,0.0305122468],[3,20487,0.0063483262],[3,20500,0.0107190386],[3,20518,0.0165345528],[3,20520,0.088978517],[3,20522,0.253519387],[3,20542,0.040798301],[3,20545,0.0056988464],[3,20551,0.0850477406],[3,20581,0.0311003944],[3,20597,0.1107590893],[3,20602,0.0269095167],[3,20603,0.014900088],[3,20612,0.2064526387],[3,20616,0.0075844967],[3,20620,0.0193749111],[3,20625,0.0790515273],[3,20632,0.0282029897],[3,20639,0.0527657356],[3,20654,0.0283215781],[3,20656,0.0488704689],[3,20657,0.0047115137],[3,20691,0.0632237872],[3,20706,0.0053003153],[3,20710,0.1056159537],[3,20723,0.0679432541],[3,20724,0.0590588829],[3,20725,0.1779203776],[3,20730,0.0072817024],[3,20735,0.1341749452],[3,20739,0.0418406802],[3,20744,0.0053003153],[3,20748,0.1640233724],[3,20762,0.0272823171],[3,20791,0.0383593681],[3,20796,0.0284095165],[3,20812,0.0320084193],[3,20824,0.1233871445],[3,20825,0.059952671],[3,20828,0.309177766],[3,20837,0.0097019034],[3,20864,0.0047115137],[3,20877,0.0829046805],[3,20885,0.0047115137],[3,20894,0.174555819],[3,20906,0.0973195458],[3,20909,0.0958910613],[3,20934,0.3498531694],[3,20978,0.0311426658],[3,20988,0.0329322186],[3,21000,0.0103202],[3,21002,0.0633957902],[3,21030,0.1548152957],[3,21054,0.0244028149],[3,21108,0.007807178],[3,21114,0.1082822527],[3,21123,0.0060393876],[3,21139,0.0053003153],[3,21155,0.2039403207],[3,21196,0.007807178],[3,21288,0.1048633473],[3,21340,0.0154277068],[3,21378,0.0795169601],[3,21394,0.0445248658],[3,21399,0.0648595804],[3,21402,0.0518901747],[3,21403,0.0042982381],[3,21406,0.1004568368],[3,21408,0.0039197471],[3,21410,0.007807178],[3,21414,0.0082308882],[3,21418,0.0370607553],[3,21423,0.0463129542],[3,21431,0.0070120871],[3,21445,0.1406544019],[3,21450,0.1411583843],[3,21460,0.2440492315],[3,21497,0.2438726276],[3,21507,0.2719401389],[3,21537,0.172969813],[3,21577,0.0988094579],[3,21622,0.0768429428],[3,21664,0.1516080313],[3,21778,0.1724751291],[3,21863,0.0506474576],[3,21869,0.1504293561],[3,21888,0.1654694737],[3,21923,0.0807335956],[3,22115,0.1310651652],[3,22161,0.0533612959],[3,22175,0.0485134329],[3,22179,0.027264497],[3,22242,0.0859332338],[3,22258,0.047125614],[3,22302,0.0492581945],[3,22499,0.0396345992],[3,22500,0.200183978],[3,22523,0.3667814612],[3,22536,0.3123417231],[3,22564,0.6206584114],[3,22567,0.1722927046],[3,22586,0.1711778223],[3,22587,0.06295919],[3,22594,0.1023873885],[3,22596,0.0647738112],[3,22679,0.3966732372],[3,22720,0.1127244328],[3,22739,0.1882800755],[3,22743,0.3695378538],[3,22793,0.0336866063],[3,22794,0.0271311122],[3,22838,0.1412927018],[3,22852,0.4302917713],[3,22854,0.0781483457],[3,22884,0.3851914426],[3,22899,0.0866887781],[3,23014,0.0959103148],[3,23027,0.1588690397],[3,23074,0.1006031646],[3,23077,0.0573149018],[3,23080,0.4617022275],[3,23081,0.3048513473],[3,23088,0.0373413259],[3,23093,0.8163660945],[3,23112,0.2257949419],[3,23136,0.2576758291],[3,23158,0.1128682705],[3,23163,0.0570348565],[3,23172,0.0594054446],[3,23177,0.0060393876],[3,23180,0.0060393876],[3,23192,0.0073806625],[3,23197,0.061959026],[3,23201,0.0137448304],[3,23207,0.0069691372],[3,23211,0.0063483262],[3,23213,0.1200378013],[3,23221,0.0069691372],[3,23224,0.0191182681],[3,23232,0.0047115137],[3,23233,0.0047115137],[3,23234,0.0069691372],[3,23235,0.0060393876],[3,23236,0.0907102597],[3,23241,0.0099718187],[3,23249,0.007807178],[3,23251,0.0042982381],[3,23254,0.0060393876],[3,23261,0.0190300919],[3,23262,0.007807178],[3,23263,0.061959026],[3,23276,0.0553305242],[3,23281,0.0154277068],[3,23293,0.0071427035],[3,23296,0.0082308882],[3,23298,0.007807178],[3,23299,0.0081755926],[3,23305,0.0060393876],[3,23310,0.013887483],[3,23315,0.007807178],[3,23320,0.0053003153],[3,23325,0.0053003153],[3,23328,0.0152291619],[3,23330,0.0047115137],[3,23331,0.0047115137],[3,23337,0.0073806625],[3,23342,0.0225934936],[3,23343,0.0039197471],[3,23345,0.0053003153],[3,23353,0.0069691372],[3,23355,0.0286658035],[3,23361,0.0072817024],[3,23364,0.0336301806],[3,23366,0.0053003153],[3,23375,0.0058639036],[3,23387,0.0039197471],[3,23390,0.0047115137],[3,23399,0.0060393876],[3,23404,0.0172804042],[3,23405,0.0485134329],[3,23407,0.0048468286],[3,23425,0.0060393876],[3,23431,0.0118393362],[3,23432,0.0047115137],[3,23434,0.0485134329],[3,23438,0.0060393876],[3,23444,0.0060393876],[3,23450,0.0053003153],[3,23459,0.0042982381],[3,23470,0.0291877485],[3,23473,0.0069691372],[3,23474,0.0099718187],[3,23478,0.0302394496],[3,23481,0.158758068],[3,23486,0.0274964386],[3,23490,0.0172804042],[3,23492,0.0053003153],[3,23511,0.0060393876],[3,23528,0.0042982381],[3,23529,0.0060393876],[3,23531,0.0047115137],[3,23534,0.0220093588],[3,23541,0.0069691372],[3,23547,0.0074064714],[3,23550,0.0135536708],[3,23551,0.0201333041],[3,23559,0.0042982381],[3,23560,0.0286658035],[3,23562,0.0280619403],[3,23565,0.0053003153],[3,23575,0.0216033731],[3,23580,0.1829090058],[3,23589,0.0472339429],[3,23595,0.0060393876],[3,23596,0.0053003153],[3,23597,0.0315490643],[3,23598,0.0053003153],[3,23603,0.0047115137],[3,23606,0.007807178],[3,23611,0.0060393876],[3,23612,0.0172804042],[3,23617,0.0039197471],[3,23631,0.0073806625],[3,23643,0.0054398903],[3,23653,0.0274964386],[3,23658,0.0060393876],[3,23659,0.0039197471],[3,23663,0.0047115137],[3,23679,0.0060393876],[3,23688,0.0047115137],[3,23692,0.0143034394],[3,23693,0.014900088],[3,23697,0.0073806625],[3,23698,0.0060393876],[3,23700,0.0172804042],[3,23707,0.0073806625],[3,23728,0.0053003153],[3,23729,0.0225934936],[3,23733,0.0071427035],[3,23749,0.0069691372],[3,23752,0.0060393876],[3,23754,0.0494282023],[3,23756,0.0075844967],[3,23770,0.061959026],[3,23777,0.0274964386],[3,23782,0.0058639036],[3,23788,0.0069691372],[3,23791,0.0177332989],[3,23813,0.0053003153],[3,23829,0.007807178],[3,23833,0.0247034435],[3,23839,0.0071427035],[3,23851,0.0172804042],[3,23852,0.0190300919],[3,23854,0.0060229982],[3,23881,0.0047115137],[3,23883,0.0047115137],[3,23914,0.0053003153],[3,23955,0.0069691372],[3,23958,0.0047115137],[3,23959,0.0109788562],[3,23965,0.0073806625],[3,23974,0.0229016923],[3,23982,0.0070120871],[3,23998,0.0047115137],[3,24010,0.0060393876],[3,24012,0.0060393876],[3,24014,0.0060393876],[3,24016,0.0073806625],[3,24026,0.0390783089],[3,24031,0.0069691372],[3,24032,0.0053003153],[3,24055,0.0507487981],[3,24068,0.0053003153],[3,24079,0.0047115137],[3,24080,0.0069691372],[3,24104,0.0095383268],[3,24108,0.0336613161],[3,24115,0.0053003153],[3,24132,0.007807178],[3,24155,0.0191987897],[3,24159,0.0060393876],[3,24165,0.0429206118],[3,24175,0.0152291619],[3,24182,0.0053003153],[3,24185,0.0053003153],[3,24203,0.0053003153],[3,24210,0.0071427035],[3,24226,0.0060923121],[3,24253,0.0336301806],[3,24260,0.007807178],[3,24269,0.0172804042],[3,24270,0.0109788562],[3,24274,0.0071427035],[3,24280,0.0336301806],[3,24337,0.0172804042],[3,24340,0.0069691372],[3,24341,0.0282493845],[3,24359,0.0063483262],[3,24365,0.0485134329],[3,24382,0.007807178],[3,24390,0.0053003153],[3,24394,0.0047115137],[3,24399,0.1177070762],[3,24411,0.0053003153],[3,24416,0.0219610535],[3,24428,0.0047115137],[3,24440,0.0071427035],[3,24444,0.0069691372],[3,24447,0.0060923121],[3,24450,0.0047115137],[3,24453,0.0340372785],[3,24458,0.0060923121],[3,24460,0.0039197471],[3,24475,0.0060923121],[3,24478,0.0437729604],[3,24492,0.0053003153],[3,24494,0.0060393876],[3,24501,0.0060393876],[3,24507,0.007807178],[3,24538,0.0047115137],[3,24543,0.0177332989],[3,24547,0.0053003153],[3,24551,0.0315490643],[3,24552,0.0060229982],[3,24555,0.0190300919],[3,24558,0.0047115137],[3,24565,0.0058639036],[3,24571,0.0047115137],[3,24580,0.0060393876],[3,24583,0.007807178],[3,24586,0.0073806625],[3,24597,0.0543671196],[3,24599,0.0063483262],[3,24606,0.0564010569],[3,24612,0.0048468286],[3,24615,0.0075844967],[3,24636,0.0060393876],[3,24644,0.0039197471],[3,24646,0.0055419896],[3,24653,0.0099718187],[3,24665,0.0547107828],[3,24667,0.0142344227],[3,24680,0.0286658035],[3,24686,0.0047115137],[3,24703,0.007807178],[3,24710,0.054044294],[3,24716,0.0073806625],[3,24749,0.0060229982],[3,24789,0.0286658035],[3,24799,0.0053003153],[3,24800,0.0053003153],[3,24814,0.0286658035],[3,24823,0.0191987897],[3,24830,0.0172804042],[3,24836,0.0073806625],[3,24841,0.0073806625],[3,24844,0.0177332989],[3,24845,0.0241537397],[3,24846,0.0069691372],[3,24849,0.0060923121],[3,24850,0.0172804042],[3,24852,0.0053003153],[3,24853,0.038917645],[3,24870,0.0553305242],[3,24875,0.0216539189],[3,24882,0.0274964386],[3,24919,0.1949612365],[3,24942,0.0461041214],[3,24964,0.0053003153],[3,24978,0.0322212997],[3,25004,0.0124557846],[3,25042,0.0053003153],[3,25048,0.0047115137],[3,25084,0.1988092463],[3,25102,0.0053003153],[3,25132,0.0047115137],[3,25155,0.0060393876],[3,25161,0.0060923121],[3,25178,0.0286658035],[3,25190,0.0053003153],[3,25241,0.0071427035],[3,25244,0.0073806625],[3,25246,0.0039197471],[3,25250,0.0191987897],[3,25260,0.0081755926],[3,25281,0.0053003153],[3,25285,0.0053003153],[3,25338,0.0191638608],[3,25354,0.0440987619],[3,25356,0.0672073408],[3,25357,0.0489566005],[3,25358,0.0095383268],[3,25396,0.0042982381],[3,25402,0.0283178049],[3,25429,0.0047115137],[3,25431,0.0053003153],[3,25438,0.0099718187],[3,25440,0.0053003153],[3,25442,0.0241537397],[3,25452,0.0047115137],[3,25454,0.100710394],[3,25462,0.0837050734],[3,25463,0.0109065181],[3,25464,0.0047115137],[3,25472,0.0047115137],[3,25479,0.0052298232],[3,25482,0.0042982381],[3,25485,0.0073806625],[3,25489,0.0052298232],[3,25490,0.0340534194],[3,25493,0.0099718187],[3,25505,0.0042982381],[3,25509,0.0053003153],[3,25512,0.0069691372],[3,25524,0.0353025079],[3,25535,0.0274964386],[3,25546,0.0047115137],[3,25548,0.0060393876],[3,25553,0.0374150529],[3,25558,0.0191987897],[3,25562,0.0633440957],[3,25567,0.0053003153],[3,25570,0.0039197471],[3,25578,0.0053003153],[3,25579,0.0053003153],[3,25584,0.0279472222],[3,25586,0.0030921879],[3,25589,0.0036796964],[3,25591,0.0160853741],[3,25592,0.0057554746],[3,25593,0.0036093591],[3,25607,0.0467979072],[3,25615,0.0518241577],[3,25616,0.049028363],[3,25617,0.0061810534],[3,25622,0.0319473423],[3,25624,0.0059588611],[3,25627,0.0521440256],[3,25636,0.0175476463],[3,25639,0.0044171455],[3,25640,0.0536000265],[3,25641,0.0030921879],[3,25642,0.0044171455],[3,25643,0.0069027762],[3,25649,0.0922138377],[3,25650,0.0053448531],[3,25662,0.0059588611],[3,25671,0.015633474],[3,25674,0.0057554746],[3,25683,0.0038189647],[3,25689,0.0279578544],[3,25696,0.0030921879],[3,25697,0.0536000265],[3,25708,0.0044171455],[3,25711,0.0030921879],[3,25714,0.0053448531],[3,25716,0.0023021602],[3,25717,0.0036796964],[3,25719,0.0030921879],[3,25727,0.0053448531],[3,25728,0.0844862876],[3,25731,0.0053448531],[3,25732,0.0055180382],[3,25742,0.0467979072],[3,25754,0.023840589],[3,25757,0.0521076334],[3,25764,0.0038189647],[3,25769,0.0160920354],[3,25772,0.0357713164],[3,25776,0.0124724499],[3,25794,0.0057554746],[3,25798,0.0036796964],[3,25799,0.0566531982],[3,25805,0.0318620531],[3,25808,0.0044171455],[3,25809,0.0030921879],[3,25817,0.0467979072],[3,25828,0.0030921879],[3,25829,0.0390722415],[3,25839,0.015633474],[3,25840,0.0030921879],[3,25841,0.0055180382],[3,25842,0.1067761202],[3,25843,0.0258270715],[3,25852,0.0057554746],[3,25886,0.0523166212],[3,25887,0.0073890087],[3,25890,0.0209348945],[3,25891,0.0258270715],[3,25893,0.004400792],[3,25898,0.0057554746],[3,25913,0.004400792],[3,25915,0.0030921879],[3,25917,0.0885241678],[3,25921,0.0258270715],[3,25929,0.0523166212],[3,25942,0.0030921879],[3,25953,0.0023021602],[3,25961,0.0319473423],[3,25962,0.0230505096],[3,25963,0.015633474],[3,25969,0.0121056651],[3,25970,0.0160853741],[3,25973,0.0044171455],[3,25977,0.0057554746],[3,25979,0.0036796964],[3,25981,0.0032272056],[3,25987,0.0083941639],[3,25988,0.0835334317],[3,25992,0.0030921879],[3,26000,0.0044171455],[3,26007,0.0055180382],[3,26009,0.0036796964],[3,26010,0.0044171455],[3,26015,0.0681195901],[3,26017,0.0178137041],[3,26030,0.015633474],[3,26036,0.0030921879],[3,26039,0.0126645309],[3,26043,0.0258270715],[3,26044,0.0669020953],[3,26054,0.0053448531],[3,26059,0.0758634727],[3,26060,0.004400792],[3,26068,0.0053448531],[3,26069,0.0550593402],[3,26070,0.0030921879],[3,26072,0.0093457658],[3,26079,0.0036796964],[3,26085,0.0032272056],[3,26086,0.0523166212],[3,26089,0.0030921879],[3,26093,0.004400792],[3,26102,0.0065486588],[3,26107,0.0030921879],[3,26109,0.0023021602],[3,26110,0.0023021602],[3,26113,0.0030921879],[3,26116,0.0544161572],[3,26117,0.0352601483],[3,26119,0.0044171455],[3,26120,0.0057554746],[3,26122,0.0036796964],[3,26140,0.0108035314],[3,26143,0.0269938681],[3,26145,0.0036796964],[3,26147,0.0467979072],[3,26150,0.0156425367],[3,26151,0.0044171455],[3,26159,0.0047266572],[3,26160,0.0087700061],[3,26168,0.0521076334],[3,26172,0.0061810534],[3,26174,0.0057554746],[3,26177,0.0173793189],[3,26178,0.0093457658],[3,26179,0.0023021602],[3,26181,0.049028363],[3,26188,0.0786723674],[3,26189,0.0044171455],[3,26191,0.0526387377],[3,26192,0.0258270715],[3,26193,0.0318637678],[3,26198,0.059415236],[3,26200,0.0030921879],[3,26203,0.0030921879],[3,26208,0.0036796964],[3,26213,0.0023021602],[3,26217,0.0160853741],[3,26218,0.0061810534],[3,26223,0.0030921879],[3,26231,0.022491714],[3,26239,0.0030921879],[3,26248,0.0924325777],[3,26253,0.0258270715],[3,26256,0.0550593402],[3,26258,0.0551635707],[3,26259,0.0055180382],[3,26261,0.0337112317],[3,26265,0.0030921879],[3,26273,0.0523166212],[3,26274,0.0879516267],[3,26277,0.0036796964],[3,26281,0.0080079245],[3,26284,0.0298132876],[3,26293,0.0602139705],[3,26296,0.0032272056],[3,26298,0.0467979072],[3,26310,0.0319473423],[3,26320,0.0059588611],[3,26331,0.0339357881],[3,26347,0.0500274706],[3,26349,0.0230402104],[3,26355,0.0030921879],[3,26356,0.025147937],[3,26361,0.0551635707],[3,26365,0.0405654481],[3,26370,0.0036796964],[3,26371,0.022491714],[3,26374,0.0032272056],[3,26377,0.0191502728],[3,26380,0.0258270715],[3,26383,0.0065486588],[3,26387,0.025301703],[3,26391,0.0030921879],[3,26399,0.015633474],[3,26405,0.0036796964],[3,26414,0.0030921879],[3,26429,0.0141749331],[3,26433,0.0258270715],[3,26437,0.0030921879],[3,26439,0.0628002121],[3,26440,0.0494908987],[3,26441,0.0030921879],[3,26448,0.0023021602],[3,26452,0.0258270715],[3,26469,0.0036796964],[3,26472,0.0175476463],[3,26481,0.015633474],[3,26490,0.0044171455],[3,26494,0.0030921879],[3,26497,0.0386134822],[3,26503,0.0137848456],[3,26510,0.0047459194],[3,26513,0.0484921985],[3,26532,0.0175476463],[3,26545,0.0057812268],[3,26549,0.0758634727],[3,26562,0.0044171455],[3,26569,0.1824205839],[3,26573,0.0036796964],[3,26575,0.0044171455],[3,26576,0.0072725643],[3,26584,0.0055180382],[3,26585,0.0145718137],[3,26588,0.0069903259],[3,26594,0.0057554746],[3,26606,0.0626555376],[3,26622,0.0209348945],[3,26632,0.0036093591],[3,26663,0.0270679059],[3,26664,0.0793450492],[3,26667,0.0069005657],[3,26673,0.0364805939],[3,26676,0.0482365191],[3,26679,0.0044171455],[3,26680,0.1231197476],[3,26691,0.0030921879],[3,26694,0.2413930268],[3,26740,0.0083409399],[3,26741,0.0098042332],[3,26750,0.0059588611],[3,26757,0.0989217089],[3,26768,0.0090597625],[3,26770,0.0265180942],[3,26771,0.0044171455],[3,26774,0.0039348743],[3,26793,0.0218528434],[3,26795,0.0036796964],[3,26801,0.037079816],[3,26804,0.0411616187],[3,26811,0.0044171455],[3,26812,0.0036796964],[3,26825,0.0593222762],[3,26832,0.0826414841],[3,26833,0.0265558971],[3,26834,0.0061810534],[3,26838,0.6136931342],[3,26839,0.0530231693],[3,26848,0.0297288897],[3,26858,0.0036093591],[3,26895,0.0023021602],[3,26900,0.0044171455],[3,26909,0.0053448531],[3,26914,0.0030921879],[3,26917,0.0061810534],[3,26926,0.06563332],[3,26931,0.0647555339],[3,26944,0.0378785646],[3,26946,0.0058112878],[3,26949,0.0549309641],[3,26957,0.0030921879],[3,26961,0.0466698909],[3,26967,0.0061810534],[3,26970,0.0068004887],[3,26973,0.0036796964],[3,26981,0.0273623091],[3,26987,0.1076323794],[3,26988,0.0210881249],[3,26994,0.0036796964],[3,26996,0.0647828487],[3,26998,0.1345064266],[3,27009,0.0258270715],[3,27010,0.0057554746],[3,27013,0.0845781847],[3,27015,0.3959578459],[3,27018,0.0376936958],[3,27022,0.0053448531],[3,27023,0.049028363],[3,27025,0.0624469607],[3,27030,0.0633604815],[3,27034,0.0036796964],[3,27035,0.0303053533],[3,27036,0.0030921879],[3,27038,0.0072423269],[3,27040,0.3590927077],[3,27044,0.01153605],[3,27045,0.2804236936],[3,27048,0.0675165114],[3,27050,0.2060410337],[3,27051,0.0614610281],[3,27053,0.0073419954],[3,27061,0.0216690987],[3,27072,0.0056567318],[3,27073,0.0529816462],[3,27083,0.0036796964],[3,27094,0.0059588611],[3,27104,0.0392079507],[3,27112,0.1426214252],[3,27120,0.0258270715],[3,27139,0.0202433353],[3,27161,0.082050027],[3,27169,0.1194912649],[3,27170,0.0036796964],[3,27179,0.0523166212],[3,27186,0.006593568],[3,27189,0.0069027762],[3,27194,0.0732142173],[3,27202,0.0567943698],[3,27203,0.0036796964],[3,27205,0.0055180382],[3,27211,0.0315767622],[3,27244,0.0763092624],[3,27264,0.0160853741],[3,27270,0.010204356],[3,27274,0.2074553594],[3,27287,0.0126645309],[3,27298,0.0093457658],[3,27319,0.0606912901],[3,27327,0.1164910715],[3,27340,0.0257399954],[3,27371,0.1300081387],[3,27372,0.0039348743],[3,27394,0.0023021602],[3,27398,0.0030921879],[3,27403,0.0072423269],[3,27409,0.0367153897],[3,27419,0.0057554746],[3,27421,0.0551635707],[3,27422,0.030109746],[3,27430,0.1475315259],[3,27432,0.0238801813],[3,27443,0.0044171455],[3,27449,0.0602139705],[3,27456,0.0239955412],[3,27457,0.0494720403],[3,27468,0.0288959162],[3,27476,0.0207244143],[3,27479,0.1250986238],[3,27481,0.0570819441],[3,27495,0.0209348945],[3,27505,0.015633474],[3,27516,0.0210881249],[3,27522,0.0036796964],[3,27523,0.0175476463],[3,27528,0.0044171455],[3,27531,0.1190962953],[3,27535,0.0550041224],[3,27536,0.0319473423],[3,27551,0.0806417349],[3,27569,0.0554515428],[3,27575,0.0550593402],[3,27581,0.0057554746],[3,27583,0.3429187071],[3,27586,0.0030921879],[3,27589,0.0281947489],[3,27590,0.0233068547],[3,27601,0.2468423478],[3,27604,0.004400792],[3,27613,0.0044171455],[3,27614,0.0255539345],[3,27620,0.0036093591],[3,27625,0.0094841895],[3,27635,0.0156425367],[3,27648,0.0093457658],[3,27652,0.4858437036],[3,27664,0.0052913172],[3,27669,0.2184823515],[3,27687,0.04694088],[3,27713,0.0044699537],[3,27715,0.0122480045],[3,27727,0.0299961821],[3,27729,0.0602139705],[3,27736,0.0055180382],[3,27741,0.0058112878],[3,27758,0.1278560916],[3,27761,0.0047254055],[3,27769,0.0646086641],[3,27776,0.0175476463],[3,27779,0.1419879579],[3,27786,0.0358182267],[3,27792,0.0191502728],[3,27794,0.0023021602],[3,27799,0.1966223685],[3,27819,0.0053877087],[3,27824,0.0220381623],[3,27827,0.0057554746],[3,27841,0.0026798199],[3,27847,0.0030921879],[3,27859,0.0044171455],[3,27868,0.0160853741],[3,27881,0.0030921879],[3,27887,0.0387232631],[3,27888,0.1071224251],[3,27891,0.0053448531],[3,27899,0.0036053407],[3,27905,0.0036796964],[3,27923,0.0042420468],[3,27924,0.0339335932],[3,27927,0.0175476463],[3,27979,0.0100311411],[3,27982,0.0400424456],[3,27985,0.0085818347],[3,27986,0.0036796964],[3,28014,0.0069251766],[3,28015,0.0321466007],[3,28066,0.0083409399],[3,28067,0.0036796964],[3,28069,0.0030921879],[3,28070,0.013185569],[3,28075,0.1239293394],[3,28076,0.1651352137],[3,28109,0.0080716175],[3,28128,0.0036796964],[3,28137,0.2382347525],[3,28156,0.1368869277],[3,28160,0.0114662253],[3,28186,0.0038189647],[3,28187,0.0044171455],[3,28193,0.0548848749],[3,28208,0.0023021602],[3,28214,0.060885396],[3,28218,0.0971422016],[3,28251,0.0964438432],[3,28252,0.0059588611],[3,28265,0.0295687398],[3,28267,0.1661658398],[3,28282,0.0574078359],[3,28284,0.2338670678],[3,28356,0.0038189647],[3,28390,0.070739309],[3,28397,0.0217364988],[3,28431,0.0433346226],[3,28454,0.0184439975],[3,28457,0.0451508506],[3,28479,0.1883026397],[3,28508,0.2905986156],[3,28511,0.0026798199],[3,28540,0.0030921879],[3,28552,0.0786599276],[3,28556,0.0044171455],[3,28571,0.0291126227],[3,28584,0.0173793189],[3,28591,0.0135867367],[3,28593,0.1648065442],[3,28595,0.0053448531],[3,28603,0.0030921879],[3,28607,0.1614042367],[3,28658,0.0036796964],[3,28707,0.0023021602],[3,28717,0.0431353711],[3,28719,0.3094123437],[3,28775,0.0038189647],[3,28776,0.0036796964],[3,28783,0.0036796964],[3,28816,0.0523166212],[3,28917,0.0442261845],[3,28925,0.0184084239],[3,28930,0.0036796964],[3,28937,0.0036796964],[3,28975,0.0036796964],[3,29000,0.0036796964],[3,29027,0.015633474],[3,29049,0.0036796964],[3,29050,0.0872300547],[3,29114,0.0036796964],[3,29121,0.1089755252],[3,29139,0.0053448531],[3,29155,0.0036796964],[3,29199,0.0030921879],[3,29206,0.0036796964],[3,29234,0.0036796964],[3,29263,0.0053448531],[3,29273,0.015633474],[3,29289,0.026431515],[3,29294,0.0147364708],[3,29299,0.004400792],[3,29316,0.0036796964],[3,29370,0.0055180382],[3,29410,0.0030921879],[3,29419,0.0550593402],[3,29446,0.0030921879],[3,29457,0.0843463187],[3,29459,0.104287498],[3,29476,0.0133546283],[3,29477,0.0190726945],[3,29481,0.0030921879],[3,29489,0.0044171455],[3,29490,0.025301703],[3,29504,0.0044171455],[3,29509,0.0036796964],[3,29513,0.0093457658],[3,29547,0.0055180382],[3,29548,0.0160853741],[3,29560,0.0643817102],[3,29607,0.0044171455],[3,29619,0.0255740115],[3,29627,0.2148376303],[3,29635,0.0044171455],[3,29638,0.0160853741],[3,29652,0.0550593402],[3,29653,0.0523166212],[3,29655,0.0030921879],[3,29676,0.0030921879],[3,29694,0.0053448531],[3,29699,0.0416922075],[3,29714,0.0039208398],[3,29716,0.0319473423],[3,29719,0.0314182301],[3,29725,0.0879516267],[3,29726,0.0909007663],[3,29753,0.0057554746],[3,29758,0.0057554746],[3,29768,0.1043719853],[3,29790,0.0128749798],[3,29795,0.028861144],[3,29808,0.1729200189],[3,29826,0.0789350382],[3,29831,0.01153605],[3,29841,0.015633474],[3,29851,0.1383604562],[3,29855,0.0264130935],[3,29867,0.0047459194],[3,29887,0.0555248934],[3,29931,0.0068004887],[3,29949,0.3025637867],[3,30035,0.0057554746],[3,30041,0.1107886549],[3,30043,0.004400792],[3,30045,0.1759127965],[3,30053,0.0269418359],[3,30058,0.0760027194],[3,30069,0.0849968819],[3,30071,0.0130182296],[3,30074,0.0275147628],[3,30086,0.3462321527],[3,30087,0.0280484997],[3,30122,0.0453319187],[3,30126,0.1285654432],[3,30152,0.0030921879],[3,30163,0.0140064378],[3,30188,0.0068004887],[3,30197,0.0334504409],[3,30259,0.3225659994],[3,30298,0.0300923047],[3,30308,0.2101131911],[3,30311,0.005878674],[3,30349,0.1581997249],[3,30400,0.0047254055],[3,30418,0.0036796964],[3,30492,0.0221296256],[3,30496,0.0061810534],[3,30499,0.3854626164],[3,30502,0.0044699537],[3,30511,0.1165964194],[3,30518,0.0529816462],[3,30521,0.0501209771],[3,30524,0.1378425925],[3,30528,0.1556795752],[3,30554,0.0044171455],[3,30589,0.0073890087],[3,30606,0.4810544118],[3,30624,0.0061810534],[3,30650,0.0146751763],[3,30696,0.0098689615],[3,30704,0.1632724325],[3,30707,0.1316944049],[3,30709,0.2094684725],[3,30721,0.0205820078],[3,30724,0.4329755249],[3,30744,0.004400792],[3,30746,0.0233946036],[3,30765,0.0534356461],[3,30790,0.0692680116],[3,30847,0.0036796964],[3,30858,0.0036796964],[3,30861,0.0147364708],[3,30868,0.0580720757],[3,30888,0.0666131642],[3,30889,0.0057554746],[3,30898,0.0036796964],[3,30905,0.0773352773],[3,30947,0.0061810534],[3,30959,0.0036796964],[3,30969,0.0529816462],[3,31022,0.0036796964],[3,31032,0.0030921879],[3,31157,0.1949874865],[3,31163,0.1044872723],[3,31165,0.0026798199],[3,31170,0.0288971465],[3,31185,0.0440060546],[3,31199,0.0162451025],[3,31208,0.0450836803],[3,31229,0.025301703],[3,31231,0.0375211591],[3,31247,0.015633474],[3,31250,0.5996537813],[3,31289,0.0023021602],[3,31297,0.0217780454],[3,31333,0.0036796964],[3,31351,0.0044171455],[3,31364,0.0030921879],[3,31386,0.0036796964],[3,31412,0.015633474],[3,31420,0.0036796964],[3,31425,0.0061810534],[3,31446,0.0026798199],[3,31456,0.0269938681],[3,31487,0.015633474],[3,31498,0.0039348743],[3,31501,0.0061810534],[3,31523,0.0042420468],[3,31533,0.0036796964],[3,31548,0.0030921879],[3,31578,0.0030921879],[3,31595,0.0523166212],[3,31622,0.0069027762],[3,31623,0.0044699537],[3,31672,0.0023021602],[3,31679,0.0065486588],[3,31728,0.0523166212],[3,31736,0.0030921879],[3,31740,0.0057554746],[3,31741,0.0521076334],[3,31751,0.0023021602],[3,31754,0.0209348945],[3,31768,0.0285640581],[3,31773,0.0030921879],[3,31799,0.0038189647],[3,31804,0.0036796964],[3,31817,0.0529816462],[3,31832,0.0047254055],[3,31900,0.0036796964],[3,31908,0.0083409399],[3,31955,0.0160853741],[3,32000,0.0036796964],[3,32017,0.0264728943],[3,32026,0.0122480045],[3,32030,0.0036796964],[3,32035,0.0042420468],[3,32052,0.0036796964],[3,32065,0.0036796964],[3,32080,0.0030921879],[3,32095,0.0023021602],[3,32102,0.0137848456],[3,32109,0.0036796964],[3,32121,0.0036093591],[3,32167,0.0044699537],[3,32172,0.0053448531],[3,32182,0.0181485356],[3,32191,0.0030921879],[3,32226,0.004400792],[3,32270,0.0285640581],[3,32301,0.015633474],[3,32305,0.0087706953],[3,32316,0.0340253685],[3,32380,0.0069903259],[3,32447,0.0184801082],[3,32451,0.0175476463],[3,32502,0.0163851739],[3,32517,0.0036796964],[3,32528,0.0036796964],[3,32550,0.0217272886],[3,32564,0.0146751763],[3,32569,0.0059588611],[3,32599,0.0319473423],[3,32711,0.0285640581],[3,32728,0.0040773522],[3,32737,0.0319473423],[3,32738,0.0068004887],[3,32746,0.0606997086],[3,32751,0.0042420468],[3,32753,0.0221100858],[3,32774,0.0030921879],[3,32783,0.0057554746],[3,32788,0.0030921879],[3,32838,0.0036796964],[3,32850,0.0030921879],[3,32860,0.0036796964],[3,32868,0.0093457658],[3,32934,0.0160853741],[3,32988,0.0038189647],[3,32990,0.0121056651],[3,32997,0.0343354603],[3,33028,0.0036796964],[3,33120,0.1686219193],[3,33152,0.0260494534],[3,33159,0.0175476463],[3,33167,0.0053448531],[3,33187,0.0030921879],[3,33188,0.0036796964],[3,33197,0.0057554746],[3,33213,0.0708355217],[3,33233,0.0030921879],[3,33243,0.0030921879],[3,33247,0.0030921879],[3,33261,0.0550593402],[3,33305,0.06171615],[3,33309,0.0044171455],[3,33312,0.0030921879],[3,33314,0.0044444651],[3,33317,0.0036053407],[3,33332,0.015633474],[3,33333,0.0044171455],[3,33336,0.0053448531],[3,33349,0.0651259951],[3,33358,0.0164920914],[3,33380,0.1937087921],[3,33382,0.0087700061],[3,33388,0.0133615665],[3,33394,0.0057554746],[3,33419,0.3693581485],[3,33431,0.0026798199],[3,33436,0.073719265],[3,33455,0.0564318415],[3,33462,0.0061810534],[3,33502,0.0258270715],[3,33516,0.0319849153],[3,33529,0.0889583274],[3,33530,0.0036796964],[3,33531,0.0023021602],[3,33537,0.0306589908],[3,33544,0.0280313672],[3,33572,0.0047254055],[3,33577,0.233771581],[3,33590,0.0269614047],[3,33591,0.149984524],[3,33600,0.0047254055],[3,33610,0.006593568],[3,33620,0.0545149981],[3,33628,0.0602139705],[3,33664,0.0030921879],[3,33690,0.0036796964],[3,33692,0.0039348743],[3,33695,0.044234231],[3,33708,0.0057554746],[3,33712,0.0465878625],[3,33717,0.0023021602],[3,33720,0.033013206],[3,33732,0.006415781],[3,33746,0.0326396117],[3,33749,0.02788973],[3,33756,0.0467979072],[3,33824,0.0047254055],[3,33828,0.052847867],[3,33838,0.0295288312],[3,33843,0.0452802203],[3,33851,0.1260850361],[3,33852,0.0036796964],[3,33872,0.0352671136],[3,33909,0.1297240849],[3,33915,0.0501196907],[3,33969,0.2357207697],[3,34004,0.0657758663],[3,34050,0.1077789669],[3,34064,0.0521250051],[3,34094,0.052847867],[3,34096,0.0090597625],[3,34098,0.0065486588],[3,34114,0.164491694],[3,34144,0.0061810534],[3,34192,0.2863804622],[3,34284,0.0072423269],[3,34389,0.0279097489],[3,34391,0.4676089464],[3,34426,0.2506964858],[3,34468,0.021099552],[3,34489,0.0036796964],[3,34492,0.0536000265],[3,34499,0.0339511357],[3,34514,0.3125002504],[3,34526,0.3065798344],[3,34605,0.2566681239],[3,34642,0.026223554],[3,34723,0.0044171455],[3,34738,0.0307633398],[3,34771,0.4699880471],[3,34805,0.0928586061],[3,34909,0.0875654886],[3,35023,0.205543385],[3,35056,0.0057554746],[3,35101,0.0373825956],[3,35142,0.0030921879],[3,35147,0.4259856049],[3,35153,0.1225936355],[3,35177,0.0030921879],[3,35185,0.0146751763],[3,35194,0.1979765093],[3,35209,0.0651708598],[3,35229,0.1745704015],[3,35517,0.010827195],[3,35654,0.0044699537],[3,35896,0.0789735905],[3,35994,0.0036796964],[3,36030,0.1673355832],[3,36035,0.0348704571],[3,36067,0.0035401774],[3,36099,0.0493179024],[3,36191,0.0124732637],[3,36208,0.0036796964],[3,36214,0.0148892607],[3,36218,0.0728293982],[3,36220,0.0030921879],[3,36235,0.0617763774],[3,36239,0.0271726162],[3,36252,0.3670036994],[3,36264,0.1427205522],[3,36289,0.0036093591],[3,36310,0.0038189647],[3,36326,0.0057554746],[3,36346,0.0579528132],[3,36412,0.0023021602],[3,36424,0.0191748504],[3,36454,0.199921775],[3,36464,0.0044171455],[3,36564,0.0055180382],[3,36594,0.0030921879],[3,36642,0.0057554746],[3,36680,0.0759344076],[3,36691,0.0032962541],[3,36713,0.0057963084],[3,36731,0.0182435946],[3,36769,0.0043414186],[3,36786,0.0268759013],[3,36789,0.0525725284],[3,36836,0.0237524596],[3,36863,0.0171569818],[3,36883,0.0053709513],[3,36891,0.0057963084],[3,36951,0.0034354499],[3,37022,0.0040169742],[3,37042,0.087780674],[3,37094,0.0032962541],[3,37158,0.0743856895],[3,37159,0.0876145905],[3,37166,0.0252752475],[3,37173,0.0463920098],[3,37201,0.6534673553],[3,37220,0.0043414186],[3,37241,0.0696953973],[3,37247,0.0521164057],[3,37249,0.1082359093],[3,37251,0.0118600999],[3,37257,0.1221020683],[3,37268,0.0285608354],[3,37278,0.0025506707],[3,37307,0.1536267722],[3,37321,0.0648471024],[3,37323,0.0065176554],[3,37324,0.0974987112],[3,37333,0.0158551163],[3,37355,0.0368041314],[3,37365,0.1436356708],[3,37366,0.0271195359],[3,37370,0.0786541181],[3,37401,0.1710074636],[3,37441,0.0528858523],[3,37457,0.0040169742],[3,37461,0.0246365234],[3,37484,0.0516041453],[3,37535,0.1049545748],[3,37539,0.1138064658],[3,37587,0.0053709513],[3,37606,0.2891021817],[3,37613,0.0377845676],[3,37664,0.0537035271],[3,37686,0.0027090516],[3,37687,0.0032962541],[3,37709,0.075031077],[3,37735,0.095016656],[3,37760,0.0644234822],[3,37767,0.0172480263],[3,37771,0.0510940375],[3,37780,0.0032962541],[3,37797,0.0179760972],[3,37800,0.0946323711],[3,37802,0.0032962541],[3,37849,0.02650266],[3,37859,0.0072535045],[3,37863,0.0684091362],[3,37869,0.6718305206],[3,37883,0.1555071228],[3,37905,0.1404990013],[3,37941,0.0916401527],[3,37942,0.0196442271],[3,37946,0.0019194353],[3,37983,0.0268316853],[3,37986,0.1644756957],[3,38021,0.0239449963],[3,38031,0.1630904246],[3,38067,0.4160620547],[3,38091,0.0342901524],[3,38097,0.3245368684],[3,38108,0.1077083869],[3,38110,0.0027090516],[3,38127,0.1502452936],[3,38153,0.0621948802],[3,38159,0.0557238576],[3,38258,0.3745660304],[3,38284,0.0032962541],[3,38313,0.0883390492],[3,38315,0.0372057834],[3,38327,0.2552632789],[3,38333,0.0317754722],[3,38372,0.0040333191],[3,38405,0.1619168503],[3,38471,0.1711094654],[3,38475,0.0040169742],[3,38476,0.1109129501],[3,38533,0.0528773479],[3,38580,0.1126120452],[3,38598,0.0562054395],[3,38604,0.1930700944],[3,38613,0.2928007095],[3,38646,0.1022861671],[3,38652,0.1763715808],[3,38690,0.0031568078],[3,38694,0.2433873207],[3,38702,0.1229225025],[3,38715,0.0179913432],[3,38723,0.0225171361],[3,38730,0.2994161944],[3,38740,0.0634945292],[3,38775,0.3970975978],[3,38785,0.0022968983],[3,38809,0.0519078497],[3,38857,0.0062086082],[3,38888,0.0671660937],[3,38943,0.0226466855],[3,38960,0.0499966532],[3,38963,0.0702556285],[3,38973,0.0305218866],[3,39011,0.0152438064],[3,39017,0.0032962541],[3,39029,0.0053709513],[3,39036,0.0526801844],[3,39047,0.0032962541],[3,39066,0.0445407212],[3,39123,0.0584270893],[3,39146,0.0032962541],[3,39150,0.0032962541],[3,39157,0.0040169742],[3,39180,0.0216940443],[3,39206,0.0051336386],[3,39246,0.0200567021],[3,39254,0.0049605436],[3,39381,0.0040333191],[3,39389,0.047940764],[3,39392,0.0055742318],[3,39439,0.0040333191],[3,39465,0.0152438064],[3,39466,0.0076605332],[3,39469,0.0032259534],[3,39482,0.0477171331],[3,39489,0.0306004668],[3,39502,0.0022968983],[3,39504,0.0525725284],[3,39514,0.0040860998],[3,39516,0.0022968983],[3,39517,0.0027090516],[3,39535,0.0607673036],[3,39566,0.0055742318],[3,39570,0.150721876],[3,39584,0.0032259534],[3,39594,0.0550609531],[3,39603,0.1934197057],[3,39608,0.0227544409],[3,39654,0.0404856892],[3,39660,0.0266288669],[3,39662,0.0310155327],[3,39679,0.0019194353],[3,39688,0.3261441137],[3,39719,0.0032962541],[3,39725,0.0919867869],[3,39761,0.0394011231],[3,39789,0.1840498135],[3,39803,0.0614414491],[3,39839,0.043778581],[3,39904,0.0195885938],[3,39926,0.0265982842],[3,39981,0.0078305976],[3,39998,0.119540292],[3,40014,0.008887231],[3,40018,0.0910872914],[3,40042,0.0660169441],[3,40144,0.2210830902],[3,40158,0.1078508314],[3,40160,0.1205634209],[3,40163,0.0083846017],[3,40164,0.2043751698],[3,40165,0.0457065197],[3,40166,0.6582449787],[3,40170,0.0040606246],[3,40173,0.2922945111],[3,40258,0.0104847441],[3,40301,0.0529579997],[3,40332,0.04130136],[3,40352,0.2796633009],[3,40412,0.0226466855],[3,40414,0.0032962541],[3,40454,0.2197000704],[3,40462,0.1677957498],[3,40537,0.0019194353],[3,40576,0.0032962541],[3,40600,0.0371708232],[3,40615,0.0171569818],[3,40654,0.1419928127],[3,40660,0.1557586073],[3,40698,0.0032962541],[3,40714,0.0032962541],[3,40722,0.0483115782],[3,40762,0.1796089545],[3,40766,0.0899031404],[3,40874,0.0573567883],[3,40880,0.0347379454],[3,40906,0.0061637224],[3,40913,0.0032962541],[3,40918,0.0050033769],[3,40936,0.0055742318],[3,40963,0.0075227556],[3,40998,0.0043426697],[3,41037,0.0053709513],[3,41057,0.0053709513],[3,41059,0.0144999806],[3,41070,0.025475926],[3,41088,0.0035512991],[3,41102,0.0133961407],[3,41118,0.0139632032],[3,41131,0.0065176554],[3,41169,0.0022968983],[3,41181,0.0046532091],[3,41189,0.0064154211],[3,41198,0.0032962541],[3,41211,0.0318724353],[3,41232,0.0032962541],[3,41256,0.0847085517],[3,41263,0.4371876303],[3,41270,0.3299521236],[3,41298,0.0959201282],[3,41301,0.1004248696],[3,41303,0.1575010081],[3,41332,0.2545717113],[3,41340,0.0278179471],[3,41363,0.0121781786],[3,41380,0.0413233014],[3,41384,0.1004926659],[3,41405,0.6068123825],[3,41427,0.3782997085],[3,41432,0.006515446],[3,41436,0.0068570292],[3,41505,0.0570374987],[3,41508,0.0670124349],[3,41513,0.0032962541],[3,41531,0.0171569818],[3,41547,0.1137696599],[3,41548,0.0794132718],[3,41565,0.0040333191],[3,41573,0.0034354499],[3,41614,0.0591314256],[3,41754,0.0342938368],[3,42515,0.0032962541],[3,43422,0.0662470607],[3,43458,0.0903425614],[3,43467,0.1385090884],[3,43479,0.0089593727],[3,43483,0.0831704347],[3,43513,0.1129188845],[3,43533,0.0258623187],[3,43582,0.0733542253],[3,43692,0.0528135632],[3,43696,0.159529068],[3,43720,0.3937743751],[3,43850,0.0032962541],[3,43863,0.1018146754],[3,43864,0.0051336386],[3,43879,0.6503630105],[3,43887,0.0757298752],[3,43890,0.037136177],[3,43892,0.0336503208],[3,43894,0.0032962541],[3,43898,0.0651984781],[3,43904,0.1475445078],[3,43914,0.0384253111],[3,43958,0.0404842126],[3,43959,0.0705286786],[3,43972,0.0089593727],[3,43986,0.0516989708],[3,43998,0.1366301477],[3,44015,0.0032814696],[3,44017,0.2679940409],[3,44046,0.0040333191],[3,44066,0.1768541426],[3,44127,0.005774245],[3,44157,0.0079550701],[3,44161,0.0116014055],[3,44204,0.0622531769],[3,44260,0.0022952113],[3,44276,0.0215070662],[3,44373,0.0054267354],[3,44388,0.0597486601],[3,44393,0.0032962541],[3,44507,0.0057963084],[3,44555,0.008127115],[3,44581,0.055417295],[3,44585,0.0297069465],[3,44619,0.0049605436],[3,44649,0.0057963084],[3,44753,0.0016641951],[3,44856,0.0748761367],[3,45125,0.0040853369],[3,45201,0.0249465094],[3,45203,0.0053709513],[3,45224,0.0054267354],[3,45272,0.02948822],[3,45275,0.0040333191],[3,45277,0.5668021814],[3,45283,0.0917965326],[3,45296,0.0832936749],[3,45301,0.0344094964],[3,45316,0.0029662444],[3,45354,0.0328539826],[3,45447,0.3691795736],[3,45450,0.0115345796],[3,45509,0.0624654074],[3,45514,0.0040169742],[3,45526,0.203673664],[3,45538,0.0315041548],[3,45560,0.0062086082],[3,45562,0.0115452745],[3,45602,0.0739576801],[3,45611,0.0884389216],[3,45632,0.0211624035],[3,45640,0.0061637224],[3,45674,0.0365180472],[3,45680,0.002041527],[3,45683,0.003179683],[3,45689,0.0032962541],[3,45749,0.1846715205],[3,45768,0.0027090516],[3,45781,0.0061586187],[3,45783,0.03922406],[3,45790,0.1426643938],[3,45800,0.1112769635],[3,45805,0.0333856929],[3,45815,0.0851110745],[3,45819,0.1105873463],[3,45857,0.0061586187],[3,45896,0.0179508316],[3,45922,0.0211754591],[3,45937,0.0393010778],[3,45960,0.0054267354],[3,46002,0.0507217218],[3,46040,0.0051402416],[3,46046,0.0171549872],[3,46052,0.0040860998],[3,46087,0.0032962541],[3,46095,0.2374412297],[3,46112,0.0075056691],[3,46151,0.0016641951],[3,46201,0.0040853369],[3,46210,0.0032962541],[3,46230,0.0022952113],[3,46337,0.5149410207],[3,46366,0.0032962541],[3,46400,0.0027090516],[3,46433,0.0205835389],[3,46535,0.002041527],[3,46754,0.0032962541],[3,46784,0.0062608174],[3,46829,0.0032962541],[3,46838,0.0215070662],[3,46900,0.0040333191],[3,46933,0.0061637224],[3,46960,0.0087221255],[3,47044,0.0244483869],[3,47100,0.0050158547],[3,47101,0.0227872336],[3,47105,0.0036023978],[3,47106,0.0036023978],[3,47134,0.0053709513],[3,47163,0.0051402416],[3,47196,0.0128477057],[3,47253,0.0032962541],[3,47404,0.0032962541],[3,47485,0.0016641951],[3,47546,0.0157047542],[3,47551,0.0036023978],[3,47595,0.0040333191],[3,47969,0.0032814696],[3,48027,0.0047470651],[3,48053,0.0062586088],[3,48075,0.0027090516],[3,48098,0.0032962541],[3,48108,0.0247641695],[3,48156,0.0283589472],[3,48161,0.0572104451],[3,48205,0.0016641951],[3,48236,0.0032962541],[3,48329,0.0032962541],[3,48332,0.0036023978],[3,48342,0.0218316053],[3,48360,0.0219822371],[3,48410,0.0032962541],[3,48420,0.0243099111],[3,48439,0.0239620298],[3,48556,0.0027090516],[3,48580,0.0040860998],[3,48644,0.0040169742],[3,48736,0.0262337062],[3,48747,0.0032814696],[3,49013,0.0040860998],[3,49092,0.0258787765],[3,49111,0.0190550181],[3,49118,0.0155950337],[3,49164,0.0032962541],[3,49170,0.0226514342],[3,49243,0.0040853369],[3,49244,0.0888223113],[3,49268,0.0032962541],[3,49516,0.0836049298],[3,49534,0.0280664296],[3,49543,0.0168616248],[3,49555,0.0027090516],[3,49571,0.0027090516],[3,49576,0.0127684228],[3,49593,0.0278916413],[3,49610,0.0681748242],[3,49614,0.0228188483],[3,49617,0.0549242503],[3,49767,0.018497682],[3,49805,0.0542986743],[3,49833,0.0040853369],[3,49839,0.0689152636],[3,49874,0.0315942076],[3,50040,0.0032962541],[3,50050,0.0862876704],[3,50051,0.0463519991],[3,50059,0.255045887],[3,50153,0.4987179628],[3,50195,0.0027090516],[3,50519,0.0249113998],[3,50686,0.1609878613],[3,50769,0.0027090516],[3,50811,0.0600562054],[3,50890,0.0440619819],[3,50914,0.0469373543],[3,50962,0.0832934726],[3,50977,0.0027090516],[3,51008,0.3682102901],[3,51088,0.0878203943],[3,51137,0.2192833416],[3,51144,0.0463920098],[3,51295,0.0158733639],[3,51416,0.0299782132],[3,51495,0.0332498897],[3,51510,0.0547204194],[3,51534,0.1079804602],[3,51635,0.0205424661],[3,51659,0.0235015812],[3,51688,0.0032962541],[3,51756,0.013786025],[3,51798,0.0325903952],[3,51845,0.0036023978],[3,51938,0.0029702593],[3,52291,0.0126662743],[3,52377,0.0060738417],[3,52557,0.0411265016],[3,52563,0.0329508484],[3,52568,0.037935021],[3,52615,0.0620822158],[3,52644,0.3442246285],[3,52660,0.0187902733],[3,52689,0.0166634878],[3,52693,0.3316031848],[3,52696,0.1080355779],[3,52704,0.2430473589],[3,52709,0.1780339008],[3,52715,0.650996275],[3,52727,0.1735414983],[3,52738,0.0955677741],[3,52750,0.1049972184],[3,52775,0.1236282256],[3,52781,0.7064710477],[3,52787,0.2453506393],[3,52789,0.0525731839],[3,52790,0.0078494963],[3,52792,0.3687575042],[3,52805,0.105572426],[3,52831,0.0798861823],[3,52839,0.7698466153],[3,52851,0.0930355902],[3,52856,0.127152167],[3,52865,0.0469124191],[3,52897,0.1118793306],[3,52911,0.1549237253],[3,52913,0.1110884963],[3,52916,0.2548457891],[3,52927,0.096199502],[3,52953,0.0365404261],[3,52956,0.1740596827],[3,52982,0.3835619663],[3,53069,0.1902074267],[3,53082,0.351965976],[3,53089,0.0040606246],[3,53106,0.2642082275],[3,53113,0.0757943225],[3,53114,0.6680644305],[3,53116,0.077707415],[3,53149,0.2344802615],[3,53160,0.0685037172],[3,53172,0.1767955593],[3,53218,0.1959432784],[3,53219,0.3975260934],[3,53231,0.1528100821],[3,53234,0.7207787105],[3,53241,0.0062188678],[3,53252,0.2412587156],[3,53255,0.1261292892],[3,53258,0.1387785106],[3,53272,0.0828168518],[3,53280,0.0821019676],[3,53297,0.0617422016],[3,53303,0.143837711],[3,53311,0.3714650082],[3,53316,0.2925970915],[3,53318,0.1617663878],[3,53319,0.0627092923],[3,53326,0.4634141382],[3,53345,0.1050730922],[3,53358,0.1461357771],[3,53394,0.2852495479],[3,53397,0.1049707239],[3,53424,0.1068792675],[3,53427,0.0632407852],[3,53438,0.2098406324],[3,53515,0.1827988414],[3,53561,0.0861093279],[3,53563,0.2659178104],[3,53566,0.0758174831],[3,53601,0.1274523423],[3,53608,0.0248327552],[3,53625,0.0671511647],[3,53648,0.2168415955],[3,53651,0.014514001],[3,53660,0.4298817745],[3,53665,0.1643497892],[3,53666,0.4113010088],[3,53679,0.1375599566],[3,53703,0.0574813325],[3,53714,0.1492719009],[3,53719,0.190689351],[3,53731,0.1670311139],[3,53754,0.257965532],[3,53757,0.204149578],[3,53769,0.8354035797],[3,53783,0.0065347688],[3,53803,0.0574634386],[3,53827,0.1366848587],[3,53828,0.1494459555],[3,53842,0.0325968969],[3,53845,0.0852819128],[3,53863,0.3894434458],[3,53868,0.138703692],[3,53902,0.09694921],[3,53907,0.0065347688],[3,53930,0.0053709513],[3,53937,0.1690621226],[3,53968,0.1630406259],[3,53998,0.0062188678],[3,54007,0.0695176165],[3,54033,0.0539982021],[3,54036,0.0655374245],[3,54053,0.0269387113],[3,54057,0.1448375608],[3,54062,0.1104346889],[3,54071,0.2252223152],[3,54074,0.1395785563],[3,54110,0.0016641951],[3,54114,0.2470993101],[3,54115,0.1763206491],[3,54119,0.2431637936],[3,54126,0.2629957824],[3,54144,0.0801895972],[3,54158,0.1507162776],[3,54174,0.0087002607],[3,54198,0.0326150763],[3,54204,0.1702939443],[3,54217,0.0068570292],[3,54231,0.0308958967],[3,54238,0.139015765],[3,54266,0.3068478734],[3,54270,0.0385852639],[3,54319,0.0755688768],[3,54321,0.0867028409],[3,54322,0.1351068371],[3,54326,0.0062188678],[3,54337,0.17448914],[3,54355,0.129773298],[3,54369,0.0670129061],[3,54383,0.0822965787],[3,54391,0.1090949228],[3,54392,0.0065347688],[3,54401,0.2492589806],[3,54409,0.024432455],[3,54444,0.1544776731],[3,54463,0.0040606246],[3,54469,0.267220338],[3,54492,0.0850918629],[3,54504,0.7644855398],[3,54530,0.0049070356],[3,54570,0.0985887835],[3,54580,0.1874636956],[3,54584,0.0946137353],[3,54634,0.2073374434],[3,54637,0.0631159621],[3,54645,0.0062086082],[3,54654,0.0820334714],[3,54673,0.1371012781],[3,54707,0.1453813541],[3,54714,0.1999743344],[3,54718,0.1543957501],[3,54724,0.2926022347],[3,54761,0.7569555763],[3,54778,0.032584413],[3,54783,0.0560787042],[3,54784,0.5132967443],[3,54796,0.0422677161],[3,54844,0.2280601898],[3,54847,0.2312244825],[3,54870,0.0901371239],[3,54877,0.1603660275],[3,54880,0.1000885844],[3,54882,0.2153605742],[3,54888,0.1408616313],[3,54899,0.6163605913],[3,54904,0.1514630282],[3,54905,0.1740723474],[3,54915,0.2448218755],[3,54916,0.0968979696],[3,54917,0.3687378917],[3,54919,0.0370769149],[3,54921,0.1198491887],[3,54922,0.1920118275],[3,54923,0.0729161583],[3,54926,0.1498144919],[3,54928,0.1905160083],[3,54931,0.0914631044],[3,54932,0.3332539908],[3,54933,0.1423637148],[3,54934,0.1141546181],[3,54936,0.6408462451],[3,54937,0.1920908371],[3,54940,0.1874580036],[3,54943,0.1235598902],[3,54955,0.0350108272],[3,54972,0.14927706],[3,54981,0.2718685564],[3,54993,0.051809823],[3,54996,0.0648615973],[3,54997,0.1459452733],[3,54998,0.1991331192],[3,54999,0.1079412302],[3,55003,0.1782391584],[3,55004,0.0959297478],[3,55005,0.1700340969],[3,55017,0.1322908075],[3,55020,0.5087604728],[3,55022,0.1686836815],[3,55027,0.1163844918],[3,55044,0.404592244],[3,55045,0.270486958],[3,55049,0.1282859982],[3,55050,0.1348782623],[3,55051,0.1245439831],[3,55052,0.5417542968],[3,55059,0.0878411762],[3,55060,0.657769494],[3,55065,0.0315491787],[3,55066,0.0310725068],[3,55071,0.2268926345],[3,55072,0.1233906854],[3,55074,0.1165565047],[3,55075,0.1188789701],[3,55077,0.0481495002],[3,55081,0.1525583919],[3,55083,0.0994913554],[3,55084,0.1226260836],[3,55085,0.4128449582],[3,55086,0.5298749502],[3,55087,0.0864181082],[3,55091,0.0623509105],[3,55093,0.1666519091],[3,55102,0.5087740678],[3,55108,0.2012467614],[3,55109,0.1406663016],[3,55110,0.1635822905],[3,55113,0.0381761421],[3,55114,0.1010295694],[3,55116,0.0688837659],[3,55117,0.2744816512],[3,55118,0.1010216114],[3,55119,0.0958662284],[3,55120,0.1910394482],[3,55121,0.1486171322],[3,55132,0.2568346312],[3,55133,0.2811557159],[3,55134,0.1523485297],[3,55137,0.1291402116],[3,55138,0.2366372419],[3,55139,0.0332218372],[3,55140,0.2912236271],[3,55141,0.236062415],[3,55142,0.0968494893],[3,55145,0.833430426],[3,55156,0.2350444507],[3,55159,0.0079550701],[3,55163,0.0854068317],[3,55165,0.1015385966],[3,55167,0.1453685733],[3,55168,0.1112965329],[3,55174,0.2432223966],[3,55176,0.065944472],[3,55178,0.0264674217],[3,55179,0.1660875633],[3,55182,0.0027090516],[3,55187,0.0216184355],[3,55189,0.0730207147],[3,55190,0.0931153745],[3,55191,0.0804863461],[3,55192,0.01851677],[3,55194,0.0653569644],[3,55195,0.0558514364],[3,55199,0.2537625294],[3,55200,0.4385223218],[3,55201,0.1050147653],[3,55204,0.2065751597],[3,55205,0.1139625626],[3,55207,0.0246933624],[3,55208,0.935347692],[3,55209,0.1243721816],[3,55210,0.1111621235],[3,55211,0.0669942992],[3,55212,0.1842668071],[3,55215,0.1147369356],[3,55219,0.0258918464],[3,55222,0.1159432764],[3,55223,0.4687927306],[3,55224,0.0597767781],[3,55228,0.1232547844],[3,55229,0.3373748993],[3,55232,0.1132681484],[3,55233,0.0836566398],[3,55235,0.1639815231],[3,55237,0.4220126455],[3,55239,0.1992344065],[3,55240,0.0949640447],[3,55242,0.1055320681],[3,55243,0.1838352567],[3,55244,0.061872545],[3,55246,0.1207696137],[3,55247,0.2476175592],[3,55251,0.1217994731],[3,55254,0.0835771139],[3,55255,0.8755078005],[3,55257,0.0310637746],[3,55260,0.0646806263],[3,55261,0.1506854216],[3,55277,0.2303520676],[3,55278,0.0445157311],[3,55280,0.1437964793],[3,55281,0.0621446626],[3,55282,0.2157096475],[3,55283,0.1123414465],[3,55284,0.3011112098],[3,55287,0.0397896868],[3,55289,0.1026155822],[3,55290,0.243245716],[3,55291,0.1078511205],[3,55292,0.1446169273],[3,55293,0.4722868689],[3,55294,0.1810789475],[3,55296,0.3393422181],[3,55297,0.0774456092],[3,55298,0.06511368],[3,55300,0.1002711975],[3,55302,0.2384659975],[3,55306,0.1011597914],[3,55308,0.1533051807],[3,55312,0.1856904697],[3,55317,0.0036023978],[3,55321,0.1174738417],[3,55326,0.2136087191],[3,55327,0.066560925],[3,55329,0.0274933201],[3,55330,0.0632730848],[3,55331,0.0843743832],[3,55332,0.1121029202],[3,55333,0.1402768323],[3,55334,0.2057353987],[3,55335,0.4065106647],[3,55337,0.0651742675],[3,55340,0.2162158799],[3,55341,0.0654357267],[3,55342,0.079576167],[3,55343,0.1050342683],[3,55344,0.1619657071],[3,55348,0.1984544828],[3,55360,0.1571069326],[3,55362,0.063331799],[3,55364,0.1664481296],[3,55365,0.0931658457],[3,55366,0.0262617121],[3,55367,0.0858128941],[3,55373,0.0364359214],[3,55379,0.4486932606],[3,55381,0.1366378353],[3,55383,0.0985887835],[3,55384,0.5449721598],[3,55385,0.2496257633],[3,55386,0.2598853045],[3,55387,0.1087587186],[3,55388,0.1495849121],[3,55389,0.1938767682],[3,55391,0.4021193767],[3,55392,0.2175888295],[3,55394,0.1453495628],[3,55395,0.0602620386],[3,55397,0.1000389993],[3,55402,0.2798207381],[3,55403,0.1576057663],[3,55405,0.0749367045],[3,55408,0.1698369977],[3,55423,0.5763839864],[3,55425,0.1252206055],[3,55426,0.0410130411],[3,55432,0.0687758695],[3,55433,0.1093646576],[3,55435,0.7141108425],[3,55436,0.130640596],[3,55438,0.074649386],[3,55439,0.0248501061],[3,55440,0.0863614758],[3,55442,0.0270493265],[3,55446,0.0255816389],[3,55447,0.2800329597],[3,55449,0.0246694903],[3,55450,0.1496747263],[3,55456,0.125067578],[3,55458,0.1870444947],[3,55461,0.2714265168],[3,55473,0.2658430281],[3,55478,0.0027090516],[3,55485,0.101501309],[3,55488,0.1532664247],[3,55489,0.0895694846],[3,55490,0.1449215118],[3,55492,0.1047268336],[3,55494,0.7121553793],[3,55495,0.208654894],[3,55498,0.2196585077],[3,55499,0.2952703554],[3,55500,0.1263897671],[3,55503,0.1272616067],[3,55504,0.0813520932],[3,55507,0.0392616405],[3,55511,0.098626764],[3,55512,0.095862972],[3,55513,0.1496227581],[3,55514,0.172173537],[3,55531,0.1242646773],[3,55532,0.1259509628],[3,55534,0.102114406],[3,55536,0.125011091],[3,55537,0.2769968711],[3,55538,0.0318708805],[3,55539,0.0399688423],[3,55541,0.0936347508],[3,55542,0.173638453],[3,55543,0.2545930846],[3,55552,0.0433815565],[3,55553,0.1313059226],[3,55554,0.2529376891],[3,55560,0.0825006016],[3,55561,0.1061443325],[3,55562,0.3265136493],[3,55563,0.1340146684],[3,55564,0.1919885997],[3,55568,0.1689312555],[3,55577,0.2771726696],[3,55578,0.112598102],[3,55579,0.0636510984],[3,55580,0.0870213313],[3,55581,0.1415822181],[3,55582,0.0094363609],[3,55584,0.0291423711],[3,55586,0.0688069021],[3,55599,0.4324324006],[3,55605,0.0891202299],[3,55606,0.1111767993],[3,55609,0.085628669],[3,55610,0.2627951548],[3,55613,0.1800268314],[3,55614,0.0950971205],[3,55615,0.0948387689],[3,55616,0.4088151813],[3,55619,0.1124125353],[3,55620,0.1447592672],[3,55624,0.1410632007],[3,55625,0.1222207256],[3,55626,0.0578947876],[3,55627,0.0689913831],[3,55631,0.1820650818],[3,55632,0.0648055105],[3,55634,0.0610724762],[3,55635,0.085267007],[3,55639,0.1400114736],[3,55643,0.0266160072],[3,55663,0.1445677105],[3,55664,0.1352721814],[3,55671,0.1374208014],[3,55672,0.2053622202],[3,55673,0.1227409046],[3,55674,0.200564034],[3,55676,0.1340268705],[3,55678,0.1872606736],[3,55683,0.0594705584],[3,55685,0.0410392483],[3,55687,0.0627579613],[3,55689,0.1086251148],[3,55691,0.0987251395],[3,55693,0.1762837115],[3,55697,0.5938457223],[3,55698,0.2131839865],[3,55718,0.8281521475],[3,55720,0.246541591],[3,55722,0.1892912511],[3,55723,0.1580050351],[3,55724,0.1254153032],[3,55725,0.0918917005],[3,55726,0.1384402131],[3,55731,0.1104856766],[3,55736,0.063734197],[3,55737,0.0366029843],[3,55739,0.0673657912],[3,55740,0.1006517193],[3,55741,0.4534314051],[3,55744,0.0964250225],[3,55746,0.0289221558],[3,55751,0.0710825593],[3,55752,0.1473954965],[3,55753,0.0466733687],[3,55755,0.1806889895],[3,55756,0.0902445467],[3,55757,0.0998132194],[3,55758,0.1451197557],[3,55762,0.2379082321],[3,55765,0.1774963901],[3,55766,0.043375145],[3,55768,0.099192021],[3,55772,0.0932263858],[3,55773,0.0298981252],[3,55794,0.1399258776],[3,55795,0.2075951961],[3,55796,0.1595147483],[3,55798,0.2504069976],[3,55799,0.2434725136],[3,55800,0.1127631073],[3,55802,0.1490397815],[3,55803,0.0984533815],[3,55806,0.2430786062],[3,55807,0.1949381031],[3,55808,0.2257504399],[3,55813,0.25368616],[3,55814,0.1852131667],[3,55816,0.0330631584],[3,55817,0.0659031218],[3,55824,0.1492989233],[3,55825,0.175818056],[3,55827,0.0535368606],[3,55828,0.0052376042],[3,55829,0.1343683513],[3,55834,0.231907352],[3,55836,0.0631859264],[3,55839,0.0525725284],[3,55863,0.1208833539],[3,55865,0.1787355214],[3,55866,0.1491369492],[3,55868,0.1350939393],[3,55869,0.0501893679],[3,55871,0.1097637708],[3,55874,0.1622494513],[3,55876,0.3240761903],[3,55877,0.1896767717],[3,55878,0.2962551657],[3,55880,0.0215356863],[3,55882,0.1618679615],[3,55884,0.1648471271],[3,55885,0.1566479326],[3,55886,0.2991445137],[3,55889,0.1354500767],[3,55891,0.1096392058],[3,55892,0.0378069802],[3,55895,0.1179837002],[3,55897,0.1648059706],[3,55915,0.6507763016],[3,55919,0.0865777383],[3,55920,0.3229116013],[3,55921,0.1257277462],[3,55924,0.1055523829],[3,55931,0.1124458282],[3,55937,0.0329229191],[3,55940,0.0032962541],[3,55947,0.0790878155],[3,55949,0.1632855892],[3,55952,0.1335910783],[3,55953,0.081556701],[3,55955,0.0313825233],[3,55956,0.0803317009],[3,55957,0.2934981568],[3,55966,0.2135501353],[3,55968,0.164770291],[3,55982,0.1492468513],[3,55983,0.1583180297],[3,56004,0.1318236271],[3,56006,0.1516778845],[3,56011,0.3711760077],[3,56017,0.060666553],[3,56020,0.1025447721],[3,56045,0.0869964303],[3,56054,0.1876964452],[3,56063,0.1447406556],[3,56087,0.0611892638],[3,56089,0.0459508846],[3,56097,0.0027090516],[3,56106,0.3106941734],[3,56107,0.1596773955],[3,56125,0.1445777058],[3,56132,0.0032962541],[3,56155,0.0182552734],[3,56158,0.0078494963],[3,56165,0.002041527],[3,56169,0.0994461373],[3,56176,0.0062188678],[3,56211,0.067482574],[3,56223,0.2082792861],[3,56253,0.0029702593],[3,56263,0.1885816817],[3,56275,0.0854711692],[3,56313,0.2343027738],[3,56327,0.0620802862],[3,56348,0.0632447971],[3,56384,0.0551334736],[3,56471,0.0283551067],[3,56511,0.0129705425],[3,56603,0.0182906032],[3,56615,0.0027090516],[3,56641,0.0055742318],[3,56644,0.0287349827],[3,56670,0.1411883265],[3,56676,0.0946542935],[3,56677,0.1222334333],[3,56678,0.0635180555],[3,56679,0.1243553783],[3,56680,0.1038270525],[3,56681,0.0985702693],[3,56682,0.1106575363],[3,56683,0.0311201834],[3,56685,0.1296528257],[3,56686,0.1807304663],[3,56687,0.121061243],[3,56689,0.0578059088],[3,56690,0.0370567904],[3,56711,0.0251813392],[3,56723,0.0149261462],[3,56735,0.0679013981],[3,56817,0.0146031257],[3,56827,0.2300576886],[3,56833,0.1349668073],[3,56834,0.0032962541],[3,56842,0.1283727918],[3,56857,0.1433907765],[3,56861,0.1188769432],[3,56888,0.1310670435],[3,56891,0.0591639651],[3,56896,0.0016641951],[3,56918,0.0055742318],[3,56938,0.031164115],[3,56959,0.0417771146],[3,56984,0.0016641951],[3,56987,0.0032962541],[3,56989,0.0469116061],[3,57152,0.1359328419],[3,57167,0.1807040781],[3,57196,0.0032962541],[3,57234,0.0040333191],[3,57312,0.0036023978],[3,57446,0.1537459089],[3,57499,0.2235316489],[3,57504,0.0295290814],[3,57521,0.005876737],[3,57524,0.0077420644],[3,57525,0.3319864892],[3,57528,0.1462074032],[3,57531,0.1037180533],[3,57538,0.1173624304],[3,57556,0.1239729114],[3,57557,0.214103869],[3,57561,0.1218456779],[3,57570,0.033095988],[3,57571,0.2276945656],[3,57572,0.0027090516],[3,57573,0.1513768532],[3,57578,0.0047470651],[3,57592,0.1212625655],[3,57593,0.0388687347],[3,57595,0.1920656185],[3,57606,0.0293962474],[3,57607,0.054506838],[3,57608,0.0641498937],[3,57611,0.0051336386],[3,57619,0.3113578584],[3,57625,0.1226690254],[3,57630,0.8205204873],[3,57654,0.017982682],[3,57660,0.3312669608],[3,57662,0.0187873431],[3,57666,0.1544901308],[3,57677,0.0074286441],[3,57801,0.0418123987],[3,57802,0.0729379416],[3,57821,0.3712897996],[3,57874,0.298341896],[3,58668,0.0340502117],[3,58758,0.0032962541],[3,58766,0.3969616278],[3,58783,0.2675867757],[3,58788,0.0803744698],[3,58798,0.1067320325],[3,58799,0.5157120349],[3,58804,0.8023033384],[3,58808,0.1329600824],[3,58809,0.1316130407],[3,58810,0.4009836874],[3,58812,0.0416120959],[3,58813,0.1617551037],[3,58814,0.157703324],[3,58820,0.2134826677],[3,58821,0.0939221842],[3,58822,0.031204581],[3,58823,0.0743708764],[3,58824,0.1085947473],[3,58827,0.0224707079],[3,58828,0.2088683992],[3,58831,0.0366029843],[3,58836,0.1043644892],[3,58837,0.094636021],[3,58838,0.0076273584],[3,58840,0.1249450364],[3,58846,0.1278193454],[3,58849,0.1609489189],[3,58854,0.0405460423],[3,58855,0.1754134893],[3,58858,0.0782873077],[3,58932,0.0040853369],[3,58945,0.0051402416],[3,58949,0.0433095889],[3,58955,0.0057745914],[3,58956,0.0032962541],[3,58982,0.0289069938],[3,59006,0.1718735474],[3,59047,0.3222209892],[3,59052,0.0665233991],[3,59095,0.0267267192],[3,59131,0.0607310209],[3,59136,0.0053435973],[3,59199,0.0502619163],[3,59229,0.2071014609],[3,59230,0.096265576],[3,59269,0.0205752747],[3,59293,0.0054267354],[3,59312,0.0016641951],[3,59313,0.0032962541],[3,59334,0.0055742318],[3,59376,0.1645597799],[3,59448,0.1883662525],[3,59449,0.053359984],[3,59478,0.0026728674],[3,59532,0.0029702593],[3,59676,0.0226466855],[3,59697,0.0272916634],[3,59701,0.0032962541],[3,59792,0.0027090516],[3,60035,0.0032962541],[3,60039,0.1044166312],[3,60043,0.0355147564],[3,60047,0.047652183],[3,60049,0.1207942191],[3,60053,0.0552256705],[3,60072,0.1030984984],[3,60078,0.3278216768],[3,60090,0.0804992124],[3,60105,0.0650507184],[3,60110,0.1292990038],[3,60115,0.1392239142],[3,60126,0.1069215895],[3,60127,0.1658692555],[3,60133,0.0999048757],[3,60138,0.0325780365],[3,60149,0.0323692911],[3,60152,0.0057745914],[3,60156,0.1243771546],[3,60157,0.2819162527],[3,60167,0.0361974129],[3,60179,0.0612463145],[3,60182,0.195304164],[3,60185,0.0027090516],[3,60192,0.0042077477],[3,60219,0.0656408519],[3,60230,0.6359713783],[3,60284,0.0027090516],[3,60297,0.0032962541],[3,60301,0.2195902286],[3,60321,0.0302842463],[3,60324,0.3897648857],[3,60342,0.0901388479],[3,60389,0.0870357815],[3,60404,0.0506078732],[3,60432,0.1078040626],[3,60433,0.1583693323],[3,60509,0.123898991],[3,60512,0.1098489298],[3,60555,0.0345933016],[3,60593,0.061979555],[3,60609,0.0036023978],[3,60618,0.0032962541],[3,60807,0.0032962541],[3,60930,0.0040333191],[3,61025,0.1064205918],[3,61203,0.0032962541],[3,61246,0.0633582213],[3,61259,0.0136995626],[3,61271,0.0851239516],[3,61283,0.2364167853],[3,61293,0.1786675989],[3,61302,0.2398665152],[3,61307,0.0925585237],[3,61339,0.040719899],[3,61366,0.1371928101],[3,61379,0.0672959834],[3,61385,0.0510175469],[3,61406,0.0494173902],[3,61412,0.0178507339],[3,61427,0.005876737],[3,61438,0.0032962541],[3,61463,0.0143371317],[3,61512,0.0032962541],[3,61551,0.0079550701],[3,61622,0.0032962541],[3,61628,0.0196853196],[3,61664,0.0032962541],[3,61689,0.0027090516],[3,61737,0.0042077477],[3,61741,0.003179683],[3,61754,0.0047470651],[3,61781,0.0206862937],[3,61821,0.0040853369],[3,61826,0.002041527],[3,62228,0.094390116],[3,62253,0.0032962541],[3,62257,0.0046324215],[3,62285,0.02948822],[3,62513,0.0032962541],[3,62547,0.003179683],[3,62556,0.0057963084],[3,62864,0.002041527],[3,63072,0.0305122091],[3,63125,0.0032962541],[3,63132,0.1052336635],[3,63145,0.0071083984],[3,63177,0.002041527],[3,63182,0.003179683],[3,63201,0.0451384658],[3,63292,0.0477054899],[3,63298,0.0036023978],[3,63345,0.0036023978],[3,63355,0.0026728674],[3,63594,0.0028356989],[3,63599,0.0309776729],[3,63759,0.0014094531],[3,63806,0.1782444634],[3,63829,0.1926896759],[3,63830,0.1409052558],[3,63831,0.1460223787],[3,63833,0.108396406],[3,63859,0.1736792726],[3,63860,0.0574898674],[3,63863,0.1044188573],[3,63864,0.1357922269],[3,63916,0.0262959912],[3,63922,0.032007497],[3,63933,0.9318990868],[3,63979,0.0814623789],[3,63989,0.0154881608],[3,64009,0.1037590894],[3,64025,0.0021161346],[3,64098,0.0421507186],[3,64117,0.0140445273],[3,64155,0.0175110329],[3,64482,0.2074415986],[3,64697,0.0631446278],[3,64703,0.013046911],[3,64728,0.0338696576],[3,64729,0.0004866929],[3,64735,0.0021161346],[3,64740,0.0104079675],[3,64742,0.0216719932],[3,64773,0.0021153737],[3,64791,0.0104079675],[3,64866,0.0021153737],[3,64917,0.0021161346],[3,64974,0.0172900355],[3,65096,0.0303237516],[3,65209,0.0021161346],[3,65319,0.0506497746],[3,65325,0.0028520176],[3,65350,0.0242081882],[3,65371,0.0479233053],[3,65374,0.193989466],[3,65377,0.0285622576],[3,65378,0.0512898922],[3,65379,0.0983733361],[3,65394,0.0008634197],[3,65397,0.0914007991],[3,65398,0.1154332242],[3,65429,0.0021161346],[3,65432,0.0513133874],[3,65433,0.0437836206],[3,65438,0.0045904975],[3,65457,0.0589962168],[3,65476,0.0015298737],[3,65484,0.0015298737],[3,65486,0.0427438652],[3,65497,0.0242164778],[3,65505,0.0152090193],[3,65519,0.2462636624],[3,65526,0.1241287516],[3,65577,0.0121467079],[3,65653,0.0021161346],[3,65682,0.0157866648],[3,65688,0.0440004294],[3,65697,0.0021161346],[3,65725,0.0021161346],[3,65734,0.2758743394],[3,65744,0.0250873245],[3,65753,0.0162939224],[3,65861,0.0214276864],[3,65952,0.0021161346],[3,66243,0.0301593999],[3,66248,0.002903952],[3,66324,0.0266929465],[3,66345,0.0253807966],[3,66369,0.0021161346],[3,66372,0.0041875047],[3,66378,0.1463663576],[3,66384,0.0052816447],[3,66397,0.1077180682],[3,66409,0.0613446625],[3,66419,0.1359372694],[3,66422,0.2208579583],[3,66426,0.0913835965],[3,66428,0.0901568341],[3,66453,0.7106604364],[3,66468,0.0519304545],[3,66501,0.0833343568],[3,66527,0.0549261092],[3,66533,0.1850874935],[3,66545,0.0004866929],[3,66596,0.2032125363],[3,66610,0.0831926223],[3,66625,0.0034501592],[3,66671,0.1882742144],[3,66690,0.1502382356],[3,66778,0.016225063],[3,66818,0.1889446372],[3,66829,0.0517577004],[3,66854,0.9042764268],[3,66890,0.0369218085],[3,66904,0.0021161346],[3,66912,0.0268014367],[3,66938,0.0214355349],[3,66951,0.0103338754],[3,66973,0.0125027599],[3,66995,0.0041875047],[3,67043,0.0021161346],[3,67147,0.0021161346],[3,67180,0.0140445273],[3,67184,0.0028520176],[3,67213,0.0364540141],[3,67214,0.1828658248],[3,67251,0.0008634197],[3,67309,0.0046924794],[3,67397,0.1987359336],[3,67398,0.081689248],[3,67409,0.298027846],[3,67418,0.0041875047],[3,67424,0.0195575526],[3,67429,0.0726570243],[3,67692,0.0021161346],[3,67728,0.024705379],[3,67731,0.0021161346],[3,67745,0.0606220022],[3,67762,0.2740708391],[3,67812,0.1794950634],[3,67816,0.2402039927],[3,67830,0.0021161346],[3,67838,0.0050737387],[3,67847,0.0039505725],[3,67854,0.2276542112],[3,67920,0.0021161346],[3,68159,0.0024217873],[3,68196,0.0015298737],[3,68298,0.0021161346],[3,68337,0.0015298737],[3,68346,0.0229981073],[3,68361,0.0084649348],[3,68368,0.0008634197],[3,68382,0.0021161346],[3,68383,0.0135748241],[3,68390,0.0021153737],[3,68400,0.0211917376],[3,68422,0.0241424716],[3,68521,0.0226748081],[3,68530,0.0238725404],[3,68553,0.0017906625],[3,68596,0.0183055104],[3,68781,0.0383488197],[3,68839,0.0021161346],[3,68947,0.0513133874],[3,69061,0.0019997504],[3,69062,0.0885425662],[3,69063,0.0172242417],[3,69211,0.0195158093],[3,69309,0.0045904975],[3,69314,0.0043540165],[3,69393,0.0252558892],[3,69455,0.0269476521],[3,69623,0.0069145436],[3,69643,0.0024217873],[3,69821,0.0772509566],[3,69832,0.1123213227],[3,69837,0.0487605966],[3,69939,0.0255107241],[3,69942,0.0015298737],[3,70035,0.0021161346],[3,70114,0.0021161346],[3,70238,0.0126550932],[3,70297,0.0120021364],[3,70644,0.0156597513],[3,70675,0.0255677584],[3,70688,0.0021161346],[3,70848,0.0017906625],[3,71191,0.0008634197],[3,71192,0.0905852052],[3,71212,0.1149977131],[3,71215,0.0037777552],[3,71218,0.0021161346],[3,71231,0.0037777552],[3,71241,0.1233075918],[3,71252,0.0015298737],[3,71257,0.0015298737],[3,71287,0.0829040333],[3,71292,0.0152090193],[3,71294,0.0046121797],[3,71299,0.1981659537],[3,71327,0.0890294173],[3,71330,0.0274340936],[3,71335,0.0029047137],[3,71352,0.0909423804],[3,71365,0.0166472743],[3,71387,0.0303241552],[3,71398,0.1276198048],[3,71401,0.0466216958],[3,71404,0.0028356989],[3,71411,0.0579546604],[3,71413,0.0389580403],[3,71417,0.002903952],[3,71418,0.1669509163],[3,71424,0.2455560701],[3,71483,0.1212022762],[3,71488,0.2205065746],[3,71496,0.0523251189],[3,71559,0.1400050343],[3,71561,0.1991042996],[3,71565,0.1044800568],[3,71590,0.0039505725],[3,71637,0.0015298737],[3,71662,0.0172931849],[3,71683,0.0024217873],[3,71690,0.0041875047],[3,71692,0.0125027599],[3,71726,0.0917366052],[3,71738,0.0021161346],[3,71740,0.1572836048],[3,71788,0.1135730575],[3,71794,0.4614816024],[3,71823,0.0021161346],[3,71844,0.0986147573],[3,71860,0.0156597513],[3,71896,0.0471954724],[3,71930,0.1157479314],[3,71939,0.0371496243],[3,71943,0.0765333319],[3,71973,0.0851312231],[3,71974,0.0857466291],[3,71977,0.0974119917],[3,71987,0.0242164778],[3,72039,0.0008634197],[3,72068,0.0021161346],[3,72133,0.2656589362],[3,72166,0.3068073225],[3,72180,0.0232736312],[3,72192,0.0717333018],[3,72376,0.0211510245],[3,72383,0.0014094531],[3,72435,0.0032299174],[3,72468,0.003564619],[3,72829,0.0033032003],[3,72856,0.0211852872],[3,72897,0.0074545296],[3,72900,0.0272117359],[3,72909,0.0230887297],[3,72913,0.0032299174],[3,72925,0.0668150584],[3,72936,0.0499403768],[3,72961,0.0033032003],[3,72970,0.0365934267],[3,72994,0.0032525662],[3,72998,0.0537055081],[3,73012,0.0032299174],[3,73014,0.1617350379],[3,73020,0.0219146465],[3,73072,0.0046121797],[3,73096,0.0032299174],[3,73140,0.1747431112],[3,73144,0.0745338116],[3,73148,0.0321910312],[3,73150,0.0084477986],[3,73185,0.0032299174],[3,73301,0.042709062],[3,73484,0.0004866929],[3,73585,0.1322807713],[3,73725,0.0053044276],[3,73751,0.0652483872],[3,73764,0.0033032003],[3,73777,0.2692678701],[3,73822,0.0190812799],[3,73866,0.0589620432],[3,73960,0.0434617408],[3,74037,0.1841339844],[3,74060,0.0026310549],[3,74067,0.061665814],[3,74091,0.0032299174],[3,74111,0.1157316452],[3,74128,0.0032299174],[3,74135,0.0372599206],[3,74148,0.0584442765],[3,74184,0.0631119718],[3,74226,0.0602643172],[3,74318,0.0032299174],[3,74328,0.0622069574],[3,74364,0.0322875428],[3,74368,0.0563949276],[3,74382,0.0004866929],[3,74440,0.3346167392],[3,74468,0.0613703102],[3,74474,0.0719018908],[3,74482,0.1234002774],[3,74485,0.0893156602],[3,74494,0.0060971273],[3,74495,0.8251772431],[3,74522,0.2272994964],[3,74528,0.0684452784],[3,74533,0.1379116518],[3,74543,0.1418825667],[3,74574,0.0555062134],[3,74585,0.0032299174],[3,74607,0.0202921945],[3,74627,0.0032299174],[3,74693,0.0212229689],[3,74762,0.0028948832],[3,74808,0.0082959424],[3,74826,0.0028948832],[3,74853,0.0057607876],[3,74862,0.0806882215],[3,74881,0.0062182674],[3,74907,0.0251662041],[3,74983,0.1691495219],[3,75047,0.0001529077],[3,75067,0.0028948832],[3,75111,0.0028948832],[3,75116,0.1410591607],[3,75123,0.0014562837],[3,75124,0.0469267066],[3,75151,0.0001529077],[3,75240,0.0001529077],[3,75243,0.005220418],[3,75262,0.2493780584],[3,75272,0.0028948832],[3,75287,0.020840578],[3,75322,0.1600465325],[3,75343,0.0028948832],[3,75358,0.1178201137],[3,75375,0.0249253978],[3,75387,0.000529463],[3,75430,0.0036842982],[3,75440,0.0775173365],[3,75541,0.0016652764],[3,75631,0.0028948832],[3,75637,0.000529463],[3,75670,0.0001529077],[3,75671,0.0510599172],[3,75698,0.1064188728],[3,75711,0.0047312656],[3,75714,0.0256991827],[3,75760,0.0241716292],[3,75769,0.2082128687],[3,75796,0.3361648958],[3,75809,0.0025690662],[3,75810,0.1508430713],[3,75812,0.2886683706],[3,75839,0.0235067971],[3,75843,0.530357406],[3,75844,0.1066144115],[3,75871,0.3297368832],[3,75913,0.0458647712],[3,75948,0.081661702],[3,76022,0.0291746021],[3,76035,0.1054154496],[3,76085,0.0029681327],[3,76091,0.0029681327],[3,76135,0.1432229604],[3,76143,0.053028798],[3,76163,0.0274822421],[3,76171,0.0622927437],[3,76182,0.51025534],[3,76190,0.4516557218],[3,76222,0.058257969],[3,76227,0.0039407281],[3,76269,0.0559307335],[3,76278,0.1974982811],[3,76282,0.114100307],[3,76305,0.0056309703],[3,76339,0.1089960532],[3,76445,0.1631834759],[3,76472,0.0132350797],[3,76506,0.0028948832],[3,76511,0.0251837188],[3,76587,0.022019933],[3,76600,0.1171012922],[3,76613,0.0017668535],[3,76747,0.0086294165],[3,76809,0.0028948832],[3,76822,0.0172528546],[3,76852,0.0029681327],[3,76860,0.0028948832],[3,76894,0.0042544982],[3,76899,0.0396588382],[3,76906,0.0203951125],[3,76908,0.0424387965],[3,77006,0.0579277217],[3,77082,0.0264785318],[3,77120,0.0639895767],[3,77126,0.0540041756],[3,77156,0.2392348982],[3,77162,0.062487849],[3,77166,0.0208763908],[3,77212,0.0206577885],[3,77216,0.0229738263],[3,77243,0.0823676366],[3,77263,0.021021156],[3,77270,0.0042548439],[3,77280,0.018420094],[3,77287,0.1751268296],[3,77341,0.2140365371],[3,77389,0.2456614497],[3,77413,0.0028948832],[3,77425,0.0028948832],[3,77427,0.0028948832],[3,77429,0.0049456764],[3,77440,0.06020312],[3,77468,0.0143003116],[3,77487,0.0492586177],[3,77496,0.0007826251],[3,77526,0.0224803576],[3,77538,0.000529463],[3,77557,0.0047378651],[3,77600,0.1185975547],[3,77611,0.0411286632],[3,77631,0.1561107245],[3,77632,0.0250917449],[3,77634,0.2165234743],[3,77640,0.0025690662],[3,77646,0.0913248376],[3,77652,0.0613085396],[3,77686,0.1279344926],[3,77697,0.0468398546],[3,77723,0.0972471048],[3,77728,0.089411631],[3,77745,0.0001529077],[3,77775,0.0236855884],[3,77786,0.0236106407],[3,77792,0.1456901071],[3,77808,0.1427384125],[3,77826,0.0025690662],[3,77834,0.0099012821],[3,77838,0.0084333361],[3,77861,0.1431496521],[3,77862,0.0639323086],[3,77872,0.0001529077],[3,77883,0.0682477408],[3,77885,0.0343217845],[3,77891,0.0043567793],[3,77892,0.0577723718],[3,77895,0.0568044435],[3,77925,0.1719943662],[3,77961,0.026700935],[3,77989,0.039313987],[3,78017,0.0409952706],[3,78018,0.0058343225],[3,78143,0.0028948832],[3,78150,0.0042928669],[3,78180,0.0042928669],[3,78203,0.0187394473],[3,78256,0.0363942301],[3,78258,0.0202796802],[3,78272,0.0036842982],[3,78278,0.0028948832],[3,78287,0.0014562837],[3,78294,0.0516692979],[3,78303,0.0029681327],[3,78310,0.0144418925],[3,78314,0.0032294325],[3,78315,0.0032294325],[3,78317,0.0162408337],[3,78327,0.5803071154],[3,78343,0.1470009569],[3,78345,0.0668084874],[3,78355,0.0020871211],[3,78381,0.0047312656],[3,78387,0.0919879205],[3,78391,0.0151791766],[3,78393,0.1429580658],[3,78437,0.2378710351],[3,78441,0.1711476859],[3,78506,0.1094110366],[3,78525,0.0272179505],[3,78531,0.0460431273],[3,78575,0.1023434129],[3,78587,0.0042548439],[3,78613,0.2550665487],[3,78620,0.2891614057],[3,78621,0.0888786862],[3,78680,0.0007826251],[3,78685,0.0028948832],[3,78687,0.0931552128],[3,78689,0.1293496763],[3,78690,0.3240567124],[3,78710,0.0611245003],[3,78715,0.2081171699],[3,78730,0.2046747053],[3,78755,0.0804136584],[3,78764,0.03713603],[3,78882,0.0029681327],[3,78884,0.1087706972],[3,79042,0.141051015],[3,79067,0.0701653356],[3,79090,0.1245116522],[3,79152,0.055155417],[3,79208,0.0029681327],[3,79214,0.0028948832],[3,79252,0.0063341084],[3,79253,0.0117206591],[3,79298,0.0415782763],[3,79372,0.0028948832],[3,79385,0.0078358867],[3,79439,0.0020871211],[3,79458,0.0057607876],[3,79467,0.0032294325],[3,79482,0.005815903],[3,79489,0.3613099602],[3,79507,0.0517378411],[3,79509,0.007872195],[3,79562,0.0383909331],[3,79627,0.0620300717],[3,79638,0.4243141537],[3,79650,0.0307647257],[3,79654,0.0570467361],[3,79657,0.0019229082],[3,79679,0.056765989],[3,79699,0.0178496591],[3,79706,0.3114246803],[3,79747,0.102565104],[3,79768,0.2227922419],[3,79785,0.2991167696],[3,79790,0.000529463],[3,79834,0.0029681327],[3,79851,0.0211753503],[3,79876,0.0058343225],[3,79883,0.0117206591],[3,79896,0.000529463],[3,79922,0.0527469685],[3,79926,0.1548843022],[3,79937,0.2286326072],[3,79943,0.0297158681],[3,79946,0.029482278],[3,79960,0.093824816],[3,79962,0.026786962],[3,79978,0.0062182674],[3,80013,0.0496776027],[3,80018,0.0028948832],[3,80042,0.0028948832],[3,80043,0.0036588368],[3,80044,0.0227749693],[3,80063,0.005393574],[3,80091,0.0028948832],[3,80109,0.0193683094],[3,80143,0.0047312656],[3,80171,0.0028948832],[3,80179,0.0415809228],[3,80180,0.0001529077],[3,80183,0.0363558574],[3,80208,0.0028948832],[3,80229,0.000529463],[3,80247,0.0020871211],[3,80305,0.016641446],[3,80312,0.0043567793],[3,80321,0.0029681327],[3,80345,0.0169997269],[3,80349,0.0845269681],[3,80358,0.0016652764],[3,80360,0.0282253235],[3,80372,0.0028948832],[3,80381,0.0067951979],[3,80434,0.0028948832],[3,80463,0.0367184152],[3,80474,0.0028948832],[3,80481,0.001780847],[3,80512,0.0043567793],[3,80515,0.0123157676],[3,80518,0.0350702758],[3,80519,0.0195639395],[3,80523,0.0278162692],[3,80528,0.0028948832],[3,80544,0.0028948832],[3,80563,0.0243113559],[3,80567,0.0969120135],[3,80595,0.0192436913],[3,80598,0.0028948832],[3,80609,0.0036842982],[3,80629,0.0028948832],[3,80649,0.0132759052],[3,80652,0.0261252391],[3,80676,0.0047378651],[3,80711,0.0028948832],[3,80712,0.0132350797],[3,80728,0.0028948832],[3,80729,0.0007826251],[3,80734,0.0205075155],[3,80735,0.0028948832],[3,80741,0.0014562837],[3,80769,0.0016652764],[3,80776,0.0442377],[3,80784,0.001780847],[3,80792,0.0028948832],[3,80794,0.0036217998],[3,80802,0.0034864663],[3,80809,0.0631814678],[3,80832,0.0123157676],[3,80835,0.0043567793],[3,80850,0.0250364626],[3,80864,0.0028948832],[3,80872,0.0326808694],[3,80875,0.0261915681],[3,80879,0.0028948832],[3,80896,0.2580524829],[3,80899,0.0028948832],[3,80901,0.0028948832],[3,80905,0.0028948832],[3,80907,0.0291461586],[3,80923,0.0028948832],[3,80928,0.0042765162],[3,80929,0.0292037625],[3,80949,0.001780847],[3,80951,0.0017668535],[3,80954,0.0387667055],[3,80956,0.0503230494],[3,80960,0.0303940247],[3,80964,0.0661984534],[3,80973,0.0036842982],[3,80974,0.0001529077],[3,80979,0.0221126758],[3,80994,0.0042548439],[3,80999,0.0132350797],[3,81019,0.0074748839],[3,81035,0.0117206591],[3,81039,0.0190354633],[3,81047,0.1687184815],[3,81062,0.0442258417],[3,81072,0.0799463845],[3,81073,0.0071478042],[3,81079,0.1747838259],[3,81101,0.0025690662],[3,81143,0.0164378075],[3,81151,0.7746633199],[3,81152,0.0042765162],[3,81153,0.0175064434],[3,81164,0.0240268388],[3,81165,0.000529463],[3,81172,0.0282234041],[3,81191,0.0028948832],[3,81220,0.0028948832],[3,81227,0.001780847],[3,81239,0.0028948832],[3,81244,0.0107428627],[3,81248,0.0010752478],[3,81252,0.1827030783],[3,81271,0.0028948832],[3,81277,0.0028948832],[3,81289,0.0031150248],[3,81291,0.0529408059],[3,81296,0.0057607876],[3,81306,0.0497472108],[3,81313,0.000529463],[3,81318,0.0028948832],[3,81319,0.0028948832],[3,81335,0.000529463],[3,81339,0.0028948832],[3,81371,0.0028948832],[3,81372,0.0265375959],[3,81380,0.020791828],[3,81384,0.1276927724],[3,81393,0.0789755697],[3,81395,0.0028948832],[3,81399,0.0280805503],[3,81401,0.0024405052],[3,81433,0.0032553123],[3,81451,0.0028948832],[3,81465,0.0082692146],[3,81468,0.0279124664],[3,81470,0.0034864663],[3,81471,0.0353150303],[3,81480,0.1969066151],[3,81483,0.0057607876],[3,81494,0.0181048043],[3,81496,0.0028948832],[3,81513,0.0042548439],[3,81515,0.0152301427],[3,81517,0.0392467413],[3,81524,0.0127616407],[3,81528,0.0192436913],[3,81552,0.0523059838],[3,81581,0.1892880382],[3,81589,0.0028948832],[3,81590,0.0028948832],[3,81592,0.0114300276],[3,81603,0.0946266817],[3,81605,0.0062545083],[3,81612,0.0691500095],[3,81636,0.0001529077],[3,81644,0.0557092718],[3,81685,0.0010752478],[3,81723,0.0198403178],[3,81730,0.0170752829],[3,81749,0.0348288552],[3,81750,0.0166616754],[3,81758,0.005393574],[3,81818,0.0105692914],[3,81830,0.0028948832],[3,81838,0.1561673949],[3,81878,0.0226875418],[3,81893,0.0029681327],[3,81924,0.0028948832],[3,81938,0.0028948832],[3,81944,0.0042548439],[3,81948,0.0028948832],[3,81958,0.0028948832],[3,81971,0.016641446],[3,81987,0.0095889198],[3,82050,0.0025690662],[3,82085,0.0451820973],[3,82094,0.0056309703],[3,82096,0.0028948832],[3,82132,0.0028948832],[3,82153,0.0272265896],[3,82165,0.0060564773],[3,82240,0.2033536262],[3,82313,0.0165341346],[3,82344,0.0001529077],[3,82367,0.0516172486],[3,82385,0.0042548439],[3,82414,0.0054001794],[3,82445,0.000529463],[3,82455,0.0042548439],[3,82514,0.0007826251],[3,82520,0.0028948832],[3,82533,0.0356553213],[3,82543,0.0028948832],[3,82567,0.0017668535],[3,82601,0.0042544982],[3,82667,0.0016652764],[3,82692,0.1233916418],[3,82698,0.0438682515],[3,82763,0.0028948832],[3,82770,0.1173718702],[3,82795,0.0151351368],[3,82800,0.0043567793],[3,82821,0.0032294325],[3,82857,0.0420106701],[3,82940,0.0320322766],[3,82968,0.0036842982],[3,82978,0.0179135188],[3,83006,0.0218882462],[3,83045,0.0028948832],[3,83084,0.0028948832],[3,83172,0.0007826251],[3,83331,0.0243067586],[3,83359,0.0136589874],[3,83389,0.1305054243],[3,83410,0.0235466703],[3,83420,0.0028948832],[3,83425,0.0031150248],[3,83429,0.172620181],[3,83436,0.0047378651],[3,83453,0.189410702],[3,83484,0.0072680459],[3,83527,0.0175532152],[3,83533,0.0340721049],[3,83592,0.0028948832],[3,83661,0.0044355964],[3,83751,0.0513614135],[3,83753,0.0001529077],[3,83861,0.0262893339],[3,83869,0.000529463],[3,83970,0.0042548439],[3,84050,0.0179636121],[3,84106,0.0396377477],[3,84190,0.3199205064],[3,84217,0.028594398],[3,84234,0.0121635037],[3,84283,0.0240798728],[3,84295,0.0057607876],[3,84306,0.0047378651],[3,84380,0.0100696651],[3,84406,0.0028948832],[3,84409,0.0375706554],[3,84510,0.0011595039],[3,84631,0.061356876],[3,84641,0.0019229082],[3,84643,0.162134616],[3,84747,0.2085444896],[3,84765,0.0028948832],[3,84859,0.0384996065],[3,84860,0.0203062892],[3,84897,0.0028948832],[3,84916,0.0362491973],[3,84966,0.0028948832],[3,85083,0.0028948832],[3,85115,0.0029681327],[3,85153,0.001780847],[3,85176,0.0138483802],[3,85218,0.0028948832],[3,85229,0.0364619841],[3,85231,0.0014562837],[3,85279,0.0519678798],[3,85299,0.0028948832],[3,85347,0.0184996151],[3,85365,0.0014562837],[3,85377,0.2165751818],[3,85412,0.0152144322],[3,85441,0.0014562837],[3,85461,0.0363645418],[3,85494,0.0063341084],[3,85654,0.0027414033],[3,85743,0.0861901842],[3,85758,0.0929022115],[3,85762,0.0027414033],[3,85799,0.0263277927],[3,85808,0.0327581941],[3,85840,0.0027414033],[3,85841,0.0],[3,85870,0.3505848435],[3,85898,0.0024156543],[3,85908,0.0030758827],[3,85917,0.1330111935],[3,85980,0.0262581564],[3,86001,0.0027414033],[3,86006,0.0041007346],[3,86022,0.0343910214],[3,86048,0.004122748],[3,86052,0.0551435699],[3,86062,0.0027414033],[3,86109,0.0027414033],[3,86121,0.0059023377],[3,86132,0.0027414033],[3,86150,0.0027414033],[3,86188,0.0424242413],[3,86211,0.0390491557],[3,86223,0.0255525172],[3,86268,0.0031150248],[3,86331,0.0027414033],[3,86338,0.0027414033],[3,86358,0.0027414033],[3,86378,0.0],[3,86386,0.0121635037],[3,86391,0.0052395727],[3,86396,0.0027414033],[3,86400,0.0027414033],[3,86440,0.0027414033],[3,86441,0.089015707],[3,86454,0.0031150248],[3,86464,0.3076492494],[3,86473,0.0016275995],[3,86474,0.0027414033],[3,86484,0.0043567793],[3,86500,0.0254772268],[3,86505,0.0027414033],[3,86514,0.0394648621],[3,86516,0.0244204695],[3,86521,0.0043567793],[3,86522,0.0121635037],[3,86527,0.0199369968],[3,86545,0.0043567793],[3,86557,0.001512053],[3,86564,0.0016275995],[3,86588,0.0031150248],[3,86596,0.0027414033],[3,86597,0.0],[3,86598,0.0027414033],[3,86636,0.0437107237],[3,86659,0.0030758827],[3,86749,0.027043792],[3,86784,0.0117206591],[3,86806,0.0027414033],[3,86807,0.001303104],[3,86809,0.0016275995],[3,86820,0.0024156543],[3,86824,0.0043567793],[3,86826,0.0024156543],[3,86849,0.0027414033],[3,86856,0.0027414033],[3,86861,0.0019338098],[3,86879,0.0016275995],[3,86881,0.0027414033],[3,86882,0.0059023377],[3,86890,0.0041007346],[3,86892,0.024095017],[3,86902,0.0080923051],[3,86903,0.0963511859],[3,86908,0.0218237535],[3,86914,0.016573715],[3,86915,0.0052395727],[3,86916,0.0194417751],[3,86922,0.0140608309],[3,86930,0.0041007346],[3,86937,0.0212229689],[3,86950,0.0030758827],[3,86955,0.0678931927],[3,86966,0.0277070484],[3,86973,0.0027414033],[3,86978,0.0027414033],[3,86984,0.0043567793],[3,86985,0.0041007346],[3,86988,0.0027414033],[3,87004,0.079518611],[3,87010,0.0027414033],[3,87012,0.0123157676],[3,87052,0.024250307],[3,87064,0.0027414033],[3,87073,0.0357550566],[3,87109,0.0230065779],[3,87124,0.0054769195],[3,87168,0.0024156543],[3,87172,0.0054769195],[3,87208,0.0027414033],[3,87240,0.0025377877],[3,87252,0.0276837067],[3,87261,0.0056067096],[3,87310,0.1009461519],[3,87311,0.0019338098],[3,87322,0.0028146375],[3,87324,0.0016275995],[3,87325,0.0144418925],[3,87335,0.0268595653],[3,87362,0.0016275995],[3,87372,0.0027414033],[3,87373,0.0221377823],[3,87379,0.0217421178],[3,87384,0.0205992121],[3,87401,0.0006295859],[3,87409,0.0016275995],[3,87420,0.0223803957],[3,87426,0.0178760468],[3,87433,0.0042548439],[3,87437,0.001303104],[3,87469,0.0073628942],[3,87489,0.0027414033],[3,87530,0.0042548439],[3,87547,0.0024156543],[3,87549,0.001303104],[3,87557,0.0117206591],[3,87565,0.0645459962],[3,87571,0.0027414033],[3,87597,0.0297316359],[3,87670,0.0183466655],[3,87691,0.0213792057],[3,87722,0.0027414033],[3,87732,0.0027414033],[3,87741,0.0878157757],[3,87754,0.0],[3,87760,0.0027414033],[3,87771,0.0199666026],[3,87820,0.0027414033],[3,87906,0.0027414033],[3,87931,0.0027414033],[3,87959,0.0003764767],[3,87966,0.0258032924],[3,87992,0.0206352603],[3,88003,0.0243067586],[3,88036,0.0042548439],[3,88072,0.0187898849],[3,88084,0.0042548439],[3,88097,0.2594415529],[3,88139,0.0027414033],[3,88145,0.001512053],[3,88190,0.001512053],[3,88199,0.0045840006],[3,88203,0.0125533627],[3,88252,0.0024156543],[3,88268,0.0031150248],[3,88279,0.0003764767],[3,88322,0.0255654101],[3,88333,0.0027414033],[3,88352,0.0],[3,88386,0.0189294164],[3,88515,0.0239896437],[3,88534,0.0056067096],[3,88574,0.2210444675],[3,88695,0.00694655],[3,88721,0.0198311914],[3,88740,0.1801619069],[3,88761,0.0],[3,88778,0.0027414033],[3,88814,0.0170337691],[3,88855,0.001512053],[3,88910,0.0043567793],[3,88919,0.0027414033],[3,88997,0.0301928152],[3,89007,0.0016275995],[3,89018,0.0183771582],[3,89093,0.0027414033],[3,89142,0.0248507884],[3,89152,0.0056802292],[3,89161,0.0137053614],[3,89167,0.0027414033],[3,89266,0.0535148886],[3,89273,0.0034863941],[3,89344,0.0238636024],[3,89381,0.1207566559],[3,89569,0.0061041103],[3,89634,0.0039092851],[3,89649,0.0464878333],[3,89654,0.0023477637],[3,89797,0.2103312621],[3,89809,0.0039092851],[3,89852,0.0042548439],[3,89913,0.0196793539],[3,89918,0.0247891251],[3,89926,0.0118236273],[3,89974,0.0055104235],[3,90001,0.0206256449],[3,90043,0.0047190506],[3,90067,0.0947644325],[3,90068,0.0047190506],[3,90092,0.0232462301],[3,90096,0.056302649],[3,90097,0.0424456671],[3,90141,0.1672639006],[3,90164,0.0047190506],[3,90197,0.0236213536],[3,90259,0.0047190506],[3,90273,0.0297564257],[3,90294,0.2253967129],[3,90309,0.0047190506],[3,90319,0.0019702745],[3,90343,0.0080870081],[3,90370,0.0178497818],[3,90427,0.0630069001],[3,90432,0.0823693849],[3,90436,0.0047190506],[3,90532,0.083990791],[3,90622,0.0916203508],[3,90667,0.0019702745],[3,90837,0.0032728666],[3,90838,0.0390130537],[3,90847,0.0047190506],[3,90878,0.0047190506],[3,90905,0.0505168805],[3,90991,0.1527449351],[3,91005,0.4732655375],[3,91024,0.0043924255],[3,91127,0.0148602001],[3,91160,0.1802778449],[3,91195,0.0117206591],[3,91254,0.0039092851],[3,91287,0.0078254308],[3,91303,0.1721066258],[3,91391,0.243612907],[3,91403,0.0366988183],[3,91425,0.0047190506],[3,91428,0.0673915156],[3,91459,0.002770503],[3,91480,0.0054812545],[3,91485,0.0236556626],[3,91535,0.008502459],[3,91536,0.031628606],[3,91540,0.0598830435],[3,91554,0.0334785366],[3,91573,0.0309076914],[3,91589,0.0035197896],[3,91758,0.0020036012],[3,91818,0.0346012002],[3,91832,0.004752502],[3,91837,0.0023811075],[3,91937,0.004752502],[3,92018,0.0044258621],[3,92021,0.0076993653],[3,92026,0.0074954996],[3,92082,0.0766004201],[3,92145,0.004752502],[3,92194,0.0195083705],[3,92211,0.0048259365],[3,92399,0.004752502],[3,92426,0.051243878],[3,92428,0.2621244536],[3,92436,0.0211426214],[3,92441,0.0536386422],[3,92458,0.0361526448],[3,92464,0.4278343357],[3,92478,0.0528016949],[3,92491,0.11873693],[3,92551,0.0323720141],[3,92558,0.0329205941],[3,92562,0.141170619],[3,92660,0.1021252529],[3,92680,0.0227106915],[3,92683,0.0074954996],[3,92693,0.0538249131],[3,92718,0.2599940046],[3,92725,0.1548220418],[3,92727,0.1268795782],[3,92737,0.2071087118],[3,92744,0.004752502],[3,92755,0.4831776991],[3,92778,0.2244599362],[3,92819,0.1261532918],[3,92827,0.0076256447],[3,92854,0.0225317377],[3,92887,0.1029921823],[3,92916,0.0689706656],[3,92923,0.086893923],[3,92924,0.8171843436],[3,92931,0.510712336],[3,92942,0.0255136972],[3,92947,0.0577096767],[3,92950,0.0239897513],[3,92953,0.0346332697],[3,92975,0.0202296451],[3,92994,0.0192778017],[3,92996,0.0656571909],[3,92999,0.1810298601],[3,93033,0.0913324455],[3,93050,0.053993078],[3,93085,0.2738526315],[3,93132,0.004752502],[3,93165,0.0292068074],[3,93213,0.4064046028],[3,93214,0.0249084356],[3,93216,0.0096073559],[3,93386,0.0801492549],[3,93399,0.004752502],[3,93403,0.3335190747],[3,93411,0.0369889555],[3,93432,0.3248291811],[3,93435,0.0055439108],[3,93438,0.0332984075],[3,93444,0.0545341487],[3,93453,0.3978719142],[3,93477,0.0048259365],[3,93483,0.3197613894],[3,93549,0.0607972705],[3,93583,0.004752502],[3,93627,0.0061155509],[3,93650,0.0031484034],[3,93653,0.0456341125],[3,93677,0.0020036012],[3,93720,0.0044258621],[3,93763,0.0467520907],[3,93787,0.0146946437],[3,93796,0.004752502],[3,93807,0.0084688195],[3,93833,0.2256951822],[3,93835,0.0499296358],[3,93845,0.3708648384],[3,93865,0.0865650046],[3,93874,0.0530187602],[3,93879,0.1589486773],[3,93901,0.0895760346],[3,93902,0.0488409381],[3,93918,0.0537453672],[3,93935,0.0231545998],[3,93937,0.0534988818],[3,94023,0.1359734426],[3,94036,0.060986876],[3,94092,0.023300847],[3,94103,0.4597464688],[3,94141,0.1073875745],[3,94163,0.070814933],[3,94220,0.0376130804],[3,94288,0.1925696484],[3,94311,0.0320871675],[3,94314,0.1418486373],[3,94470,0.0796072586],[3,94471,0.080862078],[3,94527,0.0319473041],[3,94563,0.0555884827],[3,94566,0.0162615559],[3,94599,0.1700914396],[3,94610,0.7608863566],[3,94616,0.024442326],[3,94636,0.01047946],[3,94642,0.1309136041],[3,94645,0.0119472179],[3,94654,0.0200652828],[3,94657,0.002634909],[3,94684,0.0267257789],[3,94715,0.004752502],[3,94822,0.004752502],[3,95007,0.0036356521],[3,95057,0.004752502],[3,95062,0.0023811075],[3,95074,0.012697567],[3,95082,0.0023811075],[3,95096,0.0184884195],[3,95179,0.1350828264],[3,95185,0.0585773006],[3,95200,0.0433664009],[3,95220,0.3630874978],[3,95289,0.0259783393],[3,95320,0.0132689175],[3,95346,0.4392492976],[3,95380,0.0275492706],[3,95464,0.0065935224],[3,95485,0.0221469163],[3,95515,0.0036356521],[3,95522,0.0381014544],[3,95531,0.0552208243],[3,95559,0.0866648554],[3,95562,0.1508449764],[3,95612,0.0843153146],[3,95679,0.0190146275],[3,95695,0.0883759753],[3,95710,0.1065438795],[3,95745,0.004752502],[3,95747,0.0854209892],[3,95899,0.0090161644],[3,95961,0.0044258621],[3,96106,0.064676913],[3,96108,0.0299550117],[3,96114,0.2144179108],[3,96134,0.2069489311],[3,96144,0.0031484034],[3,96146,0.3510825433],[3,96232,0.0712145518],[3,96240,0.0335205089],[3,96256,0.3120372609],[3,96319,0.0194382054],[3,96328,0.0548232555],[3,96331,0.0524096804],[3,96355,0.4107662279],[3,96379,0.034539534],[3,96418,0.1776555966],[3,96421,0.004752502],[3,96423,0.0146137613],[3,96479,0.0086626676],[3,96489,0.0270877908],[3,96518,0.0651500574],[3,96547,0.1041650598],[3,96558,0.241200989],[3,96754,0.0186766745],[3,96770,0.0190723647],[3,96839,0.0287501233],[3,96850,0.2224205575],[3,96861,0.031920595],[3,96879,0.1620972076],[3,96906,0.0342393336],[3,96910,0.1916925565],[3,96931,0.0036216233],[3,96989,0.0042882742],[3,96997,0.0260657634],[3,97022,0.0200323265],[3,97061,0.0667650753],[3,97093,0.1835517989],[3,97159,0.0939541038],[3,97196,0.1347195256],[3,97229,0.2460943922],[3,97230,0.0633198935],[3,97237,0.074777151],[3,97245,0.1053796075],[3,97285,0.0020036012],[3,97311,0.0068313048],[3,97322,0.0062517868],[3,97384,0.0274242665],[3,97394,0.0334457473],[3,97433,0.0820171815],[3,97541,0.067686934],[3,97546,0.0042882742],[3,97591,0.17066064],[3,97657,0.0316995268],[3,97703,0.0241244632],[3,97708,0.0036356521],[3,97762,0.0073964656],[3,97777,0.2512618748],[3,97872,0.1293538127],[3,97905,0.0033102691],[3,97913,0.004752502],[3,97992,0.0048259365],[3,98077,0.0044463677],[3,98129,0.0132689175],[3,98265,0.0066001386],[3,98280,0.0491823762],[3,98295,0.011857401],[3,98338,0.0073964656],[3,98366,0.0198891756],[3,98370,0.0480201792],[3,98377,0.004752502],[3,98387,0.0305899802],[3,98422,0.0507383694],[3,98426,0.0251771624],[3,98517,0.004752502],[3,98584,0.004752502],[3,98594,0.004752502],[3,98623,0.0251847323],[3,98631,0.0191601285],[3,98641,0.0031484034],[3,98649,0.0044258621],[3,98687,0.004752502],[3,98695,0.1100175485],[3,98708,0.1013672375],[3,98721,0.0346012002],[3,98725,0.004752502],[3,98754,0.002770503],[3,98769,0.0233676189],[3,98868,0.0042882742],[3,98878,0.0048259365],[3,98902,0.0084552171],[3,98939,0.007745357],[3,99016,0.1442526386],[3,99018,0.1632804137],[3,99021,0.0186702863],[3,99119,0.3328759221],[3,99150,0.004752502],[3,99233,0.0473484559],[3,99264,0.013737835],[3,99315,0.0244452052],[3,99319,0.0222906104],[3,99344,0.0035197896],[3,99355,0.0055439108],[3,99377,0.2246625497],[3,99386,0.0584058541],[3,99452,0.0954633883],[3,99485,0.0190735747],[3,99553,0.002634909],[3,99572,0.1113150268],[3,99582,0.1164499655],[3,99618,0.1762271628],[3,99630,0.4735534474],[3,99660,0.0294092573],[3,99721,0.1377587694],[3,99724,0.5784051337],[3,99734,0.0023811075],[3,99844,0.0269740242],[3,99926,0.0039426998],[3,99954,0.0035197896],[3,100001,0.004752502],[3,100009,0.0333080648],[3,100058,0.0830355388],[3,100078,0.0548576289],[3,100082,0.2111225822],[3,100151,0.0218239491],[3,100158,0.055824938],[3,100188,0.004752502],[3,100200,0.0879889518],[3,100206,0.0061155509],[3,100247,0.0054812545],[3,100251,0.1983332124],[3,100263,0.0833302312],[3,100326,0.3305135732],[3,100327,0.1611826443],[3,100380,0.0044258621],[3,100393,0.0072641257],[3,100477,0.0715034771],[3,100480,0.021894097],[3,100595,0.004752502],[3,100599,0.0117544281],[3,100612,0.0271774279],[3,100692,0.1012865943],[3,100700,0.004752502],[3,100715,0.084819752],[3,100740,0.1854734962],[3,100747,0.1418367025],[3,100772,0.1853798824],[3,100819,0.0048259365],[3,100820,0.0108201453],[3,100827,0.0160046798],[3,100831,0.0212601538],[3,100954,0.086427971],[3,101037,0.004752502],[3,101039,0.0210353954],[3,101051,0.004752502],[3,101126,0.0044258621],[3,101206,0.0042882742],[3,101237,0.0577451348],[3,101247,0.0044258621],[3,101280,0.0313936462],[3,101339,0.0020036012],[3,101371,0.0054000025],[3,101377,0.0573205613],[3,101381,0.004752502],[3,101384,0.0446398919],[3,101385,0.0320440193],[3,101420,0.004752502],[3,101426,0.0940524922],[3,101464,0.0191731414],[3,101467,0.0074954996],[3,101474,0.0200784064],[3,101489,0.0404994039],[3,101493,0.004752502],[3,101497,0.0370755475],[3,101498,0.190078287],[3,101505,0.004752502],[3,101605,0.004752502],[3,101648,0.004752502],[3,101653,0.0382202372],[3,101679,0.0076993653],[3,101697,0.204609324],[3,101700,0.004752502],[3,101752,0.004752502],[3,101765,0.0213264037],[3,101791,0.008335568],[3,101820,0.0044258621],[3,101837,0.004752502],[3,101838,0.004752502],[3,101859,0.0065935224],[3,101878,0.0149549116],[3,101925,0.0066001386],[3,101978,0.1415963583],[3,101991,0.0400972007],[3,102150,0.0703022917],[3,102187,0.1944378051],[3,102253,0.0175822241],[3,102281,0.0094738388],[3,102287,0.0212170442],[3,102309,0.004752502],[3,102345,0.004752502],[3,102357,0.0042882742],[3,102386,0.0044258621],[3,102411,0.0044258621],[3,102470,0.0065935224],[3,102474,0.0697427065],[3,102524,0.0036356521],[3,102525,0.0039426998],[3,102539,0.011857401],[3,102543,0.0041564902],[3,102544,0.004752502],[3,102566,0.0020036012],[3,102583,0.0039426998],[3,102587,0.0381781628],[3,102626,0.037540133],[3,102634,0.004752502],[3,102638,0.004752502],[3,102665,0.0178902909],[3,102673,0.0271774279],[3,102685,0.004752502],[3,102697,0.004752502],[3,102705,0.0062517868],[3,102709,0.0592676132],[3,102714,0.0035197896],[3,102720,0.004752502],[3,102728,0.0187460791],[3,102731,0.0023811075],[3,102739,0.004752502],[3,102762,0.0210276965],[3,102771,0.004752502],[3,102784,0.004752502],[3,102791,0.0023811075],[3,102794,0.0345277891],[3,102825,0.0066001386],[3,102848,0.0192388178],[3,102855,0.0732616471],[3,102892,0.004752502],[3,102977,0.0065935224],[3,102978,0.0054812545],[3,103007,0.0322380002],[3,103023,0.0039426998],[3,103024,0.0096506247],[3,103025,0.0175845686],[3,103051,0.0117544281],[3,103058,0.004752502],[3,103078,0.0074954996],[3,103081,0.0787130354],[3,103088,0.004752502],[3,103111,0.0140947061],[3,103118,0.0110529215],[3,103122,0.0042882742],[3,103130,0.0076993653],[3,103184,0.004752502],[3,103204,0.0043902142],[3,103205,0.004752502],[3,103270,0.004752502],[3,103292,0.0123495636],[3,103296,0.0979730277],[3,103343,0.0045483296],[3,103359,0.0035197896],[3,103379,0.0033102691],[3,103385,0.0126203028],[3,103441,0.0970462746],[3,103474,0.081611569],[3,103482,0.108785431],[3,103500,0.0877212737],[3,103576,0.0806988062],[3,103615,0.0291814989],[3,103630,0.0232896992],[3,103640,0.0416938961],[3,103687,0.004752502],[3,103694,0.004752502],[3,103705,0.0074954996],[3,103837,0.011857401],[3,103840,0.004752502],[3,103950,0.0039426998],[3,104048,0.0310506814],[3,104172,0.0176609068],[3,104179,0.0265835177],[3,104381,0.004752502],[3,104565,0.004752502],[3,104596,0.0044258621],[3,104755,0.0836051831],[3,104762,0.0061376245],[3,104977,0.004752502],[3,105139,0.020991702],[3,105179,0.0042882742],[3,105193,0.0306333481],[3,105202,0.004752502],[3,105245,0.007745357],[3,105311,0.0117544281],[3,105326,0.0171737575],[3,105328,0.004752502],[3,105383,0.004752502],[3,105410,0.004752502],[3,105702,0.0036216233],[3,106039,0.0180488104],[3,106052,0.004752502],[3,106221,0.0359726884],[3,106317,0.0036356521],[3,106318,0.0243292354],[3,106348,0.004752502],[3,106371,0.016534515],[3,106441,0.0068313048],[3,106492,0.004752502],[3,106558,0.0031484034],[3,106573,0.004752502],[3,106593,0.0039426998],[3,106625,0.004752502],[3,106633,0.0318469518],[3,106774,0.0023811075],[3,106861,0.0201425202],[3,106911,0.0050878962],[3,106978,0.0076256447],[3,107223,0.0291020532],[3,107262,0.004752502],[3,107266,0.0243663128],[3,107268,0.1345464745],[3,107273,0.004752502],[3,107305,0.0196271793],[3,107374,0.0156224884],[3,107411,0.004752502],[3,107562,0.004752502],[3,107662,0.1275609042],[3,107686,0.004752502],[3,107700,0.004752502],[3,107711,0.0864280829],[3,107717,0.0036356521],[3,107757,0.0489019795],[3,107781,0.0031484034],[3,107926,0.004752502],[3,107928,0.0128825886],[3,107947,0.0023811075],[3,107973,0.0020036012],[3,108051,0.004752502],[3,108101,0.0066001386],[3,108120,0.0020036012],[3,108123,0.0036216233],[3,108236,0.0621066049],[3,108246,0.0244562209],[3,108340,0.0053635799],[3,108394,0.0056906354],[3,108493,0.0048798028],[3,108501,0.0033162238],[3,108585,0.0048798028],[3,108612,0.006483051],[3,108674,0.0295376343],[3,108691,0.014318107],[3,108724,0.0057641633],[3,108733,0.0050158108],[3,108765,0.0109431594],[3,108779,0.0044563546],[3,108946,0.0056906354],[3,108982,0.0246178463],[3,109045,0.0442049011],[3,109056,0.023536109],[3,109079,0.0205842376],[3,109151,0.0042465675],[3,109152,0.0056906354],[3,109251,0.0285837667],[3,109293,0.0128926898],[3,109382,0.0044563546],[3,109443,0.0033162238],[3,109455,0.0217774882],[3,109460,0.0143705701],[3,109482,0.0319650991],[3,109497,0.0227107081],[3,109546,0.0226223362],[3,109547,0.2045803356],[3,109792,0.0056906354],[3,109808,0.0056906354],[3,109952,0.0035703482],[3,109968,0.0045723645],[3,110000,0.0073304159],[3,110011,0.0044563546],[3,110266,0.0057641633],[3,110285,0.0365156529],[3,110288,0.0056906354],[3,110394,0.0053278866],[3,110406,0.0327216195],[3,110407,0.0245244772],[3,110411,0.0286046668],[3,110811,0.0053635799],[3,110994,0.0029382372],[3,111138,0.01270147],[3,111210,0.0228489651],[3,111326,0.0048798028],[3,111434,0.0056906354],[3,111450,0.0053635799],[3,111453,0.0053278866],[3,111622,0.0056906354],[3,111730,0.0088642472],[3,111907,0.0390889959],[3,111976,0.0104909962],[3,111983,0.0194057311],[3,112019,0.0056906354],[3,112051,0.0056906354],[3,112063,0.0053635799],[3,112088,0.0277224607],[3,112111,0.0229431613],[3,112126,0.020648776],[3,112180,0.0252884306],[3,112192,0.0529008287],[3,112236,0.0047149657],[3,112338,0.0158404685],[3,112413,0.0265721321],[3,112496,0.0056906354],[3,112531,0.0042465675],[3,112551,0.0140564718],[3,112589,0.0247473735],[3,112615,0.0033162238],[3,112619,0.0194419714],[3,112642,0.22797572],[3,112679,0.0164956557],[3,112730,0.0297891023],[3,112839,0.0042465675],[3,112856,0.0053278866],[3,112861,0.053593438],[3,112920,0.0048798028],[3,113031,0.0852313513],[3,113283,0.0103882123],[3,113296,0.0201428132],[3,113317,0.0204244663],[3,113327,0.0519131068],[3,113333,0.0182744534],[3,113350,0.0040844959],[3,113352,0.0308097649],[3,113365,0.0056906354],[3,113469,0.0044563546],[3,113500,0.0033162238],[3,113503,0.0121020775],[3,113678,0.008641248],[3,113918,0.0147434783],[3,113956,0.056699366],[3,113981,0.0029382372],[3,114057,0.1915989025],[3,114122,0.0672148418],[3,114134,0.0375415367],[3,114390,0.0086085755],[3,114536,0.1285971798],[3,114572,0.0371769839],[3,114592,0.0090630309],[3,114604,0.0056906354],[3,114672,0.0193339425],[3,114895,0.0131448982],[3,114909,0.0052258169],[3,115150,0.0080250099],[3,115196,0.0052258169],[3,115217,0.0200299287],[3,115326,0.0056906354],[3,115332,0.0056906354],[3,115341,0.0934160988],[3,115347,0.0108472725],[3,115568,0.0231449016],[3,115569,0.0312235342],[3,115570,0.0326454632],[3,115571,0.0859441859],[3,115572,0.0645875924],[3,115577,0.2642182581],[3,115578,0.0991006708],[3,115580,0.0413772532],[3,115581,0.0613919306],[3,115585,0.007237205],[3,115590,0.0236875417],[3,115594,0.4820585742],[3,115601,0.083058888],[3,115603,0.0564475156],[3,115606,0.0196720638],[3,115609,0.0418910451],[3,115610,0.0787102646],[3,115611,0.039162319],[3,115623,0.1029990251],[3,115625,0.204557265],[3,115626,0.09735617],[3,115628,0.0469689305],[3,115629,0.2872289645],[3,115630,0.0483108259],[3,115631,0.0307102295],[3,115632,0.0308487106],[3,115633,0.0565952432],[3,115635,0.0288632245],[3,115636,0.0676893865],[3,115637,0.2208242523],[3,115639,0.0858796921],[3,115641,0.0605350761],[3,115643,0.0345034029],[3,115644,0.0316845902],[3,115646,0.0958897814],[3,115647,0.0218550332],[3,115648,0.0261709532],[3,115649,0.4529769276],[3,115650,0.0441844526],[3,115651,0.1950209789],[3,115652,0.1592363273],[3,115653,0.0393087455],[3,115656,0.0247178473],[3,115657,0.0488467982],[3,115658,0.049681171],[3,115659,0.0388584163],[3,115661,0.041350579],[3,115662,0.0290929187],[3,115663,0.0239743052],[3,115664,0.0525740764],[3,115673,0.0269173137],[3,115674,0.1398908372],[3,115675,0.1715079757],[3,115677,0.1690765952],[3,115679,0.1640284711],[3,115680,0.0827871363],[3,115682,0.0355201156],[3,115683,0.042891666],[3,115684,0.0404608641],[3,115687,0.0311972614],[3,115688,0.1082668617],[3,115689,0.0725665681],[3,115694,0.0542929128],[3,115695,0.0315126299],[3,115696,0.0190136901],[3,115697,0.039809649],[3,115698,0.0318373649],[3,115700,0.1960740435],[3,115701,0.1333310834],[3,115704,0.0293126069],[3,115709,0.0271087118],[3,115711,0.1072470566],[3,115712,0.1243982333],[3,115714,0.0052258169],[3,115717,0.0404338663],[3,115723,0.0733507036],[3,115725,0.0917999875],[3,115726,0.0403589937],[3,115727,0.0280112804],[3,115729,0.1750313952],[3,115730,0.0052258169],[3,115731,0.0517074353],[3,115732,0.0277939113],[3,115733,0.0277628277],[3,115736,0.7053628206],[3,115737,0.0205699162],[3,115738,0.0050938653],[3,115742,0.2883939406],[3,115743,0.0139720399],[3,115745,0.0672680021],[3,115746,0.1023101031],[3,115747,0.1659297069],[3,115749,0.0563629808],[3,115750,0.0476064083],[3,115754,0.0262741077],[3,115766,0.0486796467],[3,115767,0.0544640589],[3,115768,0.0858469661],[3,115783,0.110480822],[3,115784,0.0654536859],[3,115786,0.0052258169],[3,115907,0.0816028898],[3,116498,0.0591664178],[3,116502,0.0257480172],[3,116505,0.0556649921],[3,116517,0.0237103628],[3,116518,0.0656911894],[3,116553,0.0553291388],[3,116566,0.0296957787],[3,116576,0.0952159452],[3,116612,0.0376350873],[3,116628,0.0052655847],[3,116636,0.0037061147],[3,116644,0.0169419755],[3,116668,0.0850287605],[3,116678,0.0745940037],[3,116682,0.0460033109],[3,116686,0.0862728935],[3,116702,0.0213715029],[3,116723,0.0565014829],[3,116736,0.0457997605],[3,116737,0.1036810472],[3,116760,0.0520031584],[3,116761,0.0701739556],[3,116767,0.0233470009],[3,116811,0.0546602173],[3,116867,0.0325108411],[3,116893,0.0052655847],[3,116906,0.0324496108],[3,116967,0.0081893906],[3,116978,0.1036810472],[3,116983,0.0041479576],[3,116996,0.071303816],[3,117002,0.0052655847],[3,117012,0.0777168956],[3,117048,0.0065524933],[3,117051,0.0044744744],[3,117053,0.0308385488],[3,117066,0.0923472908],[3,117100,0.0182031584],[3,117129,0.0809198283],[3,117142,0.1148541783],[3,117145,0.0250555128],[3,117181,0.0275240294],[3,117185,0.0033580455],[3,117200,0.0519876083],[3,117213,0.0189182715],[3,117216,0.0322941748],[3,117242,0.0040844959],[3,117251,0.0044744744],[3,117259,0.0317623483],[3,117277,0.0229711307],[3,117289,0.0825923809],[3,117291,0.0172327754],[3,117300,0.0239009654],[3,117305,0.0277577044],[3,117329,0.024748351],[3,117375,0.1085017815],[3,117397,0.0357250663],[3,117401,0.030105687],[3,117402,0.0599642575],[3,117415,0.0265375689],[3,117445,0.0040844959],[3,117481,0.189456175],[3,117482,0.0074662009],[3,117488,0.0032422267],[3,117494,0.0421896148],[3,117497,0.0023576798],[3,117527,0.0035004118],[3,117530,0.0169253316],[3,117595,0.0041479576],[3,117619,0.0052655847],[3,117663,0.0023576798],[3,117717,0.1034578626],[3,117744,0.0044744744],[3,117750,0.0358653076],[3,117759,0.0174718873],[3,117780,0.0431467556],[3,117796,0.0176489971],[3,117798,0.1940805264],[3,117857,0.0203794824],[3,117858,0.0224296395],[3,117872,0.0044744744],[3,117882,0.0413439157],[3,117891,0.1079260096],[3,117899,0.0044744744],[3,117908,0.0255502553],[3,117910,0.1654299832],[3,117971,0.0533175085],[3,118000,0.1061888296],[3,118091,0.1281527882],[3,118098,0.0271731665],[3,118099,0.0033580455],[3,118131,0.0726351785],[3,118204,0.1477285885],[3,118280,0.2020184876],[3,118297,0.0492275088],[3,118308,0.0665243404],[3,118421,0.0044744744],[3,118492,0.0473080362],[3,118736,0.0427357763],[3,118769,0.031345327],[3,118772,0.0196081614],[3,118797,0.0030327852],[3,118830,0.0527048982],[3,118858,0.0237736217],[3,118883,0.0044744744],[3,118909,0.01270147],[3,118933,0.0140139151],[3,118960,0.0259703628],[3,119061,0.0262211561],[3,119089,0.0816028898],[3,119090,0.0307926559],[3,119135,0.0925298374],[3,119165,0.025266712],[3,119182,0.1271378289],[3,119189,0.0534895836],[3,119216,0.0446280979],[3,119222,0.0322111897],[3,119238,0.0044744744],[3,119283,0.326995473],[3,119300,0.0219895557],[3,119366,0.1227710454],[3,119380,0.0575097224],[3,119418,0.0823458964],[3,119419,0.0052258169],[3,119426,0.0272215651],[3,119451,0.0052655847],[3,119488,0.0900785739],[3,119536,0.2093109462],[3,119550,0.0052258169],[3,119594,0.0765360449],[3,119713,0.0282024478],[3,119737,0.0422855694],[3,119806,0.0604514747],[3,119817,0.020987479],[3,119829,0.1103674161],[3,119905,0.2723338833],[3,119906,0.0598753052],[3,119974,0.0229732679],[3,119983,0.0017266101],[3,119991,0.0234651798],[3,120027,0.0764852419],[3,120054,0.0245317855],[3,120062,0.0041479576],[3,120133,0.0781767353],[3,120134,0.0238089854],[3,120156,0.1520333709],[3,120174,0.2550672332],[3,120215,0.0271782477],[3,120271,0.002103974],[3,120301,0.0044744744],[3,120351,0.0291912161],[3,120354,0.0232875388],[3,120361,0.0914298588],[3,120382,0.0522380025],[3,120392,0.0081407266],[3,120396,0.0262259937],[3,120429,0.3616954881],[3,120430,0.0047132611],[3,120440,0.0054648471],[3,120469,0.0744871007],[3,120483,0.0402080335],[3,120529,0.754451194],[3,120554,0.0138133969],[3,120573,0.045742397],[3,120608,0.0180649406],[3,120705,0.0948643596],[3,120727,0.0276682568],[3,120732,0.0674896026],[3,120789,0.4879842509],[3,120798,0.0189526495],[3,120803,0.0598650706],[3,120859,0.0055669499],[3,120913,0.0047132611],[3,120981,0.0184466248],[3,121026,0.0191459511],[3,121049,0.0464720375],[3,121055,0.0047132611],[3,121093,0.0133864938],[3,121188,0.0234708875],[3,121204,0.0521013803],[3,121262,0.3986819329],[3,121303,0.0055669499],[3,121336,0.0570359474],[3,121524,0.0363758478],[3,121616,0.0851619576],[3,121669,0.0389458706],[3,121695,0.0192507062],[3,121843,0.0192453827],[3,121875,0.0607523054],[3,121876,0.0295889613],[3,121907,0.0262260167],[3,122055,0.0023419928],[3,122107,0.0060762375],[3,122123,0.0107924216],[3,122138,0.0453767724],[3,122164,0.0380144465],[3,122179,0.0268315987],[3,122182,0.0241522312],[3,122306,0.0055046278],[3,122334,0.0220552966],[3,122404,0.0043231563],[3,122429,0.024325555],[3,122473,0.0047132611],[3,122483,0.0081609891],[3,122601,0.4062257468],[3,122792,0.0332304979],[3,122876,0.0311899365],[3,122960,0.1077952767],[3,122978,0.0035964706],[3,122986,0.0398982421],[3,123008,0.003903502],[3,123055,0.0529513588],[3,123174,0.0023419928],[3,123362,0.0074314948],[3,123392,0.0720131932],[3,123465,0.0559082352],[3,123513,0.0047132611],[3,123540,0.0600460667],[3,123633,0.0114073249],[3,123651,0.0025957808],[3,123682,0.0250739068],[3,123717,0.0259760245],[3,123753,0.0054648471],[3,123762,0.0133864938],[3,123960,0.1251315893],[3,124114,0.1036213879],[3,124188,0.0393945911],[3,124212,0.0117896637],[3,124229,0.0309540817],[3,124245,0.0822382693],[3,124252,0.0055046278],[3,124348,0.0047132611],[3,124553,0.1316786999],[3,124555,0.0098630717],[3,124598,0.0246813947],[3,124614,0.0384865095],[3,124643,0.5620011504],[3,124658,0.0385908771],[3,124666,0.0352376837],[3,124689,0.0130460593],[3,124711,0.0086232186],[3,124780,0.0190364598],[3,124832,0.003903502],[3,124836,0.0845847337],[3,124856,0.0622149459],[3,124872,0.062669336],[3,124919,0.0249114529],[3,124920,0.1716397546],[3,124928,0.7149694185],[3,124931,0.0602993446],[3,124952,0.0720105957],[3,125026,0.003903502],[3,125029,0.0275494265],[3,125040,0.0385958192],[3,125049,0.0083807011],[3,125063,0.0691117595],[3,125085,0.3439063249],[3,125099,0.1180401384],[3,125139,0.0291912161],[3,125335,0.0299555697],[3,125353,0.1136689924],[3,125359,0.0619786335],[3,125372,0.0288182404],[3,125507,0.0468453367],[3,125544,0.1208634996],[3,125569,0.0353534809],[3,125573,0.0365423497],[3,125580,0.0315453864],[3,125628,0.0581742424],[3,125633,0.0234924915],[3,125635,0.024246773],[3,125640,0.2742042754],[3,125667,0.0217845922],[3,125701,0.4616028479],[3,125713,0.0185403053],[3,125716,0.0240973758],[3,125738,0.0244524977],[3,125739,0.039548975],[3,125754,0.0047866917],[3,125796,0.0222918225],[3,125799,0.0539176801],[3,125817,0.0047132611],[3,125867,0.0043866386],[3,125883,0.0813781],[3,125890,0.0043231563],[3,125892,0.0054791034],[3,125908,0.0446187873],[3,125963,0.0387129355],[3,125964,0.0477126045],[3,125990,0.0044071431],[3,126011,0.0047132611],[3,126054,0.1089077822],[3,126060,0.0130460593],[3,126107,0.0100582688],[3,126108,0.0292509894],[3,126109,0.1258792687],[3,126136,0.0454610374],[3,126162,0.0844070143],[3,126205,0.2247042308],[3,126250,0.1005677732],[3,126280,0.0515996479],[3,126282,0.0221873787],[3,126288,0.1722402498],[3,126308,0.0223343159],[3,126324,0.024032977],[3,126336,0.0035964706],[3,126338,0.1236899856],[3,126374,0.0988392161],[3,126397,0.0910571889],[3,126418,0.0691559497],[3,126426,0.0055669499],[3,126513,0.0047132611],[3,126547,0.0345747057],[3,126564,0.0733773825],[3,126617,0.054749947],[3,126618,0.0673561326],[3,126647,0.0291372452],[3,126686,0.0144823046],[3,126710,0.0294159818],[3,126732,0.0248604143],[3,126736,0.0243410987],[3,126828,0.0344720898],[3,126849,0.1035756361],[3,126855,0.0047132611],[3,126907,0.0298704383],[3,126946,0.0047132611],[3,126949,0.1095654916],[3,126969,0.022303744],[3,126989,0.0921848211],[3,127019,0.0047132611],[3,127030,0.1493619701],[3,127112,0.0465231797],[3,127135,0.0019645066],[3,127143,0.005114866],[3,127210,0.0050486375],[3,127230,0.0798723588],[3,127244,0.0426002391],[3,127253,0.0157345662],[3,127260,0.0019645066],[3,127279,0.3981210655],[3,127405,0.003738883],[3,127463,0.715653526],[3,127508,0.0724580968],[3,127662,0.0042577597],[3,127695,0.012942922],[3,127778,0.0910252271],[3,127889,0.0047132611],[3,127891,0.0043866386],[3,127924,0.0060678864],[3,127959,0.0389209078],[3,127999,0.0056543414],[3,128045,0.0320132456],[3,128143,0.003738883],[3,128170,0.0199709065],[3,128237,0.0189945397],[3,128244,0.0047866917],[3,128278,0.0235411009],[3,128281,0.0089463896],[3,128290,0.038439103],[3,128299,0.1289208589],[3,128336,0.0289205735],[3,128342,0.0200299287],[3,128413,0.0820543203],[3,128444,0.0124482985],[3,128455,0.1040761284],[3,128456,0.0034806143],[3,128485,0.0648018069],[3,128497,0.0572099627],[3,128527,0.0399250025],[3,128531,0.212287384],[3,128553,0.1002058194],[3,128561,0.0495618123],[3,128562,0.0809102774],[3,128577,0.1343303049],[3,128589,0.0047132611],[3,128601,0.0047132611],[3,128621,0.0348206837],[3,128634,0.0055046278],[3,128664,0.0359844355],[3,128711,0.0380144465],[3,128728,0.020898495],[3,128777,0.0047132611],[3,128790,0.0384307613],[3,128812,0.0074314948],[3,128819,0.0081609891],[3,128861,0.1609418095],[3,128900,0.0573900762],[3,128977,0.0047866917],[3,129006,0.0023419928],[3,129160,0.0668150001],[3,129310,0.0739895293],[3,129322,0.0023419928],[3,129336,0.0515628787],[3,129439,0.0154262892],[3,129442,0.1427776169],[3,129453,0.0076599676],[3,129475,0.0047132611],[3,129630,0.0055046278],[3,129637,0.1071428309],[3,129684,0.0054648471],[3,129738,0.0539285329],[3,129771,0.0032711049],[3,129775,0.0436249115],[3,129792,0.5286048939],[3,129875,0.5977788753],[3,129882,0.0032019551],[3,129896,0.0043231563],[3,129991,0.1389474054],[3,129997,0.0537140975],[3,130003,0.0266250015],[3,130007,0.0542143805],[3,130014,0.0563193512],[3,130025,0.8546358897],[3,130026,0.0149047868],[3,130147,0.0636584386],[3,130160,0.0424465371],[3,130196,0.0498756157],[3,130213,0.0583946202],[3,130243,0.0188643063],[3,130245,0.0113776643],[3,130265,0.1312987364],[3,130293,0.0329287907],[3,130354,0.0653681109],[3,130362,0.0298334273],[3,130367,0.0047132611],[3,130397,0.0235033036],[3,130480,0.1789803237],[3,130518,0.0047866917],[3,130561,0.0023419928],[3,130576,0.0623371017],[3,130593,0.0060983099],[3,130630,0.0292371114],[3,130660,0.0211206761],[3,130888,0.1361832489],[3,130896,0.0350392292],[3,130905,0.0786780297],[3,130971,0.019851851],[3,131004,0.006114701],[3,131075,0.0047132611],[3,131172,0.0055046278],[3,131235,0.0025957808],[3,131279,0.1184064524],[3,131343,0.1678944502],[3,131409,0.1155505236],[3,131506,0.0047132611],[3,131522,0.0055046278],[3,131540,0.0047132611],[3,131545,0.0043231563],[3,131592,0.0049122897],[3,131608,0.0130460593],[3,131655,0.0193323157],[3,131666,0.0461625674],[3,131675,0.0047866917],[3,131837,0.1065147311],[3,131860,0.0710781903],[3,131884,0.0075862509],[3,131901,0.085909394],[3,131914,0.0602962323],[3,131922,0.0549379097],[3,131936,0.0023419928],[3,132053,0.0376917738],[3,132149,0.0731325038],[3,132261,0.0375080351],[3,132335,0.0327670163],[3,132394,0.1434475481],[3,132437,0.0056823763],[3,132488,0.0056823763],[3,132509,0.0267430904],[3,132528,0.067836805],[3,132569,0.4717224316],[3,132590,0.0250536101],[3,132609,0.0312576828],[3,132637,0.0523831126],[3,132657,0.0210586511],[3,132689,0.0980245471],[3,132694,0.1048196509],[3,132701,0.0278941128],[3,132749,0.1785003083],[3,132805,0.0393099485],[3,132846,0.031075867],[3,132892,0.0770512647],[3,132952,0.1862078848],[3,132980,0.0143670089],[3,132986,0.0056823763],[3,133010,0.1092839395],[3,133021,0.037776063],[3,133113,0.011030851],[3,133131,0.5571352842],[3,133279,0.0570786277],[3,133283,0.0193003782],[3,133286,0.0253360123],[3,133407,0.1213160357],[3,133469,0.0629093895],[3,133486,0.044019739],[3,133490,0.1141331129],[3,133515,0.0056823763],[3,133532,0.0365062388],[3,133634,0.0922557588],[3,133642,0.026906645],[3,133643,0.0327670163],[3,133690,0.0048715528],[3,133806,0.0713038355],[3,133808,0.0056823763],[3,133869,0.0029300088],[3,133901,0.0056823763],[3,133984,0.0375685867],[3,134031,0.0064349501],[3,134044,0.0277293446],[3,134095,0.3054239737],[3,134148,0.0829317173],[3,134159,0.0052917587],[3,134176,0.0042383245],[3,134221,0.0260939067],[3,134270,0.0048715528],[3,134295,0.0346166919],[3,134365,0.0243869613],[3,134366,0.0052917587],[3,134385,0.0052917587],[3,134416,0.0099514155],[3,134514,0.0365203651],[3,134521,0.0275717191],[3,134545,0.1484431931],[3,134601,0.0056823763],[3,134621,0.0042383245],[3,134660,0.0029300088],[3,134676,0.0281947465],[3,134681,0.1208688186],[3,134816,0.1045085515],[3,134843,0.0095521961],[3,134911,0.0361522397],[3,135003,0.0274324713],[3,135082,0.0150754685],[3,135163,0.0057559034],[3,135197,0.0047067175],[3,135260,0.0207860069],[3,135319,0.0056823763],[3,135432,0.1798250162],[3,135508,0.0086790055],[3,135536,0.0056823763],[3,135594,0.0345718554],[3,135700,0.0487879867],[3,135737,0.3486184436],[3,135766,0.077760088],[3,135792,0.0514034263],[3,135798,0.0576140284],[3,135814,0.0053553244],[3,135836,0.3068668731],[3,135854,0.0227111381],[3,135863,0.0649186384],[3,135873,0.0344770118],[3,135890,0.0511117382],[3,135894,0.0077638006],[3,135896,0.0851560787],[3,135944,0.0045641179],[3,135983,0.0045641179],[3,135987,0.029740176],[3,136006,0.0171045556],[3,136009,0.0056823763],[3,136021,0.0128843501],[3,136034,0.006474783],[3,136088,0.0056823763],[3,136112,0.0056823763],[3,136119,0.0340619091],[3,136150,0.0532899526],[3,136164,0.1000967881],[3,136256,0.0044481093],[3,136279,0.059073237],[3,136317,0.0029300088],[3,136339,0.0505473885],[3,136403,0.078414982],[3,136445,0.0339529564],[3,136463,0.0048715528],[3,136513,0.0056823763],[3,136542,0.0053553244],[3,136650,0.0178399611],[3,136699,0.0095962604],[3,136720,0.049524783],[3,136833,0.0064120477],[3,136891,0.0727974263],[3,136952,0.2594550525],[3,136979,0.0799086953],[3,137027,0.0491392645],[3,137030,0.070856323],[3,137053,0.0194621268],[3,137077,0.1191979715],[3,137113,0.0407483014],[3,137149,0.0056823763],[3,137156,0.0044481093],[3,137178,0.3019468912],[3,137223,0.0056823763],[3,137302,0.0298928022],[3,137353,0.6901757258],[3,137423,0.097344064],[3,137533,0.0072289286],[3,137574,0.0318870627],[3,137575,0.0318901117],[3,137586,0.0437944884],[3,137619,0.022795847],[3,137650,0.1098342945],[3,137659,0.0070471441],[3,138213,0.0056823763],[3,138215,0.0295168097],[3,138292,0.0056823763],[3,138340,0.2074721517],[3,138379,0.0065371871],[3,138410,0.0056823763],[3,138523,0.0130593286],[3,138546,0.085531887],[3,138620,0.0516741234],[3,138684,0.0056823763],[3,138688,0.1151544407],[3,138707,0.0048715528],[3,138834,0.0215109685],[3,138907,0.4484341601],[3,138933,0.324401578],[3,138976,0.0191900044],[3,139051,0.091525219],[3,139169,0.02944389],[3,139189,0.0214225368],[3,139201,0.1371811262],[3,139216,0.0449749921],[3,139407,0.0275703356],[3,139421,0.0919361686],[3,139546,0.0056823763],[3,139555,0.0230591511],[3,139617,0.0943314384],[3,139650,0.0246186696],[3,139665,0.0057559034],[3,139718,0.1205283278],[3,139769,0.0208477115],[3,139792,0.0179423375],[3,139862,0.4203703816],[3,139894,0.0053553244],[3,139907,0.0758650395],[3,139919,0.0056823763],[3,139932,0.0741736125],[3,139960,0.0256355926],[3,139981,0.0252438212],[3,140095,0.13221842],[3,140176,0.1090561211],[3,140190,0.0755822685],[3,140201,0.1553647603],[3,140343,0.0056823763],[3,140369,0.006474783],[3,140454,0.0099210687],[3,140462,0.1285890714],[3,140488,0.0602480703],[3,140500,0.0395560806],[3,140561,0.0979504944],[3,140604,0.3927564096],[3,140607,0.0431880949],[3,140711,0.035981591],[3,140766,0.0057559034],[3,141134,0.0345121498],[3,141194,0.0284368608],[3,141228,0.0052917587],[3,141247,0.0056823763],[3,141406,0.0213357496],[3,141417,0.0044481093],[3,141457,0.074855469],[3,141558,0.1436115364],[3,141685,0.0226985571],[3,141715,0.0056823763],[3,141788,0.0057559034],[3,141941,0.0056823763],[3,141947,0.0084041827],[3,141982,0.0278989285],[3,142050,0.0153523787],[3,142162,0.0539036946],[3,142255,0.0448027241],[3,142310,0.2736902894],[3,142359,0.0056823763],[3,142482,0.0979289745],[3,142523,0.0084041827],[3,142533,0.0311206845],[3,142676,0.0029300088],[3,142718,0.0507413442],[3,142783,0.0033079912],[3,142785,0.0630792233],[3,142851,0.0736830341],[3,142855,0.0048715528],[3,142894,0.0486172172],[3,142957,0.0042383245],[3,142960,0.0915212805],[3,143082,0.033815563],[3,143086,0.0298290258],[3,143118,0.0598300947],[3,143235,0.05692071],[3,143261,0.0656286282],[3,143302,0.0153523787],[3,143422,0.0375551173],[3,143428,0.0053553244],[3,143501,0.0319226581],[3,143524,0.0387262986],[3,143525,0.3139401739],[3,143568,0.0276402694],[3,143689,0.0048715528],[3,143705,0.0110236594],[3,143938,0.0861882539],[3,144005,0.0206101896],[3,144028,0.0549240325],[3,144258,0.0164094854],[3,144309,0.0175402892],[3,144454,0.0056823763],[3,144486,0.0499738075],[3,144506,0.2835944026],[3,144709,0.0257209137],[3,144864,0.0246724593],[3,144889,0.0119159029],[3,144904,0.0548010997],[3,144910,0.0057559034],[3,145250,0.0052917587],[3,145327,0.0096548643],[3,145334,0.024313277],[3,145471,0.0619947739],[3,145600,0.0078197656],[3,145625,0.0631256583],[3,145640,0.0424466673],[3,145680,0.0907849167],[3,145732,0.011037701],[3,145771,0.018278634],[3,145865,0.0064349501],[3,145944,0.0035621128],[3,145956,0.0056823763],[3,145978,0.0369101127],[3,146145,0.1047240796],[3,146191,0.0602597923],[3,146198,0.1030116526],[3,146199,0.0053553244],[3,146233,0.0977601734],[3,146238,0.0465264617],[3,146263,0.0331525727],[3,146265,0.0756202765],[3,146273,0.023517354],[3,146279,0.0360563106],[3,146286,0.0501981136],[3,146298,0.0070856582],[3,146305,0.0705717835],[3,146322,0.0104537655],[3,146370,0.0349845245],[3,146378,0.042909026],[3,146382,0.2152474511],[3,146388,0.0596267985],[3,146413,0.0342053263],[3,146426,0.0812399136],[3,146438,0.0056823763],[3,146472,0.0586521035],[3,146480,0.259876996],[3,146528,0.0703894978],[3,146536,0.0033079912],[3,146564,0.0033079912],[3,146579,0.0745040911],[3,146580,0.0403331017],[3,146610,0.0056823763],[3,146635,0.1200145335],[3,146646,0.0304804313],[3,146714,0.0465529635],[3,146738,0.0287046738],[3,146743,0.0140261269],[3,146744,0.0693297888],[3,146757,0.0756202765],[3,146786,0.0631653797],[3,146814,0.1064791817],[3,146900,0.0351014324],[3,146921,0.0127680799],[3,146925,0.0029300088],[3,147034,0.0500945979],[3,147063,0.1466485251],[3,147080,0.0745040911],[3,147120,0.0158352466],[3,147136,0.0623609029],[3,147172,0.085413549],[3,147205,0.0095521961],[3,147226,0.0352594121],[3,147253,0.0846637526],[3,147288,0.0854966618],[3,147329,0.0756202765],[3,147477,0.0506204527],[3,147485,0.0396531615],[3,147486,0.1124627164],[3,147501,0.0537459637],[3,147623,0.0517656011],[3,147688,0.0257813584],[3,147689,0.0269118978],[3,147814,0.0744844369],[3,147821,0.2566465561],[3,147881,0.0056823763],[3,147918,0.1310595704],[3,148031,0.0332108688],[3,148095,0.0056823763],[3,148173,0.0578088597],[3,148184,0.0396991808],[3,148206,0.0095521961],[3,148251,0.0231585458],[3,148281,0.040568418],[3,148313,0.0052917587],[3,148334,0.0427045655],[3,148385,0.0035621128],[3,148515,0.0862271039],[3,148553,0.1279531148],[3,148565,0.1284706873],[3,148594,0.0200359251],[3,148643,0.0321177009],[3,148674,0.0056823763],[3,148916,0.1519343741],[3,148924,0.0164872757],[3,148938,0.0081905369],[3,149408,0.1207290859],[3,149432,0.0056823763],[3,149450,0.098697636],[3,149514,0.0300001903],[3,149610,0.107077173],[3,149627,0.0052591973],[3,149639,0.0337303537],[3,149679,0.0056823763],[3,149685,0.0056823763],[3,149768,0.0057559034],[3,149787,0.7275940138],[3,149847,0.1283417241],[3,149852,0.0190081885],[3,150077,0.0056823763],[3,150215,0.0259040949],[3,150265,0.0618382097],[3,150321,0.3051332536],[3,150328,0.0416734657],[3,150330,0.0028341976],[3,150386,0.0044680938],[3,150395,0.1873624938],[3,150434,0.021321821],[3,150442,0.0034662192],[3,150448,0.008428833],[3,150461,0.088257207],[3,150627,0.1663584073],[3,150710,0.0413234248],[3,150711,0.0551759901],[3,150798,0.6452215604],[3,150822,0.0139228541],[3,151198,0.1994044568],[3,151312,0.0660608212],[3,151341,0.006474783],[3,151384,0.0047754886],[3,151469,0.0056823763],[3,151693,0.0352717138],[3,151745,0.1043667985],[3,151799,0.006474783],[3,151821,0.0334573201],[3,151909,0.0779059404],[3,152054,0.0428144077],[3,152069,0.0564475156],[3,152095,0.032320918],[3,152096,0.062070287],[3,152107,0.0936732935],[3,152200,0.0397482953],[3,152223,0.2666535886],[3,152224,0.0812399136],[3,152244,0.0086329558],[3,152599,0.0282637609],[3,152970,0.0495361281],[3,153011,0.0612349028],[3,153110,0.0041423428],[3,153195,0.0341284423],[3,153199,0.0815121432],[3,153266,0.0063157829],[3,153302,0.063842067],[3,153382,0.0917155814],[3,153406,0.0070692455],[3,153420,0.059471778],[3,153472,0.0263505672],[3,153499,0.0322356473],[3,153588,0.250017113],[3,153658,0.1978326549],[3,153777,0.0056823763],[3,153800,0.0644470958],[3,153896,0.0056823763],[3,153952,0.0314808606],[3,154317,0.0077638006],[3,154548,0.0327432913],[3,154704,0.0750243018],[3,154728,0.0322613674],[3,155120,0.0229978142],[3,155132,0.0429530342],[3,155362,0.0057559034],[3,155570,0.0267530675],[3,155601,0.0573620107],[3,155602,0.2126176051],[3,155655,0.0057559034],[3,155794,0.5536693098],[3,155916,0.0239497637],[3,156336,0.0284122824],[3,156344,0.0232114145],[3,156373,0.0812152368],[3,156414,0.0069507965],[3,156459,0.1134113491],[3,156561,0.0664966688],[3,156601,0.0473895965],[3,156720,0.0064349501],[3,156804,0.0957081577],[3,157090,0.0057559034],[3,157097,0.0575147293],[3,157225,0.0522496255],[3,157375,0.0056823763],[3,157378,0.0086329558],[3,157494,0.0047754886],[3,157580,0.0041383216],[3,157586,0.0729701572],[3,157716,0.0337584821],[3,157717,0.0569127221],[3,157733,0.0292350709],[3,157783,0.1262356571],[3,157833,0.0385667484],[3,158040,0.0241646602],[3,158145,0.0256613929],[3,158257,0.0057559034],[3,158310,0.0667995583],[3,158311,0.0498683203],[3,158340,0.0056823763],[3,158380,0.0269986869],[3,158402,0.0052917587],[3,158470,0.0621164271],[3,158511,0.2828992756],[3,158626,0.0274829546],[3,158785,0.0203941578],[3,158937,0.0219431656],[3,159012,0.1143593765],[3,159015,0.1458126328],[3,159073,0.0259726279],[3,159134,0.0194060851],[3,159289,0.0384769366],[3,159789,0.0573289358],[3,159817,0.0074359321],[3,159822,0.0198275395],[3,159904,0.041430116],[3,159952,0.0052591973],[3,159980,0.0312748319],[3,160035,0.0056823763],[3,160055,0.0827066109],[3,160248,0.0056823763],[3,160472,0.0241476776],[3,160697,0.0056823763],[3,160822,0.0316426016],[3,160952,0.1152598378],[3,161131,0.0140261269],[3,161322,0.0568200682],[3,161404,0.1225183896],[3,161422,0.0245922375],[3,161424,0.0187010889],[3,161426,0.0047754886],[3,161544,0.1147602686],[3,161755,0.2626446711],[3,161868,0.2656624023],[3,161908,0.0249719851],[3,161914,0.0056823763],[3,161916,0.0065371871],[3,162191,0.0221261211],[3,162489,0.0217256626],[3,162671,0.028357928],[3,162719,0.0314551503],[3,162811,0.0339547942],[3,162933,0.0056823763],[3,163005,0.0705717835],[3,163077,0.022558763],[3,163126,0.0257264653],[3,163180,0.0747206141],[3,163206,0.1143344161],[3,163227,0.0140261269],[3,163256,0.0224233914],[3,163506,0.2592125757],[3,163574,0.0175655959],[3,163804,0.1104753855],[3,164374,0.0056823763],[3,164483,0.0203555825],[3,164612,0.0110236594],[3,164952,0.0056823763],[3,165056,0.0052591973],[3,165105,0.0126437732],[3,165335,0.0056823763],[3,165418,0.0037599127],[3,165629,0.0298290258],[3,165631,0.0822196552],[3,165829,0.0056823763],[3,165851,0.0032121307],[3,165936,0.0464324167],[3,165984,0.0321014565],[3,166015,0.0052591973],[3,166118,0.0343195554],[3,166156,0.0056823763],[3,166182,0.1216257124],[3,166245,0.0056823763],[3,166267,0.0917111673],[3,166396,0.0056823763],[3,166616,0.0047754886],[3,166641,0.0251562837],[3,166667,0.0064349501],[3,166839,0.0085591422],[3,166858,0.0052917587],[3,166969,0.0091346359],[3,166985,0.0056823763],[3,166998,0.0069507965],[3,167002,0.0975620504],[3,167017,0.0063157829],[3,167056,0.0251177374],[3,167280,0.0056823763],[3,167340,0.0379556814],[3,167507,0.0056823763],[3,167836,0.1004375423],[3,168011,0.0056823763],[3,168086,0.0267530675],[3,168320,0.0241587441],[3,168602,0.1735919057],[3,168640,0.0149836283],[3,169141,0.0065371871],[3,169221,0.0028341976],[3,169275,0.0056823763],[3,169427,0.0400097375],[3,169807,0.0057559034],[3,169892,0.1212462831],[3,169936,0.0056823763],[3,170059,0.0269071187],[3,170067,0.0329186026],[3,170121,0.0044680938],[3,170327,0.008428833],[3,170362,0.0064349501],[3,170522,0.0084041827],[3,170657,0.0056823763],[3,170844,0.0028341976],[3,170872,0.0028341976],[3,170874,0.0064349501],[3,171160,0.0056823763],[3,171414,0.0693859971],[3,171433,0.0085591422],[3,171468,0.0056823763],[3,171581,0.0818298145],[3,171587,0.0561963744],[3,171592,0.0388393773],[3,171787,0.0959064995],[3,172240,0.0041423428],[3,172263,0.0047754886],[3,172264,0.0075257184],[3,172651,0.0044540491],[3,173092,0.0280322781],[3,173288,0.0053818032],[3,173450,0.0341542621],[3,173704,0.006474783],[3,173742,0.1714606766],[3,173877,0.0077638006],[3,174078,0.0056823763],[3,174149,0.0028341976],[3,174442,0.0264057916],[3,174463,0.0032121307],[3,174467,0.0047754886],[3,174474,0.0202637003],[3,174571,0.185441578],[3,174574,0.0081905369],[3,174730,0.0936992025],[3,174832,0.0468152581],[3,174859,0.0828074587],[3,174900,0.0249479824],[3,174917,0.0086329558],[3,175231,0.0236778507],[3,175396,0.0465729617],[3,175656,0.0041423428],[3,175672,0.0087648109],[3,175787,0.0487916687],[3,175825,0.0216427208],[3,175939,0.1617391313],[3,175959,0.0028341976],[3,176773,0.0265284192],[3,176903,0.0234659356],[3,176941,0.0153523787],[3,176996,0.0213366976],[3,177297,0.0064349501],[3,177298,0.0162253024],[3,177340,0.0028341976],[3,177484,0.0629676104],[3,177576,0.0056823763],[3,177695,0.0065371871],[3,177799,0.0277382883],[3,177918,0.1516404386],[3,178060,0.056749873],[3,178067,0.0233560713],[3,178080,0.0052917587],[3,178381,0.0043521003],[3,178596,0.0140261269],[3,178871,0.0070856582],[3,178873,0.0028341976],[3,178876,0.0052917587],[3,179142,0.0043521003],[3,179180,0.3195710933],[3,179187,0.0116685067],[3,179261,0.0477395624],[3,179300,0.0070856582],[3,179371,0.0081905369],[3,179588,0.0676506453],[3,179623,0.019415499],[3,179629,0.006474783],[3,179654,0.0081905369],[3,179692,0.032696357],[3,179849,0.0052917587],[3,179940,0.006474783],[3,179950,0.0052591973],[3,179980,0.2546936841],[3,180000,0.0028341976],[3,180012,0.0398437568],[3,180022,0.0211220547],[3,180028,0.0093546367],[3,180097,0.0056823763],[3,180128,0.0233473862],[3,180281,0.0829932502],[3,180458,0.0056823763],[3,180497,0.0171998116],[3,180502,0.0210185767],[3,180588,0.206612449],[3,180654,0.0143670089],[3,180768,0.0142777606],[3,180806,0.4652175213],[3,180923,0.0277613191],[3,181072,0.2239614597],[3,181133,0.0064349501],[3,181195,0.0056823763],[3,181202,0.3037837195],[3,181239,0.1121403082],[3,181327,0.0273397484],[3,181383,0.0070856582],[3,181390,0.0338779705],[3,181435,0.0065371871],[3,181494,0.0900416146],[3,181541,0.0752522674],[3,181814,0.0199404806],[3,181892,0.147344885],[3,182099,0.0219820912],[3,182155,0.1936038217],[3,182487,0.0028341976],[3,182543,0.0477265012],[3,182561,0.3797043958],[3,182585,0.0043521003],[3,182660,0.045087557],[3,182776,0.1175115028],[3,182821,0.0056823763],[3,182905,0.0280809009],[3,182934,0.0536472927],[3,182957,0.0401883696],[3,182959,0.1021988663],[3,182995,0.0087229168],[3,183053,0.0057559034],[3,183067,0.0594891283],[3,183075,0.0546737911],[3,183128,0.0286419839],[3,183263,0.0128751835],[3,183314,0.0362032854],[3,183316,0.0093258563],[3,183369,0.03362588],[3,183380,0.0617981358],[3,183407,0.1420585844],[3,183523,0.0064540065],[3,183529,0.143846979],[3,183535,0.2146138924],[3,183625,0.0033236886],[3,183705,0.0229772857],[3,183776,0.1186869583],[3,184256,0.104107458],[3,184588,0.0384245257],[3,184590,0.0151603194],[3,184683,0.1398031256],[3,184924,0.0057589726],[3,185088,0.0058067545],[3,185183,0.004943748],[3,185185,0.015783193],[3,185197,0.0275274605],[3,185215,0.0050365481],[3,185257,0.0050365481],[3,185303,0.0053757033],[3,185310,0.0192523817],[3,185331,0.0221270763],[3,185343,0.0214200667],[3,185405,0.0050365481],[3,185558,0.0050365481],[3,185600,0.0226967335],[3,185712,0.1008495048],[3,185750,0.0199485212],[3,185809,0.0062026667],[3,185880,0.0045649254],[3,185895,0.0395761291],[3,185960,0.00702827],[3,185969,0.0288864289],[3,186289,0.0081741517],[3,186291,0.0071161516],[3,186306,0.0297053661],[3,186343,0.0057589726],[3,186422,0.0045649254],[3,186570,0.0050365481],[3,186606,0.006188589],[3,186666,0.0049673332],[3,186860,0.0271847681],[3,186864,0.0055775825],[3,186932,0.0050365481],[3,186955,0.0050365481],[3,186958,0.0161248772],[3,186991,0.0235691564],[3,187017,0.0050365481],[3,187088,0.0214905174],[3,187530,0.5011868031],[3,187590,0.0088734007],[3,187693,0.0959451336],[3,187744,0.0846036072],[3,187748,0.2159111795],[3,187769,0.16572474],[3,187804,0.0759388148],[3,188116,0.0155793031],[3,188123,0.0295845187],[3,188279,0.0069956319],[3,188288,0.0590109089],[3,188477,0.0050365481],[3,188577,0.1228416972],[3,188608,0.0158752662],[3,188654,0.0276465835],[3,188748,0.0735303453],[3,188781,0.0050365481],[3,188803,0.0631067799],[3,188894,0.0851194905],[3,189180,0.1123819033],[3,189245,0.0091774894],[3,189295,0.0213320236],[3,189482,0.0431617614],[3,189598,0.0204030274],[3,189604,0.3952015818],[3,189615,0.0050365481],[3,189647,0.0432279928],[3,189675,0.0079107975],[3,189708,0.0347384326],[3,189747,0.0266201592],[3,189759,0.0050365481],[3,189773,0.0074320358],[3,189780,0.0271317467],[3,189916,0.0402682654],[3,189995,0.0091732851],[3,190034,0.0103224901],[3,190283,0.0110985684],[3,190740,0.0966826595],[3,190808,0.1529659242],[3,190881,0.0069588816],[3,191043,0.0074320358],[3,191048,0.0740949503],[3,191161,0.031500639],[3,191181,0.0074320358],[3,191281,0.0103224901],[3,191300,0.0322071333],[3,191406,0.0074320358],[3,191506,0.1518534241],[3,191905,0.0082743636],[3,192020,0.1387215674],[3,192024,0.0089110604],[3,192129,0.0128664132],[3,192201,0.3780561791],[3,192447,0.031733959],[3,192469,0.0182962736],[3,192949,0.027663496],[3,192958,0.0086019415],[3,193204,0.0291440511],[3,193421,0.0081568064],[3,193531,0.2914592194],[3,193564,0.0516447721],[3,193701,0.0824506262],[3,193847,0.0110128227],[3,194016,0.0310735335],[3,194023,0.0074320358],[3,194028,0.0435704201],[3,194052,0.06460952],[3,194221,0.0170854106],[3,194268,0.0068415332],[3,194271,0.0170654002],[3,194321,0.0152582475],[3,194375,0.0074320358],[3,194429,0.0586465456],[3,194530,0.0263760047],[3,194671,0.0074320358],[3,194712,0.0547423085],[3,194722,0.0442131489],[3,195292,0.070033136],[3,195300,0.0425212923],[3,195376,0.0513429448],[3,195402,0.0352092583],[3,195452,0.0236489319],[3,195714,0.0333341943],[3,195811,0.1884286664],[3,195924,0.0352679665],[3,196001,0.0101110333],[3,196002,0.0075944479],[3,196020,0.0270221306],[3,196026,0.0069588816],[3,196044,0.1132298354],[3,196071,0.0074320358],[3,196091,0.1347064289],[3,196145,0.0719172582],[3,196146,0.0210482205],[3,196250,0.0699177552],[3,196364,0.0712980242],[3,196379,0.0702569746],[3,196572,0.1414623391],[3,196615,0.0068415332],[3,196630,0.0182065558],[3,196658,0.0091732851],[3,196913,0.0092797465],[3,197243,0.3608660287],[3,197485,0.0262321773],[3,197504,0.0068415332],[3,197864,0.0565514375],[3,197890,0.1768127999],[3,197969,0.1000694951],[3,197972,0.0084852974],[3,197973,0.0069588816],[3,198002,0.0213706613],[3,198140,0.0095183929],[3,198142,0.0074320358],[3,198148,0.0426211575],[3,198252,0.0255269741],[3,198452,0.0081568064],[3,198500,0.0074320358],[3,198508,0.0082263206],[3,198648,0.0724689746],[3,198724,0.0069588816],[3,198758,0.0420411661],[3,198889,0.0074320358],[3,198893,0.0086019415],[3,198965,0.0074320358],[3,199150,0.0084852974],[3,199821,0.027828031],[3,199864,0.0736149683],[3,199865,0.1198568522],[3,199883,0.3420872382],[3,199943,0.0074320358],[3,199968,0.0074320358],[3,199996,0.0223642403],[3,200041,0.0180586192],[3,200206,0.050262836],[3,200238,0.0074320358],[3,200761,0.0997637992],[3,200924,0.0074320358],[3,200986,0.0104599934],[3,201056,0.7640704725],[3,201361,0.0074320358],[3,201396,0.0817738111],[3,201499,0.0583092818],[3,202080,0.0438157365],[3,202084,0.0081732432],[3,202156,0.2417666406],[3,202227,0.0625872002],[3,202532,0.0986318431],[3,202544,0.024816325],[3,202819,0.0210482205],[3,202946,0.0226531521],[3,203046,0.0074320358],[3,203243,0.040011572],[3,203429,0.0277284197],[3,204054,0.0099055866],[3,204076,0.101843085],[3,204220,0.0332621604],[3,204294,0.0099055866],[3,204344,0.1384586383],[3,204381,0.0099055866],[3,204502,0.2228660453],[3,204540,0.0278559912],[3,204725,0.0443019164],[3,204815,0.0425491213],[3,204962,0.3804562149],[3,204986,0.0581921633],[3,205071,0.0135288352],[3,205284,0.2136411458],[3,205326,0.0273662882],[3,205394,0.0201192532],[3,205421,0.0523151702],[3,205524,0.0219604434],[3,205870,0.0109623687],[3,205893,0.0330370316],[3,205984,0.1358391477],[3,206088,0.0490361248],[3,206114,0.0093131103],[3,206511,0.0099055866],[3,206785,0.2395381478],[3,206862,0.0099055866],[3,207111,0.0342273543],[3,207229,0.0106327797],[3,207290,0.0243877464],[3,207348,0.0106492715],[3,207604,0.0265779419],[3,207614,0.0507769214],[3,207675,0.0872998661],[3,208167,0.028135821],[3,208169,0.0212372868],[3,208627,0.0106327797],[3,208640,0.0117594732],[3,208761,0.0295934561],[3,208828,0.0490762894],[3,208893,0.0257932327],[3,209027,0.0128057021],[3,209395,0.023025021],[3,209885,0.0279994786],[3,209999,0.0109623687],[3,210194,0.0416057403],[3,210510,0.0322921405],[3,210533,0.0315434135],[3,210709,0.009812174],[3,211144,0.0224501486],[3,211708,0.0119989173],[3,211803,0.0391620741],[3,211949,0.0107025262],[3,212224,0.0107507298],[3,212444,0.0099055866],[3,212471,0.0256006982],[3,212484,0.0608894793],[3,212502,0.0231500003],[3,212970,0.0099055866],[3,213570,0.0836735846],[3,213641,0.0099055866],[3,214310,0.0099055866],[3,214457,0.0276916072],[3,214460,0.0099055866],[3,214462,0.0240322977],[3,214485,0.0142453993],[3,214562,0.0099055866],[3,214701,0.0109623687],[3,214997,0.0358122947],[3,215043,0.0094308509],[3,215107,0.0094308509],[3,215510,0.0363728962],[3,215567,0.0259333644],[3,215576,0.0118776021],[3,215896,0.009812174],[3,215912,0.0099055866],[3,216436,0.0099055866],[3,216841,0.0941849042],[3,217361,0.0129210061],[3,217469,0.0129210061],[3,217591,0.0289277321],[3,217766,0.0144110008],[3,217809,0.0230580132],[3,217834,0.0129210061],[3,217838,0.0302850371],[3,217942,0.0140995891],[3,217964,0.0178271736],[3,218114,0.0527855873],[3,218232,0.0487908267],[3,218508,0.0296789885],[3,218509,0.0321338567],[3,219249,0.023520609],[3,219461,0.0234160229],[3,219487,0.0409233651],[3,219641,0.0129210061],[3,219678,0.0269630168],[3,219936,0.0129210061],[3,219939,0.0129210061],[3,220102,0.0129210061],[3,220491,0.187758746],[3,220646,0.044562581],[3,220892,0.0129210061],[3,220897,0.0606004989],[3,221039,0.0306093415],[3,221093,0.016614734],[3,221098,0.0161700938],[3,221161,0.1052271975],[3,221342,0.0189876531],[3,221503,0.3063769893],[3,221560,0.0144110008],[3,221635,0.0322791628],[3,221638,0.0124443425],[3,221675,0.0456076675],[3,221708,0.0146070627],[3,221739,0.5660084717],[3,221864,0.0190937594],[3,221866,0.0129210061],[3,222024,0.0316007799],[3,222103,0.0129210061],[3,222196,0.0691679695],[3,222314,0.1516494268],[3,222320,0.4723725424],[3,222601,0.0227625004],[3,222911,0.1981986662],[3,222951,0.1030523125],[3,223360,0.3110064019],[3,223394,0.0146517248],[3,223651,0.0235764502],[3,223878,0.0139820799],[3,223919,0.0498981792],[3,224019,0.0849047781],[3,224039,0.1433745278],[3,224061,0.0321632746],[3,224077,0.0184100111],[3,224100,0.0162721578],[3,224110,0.0137695815],[3,224215,0.04281043],[3,224287,0.0311655756],[3,224428,0.2008861978],[3,224586,0.0310262169],[3,224659,0.0140369827],[3,224684,0.0726266582],[3,224755,0.0688243427],[3,224813,0.1169370764],[3,224887,0.0226060334],[3,224909,0.0146786318],[3,224947,0.1771789419],[3,225030,0.035733477],[3,225311,0.0131881033],[3,225499,0.037319282],[3,225583,0.013988566],[3,225601,0.0368434799],[3,225831,0.0131881033],[3,225881,0.2993873569],[3,226091,0.0215053891],[3,226196,0.0462142662],[3,226252,0.0729795947],[3,226376,0.0153472222],[3,226482,0.1141444396],[3,226521,0.5044262661],[3,226581,0.0272351448],[3,226717,0.0425739292],[3,226796,0.0638453062],[3,226817,0.0160941157],[3,226846,0.0241368441],[3,226916,0.0717954156],[3,226992,0.0301177482],[3,227086,0.0131881033],[3,227248,0.0848028441],[3,227349,0.0185367168],[3,227567,0.0142495572],[3,227897,0.0146786318],[3,228083,0.1284268529],[3,228091,0.1202959705],[3,228100,0.0131881033],[3,228306,0.0131881033],[3,228351,0.013988566],[3,228360,0.0706675077],[3,228375,0.1001542824],[3,228424,0.0603201215],[3,228467,0.1029288563],[3,228887,0.0131881033],[3,228949,0.0133517786],[3,229298,0.0162046139],[3,229622,0.3206981902],[3,229856,0.0384741626],[3,229889,0.0832217443],[3,229928,0.0131881033],[3,229945,0.0131881033],[3,229961,0.0748285014],[3,229987,0.6243359821],[3,230023,0.0816674771],[3,230029,0.2651598006],[3,230053,0.1576170264],[3,230075,0.0691715177],[3,230085,0.2178998516],[3,230168,0.3148287942],[3,230209,0.2284432278],[3,230311,0.0226962769],[3,230359,0.0279451899],[3,230439,0.0131881033],[3,230511,0.066728153],[3,230583,0.1112060829],[3,230590,0.0131881033],[3,230594,0.6301074446],[3,230621,0.0433591362],[3,230637,0.0369542736],[3,230663,0.2581565366],[3,230670,0.1313431684],[3,230727,0.0125930077],[3,230838,0.0293544239],[3,230841,0.0957121026],[3,230844,0.0399000946],[3,230875,0.0934771638],[3,230876,0.7373046014],[3,230887,0.1945046687],[3,230981,0.0346744992],[3,231000,0.0890701803],[3,231036,0.0360008921],[3,231081,0.6947571739],[3,231095,0.0358625536],[3,231109,0.1660956764],[3,231113,0.0800365636],[3,231157,0.0215815699],[3,231224,0.0127112688],[3,231234,0.0303906244],[3,231244,0.0278331724],[3,231250,0.1019880104],[3,231291,0.1656258076],[3,231305,0.1050398046],[3,231332,0.0149428962],[3,231380,0.0149428962],[3,231390,0.11344744],[3,231451,0.0763717171],[3,231490,0.2400422445],[3,231543,0.0356642763],[3,231662,0.0125930077],[3,231716,0.0676662728],[3,231745,0.0571334139],[3,231852,0.0339895998],[3,231883,0.4298536],[3,231960,0.0622175693],[3,231968,0.5086083736],[3,231973,0.166259062],[3,231985,0.0492134629],[3,232056,0.562148127],[3,232068,0.0306496869],[3,232124,0.2507168023],[3,232170,0.1773173586],[3,232206,0.1272156583],[3,232241,0.0161500024],[3,232260,0.378288881],[3,232276,0.0307796724],[3,232338,0.0343014832],[3,232503,0.0143528753],[3,232524,0.0474630797],[3,232559,0.1036115632],[3,232594,0.2239473336],[3,232619,0.0139185112],[3,232620,0.0449666896],[3,232729,0.0508066343],[3,232743,0.0431522545],[3,232821,0.2289922044],[3,232915,0.0328040714],[3,233050,0.0503063927],[3,233063,0.0127112688],[3,233083,0.6553344361],[3,233105,0.0805475667],[3,233159,0.0399571465],[3,233260,0.0131881033],[3,233389,0.0328586672],[3,233453,0.0261126721],[3,233490,0.1022647841],[3,233492,0.0320895948],[3,233499,0.042428721],[3,233633,0.3972804256],[3,234062,0.0591847206],[3,234091,0.1907856628],[3,234102,0.3794845117],[3,234112,0.4453746306],[3,234115,0.068739012],[3,234144,0.1285409294],[3,234176,0.0146786318],[3,234187,0.0482994973],[3,234231,0.0125930077],[3,234233,0.0288387271],[3,234348,0.0333757144],[3,234431,0.0237915037],[3,234435,0.0313327786],[3,234440,0.0302110724],[3,234488,0.1352740302],[3,234521,0.0127112688],[3,234706,0.0131881033],[3,234716,0.0149428962],[3,234870,0.0480782221],[3,234969,0.0586946164],[3,234971,0.1459055172],[3,235056,0.062431305],[3,235245,0.0605310658],[3,235604,0.0440831129],[3,236090,0.0813761399],[3,236202,0.0229165629],[3,236413,0.0131881033],[3,236440,0.0180960285],[3,236597,0.0888139728],[3,236640,0.0201614336],[3,236791,0.4743334313],[3,236859,0.0182648152],[3,236880,0.0371972485],[3,237004,0.0169996261],[3,237007,0.0152400052],[3,237028,0.0722293232],[3,237032,0.0386472557],[3,237067,0.1069373669],[3,237068,0.0356670868],[3,237076,0.0852976148],[3,237151,0.0157885235],[3,237249,0.058485035],[3,237426,0.0388799257],[3,237430,0.0258725788],[3,237838,0.1754640006],[3,237978,0.1099127034],[3,238340,0.0152400052],[3,238578,0.0348263601],[3,238584,0.0361767423],[3,238586,0.0397315117],[3,238682,0.1292128099],[3,238774,0.0151459214],[3,238848,0.0408010652],[3,238866,0.0189598056],[3,238907,0.0470871672],[3,238969,0.4237506111],[3,239383,0.0152400052],[3,239540,0.0727249288],[3,239813,0.117314872],[3,239818,0.0313653894],[3,239834,0.0327223667],[3,240129,0.1357418383],[3,240268,0.0520785812],[3,240670,0.0990271469],[3,240768,0.0293256944],[3,240831,0.0327223667],[3,240836,0.0159724227],[3,241010,0.0152400052],[3,241442,0.0147618588],[3,241538,0.0698179772],[3,241781,0.1289748044],[3,241950,0.0151459214],[3,241985,0.0556317099],[3,242161,0.100168312],[3,242199,0.0432240092],[3,242200,0.0887484451],[3,242210,0.0951311515],[3,242216,0.0709949539],[3,242227,0.083021683],[3,242228,0.0915345438],[3,242266,0.0510264651],[3,242698,0.0151459214],[3,242954,0.0147618588],[3,243108,0.048187989],[3,243237,0.0167346346],[3,243330,0.1431895473],[3,243466,0.0351493927],[3,243507,0.0272198736],[3,243669,0.0441172836],[3,243685,0.0146432723],[3,243729,0.1163495064],[3,243753,0.1350963506],[3,243935,0.0169996261],[3,244111,0.0338285428],[3,244140,0.0479389251],[3,244218,0.0726076549],[3,244227,0.0173507826],[3,244373,0.0152400052],[3,244399,0.1045943871],[3,244440,0.6431318841],[3,244482,0.0394716715],[3,244525,0.2641344498],[3,244579,0.018160956],[3,244666,0.1053197347],[3,244757,0.3974630019],[3,244850,0.0172261882],[3,244994,0.2396434829],[3,244995,0.417466418],[3,244997,0.248943898],[3,244998,0.1343064483],[3,244999,0.204186613],[3,245000,0.0917793036],[3,245002,0.2665674637],[3,245003,0.1824260336],[3,245005,0.0979431836],[3,245006,0.0716830529],[3,245009,0.1931361281],[3,245622,0.0662094933],[3,245660,0.1408423494],[3,245675,0.0418331598],[3,246075,0.0184991994],[3,246292,0.0152400052],[3,246317,0.0302685666],[3,246319,0.0612940766],[3,246326,0.035035796],[3,246355,0.0665175167],[3,246361,0.0177806358],[3,246375,0.2767361762],[3,246381,0.0338181497],[3,246389,0.0528054854],[3,246437,0.1053964352],[3,246463,0.1036582325],[3,246578,0.532561495],[3,246597,0.0334015333],[3,246613,0.1265720261],[3,246660,0.0413583706],[3,246689,0.0478440501],[3,246732,0.1038004373],[3,246753,0.1920838365],[3,246799,0.2392859611],[3,246894,0.0569186288],[3,247636,0.0405104006],[3,248070,0.0334015333],[3,248072,0.0490213919],[3,248178,0.035813331],[3,248215,0.0671221969],[3,248582,0.0454757772],[3,249168,0.1590253147],[3,249299,0.1329363511],[3,249330,0.0824694883],[3,249527,0.0384352527],[3,249694,0.2009381964],[3,249753,0.0152400052],[3,249773,0.2216937455],[3,249885,0.0159890329],[3,249918,0.385692309],[3,249958,0.0781090952],[3,250014,0.0973283343],[3,250099,0.0356954953],[3,250181,0.0308239364],[3,250214,0.1253172522],[3,250240,0.0452226285],[3,250301,0.0472368572],[3,250341,0.0376934239],[3,250380,0.0419909379],[3,250406,0.0152400052],[3,250722,0.1196045128],[3,250832,0.0152400052],[3,251376,0.1011906792],[3,251516,0.0467495949],[3,251531,0.0201614336],[3,251786,0.0839612495],[3,251817,0.0427288859],[3,252281,0.0680089773],[3,252467,0.0890540378],[3,252498,0.0898606282],[3,252647,0.0312395672],[3,252797,0.4956704409],[3,252814,0.0500316449],[3,252815,0.0668268733],[3,253057,0.0167346346],[3,253061,0.0404377874],[3,253082,0.06806977],[3,253196,0.0152400052],[3,253376,0.1057478273],[3,253553,0.0342949393],[3,253557,0.0418423293],[3,253706,0.1338868827],[3,253768,0.0163043795],[3,253916,0.053845982],[3,254081,0.171343396],[3,254565,0.0164079818],[3,254967,0.0383705601],[3,255223,0.0151459214],[3,255789,0.0151459214],[3,256044,0.1230712284],[3,256216,0.0167346346],[3,256489,0.0159724227],[3,256688,0.2660538507],[3,256712,0.1070344645],[3,256842,0.0169996261],[3,256995,0.0151459214],[3,257026,0.0152400052],[3,257219,0.0232197107],[3,259039,0.0151459214],[3,259129,0.018160956],[3,259284,0.0432601821],[3,259655,0.0343026454],[3,260047,0.0457959984],[3,260076,0.0152400052],[3,260269,0.0152400052],[3,260588,0.0393071382],[3,260688,0.0406773027],[3,260725,0.0172261882],[3,260887,0.0348292936],[3,261153,0.0161406714],[3,261361,0.0138679211],[3,261690,0.0159724227],[3,261698,0.0138679211],[3,261802,0.0154390071],[3,262046,0.0138679211],[3,262291,0.0173048334],[3,262336,0.198065022],[3,262797,0.0540600586],[3,263700,0.0303316947],[3,263718,0.033068846],[3,263914,0.0146691135],[3,264047,0.0394545401],[3,264200,0.0138679211],[3,264321,0.0327105059],[3,264374,0.0388819753],[3,264375,0.0168325201],[3,264469,0.1715917136],[3,264483,0.0367752593],[3,264490,0.1918667397],[3,264495,0.0622200798],[3,264581,0.0307377697],[3,264720,0.0246390566],[3,264939,0.077701926],[3,265003,0.0777863148],[3,265153,0.0297923675],[3,265168,0.1592274297],[3,265175,0.0313765647],[3,265188,0.0323775249],[3,265231,0.0329544586],[3,265376,0.0380510265],[3,265436,0.3472936653],[3,265456,0.141275087],[3,265624,0.0791745586],[3,265680,0.0138679211],[3,265684,0.0169603414],[3,265893,0.0864630252],[3,265939,0.0719241602],[3,265996,0.5800025728],[3,266043,0.1780483102],[3,266080,0.1359347611],[3,266315,0.0164038904],[3,266381,0.0706581011],[3,266427,0.0256704756],[3,266512,0.4394113585],[3,266598,0.024548581],[3,266645,0.0845519858],[3,267032,0.0610443076],[3,267038,0.0790202929],[3,267110,0.2185575643],[3,267279,0.0313575412],[3,267457,0.0873390203],[3,267881,0.1081431747],[3,267915,0.0138679211],[3,268385,0.0723404219],[3,268590,0.0350731887],[3,268719,0.0138679211],[3,268998,0.0313402984],[3,269070,0.6545486939],[3,269357,0.0177444345],[3,269642,0.0157787981],[3,269730,0.0330840774],[3,269972,0.0173539058],[3,270079,0.0347343941],[3,270504,0.0392552633],[3,270535,0.0383554517],[3,270690,0.0315992116],[3,270753,0.0700617618],[3,270860,0.065939974],[3,270888,0.0183212591],[3,270980,0.0387374402],[3,271043,0.0151816353],[3,271107,0.0400051508],[3,271345,0.0517185317],[3,271585,0.0275789836],[3,271608,0.1051877489],[3,271637,0.0165820414],[3,271703,0.1794882216],[3,271801,0.0347713684],[3,271939,0.0185157846],[3,272203,0.0173539058],[3,272339,0.0749872141],[3,272543,0.1369772863],[3,272566,0.0436275829],[3,272569,0.0414903607],[3,272645,0.0998538469],[3,273015,0.0390222245],[3,273053,0.4677705789],[3,273102,0.0356948875],[3,273264,0.0423924798],[3,273416,0.0355726893],[3,273643,0.032935873],[3,273996,0.0157787981],[3,274051,0.0176014466],[3,274502,0.0527812421],[3,274637,0.0496270708],[3,274951,0.0755053524],[3,274970,0.0173063953],[3,275119,0.0306517656],[3,275231,0.0720577836],[3,275278,0.0227672987],[3,275297,0.0187086297],[3,275453,0.0177444345],[3,275751,0.0157787981],[3,275894,0.0319291382],[3,275968,0.0180573662],[3,276161,0.0151816353],[3,276201,0.0476285483],[3,276228,0.2172511282],[3,276631,0.0205172076],[3,276684,0.0349150325],[3,276799,0.2782858947],[3,277082,0.0240422225],[3,277158,0.0185157846],[3,277317,0.0592165835],[3,277582,0.0342376829],[3,277688,0.0289670987],[3,277831,0.0311603553],[3,277936,0.0831358017],[3,277942,0.0160811081],[3,277984,0.0559825587],[3,278200,0.035994707],[3,278323,0.1131275898],[3,278329,0.032935873],[3,278484,0.0190722454],[3,278492,0.3679244759],[3,278537,0.0189058176],[3,278569,0.1186561912],[3,278654,0.0192245084],[3,278911,0.0157787981],[3,279088,0.0165117433],[3,279246,0.1094988461],[3,279296,0.0157787981],[3,279314,0.0906696102],[3,279333,0.0276721815],[3,280047,0.0315130772],[3,280143,0.0298416145],[3,280159,0.0165283654],[3,280286,0.0396422432],[3,280287,0.1271911173],[3,280290,0.0165820414],[3,280373,0.0963536687],[3,280440,0.0165820414],[3,280771,0.1583359287],[3,280875,0.0289670987],[3,280963,0.1095061832],[3,281002,0.0594131184],[3,281124,0.0438825711],[3,281232,0.027465202],[3,281353,0.0956111276],[3,281466,0.0508445233],[3,281580,0.0194792501],[3,281821,0.0157787981],[3,282166,0.0180573662],[3,282280,0.0185497876],[3,282288,0.0501826604],[3,282586,0.0178886869],[3,282928,0.0185157846],[3,283362,0.0906968441],[3,283518,0.0844463289],[3,283537,0.0160811081],[3,283652,0.0151816353],[3,283842,0.0612049181],[3,283914,0.0525297329],[3,283961,0.0332392502],[3,283970,0.0665630269],[3,283989,0.0357488094],[3,284180,0.1368743808],[3,284425,0.033937523],[3,284446,0.0183212591],[3,284570,0.0765852661],[3,284745,0.015499325],[3,284800,0.0505439457],[3,284808,0.134457744],[3,284815,0.0418508266],[3,284915,0.0192245084],[3,284967,0.0166583306],[3,285057,0.0173539058],[3,285216,0.0157787981],[3,285231,0.0754568053],[3,285850,0.0160811081],[3,286054,0.0279290082],[3,286295,0.0369766764],[3,286386,0.1209580954],[3,287167,0.1389390806],[3,287624,0.0185157846],[3,287868,0.0185157846],[3,288031,0.0160811081],[3,288999,0.0196454405],[3,289058,0.0199264625],[3,289286,0.0199264625],[3,289336,0.0822651158],[3,289337,0.0199264625],[3,289347,0.0206801841],[3,290095,0.0222176589],[3,290216,0.0193259901],[3,290546,0.0400909032],[3,290738,0.0356892577],[3,290815,0.0904545727],[3,290894,0.0246911331],[3,290899,0.1003555269],[3,291067,0.0838211209],[3,291068,0.0340205687],[3,292151,0.0452653434],[3,292200,0.020230448],[3,292247,0.0456298128],[3,292570,0.0208108695],[3,292699,0.0303754221],[3,292745,0.046452314],[3,292826,0.0435422851],[3,293060,0.0271908215],[3,293508,0.0208774196],[3,293694,0.0199264625],[3,293817,0.0809078903],[3,293833,0.0222176589],[3,294192,0.1140261004],[3,294637,0.0313819095],[3,295592,0.0300272347],[3,295856,0.0832574239],[3,296675,0.0220480447],[3,296678,0.0199264625],[3,296863,0.0367908029],[3,297016,0.0369208354],[3,298230,0.0199264625],[3,298329,0.020510794],[3,299375,0.0199264625],[3,300378,0.1077974291],[3,300697,0.0226786178],[3,301096,0.0215695364],[3,301101,0.0199264625],[3,301309,0.0199264625],[3,301531,0.1437484959],[3,301532,0.033268935],[3,301576,0.0233912695],[3,301626,0.0620710589],[3,301907,0.0443298188],[3,301949,0.0199264625],[3,302234,0.0220480447],[3,302243,0.0439033386],[3,302527,0.0455407344],[3,302602,0.0206634698],[3,302671,0.0591590927],[3,302672,0.0312675587],[3,302688,0.0596245134],[3,302987,0.0396850832],[3,303467,0.0836727441],[3,303827,0.0341024922],[3,304350,0.0162491078],[3,304995,0.0327340311],[3,305008,0.0337861106],[3,305299,0.0203454897],[3,305505,0.0317278439],[3,305529,0.0498728331],[3,305670,0.039752393],[3,305770,0.0183603225],[3,306018,0.0170528559],[3,306225,0.0162491078],[3,306279,0.0162491078],[3,306341,0.069996424],[3,306352,0.0162491078],[3,306519,0.1436109099],[3,306680,0.0178252053],[3,307286,0.029677697],[3,307304,0.0191269214],[3,307412,0.0472555904],[3,307473,0.0508899851],[3,307552,0.1741701187],[3,307665,0.0664835135],[3,307722,0.1066848478],[3,307830,0.0171954179],[3,307861,0.1625702844],[3,308319,0.0162491078],[3,308368,0.0400473258],[3,308531,0.1489734709],[3,308538,0.0797456954],[3,308555,0.1699137986],[3,308557,0.0358186549],[3,308594,0.6079483738],[3,308607,0.0189878143],[3,308745,0.2122721167],[3,308902,0.0732054409],[3,309440,0.1126993924],[3,309494,0.0547750097],[3,309566,0.3446821224],[3,310008,0.0436630601],[3,310125,0.0842851152],[3,310542,0.1001792808],[3,310704,0.0859448857],[3,311182,0.0156515697],[3,311426,0.0197729447],[3,311473,0.0254024451],[3,312185,0.0321893808],[3,312237,0.0129251583],[3,312396,0.0154328306],[3,312618,0.0196077118],[3,312773,0.0242121624],[3,312853,0.0120685209],[3,312891,0.0340077316],[3,312892,0.0800910727],[3,312949,0.0318322117],[3,312959,0.0204680623],[3,313200,0.0331223155],[3,313306,0.0363145963],[3,313499,0.0144469251],[3,313671,0.0301308277],[3,313716,0.1973983987],[3,313778,0.0292444891],[3,313853,0.0144516739],[3,314011,0.1191084375],[3,314278,0.0892785113],[3,314322,0.0631586989],[3,314409,0.0362950798],[3,314505,0.0126042701],[3,314609,0.0123467123],[3,314671,0.186644636],[3,314752,0.2588346281],[3,314820,0.0678393115],[3,314864,0.0157500048],[3,315129,0.0678251468],[3,315297,0.0123467123],[3,315319,0.0148406913],[3,315467,0.3045933095],[3,315475,0.0348955499],[3,315566,0.0663051976],[3,315748,0.035489468],[3,315849,0.0123467123],[3,315990,0.0200026031],[3,316098,0.0123467123],[3,316222,0.0317332692],[3,316631,0.1858512024],[3,316701,0.035229749],[3,317314,0.1105251539],[3,317315,0.1183265669],[3,317425,0.0489547444],[3,317646,0.1915612269],[3,317791,0.0123467123],[3,318082,0.2018844485],[3,318465,0.0705790463],[3,319028,0.0130928421],[3,319345,0.0540924954],[3,319634,0.0156090564],[3,319691,0.0154328306],[3,319774,0.0263762616],[3,319965,0.045175393],[3,320177,0.0712686882],[3,320190,0.0307788244],[3,320273,0.0146514021],[3,320278,0.0741913437],[3,320440,0.0602372165],[3,320606,0.0348915209],[3,320706,0.4995919368],[3,320826,0.2976156701],[3,320846,0.0151237192],[3,320933,0.0225544412],[3,320945,0.0250697767],[3,321113,0.0129652454],[3,321181,0.1608760621],[3,321237,0.0129652454],[3,321617,0.0129652454],[3,321772,0.0250697767],[3,321805,0.0174559659],[3,321876,0.0494018375],[3,321893,0.0333526063],[3,322045,0.0872634508],[3,322130,0.0165606288],[3,322231,0.0182600861],[3,322232,0.1632883596],[3,322246,0.0241984678],[3,322263,0.0372135136],[3,322424,0.0238613901],[3,322618,0.0313538684],[3,322881,0.1338745068],[3,323206,0.0401949977],[3,323357,0.065356723],[3,324017,0.0986688004],[3,324103,0.2435728772],[3,324300,0.0337208221],[3,324361,0.0129652454],[3,324579,0.3727159016],[3,324699,0.0129652454],[3,324912,0.0272010225],[3,325340,0.0362045157],[3,325512,0.0143784961],[3,326023,0.0137119948],[3,326409,0.2872816952],[3,326481,0.0129652454],[3,326628,0.1271188364],[3,326636,0.0438651981],[3,326760,0.0129652454],[3,326806,0.0136954351],[3,327016,0.0375584021],[3,327116,0.0136954351],[3,327180,0.0448374043],[3,327246,0.0368495472],[3,327266,0.3574271597],[3,327408,0.0129652454],[3,327471,0.0150719548],[3,327554,0.0226213943],[3,327718,0.0667255249],[3,327824,0.0252846951],[3,327914,0.0129652454],[3,327960,0.0354742904],[3,327985,0.0166191445],[3,328419,0.2224580547],[3,328828,0.0137119948],[3,329043,0.0147425518],[3,329146,0.017504284],[3,329160,0.0864054055],[3,329172,0.1560919425],[3,329527,0.0129652454],[3,329587,0.2045610163],[3,329692,0.0615485328],[3,329775,0.0182256314],[3,329849,0.0647093765],[3,329975,0.0734883798],[3,329987,0.0229725812],[3,330034,0.0132230171],[3,330549,0.0180301125],[3,330687,0.029645894],[3,330800,0.1749630296],[3,330846,0.0129652454],[3,330995,0.24026653],[3,331190,0.0166517855],[3,331245,0.0151306858],[3,331253,0.1177054311],[3,331427,0.0169368656],[3,331736,0.0360760021],[3,332429,0.1448600536],[3,332438,0.0739068618],[3,332571,0.0129652454],[3,332795,0.258852275],[3,333042,0.0129652454],[3,333091,0.0129652454],[3,333167,0.0129652454],[3,333201,0.0229424775],[3,333292,0.0228534205],[3,333323,0.0529852794],[3,333537,0.0162641978],[3,333777,0.0159320471],[3,334201,0.0525908163],[3,334478,0.1232775737],[3,334679,0.1130352089],[3,334758,0.0150719548],[3,335374,0.0143784961],[3,335467,0.0162641978],[3,335978,0.0295579542],[3,336374,0.263119444],[3,336577,0.0131251827],[3,336808,0.0149028704],[3,336951,0.0836400562],[3,336972,0.2515412462],[3,336986,0.0303047865],[3,337282,0.2332671721],[3,337358,0.0131251827],[3,337368,0.1024115209],[3,337466,0.0322741414],[3,337650,0.1084774341],[3,337779,0.0159338144],[3,338334,0.0156217679],[3,338349,0.0481824841],[3,338637,0.0306131784],[3,338646,0.0131251827],[3,338839,0.0152275903],[3,339181,0.0131251827],[3,340776,0.0166503768],[3,340932,0.0233663214],[3,341193,0.3995733831],[3,341646,0.0230555194],[3,342272,0.0320695531],[3,342844,0.0417976771],[3,344931,0.0567598645],[3,346342,0.016214526],[3,346345,0.0143220028],[3,346633,0.0355955103],[3,346898,0.027887566],[3,347233,0.0221969394],[3,347896,0.0133830097],[3,347923,0.0133830097],[3,348065,0.038896777],[3,348365,0.0556601536],[3,348617,0.0342420783],[3,348632,0.0935808762],[3,348741,0.0152275903],[3,348880,0.0375369206],[3,348929,0.0190124423],[3,348966,0.0131251827],[3,348993,0.0291739701],[3,349076,0.0159338144],[3,349112,0.0131251827],[3,349137,0.0131251827],[3,349179,0.0156217679],[3,349352,0.0666228071],[3,349398,0.0131251827],[3,349407,0.3030693241],[3,349555,0.0139631469],[3,349577,0.5519813584],[3,349605,0.0820768718],[3,349619,0.0131251827],[3,349641,0.0530070679],[3,349651,0.2086805887],[3,349698,0.0591258584],[3,349868,0.0173576268],[3,349922,0.0438427696],[3,350035,0.0633588393],[3,350126,0.0317208191],[3,350140,0.5990377488],[3,350155,0.0405685346],[3,350207,0.1679664478],[3,350226,0.0302262365],[3,350292,0.0207000161],[3,350327,0.0624468297],[3,350355,0.0391657265],[3,350736,0.3261186315],[3,350906,0.0125301373],[3,350918,0.2442369262],[3,350925,0.1814168994],[3,350940,0.0444198495],[3,350953,0.0166503768],[3,350966,0.2379193398],[3,351188,0.0414439095],[3,351194,0.1316458067],[3,351538,0.0167798655],[3,351819,0.054092475],[3,351851,0.0156217679],[3,352217,0.0323188086],[3,352396,0.018827556],[3,352444,0.0131251827],[3,352471,0.0170976548],[3,352494,0.0312499257],[3,352505,0.0164248427],[3,352573,0.0473278797],[3,352778,0.0142411591],[3,352801,0.0165295624],[3,352840,0.2681089873],[3,352978,0.0206492112],[3,353108,0.0286262781],[3,353179,0.0471517601],[3,353612,0.284925365],[3,353764,0.0160911421],[3,354136,0.0361005687],[3,354617,0.0399107386],[3,354709,0.0194039258],[3,354815,0.0168243932],[3,354937,0.0172115571],[3,354993,0.0184667539],[3,355129,0.0160911421],[3,355348,0.0197931385],[3,355354,0.0549430621],[3,355388,0.0314737113],[3,355464,0.096527104],[3,355536,0.0336641699],[3,355675,0.0704148384],[3,355755,0.1214403747],[3,355756,0.0326475952],[3,355878,0.0154937301],[3,355982,0.0686566128],[3,356016,0.2247232762],[3,356188,0.3015240323],[3,356361,0.120121344],[3,356451,0.0517521072],[3,356755,0.0160911421],[3,357069,0.0450295738],[3,357163,0.0689498312],[3,357198,0.0310988415],[3,357310,0.0193307515],[3,357326,0.0755582364],[3,357385,0.0175765765],[3,357412,0.0189109445],[3,357473,0.4001424052],[3,357554,0.0392618303],[3,357574,0.0963850789],[3,357719,0.0163499945],[3,357849,0.0516548783],[3,358035,0.0359410376],[3,358151,0.0812247237],[3,358153,0.0160911421],[3,358865,0.0562554399],[3,358869,0.3902144977],[3,359154,0.3385544215],[3,359363,0.6987818776],[3,359731,0.0169324392],[3,360077,0.0160911421],[3,361096,0.2441673504],[3,361177,0.1182506675],[3,361437,0.0904238866],[3,361643,0.0160911421],[3,361727,0.0450958834],[3,361905,0.0184667539],[3,361950,0.0644162209],[3,362005,0.039342842],[3,362017,0.0182019115],[3,362024,0.0994580894],[3,362249,0.0637993235],[3,362478,0.1162985313],[3,362734,0.4114065866],[3,362751,0.0253736626],[3,362771,0.0470881936],[3,362915,0.0787906514],[3,363038,0.1379905851],[3,363114,0.058302556],[3,363172,0.0194039258],[3,363185,0.0844741599],[3,363365,0.1341130682],[3,363839,0.059034228],[3,364405,0.0548958879],[3,364428,0.0196303568],[3,364537,0.0546398943],[3,364638,0.0883942891],[3,364655,0.0593844982],[3,364955,0.1005229005],[3,365196,0.0241938782],[3,365816,0.0169324392],[3,366080,0.0871859458],[3,366261,0.0182019115],[3,366525,0.0416634042],[3,367251,0.104135255],[3,367276,0.0433145515],[3,367288,0.0414793361],[3,367516,0.0313336748],[3,367556,0.2391832847],[3,370025,0.0829281503],[3,370104,0.1505018929],[3,370208,0.0272960974],[3,370456,0.0697738247],[3,370468,0.1542889756],[3,370813,0.0160911421],[3,370849,0.0169324392],[3,370853,0.0365955326],[3,370940,0.1797506701],[3,371014,0.042973744],[3,371040,0.0451393847],[3,371538,0.0417580611],[3,371904,0.0670747808],[3,371946,0.064607778],[3,371984,0.2394612293],[3,372053,0.0260826378],[3,372082,0.1076543229],[3,372104,0.0326335331],[3,372133,0.1128588774],[3,372243,0.0715037196],[3,372282,0.0788810601],[3,372830,0.0160911421],[3,373750,0.0160911421],[3,374115,0.0168947207],[3,374561,0.0575147643],[3,374652,0.0186346642],[3,374684,0.1578422919],[3,375552,0.0189109445],[3,375614,0.0406098454],[3,375739,0.1577849973],[3,375779,0.0905502261],[3,375873,0.0151750163],[3,375883,0.0159239789],[3,376038,0.0151750163],[3,376206,0.0173398875],[3,376526,0.2480736441],[3,376605,0.0762937884],[3,376613,0.2732371134],[3,376658,0.0413445036],[3,376786,0.0732159464],[3,377770,0.0770498248],[3,378130,0.0691952404],[3,378245,0.1645505118],[3,378317,0.1277677106],[3,378530,0.0852050607],[3,379125,0.0433117339],[3,379433,0.0257765622],[3,379452,0.2314329542],[3,379764,0.0360263563],[3,379832,0.0638224858],[3,379926,0.0608793347],[3,380097,0.076056604],[3,381107,0.0151750163],[3,381113,0.0175477212],[3,381168,0.0500038637],[3,381237,0.0151750163],[3,381351,0.0184106616],[3,381906,0.0267686425],[3,382670,0.019727508],[3,382712,0.0145783353],[3,382781,0.0363743442],[3,383255,0.0294695478],[3,383842,0.0184106616],[3,384625,0.0151750163],[3,384714,0.0327209676],[3,384730,0.0172832029],[3,385577,0.0175477212],[3,385725,0.1167680327],[3,386541,0.0660468482],[3,387059,0.0159073702],[3,387213,0.0743268689],[3,387729,0.015433552],[3,388259,0.0605482854],[3,389360,0.0145783353],[3,389464,0.0843065324],[3,390140,0.0177154261],[3,391143,0.0388723294],[3,391219,0.0248309402],[3,391350,0.5759512378],[3,391401,0.0187932689],[3,391504,0.0708400186],[3,391518,0.0187099003],[3,391526,0.0526260034],[3,391527,0.0365048752],[3,391544,0.1628498582],[3,391553,0.0170420595],[3,391645,0.0160152839],[3,391706,0.0181448064],[3,392122,0.0480925488],[3,392194,0.0419211775],[3,392365,0.1925469379],[3,392379,0.057826906],[3,392548,0.1877432084],[3,392813,0.0242717092],[3,392884,0.3059886436],[3,392946,0.1119315136],[3,393269,0.536210919],[3,393364,0.1357809217],[3,393370,0.0188724829],[3,393421,0.2586540291],[3,393477,0.0431891894],[3,393621,0.0179913683],[3,393717,0.3386395864],[3,393753,0.0151750163],[3,393783,0.023957918],[3,393864,0.0151750163],[3,393930,0.0233078811],[3,394245,0.1279715225],[3,394256,0.0263662608],[3,394274,0.0390887953],[3,394279,0.0268210966],[3,394312,0.0811115431],[3,394437,0.0151750163],[3,394573,0.2753192038],[3,394601,0.1222021727],[3,394760,0.5711220571],[3,395189,0.1745510877],[3,395591,0.0280361152],[3,395674,0.0359190135],[3,396180,0.0848371717],[3,396543,0.2983384047],[3,396671,0.0608993172],[3,396794,0.0164943547],[3,396843,0.0151750163],[3,396893,0.0485410698],[3,396903,0.0353299649],[3,397025,0.0151750163],[3,397328,0.0573776711],[3,398121,0.0173468748],[3,398571,0.0743840136],[3,398601,0.0145783353],[3,398666,0.6056001272],[3,398742,0.0151750163],[3,398751,0.0297608967],[3,398840,0.1886323903],[3,398854,0.0504154749],[3,398868,0.0428028039],[3,398985,0.3048364225],[3,399111,0.0187099003],[3,399637,0.0170420595],[3,399764,0.0995264466],[3,399905,0.0159073702],[3,399913,0.1166941789],[3,399928,0.0369911303],[3,400024,0.1363283272],[3,400186,0.0175477212],[3,401331,0.0182728515],[3,401492,0.0151750163],[3,401500,0.017678464],[3,401674,0.019727508],[3,401863,0.4012463973],[3,401890,0.4706926381],[3,402143,0.1454689048],[3,402274,0.0322282364],[3,402282,0.0284820142],[3,402341,0.2817810963],[3,402399,0.0498666614],[3,403054,0.0547734628],[3,403454,0.0151750163],[3,403520,0.0413062055],[3,403573,0.4603670822],[3,403834,0.0145783353],[3,403919,0.0170420595],[3,404047,0.0226970668],[3,404303,0.0309282341],[3,404606,0.0326981227],[3,404750,0.0183115984],[3,404759,0.0145783353],[3,404825,0.017678464],[3,404859,0.0578803859],[3,404981,0.528697876],[3,405006,0.0662254436],[3,405110,0.1670329634],[3,405426,0.1844147033],[3,405566,0.0759970347],[3,405783,0.0403652855],[3,405793,0.150456312],[3,405846,0.1297218134],[3,405975,0.1710119968],[3,406229,0.0733331682],[3,406496,0.0377629831],[3,406499,0.1094357267],[3,406519,0.0173468748],[3,406714,0.1126315078],[3,407246,0.0145783353],[3,407281,0.0796458292],[3,407293,0.1402005277],[3,407359,0.0904720655],[3,407360,0.0188724829],[3,407450,0.038891923],[3,407454,0.0759115236],[3,407477,0.0535066008],[3,407560,0.0782539826],[3,407950,0.0151750163],[3,408176,0.0556637814],[3,408239,0.0151750163],[3,408363,0.056864556],[3,408580,0.0457200161],[3,408599,0.247507416],[3,409063,0.0309465186],[3,409272,0.3044210166],[3,410563,0.0184837463],[3,410588,0.0166586331],[3,410788,0.0159776116],[3,410854,0.0151750163],[3,411416,0.0889266345],[3,411887,0.0207487579],[3,411961,0.1423208023],[3,411986,0.1196488242],[3,412213,0.0468411406],[3,412265,0.0264249613],[3,412560,0.1420543626],[3,412630,0.0184106616],[3,412694,0.0495987001],[3,413049,0.1625870391],[3,413161,0.0151750163],[3,414205,0.0440286473],[3,414319,0.0151750163],[3,414629,0.1021601645],[3,414752,0.0242638632],[3,414840,0.0341901292],[3,415005,0.027598034],[3,415175,0.2368463047],[3,415190,0.0631470967],[3,415229,0.0258776322],[3,415267,0.0237290252],[3,415308,0.0578803859],[3,415320,0.0429259502],[3,416288,0.0282905507],[3,416489,0.0151750163],[3,416715,0.0555482075],[3,416782,0.07484264],[3,416821,0.0151750163],[3,416919,0.0151750163],[3,417210,0.0290682501],[3,417770,0.0362259778],[3,417804,0.0469811788],[3,418057,0.0187099003],[3,418313,0.0325747852],[3,418526,0.161751178],[3,418849,0.0257765622],[3,418858,0.2501038989],[3,419168,0.3545001335],[3,419180,0.1505871758],[3,419263,0.0303452997],[3,420194,0.1605378811],[3,420430,0.0151750163],[3,420431,0.0151750163],[3,420645,0.2144234701],[3,420853,0.0169575904],[3,420859,0.0451651805],[3,420894,0.0207118406],[3,421037,0.1433879825],[3,421105,0.0160152839],[3,421193,0.0184837463],[3,421772,0.0706737416],[3,421815,0.0418937717],[3,421844,0.0786632446],[3,422029,0.0452672001],[3,422036,0.0388217952],[3,422071,0.0292445694],[3,422608,0.1076741878],[3,422915,0.0160152839],[3,423015,0.0268005917],[3,423303,0.0423888157],[3,423514,0.1018606729],[3,423896,0.0170420595],[3,423948,0.0191584079],[3,424114,0.0151750163],[3,424498,0.017678464],[3,424542,0.0428312878],[3,425352,0.0389591552],[3,425791,0.0297922215],[3,426175,0.0165924557],[3,426293,0.0160152839],[3,426522,0.0303988977],[3,427153,0.1280793082],[3,427533,0.0184837463],[3,427564,0.0263662608],[3,427961,0.0151750163],[3,428578,0.0151750163],[3,429156,0.0151750163],[3,429717,0.0145783353],[3,430804,0.0292178612],[3,430853,0.0198704631],[3,430882,0.0715845484],[3,431008,0.0170470604],[3,431120,0.0490735942],[3,431448,0.0321234107],[3,432394,0.0195567755],[3,432747,0.0329469731],[3,433123,0.1023614654],[3,434025,0.0423267372],[3,434206,0.0194257054],[3,434700,0.0198704631],[3,434832,0.0177978981],[3,434918,0.2125338101],[3,434927,0.0687503474],[3,435140,0.1108150875],[3,435265,0.1006353313],[3,435266,0.1618700034],[3,435310,0.0170470604],[3,435698,0.0361645613],[3,435752,0.0546605067],[3,435826,0.0445170371],[3,435935,0.0730242159],[3,436126,0.1152062352],[3,436212,0.5705299285],[3,436307,0.3863015479],[3,436379,0.0376645701],[3,436516,0.2089173657],[3,436725,0.1255464315],[3,436819,0.1409579922],[3,436823,0.0194257054],[3,437042,0.3807786701],[3,437267,0.0215623665],[3,437285,0.0170470604],[3,437371,0.0195567755],[3,437457,0.0403790042],[3,437575,0.0674490283],[3,437607,0.0181689059],[3,437662,0.0931043888],[3,438251,0.0648274308],[3,438326,0.0309357119],[3,438429,0.2434512559],[3,438474,0.1851853433],[3,438492,0.0511307107],[3,439097,0.0252002857],[3,439261,0.0452493211],[3,439578,0.0185343914],[3,439752,0.0495022368],[3,440014,0.0191605248],[3,440196,0.0187424484],[3,440309,0.0810822756],[3,440315,0.0231152342],[3,440773,0.0198593199],[3,440865,0.0198593199],[3,440883,0.0198593199],[3,440912,0.0486922792],[3,440936,0.0215063556],[3,440996,0.0455907928],[3,441010,0.0253307631],[3,441047,0.3196291676],[3,441231,0.3124303348],[3,441406,0.0685323208],[3,441576,0.0234163474],[3,441737,0.1743224205],[3,441751,0.1443969609],[3,441816,0.0198593199],[3,441822,0.0407578272],[3,442015,0.3809188711],[3,442033,0.0198593199],[3,442134,0.2126966068],[3,442146,0.0358030833],[3,442252,0.1201385583],[3,442384,0.0618606575],[3,443027,0.1930826769],[3,443532,0.0398189618],[3,443616,0.0387643384],[3,443675,0.1031378318],[3,444120,0.1222336811],[3,444318,0.1845035962],[3,445507,0.0369068655],[3,445749,0.2170310251],[3,445772,0.0253307631],[3,445885,0.0620848239],[3,445911,0.0198593199],[3,446030,0.0214991811],[3,446266,0.0542890734],[3,446686,0.1192506789],[3,446815,0.0841049653],[3,446912,0.0477237473],[3,446973,0.0725729011],[3,447062,0.0311206696],[3,447157,0.036674571],[3,447273,0.1606397393],[3,447425,0.0219807128],[3,447544,0.0642362914],[3,447569,0.4000874131],[3,447646,0.0231152342],[3,447720,0.0315783547],[3,447758,0.039663286],[3,447841,0.1093502101],[3,448002,0.0206129742],[3,448168,0.0198593199],[3,448449,0.1560623058],[3,448931,0.0214991811],[3,449054,0.622593067],[3,449069,0.0926479385],[3,449314,0.0699476967],[3,449716,0.1937548524],[3,450067,0.0249329941],[3,450331,0.0648070048],[3,450562,0.0741397322],[3,450914,0.0382108346],[3,451226,0.0703466769],[3,451464,0.0212856385],[3,451933,0.0296668076],[3,451960,0.4824075032],[3,452023,0.0401696808],[3,452145,0.0290005776],[3,452187,0.0669726185],[3,452394,0.0932762746],[3,452530,0.0695223791],[3,452575,0.0340545899],[3,452645,0.3381579963],[3,452832,0.0480022113],[3,452885,0.1353669396],[3,453112,0.0612846157],[3,453165,0.2399888056],[3,453223,0.0368881976],[3,453442,0.0779764159],[3,453508,0.1655094165],[3,453986,0.0234280185],[3,454065,0.0245741086],[3,454115,0.203298278],[3,454201,0.0920618873],[3,454223,0.135111183],[3,454818,0.2553228856],[3,454890,0.1451217371],[3,454956,0.3000979918],[3,455064,0.0767531547],[3,455410,0.0739084973],[3,455452,0.0205666546],[3,456297,0.0367743953],[3,456460,0.0624674028],[3,456505,0.0306058896],[3,456580,0.0309099875],[3,456639,0.0492840724],[3,456849,0.060756517],[3,457408,0.0356200803],[3,457431,0.1355559952],[3,457635,0.0459264807],[3,457643,0.0238256295],[3,457978,0.0222080573],[3,457980,0.0238256295],[3,458198,0.0522991353],[3,458282,0.0434209789],[3,458392,0.032388107],[3,458634,0.0345600615],[3,458695,0.0431777078],[3,458741,0.0245606105],[3,458865,0.0235578575],[3,459033,0.0275341482],[3,459080,0.0445597906],[3,459324,0.0607151129],[3,459656,0.0205666546],[3,459667,0.0247890382],[3,459686,0.5362414176],[3,459860,0.0226900416],[3,459887,0.3789090361],[3,459975,0.1332827494],[3,459979,0.0244733002],[3,460281,0.0821392691],[3,460467,0.0407218164],[3,461136,0.0875805396],[3,461428,0.0598251377],[3,461621,0.0403229698],[3,462945,0.0834982571],[3,463548,0.0241270258],[3,463925,0.0205666546],[3,464820,0.0205666546],[3,464952,0.0205666546],[3,465900,0.0216937671],[3,465952,0.0464606971],[3,466169,0.0533207241],[3,466362,0.0222080573],[3,466619,0.0413297603],[3,466855,0.0205666546],[3,467052,0.0312446398],[3,467179,0.0241387079],[3,467235,0.054870582],[3,467257,0.0205666546],[3,467784,0.078441651],[3,468063,0.0247890382],[3,468186,0.0247890382],[3,469235,0.0205666546],[3,469446,0.021994314],[3,469773,0.0536097112],[3,469776,0.0431989013],[3,469989,0.0221803453],[3,470587,0.0382879112],[3,473043,0.0184628953],[3,473146,0.021934511],[3,473328,0.0210379804],[3,474030,0.022866975],[3,474031,0.0233820652],[3,474307,0.2518451474],[3,474336,0.0585251968],[3,474538,0.022784559],[3,474706,0.0258024878],[3,474726,0.0364316866],[3,475233,0.1704006668],[3,475333,0.0476556974],[3,475609,0.0258024878],[3,475641,0.025305312],[3,475897,0.0204786573],[3,475954,0.0357474921],[3,476122,0.0876174265],[3,476152,0.0660978223],[3,476268,0.0810655845],[3,476318,0.2494252266],[3,476364,0.0774198861],[3,476454,0.0958456326],[3,476514,0.051409865],[3,476777,0.1429623927],[3,476904,0.0250767286],[3,476963,0.0333163278],[3,477089,0.279261837],[3,477099,0.2225749359],[3,477140,0.2379671882],[3,477234,0.165453198],[3,477282,0.4215408848],[3,477283,0.0925067692],[3,477406,0.1053581366],[3,477461,0.1590449548],[3,477465,0.0651601614],[3,477677,0.0808721625],[3,477722,0.0232048847],[3,477793,0.1136436018],[3,478027,0.0580840929],[3,478053,0.0314608755],[3,478135,0.0735836116],[3,478563,0.0278770825],[3,478697,0.0383653948],[3,479020,0.0588958502],[3,479327,0.0289375574],[3,479428,0.0752978175],[3,479482,0.0385569975],[3,479897,0.0726288686],[3,480054,0.0259363187],[3,480066,0.144903671],[3,480088,0.0456492674],[3,480107,0.0356414993],[3,480192,0.0271568337],[3,480443,0.0250155829],[3,480842,0.0272892513],[3,481661,0.0250155829],[3,481709,0.0259363187],[3,481736,0.0404944466],[3,481836,0.0256047665],[3,482154,0.0252555744],[3,482280,0.0508909042],[3,482384,0.0408333781],[3,482505,0.048649409],[3,482669,0.0291856563],[3,482775,0.0250777583],[3,482800,0.0248010445],[3,483252,0.132138374],[3,483253,0.1508722804],[3,483297,0.2210692486],[3,483429,0.3596676722],[3,483473,0.027671051],[3,483525,0.1440213825],[3,483599,0.0263704036],[3,483663,0.0281048489],[3,483719,0.0616135156],[3,483818,0.0362963301],[3,483915,0.1317939129],[3,483928,0.0410831738],[3,483944,0.2358093074],[3,484026,0.1134037323],[3,484040,0.1378601106],[3,484158,0.0802360564],[3,484165,0.1097735306],[3,484396,0.2897614554],[3,484758,0.0217785889],[3,484788,0.1659327667],[3,485028,0.0472178221],[3,485404,0.1895336439],[3,485425,0.0357116184],[3,485681,0.188643584],[3,485710,0.0211766386],[3,485825,0.0321690355],[3,485832,0.1835512107],[3,485869,0.0239625779],[3,485906,0.0365248775],[3,485913,0.2069177126],[3,486048,0.0465732741],[3,486503,0.0211766386],[3,486505,0.0239053927],[3,486510,0.679202094],[3,487010,0.049993263],[3,487826,0.0544899567],[3,488465,0.0846050127],[3,488471,0.0232085456],[3,488505,0.364932094],[3,488509,0.5014359221],[3,488629,0.0260077668],[3,488722,0.0432539538],[3,488737,0.4050232203],[3,488764,0.0738767328],[3,488820,0.0234226329],[3,488827,0.169834213],[3,488893,0.0431742246],[3,489228,0.0474895749],[3,489437,0.0563481826],[3,489549,0.1102488544],[3,489571,0.0248957504],[3,489731,0.0879396919],[3,490106,0.26717802],[3,490248,0.0335275149],[3,490289,0.0432340644],[3,490324,0.4842765645],[3,490480,0.0248957504],[3,490549,0.0928803738],[3,490641,0.0284765861],[3,490864,0.1622225229],[3,491061,0.1116758835],[3,491150,0.0289126629],[3,491186,0.2027890706],[3,491257,0.0540850159],[3,492146,0.0248957504],[3,492364,0.2366227785],[3,493060,0.3752931305],[3,493636,0.397377406],[3,493640,0.4241492526],[3,493643,0.0248957504],[3,493896,0.0362456543],[3,494177,0.1284827512],[3,494519,0.207752163],[3,494767,0.1498723488],[3,494924,0.0414653396],[3,494966,0.0270433584],[3,495091,0.3842163185],[3,495114,0.1442396589],[3,495203,0.2484979979],[3,495252,0.7022023402],[3,495255,0.6536889034],[3,495438,0.1205646256],[3,495509,0.0303387837],[3,495797,0.1148351276],[3,495894,0.0270887638],[3,496013,0.0242913128],[3,496219,0.1110373817],[3,496414,0.0248957504],[3,496450,0.1231391466],[3,496962,0.0291424036],[3,497004,0.0242913128],[3,497099,0.0270887638],[3,497295,0.5533714096],[3,497559,0.1133497743],[3,497870,0.0659588694],[3,498047,0.0669044866],[3,498527,0.7938534981],[3,498945,0.0288248507],[3,498949,0.0304236142],[3,499816,0.0264328074],[3,499864,0.0767478321],[3,500093,0.0264328074],[3,500476,0.0264511431],[3,501154,0.0264733435],[3,501192,0.0345732847],[3,501402,0.1166301708],[3,501936,0.0279428517],[3,502633,0.0262333921],[3,502952,0.1035078555],[3,503114,0.0295758582],[3,503415,0.0352159096],[3,503500,0.0256857173],[3,503539,0.0393777002],[3,503543,0.0277274844],[3,503561,0.0250806494],[3,503585,0.1708349277],[3,503839,0.2434729303],[3,505056,0.0759301392],[3,505417,0.0256857173],[3,505707,0.0440350212],[3,505944,0.0367504637],[3,506852,0.0256857173],[3,507035,0.1130846451],[3,507139,0.0268795833],[3,507158,0.0250806494],[3,507262,0.0283926465],[3,507503,0.0393822975],[3,507526,0.0262333921],[3,507679,0.043080917],[3,508745,0.0725676013],[3,509210,0.0412665266],[3,510461,0.0284114447],[3,511736,0.0368539724],[3,511795,0.0443295508],[3,512065,0.040740267],[3,512680,0.0389981602],[3,512896,0.0229826328],[3,512952,0.023314238],[3,513028,0.0275213123],[3,513103,0.0229826328],[3,513502,0.1249751801],[3,513902,0.0229826328],[3,514470,0.0340130347],[3,516878,0.0261773342],[3,516909,0.0247153707],[3,517487,0.0330121455],[3,517974,0.056117992],[3,518069,0.0229826328],[3,518353,0.0256799126],[3,518555,0.0229826328],[3,518764,0.0389306381],[3,519154,0.0229826328],[3,519461,0.037876424],[3,519622,0.0497786704],[3,519738,0.0241163823],[3,519784,0.0249925243],[3,520543,0.0229826328],[3,520567,0.0228117713],[3,520697,0.0229826328],[3,520730,0.0312825709],[3,520814,0.7428480816],[3,520829,0.0694624661],[3,520987,0.0250171215],[3,521017,0.0284282515],[3,521059,0.2196612158],[3,521182,0.035217762],[3,521225,0.0488010072],[3,521530,0.2196995223],[3,521659,0.0516116927],[3,521960,0.459749023],[3,522084,0.1270302307],[3,522107,0.0229826328],[3,522210,0.7285416165],[3,522219,0.0228117713],[3,522249,0.2240467247],[3,522283,0.0841818638],[3,522390,0.0599230273],[3,522399,0.0456565136],[3,522553,0.0228819011],[3,522896,0.1345005287],[3,523032,0.1563067473],[3,523061,0.2009808208],[3,523136,0.1515006013],[3,523726,0.0417963226],[3,524284,0.032997372],[3,524522,0.0236835867],[3,524914,0.0229826328],[3,525550,0.0842516059],[3,525584,0.1035201627],[3,525669,0.1014099379],[3,525696,0.1621761154],[3,525778,0.038274561],[3,525801,0.1026875771],[3,525874,0.0735920721],[3,525976,0.702443505],[3,526042,0.0720444942],[3,526214,0.0448917942],[3,526225,0.1619979256],[3,526372,0.2873664389],[3,526558,0.0336286108],[3,526693,0.0223797218],[3,527125,0.0428845381],[3,527185,0.0229826328],[3,527194,0.0732812979],[3,528037,0.0252369628],[3,528044,0.3267091819],[3,528308,0.0604995108],[3,528498,0.0364099097],[3,529402,0.0411043759],[3,529405,0.0324416036],[3,529406,0.0384712225],[3,529408,0.1718688884],[3,530065,0.0361765526],[3,530206,0.047150007],[3,530772,0.1163693616],[3,531050,0.2850174674],[3,531283,0.4271955892],[3,531332,0.032984635],[3,532392,0.0278860584],[3,532750,0.0303936629],[3,533125,0.2046459464],[3,533126,0.0746198845],[3,533150,0.0870251181],[3,533965,0.4489311699],[3,533988,0.026768426],[3,534173,0.0996670579],[3,534174,0.0764747305],[3,534452,0.0402196212],[3,534843,0.0597066483],[3,535204,0.0388280474],[3,535544,0.3129126821],[3,535697,0.106018842],[3,535981,0.0275146573],[3,536597,0.1134638866],[3,536639,0.0728585213],[3,536930,0.1752227137],[3,536952,0.0377717693],[3,536953,0.0383914496],[3,537091,0.0873917768],[3,537118,0.0559969884],[3,537377,0.0295756114],[3,537421,0.5311991558],[3,537612,0.0542614707],[3,537655,0.102958323],[3,537717,0.142402699],[3,537727,0.10722721],[3,537783,0.027295102],[3,537870,0.6322179878],[3,538353,0.0437822053],[3,538824,0.0405624545],[3,540114,0.1532002771],[3,540256,0.054098197],[3,540392,0.0608275232],[3,540562,0.0389118747],[3,540563,0.028321259],[3,541060,0.1959793594],[3,541537,0.0741596808],[3,542131,0.0436038571],[3,542460,0.0895279618],[3,542914,0.0278860584],[3,543586,0.027295102],[3,543862,0.0476894593],[3,544739,0.0252369628],[3,544769,0.0273735167],[3,544837,0.036087437],[3,545787,0.030187441],[3,550447,0.0367574978],[3,550843,0.1584619479],[3,550849,0.0275146573],[3,551754,0.0273735167],[3,559340,0.0478391173],[3,559809,0.0252369628],[3,560047,0.0252369628],[3,560741,0.0264301223],[3,560758,0.0304339366],[3,562156,0.028321259],[3,562850,0.0252369628],[3,563129,0.0534441968],[3,563134,0.0372682981],[3,564534,0.0260503577],[3,565142,0.026768426],[3,565547,0.0250655915],[3,565634,0.0715742945],[3,565668,0.0610799262],[3,565758,0.0764023051],[3,566025,0.0252369628],[3,566505,0.0533289289],[3,566522,0.1837847444],[3,566665,0.0545803009],[3,566733,0.0873446782],[3,566775,0.2761067226],[3,566871,0.056761706],[3,566924,0.0706980912],[3,566928,0.1231373013],[3,566962,0.0295756114],[3,567215,0.0405006601],[3,567557,0.028321259],[3,567625,0.0406530469],[3,567639,0.0569478768],[3,567958,0.2456497491],[3,568205,0.047129922],[3,568243,0.0252369628],[3,568291,0.1330587471],[3,568585,0.0252369628],[3,568831,0.0625615637],[3,568910,0.0703498652],[3,569471,0.0292556848],[3,569853,0.0264301223],[3,569900,0.0473668761],[3,570027,0.5039808999],[3,570510,0.0697487926],[3,570635,0.0425969614],[3,570654,0.0403341615],[3,570724,0.1395442602],[3,571131,0.2811838165],[3,571460,0.43983536],[3,571487,0.0789673449],[3,571993,0.030187441],[3,572000,0.037443181],[3,572078,0.0495570411],[3,572204,0.0311561329],[3,572597,0.0319808348],[3,573073,0.0252369628],[3,573175,0.0334391104],[3,573497,0.0277299644],[3,574436,0.0522343244],[3,575055,0.2786812568],[3,575723,0.0486419094],[3,575915,0.0252369628],[3,576356,0.048252522],[3,576592,0.1352040355],[3,576729,0.0867543424],[3,577101,0.0446196299],[3,577232,0.2641454716],[3,577489,0.0739777285],[3,577585,0.0275146573],[3,577768,0.1188820773],[3,577904,0.1570685239],[3,577921,0.1331807148],[3,577998,0.1179061553],[3,578016,0.0402208192],[3,578030,0.080639083],[3,578049,0.2142453785],[3,578234,0.1226842716],[3,578274,0.1280441764],[3,578286,0.1133553857],[3,578319,0.0797453189],[3,578334,0.0842415722],[3,578355,0.0804460632],[3,578403,0.1244660934],[3,578445,0.1701371736],[3,578529,0.1278279578],[3,578617,0.1139444519],[3,578658,0.2371820538],[3,578737,0.1967994598],[3,578768,0.1075935461],[3,578789,0.1456718038],[3,578869,0.2297982194],[3,578874,0.1261947077],[3,578922,0.1115350782],[3,579016,0.102466186],[3,579138,0.0395799192],[3,579148,0.423905008],[3,579190,0.0735296993],[3,579489,0.0335796153],[3,580037,0.4678898779],[3,580120,0.0524833436],[3,580147,0.1768604151],[3,580275,0.2043101088],[3,580492,0.4328724846],[3,580503,0.2145790792],[3,580522,0.0491746858],[3,581250,0.1732344743],[3,582118,0.0298934773],[3,582684,0.0389748166],[3,582817,0.1887805684],[3,583023,0.0706647219],[3,583300,0.0304339366],[3,583301,0.0252369628],[3,583744,0.270493073],[3,583942,0.0339087127],[3,584196,0.2670983476],[3,584549,0.044472284],[3,584760,0.1246057377],[3,584994,0.0361435138],[3,585001,0.2348650706],[3,585008,0.1770275336],[3,585063,0.0284680554],[3,585075,0.0308440502],[3,585255,0.0709209586],[3,585257,0.0871952867],[3,585263,0.1072942064],[3,585339,0.1469231888],[3,585358,0.0855087193],[3,585390,0.1075639369],[3,585764,0.1078012629],[3,585786,0.1195238786],[3,585825,0.1344691261],[3,585963,0.1982375882],[3,585994,0.0871173131],[3,586034,0.0842040377],[3,586060,0.1205116414],[3,586094,0.1646068277],[3,586187,0.0343828536],[3,586475,0.0305408592],[3,586486,0.041275317],[3,586721,0.0332880992],[3,587406,0.0565634257],[3,587583,0.0346884564],[3,587775,0.0501887797],[3,587901,0.0286401963],[3,587982,0.0456637469],[3,587987,0.0821659814],[3,588669,0.0653850612],[3,588854,0.0284680554],[3,588958,0.0311443932],[3,589404,0.0298387139],[3,589837,0.1924726861],[3,591534,0.0810443068],[3,591847,0.0562579245],[3,592045,0.0343828536],[3,592085,0.1097552801],[3,592143,0.1148635803],[3,592240,0.0862987119],[3,592318,0.1211936465],[3,592620,0.1078012629],[3,592636,0.0890162494],[3,592660,0.0842040377],[3,592689,0.1170084236],[3,592690,0.0901148446],[3,592749,0.0611805298],[3,592750,0.0819595258],[3,592753,0.0379326885],[3,592782,0.0951887536],[3,592824,0.1197927262],[3,592869,0.091412773],[3,592910,0.1059318796],[3,593050,0.0448738354],[3,593211,0.0286952807],[3,593232,0.4854247209],[3,593256,0.0537978575],[3,593305,0.1000678892],[3,593333,0.0305408592],[3,593483,0.4387399043],[3,593572,0.0492802139],[3,593680,0.0422533342],[3,593698,0.0350892114],[3,593782,0.078224238],[3,594081,0.0326599685],[3,594235,0.0716070002],[3,594501,0.0303293576],[3,594667,0.0600036878],[3,594668,0.0543288499],[3,594873,0.0663732014],[3,595263,0.1187273876],[3,595286,0.0912753358],[3,595287,0.0303845643],[3,595729,0.1145332291],[3,596062,0.0343213095],[3,596105,0.0344596484],[3,596972,0.0298898682],[3,597937,0.0575381491],[3,598290,0.0298898682],[3,598476,0.0363245606],[3,598479,0.0326823361],[3,599551,0.0606142808],[3,599629,0.0316471503],[3,599662,0.163980382],[3,599719,0.1033801133],[3,600112,0.0349177301],[3,600125,0.0511697335],[3,600278,0.4290724395],[3,600333,0.0413507675],[3,600794,0.0292814456],[3,600820,0.0822998011],[3,600860,0.1708582208],[3,600991,0.0316471503],[3,601251,0.1076760402],[3,601662,0.0564953241],[3,601858,0.0376574325],[3,602069,0.0680907383],[3,602205,0.0387251684],[3,602223,0.042053033],[3,602232,0.0560100048],[3,602473,0.0325552283],[3,602684,0.0922385387],[3,603190,0.1545570547],[3,603369,0.0529481536],[3,603474,0.0409923029],[3,603793,0.0464368517],[3,603811,0.0899585643],[3,603836,0.0680051558],[3,604644,0.0456105672],[3,605366,0.0314386576],[3,605702,0.0314386576],[3,606560,0.0501805037],[3,606592,0.0306366334],[3,607136,0.0420896069],[3,607761,0.1275514099],[3,608813,0.0281983139],[3,608856,0.027971237],[3,609676,0.0564329623],[3,610213,0.3794278804],[3,611027,0.0298485474],[3,612833,0.0642759083],[3,612845,0.0297698773],[3,613081,0.1318898693],[3,613507,0.0336434825],[3,615692,0.0297698773],[3,616051,0.0686079072],[3,616134,0.028063016],[3,616454,0.0779813456],[3,616554,0.0333312525],[3,616862,0.0614264491],[3,617760,0.0302880131],[3,618027,0.0757674639],[3,618341,0.0312393899],[3,618567,0.1420052666],[3,619099,0.0292493869],[3,619319,0.0649774741],[3,619729,0.0767380071],[3,619806,0.0277050353],[3,619854,0.0547527207],[3,620315,0.1366524979],[3,620552,0.5166862339],[3,620553,0.3192584605],[3,620689,0.0320578136],[3,620838,0.0394148725],[3,620893,0.0277050353],[3,620900,0.0277050353],[3,620981,0.0429850959],[3,621061,0.2300761481],[3,621067,0.0277050353],[3,621212,0.2411576434],[3,621414,0.0441347211],[3,621482,0.0918429],[3,621510,0.1917399114],[3,621572,0.0778492706],[3,621752,0.0587876731],[3,622147,0.0297698773],[3,622281,0.03766695],[3,622292,0.0574952512],[3,622293,0.0313333977],[3,622642,0.0777473894],[3,623287,0.0332367509],[3,623385,0.1989905047],[3,623513,0.1927656817],[3,624101,0.0350553133],[3,624149,0.033243511],[3,624387,0.0332367509],[3,624635,0.0925183289],[3,624638,0.0638513264],[3,625181,0.03564713],[3,625182,0.0296931545],[3,625285,0.1105608846],[3,625984,0.0451330849],[3,626036,0.0296931545],[3,626503,0.0882503689],[3,626531,0.0570822295],[3,626598,0.1815256042],[3,626638,0.1287544119],[3,626696,0.0541325432],[3,626782,0.0734939871],[3,626788,0.0317633957],[3,626801,0.1058615909],[3,626829,0.0352439469],[3,626986,0.0461215873],[3,627629,0.2124497267],[3,627917,0.1327799734],[3,628073,0.0553036973],[3,628300,0.0584035152],[3,629082,0.0299600522],[3,629650,0.0349874765],[3,630058,0.2637053179],[3,630429,0.0494808796],[3,630438,0.2704211933],[3,630644,0.0337184868],[3,630825,0.1183156513],[3,630894,0.0589963979],[3,630996,0.0290848888],[3,631041,0.0433584868],[3,631193,0.0345828991],[3,631326,0.0296931545],[3,631401,0.0348801766],[3,631447,0.030439727],[3,631501,0.0290848888],[3,631862,0.1712690862],[3,632076,0.0754150773],[3,632956,0.0296931545],[3,633498,0.0296931545],[3,633514,0.0464736691],[3,633550,0.0794129491],[3,633580,0.0352151777],[3,634082,0.0296931545],[3,635061,0.0636927291],[3,636316,0.0317633957],[3,636559,0.0366986089],[3,636747,0.0902531008],[3,637290,0.0301877229],[3,638115,0.0297483151],[3,638162,0.0466333512],[3,638229,0.0450898447],[3,638868,0.0489495974],[3,639265,0.0563232601],[3,639344,0.1131212816],[3,639479,0.1424172935],[3,640074,0.0314499833],[3,640560,0.0651874484],[3,640571,0.0296931545],[3,640674,0.0513422256],[3,640730,0.0332367509],[3,640745,0.16003133],[3,640758,0.0972130492],[3,640873,0.0318422716],[3,642179,0.0466974997],[3,642247,0.1295465763],[3,642443,0.1510020476],[3,642647,0.1145283759],[3,642808,0.0296931545],[3,642919,0.0318422716],[3,643347,0.6610276653],[3,643500,0.038908968],[3,643760,0.4927658745],[3,644368,0.1974986863],[3,644583,0.0326416366],[3,645805,0.0312415443],[3,646020,0.0296931545],[3,646216,0.0332367509],[3,646313,0.0313333977],[3,646443,0.4843711785],[3,647210,0.0358750269],[3,647539,0.0820100002],[3,647616,0.0313333977],[3,647646,0.0296931545],[3,647690,0.030439727],[3,647731,0.1042136028],[3,647830,0.0330152207],[3,647858,0.0348801766],[3,648227,0.2412363471],[3,648338,0.0444044747],[3,648350,0.1930992403],[3,648356,0.0401785637],[3,648466,0.0299600522],[3,648914,0.1674014996],[3,649371,0.0355160574],[3,649967,0.0303649103],[3,650002,0.1135434014],[3,651263,0.1084124411],[3,651710,0.0986772356],[3,651863,0.0446633373],[3,653053,0.0290848888],[3,653872,0.0299600522],[3,654463,0.0340573148],[3,654671,0.0403738976],[3,654774,0.0296931545],[3,654936,0.1257657272],[3,655108,0.0759564699],[3,655274,0.0936633723],[3,655366,0.0392028933],[3,655422,0.044995505],[3,655550,0.0353141786],[3,656030,0.035614289],[3,656346,0.0296931545],[3,656600,0.0898543581],[3,657411,0.0358694218],[3,657778,0.0351990056],[3,657998,0.0474228904],[3,658834,0.0290848888],[3,659140,0.0479125161],[3,660324,0.0326416366],[3,660437,0.0402364396],[3,660464,0.0348801766],[3,660659,0.0459949711],[3,660963,0.2797481598],[3,661004,0.3998228632],[3,661320,0.0542860748],[3,661356,0.1359717392],[3,661367,0.0768099387],[3,661451,0.0296931545],[3,661453,0.1003262614],[3,661501,0.267517338],[3,661545,0.254442397],[3,661564,0.4616087963],[3,661624,0.0743884973],[3,661670,0.3780454414],[3,661709,0.0654097555],[3,661841,0.0965638479],[3,661884,0.0509304832],[3,662096,0.0956102597],[3,662333,0.0469727323],[3,662689,0.0804823421],[3,662708,0.1803198525],[3,662720,0.0698797931],[3,662723,0.0340573148],[3,662757,0.1224565234],[3,662758,0.2487840429],[3,662864,0.1511880077],[3,662900,0.0296931545],[3,662966,0.2286680894],[3,662973,0.2979238018],[3,663130,0.1274116517],[3,663222,0.0296931545],[3,663235,0.205109868],[3,663331,0.2972384734],[3,663404,0.0969457879],[3,663580,0.0760983938],[3,663660,0.0475080275],[3,663668,0.0361926476],[3,663804,0.2286193876],[3,664101,0.0318422716],[3,664711,0.1732098682],[3,664752,0.1814543579],[3,664790,0.0783741479],[3,664800,0.1395221345],[3,664995,0.3748284175],[3,665068,0.0415517528],[3,665100,0.0310205912],[3,665189,0.0322030166],[3,665191,0.0558924178],[3,665194,0.1592112934],[3,665278,0.0831701714],[3,665512,0.0317633957],[3,665628,0.1487751908],[3,665716,0.0304916542],[3,665862,0.1447051861],[3,665879,0.0296931545],[3,666000,0.0296931545],[3,666648,0.0503845808],[3,666827,0.0319000567],[3,666892,0.054259899],[3,666993,0.2207392318],[3,667147,0.0922265152],[3,667225,0.8568421053],[3,667261,0.0465860536],[3,667431,0.0507503575],[3,667883,0.050157381],[3,667889,0.12876996],[3,668086,0.2563553479],[3,668088,0.0535987518],[3,668183,0.0762602065],[3,668616,0.0304916542],[3,669044,0.0348801766],[3,669468,0.0486927695],[3,669586,0.2241391794],[3,669621,0.5425759574],[3,670850,0.0953094542],[3,671045,0.0420758122],[3,671077,0.0348665704],[3,672723,0.0332367509],[3,672752,0.0305113322],[3,672820,0.0296931545],[3,672941,0.031643079],[3,673167,0.0347445767],[3,673711,0.0340573148],[3,674055,0.1449346273],[3,674110,0.2205095012],[3,674139,0.1036060508],[3,674192,0.6546611712],[3,674228,0.0407822021],[3,674310,0.1019072565],[3,674410,0.0533779088],[3,674442,0.1924810167],[3,674447,0.0348801766],[3,674485,0.1158516377],[3,674512,0.1695098766],[3,674513,0.1657021781],[3,674517,0.0296931545],[3,674565,0.0529492202],[3,674683,0.2244663808],[3,674862,0.1537033767],[3,674883,0.1003819261],[3,674898,0.0543860336],[3,675256,0.2028894813],[3,675259,0.0830570058],[3,675260,0.3384382096],[3,675288,0.030439727],[3,675308,0.1114524219],[3,675346,0.1500520333],[3,675355,0.0821136604],[3,675378,0.39492931],[3,675541,0.1144623807],[3,675565,0.0353340836],[3,675821,0.0312415443],[3,676043,0.0535179259],[3,676426,0.1492113316],[3,676458,0.4524695712],[3,676463,0.0860096427],[3,676576,0.0742025235],[3,676612,0.0493993108],[3,676614,0.0305113322],[3,676633,0.2264404065],[3,676669,0.1923426671],[3,676781,0.2004444478],[3,676956,0.0752093408],[3,677128,0.6098804452],[3,677268,0.0923281137],[3,677273,0.0774855281],[3,677311,0.0718366837],[3,677321,0.0803818474],[3,677426,0.071399524],[3,677473,0.0356903008],[3,677490,0.0802489125],[3,677533,0.0616040038],[3,677538,0.0866108753],[3,677580,0.2238447891],[3,677621,0.2113056634],[3,677762,0.0443625685],[3,678234,0.2148922168],[3,678258,0.0296931545],[3,678260,0.0511210743],[3,678428,0.8855310428],[3,678458,0.0646118439],[3,678472,0.3337614645],[3,678477,0.0318422716],[3,678491,0.1967912673],[3,678495,0.1200310344],[3,678534,0.0301877229],[3,678624,0.0314611572],[3,678700,0.0330152207],[3,678754,0.1755105142],[3,678857,0.0330152207],[3,678976,0.1110801203],[3,678983,0.106504169],[3,679000,0.0650927595],[3,679266,0.1457384724],[3,679669,0.0349874765],[3,679722,0.0651515549],[3,679872,0.0343770498],[3,680272,0.0808455918],[3,680318,0.3017922826],[3,680374,0.0475027826],[3,680503,0.0390614403],[3,680506,0.0318422716],[3,680923,0.0748894186],[3,681036,0.0963518131],[3,681282,0.0318422716],[3,681422,0.0801009315],[3,681439,0.0532249403],[3,681441,0.580511934],[3,681468,0.1480019709],[3,681554,0.0428908233],[3,681561,0.0348801766],[3,681919,0.0499324293],[3,682363,0.2271961833],[3,682480,0.032357827],[3,682610,0.1154708565],[3,682616,0.584578156],[3,682644,0.1398273159],[3,682701,0.6328861652],[3,682772,0.0343770498],[3,682797,0.0368168366],[3,682801,0.0985506623],[3,683062,0.0913902502],[3,683178,0.0940573008],[3,683318,0.0767616253],[3,683324,0.0296931545],[3,683506,0.0483778715],[3,683662,0.7669268936],[3,683784,0.0748213381],[3,683926,0.0737455408],[3,683940,0.0300520712],[3,683992,0.0611382434],[3,684082,0.094285087],[3,684257,0.0332689337],[3,684436,0.0296931545],[3,684458,0.0586967652],[3,684483,0.0427498686],[3,684563,0.0303649103],[3,684721,0.0342541768],[3,684907,0.1048957563],[3,685057,0.0852352518],[3,686025,0.0374945903],[3,686202,0.0702171513],[3,688051,0.0340573148],[3,688388,0.0522620717],[3,688796,0.0382066315],[3,689001,0.114623497],[3,689600,0.1349507897],[3,689601,0.0342541768],[3,689835,0.0633128799],[3,690441,0.0510355261],[3,690476,0.0446388055],[3,690702,0.0614009968],[3,691083,0.0760739273],[3,691106,0.0343770498],[3,691197,0.0348513244],[3,692806,0.0305113322],[3,693054,0.0337550832],[3,693470,0.0299600522],[3,693550,0.0314499833],[3,694145,0.0431090234],[3,694206,0.1772390826],[3,694248,0.0296931545],[3,694994,0.0296931545],[3,695091,0.0318422716],[3,695455,0.0411182431],[3,695683,0.0412738686],[3,696102,0.038534635],[3,696490,0.0918422559],[3,696737,0.0322828864],[3,696998,0.0348665704],[3,697010,0.0296931545],[3,697032,0.1577533766],[3,697620,0.0322949855],[3,697634,0.0433715431],[3,697680,0.1046041177],[3,697770,0.0552820879],[3,697833,0.1231392561],[3,697890,0.0418908958],[3,697905,0.0970775786],[3,697940,0.1333375431],[3,698280,0.0364934566],[3,698333,0.0627386193],[3,698384,0.0370507085],[3,698700,0.1204502352],[3,698875,0.3564092472],[3,698881,0.0323504853],[3,698934,0.0364412524],[3,699000,0.0323504853],[3,699141,0.0679635058],[3,699161,0.034260412],[3,699166,0.0326183135],[3,699198,0.0317400993],[3,699346,0.1131549418],[3,699383,0.0671132209],[3,699621,0.0633092531],[3,699744,0.0574555726],[3,699747,0.0376886681],[3,699795,0.0757292024],[3,699814,0.1702577209],[3,700212,0.0320885666],[3,700266,0.1082451722],[3,700308,0.1533475529],[3,700316,0.0339964464],[3,701089,0.0359064346],[3,701114,0.0617777451],[3,701297,0.0364934566],[3,701324,0.0339964464],[3,701330,0.0374946477],[3,701490,0.3272402067],[3,701591,0.0623410513],[3,701606,0.0543339548],[3,701768,0.0396732043],[3,701940,0.1391719607],[3,701982,0.3145086574],[3,702163,0.0339042728],[3,702260,0.0323504853],[3,702314,0.0933035596],[3,702360,0.0373647253],[3,702375,0.0469980117],[3,702783,0.21977957],[3,703061,0.6543083949],[3,703124,0.7315438899],[3,703290,0.0750419157],[3,704021,0.1046636803],[3,704098,0.1206883991],[3,704242,0.0344279433],[3,704666,0.1510139534],[3,704682,0.0323504853],[3,705142,0.2197592057],[3,705193,0.0657413456],[3,705248,0.0502855106],[3,705665,0.0370507085],[3,706093,0.1316134838],[3,706536,0.0363898498],[3,707378,0.0779867713],[3,708009,0.0367298589],[3,708248,0.0569970794],[3,708626,0.0359064346],[3,708762,0.0507358834],[3,708844,0.0323504853],[3,709452,0.0323504853],[3,709536,0.0331607632],[3,709573,0.0370507085],[3,710119,0.2466990625],[3,710170,0.0519345607],[3,710335,0.1538721447],[3,710387,0.0887753898],[3,710891,0.395125667],[3,710893,0.0575344482],[3,711030,0.2695993553],[3,711067,0.0902201842],[3,711113,0.1173682069],[3,711443,0.0332001701],[3,711472,0.0446068801],[3,711517,0.2698697666],[3,712092,0.2533886157],[3,712224,0.2167376643],[3,712236,0.7249443912],[3,712480,0.271682168],[3,712561,0.1047910941],[3,712642,0.1168425746],[3,712867,0.1139593569],[3,713558,0.057840018],[3,713598,0.0579595328],[3,713650,0.0620867569],[3,714055,0.0546076361],[3,714193,0.1112915983],[3,714586,0.0679902651],[3,714774,0.0494676996],[3,714847,0.0324339213],[3,714855,0.072719742],[3,714857,0.0355345505],[3,715415,0.2284079057],[3,715673,0.0614857585],[3,715725,0.2067687476],[3,715730,0.2733655508],[3,716138,0.0324339213],[3,716529,0.0324339213],[3,716847,0.1864596882],[3,716849,0.1205489514],[3,716867,0.0822217834],[3,716869,0.2741460566],[3,717049,0.0332001701],[3,718295,0.0354258419],[3,718366,0.035777047],[3,718973,0.2350649028],[3,719427,0.0592018755],[3,719619,0.0340621506],[3,719627,0.4587196692],[3,719735,0.0346743909],[3,719771,0.0398176468],[3,719842,0.0368957142],[3,719859,0.0895012723],[3,719905,0.0386760672],[3,719950,0.0719653484],[3,719954,0.0368375514],[3,720802,0.1504925393],[3,720897,0.2436424187],[3,721199,0.0365901199],[3,721339,0.0340621506],[3,721464,0.0742248831],[3,721706,0.0691952789],[3,721742,0.0470617787],[3,721901,0.0996194807],[3,722120,0.0346743909],[3,722153,0.0422997006],[3,722156,0.0681993929],[3,722370,0.0346743909],[3,722425,0.096978886],[3,722600,0.0560611935],[3,722716,0.0767755405],[3,722936,0.0346743909],[3,723206,0.0364096443],[3,723250,0.1312105504],[3,723517,0.0488487185],[3,723817,0.0346743909],[3,723996,0.0354428838],[3,724011,0.0346743909],[3,724593,0.0787248093],[3,724708,0.0398176468],[3,724754,0.0346743909],[3,724940,0.0364096443],[3,725656,0.0468073026],[3,726289,0.0513057991],[3,726665,0.0852628467],[3,726724,0.8056945359],[3,726894,0.059750743],[3,726944,0.0582613484],[3,727306,0.0398176468],[3,727494,0.0354258419],[3,727540,0.0362443496],[3,727701,0.0867556911],[3,727702,0.058300223],[3,727801,0.0816592037],[3,727894,0.035497915],[3,727904,0.0346743909],[3,727985,0.0599365002],[3,728262,0.0616691757],[3,728341,0.0812857146],[3,728580,0.0661900059],[3,728582,0.1178985189],[3,728731,0.0354428838],[3,728865,0.1649067096],[3,729002,0.1985365271],[3,729746,0.0346743909],[3,729871,0.0350317209],[3,729980,0.0340621506],[3,730148,0.0434923011],[3,730314,0.0755069794],[3,730590,0.0340621506],[3,730769,0.0340621506],[3,731017,0.1335573492],[3,731082,0.3503453252],[3,731092,0.0340621506],[3,731463,0.2680715798],[3,731600,0.224424119],[3,731661,0.0955806654],[3,732217,0.1794757116],[3,732231,0.0346743909],[3,732546,0.0340621506],[3,733062,0.1224719134],[3,733315,0.1265424475],[3,733545,0.0340621506],[3,734028,0.0426076115],[3,734282,0.0566974802],[3,735304,0.0371854527],[3,735312,0.0346743909],[3,735578,0.0346743909],[3,736542,0.0419236418],[3,736856,0.110131749],[3,737231,0.0348376658],[3,737612,0.109710178],[3,737782,0.0484541257],[3,737920,0.0638113087],[3,738006,0.0869425934],[3,738015,0.0348376658],[3,739019,0.130039743],[3,739237,0.0656997946],[3,739284,0.0732773314],[3,739519,0.1186811568],[3,739852,0.2303913894],[3,739970,0.0568656371],[3,740012,0.0732958617],[3,740020,0.1160155529],[3,740358,0.0583535681],[3,741389,0.0346743909],[3,741651,0.0905533771],[3,741735,0.0346743909],[3,741748,0.1044226156],[3,741769,0.1622844219],[3,741801,0.0801673224],[3,742006,0.4067666294],[3,742633,0.0398816126],[3,744290,0.0346743909],[3,744485,0.0686840671],[3,744646,0.0447413978],[3,745005,0.0731756006],[3,745156,0.0368375514],[3,745287,0.0511070198],[3,745802,0.0410186223],[3,745949,0.0398816126],[3,746220,0.0466930748],[3,746762,0.0410967364],[3,748989,0.0364096443],[3,749798,0.0407107257],[3,749953,0.0534211499],[3,750031,0.0390449066],[3,751106,0.0377841006],[3,751245,0.0346743909],[3,752383,0.055811695],[3,752736,0.0398176468],[3,752775,0.0498447402],[3,752941,0.0372810455],[3,753619,0.0432166272],[3,754204,0.0398176468],[3,755713,0.0500677322],[3,756565,0.0629884473],[3,756992,0.0411624148],[3,757178,0.0368375514],[3,757503,0.6560294399],[3,757680,0.0371854527],[3,757890,0.0607230165],[3,758067,0.0456954375],[3,758231,0.1422127597],[3,758350,0.0368375514],[3,758426,0.339515238],[3,759436,0.0694830676],[3,759478,0.0967135395],[3,759550,0.0511366218],[3,759926,0.1754316063],[3,760104,0.0346743909],[3,760119,0.1282449482],[3,760195,0.3234691183],[3,760488,0.1903286587],[3,760542,0.035497915],[3,760756,0.0605546358],[3,760811,0.0411399473],[3,760845,0.072798838],[3,761023,0.1419698625],[3,761105,0.1010372762],[3,761323,0.0887644567],[3,761363,0.0876875783],[3,761376,0.0617320788],[3,761934,0.0362216613],[3,762145,0.0717014939],[3,762424,0.0386760672],[3,762512,0.2472661994],[3,762669,0.1560186911],[3,762886,0.1500431239],[3,763583,0.0346743909],[3,764264,0.0457082859],[3,765133,0.0547879499],[3,765231,0.0454589483],[3,765613,0.0524241478],[3,765963,0.0346743909],[3,766037,0.1059304955],[3,766205,0.0354428838],[3,766251,0.0372810455],[3,766309,0.1061469267],[3,766749,0.0346743909],[3,767221,0.0372810455],[3,767677,0.0870479709],[3,768073,0.108156125],[3,768473,0.0845564869],[3,768644,0.2378712885],[3,768843,0.0585824717],[3,768874,0.2340489239],[3,768912,0.0465638757],[3,768958,0.097794793],[3,768975,0.0346743909],[3,769034,0.0377841006],[3,769165,0.0406906932],[3,769282,0.097472841],[3,769430,0.0346743909],[3,769539,0.0440589889],[3,769601,0.0474079177],[3,770138,0.063217032],[3,770181,0.036442706],[3,770610,0.0346743909],[3,770751,0.0722074424],[3,771283,0.1368525235],[3,771498,0.6132739032],[3,771613,0.0573168457],[3,771652,0.0587859925],[3,771656,0.4204393957],[3,771702,0.0598046513],[3,771950,0.0637108407],[3,772799,0.0346743909],[3,773022,0.0346743909],[3,773124,0.0457763406],[3,774447,0.0340621506],[3,774762,0.0560501812],[3,774803,0.0371854527],[3,774960,0.0340621506],[3,774988,0.0340621506],[3,774994,0.0340621506],[3,775050,0.0346743909],[3,775927,0.0523879207],[3,776316,0.1163870338],[3,776557,0.0362216613],[3,776880,0.0422997006],[3,776961,0.1128146715],[3,776985,0.0683907949],[3,777047,0.0362443496],[3,777196,0.2200353388],[3,777532,0.0346743909],[3,777631,0.0460287491],[3,778358,0.0548145669],[3,778376,0.0405164906],[3,778742,0.0377216225],[3,779037,0.4702832975],[3,779366,0.0538960107],[3,779548,0.0627694435],[3,779572,0.0340621506],[3,779854,0.0548230933],[3,780234,0.060969729],[3,780270,0.037551486],[3,780779,0.0609098109],[3,781051,0.0599076084],[3,781192,0.0391166278],[3,781228,0.0346743909],[3,781514,0.0583888227],[3,783011,0.0368375514],[3,783848,0.0340621506],[3,784486,0.0489148806],[3,784656,0.0419317466],[3,785413,0.0924109397],[3,785419,0.0988318882],[3,785637,0.090541084],[3,785639,0.4833503646],[3,785784,0.0624247807],[3,786330,0.0909932791],[3,786448,0.0340621506],[3,786473,0.1110722935],[3,786477,0.0363035674],[3,786554,0.0543309887],[3,786637,0.3777033109],[3,786880,0.2329445197],[3,787016,0.0951335291],[3,787063,0.5931501697],[3,787091,0.1894989778],[3,787619,0.0765445773],[3,788145,0.3434410475],[3,788711,0.0608454135],[3,788921,0.0403088198],[3,789503,0.0566998926],[3,789710,0.0420282479],[3,790228,0.0613333268],[3,791636,0.0346743909],[3,792149,0.0513759938],[3,792183,0.035497915],[3,792743,0.0365901199],[3,792871,0.0346743909],[3,793180,0.0506609146],[3,793324,0.0402324979],[3,794107,0.0391796312],[3,794388,0.1401555589],[3,794500,0.0697526809],[3,794683,0.04753266],[3,794738,0.0960498999],[3,794746,0.0444787441],[3,794915,0.0696297824],[3,795015,0.0960498999],[3,795454,0.1869178185],[3,795765,0.0772287709],[3,795939,0.1438226224],[3,796183,0.0490876645],[3,796322,0.0346743909],[3,796640,0.083866387],[3,797055,0.0354258419],[3,797111,0.4444045012],[3,797227,0.0958728243],[3,797316,0.0372810455],[3,797389,0.096548153],[3,797611,0.8733907663],[3,798350,0.0429434061],[3,798674,0.388257449],[3,799449,0.0391166278],[3,801644,0.0411536915],[3,802326,0.0346743909],[3,802822,0.0400101373],[3,804532,0.0346743909],[3,805513,0.1644435957],[3,805682,0.0981357285],[3,806141,0.0368090311],[3,807198,0.0499083367],[3,808144,0.0593635468],[3,809682,0.0361501294],[3,810834,0.1778272156],[3,812594,0.0972725758],[3,813592,0.0532949189],[3,813904,0.0575059477],[3,814115,0.0361501294],[3,814131,0.0361501294],[3,814380,0.0383174504],[3,814408,0.0796209259],[3,814993,0.0406009103],[3,815102,0.0486634094],[3,815245,0.0383174504],[3,815259,0.0355367115],[3,815272,0.0729413402],[3,815380,0.1525631186],[3,815449,0.0378887203],[3,815774,0.0459556887],[3,815899,0.1656840747],[3,816521,0.0519859628],[3,816845,0.1220661858],[3,816868,0.1276970547],[3,817033,0.0673812939],[3,817067,0.0913097622],[3,817093,0.0775987638],[3,817174,0.0484667321],[3,817189,0.1248281619],[3,817289,0.3854120172],[3,817487,0.0887984075],[3,817772,0.0467831903],[3,817858,0.0807059706],[3,818181,0.1342608665],[3,818340,0.177808667],[3,818346,0.3004099314],[3,818421,0.2234909704],[3,818558,0.0631954701],[3,819013,0.0361501294],[3,819049,0.1212041135],[3,819135,0.0673145995],[3,819412,0.0434214435],[3,819843,0.0537641621],[3,820098,0.0818027344],[3,820316,0.0405229832],[3,820370,0.0470264637],[3,820497,0.4166775825],[3,821024,0.0552969],[3,821765,0.0377824393],[3,822182,0.2710695082],[3,822623,0.1075722513],[3,823483,0.0378887203],[3,823767,0.3294802909],[3,823775,0.0369752374],[3,825292,0.0559549629],[3,826875,0.1298764055],[3,826890,0.0361501294],[3,827057,0.1456307944],[3,827111,0.0355367115],[3,827444,0.2114696819],[3,827451,0.1030267192],[3,827503,0.0559447663],[3,827919,0.1216924309],[3,828327,0.2381985156],[3,828373,0.0961586783],[3,828488,0.2240806196],[3,828597,0.0451516109],[3,828627,0.0410319083],[3,828790,0.0361501294],[3,828824,0.2395445365],[3,828825,0.0928669502],[3,828836,0.0658340325],[3,828864,0.0431504402],[3,828877,0.2787086748],[3,829087,0.0588111942],[3,829371,0.0386047015],[3,829538,0.034925728],[3,829928,0.0341028174],[3,829936,0.0603785671],[3,830381,0.04883674],[3,831075,0.1695526418],[3,831120,0.0360171193],[3,831250,0.0392044532],[3,831278,0.0904077776],[3,831664,0.0848040826],[3,831903,0.0811405216],[3,832036,0.0673812939],[3,833750,0.4078567269],[3,833764,0.0402549715],[3,834218,0.1523708572],[3,834443,0.3096622291],[3,835009,0.0341028174],[3,835924,0.4680892619],[3,836005,0.4015571458],[3,836058,0.0360171193],[3,836258,0.0512035564],[3,836455,0.0764031263],[3,836785,0.091455328],[3,837103,0.1804524474],[3,837231,0.1400686032],[3,837319,0.1057717341],[3,837535,0.0780418879],[3,837710,0.0641519969],[3,838152,0.1241379173],[3,838183,0.1753632771],[3,838299,0.0341028174],[3,838639,0.0875334437],[3,838999,0.0393061601],[3,839481,0.1308294145],[3,839582,0.0601842753],[3,840392,0.0554393881],[3,841212,0.0341028174],[3,842412,0.0622075083],[3,842442,0.0419443887],[3,842443,0.0456383156],[3,842459,0.0377824393],[3,842959,0.034925728],[3,843496,0.0385417451],[3,844344,0.0882428463],[3,844612,0.325479328],[3,844952,0.0430181316],[3,844969,0.1030177132],[3,845181,0.0591103921],[3,845418,0.0852976385],[3,845727,0.044903479],[3,845812,0.0527578177],[3,845839,0.0993015042],[3,845878,0.1191907513],[3,845885,0.1293286752],[3,846096,0.10604744],[3,846695,0.3275583679],[3,846972,0.0514744169],[3,847061,0.0405772913],[3,847140,0.1398510133],[3,847207,0.1059611465],[3,847372,0.0341028174],[3,847953,0.1521984424],[3,848059,0.1251660598],[3,848123,0.0808833919],[3,848480,0.0955306894],[3,848558,0.1272494461],[3,848639,0.3075301872],[3,848683,0.0762043295],[3,848785,0.0793650513],[3,849115,0.0469515079],[3,850536,0.0505650865],[3,850896,0.0362312784],[3,851062,0.0341028174],[3,851507,0.0341028174],[3,851857,0.2171613271],[3,851910,0.0936520341],[3,851932,0.0493441395],[3,852051,0.0360171193],[3,853593,0.0534214013],[3,853735,0.040726162],[3,853768,0.3582551325],[3,853822,0.0978607503],[3,854074,0.0501299587],[3,854076,0.0341028174],[3,854135,0.0379425085],[3,854225,0.2339890329],[3,854234,0.1337125938],[3,854869,0.0369777692],[3,854926,0.0606238587],[3,855002,0.1708070761],[3,855016,0.0381951106],[3,855019,0.0481573739],[3,855034,0.0606238587],[3,855040,0.0395832871],[3,855292,0.1271732161],[3,855472,0.1083025719],[3,855511,0.2286609295],[3,855677,0.096847953],[3,856073,0.0555381128],[3,856350,0.0723568821],[3,856607,0.3648197899],[3,856669,0.1175901684],[3,856688,0.0435744656],[3,856955,0.0951106966],[3,857552,0.1620056686],[3,857628,0.0561174702],[3,857674,0.4299781199],[3,857770,0.0418850739],[3,857861,0.1610272315],[3,857913,0.5103132345],[3,858114,0.1629957403],[3,858188,0.0461674276],[3,858242,0.0562619965],[3,858257,0.1036660004],[3,858340,0.0681242673],[3,858526,0.0407367078],[3,858560,0.0388106514],[3,859063,0.4610710786],[3,859807,0.0545145733],[3,860333,0.0952446446],[3,860738,0.0773890359],[3,861217,0.0407367078],[3,862001,0.0395832871],[3,862835,0.0400064738],[3,863337,0.0402182556],[3,863720,0.0425845991],[3,863877,0.0858875651],[3,864247,0.1049150374],[3,864815,0.1045126733],[3,865079,0.0408354121],[3,865245,0.054028347],[3,865408,0.0610399529],[3,865572,0.044264247],[3,866122,0.0408354121],[3,866132,0.044264247],[3,866425,0.0402182556],[3,867118,0.0512342616],[3,867170,0.0999959146],[3,867681,0.0619242525],[3,869185,0.0408354121],[3,871663,0.0446781002],[3,872169,0.0420464009],[3,872344,0.048762544],[3,873733,0.0809150432],[3,873914,0.1558319076],[3,874344,0.0594561309],[3,874555,0.0554455697],[3,874562,0.0635849426],[3,874579,0.0428222774],[3,874624,0.296765246],[3,874649,0.0682946248],[3,874791,0.0596074057],[3,874801,0.1131260475],[3,874888,0.2916964109],[3,874916,0.1002902701],[3,874980,0.0481407328],[3,875017,0.0559125884],[3,875035,0.3325125605],[3,875049,0.0460857033],[3,875097,0.3034542958],[3,875408,0.4265099266],[3,875627,0.0481407328],[3,875775,0.0414282781],[3,875814,0.0428050719],[3,875819,0.0481411627],[3,875822,0.1888758769],[3,875828,0.0450553354],[3,875830,0.0820238317],[3,875852,0.4020815025],[3,875951,0.0466813359],[3,875953,0.3503634464],[3,875956,0.0588050907],[3,875957,0.0445293601],[3,876000,0.0445064273],[3,876034,0.4928778377],[3,876036,0.0445293601],[3,876315,0.0640747662],[3,876448,0.0445064273],[3,876579,0.0420464009],[3,876649,0.0728594776],[3,876650,0.1214315726],[3,876651,0.07978587],[3,876652,0.2076665227],[3,876658,0.5188529221],[3,876659,0.3465617105],[3,876661,0.0428778374],[3,876662,0.4207257531],[3,876667,0.0478072874],[3,876668,0.1266027997],[3,876767,0.2897610803],[3,876860,0.1452722415],[3,877154,0.1008631886],[3,877236,0.0906312423],[3,877558,0.0663587032],[3,877563,0.2770735936],[3,877872,0.0484455821],[3,877924,0.1441576401],[3,877945,0.3084203259],[3,878090,0.1046486849],[3,878208,0.0550282124],[3,878287,0.0420464009],[3,878335,0.6752089559],[3,878353,0.0505595829],[3,878374,0.1200720351],[3,878506,0.0469872816],[3,878526,0.1036570399],[3,878540,0.0970222334],[3,878596,0.2473300937],[3,878824,0.0445064273],[3,878872,0.1174374131],[3,878929,0.0638428871],[3,879011,0.1011307905],[3,879132,0.0447298539],[3,879204,0.0681367313],[3,879205,0.0638831843],[3,879299,0.0473036536],[3,879410,0.0985851043],[3,879878,0.3540347278],[3,880078,0.0481411627],[3,880143,0.3215523389],[3,880157,0.1518830257],[3,880435,0.0454806044],[3,881799,0.0474707813],[3,883760,0.0445064273],[3,884794,0.0573075878],[3,885107,0.0680683043],[3,885197,0.7343486542],[3,885240,0.0445064273],[3,885496,0.0532035907],[3,885613,0.0986608583],[3,885641,0.1092229552],[3,885990,0.2857357254],[3,886209,0.2125165927],[3,886515,0.0414282781],[3,886830,0.3486303395],[3,886884,0.1018953554],[3,886889,0.3922099087],[3,886911,0.1238290758],[3,887373,0.0905893062],[3,887491,0.4074849826],[3,887830,0.1024945117],[3,888083,0.0511197722],[3,888246,0.0445064273],[3,888305,0.1883533268],[3,888511,0.1670345365],[3,888832,0.3537105271],[3,888840,0.0414282781],[3,888920,0.0568513845],[3,889632,0.0638851234],[3,891936,0.0779577187],[3,892614,0.0467548944],[3,892620,0.1633237673],[3,892848,0.0573228594],[3,893071,0.0414282781],[3,893170,0.1003721957],[3,893192,0.0741465641],[3,893207,0.0420464009],[3,893550,0.063814524],[3,893650,0.0561236451],[3,893811,0.0478156898],[3,893837,0.1220235056],[3,893986,0.3774137932],[3,894053,0.1065580771],[3,894159,0.1112857903],[3,894192,0.4058660465],[3,894428,0.0515059627],[3,894575,0.0544487406],[3,894653,0.0823825692],[3,895017,0.2139608389],[3,895310,0.4384408138],[3,895540,0.2784168681],[3,895958,0.1536542839],[3,896541,0.0698898443],[3,896785,0.1327678414],[3,896972,0.2356018371],[3,897045,0.1934880293],[3,897140,0.071142811],[3,897164,0.0493550798],[3,897331,0.1161329885],[3,897338,0.1092777835],[3,897604,0.0420464009],[3,898321,0.1084498459],[3,899505,0.0585600192],[3,900204,0.0678926235],[3,901860,0.0707749568],[3,901972,0.0486533185],[3,902423,0.0494340356],[3,902481,0.0454806044],[3,902720,0.0545178376],[3,903486,0.0873988352],[3,904019,0.068163373],[3,904192,0.0553483742],[3,904391,0.1199698007],[3,904509,0.0985725795],[3,904570,0.3911080187],[3,904818,0.2014540229],[3,905354,0.0437983266],[3,905504,0.1278043069],[3,905505,0.1307580683],[3,905614,0.0549103487],[3,906026,0.1371410517],[3,906038,0.0879695203],[3,906156,0.0554815804],[3,906253,0.0414282781],[3,906334,0.2161674478],[3,906522,0.2375065729],[3,906525,0.0414282781],[3,907379,0.0493550798],[3,907430,0.0445293601],[3,907874,0.0511159133],[3,908248,0.1725448381],[3,908306,0.0486533185],[3,908500,0.2225528963],[3,909047,0.0439805362],[3,909049,0.0693888895],[3,909100,0.2439216416],[3,909517,0.048762544],[3,910618,0.1013783558],[3,911610,0.0460857033],[3,911873,0.17317845],[3,912324,0.2809772534],[3,912848,0.0849882115],[3,912998,0.0481411627],[3,913541,0.0454806044],[3,915195,0.0702401926],[3,915681,0.0795550585],[3,915687,0.1007662791],[3,915769,0.086278747],[3,917575,0.2073344217],[3,918291,0.0414282781],[3,918309,0.0437983266],[3,918511,0.0439805362],[3,921931,0.0437983266],[3,921998,0.0475935522],[3,922292,0.0428050719],[3,922831,0.0478156898],[3,922835,0.0526293358],[3,923170,0.0499321364],[3,923887,0.0884985749],[3,923947,0.0610066204],[3,924763,0.062411058],[3,925061,0.0497272796],[3,926126,0.0462504114],[3,926711,0.0462504114],[3,926718,0.073104285],[3,926750,0.0609285931],[3,927054,0.1015664071],[3,928005,0.0440607881],[3,928517,0.0440607881],[3,929196,0.0440607881],[3,929924,0.0524371547],[3,930007,0.0440607881],[3,930167,0.0506848863],[3,931540,0.0448943867],[3,932240,0.0440607881],[3,932428,0.0557957587],[3,932674,0.0548841933],[3,934607,0.0440607881],[3,934979,0.1327800485],[3,935166,0.217250718],[3,935251,0.1044771418],[3,935496,0.051555129],[3,935563,0.0570763573],[3,936329,0.2765079105],[3,936334,0.0448386823],[3,936375,0.1425164701],[3,936379,0.2713915245],[3,936551,0.0758457529],[3,936707,0.2639561705],[3,936751,0.0524371547],[3,937098,0.1000391684],[3,937738,0.043441058],[3,938254,0.1374475062],[3,938392,0.099840866],[3,938522,0.0969309473],[3,938762,0.3090136374],[3,938867,0.0475039219],[3,939302,0.0481105943],[3,939594,0.1449602363],[3,939626,0.1152090308],[3,939677,0.0440607881],[3,939740,0.0626026242],[3,939841,0.1013144662],[3,939926,0.0467543507],[3,940066,0.2968750964],[3,940155,0.0591723045],[3,940179,0.0524752081],[3,940534,0.0636361912],[3,940823,0.0503490396],[3,940826,0.0727439786],[3,941591,0.1946518889],[3,941649,0.1170062031],[3,941786,0.1227838978],[3,942358,0.1509196077],[3,942437,0.0653399469],[3,942607,0.0772977961],[3,942721,0.0514756073],[3,945464,0.0505772929],[3,946088,0.0468933485],[3,946794,0.065845971],[3,947133,0.1273632578],[3,947249,0.0703491208],[3,947381,0.0594780064],[3,947607,0.0728957765],[3,947797,0.5001872382],[3,947816,0.185363306],[3,947928,0.0576128747],[3,948053,0.0541817492],[3,948568,0.2843749695],[3,948690,0.0671244169],[3,948701,0.0593006216],[3,948864,0.1018844808],[3,949202,0.0645326257],[3,949260,0.1488013874],[3,949829,0.0786589557],[3,950176,0.0468933485],[3,950783,0.1675304072],[3,951142,0.0462713582],[3,951216,0.5442953067],[3,951508,0.0462713582],[3,951646,0.5747999779],[3,951720,0.083508158],[3,953726,0.0868922838],[3,954607,0.0493787512],[3,954624,0.0519200318],[3,954821,0.0506372965],[3,955080,0.0691637901],[3,955419,0.047180333],[3,955440,0.2854010496],[3,955549,0.0617745547],[3,955634,0.1599821348],[3,955698,0.0538310376],[3,955754,0.1251840474],[3,955974,0.0489438696],[3,956519,0.1975173777],[3,956897,0.2049041818],[3,957069,0.0812677464],[3,957145,0.1038141378],[3,957151,0.0465581137],[3,957206,0.1244734117],[3,957302,0.1687543699],[3,957568,0.548603341],[3,957751,0.0779691811],[3,958058,0.1344782975],[3,958624,0.1158747595],[3,958628,0.3558604984],[3,958886,0.1063984729],[3,961465,0.0558485554],[3,961496,0.0522540139],[3,961573,0.0467963725],[3,962342,0.0914579408],[3,962601,0.0467963725],[3,963038,0.0833180363],[3,963110,0.0502516338],[3,963411,0.0467963725],[3,963454,0.3221351337],[3,963461,0.1681650367],[3,963612,0.0538665284],[3,963764,0.1036630739],[3,964022,0.050860443],[3,964203,0.0587870835],[3,965414,0.0751008735],[3,965669,0.1552075158],[3,965826,0.1801461274],[3,966025,0.347711235],[3,966591,0.0810558382],[3,966908,0.0461744596],[3,967123,0.4311729385],[3,967972,0.0467963725],[3,968978,0.0579230335],[3,971696,0.5635777294],[3,973133,0.2137201503],[3,973335,0.0457965487],[3,974922,0.0450346159],[3,977561,0.0713722079],[3,978060,0.0464351101],[3,978936,0.0769918615],[3,979604,0.2061333724],[3,980148,0.0456556186],[3,980227,0.0464351101],[3,980254,0.0483547122],[3,980446,0.0550984193],[3,981675,0.0611621866],[3,981725,0.1459895323],[3,982040,0.0475987654],[3,982108,0.252245149],[3,982421,0.0573048677],[3,982493,0.0712020661],[3,982562,0.241216127],[3,982566,0.0965583184],[3,982573,0.1104122427],[3,982637,0.0475987654],[3,982730,0.160848495],[3,982783,0.0493337135],[3,982812,0.07780848],[3,982863,0.1066547656],[3,983047,0.4291730088],[3,983058,0.0450346159],[3,983277,0.0481073016],[3,983298,0.0474157069],[3,983325,0.6133842946],[3,983329,0.0602434137],[3,983346,0.0545211079],[3,983384,0.0456556186],[3,983806,0.2831763718],[3,984002,0.2477242618],[3,984083,0.1205436127],[3,984314,0.0925295749],[3,984336,0.1861070523],[3,984450,0.0503121486],[3,984470,0.0516487747],[3,984492,0.0493337135],[3,984502,0.1050755531],[3,984585,0.2694662861],[3,984647,0.0456556186],[3,984695,0.0512932728],[3,985063,0.1458748606],[3,985111,0.0948394633],[3,985727,0.0868026504],[3,985836,0.0493337135],[3,985852,0.0567659942],[3,986008,0.1333195847],[3,986231,0.0945939005],[3,986299,0.0456556186],[3,986372,0.060059756],[3,986449,0.2131126154],[3,986879,0.2331554116],[3,987069,0.0564041085],[3,987609,0.2172841802],[3,988091,0.0555585823],[3,988148,0.0766356768],[3,988279,0.0648512656],[3,988780,0.0582948955],[3,989270,0.0614370751],[3,990022,0.0497137407],[3,990046,0.0497137407],[3,990166,0.0711297008],[3,990508,0.0612928072],[3,990634,0.0712276084],[3,991119,0.0674853189],[3,991125,0.0456556186],[3,991365,0.0478497381],[3,992286,0.0863736778],[3,993080,0.1888216915],[3,993401,0.0497137407],[3,993429,0.0529413338],[3,993702,0.177664227],[3,993844,0.1186365381],[3,993978,0.3938312355],[3,994141,0.0863736778],[3,994143,0.1373968793],[3,994585,0.067053814],[3,994834,0.1813193431],[3,995502,0.2729636955],[3,995918,0.0472931133],[3,996762,0.1627803544],[3,997164,0.3568571895],[3,997443,0.3109488419],[3,999522,0.1296823558],[3,999777,0.0480742448],[3,999888,0.1612342223],[3,1000316,0.3063799184],[3,1000566,0.0584268657],[3,1001568,0.1014378414],[3,1002298,0.3015383421],[3,1003575,0.0900140868],[3,1003728,0.1160013115],[3,1004084,0.2458366591],[3,1004599,0.0645236593],[3,1004920,0.5314168458],[3,1005157,0.235420056],[3,1005521,0.0559149116],[3,1005659,0.0771148899],[3,1006048,0.0671037889],[3,1006215,0.1180608865],[3,1006248,0.100472633],[3,1007232,0.0472931133],[3,1007256,0.1927337797],[3,1007566,0.057822335],[3,1007801,0.046670804],[3,1007966,0.1142616388],[3,1008269,0.0632588564],[3,1008420,0.0481301811],[3,1008550,0.0472931133],[3,1008888,0.0494918494],[3,1009414,0.0779450989],[3,1010875,0.059210333],[3,1011484,0.0481301811],[3,1011693,0.0539059057],[3,1012438,0.1245208318],[3,1012760,0.1838205658],[3,1013136,0.3683851605],[3,1013154,0.0992362925],[3,1013407,0.2035491045],[3,1013625,0.2920454678],[3,1013654,0.1649711076],[3,1013696,0.244415439],[3,1014157,0.290787882],[3,1014394,0.1037180842],[3,1014727,0.0687165721],[3,1014734,0.2953439101],[3,1014821,0.1794702785],[3,1014900,0.0766605258],[3,1014974,0.1244174382],[3,1015368,0.0499978859],[3,1015655,0.0784903867],[3,1015678,0.2656398238],[3,1016445,0.201491622],[3,1016499,0.3031487543],[3,1016600,0.0801948375],[3,1016641,0.0544860279],[3,1016648,0.1609105753],[3,1017004,0.2641329703],[3,1017612,0.0494918494],[3,1017951,0.0472931133],[3,1019049,0.0764045882],[3,1019133,0.0601425174],[3,1019575,0.0533549666],[3,1019843,0.046670804],[3,1019851,0.0891134846],[3,1019888,0.0584207045],[3,1019982,0.0472931133],[3,1020213,0.1118817573],[3,1020451,0.0480742448],[3,1020890,0.1070408829],[3,1021419,0.046670804],[3,1021560,0.046670804],[3,1021574,0.0545175567],[3,1021671,0.1180410913],[3,1021792,0.0650817931],[3,1021808,0.0669562572],[3,1021869,0.1117472448],[3,1021872,0.444348436],[3,1021949,0.046670804],[3,1021951,0.0747660043],[3,1022305,0.1285346356],[3,1022448,0.1080996363],[3,1022481,0.1031872399],[3,1022630,0.1925604132],[3,1022648,0.0480742448],[3,1022722,0.0807802042],[3,1022762,0.120056701],[3,1022860,0.3203444484],[3,1023496,0.103302447],[3,1023812,0.0480569227],[3,1023941,0.1420118147],[3,1024052,0.26258441],[3,1024294,0.2691598451],[3,1025099,0.2931610171],[3,1025595,0.0472931133],[3,1026950,0.0565952733],[3,1027517,0.3054290954],[3,1027874,0.0865035953],[3,1027966,0.082990075],[3,1028295,0.0472931133],[3,1029843,0.0889433119],[3,1030554,0.0545175567],[3,1031137,0.056266276],[3,1032577,0.0830222531],[3,1033248,0.0526716598],[3,1034737,0.077008612],[3,1035047,0.0500128961],[3,1035423,0.1445019989],[3,1035970,0.0500128961],[3,1036454,0.0719074627],[3,1036481,0.0500128961],[3,1036573,0.054730942],[3,1036903,0.0745619062],[3,1037321,0.1022231253],[3,1037734,0.2126579818],[3,1037739,0.131579313],[3,1037952,0.0738579096],[3,1038463,0.0732129569],[3,1038539,0.0506773251],[3,1039008,0.0973029862],[3,1039091,0.0530756938],[3,1039306,0.0520879816],[3,1039317,0.0531137294],[3,1039395,0.142361762],[3,1039468,0.112103155],[3,1039841,0.080787559],[3,1040048,0.0513028339],[3,1040583,0.069768492],[3,1040744,0.0513028339],[3,1040803,0.0605903693],[3,1040827,0.0513028339],[3,1040878,0.1560883849],[3,1040997,0.1918142203],[3,1041330,0.0802214759],[3,1041349,0.0520705704],[3,1041406,0.0661927399],[3,1041520,0.1019509698],[3,1041608,0.0622981866],[3,1041886,0.0561109167],[3,1041893,0.1783031661],[3,1042355,0.1336719118],[3,1042462,0.0990560643],[3,1042817,0.0513028339],[3,1042977,0.055083094],[3,1042990,0.0781671502],[3,1043004,0.0792755067],[3,1043109,0.0902672205],[3,1043437,0.1292796705],[3,1043448,0.0613717349],[3,1043460,0.1139641781],[3,1043644,0.0586324639],[3,1043748,0.0520705704],[3,1043829,0.0683184028],[3,1044212,0.0861473931],[3,1044231,0.1562562138],[3,1044258,0.0583305253],[3,1044376,0.0513028339],[3,1044452,0.0520705704],[3,1044465,0.0708871968],[3,1044515,0.0689095162],[3,1045076,0.2929420155],[3,1045235,0.0557121212],[3,1045842,0.0807652223],[3,1045922,0.0609693307],[3,1046947,0.0694373383],[3,1047192,0.0828085829],[3,1047496,0.069730856],[3,1047609,0.077910531],[3,1047848,0.0835216472],[3,1049119,0.0590570451],[3,1049357,0.120604309],[3,1049409,0.0779762372],[3,1049564,0.1822179458],[3,1049912,0.2155057518],[3,1049938,0.2004130678],[3,1050336,0.0659964812],[3,1050585,0.3102810947],[3,1050627,0.0639586812],[3,1050920,0.0794596161],[3,1051034,0.0576513948],[3,1051419,0.0753327671],[3,1051454,0.2207851657],[3,1051567,0.368939044],[3,1051828,0.0535722977],[3,1051841,0.0897807624],[3,1052023,0.3030056541],[3,1052697,0.170828376],[3,1053200,0.1570920596],[3,1054437,0.0539659799],[3,1054475,0.0513028339],[3,1055131,0.0576513948],[3,1055301,0.2484057309],[3,1055718,0.0890983573],[3,1056507,0.0854225696],[3,1056969,0.0569641066],[3,1057078,0.0605297449],[3,1057385,0.0599134219],[3,1057650,0.054946068],[3,1057776,0.0795221498],[3,1057791,0.081047418],[3,1057889,0.0797804019],[3,1058299,0.0932453263],[3,1058365,0.0587830249],[3,1058585,0.1934142008],[3,1058588,0.0541580606],[3,1059480,0.1460109991],[3,1059537,0.1124406665],[3,1059538,0.1090186183],[3,1059796,0.0651786678],[3,1059935,0.1707441036],[3,1060104,0.1565680411],[3,1060209,0.0797804019],[3,1060223,0.075276087],[3,1060617,0.1793626691],[3,1061065,0.0626793482],[3,1061903,0.0639401861],[3,1062185,0.0541580606],[3,1062356,0.1220184155],[3,1063125,0.2145622569],[3,1063287,0.0973841727],[3,1063428,0.1339357863],[3,1063637,0.1157322893],[3,1063791,0.0906408386],[3,1064255,0.0851931768],[3,1064257,0.0897133129],[3,1064268,0.0692439018],[3,1119806,0.0971732373],[3,1119826,0.0685418524],[3,1120607,0.0825387505],[3,1120671,0.0922358563],[3,1120685,0.2255953589],[3,1120835,0.0714897542],[3,1120868,0.4869021538],[3,1120895,0.0691816273],[3,1120993,0.0924839974],[3,1121267,0.1468768006],[3,1121303,0.1749075957],[3,1121376,0.1044329815],[3,1121556,0.3209503705],[3,1121637,0.1652474781],[3,1121700,0.2286040287],[3,1121861,0.0691816273],[3,1121869,0.0711718029],[3,1122167,0.1594997049],[3,1122191,0.0770660859],[3,1122304,0.0699668738],[3,1122357,0.3712248061],[3,1122492,0.0699668738],[3,1123217,0.112001626],[3,1123272,0.159405097],[3,1123842,0.0691816273],[3,1124079,0.134190645],[3,1124194,0.0705793824],[3,1125196,0.1246092437],[3,1125252,0.1696370207],[3,1125347,0.1019361635],[3,1125944,0.0685418524],[3,1126017,0.0724522046],[3,1126044,0.0790166673],[3,1126053,0.2711696746],[3,1126055,0.2823973827],[3,1126153,0.146935681],[3,1126167,0.2133230249],[3,1126174,0.1645413114],[3,1126382,0.0756613762],[3,1126406,0.216446138],[3,1126650,0.0973394132],[3,1127179,0.213708732],[3,1127371,0.1061115698],[3,1127393,0.1641017088],[3,1127539,0.5188636097],[3,1127694,0.0695737811],[3,1127695,0.1078045479],[3,1127716,0.1274819055],[3,1127819,0.0728081586],[3,1127870,0.0699357222],[3,1127895,0.0804587394],[3,1128175,0.0907059137],[3,1128189,0.1896332399],[3,1128241,0.1720001818],[3,1128643,0.5065987638],[3,1128655,0.09267597],[3,1131803,0.3448448536],[3,1131808,0.2083280469],[3,1131813,0.9893018309],[3,1131816,0.3537435518],[3,1131819,0.8305038547],[3,1131826,0.2772562469],[3,1131842,0.3354671209],[3,1131843,0.677144548],[3,1131853,0.5166965015],[3,1131854,0.2479170801],[3,1131858,0.862144066],[3,1131861,0.3082674035],[3,1131863,0.7053388935],[3,1131867,0.4206715783],[3,1131868,0.2714066294],[3,1131876,0.7129297199],[3,1131881,0.3031704151],[3,1131892,0.503426958],[3,1131898,0.2806155165],[3,1131900,0.4491069178],[3,1131906,0.9998189765],[3,1131908,0.9869733717],[3,1131911,0.5556540868],[3,1131927,0.3411892904],[3,1131929,0.6676530466],[3,1131932,0.3198367649],[3,1131933,0.5639305801],[3,1131934,0.8849918409],[3,1131935,0.2889451725],[3,1131936,0.9893018309],[3,1131939,0.3816933562],[3,1131944,0.303051655],[3,1131951,0.9939682123],[3,1131953,0.5884445043],[3,1132128,0.4109021029],[3,1132129,0.3238389665],[3,1132130,0.3214023587],[3,1132139,0.6081975893],[3,1132147,0.3076848684],[3,1132148,0.3026030916],[3,1132152,0.3474598996],[3,1132155,0.2821254899],[3,1132157,0.5285269727],[3,1132159,0.4416315193],[3,1132170,0.7502761925],[3,1132182,0.5626572043],[3,1132189,0.7040923215],[3,1132193,0.584233961],[3,1132204,0.352059415],[3,1132208,0.7812208597],[3,1132218,0.5703099544],[3,1132224,0.7089838775],[3,1132227,0.447246549],[3,1132236,0.5994540807],[3,1132237,0.4000125312],[3,1132409,0.3172204332],[3,1132438,0.5798312886],[3,1132454,0.3054803717],[3,1132455,0.9869733717],[3,1132465,0.8692008255],[3,1132467,0.3115159985],[3,1132472,0.3189637444],[3,1132476,0.3523692866],[3,1132482,0.6045077102],[3,1132489,0.2678468047],[3,1132490,0.2517746344],[3,1132492,0.3093912206],[3,1132656,0.7542147228],[3,1132657,0.8353871326],[3,1132661,0.3856102376],[3,1132673,0.817904986],[3,1132675,0.3240392355],[3,1132690,0.2746681985],[3,1132692,0.3259350833],[3,1132698,0.5275569996],[3,1132859,0.568151795],[3,1132872,0.3731998377],[3,1132875,0.8721368583],[3,1132877,0.9939682123],[3,1132883,0.351649179],[3,1132885,0.5316011822],[3,1132889,0.6873299955],[3,1132893,0.7355145764],[3,1132900,0.9869733717],[3,1132913,0.3547608869],[3,1132917,0.5218626863],[3,1133065,0.731160643],[3,1133118,0.3490785899],[3,1133120,0.6975047259],[3,1133125,0.3954722649],[3,1133130,0.6392004861],[3,1133132,0.3502998185],[3,1133139,0.8441543274],[3,1133429,0.4243078108],[3,1133440,0.3835902436],[3,1133443,0.5318759812],[3,1133444,0.4312032985],[3,1133445,0.4168393834],[3,1133449,0.5371018563],[3,1133453,0.331116622],[3,1133465,0.56235374],[3,1133467,0.272352966],[3,1133611,0.2604381621],[3,1133617,0.4797642663],[3,1133620,0.5788050225],[3,1133624,0.8319489536],[3,1133635,0.4996765116],[3,1133642,0.3299081699],[3,1133791,0.5867535106],[3,1133809,0.5334637378],[3,1133815,0.6857550824],[3,1133818,0.7185214391],[3,1133819,0.6207198645],[3,1133966,0.3515109642],[3,1133970,0.3739724007],[3,1133989,0.3182474774],[3,1134109,0.4891400272],[3,1134114,0.8632504172],[3,1134116,0.7097529706],[3,1134124,0.3750594533],[3,1134131,0.7097529706],[3,1134248,0.5854030382],[3,1134251,0.5019364795],[3,1134356,0.9834865854],[3,1134366,0.9834865854],[3,1134380,0.9846480612],[3,1134385,0.4558196117],[3,1134387,0.9939682123],[3,1134390,0.290523441],[3,1134392,0.9834865854],[3,1134393,0.5126939466],[3,1134511,0.7054410265],[3,1134514,0.2993129539],[3,1134524,0.5984320293],[3,1134529,0.7241988281],[3,1134532,0.4867413855],[3,1134539,0.5415611148],[3,1134670,0.5210308345],[3,1134677,0.5016470005],[3,1134683,0.5675971889],[3,1134684,0.5598057203],[3,1134818,0.4770954278],[3,1134819,0.5570290178],[3,1134823,0.7565772387],[3,1135010,0.3728328286],[3,1135113,0.3600251581],[3,1135127,0.6206905271],[3,1135224,0.7479159912],[3,1135228,0.9893018309],[3,1135229,0.9881372075],[3,1135231,0.9776909793],[3,1135232,0.98116599],[3,1135233,0.9823258952],[3,1135235,0.9823258952],[3,1135239,0.9904672426],[3,1135241,0.98116599],[3,1135242,0.9893018309],[3,1135246,0.9858103232],[3,1135248,0.9788485326],[3,1135249,0.9846480612],[3,1135251,0.5652227485],[3,1135255,0.6641109491],[3,1135338,0.9834865854],[3,1135340,0.5934116978],[3,1135342,0.9846480612],[3,1135343,0.5384758148],[3,1135345,0.5200520377],[3,1135347,0.6144159915],[3,1135348,0.9823258952],[3,1135442,0.6757620443],[3,1135449,0.5493481472],[3,1135458,0.9800068693],[3,1135459,0.4566210547],[3,1135461,0.7002607262],[3,1135468,0.6737106237],[3,1135567,0.6147196113],[3,1135673,0.9881372075],[3,1135678,0.9273528138],[3,1135886,0.5003080144],[3,1135990,0.8396722106],[3,1135991,0.8367042948],[3,1135992,0.3218217498],[3,1135993,0.6315044613],[3,1135995,0.4617540534],[3,1135998,0.9423675447],[3,1136078,0.660824857],[3,1136321,0.9939682123],[3,1136324,0.6708259486],[3,1136328,0.4994517038],[3,1136330,0.5951923361],[3,1136333,0.5537519424],[3,1136411,0.517139511],[3,1136481,0.5112735502],[3,1136482,0.9869733717],[3,1136486,0.4606272294],[3,1136549,0.4964585831],[3,1136551,0.7147447635],[3,1136620,0.3952172685],[3,1136621,0.706130915],[3,1136623,0.6788419877],[3,1136689,0.4627558318],[3,1136695,0.6399839313],[3,1136696,0.9939682123],[3,1136774,0.5010135651],[3,1136781,0.7245765264],[3,1136967,0.2830738177],[3,1136975,0.7257673905],[3,1137188,0.4906025109],[3,1137189,0.6723823266],[3,1137309,0.8893755412],[3,1137312,0.3796256745],[3,1137394,0.7238649642],[3,1137809,0.631197758],[3,1137811,0.6746246616],[3,1137900,0.2717313206],[3,1138280,0.609245261],[3,1138901,0.52420026],[3,1138902,0.5340669045],[3,1138903,0.8688829261],[3,1138905,0.5746824325],[3,1138907,0.5252796868],[3,1138910,0.6318545716],[3,1138911,0.5778991221],[3,1138914,0.62678193],[3,1138921,0.5661960727],[3,1138924,0.551718492],[3,1138925,0.5364537422],[3,1138927,0.5362344056],[3,1138928,0.7064841005],[3,1138930,0.5241469935],[3,1138934,0.5763875739],[3,1138936,0.5457041392],[3,1138937,0.5157367205],[3,1138938,0.8743098187],[3,1138939,0.3856990595],[3,1138941,0.548868062],[3,1138942,0.7006539412],[3,1138943,0.5737884656],[3,1138944,0.5429755524],[3,1139110,0.5098013117],[3,1139181,0.8244862925],[3,1139286,0.6924993421],[3,1139430,0.6102490536],[3,1139433,0.437157368],[3,1139676,0.323907716],[3,1139907,0.6868015614],[3,1139986,0.9788485326],[3,1140967,0.5673342466],[3,1141446,0.8527119676],[3,1141505,0.8764857214],[3,1142232,0.66291647],[3,1142954,0.7448559911],[3,1143213,0.2828316464],[3,1143428,0.2561102651],[3,1144261,0.2733237511],[3,1144435,0.5800359243],[3,1144780,0.7637280819],[3,1145406,0.4956006683],[3,1145876,0.5248286543],[3,1145974,0.7937152016],[3,1148391,0.7771591311],[3,1148443,0.2699630446],[3,1148485,0.2994942682],[3,1148567,0.0790153039],[3,1149701,0.4080832686],[3,1150249,0.3231290496],[3,1150860,0.2663141377],[3,1151173,0.6220819001],[3,1152633,0.103788155],[3,1152660,0.1150800046],[3,1152749,0.8583998181],[3,1154032,0.2845144082],[3,1155917,0.1356271552],[3,1156331,0.7554181423],[3,1157113,0.1672086263],[3,1157444,0.6764714083],[3,1158116,0.5677962813],[3,1159670,0.3335448321],[3,1160338,0.1058774347],[3,1162975,0.6475493471],[3,1163339,1.0],[3,1163342,0.6473197422],[3,1163355,0.27005518],[3,1163514,0.8787712869],[3,1163516,0.3698175299],[3,1163917,0.6746567909],[3,1163925,0.7640778544],[3,1163976,0.4132658713],[3,1163989,0.640942202],[3,1163997,0.5042416731],[3,1163998,0.1970816645],[3,1166280,0.0990957528],[3,1166502,0.2683567293],[3,1166711,0.0781408919],[3,1166769,0.0818270036],[3,1166808,0.0819450313],[3,1167744,0.6077235317],[3,1167862,0.1661647364],[3,1168053,0.1801414673],[3,1168328,0.1664377977],[3,1168420,0.252419605],[3,1168601,0.1546290897],[3,1168714,0.2437940301],[3,1168811,0.1693435866],[3,1169121,0.1888652846],[3,1170061,0.1456148501],[3,1170120,0.4912829114],[3,1170154,0.1466280849],[3,1170177,0.8158268819],[3,1170215,0.1430455185],[3,1170231,0.2692553869],[3,1170272,0.181195147],[3,1170422,0.6478803463],[3,1170462,0.0886833122],[3,1170660,0.2133354848],[3,1170712,0.605866198],[3,1170722,0.8601809191],[3,1170738,0.6389368914],[3,1170753,0.6074689113],[3,1170796,0.2657434717],[3,1170921,0.2005021981],[3,1170963,0.1792931764],[3,1171160,0.1764858652],[3,1171712,0.151893056],[3,1171792,0.1674555406],[3,1171849,0.1462316829],[3,1171976,0.1414375661],[3,1172036,0.1875162697],[3,1186948,0.0904318902],[3,1191208,0.0814122132],[3,1192419,0.0769790186],[3,1194556,0.1823472652],[3,1194953,0.1025635299],[3,1195193,0.4646193445],[3,1195232,0.105840298],[3,1195271,0.0832132943],[3,1195514,0.1562413509],[3,1195861,0.1565101573],[3,1196060,0.1956449181],[3,1196432,0.1581011269],[3,1196648,0.1118402003],[3,1196904,0.0886833122],[3,1197303,0.1489329254],[3,1197637,0.7076254527],[3,1197722,0.3974859162],[3,1199370,0.0774617468],[3,1199910,0.0807777064],[3,1201255,0.0987716409],[3,1202184,0.1342725341],[3,1202412,0.1333810523],[3,1202413,0.0778624895],[3,1203238,0.28803639],[3,1203427,0.0832057522],[3,1203516,0.277768682],[3,1203783,0.1608504945],[3,1203882,0.1258291262],[3,1204142,0.0987716409],[3,1204407,0.3052577971],[3,1204409,0.0848401949],[3,1204499,0.1082500006],[3,1204591,0.1731407006],[3,1205418,0.1015089864],[3,1205589,0.2686061207],[3,1205590,0.0904318902],[3,1205973,0.0951645615],[3,1207292,0.2932753281],[3,1207293,0.1079927318],[3,1207522,0.0828367562],[3,1207720,0.2450625948],[3,1207956,0.232377172],[3,1208005,0.3295131904],[3,1208019,0.0855131164],[3,1208230,0.0828901688],[3,1208334,0.1194876089],[3,1210614,0.1062569736],[3,1216808,0.0970274139],[3,1217966,0.2659807428],[3,1218419,0.0789505716],[3,1218428,0.0814499687],[3,1218478,0.6387427745],[3,1218903,0.4696801205],[3,1219875,0.109489595],[3,1220106,0.1017446022],[3,1220306,0.1422077194],[3,1220620,0.1295918551],[3,1220862,0.2276626704],[3,1220944,0.0784547536],[3,1221544,0.0805513757],[3,1221885,0.1467412789],[3,1223819,0.0961783421],[3,1224341,0.2194295515],[3,1228559,0.1266941022],[3,1228741,0.5091833048],[3,1229512,0.0832054074],[3,1229623,0.4588856875],[3,1229652,0.1073068479],[3,1230187,0.2631835028],[3,1230991,0.079573047],[3,1235504,0.0813576812],[3,1237075,0.0777982176],[3,1247693,0.0777982176],[3,1254381,0.0795533408],[3,1254583,0.1304771986],[3,1255314,0.0824060502],[3,1255593,0.1141566246],[3,1255832,0.0954485083],[3,1255900,0.139938434],[3,1256086,0.0800829557],[3,1256272,0.0777982176],[3,1256440,0.1872327613],[3,1256441,0.6590593404],[3,1258800,0.0815117485],[3,1265964,0.1075021521],[3,1266284,0.0777982176],[3,1267131,0.1127149681],[3,1267212,0.1210762774],[3,1267312,0.0831044571],[3,1267740,0.1018562501],[3,1268822,0.1922056465],[3,1269181,0.1572963263],[3,1269832,0.1785348124],[3,1269836,0.1023336816],[3,1269870,0.0814170084],[3,1270195,0.1072569699],[3,1270783,0.2833834135],[3,1271038,0.3867318179],[3,1271604,0.1614054649],[3,1275357,0.0771515671],[3,1275483,0.0879520673],[3,1282762,0.0784547536],[3,1282833,0.1687958881],[3,1282834,0.1008943107],[3,1282835,0.0777982176],[3,1282843,0.1672771989],[3,1282867,0.238218565],[3,1283027,0.0843913163],[3,1283374,0.0777982176],[3,1283744,0.1841211848],[3,1284025,0.1611704661],[3,1284117,0.1450557899],[3,1284397,0.3390244377],[3,1284398,0.1248434251],[3,1285529,0.2395129991],[3,1285577,0.1214358878],[3,1285737,0.1484014584],[3,1287798,0.1838047236],[3,1288048,0.0824746396],[3,1288170,0.3187930479],[3,1288178,0.0779979115],[3,1288226,0.1266535546],[3,1288362,0.3386436191],[3,1288501,0.1053276775],[3,1288580,0.0771515671],[3,1288828,0.0777982176],[3,1290530,0.14735411],[3,1291419,0.0777982176],[3,1292276,0.0814499687],[3,1292774,0.106556483],[3,1293293,0.3066178541],[3,1327061,0.0774376194],[3,1395316,0.0892467755],[3,1406448,0.145095493],[3,1436012,0.0914360677],[3,1443419,0.0920941254],[3,1468240,0.1087481032],[3,1468408,0.1772470623],[3,1468461,0.1689921033],[3,1469974,0.1300079873],[3,1470059,0.2448694286],[3,1470290,0.1486839384],[3,1470352,0.1705800054],[3,1471342,0.3114835977],[3,1471967,0.1380120573],[3,1472211,0.3121083913],[3,1472257,0.4326830015],[3,1473053,0.1022569359],[3,1474433,0.1652496326],[3,1476713,0.1562997853],[3,1477443,0.0973166101],[3,1477515,0.1204481989],[3,1477516,0.0973166101],[3,1479914,0.2114827496],[3,1483312,0.1003755703],[3,1484805,0.1044009055],[3,1485092,0.1613742244],[3,1485103,0.1473047584],[3,1485340,0.0973166101],[3,1485679,0.1536745351],[3,1485923,0.3174219703],[3,1486077,0.2718360834],[3,1486094,0.3861964734],[3,1486123,0.341163634],[3,1486303,0.1459061089],[3,1486320,0.155584829],[3,1486435,0.1008747114],[3,1487219,0.1759555223],[3,1487448,0.1640486502],[3,1488206,0.0999627003],[3,1488491,0.2313913735],[3,1489471,0.0990271068],[3,1495905,0.1403573368],[3,1496277,0.1021219621],[3,1496884,0.0993887434],[3,1499044,0.1233883237],[3,1499095,0.3470569945],[3,1499781,0.1159373001],[3,1526647,0.110015509],[3,1537803,0.1236692697],[3,1546787,0.2181597258],[3,1552999,0.1294026113],[3,1628379,0.1391977486],[3,1631410,0.1803856649],[3,1631423,0.1167677158],[3,1631605,0.1223984166],[3,1636719,0.2126706062],[3,1654486,0.1754795317],[3,1683758,0.1243847118],[3,1685070,0.1178004478],[3,1701403,0.1256795808],[3,1702659,0.1932045403],[3,1719457,0.1841697302],[3,1726987,0.1322429833],[3,1727006,0.1480246828],[3,1727033,0.1931626043],[3,1727054,0.1558520118],[3,1727080,0.2048187693],[3,1727148,0.1465561639],[3,1727160,0.2910820819],[3,1727244,0.1304465032],[3,1727309,0.3614132805],[3,1727314,0.1874247145],[3,1727358,0.177615011],[3,1727405,0.156218727],[3,1731079,0.1835824927],[3,1740335,0.1319807923],[3,1741488,0.1503157574],[3,1741531,0.1889009971],[3,1748458,0.2032866148],[3,1757131,0.2724579909],[3,1758574,0.1300201239],[3,1758849,0.1285521385],[3,1758911,0.121371339],[3,1760045,0.126209461],[3,1760055,0.1222879155],[3,1760170,0.1947171102],[3,1760205,0.3097239123],[3,1760255,0.2184195049],[3,1760268,0.1935266654],[3,1762193,0.124268998],[3,1762506,0.1695875416],[3,1766370,0.1990479591],[3,1767234,0.2537268357],[3,1767422,0.2281380531],[3,1767423,0.3722031801],[3,1769293,0.121371339],[3,1769308,0.1665727001],[3,1769328,0.1468393362],[3,1769506,0.1518167016],[3,1769544,0.149804187],[3,1769621,0.319953778],[3,1772027,0.1281933128],[3,1778093,0.1992653371],[3,1778095,0.1952708745],[3,1778160,0.1952708745],[3,1778777,0.1992653371],[3,1779728,0.1785974759],[3,1782161,0.1696966521],[3,1784091,0.1581421232],[3,1784589,0.1923224567],[3,1785412,0.1294366766],[3,1793217,0.1708738528],[3,1793370,0.1297163447],[3,1793617,0.1740091768],[3,1793763,0.1795294989],[3,1793800,0.1812071209],[3,1793856,0.1899984757],[3,1793987,0.1977170242],[3,1794024,0.1909889856],[3,1794046,0.1962875747],[3,1794153,0.1466977655],[3,1794325,0.1333874077],[3,1794510,0.1562825247],[3,1794851,0.2158021137],[3,1794948,0.1582100128],[3,1795010,0.1437006967],[3,1795065,0.1578074463],[3,1795276,0.1443445851],[3,1795352,0.1803938774],[3,1795638,0.1306496172],[3,1795652,0.208004776],[3,1796095,0.126838168],[3,1796178,0.2052514252],[3,1796621,0.1678127083],[3,1796627,0.1283668829],[3,1796674,0.1332747081],[3,1796827,0.1745666235],[3,1796913,0.1643595257],[3,1797001,0.1404071306],[3,1797058,0.2106682252],[3,1797229,0.1477595257],[3,1797494,0.2332856864],[3,1797535,0.1938418089],[3,1797603,0.2074567971],[3,1797607,0.1799146494],[3,1797757,0.1929368083],[3,1798074,0.1707951561],[3,1798213,0.1539985199],[3,1798895,0.1862142392],[3,1799121,0.2071729942],[3,1799178,0.1484063965],[3,1799409,0.1823487502],[3,1799475,0.1580063022],[3,1799525,0.2062133034],[3,1799545,0.1892415492],[3,1799669,0.1415119233],[3,1800482,0.1835033022],[3,1800743,0.1585279604],[3,1804681,0.2213740702],[3,1812622,0.1901571394],[3,1812623,0.1346048438],[3,1812626,0.1456234475],[3,1812632,0.1415744666],[3,1815874,0.172120836],[3,1816056,0.1839179857],[3,1816103,0.2289257129],[3,1831148,0.1662465849],[3,1833513,0.1896119001],[3,1833515,0.170392322],[3,1833518,0.1887198621],[3,1833519,0.3074604091],[3,1833524,0.1452672293],[3,1833526,0.2039408829],[3,1833528,0.1769949998],[3,1833536,0.1688398909],[3,1833602,0.1696997602],[3,1839658,0.2761403396],[3,1841048,0.1303315578],[3,1842235,0.1687299614],[3,1842590,0.1680072246],[3,1844088,0.1463071802],[3,1844405,0.1792558333],[3,1844684,0.1447473159],[3,1845960,0.3079086687],[3,1846403,0.1404722993],[3,1847234,0.5321791533],[3,1847377,0.1332317686],[3,1848215,0.1579128463],[3,1848249,0.2013039366],[3,1848401,0.3955887079],[3,1848641,0.1899496038],[3,1849543,0.4247236447],[3,1849818,0.1281823593],[3,1851859,0.1351830056],[3,1852079,0.1672651504],[3,1852895,0.247033053],[3,1853172,0.3531711092],[3,1854788,0.1524341404],[3,1858045,0.214869517],[3,1859424,0.1750611078],[3,1860385,0.1359025698],[3,1862150,0.1281823593],[3,1862472,0.1352050195],[3,1867201,0.1351365374],[3,1872983,0.1893267776],[3,1873697,0.2162056215],[3,1873725,0.2398417034],[3,1873864,0.1621971787],[3,1874130,0.1558605122],[3,1875339,0.1511265902],[3,1876407,0.2056460761],[3,1876493,0.1526707546],[3,1879887,0.1482897728],[3,1880846,0.1532190707],[3,1881833,0.1577243511],[3,1882787,0.1512742732],[3,1887406,0.1813803617],[3,1889641,0.2034045555],[3,1890504,0.1785522782],[3,1890704,0.6084038206],[3,1890753,0.1464736917],[3,1893837,0.1526707546],[3,1902365,0.1452254593],[3,1903852,0.1456453927],[3,1905181,0.173529429],[3,1905362,0.2175036614],[3,1905381,0.200942793],[3,1906557,0.1533914097],[3,1906769,0.1975213811],[3,1908052,0.2610131665],[3,1908437,0.2981003841],[3,1909405,0.2984597875],[3,1909983,0.1755242409],[3,1910299,0.1970564175],[3,1910638,0.3523454355],[3,1912357,0.1915446828],[3,1912739,0.1515890887],[3,1916874,0.1442090232],[3,1919239,0.3786566797],[3,1923316,0.1534359509],[3,1923393,0.2215604493],[3,1923399,0.2391299388],[3,1923402,0.1848876291],[3,1923403,0.2143593023],[3,1924590,0.1442842901],[3,1925048,0.1569052951],[3,1929158,0.148463812],[3,1930407,0.1485394442],[3,1930689,0.1658014307],[3,1930788,0.1926444505],[3,1931951,0.1478722095],[3,1933376,0.1478722095],[3,1934749,0.1490644513],[3,1935255,0.2016333099],[3,1935464,0.150780346],[3,1937099,0.2684772354],[3,1937233,0.1528795386],[3,1937338,0.1632473046],[3,1937340,0.2437447885],[3,1937412,0.1647957652],[3,1940407,0.1523312817],[3,1940517,0.1561069152],[3,1940801,0.2497959717],[3,1940917,0.2292860581],[3,1942388,0.1684058245],[3,1943076,0.175319884],[3,1943135,0.1563023096],[3,1943177,0.1576751758],[3,1943946,0.1794191871],[3,1944885,0.1640696724],[3,1945092,0.1884321934],[3,1945116,0.1880109064],[3,1945321,0.1659176407],[3,1945349,0.2087339179],[3,1945736,0.5291744186],[3,1946290,0.189209072],[3,1947589,0.1597452855],[3,1947857,0.1881349264],[3,1947882,0.4361556222],[3,1947953,0.1592500744],[3,1948091,0.4350407363],[3,1948522,0.2450942061],[3,1948785,0.2851562997],[3,1949775,0.1579097741],[3,1950258,0.1509391104],[3,1953123,0.1789859602],[3,1954300,0.1540897859],[3,1956137,0.1719955995],[3,1956852,0.1550760452],[3,1957096,0.178062202],[3,1957839,0.227187045],[3,1958833,0.244792057],[3,1959496,0.2424496564],[3,1961090,0.226932627],[3,1961412,0.3666738626],[3,1962727,0.1630541808],[3,1962826,0.1808773653],[3,1963452,0.1934499047],[3,1964065,0.2372532353],[3,1964081,0.1939597788],[3,1964369,0.1602750037],[3,1964474,0.3590307462],[3,1965501,0.2167855873],[3,1965669,0.1780916539],[3,1966024,0.2247790716],[3,1966040,0.2226689203],[3,1966135,0.2070135618],[3,1966221,0.1770569673],[3,1966996,0.1618714187],[3,1968899,0.3498429346],[3,1969562,0.154999852],[3,1969820,0.1550760452],[3,1971181,0.1567965991],[3,1975232,0.1561890367],[3,1975868,0.1609069909],[3,1976061,0.2818898781],[3,1976437,0.1609069909],[3,1976640,0.1620025527],[3,1976968,0.2081069514],[3,1976997,0.2096976973],[3,1977026,0.2113963047],[3,1977033,0.196968294],[3,1977034,0.1663186355],[3,1977053,0.1591151269],[3,1977622,0.2238682289],[3,1977775,0.1904835658],[3,1977842,0.2755381594],[3,1978017,0.2089659255],[3,1978050,0.3329804912],[3,1979008,0.2639426431],[3,1979562,0.1789702663],[3,1980417,0.154999852],[3,1980891,0.1640154957],[3,1981452,0.1972400812],[3,1981462,0.1664407746],[3,1981600,0.3170889296],[3,1982349,0.4020257419],[3,1983100,0.162189324],[3,1986626,0.2844831587],[3,1988802,0.2817771062],[3,1989993,0.1543554268],[3,1991165,0.1546407407],[3,1994410,0.203172497],[3,1994566,0.2041700223],[3,1994648,0.1782398099],[3,1994763,0.1911903139],[3,1995255,0.1526527055],[3,1995354,0.376174191],[3,1996099,0.1573475276],[3,1996489,0.2219349532],[3,1997497,0.1946605395],[3,1997825,0.1993067326],[3,1997993,0.2552863762],[3,1998417,0.2908000108],[3,1999338,0.1576231797],[3,2000975,0.16276169],[3,2001235,0.4550845951],[3,2001261,0.1779245994],[3,2003930,0.1875227747],[3,2008595,0.1539114992],[3,2008761,0.1954142939],[3,2009624,0.1576231797],[3,2012444,0.1536028565],[3,2014423,0.4767476916],[3,2015391,0.1770026827],[3,2015802,0.2369690136],[3,2017355,0.33165658],[3,2017711,0.4648152688],[3,2019018,0.2506341795],[3,2019511,0.1612636159],[3,2020483,0.1834299764],[3,2020522,0.1846788086],[3,2020983,0.2087408658],[3,2020984,0.2400876338],[3,2020986,0.3518826411],[3,2021000,0.2122125065],[3,2021175,0.2044325873],[3,2021271,0.4936320856],[3,2021277,0.2571033719],[3,2021279,0.2232830209],[3,2021302,0.1929106465],[3,2021619,0.7682710717],[3,2021643,0.2158331744],[3,2021898,0.3198219439],[3,2022247,0.3471797381],[3,2022467,0.1853842411],[3,2022532,0.2957681857],[3,2023551,0.1526527055],[3,2023767,0.2329287654],[3,2023959,0.2080642268],[3,2025116,0.2656715391],[3,2025385,0.320000815],[3,2027835,0.2371946823],[3,2031949,0.2436768641],[3,2032161,0.1721406029],[3,2032865,0.1613111669],[3,2034356,0.201397227],[3,2036057,0.1585664945],[3,2037238,0.1683455756],[3,2037525,0.1514556114],[3,2038151,0.23859802],[3,2038334,0.1508741467],[3,2038660,0.1588581972],[3,2039137,0.1614565335],[3,2048471,0.1582844257],[3,2049277,0.2177141406],[3,2049699,0.1602534794],[3,2049754,0.1983423834],[3,2049756,0.1952147792],[3,2052613,0.1566788967],[3,2052741,0.2817662119],[3,2053717,0.1934150831],[3,2060628,0.1825303825],[3,2060629,0.3375525603],[3,2060633,0.2231268469],[3,2062001,0.2253775162],[3,2064572,0.270016032],[3,2064640,0.2260321809],[3,2064830,0.1688238754],[3,2064874,0.1514556114],[3,2065445,0.1561702837],[3,2065622,0.1596428625],[3,2065837,0.2559220986],[3,2065982,0.3979335645],[3,2065998,0.3294658235],[3,2066168,0.2087873686],[3,2066218,0.1971396314],[3,2066227,0.2491482225],[3,2066658,0.2248418105],[3,2068259,0.1978567604],[3,2068353,0.2532578453],[3,2069653,0.3197803913],[3,2069730,0.3043532301],[3,2070154,0.2033829557],[3,2070316,0.2752875829],[3,2071678,0.1944860725],[3,2071739,0.24151112],[3,2071882,0.1986318486],[3,2072111,0.3209119833],[3,2072374,0.4536458347],[3,2072724,0.1892584654],[3,2073421,0.1611607022],[3,2074054,0.459014745],[3,2074254,0.1796950025],[3,2074266,0.1821577295],[3,2075389,0.1899635966],[3,2075425,0.1932446021],[3,2076631,0.1730155198],[3,2077162,0.1801127552],[3,2077645,0.1622618903],[3,2077653,0.1732032096],[3,2078000,0.2194857099],[3,2078196,0.367134443],[3,2078260,0.1509456569],[3,2079182,0.1623648471],[3,2080879,0.1916018322],[3,2082792,0.1514556114],[3,2086590,0.1524044775],[3,2091010,0.3600182109],[3,2091025,0.2682406052],[3,2091925,0.1648796608],[3,2092094,0.1508741467],[3,2092193,0.2241159369],[3,2092522,0.3517865071],[3,2092696,0.1595917933],[3,2092880,0.5551670428],[3,2093417,0.1921760063],[3,2093780,0.2511318909],[3,2094161,0.3139046523],[3,2098449,0.4075163895],[3,2098673,0.2160977576],[3,2099101,0.192315549],[3,2100245,0.1655418644],[3,2100625,0.1660756739],[3,2102397,0.1803276333],[3,2102644,0.6843278941],[3,2103374,0.1594012078],[3,2103442,0.1507501868],[3,2103517,0.1685604632],[3,2103580,0.1835937364],[3,2104181,0.2538215703],[3,2105124,0.1838981547],[3,2122369,0.1613978357],[3,2124269,0.1583538688],[3,2124799,0.2136421143],[3,2125440,0.1798155137],[3,2125746,0.3448217811],[3,2126224,0.1788689303],[3,2126334,0.1663479411],[3,2129701,0.3260698939],[3,2131636,0.3769119785],[3,2132786,0.2324693646],[3,2138850,0.300541538],[3,2138900,0.1902155897],[3,2139932,0.1571194088],[3,2156842,0.161332998],[3,2158943,0.1625744927],[3,2189035,0.1732658071],[3,2198804,0.1993744392],[3,2238439,0.2061837903],[3,2240239,0.2496154875],[3,2240259,0.1695202355],[3,2242664,0.2635879163],[3,2242666,0.2847076623],[3,2242829,0.2011176069],[3,2243625,0.2174354274],[3,2243878,0.2032153723],[3,2243995,0.3642486968],[3,2243999,0.4833106063],[3,2244200,0.1667550584],[3,2244208,0.1874903616],[3,2244801,0.1741855354],[3,2244888,0.2797037707],[3,2245352,0.2544295488],[3,2245392,0.288178693],[3,2246585,0.1893155624],[3,2247079,0.2104979297],[3,2247281,0.1985523209],[3,2250970,0.1976841707],[3,2253491,0.2310364549],[3,2258130,0.1723664799],[3,2269535,0.1890429602],[3,2278890,0.1644348242],[3,2280583,0.2044582474],[3,2281236,0.1768441538],[3,2281340,0.214056157],[3,2281644,0.1711427277],[3,2282322,0.2083723966],[3,2284090,0.1664185104],[3,2285029,0.1842849277],[3,2289461,0.2218933195],[3,2290534,0.1949030072],[3,2291313,0.3031101489],[3,2293232,0.1662046792],[3,2294985,0.1763502088],[3,2295720,0.2065756588],[3,2295834,0.1765719274],[3,2299477,0.1751443425],[3,2299911,0.1956422078],[3,2301562,0.1695407026],[3,2303905,0.1692410649],[3,2304296,0.1689473429],[3,2304298,0.1695407026],[3,2306891,0.1766156415],[3,2307698,0.1745555257],[3,2307898,0.177068476],[3,2308773,0.1678222913],[3,2308800,0.2078625928],[3,2308940,0.2350895044],[3,2309564,0.2501408467],[3,2310214,0.1895683883],[3,2310455,0.2420613617],[3,2311053,0.2250061983],[3,2314117,0.1683375723],[3,2314525,0.1689305307],[3,2314643,0.1760257527],[3,2314796,0.2857521718],[3,2315236,0.1797041922],[3,2315353,0.1746576775],[3,2315654,0.3327886373],[3,2315683,0.1951562608],[3,2315997,0.3812245505],[3,2316109,0.2488535892],[3,2316568,0.1686310957],[3,2317821,0.1874869617],[3,2318202,0.2775192277],[3,2318275,0.2098678925],[3,2318861,0.1874808691],[3,2318907,0.1764046955],[3,2319005,0.1934639571],[3,2319008,0.1847353859],[3,2319045,0.2042200804],[3,2319127,0.1948240116],[3,2319128,0.1760257527],[3,2319183,0.2001463061],[3,2319221,0.195014382],[3,2319589,0.4363463093],[3,2319689,0.1748826293],[3,2320861,0.5845681842],[3,2320977,0.2490695876],[3,2322022,0.1689305307],[3,2322124,0.1700443328],[3,2325301,0.1799527554],[3,2326190,0.2209005997],[3,2326191,0.2199033492],[3,2326210,0.2209005997],[3,2326220,0.215213491],[3,2326232,0.2511266057],[3,2326241,0.2150797829],[3,2326274,0.2505639746],[3,2326278,0.2197107152],[3,2326280,0.2467807395],[3,2326282,0.2127797727],[3,2326286,0.2357736093],[3,2328540,0.2893452571],[3,2329940,0.1838869684],[3,2330236,0.1889596471],[3,2331326,0.2253227309],[3,2331337,0.2423853185],[3,2331361,0.2193942846],[3,2331577,0.2518952702],[3,2331685,0.2178754283],[3,2331880,0.2174487072],[3,2331923,0.2223680609],[3,2333140,0.1760714441],[3,2333262,0.1717006127],[3,2333852,0.2149948511],[3,2334600,0.1943510752],[3,2334764,0.1717006127],[3,2334765,0.1948107418],[3,2335950,0.1819661351],[3,2336735,0.1717006127],[3,2337028,0.1985536859],[3,2337266,0.1943128735],[3,2337608,0.2230471788],[3,2337711,0.5683133107],[3,2338131,0.5093971254],[3,2338437,0.3692294381],[3,2339154,0.173958467],[3,2339550,0.1717006127],[3,2342759,0.1788176357],[3,2342794,0.1797958697],[3,2344261,0.3094207026],[3,2344283,0.1792372099],[3,2344448,0.1881584255],[3,2344484,0.2054600695],[3,2344606,0.1792372099],[3,2344851,0.2650556121],[3,2345294,0.2119120626],[3,2345476,0.1870355518],[3,2345849,0.2034478336],[3,2346657,0.2774187638],[3,2346737,0.3338543632],[3,2348459,0.5936979239],[3,2349340,0.2626052811],[3,2349621,0.2183841783],[3,2349998,0.1808337042],[3,2350010,0.2160153984],[3,2350115,0.1798543575],[3,2352158,0.1798543575],[3,2355453,0.1987162421],[3,2356375,0.1811518307],[3,2356396,0.1908446875],[3,2356536,0.1864984723],[3,2357256,0.2255858038],[3,2357323,0.6599525513],[3,2357477,0.2780761748],[3,2357541,0.2170572938],[3,2357721,0.2710209716],[3,2357870,0.1798543575],[3,2357952,0.4054370577],[3,2358429,0.2052028453],[3,2359104,0.2131556011],[3,2359162,0.2065895519],[3,2360345,0.2002486925],[3,2360487,0.2129056393],[3,2360670,0.1987162421],[3,2361467,0.1879367434],[3,2361934,0.2103387702],[3,2366615,0.1792372099],[3,2368589,0.1792372099],[3,2369260,0.3354640614],[3,2369525,0.3205062391],[3,2370300,0.2315962601],[3,2370546,0.1792372099],[3,2370761,0.3705959827],[3,2371562,0.2011136346],[3,2371607,0.3105304179],[3,2372030,0.248293447],[3,2374990,0.2539650054],[3,2377298,0.2686791449],[3,2377974,0.1776195364],[3,2378093,0.1827493711],[3,2378095,0.2054236269],[3,2378106,0.2701802354],[3,2381118,0.2160128974],[3,2386376,0.1902375618],[3,2394254,0.2281835669],[3,2396417,0.1861015033],[3,2398455,0.2057315101],[3,2399312,0.1842135609],[3,2401037,0.2715404199],[3,2403371,0.2526810405],[3,2404839,0.2129878648],[3,2407042,0.2015677989],[3,2410355,0.1820585009],[3,2417547,0.1846371766],[3,2419544,0.187905857],[3,2430389,0.2024300753],[3,2434183,0.183297034],[3,2435118,0.1905460418],[3,2439390,0.1788012817],[3,2439778,0.1857762427],[3,2445777,0.179508964],[3,2446882,0.1789872936],[3,2448162,0.1914430912],[3,2451276,0.1901002964],[3,2451429,0.2679912705],[3,2451515,0.2100922006],[3,2451679,0.1754668188],[3,2452419,0.2168063955],[3,2453386,0.3520017887],[3,2453477,0.2223872351],[3,2454666,0.3763712476],[3,2455844,0.2252179179],[3,2456690,0.2413777338],[3,2456939,0.3067897895],[3,2457906,0.243226355],[3,2458118,0.2727434316],[3,2458185,0.1909213677],[3,2460263,0.1909213677],[3,2460584,0.1976694332],[3,2460772,0.2102811002],[3,2461812,0.1887813096],[3,2461868,0.1892156985],[3,2462984,0.2281765165],[3,2463218,0.2580222025],[3,2463220,0.1795735235],[3,2463989,0.1952834134],[3,2464857,0.2645907495],[3,2467046,0.2115593438],[3,2469794,0.1910090554],[3,2471131,0.1869943755],[3,2471191,0.1882554165],[3,2478544,0.1754902883],[3,2479829,0.1924723329],[3,2481482,0.2358952726],[3,2481928,0.1907310332],[3,2482435,0.1974047988],[3,2483799,0.1959781534],[3,2484596,0.2113591421],[3,2484705,0.2033986951],[3,2484922,0.2066691029],[3,2485276,0.2035192198],[3,2485713,0.2074263176],[3,2485835,0.1935717533],[3,2485926,0.2506454414],[3,2487340,0.3638920285],[3,2487564,0.1935717533],[3,2487623,0.3829353404],[3,2489600,0.2139250548],[3,2490322,0.2512405526],[3,2490359,0.4674952754],[3,2490660,0.1993385952],[3,2492005,0.1892050106],[3,2492033,0.1991818645],[3,2492639,0.2091966212],[3,2492780,0.2198614508],[3,2495187,0.2039495191],[3,2495955,0.2213228132],[3,2497884,0.3811372668],[3,2501794,0.1970147267],[3,2501847,0.1954696215],[3,2505972,0.2585872777],[3,2506699,0.2413973403],[3,2508918,0.4288963769],[3,2509597,0.2390168514],[3,2510183,0.2325845552],[3,2511080,0.2189504369],[3,2511090,0.2360694066],[3,2511181,0.2504499311],[3,2511246,0.2657637031],[3,2511481,0.2269904723],[3,2511553,0.2190438773],[3,2511688,0.4198068312],[3,2513162,0.1966815963],[3,2513170,0.2417824413],[3,2513868,0.1981253879],[3,2518195,0.3436564693],[3,2519674,0.2283974185],[3,2521127,0.2025638248],[3,2522002,0.1970147267],[3,2523454,0.2010699958],[3,2528868,0.269026237],[3,2536143,0.2098521706],[3,2537806,0.2105602642],[3,2539212,0.249242051],[3,2539404,0.2060603838],[3,2540085,0.6177519439],[3,2540138,0.5606455909],[3,2540160,0.2677747991],[3,2540898,0.1896206997],[3,2543699,0.1763494978],[3,2543749,0.1974385306],[3,2544793,0.1896206997],[3,2544867,0.2413795286],[3,2545762,0.1821508499],[3,2547255,0.182319718],[3,2547422,0.3813972461],[3,2547914,0.3120861176],[3,2548475,0.1959630651],[3,2549703,0.1986366193],[3,2557736,0.1930709656],[3,2558174,0.1891036432],[3,2559992,0.2264847164],[3,2559996,0.1915329364],[3,2560073,0.1926244788],[3,2565787,0.1871954701],[3,2565966,0.1933198139],[3,2566085,0.1886626687],[3,2566275,0.1837656808],[3,2566277,0.1886300194],[3,2567589,0.1852423189],[3,2568303,0.1907468248],[3,2568968,0.1938018206],[3,2570093,0.2009534559],[3,2570741,0.1710704065],[3,2570750,0.3007674866],[3,2571525,0.1803544998],[3,2572408,0.1862204279],[3,2572460,0.1814855902],[3,2573499,0.1954162502],[3,2574367,0.1960329524],[3,2574384,0.2180328037],[3,2574411,0.1913631768],[3,2574443,0.5368712121],[3,2574482,0.3341670478],[3,2574562,0.3845766208],[3,2574586,0.2522270446],[3,2574621,0.2965582571],[3,2574622,0.5032060942],[3,2574628,0.2787923863],[3,2574636,0.235707325],[3,2574658,0.2678234052],[3,2574675,0.260596003],[3,2574677,0.2022571865],[3,2574679,0.524959582],[3,2574692,0.4225550243],[3,2574724,0.2036531203],[3,2574758,0.5447768758],[3,2574767,0.3419855673],[3,2574768,0.5753750598],[3,2574777,0.4450878623],[3,2574779,0.2229395311],[3,2574781,0.6418152302],[3,2574784,0.1952416695],[3,2574829,0.1969100729],[3,2574831,0.4004340291],[3,2574834,0.6473196861],[3,2574853,0.2942247915],[3,2574855,0.2551640251],[3,2574861,0.1879317733],[3,2574863,0.4223787848],[3,2574864,0.2827468578],[3,2574867,0.5763493102],[3,2574874,0.4324803042],[3,2574876,0.5642963714],[3,2574878,0.1932266332],[3,2574888,0.1850513169],[3,2574892,0.3130371227],[3,2574903,0.3880612046],[3,2574904,0.2302333259],[3,2574924,0.311925328],[3,2574925,0.1942921424],[3,2574929,0.3094830214],[3,2574930,0.6933300839],[3,2574934,0.3954812611],[3,2574936,0.2418435327],[3,2574943,0.3275310626],[3,2574947,0.5540190729],[3,2574967,0.4198329877],[3,2574972,0.2383877845],[3,2574973,0.4243392812],[3,2574974,0.2900825644],[3,2574976,0.6328877637],[3,2574986,0.2707342553],[3,2574988,0.5536560362],[3,2574997,0.4613701301],[3,2575010,0.1833729491],[3,2575012,0.4900755994],[3,2575013,0.1885678113],[3,2575032,0.3091831378],[3,2575034,0.301010322],[3,2575037,0.2959435821],[3,2575038,0.4335118003],[3,2575050,0.4871976148],[3,2575054,0.3136044638],[3,2575066,0.2521381117],[3,2575090,0.3439103526],[3,2575102,0.1856618621],[3,2575103,0.1854593404],[3,2575106,0.3009994699],[3,2575122,0.2590038207],[3,2575142,0.4570588836],[3,2575161,0.2935546983],[3,2575167,0.2555053491],[3,2575191,0.2922342756],[3,2575198,0.2248349535],[3,2575200,0.2703850027],[3,2575222,0.4418807749],[3,2575224,0.3701008116],[3,2575225,0.3910992351],[3,2575226,0.6570783914],[3,2575228,0.1846300296],[3,2575229,0.1839768939],[3,2575232,0.4232793614],[3,2575234,0.4560752229],[3,2575238,0.3307123584],[3,2575251,0.5498692334],[3,2575263,0.3939362659],[3,2575264,0.4199547906],[3,2575268,0.5566893943],[3,2575269,0.4451700156],[3,2575280,0.2023549751],[3,2575282,0.6146114291],[3,2575295,0.2034198412],[3,2575296,0.2712772541],[3,2575297,0.3339414704],[3,2575298,0.3435832501],[3,2575299,0.4701150354],[3,2575300,0.4267970275],[3,2575303,0.7311214554],[3,2575305,0.3024526689],[3,2575307,0.3901686504],[3,2575308,0.2508475114],[3,2575309,0.3311710333],[3,2575310,0.1904307583],[3,2575311,0.1925961666],[3,2575312,0.4731983862],[3,2575313,0.3464301107],[3,2575322,0.5783979179],[3,2575336,0.3745308556],[3,2575340,0.5064199662],[3,2575346,0.2278403712],[3,2575347,0.2381904382],[3,2575357,0.2600704755],[3,2575358,0.3021180919],[3,2575361,0.3932835119],[3,2575363,0.1916556684],[3,2575364,0.3194710648],[3,2575365,0.4924204678],[3,2575367,0.2688270548],[3,2575370,0.3219887299],[3,2575371,0.1974237245],[3,2575375,0.1859840364],[3,2575377,0.4163385195],[3,2575382,0.2115749539],[3,2575383,0.2385932654],[3,2575384,0.357978789],[3,2575388,0.2221693066],[3,2575389,0.5660818491],[3,2575391,0.1815805418],[3,2575392,0.2594275685],[3,2575395,0.5665390753],[3,2575396,0.4190158257],[3,2575404,0.3328176142],[3,2575407,0.4323926388],[3,2575415,0.2707581261],[3,2575417,0.3225621908],[3,2575445,0.4856243019],[3,2575465,0.2887831387],[3,2575466,0.4988289566],[3,2575468,0.7824745427],[3,2575478,0.6268534523],[3,2575479,0.1889218019],[3,2575480,0.189458536],[3,2575481,0.2830318682],[3,2575483,0.5818989483],[3,2575484,0.4019786292],[3,2575489,0.5285192021],[3,2575498,0.3632390544],[3,2575510,0.3495251984],[3,2575528,0.336816912],[3,2575531,0.309747462],[3,2575532,0.1841963112],[3,2575536,0.4493632269],[3,2575537,0.5619568004],[3,2575543,0.1770316305],[3,2575553,0.2383654999],[3,2575554,0.2583659338],[3,2575556,0.2974235553],[3,2575573,0.3938118542],[3,2575576,0.2342464862],[3,2575577,0.295505587],[3,2575585,0.2360228045],[3,2575591,0.3392003052],[3,2575593,0.2892244398],[3,2575594,0.3240917852],[3,2575595,0.2470309796],[3,2575596,0.3644983605],[3,2575597,0.4699359993],[3,2575606,0.2823035976],[3,2575607,0.2117701318],[3,2575610,0.1888674854],[3,2575638,0.5096533774],[3,2575642,0.2710164419],[3,2575646,0.3275959771],[3,2575688,0.4177589663],[3,2575689,0.3964772673],[3,2575690,0.2711763904],[3,2575691,0.3141855617],[3,2575693,0.1759592284],[3,2575694,0.2043784537],[3,2575695,0.4348421884],[3,2575698,0.2349923903],[3,2575729,0.3310053186],[3,2575732,0.2568461059],[3,2575736,0.3998007688],[3,2575737,0.1931209258],[3,2575760,0.3746676794],[3,2575763,0.2023018422],[3,2575779,0.4143520867],[3,2575780,0.3563756745],[3,2575820,0.195573741],[3,2575822,0.6049821245],[3,2575823,0.2480816696],[3,2575825,0.3917546271],[3,2575826,0.2849138424],[3,2575828,0.3182661746],[3,2575829,0.24156591],[3,2575830,0.2536294685],[3,2575846,0.2806175527],[3,2575850,0.2183442362],[3,2575909,0.2402649092],[3,2575912,0.2687353661],[3,2575913,0.2300723399],[3,2575921,0.5714251818],[3,2575924,0.1958904643],[3,2575926,0.3896782072],[3,2575929,0.460031552],[3,2575930,0.5675254784],[3,2575941,0.3350722379],[3,2575945,0.1898056912],[3,2575947,0.1898225256],[3,2575960,0.4848566637],[3,2575965,0.3423559946],[3,2575983,0.2672671202],[3,2575987,0.5488939013],[3,2575988,0.3291281538],[3,2575989,0.2844529803],[3,2575990,0.2299191114],[3,2575991,0.3100036836],[3,2575992,0.1890389661],[3,2575993,0.2090157238],[3,2576005,0.3354572978],[3,2576022,0.4832825818],[3,2576033,0.2113426632],[3,2576037,0.1717909971],[3,2576044,0.3866339748],[3,2576046,0.2397243753],[3,2576047,0.365299924],[3,2576048,0.3558201339],[3,2576049,0.4118560919],[3,2576062,0.3555158545],[3,2576090,0.4089234703],[3,2576093,0.319491671],[3,2576095,0.3128352137],[3,2576108,0.1846426931],[3,2576109,0.3284787635],[3,2576124,0.3841324277],[3,2576125,0.2011283544],[3,2576127,0.6007435485],[3,2576130,0.3291987679],[3,2576131,0.2515705914],[3,2576133,0.1989576173],[3,2576134,0.4041468436],[3,2576135,0.5180761298],[3,2576149,0.326396926],[3,2576151,0.3720366217],[3,2576154,0.3535830336],[3,2576156,0.24663718],[3,2576157,0.3093456488],[3,2576159,0.3033106327],[3,2576162,0.2174837155],[3,2576163,0.3355201217],[3,2576164,0.2536172691],[3,2576166,0.5892369768],[3,2576167,0.2293766332],[3,2576169,0.3220926256],[3,2576170,0.3340056481],[3,2576172,0.2578635864],[3,2576175,0.501365965],[3,2576178,0.3412670784],[3,2576179,0.1966356042],[3,2576180,0.2479176626],[3,2576181,0.5738830729],[3,2576182,0.3597891089],[3,2576184,0.2457414081],[3,2576185,0.2284167947],[3,2576187,0.7008866227],[3,2576188,0.3126774488],[3,2576189,0.5878624109],[3,2576190,0.1855045564],[3,2576191,0.2732261796],[3,2576195,0.3008487659],[3,2576196,0.6738527865],[3,2576197,0.2633461054],[3,2576198,0.287130584],[3,2576199,0.4271425519],[3,2576200,0.2176512932],[3,2576213,0.7149557388],[3,2576239,0.3470035035],[3,2576256,0.2802666215],[3,2576258,0.1892450509],[3,2576259,0.2684575978],[3,2576260,0.5958094735],[3,2576263,0.3069067849],[3,2576264,0.242762158],[3,2576265,0.1923570132],[3,2576266,0.231384217],[3,2576268,0.3391571238],[3,2576271,0.1928658852],[3,2576272,0.6000114932],[3,2576274,0.2522722701],[3,2576275,0.2878316728],[3,2576276,0.298149841],[3,2576277,0.4449920452],[3,2576278,0.3148581517],[3,2576279,0.3199697842],[3,2576280,0.1963522408],[3,2576281,0.2659524845],[3,2576282,0.6132769955],[3,2576283,0.2249561376],[3,2576284,0.5051310113],[3,2576286,0.1869659848],[3,2576287,0.3479213462],[3,2576288,0.3404346465],[3,2576289,0.1887296153],[3,2576290,0.3145668988],[3,2576291,0.2875646009],[3,2576292,0.3927710103],[3,2576293,0.3214573477],[3,2576294,0.2564792874],[3,2576295,0.2314917411],[3,2576296,0.5760971369],[3,2576297,0.2554828416],[3,2576298,0.3751526081],[3,2576299,0.5831365829],[3,2576300,0.3632579358],[3,2576301,0.2348032309],[3,2576302,0.596181687],[3,2576303,0.3585992211],[3,2576305,0.29785497],[3,2576306,0.295359311],[3,2576307,0.3095381874],[3,2576308,0.2864537164],[3,2576309,0.3665068092],[3,2576310,0.1878778904],[3,2576311,0.1892883424],[3,2576312,0.2150350586],[3,2576317,0.2347484445],[3,2576319,0.3602695822],[3,2576320,0.2885628486],[3,2576321,0.4937475629],[3,2576323,0.2613298001],[3,2576325,0.1952426881],[3,2576327,0.2443011084],[3,2576328,0.2614855186],[3,2576329,0.2306083649],[3,2576330,0.2825732376],[3,2576331,0.2652203776],[3,2576332,0.4317230843],[3,2576333,0.1917829195],[3,2576334,0.4011214521],[3,2576335,0.2472944838],[3,2576336,0.1873993923],[3,2576337,0.5458512706],[3,2576338,0.2837426761],[3,2576339,0.2702063828],[3,2576341,0.3217589203],[3,2576342,0.2381217158],[3,2576344,0.409518148],[3,2576345,0.2280452761],[3,2576346,0.2823781628],[3,2576347,0.345974794],[3,2576348,0.1958404972],[3,2576349,0.2477155754],[3,2576350,0.3515227459],[3,2576351,0.2680748951],[3,2576352,0.2644338455],[3,2576353,0.5759285122],[3,2576354,0.3425386039],[3,2576356,0.2749815849],[3,2576357,0.3200226931],[3,2576358,0.278405302],[3,2576370,0.5232838687],[3,2576371,0.6436058745],[3,2576373,0.3114443686],[3,2576374,0.2911124633],[3,2576375,0.2568374184],[3,2576378,0.432870335],[3,2576379,0.2793911975],[3,2576380,0.3313684018],[3,2576381,0.4266751399],[3,2576382,0.4372109992],[3,2576383,0.5887858423],[3,2576384,0.2262321015],[3,2576387,0.2775930073],[3,2576388,0.2243666536],[3,2576390,0.3110433645],[3,2576403,0.6004684487],[3,2576406,0.4396928936],[3,2576407,0.5935633293],[3,2576411,0.5598351736],[3,2576412,0.4813480955],[3,2576418,0.1856623061],[3,2576419,0.4557854579],[3,2576420,0.4872984545],[3,2576421,0.4261748566],[3,2576422,0.5364218803],[3,2576423,0.4554505537],[3,2576424,0.4536196655],[3,2576425,0.6652122143],[3,2576426,0.3981415761],[3,2576427,0.3640608529],[3,2576428,0.5424802287],[3,2576429,0.5075912705],[3,2576431,0.4086841689],[3,2576432,0.7482501984],[3,2576433,0.3083997302],[3,2576456,0.5124394896],[3,2576457,0.4424059155],[3,2576458,0.392532627],[3,2576459,0.4134308022],[3,2576460,0.4584542397],[3,2576462,0.441150867],[3,2576465,0.4434874382],[3,2576466,0.652694901],[3,2576467,0.5418257747],[3,2576468,0.1968326619],[3,2576469,0.4049207359],[3,2576475,0.5156916809],[3,2576490,0.2328595578],[3,2576497,0.2084160834],[3,2576503,0.4020831022],[3,2576505,0.3072832011],[3,2576508,0.5488648362],[3,2576513,0.4640263817],[3,2576521,0.2146309904],[3,2576525,0.491540533],[3,2576530,0.5435702823],[3,2576540,0.2305517899],[3,2576544,0.4422136838],[3,2576548,0.187824071],[3,2576579,0.6039387245],[3,2576580,0.6620574623],[3,2576582,0.5734590928],[3,2576583,0.3095812315],[3,2576586,0.5118062545],[3,2576588,0.4886255616],[3,2576589,0.4256549864],[3,2576591,0.6265037147],[3,2576593,0.3822723929],[3,2576594,0.3045678906],[3,2576604,0.2234620015],[3,2576611,0.4248842489],[3,2576612,0.4400063653],[3,2576616,0.3721170926],[3,2576628,0.3033482903],[3,2576629,0.3579619479],[3,2576630,0.4817407618],[3,2576632,0.560838796],[3,2576640,0.5037768663],[3,2576653,0.6027886477],[3,2576656,0.5045924729],[3,2576660,0.2133705235],[3,2576675,0.446299979],[3,2576689,0.6845959986],[3,2576690,0.6714945875],[3,2576699,0.3978994871],[3,2576702,0.2330859012],[3,2576708,0.6482408675],[3,2576710,0.5206450881],[3,2576717,0.2043960232],[3,2576719,0.4709351083],[3,2576722,0.3629161614],[3,2576732,0.4825548169],[3,2576828,0.3399331776],[3,2576856,0.4290742881],[3,2576867,0.2151337949],[3,2576877,0.6557179754],[3,2576878,0.456121177],[3,2576893,0.3152046422],[3,2576921,0.3918830646],[3,2576965,0.2127295552],[3,2576967,0.2161786548],[3,2576983,0.4837498812],[3,2576985,0.5882183705],[3,2576998,0.2022481535],[3,2577001,0.2002111509],[3,2577011,0.350266639],[3,2577024,0.5408833277],[3,2577049,0.6837828956],[3,2577050,0.6011628147],[3,2577056,0.3587426614],[3,2577091,0.4607881032],[3,2577105,0.358443591],[3,2577118,0.3894858085],[3,2577141,0.2650922788],[3,2577142,0.2010802865],[3,2577154,0.3191268565],[3,2577168,0.7090574429],[3,2577170,0.471640186],[3,2577171,0.3219316327],[3,2577172,0.8070524794],[3,2577174,0.4681200212],[3,2577175,0.4864676397],[3,2577176,0.3758679524],[3,2577177,0.4373073822],[3,2577195,0.4836137831],[3,2577196,0.583402816],[3,2577197,0.7059757883],[3,2577198,0.4494247662],[3,2577209,0.557334979],[3,2577232,0.3793259331],[3,2577253,0.4157929563],[3,2577254,0.2404761228],[3,2577274,0.3575882276],[3,2577317,0.2649081698],[3,2577348,0.4970962656],[3,2577422,0.1844095184],[3,2577463,0.5164920908],[3,2577527,0.6183595754],[3,2577548,0.1993010649],[3,2577615,0.3564568696],[3,2577642,0.3020726498],[3,2577668,0.8226877926],[3,2578214,0.20173712],[3,2578215,0.4255938012],[3,2578299,0.2145900268],[3,2578314,0.3758327081],[3,2578405,0.1839097929],[3,2578446,0.4885065534],[3,2578625,0.3458467177],[3,2578637,0.2307285481],[3,2578716,0.2391172782],[3,2578736,0.3596224715],[3,2578769,0.5108024462],[3,2578808,0.3046766012],[3,2578887,0.2054730602],[3,2578964,0.3038434848],[3,2578970,0.213398564],[3,2578993,0.1828712845],[3,2579056,0.3798240046],[3,2579142,0.1839097929],[3,2579221,0.1842179702],[3,2579309,0.1931847304],[3,2579753,0.3989392357],[3,2580006,0.1954162502],[3,2580067,0.2413254422],[3,2580115,0.2259921506],[3,2580234,0.2138573383],[3,2580235,0.2197323892],[3,2580282,0.2016099062],[3,2580306,0.1889815867],[3,2580542,0.1808419147],[3,2580572,0.182482594],[3,2580578,0.1970200865],[3,2580614,0.2079596638],[3,2580741,0.3134654915],[3,2580769,0.1873804702],[3,2580812,0.2486365336],[3,2581082,0.3657183606],[3,2581084,0.2957553066],[3,2581456,0.1998718245],[3,2581469,0.4104231373],[3,2581471,0.4596312147],[3,2581478,0.2228817588],[3,2581654,0.1929138325],[3,2581672,0.1807482434],[3,2581695,0.3363350012],[3,2581696,0.37367507],[3,2581743,0.2621655292],[3,2581766,0.1847976004],[3,2581777,0.3657514487],[3,2581779,0.1844763936],[3,2581824,0.3712848702],[3,2581937,0.2950746185],[3,2581969,0.2860596548],[3,2582158,0.2361541578],[3,2582380,0.2781967301],[3,2582463,0.2832883609],[3,2582530,0.2588668807],[3,2582547,0.3105772085],[3,2582574,0.1880822433],[3,2582657,0.431571882],[3,2582855,0.2749526778],[3,2583091,0.2308005795],[3,2583101,0.1892131754],[3,2583241,0.2524843949],[3,2583260,0.4067894317],[3,2583269,0.1945139409],[3,2583347,0.5865579912],[3,2583699,0.3113464995],[3,2583946,0.2455163786],[3,2583975,0.1937305849],[3,2584178,0.1893892418],[3,2584240,0.2152511779],[3,2584241,0.2103416549],[3,2584265,0.2340337886],[3,2584287,0.3299110032],[3,2584367,0.3581439273],[3,2584899,0.3297261961],[3,2584943,0.1965267145],[3,2585051,0.2413971119],[3,2585286,0.1941582373],[3,2585720,0.2275383588],[3,2585756,0.2174905868],[3,2585771,0.2055028202],[3,2585815,0.2501943454],[3,2585831,0.1984319054],[3,2585893,0.3009335453],[3,2585921,0.3520767665],[3,2585949,0.3450319605],[3,2585961,0.208936153],[3,2585963,0.2079344556],[3,2586000,0.3994527902],[3,2586038,0.4144870958],[3,2586048,0.3091270729],[3,2586080,0.2096682419],[3,2586107,0.1717909971],[3,2586135,0.2015741867],[3,2586162,0.2133882815],[3,2586212,0.1919814152],[3,2586216,0.3668493107],[3,2586255,0.2114470209],[3,2586315,0.3521042505],[3,2586550,0.2844183614],[3,2586551,0.2532066995],[3,2586802,0.2483497114],[3,2586911,0.2317709686],[3,2586922,0.2196453409],[3,2586929,0.2125569688],[3,2587060,0.474489935],[3,2587061,0.1886802488],[3,2587628,0.2027382575],[3,2588021,0.3265726703],[3,2588029,0.2148651329],[3,2588917,0.3064328205],[3,2588994,0.1824666877],[3,2590755,0.1839097929],[3,2592895,0.180319482],[3,2596098,0.1743630569],[3,2598862,0.191320711],[3,2601639,0.1954162502],[3,2602676,0.180319482],[3,2603085,0.2397366155],[3,2606418,0.184677074],[3,2607139,0.1890577698],[3,2609700,0.1896983856],[3,2611958,0.2653613358],[3,2622351,0.1899243587],[3,2623093,0.3026842082],[3,2624026,0.1894982176],[3,2624040,0.1986987109],[3,2624593,0.2924955559],[3,2625183,0.2847425234],[3,2626123,0.1826145831],[3,2626767,0.1812767749],[3,2627101,0.2064070291],[3,2627306,0.1836845751],[3,2628569,0.1812292774],[3,2628577,0.1829089873],[3,2629158,0.1932129013],[3,2629200,0.1935879625],[3,2629281,0.1881201773],[3,2629477,0.1812292774],[3,2629629,0.1983139951],[3,2629638,0.1995598784],[3,2629671,0.1961588338],[3,2629723,0.1812292774],[3,2629726,0.2133648539],[3,2629786,0.1920149907],[3,2629855,0.1816874418],[3,2630018,0.1803351702],[3,2630357,0.1891688341],[3,2630495,0.1826145831],[3,2630511,0.1737849088],[3,2630516,0.1989693284],[3,2630538,0.1803351702],[3,2630574,0.1971719548],[3,2630580,0.2003427186],[3,2630600,0.196754864],[3,2630790,0.216543383],[3,2630793,0.1950085493],[3,2630825,0.1866908404],[3,2630826,0.1957508293],[3,2630827,0.1836845751],[3,2630850,0.183822511],[3,2630961,0.1852710634],[3,2630966,0.18854652],[3,2630973,0.1776080329],[3,2630989,0.207162733],[3,2631039,0.2064772729],[3,2631045,0.174303602],[3,2631047,0.2021009655],[3,2631052,0.2056238427],[3,2631056,0.1968841005],[3,2631108,0.1922650294],[3,2631115,0.1950505278],[3,2631141,0.1901414125],[3,2631158,0.1874936496],[3,2631191,0.2023064492],[3,2631255,0.1894982176],[3,2631275,0.2632845747],[3,2631333,0.1851754013],[3,2631336,0.177145151],[3,2632343,0.1744769533],[3,2632371,0.1912348786],[3,2632815,0.3006760511],[3,2633679,0.207389991],[3,2635808,0.5187264876],[3,2635859,0.1800415937],[3,2637208,0.1973368158],[3,2637667,0.1800415937],[3,2638110,0.2213453007],[3,2638817,0.1859627875],[3,2638831,0.19004],[3,2638856,0.1950598264],[3,2638882,0.1943127288],[3,2639172,0.2021276793],[3,2639252,0.5345487807],[3,2639315,0.2823360284],[3,2639332,0.2757577943],[3,2639342,0.2274766246],[3,2639677,0.1879231232],[3,2639858,0.2360138016],[3,2639983,0.1959656609],[3,2640128,0.2019551958],[3,2640318,0.1945850214],[3,2640401,0.1998343232],[3,2640458,0.2008757555],[3,2640503,0.1729714791],[3,2640584,0.2170951526],[3,2640635,0.1982414011],[3,2640762,0.5140979478],[3,2640776,0.2256161591],[3,2640793,0.2438357055],[3,2640820,0.1775499811],[3,2640901,0.1913188696],[3,2640986,0.1775499811],[3,2640987,0.1954110591],[3,2641071,0.2374084034],[3,2641103,0.1804111034],[3,2641184,0.4023192139],[3,2641205,0.4267819768],[3,2641264,0.1908499083],[3,2641303,0.263857264],[3,2641315,0.1968324048],[3,2641472,0.1792858977],[3,2641494,0.3317809768],[3,2641564,0.1964688075],[3,2641566,0.2000265671],[3,2641597,0.2418823997],[3,2641631,0.5906691985],[3,2641647,0.289073691],[3,2641710,0.3613062269],[3,2641715,0.1874693404],[3,2641817,0.188872564],[3,2642210,0.4691401946],[3,2642330,0.2461322681],[3,2642375,0.1952479597],[3,2642381,0.2067636711],[3,2642401,0.2213972175],[3,2642402,0.3421846155],[3,2642407,0.3865464133],[3,2642408,0.3354710323],[3,2642485,0.1800415937],[3,2642537,0.224839351],[3,2642581,0.2006248805],[3,2642644,0.1821273505],[3,2642651,0.1825664321],[3,2642652,0.2161963034],[3,2642656,0.191464907],[3,2642695,0.1836098731],[3,2642714,0.1899285349],[3,2642752,0.2620934297],[3,2642770,0.2027018983],[3,2642775,0.2115584841],[3,2642788,0.4157347751],[3,2642800,0.2249737076],[3,2642835,0.2155438568],[3,2642906,0.1861459362],[3,2642910,0.1830935111],[3,2642936,0.2980153941],[3,2642939,0.1897265698],[3,2642969,0.1951567594],[3,2642985,0.2097643181],[3,2643005,0.2078916669],[3,2643037,0.1976507607],[3,2643070,0.3159712986],[3,2643093,0.7922507588],[3,2643112,0.1832113229],[3,2643137,0.1793133593],[3,2643172,0.2552837915],[3,2643187,0.2753376821],[3,2643212,0.2371386789],[3,2643279,0.2511036378],[3,2643280,0.2941768228],[3,2643339,0.3852019528],[3,2643362,0.3149444911],[3,2643374,0.1974218939],[3,2643404,0.1938606539],[3,2643458,0.3020514302],[3,2643473,0.1998718621],[3,2643475,0.325317412],[3,2643516,0.7776444284],[3,2643536,0.2344214737],[3,2643582,0.1929772202],[3,2643584,0.1887643297],[3,2643585,0.1965298723],[3,2643588,0.279808462],[3,2643653,0.196773897],[3,2643671,0.1965488464],[3,2643672,0.1959203977],[3,2643679,0.1826837653],[3,2643690,0.2378475769],[3,2643705,0.1902558471],[3,2643717,0.4029614697],[3,2643751,0.1866674107],[3,2643772,0.230556352],[3,2643804,0.4180492389],[3,2643895,0.2043474664],[3,2643897,0.191017707],[3,2643962,0.2037027467],[3,2644034,0.3173289865],[3,2644109,0.5549709383],[3,2644110,0.2730029829],[3,2644111,0.3359577546],[3,2644112,0.448222554],[3,2644158,0.2756073855],[3,2644170,0.2243909947],[3,2644370,0.1956143659],[3,2644546,0.1885376591],[3,2644549,0.4711686874],[3,2644550,0.6229345424],[3,2644558,0.4874306925],[3,2644568,0.2537917213],[3,2644584,0.1832113229],[3,2644593,0.2506005934],[3,2644613,0.3019689337],[3,2644616,0.4795828572],[3,2644662,0.1880364367],[3,2644718,0.1930739356],[3,2644792,0.2043166695],[3,2644874,0.2099665124],[3,2644876,0.182645717],[3,2644898,0.1973733794],[3,2644906,0.1894209634],[3,2644933,0.7414518961],[3,2644974,0.7162897317],[3,2644980,0.3303339056],[3,2645004,0.3741946457],[3,2645011,0.253928897],[3,2645012,0.3129363063],[3,2645019,0.1764367861],[3,2645099,0.2763342965],[3,2645117,0.4361520567],[3,2645118,0.3217503379],[3,2645142,0.6135654823],[3,2645149,0.1890400276],[3,2645179,0.1895860006],[3,2645181,0.4079629568],[3,2645204,0.3736190715],[3,2645234,0.2123300043],[3,2645236,0.2731633229],[3,2645238,0.2729408296],[3,2645239,0.8082414981],[3,2645242,0.1934630818],[3,2645244,0.377722285],[3,2645345,0.5376249947],[3,2645361,0.3419630108],[3,2645365,0.229419478],[3,2645371,0.2237575443],[3,2645478,0.4970686815],[3,2645515,0.2692703105],[3,2645535,0.3008622286],[3,2645567,0.3340171427],[3,2645586,0.4497620274],[3,2645623,0.3457940356],[3,2645627,0.2306276544],[3,2645646,0.6637979602],[3,2645711,0.3213223579],[3,2645722,0.345307681],[3,2645774,0.2042966186],[3,2645843,0.1927274529],[3,2645858,0.5893190425],[3,2645860,0.5160542376],[3,2645872,0.1981936273],[3,2645979,0.2176815035],[3,2645980,0.492747852],[3,2645990,0.2052465494],[3,2645991,0.2780302641],[3,2646032,0.1814803162],[3,2646058,0.4848131211],[3,2646062,0.3460272788],[3,2646076,0.3832660354],[3,2646077,0.2262681555],[3,2646090,0.2515050875],[3,2646159,0.1763245894],[3,2646175,0.3726496138],[3,2646176,0.184126653],[3,2646223,0.6047243492],[3,2646225,0.2496160009],[3,2646227,0.7004825719],[3,2646265,0.4366305241],[3,2646275,0.6112673388],[3,2646301,0.226459347],[3,2646438,0.2942870913],[3,2646539,0.2649229303],[3,2646624,0.4628307094],[3,2646627,0.4758120621],[3,2646628,0.6424662956],[3,2646657,0.2465597744],[3,2646670,0.2569771908],[3,2646760,0.6011281947],[3,2646966,0.1771831968],[3,2646994,0.2905921947],[3,2647011,0.2316977123],[3,2647043,0.2071802098],[3,2647092,0.2093130279],[3,2647113,0.3226637674],[3,2647114,0.2964389067],[3,2647139,0.6565357583],[3,2647157,0.6448016496],[3,2647210,0.4266459577],[3,2647220,0.2105944773],[3,2647230,0.2252444687],[3,2647317,0.677527135],[3,2647374,0.3171846183],[3,2647392,0.1859887256],[3,2647396,0.2843843571],[3,2647500,0.1935420011],[3,2647811,0.2477061352],[3,2647842,0.7231990328],[3,2647859,0.2298852314],[3,2647873,0.6012010555],[3,2647979,0.3051321266],[3,2647983,0.2965708806],[3,2648036,0.1956878357],[3,2648095,0.4978073103],[3,2648130,0.2702897159],[3,2648143,0.29106917],[3,2648151,0.2593604859],[3,2648152,0.3971505996],[3,2648155,0.1874687786],[3,2648199,0.2978047524],[3,2648233,0.1927753699],[3,2648306,0.3845890097],[3,2648307,0.441837779],[3,2648441,0.1807509338],[3,2648451,0.1746638372],[3,2648614,0.2745789894],[3,2648891,0.2143593315],[3,2648922,0.1858263486],[3,2648937,0.1914067503],[3,2648940,0.2253766796],[3,2648961,0.1911369058],[3,2649001,0.2316482193],[3,2649017,0.203218868],[3,2649052,0.267907516],[3,2649058,0.2106966516],[3,2649078,0.4824798605],[3,2649088,0.1885864697],[3,2649090,0.1943181204],[3,2649106,0.2611778015],[3,2649114,0.190341296],[3,2649115,0.5027144055],[3,2649155,0.2692062081],[3,2649207,0.2167900475],[3,2649378,0.2000807108],[3,2649638,0.1983807842],[3,2649854,0.1868846545],[3,2649922,0.2036779525],[3,2650003,0.1762252579],[3,2650014,0.1868846545],[3,2650180,0.3512451757],[3,2650203,0.3680364439],[3,2650226,0.3343958984],[3,2650244,0.18291869],[3,2650257,0.2064481091],[3,2650383,0.1832476736],[3,2650596,0.1802079306],[3,2650710,0.2099298823],[3,2650729,0.2764669753],[3,2650786,0.1929681469],[3,2650848,0.2520162903],[3,2650849,0.3966125264],[3,2650871,0.2396505384],[3,2650955,0.3760526565],[3,2650959,0.1998796092],[3,2651000,0.1769285899],[3,2651044,0.1972780215],[3,2651067,0.2699765839],[3,2651094,0.1965962215],[3,2651170,0.2117565245],[3,2651228,0.3616574929],[3,2651318,0.3518199256],[3,2651347,0.2020638516],[3,2651441,0.214225599],[3,2651710,0.2121669989],[3,2651821,0.2049065557],[3,2651866,0.1820877427],[3,2651973,0.1876753365],[3,2651979,0.1854435775],[3,2651991,0.1874956724],[3,2652027,0.186067305],[3,2652101,0.2106727206],[3,2652419,0.1925250035],[3,2652504,0.3404096609],[3,2652566,0.1972361048],[3,2652699,0.2228837076],[3,2652700,0.1784939082],[3,2652704,0.1795853385],[3,2652742,0.2663664432],[3,2652750,0.2589600132],[3,2652876,0.2029753672],[3,2652907,0.3362348384],[3,2652930,0.2276638283],[3,2652932,0.1790351142],[3,2652935,0.2119942938],[3,2652940,0.2790931354],[3,2652949,0.2413330116],[3,2652976,0.1882698217],[3,2653002,0.2341127902],[3,2653003,0.3414404734],[3,2653007,0.1881962289],[3,2653011,0.1830694865],[3,2653022,0.1802079306],[3,2653034,0.1883355549],[3,2653071,0.2010735968],[3,2653146,0.3168094106],[3,2653147,0.2188491557],[3,2653208,0.264180115],[3,2653402,0.2441106205],[3,2653604,0.1883923559],[3,2653607,0.2107361217],[3,2653755,0.1914669269],[3,2653837,0.2270851039],[3,2653845,0.2476596397],[3,2653912,0.1861756293],[3,2653913,0.1820877427],[3,2654206,0.2096767809],[3,2654363,0.1948749057],[3,2654389,0.2101718508],[3,2654402,0.2063494375],[3,2654429,0.1830694865],[3,2654456,0.1935009755],[3,2654457,0.1854435775],[3,2654458,0.1962606197],[3,2654491,0.2152788441],[3,2654515,0.2028979258],[3,2654529,0.2002454524],[3,2654535,0.1830694865],[3,2654537,0.1941095891],[3,2654606,0.2754525543],[3,2654613,0.2178731397],[3,2654766,0.1982941858],[3,2654870,0.1920467984],[3,2654880,0.1973027409],[3,2654901,0.1744826844],[3,2654926,0.1980450962],[3,2654939,0.1958770787],[3,2654940,0.2041575949],[3,2654943,0.3624112868],[3,2654971,0.1977083415],[3,2654991,0.2026638898],[3,2655000,0.2358615139],[3,2655005,0.1841730664],[3,2655018,0.3315002501],[3,2655021,0.2465492294],[3,2655027,0.1820877427],[3,2655028,0.1915443352],[3,2655053,0.3358939531],[3,2655095,0.194552045],[3,2655103,0.1820877427],[3,2655120,0.1996025784],[3,2655128,0.2123678051],[3,2655143,0.1891452008],[3,2655227,0.1840254229],[3,2655231,0.1820877427],[3,2655259,0.1982494243],[3,2655261,0.1997321314],[3,2655265,0.2220205823],[3,2655350,0.1892578569],[3,2655508,0.3606078201],[3,2655509,0.1752667341],[3,2655842,0.1880801845],[3,2655901,0.1777877678],[3,2655902,0.1968217606],[3,2655981,0.3986863155],[3,2655992,0.1858037647],[3,2655993,0.2256240622],[3,2656035,0.1912683804],[3,2656113,0.2015306441],[3,2656137,0.1987913586],[3,2656570,0.1820877427],[3,2656611,0.1800428326],[3,2656635,0.1923104688],[3,2656723,0.1817788928],[3,2656996,0.2986630882],[3,2657248,0.1906826195],[3,2657252,0.2059898215],[3,2657258,0.1805480115],[3,2657259,0.2293596522],[3,2657297,0.1927393312],[3,2657314,0.2377888779],[3,2657317,0.2146898614],[3,2657346,0.1887308665],[3,2657370,0.2008977352],[3,2657410,0.2106442479],[3,2657450,0.2117856931],[3,2657512,0.2050849917],[3,2657663,0.1932369951],[3,2657826,0.2021433004],[3,2657923,0.2450003244],[3,2657940,0.2247840081],[3,2657969,0.2077096322],[3,2657981,0.2328920193],[3,2658001,0.2067223764],[3,2658024,0.2038422614],[3,2658032,0.2048240743],[3,2658041,0.2061973714],[3,2658067,0.181860319],[3,2658085,0.2223802423],[3,2658109,0.23636458],[3,2658116,0.1929103608],[3,2658160,0.2132737791],[3,2658199,0.2023720736],[3,2658298,0.2954913102],[3,2658345,0.1861650151],[3,2658346,0.2077216167],[3,2658347,0.2769035319],[3,2658353,0.2059618608],[3,2658804,0.2025290459],[3,2658831,0.2165004398],[3,2658834,0.1897197404],[3,2658900,0.1896331963],[3,2658915,0.1887308665],[3,2658927,0.207375715],[3,2659488,0.2027015726],[3,2659512,0.3510622071],[3,2659592,0.1916735882],[3,2659644,0.2041296022],[3,2659647,0.2153802741],[3,2659816,0.2119734443],[3,2659897,0.4132923404],[3,2659925,0.2092654753],[3,2659988,0.181070575],[3,2660011,0.2396388188],[3,2660068,0.2149235659],[3,2660117,0.2064263251],[3,2660168,0.2670641851],[3,2660200,0.1919307746],[3,2660211,0.205449971],[3,2660215,0.2169543889],[3,2660248,0.1887308665],[3,2660250,0.1996206138],[3,2660266,0.2134020507],[3,2660305,0.2022771705],[3,2660309,0.1813373038],[3,2660342,0.1887308665],[3,2660369,0.1965501413],[3,2660405,0.2044105063],[3,2660452,0.1859942074],[3,2660623,0.1887308665],[3,2660987,0.181070575],[3,2661022,0.2071732483],[3,2661129,0.1922502781],[3,2661159,0.1934242621],[3,2661186,0.1946327251],[3,2661197,0.1941780723],[3,2661206,0.1887308665],[3,2661749,0.2021562915],[3,2662472,0.1887308665],[3,2670045,0.1903127805],[3,2670709,0.1883857997],[3,2670953,0.1912166595],[3,2671415,0.1993237179],[3,2671787,0.1941064712],[3,2671860,0.234473502],[3,2672174,0.3151070278],[3,2672195,0.2594390175],[3,2672411,0.3959864226],[3,2672482,0.3045013612],[3,2672847,0.1935905433],[3,2672894,0.3477480003],[3,2672961,0.2006488756],[3,2672968,0.2006994293],[3,2672974,0.2402999135],[3,2672997,0.3127251047],[3,2672998,0.2343163943],[3,2673070,0.5259209527],[3,2673142,0.1885253735],[3,2673686,0.211444749],[3,2673991,0.4011222314],[3,2674186,0.2835765841],[3,2674287,0.2135790579],[3,2674375,0.2463282719],[3,2674642,0.5162043339],[3,2674646,0.1968083049],[3,2674665,0.5663153722],[3,2674711,0.3418338812],[3,2674726,0.6159750317],[3,2674743,0.2270171893],[3,2674763,0.2028092259],[3,2674811,0.1903127805],[3,2674812,0.2908315341],[3,2674822,0.278976465],[3,2674876,0.2061798078],[3,2674993,0.211200454],[3,2675257,0.3083624893],[3,2675316,0.300392133],[3,2675363,0.3451040462],[3,2675410,0.1902004956],[3,2675423,0.4411511214],[3,2675482,0.1952729459],[3,2675508,0.4237818006],[3,2675541,0.2221660892],[3,2675601,0.2276014158],[3,2675630,0.2191084441],[3,2675704,0.2103763657],[3,2675733,0.5553806186],[3,2676132,0.3174178276],[3,2676222,0.2387611817],[3,2676258,0.2249430072],[3,2676405,0.2165848043],[3,2676430,0.2272926398],[3,2676522,0.3896188709],[3,2676771,0.3490164084],[3,2676820,0.2436489297],[3,2676935,0.2582086734],[3,2676936,0.1932998053],[3,2677000,0.2396826281],[3,2677019,0.4158837666],[3,2677214,0.2622157487],[3,2677227,0.1884108091],[3,2677239,0.4259707936],[3,2677444,0.3500828871],[3,2677466,0.247205036],[3,2677697,0.2400669726],[3,2677786,0.7976147083],[3,2678172,0.270223596],[3,2678256,0.2266312139],[3,2678350,0.3666134251],[3,2678376,0.4256969732],[3,2678567,0.2057541581],[3,2678807,0.6586543138],[3,2679384,0.2225567776],[3,2679749,0.3311943318],[3,2680226,0.1909374242],[3,2680859,0.1997821676],[3,2681203,0.1999652601],[3,2681283,0.3032905674],[3,2682277,0.7957599025],[3,2682829,0.1909374242],[3,2683921,0.1909374242],[3,2684772,0.2071064037],[3,2684810,0.2037105699],[3,2686520,0.1909374242],[3,2688008,0.1845787672],[3,2689020,0.1903016602],[3,2689088,0.1909374242],[3,2690688,0.201747016],[3,2697418,0.1909374242],[3,2699605,0.388976441],[3,2699637,0.4101399269],[3,2699796,0.2187521184],[3,2700044,0.1909374242],[3,2700729,0.2045291922],[3,2700757,0.2965016991],[3,2700780,0.3253361973],[3,2700950,0.2166222006],[3,2700955,0.2140329605],[3,2701009,0.3291347006],[3,2701197,0.3467504243],[3,2701266,0.5439023867],[3,2701342,0.3133309079],[3,2701783,0.2067183673],[3,2702043,0.1896727103],[3,2702227,0.3269921127],[3,2702297,0.1909374242],[3,2703047,0.2792809596],[3,2703903,0.1943295445],[3,2703918,0.5174872306],[3,2705254,0.2244122624],[3,2705495,0.2074425327],[3,2705614,0.1909374242],[3,2705940,0.2474680449],[3,2705971,0.2048410264],[3,2706452,0.2105049462],[3,2706501,0.2241644602],[3,2706536,0.2086276024],[3,2706627,0.337351219],[3,2706658,0.1883857997],[3,2707128,0.219914549],[3,2707170,0.2052549202],[3,2707543,0.3367798925],[3,2707883,0.1935411319],[3,2707884,0.2070995798],[3,2707944,0.2403460668],[3,2708641,0.2779841155],[3,2708767,0.1871256592],[3,2709093,0.2520322688],[3,2709474,0.1902004956],[3,2710107,0.2101109921],[3,2710789,0.196974269],[3,2711074,0.2076543175],[3,2712044,0.1909374242],[3,2713106,0.1940540456],[3,2713135,0.2553338849],[3,2713816,0.1963677117],[3,2713873,0.2528470827],[3,2714742,0.2091205713],[3,2720044,0.193645146],[3,2722067,0.2335782845],[3,2723179,0.2018345383],[3,2723837,0.193645146],[3,2724367,0.1970901868],[3,2725426,0.2498065948],[3,2727352,0.2133499327],[3,2728527,0.2713467478],[3,2729452,0.2084614609],[3,2730651,0.2026010706],[3,2739031,0.1945719572],[3,2740039,0.2254633047],[3,2740803,0.2138850756],[3,2741153,0.2319366037],[3,2741453,0.1963267009],[3,2742647,0.3157341001]

In [204]:
arr_to_submit = pd.DataFrame(a) 

In [ ]:
pd.DataFrame(a).to_csv("armis_85_preds")

In [206]:
# arr_to_submit = pd.read_csv("SparkBeyond_get_preds_all.csv.gz").to_json(orient='values')
arr_to_submit = arr_to_submit.to_json(orient='values')

leaderboard_name = "armis"
host = "leaderboard.datahack.org.il"

# Name of the user
submitter = "Overfitters"

predictions = json.loads(arr_to_submit)

jsonStr = json.dumps({'submitter': submitter, 'predictions': predictions})
data = jsonStr.encode('utf-8')
req = request.Request(f"https://{host}/{leaderboard_name}/api/",
                      headers={'Content-Type': 'application/json'},
                      data=data)
resp = request.urlopen(req)

ret = json.load(resp)
print(ret['score'])

0.8510815155048088


## old brute force

In [110]:
def by_force(param, bootstrap, n_estimators, behaviour,networks_fs = []):
    if networks_fs == []:

        for i in range(4):
            print(i)
#             networks_fs.append(extract_features(df[df.network_id == i])
            networks_fs.append(extract_all_features(df[df.network_id == i]))
        ### drop or fill nans
        for i in range(4):
        #     networks_fs[i].fillna(-1,inplace=True)
            networks_fs[i].dropna(axis=1,inplace=True)

    for i in range(4):
        try: networks_fs[i].drop("index",axis=1,inplace=True)
        except:pass
        try: networks_fs[i].drop('level_0',axis=1,inplace=True)
        except:pass
   
        
#     for i in range(4):
        networks_fs[i] = detect_anomalies(feature_set=networks_fs[i], param=param,
                                          bootstrap=bootstrap, n_estimators=n_estimators, behaviour=behaviour) # ORIG
#         networks_fs[i] = detect_anomalies(feature_set=networks_fs[i])
    
    # Remove the device_id from index and Add network id to each data set
    for i in range(4):
#         networks_fs[i].fillna(0,inplace=True)
        networks_fs[i].reset_index(level=0, inplace=True,)
        networks_fs[i]["network_id"] = i 
        
    df_to_submit = pd.concat(networks_fs)[["network_id", "device_id", "confidence"]] # The column order is important!
    df_to_submit.fillna(0,inplace=True) # handle nans
    df_to_submit.head()
    arr_to_submit = df_to_submit.to_json(orient='values')


    leaderboard_name = "armis"
    host = "leaderboard.datahack.org.il"

    # Name of the user
    submitter = ".303GrimOverfitters"

    predictions = json.loads(arr_to_submit)

    jsonStr = json.dumps({'submitter': submitter, 'predictions': predictions})
    data = jsonStr.encode('utf-8')
    req = request.Request(f"https://{host}/{leaderboard_name}/api/",
                          headers={'Content-Type': 'application/json'},
                          data=data)
    resp = request.urlopen(req)
    
    ret = json.load(resp)
    return ret['score']


max_score = 0
max_param = 0.05

for i in [0.02,0.05, 0.08]:  
# while True:
# for j in range(3):
    
    i = random.random()
#     behaviour = random.choice(['old', 'new'])
    behaviour = "new"
    n_estimators = random.randint(40, 150)
    bootstrap = random.choice([True, False])
    score = by_force(i,  bootstrap, n_estimators, behaviour,networks_fs=networks_fs)
    print(score)
    if score > max_score:
        max_score = score
        max_param = i 
        max_n_estimators = n_estimators
        max_boostrap = bootstrap
    else:   
        print('resubmitting')
        score = by_force(i,  bootstrap, n_estimators, behaviour,networks_fs=networks_fs)        
        
        

0


KeyboardInterrupt: 

### feat stats
* subtract features , get stats of perf

In [ ]:
networks_fs[0].head()

In [ ]:
networks_fs[0].reset_index(level=0).columns

In [ ]:
# def select_features_overfit(networks_fs):
feat_stats = {}
# col_counts = networks_fs[0].shape[1] -1 # ignore first col of device id
f_names = list(networks_fs[0].columns).remove("device_id")

#baseline best score
baseline_best = 60
for f in range(5): # try only a few cols to speedup col_counts//120
    print(f)
    f_name = networks_fs[0].columns[f][0]
    print("col name",f_name)

    for i in range(4):
        try: networks_fs[i].drop("index",axis=1,inplace=True)
        except:pass
        try: networks_fs[i].drop('level_0',axis=1,inplace=True)
        except:pass
        
        networks_fs[i] = detect_anomalies(feature_set=networks_fs[i].drop([f_name],axis=1), param=param,  bootstrap=bootstrap, n_estimators=n_estimators, behaviour=behaviour) # ORIG


    # Remove the device_id from index and Add network id to each data set
    for i in range(4):  # DOES this fuck up feature index order ???? 
#         networks_fs[i].fillna(0,inplace=True)  # 
        networks_fs[i].reset_index(level=0, inplace=True)
        networks_fs[i]["network_id"] = i 

    df_to_submit = pd.concat(networks_fs)[["network_id", "device_id", "confidence"]] # The column order is important!
    df_to_submit.fillna(0,inplace=True)
    df_to_submit.head()
    arr_to_submit = df_to_submit.to_json(orient='values')


    leaderboard_name = "armis"
    host = "leaderboard.datahack.org.il"

    # Name of the user
    submitter = ".303BookwormOverfitter"

    predictions = json.loads(arr_to_submit)

    jsonStr = json.dumps({'submitter': submitter, 'predictions': predictions})
    data = jsonStr.encode('utf-8')
    req = request.Request(f"https://{host}/{leaderboard_name}/api/",
                          headers={'Content-Type': 'application/json'},
                          data=data)
    resp = request.urlopen(req)

    ret = json.load(resp)
    print("score",ret['score'])

    if  ret['score']>baseline_best:
        print("\n best score surpassed! \n")
    # save value to dict

#     feat_stats[networks_fs[0].columns[f][0]] = ret['score']

    feat_stats[f_name]= ret['score']
    
#     return(feat_stats)
        
        
        
        
        
# select_features_overfit(networks_fs)

In [ ]:
for i in range(4):
    networks_fs[i].drop(["level_0","index"],inplace=True,axis=1)

In [ ]:
def by_force(param, bootstrap, n_estimators, behaviour,networks_fs = []):
    if networks_fs == []:

        for i in range(4):
            print(i)
            networks_fs.append(extract_features(df[df.network_id == i]))
        ### drop or fill nans
        for i in range(4):
        #     networks_fs[i].fillna(-1,inplace=True)
            networks_fs[i].dropna(axis=1,inplace=True)
        
        
    for i in range(4):
        networks_fs[i] = detect_anomalies(feature_set=networks_fs[i], param=param,  bootstrap=bootstrap, n_estimators=n_estimators, behaviour=behaviour) # ORIG
#         networks_fs[i] = detect_anomalies(feature_set=networks_fs[i])
    
    # Remove the device_id from index and Add network id to each data set
    for i in range(4):
        networks_fs[i].reset_index(level=0, inplace=True)
        networks_fs[i]["network_id"] = i 
        
    df_to_submit = pd.concat(networks_fs)[["network_id", "device_id", "confidence"]] # The column order is important!
    df_to_submit.fillna(0,inplace=True)
    df_to_submit.head()
    arr_to_submit = df_to_submit.to_json(orient='values')


    leaderboard_name = "armis"
    host = "leaderboard.datahack.org.il"

    # Name of the user
    submitter = "Overfitters"

    predictions = json.loads(arr_to_submit)

    jsonStr = json.dumps({'submitter': submitter, 'predictions': predictions})
    data = jsonStr.encode('utf-8')
    req = request.Request(f"https://{host}/{leaderboard_name}/api/",
                          headers={'Content-Type': 'application/json'},
                          data=data)
    resp = request.urlopen(req)
    
    ret = json.load(resp)
    return ret['score']



max_score = 0.5
max_param = 0.05
for i in [0.03,0.05, 0.09]:  

# while True:
# for j in range(2):
    
    i = random.random()
#     behaviour = random.choice(['old', 'new'])
    behaviour = "new"
    n_estimators = random.randint(50, 250)
    bootstrap = random.choice([True, False])
    score = by_force(i,  bootstrap, n_estimators, behaviour,networks_fs=networks_fs)
    print(score)
    if score > max_score:
        max_score = score
        max_param = i 
        max_n_estimators = n_estimators
        max_boostrap = bootstrap
    else:   
        print('resubmitting')
        score = by_force(i,  bootstrap, n_estimators, behaviour,networks_fs=networks_fs)        
        

In [ ]:
by_force(1,  bootstrap, n_estimators, behaviour,networks_fs=networks_fs)

In [ ]:

# for i in range(4):
#     networks_fs[i] = detect_anomalies(feature_set=networks_fs[i], param=param,  bootstrap=bootstrap, n_estimators=n_estimators, behaviour=behaviour) # ORIG

# Remove the device_id from index and Add network id to each data set
# for i in range(4):
#     networks_fs[i].reset_index(level=0, inplace=True)
#     networks_fs[i]["network_id"] = i ## bug??
    
    
df_to_submit = pd.concat(networks_fs)[["network_id", "device_id", "confidence"]] # The column order is important!
df_to_submit.head()



## Submissions
* In order to update the Leader Board you need to send a POST request to following url: "https://leaderboard.datahack.org.il/armis/api/". Our Leader Board receives your anomalies results in the json form of [["network_id", "device_id", "confidence"]] - The order is important!

In [ ]:
arr_to_submit = df_to_submit.to_json(orient='values')

In [ ]:
df_to_submit.to_csv("sub_83.csv",index=False)

In [ ]:
json.dump(arr_to_submit, open('sub.json', 'w'))

In [ ]:
from urllib import request
import json

leaderboard_name = "armis"
host = "leaderboard.datahack.org.il"

# Name of the user
submitter = "Overfitters"

predictions = json.loads(arr_to_submit)

jsonStr = json.dumps({'submitter': submitter, 'predictions': predictions})
data = jsonStr.encode('utf-8')
req = request.Request(f"https://{host}/{leaderboard_name}/api/",
                      headers={'Content-Type': 'application/json'},
                      data=data)
resp = request.urlopen(req)
print(json.load(resp))

In [ ]:
df_to_submit.to_json(orient='values')